In [1]:
import physics_environments
from physics_environments.envs.rl_pendulum.cart_pendulum_math.param_manager import CartPendulumParameterManager
from physics_environments.envs.rl_pendulum.cart_pendulum_math.robotic_eq_solutions import robotic_equation_n5
from physics_environments import CartPendulumPhysicsConstants, CartPendulumPhysicsEnvironmentParams, CartPendulumPhysicsEnvironment, \
                                 RLCartPendulumEnvironment, RLCartPendulumEnvironmentParams


import os
import numba
import numpy as np
import jax
import jax.numpy as jnp
import time

In [2]:
# Note the GPU installation:
jax.devices()

[CudaDevice(id=0)]

In [3]:
# >>> nvidia-smi
# +-----------------------------------------------------------------------------------------+
# | NVIDIA-SMI 560.94                 Driver Version: 560.94         CUDA Version: 12.6     |
# |-----------------------------------------+------------------------+----------------------+
# | GPU  Name                  Driver-Model | Bus-Id          Disp.A | Volatile Uncorr. ECC |
# | Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
# |                                         |                        |               MIG M. |
# |=========================================+========================+======================|
# |   0  NVIDIA GeForce RTX 2080 Ti   WDDM  |   00000000:08:00.0  On |                  N/A |
# | 33%   34C    P8             31W /  250W |   10285MiB /  11264MiB |      1%      Default |
# |                                         |                        |                  N/A |
# +-----------------------------------------+------------------------+----------------------+

In [4]:
n = 5
constants = CartPendulumPhysicsConstants(n    = n,         
                                         g    = 9.81,       
                                         l    = n*[0.300],
                                         r    = n*[0.200],
                                         m    = n*[0.800],
                                         I    = n*[0.011],
                                         mu   = n*[0.015])

env_type             = 'compound pendulum'
cart_pendulum_params = CartPendulumPhysicsEnvironmentParams(constants       = constants,         
                                                            env_type        = env_type,      
                                                            save_after_init = False, 
                                                            save_filename   = f"env_{n}n_{env_type}_obj.pkl") 

rl_env_params = RLCartPendulumEnvironmentParams(physics_env_params          = cart_pendulum_params,
                                                training_generator_type     = "DefaultGenerator",
                                                dt                          = 0.010,
                                                t_episode                   = 10.00,
                                                max_cart_acceleration       = 25.00,
                                                cart_width                  = 0.250,
                                                track_width                 = 0.800
                                                )

rl_env = RLCartPendulumEnvironment(params = rl_env_params)

<class 'physics_environments.envs.rl_pendulum.cart_pendulum_math.types.CartPendulumPhysicsConstants'> {'n': 5, 'g': 9.81, 'l': [0.3, 0.3, 0.3, 0.3, 0.3], 'r': [0.2, 0.2, 0.2, 0.2, 0.2], 'm': [0.8, 0.8, 0.8, 0.8, 0.8], 'I': [0.011, 0.011, 0.011, 0.011, 0.011], 'mu': [0.015, 0.015, 0.015, 0.015, 0.015]}


#### Regular Python Function calling (With Numpy)

---

In [5]:
gpu = jax.devices("gpu")[0]  # First GPU
cpu = jax.devices("cpu")[0]  # First CPU

inputs_np_cpu  = np.array(2), np.array([1, 2, 3, 4, 1, 11, 22, 33, 44, 55, 1]), np.array(rl_env.constants.ode_constants)
inputs_jax_gpu = [jnp.array(2, device=gpu),jnp.array([1, 2, 3, 4, 1, 11, 22, 33, 44, 55, 1], device=gpu), jnp.array(rl_env.constants.ode_constants, device=gpu)]
inputs_jax_cpu = [jnp.array(2, device=cpu),jnp.array([1, 2, 3, 4, 1, 11, 22, 33, 44, 55, 1], device=cpu), jnp.array(rl_env.constants.ode_constants, device=cpu)]

In [6]:
math_env = CartPendulumParameterManager.load_env_obj_from_file(
    filename = "/home/hbeyer/RL_Pendulum_Project/physics_environments/experiments/done/bu/bu_obj/env_5n_compound pendulum_obj.pkl"
)

In [7]:
def func_jax(t,y,args):
    return robotic_equation_n5(t=t, 
                    y=y,
                    args=args)

@jax.jit
def func_jax_jit(t, y, args):
    return robotic_equation_n5(t=t, 
                    y=y,
                    args=args)

def robotic_equation_n5_numpy(t, y, args):
    theta_1, theta_2, theta_3, theta_4, theta_5, dtheta_1, dtheta_2, dtheta_3, dtheta_4, dtheta_5, ddx_c                           = y
    g, l_1, l_2, l_3, l_4, r_1, r_2, r_3, r_4, r_5, m_1, m_2, m_3, m_4, m_5, mu_1, mu_2, mu_3, mu_4, mu_5, I_1, I_2, I_3, I_4, I_5 = args

    return np.array([dtheta_1,
                      dtheta_2,
                      dtheta_3,
                      dtheta_4,
                      dtheta_5,

                      (dtheta_5**2*l_1*m_5*r_5*np.sin(theta_5 - theta_1) + dtheta_4*(dtheta_4*l_1*l_4*m_5*np.sin(theta_4 - theta_1) + dtheta_4*l_1*m_4*r_4*np.sin(theta_4 - theta_1)) + dtheta_3*(dtheta_3*l_1*l_3*m_4*np.sin(theta_3 - theta_1) + dtheta_3*l_1*l_3*m_5*np.sin(theta_3 - theta_1) + dtheta_3*l_1*m_3*r_3*np.sin(theta_3 - theta_1)) + dtheta_2*mu_2 + dtheta_2*(dtheta_2*l_1*l_2*m_3*np.sin(theta_2 - theta_1) + dtheta_2*l_1*l_2*m_4*np.sin(theta_2 - theta_1) + dtheta_2*l_1*l_2*m_5*np.sin(theta_2 - theta_1) + dtheta_2*l_1*m_2*r_2*np.sin(theta_2 - theta_1)) - dtheta_1*(mu_1 + mu_2) - ddx_c*(-l_1*m_2 - l_1*m_3 - l_1*m_4 - l_1*m_5 - m_1*r_1)*np.cos(theta_1) - g*(-l_1*m_2 - l_1*m_3 - l_1*m_4 - l_1*m_5 - m_1*r_1)*np.sin(theta_1) - l_1*m_5*r_5*(-dtheta_5*mu_5 - dtheta_4**2*l_4*m_5*r_5*np.sin(theta_5 - theta_4) + dtheta_4*mu_5 - dtheta_3**2*l_3*m_5*r_5*np.sin(theta_5 - theta_3) - dtheta_2**2*l_2*m_5*r_5*np.sin(theta_5 - theta_2) - dtheta_1**2*l_1*m_5*r_5*np.sin(theta_5 - theta_1) + ddx_c*m_5*r_5*np.cos(theta_5) + g*m_5*r_5*np.sin(theta_5) - l_1*m_5*r_5*(dtheta_5**2*l_1*m_5*r_5*np.sin(theta_5 - theta_1) + dtheta_4*(dtheta_4*l_1*l_4*m_5*np.sin(theta_4 - theta_1) + dtheta_4*l_1*m_4*r_4*np.sin(theta_4 - theta_1)) + dtheta_3*(dtheta_3*l_1*l_3*m_4*np.sin(theta_3 - theta_1) + dtheta_3*l_1*l_3*m_5*np.sin(theta_3 - theta_1) + dtheta_3*l_1*m_3*r_3*np.sin(theta_3 - theta_1)) + dtheta_2*mu_2 + dtheta_2*(dtheta_2*l_1*l_2*m_3*np.sin(theta_2 - theta_1) + dtheta_2*l_1*l_2*m_4*np.sin(theta_2 - theta_1) + dtheta_2*l_1*l_2*m_5*np.sin(theta_2 - theta_1) + dtheta_2*l_1*m_2*r_2*np.sin(theta_2 - theta_1)) - dtheta_1*(mu_1 + mu_2) - ddx_c*(-l_1*m_2 - l_1*m_3 - l_1*m_4 - l_1*m_5 - m_1*r_1)*np.cos(theta_1) - g*(-l_1*m_2 - l_1*m_3 - l_1*m_4 - l_1*m_5 - m_1*r_1)*np.sin(theta_1))*np.cos(theta_5 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) - (-l_1**2*m_5*r_5*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_5 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*m_5*r_5*np.cos(theta_5 - theta_2))*(dtheta_5**2*l_2*m_5*r_5*np.sin(theta_5 - theta_2) + dtheta_4*(dtheta_4*l_2*l_4*m_5*np.sin(theta_4 - theta_2) + dtheta_4*l_2*m_4*r_4*np.sin(theta_4 - theta_2)) + dtheta_3*mu_3 + dtheta_3*(dtheta_3*l_2*l_3*m_4*np.sin(theta_3 - theta_2) + dtheta_3*l_2*l_3*m_5*np.sin(theta_3 - theta_2) + dtheta_3*l_2*m_3*r_3*np.sin(theta_3 - theta_2)) - dtheta_2*(mu_2 + mu_3) + dtheta_1*mu_2 + dtheta_1*(-dtheta_1*l_1*l_2*m_3*np.sin(theta_2 - theta_1) - dtheta_1*l_1*l_2*m_4*np.sin(theta_2 - theta_1) - dtheta_1*l_1*l_2*m_5*np.sin(theta_2 - theta_1) - dtheta_1*l_1*m_2*r_2*np.sin(theta_2 - theta_1)) - ddx_c*(-l_2*m_3 - l_2*m_4 - l_2*m_5 - m_2*r_2)*np.cos(theta_2) - g*(-l_2*m_3 - l_2*m_4 - l_2*m_5 - m_2*r_2)*np.sin(theta_2) - l_1*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*(dtheta_5**2*l_1*m_5*r_5*np.sin(theta_5 - theta_1) + dtheta_4*(dtheta_4*l_1*l_4*m_5*np.sin(theta_4 - theta_1) + dtheta_4*l_1*m_4*r_4*np.sin(theta_4 - theta_1)) + dtheta_3*(dtheta_3*l_1*l_3*m_4*np.sin(theta_3 - theta_1) + dtheta_3*l_1*l_3*m_5*np.sin(theta_3 - theta_1) + dtheta_3*l_1*m_3*r_3*np.sin(theta_3 - theta_1)) + dtheta_2*mu_2 + dtheta_2*(dtheta_2*l_1*l_2*m_3*np.sin(theta_2 - theta_1) + dtheta_2*l_1*l_2*m_4*np.sin(theta_2 - theta_1) + dtheta_2*l_1*l_2*m_5*np.sin(theta_2 - theta_1) + dtheta_2*l_1*m_2*r_2*np.sin(theta_2 - theta_1)) - dtheta_1*(mu_1 + mu_2) - ddx_c*(-l_1*m_2 - l_1*m_3 - l_1*m_4 - l_1*m_5 - m_1*r_1)*np.cos(theta_1) - g*(-l_1*m_2 - l_1*m_3 - l_1*m_4 - l_1*m_5 - m_1*r_1)*np.sin(theta_1))*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2))/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2) - (-l_1**2*m_5*r_5*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_5 - theta_1)*np.cos(theta_3 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_3*m_5*r_5*np.cos(theta_5 - theta_3) - (-l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_3 - theta_2))*(-l_1**2*m_5*r_5*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_5 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*m_5*r_5*np.cos(theta_5 - theta_2))/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2))*(dtheta_5**2*l_3*m_5*r_5*np.sin(theta_5 - theta_3) + dtheta_4*mu_4 + dtheta_4*(dtheta_4*l_3*l_4*m_5*np.sin(theta_4 - theta_3) + dtheta_4*l_3*m_4*r_4*np.sin(theta_4 - theta_3)) - dtheta_3*(mu_3 + mu_4) + dtheta_2*mu_3 + dtheta_2*(-dtheta_2*l_2*l_3*m_4*np.sin(theta_3 - theta_2) - dtheta_2*l_2*l_3*m_5*np.sin(theta_3 - theta_2) - dtheta_2*l_2*m_3*r_3*np.sin(theta_3 - theta_2)) + dtheta_1*(-dtheta_1*l_1*l_3*m_4*np.sin(theta_3 - theta_1) - dtheta_1*l_1*l_3*m_5*np.sin(theta_3 - theta_1) - dtheta_1*l_1*m_3*r_3*np.sin(theta_3 - theta_1)) - ddx_c*(-l_3*m_4 - l_3*m_5 - m_3*r_3)*np.cos(theta_3) - g*(-l_3*m_4 - l_3*m_5 - m_3*r_3)*np.sin(theta_3) - l_1*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(dtheta_5**2*l_1*m_5*r_5*np.sin(theta_5 - theta_1) + dtheta_4*(dtheta_4*l_1*l_4*m_5*np.sin(theta_4 - theta_1) + dtheta_4*l_1*m_4*r_4*np.sin(theta_4 - theta_1)) + dtheta_3*(dtheta_3*l_1*l_3*m_4*np.sin(theta_3 - theta_1) + dtheta_3*l_1*l_3*m_5*np.sin(theta_3 - theta_1) + dtheta_3*l_1*m_3*r_3*np.sin(theta_3 - theta_1)) + dtheta_2*mu_2 + dtheta_2*(dtheta_2*l_1*l_2*m_3*np.sin(theta_2 - theta_1) + dtheta_2*l_1*l_2*m_4*np.sin(theta_2 - theta_1) + dtheta_2*l_1*l_2*m_5*np.sin(theta_2 - theta_1) + dtheta_2*l_1*m_2*r_2*np.sin(theta_2 - theta_1)) - dtheta_1*(mu_1 + mu_2) - ddx_c*(-l_1*m_2 - l_1*m_3 - l_1*m_4 - l_1*m_5 - m_1*r_1)*np.cos(theta_1) - g*(-l_1*m_2 - l_1*m_3 - l_1*m_4 - l_1*m_5 - m_1*r_1)*np.sin(theta_1))*np.cos(theta_3 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) - (-l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_3 - theta_2))*(dtheta_5**2*l_2*m_5*r_5*np.sin(theta_5 - theta_2) + dtheta_4*(dtheta_4*l_2*l_4*m_5*np.sin(theta_4 - theta_2) + dtheta_4*l_2*m_4*r_4*np.sin(theta_4 - theta_2)) + dtheta_3*mu_3 + dtheta_3*(dtheta_3*l_2*l_3*m_4*np.sin(theta_3 - theta_2) + dtheta_3*l_2*l_3*m_5*np.sin(theta_3 - theta_2) + dtheta_3*l_2*m_3*r_3*np.sin(theta_3 - theta_2)) - dtheta_2*(mu_2 + mu_3) + dtheta_1*mu_2 + dtheta_1*(-dtheta_1*l_1*l_2*m_3*np.sin(theta_2 - theta_1) - dtheta_1*l_1*l_2*m_4*np.sin(theta_2 - theta_1) - dtheta_1*l_1*l_2*m_5*np.sin(theta_2 - theta_1) - dtheta_1*l_1*m_2*r_2*np.sin(theta_2 - theta_1)) - ddx_c*(-l_2*m_3 - l_2*m_4 - l_2*m_5 - m_2*r_2)*np.cos(theta_2) - g*(-l_2*m_3 - l_2*m_4 - l_2*m_5 - m_2*r_2)*np.sin(theta_2) - l_1*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*(dtheta_5**2*l_1*m_5*r_5*np.sin(theta_5 - theta_1) + dtheta_4*(dtheta_4*l_1*l_4*m_5*np.sin(theta_4 - theta_1) + dtheta_4*l_1*m_4*r_4*np.sin(theta_4 - theta_1)) + dtheta_3*(dtheta_3*l_1*l_3*m_4*np.sin(theta_3 - theta_1) + dtheta_3*l_1*l_3*m_5*np.sin(theta_3 - theta_1) + dtheta_3*l_1*m_3*r_3*np.sin(theta_3 - theta_1)) + dtheta_2*mu_2 + dtheta_2*(dtheta_2*l_1*l_2*m_3*np.sin(theta_2 - theta_1) + dtheta_2*l_1*l_2*m_4*np.sin(theta_2 - theta_1) + dtheta_2*l_1*l_2*m_5*np.sin(theta_2 - theta_1) + dtheta_2*l_1*m_2*r_2*np.sin(theta_2 - theta_1)) - dtheta_1*(mu_1 + mu_2) - ddx_c*(-l_1*m_2 - l_1*m_3 - l_1*m_4 - l_1*m_5 - m_1*r_1)*np.cos(theta_1) - g*(-l_1*m_2 - l_1*m_3 - l_1*m_4 - l_1*m_5 - m_1*r_1)*np.sin(theta_1))*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2))/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2))/(I_3 - l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)**2*np.cos(theta_3 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_3**2*m_4 + l_3**2*m_5 + m_3*r_3**2 - (-l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_3 - theta_2))**2/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2)) - (-l_1**2*m_5*r_5*(l_4*m_5 + m_4*r_4)*np.cos(theta_5 - theta_1)*np.cos(theta_4 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_4*m_5*r_5*np.cos(theta_5 - theta_4) - (-l_1**2*(l_4*m_5 + m_4*r_4)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_4 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_4*m_5 + m_4*r_4)*np.cos(theta_4 - theta_2))*(-l_1**2*m_5*r_5*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_5 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*m_5*r_5*np.cos(theta_5 - theta_2))/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2) - (-l_1**2*(l_4*m_5 + m_4*r_4)*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_4 - theta_1)*np.cos(theta_3 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_3*(l_4*m_5 + m_4*r_4)*np.cos(theta_4 - theta_3) - (-l_1**2*(l_4*m_5 + m_4*r_4)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_4 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_4*m_5 + m_4*r_4)*np.cos(theta_4 - theta_2))*(-l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_3 - theta_2))/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2))*(-l_1**2*m_5*r_5*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_5 - theta_1)*np.cos(theta_3 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_3*m_5*r_5*np.cos(theta_5 - theta_3) - (-l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_3 - theta_2))*(-l_1**2*m_5*r_5*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_5 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*m_5*r_5*np.cos(theta_5 - theta_2))/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2))/(I_3 - l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)**2*np.cos(theta_3 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_3**2*m_4 + l_3**2*m_5 + m_3*r_3**2 - (-l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_3 - theta_2))**2/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2)))*(dtheta_5**2*l_4*m_5*r_5*np.sin(theta_5 - theta_4) + dtheta_5*mu_5 - dtheta_4*(mu_4 + mu_5) + dtheta_3*mu_4 + dtheta_3*(-dtheta_3*l_3*l_4*m_5*np.sin(theta_4 - theta_3) - dtheta_3*l_3*m_4*r_4*np.sin(theta_4 - theta_3)) + dtheta_2*(-dtheta_2*l_2*l_4*m_5*np.sin(theta_4 - theta_2) - dtheta_2*l_2*m_4*r_4*np.sin(theta_4 - theta_2)) + dtheta_1*(-dtheta_1*l_1*l_4*m_5*np.sin(theta_4 - theta_1) - dtheta_1*l_1*m_4*r_4*np.sin(theta_4 - theta_1)) - ddx_c*(-l_4*m_5 - m_4*r_4)*np.cos(theta_4) - g*(-l_4*m_5 - m_4*r_4)*np.sin(theta_4) - l_1*(l_4*m_5 + m_4*r_4)*(dtheta_5**2*l_1*m_5*r_5*np.sin(theta_5 - theta_1) + dtheta_4*(dtheta_4*l_1*l_4*m_5*np.sin(theta_4 - theta_1) + dtheta_4*l_1*m_4*r_4*np.sin(theta_4 - theta_1)) + dtheta_3*(dtheta_3*l_1*l_3*m_4*np.sin(theta_3 - theta_1) + dtheta_3*l_1*l_3*m_5*np.sin(theta_3 - theta_1) + dtheta_3*l_1*m_3*r_3*np.sin(theta_3 - theta_1)) + dtheta_2*mu_2 + dtheta_2*(dtheta_2*l_1*l_2*m_3*np.sin(theta_2 - theta_1) + dtheta_2*l_1*l_2*m_4*np.sin(theta_2 - theta_1) + dtheta_2*l_1*l_2*m_5*np.sin(theta_2 - theta_1) + dtheta_2*l_1*m_2*r_2*np.sin(theta_2 - theta_1)) - dtheta_1*(mu_1 + mu_2) - ddx_c*(-l_1*m_2 - l_1*m_3 - l_1*m_4 - l_1*m_5 - m_1*r_1)*np.cos(theta_1) - g*(-l_1*m_2 - l_1*m_3 - l_1*m_4 - l_1*m_5 - m_1*r_1)*np.sin(theta_1))*np.cos(theta_4 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) - (-l_1**2*(l_4*m_5 + m_4*r_4)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_4 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_4*m_5 + m_4*r_4)*np.cos(theta_4 - theta_2))*(dtheta_5**2*l_2*m_5*r_5*np.sin(theta_5 - theta_2) + dtheta_4*(dtheta_4*l_2*l_4*m_5*np.sin(theta_4 - theta_2) + dtheta_4*l_2*m_4*r_4*np.sin(theta_4 - theta_2)) + dtheta_3*mu_3 + dtheta_3*(dtheta_3*l_2*l_3*m_4*np.sin(theta_3 - theta_2) + dtheta_3*l_2*l_3*m_5*np.sin(theta_3 - theta_2) + dtheta_3*l_2*m_3*r_3*np.sin(theta_3 - theta_2)) - dtheta_2*(mu_2 + mu_3) + dtheta_1*mu_2 + dtheta_1*(-dtheta_1*l_1*l_2*m_3*np.sin(theta_2 - theta_1) - dtheta_1*l_1*l_2*m_4*np.sin(theta_2 - theta_1) - dtheta_1*l_1*l_2*m_5*np.sin(theta_2 - theta_1) - dtheta_1*l_1*m_2*r_2*np.sin(theta_2 - theta_1)) - ddx_c*(-l_2*m_3 - l_2*m_4 - l_2*m_5 - m_2*r_2)*np.cos(theta_2) - g*(-l_2*m_3 - l_2*m_4 - l_2*m_5 - m_2*r_2)*np.sin(theta_2) - l_1*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*(dtheta_5**2*l_1*m_5*r_5*np.sin(theta_5 - theta_1) + dtheta_4*(dtheta_4*l_1*l_4*m_5*np.sin(theta_4 - theta_1) + dtheta_4*l_1*m_4*r_4*np.sin(theta_4 - theta_1)) + dtheta_3*(dtheta_3*l_1*l_3*m_4*np.sin(theta_3 - theta_1) + dtheta_3*l_1*l_3*m_5*np.sin(theta_3 - theta_1) + dtheta_3*l_1*m_3*r_3*np.sin(theta_3 - theta_1)) + dtheta_2*mu_2 + dtheta_2*(dtheta_2*l_1*l_2*m_3*np.sin(theta_2 - theta_1) + dtheta_2*l_1*l_2*m_4*np.sin(theta_2 - theta_1) + dtheta_2*l_1*l_2*m_5*np.sin(theta_2 - theta_1) + dtheta_2*l_1*m_2*r_2*np.sin(theta_2 - theta_1)) - dtheta_1*(mu_1 + mu_2) - ddx_c*(-l_1*m_2 - l_1*m_3 - l_1*m_4 - l_1*m_5 - m_1*r_1)*np.cos(theta_1) - g*(-l_1*m_2 - l_1*m_3 - l_1*m_4 - l_1*m_5 - m_1*r_1)*np.sin(theta_1))*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2))/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2) - (-l_1**2*(l_4*m_5 + m_4*r_4)*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_4 - theta_1)*np.cos(theta_3 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_3*(l_4*m_5 + m_4*r_4)*np.cos(theta_4 - theta_3) - (-l_1**2*(l_4*m_5 + m_4*r_4)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_4 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_4*m_5 + m_4*r_4)*np.cos(theta_4 - theta_2))*(-l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_3 - theta_2))/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2))*(dtheta_5**2*l_3*m_5*r_5*np.sin(theta_5 - theta_3) + dtheta_4*mu_4 + dtheta_4*(dtheta_4*l_3*l_4*m_5*np.sin(theta_4 - theta_3) + dtheta_4*l_3*m_4*r_4*np.sin(theta_4 - theta_3)) - dtheta_3*(mu_3 + mu_4) + dtheta_2*mu_3 + dtheta_2*(-dtheta_2*l_2*l_3*m_4*np.sin(theta_3 - theta_2) - dtheta_2*l_2*l_3*m_5*np.sin(theta_3 - theta_2) - dtheta_2*l_2*m_3*r_3*np.sin(theta_3 - theta_2)) + dtheta_1*(-dtheta_1*l_1*l_3*m_4*np.sin(theta_3 - theta_1) - dtheta_1*l_1*l_3*m_5*np.sin(theta_3 - theta_1) - dtheta_1*l_1*m_3*r_3*np.sin(theta_3 - theta_1)) - ddx_c*(-l_3*m_4 - l_3*m_5 - m_3*r_3)*np.cos(theta_3) - g*(-l_3*m_4 - l_3*m_5 - m_3*r_3)*np.sin(theta_3) - l_1*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(dtheta_5**2*l_1*m_5*r_5*np.sin(theta_5 - theta_1) + dtheta_4*(dtheta_4*l_1*l_4*m_5*np.sin(theta_4 - theta_1) + dtheta_4*l_1*m_4*r_4*np.sin(theta_4 - theta_1)) + dtheta_3*(dtheta_3*l_1*l_3*m_4*np.sin(theta_3 - theta_1) + dtheta_3*l_1*l_3*m_5*np.sin(theta_3 - theta_1) + dtheta_3*l_1*m_3*r_3*np.sin(theta_3 - theta_1)) + dtheta_2*mu_2 + dtheta_2*(dtheta_2*l_1*l_2*m_3*np.sin(theta_2 - theta_1) + dtheta_2*l_1*l_2*m_4*np.sin(theta_2 - theta_1) + dtheta_2*l_1*l_2*m_5*np.sin(theta_2 - theta_1) + dtheta_2*l_1*m_2*r_2*np.sin(theta_2 - theta_1)) - dtheta_1*(mu_1 + mu_2) - ddx_c*(-l_1*m_2 - l_1*m_3 - l_1*m_4 - l_1*m_5 - m_1*r_1)*np.cos(theta_1) - g*(-l_1*m_2 - l_1*m_3 - l_1*m_4 - l_1*m_5 - m_1*r_1)*np.sin(theta_1))*np.cos(theta_3 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) - (-l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_3 - theta_2))*(dtheta_5**2*l_2*m_5*r_5*np.sin(theta_5 - theta_2) + dtheta_4*(dtheta_4*l_2*l_4*m_5*np.sin(theta_4 - theta_2) + dtheta_4*l_2*m_4*r_4*np.sin(theta_4 - theta_2)) + dtheta_3*mu_3 + dtheta_3*(dtheta_3*l_2*l_3*m_4*np.sin(theta_3 - theta_2) + dtheta_3*l_2*l_3*m_5*np.sin(theta_3 - theta_2) + dtheta_3*l_2*m_3*r_3*np.sin(theta_3 - theta_2)) - dtheta_2*(mu_2 + mu_3) + dtheta_1*mu_2 + dtheta_1*(-dtheta_1*l_1*l_2*m_3*np.sin(theta_2 - theta_1) - dtheta_1*l_1*l_2*m_4*np.sin(theta_2 - theta_1) - dtheta_1*l_1*l_2*m_5*np.sin(theta_2 - theta_1) - dtheta_1*l_1*m_2*r_2*np.sin(theta_2 - theta_1)) - ddx_c*(-l_2*m_3 - l_2*m_4 - l_2*m_5 - m_2*r_2)*np.cos(theta_2) - g*(-l_2*m_3 - l_2*m_4 - l_2*m_5 - m_2*r_2)*np.sin(theta_2) - l_1*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*(dtheta_5**2*l_1*m_5*r_5*np.sin(theta_5 - theta_1) + dtheta_4*(dtheta_4*l_1*l_4*m_5*np.sin(theta_4 - theta_1) + dtheta_4*l_1*m_4*r_4*np.sin(theta_4 - theta_1)) + dtheta_3*(dtheta_3*l_1*l_3*m_4*np.sin(theta_3 - theta_1) + dtheta_3*l_1*l_3*m_5*np.sin(theta_3 - theta_1) + dtheta_3*l_1*m_3*r_3*np.sin(theta_3 - theta_1)) + dtheta_2*mu_2 + dtheta_2*(dtheta_2*l_1*l_2*m_3*np.sin(theta_2 - theta_1) + dtheta_2*l_1*l_2*m_4*np.sin(theta_2 - theta_1) + dtheta_2*l_1*l_2*m_5*np.sin(theta_2 - theta_1) + dtheta_2*l_1*m_2*r_2*np.sin(theta_2 - theta_1)) - dtheta_1*(mu_1 + mu_2) - ddx_c*(-l_1*m_2 - l_1*m_3 - l_1*m_4 - l_1*m_5 - m_1*r_1)*np.cos(theta_1) - g*(-l_1*m_2 - l_1*m_3 - l_1*m_4 - l_1*m_5 - m_1*r_1)*np.sin(theta_1))*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2))/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2))/(I_3 - l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)**2*np.cos(theta_3 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_3**2*m_4 + l_3**2*m_5 + m_3*r_3**2 - (-l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_3 - theta_2))**2/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2)))/(I_4 - l_1**2*(l_4*m_5 + m_4*r_4)**2*np.cos(theta_4 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_4**2*m_5 + m_4*r_4**2 - (-l_1**2*(l_4*m_5 + m_4*r_4)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_4 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_4*m_5 + m_4*r_4)*np.cos(theta_4 - theta_2))**2/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2) - (-l_1**2*(l_4*m_5 + m_4*r_4)*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_4 - theta_1)*np.cos(theta_3 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_3*(l_4*m_5 + m_4*r_4)*np.cos(theta_4 - theta_3) - (-l_1**2*(l_4*m_5 + m_4*r_4)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_4 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_4*m_5 + m_4*r_4)*np.cos(theta_4 - theta_2))*(-l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_3 - theta_2))/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2))**2/(I_3 - l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)**2*np.cos(theta_3 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_3**2*m_4 + l_3**2*m_5 + m_3*r_3**2 - (-l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_3 - theta_2))**2/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2))))*np.cos(theta_5 - theta_1)/(I_5 - l_1**2*m_5**2*r_5**2*np.cos(theta_5 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + m_5*r_5**2 - (-l_1**2*m_5*r_5*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_5 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*m_5*r_5*np.cos(theta_5 - theta_2))**2/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2) - (-l_1**2*m_5*r_5*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_5 - theta_1)*np.cos(theta_3 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_3*m_5*r_5*np.cos(theta_5 - theta_3) - (-l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_3 - theta_2))*(-l_1**2*m_5*r_5*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_5 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*m_5*r_5*np.cos(theta_5 - theta_2))/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2))**2/(I_3 - l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)**2*np.cos(theta_3 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_3**2*m_4 + l_3**2*m_5 + m_3*r_3**2 - (-l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_3 - theta_2))**2/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2)) - (-l_1**2*m_5*r_5*(l_4*m_5 + m_4*r_4)*np.cos(theta_5 - theta_1)*np.cos(theta_4 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_4*m_5*r_5*np.cos(theta_5 - theta_4) - (-l_1**2*(l_4*m_5 + m_4*r_4)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_4 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_4*m_5 + m_4*r_4)*np.cos(theta_4 - theta_2))*(-l_1**2*m_5*r_5*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_5 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*m_5*r_5*np.cos(theta_5 - theta_2))/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2) - (-l_1**2*(l_4*m_5 + m_4*r_4)*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_4 - theta_1)*np.cos(theta_3 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_3*(l_4*m_5 + m_4*r_4)*np.cos(theta_4 - theta_3) - (-l_1**2*(l_4*m_5 + m_4*r_4)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_4 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_4*m_5 + m_4*r_4)*np.cos(theta_4 - theta_2))*(-l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_3 - theta_2))/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2))*(-l_1**2*m_5*r_5*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_5 - theta_1)*np.cos(theta_3 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_3*m_5*r_5*np.cos(theta_5 - theta_3) - (-l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_3 - theta_2))*(-l_1**2*m_5*r_5*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_5 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*m_5*r_5*np.cos(theta_5 - theta_2))/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2))/(I_3 - l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)**2*np.cos(theta_3 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_3**2*m_4 + l_3**2*m_5 + m_3*r_3**2 - (-l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_3 - theta_2))**2/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2)))**2/(I_4 - l_1**2*(l_4*m_5 + m_4*r_4)**2*np.cos(theta_4 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_4**2*m_5 + m_4*r_4**2 - (-l_1**2*(l_4*m_5 + m_4*r_4)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_4 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_4*m_5 + m_4*r_4)*np.cos(theta_4 - theta_2))**2/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2) - (-l_1**2*(l_4*m_5 + m_4*r_4)*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_4 - theta_1)*np.cos(theta_3 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_3*(l_4*m_5 + m_4*r_4)*np.cos(theta_4 - theta_3) - (-l_1**2*(l_4*m_5 + m_4*r_4)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_4 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_4*m_5 + m_4*r_4)*np.cos(theta_4 - theta_2))*(-l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_3 - theta_2))/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2))**2/(I_3 - l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)**2*np.cos(theta_3 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_3**2*m_4 + l_3**2*m_5 + m_3*r_3**2 - (-l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_3 - theta_2))**2/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2)))) - l_1*(l_4*m_5 + m_4*r_4)*(dtheta_5**2*l_4*m_5*r_5*np.sin(theta_5 - theta_4) + dtheta_5*mu_5 - dtheta_4*(mu_4 + mu_5) + dtheta_3*mu_4 + dtheta_3*(-dtheta_3*l_3*l_4*m_5*np.sin(theta_4 - theta_3) - dtheta_3*l_3*m_4*r_4*np.sin(theta_4 - theta_3)) + dtheta_2*(-dtheta_2*l_2*l_4*m_5*np.sin(theta_4 - theta_2) - dtheta_2*l_2*m_4*r_4*np.sin(theta_4 - theta_2)) + dtheta_1*(-dtheta_1*l_1*l_4*m_5*np.sin(theta_4 - theta_1) - dtheta_1*l_1*m_4*r_4*np.sin(theta_4 - theta_1)) - ddx_c*(-l_4*m_5 - m_4*r_4)*np.cos(theta_4) - g*(-l_4*m_5 - m_4*r_4)*np.sin(theta_4) - l_1*(l_4*m_5 + m_4*r_4)*(dtheta_5**2*l_1*m_5*r_5*np.sin(theta_5 - theta_1) + dtheta_4*(dtheta_4*l_1*l_4*m_5*np.sin(theta_4 - theta_1) + dtheta_4*l_1*m_4*r_4*np.sin(theta_4 - theta_1)) + dtheta_3*(dtheta_3*l_1*l_3*m_4*np.sin(theta_3 - theta_1) + dtheta_3*l_1*l_3*m_5*np.sin(theta_3 - theta_1) + dtheta_3*l_1*m_3*r_3*np.sin(theta_3 - theta_1)) + dtheta_2*mu_2 + dtheta_2*(dtheta_2*l_1*l_2*m_3*np.sin(theta_2 - theta_1) + dtheta_2*l_1*l_2*m_4*np.sin(theta_2 - theta_1) + dtheta_2*l_1*l_2*m_5*np.sin(theta_2 - theta_1) + dtheta_2*l_1*m_2*r_2*np.sin(theta_2 - theta_1)) - dtheta_1*(mu_1 + mu_2) - ddx_c*(-l_1*m_2 - l_1*m_3 - l_1*m_4 - l_1*m_5 - m_1*r_1)*np.cos(theta_1) - g*(-l_1*m_2 - l_1*m_3 - l_1*m_4 - l_1*m_5 - m_1*r_1)*np.sin(theta_1))*np.cos(theta_4 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) - (-l_1**2*(l_4*m_5 + m_4*r_4)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_4 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_4*m_5 + m_4*r_4)*np.cos(theta_4 - theta_2))*(dtheta_5**2*l_2*m_5*r_5*np.sin(theta_5 - theta_2) + dtheta_4*(dtheta_4*l_2*l_4*m_5*np.sin(theta_4 - theta_2) + dtheta_4*l_2*m_4*r_4*np.sin(theta_4 - theta_2)) + dtheta_3*mu_3 + dtheta_3*(dtheta_3*l_2*l_3*m_4*np.sin(theta_3 - theta_2) + dtheta_3*l_2*l_3*m_5*np.sin(theta_3 - theta_2) + dtheta_3*l_2*m_3*r_3*np.sin(theta_3 - theta_2)) - dtheta_2*(mu_2 + mu_3) + dtheta_1*mu_2 + dtheta_1*(-dtheta_1*l_1*l_2*m_3*np.sin(theta_2 - theta_1) - dtheta_1*l_1*l_2*m_4*np.sin(theta_2 - theta_1) - dtheta_1*l_1*l_2*m_5*np.sin(theta_2 - theta_1) - dtheta_1*l_1*m_2*r_2*np.sin(theta_2 - theta_1)) - ddx_c*(-l_2*m_3 - l_2*m_4 - l_2*m_5 - m_2*r_2)*np.cos(theta_2) - g*(-l_2*m_3 - l_2*m_4 - l_2*m_5 - m_2*r_2)*np.sin(theta_2) - l_1*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*(dtheta_5**2*l_1*m_5*r_5*np.sin(theta_5 - theta_1) + dtheta_4*(dtheta_4*l_1*l_4*m_5*np.sin(theta_4 - theta_1) + dtheta_4*l_1*m_4*r_4*np.sin(theta_4 - theta_1)) + dtheta_3*(dtheta_3*l_1*l_3*m_4*np.sin(theta_3 - theta_1) + dtheta_3*l_1*l_3*m_5*np.sin(theta_3 - theta_1) + dtheta_3*l_1*m_3*r_3*np.sin(theta_3 - theta_1)) + dtheta_2*mu_2 + dtheta_2*(dtheta_2*l_1*l_2*m_3*np.sin(theta_2 - theta_1) + dtheta_2*l_1*l_2*m_4*np.sin(theta_2 - theta_1) + dtheta_2*l_1*l_2*m_5*np.sin(theta_2 - theta_1) + dtheta_2*l_1*m_2*r_2*np.sin(theta_2 - theta_1)) - dtheta_1*(mu_1 + mu_2) - ddx_c*(-l_1*m_2 - l_1*m_3 - l_1*m_4 - l_1*m_5 - m_1*r_1)*np.cos(theta_1) - g*(-l_1*m_2 - l_1*m_3 - l_1*m_4 - l_1*m_5 - m_1*r_1)*np.sin(theta_1))*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2))/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2) - (-l_1**2*(l_4*m_5 + m_4*r_4)*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_4 - theta_1)*np.cos(theta_3 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_3*(l_4*m_5 + m_4*r_4)*np.cos(theta_4 - theta_3) - (-l_1**2*(l_4*m_5 + m_4*r_4)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_4 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_4*m_5 + m_4*r_4)*np.cos(theta_4 - theta_2))*(-l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_3 - theta_2))/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2))*(dtheta_5**2*l_3*m_5*r_5*np.sin(theta_5 - theta_3) + dtheta_4*mu_4 + dtheta_4*(dtheta_4*l_3*l_4*m_5*np.sin(theta_4 - theta_3) + dtheta_4*l_3*m_4*r_4*np.sin(theta_4 - theta_3)) - dtheta_3*(mu_3 + mu_4) + dtheta_2*mu_3 + dtheta_2*(-dtheta_2*l_2*l_3*m_4*np.sin(theta_3 - theta_2) - dtheta_2*l_2*l_3*m_5*np.sin(theta_3 - theta_2) - dtheta_2*l_2*m_3*r_3*np.sin(theta_3 - theta_2)) + dtheta_1*(-dtheta_1*l_1*l_3*m_4*np.sin(theta_3 - theta_1) - dtheta_1*l_1*l_3*m_5*np.sin(theta_3 - theta_1) - dtheta_1*l_1*m_3*r_3*np.sin(theta_3 - theta_1)) - ddx_c*(-l_3*m_4 - l_3*m_5 - m_3*r_3)*np.cos(theta_3) - g*(-l_3*m_4 - l_3*m_5 - m_3*r_3)*np.sin(theta_3) - l_1*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(dtheta_5**2*l_1*m_5*r_5*np.sin(theta_5 - theta_1) + dtheta_4*(dtheta_4*l_1*l_4*m_5*np.sin(theta_4 - theta_1) + dtheta_4*l_1*m_4*r_4*np.sin(theta_4 - theta_1)) + dtheta_3*(dtheta_3*l_1*l_3*m_4*np.sin(theta_3 - theta_1) + dtheta_3*l_1*l_3*m_5*np.sin(theta_3 - theta_1) + dtheta_3*l_1*m_3*r_3*np.sin(theta_3 - theta_1)) + dtheta_2*mu_2 + dtheta_2*(dtheta_2*l_1*l_2*m_3*np.sin(theta_2 - theta_1) + dtheta_2*l_1*l_2*m_4*np.sin(theta_2 - theta_1) + dtheta_2*l_1*l_2*m_5*np.sin(theta_2 - theta_1) + dtheta_2*l_1*m_2*r_2*np.sin(theta_2 - theta_1)) - dtheta_1*(mu_1 + mu_2) - ddx_c*(-l_1*m_2 - l_1*m_3 - l_1*m_4 - l_1*m_5 - m_1*r_1)*np.cos(theta_1) - g*(-l_1*m_2 - l_1*m_3 - l_1*m_4 - l_1*m_5 - m_1*r_1)*np.sin(theta_1))*np.cos(theta_3 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) - (-l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_3 - theta_2))*(dtheta_5**2*l_2*m_5*r_5*np.sin(theta_5 - theta_2) + dtheta_4*(dtheta_4*l_2*l_4*m_5*np.sin(theta_4 - theta_2) + dtheta_4*l_2*m_4*r_4*np.sin(theta_4 - theta_2)) + dtheta_3*mu_3 + dtheta_3*(dtheta_3*l_2*l_3*m_4*np.sin(theta_3 - theta_2) + dtheta_3*l_2*l_3*m_5*np.sin(theta_3 - theta_2) + dtheta_3*l_2*m_3*r_3*np.sin(theta_3 - theta_2)) - dtheta_2*(mu_2 + mu_3) + dtheta_1*mu_2 + dtheta_1*(-dtheta_1*l_1*l_2*m_3*np.sin(theta_2 - theta_1) - dtheta_1*l_1*l_2*m_4*np.sin(theta_2 - theta_1) - dtheta_1*l_1*l_2*m_5*np.sin(theta_2 - theta_1) - dtheta_1*l_1*m_2*r_2*np.sin(theta_2 - theta_1)) - ddx_c*(-l_2*m_3 - l_2*m_4 - l_2*m_5 - m_2*r_2)*np.cos(theta_2) - g*(-l_2*m_3 - l_2*m_4 - l_2*m_5 - m_2*r_2)*np.sin(theta_2) - l_1*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*(dtheta_5**2*l_1*m_5*r_5*np.sin(theta_5 - theta_1) + dtheta_4*(dtheta_4*l_1*l_4*m_5*np.sin(theta_4 - theta_1) + dtheta_4*l_1*m_4*r_4*np.sin(theta_4 - theta_1)) + dtheta_3*(dtheta_3*l_1*l_3*m_4*np.sin(theta_3 - theta_1) + dtheta_3*l_1*l_3*m_5*np.sin(theta_3 - theta_1) + dtheta_3*l_1*m_3*r_3*np.sin(theta_3 - theta_1)) + dtheta_2*mu_2 + dtheta_2*(dtheta_2*l_1*l_2*m_3*np.sin(theta_2 - theta_1) + dtheta_2*l_1*l_2*m_4*np.sin(theta_2 - theta_1) + dtheta_2*l_1*l_2*m_5*np.sin(theta_2 - theta_1) + dtheta_2*l_1*m_2*r_2*np.sin(theta_2 - theta_1)) - dtheta_1*(mu_1 + mu_2) - ddx_c*(-l_1*m_2 - l_1*m_3 - l_1*m_4 - l_1*m_5 - m_1*r_1)*np.cos(theta_1) - g*(-l_1*m_2 - l_1*m_3 - l_1*m_4 - l_1*m_5 - m_1*r_1)*np.sin(theta_1))*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2))/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2))/(I_3 - l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)**2*np.cos(theta_3 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_3**2*m_4 + l_3**2*m_5 + m_3*r_3**2 - (-l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_3 - theta_2))**2/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2)) - (-l_1**2*m_5*r_5*(l_4*m_5 + m_4*r_4)*np.cos(theta_5 - theta_1)*np.cos(theta_4 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_4*m_5*r_5*np.cos(theta_5 - theta_4) - (-l_1**2*(l_4*m_5 + m_4*r_4)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_4 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_4*m_5 + m_4*r_4)*np.cos(theta_4 - theta_2))*(-l_1**2*m_5*r_5*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_5 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*m_5*r_5*np.cos(theta_5 - theta_2))/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2) - (-l_1**2*(l_4*m_5 + m_4*r_4)*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_4 - theta_1)*np.cos(theta_3 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_3*(l_4*m_5 + m_4*r_4)*np.cos(theta_4 - theta_3) - (-l_1**2*(l_4*m_5 + m_4*r_4)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_4 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_4*m_5 + m_4*r_4)*np.cos(theta_4 - theta_2))*(-l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_3 - theta_2))/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2))*(-l_1**2*m_5*r_5*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_5 - theta_1)*np.cos(theta_3 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_3*m_5*r_5*np.cos(theta_5 - theta_3) - (-l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_3 - theta_2))*(-l_1**2*m_5*r_5*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_5 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*m_5*r_5*np.cos(theta_5 - theta_2))/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2))/(I_3 - l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)**2*np.cos(theta_3 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_3**2*m_4 + l_3**2*m_5 + m_3*r_3**2 - (-l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_3 - theta_2))**2/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2)))*(-dtheta_5*mu_5 - dtheta_4**2*l_4*m_5*r_5*np.sin(theta_5 - theta_4) + dtheta_4*mu_5 - dtheta_3**2*l_3*m_5*r_5*np.sin(theta_5 - theta_3) - dtheta_2**2*l_2*m_5*r_5*np.sin(theta_5 - theta_2) - dtheta_1**2*l_1*m_5*r_5*np.sin(theta_5 - theta_1) + ddx_c*m_5*r_5*np.cos(theta_5) + g*m_5*r_5*np.sin(theta_5) - l_1*m_5*r_5*(dtheta_5**2*l_1*m_5*r_5*np.sin(theta_5 - theta_1) + dtheta_4*(dtheta_4*l_1*l_4*m_5*np.sin(theta_4 - theta_1) + dtheta_4*l_1*m_4*r_4*np.sin(theta_4 - theta_1)) + dtheta_3*(dtheta_3*l_1*l_3*m_4*np.sin(theta_3 - theta_1) + dtheta_3*l_1*l_3*m_5*np.sin(theta_3 - theta_1) + dtheta_3*l_1*m_3*r_3*np.sin(theta_3 - theta_1)) + dtheta_2*mu_2 + dtheta_2*(dtheta_2*l_1*l_2*m_3*np.sin(theta_2 - theta_1) + dtheta_2*l_1*l_2*m_4*np.sin(theta_2 - theta_1) + dtheta_2*l_1*l_2*m_5*np.sin(theta_2 - theta_1) + dtheta_2*l_1*m_2*r_2*np.sin(theta_2 - theta_1)) - dtheta_1*(mu_1 + mu_2) - ddx_c*(-l_1*m_2 - l_1*m_3 - l_1*m_4 - l_1*m_5 - m_1*r_1)*np.cos(theta_1) - g*(-l_1*m_2 - l_1*m_3 - l_1*m_4 - l_1*m_5 - m_1*r_1)*np.sin(theta_1))*np.cos(theta_5 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) - (-l_1**2*m_5*r_5*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_5 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*m_5*r_5*np.cos(theta_5 - theta_2))*(dtheta_5**2*l_2*m_5*r_5*np.sin(theta_5 - theta_2) + dtheta_4*(dtheta_4*l_2*l_4*m_5*np.sin(theta_4 - theta_2) + dtheta_4*l_2*m_4*r_4*np.sin(theta_4 - theta_2)) + dtheta_3*mu_3 + dtheta_3*(dtheta_3*l_2*l_3*m_4*np.sin(theta_3 - theta_2) + dtheta_3*l_2*l_3*m_5*np.sin(theta_3 - theta_2) + dtheta_3*l_2*m_3*r_3*np.sin(theta_3 - theta_2)) - dtheta_2*(mu_2 + mu_3) + dtheta_1*mu_2 + dtheta_1*(-dtheta_1*l_1*l_2*m_3*np.sin(theta_2 - theta_1) - dtheta_1*l_1*l_2*m_4*np.sin(theta_2 - theta_1) - dtheta_1*l_1*l_2*m_5*np.sin(theta_2 - theta_1) - dtheta_1*l_1*m_2*r_2*np.sin(theta_2 - theta_1)) - ddx_c*(-l_2*m_3 - l_2*m_4 - l_2*m_5 - m_2*r_2)*np.cos(theta_2) - g*(-l_2*m_3 - l_2*m_4 - l_2*m_5 - m_2*r_2)*np.sin(theta_2) - l_1*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*(dtheta_5**2*l_1*m_5*r_5*np.sin(theta_5 - theta_1) + dtheta_4*(dtheta_4*l_1*l_4*m_5*np.sin(theta_4 - theta_1) + dtheta_4*l_1*m_4*r_4*np.sin(theta_4 - theta_1)) + dtheta_3*(dtheta_3*l_1*l_3*m_4*np.sin(theta_3 - theta_1) + dtheta_3*l_1*l_3*m_5*np.sin(theta_3 - theta_1) + dtheta_3*l_1*m_3*r_3*np.sin(theta_3 - theta_1)) + dtheta_2*mu_2 + dtheta_2*(dtheta_2*l_1*l_2*m_3*np.sin(theta_2 - theta_1) + dtheta_2*l_1*l_2*m_4*np.sin(theta_2 - theta_1) + dtheta_2*l_1*l_2*m_5*np.sin(theta_2 - theta_1) + dtheta_2*l_1*m_2*r_2*np.sin(theta_2 - theta_1)) - dtheta_1*(mu_1 + mu_2) - ddx_c*(-l_1*m_2 - l_1*m_3 - l_1*m_4 - l_1*m_5 - m_1*r_1)*np.cos(theta_1) - g*(-l_1*m_2 - l_1*m_3 - l_1*m_4 - l_1*m_5 - m_1*r_1)*np.sin(theta_1))*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2))/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2) - (-l_1**2*m_5*r_5*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_5 - theta_1)*np.cos(theta_3 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_3*m_5*r_5*np.cos(theta_5 - theta_3) - (-l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_3 - theta_2))*(-l_1**2*m_5*r_5*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_5 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*m_5*r_5*np.cos(theta_5 - theta_2))/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2))*(dtheta_5**2*l_3*m_5*r_5*np.sin(theta_5 - theta_3) + dtheta_4*mu_4 + dtheta_4*(dtheta_4*l_3*l_4*m_5*np.sin(theta_4 - theta_3) + dtheta_4*l_3*m_4*r_4*np.sin(theta_4 - theta_3)) - dtheta_3*(mu_3 + mu_4) + dtheta_2*mu_3 + dtheta_2*(-dtheta_2*l_2*l_3*m_4*np.sin(theta_3 - theta_2) - dtheta_2*l_2*l_3*m_5*np.sin(theta_3 - theta_2) - dtheta_2*l_2*m_3*r_3*np.sin(theta_3 - theta_2)) + dtheta_1*(-dtheta_1*l_1*l_3*m_4*np.sin(theta_3 - theta_1) - dtheta_1*l_1*l_3*m_5*np.sin(theta_3 - theta_1) - dtheta_1*l_1*m_3*r_3*np.sin(theta_3 - theta_1)) - ddx_c*(-l_3*m_4 - l_3*m_5 - m_3*r_3)*np.cos(theta_3) - g*(-l_3*m_4 - l_3*m_5 - m_3*r_3)*np.sin(theta_3) - l_1*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(dtheta_5**2*l_1*m_5*r_5*np.sin(theta_5 - theta_1) + dtheta_4*(dtheta_4*l_1*l_4*m_5*np.sin(theta_4 - theta_1) + dtheta_4*l_1*m_4*r_4*np.sin(theta_4 - theta_1)) + dtheta_3*(dtheta_3*l_1*l_3*m_4*np.sin(theta_3 - theta_1) + dtheta_3*l_1*l_3*m_5*np.sin(theta_3 - theta_1) + dtheta_3*l_1*m_3*r_3*np.sin(theta_3 - theta_1)) + dtheta_2*mu_2 + dtheta_2*(dtheta_2*l_1*l_2*m_3*np.sin(theta_2 - theta_1) + dtheta_2*l_1*l_2*m_4*np.sin(theta_2 - theta_1) + dtheta_2*l_1*l_2*m_5*np.sin(theta_2 - theta_1) + dtheta_2*l_1*m_2*r_2*np.sin(theta_2 - theta_1)) - dtheta_1*(mu_1 + mu_2) - ddx_c*(-l_1*m_2 - l_1*m_3 - l_1*m_4 - l_1*m_5 - m_1*r_1)*np.cos(theta_1) - g*(-l_1*m_2 - l_1*m_3 - l_1*m_4 - l_1*m_5 - m_1*r_1)*np.sin(theta_1))*np.cos(theta_3 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) - (-l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_3 - theta_2))*(dtheta_5**2*l_2*m_5*r_5*np.sin(theta_5 - theta_2) + dtheta_4*(dtheta_4*l_2*l_4*m_5*np.sin(theta_4 - theta_2) + dtheta_4*l_2*m_4*r_4*np.sin(theta_4 - theta_2)) + dtheta_3*mu_3 + dtheta_3*(dtheta_3*l_2*l_3*m_4*np.sin(theta_3 - theta_2) + dtheta_3*l_2*l_3*m_5*np.sin(theta_3 - theta_2) + dtheta_3*l_2*m_3*r_3*np.sin(theta_3 - theta_2)) - dtheta_2*(mu_2 + mu_3) + dtheta_1*mu_2 + dtheta_1*(-dtheta_1*l_1*l_2*m_3*np.sin(theta_2 - theta_1) - dtheta_1*l_1*l_2*m_4*np.sin(theta_2 - theta_1) - dtheta_1*l_1*l_2*m_5*np.sin(theta_2 - theta_1) - dtheta_1*l_1*m_2*r_2*np.sin(theta_2 - theta_1)) - ddx_c*(-l_2*m_3 - l_2*m_4 - l_2*m_5 - m_2*r_2)*np.cos(theta_2) - g*(-l_2*m_3 - l_2*m_4 - l_2*m_5 - m_2*r_2)*np.sin(theta_2) - l_1*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*(dtheta_5**2*l_1*m_5*r_5*np.sin(theta_5 - theta_1) + dtheta_4*(dtheta_4*l_1*l_4*m_5*np.sin(theta_4 - theta_1) + dtheta_4*l_1*m_4*r_4*np.sin(theta_4 - theta_1)) + dtheta_3*(dtheta_3*l_1*l_3*m_4*np.sin(theta_3 - theta_1) + dtheta_3*l_1*l_3*m_5*np.sin(theta_3 - theta_1) + dtheta_3*l_1*m_3*r_3*np.sin(theta_3 - theta_1)) + dtheta_2*mu_2 + dtheta_2*(dtheta_2*l_1*l_2*m_3*np.sin(theta_2 - theta_1) + dtheta_2*l_1*l_2*m_4*np.sin(theta_2 - theta_1) + dtheta_2*l_1*l_2*m_5*np.sin(theta_2 - theta_1) + dtheta_2*l_1*m_2*r_2*np.sin(theta_2 - theta_1)) - dtheta_1*(mu_1 + mu_2) - ddx_c*(-l_1*m_2 - l_1*m_3 - l_1*m_4 - l_1*m_5 - m_1*r_1)*np.cos(theta_1) - g*(-l_1*m_2 - l_1*m_3 - l_1*m_4 - l_1*m_5 - m_1*r_1)*np.sin(theta_1))*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2))/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2))/(I_3 - l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)**2*np.cos(theta_3 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_3**2*m_4 + l_3**2*m_5 + m_3*r_3**2 - (-l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_3 - theta_2))**2/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2)) - (-l_1**2*m_5*r_5*(l_4*m_5 + m_4*r_4)*np.cos(theta_5 - theta_1)*np.cos(theta_4 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_4*m_5*r_5*np.cos(theta_5 - theta_4) - (-l_1**2*(l_4*m_5 + m_4*r_4)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_4 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_4*m_5 + m_4*r_4)*np.cos(theta_4 - theta_2))*(-l_1**2*m_5*r_5*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_5 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*m_5*r_5*np.cos(theta_5 - theta_2))/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2) - (-l_1**2*(l_4*m_5 + m_4*r_4)*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_4 - theta_1)*np.cos(theta_3 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_3*(l_4*m_5 + m_4*r_4)*np.cos(theta_4 - theta_3) - (-l_1**2*(l_4*m_5 + m_4*r_4)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_4 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_4*m_5 + m_4*r_4)*np.cos(theta_4 - theta_2))*(-l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_3 - theta_2))/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2))*(-l_1**2*m_5*r_5*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_5 - theta_1)*np.cos(theta_3 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_3*m_5*r_5*np.cos(theta_5 - theta_3) - (-l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_3 - theta_2))*(-l_1**2*m_5*r_5*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_5 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*m_5*r_5*np.cos(theta_5 - theta_2))/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2))/(I_3 - l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)**2*np.cos(theta_3 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_3**2*m_4 + l_3**2*m_5 + m_3*r_3**2 - (-l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_3 - theta_2))**2/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2)))*(dtheta_5**2*l_4*m_5*r_5*np.sin(theta_5 - theta_4) + dtheta_5*mu_5 - dtheta_4*(mu_4 + mu_5) + dtheta_3*mu_4 + dtheta_3*(-dtheta_3*l_3*l_4*m_5*np.sin(theta_4 - theta_3) - dtheta_3*l_3*m_4*r_4*np.sin(theta_4 - theta_3)) + dtheta_2*(-dtheta_2*l_2*l_4*m_5*np.sin(theta_4 - theta_2) - dtheta_2*l_2*m_4*r_4*np.sin(theta_4 - theta_2)) + dtheta_1*(-dtheta_1*l_1*l_4*m_5*np.sin(theta_4 - theta_1) - dtheta_1*l_1*m_4*r_4*np.sin(theta_4 - theta_1)) - ddx_c*(-l_4*m_5 - m_4*r_4)*np.cos(theta_4) - g*(-l_4*m_5 - m_4*r_4)*np.sin(theta_4) - l_1*(l_4*m_5 + m_4*r_4)*(dtheta_5**2*l_1*m_5*r_5*np.sin(theta_5 - theta_1) + dtheta_4*(dtheta_4*l_1*l_4*m_5*np.sin(theta_4 - theta_1) + dtheta_4*l_1*m_4*r_4*np.sin(theta_4 - theta_1)) + dtheta_3*(dtheta_3*l_1*l_3*m_4*np.sin(theta_3 - theta_1) + dtheta_3*l_1*l_3*m_5*np.sin(theta_3 - theta_1) + dtheta_3*l_1*m_3*r_3*np.sin(theta_3 - theta_1)) + dtheta_2*mu_2 + dtheta_2*(dtheta_2*l_1*l_2*m_3*np.sin(theta_2 - theta_1) + dtheta_2*l_1*l_2*m_4*np.sin(theta_2 - theta_1) + dtheta_2*l_1*l_2*m_5*np.sin(theta_2 - theta_1) + dtheta_2*l_1*m_2*r_2*np.sin(theta_2 - theta_1)) - dtheta_1*(mu_1 + mu_2) - ddx_c*(-l_1*m_2 - l_1*m_3 - l_1*m_4 - l_1*m_5 - m_1*r_1)*np.cos(theta_1) - g*(-l_1*m_2 - l_1*m_3 - l_1*m_4 - l_1*m_5 - m_1*r_1)*np.sin(theta_1))*np.cos(theta_4 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) - (-l_1**2*(l_4*m_5 + m_4*r_4)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_4 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_4*m_5 + m_4*r_4)*np.cos(theta_4 - theta_2))*(dtheta_5**2*l_2*m_5*r_5*np.sin(theta_5 - theta_2) + dtheta_4*(dtheta_4*l_2*l_4*m_5*np.sin(theta_4 - theta_2) + dtheta_4*l_2*m_4*r_4*np.sin(theta_4 - theta_2)) + dtheta_3*mu_3 + dtheta_3*(dtheta_3*l_2*l_3*m_4*np.sin(theta_3 - theta_2) + dtheta_3*l_2*l_3*m_5*np.sin(theta_3 - theta_2) + dtheta_3*l_2*m_3*r_3*np.sin(theta_3 - theta_2)) - dtheta_2*(mu_2 + mu_3) + dtheta_1*mu_2 + dtheta_1*(-dtheta_1*l_1*l_2*m_3*np.sin(theta_2 - theta_1) - dtheta_1*l_1*l_2*m_4*np.sin(theta_2 - theta_1) - dtheta_1*l_1*l_2*m_5*np.sin(theta_2 - theta_1) - dtheta_1*l_1*m_2*r_2*np.sin(theta_2 - theta_1)) - ddx_c*(-l_2*m_3 - l_2*m_4 - l_2*m_5 - m_2*r_2)*np.cos(theta_2) - g*(-l_2*m_3 - l_2*m_4 - l_2*m_5 - m_2*r_2)*np.sin(theta_2) - l_1*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*(dtheta_5**2*l_1*m_5*r_5*np.sin(theta_5 - theta_1) + dtheta_4*(dtheta_4*l_1*l_4*m_5*np.sin(theta_4 - theta_1) + dtheta_4*l_1*m_4*r_4*np.sin(theta_4 - theta_1)) + dtheta_3*(dtheta_3*l_1*l_3*m_4*np.sin(theta_3 - theta_1) + dtheta_3*l_1*l_3*m_5*np.sin(theta_3 - theta_1) + dtheta_3*l_1*m_3*r_3*np.sin(theta_3 - theta_1)) + dtheta_2*mu_2 + dtheta_2*(dtheta_2*l_1*l_2*m_3*np.sin(theta_2 - theta_1) + dtheta_2*l_1*l_2*m_4*np.sin(theta_2 - theta_1) + dtheta_2*l_1*l_2*m_5*np.sin(theta_2 - theta_1) + dtheta_2*l_1*m_2*r_2*np.sin(theta_2 - theta_1)) - dtheta_1*(mu_1 + mu_2) - ddx_c*(-l_1*m_2 - l_1*m_3 - l_1*m_4 - l_1*m_5 - m_1*r_1)*np.cos(theta_1) - g*(-l_1*m_2 - l_1*m_3 - l_1*m_4 - l_1*m_5 - m_1*r_1)*np.sin(theta_1))*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2))/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2) - (-l_1**2*(l_4*m_5 + m_4*r_4)*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_4 - theta_1)*np.cos(theta_3 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_3*(l_4*m_5 + m_4*r_4)*np.cos(theta_4 - theta_3) - (-l_1**2*(l_4*m_5 + m_4*r_4)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_4 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_4*m_5 + m_4*r_4)*np.cos(theta_4 - theta_2))*(-l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_3 - theta_2))/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2))*(dtheta_5**2*l_3*m_5*r_5*np.sin(theta_5 - theta_3) + dtheta_4*mu_4 + dtheta_4*(dtheta_4*l_3*l_4*m_5*np.sin(theta_4 - theta_3) + dtheta_4*l_3*m_4*r_4*np.sin(theta_4 - theta_3)) - dtheta_3*(mu_3 + mu_4) + dtheta_2*mu_3 + dtheta_2*(-dtheta_2*l_2*l_3*m_4*np.sin(theta_3 - theta_2) - dtheta_2*l_2*l_3*m_5*np.sin(theta_3 - theta_2) - dtheta_2*l_2*m_3*r_3*np.sin(theta_3 - theta_2)) + dtheta_1*(-dtheta_1*l_1*l_3*m_4*np.sin(theta_3 - theta_1) - dtheta_1*l_1*l_3*m_5*np.sin(theta_3 - theta_1) - dtheta_1*l_1*m_3*r_3*np.sin(theta_3 - theta_1)) - ddx_c*(-l_3*m_4 - l_3*m_5 - m_3*r_3)*np.cos(theta_3) - g*(-l_3*m_4 - l_3*m_5 - m_3*r_3)*np.sin(theta_3) - l_1*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(dtheta_5**2*l_1*m_5*r_5*np.sin(theta_5 - theta_1) + dtheta_4*(dtheta_4*l_1*l_4*m_5*np.sin(theta_4 - theta_1) + dtheta_4*l_1*m_4*r_4*np.sin(theta_4 - theta_1)) + dtheta_3*(dtheta_3*l_1*l_3*m_4*np.sin(theta_3 - theta_1) + dtheta_3*l_1*l_3*m_5*np.sin(theta_3 - theta_1) + dtheta_3*l_1*m_3*r_3*np.sin(theta_3 - theta_1)) + dtheta_2*mu_2 + dtheta_2*(dtheta_2*l_1*l_2*m_3*np.sin(theta_2 - theta_1) + dtheta_2*l_1*l_2*m_4*np.sin(theta_2 - theta_1) + dtheta_2*l_1*l_2*m_5*np.sin(theta_2 - theta_1) + dtheta_2*l_1*m_2*r_2*np.sin(theta_2 - theta_1)) - dtheta_1*(mu_1 + mu_2) - ddx_c*(-l_1*m_2 - l_1*m_3 - l_1*m_4 - l_1*m_5 - m_1*r_1)*np.cos(theta_1) - g*(-l_1*m_2 - l_1*m_3 - l_1*m_4 - l_1*m_5 - m_1*r_1)*np.sin(theta_1))*np.cos(theta_3 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) - (-l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_3 - theta_2))*(dtheta_5**2*l_2*m_5*r_5*np.sin(theta_5 - theta_2) + dtheta_4*(dtheta_4*l_2*l_4*m_5*np.sin(theta_4 - theta_2) + dtheta_4*l_2*m_4*r_4*np.sin(theta_4 - theta_2)) + dtheta_3*mu_3 + dtheta_3*(dtheta_3*l_2*l_3*m_4*np.sin(theta_3 - theta_2) + dtheta_3*l_2*l_3*m_5*np.sin(theta_3 - theta_2) + dtheta_3*l_2*m_3*r_3*np.sin(theta_3 - theta_2)) - dtheta_2*(mu_2 + mu_3) + dtheta_1*mu_2 + dtheta_1*(-dtheta_1*l_1*l_2*m_3*np.sin(theta_2 - theta_1) - dtheta_1*l_1*l_2*m_4*np.sin(theta_2 - theta_1) - dtheta_1*l_1*l_2*m_5*np.sin(theta_2 - theta_1) - dtheta_1*l_1*m_2*r_2*np.sin(theta_2 - theta_1)) - ddx_c*(-l_2*m_3 - l_2*m_4 - l_2*m_5 - m_2*r_2)*np.cos(theta_2) - g*(-l_2*m_3 - l_2*m_4 - l_2*m_5 - m_2*r_2)*np.sin(theta_2) - l_1*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*(dtheta_5**2*l_1*m_5*r_5*np.sin(theta_5 - theta_1) + dtheta_4*(dtheta_4*l_1*l_4*m_5*np.sin(theta_4 - theta_1) + dtheta_4*l_1*m_4*r_4*np.sin(theta_4 - theta_1)) + dtheta_3*(dtheta_3*l_1*l_3*m_4*np.sin(theta_3 - theta_1) + dtheta_3*l_1*l_3*m_5*np.sin(theta_3 - theta_1) + dtheta_3*l_1*m_3*r_3*np.sin(theta_3 - theta_1)) + dtheta_2*mu_2 + dtheta_2*(dtheta_2*l_1*l_2*m_3*np.sin(theta_2 - theta_1) + dtheta_2*l_1*l_2*m_4*np.sin(theta_2 - theta_1) + dtheta_2*l_1*l_2*m_5*np.sin(theta_2 - theta_1) + dtheta_2*l_1*m_2*r_2*np.sin(theta_2 - theta_1)) - dtheta_1*(mu_1 + mu_2) - ddx_c*(-l_1*m_2 - l_1*m_3 - l_1*m_4 - l_1*m_5 - m_1*r_1)*np.cos(theta_1) - g*(-l_1*m_2 - l_1*m_3 - l_1*m_4 - l_1*m_5 - m_1*r_1)*np.sin(theta_1))*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2))/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2))/(I_3 - l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)**2*np.cos(theta_3 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_3**2*m_4 + l_3**2*m_5 + m_3*r_3**2 - (-l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_3 - theta_2))**2/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2)))/(I_4 - l_1**2*(l_4*m_5 + m_4*r_4)**2*np.cos(theta_4 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_4**2*m_5 + m_4*r_4**2 - (-l_1**2*(l_4*m_5 + m_4*r_4)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_4 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_4*m_5 + m_4*r_4)*np.cos(theta_4 - theta_2))**2/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2) - (-l_1**2*(l_4*m_5 + m_4*r_4)*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_4 - theta_1)*np.cos(theta_3 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_3*(l_4*m_5 + m_4*r_4)*np.cos(theta_4 - theta_3) - (-l_1**2*(l_4*m_5 + m_4*r_4)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_4 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_4*m_5 + m_4*r_4)*np.cos(theta_4 - theta_2))*(-l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_3 - theta_2))/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2))**2/(I_3 - l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)**2*np.cos(theta_3 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_3**2*m_4 + l_3**2*m_5 + m_3*r_3**2 - (-l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_3 - theta_2))**2/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2))))/(I_5 - l_1**2*m_5**2*r_5**2*np.cos(theta_5 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + m_5*r_5**2 - (-l_1**2*m_5*r_5*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_5 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*m_5*r_5*np.cos(theta_5 - theta_2))**2/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2) - (-l_1**2*m_5*r_5*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_5 - theta_1)*np.cos(theta_3 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_3*m_5*r_5*np.cos(theta_5 - theta_3) - (-l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_3 - theta_2))*(-l_1**2*m_5*r_5*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_5 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*m_5*r_5*np.cos(theta_5 - theta_2))/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2))**2/(I_3 - l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)**2*np.cos(theta_3 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_3**2*m_4 + l_3**2*m_5 + m_3*r_3**2 - (-l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_3 - theta_2))**2/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2)) - (-l_1**2*m_5*r_5*(l_4*m_5 + m_4*r_4)*np.cos(theta_5 - theta_1)*np.cos(theta_4 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_4*m_5*r_5*np.cos(theta_5 - theta_4) - (-l_1**2*(l_4*m_5 + m_4*r_4)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_4 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_4*m_5 + m_4*r_4)*np.cos(theta_4 - theta_2))*(-l_1**2*m_5*r_5*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_5 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*m_5*r_5*np.cos(theta_5 - theta_2))/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2) - (-l_1**2*(l_4*m_5 + m_4*r_4)*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_4 - theta_1)*np.cos(theta_3 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_3*(l_4*m_5 + m_4*r_4)*np.cos(theta_4 - theta_3) - (-l_1**2*(l_4*m_5 + m_4*r_4)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_4 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_4*m_5 + m_4*r_4)*np.cos(theta_4 - theta_2))*(-l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_3 - theta_2))/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2))*(-l_1**2*m_5*r_5*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_5 - theta_1)*np.cos(theta_3 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_3*m_5*r_5*np.cos(theta_5 - theta_3) - (-l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_3 - theta_2))*(-l_1**2*m_5*r_5*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_5 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*m_5*r_5*np.cos(theta_5 - theta_2))/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2))/(I_3 - l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)**2*np.cos(theta_3 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_3**2*m_4 + l_3**2*m_5 + m_3*r_3**2 - (-l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_3 - theta_2))**2/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2)))**2/(I_4 - l_1**2*(l_4*m_5 + m_4*r_4)**2*np.cos(theta_4 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_4**2*m_5 + m_4*r_4**2 - (-l_1**2*(l_4*m_5 + m_4*r_4)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_4 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_4*m_5 + m_4*r_4)*np.cos(theta_4 - theta_2))**2/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2) - (-l_1**2*(l_4*m_5 + m_4*r_4)*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_4 - theta_1)*np.cos(theta_3 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_3*(l_4*m_5 + m_4*r_4)*np.cos(theta_4 - theta_3) - (-l_1**2*(l_4*m_5 + m_4*r_4)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_4 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_4*m_5 + m_4*r_4)*np.cos(theta_4 - theta_2))*(-l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_3 - theta_2))/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2))**2/(I_3 - l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)**2*np.cos(theta_3 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_3**2*m_4 + l_3**2*m_5 + m_3*r_3**2 - (-l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_3 - theta_2))**2/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2)))))*np.cos(theta_4 - theta_1)/(I_4 - l_1**2*(l_4*m_5 + m_4*r_4)**2*np.cos(theta_4 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_4**2*m_5 + m_4*r_4**2 - (-l_1**2*(l_4*m_5 + m_4*r_4)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_4 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_4*m_5 + m_4*r_4)*np.cos(theta_4 - theta_2))**2/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2) - (-l_1**2*(l_4*m_5 + m_4*r_4)*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_4 - theta_1)*np.cos(theta_3 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_3*(l_4*m_5 + m_4*r_4)*np.cos(theta_4 - theta_3) - (-l_1**2*(l_4*m_5 + m_4*r_4)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_4 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_4*m_5 + m_4*r_4)*np.cos(theta_4 - theta_2))*(-l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_3 - theta_2))/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2))**2/(I_3 - l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)**2*np.cos(theta_3 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_3**2*m_4 + l_3**2*m_5 + m_3*r_3**2 - (-l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_3 - theta_2))**2/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2))) - l_1*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(dtheta_5**2*l_3*m_5*r_5*np.sin(theta_5 - theta_3) + dtheta_4*mu_4 + dtheta_4*(dtheta_4*l_3*l_4*m_5*np.sin(theta_4 - theta_3) + dtheta_4*l_3*m_4*r_4*np.sin(theta_4 - theta_3)) - dtheta_3*(mu_3 + mu_4) + dtheta_2*mu_3 + dtheta_2*(-dtheta_2*l_2*l_3*m_4*np.sin(theta_3 - theta_2) - dtheta_2*l_2*l_3*m_5*np.sin(theta_3 - theta_2) - dtheta_2*l_2*m_3*r_3*np.sin(theta_3 - theta_2)) + dtheta_1*(-dtheta_1*l_1*l_3*m_4*np.sin(theta_3 - theta_1) - dtheta_1*l_1*l_3*m_5*np.sin(theta_3 - theta_1) - dtheta_1*l_1*m_3*r_3*np.sin(theta_3 - theta_1)) - ddx_c*(-l_3*m_4 - l_3*m_5 - m_3*r_3)*np.cos(theta_3) - g*(-l_3*m_4 - l_3*m_5 - m_3*r_3)*np.sin(theta_3) - l_1*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(dtheta_5**2*l_1*m_5*r_5*np.sin(theta_5 - theta_1) + dtheta_4*(dtheta_4*l_1*l_4*m_5*np.sin(theta_4 - theta_1) + dtheta_4*l_1*m_4*r_4*np.sin(theta_4 - theta_1)) + dtheta_3*(dtheta_3*l_1*l_3*m_4*np.sin(theta_3 - theta_1) + dtheta_3*l_1*l_3*m_5*np.sin(theta_3 - theta_1) + dtheta_3*l_1*m_3*r_3*np.sin(theta_3 - theta_1)) + dtheta_2*mu_2 + dtheta_2*(dtheta_2*l_1*l_2*m_3*np.sin(theta_2 - theta_1) + dtheta_2*l_1*l_2*m_4*np.sin(theta_2 - theta_1) + dtheta_2*l_1*l_2*m_5*np.sin(theta_2 - theta_1) + dtheta_2*l_1*m_2*r_2*np.sin(theta_2 - theta_1)) - dtheta_1*(mu_1 + mu_2) - ddx_c*(-l_1*m_2 - l_1*m_3 - l_1*m_4 - l_1*m_5 - m_1*r_1)*np.cos(theta_1) - g*(-l_1*m_2 - l_1*m_3 - l_1*m_4 - l_1*m_5 - m_1*r_1)*np.sin(theta_1))*np.cos(theta_3 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) - (-l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_3 - theta_2))*(dtheta_5**2*l_2*m_5*r_5*np.sin(theta_5 - theta_2) + dtheta_4*(dtheta_4*l_2*l_4*m_5*np.sin(theta_4 - theta_2) + dtheta_4*l_2*m_4*r_4*np.sin(theta_4 - theta_2)) + dtheta_3*mu_3 + dtheta_3*(dtheta_3*l_2*l_3*m_4*np.sin(theta_3 - theta_2) + dtheta_3*l_2*l_3*m_5*np.sin(theta_3 - theta_2) + dtheta_3*l_2*m_3*r_3*np.sin(theta_3 - theta_2)) - dtheta_2*(mu_2 + mu_3) + dtheta_1*mu_2 + dtheta_1*(-dtheta_1*l_1*l_2*m_3*np.sin(theta_2 - theta_1) - dtheta_1*l_1*l_2*m_4*np.sin(theta_2 - theta_1) - dtheta_1*l_1*l_2*m_5*np.sin(theta_2 - theta_1) - dtheta_1*l_1*m_2*r_2*np.sin(theta_2 - theta_1)) - ddx_c*(-l_2*m_3 - l_2*m_4 - l_2*m_5 - m_2*r_2)*np.cos(theta_2) - g*(-l_2*m_3 - l_2*m_4 - l_2*m_5 - m_2*r_2)*np.sin(theta_2) - l_1*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*(dtheta_5**2*l_1*m_5*r_5*np.sin(theta_5 - theta_1) + dtheta_4*(dtheta_4*l_1*l_4*m_5*np.sin(theta_4 - theta_1) + dtheta_4*l_1*m_4*r_4*np.sin(theta_4 - theta_1)) + dtheta_3*(dtheta_3*l_1*l_3*m_4*np.sin(theta_3 - theta_1) + dtheta_3*l_1*l_3*m_5*np.sin(theta_3 - theta_1) + dtheta_3*l_1*m_3*r_3*np.sin(theta_3 - theta_1)) + dtheta_2*mu_2 + dtheta_2*(dtheta_2*l_1*l_2*m_3*np.sin(theta_2 - theta_1) + dtheta_2*l_1*l_2*m_4*np.sin(theta_2 - theta_1) + dtheta_2*l_1*l_2*m_5*np.sin(theta_2 - theta_1) + dtheta_2*l_1*m_2*r_2*np.sin(theta_2 - theta_1)) - dtheta_1*(mu_1 + mu_2) - ddx_c*(-l_1*m_2 - l_1*m_3 - l_1*m_4 - l_1*m_5 - m_1*r_1)*np.cos(theta_1) - g*(-l_1*m_2 - l_1*m_3 - l_1*m_4 - l_1*m_5 - m_1*r_1)*np.sin(theta_1))*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2))/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2) - (-l_1**2*(l_4*m_5 + m_4*r_4)*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_4 - theta_1)*np.cos(theta_3 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_3*(l_4*m_5 + m_4*r_4)*np.cos(theta_4 - theta_3) - (-l_1**2*(l_4*m_5 + m_4*r_4)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_4 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_4*m_5 + m_4*r_4)*np.cos(theta_4 - theta_2))*(-l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_3 - theta_2))/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2))*(dtheta_5**2*l_4*m_5*r_5*np.sin(theta_5 - theta_4) + dtheta_5*mu_5 - dtheta_4*(mu_4 + mu_5) + dtheta_3*mu_4 + dtheta_3*(-dtheta_3*l_3*l_4*m_5*np.sin(theta_4 - theta_3) - dtheta_3*l_3*m_4*r_4*np.sin(theta_4 - theta_3)) + dtheta_2*(-dtheta_2*l_2*l_4*m_5*np.sin(theta_4 - theta_2) - dtheta_2*l_2*m_4*r_4*np.sin(theta_4 - theta_2)) + dtheta_1*(-dtheta_1*l_1*l_4*m_5*np.sin(theta_4 - theta_1) - dtheta_1*l_1*m_4*r_4*np.sin(theta_4 - theta_1)) - ddx_c*(-l_4*m_5 - m_4*r_4)*np.cos(theta_4) - g*(-l_4*m_5 - m_4*r_4)*np.sin(theta_4) - l_1*(l_4*m_5 + m_4*r_4)*(dtheta_5**2*l_1*m_5*r_5*np.sin(theta_5 - theta_1) + dtheta_4*(dtheta_4*l_1*l_4*m_5*np.sin(theta_4 - theta_1) + dtheta_4*l_1*m_4*r_4*np.sin(theta_4 - theta_1)) + dtheta_3*(dtheta_3*l_1*l_3*m_4*np.sin(theta_3 - theta_1) + dtheta_3*l_1*l_3*m_5*np.sin(theta_3 - theta_1) + dtheta_3*l_1*m_3*r_3*np.sin(theta_3 - theta_1)) + dtheta_2*mu_2 + dtheta_2*(dtheta_2*l_1*l_2*m_3*np.sin(theta_2 - theta_1) + dtheta_2*l_1*l_2*m_4*np.sin(theta_2 - theta_1) + dtheta_2*l_1*l_2*m_5*np.sin(theta_2 - theta_1) + dtheta_2*l_1*m_2*r_2*np.sin(theta_2 - theta_1)) - dtheta_1*(mu_1 + mu_2) - ddx_c*(-l_1*m_2 - l_1*m_3 - l_1*m_4 - l_1*m_5 - m_1*r_1)*np.cos(theta_1) - g*(-l_1*m_2 - l_1*m_3 - l_1*m_4 - l_1*m_5 - m_1*r_1)*np.sin(theta_1))*np.cos(theta_4 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) - (-l_1**2*(l_4*m_5 + m_4*r_4)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_4 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_4*m_5 + m_4*r_4)*np.cos(theta_4 - theta_2))*(dtheta_5**2*l_2*m_5*r_5*np.sin(theta_5 - theta_2) + dtheta_4*(dtheta_4*l_2*l_4*m_5*np.sin(theta_4 - theta_2) + dtheta_4*l_2*m_4*r_4*np.sin(theta_4 - theta_2)) + dtheta_3*mu_3 + dtheta_3*(dtheta_3*l_2*l_3*m_4*np.sin(theta_3 - theta_2) + dtheta_3*l_2*l_3*m_5*np.sin(theta_3 - theta_2) + dtheta_3*l_2*m_3*r_3*np.sin(theta_3 - theta_2)) - dtheta_2*(mu_2 + mu_3) + dtheta_1*mu_2 + dtheta_1*(-dtheta_1*l_1*l_2*m_3*np.sin(theta_2 - theta_1) - dtheta_1*l_1*l_2*m_4*np.sin(theta_2 - theta_1) - dtheta_1*l_1*l_2*m_5*np.sin(theta_2 - theta_1) - dtheta_1*l_1*m_2*r_2*np.sin(theta_2 - theta_1)) - ddx_c*(-l_2*m_3 - l_2*m_4 - l_2*m_5 - m_2*r_2)*np.cos(theta_2) - g*(-l_2*m_3 - l_2*m_4 - l_2*m_5 - m_2*r_2)*np.sin(theta_2) - l_1*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*(dtheta_5**2*l_1*m_5*r_5*np.sin(theta_5 - theta_1) + dtheta_4*(dtheta_4*l_1*l_4*m_5*np.sin(theta_4 - theta_1) + dtheta_4*l_1*m_4*r_4*np.sin(theta_4 - theta_1)) + dtheta_3*(dtheta_3*l_1*l_3*m_4*np.sin(theta_3 - theta_1) + dtheta_3*l_1*l_3*m_5*np.sin(theta_3 - theta_1) + dtheta_3*l_1*m_3*r_3*np.sin(theta_3 - theta_1)) + dtheta_2*mu_2 + dtheta_2*(dtheta_2*l_1*l_2*m_3*np.sin(theta_2 - theta_1) + dtheta_2*l_1*l_2*m_4*np.sin(theta_2 - theta_1) + dtheta_2*l_1*l_2*m_5*np.sin(theta_2 - theta_1) + dtheta_2*l_1*m_2*r_2*np.sin(theta_2 - theta_1)) - dtheta_1*(mu_1 + mu_2) - ddx_c*(-l_1*m_2 - l_1*m_3 - l_1*m_4 - l_1*m_5 - m_1*r_1)*np.cos(theta_1) - g*(-l_1*m_2 - l_1*m_3 - l_1*m_4 - l_1*m_5 - m_1*r_1)*np.sin(theta_1))*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2))/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2) - (-l_1**2*(l_4*m_5 + m_4*r_4)*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_4 - theta_1)*np.cos(theta_3 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_3*(l_4*m_5 + m_4*r_4)*np.cos(theta_4 - theta_3) - (-l_1**2*(l_4*m_5 + m_4*r_4)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_4 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_4*m_5 + m_4*r_4)*np.cos(theta_4 - theta_2))*(-l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_3 - theta_2))/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2))*(dtheta_5**2*l_3*m_5*r_5*np.sin(theta_5 - theta_3) + dtheta_4*mu_4 + dtheta_4*(dtheta_4*l_3*l_4*m_5*np.sin(theta_4 - theta_3) + dtheta_4*l_3*m_4*r_4*np.sin(theta_4 - theta_3)) - dtheta_3*(mu_3 + mu_4) + dtheta_2*mu_3 + dtheta_2*(-dtheta_2*l_2*l_3*m_4*np.sin(theta_3 - theta_2) - dtheta_2*l_2*l_3*m_5*np.sin(theta_3 - theta_2) - dtheta_2*l_2*m_3*r_3*np.sin(theta_3 - theta_2)) + dtheta_1*(-dtheta_1*l_1*l_3*m_4*np.sin(theta_3 - theta_1) - dtheta_1*l_1*l_3*m_5*np.sin(theta_3 - theta_1) - dtheta_1*l_1*m_3*r_3*np.sin(theta_3 - theta_1)) - ddx_c*(-l_3*m_4 - l_3*m_5 - m_3*r_3)*np.cos(theta_3) - g*(-l_3*m_4 - l_3*m_5 - m_3*r_3)*np.sin(theta_3) - l_1*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(dtheta_5**2*l_1*m_5*r_5*np.sin(theta_5 - theta_1) + dtheta_4*(dtheta_4*l_1*l_4*m_5*np.sin(theta_4 - theta_1) + dtheta_4*l_1*m_4*r_4*np.sin(theta_4 - theta_1)) + dtheta_3*(dtheta_3*l_1*l_3*m_4*np.sin(theta_3 - theta_1) + dtheta_3*l_1*l_3*m_5*np.sin(theta_3 - theta_1) + dtheta_3*l_1*m_3*r_3*np.sin(theta_3 - theta_1)) + dtheta_2*mu_2 + dtheta_2*(dtheta_2*l_1*l_2*m_3*np.sin(theta_2 - theta_1) + dtheta_2*l_1*l_2*m_4*np.sin(theta_2 - theta_1) + dtheta_2*l_1*l_2*m_5*np.sin(theta_2 - theta_1) + dtheta_2*l_1*m_2*r_2*np.sin(theta_2 - theta_1)) - dtheta_1*(mu_1 + mu_2) - ddx_c*(-l_1*m_2 - l_1*m_3 - l_1*m_4 - l_1*m_5 - m_1*r_1)*np.cos(theta_1) - g*(-l_1*m_2 - l_1*m_3 - l_1*m_4 - l_1*m_5 - m_1*r_1)*np.sin(theta_1))*np.cos(theta_3 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) - (-l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_3 - theta_2))*(dtheta_5**2*l_2*m_5*r_5*np.sin(theta_5 - theta_2) + dtheta_4*(dtheta_4*l_2*l_4*m_5*np.sin(theta_4 - theta_2) + dtheta_4*l_2*m_4*r_4*np.sin(theta_4 - theta_2)) + dtheta_3*mu_3 + dtheta_3*(dtheta_3*l_2*l_3*m_4*np.sin(theta_3 - theta_2) + dtheta_3*l_2*l_3*m_5*np.sin(theta_3 - theta_2) + dtheta_3*l_2*m_3*r_3*np.sin(theta_3 - theta_2)) - dtheta_2*(mu_2 + mu_3) + dtheta_1*mu_2 + dtheta_1*(-dtheta_1*l_1*l_2*m_3*np.sin(theta_2 - theta_1) - dtheta_1*l_1*l_2*m_4*np.sin(theta_2 - theta_1) - dtheta_1*l_1*l_2*m_5*np.sin(theta_2 - theta_1) - dtheta_1*l_1*m_2*r_2*np.sin(theta_2 - theta_1)) - ddx_c*(-l_2*m_3 - l_2*m_4 - l_2*m_5 - m_2*r_2)*np.cos(theta_2) - g*(-l_2*m_3 - l_2*m_4 - l_2*m_5 - m_2*r_2)*np.sin(theta_2) - l_1*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*(dtheta_5**2*l_1*m_5*r_5*np.sin(theta_5 - theta_1) + dtheta_4*(dtheta_4*l_1*l_4*m_5*np.sin(theta_4 - theta_1) + dtheta_4*l_1*m_4*r_4*np.sin(theta_4 - theta_1)) + dtheta_3*(dtheta_3*l_1*l_3*m_4*np.sin(theta_3 - theta_1) + dtheta_3*l_1*l_3*m_5*np.sin(theta_3 - theta_1) + dtheta_3*l_1*m_3*r_3*np.sin(theta_3 - theta_1)) + dtheta_2*mu_2 + dtheta_2*(dtheta_2*l_1*l_2*m_3*np.sin(theta_2 - theta_1) + dtheta_2*l_1*l_2*m_4*np.sin(theta_2 - theta_1) + dtheta_2*l_1*l_2*m_5*np.sin(theta_2 - theta_1) + dtheta_2*l_1*m_2*r_2*np.sin(theta_2 - theta_1)) - dtheta_1*(mu_1 + mu_2) - ddx_c*(-l_1*m_2 - l_1*m_3 - l_1*m_4 - l_1*m_5 - m_1*r_1)*np.cos(theta_1) - g*(-l_1*m_2 - l_1*m_3 - l_1*m_4 - l_1*m_5 - m_1*r_1)*np.sin(theta_1))*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2))/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2))/(I_3 - l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)**2*np.cos(theta_3 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_3**2*m_4 + l_3**2*m_5 + m_3*r_3**2 - (-l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_3 - theta_2))**2/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2)) - (-l_1**2*m_5*r_5*(l_4*m_5 + m_4*r_4)*np.cos(theta_5 - theta_1)*np.cos(theta_4 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_4*m_5*r_5*np.cos(theta_5 - theta_4) - (-l_1**2*(l_4*m_5 + m_4*r_4)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_4 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_4*m_5 + m_4*r_4)*np.cos(theta_4 - theta_2))*(-l_1**2*m_5*r_5*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_5 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*m_5*r_5*np.cos(theta_5 - theta_2))/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2) - (-l_1**2*(l_4*m_5 + m_4*r_4)*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_4 - theta_1)*np.cos(theta_3 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_3*(l_4*m_5 + m_4*r_4)*np.cos(theta_4 - theta_3) - (-l_1**2*(l_4*m_5 + m_4*r_4)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_4 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_4*m_5 + m_4*r_4)*np.cos(theta_4 - theta_2))*(-l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_3 - theta_2))/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2))*(-l_1**2*m_5*r_5*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_5 - theta_1)*np.cos(theta_3 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_3*m_5*r_5*np.cos(theta_5 - theta_3) - (-l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_3 - theta_2))*(-l_1**2*m_5*r_5*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_5 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*m_5*r_5*np.cos(theta_5 - theta_2))/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2))/(I_3 - l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)**2*np.cos(theta_3 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_3**2*m_4 + l_3**2*m_5 + m_3*r_3**2 - (-l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_3 - theta_2))**2/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2)))*(-dtheta_5*mu_5 - dtheta_4**2*l_4*m_5*r_5*np.sin(theta_5 - theta_4) + dtheta_4*mu_5 - dtheta_3**2*l_3*m_5*r_5*np.sin(theta_5 - theta_3) - dtheta_2**2*l_2*m_5*r_5*np.sin(theta_5 - theta_2) - dtheta_1**2*l_1*m_5*r_5*np.sin(theta_5 - theta_1) + ddx_c*m_5*r_5*np.cos(theta_5) + g*m_5*r_5*np.sin(theta_5) - l_1*m_5*r_5*(dtheta_5**2*l_1*m_5*r_5*np.sin(theta_5 - theta_1) + dtheta_4*(dtheta_4*l_1*l_4*m_5*np.sin(theta_4 - theta_1) + dtheta_4*l_1*m_4*r_4*np.sin(theta_4 - theta_1)) + dtheta_3*(dtheta_3*l_1*l_3*m_4*np.sin(theta_3 - theta_1) + dtheta_3*l_1*l_3*m_5*np.sin(theta_3 - theta_1) + dtheta_3*l_1*m_3*r_3*np.sin(theta_3 - theta_1)) + dtheta_2*mu_2 + dtheta_2*(dtheta_2*l_1*l_2*m_3*np.sin(theta_2 - theta_1) + dtheta_2*l_1*l_2*m_4*np.sin(theta_2 - theta_1) + dtheta_2*l_1*l_2*m_5*np.sin(theta_2 - theta_1) + dtheta_2*l_1*m_2*r_2*np.sin(theta_2 - theta_1)) - dtheta_1*(mu_1 + mu_2) - ddx_c*(-l_1*m_2 - l_1*m_3 - l_1*m_4 - l_1*m_5 - m_1*r_1)*np.cos(theta_1) - g*(-l_1*m_2 - l_1*m_3 - l_1*m_4 - l_1*m_5 - m_1*r_1)*np.sin(theta_1))*np.cos(theta_5 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) - (-l_1**2*m_5*r_5*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_5 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*m_5*r_5*np.cos(theta_5 - theta_2))*(dtheta_5**2*l_2*m_5*r_5*np.sin(theta_5 - theta_2) + dtheta_4*(dtheta_4*l_2*l_4*m_5*np.sin(theta_4 - theta_2) + dtheta_4*l_2*m_4*r_4*np.sin(theta_4 - theta_2)) + dtheta_3*mu_3 + dtheta_3*(dtheta_3*l_2*l_3*m_4*np.sin(theta_3 - theta_2) + dtheta_3*l_2*l_3*m_5*np.sin(theta_3 - theta_2) + dtheta_3*l_2*m_3*r_3*np.sin(theta_3 - theta_2)) - dtheta_2*(mu_2 + mu_3) + dtheta_1*mu_2 + dtheta_1*(-dtheta_1*l_1*l_2*m_3*np.sin(theta_2 - theta_1) - dtheta_1*l_1*l_2*m_4*np.sin(theta_2 - theta_1) - dtheta_1*l_1*l_2*m_5*np.sin(theta_2 - theta_1) - dtheta_1*l_1*m_2*r_2*np.sin(theta_2 - theta_1)) - ddx_c*(-l_2*m_3 - l_2*m_4 - l_2*m_5 - m_2*r_2)*np.cos(theta_2) - g*(-l_2*m_3 - l_2*m_4 - l_2*m_5 - m_2*r_2)*np.sin(theta_2) - l_1*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*(dtheta_5**2*l_1*m_5*r_5*np.sin(theta_5 - theta_1) + dtheta_4*(dtheta_4*l_1*l_4*m_5*np.sin(theta_4 - theta_1) + dtheta_4*l_1*m_4*r_4*np.sin(theta_4 - theta_1)) + dtheta_3*(dtheta_3*l_1*l_3*m_4*np.sin(theta_3 - theta_1) + dtheta_3*l_1*l_3*m_5*np.sin(theta_3 - theta_1) + dtheta_3*l_1*m_3*r_3*np.sin(theta_3 - theta_1)) + dtheta_2*mu_2 + dtheta_2*(dtheta_2*l_1*l_2*m_3*np.sin(theta_2 - theta_1) + dtheta_2*l_1*l_2*m_4*np.sin(theta_2 - theta_1) + dtheta_2*l_1*l_2*m_5*np.sin(theta_2 - theta_1) + dtheta_2*l_1*m_2*r_2*np.sin(theta_2 - theta_1)) - dtheta_1*(mu_1 + mu_2) - ddx_c*(-l_1*m_2 - l_1*m_3 - l_1*m_4 - l_1*m_5 - m_1*r_1)*np.cos(theta_1) - g*(-l_1*m_2 - l_1*m_3 - l_1*m_4 - l_1*m_5 - m_1*r_1)*np.sin(theta_1))*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2))/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2) - (-l_1**2*m_5*r_5*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_5 - theta_1)*np.cos(theta_3 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_3*m_5*r_5*np.cos(theta_5 - theta_3) - (-l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_3 - theta_2))*(-l_1**2*m_5*r_5*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_5 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*m_5*r_5*np.cos(theta_5 - theta_2))/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2))*(dtheta_5**2*l_3*m_5*r_5*np.sin(theta_5 - theta_3) + dtheta_4*mu_4 + dtheta_4*(dtheta_4*l_3*l_4*m_5*np.sin(theta_4 - theta_3) + dtheta_4*l_3*m_4*r_4*np.sin(theta_4 - theta_3)) - dtheta_3*(mu_3 + mu_4) + dtheta_2*mu_3 + dtheta_2*(-dtheta_2*l_2*l_3*m_4*np.sin(theta_3 - theta_2) - dtheta_2*l_2*l_3*m_5*np.sin(theta_3 - theta_2) - dtheta_2*l_2*m_3*r_3*np.sin(theta_3 - theta_2)) + dtheta_1*(-dtheta_1*l_1*l_3*m_4*np.sin(theta_3 - theta_1) - dtheta_1*l_1*l_3*m_5*np.sin(theta_3 - theta_1) - dtheta_1*l_1*m_3*r_3*np.sin(theta_3 - theta_1)) - ddx_c*(-l_3*m_4 - l_3*m_5 - m_3*r_3)*np.cos(theta_3) - g*(-l_3*m_4 - l_3*m_5 - m_3*r_3)*np.sin(theta_3) - l_1*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(dtheta_5**2*l_1*m_5*r_5*np.sin(theta_5 - theta_1) + dtheta_4*(dtheta_4*l_1*l_4*m_5*np.sin(theta_4 - theta_1) + dtheta_4*l_1*m_4*r_4*np.sin(theta_4 - theta_1)) + dtheta_3*(dtheta_3*l_1*l_3*m_4*np.sin(theta_3 - theta_1) + dtheta_3*l_1*l_3*m_5*np.sin(theta_3 - theta_1) + dtheta_3*l_1*m_3*r_3*np.sin(theta_3 - theta_1)) + dtheta_2*mu_2 + dtheta_2*(dtheta_2*l_1*l_2*m_3*np.sin(theta_2 - theta_1) + dtheta_2*l_1*l_2*m_4*np.sin(theta_2 - theta_1) + dtheta_2*l_1*l_2*m_5*np.sin(theta_2 - theta_1) + dtheta_2*l_1*m_2*r_2*np.sin(theta_2 - theta_1)) - dtheta_1*(mu_1 + mu_2) - ddx_c*(-l_1*m_2 - l_1*m_3 - l_1*m_4 - l_1*m_5 - m_1*r_1)*np.cos(theta_1) - g*(-l_1*m_2 - l_1*m_3 - l_1*m_4 - l_1*m_5 - m_1*r_1)*np.sin(theta_1))*np.cos(theta_3 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) - (-l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_3 - theta_2))*(dtheta_5**2*l_2*m_5*r_5*np.sin(theta_5 - theta_2) + dtheta_4*(dtheta_4*l_2*l_4*m_5*np.sin(theta_4 - theta_2) + dtheta_4*l_2*m_4*r_4*np.sin(theta_4 - theta_2)) + dtheta_3*mu_3 + dtheta_3*(dtheta_3*l_2*l_3*m_4*np.sin(theta_3 - theta_2) + dtheta_3*l_2*l_3*m_5*np.sin(theta_3 - theta_2) + dtheta_3*l_2*m_3*r_3*np.sin(theta_3 - theta_2)) - dtheta_2*(mu_2 + mu_3) + dtheta_1*mu_2 + dtheta_1*(-dtheta_1*l_1*l_2*m_3*np.sin(theta_2 - theta_1) - dtheta_1*l_1*l_2*m_4*np.sin(theta_2 - theta_1) - dtheta_1*l_1*l_2*m_5*np.sin(theta_2 - theta_1) - dtheta_1*l_1*m_2*r_2*np.sin(theta_2 - theta_1)) - ddx_c*(-l_2*m_3 - l_2*m_4 - l_2*m_5 - m_2*r_2)*np.cos(theta_2) - g*(-l_2*m_3 - l_2*m_4 - l_2*m_5 - m_2*r_2)*np.sin(theta_2) - l_1*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*(dtheta_5**2*l_1*m_5*r_5*np.sin(theta_5 - theta_1) + dtheta_4*(dtheta_4*l_1*l_4*m_5*np.sin(theta_4 - theta_1) + dtheta_4*l_1*m_4*r_4*np.sin(theta_4 - theta_1)) + dtheta_3*(dtheta_3*l_1*l_3*m_4*np.sin(theta_3 - theta_1) + dtheta_3*l_1*l_3*m_5*np.sin(theta_3 - theta_1) + dtheta_3*l_1*m_3*r_3*np.sin(theta_3 - theta_1)) + dtheta_2*mu_2 + dtheta_2*(dtheta_2*l_1*l_2*m_3*np.sin(theta_2 - theta_1) + dtheta_2*l_1*l_2*m_4*np.sin(theta_2 - theta_1) + dtheta_2*l_1*l_2*m_5*np.sin(theta_2 - theta_1) + dtheta_2*l_1*m_2*r_2*np.sin(theta_2 - theta_1)) - dtheta_1*(mu_1 + mu_2) - ddx_c*(-l_1*m_2 - l_1*m_3 - l_1*m_4 - l_1*m_5 - m_1*r_1)*np.cos(theta_1) - g*(-l_1*m_2 - l_1*m_3 - l_1*m_4 - l_1*m_5 - m_1*r_1)*np.sin(theta_1))*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2))/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2))/(I_3 - l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)**2*np.cos(theta_3 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_3**2*m_4 + l_3**2*m_5 + m_3*r_3**2 - (-l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_3 - theta_2))**2/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2)) - (-l_1**2*m_5*r_5*(l_4*m_5 + m_4*r_4)*np.cos(theta_5 - theta_1)*np.cos(theta_4 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_4*m_5*r_5*np.cos(theta_5 - theta_4) - (-l_1**2*(l_4*m_5 + m_4*r_4)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_4 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_4*m_5 + m_4*r_4)*np.cos(theta_4 - theta_2))*(-l_1**2*m_5*r_5*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_5 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*m_5*r_5*np.cos(theta_5 - theta_2))/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2) - (-l_1**2*(l_4*m_5 + m_4*r_4)*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_4 - theta_1)*np.cos(theta_3 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_3*(l_4*m_5 + m_4*r_4)*np.cos(theta_4 - theta_3) - (-l_1**2*(l_4*m_5 + m_4*r_4)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_4 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_4*m_5 + m_4*r_4)*np.cos(theta_4 - theta_2))*(-l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_3 - theta_2))/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2))*(-l_1**2*m_5*r_5*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_5 - theta_1)*np.cos(theta_3 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_3*m_5*r_5*np.cos(theta_5 - theta_3) - (-l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_3 - theta_2))*(-l_1**2*m_5*r_5*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_5 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*m_5*r_5*np.cos(theta_5 - theta_2))/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2))/(I_3 - l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)**2*np.cos(theta_3 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_3**2*m_4 + l_3**2*m_5 + m_3*r_3**2 - (-l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_3 - theta_2))**2/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2)))*(dtheta_5**2*l_4*m_5*r_5*np.sin(theta_5 - theta_4) + dtheta_5*mu_5 - dtheta_4*(mu_4 + mu_5) + dtheta_3*mu_4 + dtheta_3*(-dtheta_3*l_3*l_4*m_5*np.sin(theta_4 - theta_3) - dtheta_3*l_3*m_4*r_4*np.sin(theta_4 - theta_3)) + dtheta_2*(-dtheta_2*l_2*l_4*m_5*np.sin(theta_4 - theta_2) - dtheta_2*l_2*m_4*r_4*np.sin(theta_4 - theta_2)) + dtheta_1*(-dtheta_1*l_1*l_4*m_5*np.sin(theta_4 - theta_1) - dtheta_1*l_1*m_4*r_4*np.sin(theta_4 - theta_1)) - ddx_c*(-l_4*m_5 - m_4*r_4)*np.cos(theta_4) - g*(-l_4*m_5 - m_4*r_4)*np.sin(theta_4) - l_1*(l_4*m_5 + m_4*r_4)*(dtheta_5**2*l_1*m_5*r_5*np.sin(theta_5 - theta_1) + dtheta_4*(dtheta_4*l_1*l_4*m_5*np.sin(theta_4 - theta_1) + dtheta_4*l_1*m_4*r_4*np.sin(theta_4 - theta_1)) + dtheta_3*(dtheta_3*l_1*l_3*m_4*np.sin(theta_3 - theta_1) + dtheta_3*l_1*l_3*m_5*np.sin(theta_3 - theta_1) + dtheta_3*l_1*m_3*r_3*np.sin(theta_3 - theta_1)) + dtheta_2*mu_2 + dtheta_2*(dtheta_2*l_1*l_2*m_3*np.sin(theta_2 - theta_1) + dtheta_2*l_1*l_2*m_4*np.sin(theta_2 - theta_1) + dtheta_2*l_1*l_2*m_5*np.sin(theta_2 - theta_1) + dtheta_2*l_1*m_2*r_2*np.sin(theta_2 - theta_1)) - dtheta_1*(mu_1 + mu_2) - ddx_c*(-l_1*m_2 - l_1*m_3 - l_1*m_4 - l_1*m_5 - m_1*r_1)*np.cos(theta_1) - g*(-l_1*m_2 - l_1*m_3 - l_1*m_4 - l_1*m_5 - m_1*r_1)*np.sin(theta_1))*np.cos(theta_4 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) - (-l_1**2*(l_4*m_5 + m_4*r_4)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_4 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_4*m_5 + m_4*r_4)*np.cos(theta_4 - theta_2))*(dtheta_5**2*l_2*m_5*r_5*np.sin(theta_5 - theta_2) + dtheta_4*(dtheta_4*l_2*l_4*m_5*np.sin(theta_4 - theta_2) + dtheta_4*l_2*m_4*r_4*np.sin(theta_4 - theta_2)) + dtheta_3*mu_3 + dtheta_3*(dtheta_3*l_2*l_3*m_4*np.sin(theta_3 - theta_2) + dtheta_3*l_2*l_3*m_5*np.sin(theta_3 - theta_2) + dtheta_3*l_2*m_3*r_3*np.sin(theta_3 - theta_2)) - dtheta_2*(mu_2 + mu_3) + dtheta_1*mu_2 + dtheta_1*(-dtheta_1*l_1*l_2*m_3*np.sin(theta_2 - theta_1) - dtheta_1*l_1*l_2*m_4*np.sin(theta_2 - theta_1) - dtheta_1*l_1*l_2*m_5*np.sin(theta_2 - theta_1) - dtheta_1*l_1*m_2*r_2*np.sin(theta_2 - theta_1)) - ddx_c*(-l_2*m_3 - l_2*m_4 - l_2*m_5 - m_2*r_2)*np.cos(theta_2) - g*(-l_2*m_3 - l_2*m_4 - l_2*m_5 - m_2*r_2)*np.sin(theta_2) - l_1*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*(dtheta_5**2*l_1*m_5*r_5*np.sin(theta_5 - theta_1) + dtheta_4*(dtheta_4*l_1*l_4*m_5*np.sin(theta_4 - theta_1) + dtheta_4*l_1*m_4*r_4*np.sin(theta_4 - theta_1)) + dtheta_3*(dtheta_3*l_1*l_3*m_4*np.sin(theta_3 - theta_1) + dtheta_3*l_1*l_3*m_5*np.sin(theta_3 - theta_1) + dtheta_3*l_1*m_3*r_3*np.sin(theta_3 - theta_1)) + dtheta_2*mu_2 + dtheta_2*(dtheta_2*l_1*l_2*m_3*np.sin(theta_2 - theta_1) + dtheta_2*l_1*l_2*m_4*np.sin(theta_2 - theta_1) + dtheta_2*l_1*l_2*m_5*np.sin(theta_2 - theta_1) + dtheta_2*l_1*m_2*r_2*np.sin(theta_2 - theta_1)) - dtheta_1*(mu_1 + mu_2) - ddx_c*(-l_1*m_2 - l_1*m_3 - l_1*m_4 - l_1*m_5 - m_1*r_1)*np.cos(theta_1) - g*(-l_1*m_2 - l_1*m_3 - l_1*m_4 - l_1*m_5 - m_1*r_1)*np.sin(theta_1))*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2))/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2) - (-l_1**2*(l_4*m_5 + m_4*r_4)*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_4 - theta_1)*np.cos(theta_3 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_3*(l_4*m_5 + m_4*r_4)*np.cos(theta_4 - theta_3) - (-l_1**2*(l_4*m_5 + m_4*r_4)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_4 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_4*m_5 + m_4*r_4)*np.cos(theta_4 - theta_2))*(-l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_3 - theta_2))/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2))*(dtheta_5**2*l_3*m_5*r_5*np.sin(theta_5 - theta_3) + dtheta_4*mu_4 + dtheta_4*(dtheta_4*l_3*l_4*m_5*np.sin(theta_4 - theta_3) + dtheta_4*l_3*m_4*r_4*np.sin(theta_4 - theta_3)) - dtheta_3*(mu_3 + mu_4) + dtheta_2*mu_3 + dtheta_2*(-dtheta_2*l_2*l_3*m_4*np.sin(theta_3 - theta_2) - dtheta_2*l_2*l_3*m_5*np.sin(theta_3 - theta_2) - dtheta_2*l_2*m_3*r_3*np.sin(theta_3 - theta_2)) + dtheta_1*(-dtheta_1*l_1*l_3*m_4*np.sin(theta_3 - theta_1) - dtheta_1*l_1*l_3*m_5*np.sin(theta_3 - theta_1) - dtheta_1*l_1*m_3*r_3*np.sin(theta_3 - theta_1)) - ddx_c*(-l_3*m_4 - l_3*m_5 - m_3*r_3)*np.cos(theta_3) - g*(-l_3*m_4 - l_3*m_5 - m_3*r_3)*np.sin(theta_3) - l_1*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(dtheta_5**2*l_1*m_5*r_5*np.sin(theta_5 - theta_1) + dtheta_4*(dtheta_4*l_1*l_4*m_5*np.sin(theta_4 - theta_1) + dtheta_4*l_1*m_4*r_4*np.sin(theta_4 - theta_1)) + dtheta_3*(dtheta_3*l_1*l_3*m_4*np.sin(theta_3 - theta_1) + dtheta_3*l_1*l_3*m_5*np.sin(theta_3 - theta_1) + dtheta_3*l_1*m_3*r_3*np.sin(theta_3 - theta_1)) + dtheta_2*mu_2 + dtheta_2*(dtheta_2*l_1*l_2*m_3*np.sin(theta_2 - theta_1) + dtheta_2*l_1*l_2*m_4*np.sin(theta_2 - theta_1) + dtheta_2*l_1*l_2*m_5*np.sin(theta_2 - theta_1) + dtheta_2*l_1*m_2*r_2*np.sin(theta_2 - theta_1)) - dtheta_1*(mu_1 + mu_2) - ddx_c*(-l_1*m_2 - l_1*m_3 - l_1*m_4 - l_1*m_5 - m_1*r_1)*np.cos(theta_1) - g*(-l_1*m_2 - l_1*m_3 - l_1*m_4 - l_1*m_5 - m_1*r_1)*np.sin(theta_1))*np.cos(theta_3 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) - (-l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_3 - theta_2))*(dtheta_5**2*l_2*m_5*r_5*np.sin(theta_5 - theta_2) + dtheta_4*(dtheta_4*l_2*l_4*m_5*np.sin(theta_4 - theta_2) + dtheta_4*l_2*m_4*r_4*np.sin(theta_4 - theta_2)) + dtheta_3*mu_3 + dtheta_3*(dtheta_3*l_2*l_3*m_4*np.sin(theta_3 - theta_2) + dtheta_3*l_2*l_3*m_5*np.sin(theta_3 - theta_2) + dtheta_3*l_2*m_3*r_3*np.sin(theta_3 - theta_2)) - dtheta_2*(mu_2 + mu_3) + dtheta_1*mu_2 + dtheta_1*(-dtheta_1*l_1*l_2*m_3*np.sin(theta_2 - theta_1) - dtheta_1*l_1*l_2*m_4*np.sin(theta_2 - theta_1) - dtheta_1*l_1*l_2*m_5*np.sin(theta_2 - theta_1) - dtheta_1*l_1*m_2*r_2*np.sin(theta_2 - theta_1)) - ddx_c*(-l_2*m_3 - l_2*m_4 - l_2*m_5 - m_2*r_2)*np.cos(theta_2) - g*(-l_2*m_3 - l_2*m_4 - l_2*m_5 - m_2*r_2)*np.sin(theta_2) - l_1*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*(dtheta_5**2*l_1*m_5*r_5*np.sin(theta_5 - theta_1) + dtheta_4*(dtheta_4*l_1*l_4*m_5*np.sin(theta_4 - theta_1) + dtheta_4*l_1*m_4*r_4*np.sin(theta_4 - theta_1)) + dtheta_3*(dtheta_3*l_1*l_3*m_4*np.sin(theta_3 - theta_1) + dtheta_3*l_1*l_3*m_5*np.sin(theta_3 - theta_1) + dtheta_3*l_1*m_3*r_3*np.sin(theta_3 - theta_1)) + dtheta_2*mu_2 + dtheta_2*(dtheta_2*l_1*l_2*m_3*np.sin(theta_2 - theta_1) + dtheta_2*l_1*l_2*m_4*np.sin(theta_2 - theta_1) + dtheta_2*l_1*l_2*m_5*np.sin(theta_2 - theta_1) + dtheta_2*l_1*m_2*r_2*np.sin(theta_2 - theta_1)) - dtheta_1*(mu_1 + mu_2) - ddx_c*(-l_1*m_2 - l_1*m_3 - l_1*m_4 - l_1*m_5 - m_1*r_1)*np.cos(theta_1) - g*(-l_1*m_2 - l_1*m_3 - l_1*m_4 - l_1*m_5 - m_1*r_1)*np.sin(theta_1))*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2))/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2))/(I_3 - l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)**2*np.cos(theta_3 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_3**2*m_4 + l_3**2*m_5 + m_3*r_3**2 - (-l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_3 - theta_2))**2/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2)))/(I_4 - l_1**2*(l_4*m_5 + m_4*r_4)**2*np.cos(theta_4 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_4**2*m_5 + m_4*r_4**2 - (-l_1**2*(l_4*m_5 + m_4*r_4)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_4 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_4*m_5 + m_4*r_4)*np.cos(theta_4 - theta_2))**2/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2) - (-l_1**2*(l_4*m_5 + m_4*r_4)*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_4 - theta_1)*np.cos(theta_3 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_3*(l_4*m_5 + m_4*r_4)*np.cos(theta_4 - theta_3) - (-l_1**2*(l_4*m_5 + m_4*r_4)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_4 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_4*m_5 + m_4*r_4)*np.cos(theta_4 - theta_2))*(-l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_3 - theta_2))/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2))**2/(I_3 - l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)**2*np.cos(theta_3 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_3**2*m_4 + l_3**2*m_5 + m_3*r_3**2 - (-l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_3 - theta_2))**2/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2))))/(I_5 - l_1**2*m_5**2*r_5**2*np.cos(theta_5 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + m_5*r_5**2 - (-l_1**2*m_5*r_5*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_5 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*m_5*r_5*np.cos(theta_5 - theta_2))**2/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2) - (-l_1**2*m_5*r_5*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_5 - theta_1)*np.cos(theta_3 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_3*m_5*r_5*np.cos(theta_5 - theta_3) - (-l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_3 - theta_2))*(-l_1**2*m_5*r_5*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_5 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*m_5*r_5*np.cos(theta_5 - theta_2))/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2))**2/(I_3 - l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)**2*np.cos(theta_3 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_3**2*m_4 + l_3**2*m_5 + m_3*r_3**2 - (-l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_3 - theta_2))**2/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2)) - (-l_1**2*m_5*r_5*(l_4*m_5 + m_4*r_4)*np.cos(theta_5 - theta_1)*np.cos(theta_4 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_4*m_5*r_5*np.cos(theta_5 - theta_4) - (-l_1**2*(l_4*m_5 + m_4*r_4)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_4 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_4*m_5 + m_4*r_4)*np.cos(theta_4 - theta_2))*(-l_1**2*m_5*r_5*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_5 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*m_5*r_5*np.cos(theta_5 - theta_2))/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2) - (-l_1**2*(l_4*m_5 + m_4*r_4)*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_4 - theta_1)*np.cos(theta_3 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_3*(l_4*m_5 + m_4*r_4)*np.cos(theta_4 - theta_3) - (-l_1**2*(l_4*m_5 + m_4*r_4)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_4 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_4*m_5 + m_4*r_4)*np.cos(theta_4 - theta_2))*(-l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_3 - theta_2))/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2))*(-l_1**2*m_5*r_5*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_5 - theta_1)*np.cos(theta_3 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_3*m_5*r_5*np.cos(theta_5 - theta_3) - (-l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_3 - theta_2))*(-l_1**2*m_5*r_5*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_5 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*m_5*r_5*np.cos(theta_5 - theta_2))/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2))/(I_3 - l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)**2*np.cos(theta_3 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_3**2*m_4 + l_3**2*m_5 + m_3*r_3**2 - (-l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_3 - theta_2))**2/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2)))**2/(I_4 - l_1**2*(l_4*m_5 + m_4*r_4)**2*np.cos(theta_4 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_4**2*m_5 + m_4*r_4**2 - (-l_1**2*(l_4*m_5 + m_4*r_4)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_4 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_4*m_5 + m_4*r_4)*np.cos(theta_4 - theta_2))**2/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2) - (-l_1**2*(l_4*m_5 + m_4*r_4)*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_4 - theta_1)*np.cos(theta_3 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_3*(l_4*m_5 + m_4*r_4)*np.cos(theta_4 - theta_3) - (-l_1**2*(l_4*m_5 + m_4*r_4)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_4 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_4*m_5 + m_4*r_4)*np.cos(theta_4 - theta_2))*(-l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_3 - theta_2))/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2))**2/(I_3 - l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)**2*np.cos(theta_3 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_3**2*m_4 + l_3**2*m_5 + m_3*r_3**2 - (-l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_3 - theta_2))**2/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2)))))/(I_4 - l_1**2*(l_4*m_5 + m_4*r_4)**2*np.cos(theta_4 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_4**2*m_5 + m_4*r_4**2 - (-l_1**2*(l_4*m_5 + m_4*r_4)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_4 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_4*m_5 + m_4*r_4)*np.cos(theta_4 - theta_2))**2/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2) - (-l_1**2*(l_4*m_5 + m_4*r_4)*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_4 - theta_1)*np.cos(theta_3 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_3*(l_4*m_5 + m_4*r_4)*np.cos(theta_4 - theta_3) - (-l_1**2*(l_4*m_5 + m_4*r_4)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_4 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_4*m_5 + m_4*r_4)*np.cos(theta_4 - theta_2))*(-l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_3 - theta_2))/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2))**2/(I_3 - l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)**2*np.cos(theta_3 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_3**2*m_4 + l_3**2*m_5 + m_3*r_3**2 - (-l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_3 - theta_2))**2/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2))) - (-l_1**2*m_5*r_5*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_5 - theta_1)*np.cos(theta_3 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_3*m_5*r_5*np.cos(theta_5 - theta_3) - (-l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_3 - theta_2))*(-l_1**2*m_5*r_5*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_5 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*m_5*r_5*np.cos(theta_5 - theta_2))/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2))*(-dtheta_5*mu_5 - dtheta_4**2*l_4*m_5*r_5*np.sin(theta_5 - theta_4) + dtheta_4*mu_5 - dtheta_3**2*l_3*m_5*r_5*np.sin(theta_5 - theta_3) - dtheta_2**2*l_2*m_5*r_5*np.sin(theta_5 - theta_2) - dtheta_1**2*l_1*m_5*r_5*np.sin(theta_5 - theta_1) + ddx_c*m_5*r_5*np.cos(theta_5) + g*m_5*r_5*np.sin(theta_5) - l_1*m_5*r_5*(dtheta_5**2*l_1*m_5*r_5*np.sin(theta_5 - theta_1) + dtheta_4*(dtheta_4*l_1*l_4*m_5*np.sin(theta_4 - theta_1) + dtheta_4*l_1*m_4*r_4*np.sin(theta_4 - theta_1)) + dtheta_3*(dtheta_3*l_1*l_3*m_4*np.sin(theta_3 - theta_1) + dtheta_3*l_1*l_3*m_5*np.sin(theta_3 - theta_1) + dtheta_3*l_1*m_3*r_3*np.sin(theta_3 - theta_1)) + dtheta_2*mu_2 + dtheta_2*(dtheta_2*l_1*l_2*m_3*np.sin(theta_2 - theta_1) + dtheta_2*l_1*l_2*m_4*np.sin(theta_2 - theta_1) + dtheta_2*l_1*l_2*m_5*np.sin(theta_2 - theta_1) + dtheta_2*l_1*m_2*r_2*np.sin(theta_2 - theta_1)) - dtheta_1*(mu_1 + mu_2) - ddx_c*(-l_1*m_2 - l_1*m_3 - l_1*m_4 - l_1*m_5 - m_1*r_1)*np.cos(theta_1) - g*(-l_1*m_2 - l_1*m_3 - l_1*m_4 - l_1*m_5 - m_1*r_1)*np.sin(theta_1))*np.cos(theta_5 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) - (-l_1**2*m_5*r_5*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_5 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*m_5*r_5*np.cos(theta_5 - theta_2))*(dtheta_5**2*l_2*m_5*r_5*np.sin(theta_5 - theta_2) + dtheta_4*(dtheta_4*l_2*l_4*m_5*np.sin(theta_4 - theta_2) + dtheta_4*l_2*m_4*r_4*np.sin(theta_4 - theta_2)) + dtheta_3*mu_3 + dtheta_3*(dtheta_3*l_2*l_3*m_4*np.sin(theta_3 - theta_2) + dtheta_3*l_2*l_3*m_5*np.sin(theta_3 - theta_2) + dtheta_3*l_2*m_3*r_3*np.sin(theta_3 - theta_2)) - dtheta_2*(mu_2 + mu_3) + dtheta_1*mu_2 + dtheta_1*(-dtheta_1*l_1*l_2*m_3*np.sin(theta_2 - theta_1) - dtheta_1*l_1*l_2*m_4*np.sin(theta_2 - theta_1) - dtheta_1*l_1*l_2*m_5*np.sin(theta_2 - theta_1) - dtheta_1*l_1*m_2*r_2*np.sin(theta_2 - theta_1)) - ddx_c*(-l_2*m_3 - l_2*m_4 - l_2*m_5 - m_2*r_2)*np.cos(theta_2) - g*(-l_2*m_3 - l_2*m_4 - l_2*m_5 - m_2*r_2)*np.sin(theta_2) - l_1*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*(dtheta_5**2*l_1*m_5*r_5*np.sin(theta_5 - theta_1) + dtheta_4*(dtheta_4*l_1*l_4*m_5*np.sin(theta_4 - theta_1) + dtheta_4*l_1*m_4*r_4*np.sin(theta_4 - theta_1)) + dtheta_3*(dtheta_3*l_1*l_3*m_4*np.sin(theta_3 - theta_1) + dtheta_3*l_1*l_3*m_5*np.sin(theta_3 - theta_1) + dtheta_3*l_1*m_3*r_3*np.sin(theta_3 - theta_1)) + dtheta_2*mu_2 + dtheta_2*(dtheta_2*l_1*l_2*m_3*np.sin(theta_2 - theta_1) + dtheta_2*l_1*l_2*m_4*np.sin(theta_2 - theta_1) + dtheta_2*l_1*l_2*m_5*np.sin(theta_2 - theta_1) + dtheta_2*l_1*m_2*r_2*np.sin(theta_2 - theta_1)) - dtheta_1*(mu_1 + mu_2) - ddx_c*(-l_1*m_2 - l_1*m_3 - l_1*m_4 - l_1*m_5 - m_1*r_1)*np.cos(theta_1) - g*(-l_1*m_2 - l_1*m_3 - l_1*m_4 - l_1*m_5 - m_1*r_1)*np.sin(theta_1))*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2))/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2) - (-l_1**2*m_5*r_5*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_5 - theta_1)*np.cos(theta_3 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_3*m_5*r_5*np.cos(theta_5 - theta_3) - (-l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_3 - theta_2))*(-l_1**2*m_5*r_5*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_5 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*m_5*r_5*np.cos(theta_5 - theta_2))/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2))*(dtheta_5**2*l_3*m_5*r_5*np.sin(theta_5 - theta_3) + dtheta_4*mu_4 + dtheta_4*(dtheta_4*l_3*l_4*m_5*np.sin(theta_4 - theta_3) + dtheta_4*l_3*m_4*r_4*np.sin(theta_4 - theta_3)) - dtheta_3*(mu_3 + mu_4) + dtheta_2*mu_3 + dtheta_2*(-dtheta_2*l_2*l_3*m_4*np.sin(theta_3 - theta_2) - dtheta_2*l_2*l_3*m_5*np.sin(theta_3 - theta_2) - dtheta_2*l_2*m_3*r_3*np.sin(theta_3 - theta_2)) + dtheta_1*(-dtheta_1*l_1*l_3*m_4*np.sin(theta_3 - theta_1) - dtheta_1*l_1*l_3*m_5*np.sin(theta_3 - theta_1) - dtheta_1*l_1*m_3*r_3*np.sin(theta_3 - theta_1)) - ddx_c*(-l_3*m_4 - l_3*m_5 - m_3*r_3)*np.cos(theta_3) - g*(-l_3*m_4 - l_3*m_5 - m_3*r_3)*np.sin(theta_3) - l_1*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(dtheta_5**2*l_1*m_5*r_5*np.sin(theta_5 - theta_1) + dtheta_4*(dtheta_4*l_1*l_4*m_5*np.sin(theta_4 - theta_1) + dtheta_4*l_1*m_4*r_4*np.sin(theta_4 - theta_1)) + dtheta_3*(dtheta_3*l_1*l_3*m_4*np.sin(theta_3 - theta_1) + dtheta_3*l_1*l_3*m_5*np.sin(theta_3 - theta_1) + dtheta_3*l_1*m_3*r_3*np.sin(theta_3 - theta_1)) + dtheta_2*mu_2 + dtheta_2*(dtheta_2*l_1*l_2*m_3*np.sin(theta_2 - theta_1) + dtheta_2*l_1*l_2*m_4*np.sin(theta_2 - theta_1) + dtheta_2*l_1*l_2*m_5*np.sin(theta_2 - theta_1) + dtheta_2*l_1*m_2*r_2*np.sin(theta_2 - theta_1)) - dtheta_1*(mu_1 + mu_2) - ddx_c*(-l_1*m_2 - l_1*m_3 - l_1*m_4 - l_1*m_5 - m_1*r_1)*np.cos(theta_1) - g*(-l_1*m_2 - l_1*m_3 - l_1*m_4 - l_1*m_5 - m_1*r_1)*np.sin(theta_1))*np.cos(theta_3 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) - (-l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_3 - theta_2))*(dtheta_5**2*l_2*m_5*r_5*np.sin(theta_5 - theta_2) + dtheta_4*(dtheta_4*l_2*l_4*m_5*np.sin(theta_4 - theta_2) + dtheta_4*l_2*m_4*r_4*np.sin(theta_4 - theta_2)) + dtheta_3*mu_3 + dtheta_3*(dtheta_3*l_2*l_3*m_4*np.sin(theta_3 - theta_2) + dtheta_3*l_2*l_3*m_5*np.sin(theta_3 - theta_2) + dtheta_3*l_2*m_3*r_3*np.sin(theta_3 - theta_2)) - dtheta_2*(mu_2 + mu_3) + dtheta_1*mu_2 + dtheta_1*(-dtheta_1*l_1*l_2*m_3*np.sin(theta_2 - theta_1) - dtheta_1*l_1*l_2*m_4*np.sin(theta_2 - theta_1) - dtheta_1*l_1*l_2*m_5*np.sin(theta_2 - theta_1) - dtheta_1*l_1*m_2*r_2*np.sin(theta_2 - theta_1)) - ddx_c*(-l_2*m_3 - l_2*m_4 - l_2*m_5 - m_2*r_2)*np.cos(theta_2) - g*(-l_2*m_3 - l_2*m_4 - l_2*m_5 - m_2*r_2)*np.sin(theta_2) - l_1*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*(dtheta_5**2*l_1*m_5*r_5*np.sin(theta_5 - theta_1) + dtheta_4*(dtheta_4*l_1*l_4*m_5*np.sin(theta_4 - theta_1) + dtheta_4*l_1*m_4*r_4*np.sin(theta_4 - theta_1)) + dtheta_3*(dtheta_3*l_1*l_3*m_4*np.sin(theta_3 - theta_1) + dtheta_3*l_1*l_3*m_5*np.sin(theta_3 - theta_1) + dtheta_3*l_1*m_3*r_3*np.sin(theta_3 - theta_1)) + dtheta_2*mu_2 + dtheta_2*(dtheta_2*l_1*l_2*m_3*np.sin(theta_2 - theta_1) + dtheta_2*l_1*l_2*m_4*np.sin(theta_2 - theta_1) + dtheta_2*l_1*l_2*m_5*np.sin(theta_2 - theta_1) + dtheta_2*l_1*m_2*r_2*np.sin(theta_2 - theta_1)) - dtheta_1*(mu_1 + mu_2) - ddx_c*(-l_1*m_2 - l_1*m_3 - l_1*m_4 - l_1*m_5 - m_1*r_1)*np.cos(theta_1) - g*(-l_1*m_2 - l_1*m_3 - l_1*m_4 - l_1*m_5 - m_1*r_1)*np.sin(theta_1))*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2))/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2))/(I_3 - l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)**2*np.cos(theta_3 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_3**2*m_4 + l_3**2*m_5 + m_3*r_3**2 - (-l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_3 - theta_2))**2/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2)) - (-l_1**2*m_5*r_5*(l_4*m_5 + m_4*r_4)*np.cos(theta_5 - theta_1)*np.cos(theta_4 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_4*m_5*r_5*np.cos(theta_5 - theta_4) - (-l_1**2*(l_4*m_5 + m_4*r_4)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_4 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_4*m_5 + m_4*r_4)*np.cos(theta_4 - theta_2))*(-l_1**2*m_5*r_5*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_5 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*m_5*r_5*np.cos(theta_5 - theta_2))/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2) - (-l_1**2*(l_4*m_5 + m_4*r_4)*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_4 - theta_1)*np.cos(theta_3 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_3*(l_4*m_5 + m_4*r_4)*np.cos(theta_4 - theta_3) - (-l_1**2*(l_4*m_5 + m_4*r_4)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_4 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_4*m_5 + m_4*r_4)*np.cos(theta_4 - theta_2))*(-l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_3 - theta_2))/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2))*(-l_1**2*m_5*r_5*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_5 - theta_1)*np.cos(theta_3 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_3*m_5*r_5*np.cos(theta_5 - theta_3) - (-l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_3 - theta_2))*(-l_1**2*m_5*r_5*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_5 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*m_5*r_5*np.cos(theta_5 - theta_2))/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2))/(I_3 - l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)**2*np.cos(theta_3 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_3**2*m_4 + l_3**2*m_5 + m_3*r_3**2 - (-l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_3 - theta_2))**2/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2)))*(dtheta_5**2*l_4*m_5*r_5*np.sin(theta_5 - theta_4) + dtheta_5*mu_5 - dtheta_4*(mu_4 + mu_5) + dtheta_3*mu_4 + dtheta_3*(-dtheta_3*l_3*l_4*m_5*np.sin(theta_4 - theta_3) - dtheta_3*l_3*m_4*r_4*np.sin(theta_4 - theta_3)) + dtheta_2*(-dtheta_2*l_2*l_4*m_5*np.sin(theta_4 - theta_2) - dtheta_2*l_2*m_4*r_4*np.sin(theta_4 - theta_2)) + dtheta_1*(-dtheta_1*l_1*l_4*m_5*np.sin(theta_4 - theta_1) - dtheta_1*l_1*m_4*r_4*np.sin(theta_4 - theta_1)) - ddx_c*(-l_4*m_5 - m_4*r_4)*np.cos(theta_4) - g*(-l_4*m_5 - m_4*r_4)*np.sin(theta_4) - l_1*(l_4*m_5 + m_4*r_4)*(dtheta_5**2*l_1*m_5*r_5*np.sin(theta_5 - theta_1) + dtheta_4*(dtheta_4*l_1*l_4*m_5*np.sin(theta_4 - theta_1) + dtheta_4*l_1*m_4*r_4*np.sin(theta_4 - theta_1)) + dtheta_3*(dtheta_3*l_1*l_3*m_4*np.sin(theta_3 - theta_1) + dtheta_3*l_1*l_3*m_5*np.sin(theta_3 - theta_1) + dtheta_3*l_1*m_3*r_3*np.sin(theta_3 - theta_1)) + dtheta_2*mu_2 + dtheta_2*(dtheta_2*l_1*l_2*m_3*np.sin(theta_2 - theta_1) + dtheta_2*l_1*l_2*m_4*np.sin(theta_2 - theta_1) + dtheta_2*l_1*l_2*m_5*np.sin(theta_2 - theta_1) + dtheta_2*l_1*m_2*r_2*np.sin(theta_2 - theta_1)) - dtheta_1*(mu_1 + mu_2) - ddx_c*(-l_1*m_2 - l_1*m_3 - l_1*m_4 - l_1*m_5 - m_1*r_1)*np.cos(theta_1) - g*(-l_1*m_2 - l_1*m_3 - l_1*m_4 - l_1*m_5 - m_1*r_1)*np.sin(theta_1))*np.cos(theta_4 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) - (-l_1**2*(l_4*m_5 + m_4*r_4)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_4 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_4*m_5 + m_4*r_4)*np.cos(theta_4 - theta_2))*(dtheta_5**2*l_2*m_5*r_5*np.sin(theta_5 - theta_2) + dtheta_4*(dtheta_4*l_2*l_4*m_5*np.sin(theta_4 - theta_2) + dtheta_4*l_2*m_4*r_4*np.sin(theta_4 - theta_2)) + dtheta_3*mu_3 + dtheta_3*(dtheta_3*l_2*l_3*m_4*np.sin(theta_3 - theta_2) + dtheta_3*l_2*l_3*m_5*np.sin(theta_3 - theta_2) + dtheta_3*l_2*m_3*r_3*np.sin(theta_3 - theta_2)) - dtheta_2*(mu_2 + mu_3) + dtheta_1*mu_2 + dtheta_1*(-dtheta_1*l_1*l_2*m_3*np.sin(theta_2 - theta_1) - dtheta_1*l_1*l_2*m_4*np.sin(theta_2 - theta_1) - dtheta_1*l_1*l_2*m_5*np.sin(theta_2 - theta_1) - dtheta_1*l_1*m_2*r_2*np.sin(theta_2 - theta_1)) - ddx_c*(-l_2*m_3 - l_2*m_4 - l_2*m_5 - m_2*r_2)*np.cos(theta_2) - g*(-l_2*m_3 - l_2*m_4 - l_2*m_5 - m_2*r_2)*np.sin(theta_2) - l_1*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*(dtheta_5**2*l_1*m_5*r_5*np.sin(theta_5 - theta_1) + dtheta_4*(dtheta_4*l_1*l_4*m_5*np.sin(theta_4 - theta_1) + dtheta_4*l_1*m_4*r_4*np.sin(theta_4 - theta_1)) + dtheta_3*(dtheta_3*l_1*l_3*m_4*np.sin(theta_3 - theta_1) + dtheta_3*l_1*l_3*m_5*np.sin(theta_3 - theta_1) + dtheta_3*l_1*m_3*r_3*np.sin(theta_3 - theta_1)) + dtheta_2*mu_2 + dtheta_2*(dtheta_2*l_1*l_2*m_3*np.sin(theta_2 - theta_1) + dtheta_2*l_1*l_2*m_4*np.sin(theta_2 - theta_1) + dtheta_2*l_1*l_2*m_5*np.sin(theta_2 - theta_1) + dtheta_2*l_1*m_2*r_2*np.sin(theta_2 - theta_1)) - dtheta_1*(mu_1 + mu_2) - ddx_c*(-l_1*m_2 - l_1*m_3 - l_1*m_4 - l_1*m_5 - m_1*r_1)*np.cos(theta_1) - g*(-l_1*m_2 - l_1*m_3 - l_1*m_4 - l_1*m_5 - m_1*r_1)*np.sin(theta_1))*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2))/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2) - (-l_1**2*(l_4*m_5 + m_4*r_4)*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_4 - theta_1)*np.cos(theta_3 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_3*(l_4*m_5 + m_4*r_4)*np.cos(theta_4 - theta_3) - (-l_1**2*(l_4*m_5 + m_4*r_4)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_4 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_4*m_5 + m_4*r_4)*np.cos(theta_4 - theta_2))*(-l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_3 - theta_2))/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2))*(dtheta_5**2*l_3*m_5*r_5*np.sin(theta_5 - theta_3) + dtheta_4*mu_4 + dtheta_4*(dtheta_4*l_3*l_4*m_5*np.sin(theta_4 - theta_3) + dtheta_4*l_3*m_4*r_4*np.sin(theta_4 - theta_3)) - dtheta_3*(mu_3 + mu_4) + dtheta_2*mu_3 + dtheta_2*(-dtheta_2*l_2*l_3*m_4*np.sin(theta_3 - theta_2) - dtheta_2*l_2*l_3*m_5*np.sin(theta_3 - theta_2) - dtheta_2*l_2*m_3*r_3*np.sin(theta_3 - theta_2)) + dtheta_1*(-dtheta_1*l_1*l_3*m_4*np.sin(theta_3 - theta_1) - dtheta_1*l_1*l_3*m_5*np.sin(theta_3 - theta_1) - dtheta_1*l_1*m_3*r_3*np.sin(theta_3 - theta_1)) - ddx_c*(-l_3*m_4 - l_3*m_5 - m_3*r_3)*np.cos(theta_3) - g*(-l_3*m_4 - l_3*m_5 - m_3*r_3)*np.sin(theta_3) - l_1*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(dtheta_5**2*l_1*m_5*r_5*np.sin(theta_5 - theta_1) + dtheta_4*(dtheta_4*l_1*l_4*m_5*np.sin(theta_4 - theta_1) + dtheta_4*l_1*m_4*r_4*np.sin(theta_4 - theta_1)) + dtheta_3*(dtheta_3*l_1*l_3*m_4*np.sin(theta_3 - theta_1) + dtheta_3*l_1*l_3*m_5*np.sin(theta_3 - theta_1) + dtheta_3*l_1*m_3*r_3*np.sin(theta_3 - theta_1)) + dtheta_2*mu_2 + dtheta_2*(dtheta_2*l_1*l_2*m_3*np.sin(theta_2 - theta_1) + dtheta_2*l_1*l_2*m_4*np.sin(theta_2 - theta_1) + dtheta_2*l_1*l_2*m_5*np.sin(theta_2 - theta_1) + dtheta_2*l_1*m_2*r_2*np.sin(theta_2 - theta_1)) - dtheta_1*(mu_1 + mu_2) - ddx_c*(-l_1*m_2 - l_1*m_3 - l_1*m_4 - l_1*m_5 - m_1*r_1)*np.cos(theta_1) - g*(-l_1*m_2 - l_1*m_3 - l_1*m_4 - l_1*m_5 - m_1*r_1)*np.sin(theta_1))*np.cos(theta_3 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) - (-l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_3 - theta_2))*(dtheta_5**2*l_2*m_5*r_5*np.sin(theta_5 - theta_2) + dtheta_4*(dtheta_4*l_2*l_4*m_5*np.sin(theta_4 - theta_2) + dtheta_4*l_2*m_4*r_4*np.sin(theta_4 - theta_2)) + dtheta_3*mu_3 + dtheta_3*(dtheta_3*l_2*l_3*m_4*np.sin(theta_3 - theta_2) + dtheta_3*l_2*l_3*m_5*np.sin(theta_3 - theta_2) + dtheta_3*l_2*m_3*r_3*np.sin(theta_3 - theta_2)) - dtheta_2*(mu_2 + mu_3) + dtheta_1*mu_2 + dtheta_1*(-dtheta_1*l_1*l_2*m_3*np.sin(theta_2 - theta_1) - dtheta_1*l_1*l_2*m_4*np.sin(theta_2 - theta_1) - dtheta_1*l_1*l_2*m_5*np.sin(theta_2 - theta_1) - dtheta_1*l_1*m_2*r_2*np.sin(theta_2 - theta_1)) - ddx_c*(-l_2*m_3 - l_2*m_4 - l_2*m_5 - m_2*r_2)*np.cos(theta_2) - g*(-l_2*m_3 - l_2*m_4 - l_2*m_5 - m_2*r_2)*np.sin(theta_2) - l_1*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*(dtheta_5**2*l_1*m_5*r_5*np.sin(theta_5 - theta_1) + dtheta_4*(dtheta_4*l_1*l_4*m_5*np.sin(theta_4 - theta_1) + dtheta_4*l_1*m_4*r_4*np.sin(theta_4 - theta_1)) + dtheta_3*(dtheta_3*l_1*l_3*m_4*np.sin(theta_3 - theta_1) + dtheta_3*l_1*l_3*m_5*np.sin(theta_3 - theta_1) + dtheta_3*l_1*m_3*r_3*np.sin(theta_3 - theta_1)) + dtheta_2*mu_2 + dtheta_2*(dtheta_2*l_1*l_2*m_3*np.sin(theta_2 - theta_1) + dtheta_2*l_1*l_2*m_4*np.sin(theta_2 - theta_1) + dtheta_2*l_1*l_2*m_5*np.sin(theta_2 - theta_1) + dtheta_2*l_1*m_2*r_2*np.sin(theta_2 - theta_1)) - dtheta_1*(mu_1 + mu_2) - ddx_c*(-l_1*m_2 - l_1*m_3 - l_1*m_4 - l_1*m_5 - m_1*r_1)*np.cos(theta_1) - g*(-l_1*m_2 - l_1*m_3 - l_1*m_4 - l_1*m_5 - m_1*r_1)*np.sin(theta_1))*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2))/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2))/(I_3 - l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)**2*np.cos(theta_3 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_3**2*m_4 + l_3**2*m_5 + m_3*r_3**2 - (-l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_3 - theta_2))**2/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2)))/(I_4 - l_1**2*(l_4*m_5 + m_4*r_4)**2*np.cos(theta_4 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_4**2*m_5 + m_4*r_4**2 - (-l_1**2*(l_4*m_5 + m_4*r_4)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_4 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_4*m_5 + m_4*r_4)*np.cos(theta_4 - theta_2))**2/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2) - (-l_1**2*(l_4*m_5 + m_4*r_4)*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_4 - theta_1)*np.cos(theta_3 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_3*(l_4*m_5 + m_4*r_4)*np.cos(theta_4 - theta_3) - (-l_1**2*(l_4*m_5 + m_4*r_4)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_4 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_4*m_5 + m_4*r_4)*np.cos(theta_4 - theta_2))*(-l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_3 - theta_2))/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2))**2/(I_3 - l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)**2*np.cos(theta_3 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_3**2*m_4 + l_3**2*m_5 + m_3*r_3**2 - (-l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_3 - theta_2))**2/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2))))/(I_5 - l_1**2*m_5**2*r_5**2*np.cos(theta_5 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + m_5*r_5**2 - (-l_1**2*m_5*r_5*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_5 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*m_5*r_5*np.cos(theta_5 - theta_2))**2/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2) - (-l_1**2*m_5*r_5*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_5 - theta_1)*np.cos(theta_3 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_3*m_5*r_5*np.cos(theta_5 - theta_3) - (-l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_3 - theta_2))*(-l_1**2*m_5*r_5*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_5 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*m_5*r_5*np.cos(theta_5 - theta_2))/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2))**2/(I_3 - l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)**2*np.cos(theta_3 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_3**2*m_4 + l_3**2*m_5 + m_3*r_3**2 - (-l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_3 - theta_2))**2/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2)) - (-l_1**2*m_5*r_5*(l_4*m_5 + m_4*r_4)*np.cos(theta_5 - theta_1)*np.cos(theta_4 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_4*m_5*r_5*np.cos(theta_5 - theta_4) - (-l_1**2*(l_4*m_5 + m_4*r_4)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_4 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_4*m_5 + m_4*r_4)*np.cos(theta_4 - theta_2))*(-l_1**2*m_5*r_5*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_5 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*m_5*r_5*np.cos(theta_5 - theta_2))/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2) - (-l_1**2*(l_4*m_5 + m_4*r_4)*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_4 - theta_1)*np.cos(theta_3 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_3*(l_4*m_5 + m_4*r_4)*np.cos(theta_4 - theta_3) - (-l_1**2*(l_4*m_5 + m_4*r_4)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_4 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_4*m_5 + m_4*r_4)*np.cos(theta_4 - theta_2))*(-l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_3 - theta_2))/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2))*(-l_1**2*m_5*r_5*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_5 - theta_1)*np.cos(theta_3 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_3*m_5*r_5*np.cos(theta_5 - theta_3) - (-l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_3 - theta_2))*(-l_1**2*m_5*r_5*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_5 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*m_5*r_5*np.cos(theta_5 - theta_2))/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2))/(I_3 - l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)**2*np.cos(theta_3 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_3**2*m_4 + l_3**2*m_5 + m_3*r_3**2 - (-l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_3 - theta_2))**2/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2)))**2/(I_4 - l_1**2*(l_4*m_5 + m_4*r_4)**2*np.cos(theta_4 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_4**2*m_5 + m_4*r_4**2 - (-l_1**2*(l_4*m_5 + m_4*r_4)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_4 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_4*m_5 + m_4*r_4)*np.cos(theta_4 - theta_2))**2/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2) - (-l_1**2*(l_4*m_5 + m_4*r_4)*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_4 - theta_1)*np.cos(theta_3 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_3*(l_4*m_5 + m_4*r_4)*np.cos(theta_4 - theta_3) - (-l_1**2*(l_4*m_5 + m_4*r_4)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_4 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_4*m_5 + m_4*r_4)*np.cos(theta_4 - theta_2))*(-l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_3 - theta_2))/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2))**2/(I_3 - l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)**2*np.cos(theta_3 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_3**2*m_4 + l_3**2*m_5 + m_3*r_3**2 - (-l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_3 - theta_2))**2/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2)))))*np.cos(theta_3 - theta_1)/(I_3 - l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)**2*np.cos(theta_3 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_3**2*m_4 + l_3**2*m_5 + m_3*r_3**2 - (-l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_3 - theta_2))**2/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2)) - l_1*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*(dtheta_5**2*l_2*m_5*r_5*np.sin(theta_5 - theta_2) + dtheta_4*(dtheta_4*l_2*l_4*m_5*np.sin(theta_4 - theta_2) + dtheta_4*l_2*m_4*r_4*np.sin(theta_4 - theta_2)) + dtheta_3*mu_3 + dtheta_3*(dtheta_3*l_2*l_3*m_4*np.sin(theta_3 - theta_2) + dtheta_3*l_2*l_3*m_5*np.sin(theta_3 - theta_2) + dtheta_3*l_2*m_3*r_3*np.sin(theta_3 - theta_2)) - dtheta_2*(mu_2 + mu_3) + dtheta_1*mu_2 + dtheta_1*(-dtheta_1*l_1*l_2*m_3*np.sin(theta_2 - theta_1) - dtheta_1*l_1*l_2*m_4*np.sin(theta_2 - theta_1) - dtheta_1*l_1*l_2*m_5*np.sin(theta_2 - theta_1) - dtheta_1*l_1*m_2*r_2*np.sin(theta_2 - theta_1)) - ddx_c*(-l_2*m_3 - l_2*m_4 - l_2*m_5 - m_2*r_2)*np.cos(theta_2) - g*(-l_2*m_3 - l_2*m_4 - l_2*m_5 - m_2*r_2)*np.sin(theta_2) - l_1*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*(dtheta_5**2*l_1*m_5*r_5*np.sin(theta_5 - theta_1) + dtheta_4*(dtheta_4*l_1*l_4*m_5*np.sin(theta_4 - theta_1) + dtheta_4*l_1*m_4*r_4*np.sin(theta_4 - theta_1)) + dtheta_3*(dtheta_3*l_1*l_3*m_4*np.sin(theta_3 - theta_1) + dtheta_3*l_1*l_3*m_5*np.sin(theta_3 - theta_1) + dtheta_3*l_1*m_3*r_3*np.sin(theta_3 - theta_1)) + dtheta_2*mu_2 + dtheta_2*(dtheta_2*l_1*l_2*m_3*np.sin(theta_2 - theta_1) + dtheta_2*l_1*l_2*m_4*np.sin(theta_2 - theta_1) + dtheta_2*l_1*l_2*m_5*np.sin(theta_2 - theta_1) + dtheta_2*l_1*m_2*r_2*np.sin(theta_2 - theta_1)) - dtheta_1*(mu_1 + mu_2) - ddx_c*(-l_1*m_2 - l_1*m_3 - l_1*m_4 - l_1*m_5 - m_1*r_1)*np.cos(theta_1) - g*(-l_1*m_2 - l_1*m_3 - l_1*m_4 - l_1*m_5 - m_1*r_1)*np.sin(theta_1))*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) - (-l_1**2*(l_4*m_5 + m_4*r_4)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_4 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_4*m_5 + m_4*r_4)*np.cos(theta_4 - theta_2))*(dtheta_5**2*l_4*m_5*r_5*np.sin(theta_5 - theta_4) + dtheta_5*mu_5 - dtheta_4*(mu_4 + mu_5) + dtheta_3*mu_4 + dtheta_3*(-dtheta_3*l_3*l_4*m_5*np.sin(theta_4 - theta_3) - dtheta_3*l_3*m_4*r_4*np.sin(theta_4 - theta_3)) + dtheta_2*(-dtheta_2*l_2*l_4*m_5*np.sin(theta_4 - theta_2) - dtheta_2*l_2*m_4*r_4*np.sin(theta_4 - theta_2)) + dtheta_1*(-dtheta_1*l_1*l_4*m_5*np.sin(theta_4 - theta_1) - dtheta_1*l_1*m_4*r_4*np.sin(theta_4 - theta_1)) - ddx_c*(-l_4*m_5 - m_4*r_4)*np.cos(theta_4) - g*(-l_4*m_5 - m_4*r_4)*np.sin(theta_4) - l_1*(l_4*m_5 + m_4*r_4)*(dtheta_5**2*l_1*m_5*r_5*np.sin(theta_5 - theta_1) + dtheta_4*(dtheta_4*l_1*l_4*m_5*np.sin(theta_4 - theta_1) + dtheta_4*l_1*m_4*r_4*np.sin(theta_4 - theta_1)) + dtheta_3*(dtheta_3*l_1*l_3*m_4*np.sin(theta_3 - theta_1) + dtheta_3*l_1*l_3*m_5*np.sin(theta_3 - theta_1) + dtheta_3*l_1*m_3*r_3*np.sin(theta_3 - theta_1)) + dtheta_2*mu_2 + dtheta_2*(dtheta_2*l_1*l_2*m_3*np.sin(theta_2 - theta_1) + dtheta_2*l_1*l_2*m_4*np.sin(theta_2 - theta_1) + dtheta_2*l_1*l_2*m_5*np.sin(theta_2 - theta_1) + dtheta_2*l_1*m_2*r_2*np.sin(theta_2 - theta_1)) - dtheta_1*(mu_1 + mu_2) - ddx_c*(-l_1*m_2 - l_1*m_3 - l_1*m_4 - l_1*m_5 - m_1*r_1)*np.cos(theta_1) - g*(-l_1*m_2 - l_1*m_3 - l_1*m_4 - l_1*m_5 - m_1*r_1)*np.sin(theta_1))*np.cos(theta_4 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) - (-l_1**2*(l_4*m_5 + m_4*r_4)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_4 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_4*m_5 + m_4*r_4)*np.cos(theta_4 - theta_2))*(dtheta_5**2*l_2*m_5*r_5*np.sin(theta_5 - theta_2) + dtheta_4*(dtheta_4*l_2*l_4*m_5*np.sin(theta_4 - theta_2) + dtheta_4*l_2*m_4*r_4*np.sin(theta_4 - theta_2)) + dtheta_3*mu_3 + dtheta_3*(dtheta_3*l_2*l_3*m_4*np.sin(theta_3 - theta_2) + dtheta_3*l_2*l_3*m_5*np.sin(theta_3 - theta_2) + dtheta_3*l_2*m_3*r_3*np.sin(theta_3 - theta_2)) - dtheta_2*(mu_2 + mu_3) + dtheta_1*mu_2 + dtheta_1*(-dtheta_1*l_1*l_2*m_3*np.sin(theta_2 - theta_1) - dtheta_1*l_1*l_2*m_4*np.sin(theta_2 - theta_1) - dtheta_1*l_1*l_2*m_5*np.sin(theta_2 - theta_1) - dtheta_1*l_1*m_2*r_2*np.sin(theta_2 - theta_1)) - ddx_c*(-l_2*m_3 - l_2*m_4 - l_2*m_5 - m_2*r_2)*np.cos(theta_2) - g*(-l_2*m_3 - l_2*m_4 - l_2*m_5 - m_2*r_2)*np.sin(theta_2) - l_1*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*(dtheta_5**2*l_1*m_5*r_5*np.sin(theta_5 - theta_1) + dtheta_4*(dtheta_4*l_1*l_4*m_5*np.sin(theta_4 - theta_1) + dtheta_4*l_1*m_4*r_4*np.sin(theta_4 - theta_1)) + dtheta_3*(dtheta_3*l_1*l_3*m_4*np.sin(theta_3 - theta_1) + dtheta_3*l_1*l_3*m_5*np.sin(theta_3 - theta_1) + dtheta_3*l_1*m_3*r_3*np.sin(theta_3 - theta_1)) + dtheta_2*mu_2 + dtheta_2*(dtheta_2*l_1*l_2*m_3*np.sin(theta_2 - theta_1) + dtheta_2*l_1*l_2*m_4*np.sin(theta_2 - theta_1) + dtheta_2*l_1*l_2*m_5*np.sin(theta_2 - theta_1) + dtheta_2*l_1*m_2*r_2*np.sin(theta_2 - theta_1)) - dtheta_1*(mu_1 + mu_2) - ddx_c*(-l_1*m_2 - l_1*m_3 - l_1*m_4 - l_1*m_5 - m_1*r_1)*np.cos(theta_1) - g*(-l_1*m_2 - l_1*m_3 - l_1*m_4 - l_1*m_5 - m_1*r_1)*np.sin(theta_1))*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2))/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2) - (-l_1**2*(l_4*m_5 + m_4*r_4)*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_4 - theta_1)*np.cos(theta_3 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_3*(l_4*m_5 + m_4*r_4)*np.cos(theta_4 - theta_3) - (-l_1**2*(l_4*m_5 + m_4*r_4)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_4 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_4*m_5 + m_4*r_4)*np.cos(theta_4 - theta_2))*(-l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_3 - theta_2))/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2))*(dtheta_5**2*l_3*m_5*r_5*np.sin(theta_5 - theta_3) + dtheta_4*mu_4 + dtheta_4*(dtheta_4*l_3*l_4*m_5*np.sin(theta_4 - theta_3) + dtheta_4*l_3*m_4*r_4*np.sin(theta_4 - theta_3)) - dtheta_3*(mu_3 + mu_4) + dtheta_2*mu_3 + dtheta_2*(-dtheta_2*l_2*l_3*m_4*np.sin(theta_3 - theta_2) - dtheta_2*l_2*l_3*m_5*np.sin(theta_3 - theta_2) - dtheta_2*l_2*m_3*r_3*np.sin(theta_3 - theta_2)) + dtheta_1*(-dtheta_1*l_1*l_3*m_4*np.sin(theta_3 - theta_1) - dtheta_1*l_1*l_3*m_5*np.sin(theta_3 - theta_1) - dtheta_1*l_1*m_3*r_3*np.sin(theta_3 - theta_1)) - ddx_c*(-l_3*m_4 - l_3*m_5 - m_3*r_3)*np.cos(theta_3) - g*(-l_3*m_4 - l_3*m_5 - m_3*r_3)*np.sin(theta_3) - l_1*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(dtheta_5**2*l_1*m_5*r_5*np.sin(theta_5 - theta_1) + dtheta_4*(dtheta_4*l_1*l_4*m_5*np.sin(theta_4 - theta_1) + dtheta_4*l_1*m_4*r_4*np.sin(theta_4 - theta_1)) + dtheta_3*(dtheta_3*l_1*l_3*m_4*np.sin(theta_3 - theta_1) + dtheta_3*l_1*l_3*m_5*np.sin(theta_3 - theta_1) + dtheta_3*l_1*m_3*r_3*np.sin(theta_3 - theta_1)) + dtheta_2*mu_2 + dtheta_2*(dtheta_2*l_1*l_2*m_3*np.sin(theta_2 - theta_1) + dtheta_2*l_1*l_2*m_4*np.sin(theta_2 - theta_1) + dtheta_2*l_1*l_2*m_5*np.sin(theta_2 - theta_1) + dtheta_2*l_1*m_2*r_2*np.sin(theta_2 - theta_1)) - dtheta_1*(mu_1 + mu_2) - ddx_c*(-l_1*m_2 - l_1*m_3 - l_1*m_4 - l_1*m_5 - m_1*r_1)*np.cos(theta_1) - g*(-l_1*m_2 - l_1*m_3 - l_1*m_4 - l_1*m_5 - m_1*r_1)*np.sin(theta_1))*np.cos(theta_3 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) - (-l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_3 - theta_2))*(dtheta_5**2*l_2*m_5*r_5*np.sin(theta_5 - theta_2) + dtheta_4*(dtheta_4*l_2*l_4*m_5*np.sin(theta_4 - theta_2) + dtheta_4*l_2*m_4*r_4*np.sin(theta_4 - theta_2)) + dtheta_3*mu_3 + dtheta_3*(dtheta_3*l_2*l_3*m_4*np.sin(theta_3 - theta_2) + dtheta_3*l_2*l_3*m_5*np.sin(theta_3 - theta_2) + dtheta_3*l_2*m_3*r_3*np.sin(theta_3 - theta_2)) - dtheta_2*(mu_2 + mu_3) + dtheta_1*mu_2 + dtheta_1*(-dtheta_1*l_1*l_2*m_3*np.sin(theta_2 - theta_1) - dtheta_1*l_1*l_2*m_4*np.sin(theta_2 - theta_1) - dtheta_1*l_1*l_2*m_5*np.sin(theta_2 - theta_1) - dtheta_1*l_1*m_2*r_2*np.sin(theta_2 - theta_1)) - ddx_c*(-l_2*m_3 - l_2*m_4 - l_2*m_5 - m_2*r_2)*np.cos(theta_2) - g*(-l_2*m_3 - l_2*m_4 - l_2*m_5 - m_2*r_2)*np.sin(theta_2) - l_1*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*(dtheta_5**2*l_1*m_5*r_5*np.sin(theta_5 - theta_1) + dtheta_4*(dtheta_4*l_1*l_4*m_5*np.sin(theta_4 - theta_1) + dtheta_4*l_1*m_4*r_4*np.sin(theta_4 - theta_1)) + dtheta_3*(dtheta_3*l_1*l_3*m_4*np.sin(theta_3 - theta_1) + dtheta_3*l_1*l_3*m_5*np.sin(theta_3 - theta_1) + dtheta_3*l_1*m_3*r_3*np.sin(theta_3 - theta_1)) + dtheta_2*mu_2 + dtheta_2*(dtheta_2*l_1*l_2*m_3*np.sin(theta_2 - theta_1) + dtheta_2*l_1*l_2*m_4*np.sin(theta_2 - theta_1) + dtheta_2*l_1*l_2*m_5*np.sin(theta_2 - theta_1) + dtheta_2*l_1*m_2*r_2*np.sin(theta_2 - theta_1)) - dtheta_1*(mu_1 + mu_2) - ddx_c*(-l_1*m_2 - l_1*m_3 - l_1*m_4 - l_1*m_5 - m_1*r_1)*np.cos(theta_1) - g*(-l_1*m_2 - l_1*m_3 - l_1*m_4 - l_1*m_5 - m_1*r_1)*np.sin(theta_1))*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2))/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2))/(I_3 - l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)**2*np.cos(theta_3 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_3**2*m_4 + l_3**2*m_5 + m_3*r_3**2 - (-l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_3 - theta_2))**2/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2)) - (-l_1**2*m_5*r_5*(l_4*m_5 + m_4*r_4)*np.cos(theta_5 - theta_1)*np.cos(theta_4 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_4*m_5*r_5*np.cos(theta_5 - theta_4) - (-l_1**2*(l_4*m_5 + m_4*r_4)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_4 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_4*m_5 + m_4*r_4)*np.cos(theta_4 - theta_2))*(-l_1**2*m_5*r_5*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_5 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*m_5*r_5*np.cos(theta_5 - theta_2))/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2) - (-l_1**2*(l_4*m_5 + m_4*r_4)*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_4 - theta_1)*np.cos(theta_3 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_3*(l_4*m_5 + m_4*r_4)*np.cos(theta_4 - theta_3) - (-l_1**2*(l_4*m_5 + m_4*r_4)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_4 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_4*m_5 + m_4*r_4)*np.cos(theta_4 - theta_2))*(-l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_3 - theta_2))/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2))*(-l_1**2*m_5*r_5*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_5 - theta_1)*np.cos(theta_3 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_3*m_5*r_5*np.cos(theta_5 - theta_3) - (-l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_3 - theta_2))*(-l_1**2*m_5*r_5*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_5 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*m_5*r_5*np.cos(theta_5 - theta_2))/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2))/(I_3 - l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)**2*np.cos(theta_3 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_3**2*m_4 + l_3**2*m_5 + m_3*r_3**2 - (-l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_3 - theta_2))**2/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2)))*(-dtheta_5*mu_5 - dtheta_4**2*l_4*m_5*r_5*np.sin(theta_5 - theta_4) + dtheta_4*mu_5 - dtheta_3**2*l_3*m_5*r_5*np.sin(theta_5 - theta_3) - dtheta_2**2*l_2*m_5*r_5*np.sin(theta_5 - theta_2) - dtheta_1**2*l_1*m_5*r_5*np.sin(theta_5 - theta_1) + ddx_c*m_5*r_5*np.cos(theta_5) + g*m_5*r_5*np.sin(theta_5) - l_1*m_5*r_5*(dtheta_5**2*l_1*m_5*r_5*np.sin(theta_5 - theta_1) + dtheta_4*(dtheta_4*l_1*l_4*m_5*np.sin(theta_4 - theta_1) + dtheta_4*l_1*m_4*r_4*np.sin(theta_4 - theta_1)) + dtheta_3*(dtheta_3*l_1*l_3*m_4*np.sin(theta_3 - theta_1) + dtheta_3*l_1*l_3*m_5*np.sin(theta_3 - theta_1) + dtheta_3*l_1*m_3*r_3*np.sin(theta_3 - theta_1)) + dtheta_2*mu_2 + dtheta_2*(dtheta_2*l_1*l_2*m_3*np.sin(theta_2 - theta_1) + dtheta_2*l_1*l_2*m_4*np.sin(theta_2 - theta_1) + dtheta_2*l_1*l_2*m_5*np.sin(theta_2 - theta_1) + dtheta_2*l_1*m_2*r_2*np.sin(theta_2 - theta_1)) - dtheta_1*(mu_1 + mu_2) - ddx_c*(-l_1*m_2 - l_1*m_3 - l_1*m_4 - l_1*m_5 - m_1*r_1)*np.cos(theta_1) - g*(-l_1*m_2 - l_1*m_3 - l_1*m_4 - l_1*m_5 - m_1*r_1)*np.sin(theta_1))*np.cos(theta_5 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) - (-l_1**2*m_5*r_5*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_5 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*m_5*r_5*np.cos(theta_5 - theta_2))*(dtheta_5**2*l_2*m_5*r_5*np.sin(theta_5 - theta_2) + dtheta_4*(dtheta_4*l_2*l_4*m_5*np.sin(theta_4 - theta_2) + dtheta_4*l_2*m_4*r_4*np.sin(theta_4 - theta_2)) + dtheta_3*mu_3 + dtheta_3*(dtheta_3*l_2*l_3*m_4*np.sin(theta_3 - theta_2) + dtheta_3*l_2*l_3*m_5*np.sin(theta_3 - theta_2) + dtheta_3*l_2*m_3*r_3*np.sin(theta_3 - theta_2)) - dtheta_2*(mu_2 + mu_3) + dtheta_1*mu_2 + dtheta_1*(-dtheta_1*l_1*l_2*m_3*np.sin(theta_2 - theta_1) - dtheta_1*l_1*l_2*m_4*np.sin(theta_2 - theta_1) - dtheta_1*l_1*l_2*m_5*np.sin(theta_2 - theta_1) - dtheta_1*l_1*m_2*r_2*np.sin(theta_2 - theta_1)) - ddx_c*(-l_2*m_3 - l_2*m_4 - l_2*m_5 - m_2*r_2)*np.cos(theta_2) - g*(-l_2*m_3 - l_2*m_4 - l_2*m_5 - m_2*r_2)*np.sin(theta_2) - l_1*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*(dtheta_5**2*l_1*m_5*r_5*np.sin(theta_5 - theta_1) + dtheta_4*(dtheta_4*l_1*l_4*m_5*np.sin(theta_4 - theta_1) + dtheta_4*l_1*m_4*r_4*np.sin(theta_4 - theta_1)) + dtheta_3*(dtheta_3*l_1*l_3*m_4*np.sin(theta_3 - theta_1) + dtheta_3*l_1*l_3*m_5*np.sin(theta_3 - theta_1) + dtheta_3*l_1*m_3*r_3*np.sin(theta_3 - theta_1)) + dtheta_2*mu_2 + dtheta_2*(dtheta_2*l_1*l_2*m_3*np.sin(theta_2 - theta_1) + dtheta_2*l_1*l_2*m_4*np.sin(theta_2 - theta_1) + dtheta_2*l_1*l_2*m_5*np.sin(theta_2 - theta_1) + dtheta_2*l_1*m_2*r_2*np.sin(theta_2 - theta_1)) - dtheta_1*(mu_1 + mu_2) - ddx_c*(-l_1*m_2 - l_1*m_3 - l_1*m_4 - l_1*m_5 - m_1*r_1)*np.cos(theta_1) - g*(-l_1*m_2 - l_1*m_3 - l_1*m_4 - l_1*m_5 - m_1*r_1)*np.sin(theta_1))*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2))/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2) - (-l_1**2*m_5*r_5*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_5 - theta_1)*np.cos(theta_3 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_3*m_5*r_5*np.cos(theta_5 - theta_3) - (-l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_3 - theta_2))*(-l_1**2*m_5*r_5*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_5 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*m_5*r_5*np.cos(theta_5 - theta_2))/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2))*(dtheta_5**2*l_3*m_5*r_5*np.sin(theta_5 - theta_3) + dtheta_4*mu_4 + dtheta_4*(dtheta_4*l_3*l_4*m_5*np.sin(theta_4 - theta_3) + dtheta_4*l_3*m_4*r_4*np.sin(theta_4 - theta_3)) - dtheta_3*(mu_3 + mu_4) + dtheta_2*mu_3 + dtheta_2*(-dtheta_2*l_2*l_3*m_4*np.sin(theta_3 - theta_2) - dtheta_2*l_2*l_3*m_5*np.sin(theta_3 - theta_2) - dtheta_2*l_2*m_3*r_3*np.sin(theta_3 - theta_2)) + dtheta_1*(-dtheta_1*l_1*l_3*m_4*np.sin(theta_3 - theta_1) - dtheta_1*l_1*l_3*m_5*np.sin(theta_3 - theta_1) - dtheta_1*l_1*m_3*r_3*np.sin(theta_3 - theta_1)) - ddx_c*(-l_3*m_4 - l_3*m_5 - m_3*r_3)*np.cos(theta_3) - g*(-l_3*m_4 - l_3*m_5 - m_3*r_3)*np.sin(theta_3) - l_1*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(dtheta_5**2*l_1*m_5*r_5*np.sin(theta_5 - theta_1) + dtheta_4*(dtheta_4*l_1*l_4*m_5*np.sin(theta_4 - theta_1) + dtheta_4*l_1*m_4*r_4*np.sin(theta_4 - theta_1)) + dtheta_3*(dtheta_3*l_1*l_3*m_4*np.sin(theta_3 - theta_1) + dtheta_3*l_1*l_3*m_5*np.sin(theta_3 - theta_1) + dtheta_3*l_1*m_3*r_3*np.sin(theta_3 - theta_1)) + dtheta_2*mu_2 + dtheta_2*(dtheta_2*l_1*l_2*m_3*np.sin(theta_2 - theta_1) + dtheta_2*l_1*l_2*m_4*np.sin(theta_2 - theta_1) + dtheta_2*l_1*l_2*m_5*np.sin(theta_2 - theta_1) + dtheta_2*l_1*m_2*r_2*np.sin(theta_2 - theta_1)) - dtheta_1*(mu_1 + mu_2) - ddx_c*(-l_1*m_2 - l_1*m_3 - l_1*m_4 - l_1*m_5 - m_1*r_1)*np.cos(theta_1) - g*(-l_1*m_2 - l_1*m_3 - l_1*m_4 - l_1*m_5 - m_1*r_1)*np.sin(theta_1))*np.cos(theta_3 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) - (-l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_3 - theta_2))*(dtheta_5**2*l_2*m_5*r_5*np.sin(theta_5 - theta_2) + dtheta_4*(dtheta_4*l_2*l_4*m_5*np.sin(theta_4 - theta_2) + dtheta_4*l_2*m_4*r_4*np.sin(theta_4 - theta_2)) + dtheta_3*mu_3 + dtheta_3*(dtheta_3*l_2*l_3*m_4*np.sin(theta_3 - theta_2) + dtheta_3*l_2*l_3*m_5*np.sin(theta_3 - theta_2) + dtheta_3*l_2*m_3*r_3*np.sin(theta_3 - theta_2)) - dtheta_2*(mu_2 + mu_3) + dtheta_1*mu_2 + dtheta_1*(-dtheta_1*l_1*l_2*m_3*np.sin(theta_2 - theta_1) - dtheta_1*l_1*l_2*m_4*np.sin(theta_2 - theta_1) - dtheta_1*l_1*l_2*m_5*np.sin(theta_2 - theta_1) - dtheta_1*l_1*m_2*r_2*np.sin(theta_2 - theta_1)) - ddx_c*(-l_2*m_3 - l_2*m_4 - l_2*m_5 - m_2*r_2)*np.cos(theta_2) - g*(-l_2*m_3 - l_2*m_4 - l_2*m_5 - m_2*r_2)*np.sin(theta_2) - l_1*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*(dtheta_5**2*l_1*m_5*r_5*np.sin(theta_5 - theta_1) + dtheta_4*(dtheta_4*l_1*l_4*m_5*np.sin(theta_4 - theta_1) + dtheta_4*l_1*m_4*r_4*np.sin(theta_4 - theta_1)) + dtheta_3*(dtheta_3*l_1*l_3*m_4*np.sin(theta_3 - theta_1) + dtheta_3*l_1*l_3*m_5*np.sin(theta_3 - theta_1) + dtheta_3*l_1*m_3*r_3*np.sin(theta_3 - theta_1)) + dtheta_2*mu_2 + dtheta_2*(dtheta_2*l_1*l_2*m_3*np.sin(theta_2 - theta_1) + dtheta_2*l_1*l_2*m_4*np.sin(theta_2 - theta_1) + dtheta_2*l_1*l_2*m_5*np.sin(theta_2 - theta_1) + dtheta_2*l_1*m_2*r_2*np.sin(theta_2 - theta_1)) - dtheta_1*(mu_1 + mu_2) - ddx_c*(-l_1*m_2 - l_1*m_3 - l_1*m_4 - l_1*m_5 - m_1*r_1)*np.cos(theta_1) - g*(-l_1*m_2 - l_1*m_3 - l_1*m_4 - l_1*m_5 - m_1*r_1)*np.sin(theta_1))*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2))/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2))/(I_3 - l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)**2*np.cos(theta_3 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_3**2*m_4 + l_3**2*m_5 + m_3*r_3**2 - (-l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_3 - theta_2))**2/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2)) - (-l_1**2*m_5*r_5*(l_4*m_5 + m_4*r_4)*np.cos(theta_5 - theta_1)*np.cos(theta_4 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_4*m_5*r_5*np.cos(theta_5 - theta_4) - (-l_1**2*(l_4*m_5 + m_4*r_4)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_4 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_4*m_5 + m_4*r_4)*np.cos(theta_4 - theta_2))*(-l_1**2*m_5*r_5*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_5 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*m_5*r_5*np.cos(theta_5 - theta_2))/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2) - (-l_1**2*(l_4*m_5 + m_4*r_4)*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_4 - theta_1)*np.cos(theta_3 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_3*(l_4*m_5 + m_4*r_4)*np.cos(theta_4 - theta_3) - (-l_1**2*(l_4*m_5 + m_4*r_4)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_4 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_4*m_5 + m_4*r_4)*np.cos(theta_4 - theta_2))*(-l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_3 - theta_2))/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2))*(-l_1**2*m_5*r_5*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_5 - theta_1)*np.cos(theta_3 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_3*m_5*r_5*np.cos(theta_5 - theta_3) - (-l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_3 - theta_2))*(-l_1**2*m_5*r_5*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_5 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*m_5*r_5*np.cos(theta_5 - theta_2))/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2))/(I_3 - l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)**2*np.cos(theta_3 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_3**2*m_4 + l_3**2*m_5 + m_3*r_3**2 - (-l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_3 - theta_2))**2/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2)))*(dtheta_5**2*l_4*m_5*r_5*np.sin(theta_5 - theta_4) + dtheta_5*mu_5 - dtheta_4*(mu_4 + mu_5) + dtheta_3*mu_4 + dtheta_3*(-dtheta_3*l_3*l_4*m_5*np.sin(theta_4 - theta_3) - dtheta_3*l_3*m_4*r_4*np.sin(theta_4 - theta_3)) + dtheta_2*(-dtheta_2*l_2*l_4*m_5*np.sin(theta_4 - theta_2) - dtheta_2*l_2*m_4*r_4*np.sin(theta_4 - theta_2)) + dtheta_1*(-dtheta_1*l_1*l_4*m_5*np.sin(theta_4 - theta_1) - dtheta_1*l_1*m_4*r_4*np.sin(theta_4 - theta_1)) - ddx_c*(-l_4*m_5 - m_4*r_4)*np.cos(theta_4) - g*(-l_4*m_5 - m_4*r_4)*np.sin(theta_4) - l_1*(l_4*m_5 + m_4*r_4)*(dtheta_5**2*l_1*m_5*r_5*np.sin(theta_5 - theta_1) + dtheta_4*(dtheta_4*l_1*l_4*m_5*np.sin(theta_4 - theta_1) + dtheta_4*l_1*m_4*r_4*np.sin(theta_4 - theta_1)) + dtheta_3*(dtheta_3*l_1*l_3*m_4*np.sin(theta_3 - theta_1) + dtheta_3*l_1*l_3*m_5*np.sin(theta_3 - theta_1) + dtheta_3*l_1*m_3*r_3*np.sin(theta_3 - theta_1)) + dtheta_2*mu_2 + dtheta_2*(dtheta_2*l_1*l_2*m_3*np.sin(theta_2 - theta_1) + dtheta_2*l_1*l_2*m_4*np.sin(theta_2 - theta_1) + dtheta_2*l_1*l_2*m_5*np.sin(theta_2 - theta_1) + dtheta_2*l_1*m_2*r_2*np.sin(theta_2 - theta_1)) - dtheta_1*(mu_1 + mu_2) - ddx_c*(-l_1*m_2 - l_1*m_3 - l_1*m_4 - l_1*m_5 - m_1*r_1)*np.cos(theta_1) - g*(-l_1*m_2 - l_1*m_3 - l_1*m_4 - l_1*m_5 - m_1*r_1)*np.sin(theta_1))*np.cos(theta_4 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) - (-l_1**2*(l_4*m_5 + m_4*r_4)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_4 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_4*m_5 + m_4*r_4)*np.cos(theta_4 - theta_2))*(dtheta_5**2*l_2*m_5*r_5*np.sin(theta_5 - theta_2) + dtheta_4*(dtheta_4*l_2*l_4*m_5*np.sin(theta_4 - theta_2) + dtheta_4*l_2*m_4*r_4*np.sin(theta_4 - theta_2)) + dtheta_3*mu_3 + dtheta_3*(dtheta_3*l_2*l_3*m_4*np.sin(theta_3 - theta_2) + dtheta_3*l_2*l_3*m_5*np.sin(theta_3 - theta_2) + dtheta_3*l_2*m_3*r_3*np.sin(theta_3 - theta_2)) - dtheta_2*(mu_2 + mu_3) + dtheta_1*mu_2 + dtheta_1*(-dtheta_1*l_1*l_2*m_3*np.sin(theta_2 - theta_1) - dtheta_1*l_1*l_2*m_4*np.sin(theta_2 - theta_1) - dtheta_1*l_1*l_2*m_5*np.sin(theta_2 - theta_1) - dtheta_1*l_1*m_2*r_2*np.sin(theta_2 - theta_1)) - ddx_c*(-l_2*m_3 - l_2*m_4 - l_2*m_5 - m_2*r_2)*np.cos(theta_2) - g*(-l_2*m_3 - l_2*m_4 - l_2*m_5 - m_2*r_2)*np.sin(theta_2) - l_1*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*(dtheta_5**2*l_1*m_5*r_5*np.sin(theta_5 - theta_1) + dtheta_4*(dtheta_4*l_1*l_4*m_5*np.sin(theta_4 - theta_1) + dtheta_4*l_1*m_4*r_4*np.sin(theta_4 - theta_1)) + dtheta_3*(dtheta_3*l_1*l_3*m_4*np.sin(theta_3 - theta_1) + dtheta_3*l_1*l_3*m_5*np.sin(theta_3 - theta_1) + dtheta_3*l_1*m_3*r_3*np.sin(theta_3 - theta_1)) + dtheta_2*mu_2 + dtheta_2*(dtheta_2*l_1*l_2*m_3*np.sin(theta_2 - theta_1) + dtheta_2*l_1*l_2*m_4*np.sin(theta_2 - theta_1) + dtheta_2*l_1*l_2*m_5*np.sin(theta_2 - theta_1) + dtheta_2*l_1*m_2*r_2*np.sin(theta_2 - theta_1)) - dtheta_1*(mu_1 + mu_2) - ddx_c*(-l_1*m_2 - l_1*m_3 - l_1*m_4 - l_1*m_5 - m_1*r_1)*np.cos(theta_1) - g*(-l_1*m_2 - l_1*m_3 - l_1*m_4 - l_1*m_5 - m_1*r_1)*np.sin(theta_1))*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2))/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2) - (-l_1**2*(l_4*m_5 + m_4*r_4)*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_4 - theta_1)*np.cos(theta_3 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_3*(l_4*m_5 + m_4*r_4)*np.cos(theta_4 - theta_3) - (-l_1**2*(l_4*m_5 + m_4*r_4)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_4 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_4*m_5 + m_4*r_4)*np.cos(theta_4 - theta_2))*(-l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_3 - theta_2))/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2))*(dtheta_5**2*l_3*m_5*r_5*np.sin(theta_5 - theta_3) + dtheta_4*mu_4 + dtheta_4*(dtheta_4*l_3*l_4*m_5*np.sin(theta_4 - theta_3) + dtheta_4*l_3*m_4*r_4*np.sin(theta_4 - theta_3)) - dtheta_3*(mu_3 + mu_4) + dtheta_2*mu_3 + dtheta_2*(-dtheta_2*l_2*l_3*m_4*np.sin(theta_3 - theta_2) - dtheta_2*l_2*l_3*m_5*np.sin(theta_3 - theta_2) - dtheta_2*l_2*m_3*r_3*np.sin(theta_3 - theta_2)) + dtheta_1*(-dtheta_1*l_1*l_3*m_4*np.sin(theta_3 - theta_1) - dtheta_1*l_1*l_3*m_5*np.sin(theta_3 - theta_1) - dtheta_1*l_1*m_3*r_3*np.sin(theta_3 - theta_1)) - ddx_c*(-l_3*m_4 - l_3*m_5 - m_3*r_3)*np.cos(theta_3) - g*(-l_3*m_4 - l_3*m_5 - m_3*r_3)*np.sin(theta_3) - l_1*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(dtheta_5**2*l_1*m_5*r_5*np.sin(theta_5 - theta_1) + dtheta_4*(dtheta_4*l_1*l_4*m_5*np.sin(theta_4 - theta_1) + dtheta_4*l_1*m_4*r_4*np.sin(theta_4 - theta_1)) + dtheta_3*(dtheta_3*l_1*l_3*m_4*np.sin(theta_3 - theta_1) + dtheta_3*l_1*l_3*m_5*np.sin(theta_3 - theta_1) + dtheta_3*l_1*m_3*r_3*np.sin(theta_3 - theta_1)) + dtheta_2*mu_2 + dtheta_2*(dtheta_2*l_1*l_2*m_3*np.sin(theta_2 - theta_1) + dtheta_2*l_1*l_2*m_4*np.sin(theta_2 - theta_1) + dtheta_2*l_1*l_2*m_5*np.sin(theta_2 - theta_1) + dtheta_2*l_1*m_2*r_2*np.sin(theta_2 - theta_1)) - dtheta_1*(mu_1 + mu_2) - ddx_c*(-l_1*m_2 - l_1*m_3 - l_1*m_4 - l_1*m_5 - m_1*r_1)*np.cos(theta_1) - g*(-l_1*m_2 - l_1*m_3 - l_1*m_4 - l_1*m_5 - m_1*r_1)*np.sin(theta_1))*np.cos(theta_3 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) - (-l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_3 - theta_2))*(dtheta_5**2*l_2*m_5*r_5*np.sin(theta_5 - theta_2) + dtheta_4*(dtheta_4*l_2*l_4*m_5*np.sin(theta_4 - theta_2) + dtheta_4*l_2*m_4*r_4*np.sin(theta_4 - theta_2)) + dtheta_3*mu_3 + dtheta_3*(dtheta_3*l_2*l_3*m_4*np.sin(theta_3 - theta_2) + dtheta_3*l_2*l_3*m_5*np.sin(theta_3 - theta_2) + dtheta_3*l_2*m_3*r_3*np.sin(theta_3 - theta_2)) - dtheta_2*(mu_2 + mu_3) + dtheta_1*mu_2 + dtheta_1*(-dtheta_1*l_1*l_2*m_3*np.sin(theta_2 - theta_1) - dtheta_1*l_1*l_2*m_4*np.sin(theta_2 - theta_1) - dtheta_1*l_1*l_2*m_5*np.sin(theta_2 - theta_1) - dtheta_1*l_1*m_2*r_2*np.sin(theta_2 - theta_1)) - ddx_c*(-l_2*m_3 - l_2*m_4 - l_2*m_5 - m_2*r_2)*np.cos(theta_2) - g*(-l_2*m_3 - l_2*m_4 - l_2*m_5 - m_2*r_2)*np.sin(theta_2) - l_1*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*(dtheta_5**2*l_1*m_5*r_5*np.sin(theta_5 - theta_1) + dtheta_4*(dtheta_4*l_1*l_4*m_5*np.sin(theta_4 - theta_1) + dtheta_4*l_1*m_4*r_4*np.sin(theta_4 - theta_1)) + dtheta_3*(dtheta_3*l_1*l_3*m_4*np.sin(theta_3 - theta_1) + dtheta_3*l_1*l_3*m_5*np.sin(theta_3 - theta_1) + dtheta_3*l_1*m_3*r_3*np.sin(theta_3 - theta_1)) + dtheta_2*mu_2 + dtheta_2*(dtheta_2*l_1*l_2*m_3*np.sin(theta_2 - theta_1) + dtheta_2*l_1*l_2*m_4*np.sin(theta_2 - theta_1) + dtheta_2*l_1*l_2*m_5*np.sin(theta_2 - theta_1) + dtheta_2*l_1*m_2*r_2*np.sin(theta_2 - theta_1)) - dtheta_1*(mu_1 + mu_2) - ddx_c*(-l_1*m_2 - l_1*m_3 - l_1*m_4 - l_1*m_5 - m_1*r_1)*np.cos(theta_1) - g*(-l_1*m_2 - l_1*m_3 - l_1*m_4 - l_1*m_5 - m_1*r_1)*np.sin(theta_1))*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2))/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2))/(I_3 - l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)**2*np.cos(theta_3 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_3**2*m_4 + l_3**2*m_5 + m_3*r_3**2 - (-l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_3 - theta_2))**2/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2)))/(I_4 - l_1**2*(l_4*m_5 + m_4*r_4)**2*np.cos(theta_4 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_4**2*m_5 + m_4*r_4**2 - (-l_1**2*(l_4*m_5 + m_4*r_4)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_4 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_4*m_5 + m_4*r_4)*np.cos(theta_4 - theta_2))**2/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2) - (-l_1**2*(l_4*m_5 + m_4*r_4)*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_4 - theta_1)*np.cos(theta_3 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_3*(l_4*m_5 + m_4*r_4)*np.cos(theta_4 - theta_3) - (-l_1**2*(l_4*m_5 + m_4*r_4)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_4 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_4*m_5 + m_4*r_4)*np.cos(theta_4 - theta_2))*(-l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_3 - theta_2))/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2))**2/(I_3 - l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)**2*np.cos(theta_3 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_3**2*m_4 + l_3**2*m_5 + m_3*r_3**2 - (-l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_3 - theta_2))**2/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2))))/(I_5 - l_1**2*m_5**2*r_5**2*np.cos(theta_5 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + m_5*r_5**2 - (-l_1**2*m_5*r_5*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_5 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*m_5*r_5*np.cos(theta_5 - theta_2))**2/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2) - (-l_1**2*m_5*r_5*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_5 - theta_1)*np.cos(theta_3 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_3*m_5*r_5*np.cos(theta_5 - theta_3) - (-l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_3 - theta_2))*(-l_1**2*m_5*r_5*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_5 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*m_5*r_5*np.cos(theta_5 - theta_2))/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2))**2/(I_3 - l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)**2*np.cos(theta_3 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_3**2*m_4 + l_3**2*m_5 + m_3*r_3**2 - (-l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_3 - theta_2))**2/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2)) - (-l_1**2*m_5*r_5*(l_4*m_5 + m_4*r_4)*np.cos(theta_5 - theta_1)*np.cos(theta_4 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_4*m_5*r_5*np.cos(theta_5 - theta_4) - (-l_1**2*(l_4*m_5 + m_4*r_4)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_4 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_4*m_5 + m_4*r_4)*np.cos(theta_4 - theta_2))*(-l_1**2*m_5*r_5*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_5 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*m_5*r_5*np.cos(theta_5 - theta_2))/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2) - (-l_1**2*(l_4*m_5 + m_4*r_4)*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_4 - theta_1)*np.cos(theta_3 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_3*(l_4*m_5 + m_4*r_4)*np.cos(theta_4 - theta_3) - (-l_1**2*(l_4*m_5 + m_4*r_4)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_4 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_4*m_5 + m_4*r_4)*np.cos(theta_4 - theta_2))*(-l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_3 - theta_2))/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2))*(-l_1**2*m_5*r_5*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_5 - theta_1)*np.cos(theta_3 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_3*m_5*r_5*np.cos(theta_5 - theta_3) - (-l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_3 - theta_2))*(-l_1**2*m_5*r_5*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_5 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*m_5*r_5*np.cos(theta_5 - theta_2))/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2))/(I_3 - l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)**2*np.cos(theta_3 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_3**2*m_4 + l_3**2*m_5 + m_3*r_3**2 - (-l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_3 - theta_2))**2/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2)))**2/(I_4 - l_1**2*(l_4*m_5 + m_4*r_4)**2*np.cos(theta_4 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_4**2*m_5 + m_4*r_4**2 - (-l_1**2*(l_4*m_5 + m_4*r_4)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_4 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_4*m_5 + m_4*r_4)*np.cos(theta_4 - theta_2))**2/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2) - (-l_1**2*(l_4*m_5 + m_4*r_4)*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_4 - theta_1)*np.cos(theta_3 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_3*(l_4*m_5 + m_4*r_4)*np.cos(theta_4 - theta_3) - (-l_1**2*(l_4*m_5 + m_4*r_4)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_4 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_4*m_5 + m_4*r_4)*np.cos(theta_4 - theta_2))*(-l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_3 - theta_2))/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2))**2/(I_3 - l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)**2*np.cos(theta_3 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_3**2*m_4 + l_3**2*m_5 + m_3*r_3**2 - (-l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_3 - theta_2))**2/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2)))))/(I_4 - l_1**2*(l_4*m_5 + m_4*r_4)**2*np.cos(theta_4 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_4**2*m_5 + m_4*r_4**2 - (-l_1**2*(l_4*m_5 + m_4*r_4)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_4 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_4*m_5 + m_4*r_4)*np.cos(theta_4 - theta_2))**2/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2) - (-l_1**2*(l_4*m_5 + m_4*r_4)*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_4 - theta_1)*np.cos(theta_3 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_3*(l_4*m_5 + m_4*r_4)*np.cos(theta_4 - theta_3) - (-l_1**2*(l_4*m_5 + m_4*r_4)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_4 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_4*m_5 + m_4*r_4)*np.cos(theta_4 - theta_2))*(-l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_3 - theta_2))/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2))**2/(I_3 - l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)**2*np.cos(theta_3 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_3**2*m_4 + l_3**2*m_5 + m_3*r_3**2 - (-l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_3 - theta_2))**2/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2))) - (-l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_3 - theta_2))*(dtheta_5**2*l_3*m_5*r_5*np.sin(theta_5 - theta_3) + dtheta_4*mu_4 + dtheta_4*(dtheta_4*l_3*l_4*m_5*np.sin(theta_4 - theta_3) + dtheta_4*l_3*m_4*r_4*np.sin(theta_4 - theta_3)) - dtheta_3*(mu_3 + mu_4) + dtheta_2*mu_3 + dtheta_2*(-dtheta_2*l_2*l_3*m_4*np.sin(theta_3 - theta_2) - dtheta_2*l_2*l_3*m_5*np.sin(theta_3 - theta_2) - dtheta_2*l_2*m_3*r_3*np.sin(theta_3 - theta_2)) + dtheta_1*(-dtheta_1*l_1*l_3*m_4*np.sin(theta_3 - theta_1) - dtheta_1*l_1*l_3*m_5*np.sin(theta_3 - theta_1) - dtheta_1*l_1*m_3*r_3*np.sin(theta_3 - theta_1)) - ddx_c*(-l_3*m_4 - l_3*m_5 - m_3*r_3)*np.cos(theta_3) - g*(-l_3*m_4 - l_3*m_5 - m_3*r_3)*np.sin(theta_3) - l_1*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(dtheta_5**2*l_1*m_5*r_5*np.sin(theta_5 - theta_1) + dtheta_4*(dtheta_4*l_1*l_4*m_5*np.sin(theta_4 - theta_1) + dtheta_4*l_1*m_4*r_4*np.sin(theta_4 - theta_1)) + dtheta_3*(dtheta_3*l_1*l_3*m_4*np.sin(theta_3 - theta_1) + dtheta_3*l_1*l_3*m_5*np.sin(theta_3 - theta_1) + dtheta_3*l_1*m_3*r_3*np.sin(theta_3 - theta_1)) + dtheta_2*mu_2 + dtheta_2*(dtheta_2*l_1*l_2*m_3*np.sin(theta_2 - theta_1) + dtheta_2*l_1*l_2*m_4*np.sin(theta_2 - theta_1) + dtheta_2*l_1*l_2*m_5*np.sin(theta_2 - theta_1) + dtheta_2*l_1*m_2*r_2*np.sin(theta_2 - theta_1)) - dtheta_1*(mu_1 + mu_2) - ddx_c*(-l_1*m_2 - l_1*m_3 - l_1*m_4 - l_1*m_5 - m_1*r_1)*np.cos(theta_1) - g*(-l_1*m_2 - l_1*m_3 - l_1*m_4 - l_1*m_5 - m_1*r_1)*np.sin(theta_1))*np.cos(theta_3 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) - (-l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_3 - theta_2))*(dtheta_5**2*l_2*m_5*r_5*np.sin(theta_5 - theta_2) + dtheta_4*(dtheta_4*l_2*l_4*m_5*np.sin(theta_4 - theta_2) + dtheta_4*l_2*m_4*r_4*np.sin(theta_4 - theta_2)) + dtheta_3*mu_3 + dtheta_3*(dtheta_3*l_2*l_3*m_4*np.sin(theta_3 - theta_2) + dtheta_3*l_2*l_3*m_5*np.sin(theta_3 - theta_2) + dtheta_3*l_2*m_3*r_3*np.sin(theta_3 - theta_2)) - dtheta_2*(mu_2 + mu_3) + dtheta_1*mu_2 + dtheta_1*(-dtheta_1*l_1*l_2*m_3*np.sin(theta_2 - theta_1) - dtheta_1*l_1*l_2*m_4*np.sin(theta_2 - theta_1) - dtheta_1*l_1*l_2*m_5*np.sin(theta_2 - theta_1) - dtheta_1*l_1*m_2*r_2*np.sin(theta_2 - theta_1)) - ddx_c*(-l_2*m_3 - l_2*m_4 - l_2*m_5 - m_2*r_2)*np.cos(theta_2) - g*(-l_2*m_3 - l_2*m_4 - l_2*m_5 - m_2*r_2)*np.sin(theta_2) - l_1*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*(dtheta_5**2*l_1*m_5*r_5*np.sin(theta_5 - theta_1) + dtheta_4*(dtheta_4*l_1*l_4*m_5*np.sin(theta_4 - theta_1) + dtheta_4*l_1*m_4*r_4*np.sin(theta_4 - theta_1)) + dtheta_3*(dtheta_3*l_1*l_3*m_4*np.sin(theta_3 - theta_1) + dtheta_3*l_1*l_3*m_5*np.sin(theta_3 - theta_1) + dtheta_3*l_1*m_3*r_3*np.sin(theta_3 - theta_1)) + dtheta_2*mu_2 + dtheta_2*(dtheta_2*l_1*l_2*m_3*np.sin(theta_2 - theta_1) + dtheta_2*l_1*l_2*m_4*np.sin(theta_2 - theta_1) + dtheta_2*l_1*l_2*m_5*np.sin(theta_2 - theta_1) + dtheta_2*l_1*m_2*r_2*np.sin(theta_2 - theta_1)) - dtheta_1*(mu_1 + mu_2) - ddx_c*(-l_1*m_2 - l_1*m_3 - l_1*m_4 - l_1*m_5 - m_1*r_1)*np.cos(theta_1) - g*(-l_1*m_2 - l_1*m_3 - l_1*m_4 - l_1*m_5 - m_1*r_1)*np.sin(theta_1))*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2))/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2) - (-l_1**2*(l_4*m_5 + m_4*r_4)*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_4 - theta_1)*np.cos(theta_3 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_3*(l_4*m_5 + m_4*r_4)*np.cos(theta_4 - theta_3) - (-l_1**2*(l_4*m_5 + m_4*r_4)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_4 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_4*m_5 + m_4*r_4)*np.cos(theta_4 - theta_2))*(-l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_3 - theta_2))/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2))*(dtheta_5**2*l_4*m_5*r_5*np.sin(theta_5 - theta_4) + dtheta_5*mu_5 - dtheta_4*(mu_4 + mu_5) + dtheta_3*mu_4 + dtheta_3*(-dtheta_3*l_3*l_4*m_5*np.sin(theta_4 - theta_3) - dtheta_3*l_3*m_4*r_4*np.sin(theta_4 - theta_3)) + dtheta_2*(-dtheta_2*l_2*l_4*m_5*np.sin(theta_4 - theta_2) - dtheta_2*l_2*m_4*r_4*np.sin(theta_4 - theta_2)) + dtheta_1*(-dtheta_1*l_1*l_4*m_5*np.sin(theta_4 - theta_1) - dtheta_1*l_1*m_4*r_4*np.sin(theta_4 - theta_1)) - ddx_c*(-l_4*m_5 - m_4*r_4)*np.cos(theta_4) - g*(-l_4*m_5 - m_4*r_4)*np.sin(theta_4) - l_1*(l_4*m_5 + m_4*r_4)*(dtheta_5**2*l_1*m_5*r_5*np.sin(theta_5 - theta_1) + dtheta_4*(dtheta_4*l_1*l_4*m_5*np.sin(theta_4 - theta_1) + dtheta_4*l_1*m_4*r_4*np.sin(theta_4 - theta_1)) + dtheta_3*(dtheta_3*l_1*l_3*m_4*np.sin(theta_3 - theta_1) + dtheta_3*l_1*l_3*m_5*np.sin(theta_3 - theta_1) + dtheta_3*l_1*m_3*r_3*np.sin(theta_3 - theta_1)) + dtheta_2*mu_2 + dtheta_2*(dtheta_2*l_1*l_2*m_3*np.sin(theta_2 - theta_1) + dtheta_2*l_1*l_2*m_4*np.sin(theta_2 - theta_1) + dtheta_2*l_1*l_2*m_5*np.sin(theta_2 - theta_1) + dtheta_2*l_1*m_2*r_2*np.sin(theta_2 - theta_1)) - dtheta_1*(mu_1 + mu_2) - ddx_c*(-l_1*m_2 - l_1*m_3 - l_1*m_4 - l_1*m_5 - m_1*r_1)*np.cos(theta_1) - g*(-l_1*m_2 - l_1*m_3 - l_1*m_4 - l_1*m_5 - m_1*r_1)*np.sin(theta_1))*np.cos(theta_4 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) - (-l_1**2*(l_4*m_5 + m_4*r_4)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_4 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_4*m_5 + m_4*r_4)*np.cos(theta_4 - theta_2))*(dtheta_5**2*l_2*m_5*r_5*np.sin(theta_5 - theta_2) + dtheta_4*(dtheta_4*l_2*l_4*m_5*np.sin(theta_4 - theta_2) + dtheta_4*l_2*m_4*r_4*np.sin(theta_4 - theta_2)) + dtheta_3*mu_3 + dtheta_3*(dtheta_3*l_2*l_3*m_4*np.sin(theta_3 - theta_2) + dtheta_3*l_2*l_3*m_5*np.sin(theta_3 - theta_2) + dtheta_3*l_2*m_3*r_3*np.sin(theta_3 - theta_2)) - dtheta_2*(mu_2 + mu_3) + dtheta_1*mu_2 + dtheta_1*(-dtheta_1*l_1*l_2*m_3*np.sin(theta_2 - theta_1) - dtheta_1*l_1*l_2*m_4*np.sin(theta_2 - theta_1) - dtheta_1*l_1*l_2*m_5*np.sin(theta_2 - theta_1) - dtheta_1*l_1*m_2*r_2*np.sin(theta_2 - theta_1)) - ddx_c*(-l_2*m_3 - l_2*m_4 - l_2*m_5 - m_2*r_2)*np.cos(theta_2) - g*(-l_2*m_3 - l_2*m_4 - l_2*m_5 - m_2*r_2)*np.sin(theta_2) - l_1*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*(dtheta_5**2*l_1*m_5*r_5*np.sin(theta_5 - theta_1) + dtheta_4*(dtheta_4*l_1*l_4*m_5*np.sin(theta_4 - theta_1) + dtheta_4*l_1*m_4*r_4*np.sin(theta_4 - theta_1)) + dtheta_3*(dtheta_3*l_1*l_3*m_4*np.sin(theta_3 - theta_1) + dtheta_3*l_1*l_3*m_5*np.sin(theta_3 - theta_1) + dtheta_3*l_1*m_3*r_3*np.sin(theta_3 - theta_1)) + dtheta_2*mu_2 + dtheta_2*(dtheta_2*l_1*l_2*m_3*np.sin(theta_2 - theta_1) + dtheta_2*l_1*l_2*m_4*np.sin(theta_2 - theta_1) + dtheta_2*l_1*l_2*m_5*np.sin(theta_2 - theta_1) + dtheta_2*l_1*m_2*r_2*np.sin(theta_2 - theta_1)) - dtheta_1*(mu_1 + mu_2) - ddx_c*(-l_1*m_2 - l_1*m_3 - l_1*m_4 - l_1*m_5 - m_1*r_1)*np.cos(theta_1) - g*(-l_1*m_2 - l_1*m_3 - l_1*m_4 - l_1*m_5 - m_1*r_1)*np.sin(theta_1))*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2))/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2) - (-l_1**2*(l_4*m_5 + m_4*r_4)*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_4 - theta_1)*np.cos(theta_3 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_3*(l_4*m_5 + m_4*r_4)*np.cos(theta_4 - theta_3) - (-l_1**2*(l_4*m_5 + m_4*r_4)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_4 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_4*m_5 + m_4*r_4)*np.cos(theta_4 - theta_2))*(-l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_3 - theta_2))/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2))*(dtheta_5**2*l_3*m_5*r_5*np.sin(theta_5 - theta_3) + dtheta_4*mu_4 + dtheta_4*(dtheta_4*l_3*l_4*m_5*np.sin(theta_4 - theta_3) + dtheta_4*l_3*m_4*r_4*np.sin(theta_4 - theta_3)) - dtheta_3*(mu_3 + mu_4) + dtheta_2*mu_3 + dtheta_2*(-dtheta_2*l_2*l_3*m_4*np.sin(theta_3 - theta_2) - dtheta_2*l_2*l_3*m_5*np.sin(theta_3 - theta_2) - dtheta_2*l_2*m_3*r_3*np.sin(theta_3 - theta_2)) + dtheta_1*(-dtheta_1*l_1*l_3*m_4*np.sin(theta_3 - theta_1) - dtheta_1*l_1*l_3*m_5*np.sin(theta_3 - theta_1) - dtheta_1*l_1*m_3*r_3*np.sin(theta_3 - theta_1)) - ddx_c*(-l_3*m_4 - l_3*m_5 - m_3*r_3)*np.cos(theta_3) - g*(-l_3*m_4 - l_3*m_5 - m_3*r_3)*np.sin(theta_3) - l_1*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(dtheta_5**2*l_1*m_5*r_5*np.sin(theta_5 - theta_1) + dtheta_4*(dtheta_4*l_1*l_4*m_5*np.sin(theta_4 - theta_1) + dtheta_4*l_1*m_4*r_4*np.sin(theta_4 - theta_1)) + dtheta_3*(dtheta_3*l_1*l_3*m_4*np.sin(theta_3 - theta_1) + dtheta_3*l_1*l_3*m_5*np.sin(theta_3 - theta_1) + dtheta_3*l_1*m_3*r_3*np.sin(theta_3 - theta_1)) + dtheta_2*mu_2 + dtheta_2*(dtheta_2*l_1*l_2*m_3*np.sin(theta_2 - theta_1) + dtheta_2*l_1*l_2*m_4*np.sin(theta_2 - theta_1) + dtheta_2*l_1*l_2*m_5*np.sin(theta_2 - theta_1) + dtheta_2*l_1*m_2*r_2*np.sin(theta_2 - theta_1)) - dtheta_1*(mu_1 + mu_2) - ddx_c*(-l_1*m_2 - l_1*m_3 - l_1*m_4 - l_1*m_5 - m_1*r_1)*np.cos(theta_1) - g*(-l_1*m_2 - l_1*m_3 - l_1*m_4 - l_1*m_5 - m_1*r_1)*np.sin(theta_1))*np.cos(theta_3 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) - (-l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_3 - theta_2))*(dtheta_5**2*l_2*m_5*r_5*np.sin(theta_5 - theta_2) + dtheta_4*(dtheta_4*l_2*l_4*m_5*np.sin(theta_4 - theta_2) + dtheta_4*l_2*m_4*r_4*np.sin(theta_4 - theta_2)) + dtheta_3*mu_3 + dtheta_3*(dtheta_3*l_2*l_3*m_4*np.sin(theta_3 - theta_2) + dtheta_3*l_2*l_3*m_5*np.sin(theta_3 - theta_2) + dtheta_3*l_2*m_3*r_3*np.sin(theta_3 - theta_2)) - dtheta_2*(mu_2 + mu_3) + dtheta_1*mu_2 + dtheta_1*(-dtheta_1*l_1*l_2*m_3*np.sin(theta_2 - theta_1) - dtheta_1*l_1*l_2*m_4*np.sin(theta_2 - theta_1) - dtheta_1*l_1*l_2*m_5*np.sin(theta_2 - theta_1) - dtheta_1*l_1*m_2*r_2*np.sin(theta_2 - theta_1)) - ddx_c*(-l_2*m_3 - l_2*m_4 - l_2*m_5 - m_2*r_2)*np.cos(theta_2) - g*(-l_2*m_3 - l_2*m_4 - l_2*m_5 - m_2*r_2)*np.sin(theta_2) - l_1*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*(dtheta_5**2*l_1*m_5*r_5*np.sin(theta_5 - theta_1) + dtheta_4*(dtheta_4*l_1*l_4*m_5*np.sin(theta_4 - theta_1) + dtheta_4*l_1*m_4*r_4*np.sin(theta_4 - theta_1)) + dtheta_3*(dtheta_3*l_1*l_3*m_4*np.sin(theta_3 - theta_1) + dtheta_3*l_1*l_3*m_5*np.sin(theta_3 - theta_1) + dtheta_3*l_1*m_3*r_3*np.sin(theta_3 - theta_1)) + dtheta_2*mu_2 + dtheta_2*(dtheta_2*l_1*l_2*m_3*np.sin(theta_2 - theta_1) + dtheta_2*l_1*l_2*m_4*np.sin(theta_2 - theta_1) + dtheta_2*l_1*l_2*m_5*np.sin(theta_2 - theta_1) + dtheta_2*l_1*m_2*r_2*np.sin(theta_2 - theta_1)) - dtheta_1*(mu_1 + mu_2) - ddx_c*(-l_1*m_2 - l_1*m_3 - l_1*m_4 - l_1*m_5 - m_1*r_1)*np.cos(theta_1) - g*(-l_1*m_2 - l_1*m_3 - l_1*m_4 - l_1*m_5 - m_1*r_1)*np.sin(theta_1))*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2))/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2))/(I_3 - l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)**2*np.cos(theta_3 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_3**2*m_4 + l_3**2*m_5 + m_3*r_3**2 - (-l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_3 - theta_2))**2/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2)) - (-l_1**2*m_5*r_5*(l_4*m_5 + m_4*r_4)*np.cos(theta_5 - theta_1)*np.cos(theta_4 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_4*m_5*r_5*np.cos(theta_5 - theta_4) - (-l_1**2*(l_4*m_5 + m_4*r_4)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_4 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_4*m_5 + m_4*r_4)*np.cos(theta_4 - theta_2))*(-l_1**2*m_5*r_5*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_5 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*m_5*r_5*np.cos(theta_5 - theta_2))/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2) - (-l_1**2*(l_4*m_5 + m_4*r_4)*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_4 - theta_1)*np.cos(theta_3 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_3*(l_4*m_5 + m_4*r_4)*np.cos(theta_4 - theta_3) - (-l_1**2*(l_4*m_5 + m_4*r_4)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_4 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_4*m_5 + m_4*r_4)*np.cos(theta_4 - theta_2))*(-l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_3 - theta_2))/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2))*(-l_1**2*m_5*r_5*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_5 - theta_1)*np.cos(theta_3 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_3*m_5*r_5*np.cos(theta_5 - theta_3) - (-l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_3 - theta_2))*(-l_1**2*m_5*r_5*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_5 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*m_5*r_5*np.cos(theta_5 - theta_2))/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2))/(I_3 - l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)**2*np.cos(theta_3 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_3**2*m_4 + l_3**2*m_5 + m_3*r_3**2 - (-l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_3 - theta_2))**2/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2)))*(-dtheta_5*mu_5 - dtheta_4**2*l_4*m_5*r_5*np.sin(theta_5 - theta_4) + dtheta_4*mu_5 - dtheta_3**2*l_3*m_5*r_5*np.sin(theta_5 - theta_3) - dtheta_2**2*l_2*m_5*r_5*np.sin(theta_5 - theta_2) - dtheta_1**2*l_1*m_5*r_5*np.sin(theta_5 - theta_1) + ddx_c*m_5*r_5*np.cos(theta_5) + g*m_5*r_5*np.sin(theta_5) - l_1*m_5*r_5*(dtheta_5**2*l_1*m_5*r_5*np.sin(theta_5 - theta_1) + dtheta_4*(dtheta_4*l_1*l_4*m_5*np.sin(theta_4 - theta_1) + dtheta_4*l_1*m_4*r_4*np.sin(theta_4 - theta_1)) + dtheta_3*(dtheta_3*l_1*l_3*m_4*np.sin(theta_3 - theta_1) + dtheta_3*l_1*l_3*m_5*np.sin(theta_3 - theta_1) + dtheta_3*l_1*m_3*r_3*np.sin(theta_3 - theta_1)) + dtheta_2*mu_2 + dtheta_2*(dtheta_2*l_1*l_2*m_3*np.sin(theta_2 - theta_1) + dtheta_2*l_1*l_2*m_4*np.sin(theta_2 - theta_1) + dtheta_2*l_1*l_2*m_5*np.sin(theta_2 - theta_1) + dtheta_2*l_1*m_2*r_2*np.sin(theta_2 - theta_1)) - dtheta_1*(mu_1 + mu_2) - ddx_c*(-l_1*m_2 - l_1*m_3 - l_1*m_4 - l_1*m_5 - m_1*r_1)*np.cos(theta_1) - g*(-l_1*m_2 - l_1*m_3 - l_1*m_4 - l_1*m_5 - m_1*r_1)*np.sin(theta_1))*np.cos(theta_5 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) - (-l_1**2*m_5*r_5*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_5 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*m_5*r_5*np.cos(theta_5 - theta_2))*(dtheta_5**2*l_2*m_5*r_5*np.sin(theta_5 - theta_2) + dtheta_4*(dtheta_4*l_2*l_4*m_5*np.sin(theta_4 - theta_2) + dtheta_4*l_2*m_4*r_4*np.sin(theta_4 - theta_2)) + dtheta_3*mu_3 + dtheta_3*(dtheta_3*l_2*l_3*m_4*np.sin(theta_3 - theta_2) + dtheta_3*l_2*l_3*m_5*np.sin(theta_3 - theta_2) + dtheta_3*l_2*m_3*r_3*np.sin(theta_3 - theta_2)) - dtheta_2*(mu_2 + mu_3) + dtheta_1*mu_2 + dtheta_1*(-dtheta_1*l_1*l_2*m_3*np.sin(theta_2 - theta_1) - dtheta_1*l_1*l_2*m_4*np.sin(theta_2 - theta_1) - dtheta_1*l_1*l_2*m_5*np.sin(theta_2 - theta_1) - dtheta_1*l_1*m_2*r_2*np.sin(theta_2 - theta_1)) - ddx_c*(-l_2*m_3 - l_2*m_4 - l_2*m_5 - m_2*r_2)*np.cos(theta_2) - g*(-l_2*m_3 - l_2*m_4 - l_2*m_5 - m_2*r_2)*np.sin(theta_2) - l_1*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*(dtheta_5**2*l_1*m_5*r_5*np.sin(theta_5 - theta_1) + dtheta_4*(dtheta_4*l_1*l_4*m_5*np.sin(theta_4 - theta_1) + dtheta_4*l_1*m_4*r_4*np.sin(theta_4 - theta_1)) + dtheta_3*(dtheta_3*l_1*l_3*m_4*np.sin(theta_3 - theta_1) + dtheta_3*l_1*l_3*m_5*np.sin(theta_3 - theta_1) + dtheta_3*l_1*m_3*r_3*np.sin(theta_3 - theta_1)) + dtheta_2*mu_2 + dtheta_2*(dtheta_2*l_1*l_2*m_3*np.sin(theta_2 - theta_1) + dtheta_2*l_1*l_2*m_4*np.sin(theta_2 - theta_1) + dtheta_2*l_1*l_2*m_5*np.sin(theta_2 - theta_1) + dtheta_2*l_1*m_2*r_2*np.sin(theta_2 - theta_1)) - dtheta_1*(mu_1 + mu_2) - ddx_c*(-l_1*m_2 - l_1*m_3 - l_1*m_4 - l_1*m_5 - m_1*r_1)*np.cos(theta_1) - g*(-l_1*m_2 - l_1*m_3 - l_1*m_4 - l_1*m_5 - m_1*r_1)*np.sin(theta_1))*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2))/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2) - (-l_1**2*m_5*r_5*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_5 - theta_1)*np.cos(theta_3 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_3*m_5*r_5*np.cos(theta_5 - theta_3) - (-l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_3 - theta_2))*(-l_1**2*m_5*r_5*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_5 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*m_5*r_5*np.cos(theta_5 - theta_2))/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2))*(dtheta_5**2*l_3*m_5*r_5*np.sin(theta_5 - theta_3) + dtheta_4*mu_4 + dtheta_4*(dtheta_4*l_3*l_4*m_5*np.sin(theta_4 - theta_3) + dtheta_4*l_3*m_4*r_4*np.sin(theta_4 - theta_3)) - dtheta_3*(mu_3 + mu_4) + dtheta_2*mu_3 + dtheta_2*(-dtheta_2*l_2*l_3*m_4*np.sin(theta_3 - theta_2) - dtheta_2*l_2*l_3*m_5*np.sin(theta_3 - theta_2) - dtheta_2*l_2*m_3*r_3*np.sin(theta_3 - theta_2)) + dtheta_1*(-dtheta_1*l_1*l_3*m_4*np.sin(theta_3 - theta_1) - dtheta_1*l_1*l_3*m_5*np.sin(theta_3 - theta_1) - dtheta_1*l_1*m_3*r_3*np.sin(theta_3 - theta_1)) - ddx_c*(-l_3*m_4 - l_3*m_5 - m_3*r_3)*np.cos(theta_3) - g*(-l_3*m_4 - l_3*m_5 - m_3*r_3)*np.sin(theta_3) - l_1*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(dtheta_5**2*l_1*m_5*r_5*np.sin(theta_5 - theta_1) + dtheta_4*(dtheta_4*l_1*l_4*m_5*np.sin(theta_4 - theta_1) + dtheta_4*l_1*m_4*r_4*np.sin(theta_4 - theta_1)) + dtheta_3*(dtheta_3*l_1*l_3*m_4*np.sin(theta_3 - theta_1) + dtheta_3*l_1*l_3*m_5*np.sin(theta_3 - theta_1) + dtheta_3*l_1*m_3*r_3*np.sin(theta_3 - theta_1)) + dtheta_2*mu_2 + dtheta_2*(dtheta_2*l_1*l_2*m_3*np.sin(theta_2 - theta_1) + dtheta_2*l_1*l_2*m_4*np.sin(theta_2 - theta_1) + dtheta_2*l_1*l_2*m_5*np.sin(theta_2 - theta_1) + dtheta_2*l_1*m_2*r_2*np.sin(theta_2 - theta_1)) - dtheta_1*(mu_1 + mu_2) - ddx_c*(-l_1*m_2 - l_1*m_3 - l_1*m_4 - l_1*m_5 - m_1*r_1)*np.cos(theta_1) - g*(-l_1*m_2 - l_1*m_3 - l_1*m_4 - l_1*m_5 - m_1*r_1)*np.sin(theta_1))*np.cos(theta_3 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) - (-l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_3 - theta_2))*(dtheta_5**2*l_2*m_5*r_5*np.sin(theta_5 - theta_2) + dtheta_4*(dtheta_4*l_2*l_4*m_5*np.sin(theta_4 - theta_2) + dtheta_4*l_2*m_4*r_4*np.sin(theta_4 - theta_2)) + dtheta_3*mu_3 + dtheta_3*(dtheta_3*l_2*l_3*m_4*np.sin(theta_3 - theta_2) + dtheta_3*l_2*l_3*m_5*np.sin(theta_3 - theta_2) + dtheta_3*l_2*m_3*r_3*np.sin(theta_3 - theta_2)) - dtheta_2*(mu_2 + mu_3) + dtheta_1*mu_2 + dtheta_1*(-dtheta_1*l_1*l_2*m_3*np.sin(theta_2 - theta_1) - dtheta_1*l_1*l_2*m_4*np.sin(theta_2 - theta_1) - dtheta_1*l_1*l_2*m_5*np.sin(theta_2 - theta_1) - dtheta_1*l_1*m_2*r_2*np.sin(theta_2 - theta_1)) - ddx_c*(-l_2*m_3 - l_2*m_4 - l_2*m_5 - m_2*r_2)*np.cos(theta_2) - g*(-l_2*m_3 - l_2*m_4 - l_2*m_5 - m_2*r_2)*np.sin(theta_2) - l_1*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*(dtheta_5**2*l_1*m_5*r_5*np.sin(theta_5 - theta_1) + dtheta_4*(dtheta_4*l_1*l_4*m_5*np.sin(theta_4 - theta_1) + dtheta_4*l_1*m_4*r_4*np.sin(theta_4 - theta_1)) + dtheta_3*(dtheta_3*l_1*l_3*m_4*np.sin(theta_3 - theta_1) + dtheta_3*l_1*l_3*m_5*np.sin(theta_3 - theta_1) + dtheta_3*l_1*m_3*r_3*np.sin(theta_3 - theta_1)) + dtheta_2*mu_2 + dtheta_2*(dtheta_2*l_1*l_2*m_3*np.sin(theta_2 - theta_1) + dtheta_2*l_1*l_2*m_4*np.sin(theta_2 - theta_1) + dtheta_2*l_1*l_2*m_5*np.sin(theta_2 - theta_1) + dtheta_2*l_1*m_2*r_2*np.sin(theta_2 - theta_1)) - dtheta_1*(mu_1 + mu_2) - ddx_c*(-l_1*m_2 - l_1*m_3 - l_1*m_4 - l_1*m_5 - m_1*r_1)*np.cos(theta_1) - g*(-l_1*m_2 - l_1*m_3 - l_1*m_4 - l_1*m_5 - m_1*r_1)*np.sin(theta_1))*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2))/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2))/(I_3 - l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)**2*np.cos(theta_3 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_3**2*m_4 + l_3**2*m_5 + m_3*r_3**2 - (-l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_3 - theta_2))**2/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2)) - (-l_1**2*m_5*r_5*(l_4*m_5 + m_4*r_4)*np.cos(theta_5 - theta_1)*np.cos(theta_4 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_4*m_5*r_5*np.cos(theta_5 - theta_4) - (-l_1**2*(l_4*m_5 + m_4*r_4)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_4 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_4*m_5 + m_4*r_4)*np.cos(theta_4 - theta_2))*(-l_1**2*m_5*r_5*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_5 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*m_5*r_5*np.cos(theta_5 - theta_2))/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2) - (-l_1**2*(l_4*m_5 + m_4*r_4)*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_4 - theta_1)*np.cos(theta_3 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_3*(l_4*m_5 + m_4*r_4)*np.cos(theta_4 - theta_3) - (-l_1**2*(l_4*m_5 + m_4*r_4)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_4 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_4*m_5 + m_4*r_4)*np.cos(theta_4 - theta_2))*(-l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_3 - theta_2))/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2))*(-l_1**2*m_5*r_5*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_5 - theta_1)*np.cos(theta_3 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_3*m_5*r_5*np.cos(theta_5 - theta_3) - (-l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_3 - theta_2))*(-l_1**2*m_5*r_5*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_5 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*m_5*r_5*np.cos(theta_5 - theta_2))/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2))/(I_3 - l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)**2*np.cos(theta_3 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_3**2*m_4 + l_3**2*m_5 + m_3*r_3**2 - (-l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_3 - theta_2))**2/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2)))*(dtheta_5**2*l_4*m_5*r_5*np.sin(theta_5 - theta_4) + dtheta_5*mu_5 - dtheta_4*(mu_4 + mu_5) + dtheta_3*mu_4 + dtheta_3*(-dtheta_3*l_3*l_4*m_5*np.sin(theta_4 - theta_3) - dtheta_3*l_3*m_4*r_4*np.sin(theta_4 - theta_3)) + dtheta_2*(-dtheta_2*l_2*l_4*m_5*np.sin(theta_4 - theta_2) - dtheta_2*l_2*m_4*r_4*np.sin(theta_4 - theta_2)) + dtheta_1*(-dtheta_1*l_1*l_4*m_5*np.sin(theta_4 - theta_1) - dtheta_1*l_1*m_4*r_4*np.sin(theta_4 - theta_1)) - ddx_c*(-l_4*m_5 - m_4*r_4)*np.cos(theta_4) - g*(-l_4*m_5 - m_4*r_4)*np.sin(theta_4) - l_1*(l_4*m_5 + m_4*r_4)*(dtheta_5**2*l_1*m_5*r_5*np.sin(theta_5 - theta_1) + dtheta_4*(dtheta_4*l_1*l_4*m_5*np.sin(theta_4 - theta_1) + dtheta_4*l_1*m_4*r_4*np.sin(theta_4 - theta_1)) + dtheta_3*(dtheta_3*l_1*l_3*m_4*np.sin(theta_3 - theta_1) + dtheta_3*l_1*l_3*m_5*np.sin(theta_3 - theta_1) + dtheta_3*l_1*m_3*r_3*np.sin(theta_3 - theta_1)) + dtheta_2*mu_2 + dtheta_2*(dtheta_2*l_1*l_2*m_3*np.sin(theta_2 - theta_1) + dtheta_2*l_1*l_2*m_4*np.sin(theta_2 - theta_1) + dtheta_2*l_1*l_2*m_5*np.sin(theta_2 - theta_1) + dtheta_2*l_1*m_2*r_2*np.sin(theta_2 - theta_1)) - dtheta_1*(mu_1 + mu_2) - ddx_c*(-l_1*m_2 - l_1*m_3 - l_1*m_4 - l_1*m_5 - m_1*r_1)*np.cos(theta_1) - g*(-l_1*m_2 - l_1*m_3 - l_1*m_4 - l_1*m_5 - m_1*r_1)*np.sin(theta_1))*np.cos(theta_4 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) - (-l_1**2*(l_4*m_5 + m_4*r_4)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_4 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_4*m_5 + m_4*r_4)*np.cos(theta_4 - theta_2))*(dtheta_5**2*l_2*m_5*r_5*np.sin(theta_5 - theta_2) + dtheta_4*(dtheta_4*l_2*l_4*m_5*np.sin(theta_4 - theta_2) + dtheta_4*l_2*m_4*r_4*np.sin(theta_4 - theta_2)) + dtheta_3*mu_3 + dtheta_3*(dtheta_3*l_2*l_3*m_4*np.sin(theta_3 - theta_2) + dtheta_3*l_2*l_3*m_5*np.sin(theta_3 - theta_2) + dtheta_3*l_2*m_3*r_3*np.sin(theta_3 - theta_2)) - dtheta_2*(mu_2 + mu_3) + dtheta_1*mu_2 + dtheta_1*(-dtheta_1*l_1*l_2*m_3*np.sin(theta_2 - theta_1) - dtheta_1*l_1*l_2*m_4*np.sin(theta_2 - theta_1) - dtheta_1*l_1*l_2*m_5*np.sin(theta_2 - theta_1) - dtheta_1*l_1*m_2*r_2*np.sin(theta_2 - theta_1)) - ddx_c*(-l_2*m_3 - l_2*m_4 - l_2*m_5 - m_2*r_2)*np.cos(theta_2) - g*(-l_2*m_3 - l_2*m_4 - l_2*m_5 - m_2*r_2)*np.sin(theta_2) - l_1*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*(dtheta_5**2*l_1*m_5*r_5*np.sin(theta_5 - theta_1) + dtheta_4*(dtheta_4*l_1*l_4*m_5*np.sin(theta_4 - theta_1) + dtheta_4*l_1*m_4*r_4*np.sin(theta_4 - theta_1)) + dtheta_3*(dtheta_3*l_1*l_3*m_4*np.sin(theta_3 - theta_1) + dtheta_3*l_1*l_3*m_5*np.sin(theta_3 - theta_1) + dtheta_3*l_1*m_3*r_3*np.sin(theta_3 - theta_1)) + dtheta_2*mu_2 + dtheta_2*(dtheta_2*l_1*l_2*m_3*np.sin(theta_2 - theta_1) + dtheta_2*l_1*l_2*m_4*np.sin(theta_2 - theta_1) + dtheta_2*l_1*l_2*m_5*np.sin(theta_2 - theta_1) + dtheta_2*l_1*m_2*r_2*np.sin(theta_2 - theta_1)) - dtheta_1*(mu_1 + mu_2) - ddx_c*(-l_1*m_2 - l_1*m_3 - l_1*m_4 - l_1*m_5 - m_1*r_1)*np.cos(theta_1) - g*(-l_1*m_2 - l_1*m_3 - l_1*m_4 - l_1*m_5 - m_1*r_1)*np.sin(theta_1))*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2))/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2) - (-l_1**2*(l_4*m_5 + m_4*r_4)*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_4 - theta_1)*np.cos(theta_3 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_3*(l_4*m_5 + m_4*r_4)*np.cos(theta_4 - theta_3) - (-l_1**2*(l_4*m_5 + m_4*r_4)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_4 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_4*m_5 + m_4*r_4)*np.cos(theta_4 - theta_2))*(-l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_3 - theta_2))/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2))*(dtheta_5**2*l_3*m_5*r_5*np.sin(theta_5 - theta_3) + dtheta_4*mu_4 + dtheta_4*(dtheta_4*l_3*l_4*m_5*np.sin(theta_4 - theta_3) + dtheta_4*l_3*m_4*r_4*np.sin(theta_4 - theta_3)) - dtheta_3*(mu_3 + mu_4) + dtheta_2*mu_3 + dtheta_2*(-dtheta_2*l_2*l_3*m_4*np.sin(theta_3 - theta_2) - dtheta_2*l_2*l_3*m_5*np.sin(theta_3 - theta_2) - dtheta_2*l_2*m_3*r_3*np.sin(theta_3 - theta_2)) + dtheta_1*(-dtheta_1*l_1*l_3*m_4*np.sin(theta_3 - theta_1) - dtheta_1*l_1*l_3*m_5*np.sin(theta_3 - theta_1) - dtheta_1*l_1*m_3*r_3*np.sin(theta_3 - theta_1)) - ddx_c*(-l_3*m_4 - l_3*m_5 - m_3*r_3)*np.cos(theta_3) - g*(-l_3*m_4 - l_3*m_5 - m_3*r_3)*np.sin(theta_3) - l_1*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(dtheta_5**2*l_1*m_5*r_5*np.sin(theta_5 - theta_1) + dtheta_4*(dtheta_4*l_1*l_4*m_5*np.sin(theta_4 - theta_1) + dtheta_4*l_1*m_4*r_4*np.sin(theta_4 - theta_1)) + dtheta_3*(dtheta_3*l_1*l_3*m_4*np.sin(theta_3 - theta_1) + dtheta_3*l_1*l_3*m_5*np.sin(theta_3 - theta_1) + dtheta_3*l_1*m_3*r_3*np.sin(theta_3 - theta_1)) + dtheta_2*mu_2 + dtheta_2*(dtheta_2*l_1*l_2*m_3*np.sin(theta_2 - theta_1) + dtheta_2*l_1*l_2*m_4*np.sin(theta_2 - theta_1) + dtheta_2*l_1*l_2*m_5*np.sin(theta_2 - theta_1) + dtheta_2*l_1*m_2*r_2*np.sin(theta_2 - theta_1)) - dtheta_1*(mu_1 + mu_2) - ddx_c*(-l_1*m_2 - l_1*m_3 - l_1*m_4 - l_1*m_5 - m_1*r_1)*np.cos(theta_1) - g*(-l_1*m_2 - l_1*m_3 - l_1*m_4 - l_1*m_5 - m_1*r_1)*np.sin(theta_1))*np.cos(theta_3 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) - (-l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_3 - theta_2))*(dtheta_5**2*l_2*m_5*r_5*np.sin(theta_5 - theta_2) + dtheta_4*(dtheta_4*l_2*l_4*m_5*np.sin(theta_4 - theta_2) + dtheta_4*l_2*m_4*r_4*np.sin(theta_4 - theta_2)) + dtheta_3*mu_3 + dtheta_3*(dtheta_3*l_2*l_3*m_4*np.sin(theta_3 - theta_2) + dtheta_3*l_2*l_3*m_5*np.sin(theta_3 - theta_2) + dtheta_3*l_2*m_3*r_3*np.sin(theta_3 - theta_2)) - dtheta_2*(mu_2 + mu_3) + dtheta_1*mu_2 + dtheta_1*(-dtheta_1*l_1*l_2*m_3*np.sin(theta_2 - theta_1) - dtheta_1*l_1*l_2*m_4*np.sin(theta_2 - theta_1) - dtheta_1*l_1*l_2*m_5*np.sin(theta_2 - theta_1) - dtheta_1*l_1*m_2*r_2*np.sin(theta_2 - theta_1)) - ddx_c*(-l_2*m_3 - l_2*m_4 - l_2*m_5 - m_2*r_2)*np.cos(theta_2) - g*(-l_2*m_3 - l_2*m_4 - l_2*m_5 - m_2*r_2)*np.sin(theta_2) - l_1*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*(dtheta_5**2*l_1*m_5*r_5*np.sin(theta_5 - theta_1) + dtheta_4*(dtheta_4*l_1*l_4*m_5*np.sin(theta_4 - theta_1) + dtheta_4*l_1*m_4*r_4*np.sin(theta_4 - theta_1)) + dtheta_3*(dtheta_3*l_1*l_3*m_4*np.sin(theta_3 - theta_1) + dtheta_3*l_1*l_3*m_5*np.sin(theta_3 - theta_1) + dtheta_3*l_1*m_3*r_3*np.sin(theta_3 - theta_1)) + dtheta_2*mu_2 + dtheta_2*(dtheta_2*l_1*l_2*m_3*np.sin(theta_2 - theta_1) + dtheta_2*l_1*l_2*m_4*np.sin(theta_2 - theta_1) + dtheta_2*l_1*l_2*m_5*np.sin(theta_2 - theta_1) + dtheta_2*l_1*m_2*r_2*np.sin(theta_2 - theta_1)) - dtheta_1*(mu_1 + mu_2) - ddx_c*(-l_1*m_2 - l_1*m_3 - l_1*m_4 - l_1*m_5 - m_1*r_1)*np.cos(theta_1) - g*(-l_1*m_2 - l_1*m_3 - l_1*m_4 - l_1*m_5 - m_1*r_1)*np.sin(theta_1))*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2))/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2))/(I_3 - l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)**2*np.cos(theta_3 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_3**2*m_4 + l_3**2*m_5 + m_3*r_3**2 - (-l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_3 - theta_2))**2/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2)))/(I_4 - l_1**2*(l_4*m_5 + m_4*r_4)**2*np.cos(theta_4 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_4**2*m_5 + m_4*r_4**2 - (-l_1**2*(l_4*m_5 + m_4*r_4)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_4 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_4*m_5 + m_4*r_4)*np.cos(theta_4 - theta_2))**2/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2) - (-l_1**2*(l_4*m_5 + m_4*r_4)*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_4 - theta_1)*np.cos(theta_3 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_3*(l_4*m_5 + m_4*r_4)*np.cos(theta_4 - theta_3) - (-l_1**2*(l_4*m_5 + m_4*r_4)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_4 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_4*m_5 + m_4*r_4)*np.cos(theta_4 - theta_2))*(-l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_3 - theta_2))/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2))**2/(I_3 - l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)**2*np.cos(theta_3 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_3**2*m_4 + l_3**2*m_5 + m_3*r_3**2 - (-l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_3 - theta_2))**2/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2))))/(I_5 - l_1**2*m_5**2*r_5**2*np.cos(theta_5 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + m_5*r_5**2 - (-l_1**2*m_5*r_5*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_5 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*m_5*r_5*np.cos(theta_5 - theta_2))**2/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2) - (-l_1**2*m_5*r_5*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_5 - theta_1)*np.cos(theta_3 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_3*m_5*r_5*np.cos(theta_5 - theta_3) - (-l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_3 - theta_2))*(-l_1**2*m_5*r_5*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_5 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*m_5*r_5*np.cos(theta_5 - theta_2))/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2))**2/(I_3 - l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)**2*np.cos(theta_3 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_3**2*m_4 + l_3**2*m_5 + m_3*r_3**2 - (-l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_3 - theta_2))**2/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2)) - (-l_1**2*m_5*r_5*(l_4*m_5 + m_4*r_4)*np.cos(theta_5 - theta_1)*np.cos(theta_4 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_4*m_5*r_5*np.cos(theta_5 - theta_4) - (-l_1**2*(l_4*m_5 + m_4*r_4)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_4 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_4*m_5 + m_4*r_4)*np.cos(theta_4 - theta_2))*(-l_1**2*m_5*r_5*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_5 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*m_5*r_5*np.cos(theta_5 - theta_2))/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2) - (-l_1**2*(l_4*m_5 + m_4*r_4)*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_4 - theta_1)*np.cos(theta_3 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_3*(l_4*m_5 + m_4*r_4)*np.cos(theta_4 - theta_3) - (-l_1**2*(l_4*m_5 + m_4*r_4)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_4 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_4*m_5 + m_4*r_4)*np.cos(theta_4 - theta_2))*(-l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_3 - theta_2))/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2))*(-l_1**2*m_5*r_5*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_5 - theta_1)*np.cos(theta_3 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_3*m_5*r_5*np.cos(theta_5 - theta_3) - (-l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_3 - theta_2))*(-l_1**2*m_5*r_5*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_5 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*m_5*r_5*np.cos(theta_5 - theta_2))/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2))/(I_3 - l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)**2*np.cos(theta_3 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_3**2*m_4 + l_3**2*m_5 + m_3*r_3**2 - (-l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_3 - theta_2))**2/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2)))**2/(I_4 - l_1**2*(l_4*m_5 + m_4*r_4)**2*np.cos(theta_4 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_4**2*m_5 + m_4*r_4**2 - (-l_1**2*(l_4*m_5 + m_4*r_4)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_4 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_4*m_5 + m_4*r_4)*np.cos(theta_4 - theta_2))**2/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2) - (-l_1**2*(l_4*m_5 + m_4*r_4)*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_4 - theta_1)*np.cos(theta_3 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_3*(l_4*m_5 + m_4*r_4)*np.cos(theta_4 - theta_3) - (-l_1**2*(l_4*m_5 + m_4*r_4)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_4 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_4*m_5 + m_4*r_4)*np.cos(theta_4 - theta_2))*(-l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_3 - theta_2))/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2))**2/(I_3 - l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)**2*np.cos(theta_3 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_3**2*m_4 + l_3**2*m_5 + m_3*r_3**2 - (-l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_3 - theta_2))**2/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2)))))/(I_4 - l_1**2*(l_4*m_5 + m_4*r_4)**2*np.cos(theta_4 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_4**2*m_5 + m_4*r_4**2 - (-l_1**2*(l_4*m_5 + m_4*r_4)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_4 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_4*m_5 + m_4*r_4)*np.cos(theta_4 - theta_2))**2/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2) - (-l_1**2*(l_4*m_5 + m_4*r_4)*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_4 - theta_1)*np.cos(theta_3 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_3*(l_4*m_5 + m_4*r_4)*np.cos(theta_4 - theta_3) - (-l_1**2*(l_4*m_5 + m_4*r_4)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_4 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_4*m_5 + m_4*r_4)*np.cos(theta_4 - theta_2))*(-l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_3 - theta_2))/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2))**2/(I_3 - l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)**2*np.cos(theta_3 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_3**2*m_4 + l_3**2*m_5 + m_3*r_3**2 - (-l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_3 - theta_2))**2/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2))) - (-l_1**2*m_5*r_5*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_5 - theta_1)*np.cos(theta_3 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_3*m_5*r_5*np.cos(theta_5 - theta_3) - (-l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_3 - theta_2))*(-l_1**2*m_5*r_5*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_5 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*m_5*r_5*np.cos(theta_5 - theta_2))/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2))*(-dtheta_5*mu_5 - dtheta_4**2*l_4*m_5*r_5*np.sin(theta_5 - theta_4) + dtheta_4*mu_5 - dtheta_3**2*l_3*m_5*r_5*np.sin(theta_5 - theta_3) - dtheta_2**2*l_2*m_5*r_5*np.sin(theta_5 - theta_2) - dtheta_1**2*l_1*m_5*r_5*np.sin(theta_5 - theta_1) + ddx_c*m_5*r_5*np.cos(theta_5) + g*m_5*r_5*np.sin(theta_5) - l_1*m_5*r_5*(dtheta_5**2*l_1*m_5*r_5*np.sin(theta_5 - theta_1) + dtheta_4*(dtheta_4*l_1*l_4*m_5*np.sin(theta_4 - theta_1) + dtheta_4*l_1*m_4*r_4*np.sin(theta_4 - theta_1)) + dtheta_3*(dtheta_3*l_1*l_3*m_4*np.sin(theta_3 - theta_1) + dtheta_3*l_1*l_3*m_5*np.sin(theta_3 - theta_1) + dtheta_3*l_1*m_3*r_3*np.sin(theta_3 - theta_1)) + dtheta_2*mu_2 + dtheta_2*(dtheta_2*l_1*l_2*m_3*np.sin(theta_2 - theta_1) + dtheta_2*l_1*l_2*m_4*np.sin(theta_2 - theta_1) + dtheta_2*l_1*l_2*m_5*np.sin(theta_2 - theta_1) + dtheta_2*l_1*m_2*r_2*np.sin(theta_2 - theta_1)) - dtheta_1*(mu_1 + mu_2) - ddx_c*(-l_1*m_2 - l_1*m_3 - l_1*m_4 - l_1*m_5 - m_1*r_1)*np.cos(theta_1) - g*(-l_1*m_2 - l_1*m_3 - l_1*m_4 - l_1*m_5 - m_1*r_1)*np.sin(theta_1))*np.cos(theta_5 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) - (-l_1**2*m_5*r_5*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_5 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*m_5*r_5*np.cos(theta_5 - theta_2))*(dtheta_5**2*l_2*m_5*r_5*np.sin(theta_5 - theta_2) + dtheta_4*(dtheta_4*l_2*l_4*m_5*np.sin(theta_4 - theta_2) + dtheta_4*l_2*m_4*r_4*np.sin(theta_4 - theta_2)) + dtheta_3*mu_3 + dtheta_3*(dtheta_3*l_2*l_3*m_4*np.sin(theta_3 - theta_2) + dtheta_3*l_2*l_3*m_5*np.sin(theta_3 - theta_2) + dtheta_3*l_2*m_3*r_3*np.sin(theta_3 - theta_2)) - dtheta_2*(mu_2 + mu_3) + dtheta_1*mu_2 + dtheta_1*(-dtheta_1*l_1*l_2*m_3*np.sin(theta_2 - theta_1) - dtheta_1*l_1*l_2*m_4*np.sin(theta_2 - theta_1) - dtheta_1*l_1*l_2*m_5*np.sin(theta_2 - theta_1) - dtheta_1*l_1*m_2*r_2*np.sin(theta_2 - theta_1)) - ddx_c*(-l_2*m_3 - l_2*m_4 - l_2*m_5 - m_2*r_2)*np.cos(theta_2) - g*(-l_2*m_3 - l_2*m_4 - l_2*m_5 - m_2*r_2)*np.sin(theta_2) - l_1*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*(dtheta_5**2*l_1*m_5*r_5*np.sin(theta_5 - theta_1) + dtheta_4*(dtheta_4*l_1*l_4*m_5*np.sin(theta_4 - theta_1) + dtheta_4*l_1*m_4*r_4*np.sin(theta_4 - theta_1)) + dtheta_3*(dtheta_3*l_1*l_3*m_4*np.sin(theta_3 - theta_1) + dtheta_3*l_1*l_3*m_5*np.sin(theta_3 - theta_1) + dtheta_3*l_1*m_3*r_3*np.sin(theta_3 - theta_1)) + dtheta_2*mu_2 + dtheta_2*(dtheta_2*l_1*l_2*m_3*np.sin(theta_2 - theta_1) + dtheta_2*l_1*l_2*m_4*np.sin(theta_2 - theta_1) + dtheta_2*l_1*l_2*m_5*np.sin(theta_2 - theta_1) + dtheta_2*l_1*m_2*r_2*np.sin(theta_2 - theta_1)) - dtheta_1*(mu_1 + mu_2) - ddx_c*(-l_1*m_2 - l_1*m_3 - l_1*m_4 - l_1*m_5 - m_1*r_1)*np.cos(theta_1) - g*(-l_1*m_2 - l_1*m_3 - l_1*m_4 - l_1*m_5 - m_1*r_1)*np.sin(theta_1))*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2))/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2) - (-l_1**2*m_5*r_5*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_5 - theta_1)*np.cos(theta_3 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_3*m_5*r_5*np.cos(theta_5 - theta_3) - (-l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_3 - theta_2))*(-l_1**2*m_5*r_5*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_5 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*m_5*r_5*np.cos(theta_5 - theta_2))/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2))*(dtheta_5**2*l_3*m_5*r_5*np.sin(theta_5 - theta_3) + dtheta_4*mu_4 + dtheta_4*(dtheta_4*l_3*l_4*m_5*np.sin(theta_4 - theta_3) + dtheta_4*l_3*m_4*r_4*np.sin(theta_4 - theta_3)) - dtheta_3*(mu_3 + mu_4) + dtheta_2*mu_3 + dtheta_2*(-dtheta_2*l_2*l_3*m_4*np.sin(theta_3 - theta_2) - dtheta_2*l_2*l_3*m_5*np.sin(theta_3 - theta_2) - dtheta_2*l_2*m_3*r_3*np.sin(theta_3 - theta_2)) + dtheta_1*(-dtheta_1*l_1*l_3*m_4*np.sin(theta_3 - theta_1) - dtheta_1*l_1*l_3*m_5*np.sin(theta_3 - theta_1) - dtheta_1*l_1*m_3*r_3*np.sin(theta_3 - theta_1)) - ddx_c*(-l_3*m_4 - l_3*m_5 - m_3*r_3)*np.cos(theta_3) - g*(-l_3*m_4 - l_3*m_5 - m_3*r_3)*np.sin(theta_3) - l_1*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(dtheta_5**2*l_1*m_5*r_5*np.sin(theta_5 - theta_1) + dtheta_4*(dtheta_4*l_1*l_4*m_5*np.sin(theta_4 - theta_1) + dtheta_4*l_1*m_4*r_4*np.sin(theta_4 - theta_1)) + dtheta_3*(dtheta_3*l_1*l_3*m_4*np.sin(theta_3 - theta_1) + dtheta_3*l_1*l_3*m_5*np.sin(theta_3 - theta_1) + dtheta_3*l_1*m_3*r_3*np.sin(theta_3 - theta_1)) + dtheta_2*mu_2 + dtheta_2*(dtheta_2*l_1*l_2*m_3*np.sin(theta_2 - theta_1) + dtheta_2*l_1*l_2*m_4*np.sin(theta_2 - theta_1) + dtheta_2*l_1*l_2*m_5*np.sin(theta_2 - theta_1) + dtheta_2*l_1*m_2*r_2*np.sin(theta_2 - theta_1)) - dtheta_1*(mu_1 + mu_2) - ddx_c*(-l_1*m_2 - l_1*m_3 - l_1*m_4 - l_1*m_5 - m_1*r_1)*np.cos(theta_1) - g*(-l_1*m_2 - l_1*m_3 - l_1*m_4 - l_1*m_5 - m_1*r_1)*np.sin(theta_1))*np.cos(theta_3 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) - (-l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_3 - theta_2))*(dtheta_5**2*l_2*m_5*r_5*np.sin(theta_5 - theta_2) + dtheta_4*(dtheta_4*l_2*l_4*m_5*np.sin(theta_4 - theta_2) + dtheta_4*l_2*m_4*r_4*np.sin(theta_4 - theta_2)) + dtheta_3*mu_3 + dtheta_3*(dtheta_3*l_2*l_3*m_4*np.sin(theta_3 - theta_2) + dtheta_3*l_2*l_3*m_5*np.sin(theta_3 - theta_2) + dtheta_3*l_2*m_3*r_3*np.sin(theta_3 - theta_2)) - dtheta_2*(mu_2 + mu_3) + dtheta_1*mu_2 + dtheta_1*(-dtheta_1*l_1*l_2*m_3*np.sin(theta_2 - theta_1) - dtheta_1*l_1*l_2*m_4*np.sin(theta_2 - theta_1) - dtheta_1*l_1*l_2*m_5*np.sin(theta_2 - theta_1) - dtheta_1*l_1*m_2*r_2*np.sin(theta_2 - theta_1)) - ddx_c*(-l_2*m_3 - l_2*m_4 - l_2*m_5 - m_2*r_2)*np.cos(theta_2) - g*(-l_2*m_3 - l_2*m_4 - l_2*m_5 - m_2*r_2)*np.sin(theta_2) - l_1*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*(dtheta_5**2*l_1*m_5*r_5*np.sin(theta_5 - theta_1) + dtheta_4*(dtheta_4*l_1*l_4*m_5*np.sin(theta_4 - theta_1) + dtheta_4*l_1*m_4*r_4*np.sin(theta_4 - theta_1)) + dtheta_3*(dtheta_3*l_1*l_3*m_4*np.sin(theta_3 - theta_1) + dtheta_3*l_1*l_3*m_5*np.sin(theta_3 - theta_1) + dtheta_3*l_1*m_3*r_3*np.sin(theta_3 - theta_1)) + dtheta_2*mu_2 + dtheta_2*(dtheta_2*l_1*l_2*m_3*np.sin(theta_2 - theta_1) + dtheta_2*l_1*l_2*m_4*np.sin(theta_2 - theta_1) + dtheta_2*l_1*l_2*m_5*np.sin(theta_2 - theta_1) + dtheta_2*l_1*m_2*r_2*np.sin(theta_2 - theta_1)) - dtheta_1*(mu_1 + mu_2) - ddx_c*(-l_1*m_2 - l_1*m_3 - l_1*m_4 - l_1*m_5 - m_1*r_1)*np.cos(theta_1) - g*(-l_1*m_2 - l_1*m_3 - l_1*m_4 - l_1*m_5 - m_1*r_1)*np.sin(theta_1))*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2))/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2))/(I_3 - l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)**2*np.cos(theta_3 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_3**2*m_4 + l_3**2*m_5 + m_3*r_3**2 - (-l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_3 - theta_2))**2/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2)) - (-l_1**2*m_5*r_5*(l_4*m_5 + m_4*r_4)*np.cos(theta_5 - theta_1)*np.cos(theta_4 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_4*m_5*r_5*np.cos(theta_5 - theta_4) - (-l_1**2*(l_4*m_5 + m_4*r_4)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_4 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_4*m_5 + m_4*r_4)*np.cos(theta_4 - theta_2))*(-l_1**2*m_5*r_5*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_5 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*m_5*r_5*np.cos(theta_5 - theta_2))/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2) - (-l_1**2*(l_4*m_5 + m_4*r_4)*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_4 - theta_1)*np.cos(theta_3 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_3*(l_4*m_5 + m_4*r_4)*np.cos(theta_4 - theta_3) - (-l_1**2*(l_4*m_5 + m_4*r_4)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_4 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_4*m_5 + m_4*r_4)*np.cos(theta_4 - theta_2))*(-l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_3 - theta_2))/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2))*(-l_1**2*m_5*r_5*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_5 - theta_1)*np.cos(theta_3 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_3*m_5*r_5*np.cos(theta_5 - theta_3) - (-l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_3 - theta_2))*(-l_1**2*m_5*r_5*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_5 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*m_5*r_5*np.cos(theta_5 - theta_2))/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2))/(I_3 - l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)**2*np.cos(theta_3 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_3**2*m_4 + l_3**2*m_5 + m_3*r_3**2 - (-l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_3 - theta_2))**2/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2)))*(dtheta_5**2*l_4*m_5*r_5*np.sin(theta_5 - theta_4) + dtheta_5*mu_5 - dtheta_4*(mu_4 + mu_5) + dtheta_3*mu_4 + dtheta_3*(-dtheta_3*l_3*l_4*m_5*np.sin(theta_4 - theta_3) - dtheta_3*l_3*m_4*r_4*np.sin(theta_4 - theta_3)) + dtheta_2*(-dtheta_2*l_2*l_4*m_5*np.sin(theta_4 - theta_2) - dtheta_2*l_2*m_4*r_4*np.sin(theta_4 - theta_2)) + dtheta_1*(-dtheta_1*l_1*l_4*m_5*np.sin(theta_4 - theta_1) - dtheta_1*l_1*m_4*r_4*np.sin(theta_4 - theta_1)) - ddx_c*(-l_4*m_5 - m_4*r_4)*np.cos(theta_4) - g*(-l_4*m_5 - m_4*r_4)*np.sin(theta_4) - l_1*(l_4*m_5 + m_4*r_4)*(dtheta_5**2*l_1*m_5*r_5*np.sin(theta_5 - theta_1) + dtheta_4*(dtheta_4*l_1*l_4*m_5*np.sin(theta_4 - theta_1) + dtheta_4*l_1*m_4*r_4*np.sin(theta_4 - theta_1)) + dtheta_3*(dtheta_3*l_1*l_3*m_4*np.sin(theta_3 - theta_1) + dtheta_3*l_1*l_3*m_5*np.sin(theta_3 - theta_1) + dtheta_3*l_1*m_3*r_3*np.sin(theta_3 - theta_1)) + dtheta_2*mu_2 + dtheta_2*(dtheta_2*l_1*l_2*m_3*np.sin(theta_2 - theta_1) + dtheta_2*l_1*l_2*m_4*np.sin(theta_2 - theta_1) + dtheta_2*l_1*l_2*m_5*np.sin(theta_2 - theta_1) + dtheta_2*l_1*m_2*r_2*np.sin(theta_2 - theta_1)) - dtheta_1*(mu_1 + mu_2) - ddx_c*(-l_1*m_2 - l_1*m_3 - l_1*m_4 - l_1*m_5 - m_1*r_1)*np.cos(theta_1) - g*(-l_1*m_2 - l_1*m_3 - l_1*m_4 - l_1*m_5 - m_1*r_1)*np.sin(theta_1))*np.cos(theta_4 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) - (-l_1**2*(l_4*m_5 + m_4*r_4)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_4 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_4*m_5 + m_4*r_4)*np.cos(theta_4 - theta_2))*(dtheta_5**2*l_2*m_5*r_5*np.sin(theta_5 - theta_2) + dtheta_4*(dtheta_4*l_2*l_4*m_5*np.sin(theta_4 - theta_2) + dtheta_4*l_2*m_4*r_4*np.sin(theta_4 - theta_2)) + dtheta_3*mu_3 + dtheta_3*(dtheta_3*l_2*l_3*m_4*np.sin(theta_3 - theta_2) + dtheta_3*l_2*l_3*m_5*np.sin(theta_3 - theta_2) + dtheta_3*l_2*m_3*r_3*np.sin(theta_3 - theta_2)) - dtheta_2*(mu_2 + mu_3) + dtheta_1*mu_2 + dtheta_1*(-dtheta_1*l_1*l_2*m_3*np.sin(theta_2 - theta_1) - dtheta_1*l_1*l_2*m_4*np.sin(theta_2 - theta_1) - dtheta_1*l_1*l_2*m_5*np.sin(theta_2 - theta_1) - dtheta_1*l_1*m_2*r_2*np.sin(theta_2 - theta_1)) - ddx_c*(-l_2*m_3 - l_2*m_4 - l_2*m_5 - m_2*r_2)*np.cos(theta_2) - g*(-l_2*m_3 - l_2*m_4 - l_2*m_5 - m_2*r_2)*np.sin(theta_2) - l_1*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*(dtheta_5**2*l_1*m_5*r_5*np.sin(theta_5 - theta_1) + dtheta_4*(dtheta_4*l_1*l_4*m_5*np.sin(theta_4 - theta_1) + dtheta_4*l_1*m_4*r_4*np.sin(theta_4 - theta_1)) + dtheta_3*(dtheta_3*l_1*l_3*m_4*np.sin(theta_3 - theta_1) + dtheta_3*l_1*l_3*m_5*np.sin(theta_3 - theta_1) + dtheta_3*l_1*m_3*r_3*np.sin(theta_3 - theta_1)) + dtheta_2*mu_2 + dtheta_2*(dtheta_2*l_1*l_2*m_3*np.sin(theta_2 - theta_1) + dtheta_2*l_1*l_2*m_4*np.sin(theta_2 - theta_1) + dtheta_2*l_1*l_2*m_5*np.sin(theta_2 - theta_1) + dtheta_2*l_1*m_2*r_2*np.sin(theta_2 - theta_1)) - dtheta_1*(mu_1 + mu_2) - ddx_c*(-l_1*m_2 - l_1*m_3 - l_1*m_4 - l_1*m_5 - m_1*r_1)*np.cos(theta_1) - g*(-l_1*m_2 - l_1*m_3 - l_1*m_4 - l_1*m_5 - m_1*r_1)*np.sin(theta_1))*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2))/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2) - (-l_1**2*(l_4*m_5 + m_4*r_4)*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_4 - theta_1)*np.cos(theta_3 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_3*(l_4*m_5 + m_4*r_4)*np.cos(theta_4 - theta_3) - (-l_1**2*(l_4*m_5 + m_4*r_4)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_4 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_4*m_5 + m_4*r_4)*np.cos(theta_4 - theta_2))*(-l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_3 - theta_2))/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2))*(dtheta_5**2*l_3*m_5*r_5*np.sin(theta_5 - theta_3) + dtheta_4*mu_4 + dtheta_4*(dtheta_4*l_3*l_4*m_5*np.sin(theta_4 - theta_3) + dtheta_4*l_3*m_4*r_4*np.sin(theta_4 - theta_3)) - dtheta_3*(mu_3 + mu_4) + dtheta_2*mu_3 + dtheta_2*(-dtheta_2*l_2*l_3*m_4*np.sin(theta_3 - theta_2) - dtheta_2*l_2*l_3*m_5*np.sin(theta_3 - theta_2) - dtheta_2*l_2*m_3*r_3*np.sin(theta_3 - theta_2)) + dtheta_1*(-dtheta_1*l_1*l_3*m_4*np.sin(theta_3 - theta_1) - dtheta_1*l_1*l_3*m_5*np.sin(theta_3 - theta_1) - dtheta_1*l_1*m_3*r_3*np.sin(theta_3 - theta_1)) - ddx_c*(-l_3*m_4 - l_3*m_5 - m_3*r_3)*np.cos(theta_3) - g*(-l_3*m_4 - l_3*m_5 - m_3*r_3)*np.sin(theta_3) - l_1*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(dtheta_5**2*l_1*m_5*r_5*np.sin(theta_5 - theta_1) + dtheta_4*(dtheta_4*l_1*l_4*m_5*np.sin(theta_4 - theta_1) + dtheta_4*l_1*m_4*r_4*np.sin(theta_4 - theta_1)) + dtheta_3*(dtheta_3*l_1*l_3*m_4*np.sin(theta_3 - theta_1) + dtheta_3*l_1*l_3*m_5*np.sin(theta_3 - theta_1) + dtheta_3*l_1*m_3*r_3*np.sin(theta_3 - theta_1)) + dtheta_2*mu_2 + dtheta_2*(dtheta_2*l_1*l_2*m_3*np.sin(theta_2 - theta_1) + dtheta_2*l_1*l_2*m_4*np.sin(theta_2 - theta_1) + dtheta_2*l_1*l_2*m_5*np.sin(theta_2 - theta_1) + dtheta_2*l_1*m_2*r_2*np.sin(theta_2 - theta_1)) - dtheta_1*(mu_1 + mu_2) - ddx_c*(-l_1*m_2 - l_1*m_3 - l_1*m_4 - l_1*m_5 - m_1*r_1)*np.cos(theta_1) - g*(-l_1*m_2 - l_1*m_3 - l_1*m_4 - l_1*m_5 - m_1*r_1)*np.sin(theta_1))*np.cos(theta_3 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) - (-l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_3 - theta_2))*(dtheta_5**2*l_2*m_5*r_5*np.sin(theta_5 - theta_2) + dtheta_4*(dtheta_4*l_2*l_4*m_5*np.sin(theta_4 - theta_2) + dtheta_4*l_2*m_4*r_4*np.sin(theta_4 - theta_2)) + dtheta_3*mu_3 + dtheta_3*(dtheta_3*l_2*l_3*m_4*np.sin(theta_3 - theta_2) + dtheta_3*l_2*l_3*m_5*np.sin(theta_3 - theta_2) + dtheta_3*l_2*m_3*r_3*np.sin(theta_3 - theta_2)) - dtheta_2*(mu_2 + mu_3) + dtheta_1*mu_2 + dtheta_1*(-dtheta_1*l_1*l_2*m_3*np.sin(theta_2 - theta_1) - dtheta_1*l_1*l_2*m_4*np.sin(theta_2 - theta_1) - dtheta_1*l_1*l_2*m_5*np.sin(theta_2 - theta_1) - dtheta_1*l_1*m_2*r_2*np.sin(theta_2 - theta_1)) - ddx_c*(-l_2*m_3 - l_2*m_4 - l_2*m_5 - m_2*r_2)*np.cos(theta_2) - g*(-l_2*m_3 - l_2*m_4 - l_2*m_5 - m_2*r_2)*np.sin(theta_2) - l_1*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*(dtheta_5**2*l_1*m_5*r_5*np.sin(theta_5 - theta_1) + dtheta_4*(dtheta_4*l_1*l_4*m_5*np.sin(theta_4 - theta_1) + dtheta_4*l_1*m_4*r_4*np.sin(theta_4 - theta_1)) + dtheta_3*(dtheta_3*l_1*l_3*m_4*np.sin(theta_3 - theta_1) + dtheta_3*l_1*l_3*m_5*np.sin(theta_3 - theta_1) + dtheta_3*l_1*m_3*r_3*np.sin(theta_3 - theta_1)) + dtheta_2*mu_2 + dtheta_2*(dtheta_2*l_1*l_2*m_3*np.sin(theta_2 - theta_1) + dtheta_2*l_1*l_2*m_4*np.sin(theta_2 - theta_1) + dtheta_2*l_1*l_2*m_5*np.sin(theta_2 - theta_1) + dtheta_2*l_1*m_2*r_2*np.sin(theta_2 - theta_1)) - dtheta_1*(mu_1 + mu_2) - ddx_c*(-l_1*m_2 - l_1*m_3 - l_1*m_4 - l_1*m_5 - m_1*r_1)*np.cos(theta_1) - g*(-l_1*m_2 - l_1*m_3 - l_1*m_4 - l_1*m_5 - m_1*r_1)*np.sin(theta_1))*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2))/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2))/(I_3 - l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)**2*np.cos(theta_3 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_3**2*m_4 + l_3**2*m_5 + m_3*r_3**2 - (-l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_3 - theta_2))**2/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2)))/(I_4 - l_1**2*(l_4*m_5 + m_4*r_4)**2*np.cos(theta_4 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_4**2*m_5 + m_4*r_4**2 - (-l_1**2*(l_4*m_5 + m_4*r_4)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_4 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_4*m_5 + m_4*r_4)*np.cos(theta_4 - theta_2))**2/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2) - (-l_1**2*(l_4*m_5 + m_4*r_4)*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_4 - theta_1)*np.cos(theta_3 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_3*(l_4*m_5 + m_4*r_4)*np.cos(theta_4 - theta_3) - (-l_1**2*(l_4*m_5 + m_4*r_4)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_4 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_4*m_5 + m_4*r_4)*np.cos(theta_4 - theta_2))*(-l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_3 - theta_2))/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2))**2/(I_3 - l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)**2*np.cos(theta_3 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_3**2*m_4 + l_3**2*m_5 + m_3*r_3**2 - (-l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_3 - theta_2))**2/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2))))/(I_5 - l_1**2*m_5**2*r_5**2*np.cos(theta_5 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + m_5*r_5**2 - (-l_1**2*m_5*r_5*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_5 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*m_5*r_5*np.cos(theta_5 - theta_2))**2/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2) - (-l_1**2*m_5*r_5*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_5 - theta_1)*np.cos(theta_3 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_3*m_5*r_5*np.cos(theta_5 - theta_3) - (-l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_3 - theta_2))*(-l_1**2*m_5*r_5*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_5 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*m_5*r_5*np.cos(theta_5 - theta_2))/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2))**2/(I_3 - l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)**2*np.cos(theta_3 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_3**2*m_4 + l_3**2*m_5 + m_3*r_3**2 - (-l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_3 - theta_2))**2/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2)) - (-l_1**2*m_5*r_5*(l_4*m_5 + m_4*r_4)*np.cos(theta_5 - theta_1)*np.cos(theta_4 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_4*m_5*r_5*np.cos(theta_5 - theta_4) - (-l_1**2*(l_4*m_5 + m_4*r_4)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_4 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_4*m_5 + m_4*r_4)*np.cos(theta_4 - theta_2))*(-l_1**2*m_5*r_5*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_5 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*m_5*r_5*np.cos(theta_5 - theta_2))/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2) - (-l_1**2*(l_4*m_5 + m_4*r_4)*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_4 - theta_1)*np.cos(theta_3 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_3*(l_4*m_5 + m_4*r_4)*np.cos(theta_4 - theta_3) - (-l_1**2*(l_4*m_5 + m_4*r_4)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_4 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_4*m_5 + m_4*r_4)*np.cos(theta_4 - theta_2))*(-l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_3 - theta_2))/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2))*(-l_1**2*m_5*r_5*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_5 - theta_1)*np.cos(theta_3 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_3*m_5*r_5*np.cos(theta_5 - theta_3) - (-l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_3 - theta_2))*(-l_1**2*m_5*r_5*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_5 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*m_5*r_5*np.cos(theta_5 - theta_2))/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2))/(I_3 - l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)**2*np.cos(theta_3 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_3**2*m_4 + l_3**2*m_5 + m_3*r_3**2 - (-l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_3 - theta_2))**2/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2)))**2/(I_4 - l_1**2*(l_4*m_5 + m_4*r_4)**2*np.cos(theta_4 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_4**2*m_5 + m_4*r_4**2 - (-l_1**2*(l_4*m_5 + m_4*r_4)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_4 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_4*m_5 + m_4*r_4)*np.cos(theta_4 - theta_2))**2/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2) - (-l_1**2*(l_4*m_5 + m_4*r_4)*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_4 - theta_1)*np.cos(theta_3 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_3*(l_4*m_5 + m_4*r_4)*np.cos(theta_4 - theta_3) - (-l_1**2*(l_4*m_5 + m_4*r_4)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_4 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_4*m_5 + m_4*r_4)*np.cos(theta_4 - theta_2))*(-l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_3 - theta_2))/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2))**2/(I_3 - l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)**2*np.cos(theta_3 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_3**2*m_4 + l_3**2*m_5 + m_3*r_3**2 - (-l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_3 - theta_2))**2/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2)))))/(I_3 - l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)**2*np.cos(theta_3 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_3**2*m_4 + l_3**2*m_5 + m_3*r_3**2 - (-l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_3 - theta_2))**2/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2)) - (-l_1**2*m_5*r_5*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_5 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*m_5*r_5*np.cos(theta_5 - theta_2))*(-dtheta_5*mu_5 - dtheta_4**2*l_4*m_5*r_5*np.sin(theta_5 - theta_4) + dtheta_4*mu_5 - dtheta_3**2*l_3*m_5*r_5*np.sin(theta_5 - theta_3) - dtheta_2**2*l_2*m_5*r_5*np.sin(theta_5 - theta_2) - dtheta_1**2*l_1*m_5*r_5*np.sin(theta_5 - theta_1) + ddx_c*m_5*r_5*np.cos(theta_5) + g*m_5*r_5*np.sin(theta_5) - l_1*m_5*r_5*(dtheta_5**2*l_1*m_5*r_5*np.sin(theta_5 - theta_1) + dtheta_4*(dtheta_4*l_1*l_4*m_5*np.sin(theta_4 - theta_1) + dtheta_4*l_1*m_4*r_4*np.sin(theta_4 - theta_1)) + dtheta_3*(dtheta_3*l_1*l_3*m_4*np.sin(theta_3 - theta_1) + dtheta_3*l_1*l_3*m_5*np.sin(theta_3 - theta_1) + dtheta_3*l_1*m_3*r_3*np.sin(theta_3 - theta_1)) + dtheta_2*mu_2 + dtheta_2*(dtheta_2*l_1*l_2*m_3*np.sin(theta_2 - theta_1) + dtheta_2*l_1*l_2*m_4*np.sin(theta_2 - theta_1) + dtheta_2*l_1*l_2*m_5*np.sin(theta_2 - theta_1) + dtheta_2*l_1*m_2*r_2*np.sin(theta_2 - theta_1)) - dtheta_1*(mu_1 + mu_2) - ddx_c*(-l_1*m_2 - l_1*m_3 - l_1*m_4 - l_1*m_5 - m_1*r_1)*np.cos(theta_1) - g*(-l_1*m_2 - l_1*m_3 - l_1*m_4 - l_1*m_5 - m_1*r_1)*np.sin(theta_1))*np.cos(theta_5 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) - (-l_1**2*m_5*r_5*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_5 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*m_5*r_5*np.cos(theta_5 - theta_2))*(dtheta_5**2*l_2*m_5*r_5*np.sin(theta_5 - theta_2) + dtheta_4*(dtheta_4*l_2*l_4*m_5*np.sin(theta_4 - theta_2) + dtheta_4*l_2*m_4*r_4*np.sin(theta_4 - theta_2)) + dtheta_3*mu_3 + dtheta_3*(dtheta_3*l_2*l_3*m_4*np.sin(theta_3 - theta_2) + dtheta_3*l_2*l_3*m_5*np.sin(theta_3 - theta_2) + dtheta_3*l_2*m_3*r_3*np.sin(theta_3 - theta_2)) - dtheta_2*(mu_2 + mu_3) + dtheta_1*mu_2 + dtheta_1*(-dtheta_1*l_1*l_2*m_3*np.sin(theta_2 - theta_1) - dtheta_1*l_1*l_2*m_4*np.sin(theta_2 - theta_1) - dtheta_1*l_1*l_2*m_5*np.sin(theta_2 - theta_1) - dtheta_1*l_1*m_2*r_2*np.sin(theta_2 - theta_1)) - ddx_c*(-l_2*m_3 - l_2*m_4 - l_2*m_5 - m_2*r_2)*np.cos(theta_2) - g*(-l_2*m_3 - l_2*m_4 - l_2*m_5 - m_2*r_2)*np.sin(theta_2) - l_1*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*(dtheta_5**2*l_1*m_5*r_5*np.sin(theta_5 - theta_1) + dtheta_4*(dtheta_4*l_1*l_4*m_5*np.sin(theta_4 - theta_1) + dtheta_4*l_1*m_4*r_4*np.sin(theta_4 - theta_1)) + dtheta_3*(dtheta_3*l_1*l_3*m_4*np.sin(theta_3 - theta_1) + dtheta_3*l_1*l_3*m_5*np.sin(theta_3 - theta_1) + dtheta_3*l_1*m_3*r_3*np.sin(theta_3 - theta_1)) + dtheta_2*mu_2 + dtheta_2*(dtheta_2*l_1*l_2*m_3*np.sin(theta_2 - theta_1) + dtheta_2*l_1*l_2*m_4*np.sin(theta_2 - theta_1) + dtheta_2*l_1*l_2*m_5*np.sin(theta_2 - theta_1) + dtheta_2*l_1*m_2*r_2*np.sin(theta_2 - theta_1)) - dtheta_1*(mu_1 + mu_2) - ddx_c*(-l_1*m_2 - l_1*m_3 - l_1*m_4 - l_1*m_5 - m_1*r_1)*np.cos(theta_1) - g*(-l_1*m_2 - l_1*m_3 - l_1*m_4 - l_1*m_5 - m_1*r_1)*np.sin(theta_1))*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2))/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2) - (-l_1**2*m_5*r_5*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_5 - theta_1)*np.cos(theta_3 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_3*m_5*r_5*np.cos(theta_5 - theta_3) - (-l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_3 - theta_2))*(-l_1**2*m_5*r_5*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_5 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*m_5*r_5*np.cos(theta_5 - theta_2))/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2))*(dtheta_5**2*l_3*m_5*r_5*np.sin(theta_5 - theta_3) + dtheta_4*mu_4 + dtheta_4*(dtheta_4*l_3*l_4*m_5*np.sin(theta_4 - theta_3) + dtheta_4*l_3*m_4*r_4*np.sin(theta_4 - theta_3)) - dtheta_3*(mu_3 + mu_4) + dtheta_2*mu_3 + dtheta_2*(-dtheta_2*l_2*l_3*m_4*np.sin(theta_3 - theta_2) - dtheta_2*l_2*l_3*m_5*np.sin(theta_3 - theta_2) - dtheta_2*l_2*m_3*r_3*np.sin(theta_3 - theta_2)) + dtheta_1*(-dtheta_1*l_1*l_3*m_4*np.sin(theta_3 - theta_1) - dtheta_1*l_1*l_3*m_5*np.sin(theta_3 - theta_1) - dtheta_1*l_1*m_3*r_3*np.sin(theta_3 - theta_1)) - ddx_c*(-l_3*m_4 - l_3*m_5 - m_3*r_3)*np.cos(theta_3) - g*(-l_3*m_4 - l_3*m_5 - m_3*r_3)*np.sin(theta_3) - l_1*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(dtheta_5**2*l_1*m_5*r_5*np.sin(theta_5 - theta_1) + dtheta_4*(dtheta_4*l_1*l_4*m_5*np.sin(theta_4 - theta_1) + dtheta_4*l_1*m_4*r_4*np.sin(theta_4 - theta_1)) + dtheta_3*(dtheta_3*l_1*l_3*m_4*np.sin(theta_3 - theta_1) + dtheta_3*l_1*l_3*m_5*np.sin(theta_3 - theta_1) + dtheta_3*l_1*m_3*r_3*np.sin(theta_3 - theta_1)) + dtheta_2*mu_2 + dtheta_2*(dtheta_2*l_1*l_2*m_3*np.sin(theta_2 - theta_1) + dtheta_2*l_1*l_2*m_4*np.sin(theta_2 - theta_1) + dtheta_2*l_1*l_2*m_5*np.sin(theta_2 - theta_1) + dtheta_2*l_1*m_2*r_2*np.sin(theta_2 - theta_1)) - dtheta_1*(mu_1 + mu_2) - ddx_c*(-l_1*m_2 - l_1*m_3 - l_1*m_4 - l_1*m_5 - m_1*r_1)*np.cos(theta_1) - g*(-l_1*m_2 - l_1*m_3 - l_1*m_4 - l_1*m_5 - m_1*r_1)*np.sin(theta_1))*np.cos(theta_3 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) - (-l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_3 - theta_2))*(dtheta_5**2*l_2*m_5*r_5*np.sin(theta_5 - theta_2) + dtheta_4*(dtheta_4*l_2*l_4*m_5*np.sin(theta_4 - theta_2) + dtheta_4*l_2*m_4*r_4*np.sin(theta_4 - theta_2)) + dtheta_3*mu_3 + dtheta_3*(dtheta_3*l_2*l_3*m_4*np.sin(theta_3 - theta_2) + dtheta_3*l_2*l_3*m_5*np.sin(theta_3 - theta_2) + dtheta_3*l_2*m_3*r_3*np.sin(theta_3 - theta_2)) - dtheta_2*(mu_2 + mu_3) + dtheta_1*mu_2 + dtheta_1*(-dtheta_1*l_1*l_2*m_3*np.sin(theta_2 - theta_1) - dtheta_1*l_1*l_2*m_4*np.sin(theta_2 - theta_1) - dtheta_1*l_1*l_2*m_5*np.sin(theta_2 - theta_1) - dtheta_1*l_1*m_2*r_2*np.sin(theta_2 - theta_1)) - ddx_c*(-l_2*m_3 - l_2*m_4 - l_2*m_5 - m_2*r_2)*np.cos(theta_2) - g*(-l_2*m_3 - l_2*m_4 - l_2*m_5 - m_2*r_2)*np.sin(theta_2) - l_1*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*(dtheta_5**2*l_1*m_5*r_5*np.sin(theta_5 - theta_1) + dtheta_4*(dtheta_4*l_1*l_4*m_5*np.sin(theta_4 - theta_1) + dtheta_4*l_1*m_4*r_4*np.sin(theta_4 - theta_1)) + dtheta_3*(dtheta_3*l_1*l_3*m_4*np.sin(theta_3 - theta_1) + dtheta_3*l_1*l_3*m_5*np.sin(theta_3 - theta_1) + dtheta_3*l_1*m_3*r_3*np.sin(theta_3 - theta_1)) + dtheta_2*mu_2 + dtheta_2*(dtheta_2*l_1*l_2*m_3*np.sin(theta_2 - theta_1) + dtheta_2*l_1*l_2*m_4*np.sin(theta_2 - theta_1) + dtheta_2*l_1*l_2*m_5*np.sin(theta_2 - theta_1) + dtheta_2*l_1*m_2*r_2*np.sin(theta_2 - theta_1)) - dtheta_1*(mu_1 + mu_2) - ddx_c*(-l_1*m_2 - l_1*m_3 - l_1*m_4 - l_1*m_5 - m_1*r_1)*np.cos(theta_1) - g*(-l_1*m_2 - l_1*m_3 - l_1*m_4 - l_1*m_5 - m_1*r_1)*np.sin(theta_1))*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2))/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2))/(I_3 - l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)**2*np.cos(theta_3 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_3**2*m_4 + l_3**2*m_5 + m_3*r_3**2 - (-l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_3 - theta_2))**2/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2)) - (-l_1**2*m_5*r_5*(l_4*m_5 + m_4*r_4)*np.cos(theta_5 - theta_1)*np.cos(theta_4 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_4*m_5*r_5*np.cos(theta_5 - theta_4) - (-l_1**2*(l_4*m_5 + m_4*r_4)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_4 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_4*m_5 + m_4*r_4)*np.cos(theta_4 - theta_2))*(-l_1**2*m_5*r_5*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_5 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*m_5*r_5*np.cos(theta_5 - theta_2))/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2) - (-l_1**2*(l_4*m_5 + m_4*r_4)*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_4 - theta_1)*np.cos(theta_3 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_3*(l_4*m_5 + m_4*r_4)*np.cos(theta_4 - theta_3) - (-l_1**2*(l_4*m_5 + m_4*r_4)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_4 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_4*m_5 + m_4*r_4)*np.cos(theta_4 - theta_2))*(-l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_3 - theta_2))/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2))*(-l_1**2*m_5*r_5*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_5 - theta_1)*np.cos(theta_3 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_3*m_5*r_5*np.cos(theta_5 - theta_3) - (-l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_3 - theta_2))*(-l_1**2*m_5*r_5*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_5 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*m_5*r_5*np.cos(theta_5 - theta_2))/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2))/(I_3 - l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)**2*np.cos(theta_3 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_3**2*m_4 + l_3**2*m_5 + m_3*r_3**2 - (-l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_3 - theta_2))**2/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2)))*(dtheta_5**2*l_4*m_5*r_5*np.sin(theta_5 - theta_4) + dtheta_5*mu_5 - dtheta_4*(mu_4 + mu_5) + dtheta_3*mu_4 + dtheta_3*(-dtheta_3*l_3*l_4*m_5*np.sin(theta_4 - theta_3) - dtheta_3*l_3*m_4*r_4*np.sin(theta_4 - theta_3)) + dtheta_2*(-dtheta_2*l_2*l_4*m_5*np.sin(theta_4 - theta_2) - dtheta_2*l_2*m_4*r_4*np.sin(theta_4 - theta_2)) + dtheta_1*(-dtheta_1*l_1*l_4*m_5*np.sin(theta_4 - theta_1) - dtheta_1*l_1*m_4*r_4*np.sin(theta_4 - theta_1)) - ddx_c*(-l_4*m_5 - m_4*r_4)*np.cos(theta_4) - g*(-l_4*m_5 - m_4*r_4)*np.sin(theta_4) - l_1*(l_4*m_5 + m_4*r_4)*(dtheta_5**2*l_1*m_5*r_5*np.sin(theta_5 - theta_1) + dtheta_4*(dtheta_4*l_1*l_4*m_5*np.sin(theta_4 - theta_1) + dtheta_4*l_1*m_4*r_4*np.sin(theta_4 - theta_1)) + dtheta_3*(dtheta_3*l_1*l_3*m_4*np.sin(theta_3 - theta_1) + dtheta_3*l_1*l_3*m_5*np.sin(theta_3 - theta_1) + dtheta_3*l_1*m_3*r_3*np.sin(theta_3 - theta_1)) + dtheta_2*mu_2 + dtheta_2*(dtheta_2*l_1*l_2*m_3*np.sin(theta_2 - theta_1) + dtheta_2*l_1*l_2*m_4*np.sin(theta_2 - theta_1) + dtheta_2*l_1*l_2*m_5*np.sin(theta_2 - theta_1) + dtheta_2*l_1*m_2*r_2*np.sin(theta_2 - theta_1)) - dtheta_1*(mu_1 + mu_2) - ddx_c*(-l_1*m_2 - l_1*m_3 - l_1*m_4 - l_1*m_5 - m_1*r_1)*np.cos(theta_1) - g*(-l_1*m_2 - l_1*m_3 - l_1*m_4 - l_1*m_5 - m_1*r_1)*np.sin(theta_1))*np.cos(theta_4 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) - (-l_1**2*(l_4*m_5 + m_4*r_4)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_4 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_4*m_5 + m_4*r_4)*np.cos(theta_4 - theta_2))*(dtheta_5**2*l_2*m_5*r_5*np.sin(theta_5 - theta_2) + dtheta_4*(dtheta_4*l_2*l_4*m_5*np.sin(theta_4 - theta_2) + dtheta_4*l_2*m_4*r_4*np.sin(theta_4 - theta_2)) + dtheta_3*mu_3 + dtheta_3*(dtheta_3*l_2*l_3*m_4*np.sin(theta_3 - theta_2) + dtheta_3*l_2*l_3*m_5*np.sin(theta_3 - theta_2) + dtheta_3*l_2*m_3*r_3*np.sin(theta_3 - theta_2)) - dtheta_2*(mu_2 + mu_3) + dtheta_1*mu_2 + dtheta_1*(-dtheta_1*l_1*l_2*m_3*np.sin(theta_2 - theta_1) - dtheta_1*l_1*l_2*m_4*np.sin(theta_2 - theta_1) - dtheta_1*l_1*l_2*m_5*np.sin(theta_2 - theta_1) - dtheta_1*l_1*m_2*r_2*np.sin(theta_2 - theta_1)) - ddx_c*(-l_2*m_3 - l_2*m_4 - l_2*m_5 - m_2*r_2)*np.cos(theta_2) - g*(-l_2*m_3 - l_2*m_4 - l_2*m_5 - m_2*r_2)*np.sin(theta_2) - l_1*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*(dtheta_5**2*l_1*m_5*r_5*np.sin(theta_5 - theta_1) + dtheta_4*(dtheta_4*l_1*l_4*m_5*np.sin(theta_4 - theta_1) + dtheta_4*l_1*m_4*r_4*np.sin(theta_4 - theta_1)) + dtheta_3*(dtheta_3*l_1*l_3*m_4*np.sin(theta_3 - theta_1) + dtheta_3*l_1*l_3*m_5*np.sin(theta_3 - theta_1) + dtheta_3*l_1*m_3*r_3*np.sin(theta_3 - theta_1)) + dtheta_2*mu_2 + dtheta_2*(dtheta_2*l_1*l_2*m_3*np.sin(theta_2 - theta_1) + dtheta_2*l_1*l_2*m_4*np.sin(theta_2 - theta_1) + dtheta_2*l_1*l_2*m_5*np.sin(theta_2 - theta_1) + dtheta_2*l_1*m_2*r_2*np.sin(theta_2 - theta_1)) - dtheta_1*(mu_1 + mu_2) - ddx_c*(-l_1*m_2 - l_1*m_3 - l_1*m_4 - l_1*m_5 - m_1*r_1)*np.cos(theta_1) - g*(-l_1*m_2 - l_1*m_3 - l_1*m_4 - l_1*m_5 - m_1*r_1)*np.sin(theta_1))*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2))/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2) - (-l_1**2*(l_4*m_5 + m_4*r_4)*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_4 - theta_1)*np.cos(theta_3 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_3*(l_4*m_5 + m_4*r_4)*np.cos(theta_4 - theta_3) - (-l_1**2*(l_4*m_5 + m_4*r_4)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_4 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_4*m_5 + m_4*r_4)*np.cos(theta_4 - theta_2))*(-l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_3 - theta_2))/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2))*(dtheta_5**2*l_3*m_5*r_5*np.sin(theta_5 - theta_3) + dtheta_4*mu_4 + dtheta_4*(dtheta_4*l_3*l_4*m_5*np.sin(theta_4 - theta_3) + dtheta_4*l_3*m_4*r_4*np.sin(theta_4 - theta_3)) - dtheta_3*(mu_3 + mu_4) + dtheta_2*mu_3 + dtheta_2*(-dtheta_2*l_2*l_3*m_4*np.sin(theta_3 - theta_2) - dtheta_2*l_2*l_3*m_5*np.sin(theta_3 - theta_2) - dtheta_2*l_2*m_3*r_3*np.sin(theta_3 - theta_2)) + dtheta_1*(-dtheta_1*l_1*l_3*m_4*np.sin(theta_3 - theta_1) - dtheta_1*l_1*l_3*m_5*np.sin(theta_3 - theta_1) - dtheta_1*l_1*m_3*r_3*np.sin(theta_3 - theta_1)) - ddx_c*(-l_3*m_4 - l_3*m_5 - m_3*r_3)*np.cos(theta_3) - g*(-l_3*m_4 - l_3*m_5 - m_3*r_3)*np.sin(theta_3) - l_1*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(dtheta_5**2*l_1*m_5*r_5*np.sin(theta_5 - theta_1) + dtheta_4*(dtheta_4*l_1*l_4*m_5*np.sin(theta_4 - theta_1) + dtheta_4*l_1*m_4*r_4*np.sin(theta_4 - theta_1)) + dtheta_3*(dtheta_3*l_1*l_3*m_4*np.sin(theta_3 - theta_1) + dtheta_3*l_1*l_3*m_5*np.sin(theta_3 - theta_1) + dtheta_3*l_1*m_3*r_3*np.sin(theta_3 - theta_1)) + dtheta_2*mu_2 + dtheta_2*(dtheta_2*l_1*l_2*m_3*np.sin(theta_2 - theta_1) + dtheta_2*l_1*l_2*m_4*np.sin(theta_2 - theta_1) + dtheta_2*l_1*l_2*m_5*np.sin(theta_2 - theta_1) + dtheta_2*l_1*m_2*r_2*np.sin(theta_2 - theta_1)) - dtheta_1*(mu_1 + mu_2) - ddx_c*(-l_1*m_2 - l_1*m_3 - l_1*m_4 - l_1*m_5 - m_1*r_1)*np.cos(theta_1) - g*(-l_1*m_2 - l_1*m_3 - l_1*m_4 - l_1*m_5 - m_1*r_1)*np.sin(theta_1))*np.cos(theta_3 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) - (-l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_3 - theta_2))*(dtheta_5**2*l_2*m_5*r_5*np.sin(theta_5 - theta_2) + dtheta_4*(dtheta_4*l_2*l_4*m_5*np.sin(theta_4 - theta_2) + dtheta_4*l_2*m_4*r_4*np.sin(theta_4 - theta_2)) + dtheta_3*mu_3 + dtheta_3*(dtheta_3*l_2*l_3*m_4*np.sin(theta_3 - theta_2) + dtheta_3*l_2*l_3*m_5*np.sin(theta_3 - theta_2) + dtheta_3*l_2*m_3*r_3*np.sin(theta_3 - theta_2)) - dtheta_2*(mu_2 + mu_3) + dtheta_1*mu_2 + dtheta_1*(-dtheta_1*l_1*l_2*m_3*np.sin(theta_2 - theta_1) - dtheta_1*l_1*l_2*m_4*np.sin(theta_2 - theta_1) - dtheta_1*l_1*l_2*m_5*np.sin(theta_2 - theta_1) - dtheta_1*l_1*m_2*r_2*np.sin(theta_2 - theta_1)) - ddx_c*(-l_2*m_3 - l_2*m_4 - l_2*m_5 - m_2*r_2)*np.cos(theta_2) - g*(-l_2*m_3 - l_2*m_4 - l_2*m_5 - m_2*r_2)*np.sin(theta_2) - l_1*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*(dtheta_5**2*l_1*m_5*r_5*np.sin(theta_5 - theta_1) + dtheta_4*(dtheta_4*l_1*l_4*m_5*np.sin(theta_4 - theta_1) + dtheta_4*l_1*m_4*r_4*np.sin(theta_4 - theta_1)) + dtheta_3*(dtheta_3*l_1*l_3*m_4*np.sin(theta_3 - theta_1) + dtheta_3*l_1*l_3*m_5*np.sin(theta_3 - theta_1) + dtheta_3*l_1*m_3*r_3*np.sin(theta_3 - theta_1)) + dtheta_2*mu_2 + dtheta_2*(dtheta_2*l_1*l_2*m_3*np.sin(theta_2 - theta_1) + dtheta_2*l_1*l_2*m_4*np.sin(theta_2 - theta_1) + dtheta_2*l_1*l_2*m_5*np.sin(theta_2 - theta_1) + dtheta_2*l_1*m_2*r_2*np.sin(theta_2 - theta_1)) - dtheta_1*(mu_1 + mu_2) - ddx_c*(-l_1*m_2 - l_1*m_3 - l_1*m_4 - l_1*m_5 - m_1*r_1)*np.cos(theta_1) - g*(-l_1*m_2 - l_1*m_3 - l_1*m_4 - l_1*m_5 - m_1*r_1)*np.sin(theta_1))*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2))/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2))/(I_3 - l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)**2*np.cos(theta_3 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_3**2*m_4 + l_3**2*m_5 + m_3*r_3**2 - (-l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_3 - theta_2))**2/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2)))/(I_4 - l_1**2*(l_4*m_5 + m_4*r_4)**2*np.cos(theta_4 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_4**2*m_5 + m_4*r_4**2 - (-l_1**2*(l_4*m_5 + m_4*r_4)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_4 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_4*m_5 + m_4*r_4)*np.cos(theta_4 - theta_2))**2/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2) - (-l_1**2*(l_4*m_5 + m_4*r_4)*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_4 - theta_1)*np.cos(theta_3 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_3*(l_4*m_5 + m_4*r_4)*np.cos(theta_4 - theta_3) - (-l_1**2*(l_4*m_5 + m_4*r_4)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_4 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_4*m_5 + m_4*r_4)*np.cos(theta_4 - theta_2))*(-l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_3 - theta_2))/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2))**2/(I_3 - l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)**2*np.cos(theta_3 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_3**2*m_4 + l_3**2*m_5 + m_3*r_3**2 - (-l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_3 - theta_2))**2/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2))))/(I_5 - l_1**2*m_5**2*r_5**2*np.cos(theta_5 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + m_5*r_5**2 - (-l_1**2*m_5*r_5*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_5 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*m_5*r_5*np.cos(theta_5 - theta_2))**2/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2) - (-l_1**2*m_5*r_5*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_5 - theta_1)*np.cos(theta_3 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_3*m_5*r_5*np.cos(theta_5 - theta_3) - (-l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_3 - theta_2))*(-l_1**2*m_5*r_5*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_5 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*m_5*r_5*np.cos(theta_5 - theta_2))/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2))**2/(I_3 - l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)**2*np.cos(theta_3 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_3**2*m_4 + l_3**2*m_5 + m_3*r_3**2 - (-l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_3 - theta_2))**2/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2)) - (-l_1**2*m_5*r_5*(l_4*m_5 + m_4*r_4)*np.cos(theta_5 - theta_1)*np.cos(theta_4 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_4*m_5*r_5*np.cos(theta_5 - theta_4) - (-l_1**2*(l_4*m_5 + m_4*r_4)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_4 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_4*m_5 + m_4*r_4)*np.cos(theta_4 - theta_2))*(-l_1**2*m_5*r_5*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_5 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*m_5*r_5*np.cos(theta_5 - theta_2))/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2) - (-l_1**2*(l_4*m_5 + m_4*r_4)*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_4 - theta_1)*np.cos(theta_3 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_3*(l_4*m_5 + m_4*r_4)*np.cos(theta_4 - theta_3) - (-l_1**2*(l_4*m_5 + m_4*r_4)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_4 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_4*m_5 + m_4*r_4)*np.cos(theta_4 - theta_2))*(-l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_3 - theta_2))/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2))*(-l_1**2*m_5*r_5*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_5 - theta_1)*np.cos(theta_3 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_3*m_5*r_5*np.cos(theta_5 - theta_3) - (-l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_3 - theta_2))*(-l_1**2*m_5*r_5*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_5 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*m_5*r_5*np.cos(theta_5 - theta_2))/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2))/(I_3 - l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)**2*np.cos(theta_3 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_3**2*m_4 + l_3**2*m_5 + m_3*r_3**2 - (-l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_3 - theta_2))**2/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2)))**2/(I_4 - l_1**2*(l_4*m_5 + m_4*r_4)**2*np.cos(theta_4 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_4**2*m_5 + m_4*r_4**2 - (-l_1**2*(l_4*m_5 + m_4*r_4)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_4 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_4*m_5 + m_4*r_4)*np.cos(theta_4 - theta_2))**2/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2) - (-l_1**2*(l_4*m_5 + m_4*r_4)*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_4 - theta_1)*np.cos(theta_3 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_3*(l_4*m_5 + m_4*r_4)*np.cos(theta_4 - theta_3) - (-l_1**2*(l_4*m_5 + m_4*r_4)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_4 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_4*m_5 + m_4*r_4)*np.cos(theta_4 - theta_2))*(-l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_3 - theta_2))/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2))**2/(I_3 - l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)**2*np.cos(theta_3 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_3**2*m_4 + l_3**2*m_5 + m_3*r_3**2 - (-l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_3 - theta_2))**2/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2)))))*np.cos(theta_2 - theta_1)/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2))/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2),



                      (dtheta_5**2*l_2*m_5*r_5*np.sin(theta_5 - theta_2) + dtheta_4*(dtheta_4*l_2*l_4*m_5*np.sin(theta_4 - theta_2) + dtheta_4*l_2*m_4*r_4*np.sin(theta_4 - theta_2)) + dtheta_3*mu_3 + dtheta_3*(dtheta_3*l_2*l_3*m_4*np.sin(theta_3 - theta_2) + dtheta_3*l_2*l_3*m_5*np.sin(theta_3 - theta_2) + dtheta_3*l_2*m_3*r_3*np.sin(theta_3 - theta_2)) - dtheta_2*(mu_2 + mu_3) + dtheta_1*mu_2 + dtheta_1*(-dtheta_1*l_1*l_2*m_3*np.sin(theta_2 - theta_1) - dtheta_1*l_1*l_2*m_4*np.sin(theta_2 - theta_1) - dtheta_1*l_1*l_2*m_5*np.sin(theta_2 - theta_1) - dtheta_1*l_1*m_2*r_2*np.sin(theta_2 - theta_1)) - ddx_c*(-l_2*m_3 - l_2*m_4 - l_2*m_5 - m_2*r_2)*np.cos(theta_2) - g*(-l_2*m_3 - l_2*m_4 - l_2*m_5 - m_2*r_2)*np.sin(theta_2) - l_1*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*(dtheta_5**2*l_1*m_5*r_5*np.sin(theta_5 - theta_1) + dtheta_4*(dtheta_4*l_1*l_4*m_5*np.sin(theta_4 - theta_1) + dtheta_4*l_1*m_4*r_4*np.sin(theta_4 - theta_1)) + dtheta_3*(dtheta_3*l_1*l_3*m_4*np.sin(theta_3 - theta_1) + dtheta_3*l_1*l_3*m_5*np.sin(theta_3 - theta_1) + dtheta_3*l_1*m_3*r_3*np.sin(theta_3 - theta_1)) + dtheta_2*mu_2 + dtheta_2*(dtheta_2*l_1*l_2*m_3*np.sin(theta_2 - theta_1) + dtheta_2*l_1*l_2*m_4*np.sin(theta_2 - theta_1) + dtheta_2*l_1*l_2*m_5*np.sin(theta_2 - theta_1) + dtheta_2*l_1*m_2*r_2*np.sin(theta_2 - theta_1)) - dtheta_1*(mu_1 + mu_2) - ddx_c*(-l_1*m_2 - l_1*m_3 - l_1*m_4 - l_1*m_5 - m_1*r_1)*np.cos(theta_1) - g*(-l_1*m_2 - l_1*m_3 - l_1*m_4 - l_1*m_5 - m_1*r_1)*np.sin(theta_1))*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) - (-l_1**2*(l_4*m_5 + m_4*r_4)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_4 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_4*m_5 + m_4*r_4)*np.cos(theta_4 - theta_2))*(dtheta_5**2*l_4*m_5*r_5*np.sin(theta_5 - theta_4) + dtheta_5*mu_5 - dtheta_4*(mu_4 + mu_5) + dtheta_3*mu_4 + dtheta_3*(-dtheta_3*l_3*l_4*m_5*np.sin(theta_4 - theta_3) - dtheta_3*l_3*m_4*r_4*np.sin(theta_4 - theta_3)) + dtheta_2*(-dtheta_2*l_2*l_4*m_5*np.sin(theta_4 - theta_2) - dtheta_2*l_2*m_4*r_4*np.sin(theta_4 - theta_2)) + dtheta_1*(-dtheta_1*l_1*l_4*m_5*np.sin(theta_4 - theta_1) - dtheta_1*l_1*m_4*r_4*np.sin(theta_4 - theta_1)) - ddx_c*(-l_4*m_5 - m_4*r_4)*np.cos(theta_4) - g*(-l_4*m_5 - m_4*r_4)*np.sin(theta_4) - l_1*(l_4*m_5 + m_4*r_4)*(dtheta_5**2*l_1*m_5*r_5*np.sin(theta_5 - theta_1) + dtheta_4*(dtheta_4*l_1*l_4*m_5*np.sin(theta_4 - theta_1) + dtheta_4*l_1*m_4*r_4*np.sin(theta_4 - theta_1)) + dtheta_3*(dtheta_3*l_1*l_3*m_4*np.sin(theta_3 - theta_1) + dtheta_3*l_1*l_3*m_5*np.sin(theta_3 - theta_1) + dtheta_3*l_1*m_3*r_3*np.sin(theta_3 - theta_1)) + dtheta_2*mu_2 + dtheta_2*(dtheta_2*l_1*l_2*m_3*np.sin(theta_2 - theta_1) + dtheta_2*l_1*l_2*m_4*np.sin(theta_2 - theta_1) + dtheta_2*l_1*l_2*m_5*np.sin(theta_2 - theta_1) + dtheta_2*l_1*m_2*r_2*np.sin(theta_2 - theta_1)) - dtheta_1*(mu_1 + mu_2) - ddx_c*(-l_1*m_2 - l_1*m_3 - l_1*m_4 - l_1*m_5 - m_1*r_1)*np.cos(theta_1) - g*(-l_1*m_2 - l_1*m_3 - l_1*m_4 - l_1*m_5 - m_1*r_1)*np.sin(theta_1))*np.cos(theta_4 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) - (-l_1**2*(l_4*m_5 + m_4*r_4)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_4 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_4*m_5 + m_4*r_4)*np.cos(theta_4 - theta_2))*(dtheta_5**2*l_2*m_5*r_5*np.sin(theta_5 - theta_2) + dtheta_4*(dtheta_4*l_2*l_4*m_5*np.sin(theta_4 - theta_2) + dtheta_4*l_2*m_4*r_4*np.sin(theta_4 - theta_2)) + dtheta_3*mu_3 + dtheta_3*(dtheta_3*l_2*l_3*m_4*np.sin(theta_3 - theta_2) + dtheta_3*l_2*l_3*m_5*np.sin(theta_3 - theta_2) + dtheta_3*l_2*m_3*r_3*np.sin(theta_3 - theta_2)) - dtheta_2*(mu_2 + mu_3) + dtheta_1*mu_2 + dtheta_1*(-dtheta_1*l_1*l_2*m_3*np.sin(theta_2 - theta_1) - dtheta_1*l_1*l_2*m_4*np.sin(theta_2 - theta_1) - dtheta_1*l_1*l_2*m_5*np.sin(theta_2 - theta_1) - dtheta_1*l_1*m_2*r_2*np.sin(theta_2 - theta_1)) - ddx_c*(-l_2*m_3 - l_2*m_4 - l_2*m_5 - m_2*r_2)*np.cos(theta_2) - g*(-l_2*m_3 - l_2*m_4 - l_2*m_5 - m_2*r_2)*np.sin(theta_2) - l_1*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*(dtheta_5**2*l_1*m_5*r_5*np.sin(theta_5 - theta_1) + dtheta_4*(dtheta_4*l_1*l_4*m_5*np.sin(theta_4 - theta_1) + dtheta_4*l_1*m_4*r_4*np.sin(theta_4 - theta_1)) + dtheta_3*(dtheta_3*l_1*l_3*m_4*np.sin(theta_3 - theta_1) + dtheta_3*l_1*l_3*m_5*np.sin(theta_3 - theta_1) + dtheta_3*l_1*m_3*r_3*np.sin(theta_3 - theta_1)) + dtheta_2*mu_2 + dtheta_2*(dtheta_2*l_1*l_2*m_3*np.sin(theta_2 - theta_1) + dtheta_2*l_1*l_2*m_4*np.sin(theta_2 - theta_1) + dtheta_2*l_1*l_2*m_5*np.sin(theta_2 - theta_1) + dtheta_2*l_1*m_2*r_2*np.sin(theta_2 - theta_1)) - dtheta_1*(mu_1 + mu_2) - ddx_c*(-l_1*m_2 - l_1*m_3 - l_1*m_4 - l_1*m_5 - m_1*r_1)*np.cos(theta_1) - g*(-l_1*m_2 - l_1*m_3 - l_1*m_4 - l_1*m_5 - m_1*r_1)*np.sin(theta_1))*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2))/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2) - (-l_1**2*(l_4*m_5 + m_4*r_4)*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_4 - theta_1)*np.cos(theta_3 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_3*(l_4*m_5 + m_4*r_4)*np.cos(theta_4 - theta_3) - (-l_1**2*(l_4*m_5 + m_4*r_4)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_4 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_4*m_5 + m_4*r_4)*np.cos(theta_4 - theta_2))*(-l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_3 - theta_2))/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2))*(dtheta_5**2*l_3*m_5*r_5*np.sin(theta_5 - theta_3) + dtheta_4*mu_4 + dtheta_4*(dtheta_4*l_3*l_4*m_5*np.sin(theta_4 - theta_3) + dtheta_4*l_3*m_4*r_4*np.sin(theta_4 - theta_3)) - dtheta_3*(mu_3 + mu_4) + dtheta_2*mu_3 + dtheta_2*(-dtheta_2*l_2*l_3*m_4*np.sin(theta_3 - theta_2) - dtheta_2*l_2*l_3*m_5*np.sin(theta_3 - theta_2) - dtheta_2*l_2*m_3*r_3*np.sin(theta_3 - theta_2)) + dtheta_1*(-dtheta_1*l_1*l_3*m_4*np.sin(theta_3 - theta_1) - dtheta_1*l_1*l_3*m_5*np.sin(theta_3 - theta_1) - dtheta_1*l_1*m_3*r_3*np.sin(theta_3 - theta_1)) - ddx_c*(-l_3*m_4 - l_3*m_5 - m_3*r_3)*np.cos(theta_3) - g*(-l_3*m_4 - l_3*m_5 - m_3*r_3)*np.sin(theta_3) - l_1*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(dtheta_5**2*l_1*m_5*r_5*np.sin(theta_5 - theta_1) + dtheta_4*(dtheta_4*l_1*l_4*m_5*np.sin(theta_4 - theta_1) + dtheta_4*l_1*m_4*r_4*np.sin(theta_4 - theta_1)) + dtheta_3*(dtheta_3*l_1*l_3*m_4*np.sin(theta_3 - theta_1) + dtheta_3*l_1*l_3*m_5*np.sin(theta_3 - theta_1) + dtheta_3*l_1*m_3*r_3*np.sin(theta_3 - theta_1)) + dtheta_2*mu_2 + dtheta_2*(dtheta_2*l_1*l_2*m_3*np.sin(theta_2 - theta_1) + dtheta_2*l_1*l_2*m_4*np.sin(theta_2 - theta_1) + dtheta_2*l_1*l_2*m_5*np.sin(theta_2 - theta_1) + dtheta_2*l_1*m_2*r_2*np.sin(theta_2 - theta_1)) - dtheta_1*(mu_1 + mu_2) - ddx_c*(-l_1*m_2 - l_1*m_3 - l_1*m_4 - l_1*m_5 - m_1*r_1)*np.cos(theta_1) - g*(-l_1*m_2 - l_1*m_3 - l_1*m_4 - l_1*m_5 - m_1*r_1)*np.sin(theta_1))*np.cos(theta_3 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) - (-l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_3 - theta_2))*(dtheta_5**2*l_2*m_5*r_5*np.sin(theta_5 - theta_2) + dtheta_4*(dtheta_4*l_2*l_4*m_5*np.sin(theta_4 - theta_2) + dtheta_4*l_2*m_4*r_4*np.sin(theta_4 - theta_2)) + dtheta_3*mu_3 + dtheta_3*(dtheta_3*l_2*l_3*m_4*np.sin(theta_3 - theta_2) + dtheta_3*l_2*l_3*m_5*np.sin(theta_3 - theta_2) + dtheta_3*l_2*m_3*r_3*np.sin(theta_3 - theta_2)) - dtheta_2*(mu_2 + mu_3) + dtheta_1*mu_2 + dtheta_1*(-dtheta_1*l_1*l_2*m_3*np.sin(theta_2 - theta_1) - dtheta_1*l_1*l_2*m_4*np.sin(theta_2 - theta_1) - dtheta_1*l_1*l_2*m_5*np.sin(theta_2 - theta_1) - dtheta_1*l_1*m_2*r_2*np.sin(theta_2 - theta_1)) - ddx_c*(-l_2*m_3 - l_2*m_4 - l_2*m_5 - m_2*r_2)*np.cos(theta_2) - g*(-l_2*m_3 - l_2*m_4 - l_2*m_5 - m_2*r_2)*np.sin(theta_2) - l_1*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*(dtheta_5**2*l_1*m_5*r_5*np.sin(theta_5 - theta_1) + dtheta_4*(dtheta_4*l_1*l_4*m_5*np.sin(theta_4 - theta_1) + dtheta_4*l_1*m_4*r_4*np.sin(theta_4 - theta_1)) + dtheta_3*(dtheta_3*l_1*l_3*m_4*np.sin(theta_3 - theta_1) + dtheta_3*l_1*l_3*m_5*np.sin(theta_3 - theta_1) + dtheta_3*l_1*m_3*r_3*np.sin(theta_3 - theta_1)) + dtheta_2*mu_2 + dtheta_2*(dtheta_2*l_1*l_2*m_3*np.sin(theta_2 - theta_1) + dtheta_2*l_1*l_2*m_4*np.sin(theta_2 - theta_1) + dtheta_2*l_1*l_2*m_5*np.sin(theta_2 - theta_1) + dtheta_2*l_1*m_2*r_2*np.sin(theta_2 - theta_1)) - dtheta_1*(mu_1 + mu_2) - ddx_c*(-l_1*m_2 - l_1*m_3 - l_1*m_4 - l_1*m_5 - m_1*r_1)*np.cos(theta_1) - g*(-l_1*m_2 - l_1*m_3 - l_1*m_4 - l_1*m_5 - m_1*r_1)*np.sin(theta_1))*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2))/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2))/(I_3 - l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)**2*np.cos(theta_3 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_3**2*m_4 + l_3**2*m_5 + m_3*r_3**2 - (-l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_3 - theta_2))**2/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2)) - (-l_1**2*m_5*r_5*(l_4*m_5 + m_4*r_4)*np.cos(theta_5 - theta_1)*np.cos(theta_4 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_4*m_5*r_5*np.cos(theta_5 - theta_4) - (-l_1**2*(l_4*m_5 + m_4*r_4)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_4 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_4*m_5 + m_4*r_4)*np.cos(theta_4 - theta_2))*(-l_1**2*m_5*r_5*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_5 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*m_5*r_5*np.cos(theta_5 - theta_2))/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2) - (-l_1**2*(l_4*m_5 + m_4*r_4)*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_4 - theta_1)*np.cos(theta_3 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_3*(l_4*m_5 + m_4*r_4)*np.cos(theta_4 - theta_3) - (-l_1**2*(l_4*m_5 + m_4*r_4)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_4 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_4*m_5 + m_4*r_4)*np.cos(theta_4 - theta_2))*(-l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_3 - theta_2))/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2))*(-l_1**2*m_5*r_5*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_5 - theta_1)*np.cos(theta_3 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_3*m_5*r_5*np.cos(theta_5 - theta_3) - (-l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_3 - theta_2))*(-l_1**2*m_5*r_5*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_5 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*m_5*r_5*np.cos(theta_5 - theta_2))/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2))/(I_3 - l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)**2*np.cos(theta_3 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_3**2*m_4 + l_3**2*m_5 + m_3*r_3**2 - (-l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_3 - theta_2))**2/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2)))*(-dtheta_5*mu_5 - dtheta_4**2*l_4*m_5*r_5*np.sin(theta_5 - theta_4) + dtheta_4*mu_5 - dtheta_3**2*l_3*m_5*r_5*np.sin(theta_5 - theta_3) - dtheta_2**2*l_2*m_5*r_5*np.sin(theta_5 - theta_2) - dtheta_1**2*l_1*m_5*r_5*np.sin(theta_5 - theta_1) + ddx_c*m_5*r_5*np.cos(theta_5) + g*m_5*r_5*np.sin(theta_5) - l_1*m_5*r_5*(dtheta_5**2*l_1*m_5*r_5*np.sin(theta_5 - theta_1) + dtheta_4*(dtheta_4*l_1*l_4*m_5*np.sin(theta_4 - theta_1) + dtheta_4*l_1*m_4*r_4*np.sin(theta_4 - theta_1)) + dtheta_3*(dtheta_3*l_1*l_3*m_4*np.sin(theta_3 - theta_1) + dtheta_3*l_1*l_3*m_5*np.sin(theta_3 - theta_1) + dtheta_3*l_1*m_3*r_3*np.sin(theta_3 - theta_1)) + dtheta_2*mu_2 + dtheta_2*(dtheta_2*l_1*l_2*m_3*np.sin(theta_2 - theta_1) + dtheta_2*l_1*l_2*m_4*np.sin(theta_2 - theta_1) + dtheta_2*l_1*l_2*m_5*np.sin(theta_2 - theta_1) + dtheta_2*l_1*m_2*r_2*np.sin(theta_2 - theta_1)) - dtheta_1*(mu_1 + mu_2) - ddx_c*(-l_1*m_2 - l_1*m_3 - l_1*m_4 - l_1*m_5 - m_1*r_1)*np.cos(theta_1) - g*(-l_1*m_2 - l_1*m_3 - l_1*m_4 - l_1*m_5 - m_1*r_1)*np.sin(theta_1))*np.cos(theta_5 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) - (-l_1**2*m_5*r_5*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_5 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*m_5*r_5*np.cos(theta_5 - theta_2))*(dtheta_5**2*l_2*m_5*r_5*np.sin(theta_5 - theta_2) + dtheta_4*(dtheta_4*l_2*l_4*m_5*np.sin(theta_4 - theta_2) + dtheta_4*l_2*m_4*r_4*np.sin(theta_4 - theta_2)) + dtheta_3*mu_3 + dtheta_3*(dtheta_3*l_2*l_3*m_4*np.sin(theta_3 - theta_2) + dtheta_3*l_2*l_3*m_5*np.sin(theta_3 - theta_2) + dtheta_3*l_2*m_3*r_3*np.sin(theta_3 - theta_2)) - dtheta_2*(mu_2 + mu_3) + dtheta_1*mu_2 + dtheta_1*(-dtheta_1*l_1*l_2*m_3*np.sin(theta_2 - theta_1) - dtheta_1*l_1*l_2*m_4*np.sin(theta_2 - theta_1) - dtheta_1*l_1*l_2*m_5*np.sin(theta_2 - theta_1) - dtheta_1*l_1*m_2*r_2*np.sin(theta_2 - theta_1)) - ddx_c*(-l_2*m_3 - l_2*m_4 - l_2*m_5 - m_2*r_2)*np.cos(theta_2) - g*(-l_2*m_3 - l_2*m_4 - l_2*m_5 - m_2*r_2)*np.sin(theta_2) - l_1*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*(dtheta_5**2*l_1*m_5*r_5*np.sin(theta_5 - theta_1) + dtheta_4*(dtheta_4*l_1*l_4*m_5*np.sin(theta_4 - theta_1) + dtheta_4*l_1*m_4*r_4*np.sin(theta_4 - theta_1)) + dtheta_3*(dtheta_3*l_1*l_3*m_4*np.sin(theta_3 - theta_1) + dtheta_3*l_1*l_3*m_5*np.sin(theta_3 - theta_1) + dtheta_3*l_1*m_3*r_3*np.sin(theta_3 - theta_1)) + dtheta_2*mu_2 + dtheta_2*(dtheta_2*l_1*l_2*m_3*np.sin(theta_2 - theta_1) + dtheta_2*l_1*l_2*m_4*np.sin(theta_2 - theta_1) + dtheta_2*l_1*l_2*m_5*np.sin(theta_2 - theta_1) + dtheta_2*l_1*m_2*r_2*np.sin(theta_2 - theta_1)) - dtheta_1*(mu_1 + mu_2) - ddx_c*(-l_1*m_2 - l_1*m_3 - l_1*m_4 - l_1*m_5 - m_1*r_1)*np.cos(theta_1) - g*(-l_1*m_2 - l_1*m_3 - l_1*m_4 - l_1*m_5 - m_1*r_1)*np.sin(theta_1))*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2))/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2) - (-l_1**2*m_5*r_5*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_5 - theta_1)*np.cos(theta_3 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_3*m_5*r_5*np.cos(theta_5 - theta_3) - (-l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_3 - theta_2))*(-l_1**2*m_5*r_5*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_5 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*m_5*r_5*np.cos(theta_5 - theta_2))/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2))*(dtheta_5**2*l_3*m_5*r_5*np.sin(theta_5 - theta_3) + dtheta_4*mu_4 + dtheta_4*(dtheta_4*l_3*l_4*m_5*np.sin(theta_4 - theta_3) + dtheta_4*l_3*m_4*r_4*np.sin(theta_4 - theta_3)) - dtheta_3*(mu_3 + mu_4) + dtheta_2*mu_3 + dtheta_2*(-dtheta_2*l_2*l_3*m_4*np.sin(theta_3 - theta_2) - dtheta_2*l_2*l_3*m_5*np.sin(theta_3 - theta_2) - dtheta_2*l_2*m_3*r_3*np.sin(theta_3 - theta_2)) + dtheta_1*(-dtheta_1*l_1*l_3*m_4*np.sin(theta_3 - theta_1) - dtheta_1*l_1*l_3*m_5*np.sin(theta_3 - theta_1) - dtheta_1*l_1*m_3*r_3*np.sin(theta_3 - theta_1)) - ddx_c*(-l_3*m_4 - l_3*m_5 - m_3*r_3)*np.cos(theta_3) - g*(-l_3*m_4 - l_3*m_5 - m_3*r_3)*np.sin(theta_3) - l_1*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(dtheta_5**2*l_1*m_5*r_5*np.sin(theta_5 - theta_1) + dtheta_4*(dtheta_4*l_1*l_4*m_5*np.sin(theta_4 - theta_1) + dtheta_4*l_1*m_4*r_4*np.sin(theta_4 - theta_1)) + dtheta_3*(dtheta_3*l_1*l_3*m_4*np.sin(theta_3 - theta_1) + dtheta_3*l_1*l_3*m_5*np.sin(theta_3 - theta_1) + dtheta_3*l_1*m_3*r_3*np.sin(theta_3 - theta_1)) + dtheta_2*mu_2 + dtheta_2*(dtheta_2*l_1*l_2*m_3*np.sin(theta_2 - theta_1) + dtheta_2*l_1*l_2*m_4*np.sin(theta_2 - theta_1) + dtheta_2*l_1*l_2*m_5*np.sin(theta_2 - theta_1) + dtheta_2*l_1*m_2*r_2*np.sin(theta_2 - theta_1)) - dtheta_1*(mu_1 + mu_2) - ddx_c*(-l_1*m_2 - l_1*m_3 - l_1*m_4 - l_1*m_5 - m_1*r_1)*np.cos(theta_1) - g*(-l_1*m_2 - l_1*m_3 - l_1*m_4 - l_1*m_5 - m_1*r_1)*np.sin(theta_1))*np.cos(theta_3 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) - (-l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_3 - theta_2))*(dtheta_5**2*l_2*m_5*r_5*np.sin(theta_5 - theta_2) + dtheta_4*(dtheta_4*l_2*l_4*m_5*np.sin(theta_4 - theta_2) + dtheta_4*l_2*m_4*r_4*np.sin(theta_4 - theta_2)) + dtheta_3*mu_3 + dtheta_3*(dtheta_3*l_2*l_3*m_4*np.sin(theta_3 - theta_2) + dtheta_3*l_2*l_3*m_5*np.sin(theta_3 - theta_2) + dtheta_3*l_2*m_3*r_3*np.sin(theta_3 - theta_2)) - dtheta_2*(mu_2 + mu_3) + dtheta_1*mu_2 + dtheta_1*(-dtheta_1*l_1*l_2*m_3*np.sin(theta_2 - theta_1) - dtheta_1*l_1*l_2*m_4*np.sin(theta_2 - theta_1) - dtheta_1*l_1*l_2*m_5*np.sin(theta_2 - theta_1) - dtheta_1*l_1*m_2*r_2*np.sin(theta_2 - theta_1)) - ddx_c*(-l_2*m_3 - l_2*m_4 - l_2*m_5 - m_2*r_2)*np.cos(theta_2) - g*(-l_2*m_3 - l_2*m_4 - l_2*m_5 - m_2*r_2)*np.sin(theta_2) - l_1*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*(dtheta_5**2*l_1*m_5*r_5*np.sin(theta_5 - theta_1) + dtheta_4*(dtheta_4*l_1*l_4*m_5*np.sin(theta_4 - theta_1) + dtheta_4*l_1*m_4*r_4*np.sin(theta_4 - theta_1)) + dtheta_3*(dtheta_3*l_1*l_3*m_4*np.sin(theta_3 - theta_1) + dtheta_3*l_1*l_3*m_5*np.sin(theta_3 - theta_1) + dtheta_3*l_1*m_3*r_3*np.sin(theta_3 - theta_1)) + dtheta_2*mu_2 + dtheta_2*(dtheta_2*l_1*l_2*m_3*np.sin(theta_2 - theta_1) + dtheta_2*l_1*l_2*m_4*np.sin(theta_2 - theta_1) + dtheta_2*l_1*l_2*m_5*np.sin(theta_2 - theta_1) + dtheta_2*l_1*m_2*r_2*np.sin(theta_2 - theta_1)) - dtheta_1*(mu_1 + mu_2) - ddx_c*(-l_1*m_2 - l_1*m_3 - l_1*m_4 - l_1*m_5 - m_1*r_1)*np.cos(theta_1) - g*(-l_1*m_2 - l_1*m_3 - l_1*m_4 - l_1*m_5 - m_1*r_1)*np.sin(theta_1))*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2))/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2))/(I_3 - l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)**2*np.cos(theta_3 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_3**2*m_4 + l_3**2*m_5 + m_3*r_3**2 - (-l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_3 - theta_2))**2/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2)) - (-l_1**2*m_5*r_5*(l_4*m_5 + m_4*r_4)*np.cos(theta_5 - theta_1)*np.cos(theta_4 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_4*m_5*r_5*np.cos(theta_5 - theta_4) - (-l_1**2*(l_4*m_5 + m_4*r_4)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_4 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_4*m_5 + m_4*r_4)*np.cos(theta_4 - theta_2))*(-l_1**2*m_5*r_5*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_5 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*m_5*r_5*np.cos(theta_5 - theta_2))/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2) - (-l_1**2*(l_4*m_5 + m_4*r_4)*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_4 - theta_1)*np.cos(theta_3 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_3*(l_4*m_5 + m_4*r_4)*np.cos(theta_4 - theta_3) - (-l_1**2*(l_4*m_5 + m_4*r_4)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_4 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_4*m_5 + m_4*r_4)*np.cos(theta_4 - theta_2))*(-l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_3 - theta_2))/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2))*(-l_1**2*m_5*r_5*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_5 - theta_1)*np.cos(theta_3 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_3*m_5*r_5*np.cos(theta_5 - theta_3) - (-l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_3 - theta_2))*(-l_1**2*m_5*r_5*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_5 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*m_5*r_5*np.cos(theta_5 - theta_2))/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2))/(I_3 - l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)**2*np.cos(theta_3 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_3**2*m_4 + l_3**2*m_5 + m_3*r_3**2 - (-l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_3 - theta_2))**2/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2)))*(dtheta_5**2*l_4*m_5*r_5*np.sin(theta_5 - theta_4) + dtheta_5*mu_5 - dtheta_4*(mu_4 + mu_5) + dtheta_3*mu_4 + dtheta_3*(-dtheta_3*l_3*l_4*m_5*np.sin(theta_4 - theta_3) - dtheta_3*l_3*m_4*r_4*np.sin(theta_4 - theta_3)) + dtheta_2*(-dtheta_2*l_2*l_4*m_5*np.sin(theta_4 - theta_2) - dtheta_2*l_2*m_4*r_4*np.sin(theta_4 - theta_2)) + dtheta_1*(-dtheta_1*l_1*l_4*m_5*np.sin(theta_4 - theta_1) - dtheta_1*l_1*m_4*r_4*np.sin(theta_4 - theta_1)) - ddx_c*(-l_4*m_5 - m_4*r_4)*np.cos(theta_4) - g*(-l_4*m_5 - m_4*r_4)*np.sin(theta_4) - l_1*(l_4*m_5 + m_4*r_4)*(dtheta_5**2*l_1*m_5*r_5*np.sin(theta_5 - theta_1) + dtheta_4*(dtheta_4*l_1*l_4*m_5*np.sin(theta_4 - theta_1) + dtheta_4*l_1*m_4*r_4*np.sin(theta_4 - theta_1)) + dtheta_3*(dtheta_3*l_1*l_3*m_4*np.sin(theta_3 - theta_1) + dtheta_3*l_1*l_3*m_5*np.sin(theta_3 - theta_1) + dtheta_3*l_1*m_3*r_3*np.sin(theta_3 - theta_1)) + dtheta_2*mu_2 + dtheta_2*(dtheta_2*l_1*l_2*m_3*np.sin(theta_2 - theta_1) + dtheta_2*l_1*l_2*m_4*np.sin(theta_2 - theta_1) + dtheta_2*l_1*l_2*m_5*np.sin(theta_2 - theta_1) + dtheta_2*l_1*m_2*r_2*np.sin(theta_2 - theta_1)) - dtheta_1*(mu_1 + mu_2) - ddx_c*(-l_1*m_2 - l_1*m_3 - l_1*m_4 - l_1*m_5 - m_1*r_1)*np.cos(theta_1) - g*(-l_1*m_2 - l_1*m_3 - l_1*m_4 - l_1*m_5 - m_1*r_1)*np.sin(theta_1))*np.cos(theta_4 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) - (-l_1**2*(l_4*m_5 + m_4*r_4)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_4 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_4*m_5 + m_4*r_4)*np.cos(theta_4 - theta_2))*(dtheta_5**2*l_2*m_5*r_5*np.sin(theta_5 - theta_2) + dtheta_4*(dtheta_4*l_2*l_4*m_5*np.sin(theta_4 - theta_2) + dtheta_4*l_2*m_4*r_4*np.sin(theta_4 - theta_2)) + dtheta_3*mu_3 + dtheta_3*(dtheta_3*l_2*l_3*m_4*np.sin(theta_3 - theta_2) + dtheta_3*l_2*l_3*m_5*np.sin(theta_3 - theta_2) + dtheta_3*l_2*m_3*r_3*np.sin(theta_3 - theta_2)) - dtheta_2*(mu_2 + mu_3) + dtheta_1*mu_2 + dtheta_1*(-dtheta_1*l_1*l_2*m_3*np.sin(theta_2 - theta_1) - dtheta_1*l_1*l_2*m_4*np.sin(theta_2 - theta_1) - dtheta_1*l_1*l_2*m_5*np.sin(theta_2 - theta_1) - dtheta_1*l_1*m_2*r_2*np.sin(theta_2 - theta_1)) - ddx_c*(-l_2*m_3 - l_2*m_4 - l_2*m_5 - m_2*r_2)*np.cos(theta_2) - g*(-l_2*m_3 - l_2*m_4 - l_2*m_5 - m_2*r_2)*np.sin(theta_2) - l_1*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*(dtheta_5**2*l_1*m_5*r_5*np.sin(theta_5 - theta_1) + dtheta_4*(dtheta_4*l_1*l_4*m_5*np.sin(theta_4 - theta_1) + dtheta_4*l_1*m_4*r_4*np.sin(theta_4 - theta_1)) + dtheta_3*(dtheta_3*l_1*l_3*m_4*np.sin(theta_3 - theta_1) + dtheta_3*l_1*l_3*m_5*np.sin(theta_3 - theta_1) + dtheta_3*l_1*m_3*r_3*np.sin(theta_3 - theta_1)) + dtheta_2*mu_2 + dtheta_2*(dtheta_2*l_1*l_2*m_3*np.sin(theta_2 - theta_1) + dtheta_2*l_1*l_2*m_4*np.sin(theta_2 - theta_1) + dtheta_2*l_1*l_2*m_5*np.sin(theta_2 - theta_1) + dtheta_2*l_1*m_2*r_2*np.sin(theta_2 - theta_1)) - dtheta_1*(mu_1 + mu_2) - ddx_c*(-l_1*m_2 - l_1*m_3 - l_1*m_4 - l_1*m_5 - m_1*r_1)*np.cos(theta_1) - g*(-l_1*m_2 - l_1*m_3 - l_1*m_4 - l_1*m_5 - m_1*r_1)*np.sin(theta_1))*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2))/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2) - (-l_1**2*(l_4*m_5 + m_4*r_4)*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_4 - theta_1)*np.cos(theta_3 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_3*(l_4*m_5 + m_4*r_4)*np.cos(theta_4 - theta_3) - (-l_1**2*(l_4*m_5 + m_4*r_4)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_4 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_4*m_5 + m_4*r_4)*np.cos(theta_4 - theta_2))*(-l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_3 - theta_2))/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2))*(dtheta_5**2*l_3*m_5*r_5*np.sin(theta_5 - theta_3) + dtheta_4*mu_4 + dtheta_4*(dtheta_4*l_3*l_4*m_5*np.sin(theta_4 - theta_3) + dtheta_4*l_3*m_4*r_4*np.sin(theta_4 - theta_3)) - dtheta_3*(mu_3 + mu_4) + dtheta_2*mu_3 + dtheta_2*(-dtheta_2*l_2*l_3*m_4*np.sin(theta_3 - theta_2) - dtheta_2*l_2*l_3*m_5*np.sin(theta_3 - theta_2) - dtheta_2*l_2*m_3*r_3*np.sin(theta_3 - theta_2)) + dtheta_1*(-dtheta_1*l_1*l_3*m_4*np.sin(theta_3 - theta_1) - dtheta_1*l_1*l_3*m_5*np.sin(theta_3 - theta_1) - dtheta_1*l_1*m_3*r_3*np.sin(theta_3 - theta_1)) - ddx_c*(-l_3*m_4 - l_3*m_5 - m_3*r_3)*np.cos(theta_3) - g*(-l_3*m_4 - l_3*m_5 - m_3*r_3)*np.sin(theta_3) - l_1*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(dtheta_5**2*l_1*m_5*r_5*np.sin(theta_5 - theta_1) + dtheta_4*(dtheta_4*l_1*l_4*m_5*np.sin(theta_4 - theta_1) + dtheta_4*l_1*m_4*r_4*np.sin(theta_4 - theta_1)) + dtheta_3*(dtheta_3*l_1*l_3*m_4*np.sin(theta_3 - theta_1) + dtheta_3*l_1*l_3*m_5*np.sin(theta_3 - theta_1) + dtheta_3*l_1*m_3*r_3*np.sin(theta_3 - theta_1)) + dtheta_2*mu_2 + dtheta_2*(dtheta_2*l_1*l_2*m_3*np.sin(theta_2 - theta_1) + dtheta_2*l_1*l_2*m_4*np.sin(theta_2 - theta_1) + dtheta_2*l_1*l_2*m_5*np.sin(theta_2 - theta_1) + dtheta_2*l_1*m_2*r_2*np.sin(theta_2 - theta_1)) - dtheta_1*(mu_1 + mu_2) - ddx_c*(-l_1*m_2 - l_1*m_3 - l_1*m_4 - l_1*m_5 - m_1*r_1)*np.cos(theta_1) - g*(-l_1*m_2 - l_1*m_3 - l_1*m_4 - l_1*m_5 - m_1*r_1)*np.sin(theta_1))*np.cos(theta_3 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) - (-l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_3 - theta_2))*(dtheta_5**2*l_2*m_5*r_5*np.sin(theta_5 - theta_2) + dtheta_4*(dtheta_4*l_2*l_4*m_5*np.sin(theta_4 - theta_2) + dtheta_4*l_2*m_4*r_4*np.sin(theta_4 - theta_2)) + dtheta_3*mu_3 + dtheta_3*(dtheta_3*l_2*l_3*m_4*np.sin(theta_3 - theta_2) + dtheta_3*l_2*l_3*m_5*np.sin(theta_3 - theta_2) + dtheta_3*l_2*m_3*r_3*np.sin(theta_3 - theta_2)) - dtheta_2*(mu_2 + mu_3) + dtheta_1*mu_2 + dtheta_1*(-dtheta_1*l_1*l_2*m_3*np.sin(theta_2 - theta_1) - dtheta_1*l_1*l_2*m_4*np.sin(theta_2 - theta_1) - dtheta_1*l_1*l_2*m_5*np.sin(theta_2 - theta_1) - dtheta_1*l_1*m_2*r_2*np.sin(theta_2 - theta_1)) - ddx_c*(-l_2*m_3 - l_2*m_4 - l_2*m_5 - m_2*r_2)*np.cos(theta_2) - g*(-l_2*m_3 - l_2*m_4 - l_2*m_5 - m_2*r_2)*np.sin(theta_2) - l_1*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*(dtheta_5**2*l_1*m_5*r_5*np.sin(theta_5 - theta_1) + dtheta_4*(dtheta_4*l_1*l_4*m_5*np.sin(theta_4 - theta_1) + dtheta_4*l_1*m_4*r_4*np.sin(theta_4 - theta_1)) + dtheta_3*(dtheta_3*l_1*l_3*m_4*np.sin(theta_3 - theta_1) + dtheta_3*l_1*l_3*m_5*np.sin(theta_3 - theta_1) + dtheta_3*l_1*m_3*r_3*np.sin(theta_3 - theta_1)) + dtheta_2*mu_2 + dtheta_2*(dtheta_2*l_1*l_2*m_3*np.sin(theta_2 - theta_1) + dtheta_2*l_1*l_2*m_4*np.sin(theta_2 - theta_1) + dtheta_2*l_1*l_2*m_5*np.sin(theta_2 - theta_1) + dtheta_2*l_1*m_2*r_2*np.sin(theta_2 - theta_1)) - dtheta_1*(mu_1 + mu_2) - ddx_c*(-l_1*m_2 - l_1*m_3 - l_1*m_4 - l_1*m_5 - m_1*r_1)*np.cos(theta_1) - g*(-l_1*m_2 - l_1*m_3 - l_1*m_4 - l_1*m_5 - m_1*r_1)*np.sin(theta_1))*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2))/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2))/(I_3 - l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)**2*np.cos(theta_3 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_3**2*m_4 + l_3**2*m_5 + m_3*r_3**2 - (-l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_3 - theta_2))**2/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2)))/(I_4 - l_1**2*(l_4*m_5 + m_4*r_4)**2*np.cos(theta_4 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_4**2*m_5 + m_4*r_4**2 - (-l_1**2*(l_4*m_5 + m_4*r_4)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_4 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_4*m_5 + m_4*r_4)*np.cos(theta_4 - theta_2))**2/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2) - (-l_1**2*(l_4*m_5 + m_4*r_4)*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_4 - theta_1)*np.cos(theta_3 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_3*(l_4*m_5 + m_4*r_4)*np.cos(theta_4 - theta_3) - (-l_1**2*(l_4*m_5 + m_4*r_4)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_4 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_4*m_5 + m_4*r_4)*np.cos(theta_4 - theta_2))*(-l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_3 - theta_2))/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2))**2/(I_3 - l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)**2*np.cos(theta_3 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_3**2*m_4 + l_3**2*m_5 + m_3*r_3**2 - (-l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_3 - theta_2))**2/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2))))/(I_5 - l_1**2*m_5**2*r_5**2*np.cos(theta_5 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + m_5*r_5**2 - (-l_1**2*m_5*r_5*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_5 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*m_5*r_5*np.cos(theta_5 - theta_2))**2/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2) - (-l_1**2*m_5*r_5*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_5 - theta_1)*np.cos(theta_3 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_3*m_5*r_5*np.cos(theta_5 - theta_3) - (-l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_3 - theta_2))*(-l_1**2*m_5*r_5*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_5 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*m_5*r_5*np.cos(theta_5 - theta_2))/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2))**2/(I_3 - l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)**2*np.cos(theta_3 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_3**2*m_4 + l_3**2*m_5 + m_3*r_3**2 - (-l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_3 - theta_2))**2/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2)) - (-l_1**2*m_5*r_5*(l_4*m_5 + m_4*r_4)*np.cos(theta_5 - theta_1)*np.cos(theta_4 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_4*m_5*r_5*np.cos(theta_5 - theta_4) - (-l_1**2*(l_4*m_5 + m_4*r_4)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_4 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_4*m_5 + m_4*r_4)*np.cos(theta_4 - theta_2))*(-l_1**2*m_5*r_5*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_5 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*m_5*r_5*np.cos(theta_5 - theta_2))/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2) - (-l_1**2*(l_4*m_5 + m_4*r_4)*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_4 - theta_1)*np.cos(theta_3 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_3*(l_4*m_5 + m_4*r_4)*np.cos(theta_4 - theta_3) - (-l_1**2*(l_4*m_5 + m_4*r_4)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_4 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_4*m_5 + m_4*r_4)*np.cos(theta_4 - theta_2))*(-l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_3 - theta_2))/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2))*(-l_1**2*m_5*r_5*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_5 - theta_1)*np.cos(theta_3 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_3*m_5*r_5*np.cos(theta_5 - theta_3) - (-l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_3 - theta_2))*(-l_1**2*m_5*r_5*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_5 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*m_5*r_5*np.cos(theta_5 - theta_2))/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2))/(I_3 - l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)**2*np.cos(theta_3 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_3**2*m_4 + l_3**2*m_5 + m_3*r_3**2 - (-l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_3 - theta_2))**2/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2)))**2/(I_4 - l_1**2*(l_4*m_5 + m_4*r_4)**2*np.cos(theta_4 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_4**2*m_5 + m_4*r_4**2 - (-l_1**2*(l_4*m_5 + m_4*r_4)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_4 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_4*m_5 + m_4*r_4)*np.cos(theta_4 - theta_2))**2/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2) - (-l_1**2*(l_4*m_5 + m_4*r_4)*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_4 - theta_1)*np.cos(theta_3 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_3*(l_4*m_5 + m_4*r_4)*np.cos(theta_4 - theta_3) - (-l_1**2*(l_4*m_5 + m_4*r_4)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_4 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_4*m_5 + m_4*r_4)*np.cos(theta_4 - theta_2))*(-l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_3 - theta_2))/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2))**2/(I_3 - l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)**2*np.cos(theta_3 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_3**2*m_4 + l_3**2*m_5 + m_3*r_3**2 - (-l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_3 - theta_2))**2/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2)))))/(I_4 - l_1**2*(l_4*m_5 + m_4*r_4)**2*np.cos(theta_4 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_4**2*m_5 + m_4*r_4**2 - (-l_1**2*(l_4*m_5 + m_4*r_4)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_4 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_4*m_5 + m_4*r_4)*np.cos(theta_4 - theta_2))**2/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2) - (-l_1**2*(l_4*m_5 + m_4*r_4)*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_4 - theta_1)*np.cos(theta_3 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_3*(l_4*m_5 + m_4*r_4)*np.cos(theta_4 - theta_3) - (-l_1**2*(l_4*m_5 + m_4*r_4)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_4 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_4*m_5 + m_4*r_4)*np.cos(theta_4 - theta_2))*(-l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_3 - theta_2))/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2))**2/(I_3 - l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)**2*np.cos(theta_3 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_3**2*m_4 + l_3**2*m_5 + m_3*r_3**2 - (-l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_3 - theta_2))**2/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2))) - (-l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_3 - theta_2))*(dtheta_5**2*l_3*m_5*r_5*np.sin(theta_5 - theta_3) + dtheta_4*mu_4 + dtheta_4*(dtheta_4*l_3*l_4*m_5*np.sin(theta_4 - theta_3) + dtheta_4*l_3*m_4*r_4*np.sin(theta_4 - theta_3)) - dtheta_3*(mu_3 + mu_4) + dtheta_2*mu_3 + dtheta_2*(-dtheta_2*l_2*l_3*m_4*np.sin(theta_3 - theta_2) - dtheta_2*l_2*l_3*m_5*np.sin(theta_3 - theta_2) - dtheta_2*l_2*m_3*r_3*np.sin(theta_3 - theta_2)) + dtheta_1*(-dtheta_1*l_1*l_3*m_4*np.sin(theta_3 - theta_1) - dtheta_1*l_1*l_3*m_5*np.sin(theta_3 - theta_1) - dtheta_1*l_1*m_3*r_3*np.sin(theta_3 - theta_1)) - ddx_c*(-l_3*m_4 - l_3*m_5 - m_3*r_3)*np.cos(theta_3) - g*(-l_3*m_4 - l_3*m_5 - m_3*r_3)*np.sin(theta_3) - l_1*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(dtheta_5**2*l_1*m_5*r_5*np.sin(theta_5 - theta_1) + dtheta_4*(dtheta_4*l_1*l_4*m_5*np.sin(theta_4 - theta_1) + dtheta_4*l_1*m_4*r_4*np.sin(theta_4 - theta_1)) + dtheta_3*(dtheta_3*l_1*l_3*m_4*np.sin(theta_3 - theta_1) + dtheta_3*l_1*l_3*m_5*np.sin(theta_3 - theta_1) + dtheta_3*l_1*m_3*r_3*np.sin(theta_3 - theta_1)) + dtheta_2*mu_2 + dtheta_2*(dtheta_2*l_1*l_2*m_3*np.sin(theta_2 - theta_1) + dtheta_2*l_1*l_2*m_4*np.sin(theta_2 - theta_1) + dtheta_2*l_1*l_2*m_5*np.sin(theta_2 - theta_1) + dtheta_2*l_1*m_2*r_2*np.sin(theta_2 - theta_1)) - dtheta_1*(mu_1 + mu_2) - ddx_c*(-l_1*m_2 - l_1*m_3 - l_1*m_4 - l_1*m_5 - m_1*r_1)*np.cos(theta_1) - g*(-l_1*m_2 - l_1*m_3 - l_1*m_4 - l_1*m_5 - m_1*r_1)*np.sin(theta_1))*np.cos(theta_3 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) - (-l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_3 - theta_2))*(dtheta_5**2*l_2*m_5*r_5*np.sin(theta_5 - theta_2) + dtheta_4*(dtheta_4*l_2*l_4*m_5*np.sin(theta_4 - theta_2) + dtheta_4*l_2*m_4*r_4*np.sin(theta_4 - theta_2)) + dtheta_3*mu_3 + dtheta_3*(dtheta_3*l_2*l_3*m_4*np.sin(theta_3 - theta_2) + dtheta_3*l_2*l_3*m_5*np.sin(theta_3 - theta_2) + dtheta_3*l_2*m_3*r_3*np.sin(theta_3 - theta_2)) - dtheta_2*(mu_2 + mu_3) + dtheta_1*mu_2 + dtheta_1*(-dtheta_1*l_1*l_2*m_3*np.sin(theta_2 - theta_1) - dtheta_1*l_1*l_2*m_4*np.sin(theta_2 - theta_1) - dtheta_1*l_1*l_2*m_5*np.sin(theta_2 - theta_1) - dtheta_1*l_1*m_2*r_2*np.sin(theta_2 - theta_1)) - ddx_c*(-l_2*m_3 - l_2*m_4 - l_2*m_5 - m_2*r_2)*np.cos(theta_2) - g*(-l_2*m_3 - l_2*m_4 - l_2*m_5 - m_2*r_2)*np.sin(theta_2) - l_1*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*(dtheta_5**2*l_1*m_5*r_5*np.sin(theta_5 - theta_1) + dtheta_4*(dtheta_4*l_1*l_4*m_5*np.sin(theta_4 - theta_1) + dtheta_4*l_1*m_4*r_4*np.sin(theta_4 - theta_1)) + dtheta_3*(dtheta_3*l_1*l_3*m_4*np.sin(theta_3 - theta_1) + dtheta_3*l_1*l_3*m_5*np.sin(theta_3 - theta_1) + dtheta_3*l_1*m_3*r_3*np.sin(theta_3 - theta_1)) + dtheta_2*mu_2 + dtheta_2*(dtheta_2*l_1*l_2*m_3*np.sin(theta_2 - theta_1) + dtheta_2*l_1*l_2*m_4*np.sin(theta_2 - theta_1) + dtheta_2*l_1*l_2*m_5*np.sin(theta_2 - theta_1) + dtheta_2*l_1*m_2*r_2*np.sin(theta_2 - theta_1)) - dtheta_1*(mu_1 + mu_2) - ddx_c*(-l_1*m_2 - l_1*m_3 - l_1*m_4 - l_1*m_5 - m_1*r_1)*np.cos(theta_1) - g*(-l_1*m_2 - l_1*m_3 - l_1*m_4 - l_1*m_5 - m_1*r_1)*np.sin(theta_1))*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2))/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2) - (-l_1**2*(l_4*m_5 + m_4*r_4)*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_4 - theta_1)*np.cos(theta_3 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_3*(l_4*m_5 + m_4*r_4)*np.cos(theta_4 - theta_3) - (-l_1**2*(l_4*m_5 + m_4*r_4)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_4 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_4*m_5 + m_4*r_4)*np.cos(theta_4 - theta_2))*(-l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_3 - theta_2))/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2))*(dtheta_5**2*l_4*m_5*r_5*np.sin(theta_5 - theta_4) + dtheta_5*mu_5 - dtheta_4*(mu_4 + mu_5) + dtheta_3*mu_4 + dtheta_3*(-dtheta_3*l_3*l_4*m_5*np.sin(theta_4 - theta_3) - dtheta_3*l_3*m_4*r_4*np.sin(theta_4 - theta_3)) + dtheta_2*(-dtheta_2*l_2*l_4*m_5*np.sin(theta_4 - theta_2) - dtheta_2*l_2*m_4*r_4*np.sin(theta_4 - theta_2)) + dtheta_1*(-dtheta_1*l_1*l_4*m_5*np.sin(theta_4 - theta_1) - dtheta_1*l_1*m_4*r_4*np.sin(theta_4 - theta_1)) - ddx_c*(-l_4*m_5 - m_4*r_4)*np.cos(theta_4) - g*(-l_4*m_5 - m_4*r_4)*np.sin(theta_4) - l_1*(l_4*m_5 + m_4*r_4)*(dtheta_5**2*l_1*m_5*r_5*np.sin(theta_5 - theta_1) + dtheta_4*(dtheta_4*l_1*l_4*m_5*np.sin(theta_4 - theta_1) + dtheta_4*l_1*m_4*r_4*np.sin(theta_4 - theta_1)) + dtheta_3*(dtheta_3*l_1*l_3*m_4*np.sin(theta_3 - theta_1) + dtheta_3*l_1*l_3*m_5*np.sin(theta_3 - theta_1) + dtheta_3*l_1*m_3*r_3*np.sin(theta_3 - theta_1)) + dtheta_2*mu_2 + dtheta_2*(dtheta_2*l_1*l_2*m_3*np.sin(theta_2 - theta_1) + dtheta_2*l_1*l_2*m_4*np.sin(theta_2 - theta_1) + dtheta_2*l_1*l_2*m_5*np.sin(theta_2 - theta_1) + dtheta_2*l_1*m_2*r_2*np.sin(theta_2 - theta_1)) - dtheta_1*(mu_1 + mu_2) - ddx_c*(-l_1*m_2 - l_1*m_3 - l_1*m_4 - l_1*m_5 - m_1*r_1)*np.cos(theta_1) - g*(-l_1*m_2 - l_1*m_3 - l_1*m_4 - l_1*m_5 - m_1*r_1)*np.sin(theta_1))*np.cos(theta_4 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) - (-l_1**2*(l_4*m_5 + m_4*r_4)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_4 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_4*m_5 + m_4*r_4)*np.cos(theta_4 - theta_2))*(dtheta_5**2*l_2*m_5*r_5*np.sin(theta_5 - theta_2) + dtheta_4*(dtheta_4*l_2*l_4*m_5*np.sin(theta_4 - theta_2) + dtheta_4*l_2*m_4*r_4*np.sin(theta_4 - theta_2)) + dtheta_3*mu_3 + dtheta_3*(dtheta_3*l_2*l_3*m_4*np.sin(theta_3 - theta_2) + dtheta_3*l_2*l_3*m_5*np.sin(theta_3 - theta_2) + dtheta_3*l_2*m_3*r_3*np.sin(theta_3 - theta_2)) - dtheta_2*(mu_2 + mu_3) + dtheta_1*mu_2 + dtheta_1*(-dtheta_1*l_1*l_2*m_3*np.sin(theta_2 - theta_1) - dtheta_1*l_1*l_2*m_4*np.sin(theta_2 - theta_1) - dtheta_1*l_1*l_2*m_5*np.sin(theta_2 - theta_1) - dtheta_1*l_1*m_2*r_2*np.sin(theta_2 - theta_1)) - ddx_c*(-l_2*m_3 - l_2*m_4 - l_2*m_5 - m_2*r_2)*np.cos(theta_2) - g*(-l_2*m_3 - l_2*m_4 - l_2*m_5 - m_2*r_2)*np.sin(theta_2) - l_1*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*(dtheta_5**2*l_1*m_5*r_5*np.sin(theta_5 - theta_1) + dtheta_4*(dtheta_4*l_1*l_4*m_5*np.sin(theta_4 - theta_1) + dtheta_4*l_1*m_4*r_4*np.sin(theta_4 - theta_1)) + dtheta_3*(dtheta_3*l_1*l_3*m_4*np.sin(theta_3 - theta_1) + dtheta_3*l_1*l_3*m_5*np.sin(theta_3 - theta_1) + dtheta_3*l_1*m_3*r_3*np.sin(theta_3 - theta_1)) + dtheta_2*mu_2 + dtheta_2*(dtheta_2*l_1*l_2*m_3*np.sin(theta_2 - theta_1) + dtheta_2*l_1*l_2*m_4*np.sin(theta_2 - theta_1) + dtheta_2*l_1*l_2*m_5*np.sin(theta_2 - theta_1) + dtheta_2*l_1*m_2*r_2*np.sin(theta_2 - theta_1)) - dtheta_1*(mu_1 + mu_2) - ddx_c*(-l_1*m_2 - l_1*m_3 - l_1*m_4 - l_1*m_5 - m_1*r_1)*np.cos(theta_1) - g*(-l_1*m_2 - l_1*m_3 - l_1*m_4 - l_1*m_5 - m_1*r_1)*np.sin(theta_1))*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2))/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2) - (-l_1**2*(l_4*m_5 + m_4*r_4)*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_4 - theta_1)*np.cos(theta_3 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_3*(l_4*m_5 + m_4*r_4)*np.cos(theta_4 - theta_3) - (-l_1**2*(l_4*m_5 + m_4*r_4)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_4 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_4*m_5 + m_4*r_4)*np.cos(theta_4 - theta_2))*(-l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_3 - theta_2))/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2))*(dtheta_5**2*l_3*m_5*r_5*np.sin(theta_5 - theta_3) + dtheta_4*mu_4 + dtheta_4*(dtheta_4*l_3*l_4*m_5*np.sin(theta_4 - theta_3) + dtheta_4*l_3*m_4*r_4*np.sin(theta_4 - theta_3)) - dtheta_3*(mu_3 + mu_4) + dtheta_2*mu_3 + dtheta_2*(-dtheta_2*l_2*l_3*m_4*np.sin(theta_3 - theta_2) - dtheta_2*l_2*l_3*m_5*np.sin(theta_3 - theta_2) - dtheta_2*l_2*m_3*r_3*np.sin(theta_3 - theta_2)) + dtheta_1*(-dtheta_1*l_1*l_3*m_4*np.sin(theta_3 - theta_1) - dtheta_1*l_1*l_3*m_5*np.sin(theta_3 - theta_1) - dtheta_1*l_1*m_3*r_3*np.sin(theta_3 - theta_1)) - ddx_c*(-l_3*m_4 - l_3*m_5 - m_3*r_3)*np.cos(theta_3) - g*(-l_3*m_4 - l_3*m_5 - m_3*r_3)*np.sin(theta_3) - l_1*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(dtheta_5**2*l_1*m_5*r_5*np.sin(theta_5 - theta_1) + dtheta_4*(dtheta_4*l_1*l_4*m_5*np.sin(theta_4 - theta_1) + dtheta_4*l_1*m_4*r_4*np.sin(theta_4 - theta_1)) + dtheta_3*(dtheta_3*l_1*l_3*m_4*np.sin(theta_3 - theta_1) + dtheta_3*l_1*l_3*m_5*np.sin(theta_3 - theta_1) + dtheta_3*l_1*m_3*r_3*np.sin(theta_3 - theta_1)) + dtheta_2*mu_2 + dtheta_2*(dtheta_2*l_1*l_2*m_3*np.sin(theta_2 - theta_1) + dtheta_2*l_1*l_2*m_4*np.sin(theta_2 - theta_1) + dtheta_2*l_1*l_2*m_5*np.sin(theta_2 - theta_1) + dtheta_2*l_1*m_2*r_2*np.sin(theta_2 - theta_1)) - dtheta_1*(mu_1 + mu_2) - ddx_c*(-l_1*m_2 - l_1*m_3 - l_1*m_4 - l_1*m_5 - m_1*r_1)*np.cos(theta_1) - g*(-l_1*m_2 - l_1*m_3 - l_1*m_4 - l_1*m_5 - m_1*r_1)*np.sin(theta_1))*np.cos(theta_3 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) - (-l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_3 - theta_2))*(dtheta_5**2*l_2*m_5*r_5*np.sin(theta_5 - theta_2) + dtheta_4*(dtheta_4*l_2*l_4*m_5*np.sin(theta_4 - theta_2) + dtheta_4*l_2*m_4*r_4*np.sin(theta_4 - theta_2)) + dtheta_3*mu_3 + dtheta_3*(dtheta_3*l_2*l_3*m_4*np.sin(theta_3 - theta_2) + dtheta_3*l_2*l_3*m_5*np.sin(theta_3 - theta_2) + dtheta_3*l_2*m_3*r_3*np.sin(theta_3 - theta_2)) - dtheta_2*(mu_2 + mu_3) + dtheta_1*mu_2 + dtheta_1*(-dtheta_1*l_1*l_2*m_3*np.sin(theta_2 - theta_1) - dtheta_1*l_1*l_2*m_4*np.sin(theta_2 - theta_1) - dtheta_1*l_1*l_2*m_5*np.sin(theta_2 - theta_1) - dtheta_1*l_1*m_2*r_2*np.sin(theta_2 - theta_1)) - ddx_c*(-l_2*m_3 - l_2*m_4 - l_2*m_5 - m_2*r_2)*np.cos(theta_2) - g*(-l_2*m_3 - l_2*m_4 - l_2*m_5 - m_2*r_2)*np.sin(theta_2) - l_1*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*(dtheta_5**2*l_1*m_5*r_5*np.sin(theta_5 - theta_1) + dtheta_4*(dtheta_4*l_1*l_4*m_5*np.sin(theta_4 - theta_1) + dtheta_4*l_1*m_4*r_4*np.sin(theta_4 - theta_1)) + dtheta_3*(dtheta_3*l_1*l_3*m_4*np.sin(theta_3 - theta_1) + dtheta_3*l_1*l_3*m_5*np.sin(theta_3 - theta_1) + dtheta_3*l_1*m_3*r_3*np.sin(theta_3 - theta_1)) + dtheta_2*mu_2 + dtheta_2*(dtheta_2*l_1*l_2*m_3*np.sin(theta_2 - theta_1) + dtheta_2*l_1*l_2*m_4*np.sin(theta_2 - theta_1) + dtheta_2*l_1*l_2*m_5*np.sin(theta_2 - theta_1) + dtheta_2*l_1*m_2*r_2*np.sin(theta_2 - theta_1)) - dtheta_1*(mu_1 + mu_2) - ddx_c*(-l_1*m_2 - l_1*m_3 - l_1*m_4 - l_1*m_5 - m_1*r_1)*np.cos(theta_1) - g*(-l_1*m_2 - l_1*m_3 - l_1*m_4 - l_1*m_5 - m_1*r_1)*np.sin(theta_1))*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2))/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2))/(I_3 - l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)**2*np.cos(theta_3 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_3**2*m_4 + l_3**2*m_5 + m_3*r_3**2 - (-l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_3 - theta_2))**2/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2)) - (-l_1**2*m_5*r_5*(l_4*m_5 + m_4*r_4)*np.cos(theta_5 - theta_1)*np.cos(theta_4 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_4*m_5*r_5*np.cos(theta_5 - theta_4) - (-l_1**2*(l_4*m_5 + m_4*r_4)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_4 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_4*m_5 + m_4*r_4)*np.cos(theta_4 - theta_2))*(-l_1**2*m_5*r_5*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_5 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*m_5*r_5*np.cos(theta_5 - theta_2))/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2) - (-l_1**2*(l_4*m_5 + m_4*r_4)*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_4 - theta_1)*np.cos(theta_3 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_3*(l_4*m_5 + m_4*r_4)*np.cos(theta_4 - theta_3) - (-l_1**2*(l_4*m_5 + m_4*r_4)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_4 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_4*m_5 + m_4*r_4)*np.cos(theta_4 - theta_2))*(-l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_3 - theta_2))/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2))*(-l_1**2*m_5*r_5*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_5 - theta_1)*np.cos(theta_3 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_3*m_5*r_5*np.cos(theta_5 - theta_3) - (-l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_3 - theta_2))*(-l_1**2*m_5*r_5*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_5 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*m_5*r_5*np.cos(theta_5 - theta_2))/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2))/(I_3 - l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)**2*np.cos(theta_3 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_3**2*m_4 + l_3**2*m_5 + m_3*r_3**2 - (-l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_3 - theta_2))**2/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2)))*(-dtheta_5*mu_5 - dtheta_4**2*l_4*m_5*r_5*np.sin(theta_5 - theta_4) + dtheta_4*mu_5 - dtheta_3**2*l_3*m_5*r_5*np.sin(theta_5 - theta_3) - dtheta_2**2*l_2*m_5*r_5*np.sin(theta_5 - theta_2) - dtheta_1**2*l_1*m_5*r_5*np.sin(theta_5 - theta_1) + ddx_c*m_5*r_5*np.cos(theta_5) + g*m_5*r_5*np.sin(theta_5) - l_1*m_5*r_5*(dtheta_5**2*l_1*m_5*r_5*np.sin(theta_5 - theta_1) + dtheta_4*(dtheta_4*l_1*l_4*m_5*np.sin(theta_4 - theta_1) + dtheta_4*l_1*m_4*r_4*np.sin(theta_4 - theta_1)) + dtheta_3*(dtheta_3*l_1*l_3*m_4*np.sin(theta_3 - theta_1) + dtheta_3*l_1*l_3*m_5*np.sin(theta_3 - theta_1) + dtheta_3*l_1*m_3*r_3*np.sin(theta_3 - theta_1)) + dtheta_2*mu_2 + dtheta_2*(dtheta_2*l_1*l_2*m_3*np.sin(theta_2 - theta_1) + dtheta_2*l_1*l_2*m_4*np.sin(theta_2 - theta_1) + dtheta_2*l_1*l_2*m_5*np.sin(theta_2 - theta_1) + dtheta_2*l_1*m_2*r_2*np.sin(theta_2 - theta_1)) - dtheta_1*(mu_1 + mu_2) - ddx_c*(-l_1*m_2 - l_1*m_3 - l_1*m_4 - l_1*m_5 - m_1*r_1)*np.cos(theta_1) - g*(-l_1*m_2 - l_1*m_3 - l_1*m_4 - l_1*m_5 - m_1*r_1)*np.sin(theta_1))*np.cos(theta_5 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) - (-l_1**2*m_5*r_5*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_5 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*m_5*r_5*np.cos(theta_5 - theta_2))*(dtheta_5**2*l_2*m_5*r_5*np.sin(theta_5 - theta_2) + dtheta_4*(dtheta_4*l_2*l_4*m_5*np.sin(theta_4 - theta_2) + dtheta_4*l_2*m_4*r_4*np.sin(theta_4 - theta_2)) + dtheta_3*mu_3 + dtheta_3*(dtheta_3*l_2*l_3*m_4*np.sin(theta_3 - theta_2) + dtheta_3*l_2*l_3*m_5*np.sin(theta_3 - theta_2) + dtheta_3*l_2*m_3*r_3*np.sin(theta_3 - theta_2)) - dtheta_2*(mu_2 + mu_3) + dtheta_1*mu_2 + dtheta_1*(-dtheta_1*l_1*l_2*m_3*np.sin(theta_2 - theta_1) - dtheta_1*l_1*l_2*m_4*np.sin(theta_2 - theta_1) - dtheta_1*l_1*l_2*m_5*np.sin(theta_2 - theta_1) - dtheta_1*l_1*m_2*r_2*np.sin(theta_2 - theta_1)) - ddx_c*(-l_2*m_3 - l_2*m_4 - l_2*m_5 - m_2*r_2)*np.cos(theta_2) - g*(-l_2*m_3 - l_2*m_4 - l_2*m_5 - m_2*r_2)*np.sin(theta_2) - l_1*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*(dtheta_5**2*l_1*m_5*r_5*np.sin(theta_5 - theta_1) + dtheta_4*(dtheta_4*l_1*l_4*m_5*np.sin(theta_4 - theta_1) + dtheta_4*l_1*m_4*r_4*np.sin(theta_4 - theta_1)) + dtheta_3*(dtheta_3*l_1*l_3*m_4*np.sin(theta_3 - theta_1) + dtheta_3*l_1*l_3*m_5*np.sin(theta_3 - theta_1) + dtheta_3*l_1*m_3*r_3*np.sin(theta_3 - theta_1)) + dtheta_2*mu_2 + dtheta_2*(dtheta_2*l_1*l_2*m_3*np.sin(theta_2 - theta_1) + dtheta_2*l_1*l_2*m_4*np.sin(theta_2 - theta_1) + dtheta_2*l_1*l_2*m_5*np.sin(theta_2 - theta_1) + dtheta_2*l_1*m_2*r_2*np.sin(theta_2 - theta_1)) - dtheta_1*(mu_1 + mu_2) - ddx_c*(-l_1*m_2 - l_1*m_3 - l_1*m_4 - l_1*m_5 - m_1*r_1)*np.cos(theta_1) - g*(-l_1*m_2 - l_1*m_3 - l_1*m_4 - l_1*m_5 - m_1*r_1)*np.sin(theta_1))*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2))/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2) - (-l_1**2*m_5*r_5*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_5 - theta_1)*np.cos(theta_3 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_3*m_5*r_5*np.cos(theta_5 - theta_3) - (-l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_3 - theta_2))*(-l_1**2*m_5*r_5*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_5 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*m_5*r_5*np.cos(theta_5 - theta_2))/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2))*(dtheta_5**2*l_3*m_5*r_5*np.sin(theta_5 - theta_3) + dtheta_4*mu_4 + dtheta_4*(dtheta_4*l_3*l_4*m_5*np.sin(theta_4 - theta_3) + dtheta_4*l_3*m_4*r_4*np.sin(theta_4 - theta_3)) - dtheta_3*(mu_3 + mu_4) + dtheta_2*mu_3 + dtheta_2*(-dtheta_2*l_2*l_3*m_4*np.sin(theta_3 - theta_2) - dtheta_2*l_2*l_3*m_5*np.sin(theta_3 - theta_2) - dtheta_2*l_2*m_3*r_3*np.sin(theta_3 - theta_2)) + dtheta_1*(-dtheta_1*l_1*l_3*m_4*np.sin(theta_3 - theta_1) - dtheta_1*l_1*l_3*m_5*np.sin(theta_3 - theta_1) - dtheta_1*l_1*m_3*r_3*np.sin(theta_3 - theta_1)) - ddx_c*(-l_3*m_4 - l_3*m_5 - m_3*r_3)*np.cos(theta_3) - g*(-l_3*m_4 - l_3*m_5 - m_3*r_3)*np.sin(theta_3) - l_1*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(dtheta_5**2*l_1*m_5*r_5*np.sin(theta_5 - theta_1) + dtheta_4*(dtheta_4*l_1*l_4*m_5*np.sin(theta_4 - theta_1) + dtheta_4*l_1*m_4*r_4*np.sin(theta_4 - theta_1)) + dtheta_3*(dtheta_3*l_1*l_3*m_4*np.sin(theta_3 - theta_1) + dtheta_3*l_1*l_3*m_5*np.sin(theta_3 - theta_1) + dtheta_3*l_1*m_3*r_3*np.sin(theta_3 - theta_1)) + dtheta_2*mu_2 + dtheta_2*(dtheta_2*l_1*l_2*m_3*np.sin(theta_2 - theta_1) + dtheta_2*l_1*l_2*m_4*np.sin(theta_2 - theta_1) + dtheta_2*l_1*l_2*m_5*np.sin(theta_2 - theta_1) + dtheta_2*l_1*m_2*r_2*np.sin(theta_2 - theta_1)) - dtheta_1*(mu_1 + mu_2) - ddx_c*(-l_1*m_2 - l_1*m_3 - l_1*m_4 - l_1*m_5 - m_1*r_1)*np.cos(theta_1) - g*(-l_1*m_2 - l_1*m_3 - l_1*m_4 - l_1*m_5 - m_1*r_1)*np.sin(theta_1))*np.cos(theta_3 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) - (-l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_3 - theta_2))*(dtheta_5**2*l_2*m_5*r_5*np.sin(theta_5 - theta_2) + dtheta_4*(dtheta_4*l_2*l_4*m_5*np.sin(theta_4 - theta_2) + dtheta_4*l_2*m_4*r_4*np.sin(theta_4 - theta_2)) + dtheta_3*mu_3 + dtheta_3*(dtheta_3*l_2*l_3*m_4*np.sin(theta_3 - theta_2) + dtheta_3*l_2*l_3*m_5*np.sin(theta_3 - theta_2) + dtheta_3*l_2*m_3*r_3*np.sin(theta_3 - theta_2)) - dtheta_2*(mu_2 + mu_3) + dtheta_1*mu_2 + dtheta_1*(-dtheta_1*l_1*l_2*m_3*np.sin(theta_2 - theta_1) - dtheta_1*l_1*l_2*m_4*np.sin(theta_2 - theta_1) - dtheta_1*l_1*l_2*m_5*np.sin(theta_2 - theta_1) - dtheta_1*l_1*m_2*r_2*np.sin(theta_2 - theta_1)) - ddx_c*(-l_2*m_3 - l_2*m_4 - l_2*m_5 - m_2*r_2)*np.cos(theta_2) - g*(-l_2*m_3 - l_2*m_4 - l_2*m_5 - m_2*r_2)*np.sin(theta_2) - l_1*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*(dtheta_5**2*l_1*m_5*r_5*np.sin(theta_5 - theta_1) + dtheta_4*(dtheta_4*l_1*l_4*m_5*np.sin(theta_4 - theta_1) + dtheta_4*l_1*m_4*r_4*np.sin(theta_4 - theta_1)) + dtheta_3*(dtheta_3*l_1*l_3*m_4*np.sin(theta_3 - theta_1) + dtheta_3*l_1*l_3*m_5*np.sin(theta_3 - theta_1) + dtheta_3*l_1*m_3*r_3*np.sin(theta_3 - theta_1)) + dtheta_2*mu_2 + dtheta_2*(dtheta_2*l_1*l_2*m_3*np.sin(theta_2 - theta_1) + dtheta_2*l_1*l_2*m_4*np.sin(theta_2 - theta_1) + dtheta_2*l_1*l_2*m_5*np.sin(theta_2 - theta_1) + dtheta_2*l_1*m_2*r_2*np.sin(theta_2 - theta_1)) - dtheta_1*(mu_1 + mu_2) - ddx_c*(-l_1*m_2 - l_1*m_3 - l_1*m_4 - l_1*m_5 - m_1*r_1)*np.cos(theta_1) - g*(-l_1*m_2 - l_1*m_3 - l_1*m_4 - l_1*m_5 - m_1*r_1)*np.sin(theta_1))*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2))/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2))/(I_3 - l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)**2*np.cos(theta_3 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_3**2*m_4 + l_3**2*m_5 + m_3*r_3**2 - (-l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_3 - theta_2))**2/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2)) - (-l_1**2*m_5*r_5*(l_4*m_5 + m_4*r_4)*np.cos(theta_5 - theta_1)*np.cos(theta_4 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_4*m_5*r_5*np.cos(theta_5 - theta_4) - (-l_1**2*(l_4*m_5 + m_4*r_4)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_4 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_4*m_5 + m_4*r_4)*np.cos(theta_4 - theta_2))*(-l_1**2*m_5*r_5*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_5 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*m_5*r_5*np.cos(theta_5 - theta_2))/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2) - (-l_1**2*(l_4*m_5 + m_4*r_4)*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_4 - theta_1)*np.cos(theta_3 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_3*(l_4*m_5 + m_4*r_4)*np.cos(theta_4 - theta_3) - (-l_1**2*(l_4*m_5 + m_4*r_4)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_4 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_4*m_5 + m_4*r_4)*np.cos(theta_4 - theta_2))*(-l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_3 - theta_2))/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2))*(-l_1**2*m_5*r_5*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_5 - theta_1)*np.cos(theta_3 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_3*m_5*r_5*np.cos(theta_5 - theta_3) - (-l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_3 - theta_2))*(-l_1**2*m_5*r_5*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_5 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*m_5*r_5*np.cos(theta_5 - theta_2))/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2))/(I_3 - l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)**2*np.cos(theta_3 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_3**2*m_4 + l_3**2*m_5 + m_3*r_3**2 - (-l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_3 - theta_2))**2/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2)))*(dtheta_5**2*l_4*m_5*r_5*np.sin(theta_5 - theta_4) + dtheta_5*mu_5 - dtheta_4*(mu_4 + mu_5) + dtheta_3*mu_4 + dtheta_3*(-dtheta_3*l_3*l_4*m_5*np.sin(theta_4 - theta_3) - dtheta_3*l_3*m_4*r_4*np.sin(theta_4 - theta_3)) + dtheta_2*(-dtheta_2*l_2*l_4*m_5*np.sin(theta_4 - theta_2) - dtheta_2*l_2*m_4*r_4*np.sin(theta_4 - theta_2)) + dtheta_1*(-dtheta_1*l_1*l_4*m_5*np.sin(theta_4 - theta_1) - dtheta_1*l_1*m_4*r_4*np.sin(theta_4 - theta_1)) - ddx_c*(-l_4*m_5 - m_4*r_4)*np.cos(theta_4) - g*(-l_4*m_5 - m_4*r_4)*np.sin(theta_4) - l_1*(l_4*m_5 + m_4*r_4)*(dtheta_5**2*l_1*m_5*r_5*np.sin(theta_5 - theta_1) + dtheta_4*(dtheta_4*l_1*l_4*m_5*np.sin(theta_4 - theta_1) + dtheta_4*l_1*m_4*r_4*np.sin(theta_4 - theta_1)) + dtheta_3*(dtheta_3*l_1*l_3*m_4*np.sin(theta_3 - theta_1) + dtheta_3*l_1*l_3*m_5*np.sin(theta_3 - theta_1) + dtheta_3*l_1*m_3*r_3*np.sin(theta_3 - theta_1)) + dtheta_2*mu_2 + dtheta_2*(dtheta_2*l_1*l_2*m_3*np.sin(theta_2 - theta_1) + dtheta_2*l_1*l_2*m_4*np.sin(theta_2 - theta_1) + dtheta_2*l_1*l_2*m_5*np.sin(theta_2 - theta_1) + dtheta_2*l_1*m_2*r_2*np.sin(theta_2 - theta_1)) - dtheta_1*(mu_1 + mu_2) - ddx_c*(-l_1*m_2 - l_1*m_3 - l_1*m_4 - l_1*m_5 - m_1*r_1)*np.cos(theta_1) - g*(-l_1*m_2 - l_1*m_3 - l_1*m_4 - l_1*m_5 - m_1*r_1)*np.sin(theta_1))*np.cos(theta_4 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) - (-l_1**2*(l_4*m_5 + m_4*r_4)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_4 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_4*m_5 + m_4*r_4)*np.cos(theta_4 - theta_2))*(dtheta_5**2*l_2*m_5*r_5*np.sin(theta_5 - theta_2) + dtheta_4*(dtheta_4*l_2*l_4*m_5*np.sin(theta_4 - theta_2) + dtheta_4*l_2*m_4*r_4*np.sin(theta_4 - theta_2)) + dtheta_3*mu_3 + dtheta_3*(dtheta_3*l_2*l_3*m_4*np.sin(theta_3 - theta_2) + dtheta_3*l_2*l_3*m_5*np.sin(theta_3 - theta_2) + dtheta_3*l_2*m_3*r_3*np.sin(theta_3 - theta_2)) - dtheta_2*(mu_2 + mu_3) + dtheta_1*mu_2 + dtheta_1*(-dtheta_1*l_1*l_2*m_3*np.sin(theta_2 - theta_1) - dtheta_1*l_1*l_2*m_4*np.sin(theta_2 - theta_1) - dtheta_1*l_1*l_2*m_5*np.sin(theta_2 - theta_1) - dtheta_1*l_1*m_2*r_2*np.sin(theta_2 - theta_1)) - ddx_c*(-l_2*m_3 - l_2*m_4 - l_2*m_5 - m_2*r_2)*np.cos(theta_2) - g*(-l_2*m_3 - l_2*m_4 - l_2*m_5 - m_2*r_2)*np.sin(theta_2) - l_1*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*(dtheta_5**2*l_1*m_5*r_5*np.sin(theta_5 - theta_1) + dtheta_4*(dtheta_4*l_1*l_4*m_5*np.sin(theta_4 - theta_1) + dtheta_4*l_1*m_4*r_4*np.sin(theta_4 - theta_1)) + dtheta_3*(dtheta_3*l_1*l_3*m_4*np.sin(theta_3 - theta_1) + dtheta_3*l_1*l_3*m_5*np.sin(theta_3 - theta_1) + dtheta_3*l_1*m_3*r_3*np.sin(theta_3 - theta_1)) + dtheta_2*mu_2 + dtheta_2*(dtheta_2*l_1*l_2*m_3*np.sin(theta_2 - theta_1) + dtheta_2*l_1*l_2*m_4*np.sin(theta_2 - theta_1) + dtheta_2*l_1*l_2*m_5*np.sin(theta_2 - theta_1) + dtheta_2*l_1*m_2*r_2*np.sin(theta_2 - theta_1)) - dtheta_1*(mu_1 + mu_2) - ddx_c*(-l_1*m_2 - l_1*m_3 - l_1*m_4 - l_1*m_5 - m_1*r_1)*np.cos(theta_1) - g*(-l_1*m_2 - l_1*m_3 - l_1*m_4 - l_1*m_5 - m_1*r_1)*np.sin(theta_1))*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2))/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2) - (-l_1**2*(l_4*m_5 + m_4*r_4)*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_4 - theta_1)*np.cos(theta_3 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_3*(l_4*m_5 + m_4*r_4)*np.cos(theta_4 - theta_3) - (-l_1**2*(l_4*m_5 + m_4*r_4)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_4 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_4*m_5 + m_4*r_4)*np.cos(theta_4 - theta_2))*(-l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_3 - theta_2))/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2))*(dtheta_5**2*l_3*m_5*r_5*np.sin(theta_5 - theta_3) + dtheta_4*mu_4 + dtheta_4*(dtheta_4*l_3*l_4*m_5*np.sin(theta_4 - theta_3) + dtheta_4*l_3*m_4*r_4*np.sin(theta_4 - theta_3)) - dtheta_3*(mu_3 + mu_4) + dtheta_2*mu_3 + dtheta_2*(-dtheta_2*l_2*l_3*m_4*np.sin(theta_3 - theta_2) - dtheta_2*l_2*l_3*m_5*np.sin(theta_3 - theta_2) - dtheta_2*l_2*m_3*r_3*np.sin(theta_3 - theta_2)) + dtheta_1*(-dtheta_1*l_1*l_3*m_4*np.sin(theta_3 - theta_1) - dtheta_1*l_1*l_3*m_5*np.sin(theta_3 - theta_1) - dtheta_1*l_1*m_3*r_3*np.sin(theta_3 - theta_1)) - ddx_c*(-l_3*m_4 - l_3*m_5 - m_3*r_3)*np.cos(theta_3) - g*(-l_3*m_4 - l_3*m_5 - m_3*r_3)*np.sin(theta_3) - l_1*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(dtheta_5**2*l_1*m_5*r_5*np.sin(theta_5 - theta_1) + dtheta_4*(dtheta_4*l_1*l_4*m_5*np.sin(theta_4 - theta_1) + dtheta_4*l_1*m_4*r_4*np.sin(theta_4 - theta_1)) + dtheta_3*(dtheta_3*l_1*l_3*m_4*np.sin(theta_3 - theta_1) + dtheta_3*l_1*l_3*m_5*np.sin(theta_3 - theta_1) + dtheta_3*l_1*m_3*r_3*np.sin(theta_3 - theta_1)) + dtheta_2*mu_2 + dtheta_2*(dtheta_2*l_1*l_2*m_3*np.sin(theta_2 - theta_1) + dtheta_2*l_1*l_2*m_4*np.sin(theta_2 - theta_1) + dtheta_2*l_1*l_2*m_5*np.sin(theta_2 - theta_1) + dtheta_2*l_1*m_2*r_2*np.sin(theta_2 - theta_1)) - dtheta_1*(mu_1 + mu_2) - ddx_c*(-l_1*m_2 - l_1*m_3 - l_1*m_4 - l_1*m_5 - m_1*r_1)*np.cos(theta_1) - g*(-l_1*m_2 - l_1*m_3 - l_1*m_4 - l_1*m_5 - m_1*r_1)*np.sin(theta_1))*np.cos(theta_3 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) - (-l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_3 - theta_2))*(dtheta_5**2*l_2*m_5*r_5*np.sin(theta_5 - theta_2) + dtheta_4*(dtheta_4*l_2*l_4*m_5*np.sin(theta_4 - theta_2) + dtheta_4*l_2*m_4*r_4*np.sin(theta_4 - theta_2)) + dtheta_3*mu_3 + dtheta_3*(dtheta_3*l_2*l_3*m_4*np.sin(theta_3 - theta_2) + dtheta_3*l_2*l_3*m_5*np.sin(theta_3 - theta_2) + dtheta_3*l_2*m_3*r_3*np.sin(theta_3 - theta_2)) - dtheta_2*(mu_2 + mu_3) + dtheta_1*mu_2 + dtheta_1*(-dtheta_1*l_1*l_2*m_3*np.sin(theta_2 - theta_1) - dtheta_1*l_1*l_2*m_4*np.sin(theta_2 - theta_1) - dtheta_1*l_1*l_2*m_5*np.sin(theta_2 - theta_1) - dtheta_1*l_1*m_2*r_2*np.sin(theta_2 - theta_1)) - ddx_c*(-l_2*m_3 - l_2*m_4 - l_2*m_5 - m_2*r_2)*np.cos(theta_2) - g*(-l_2*m_3 - l_2*m_4 - l_2*m_5 - m_2*r_2)*np.sin(theta_2) - l_1*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*(dtheta_5**2*l_1*m_5*r_5*np.sin(theta_5 - theta_1) + dtheta_4*(dtheta_4*l_1*l_4*m_5*np.sin(theta_4 - theta_1) + dtheta_4*l_1*m_4*r_4*np.sin(theta_4 - theta_1)) + dtheta_3*(dtheta_3*l_1*l_3*m_4*np.sin(theta_3 - theta_1) + dtheta_3*l_1*l_3*m_5*np.sin(theta_3 - theta_1) + dtheta_3*l_1*m_3*r_3*np.sin(theta_3 - theta_1)) + dtheta_2*mu_2 + dtheta_2*(dtheta_2*l_1*l_2*m_3*np.sin(theta_2 - theta_1) + dtheta_2*l_1*l_2*m_4*np.sin(theta_2 - theta_1) + dtheta_2*l_1*l_2*m_5*np.sin(theta_2 - theta_1) + dtheta_2*l_1*m_2*r_2*np.sin(theta_2 - theta_1)) - dtheta_1*(mu_1 + mu_2) - ddx_c*(-l_1*m_2 - l_1*m_3 - l_1*m_4 - l_1*m_5 - m_1*r_1)*np.cos(theta_1) - g*(-l_1*m_2 - l_1*m_3 - l_1*m_4 - l_1*m_5 - m_1*r_1)*np.sin(theta_1))*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2))/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2))/(I_3 - l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)**2*np.cos(theta_3 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_3**2*m_4 + l_3**2*m_5 + m_3*r_3**2 - (-l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_3 - theta_2))**2/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2)))/(I_4 - l_1**2*(l_4*m_5 + m_4*r_4)**2*np.cos(theta_4 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_4**2*m_5 + m_4*r_4**2 - (-l_1**2*(l_4*m_5 + m_4*r_4)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_4 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_4*m_5 + m_4*r_4)*np.cos(theta_4 - theta_2))**2/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2) - (-l_1**2*(l_4*m_5 + m_4*r_4)*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_4 - theta_1)*np.cos(theta_3 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_3*(l_4*m_5 + m_4*r_4)*np.cos(theta_4 - theta_3) - (-l_1**2*(l_4*m_5 + m_4*r_4)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_4 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_4*m_5 + m_4*r_4)*np.cos(theta_4 - theta_2))*(-l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_3 - theta_2))/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2))**2/(I_3 - l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)**2*np.cos(theta_3 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_3**2*m_4 + l_3**2*m_5 + m_3*r_3**2 - (-l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_3 - theta_2))**2/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2))))/(I_5 - l_1**2*m_5**2*r_5**2*np.cos(theta_5 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + m_5*r_5**2 - (-l_1**2*m_5*r_5*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_5 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*m_5*r_5*np.cos(theta_5 - theta_2))**2/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2) - (-l_1**2*m_5*r_5*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_5 - theta_1)*np.cos(theta_3 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_3*m_5*r_5*np.cos(theta_5 - theta_3) - (-l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_3 - theta_2))*(-l_1**2*m_5*r_5*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_5 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*m_5*r_5*np.cos(theta_5 - theta_2))/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2))**2/(I_3 - l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)**2*np.cos(theta_3 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_3**2*m_4 + l_3**2*m_5 + m_3*r_3**2 - (-l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_3 - theta_2))**2/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2)) - (-l_1**2*m_5*r_5*(l_4*m_5 + m_4*r_4)*np.cos(theta_5 - theta_1)*np.cos(theta_4 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_4*m_5*r_5*np.cos(theta_5 - theta_4) - (-l_1**2*(l_4*m_5 + m_4*r_4)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_4 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_4*m_5 + m_4*r_4)*np.cos(theta_4 - theta_2))*(-l_1**2*m_5*r_5*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_5 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*m_5*r_5*np.cos(theta_5 - theta_2))/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2) - (-l_1**2*(l_4*m_5 + m_4*r_4)*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_4 - theta_1)*np.cos(theta_3 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_3*(l_4*m_5 + m_4*r_4)*np.cos(theta_4 - theta_3) - (-l_1**2*(l_4*m_5 + m_4*r_4)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_4 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_4*m_5 + m_4*r_4)*np.cos(theta_4 - theta_2))*(-l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_3 - theta_2))/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2))*(-l_1**2*m_5*r_5*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_5 - theta_1)*np.cos(theta_3 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_3*m_5*r_5*np.cos(theta_5 - theta_3) - (-l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_3 - theta_2))*(-l_1**2*m_5*r_5*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_5 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*m_5*r_5*np.cos(theta_5 - theta_2))/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2))/(I_3 - l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)**2*np.cos(theta_3 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_3**2*m_4 + l_3**2*m_5 + m_3*r_3**2 - (-l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_3 - theta_2))**2/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2)))**2/(I_4 - l_1**2*(l_4*m_5 + m_4*r_4)**2*np.cos(theta_4 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_4**2*m_5 + m_4*r_4**2 - (-l_1**2*(l_4*m_5 + m_4*r_4)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_4 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_4*m_5 + m_4*r_4)*np.cos(theta_4 - theta_2))**2/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2) - (-l_1**2*(l_4*m_5 + m_4*r_4)*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_4 - theta_1)*np.cos(theta_3 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_3*(l_4*m_5 + m_4*r_4)*np.cos(theta_4 - theta_3) - (-l_1**2*(l_4*m_5 + m_4*r_4)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_4 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_4*m_5 + m_4*r_4)*np.cos(theta_4 - theta_2))*(-l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_3 - theta_2))/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2))**2/(I_3 - l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)**2*np.cos(theta_3 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_3**2*m_4 + l_3**2*m_5 + m_3*r_3**2 - (-l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_3 - theta_2))**2/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2)))))/(I_4 - l_1**2*(l_4*m_5 + m_4*r_4)**2*np.cos(theta_4 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_4**2*m_5 + m_4*r_4**2 - (-l_1**2*(l_4*m_5 + m_4*r_4)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_4 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_4*m_5 + m_4*r_4)*np.cos(theta_4 - theta_2))**2/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2) - (-l_1**2*(l_4*m_5 + m_4*r_4)*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_4 - theta_1)*np.cos(theta_3 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_3*(l_4*m_5 + m_4*r_4)*np.cos(theta_4 - theta_3) - (-l_1**2*(l_4*m_5 + m_4*r_4)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_4 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_4*m_5 + m_4*r_4)*np.cos(theta_4 - theta_2))*(-l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_3 - theta_2))/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2))**2/(I_3 - l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)**2*np.cos(theta_3 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_3**2*m_4 + l_3**2*m_5 + m_3*r_3**2 - (-l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_3 - theta_2))**2/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2))) - (-l_1**2*m_5*r_5*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_5 - theta_1)*np.cos(theta_3 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_3*m_5*r_5*np.cos(theta_5 - theta_3) - (-l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_3 - theta_2))*(-l_1**2*m_5*r_5*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_5 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*m_5*r_5*np.cos(theta_5 - theta_2))/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2))*(-dtheta_5*mu_5 - dtheta_4**2*l_4*m_5*r_5*np.sin(theta_5 - theta_4) + dtheta_4*mu_5 - dtheta_3**2*l_3*m_5*r_5*np.sin(theta_5 - theta_3) - dtheta_2**2*l_2*m_5*r_5*np.sin(theta_5 - theta_2) - dtheta_1**2*l_1*m_5*r_5*np.sin(theta_5 - theta_1) + ddx_c*m_5*r_5*np.cos(theta_5) + g*m_5*r_5*np.sin(theta_5) - l_1*m_5*r_5*(dtheta_5**2*l_1*m_5*r_5*np.sin(theta_5 - theta_1) + dtheta_4*(dtheta_4*l_1*l_4*m_5*np.sin(theta_4 - theta_1) + dtheta_4*l_1*m_4*r_4*np.sin(theta_4 - theta_1)) + dtheta_3*(dtheta_3*l_1*l_3*m_4*np.sin(theta_3 - theta_1) + dtheta_3*l_1*l_3*m_5*np.sin(theta_3 - theta_1) + dtheta_3*l_1*m_3*r_3*np.sin(theta_3 - theta_1)) + dtheta_2*mu_2 + dtheta_2*(dtheta_2*l_1*l_2*m_3*np.sin(theta_2 - theta_1) + dtheta_2*l_1*l_2*m_4*np.sin(theta_2 - theta_1) + dtheta_2*l_1*l_2*m_5*np.sin(theta_2 - theta_1) + dtheta_2*l_1*m_2*r_2*np.sin(theta_2 - theta_1)) - dtheta_1*(mu_1 + mu_2) - ddx_c*(-l_1*m_2 - l_1*m_3 - l_1*m_4 - l_1*m_5 - m_1*r_1)*np.cos(theta_1) - g*(-l_1*m_2 - l_1*m_3 - l_1*m_4 - l_1*m_5 - m_1*r_1)*np.sin(theta_1))*np.cos(theta_5 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) - (-l_1**2*m_5*r_5*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_5 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*m_5*r_5*np.cos(theta_5 - theta_2))*(dtheta_5**2*l_2*m_5*r_5*np.sin(theta_5 - theta_2) + dtheta_4*(dtheta_4*l_2*l_4*m_5*np.sin(theta_4 - theta_2) + dtheta_4*l_2*m_4*r_4*np.sin(theta_4 - theta_2)) + dtheta_3*mu_3 + dtheta_3*(dtheta_3*l_2*l_3*m_4*np.sin(theta_3 - theta_2) + dtheta_3*l_2*l_3*m_5*np.sin(theta_3 - theta_2) + dtheta_3*l_2*m_3*r_3*np.sin(theta_3 - theta_2)) - dtheta_2*(mu_2 + mu_3) + dtheta_1*mu_2 + dtheta_1*(-dtheta_1*l_1*l_2*m_3*np.sin(theta_2 - theta_1) - dtheta_1*l_1*l_2*m_4*np.sin(theta_2 - theta_1) - dtheta_1*l_1*l_2*m_5*np.sin(theta_2 - theta_1) - dtheta_1*l_1*m_2*r_2*np.sin(theta_2 - theta_1)) - ddx_c*(-l_2*m_3 - l_2*m_4 - l_2*m_5 - m_2*r_2)*np.cos(theta_2) - g*(-l_2*m_3 - l_2*m_4 - l_2*m_5 - m_2*r_2)*np.sin(theta_2) - l_1*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*(dtheta_5**2*l_1*m_5*r_5*np.sin(theta_5 - theta_1) + dtheta_4*(dtheta_4*l_1*l_4*m_5*np.sin(theta_4 - theta_1) + dtheta_4*l_1*m_4*r_4*np.sin(theta_4 - theta_1)) + dtheta_3*(dtheta_3*l_1*l_3*m_4*np.sin(theta_3 - theta_1) + dtheta_3*l_1*l_3*m_5*np.sin(theta_3 - theta_1) + dtheta_3*l_1*m_3*r_3*np.sin(theta_3 - theta_1)) + dtheta_2*mu_2 + dtheta_2*(dtheta_2*l_1*l_2*m_3*np.sin(theta_2 - theta_1) + dtheta_2*l_1*l_2*m_4*np.sin(theta_2 - theta_1) + dtheta_2*l_1*l_2*m_5*np.sin(theta_2 - theta_1) + dtheta_2*l_1*m_2*r_2*np.sin(theta_2 - theta_1)) - dtheta_1*(mu_1 + mu_2) - ddx_c*(-l_1*m_2 - l_1*m_3 - l_1*m_4 - l_1*m_5 - m_1*r_1)*np.cos(theta_1) - g*(-l_1*m_2 - l_1*m_3 - l_1*m_4 - l_1*m_5 - m_1*r_1)*np.sin(theta_1))*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2))/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2) - (-l_1**2*m_5*r_5*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_5 - theta_1)*np.cos(theta_3 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_3*m_5*r_5*np.cos(theta_5 - theta_3) - (-l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_3 - theta_2))*(-l_1**2*m_5*r_5*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_5 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*m_5*r_5*np.cos(theta_5 - theta_2))/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2))*(dtheta_5**2*l_3*m_5*r_5*np.sin(theta_5 - theta_3) + dtheta_4*mu_4 + dtheta_4*(dtheta_4*l_3*l_4*m_5*np.sin(theta_4 - theta_3) + dtheta_4*l_3*m_4*r_4*np.sin(theta_4 - theta_3)) - dtheta_3*(mu_3 + mu_4) + dtheta_2*mu_3 + dtheta_2*(-dtheta_2*l_2*l_3*m_4*np.sin(theta_3 - theta_2) - dtheta_2*l_2*l_3*m_5*np.sin(theta_3 - theta_2) - dtheta_2*l_2*m_3*r_3*np.sin(theta_3 - theta_2)) + dtheta_1*(-dtheta_1*l_1*l_3*m_4*np.sin(theta_3 - theta_1) - dtheta_1*l_1*l_3*m_5*np.sin(theta_3 - theta_1) - dtheta_1*l_1*m_3*r_3*np.sin(theta_3 - theta_1)) - ddx_c*(-l_3*m_4 - l_3*m_5 - m_3*r_3)*np.cos(theta_3) - g*(-l_3*m_4 - l_3*m_5 - m_3*r_3)*np.sin(theta_3) - l_1*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(dtheta_5**2*l_1*m_5*r_5*np.sin(theta_5 - theta_1) + dtheta_4*(dtheta_4*l_1*l_4*m_5*np.sin(theta_4 - theta_1) + dtheta_4*l_1*m_4*r_4*np.sin(theta_4 - theta_1)) + dtheta_3*(dtheta_3*l_1*l_3*m_4*np.sin(theta_3 - theta_1) + dtheta_3*l_1*l_3*m_5*np.sin(theta_3 - theta_1) + dtheta_3*l_1*m_3*r_3*np.sin(theta_3 - theta_1)) + dtheta_2*mu_2 + dtheta_2*(dtheta_2*l_1*l_2*m_3*np.sin(theta_2 - theta_1) + dtheta_2*l_1*l_2*m_4*np.sin(theta_2 - theta_1) + dtheta_2*l_1*l_2*m_5*np.sin(theta_2 - theta_1) + dtheta_2*l_1*m_2*r_2*np.sin(theta_2 - theta_1)) - dtheta_1*(mu_1 + mu_2) - ddx_c*(-l_1*m_2 - l_1*m_3 - l_1*m_4 - l_1*m_5 - m_1*r_1)*np.cos(theta_1) - g*(-l_1*m_2 - l_1*m_3 - l_1*m_4 - l_1*m_5 - m_1*r_1)*np.sin(theta_1))*np.cos(theta_3 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) - (-l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_3 - theta_2))*(dtheta_5**2*l_2*m_5*r_5*np.sin(theta_5 - theta_2) + dtheta_4*(dtheta_4*l_2*l_4*m_5*np.sin(theta_4 - theta_2) + dtheta_4*l_2*m_4*r_4*np.sin(theta_4 - theta_2)) + dtheta_3*mu_3 + dtheta_3*(dtheta_3*l_2*l_3*m_4*np.sin(theta_3 - theta_2) + dtheta_3*l_2*l_3*m_5*np.sin(theta_3 - theta_2) + dtheta_3*l_2*m_3*r_3*np.sin(theta_3 - theta_2)) - dtheta_2*(mu_2 + mu_3) + dtheta_1*mu_2 + dtheta_1*(-dtheta_1*l_1*l_2*m_3*np.sin(theta_2 - theta_1) - dtheta_1*l_1*l_2*m_4*np.sin(theta_2 - theta_1) - dtheta_1*l_1*l_2*m_5*np.sin(theta_2 - theta_1) - dtheta_1*l_1*m_2*r_2*np.sin(theta_2 - theta_1)) - ddx_c*(-l_2*m_3 - l_2*m_4 - l_2*m_5 - m_2*r_2)*np.cos(theta_2) - g*(-l_2*m_3 - l_2*m_4 - l_2*m_5 - m_2*r_2)*np.sin(theta_2) - l_1*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*(dtheta_5**2*l_1*m_5*r_5*np.sin(theta_5 - theta_1) + dtheta_4*(dtheta_4*l_1*l_4*m_5*np.sin(theta_4 - theta_1) + dtheta_4*l_1*m_4*r_4*np.sin(theta_4 - theta_1)) + dtheta_3*(dtheta_3*l_1*l_3*m_4*np.sin(theta_3 - theta_1) + dtheta_3*l_1*l_3*m_5*np.sin(theta_3 - theta_1) + dtheta_3*l_1*m_3*r_3*np.sin(theta_3 - theta_1)) + dtheta_2*mu_2 + dtheta_2*(dtheta_2*l_1*l_2*m_3*np.sin(theta_2 - theta_1) + dtheta_2*l_1*l_2*m_4*np.sin(theta_2 - theta_1) + dtheta_2*l_1*l_2*m_5*np.sin(theta_2 - theta_1) + dtheta_2*l_1*m_2*r_2*np.sin(theta_2 - theta_1)) - dtheta_1*(mu_1 + mu_2) - ddx_c*(-l_1*m_2 - l_1*m_3 - l_1*m_4 - l_1*m_5 - m_1*r_1)*np.cos(theta_1) - g*(-l_1*m_2 - l_1*m_3 - l_1*m_4 - l_1*m_5 - m_1*r_1)*np.sin(theta_1))*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2))/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2))/(I_3 - l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)**2*np.cos(theta_3 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_3**2*m_4 + l_3**2*m_5 + m_3*r_3**2 - (-l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_3 - theta_2))**2/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2)) - (-l_1**2*m_5*r_5*(l_4*m_5 + m_4*r_4)*np.cos(theta_5 - theta_1)*np.cos(theta_4 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_4*m_5*r_5*np.cos(theta_5 - theta_4) - (-l_1**2*(l_4*m_5 + m_4*r_4)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_4 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_4*m_5 + m_4*r_4)*np.cos(theta_4 - theta_2))*(-l_1**2*m_5*r_5*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_5 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*m_5*r_5*np.cos(theta_5 - theta_2))/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2) - (-l_1**2*(l_4*m_5 + m_4*r_4)*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_4 - theta_1)*np.cos(theta_3 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_3*(l_4*m_5 + m_4*r_4)*np.cos(theta_4 - theta_3) - (-l_1**2*(l_4*m_5 + m_4*r_4)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_4 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_4*m_5 + m_4*r_4)*np.cos(theta_4 - theta_2))*(-l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_3 - theta_2))/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2))*(-l_1**2*m_5*r_5*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_5 - theta_1)*np.cos(theta_3 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_3*m_5*r_5*np.cos(theta_5 - theta_3) - (-l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_3 - theta_2))*(-l_1**2*m_5*r_5*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_5 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*m_5*r_5*np.cos(theta_5 - theta_2))/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2))/(I_3 - l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)**2*np.cos(theta_3 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_3**2*m_4 + l_3**2*m_5 + m_3*r_3**2 - (-l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_3 - theta_2))**2/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2)))*(dtheta_5**2*l_4*m_5*r_5*np.sin(theta_5 - theta_4) + dtheta_5*mu_5 - dtheta_4*(mu_4 + mu_5) + dtheta_3*mu_4 + dtheta_3*(-dtheta_3*l_3*l_4*m_5*np.sin(theta_4 - theta_3) - dtheta_3*l_3*m_4*r_4*np.sin(theta_4 - theta_3)) + dtheta_2*(-dtheta_2*l_2*l_4*m_5*np.sin(theta_4 - theta_2) - dtheta_2*l_2*m_4*r_4*np.sin(theta_4 - theta_2)) + dtheta_1*(-dtheta_1*l_1*l_4*m_5*np.sin(theta_4 - theta_1) - dtheta_1*l_1*m_4*r_4*np.sin(theta_4 - theta_1)) - ddx_c*(-l_4*m_5 - m_4*r_4)*np.cos(theta_4) - g*(-l_4*m_5 - m_4*r_4)*np.sin(theta_4) - l_1*(l_4*m_5 + m_4*r_4)*(dtheta_5**2*l_1*m_5*r_5*np.sin(theta_5 - theta_1) + dtheta_4*(dtheta_4*l_1*l_4*m_5*np.sin(theta_4 - theta_1) + dtheta_4*l_1*m_4*r_4*np.sin(theta_4 - theta_1)) + dtheta_3*(dtheta_3*l_1*l_3*m_4*np.sin(theta_3 - theta_1) + dtheta_3*l_1*l_3*m_5*np.sin(theta_3 - theta_1) + dtheta_3*l_1*m_3*r_3*np.sin(theta_3 - theta_1)) + dtheta_2*mu_2 + dtheta_2*(dtheta_2*l_1*l_2*m_3*np.sin(theta_2 - theta_1) + dtheta_2*l_1*l_2*m_4*np.sin(theta_2 - theta_1) + dtheta_2*l_1*l_2*m_5*np.sin(theta_2 - theta_1) + dtheta_2*l_1*m_2*r_2*np.sin(theta_2 - theta_1)) - dtheta_1*(mu_1 + mu_2) - ddx_c*(-l_1*m_2 - l_1*m_3 - l_1*m_4 - l_1*m_5 - m_1*r_1)*np.cos(theta_1) - g*(-l_1*m_2 - l_1*m_3 - l_1*m_4 - l_1*m_5 - m_1*r_1)*np.sin(theta_1))*np.cos(theta_4 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) - (-l_1**2*(l_4*m_5 + m_4*r_4)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_4 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_4*m_5 + m_4*r_4)*np.cos(theta_4 - theta_2))*(dtheta_5**2*l_2*m_5*r_5*np.sin(theta_5 - theta_2) + dtheta_4*(dtheta_4*l_2*l_4*m_5*np.sin(theta_4 - theta_2) + dtheta_4*l_2*m_4*r_4*np.sin(theta_4 - theta_2)) + dtheta_3*mu_3 + dtheta_3*(dtheta_3*l_2*l_3*m_4*np.sin(theta_3 - theta_2) + dtheta_3*l_2*l_3*m_5*np.sin(theta_3 - theta_2) + dtheta_3*l_2*m_3*r_3*np.sin(theta_3 - theta_2)) - dtheta_2*(mu_2 + mu_3) + dtheta_1*mu_2 + dtheta_1*(-dtheta_1*l_1*l_2*m_3*np.sin(theta_2 - theta_1) - dtheta_1*l_1*l_2*m_4*np.sin(theta_2 - theta_1) - dtheta_1*l_1*l_2*m_5*np.sin(theta_2 - theta_1) - dtheta_1*l_1*m_2*r_2*np.sin(theta_2 - theta_1)) - ddx_c*(-l_2*m_3 - l_2*m_4 - l_2*m_5 - m_2*r_2)*np.cos(theta_2) - g*(-l_2*m_3 - l_2*m_4 - l_2*m_5 - m_2*r_2)*np.sin(theta_2) - l_1*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*(dtheta_5**2*l_1*m_5*r_5*np.sin(theta_5 - theta_1) + dtheta_4*(dtheta_4*l_1*l_4*m_5*np.sin(theta_4 - theta_1) + dtheta_4*l_1*m_4*r_4*np.sin(theta_4 - theta_1)) + dtheta_3*(dtheta_3*l_1*l_3*m_4*np.sin(theta_3 - theta_1) + dtheta_3*l_1*l_3*m_5*np.sin(theta_3 - theta_1) + dtheta_3*l_1*m_3*r_3*np.sin(theta_3 - theta_1)) + dtheta_2*mu_2 + dtheta_2*(dtheta_2*l_1*l_2*m_3*np.sin(theta_2 - theta_1) + dtheta_2*l_1*l_2*m_4*np.sin(theta_2 - theta_1) + dtheta_2*l_1*l_2*m_5*np.sin(theta_2 - theta_1) + dtheta_2*l_1*m_2*r_2*np.sin(theta_2 - theta_1)) - dtheta_1*(mu_1 + mu_2) - ddx_c*(-l_1*m_2 - l_1*m_3 - l_1*m_4 - l_1*m_5 - m_1*r_1)*np.cos(theta_1) - g*(-l_1*m_2 - l_1*m_3 - l_1*m_4 - l_1*m_5 - m_1*r_1)*np.sin(theta_1))*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2))/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2) - (-l_1**2*(l_4*m_5 + m_4*r_4)*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_4 - theta_1)*np.cos(theta_3 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_3*(l_4*m_5 + m_4*r_4)*np.cos(theta_4 - theta_3) - (-l_1**2*(l_4*m_5 + m_4*r_4)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_4 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_4*m_5 + m_4*r_4)*np.cos(theta_4 - theta_2))*(-l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_3 - theta_2))/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2))*(dtheta_5**2*l_3*m_5*r_5*np.sin(theta_5 - theta_3) + dtheta_4*mu_4 + dtheta_4*(dtheta_4*l_3*l_4*m_5*np.sin(theta_4 - theta_3) + dtheta_4*l_3*m_4*r_4*np.sin(theta_4 - theta_3)) - dtheta_3*(mu_3 + mu_4) + dtheta_2*mu_3 + dtheta_2*(-dtheta_2*l_2*l_3*m_4*np.sin(theta_3 - theta_2) - dtheta_2*l_2*l_3*m_5*np.sin(theta_3 - theta_2) - dtheta_2*l_2*m_3*r_3*np.sin(theta_3 - theta_2)) + dtheta_1*(-dtheta_1*l_1*l_3*m_4*np.sin(theta_3 - theta_1) - dtheta_1*l_1*l_3*m_5*np.sin(theta_3 - theta_1) - dtheta_1*l_1*m_3*r_3*np.sin(theta_3 - theta_1)) - ddx_c*(-l_3*m_4 - l_3*m_5 - m_3*r_3)*np.cos(theta_3) - g*(-l_3*m_4 - l_3*m_5 - m_3*r_3)*np.sin(theta_3) - l_1*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(dtheta_5**2*l_1*m_5*r_5*np.sin(theta_5 - theta_1) + dtheta_4*(dtheta_4*l_1*l_4*m_5*np.sin(theta_4 - theta_1) + dtheta_4*l_1*m_4*r_4*np.sin(theta_4 - theta_1)) + dtheta_3*(dtheta_3*l_1*l_3*m_4*np.sin(theta_3 - theta_1) + dtheta_3*l_1*l_3*m_5*np.sin(theta_3 - theta_1) + dtheta_3*l_1*m_3*r_3*np.sin(theta_3 - theta_1)) + dtheta_2*mu_2 + dtheta_2*(dtheta_2*l_1*l_2*m_3*np.sin(theta_2 - theta_1) + dtheta_2*l_1*l_2*m_4*np.sin(theta_2 - theta_1) + dtheta_2*l_1*l_2*m_5*np.sin(theta_2 - theta_1) + dtheta_2*l_1*m_2*r_2*np.sin(theta_2 - theta_1)) - dtheta_1*(mu_1 + mu_2) - ddx_c*(-l_1*m_2 - l_1*m_3 - l_1*m_4 - l_1*m_5 - m_1*r_1)*np.cos(theta_1) - g*(-l_1*m_2 - l_1*m_3 - l_1*m_4 - l_1*m_5 - m_1*r_1)*np.sin(theta_1))*np.cos(theta_3 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) - (-l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_3 - theta_2))*(dtheta_5**2*l_2*m_5*r_5*np.sin(theta_5 - theta_2) + dtheta_4*(dtheta_4*l_2*l_4*m_5*np.sin(theta_4 - theta_2) + dtheta_4*l_2*m_4*r_4*np.sin(theta_4 - theta_2)) + dtheta_3*mu_3 + dtheta_3*(dtheta_3*l_2*l_3*m_4*np.sin(theta_3 - theta_2) + dtheta_3*l_2*l_3*m_5*np.sin(theta_3 - theta_2) + dtheta_3*l_2*m_3*r_3*np.sin(theta_3 - theta_2)) - dtheta_2*(mu_2 + mu_3) + dtheta_1*mu_2 + dtheta_1*(-dtheta_1*l_1*l_2*m_3*np.sin(theta_2 - theta_1) - dtheta_1*l_1*l_2*m_4*np.sin(theta_2 - theta_1) - dtheta_1*l_1*l_2*m_5*np.sin(theta_2 - theta_1) - dtheta_1*l_1*m_2*r_2*np.sin(theta_2 - theta_1)) - ddx_c*(-l_2*m_3 - l_2*m_4 - l_2*m_5 - m_2*r_2)*np.cos(theta_2) - g*(-l_2*m_3 - l_2*m_4 - l_2*m_5 - m_2*r_2)*np.sin(theta_2) - l_1*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*(dtheta_5**2*l_1*m_5*r_5*np.sin(theta_5 - theta_1) + dtheta_4*(dtheta_4*l_1*l_4*m_5*np.sin(theta_4 - theta_1) + dtheta_4*l_1*m_4*r_4*np.sin(theta_4 - theta_1)) + dtheta_3*(dtheta_3*l_1*l_3*m_4*np.sin(theta_3 - theta_1) + dtheta_3*l_1*l_3*m_5*np.sin(theta_3 - theta_1) + dtheta_3*l_1*m_3*r_3*np.sin(theta_3 - theta_1)) + dtheta_2*mu_2 + dtheta_2*(dtheta_2*l_1*l_2*m_3*np.sin(theta_2 - theta_1) + dtheta_2*l_1*l_2*m_4*np.sin(theta_2 - theta_1) + dtheta_2*l_1*l_2*m_5*np.sin(theta_2 - theta_1) + dtheta_2*l_1*m_2*r_2*np.sin(theta_2 - theta_1)) - dtheta_1*(mu_1 + mu_2) - ddx_c*(-l_1*m_2 - l_1*m_3 - l_1*m_4 - l_1*m_5 - m_1*r_1)*np.cos(theta_1) - g*(-l_1*m_2 - l_1*m_3 - l_1*m_4 - l_1*m_5 - m_1*r_1)*np.sin(theta_1))*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2))/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2))/(I_3 - l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)**2*np.cos(theta_3 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_3**2*m_4 + l_3**2*m_5 + m_3*r_3**2 - (-l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_3 - theta_2))**2/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2)))/(I_4 - l_1**2*(l_4*m_5 + m_4*r_4)**2*np.cos(theta_4 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_4**2*m_5 + m_4*r_4**2 - (-l_1**2*(l_4*m_5 + m_4*r_4)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_4 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_4*m_5 + m_4*r_4)*np.cos(theta_4 - theta_2))**2/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2) - (-l_1**2*(l_4*m_5 + m_4*r_4)*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_4 - theta_1)*np.cos(theta_3 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_3*(l_4*m_5 + m_4*r_4)*np.cos(theta_4 - theta_3) - (-l_1**2*(l_4*m_5 + m_4*r_4)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_4 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_4*m_5 + m_4*r_4)*np.cos(theta_4 - theta_2))*(-l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_3 - theta_2))/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2))**2/(I_3 - l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)**2*np.cos(theta_3 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_3**2*m_4 + l_3**2*m_5 + m_3*r_3**2 - (-l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_3 - theta_2))**2/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2))))/(I_5 - l_1**2*m_5**2*r_5**2*np.cos(theta_5 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + m_5*r_5**2 - (-l_1**2*m_5*r_5*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_5 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*m_5*r_5*np.cos(theta_5 - theta_2))**2/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2) - (-l_1**2*m_5*r_5*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_5 - theta_1)*np.cos(theta_3 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_3*m_5*r_5*np.cos(theta_5 - theta_3) - (-l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_3 - theta_2))*(-l_1**2*m_5*r_5*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_5 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*m_5*r_5*np.cos(theta_5 - theta_2))/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2))**2/(I_3 - l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)**2*np.cos(theta_3 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_3**2*m_4 + l_3**2*m_5 + m_3*r_3**2 - (-l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_3 - theta_2))**2/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2)) - (-l_1**2*m_5*r_5*(l_4*m_5 + m_4*r_4)*np.cos(theta_5 - theta_1)*np.cos(theta_4 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_4*m_5*r_5*np.cos(theta_5 - theta_4) - (-l_1**2*(l_4*m_5 + m_4*r_4)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_4 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_4*m_5 + m_4*r_4)*np.cos(theta_4 - theta_2))*(-l_1**2*m_5*r_5*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_5 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*m_5*r_5*np.cos(theta_5 - theta_2))/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2) - (-l_1**2*(l_4*m_5 + m_4*r_4)*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_4 - theta_1)*np.cos(theta_3 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_3*(l_4*m_5 + m_4*r_4)*np.cos(theta_4 - theta_3) - (-l_1**2*(l_4*m_5 + m_4*r_4)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_4 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_4*m_5 + m_4*r_4)*np.cos(theta_4 - theta_2))*(-l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_3 - theta_2))/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2))*(-l_1**2*m_5*r_5*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_5 - theta_1)*np.cos(theta_3 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_3*m_5*r_5*np.cos(theta_5 - theta_3) - (-l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_3 - theta_2))*(-l_1**2*m_5*r_5*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_5 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*m_5*r_5*np.cos(theta_5 - theta_2))/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2))/(I_3 - l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)**2*np.cos(theta_3 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_3**2*m_4 + l_3**2*m_5 + m_3*r_3**2 - (-l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_3 - theta_2))**2/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2)))**2/(I_4 - l_1**2*(l_4*m_5 + m_4*r_4)**2*np.cos(theta_4 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_4**2*m_5 + m_4*r_4**2 - (-l_1**2*(l_4*m_5 + m_4*r_4)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_4 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_4*m_5 + m_4*r_4)*np.cos(theta_4 - theta_2))**2/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2) - (-l_1**2*(l_4*m_5 + m_4*r_4)*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_4 - theta_1)*np.cos(theta_3 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_3*(l_4*m_5 + m_4*r_4)*np.cos(theta_4 - theta_3) - (-l_1**2*(l_4*m_5 + m_4*r_4)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_4 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_4*m_5 + m_4*r_4)*np.cos(theta_4 - theta_2))*(-l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_3 - theta_2))/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2))**2/(I_3 - l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)**2*np.cos(theta_3 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_3**2*m_4 + l_3**2*m_5 + m_3*r_3**2 - (-l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_3 - theta_2))**2/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2)))))/(I_3 - l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)**2*np.cos(theta_3 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_3**2*m_4 + l_3**2*m_5 + m_3*r_3**2 - (-l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_3 - theta_2))**2/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2)) - (-l_1**2*m_5*r_5*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_5 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*m_5*r_5*np.cos(theta_5 - theta_2))*(-dtheta_5*mu_5 - dtheta_4**2*l_4*m_5*r_5*np.sin(theta_5 - theta_4) + dtheta_4*mu_5 - dtheta_3**2*l_3*m_5*r_5*np.sin(theta_5 - theta_3) - dtheta_2**2*l_2*m_5*r_5*np.sin(theta_5 - theta_2) - dtheta_1**2*l_1*m_5*r_5*np.sin(theta_5 - theta_1) + ddx_c*m_5*r_5*np.cos(theta_5) + g*m_5*r_5*np.sin(theta_5) - l_1*m_5*r_5*(dtheta_5**2*l_1*m_5*r_5*np.sin(theta_5 - theta_1) + dtheta_4*(dtheta_4*l_1*l_4*m_5*np.sin(theta_4 - theta_1) + dtheta_4*l_1*m_4*r_4*np.sin(theta_4 - theta_1)) + dtheta_3*(dtheta_3*l_1*l_3*m_4*np.sin(theta_3 - theta_1) + dtheta_3*l_1*l_3*m_5*np.sin(theta_3 - theta_1) + dtheta_3*l_1*m_3*r_3*np.sin(theta_3 - theta_1)) + dtheta_2*mu_2 + dtheta_2*(dtheta_2*l_1*l_2*m_3*np.sin(theta_2 - theta_1) + dtheta_2*l_1*l_2*m_4*np.sin(theta_2 - theta_1) + dtheta_2*l_1*l_2*m_5*np.sin(theta_2 - theta_1) + dtheta_2*l_1*m_2*r_2*np.sin(theta_2 - theta_1)) - dtheta_1*(mu_1 + mu_2) - ddx_c*(-l_1*m_2 - l_1*m_3 - l_1*m_4 - l_1*m_5 - m_1*r_1)*np.cos(theta_1) - g*(-l_1*m_2 - l_1*m_3 - l_1*m_4 - l_1*m_5 - m_1*r_1)*np.sin(theta_1))*np.cos(theta_5 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) - (-l_1**2*m_5*r_5*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_5 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*m_5*r_5*np.cos(theta_5 - theta_2))*(dtheta_5**2*l_2*m_5*r_5*np.sin(theta_5 - theta_2) + dtheta_4*(dtheta_4*l_2*l_4*m_5*np.sin(theta_4 - theta_2) + dtheta_4*l_2*m_4*r_4*np.sin(theta_4 - theta_2)) + dtheta_3*mu_3 + dtheta_3*(dtheta_3*l_2*l_3*m_4*np.sin(theta_3 - theta_2) + dtheta_3*l_2*l_3*m_5*np.sin(theta_3 - theta_2) + dtheta_3*l_2*m_3*r_3*np.sin(theta_3 - theta_2)) - dtheta_2*(mu_2 + mu_3) + dtheta_1*mu_2 + dtheta_1*(-dtheta_1*l_1*l_2*m_3*np.sin(theta_2 - theta_1) - dtheta_1*l_1*l_2*m_4*np.sin(theta_2 - theta_1) - dtheta_1*l_1*l_2*m_5*np.sin(theta_2 - theta_1) - dtheta_1*l_1*m_2*r_2*np.sin(theta_2 - theta_1)) - ddx_c*(-l_2*m_3 - l_2*m_4 - l_2*m_5 - m_2*r_2)*np.cos(theta_2) - g*(-l_2*m_3 - l_2*m_4 - l_2*m_5 - m_2*r_2)*np.sin(theta_2) - l_1*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*(dtheta_5**2*l_1*m_5*r_5*np.sin(theta_5 - theta_1) + dtheta_4*(dtheta_4*l_1*l_4*m_5*np.sin(theta_4 - theta_1) + dtheta_4*l_1*m_4*r_4*np.sin(theta_4 - theta_1)) + dtheta_3*(dtheta_3*l_1*l_3*m_4*np.sin(theta_3 - theta_1) + dtheta_3*l_1*l_3*m_5*np.sin(theta_3 - theta_1) + dtheta_3*l_1*m_3*r_3*np.sin(theta_3 - theta_1)) + dtheta_2*mu_2 + dtheta_2*(dtheta_2*l_1*l_2*m_3*np.sin(theta_2 - theta_1) + dtheta_2*l_1*l_2*m_4*np.sin(theta_2 - theta_1) + dtheta_2*l_1*l_2*m_5*np.sin(theta_2 - theta_1) + dtheta_2*l_1*m_2*r_2*np.sin(theta_2 - theta_1)) - dtheta_1*(mu_1 + mu_2) - ddx_c*(-l_1*m_2 - l_1*m_3 - l_1*m_4 - l_1*m_5 - m_1*r_1)*np.cos(theta_1) - g*(-l_1*m_2 - l_1*m_3 - l_1*m_4 - l_1*m_5 - m_1*r_1)*np.sin(theta_1))*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2))/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2) - (-l_1**2*m_5*r_5*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_5 - theta_1)*np.cos(theta_3 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_3*m_5*r_5*np.cos(theta_5 - theta_3) - (-l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_3 - theta_2))*(-l_1**2*m_5*r_5*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_5 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*m_5*r_5*np.cos(theta_5 - theta_2))/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2))*(dtheta_5**2*l_3*m_5*r_5*np.sin(theta_5 - theta_3) + dtheta_4*mu_4 + dtheta_4*(dtheta_4*l_3*l_4*m_5*np.sin(theta_4 - theta_3) + dtheta_4*l_3*m_4*r_4*np.sin(theta_4 - theta_3)) - dtheta_3*(mu_3 + mu_4) + dtheta_2*mu_3 + dtheta_2*(-dtheta_2*l_2*l_3*m_4*np.sin(theta_3 - theta_2) - dtheta_2*l_2*l_3*m_5*np.sin(theta_3 - theta_2) - dtheta_2*l_2*m_3*r_3*np.sin(theta_3 - theta_2)) + dtheta_1*(-dtheta_1*l_1*l_3*m_4*np.sin(theta_3 - theta_1) - dtheta_1*l_1*l_3*m_5*np.sin(theta_3 - theta_1) - dtheta_1*l_1*m_3*r_3*np.sin(theta_3 - theta_1)) - ddx_c*(-l_3*m_4 - l_3*m_5 - m_3*r_3)*np.cos(theta_3) - g*(-l_3*m_4 - l_3*m_5 - m_3*r_3)*np.sin(theta_3) - l_1*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(dtheta_5**2*l_1*m_5*r_5*np.sin(theta_5 - theta_1) + dtheta_4*(dtheta_4*l_1*l_4*m_5*np.sin(theta_4 - theta_1) + dtheta_4*l_1*m_4*r_4*np.sin(theta_4 - theta_1)) + dtheta_3*(dtheta_3*l_1*l_3*m_4*np.sin(theta_3 - theta_1) + dtheta_3*l_1*l_3*m_5*np.sin(theta_3 - theta_1) + dtheta_3*l_1*m_3*r_3*np.sin(theta_3 - theta_1)) + dtheta_2*mu_2 + dtheta_2*(dtheta_2*l_1*l_2*m_3*np.sin(theta_2 - theta_1) + dtheta_2*l_1*l_2*m_4*np.sin(theta_2 - theta_1) + dtheta_2*l_1*l_2*m_5*np.sin(theta_2 - theta_1) + dtheta_2*l_1*m_2*r_2*np.sin(theta_2 - theta_1)) - dtheta_1*(mu_1 + mu_2) - ddx_c*(-l_1*m_2 - l_1*m_3 - l_1*m_4 - l_1*m_5 - m_1*r_1)*np.cos(theta_1) - g*(-l_1*m_2 - l_1*m_3 - l_1*m_4 - l_1*m_5 - m_1*r_1)*np.sin(theta_1))*np.cos(theta_3 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) - (-l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_3 - theta_2))*(dtheta_5**2*l_2*m_5*r_5*np.sin(theta_5 - theta_2) + dtheta_4*(dtheta_4*l_2*l_4*m_5*np.sin(theta_4 - theta_2) + dtheta_4*l_2*m_4*r_4*np.sin(theta_4 - theta_2)) + dtheta_3*mu_3 + dtheta_3*(dtheta_3*l_2*l_3*m_4*np.sin(theta_3 - theta_2) + dtheta_3*l_2*l_3*m_5*np.sin(theta_3 - theta_2) + dtheta_3*l_2*m_3*r_3*np.sin(theta_3 - theta_2)) - dtheta_2*(mu_2 + mu_3) + dtheta_1*mu_2 + dtheta_1*(-dtheta_1*l_1*l_2*m_3*np.sin(theta_2 - theta_1) - dtheta_1*l_1*l_2*m_4*np.sin(theta_2 - theta_1) - dtheta_1*l_1*l_2*m_5*np.sin(theta_2 - theta_1) - dtheta_1*l_1*m_2*r_2*np.sin(theta_2 - theta_1)) - ddx_c*(-l_2*m_3 - l_2*m_4 - l_2*m_5 - m_2*r_2)*np.cos(theta_2) - g*(-l_2*m_3 - l_2*m_4 - l_2*m_5 - m_2*r_2)*np.sin(theta_2) - l_1*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*(dtheta_5**2*l_1*m_5*r_5*np.sin(theta_5 - theta_1) + dtheta_4*(dtheta_4*l_1*l_4*m_5*np.sin(theta_4 - theta_1) + dtheta_4*l_1*m_4*r_4*np.sin(theta_4 - theta_1)) + dtheta_3*(dtheta_3*l_1*l_3*m_4*np.sin(theta_3 - theta_1) + dtheta_3*l_1*l_3*m_5*np.sin(theta_3 - theta_1) + dtheta_3*l_1*m_3*r_3*np.sin(theta_3 - theta_1)) + dtheta_2*mu_2 + dtheta_2*(dtheta_2*l_1*l_2*m_3*np.sin(theta_2 - theta_1) + dtheta_2*l_1*l_2*m_4*np.sin(theta_2 - theta_1) + dtheta_2*l_1*l_2*m_5*np.sin(theta_2 - theta_1) + dtheta_2*l_1*m_2*r_2*np.sin(theta_2 - theta_1)) - dtheta_1*(mu_1 + mu_2) - ddx_c*(-l_1*m_2 - l_1*m_3 - l_1*m_4 - l_1*m_5 - m_1*r_1)*np.cos(theta_1) - g*(-l_1*m_2 - l_1*m_3 - l_1*m_4 - l_1*m_5 - m_1*r_1)*np.sin(theta_1))*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2))/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2))/(I_3 - l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)**2*np.cos(theta_3 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_3**2*m_4 + l_3**2*m_5 + m_3*r_3**2 - (-l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_3 - theta_2))**2/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2)) - (-l_1**2*m_5*r_5*(l_4*m_5 + m_4*r_4)*np.cos(theta_5 - theta_1)*np.cos(theta_4 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_4*m_5*r_5*np.cos(theta_5 - theta_4) - (-l_1**2*(l_4*m_5 + m_4*r_4)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_4 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_4*m_5 + m_4*r_4)*np.cos(theta_4 - theta_2))*(-l_1**2*m_5*r_5*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_5 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*m_5*r_5*np.cos(theta_5 - theta_2))/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2) - (-l_1**2*(l_4*m_5 + m_4*r_4)*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_4 - theta_1)*np.cos(theta_3 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_3*(l_4*m_5 + m_4*r_4)*np.cos(theta_4 - theta_3) - (-l_1**2*(l_4*m_5 + m_4*r_4)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_4 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_4*m_5 + m_4*r_4)*np.cos(theta_4 - theta_2))*(-l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_3 - theta_2))/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2))*(-l_1**2*m_5*r_5*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_5 - theta_1)*np.cos(theta_3 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_3*m_5*r_5*np.cos(theta_5 - theta_3) - (-l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_3 - theta_2))*(-l_1**2*m_5*r_5*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_5 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*m_5*r_5*np.cos(theta_5 - theta_2))/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2))/(I_3 - l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)**2*np.cos(theta_3 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_3**2*m_4 + l_3**2*m_5 + m_3*r_3**2 - (-l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_3 - theta_2))**2/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2)))*(dtheta_5**2*l_4*m_5*r_5*np.sin(theta_5 - theta_4) + dtheta_5*mu_5 - dtheta_4*(mu_4 + mu_5) + dtheta_3*mu_4 + dtheta_3*(-dtheta_3*l_3*l_4*m_5*np.sin(theta_4 - theta_3) - dtheta_3*l_3*m_4*r_4*np.sin(theta_4 - theta_3)) + dtheta_2*(-dtheta_2*l_2*l_4*m_5*np.sin(theta_4 - theta_2) - dtheta_2*l_2*m_4*r_4*np.sin(theta_4 - theta_2)) + dtheta_1*(-dtheta_1*l_1*l_4*m_5*np.sin(theta_4 - theta_1) - dtheta_1*l_1*m_4*r_4*np.sin(theta_4 - theta_1)) - ddx_c*(-l_4*m_5 - m_4*r_4)*np.cos(theta_4) - g*(-l_4*m_5 - m_4*r_4)*np.sin(theta_4) - l_1*(l_4*m_5 + m_4*r_4)*(dtheta_5**2*l_1*m_5*r_5*np.sin(theta_5 - theta_1) + dtheta_4*(dtheta_4*l_1*l_4*m_5*np.sin(theta_4 - theta_1) + dtheta_4*l_1*m_4*r_4*np.sin(theta_4 - theta_1)) + dtheta_3*(dtheta_3*l_1*l_3*m_4*np.sin(theta_3 - theta_1) + dtheta_3*l_1*l_3*m_5*np.sin(theta_3 - theta_1) + dtheta_3*l_1*m_3*r_3*np.sin(theta_3 - theta_1)) + dtheta_2*mu_2 + dtheta_2*(dtheta_2*l_1*l_2*m_3*np.sin(theta_2 - theta_1) + dtheta_2*l_1*l_2*m_4*np.sin(theta_2 - theta_1) + dtheta_2*l_1*l_2*m_5*np.sin(theta_2 - theta_1) + dtheta_2*l_1*m_2*r_2*np.sin(theta_2 - theta_1)) - dtheta_1*(mu_1 + mu_2) - ddx_c*(-l_1*m_2 - l_1*m_3 - l_1*m_4 - l_1*m_5 - m_1*r_1)*np.cos(theta_1) - g*(-l_1*m_2 - l_1*m_3 - l_1*m_4 - l_1*m_5 - m_1*r_1)*np.sin(theta_1))*np.cos(theta_4 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) - (-l_1**2*(l_4*m_5 + m_4*r_4)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_4 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_4*m_5 + m_4*r_4)*np.cos(theta_4 - theta_2))*(dtheta_5**2*l_2*m_5*r_5*np.sin(theta_5 - theta_2) + dtheta_4*(dtheta_4*l_2*l_4*m_5*np.sin(theta_4 - theta_2) + dtheta_4*l_2*m_4*r_4*np.sin(theta_4 - theta_2)) + dtheta_3*mu_3 + dtheta_3*(dtheta_3*l_2*l_3*m_4*np.sin(theta_3 - theta_2) + dtheta_3*l_2*l_3*m_5*np.sin(theta_3 - theta_2) + dtheta_3*l_2*m_3*r_3*np.sin(theta_3 - theta_2)) - dtheta_2*(mu_2 + mu_3) + dtheta_1*mu_2 + dtheta_1*(-dtheta_1*l_1*l_2*m_3*np.sin(theta_2 - theta_1) - dtheta_1*l_1*l_2*m_4*np.sin(theta_2 - theta_1) - dtheta_1*l_1*l_2*m_5*np.sin(theta_2 - theta_1) - dtheta_1*l_1*m_2*r_2*np.sin(theta_2 - theta_1)) - ddx_c*(-l_2*m_3 - l_2*m_4 - l_2*m_5 - m_2*r_2)*np.cos(theta_2) - g*(-l_2*m_3 - l_2*m_4 - l_2*m_5 - m_2*r_2)*np.sin(theta_2) - l_1*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*(dtheta_5**2*l_1*m_5*r_5*np.sin(theta_5 - theta_1) + dtheta_4*(dtheta_4*l_1*l_4*m_5*np.sin(theta_4 - theta_1) + dtheta_4*l_1*m_4*r_4*np.sin(theta_4 - theta_1)) + dtheta_3*(dtheta_3*l_1*l_3*m_4*np.sin(theta_3 - theta_1) + dtheta_3*l_1*l_3*m_5*np.sin(theta_3 - theta_1) + dtheta_3*l_1*m_3*r_3*np.sin(theta_3 - theta_1)) + dtheta_2*mu_2 + dtheta_2*(dtheta_2*l_1*l_2*m_3*np.sin(theta_2 - theta_1) + dtheta_2*l_1*l_2*m_4*np.sin(theta_2 - theta_1) + dtheta_2*l_1*l_2*m_5*np.sin(theta_2 - theta_1) + dtheta_2*l_1*m_2*r_2*np.sin(theta_2 - theta_1)) - dtheta_1*(mu_1 + mu_2) - ddx_c*(-l_1*m_2 - l_1*m_3 - l_1*m_4 - l_1*m_5 - m_1*r_1)*np.cos(theta_1) - g*(-l_1*m_2 - l_1*m_3 - l_1*m_4 - l_1*m_5 - m_1*r_1)*np.sin(theta_1))*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2))/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2) - (-l_1**2*(l_4*m_5 + m_4*r_4)*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_4 - theta_1)*np.cos(theta_3 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_3*(l_4*m_5 + m_4*r_4)*np.cos(theta_4 - theta_3) - (-l_1**2*(l_4*m_5 + m_4*r_4)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_4 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_4*m_5 + m_4*r_4)*np.cos(theta_4 - theta_2))*(-l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_3 - theta_2))/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2))*(dtheta_5**2*l_3*m_5*r_5*np.sin(theta_5 - theta_3) + dtheta_4*mu_4 + dtheta_4*(dtheta_4*l_3*l_4*m_5*np.sin(theta_4 - theta_3) + dtheta_4*l_3*m_4*r_4*np.sin(theta_4 - theta_3)) - dtheta_3*(mu_3 + mu_4) + dtheta_2*mu_3 + dtheta_2*(-dtheta_2*l_2*l_3*m_4*np.sin(theta_3 - theta_2) - dtheta_2*l_2*l_3*m_5*np.sin(theta_3 - theta_2) - dtheta_2*l_2*m_3*r_3*np.sin(theta_3 - theta_2)) + dtheta_1*(-dtheta_1*l_1*l_3*m_4*np.sin(theta_3 - theta_1) - dtheta_1*l_1*l_3*m_5*np.sin(theta_3 - theta_1) - dtheta_1*l_1*m_3*r_3*np.sin(theta_3 - theta_1)) - ddx_c*(-l_3*m_4 - l_3*m_5 - m_3*r_3)*np.cos(theta_3) - g*(-l_3*m_4 - l_3*m_5 - m_3*r_3)*np.sin(theta_3) - l_1*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(dtheta_5**2*l_1*m_5*r_5*np.sin(theta_5 - theta_1) + dtheta_4*(dtheta_4*l_1*l_4*m_5*np.sin(theta_4 - theta_1) + dtheta_4*l_1*m_4*r_4*np.sin(theta_4 - theta_1)) + dtheta_3*(dtheta_3*l_1*l_3*m_4*np.sin(theta_3 - theta_1) + dtheta_3*l_1*l_3*m_5*np.sin(theta_3 - theta_1) + dtheta_3*l_1*m_3*r_3*np.sin(theta_3 - theta_1)) + dtheta_2*mu_2 + dtheta_2*(dtheta_2*l_1*l_2*m_3*np.sin(theta_2 - theta_1) + dtheta_2*l_1*l_2*m_4*np.sin(theta_2 - theta_1) + dtheta_2*l_1*l_2*m_5*np.sin(theta_2 - theta_1) + dtheta_2*l_1*m_2*r_2*np.sin(theta_2 - theta_1)) - dtheta_1*(mu_1 + mu_2) - ddx_c*(-l_1*m_2 - l_1*m_3 - l_1*m_4 - l_1*m_5 - m_1*r_1)*np.cos(theta_1) - g*(-l_1*m_2 - l_1*m_3 - l_1*m_4 - l_1*m_5 - m_1*r_1)*np.sin(theta_1))*np.cos(theta_3 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) - (-l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_3 - theta_2))*(dtheta_5**2*l_2*m_5*r_5*np.sin(theta_5 - theta_2) + dtheta_4*(dtheta_4*l_2*l_4*m_5*np.sin(theta_4 - theta_2) + dtheta_4*l_2*m_4*r_4*np.sin(theta_4 - theta_2)) + dtheta_3*mu_3 + dtheta_3*(dtheta_3*l_2*l_3*m_4*np.sin(theta_3 - theta_2) + dtheta_3*l_2*l_3*m_5*np.sin(theta_3 - theta_2) + dtheta_3*l_2*m_3*r_3*np.sin(theta_3 - theta_2)) - dtheta_2*(mu_2 + mu_3) + dtheta_1*mu_2 + dtheta_1*(-dtheta_1*l_1*l_2*m_3*np.sin(theta_2 - theta_1) - dtheta_1*l_1*l_2*m_4*np.sin(theta_2 - theta_1) - dtheta_1*l_1*l_2*m_5*np.sin(theta_2 - theta_1) - dtheta_1*l_1*m_2*r_2*np.sin(theta_2 - theta_1)) - ddx_c*(-l_2*m_3 - l_2*m_4 - l_2*m_5 - m_2*r_2)*np.cos(theta_2) - g*(-l_2*m_3 - l_2*m_4 - l_2*m_5 - m_2*r_2)*np.sin(theta_2) - l_1*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*(dtheta_5**2*l_1*m_5*r_5*np.sin(theta_5 - theta_1) + dtheta_4*(dtheta_4*l_1*l_4*m_5*np.sin(theta_4 - theta_1) + dtheta_4*l_1*m_4*r_4*np.sin(theta_4 - theta_1)) + dtheta_3*(dtheta_3*l_1*l_3*m_4*np.sin(theta_3 - theta_1) + dtheta_3*l_1*l_3*m_5*np.sin(theta_3 - theta_1) + dtheta_3*l_1*m_3*r_3*np.sin(theta_3 - theta_1)) + dtheta_2*mu_2 + dtheta_2*(dtheta_2*l_1*l_2*m_3*np.sin(theta_2 - theta_1) + dtheta_2*l_1*l_2*m_4*np.sin(theta_2 - theta_1) + dtheta_2*l_1*l_2*m_5*np.sin(theta_2 - theta_1) + dtheta_2*l_1*m_2*r_2*np.sin(theta_2 - theta_1)) - dtheta_1*(mu_1 + mu_2) - ddx_c*(-l_1*m_2 - l_1*m_3 - l_1*m_4 - l_1*m_5 - m_1*r_1)*np.cos(theta_1) - g*(-l_1*m_2 - l_1*m_3 - l_1*m_4 - l_1*m_5 - m_1*r_1)*np.sin(theta_1))*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2))/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2))/(I_3 - l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)**2*np.cos(theta_3 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_3**2*m_4 + l_3**2*m_5 + m_3*r_3**2 - (-l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_3 - theta_2))**2/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2)))/(I_4 - l_1**2*(l_4*m_5 + m_4*r_4)**2*np.cos(theta_4 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_4**2*m_5 + m_4*r_4**2 - (-l_1**2*(l_4*m_5 + m_4*r_4)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_4 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_4*m_5 + m_4*r_4)*np.cos(theta_4 - theta_2))**2/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2) - (-l_1**2*(l_4*m_5 + m_4*r_4)*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_4 - theta_1)*np.cos(theta_3 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_3*(l_4*m_5 + m_4*r_4)*np.cos(theta_4 - theta_3) - (-l_1**2*(l_4*m_5 + m_4*r_4)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_4 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_4*m_5 + m_4*r_4)*np.cos(theta_4 - theta_2))*(-l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_3 - theta_2))/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2))**2/(I_3 - l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)**2*np.cos(theta_3 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_3**2*m_4 + l_3**2*m_5 + m_3*r_3**2 - (-l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_3 - theta_2))**2/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2))))/(I_5 - l_1**2*m_5**2*r_5**2*np.cos(theta_5 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + m_5*r_5**2 - (-l_1**2*m_5*r_5*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_5 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*m_5*r_5*np.cos(theta_5 - theta_2))**2/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2) - (-l_1**2*m_5*r_5*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_5 - theta_1)*np.cos(theta_3 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_3*m_5*r_5*np.cos(theta_5 - theta_3) - (-l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_3 - theta_2))*(-l_1**2*m_5*r_5*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_5 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*m_5*r_5*np.cos(theta_5 - theta_2))/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2))**2/(I_3 - l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)**2*np.cos(theta_3 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_3**2*m_4 + l_3**2*m_5 + m_3*r_3**2 - (-l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_3 - theta_2))**2/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2)) - (-l_1**2*m_5*r_5*(l_4*m_5 + m_4*r_4)*np.cos(theta_5 - theta_1)*np.cos(theta_4 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_4*m_5*r_5*np.cos(theta_5 - theta_4) - (-l_1**2*(l_4*m_5 + m_4*r_4)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_4 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_4*m_5 + m_4*r_4)*np.cos(theta_4 - theta_2))*(-l_1**2*m_5*r_5*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_5 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*m_5*r_5*np.cos(theta_5 - theta_2))/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2) - (-l_1**2*(l_4*m_5 + m_4*r_4)*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_4 - theta_1)*np.cos(theta_3 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_3*(l_4*m_5 + m_4*r_4)*np.cos(theta_4 - theta_3) - (-l_1**2*(l_4*m_5 + m_4*r_4)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_4 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_4*m_5 + m_4*r_4)*np.cos(theta_4 - theta_2))*(-l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_3 - theta_2))/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2))*(-l_1**2*m_5*r_5*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_5 - theta_1)*np.cos(theta_3 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_3*m_5*r_5*np.cos(theta_5 - theta_3) - (-l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_3 - theta_2))*(-l_1**2*m_5*r_5*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_5 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*m_5*r_5*np.cos(theta_5 - theta_2))/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2))/(I_3 - l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)**2*np.cos(theta_3 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_3**2*m_4 + l_3**2*m_5 + m_3*r_3**2 - (-l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_3 - theta_2))**2/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2)))**2/(I_4 - l_1**2*(l_4*m_5 + m_4*r_4)**2*np.cos(theta_4 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_4**2*m_5 + m_4*r_4**2 - (-l_1**2*(l_4*m_5 + m_4*r_4)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_4 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_4*m_5 + m_4*r_4)*np.cos(theta_4 - theta_2))**2/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2) - (-l_1**2*(l_4*m_5 + m_4*r_4)*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_4 - theta_1)*np.cos(theta_3 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_3*(l_4*m_5 + m_4*r_4)*np.cos(theta_4 - theta_3) - (-l_1**2*(l_4*m_5 + m_4*r_4)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_4 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_4*m_5 + m_4*r_4)*np.cos(theta_4 - theta_2))*(-l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_3 - theta_2))/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2))**2/(I_3 - l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)**2*np.cos(theta_3 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_3**2*m_4 + l_3**2*m_5 + m_3*r_3**2 - (-l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_3 - theta_2))**2/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2)))))/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2),


                      (dtheta_5**2*l_3*m_5*r_5*np.sin(theta_5 - theta_3) + dtheta_4*mu_4 + dtheta_4*(dtheta_4*l_3*l_4*m_5*np.sin(theta_4 - theta_3) + dtheta_4*l_3*m_4*r_4*np.sin(theta_4 - theta_3)) - dtheta_3*(mu_3 + mu_4) + dtheta_2*mu_3 + dtheta_2*(-dtheta_2*l_2*l_3*m_4*np.sin(theta_3 - theta_2) - dtheta_2*l_2*l_3*m_5*np.sin(theta_3 - theta_2) - dtheta_2*l_2*m_3*r_3*np.sin(theta_3 - theta_2)) + dtheta_1*(-dtheta_1*l_1*l_3*m_4*np.sin(theta_3 - theta_1) - dtheta_1*l_1*l_3*m_5*np.sin(theta_3 - theta_1) - dtheta_1*l_1*m_3*r_3*np.sin(theta_3 - theta_1)) - ddx_c*(-l_3*m_4 - l_3*m_5 - m_3*r_3)*np.cos(theta_3) - g*(-l_3*m_4 - l_3*m_5 - m_3*r_3)*np.sin(theta_3) - l_1*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(dtheta_5**2*l_1*m_5*r_5*np.sin(theta_5 - theta_1) + dtheta_4*(dtheta_4*l_1*l_4*m_5*np.sin(theta_4 - theta_1) + dtheta_4*l_1*m_4*r_4*np.sin(theta_4 - theta_1)) + dtheta_3*(dtheta_3*l_1*l_3*m_4*np.sin(theta_3 - theta_1) + dtheta_3*l_1*l_3*m_5*np.sin(theta_3 - theta_1) + dtheta_3*l_1*m_3*r_3*np.sin(theta_3 - theta_1)) + dtheta_2*mu_2 + dtheta_2*(dtheta_2*l_1*l_2*m_3*np.sin(theta_2 - theta_1) + dtheta_2*l_1*l_2*m_4*np.sin(theta_2 - theta_1) + dtheta_2*l_1*l_2*m_5*np.sin(theta_2 - theta_1) + dtheta_2*l_1*m_2*r_2*np.sin(theta_2 - theta_1)) - dtheta_1*(mu_1 + mu_2) - ddx_c*(-l_1*m_2 - l_1*m_3 - l_1*m_4 - l_1*m_5 - m_1*r_1)*np.cos(theta_1) - g*(-l_1*m_2 - l_1*m_3 - l_1*m_4 - l_1*m_5 - m_1*r_1)*np.sin(theta_1))*np.cos(theta_3 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) - (-l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_3 - theta_2))*(dtheta_5**2*l_2*m_5*r_5*np.sin(theta_5 - theta_2) + dtheta_4*(dtheta_4*l_2*l_4*m_5*np.sin(theta_4 - theta_2) + dtheta_4*l_2*m_4*r_4*np.sin(theta_4 - theta_2)) + dtheta_3*mu_3 + dtheta_3*(dtheta_3*l_2*l_3*m_4*np.sin(theta_3 - theta_2) + dtheta_3*l_2*l_3*m_5*np.sin(theta_3 - theta_2) + dtheta_3*l_2*m_3*r_3*np.sin(theta_3 - theta_2)) - dtheta_2*(mu_2 + mu_3) + dtheta_1*mu_2 + dtheta_1*(-dtheta_1*l_1*l_2*m_3*np.sin(theta_2 - theta_1) - dtheta_1*l_1*l_2*m_4*np.sin(theta_2 - theta_1) - dtheta_1*l_1*l_2*m_5*np.sin(theta_2 - theta_1) - dtheta_1*l_1*m_2*r_2*np.sin(theta_2 - theta_1)) - ddx_c*(-l_2*m_3 - l_2*m_4 - l_2*m_5 - m_2*r_2)*np.cos(theta_2) - g*(-l_2*m_3 - l_2*m_4 - l_2*m_5 - m_2*r_2)*np.sin(theta_2) - l_1*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*(dtheta_5**2*l_1*m_5*r_5*np.sin(theta_5 - theta_1) + dtheta_4*(dtheta_4*l_1*l_4*m_5*np.sin(theta_4 - theta_1) + dtheta_4*l_1*m_4*r_4*np.sin(theta_4 - theta_1)) + dtheta_3*(dtheta_3*l_1*l_3*m_4*np.sin(theta_3 - theta_1) + dtheta_3*l_1*l_3*m_5*np.sin(theta_3 - theta_1) + dtheta_3*l_1*m_3*r_3*np.sin(theta_3 - theta_1)) + dtheta_2*mu_2 + dtheta_2*(dtheta_2*l_1*l_2*m_3*np.sin(theta_2 - theta_1) + dtheta_2*l_1*l_2*m_4*np.sin(theta_2 - theta_1) + dtheta_2*l_1*l_2*m_5*np.sin(theta_2 - theta_1) + dtheta_2*l_1*m_2*r_2*np.sin(theta_2 - theta_1)) - dtheta_1*(mu_1 + mu_2) - ddx_c*(-l_1*m_2 - l_1*m_3 - l_1*m_4 - l_1*m_5 - m_1*r_1)*np.cos(theta_1) - g*(-l_1*m_2 - l_1*m_3 - l_1*m_4 - l_1*m_5 - m_1*r_1)*np.sin(theta_1))*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2))/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2) - (-l_1**2*(l_4*m_5 + m_4*r_4)*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_4 - theta_1)*np.cos(theta_3 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_3*(l_4*m_5 + m_4*r_4)*np.cos(theta_4 - theta_3) - (-l_1**2*(l_4*m_5 + m_4*r_4)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_4 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_4*m_5 + m_4*r_4)*np.cos(theta_4 - theta_2))*(-l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_3 - theta_2))/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2))*(dtheta_5**2*l_4*m_5*r_5*np.sin(theta_5 - theta_4) + dtheta_5*mu_5 - dtheta_4*(mu_4 + mu_5) + dtheta_3*mu_4 + dtheta_3*(-dtheta_3*l_3*l_4*m_5*np.sin(theta_4 - theta_3) - dtheta_3*l_3*m_4*r_4*np.sin(theta_4 - theta_3)) + dtheta_2*(-dtheta_2*l_2*l_4*m_5*np.sin(theta_4 - theta_2) - dtheta_2*l_2*m_4*r_4*np.sin(theta_4 - theta_2)) + dtheta_1*(-dtheta_1*l_1*l_4*m_5*np.sin(theta_4 - theta_1) - dtheta_1*l_1*m_4*r_4*np.sin(theta_4 - theta_1)) - ddx_c*(-l_4*m_5 - m_4*r_4)*np.cos(theta_4) - g*(-l_4*m_5 - m_4*r_4)*np.sin(theta_4) - l_1*(l_4*m_5 + m_4*r_4)*(dtheta_5**2*l_1*m_5*r_5*np.sin(theta_5 - theta_1) + dtheta_4*(dtheta_4*l_1*l_4*m_5*np.sin(theta_4 - theta_1) + dtheta_4*l_1*m_4*r_4*np.sin(theta_4 - theta_1)) + dtheta_3*(dtheta_3*l_1*l_3*m_4*np.sin(theta_3 - theta_1) + dtheta_3*l_1*l_3*m_5*np.sin(theta_3 - theta_1) + dtheta_3*l_1*m_3*r_3*np.sin(theta_3 - theta_1)) + dtheta_2*mu_2 + dtheta_2*(dtheta_2*l_1*l_2*m_3*np.sin(theta_2 - theta_1) + dtheta_2*l_1*l_2*m_4*np.sin(theta_2 - theta_1) + dtheta_2*l_1*l_2*m_5*np.sin(theta_2 - theta_1) + dtheta_2*l_1*m_2*r_2*np.sin(theta_2 - theta_1)) - dtheta_1*(mu_1 + mu_2) - ddx_c*(-l_1*m_2 - l_1*m_3 - l_1*m_4 - l_1*m_5 - m_1*r_1)*np.cos(theta_1) - g*(-l_1*m_2 - l_1*m_3 - l_1*m_4 - l_1*m_5 - m_1*r_1)*np.sin(theta_1))*np.cos(theta_4 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) - (-l_1**2*(l_4*m_5 + m_4*r_4)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_4 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_4*m_5 + m_4*r_4)*np.cos(theta_4 - theta_2))*(dtheta_5**2*l_2*m_5*r_5*np.sin(theta_5 - theta_2) + dtheta_4*(dtheta_4*l_2*l_4*m_5*np.sin(theta_4 - theta_2) + dtheta_4*l_2*m_4*r_4*np.sin(theta_4 - theta_2)) + dtheta_3*mu_3 + dtheta_3*(dtheta_3*l_2*l_3*m_4*np.sin(theta_3 - theta_2) + dtheta_3*l_2*l_3*m_5*np.sin(theta_3 - theta_2) + dtheta_3*l_2*m_3*r_3*np.sin(theta_3 - theta_2)) - dtheta_2*(mu_2 + mu_3) + dtheta_1*mu_2 + dtheta_1*(-dtheta_1*l_1*l_2*m_3*np.sin(theta_2 - theta_1) - dtheta_1*l_1*l_2*m_4*np.sin(theta_2 - theta_1) - dtheta_1*l_1*l_2*m_5*np.sin(theta_2 - theta_1) - dtheta_1*l_1*m_2*r_2*np.sin(theta_2 - theta_1)) - ddx_c*(-l_2*m_3 - l_2*m_4 - l_2*m_5 - m_2*r_2)*np.cos(theta_2) - g*(-l_2*m_3 - l_2*m_4 - l_2*m_5 - m_2*r_2)*np.sin(theta_2) - l_1*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*(dtheta_5**2*l_1*m_5*r_5*np.sin(theta_5 - theta_1) + dtheta_4*(dtheta_4*l_1*l_4*m_5*np.sin(theta_4 - theta_1) + dtheta_4*l_1*m_4*r_4*np.sin(theta_4 - theta_1)) + dtheta_3*(dtheta_3*l_1*l_3*m_4*np.sin(theta_3 - theta_1) + dtheta_3*l_1*l_3*m_5*np.sin(theta_3 - theta_1) + dtheta_3*l_1*m_3*r_3*np.sin(theta_3 - theta_1)) + dtheta_2*mu_2 + dtheta_2*(dtheta_2*l_1*l_2*m_3*np.sin(theta_2 - theta_1) + dtheta_2*l_1*l_2*m_4*np.sin(theta_2 - theta_1) + dtheta_2*l_1*l_2*m_5*np.sin(theta_2 - theta_1) + dtheta_2*l_1*m_2*r_2*np.sin(theta_2 - theta_1)) - dtheta_1*(mu_1 + mu_2) - ddx_c*(-l_1*m_2 - l_1*m_3 - l_1*m_4 - l_1*m_5 - m_1*r_1)*np.cos(theta_1) - g*(-l_1*m_2 - l_1*m_3 - l_1*m_4 - l_1*m_5 - m_1*r_1)*np.sin(theta_1))*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2))/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2) - (-l_1**2*(l_4*m_5 + m_4*r_4)*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_4 - theta_1)*np.cos(theta_3 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_3*(l_4*m_5 + m_4*r_4)*np.cos(theta_4 - theta_3) - (-l_1**2*(l_4*m_5 + m_4*r_4)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_4 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_4*m_5 + m_4*r_4)*np.cos(theta_4 - theta_2))*(-l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_3 - theta_2))/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2))*(dtheta_5**2*l_3*m_5*r_5*np.sin(theta_5 - theta_3) + dtheta_4*mu_4 + dtheta_4*(dtheta_4*l_3*l_4*m_5*np.sin(theta_4 - theta_3) + dtheta_4*l_3*m_4*r_4*np.sin(theta_4 - theta_3)) - dtheta_3*(mu_3 + mu_4) + dtheta_2*mu_3 + dtheta_2*(-dtheta_2*l_2*l_3*m_4*np.sin(theta_3 - theta_2) - dtheta_2*l_2*l_3*m_5*np.sin(theta_3 - theta_2) - dtheta_2*l_2*m_3*r_3*np.sin(theta_3 - theta_2)) + dtheta_1*(-dtheta_1*l_1*l_3*m_4*np.sin(theta_3 - theta_1) - dtheta_1*l_1*l_3*m_5*np.sin(theta_3 - theta_1) - dtheta_1*l_1*m_3*r_3*np.sin(theta_3 - theta_1)) - ddx_c*(-l_3*m_4 - l_3*m_5 - m_3*r_3)*np.cos(theta_3) - g*(-l_3*m_4 - l_3*m_5 - m_3*r_3)*np.sin(theta_3) - l_1*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(dtheta_5**2*l_1*m_5*r_5*np.sin(theta_5 - theta_1) + dtheta_4*(dtheta_4*l_1*l_4*m_5*np.sin(theta_4 - theta_1) + dtheta_4*l_1*m_4*r_4*np.sin(theta_4 - theta_1)) + dtheta_3*(dtheta_3*l_1*l_3*m_4*np.sin(theta_3 - theta_1) + dtheta_3*l_1*l_3*m_5*np.sin(theta_3 - theta_1) + dtheta_3*l_1*m_3*r_3*np.sin(theta_3 - theta_1)) + dtheta_2*mu_2 + dtheta_2*(dtheta_2*l_1*l_2*m_3*np.sin(theta_2 - theta_1) + dtheta_2*l_1*l_2*m_4*np.sin(theta_2 - theta_1) + dtheta_2*l_1*l_2*m_5*np.sin(theta_2 - theta_1) + dtheta_2*l_1*m_2*r_2*np.sin(theta_2 - theta_1)) - dtheta_1*(mu_1 + mu_2) - ddx_c*(-l_1*m_2 - l_1*m_3 - l_1*m_4 - l_1*m_5 - m_1*r_1)*np.cos(theta_1) - g*(-l_1*m_2 - l_1*m_3 - l_1*m_4 - l_1*m_5 - m_1*r_1)*np.sin(theta_1))*np.cos(theta_3 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) - (-l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_3 - theta_2))*(dtheta_5**2*l_2*m_5*r_5*np.sin(theta_5 - theta_2) + dtheta_4*(dtheta_4*l_2*l_4*m_5*np.sin(theta_4 - theta_2) + dtheta_4*l_2*m_4*r_4*np.sin(theta_4 - theta_2)) + dtheta_3*mu_3 + dtheta_3*(dtheta_3*l_2*l_3*m_4*np.sin(theta_3 - theta_2) + dtheta_3*l_2*l_3*m_5*np.sin(theta_3 - theta_2) + dtheta_3*l_2*m_3*r_3*np.sin(theta_3 - theta_2)) - dtheta_2*(mu_2 + mu_3) + dtheta_1*mu_2 + dtheta_1*(-dtheta_1*l_1*l_2*m_3*np.sin(theta_2 - theta_1) - dtheta_1*l_1*l_2*m_4*np.sin(theta_2 - theta_1) - dtheta_1*l_1*l_2*m_5*np.sin(theta_2 - theta_1) - dtheta_1*l_1*m_2*r_2*np.sin(theta_2 - theta_1)) - ddx_c*(-l_2*m_3 - l_2*m_4 - l_2*m_5 - m_2*r_2)*np.cos(theta_2) - g*(-l_2*m_3 - l_2*m_4 - l_2*m_5 - m_2*r_2)*np.sin(theta_2) - l_1*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*(dtheta_5**2*l_1*m_5*r_5*np.sin(theta_5 - theta_1) + dtheta_4*(dtheta_4*l_1*l_4*m_5*np.sin(theta_4 - theta_1) + dtheta_4*l_1*m_4*r_4*np.sin(theta_4 - theta_1)) + dtheta_3*(dtheta_3*l_1*l_3*m_4*np.sin(theta_3 - theta_1) + dtheta_3*l_1*l_3*m_5*np.sin(theta_3 - theta_1) + dtheta_3*l_1*m_3*r_3*np.sin(theta_3 - theta_1)) + dtheta_2*mu_2 + dtheta_2*(dtheta_2*l_1*l_2*m_3*np.sin(theta_2 - theta_1) + dtheta_2*l_1*l_2*m_4*np.sin(theta_2 - theta_1) + dtheta_2*l_1*l_2*m_5*np.sin(theta_2 - theta_1) + dtheta_2*l_1*m_2*r_2*np.sin(theta_2 - theta_1)) - dtheta_1*(mu_1 + mu_2) - ddx_c*(-l_1*m_2 - l_1*m_3 - l_1*m_4 - l_1*m_5 - m_1*r_1)*np.cos(theta_1) - g*(-l_1*m_2 - l_1*m_3 - l_1*m_4 - l_1*m_5 - m_1*r_1)*np.sin(theta_1))*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2))/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2))/(I_3 - l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)**2*np.cos(theta_3 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_3**2*m_4 + l_3**2*m_5 + m_3*r_3**2 - (-l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_3 - theta_2))**2/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2)) - (-l_1**2*m_5*r_5*(l_4*m_5 + m_4*r_4)*np.cos(theta_5 - theta_1)*np.cos(theta_4 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_4*m_5*r_5*np.cos(theta_5 - theta_4) - (-l_1**2*(l_4*m_5 + m_4*r_4)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_4 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_4*m_5 + m_4*r_4)*np.cos(theta_4 - theta_2))*(-l_1**2*m_5*r_5*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_5 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*m_5*r_5*np.cos(theta_5 - theta_2))/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2) - (-l_1**2*(l_4*m_5 + m_4*r_4)*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_4 - theta_1)*np.cos(theta_3 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_3*(l_4*m_5 + m_4*r_4)*np.cos(theta_4 - theta_3) - (-l_1**2*(l_4*m_5 + m_4*r_4)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_4 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_4*m_5 + m_4*r_4)*np.cos(theta_4 - theta_2))*(-l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_3 - theta_2))/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2))*(-l_1**2*m_5*r_5*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_5 - theta_1)*np.cos(theta_3 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_3*m_5*r_5*np.cos(theta_5 - theta_3) - (-l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_3 - theta_2))*(-l_1**2*m_5*r_5*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_5 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*m_5*r_5*np.cos(theta_5 - theta_2))/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2))/(I_3 - l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)**2*np.cos(theta_3 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_3**2*m_4 + l_3**2*m_5 + m_3*r_3**2 - (-l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_3 - theta_2))**2/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2)))*(-dtheta_5*mu_5 - dtheta_4**2*l_4*m_5*r_5*np.sin(theta_5 - theta_4) + dtheta_4*mu_5 - dtheta_3**2*l_3*m_5*r_5*np.sin(theta_5 - theta_3) - dtheta_2**2*l_2*m_5*r_5*np.sin(theta_5 - theta_2) - dtheta_1**2*l_1*m_5*r_5*np.sin(theta_5 - theta_1) + ddx_c*m_5*r_5*np.cos(theta_5) + g*m_5*r_5*np.sin(theta_5) - l_1*m_5*r_5*(dtheta_5**2*l_1*m_5*r_5*np.sin(theta_5 - theta_1) + dtheta_4*(dtheta_4*l_1*l_4*m_5*np.sin(theta_4 - theta_1) + dtheta_4*l_1*m_4*r_4*np.sin(theta_4 - theta_1)) + dtheta_3*(dtheta_3*l_1*l_3*m_4*np.sin(theta_3 - theta_1) + dtheta_3*l_1*l_3*m_5*np.sin(theta_3 - theta_1) + dtheta_3*l_1*m_3*r_3*np.sin(theta_3 - theta_1)) + dtheta_2*mu_2 + dtheta_2*(dtheta_2*l_1*l_2*m_3*np.sin(theta_2 - theta_1) + dtheta_2*l_1*l_2*m_4*np.sin(theta_2 - theta_1) + dtheta_2*l_1*l_2*m_5*np.sin(theta_2 - theta_1) + dtheta_2*l_1*m_2*r_2*np.sin(theta_2 - theta_1)) - dtheta_1*(mu_1 + mu_2) - ddx_c*(-l_1*m_2 - l_1*m_3 - l_1*m_4 - l_1*m_5 - m_1*r_1)*np.cos(theta_1) - g*(-l_1*m_2 - l_1*m_3 - l_1*m_4 - l_1*m_5 - m_1*r_1)*np.sin(theta_1))*np.cos(theta_5 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) - (-l_1**2*m_5*r_5*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_5 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*m_5*r_5*np.cos(theta_5 - theta_2))*(dtheta_5**2*l_2*m_5*r_5*np.sin(theta_5 - theta_2) + dtheta_4*(dtheta_4*l_2*l_4*m_5*np.sin(theta_4 - theta_2) + dtheta_4*l_2*m_4*r_4*np.sin(theta_4 - theta_2)) + dtheta_3*mu_3 + dtheta_3*(dtheta_3*l_2*l_3*m_4*np.sin(theta_3 - theta_2) + dtheta_3*l_2*l_3*m_5*np.sin(theta_3 - theta_2) + dtheta_3*l_2*m_3*r_3*np.sin(theta_3 - theta_2)) - dtheta_2*(mu_2 + mu_3) + dtheta_1*mu_2 + dtheta_1*(-dtheta_1*l_1*l_2*m_3*np.sin(theta_2 - theta_1) - dtheta_1*l_1*l_2*m_4*np.sin(theta_2 - theta_1) - dtheta_1*l_1*l_2*m_5*np.sin(theta_2 - theta_1) - dtheta_1*l_1*m_2*r_2*np.sin(theta_2 - theta_1)) - ddx_c*(-l_2*m_3 - l_2*m_4 - l_2*m_5 - m_2*r_2)*np.cos(theta_2) - g*(-l_2*m_3 - l_2*m_4 - l_2*m_5 - m_2*r_2)*np.sin(theta_2) - l_1*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*(dtheta_5**2*l_1*m_5*r_5*np.sin(theta_5 - theta_1) + dtheta_4*(dtheta_4*l_1*l_4*m_5*np.sin(theta_4 - theta_1) + dtheta_4*l_1*m_4*r_4*np.sin(theta_4 - theta_1)) + dtheta_3*(dtheta_3*l_1*l_3*m_4*np.sin(theta_3 - theta_1) + dtheta_3*l_1*l_3*m_5*np.sin(theta_3 - theta_1) + dtheta_3*l_1*m_3*r_3*np.sin(theta_3 - theta_1)) + dtheta_2*mu_2 + dtheta_2*(dtheta_2*l_1*l_2*m_3*np.sin(theta_2 - theta_1) + dtheta_2*l_1*l_2*m_4*np.sin(theta_2 - theta_1) + dtheta_2*l_1*l_2*m_5*np.sin(theta_2 - theta_1) + dtheta_2*l_1*m_2*r_2*np.sin(theta_2 - theta_1)) - dtheta_1*(mu_1 + mu_2) - ddx_c*(-l_1*m_2 - l_1*m_3 - l_1*m_4 - l_1*m_5 - m_1*r_1)*np.cos(theta_1) - g*(-l_1*m_2 - l_1*m_3 - l_1*m_4 - l_1*m_5 - m_1*r_1)*np.sin(theta_1))*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2))/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2) - (-l_1**2*m_5*r_5*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_5 - theta_1)*np.cos(theta_3 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_3*m_5*r_5*np.cos(theta_5 - theta_3) - (-l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_3 - theta_2))*(-l_1**2*m_5*r_5*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_5 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*m_5*r_5*np.cos(theta_5 - theta_2))/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2))*(dtheta_5**2*l_3*m_5*r_5*np.sin(theta_5 - theta_3) + dtheta_4*mu_4 + dtheta_4*(dtheta_4*l_3*l_4*m_5*np.sin(theta_4 - theta_3) + dtheta_4*l_3*m_4*r_4*np.sin(theta_4 - theta_3)) - dtheta_3*(mu_3 + mu_4) + dtheta_2*mu_3 + dtheta_2*(-dtheta_2*l_2*l_3*m_4*np.sin(theta_3 - theta_2) - dtheta_2*l_2*l_3*m_5*np.sin(theta_3 - theta_2) - dtheta_2*l_2*m_3*r_3*np.sin(theta_3 - theta_2)) + dtheta_1*(-dtheta_1*l_1*l_3*m_4*np.sin(theta_3 - theta_1) - dtheta_1*l_1*l_3*m_5*np.sin(theta_3 - theta_1) - dtheta_1*l_1*m_3*r_3*np.sin(theta_3 - theta_1)) - ddx_c*(-l_3*m_4 - l_3*m_5 - m_3*r_3)*np.cos(theta_3) - g*(-l_3*m_4 - l_3*m_5 - m_3*r_3)*np.sin(theta_3) - l_1*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(dtheta_5**2*l_1*m_5*r_5*np.sin(theta_5 - theta_1) + dtheta_4*(dtheta_4*l_1*l_4*m_5*np.sin(theta_4 - theta_1) + dtheta_4*l_1*m_4*r_4*np.sin(theta_4 - theta_1)) + dtheta_3*(dtheta_3*l_1*l_3*m_4*np.sin(theta_3 - theta_1) + dtheta_3*l_1*l_3*m_5*np.sin(theta_3 - theta_1) + dtheta_3*l_1*m_3*r_3*np.sin(theta_3 - theta_1)) + dtheta_2*mu_2 + dtheta_2*(dtheta_2*l_1*l_2*m_3*np.sin(theta_2 - theta_1) + dtheta_2*l_1*l_2*m_4*np.sin(theta_2 - theta_1) + dtheta_2*l_1*l_2*m_5*np.sin(theta_2 - theta_1) + dtheta_2*l_1*m_2*r_2*np.sin(theta_2 - theta_1)) - dtheta_1*(mu_1 + mu_2) - ddx_c*(-l_1*m_2 - l_1*m_3 - l_1*m_4 - l_1*m_5 - m_1*r_1)*np.cos(theta_1) - g*(-l_1*m_2 - l_1*m_3 - l_1*m_4 - l_1*m_5 - m_1*r_1)*np.sin(theta_1))*np.cos(theta_3 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) - (-l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_3 - theta_2))*(dtheta_5**2*l_2*m_5*r_5*np.sin(theta_5 - theta_2) + dtheta_4*(dtheta_4*l_2*l_4*m_5*np.sin(theta_4 - theta_2) + dtheta_4*l_2*m_4*r_4*np.sin(theta_4 - theta_2)) + dtheta_3*mu_3 + dtheta_3*(dtheta_3*l_2*l_3*m_4*np.sin(theta_3 - theta_2) + dtheta_3*l_2*l_3*m_5*np.sin(theta_3 - theta_2) + dtheta_3*l_2*m_3*r_3*np.sin(theta_3 - theta_2)) - dtheta_2*(mu_2 + mu_3) + dtheta_1*mu_2 + dtheta_1*(-dtheta_1*l_1*l_2*m_3*np.sin(theta_2 - theta_1) - dtheta_1*l_1*l_2*m_4*np.sin(theta_2 - theta_1) - dtheta_1*l_1*l_2*m_5*np.sin(theta_2 - theta_1) - dtheta_1*l_1*m_2*r_2*np.sin(theta_2 - theta_1)) - ddx_c*(-l_2*m_3 - l_2*m_4 - l_2*m_5 - m_2*r_2)*np.cos(theta_2) - g*(-l_2*m_3 - l_2*m_4 - l_2*m_5 - m_2*r_2)*np.sin(theta_2) - l_1*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*(dtheta_5**2*l_1*m_5*r_5*np.sin(theta_5 - theta_1) + dtheta_4*(dtheta_4*l_1*l_4*m_5*np.sin(theta_4 - theta_1) + dtheta_4*l_1*m_4*r_4*np.sin(theta_4 - theta_1)) + dtheta_3*(dtheta_3*l_1*l_3*m_4*np.sin(theta_3 - theta_1) + dtheta_3*l_1*l_3*m_5*np.sin(theta_3 - theta_1) + dtheta_3*l_1*m_3*r_3*np.sin(theta_3 - theta_1)) + dtheta_2*mu_2 + dtheta_2*(dtheta_2*l_1*l_2*m_3*np.sin(theta_2 - theta_1) + dtheta_2*l_1*l_2*m_4*np.sin(theta_2 - theta_1) + dtheta_2*l_1*l_2*m_5*np.sin(theta_2 - theta_1) + dtheta_2*l_1*m_2*r_2*np.sin(theta_2 - theta_1)) - dtheta_1*(mu_1 + mu_2) - ddx_c*(-l_1*m_2 - l_1*m_3 - l_1*m_4 - l_1*m_5 - m_1*r_1)*np.cos(theta_1) - g*(-l_1*m_2 - l_1*m_3 - l_1*m_4 - l_1*m_5 - m_1*r_1)*np.sin(theta_1))*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2))/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2))/(I_3 - l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)**2*np.cos(theta_3 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_3**2*m_4 + l_3**2*m_5 + m_3*r_3**2 - (-l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_3 - theta_2))**2/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2)) - (-l_1**2*m_5*r_5*(l_4*m_5 + m_4*r_4)*np.cos(theta_5 - theta_1)*np.cos(theta_4 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_4*m_5*r_5*np.cos(theta_5 - theta_4) - (-l_1**2*(l_4*m_5 + m_4*r_4)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_4 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_4*m_5 + m_4*r_4)*np.cos(theta_4 - theta_2))*(-l_1**2*m_5*r_5*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_5 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*m_5*r_5*np.cos(theta_5 - theta_2))/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2) - (-l_1**2*(l_4*m_5 + m_4*r_4)*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_4 - theta_1)*np.cos(theta_3 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_3*(l_4*m_5 + m_4*r_4)*np.cos(theta_4 - theta_3) - (-l_1**2*(l_4*m_5 + m_4*r_4)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_4 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_4*m_5 + m_4*r_4)*np.cos(theta_4 - theta_2))*(-l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_3 - theta_2))/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2))*(-l_1**2*m_5*r_5*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_5 - theta_1)*np.cos(theta_3 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_3*m_5*r_5*np.cos(theta_5 - theta_3) - (-l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_3 - theta_2))*(-l_1**2*m_5*r_5*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_5 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*m_5*r_5*np.cos(theta_5 - theta_2))/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2))/(I_3 - l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)**2*np.cos(theta_3 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_3**2*m_4 + l_3**2*m_5 + m_3*r_3**2 - (-l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_3 - theta_2))**2/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2)))*(dtheta_5**2*l_4*m_5*r_5*np.sin(theta_5 - theta_4) + dtheta_5*mu_5 - dtheta_4*(mu_4 + mu_5) + dtheta_3*mu_4 + dtheta_3*(-dtheta_3*l_3*l_4*m_5*np.sin(theta_4 - theta_3) - dtheta_3*l_3*m_4*r_4*np.sin(theta_4 - theta_3)) + dtheta_2*(-dtheta_2*l_2*l_4*m_5*np.sin(theta_4 - theta_2) - dtheta_2*l_2*m_4*r_4*np.sin(theta_4 - theta_2)) + dtheta_1*(-dtheta_1*l_1*l_4*m_5*np.sin(theta_4 - theta_1) - dtheta_1*l_1*m_4*r_4*np.sin(theta_4 - theta_1)) - ddx_c*(-l_4*m_5 - m_4*r_4)*np.cos(theta_4) - g*(-l_4*m_5 - m_4*r_4)*np.sin(theta_4) - l_1*(l_4*m_5 + m_4*r_4)*(dtheta_5**2*l_1*m_5*r_5*np.sin(theta_5 - theta_1) + dtheta_4*(dtheta_4*l_1*l_4*m_5*np.sin(theta_4 - theta_1) + dtheta_4*l_1*m_4*r_4*np.sin(theta_4 - theta_1)) + dtheta_3*(dtheta_3*l_1*l_3*m_4*np.sin(theta_3 - theta_1) + dtheta_3*l_1*l_3*m_5*np.sin(theta_3 - theta_1) + dtheta_3*l_1*m_3*r_3*np.sin(theta_3 - theta_1)) + dtheta_2*mu_2 + dtheta_2*(dtheta_2*l_1*l_2*m_3*np.sin(theta_2 - theta_1) + dtheta_2*l_1*l_2*m_4*np.sin(theta_2 - theta_1) + dtheta_2*l_1*l_2*m_5*np.sin(theta_2 - theta_1) + dtheta_2*l_1*m_2*r_2*np.sin(theta_2 - theta_1)) - dtheta_1*(mu_1 + mu_2) - ddx_c*(-l_1*m_2 - l_1*m_3 - l_1*m_4 - l_1*m_5 - m_1*r_1)*np.cos(theta_1) - g*(-l_1*m_2 - l_1*m_3 - l_1*m_4 - l_1*m_5 - m_1*r_1)*np.sin(theta_1))*np.cos(theta_4 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) - (-l_1**2*(l_4*m_5 + m_4*r_4)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_4 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_4*m_5 + m_4*r_4)*np.cos(theta_4 - theta_2))*(dtheta_5**2*l_2*m_5*r_5*np.sin(theta_5 - theta_2) + dtheta_4*(dtheta_4*l_2*l_4*m_5*np.sin(theta_4 - theta_2) + dtheta_4*l_2*m_4*r_4*np.sin(theta_4 - theta_2)) + dtheta_3*mu_3 + dtheta_3*(dtheta_3*l_2*l_3*m_4*np.sin(theta_3 - theta_2) + dtheta_3*l_2*l_3*m_5*np.sin(theta_3 - theta_2) + dtheta_3*l_2*m_3*r_3*np.sin(theta_3 - theta_2)) - dtheta_2*(mu_2 + mu_3) + dtheta_1*mu_2 + dtheta_1*(-dtheta_1*l_1*l_2*m_3*np.sin(theta_2 - theta_1) - dtheta_1*l_1*l_2*m_4*np.sin(theta_2 - theta_1) - dtheta_1*l_1*l_2*m_5*np.sin(theta_2 - theta_1) - dtheta_1*l_1*m_2*r_2*np.sin(theta_2 - theta_1)) - ddx_c*(-l_2*m_3 - l_2*m_4 - l_2*m_5 - m_2*r_2)*np.cos(theta_2) - g*(-l_2*m_3 - l_2*m_4 - l_2*m_5 - m_2*r_2)*np.sin(theta_2) - l_1*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*(dtheta_5**2*l_1*m_5*r_5*np.sin(theta_5 - theta_1) + dtheta_4*(dtheta_4*l_1*l_4*m_5*np.sin(theta_4 - theta_1) + dtheta_4*l_1*m_4*r_4*np.sin(theta_4 - theta_1)) + dtheta_3*(dtheta_3*l_1*l_3*m_4*np.sin(theta_3 - theta_1) + dtheta_3*l_1*l_3*m_5*np.sin(theta_3 - theta_1) + dtheta_3*l_1*m_3*r_3*np.sin(theta_3 - theta_1)) + dtheta_2*mu_2 + dtheta_2*(dtheta_2*l_1*l_2*m_3*np.sin(theta_2 - theta_1) + dtheta_2*l_1*l_2*m_4*np.sin(theta_2 - theta_1) + dtheta_2*l_1*l_2*m_5*np.sin(theta_2 - theta_1) + dtheta_2*l_1*m_2*r_2*np.sin(theta_2 - theta_1)) - dtheta_1*(mu_1 + mu_2) - ddx_c*(-l_1*m_2 - l_1*m_3 - l_1*m_4 - l_1*m_5 - m_1*r_1)*np.cos(theta_1) - g*(-l_1*m_2 - l_1*m_3 - l_1*m_4 - l_1*m_5 - m_1*r_1)*np.sin(theta_1))*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2))/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2) - (-l_1**2*(l_4*m_5 + m_4*r_4)*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_4 - theta_1)*np.cos(theta_3 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_3*(l_4*m_5 + m_4*r_4)*np.cos(theta_4 - theta_3) - (-l_1**2*(l_4*m_5 + m_4*r_4)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_4 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_4*m_5 + m_4*r_4)*np.cos(theta_4 - theta_2))*(-l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_3 - theta_2))/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2))*(dtheta_5**2*l_3*m_5*r_5*np.sin(theta_5 - theta_3) + dtheta_4*mu_4 + dtheta_4*(dtheta_4*l_3*l_4*m_5*np.sin(theta_4 - theta_3) + dtheta_4*l_3*m_4*r_4*np.sin(theta_4 - theta_3)) - dtheta_3*(mu_3 + mu_4) + dtheta_2*mu_3 + dtheta_2*(-dtheta_2*l_2*l_3*m_4*np.sin(theta_3 - theta_2) - dtheta_2*l_2*l_3*m_5*np.sin(theta_3 - theta_2) - dtheta_2*l_2*m_3*r_3*np.sin(theta_3 - theta_2)) + dtheta_1*(-dtheta_1*l_1*l_3*m_4*np.sin(theta_3 - theta_1) - dtheta_1*l_1*l_3*m_5*np.sin(theta_3 - theta_1) - dtheta_1*l_1*m_3*r_3*np.sin(theta_3 - theta_1)) - ddx_c*(-l_3*m_4 - l_3*m_5 - m_3*r_3)*np.cos(theta_3) - g*(-l_3*m_4 - l_3*m_5 - m_3*r_3)*np.sin(theta_3) - l_1*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(dtheta_5**2*l_1*m_5*r_5*np.sin(theta_5 - theta_1) + dtheta_4*(dtheta_4*l_1*l_4*m_5*np.sin(theta_4 - theta_1) + dtheta_4*l_1*m_4*r_4*np.sin(theta_4 - theta_1)) + dtheta_3*(dtheta_3*l_1*l_3*m_4*np.sin(theta_3 - theta_1) + dtheta_3*l_1*l_3*m_5*np.sin(theta_3 - theta_1) + dtheta_3*l_1*m_3*r_3*np.sin(theta_3 - theta_1)) + dtheta_2*mu_2 + dtheta_2*(dtheta_2*l_1*l_2*m_3*np.sin(theta_2 - theta_1) + dtheta_2*l_1*l_2*m_4*np.sin(theta_2 - theta_1) + dtheta_2*l_1*l_2*m_5*np.sin(theta_2 - theta_1) + dtheta_2*l_1*m_2*r_2*np.sin(theta_2 - theta_1)) - dtheta_1*(mu_1 + mu_2) - ddx_c*(-l_1*m_2 - l_1*m_3 - l_1*m_4 - l_1*m_5 - m_1*r_1)*np.cos(theta_1) - g*(-l_1*m_2 - l_1*m_3 - l_1*m_4 - l_1*m_5 - m_1*r_1)*np.sin(theta_1))*np.cos(theta_3 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) - (-l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_3 - theta_2))*(dtheta_5**2*l_2*m_5*r_5*np.sin(theta_5 - theta_2) + dtheta_4*(dtheta_4*l_2*l_4*m_5*np.sin(theta_4 - theta_2) + dtheta_4*l_2*m_4*r_4*np.sin(theta_4 - theta_2)) + dtheta_3*mu_3 + dtheta_3*(dtheta_3*l_2*l_3*m_4*np.sin(theta_3 - theta_2) + dtheta_3*l_2*l_3*m_5*np.sin(theta_3 - theta_2) + dtheta_3*l_2*m_3*r_3*np.sin(theta_3 - theta_2)) - dtheta_2*(mu_2 + mu_3) + dtheta_1*mu_2 + dtheta_1*(-dtheta_1*l_1*l_2*m_3*np.sin(theta_2 - theta_1) - dtheta_1*l_1*l_2*m_4*np.sin(theta_2 - theta_1) - dtheta_1*l_1*l_2*m_5*np.sin(theta_2 - theta_1) - dtheta_1*l_1*m_2*r_2*np.sin(theta_2 - theta_1)) - ddx_c*(-l_2*m_3 - l_2*m_4 - l_2*m_5 - m_2*r_2)*np.cos(theta_2) - g*(-l_2*m_3 - l_2*m_4 - l_2*m_5 - m_2*r_2)*np.sin(theta_2) - l_1*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*(dtheta_5**2*l_1*m_5*r_5*np.sin(theta_5 - theta_1) + dtheta_4*(dtheta_4*l_1*l_4*m_5*np.sin(theta_4 - theta_1) + dtheta_4*l_1*m_4*r_4*np.sin(theta_4 - theta_1)) + dtheta_3*(dtheta_3*l_1*l_3*m_4*np.sin(theta_3 - theta_1) + dtheta_3*l_1*l_3*m_5*np.sin(theta_3 - theta_1) + dtheta_3*l_1*m_3*r_3*np.sin(theta_3 - theta_1)) + dtheta_2*mu_2 + dtheta_2*(dtheta_2*l_1*l_2*m_3*np.sin(theta_2 - theta_1) + dtheta_2*l_1*l_2*m_4*np.sin(theta_2 - theta_1) + dtheta_2*l_1*l_2*m_5*np.sin(theta_2 - theta_1) + dtheta_2*l_1*m_2*r_2*np.sin(theta_2 - theta_1)) - dtheta_1*(mu_1 + mu_2) - ddx_c*(-l_1*m_2 - l_1*m_3 - l_1*m_4 - l_1*m_5 - m_1*r_1)*np.cos(theta_1) - g*(-l_1*m_2 - l_1*m_3 - l_1*m_4 - l_1*m_5 - m_1*r_1)*np.sin(theta_1))*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2))/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2))/(I_3 - l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)**2*np.cos(theta_3 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_3**2*m_4 + l_3**2*m_5 + m_3*r_3**2 - (-l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_3 - theta_2))**2/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2)))/(I_4 - l_1**2*(l_4*m_5 + m_4*r_4)**2*np.cos(theta_4 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_4**2*m_5 + m_4*r_4**2 - (-l_1**2*(l_4*m_5 + m_4*r_4)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_4 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_4*m_5 + m_4*r_4)*np.cos(theta_4 - theta_2))**2/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2) - (-l_1**2*(l_4*m_5 + m_4*r_4)*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_4 - theta_1)*np.cos(theta_3 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_3*(l_4*m_5 + m_4*r_4)*np.cos(theta_4 - theta_3) - (-l_1**2*(l_4*m_5 + m_4*r_4)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_4 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_4*m_5 + m_4*r_4)*np.cos(theta_4 - theta_2))*(-l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_3 - theta_2))/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2))**2/(I_3 - l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)**2*np.cos(theta_3 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_3**2*m_4 + l_3**2*m_5 + m_3*r_3**2 - (-l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_3 - theta_2))**2/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2))))/(I_5 - l_1**2*m_5**2*r_5**2*np.cos(theta_5 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + m_5*r_5**2 - (-l_1**2*m_5*r_5*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_5 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*m_5*r_5*np.cos(theta_5 - theta_2))**2/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2) - (-l_1**2*m_5*r_5*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_5 - theta_1)*np.cos(theta_3 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_3*m_5*r_5*np.cos(theta_5 - theta_3) - (-l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_3 - theta_2))*(-l_1**2*m_5*r_5*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_5 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*m_5*r_5*np.cos(theta_5 - theta_2))/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2))**2/(I_3 - l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)**2*np.cos(theta_3 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_3**2*m_4 + l_3**2*m_5 + m_3*r_3**2 - (-l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_3 - theta_2))**2/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2)) - (-l_1**2*m_5*r_5*(l_4*m_5 + m_4*r_4)*np.cos(theta_5 - theta_1)*np.cos(theta_4 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_4*m_5*r_5*np.cos(theta_5 - theta_4) - (-l_1**2*(l_4*m_5 + m_4*r_4)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_4 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_4*m_5 + m_4*r_4)*np.cos(theta_4 - theta_2))*(-l_1**2*m_5*r_5*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_5 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*m_5*r_5*np.cos(theta_5 - theta_2))/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2) - (-l_1**2*(l_4*m_5 + m_4*r_4)*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_4 - theta_1)*np.cos(theta_3 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_3*(l_4*m_5 + m_4*r_4)*np.cos(theta_4 - theta_3) - (-l_1**2*(l_4*m_5 + m_4*r_4)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_4 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_4*m_5 + m_4*r_4)*np.cos(theta_4 - theta_2))*(-l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_3 - theta_2))/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2))*(-l_1**2*m_5*r_5*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_5 - theta_1)*np.cos(theta_3 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_3*m_5*r_5*np.cos(theta_5 - theta_3) - (-l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_3 - theta_2))*(-l_1**2*m_5*r_5*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_5 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*m_5*r_5*np.cos(theta_5 - theta_2))/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2))/(I_3 - l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)**2*np.cos(theta_3 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_3**2*m_4 + l_3**2*m_5 + m_3*r_3**2 - (-l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_3 - theta_2))**2/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2)))**2/(I_4 - l_1**2*(l_4*m_5 + m_4*r_4)**2*np.cos(theta_4 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_4**2*m_5 + m_4*r_4**2 - (-l_1**2*(l_4*m_5 + m_4*r_4)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_4 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_4*m_5 + m_4*r_4)*np.cos(theta_4 - theta_2))**2/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2) - (-l_1**2*(l_4*m_5 + m_4*r_4)*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_4 - theta_1)*np.cos(theta_3 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_3*(l_4*m_5 + m_4*r_4)*np.cos(theta_4 - theta_3) - (-l_1**2*(l_4*m_5 + m_4*r_4)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_4 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_4*m_5 + m_4*r_4)*np.cos(theta_4 - theta_2))*(-l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_3 - theta_2))/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2))**2/(I_3 - l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)**2*np.cos(theta_3 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_3**2*m_4 + l_3**2*m_5 + m_3*r_3**2 - (-l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_3 - theta_2))**2/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2)))))/(I_4 - l_1**2*(l_4*m_5 + m_4*r_4)**2*np.cos(theta_4 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_4**2*m_5 + m_4*r_4**2 - (-l_1**2*(l_4*m_5 + m_4*r_4)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_4 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_4*m_5 + m_4*r_4)*np.cos(theta_4 - theta_2))**2/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2) - (-l_1**2*(l_4*m_5 + m_4*r_4)*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_4 - theta_1)*np.cos(theta_3 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_3*(l_4*m_5 + m_4*r_4)*np.cos(theta_4 - theta_3) - (-l_1**2*(l_4*m_5 + m_4*r_4)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_4 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_4*m_5 + m_4*r_4)*np.cos(theta_4 - theta_2))*(-l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_3 - theta_2))/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2))**2/(I_3 - l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)**2*np.cos(theta_3 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_3**2*m_4 + l_3**2*m_5 + m_3*r_3**2 - (-l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_3 - theta_2))**2/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2))) - (-l_1**2*m_5*r_5*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_5 - theta_1)*np.cos(theta_3 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_3*m_5*r_5*np.cos(theta_5 - theta_3) - (-l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_3 - theta_2))*(-l_1**2*m_5*r_5*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_5 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*m_5*r_5*np.cos(theta_5 - theta_2))/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2))*(-dtheta_5*mu_5 - dtheta_4**2*l_4*m_5*r_5*np.sin(theta_5 - theta_4) + dtheta_4*mu_5 - dtheta_3**2*l_3*m_5*r_5*np.sin(theta_5 - theta_3) - dtheta_2**2*l_2*m_5*r_5*np.sin(theta_5 - theta_2) - dtheta_1**2*l_1*m_5*r_5*np.sin(theta_5 - theta_1) + ddx_c*m_5*r_5*np.cos(theta_5) + g*m_5*r_5*np.sin(theta_5) - l_1*m_5*r_5*(dtheta_5**2*l_1*m_5*r_5*np.sin(theta_5 - theta_1) + dtheta_4*(dtheta_4*l_1*l_4*m_5*np.sin(theta_4 - theta_1) + dtheta_4*l_1*m_4*r_4*np.sin(theta_4 - theta_1)) + dtheta_3*(dtheta_3*l_1*l_3*m_4*np.sin(theta_3 - theta_1) + dtheta_3*l_1*l_3*m_5*np.sin(theta_3 - theta_1) + dtheta_3*l_1*m_3*r_3*np.sin(theta_3 - theta_1)) + dtheta_2*mu_2 + dtheta_2*(dtheta_2*l_1*l_2*m_3*np.sin(theta_2 - theta_1) + dtheta_2*l_1*l_2*m_4*np.sin(theta_2 - theta_1) + dtheta_2*l_1*l_2*m_5*np.sin(theta_2 - theta_1) + dtheta_2*l_1*m_2*r_2*np.sin(theta_2 - theta_1)) - dtheta_1*(mu_1 + mu_2) - ddx_c*(-l_1*m_2 - l_1*m_3 - l_1*m_4 - l_1*m_5 - m_1*r_1)*np.cos(theta_1) - g*(-l_1*m_2 - l_1*m_3 - l_1*m_4 - l_1*m_5 - m_1*r_1)*np.sin(theta_1))*np.cos(theta_5 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) - (-l_1**2*m_5*r_5*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_5 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*m_5*r_5*np.cos(theta_5 - theta_2))*(dtheta_5**2*l_2*m_5*r_5*np.sin(theta_5 - theta_2) + dtheta_4*(dtheta_4*l_2*l_4*m_5*np.sin(theta_4 - theta_2) + dtheta_4*l_2*m_4*r_4*np.sin(theta_4 - theta_2)) + dtheta_3*mu_3 + dtheta_3*(dtheta_3*l_2*l_3*m_4*np.sin(theta_3 - theta_2) + dtheta_3*l_2*l_3*m_5*np.sin(theta_3 - theta_2) + dtheta_3*l_2*m_3*r_3*np.sin(theta_3 - theta_2)) - dtheta_2*(mu_2 + mu_3) + dtheta_1*mu_2 + dtheta_1*(-dtheta_1*l_1*l_2*m_3*np.sin(theta_2 - theta_1) - dtheta_1*l_1*l_2*m_4*np.sin(theta_2 - theta_1) - dtheta_1*l_1*l_2*m_5*np.sin(theta_2 - theta_1) - dtheta_1*l_1*m_2*r_2*np.sin(theta_2 - theta_1)) - ddx_c*(-l_2*m_3 - l_2*m_4 - l_2*m_5 - m_2*r_2)*np.cos(theta_2) - g*(-l_2*m_3 - l_2*m_4 - l_2*m_5 - m_2*r_2)*np.sin(theta_2) - l_1*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*(dtheta_5**2*l_1*m_5*r_5*np.sin(theta_5 - theta_1) + dtheta_4*(dtheta_4*l_1*l_4*m_5*np.sin(theta_4 - theta_1) + dtheta_4*l_1*m_4*r_4*np.sin(theta_4 - theta_1)) + dtheta_3*(dtheta_3*l_1*l_3*m_4*np.sin(theta_3 - theta_1) + dtheta_3*l_1*l_3*m_5*np.sin(theta_3 - theta_1) + dtheta_3*l_1*m_3*r_3*np.sin(theta_3 - theta_1)) + dtheta_2*mu_2 + dtheta_2*(dtheta_2*l_1*l_2*m_3*np.sin(theta_2 - theta_1) + dtheta_2*l_1*l_2*m_4*np.sin(theta_2 - theta_1) + dtheta_2*l_1*l_2*m_5*np.sin(theta_2 - theta_1) + dtheta_2*l_1*m_2*r_2*np.sin(theta_2 - theta_1)) - dtheta_1*(mu_1 + mu_2) - ddx_c*(-l_1*m_2 - l_1*m_3 - l_1*m_4 - l_1*m_5 - m_1*r_1)*np.cos(theta_1) - g*(-l_1*m_2 - l_1*m_3 - l_1*m_4 - l_1*m_5 - m_1*r_1)*np.sin(theta_1))*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2))/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2) - (-l_1**2*m_5*r_5*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_5 - theta_1)*np.cos(theta_3 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_3*m_5*r_5*np.cos(theta_5 - theta_3) - (-l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_3 - theta_2))*(-l_1**2*m_5*r_5*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_5 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*m_5*r_5*np.cos(theta_5 - theta_2))/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2))*(dtheta_5**2*l_3*m_5*r_5*np.sin(theta_5 - theta_3) + dtheta_4*mu_4 + dtheta_4*(dtheta_4*l_3*l_4*m_5*np.sin(theta_4 - theta_3) + dtheta_4*l_3*m_4*r_4*np.sin(theta_4 - theta_3)) - dtheta_3*(mu_3 + mu_4) + dtheta_2*mu_3 + dtheta_2*(-dtheta_2*l_2*l_3*m_4*np.sin(theta_3 - theta_2) - dtheta_2*l_2*l_3*m_5*np.sin(theta_3 - theta_2) - dtheta_2*l_2*m_3*r_3*np.sin(theta_3 - theta_2)) + dtheta_1*(-dtheta_1*l_1*l_3*m_4*np.sin(theta_3 - theta_1) - dtheta_1*l_1*l_3*m_5*np.sin(theta_3 - theta_1) - dtheta_1*l_1*m_3*r_3*np.sin(theta_3 - theta_1)) - ddx_c*(-l_3*m_4 - l_3*m_5 - m_3*r_3)*np.cos(theta_3) - g*(-l_3*m_4 - l_3*m_5 - m_3*r_3)*np.sin(theta_3) - l_1*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(dtheta_5**2*l_1*m_5*r_5*np.sin(theta_5 - theta_1) + dtheta_4*(dtheta_4*l_1*l_4*m_5*np.sin(theta_4 - theta_1) + dtheta_4*l_1*m_4*r_4*np.sin(theta_4 - theta_1)) + dtheta_3*(dtheta_3*l_1*l_3*m_4*np.sin(theta_3 - theta_1) + dtheta_3*l_1*l_3*m_5*np.sin(theta_3 - theta_1) + dtheta_3*l_1*m_3*r_3*np.sin(theta_3 - theta_1)) + dtheta_2*mu_2 + dtheta_2*(dtheta_2*l_1*l_2*m_3*np.sin(theta_2 - theta_1) + dtheta_2*l_1*l_2*m_4*np.sin(theta_2 - theta_1) + dtheta_2*l_1*l_2*m_5*np.sin(theta_2 - theta_1) + dtheta_2*l_1*m_2*r_2*np.sin(theta_2 - theta_1)) - dtheta_1*(mu_1 + mu_2) - ddx_c*(-l_1*m_2 - l_1*m_3 - l_1*m_4 - l_1*m_5 - m_1*r_1)*np.cos(theta_1) - g*(-l_1*m_2 - l_1*m_3 - l_1*m_4 - l_1*m_5 - m_1*r_1)*np.sin(theta_1))*np.cos(theta_3 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) - (-l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_3 - theta_2))*(dtheta_5**2*l_2*m_5*r_5*np.sin(theta_5 - theta_2) + dtheta_4*(dtheta_4*l_2*l_4*m_5*np.sin(theta_4 - theta_2) + dtheta_4*l_2*m_4*r_4*np.sin(theta_4 - theta_2)) + dtheta_3*mu_3 + dtheta_3*(dtheta_3*l_2*l_3*m_4*np.sin(theta_3 - theta_2) + dtheta_3*l_2*l_3*m_5*np.sin(theta_3 - theta_2) + dtheta_3*l_2*m_3*r_3*np.sin(theta_3 - theta_2)) - dtheta_2*(mu_2 + mu_3) + dtheta_1*mu_2 + dtheta_1*(-dtheta_1*l_1*l_2*m_3*np.sin(theta_2 - theta_1) - dtheta_1*l_1*l_2*m_4*np.sin(theta_2 - theta_1) - dtheta_1*l_1*l_2*m_5*np.sin(theta_2 - theta_1) - dtheta_1*l_1*m_2*r_2*np.sin(theta_2 - theta_1)) - ddx_c*(-l_2*m_3 - l_2*m_4 - l_2*m_5 - m_2*r_2)*np.cos(theta_2) - g*(-l_2*m_3 - l_2*m_4 - l_2*m_5 - m_2*r_2)*np.sin(theta_2) - l_1*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*(dtheta_5**2*l_1*m_5*r_5*np.sin(theta_5 - theta_1) + dtheta_4*(dtheta_4*l_1*l_4*m_5*np.sin(theta_4 - theta_1) + dtheta_4*l_1*m_4*r_4*np.sin(theta_4 - theta_1)) + dtheta_3*(dtheta_3*l_1*l_3*m_4*np.sin(theta_3 - theta_1) + dtheta_3*l_1*l_3*m_5*np.sin(theta_3 - theta_1) + dtheta_3*l_1*m_3*r_3*np.sin(theta_3 - theta_1)) + dtheta_2*mu_2 + dtheta_2*(dtheta_2*l_1*l_2*m_3*np.sin(theta_2 - theta_1) + dtheta_2*l_1*l_2*m_4*np.sin(theta_2 - theta_1) + dtheta_2*l_1*l_2*m_5*np.sin(theta_2 - theta_1) + dtheta_2*l_1*m_2*r_2*np.sin(theta_2 - theta_1)) - dtheta_1*(mu_1 + mu_2) - ddx_c*(-l_1*m_2 - l_1*m_3 - l_1*m_4 - l_1*m_5 - m_1*r_1)*np.cos(theta_1) - g*(-l_1*m_2 - l_1*m_3 - l_1*m_4 - l_1*m_5 - m_1*r_1)*np.sin(theta_1))*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2))/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2))/(I_3 - l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)**2*np.cos(theta_3 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_3**2*m_4 + l_3**2*m_5 + m_3*r_3**2 - (-l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_3 - theta_2))**2/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2)) - (-l_1**2*m_5*r_5*(l_4*m_5 + m_4*r_4)*np.cos(theta_5 - theta_1)*np.cos(theta_4 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_4*m_5*r_5*np.cos(theta_5 - theta_4) - (-l_1**2*(l_4*m_5 + m_4*r_4)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_4 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_4*m_5 + m_4*r_4)*np.cos(theta_4 - theta_2))*(-l_1**2*m_5*r_5*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_5 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*m_5*r_5*np.cos(theta_5 - theta_2))/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2) - (-l_1**2*(l_4*m_5 + m_4*r_4)*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_4 - theta_1)*np.cos(theta_3 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_3*(l_4*m_5 + m_4*r_4)*np.cos(theta_4 - theta_3) - (-l_1**2*(l_4*m_5 + m_4*r_4)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_4 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_4*m_5 + m_4*r_4)*np.cos(theta_4 - theta_2))*(-l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_3 - theta_2))/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2))*(-l_1**2*m_5*r_5*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_5 - theta_1)*np.cos(theta_3 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_3*m_5*r_5*np.cos(theta_5 - theta_3) - (-l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_3 - theta_2))*(-l_1**2*m_5*r_5*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_5 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*m_5*r_5*np.cos(theta_5 - theta_2))/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2))/(I_3 - l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)**2*np.cos(theta_3 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_3**2*m_4 + l_3**2*m_5 + m_3*r_3**2 - (-l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_3 - theta_2))**2/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2)))*(dtheta_5**2*l_4*m_5*r_5*np.sin(theta_5 - theta_4) + dtheta_5*mu_5 - dtheta_4*(mu_4 + mu_5) + dtheta_3*mu_4 + dtheta_3*(-dtheta_3*l_3*l_4*m_5*np.sin(theta_4 - theta_3) - dtheta_3*l_3*m_4*r_4*np.sin(theta_4 - theta_3)) + dtheta_2*(-dtheta_2*l_2*l_4*m_5*np.sin(theta_4 - theta_2) - dtheta_2*l_2*m_4*r_4*np.sin(theta_4 - theta_2)) + dtheta_1*(-dtheta_1*l_1*l_4*m_5*np.sin(theta_4 - theta_1) - dtheta_1*l_1*m_4*r_4*np.sin(theta_4 - theta_1)) - ddx_c*(-l_4*m_5 - m_4*r_4)*np.cos(theta_4) - g*(-l_4*m_5 - m_4*r_4)*np.sin(theta_4) - l_1*(l_4*m_5 + m_4*r_4)*(dtheta_5**2*l_1*m_5*r_5*np.sin(theta_5 - theta_1) + dtheta_4*(dtheta_4*l_1*l_4*m_5*np.sin(theta_4 - theta_1) + dtheta_4*l_1*m_4*r_4*np.sin(theta_4 - theta_1)) + dtheta_3*(dtheta_3*l_1*l_3*m_4*np.sin(theta_3 - theta_1) + dtheta_3*l_1*l_3*m_5*np.sin(theta_3 - theta_1) + dtheta_3*l_1*m_3*r_3*np.sin(theta_3 - theta_1)) + dtheta_2*mu_2 + dtheta_2*(dtheta_2*l_1*l_2*m_3*np.sin(theta_2 - theta_1) + dtheta_2*l_1*l_2*m_4*np.sin(theta_2 - theta_1) + dtheta_2*l_1*l_2*m_5*np.sin(theta_2 - theta_1) + dtheta_2*l_1*m_2*r_2*np.sin(theta_2 - theta_1)) - dtheta_1*(mu_1 + mu_2) - ddx_c*(-l_1*m_2 - l_1*m_3 - l_1*m_4 - l_1*m_5 - m_1*r_1)*np.cos(theta_1) - g*(-l_1*m_2 - l_1*m_3 - l_1*m_4 - l_1*m_5 - m_1*r_1)*np.sin(theta_1))*np.cos(theta_4 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) - (-l_1**2*(l_4*m_5 + m_4*r_4)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_4 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_4*m_5 + m_4*r_4)*np.cos(theta_4 - theta_2))*(dtheta_5**2*l_2*m_5*r_5*np.sin(theta_5 - theta_2) + dtheta_4*(dtheta_4*l_2*l_4*m_5*np.sin(theta_4 - theta_2) + dtheta_4*l_2*m_4*r_4*np.sin(theta_4 - theta_2)) + dtheta_3*mu_3 + dtheta_3*(dtheta_3*l_2*l_3*m_4*np.sin(theta_3 - theta_2) + dtheta_3*l_2*l_3*m_5*np.sin(theta_3 - theta_2) + dtheta_3*l_2*m_3*r_3*np.sin(theta_3 - theta_2)) - dtheta_2*(mu_2 + mu_3) + dtheta_1*mu_2 + dtheta_1*(-dtheta_1*l_1*l_2*m_3*np.sin(theta_2 - theta_1) - dtheta_1*l_1*l_2*m_4*np.sin(theta_2 - theta_1) - dtheta_1*l_1*l_2*m_5*np.sin(theta_2 - theta_1) - dtheta_1*l_1*m_2*r_2*np.sin(theta_2 - theta_1)) - ddx_c*(-l_2*m_3 - l_2*m_4 - l_2*m_5 - m_2*r_2)*np.cos(theta_2) - g*(-l_2*m_3 - l_2*m_4 - l_2*m_5 - m_2*r_2)*np.sin(theta_2) - l_1*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*(dtheta_5**2*l_1*m_5*r_5*np.sin(theta_5 - theta_1) + dtheta_4*(dtheta_4*l_1*l_4*m_5*np.sin(theta_4 - theta_1) + dtheta_4*l_1*m_4*r_4*np.sin(theta_4 - theta_1)) + dtheta_3*(dtheta_3*l_1*l_3*m_4*np.sin(theta_3 - theta_1) + dtheta_3*l_1*l_3*m_5*np.sin(theta_3 - theta_1) + dtheta_3*l_1*m_3*r_3*np.sin(theta_3 - theta_1)) + dtheta_2*mu_2 + dtheta_2*(dtheta_2*l_1*l_2*m_3*np.sin(theta_2 - theta_1) + dtheta_2*l_1*l_2*m_4*np.sin(theta_2 - theta_1) + dtheta_2*l_1*l_2*m_5*np.sin(theta_2 - theta_1) + dtheta_2*l_1*m_2*r_2*np.sin(theta_2 - theta_1)) - dtheta_1*(mu_1 + mu_2) - ddx_c*(-l_1*m_2 - l_1*m_3 - l_1*m_4 - l_1*m_5 - m_1*r_1)*np.cos(theta_1) - g*(-l_1*m_2 - l_1*m_3 - l_1*m_4 - l_1*m_5 - m_1*r_1)*np.sin(theta_1))*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2))/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2) - (-l_1**2*(l_4*m_5 + m_4*r_4)*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_4 - theta_1)*np.cos(theta_3 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_3*(l_4*m_5 + m_4*r_4)*np.cos(theta_4 - theta_3) - (-l_1**2*(l_4*m_5 + m_4*r_4)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_4 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_4*m_5 + m_4*r_4)*np.cos(theta_4 - theta_2))*(-l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_3 - theta_2))/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2))*(dtheta_5**2*l_3*m_5*r_5*np.sin(theta_5 - theta_3) + dtheta_4*mu_4 + dtheta_4*(dtheta_4*l_3*l_4*m_5*np.sin(theta_4 - theta_3) + dtheta_4*l_3*m_4*r_4*np.sin(theta_4 - theta_3)) - dtheta_3*(mu_3 + mu_4) + dtheta_2*mu_3 + dtheta_2*(-dtheta_2*l_2*l_3*m_4*np.sin(theta_3 - theta_2) - dtheta_2*l_2*l_3*m_5*np.sin(theta_3 - theta_2) - dtheta_2*l_2*m_3*r_3*np.sin(theta_3 - theta_2)) + dtheta_1*(-dtheta_1*l_1*l_3*m_4*np.sin(theta_3 - theta_1) - dtheta_1*l_1*l_3*m_5*np.sin(theta_3 - theta_1) - dtheta_1*l_1*m_3*r_3*np.sin(theta_3 - theta_1)) - ddx_c*(-l_3*m_4 - l_3*m_5 - m_3*r_3)*np.cos(theta_3) - g*(-l_3*m_4 - l_3*m_5 - m_3*r_3)*np.sin(theta_3) - l_1*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(dtheta_5**2*l_1*m_5*r_5*np.sin(theta_5 - theta_1) + dtheta_4*(dtheta_4*l_1*l_4*m_5*np.sin(theta_4 - theta_1) + dtheta_4*l_1*m_4*r_4*np.sin(theta_4 - theta_1)) + dtheta_3*(dtheta_3*l_1*l_3*m_4*np.sin(theta_3 - theta_1) + dtheta_3*l_1*l_3*m_5*np.sin(theta_3 - theta_1) + dtheta_3*l_1*m_3*r_3*np.sin(theta_3 - theta_1)) + dtheta_2*mu_2 + dtheta_2*(dtheta_2*l_1*l_2*m_3*np.sin(theta_2 - theta_1) + dtheta_2*l_1*l_2*m_4*np.sin(theta_2 - theta_1) + dtheta_2*l_1*l_2*m_5*np.sin(theta_2 - theta_1) + dtheta_2*l_1*m_2*r_2*np.sin(theta_2 - theta_1)) - dtheta_1*(mu_1 + mu_2) - ddx_c*(-l_1*m_2 - l_1*m_3 - l_1*m_4 - l_1*m_5 - m_1*r_1)*np.cos(theta_1) - g*(-l_1*m_2 - l_1*m_3 - l_1*m_4 - l_1*m_5 - m_1*r_1)*np.sin(theta_1))*np.cos(theta_3 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) - (-l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_3 - theta_2))*(dtheta_5**2*l_2*m_5*r_5*np.sin(theta_5 - theta_2) + dtheta_4*(dtheta_4*l_2*l_4*m_5*np.sin(theta_4 - theta_2) + dtheta_4*l_2*m_4*r_4*np.sin(theta_4 - theta_2)) + dtheta_3*mu_3 + dtheta_3*(dtheta_3*l_2*l_3*m_4*np.sin(theta_3 - theta_2) + dtheta_3*l_2*l_3*m_5*np.sin(theta_3 - theta_2) + dtheta_3*l_2*m_3*r_3*np.sin(theta_3 - theta_2)) - dtheta_2*(mu_2 + mu_3) + dtheta_1*mu_2 + dtheta_1*(-dtheta_1*l_1*l_2*m_3*np.sin(theta_2 - theta_1) - dtheta_1*l_1*l_2*m_4*np.sin(theta_2 - theta_1) - dtheta_1*l_1*l_2*m_5*np.sin(theta_2 - theta_1) - dtheta_1*l_1*m_2*r_2*np.sin(theta_2 - theta_1)) - ddx_c*(-l_2*m_3 - l_2*m_4 - l_2*m_5 - m_2*r_2)*np.cos(theta_2) - g*(-l_2*m_3 - l_2*m_4 - l_2*m_5 - m_2*r_2)*np.sin(theta_2) - l_1*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*(dtheta_5**2*l_1*m_5*r_5*np.sin(theta_5 - theta_1) + dtheta_4*(dtheta_4*l_1*l_4*m_5*np.sin(theta_4 - theta_1) + dtheta_4*l_1*m_4*r_4*np.sin(theta_4 - theta_1)) + dtheta_3*(dtheta_3*l_1*l_3*m_4*np.sin(theta_3 - theta_1) + dtheta_3*l_1*l_3*m_5*np.sin(theta_3 - theta_1) + dtheta_3*l_1*m_3*r_3*np.sin(theta_3 - theta_1)) + dtheta_2*mu_2 + dtheta_2*(dtheta_2*l_1*l_2*m_3*np.sin(theta_2 - theta_1) + dtheta_2*l_1*l_2*m_4*np.sin(theta_2 - theta_1) + dtheta_2*l_1*l_2*m_5*np.sin(theta_2 - theta_1) + dtheta_2*l_1*m_2*r_2*np.sin(theta_2 - theta_1)) - dtheta_1*(mu_1 + mu_2) - ddx_c*(-l_1*m_2 - l_1*m_3 - l_1*m_4 - l_1*m_5 - m_1*r_1)*np.cos(theta_1) - g*(-l_1*m_2 - l_1*m_3 - l_1*m_4 - l_1*m_5 - m_1*r_1)*np.sin(theta_1))*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2))/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2))/(I_3 - l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)**2*np.cos(theta_3 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_3**2*m_4 + l_3**2*m_5 + m_3*r_3**2 - (-l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_3 - theta_2))**2/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2)))/(I_4 - l_1**2*(l_4*m_5 + m_4*r_4)**2*np.cos(theta_4 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_4**2*m_5 + m_4*r_4**2 - (-l_1**2*(l_4*m_5 + m_4*r_4)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_4 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_4*m_5 + m_4*r_4)*np.cos(theta_4 - theta_2))**2/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2) - (-l_1**2*(l_4*m_5 + m_4*r_4)*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_4 - theta_1)*np.cos(theta_3 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_3*(l_4*m_5 + m_4*r_4)*np.cos(theta_4 - theta_3) - (-l_1**2*(l_4*m_5 + m_4*r_4)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_4 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_4*m_5 + m_4*r_4)*np.cos(theta_4 - theta_2))*(-l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_3 - theta_2))/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2))**2/(I_3 - l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)**2*np.cos(theta_3 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_3**2*m_4 + l_3**2*m_5 + m_3*r_3**2 - (-l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_3 - theta_2))**2/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2))))/(I_5 - l_1**2*m_5**2*r_5**2*np.cos(theta_5 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + m_5*r_5**2 - (-l_1**2*m_5*r_5*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_5 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*m_5*r_5*np.cos(theta_5 - theta_2))**2/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2) - (-l_1**2*m_5*r_5*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_5 - theta_1)*np.cos(theta_3 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_3*m_5*r_5*np.cos(theta_5 - theta_3) - (-l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_3 - theta_2))*(-l_1**2*m_5*r_5*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_5 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*m_5*r_5*np.cos(theta_5 - theta_2))/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2))**2/(I_3 - l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)**2*np.cos(theta_3 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_3**2*m_4 + l_3**2*m_5 + m_3*r_3**2 - (-l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_3 - theta_2))**2/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2)) - (-l_1**2*m_5*r_5*(l_4*m_5 + m_4*r_4)*np.cos(theta_5 - theta_1)*np.cos(theta_4 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_4*m_5*r_5*np.cos(theta_5 - theta_4) - (-l_1**2*(l_4*m_5 + m_4*r_4)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_4 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_4*m_5 + m_4*r_4)*np.cos(theta_4 - theta_2))*(-l_1**2*m_5*r_5*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_5 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*m_5*r_5*np.cos(theta_5 - theta_2))/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2) - (-l_1**2*(l_4*m_5 + m_4*r_4)*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_4 - theta_1)*np.cos(theta_3 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_3*(l_4*m_5 + m_4*r_4)*np.cos(theta_4 - theta_3) - (-l_1**2*(l_4*m_5 + m_4*r_4)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_4 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_4*m_5 + m_4*r_4)*np.cos(theta_4 - theta_2))*(-l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_3 - theta_2))/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2))*(-l_1**2*m_5*r_5*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_5 - theta_1)*np.cos(theta_3 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_3*m_5*r_5*np.cos(theta_5 - theta_3) - (-l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_3 - theta_2))*(-l_1**2*m_5*r_5*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_5 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*m_5*r_5*np.cos(theta_5 - theta_2))/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2))/(I_3 - l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)**2*np.cos(theta_3 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_3**2*m_4 + l_3**2*m_5 + m_3*r_3**2 - (-l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_3 - theta_2))**2/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2)))**2/(I_4 - l_1**2*(l_4*m_5 + m_4*r_4)**2*np.cos(theta_4 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_4**2*m_5 + m_4*r_4**2 - (-l_1**2*(l_4*m_5 + m_4*r_4)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_4 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_4*m_5 + m_4*r_4)*np.cos(theta_4 - theta_2))**2/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2) - (-l_1**2*(l_4*m_5 + m_4*r_4)*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_4 - theta_1)*np.cos(theta_3 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_3*(l_4*m_5 + m_4*r_4)*np.cos(theta_4 - theta_3) - (-l_1**2*(l_4*m_5 + m_4*r_4)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_4 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_4*m_5 + m_4*r_4)*np.cos(theta_4 - theta_2))*(-l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_3 - theta_2))/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2))**2/(I_3 - l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)**2*np.cos(theta_3 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_3**2*m_4 + l_3**2*m_5 + m_3*r_3**2 - (-l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_3 - theta_2))**2/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2)))))/(I_3 - l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)**2*np.cos(theta_3 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_3**2*m_4 + l_3**2*m_5 + m_3*r_3**2 - (-l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_3 - theta_2))**2/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2)),


                      (dtheta_5**2*l_4*m_5*r_5*np.sin(theta_5 - theta_4) + dtheta_5*mu_5 - dtheta_4*(mu_4 + mu_5) + dtheta_3*mu_4 + dtheta_3*(-dtheta_3*l_3*l_4*m_5*np.sin(theta_4 - theta_3) - dtheta_3*l_3*m_4*r_4*np.sin(theta_4 - theta_3)) + dtheta_2*(-dtheta_2*l_2*l_4*m_5*np.sin(theta_4 - theta_2) - dtheta_2*l_2*m_4*r_4*np.sin(theta_4 - theta_2)) + dtheta_1*(-dtheta_1*l_1*l_4*m_5*np.sin(theta_4 - theta_1) - dtheta_1*l_1*m_4*r_4*np.sin(theta_4 - theta_1)) - ddx_c*(-l_4*m_5 - m_4*r_4)*np.cos(theta_4) - g*(-l_4*m_5 - m_4*r_4)*np.sin(theta_4) - l_1*(l_4*m_5 + m_4*r_4)*(dtheta_5**2*l_1*m_5*r_5*np.sin(theta_5 - theta_1) + dtheta_4*(dtheta_4*l_1*l_4*m_5*np.sin(theta_4 - theta_1) + dtheta_4*l_1*m_4*r_4*np.sin(theta_4 - theta_1)) + dtheta_3*(dtheta_3*l_1*l_3*m_4*np.sin(theta_3 - theta_1) + dtheta_3*l_1*l_3*m_5*np.sin(theta_3 - theta_1) + dtheta_3*l_1*m_3*r_3*np.sin(theta_3 - theta_1)) + dtheta_2*mu_2 + dtheta_2*(dtheta_2*l_1*l_2*m_3*np.sin(theta_2 - theta_1) + dtheta_2*l_1*l_2*m_4*np.sin(theta_2 - theta_1) + dtheta_2*l_1*l_2*m_5*np.sin(theta_2 - theta_1) + dtheta_2*l_1*m_2*r_2*np.sin(theta_2 - theta_1)) - dtheta_1*(mu_1 + mu_2) - ddx_c*(-l_1*m_2 - l_1*m_3 - l_1*m_4 - l_1*m_5 - m_1*r_1)*np.cos(theta_1) - g*(-l_1*m_2 - l_1*m_3 - l_1*m_4 - l_1*m_5 - m_1*r_1)*np.sin(theta_1))*np.cos(theta_4 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) - (-l_1**2*(l_4*m_5 + m_4*r_4)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_4 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_4*m_5 + m_4*r_4)*np.cos(theta_4 - theta_2))*(dtheta_5**2*l_2*m_5*r_5*np.sin(theta_5 - theta_2) + dtheta_4*(dtheta_4*l_2*l_4*m_5*np.sin(theta_4 - theta_2) + dtheta_4*l_2*m_4*r_4*np.sin(theta_4 - theta_2)) + dtheta_3*mu_3 + dtheta_3*(dtheta_3*l_2*l_3*m_4*np.sin(theta_3 - theta_2) + dtheta_3*l_2*l_3*m_5*np.sin(theta_3 - theta_2) + dtheta_3*l_2*m_3*r_3*np.sin(theta_3 - theta_2)) - dtheta_2*(mu_2 + mu_3) + dtheta_1*mu_2 + dtheta_1*(-dtheta_1*l_1*l_2*m_3*np.sin(theta_2 - theta_1) - dtheta_1*l_1*l_2*m_4*np.sin(theta_2 - theta_1) - dtheta_1*l_1*l_2*m_5*np.sin(theta_2 - theta_1) - dtheta_1*l_1*m_2*r_2*np.sin(theta_2 - theta_1)) - ddx_c*(-l_2*m_3 - l_2*m_4 - l_2*m_5 - m_2*r_2)*np.cos(theta_2) - g*(-l_2*m_3 - l_2*m_4 - l_2*m_5 - m_2*r_2)*np.sin(theta_2) - l_1*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*(dtheta_5**2*l_1*m_5*r_5*np.sin(theta_5 - theta_1) + dtheta_4*(dtheta_4*l_1*l_4*m_5*np.sin(theta_4 - theta_1) + dtheta_4*l_1*m_4*r_4*np.sin(theta_4 - theta_1)) + dtheta_3*(dtheta_3*l_1*l_3*m_4*np.sin(theta_3 - theta_1) + dtheta_3*l_1*l_3*m_5*np.sin(theta_3 - theta_1) + dtheta_3*l_1*m_3*r_3*np.sin(theta_3 - theta_1)) + dtheta_2*mu_2 + dtheta_2*(dtheta_2*l_1*l_2*m_3*np.sin(theta_2 - theta_1) + dtheta_2*l_1*l_2*m_4*np.sin(theta_2 - theta_1) + dtheta_2*l_1*l_2*m_5*np.sin(theta_2 - theta_1) + dtheta_2*l_1*m_2*r_2*np.sin(theta_2 - theta_1)) - dtheta_1*(mu_1 + mu_2) - ddx_c*(-l_1*m_2 - l_1*m_3 - l_1*m_4 - l_1*m_5 - m_1*r_1)*np.cos(theta_1) - g*(-l_1*m_2 - l_1*m_3 - l_1*m_4 - l_1*m_5 - m_1*r_1)*np.sin(theta_1))*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2))/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2) - (-l_1**2*(l_4*m_5 + m_4*r_4)*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_4 - theta_1)*np.cos(theta_3 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_3*(l_4*m_5 + m_4*r_4)*np.cos(theta_4 - theta_3) - (-l_1**2*(l_4*m_5 + m_4*r_4)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_4 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_4*m_5 + m_4*r_4)*np.cos(theta_4 - theta_2))*(-l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_3 - theta_2))/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2))*(dtheta_5**2*l_3*m_5*r_5*np.sin(theta_5 - theta_3) + dtheta_4*mu_4 + dtheta_4*(dtheta_4*l_3*l_4*m_5*np.sin(theta_4 - theta_3) + dtheta_4*l_3*m_4*r_4*np.sin(theta_4 - theta_3)) - dtheta_3*(mu_3 + mu_4) + dtheta_2*mu_3 + dtheta_2*(-dtheta_2*l_2*l_3*m_4*np.sin(theta_3 - theta_2) - dtheta_2*l_2*l_3*m_5*np.sin(theta_3 - theta_2) - dtheta_2*l_2*m_3*r_3*np.sin(theta_3 - theta_2)) + dtheta_1*(-dtheta_1*l_1*l_3*m_4*np.sin(theta_3 - theta_1) - dtheta_1*l_1*l_3*m_5*np.sin(theta_3 - theta_1) - dtheta_1*l_1*m_3*r_3*np.sin(theta_3 - theta_1)) - ddx_c*(-l_3*m_4 - l_3*m_5 - m_3*r_3)*np.cos(theta_3) - g*(-l_3*m_4 - l_3*m_5 - m_3*r_3)*np.sin(theta_3) - l_1*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(dtheta_5**2*l_1*m_5*r_5*np.sin(theta_5 - theta_1) + dtheta_4*(dtheta_4*l_1*l_4*m_5*np.sin(theta_4 - theta_1) + dtheta_4*l_1*m_4*r_4*np.sin(theta_4 - theta_1)) + dtheta_3*(dtheta_3*l_1*l_3*m_4*np.sin(theta_3 - theta_1) + dtheta_3*l_1*l_3*m_5*np.sin(theta_3 - theta_1) + dtheta_3*l_1*m_3*r_3*np.sin(theta_3 - theta_1)) + dtheta_2*mu_2 + dtheta_2*(dtheta_2*l_1*l_2*m_3*np.sin(theta_2 - theta_1) + dtheta_2*l_1*l_2*m_4*np.sin(theta_2 - theta_1) + dtheta_2*l_1*l_2*m_5*np.sin(theta_2 - theta_1) + dtheta_2*l_1*m_2*r_2*np.sin(theta_2 - theta_1)) - dtheta_1*(mu_1 + mu_2) - ddx_c*(-l_1*m_2 - l_1*m_3 - l_1*m_4 - l_1*m_5 - m_1*r_1)*np.cos(theta_1) - g*(-l_1*m_2 - l_1*m_3 - l_1*m_4 - l_1*m_5 - m_1*r_1)*np.sin(theta_1))*np.cos(theta_3 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) - (-l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_3 - theta_2))*(dtheta_5**2*l_2*m_5*r_5*np.sin(theta_5 - theta_2) + dtheta_4*(dtheta_4*l_2*l_4*m_5*np.sin(theta_4 - theta_2) + dtheta_4*l_2*m_4*r_4*np.sin(theta_4 - theta_2)) + dtheta_3*mu_3 + dtheta_3*(dtheta_3*l_2*l_3*m_4*np.sin(theta_3 - theta_2) + dtheta_3*l_2*l_3*m_5*np.sin(theta_3 - theta_2) + dtheta_3*l_2*m_3*r_3*np.sin(theta_3 - theta_2)) - dtheta_2*(mu_2 + mu_3) + dtheta_1*mu_2 + dtheta_1*(-dtheta_1*l_1*l_2*m_3*np.sin(theta_2 - theta_1) - dtheta_1*l_1*l_2*m_4*np.sin(theta_2 - theta_1) - dtheta_1*l_1*l_2*m_5*np.sin(theta_2 - theta_1) - dtheta_1*l_1*m_2*r_2*np.sin(theta_2 - theta_1)) - ddx_c*(-l_2*m_3 - l_2*m_4 - l_2*m_5 - m_2*r_2)*np.cos(theta_2) - g*(-l_2*m_3 - l_2*m_4 - l_2*m_5 - m_2*r_2)*np.sin(theta_2) - l_1*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*(dtheta_5**2*l_1*m_5*r_5*np.sin(theta_5 - theta_1) + dtheta_4*(dtheta_4*l_1*l_4*m_5*np.sin(theta_4 - theta_1) + dtheta_4*l_1*m_4*r_4*np.sin(theta_4 - theta_1)) + dtheta_3*(dtheta_3*l_1*l_3*m_4*np.sin(theta_3 - theta_1) + dtheta_3*l_1*l_3*m_5*np.sin(theta_3 - theta_1) + dtheta_3*l_1*m_3*r_3*np.sin(theta_3 - theta_1)) + dtheta_2*mu_2 + dtheta_2*(dtheta_2*l_1*l_2*m_3*np.sin(theta_2 - theta_1) + dtheta_2*l_1*l_2*m_4*np.sin(theta_2 - theta_1) + dtheta_2*l_1*l_2*m_5*np.sin(theta_2 - theta_1) + dtheta_2*l_1*m_2*r_2*np.sin(theta_2 - theta_1)) - dtheta_1*(mu_1 + mu_2) - ddx_c*(-l_1*m_2 - l_1*m_3 - l_1*m_4 - l_1*m_5 - m_1*r_1)*np.cos(theta_1) - g*(-l_1*m_2 - l_1*m_3 - l_1*m_4 - l_1*m_5 - m_1*r_1)*np.sin(theta_1))*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2))/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2))/(I_3 - l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)**2*np.cos(theta_3 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_3**2*m_4 + l_3**2*m_5 + m_3*r_3**2 - (-l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_3 - theta_2))**2/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2)) - (-l_1**2*m_5*r_5*(l_4*m_5 + m_4*r_4)*np.cos(theta_5 - theta_1)*np.cos(theta_4 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_4*m_5*r_5*np.cos(theta_5 - theta_4) - (-l_1**2*(l_4*m_5 + m_4*r_4)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_4 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_4*m_5 + m_4*r_4)*np.cos(theta_4 - theta_2))*(-l_1**2*m_5*r_5*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_5 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*m_5*r_5*np.cos(theta_5 - theta_2))/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2) - (-l_1**2*(l_4*m_5 + m_4*r_4)*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_4 - theta_1)*np.cos(theta_3 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_3*(l_4*m_5 + m_4*r_4)*np.cos(theta_4 - theta_3) - (-l_1**2*(l_4*m_5 + m_4*r_4)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_4 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_4*m_5 + m_4*r_4)*np.cos(theta_4 - theta_2))*(-l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_3 - theta_2))/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2))*(-l_1**2*m_5*r_5*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_5 - theta_1)*np.cos(theta_3 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_3*m_5*r_5*np.cos(theta_5 - theta_3) - (-l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_3 - theta_2))*(-l_1**2*m_5*r_5*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_5 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*m_5*r_5*np.cos(theta_5 - theta_2))/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2))/(I_3 - l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)**2*np.cos(theta_3 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_3**2*m_4 + l_3**2*m_5 + m_3*r_3**2 - (-l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_3 - theta_2))**2/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2)))*(-dtheta_5*mu_5 - dtheta_4**2*l_4*m_5*r_5*np.sin(theta_5 - theta_4) + dtheta_4*mu_5 - dtheta_3**2*l_3*m_5*r_5*np.sin(theta_5 - theta_3) - dtheta_2**2*l_2*m_5*r_5*np.sin(theta_5 - theta_2) - dtheta_1**2*l_1*m_5*r_5*np.sin(theta_5 - theta_1) + ddx_c*m_5*r_5*np.cos(theta_5) + g*m_5*r_5*np.sin(theta_5) - l_1*m_5*r_5*(dtheta_5**2*l_1*m_5*r_5*np.sin(theta_5 - theta_1) + dtheta_4*(dtheta_4*l_1*l_4*m_5*np.sin(theta_4 - theta_1) + dtheta_4*l_1*m_4*r_4*np.sin(theta_4 - theta_1)) + dtheta_3*(dtheta_3*l_1*l_3*m_4*np.sin(theta_3 - theta_1) + dtheta_3*l_1*l_3*m_5*np.sin(theta_3 - theta_1) + dtheta_3*l_1*m_3*r_3*np.sin(theta_3 - theta_1)) + dtheta_2*mu_2 + dtheta_2*(dtheta_2*l_1*l_2*m_3*np.sin(theta_2 - theta_1) + dtheta_2*l_1*l_2*m_4*np.sin(theta_2 - theta_1) + dtheta_2*l_1*l_2*m_5*np.sin(theta_2 - theta_1) + dtheta_2*l_1*m_2*r_2*np.sin(theta_2 - theta_1)) - dtheta_1*(mu_1 + mu_2) - ddx_c*(-l_1*m_2 - l_1*m_3 - l_1*m_4 - l_1*m_5 - m_1*r_1)*np.cos(theta_1) - g*(-l_1*m_2 - l_1*m_3 - l_1*m_4 - l_1*m_5 - m_1*r_1)*np.sin(theta_1))*np.cos(theta_5 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) - (-l_1**2*m_5*r_5*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_5 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*m_5*r_5*np.cos(theta_5 - theta_2))*(dtheta_5**2*l_2*m_5*r_5*np.sin(theta_5 - theta_2) + dtheta_4*(dtheta_4*l_2*l_4*m_5*np.sin(theta_4 - theta_2) + dtheta_4*l_2*m_4*r_4*np.sin(theta_4 - theta_2)) + dtheta_3*mu_3 + dtheta_3*(dtheta_3*l_2*l_3*m_4*np.sin(theta_3 - theta_2) + dtheta_3*l_2*l_3*m_5*np.sin(theta_3 - theta_2) + dtheta_3*l_2*m_3*r_3*np.sin(theta_3 - theta_2)) - dtheta_2*(mu_2 + mu_3) + dtheta_1*mu_2 + dtheta_1*(-dtheta_1*l_1*l_2*m_3*np.sin(theta_2 - theta_1) - dtheta_1*l_1*l_2*m_4*np.sin(theta_2 - theta_1) - dtheta_1*l_1*l_2*m_5*np.sin(theta_2 - theta_1) - dtheta_1*l_1*m_2*r_2*np.sin(theta_2 - theta_1)) - ddx_c*(-l_2*m_3 - l_2*m_4 - l_2*m_5 - m_2*r_2)*np.cos(theta_2) - g*(-l_2*m_3 - l_2*m_4 - l_2*m_5 - m_2*r_2)*np.sin(theta_2) - l_1*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*(dtheta_5**2*l_1*m_5*r_5*np.sin(theta_5 - theta_1) + dtheta_4*(dtheta_4*l_1*l_4*m_5*np.sin(theta_4 - theta_1) + dtheta_4*l_1*m_4*r_4*np.sin(theta_4 - theta_1)) + dtheta_3*(dtheta_3*l_1*l_3*m_4*np.sin(theta_3 - theta_1) + dtheta_3*l_1*l_3*m_5*np.sin(theta_3 - theta_1) + dtheta_3*l_1*m_3*r_3*np.sin(theta_3 - theta_1)) + dtheta_2*mu_2 + dtheta_2*(dtheta_2*l_1*l_2*m_3*np.sin(theta_2 - theta_1) + dtheta_2*l_1*l_2*m_4*np.sin(theta_2 - theta_1) + dtheta_2*l_1*l_2*m_5*np.sin(theta_2 - theta_1) + dtheta_2*l_1*m_2*r_2*np.sin(theta_2 - theta_1)) - dtheta_1*(mu_1 + mu_2) - ddx_c*(-l_1*m_2 - l_1*m_3 - l_1*m_4 - l_1*m_5 - m_1*r_1)*np.cos(theta_1) - g*(-l_1*m_2 - l_1*m_3 - l_1*m_4 - l_1*m_5 - m_1*r_1)*np.sin(theta_1))*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2))/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2) - (-l_1**2*m_5*r_5*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_5 - theta_1)*np.cos(theta_3 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_3*m_5*r_5*np.cos(theta_5 - theta_3) - (-l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_3 - theta_2))*(-l_1**2*m_5*r_5*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_5 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*m_5*r_5*np.cos(theta_5 - theta_2))/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2))*(dtheta_5**2*l_3*m_5*r_5*np.sin(theta_5 - theta_3) + dtheta_4*mu_4 + dtheta_4*(dtheta_4*l_3*l_4*m_5*np.sin(theta_4 - theta_3) + dtheta_4*l_3*m_4*r_4*np.sin(theta_4 - theta_3)) - dtheta_3*(mu_3 + mu_4) + dtheta_2*mu_3 + dtheta_2*(-dtheta_2*l_2*l_3*m_4*np.sin(theta_3 - theta_2) - dtheta_2*l_2*l_3*m_5*np.sin(theta_3 - theta_2) - dtheta_2*l_2*m_3*r_3*np.sin(theta_3 - theta_2)) + dtheta_1*(-dtheta_1*l_1*l_3*m_4*np.sin(theta_3 - theta_1) - dtheta_1*l_1*l_3*m_5*np.sin(theta_3 - theta_1) - dtheta_1*l_1*m_3*r_3*np.sin(theta_3 - theta_1)) - ddx_c*(-l_3*m_4 - l_3*m_5 - m_3*r_3)*np.cos(theta_3) - g*(-l_3*m_4 - l_3*m_5 - m_3*r_3)*np.sin(theta_3) - l_1*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(dtheta_5**2*l_1*m_5*r_5*np.sin(theta_5 - theta_1) + dtheta_4*(dtheta_4*l_1*l_4*m_5*np.sin(theta_4 - theta_1) + dtheta_4*l_1*m_4*r_4*np.sin(theta_4 - theta_1)) + dtheta_3*(dtheta_3*l_1*l_3*m_4*np.sin(theta_3 - theta_1) + dtheta_3*l_1*l_3*m_5*np.sin(theta_3 - theta_1) + dtheta_3*l_1*m_3*r_3*np.sin(theta_3 - theta_1)) + dtheta_2*mu_2 + dtheta_2*(dtheta_2*l_1*l_2*m_3*np.sin(theta_2 - theta_1) + dtheta_2*l_1*l_2*m_4*np.sin(theta_2 - theta_1) + dtheta_2*l_1*l_2*m_5*np.sin(theta_2 - theta_1) + dtheta_2*l_1*m_2*r_2*np.sin(theta_2 - theta_1)) - dtheta_1*(mu_1 + mu_2) - ddx_c*(-l_1*m_2 - l_1*m_3 - l_1*m_4 - l_1*m_5 - m_1*r_1)*np.cos(theta_1) - g*(-l_1*m_2 - l_1*m_3 - l_1*m_4 - l_1*m_5 - m_1*r_1)*np.sin(theta_1))*np.cos(theta_3 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) - (-l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_3 - theta_2))*(dtheta_5**2*l_2*m_5*r_5*np.sin(theta_5 - theta_2) + dtheta_4*(dtheta_4*l_2*l_4*m_5*np.sin(theta_4 - theta_2) + dtheta_4*l_2*m_4*r_4*np.sin(theta_4 - theta_2)) + dtheta_3*mu_3 + dtheta_3*(dtheta_3*l_2*l_3*m_4*np.sin(theta_3 - theta_2) + dtheta_3*l_2*l_3*m_5*np.sin(theta_3 - theta_2) + dtheta_3*l_2*m_3*r_3*np.sin(theta_3 - theta_2)) - dtheta_2*(mu_2 + mu_3) + dtheta_1*mu_2 + dtheta_1*(-dtheta_1*l_1*l_2*m_3*np.sin(theta_2 - theta_1) - dtheta_1*l_1*l_2*m_4*np.sin(theta_2 - theta_1) - dtheta_1*l_1*l_2*m_5*np.sin(theta_2 - theta_1) - dtheta_1*l_1*m_2*r_2*np.sin(theta_2 - theta_1)) - ddx_c*(-l_2*m_3 - l_2*m_4 - l_2*m_5 - m_2*r_2)*np.cos(theta_2) - g*(-l_2*m_3 - l_2*m_4 - l_2*m_5 - m_2*r_2)*np.sin(theta_2) - l_1*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*(dtheta_5**2*l_1*m_5*r_5*np.sin(theta_5 - theta_1) + dtheta_4*(dtheta_4*l_1*l_4*m_5*np.sin(theta_4 - theta_1) + dtheta_4*l_1*m_4*r_4*np.sin(theta_4 - theta_1)) + dtheta_3*(dtheta_3*l_1*l_3*m_4*np.sin(theta_3 - theta_1) + dtheta_3*l_1*l_3*m_5*np.sin(theta_3 - theta_1) + dtheta_3*l_1*m_3*r_3*np.sin(theta_3 - theta_1)) + dtheta_2*mu_2 + dtheta_2*(dtheta_2*l_1*l_2*m_3*np.sin(theta_2 - theta_1) + dtheta_2*l_1*l_2*m_4*np.sin(theta_2 - theta_1) + dtheta_2*l_1*l_2*m_5*np.sin(theta_2 - theta_1) + dtheta_2*l_1*m_2*r_2*np.sin(theta_2 - theta_1)) - dtheta_1*(mu_1 + mu_2) - ddx_c*(-l_1*m_2 - l_1*m_3 - l_1*m_4 - l_1*m_5 - m_1*r_1)*np.cos(theta_1) - g*(-l_1*m_2 - l_1*m_3 - l_1*m_4 - l_1*m_5 - m_1*r_1)*np.sin(theta_1))*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2))/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2))/(I_3 - l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)**2*np.cos(theta_3 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_3**2*m_4 + l_3**2*m_5 + m_3*r_3**2 - (-l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_3 - theta_2))**2/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2)) - (-l_1**2*m_5*r_5*(l_4*m_5 + m_4*r_4)*np.cos(theta_5 - theta_1)*np.cos(theta_4 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_4*m_5*r_5*np.cos(theta_5 - theta_4) - (-l_1**2*(l_4*m_5 + m_4*r_4)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_4 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_4*m_5 + m_4*r_4)*np.cos(theta_4 - theta_2))*(-l_1**2*m_5*r_5*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_5 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*m_5*r_5*np.cos(theta_5 - theta_2))/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2) - (-l_1**2*(l_4*m_5 + m_4*r_4)*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_4 - theta_1)*np.cos(theta_3 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_3*(l_4*m_5 + m_4*r_4)*np.cos(theta_4 - theta_3) - (-l_1**2*(l_4*m_5 + m_4*r_4)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_4 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_4*m_5 + m_4*r_4)*np.cos(theta_4 - theta_2))*(-l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_3 - theta_2))/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2))*(-l_1**2*m_5*r_5*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_5 - theta_1)*np.cos(theta_3 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_3*m_5*r_5*np.cos(theta_5 - theta_3) - (-l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_3 - theta_2))*(-l_1**2*m_5*r_5*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_5 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*m_5*r_5*np.cos(theta_5 - theta_2))/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2))/(I_3 - l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)**2*np.cos(theta_3 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_3**2*m_4 + l_3**2*m_5 + m_3*r_3**2 - (-l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_3 - theta_2))**2/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2)))*(dtheta_5**2*l_4*m_5*r_5*np.sin(theta_5 - theta_4) + dtheta_5*mu_5 - dtheta_4*(mu_4 + mu_5) + dtheta_3*mu_4 + dtheta_3*(-dtheta_3*l_3*l_4*m_5*np.sin(theta_4 - theta_3) - dtheta_3*l_3*m_4*r_4*np.sin(theta_4 - theta_3)) + dtheta_2*(-dtheta_2*l_2*l_4*m_5*np.sin(theta_4 - theta_2) - dtheta_2*l_2*m_4*r_4*np.sin(theta_4 - theta_2)) + dtheta_1*(-dtheta_1*l_1*l_4*m_5*np.sin(theta_4 - theta_1) - dtheta_1*l_1*m_4*r_4*np.sin(theta_4 - theta_1)) - ddx_c*(-l_4*m_5 - m_4*r_4)*np.cos(theta_4) - g*(-l_4*m_5 - m_4*r_4)*np.sin(theta_4) - l_1*(l_4*m_5 + m_4*r_4)*(dtheta_5**2*l_1*m_5*r_5*np.sin(theta_5 - theta_1) + dtheta_4*(dtheta_4*l_1*l_4*m_5*np.sin(theta_4 - theta_1) + dtheta_4*l_1*m_4*r_4*np.sin(theta_4 - theta_1)) + dtheta_3*(dtheta_3*l_1*l_3*m_4*np.sin(theta_3 - theta_1) + dtheta_3*l_1*l_3*m_5*np.sin(theta_3 - theta_1) + dtheta_3*l_1*m_3*r_3*np.sin(theta_3 - theta_1)) + dtheta_2*mu_2 + dtheta_2*(dtheta_2*l_1*l_2*m_3*np.sin(theta_2 - theta_1) + dtheta_2*l_1*l_2*m_4*np.sin(theta_2 - theta_1) + dtheta_2*l_1*l_2*m_5*np.sin(theta_2 - theta_1) + dtheta_2*l_1*m_2*r_2*np.sin(theta_2 - theta_1)) - dtheta_1*(mu_1 + mu_2) - ddx_c*(-l_1*m_2 - l_1*m_3 - l_1*m_4 - l_1*m_5 - m_1*r_1)*np.cos(theta_1) - g*(-l_1*m_2 - l_1*m_3 - l_1*m_4 - l_1*m_5 - m_1*r_1)*np.sin(theta_1))*np.cos(theta_4 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) - (-l_1**2*(l_4*m_5 + m_4*r_4)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_4 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_4*m_5 + m_4*r_4)*np.cos(theta_4 - theta_2))*(dtheta_5**2*l_2*m_5*r_5*np.sin(theta_5 - theta_2) + dtheta_4*(dtheta_4*l_2*l_4*m_5*np.sin(theta_4 - theta_2) + dtheta_4*l_2*m_4*r_4*np.sin(theta_4 - theta_2)) + dtheta_3*mu_3 + dtheta_3*(dtheta_3*l_2*l_3*m_4*np.sin(theta_3 - theta_2) + dtheta_3*l_2*l_3*m_5*np.sin(theta_3 - theta_2) + dtheta_3*l_2*m_3*r_3*np.sin(theta_3 - theta_2)) - dtheta_2*(mu_2 + mu_3) + dtheta_1*mu_2 + dtheta_1*(-dtheta_1*l_1*l_2*m_3*np.sin(theta_2 - theta_1) - dtheta_1*l_1*l_2*m_4*np.sin(theta_2 - theta_1) - dtheta_1*l_1*l_2*m_5*np.sin(theta_2 - theta_1) - dtheta_1*l_1*m_2*r_2*np.sin(theta_2 - theta_1)) - ddx_c*(-l_2*m_3 - l_2*m_4 - l_2*m_5 - m_2*r_2)*np.cos(theta_2) - g*(-l_2*m_3 - l_2*m_4 - l_2*m_5 - m_2*r_2)*np.sin(theta_2) - l_1*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*(dtheta_5**2*l_1*m_5*r_5*np.sin(theta_5 - theta_1) + dtheta_4*(dtheta_4*l_1*l_4*m_5*np.sin(theta_4 - theta_1) + dtheta_4*l_1*m_4*r_4*np.sin(theta_4 - theta_1)) + dtheta_3*(dtheta_3*l_1*l_3*m_4*np.sin(theta_3 - theta_1) + dtheta_3*l_1*l_3*m_5*np.sin(theta_3 - theta_1) + dtheta_3*l_1*m_3*r_3*np.sin(theta_3 - theta_1)) + dtheta_2*mu_2 + dtheta_2*(dtheta_2*l_1*l_2*m_3*np.sin(theta_2 - theta_1) + dtheta_2*l_1*l_2*m_4*np.sin(theta_2 - theta_1) + dtheta_2*l_1*l_2*m_5*np.sin(theta_2 - theta_1) + dtheta_2*l_1*m_2*r_2*np.sin(theta_2 - theta_1)) - dtheta_1*(mu_1 + mu_2) - ddx_c*(-l_1*m_2 - l_1*m_3 - l_1*m_4 - l_1*m_5 - m_1*r_1)*np.cos(theta_1) - g*(-l_1*m_2 - l_1*m_3 - l_1*m_4 - l_1*m_5 - m_1*r_1)*np.sin(theta_1))*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2))/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2) - (-l_1**2*(l_4*m_5 + m_4*r_4)*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_4 - theta_1)*np.cos(theta_3 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_3*(l_4*m_5 + m_4*r_4)*np.cos(theta_4 - theta_3) - (-l_1**2*(l_4*m_5 + m_4*r_4)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_4 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_4*m_5 + m_4*r_4)*np.cos(theta_4 - theta_2))*(-l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_3 - theta_2))/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2))*(dtheta_5**2*l_3*m_5*r_5*np.sin(theta_5 - theta_3) + dtheta_4*mu_4 + dtheta_4*(dtheta_4*l_3*l_4*m_5*np.sin(theta_4 - theta_3) + dtheta_4*l_3*m_4*r_4*np.sin(theta_4 - theta_3)) - dtheta_3*(mu_3 + mu_4) + dtheta_2*mu_3 + dtheta_2*(-dtheta_2*l_2*l_3*m_4*np.sin(theta_3 - theta_2) - dtheta_2*l_2*l_3*m_5*np.sin(theta_3 - theta_2) - dtheta_2*l_2*m_3*r_3*np.sin(theta_3 - theta_2)) + dtheta_1*(-dtheta_1*l_1*l_3*m_4*np.sin(theta_3 - theta_1) - dtheta_1*l_1*l_3*m_5*np.sin(theta_3 - theta_1) - dtheta_1*l_1*m_3*r_3*np.sin(theta_3 - theta_1)) - ddx_c*(-l_3*m_4 - l_3*m_5 - m_3*r_3)*np.cos(theta_3) - g*(-l_3*m_4 - l_3*m_5 - m_3*r_3)*np.sin(theta_3) - l_1*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(dtheta_5**2*l_1*m_5*r_5*np.sin(theta_5 - theta_1) + dtheta_4*(dtheta_4*l_1*l_4*m_5*np.sin(theta_4 - theta_1) + dtheta_4*l_1*m_4*r_4*np.sin(theta_4 - theta_1)) + dtheta_3*(dtheta_3*l_1*l_3*m_4*np.sin(theta_3 - theta_1) + dtheta_3*l_1*l_3*m_5*np.sin(theta_3 - theta_1) + dtheta_3*l_1*m_3*r_3*np.sin(theta_3 - theta_1)) + dtheta_2*mu_2 + dtheta_2*(dtheta_2*l_1*l_2*m_3*np.sin(theta_2 - theta_1) + dtheta_2*l_1*l_2*m_4*np.sin(theta_2 - theta_1) + dtheta_2*l_1*l_2*m_5*np.sin(theta_2 - theta_1) + dtheta_2*l_1*m_2*r_2*np.sin(theta_2 - theta_1)) - dtheta_1*(mu_1 + mu_2) - ddx_c*(-l_1*m_2 - l_1*m_3 - l_1*m_4 - l_1*m_5 - m_1*r_1)*np.cos(theta_1) - g*(-l_1*m_2 - l_1*m_3 - l_1*m_4 - l_1*m_5 - m_1*r_1)*np.sin(theta_1))*np.cos(theta_3 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) - (-l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_3 - theta_2))*(dtheta_5**2*l_2*m_5*r_5*np.sin(theta_5 - theta_2) + dtheta_4*(dtheta_4*l_2*l_4*m_5*np.sin(theta_4 - theta_2) + dtheta_4*l_2*m_4*r_4*np.sin(theta_4 - theta_2)) + dtheta_3*mu_3 + dtheta_3*(dtheta_3*l_2*l_3*m_4*np.sin(theta_3 - theta_2) + dtheta_3*l_2*l_3*m_5*np.sin(theta_3 - theta_2) + dtheta_3*l_2*m_3*r_3*np.sin(theta_3 - theta_2)) - dtheta_2*(mu_2 + mu_3) + dtheta_1*mu_2 + dtheta_1*(-dtheta_1*l_1*l_2*m_3*np.sin(theta_2 - theta_1) - dtheta_1*l_1*l_2*m_4*np.sin(theta_2 - theta_1) - dtheta_1*l_1*l_2*m_5*np.sin(theta_2 - theta_1) - dtheta_1*l_1*m_2*r_2*np.sin(theta_2 - theta_1)) - ddx_c*(-l_2*m_3 - l_2*m_4 - l_2*m_5 - m_2*r_2)*np.cos(theta_2) - g*(-l_2*m_3 - l_2*m_4 - l_2*m_5 - m_2*r_2)*np.sin(theta_2) - l_1*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*(dtheta_5**2*l_1*m_5*r_5*np.sin(theta_5 - theta_1) + dtheta_4*(dtheta_4*l_1*l_4*m_5*np.sin(theta_4 - theta_1) + dtheta_4*l_1*m_4*r_4*np.sin(theta_4 - theta_1)) + dtheta_3*(dtheta_3*l_1*l_3*m_4*np.sin(theta_3 - theta_1) + dtheta_3*l_1*l_3*m_5*np.sin(theta_3 - theta_1) + dtheta_3*l_1*m_3*r_3*np.sin(theta_3 - theta_1)) + dtheta_2*mu_2 + dtheta_2*(dtheta_2*l_1*l_2*m_3*np.sin(theta_2 - theta_1) + dtheta_2*l_1*l_2*m_4*np.sin(theta_2 - theta_1) + dtheta_2*l_1*l_2*m_5*np.sin(theta_2 - theta_1) + dtheta_2*l_1*m_2*r_2*np.sin(theta_2 - theta_1)) - dtheta_1*(mu_1 + mu_2) - ddx_c*(-l_1*m_2 - l_1*m_3 - l_1*m_4 - l_1*m_5 - m_1*r_1)*np.cos(theta_1) - g*(-l_1*m_2 - l_1*m_3 - l_1*m_4 - l_1*m_5 - m_1*r_1)*np.sin(theta_1))*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2))/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2))/(I_3 - l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)**2*np.cos(theta_3 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_3**2*m_4 + l_3**2*m_5 + m_3*r_3**2 - (-l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_3 - theta_2))**2/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2)))/(I_4 - l_1**2*(l_4*m_5 + m_4*r_4)**2*np.cos(theta_4 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_4**2*m_5 + m_4*r_4**2 - (-l_1**2*(l_4*m_5 + m_4*r_4)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_4 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_4*m_5 + m_4*r_4)*np.cos(theta_4 - theta_2))**2/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2) - (-l_1**2*(l_4*m_5 + m_4*r_4)*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_4 - theta_1)*np.cos(theta_3 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_3*(l_4*m_5 + m_4*r_4)*np.cos(theta_4 - theta_3) - (-l_1**2*(l_4*m_5 + m_4*r_4)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_4 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_4*m_5 + m_4*r_4)*np.cos(theta_4 - theta_2))*(-l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_3 - theta_2))/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2))**2/(I_3 - l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)**2*np.cos(theta_3 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_3**2*m_4 + l_3**2*m_5 + m_3*r_3**2 - (-l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_3 - theta_2))**2/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2))))/(I_5 - l_1**2*m_5**2*r_5**2*np.cos(theta_5 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + m_5*r_5**2 - (-l_1**2*m_5*r_5*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_5 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*m_5*r_5*np.cos(theta_5 - theta_2))**2/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2) - (-l_1**2*m_5*r_5*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_5 - theta_1)*np.cos(theta_3 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_3*m_5*r_5*np.cos(theta_5 - theta_3) - (-l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_3 - theta_2))*(-l_1**2*m_5*r_5*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_5 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*m_5*r_5*np.cos(theta_5 - theta_2))/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2))**2/(I_3 - l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)**2*np.cos(theta_3 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_3**2*m_4 + l_3**2*m_5 + m_3*r_3**2 - (-l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_3 - theta_2))**2/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2)) - (-l_1**2*m_5*r_5*(l_4*m_5 + m_4*r_4)*np.cos(theta_5 - theta_1)*np.cos(theta_4 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_4*m_5*r_5*np.cos(theta_5 - theta_4) - (-l_1**2*(l_4*m_5 + m_4*r_4)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_4 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_4*m_5 + m_4*r_4)*np.cos(theta_4 - theta_2))*(-l_1**2*m_5*r_5*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_5 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*m_5*r_5*np.cos(theta_5 - theta_2))/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2) - (-l_1**2*(l_4*m_5 + m_4*r_4)*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_4 - theta_1)*np.cos(theta_3 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_3*(l_4*m_5 + m_4*r_4)*np.cos(theta_4 - theta_3) - (-l_1**2*(l_4*m_5 + m_4*r_4)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_4 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_4*m_5 + m_4*r_4)*np.cos(theta_4 - theta_2))*(-l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_3 - theta_2))/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2))*(-l_1**2*m_5*r_5*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_5 - theta_1)*np.cos(theta_3 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_3*m_5*r_5*np.cos(theta_5 - theta_3) - (-l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_3 - theta_2))*(-l_1**2*m_5*r_5*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_5 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*m_5*r_5*np.cos(theta_5 - theta_2))/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2))/(I_3 - l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)**2*np.cos(theta_3 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_3**2*m_4 + l_3**2*m_5 + m_3*r_3**2 - (-l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_3 - theta_2))**2/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2)))**2/(I_4 - l_1**2*(l_4*m_5 + m_4*r_4)**2*np.cos(theta_4 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_4**2*m_5 + m_4*r_4**2 - (-l_1**2*(l_4*m_5 + m_4*r_4)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_4 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_4*m_5 + m_4*r_4)*np.cos(theta_4 - theta_2))**2/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2) - (-l_1**2*(l_4*m_5 + m_4*r_4)*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_4 - theta_1)*np.cos(theta_3 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_3*(l_4*m_5 + m_4*r_4)*np.cos(theta_4 - theta_3) - (-l_1**2*(l_4*m_5 + m_4*r_4)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_4 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_4*m_5 + m_4*r_4)*np.cos(theta_4 - theta_2))*(-l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_3 - theta_2))/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2))**2/(I_3 - l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)**2*np.cos(theta_3 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_3**2*m_4 + l_3**2*m_5 + m_3*r_3**2 - (-l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_3 - theta_2))**2/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2)))))/(I_4 - l_1**2*(l_4*m_5 + m_4*r_4)**2*np.cos(theta_4 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_4**2*m_5 + m_4*r_4**2 - (-l_1**2*(l_4*m_5 + m_4*r_4)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_4 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_4*m_5 + m_4*r_4)*np.cos(theta_4 - theta_2))**2/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2) - (-l_1**2*(l_4*m_5 + m_4*r_4)*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_4 - theta_1)*np.cos(theta_3 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_3*(l_4*m_5 + m_4*r_4)*np.cos(theta_4 - theta_3) - (-l_1**2*(l_4*m_5 + m_4*r_4)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_4 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_4*m_5 + m_4*r_4)*np.cos(theta_4 - theta_2))*(-l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_3 - theta_2))/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2))**2/(I_3 - l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)**2*np.cos(theta_3 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_3**2*m_4 + l_3**2*m_5 + m_3*r_3**2 - (-l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_3 - theta_2))**2/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2))),


                      (-dtheta_5*mu_5 - dtheta_4**2*l_4*m_5*r_5*np.sin(theta_5 - theta_4) + dtheta_4*mu_5 - dtheta_3**2*l_3*m_5*r_5*np.sin(theta_5 - theta_3) - dtheta_2**2*l_2*m_5*r_5*np.sin(theta_5 - theta_2) - dtheta_1**2*l_1*m_5*r_5*np.sin(theta_5 - theta_1) + ddx_c*m_5*r_5*np.cos(theta_5) + g*m_5*r_5*np.sin(theta_5) - l_1*m_5*r_5*(dtheta_5**2*l_1*m_5*r_5*np.sin(theta_5 - theta_1) + dtheta_4*(dtheta_4*l_1*l_4*m_5*np.sin(theta_4 - theta_1) + dtheta_4*l_1*m_4*r_4*np.sin(theta_4 - theta_1)) + dtheta_3*(dtheta_3*l_1*l_3*m_4*np.sin(theta_3 - theta_1) + dtheta_3*l_1*l_3*m_5*np.sin(theta_3 - theta_1) + dtheta_3*l_1*m_3*r_3*np.sin(theta_3 - theta_1)) + dtheta_2*mu_2 + dtheta_2*(dtheta_2*l_1*l_2*m_3*np.sin(theta_2 - theta_1) + dtheta_2*l_1*l_2*m_4*np.sin(theta_2 - theta_1) + dtheta_2*l_1*l_2*m_5*np.sin(theta_2 - theta_1) + dtheta_2*l_1*m_2*r_2*np.sin(theta_2 - theta_1)) - dtheta_1*(mu_1 + mu_2) - ddx_c*(-l_1*m_2 - l_1*m_3 - l_1*m_4 - l_1*m_5 - m_1*r_1)*np.cos(theta_1) - g*(-l_1*m_2 - l_1*m_3 - l_1*m_4 - l_1*m_5 - m_1*r_1)*np.sin(theta_1))*np.cos(theta_5 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) - (-l_1**2*m_5*r_5*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_5 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*m_5*r_5*np.cos(theta_5 - theta_2))*(dtheta_5**2*l_2*m_5*r_5*np.sin(theta_5 - theta_2) + dtheta_4*(dtheta_4*l_2*l_4*m_5*np.sin(theta_4 - theta_2) + dtheta_4*l_2*m_4*r_4*np.sin(theta_4 - theta_2)) + dtheta_3*mu_3 + dtheta_3*(dtheta_3*l_2*l_3*m_4*np.sin(theta_3 - theta_2) + dtheta_3*l_2*l_3*m_5*np.sin(theta_3 - theta_2) + dtheta_3*l_2*m_3*r_3*np.sin(theta_3 - theta_2)) - dtheta_2*(mu_2 + mu_3) + dtheta_1*mu_2 + dtheta_1*(-dtheta_1*l_1*l_2*m_3*np.sin(theta_2 - theta_1) - dtheta_1*l_1*l_2*m_4*np.sin(theta_2 - theta_1) - dtheta_1*l_1*l_2*m_5*np.sin(theta_2 - theta_1) - dtheta_1*l_1*m_2*r_2*np.sin(theta_2 - theta_1)) - ddx_c*(-l_2*m_3 - l_2*m_4 - l_2*m_5 - m_2*r_2)*np.cos(theta_2) - g*(-l_2*m_3 - l_2*m_4 - l_2*m_5 - m_2*r_2)*np.sin(theta_2) - l_1*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*(dtheta_5**2*l_1*m_5*r_5*np.sin(theta_5 - theta_1) + dtheta_4*(dtheta_4*l_1*l_4*m_5*np.sin(theta_4 - theta_1) + dtheta_4*l_1*m_4*r_4*np.sin(theta_4 - theta_1)) + dtheta_3*(dtheta_3*l_1*l_3*m_4*np.sin(theta_3 - theta_1) + dtheta_3*l_1*l_3*m_5*np.sin(theta_3 - theta_1) + dtheta_3*l_1*m_3*r_3*np.sin(theta_3 - theta_1)) + dtheta_2*mu_2 + dtheta_2*(dtheta_2*l_1*l_2*m_3*np.sin(theta_2 - theta_1) + dtheta_2*l_1*l_2*m_4*np.sin(theta_2 - theta_1) + dtheta_2*l_1*l_2*m_5*np.sin(theta_2 - theta_1) + dtheta_2*l_1*m_2*r_2*np.sin(theta_2 - theta_1)) - dtheta_1*(mu_1 + mu_2) - ddx_c*(-l_1*m_2 - l_1*m_3 - l_1*m_4 - l_1*m_5 - m_1*r_1)*np.cos(theta_1) - g*(-l_1*m_2 - l_1*m_3 - l_1*m_4 - l_1*m_5 - m_1*r_1)*np.sin(theta_1))*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2))/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2) - (-l_1**2*m_5*r_5*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_5 - theta_1)*np.cos(theta_3 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_3*m_5*r_5*np.cos(theta_5 - theta_3) - (-l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_3 - theta_2))*(-l_1**2*m_5*r_5*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_5 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*m_5*r_5*np.cos(theta_5 - theta_2))/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2))*(dtheta_5**2*l_3*m_5*r_5*np.sin(theta_5 - theta_3) + dtheta_4*mu_4 + dtheta_4*(dtheta_4*l_3*l_4*m_5*np.sin(theta_4 - theta_3) + dtheta_4*l_3*m_4*r_4*np.sin(theta_4 - theta_3)) - dtheta_3*(mu_3 + mu_4) + dtheta_2*mu_3 + dtheta_2*(-dtheta_2*l_2*l_3*m_4*np.sin(theta_3 - theta_2) - dtheta_2*l_2*l_3*m_5*np.sin(theta_3 - theta_2) - dtheta_2*l_2*m_3*r_3*np.sin(theta_3 - theta_2)) + dtheta_1*(-dtheta_1*l_1*l_3*m_4*np.sin(theta_3 - theta_1) - dtheta_1*l_1*l_3*m_5*np.sin(theta_3 - theta_1) - dtheta_1*l_1*m_3*r_3*np.sin(theta_3 - theta_1)) - ddx_c*(-l_3*m_4 - l_3*m_5 - m_3*r_3)*np.cos(theta_3) - g*(-l_3*m_4 - l_3*m_5 - m_3*r_3)*np.sin(theta_3) - l_1*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(dtheta_5**2*l_1*m_5*r_5*np.sin(theta_5 - theta_1) + dtheta_4*(dtheta_4*l_1*l_4*m_5*np.sin(theta_4 - theta_1) + dtheta_4*l_1*m_4*r_4*np.sin(theta_4 - theta_1)) + dtheta_3*(dtheta_3*l_1*l_3*m_4*np.sin(theta_3 - theta_1) + dtheta_3*l_1*l_3*m_5*np.sin(theta_3 - theta_1) + dtheta_3*l_1*m_3*r_3*np.sin(theta_3 - theta_1)) + dtheta_2*mu_2 + dtheta_2*(dtheta_2*l_1*l_2*m_3*np.sin(theta_2 - theta_1) + dtheta_2*l_1*l_2*m_4*np.sin(theta_2 - theta_1) + dtheta_2*l_1*l_2*m_5*np.sin(theta_2 - theta_1) + dtheta_2*l_1*m_2*r_2*np.sin(theta_2 - theta_1)) - dtheta_1*(mu_1 + mu_2) - ddx_c*(-l_1*m_2 - l_1*m_3 - l_1*m_4 - l_1*m_5 - m_1*r_1)*np.cos(theta_1) - g*(-l_1*m_2 - l_1*m_3 - l_1*m_4 - l_1*m_5 - m_1*r_1)*np.sin(theta_1))*np.cos(theta_3 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) - (-l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_3 - theta_2))*(dtheta_5**2*l_2*m_5*r_5*np.sin(theta_5 - theta_2) + dtheta_4*(dtheta_4*l_2*l_4*m_5*np.sin(theta_4 - theta_2) + dtheta_4*l_2*m_4*r_4*np.sin(theta_4 - theta_2)) + dtheta_3*mu_3 + dtheta_3*(dtheta_3*l_2*l_3*m_4*np.sin(theta_3 - theta_2) + dtheta_3*l_2*l_3*m_5*np.sin(theta_3 - theta_2) + dtheta_3*l_2*m_3*r_3*np.sin(theta_3 - theta_2)) - dtheta_2*(mu_2 + mu_3) + dtheta_1*mu_2 + dtheta_1*(-dtheta_1*l_1*l_2*m_3*np.sin(theta_2 - theta_1) - dtheta_1*l_1*l_2*m_4*np.sin(theta_2 - theta_1) - dtheta_1*l_1*l_2*m_5*np.sin(theta_2 - theta_1) - dtheta_1*l_1*m_2*r_2*np.sin(theta_2 - theta_1)) - ddx_c*(-l_2*m_3 - l_2*m_4 - l_2*m_5 - m_2*r_2)*np.cos(theta_2) - g*(-l_2*m_3 - l_2*m_4 - l_2*m_5 - m_2*r_2)*np.sin(theta_2) - l_1*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*(dtheta_5**2*l_1*m_5*r_5*np.sin(theta_5 - theta_1) + dtheta_4*(dtheta_4*l_1*l_4*m_5*np.sin(theta_4 - theta_1) + dtheta_4*l_1*m_4*r_4*np.sin(theta_4 - theta_1)) + dtheta_3*(dtheta_3*l_1*l_3*m_4*np.sin(theta_3 - theta_1) + dtheta_3*l_1*l_3*m_5*np.sin(theta_3 - theta_1) + dtheta_3*l_1*m_3*r_3*np.sin(theta_3 - theta_1)) + dtheta_2*mu_2 + dtheta_2*(dtheta_2*l_1*l_2*m_3*np.sin(theta_2 - theta_1) + dtheta_2*l_1*l_2*m_4*np.sin(theta_2 - theta_1) + dtheta_2*l_1*l_2*m_5*np.sin(theta_2 - theta_1) + dtheta_2*l_1*m_2*r_2*np.sin(theta_2 - theta_1)) - dtheta_1*(mu_1 + mu_2) - ddx_c*(-l_1*m_2 - l_1*m_3 - l_1*m_4 - l_1*m_5 - m_1*r_1)*np.cos(theta_1) - g*(-l_1*m_2 - l_1*m_3 - l_1*m_4 - l_1*m_5 - m_1*r_1)*np.sin(theta_1))*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2))/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2))/(I_3 - l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)**2*np.cos(theta_3 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_3**2*m_4 + l_3**2*m_5 + m_3*r_3**2 - (-l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_3 - theta_2))**2/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2)) - (-l_1**2*m_5*r_5*(l_4*m_5 + m_4*r_4)*np.cos(theta_5 - theta_1)*np.cos(theta_4 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_4*m_5*r_5*np.cos(theta_5 - theta_4) - (-l_1**2*(l_4*m_5 + m_4*r_4)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_4 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_4*m_5 + m_4*r_4)*np.cos(theta_4 - theta_2))*(-l_1**2*m_5*r_5*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_5 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*m_5*r_5*np.cos(theta_5 - theta_2))/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2) - (-l_1**2*(l_4*m_5 + m_4*r_4)*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_4 - theta_1)*np.cos(theta_3 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_3*(l_4*m_5 + m_4*r_4)*np.cos(theta_4 - theta_3) - (-l_1**2*(l_4*m_5 + m_4*r_4)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_4 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_4*m_5 + m_4*r_4)*np.cos(theta_4 - theta_2))*(-l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_3 - theta_2))/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2))*(-l_1**2*m_5*r_5*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_5 - theta_1)*np.cos(theta_3 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_3*m_5*r_5*np.cos(theta_5 - theta_3) - (-l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_3 - theta_2))*(-l_1**2*m_5*r_5*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_5 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*m_5*r_5*np.cos(theta_5 - theta_2))/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2))/(I_3 - l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)**2*np.cos(theta_3 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_3**2*m_4 + l_3**2*m_5 + m_3*r_3**2 - (-l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_3 - theta_2))**2/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2)))*(dtheta_5**2*l_4*m_5*r_5*np.sin(theta_5 - theta_4) + dtheta_5*mu_5 - dtheta_4*(mu_4 + mu_5) + dtheta_3*mu_4 + dtheta_3*(-dtheta_3*l_3*l_4*m_5*np.sin(theta_4 - theta_3) - dtheta_3*l_3*m_4*r_4*np.sin(theta_4 - theta_3)) + dtheta_2*(-dtheta_2*l_2*l_4*m_5*np.sin(theta_4 - theta_2) - dtheta_2*l_2*m_4*r_4*np.sin(theta_4 - theta_2)) + dtheta_1*(-dtheta_1*l_1*l_4*m_5*np.sin(theta_4 - theta_1) - dtheta_1*l_1*m_4*r_4*np.sin(theta_4 - theta_1)) - ddx_c*(-l_4*m_5 - m_4*r_4)*np.cos(theta_4) - g*(-l_4*m_5 - m_4*r_4)*np.sin(theta_4) - l_1*(l_4*m_5 + m_4*r_4)*(dtheta_5**2*l_1*m_5*r_5*np.sin(theta_5 - theta_1) + dtheta_4*(dtheta_4*l_1*l_4*m_5*np.sin(theta_4 - theta_1) + dtheta_4*l_1*m_4*r_4*np.sin(theta_4 - theta_1)) + dtheta_3*(dtheta_3*l_1*l_3*m_4*np.sin(theta_3 - theta_1) + dtheta_3*l_1*l_3*m_5*np.sin(theta_3 - theta_1) + dtheta_3*l_1*m_3*r_3*np.sin(theta_3 - theta_1)) + dtheta_2*mu_2 + dtheta_2*(dtheta_2*l_1*l_2*m_3*np.sin(theta_2 - theta_1) + dtheta_2*l_1*l_2*m_4*np.sin(theta_2 - theta_1) + dtheta_2*l_1*l_2*m_5*np.sin(theta_2 - theta_1) + dtheta_2*l_1*m_2*r_2*np.sin(theta_2 - theta_1)) - dtheta_1*(mu_1 + mu_2) - ddx_c*(-l_1*m_2 - l_1*m_3 - l_1*m_4 - l_1*m_5 - m_1*r_1)*np.cos(theta_1) - g*(-l_1*m_2 - l_1*m_3 - l_1*m_4 - l_1*m_5 - m_1*r_1)*np.sin(theta_1))*np.cos(theta_4 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) - (-l_1**2*(l_4*m_5 + m_4*r_4)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_4 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_4*m_5 + m_4*r_4)*np.cos(theta_4 - theta_2))*(dtheta_5**2*l_2*m_5*r_5*np.sin(theta_5 - theta_2) + dtheta_4*(dtheta_4*l_2*l_4*m_5*np.sin(theta_4 - theta_2) + dtheta_4*l_2*m_4*r_4*np.sin(theta_4 - theta_2)) + dtheta_3*mu_3 + dtheta_3*(dtheta_3*l_2*l_3*m_4*np.sin(theta_3 - theta_2) + dtheta_3*l_2*l_3*m_5*np.sin(theta_3 - theta_2) + dtheta_3*l_2*m_3*r_3*np.sin(theta_3 - theta_2)) - dtheta_2*(mu_2 + mu_3) + dtheta_1*mu_2 + dtheta_1*(-dtheta_1*l_1*l_2*m_3*np.sin(theta_2 - theta_1) - dtheta_1*l_1*l_2*m_4*np.sin(theta_2 - theta_1) - dtheta_1*l_1*l_2*m_5*np.sin(theta_2 - theta_1) - dtheta_1*l_1*m_2*r_2*np.sin(theta_2 - theta_1)) - ddx_c*(-l_2*m_3 - l_2*m_4 - l_2*m_5 - m_2*r_2)*np.cos(theta_2) - g*(-l_2*m_3 - l_2*m_4 - l_2*m_5 - m_2*r_2)*np.sin(theta_2) - l_1*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*(dtheta_5**2*l_1*m_5*r_5*np.sin(theta_5 - theta_1) + dtheta_4*(dtheta_4*l_1*l_4*m_5*np.sin(theta_4 - theta_1) + dtheta_4*l_1*m_4*r_4*np.sin(theta_4 - theta_1)) + dtheta_3*(dtheta_3*l_1*l_3*m_4*np.sin(theta_3 - theta_1) + dtheta_3*l_1*l_3*m_5*np.sin(theta_3 - theta_1) + dtheta_3*l_1*m_3*r_3*np.sin(theta_3 - theta_1)) + dtheta_2*mu_2 + dtheta_2*(dtheta_2*l_1*l_2*m_3*np.sin(theta_2 - theta_1) + dtheta_2*l_1*l_2*m_4*np.sin(theta_2 - theta_1) + dtheta_2*l_1*l_2*m_5*np.sin(theta_2 - theta_1) + dtheta_2*l_1*m_2*r_2*np.sin(theta_2 - theta_1)) - dtheta_1*(mu_1 + mu_2) - ddx_c*(-l_1*m_2 - l_1*m_3 - l_1*m_4 - l_1*m_5 - m_1*r_1)*np.cos(theta_1) - g*(-l_1*m_2 - l_1*m_3 - l_1*m_4 - l_1*m_5 - m_1*r_1)*np.sin(theta_1))*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2))/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2) - (-l_1**2*(l_4*m_5 + m_4*r_4)*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_4 - theta_1)*np.cos(theta_3 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_3*(l_4*m_5 + m_4*r_4)*np.cos(theta_4 - theta_3) - (-l_1**2*(l_4*m_5 + m_4*r_4)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_4 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_4*m_5 + m_4*r_4)*np.cos(theta_4 - theta_2))*(-l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_3 - theta_2))/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2))*(dtheta_5**2*l_3*m_5*r_5*np.sin(theta_5 - theta_3) + dtheta_4*mu_4 + dtheta_4*(dtheta_4*l_3*l_4*m_5*np.sin(theta_4 - theta_3) + dtheta_4*l_3*m_4*r_4*np.sin(theta_4 - theta_3)) - dtheta_3*(mu_3 + mu_4) + dtheta_2*mu_3 + dtheta_2*(-dtheta_2*l_2*l_3*m_4*np.sin(theta_3 - theta_2) - dtheta_2*l_2*l_3*m_5*np.sin(theta_3 - theta_2) - dtheta_2*l_2*m_3*r_3*np.sin(theta_3 - theta_2)) + dtheta_1*(-dtheta_1*l_1*l_3*m_4*np.sin(theta_3 - theta_1) - dtheta_1*l_1*l_3*m_5*np.sin(theta_3 - theta_1) - dtheta_1*l_1*m_3*r_3*np.sin(theta_3 - theta_1)) - ddx_c*(-l_3*m_4 - l_3*m_5 - m_3*r_3)*np.cos(theta_3) - g*(-l_3*m_4 - l_3*m_5 - m_3*r_3)*np.sin(theta_3) - l_1*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(dtheta_5**2*l_1*m_5*r_5*np.sin(theta_5 - theta_1) + dtheta_4*(dtheta_4*l_1*l_4*m_5*np.sin(theta_4 - theta_1) + dtheta_4*l_1*m_4*r_4*np.sin(theta_4 - theta_1)) + dtheta_3*(dtheta_3*l_1*l_3*m_4*np.sin(theta_3 - theta_1) + dtheta_3*l_1*l_3*m_5*np.sin(theta_3 - theta_1) + dtheta_3*l_1*m_3*r_3*np.sin(theta_3 - theta_1)) + dtheta_2*mu_2 + dtheta_2*(dtheta_2*l_1*l_2*m_3*np.sin(theta_2 - theta_1) + dtheta_2*l_1*l_2*m_4*np.sin(theta_2 - theta_1) + dtheta_2*l_1*l_2*m_5*np.sin(theta_2 - theta_1) + dtheta_2*l_1*m_2*r_2*np.sin(theta_2 - theta_1)) - dtheta_1*(mu_1 + mu_2) - ddx_c*(-l_1*m_2 - l_1*m_3 - l_1*m_4 - l_1*m_5 - m_1*r_1)*np.cos(theta_1) - g*(-l_1*m_2 - l_1*m_3 - l_1*m_4 - l_1*m_5 - m_1*r_1)*np.sin(theta_1))*np.cos(theta_3 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) - (-l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_3 - theta_2))*(dtheta_5**2*l_2*m_5*r_5*np.sin(theta_5 - theta_2) + dtheta_4*(dtheta_4*l_2*l_4*m_5*np.sin(theta_4 - theta_2) + dtheta_4*l_2*m_4*r_4*np.sin(theta_4 - theta_2)) + dtheta_3*mu_3 + dtheta_3*(dtheta_3*l_2*l_3*m_4*np.sin(theta_3 - theta_2) + dtheta_3*l_2*l_3*m_5*np.sin(theta_3 - theta_2) + dtheta_3*l_2*m_3*r_3*np.sin(theta_3 - theta_2)) - dtheta_2*(mu_2 + mu_3) + dtheta_1*mu_2 + dtheta_1*(-dtheta_1*l_1*l_2*m_3*np.sin(theta_2 - theta_1) - dtheta_1*l_1*l_2*m_4*np.sin(theta_2 - theta_1) - dtheta_1*l_1*l_2*m_5*np.sin(theta_2 - theta_1) - dtheta_1*l_1*m_2*r_2*np.sin(theta_2 - theta_1)) - ddx_c*(-l_2*m_3 - l_2*m_4 - l_2*m_5 - m_2*r_2)*np.cos(theta_2) - g*(-l_2*m_3 - l_2*m_4 - l_2*m_5 - m_2*r_2)*np.sin(theta_2) - l_1*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*(dtheta_5**2*l_1*m_5*r_5*np.sin(theta_5 - theta_1) + dtheta_4*(dtheta_4*l_1*l_4*m_5*np.sin(theta_4 - theta_1) + dtheta_4*l_1*m_4*r_4*np.sin(theta_4 - theta_1)) + dtheta_3*(dtheta_3*l_1*l_3*m_4*np.sin(theta_3 - theta_1) + dtheta_3*l_1*l_3*m_5*np.sin(theta_3 - theta_1) + dtheta_3*l_1*m_3*r_3*np.sin(theta_3 - theta_1)) + dtheta_2*mu_2 + dtheta_2*(dtheta_2*l_1*l_2*m_3*np.sin(theta_2 - theta_1) + dtheta_2*l_1*l_2*m_4*np.sin(theta_2 - theta_1) + dtheta_2*l_1*l_2*m_5*np.sin(theta_2 - theta_1) + dtheta_2*l_1*m_2*r_2*np.sin(theta_2 - theta_1)) - dtheta_1*(mu_1 + mu_2) - ddx_c*(-l_1*m_2 - l_1*m_3 - l_1*m_4 - l_1*m_5 - m_1*r_1)*np.cos(theta_1) - g*(-l_1*m_2 - l_1*m_3 - l_1*m_4 - l_1*m_5 - m_1*r_1)*np.sin(theta_1))*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2))/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2))/(I_3 - l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)**2*np.cos(theta_3 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_3**2*m_4 + l_3**2*m_5 + m_3*r_3**2 - (-l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_3 - theta_2))**2/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2)))/(I_4 - l_1**2*(l_4*m_5 + m_4*r_4)**2*np.cos(theta_4 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_4**2*m_5 + m_4*r_4**2 - (-l_1**2*(l_4*m_5 + m_4*r_4)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_4 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_4*m_5 + m_4*r_4)*np.cos(theta_4 - theta_2))**2/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2) - (-l_1**2*(l_4*m_5 + m_4*r_4)*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_4 - theta_1)*np.cos(theta_3 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_3*(l_4*m_5 + m_4*r_4)*np.cos(theta_4 - theta_3) - (-l_1**2*(l_4*m_5 + m_4*r_4)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_4 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_4*m_5 + m_4*r_4)*np.cos(theta_4 - theta_2))*(-l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_3 - theta_2))/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2))**2/(I_3 - l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)**2*np.cos(theta_3 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_3**2*m_4 + l_3**2*m_5 + m_3*r_3**2 - (-l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_3 - theta_2))**2/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2))))/(I_5 - l_1**2*m_5**2*r_5**2*np.cos(theta_5 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + m_5*r_5**2 - (-l_1**2*m_5*r_5*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_5 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*m_5*r_5*np.cos(theta_5 - theta_2))**2/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2) - (-l_1**2*m_5*r_5*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_5 - theta_1)*np.cos(theta_3 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_3*m_5*r_5*np.cos(theta_5 - theta_3) - (-l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_3 - theta_2))*(-l_1**2*m_5*r_5*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_5 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*m_5*r_5*np.cos(theta_5 - theta_2))/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2))**2/(I_3 - l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)**2*np.cos(theta_3 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_3**2*m_4 + l_3**2*m_5 + m_3*r_3**2 - (-l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_3 - theta_2))**2/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2)) - (-l_1**2*m_5*r_5*(l_4*m_5 + m_4*r_4)*np.cos(theta_5 - theta_1)*np.cos(theta_4 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_4*m_5*r_5*np.cos(theta_5 - theta_4) - (-l_1**2*(l_4*m_5 + m_4*r_4)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_4 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_4*m_5 + m_4*r_4)*np.cos(theta_4 - theta_2))*(-l_1**2*m_5*r_5*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_5 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*m_5*r_5*np.cos(theta_5 - theta_2))/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2) - (-l_1**2*(l_4*m_5 + m_4*r_4)*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_4 - theta_1)*np.cos(theta_3 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_3*(l_4*m_5 + m_4*r_4)*np.cos(theta_4 - theta_3) - (-l_1**2*(l_4*m_5 + m_4*r_4)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_4 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_4*m_5 + m_4*r_4)*np.cos(theta_4 - theta_2))*(-l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_3 - theta_2))/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2))*(-l_1**2*m_5*r_5*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_5 - theta_1)*np.cos(theta_3 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_3*m_5*r_5*np.cos(theta_5 - theta_3) - (-l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_3 - theta_2))*(-l_1**2*m_5*r_5*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_5 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*m_5*r_5*np.cos(theta_5 - theta_2))/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2))/(I_3 - l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)**2*np.cos(theta_3 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_3**2*m_4 + l_3**2*m_5 + m_3*r_3**2 - (-l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_3 - theta_2))**2/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2)))**2/(I_4 - l_1**2*(l_4*m_5 + m_4*r_4)**2*np.cos(theta_4 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_4**2*m_5 + m_4*r_4**2 - (-l_1**2*(l_4*m_5 + m_4*r_4)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_4 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_4*m_5 + m_4*r_4)*np.cos(theta_4 - theta_2))**2/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2) - (-l_1**2*(l_4*m_5 + m_4*r_4)*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_4 - theta_1)*np.cos(theta_3 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_3*(l_4*m_5 + m_4*r_4)*np.cos(theta_4 - theta_3) - (-l_1**2*(l_4*m_5 + m_4*r_4)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_4 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_4*m_5 + m_4*r_4)*np.cos(theta_4 - theta_2))*(-l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_3 - theta_2))/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2))**2/(I_3 - l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)**2*np.cos(theta_3 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_3**2*m_4 + l_3**2*m_5 + m_3*r_3**2 - (-l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_3 - theta_2))**2/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2)))),
                      ddx_c
    ])
    
    
@numba.jit()
def robotic_equation_n5_numpy_jit(t, y, args):
    theta_1, theta_2, theta_3, theta_4, theta_5, dtheta_1, dtheta_2, dtheta_3, dtheta_4, dtheta_5, ddx_c                           = y
    g, l_1, l_2, l_3, l_4, r_1, r_2, r_3, r_4, r_5, m_1, m_2, m_3, m_4, m_5, mu_1, mu_2, mu_3, mu_4, mu_5, I_1, I_2, I_3, I_4, I_5 = args

    return np.array([dtheta_1,
                      dtheta_2,
                      dtheta_3,
                      dtheta_4,
                      dtheta_5,

                      (dtheta_5**2*l_1*m_5*r_5*np.sin(theta_5 - theta_1) + dtheta_4*(dtheta_4*l_1*l_4*m_5*np.sin(theta_4 - theta_1) + dtheta_4*l_1*m_4*r_4*np.sin(theta_4 - theta_1)) + dtheta_3*(dtheta_3*l_1*l_3*m_4*np.sin(theta_3 - theta_1) + dtheta_3*l_1*l_3*m_5*np.sin(theta_3 - theta_1) + dtheta_3*l_1*m_3*r_3*np.sin(theta_3 - theta_1)) + dtheta_2*mu_2 + dtheta_2*(dtheta_2*l_1*l_2*m_3*np.sin(theta_2 - theta_1) + dtheta_2*l_1*l_2*m_4*np.sin(theta_2 - theta_1) + dtheta_2*l_1*l_2*m_5*np.sin(theta_2 - theta_1) + dtheta_2*l_1*m_2*r_2*np.sin(theta_2 - theta_1)) - dtheta_1*(mu_1 + mu_2) - ddx_c*(-l_1*m_2 - l_1*m_3 - l_1*m_4 - l_1*m_5 - m_1*r_1)*np.cos(theta_1) - g*(-l_1*m_2 - l_1*m_3 - l_1*m_4 - l_1*m_5 - m_1*r_1)*np.sin(theta_1) - l_1*m_5*r_5*(-dtheta_5*mu_5 - dtheta_4**2*l_4*m_5*r_5*np.sin(theta_5 - theta_4) + dtheta_4*mu_5 - dtheta_3**2*l_3*m_5*r_5*np.sin(theta_5 - theta_3) - dtheta_2**2*l_2*m_5*r_5*np.sin(theta_5 - theta_2) - dtheta_1**2*l_1*m_5*r_5*np.sin(theta_5 - theta_1) + ddx_c*m_5*r_5*np.cos(theta_5) + g*m_5*r_5*np.sin(theta_5) - l_1*m_5*r_5*(dtheta_5**2*l_1*m_5*r_5*np.sin(theta_5 - theta_1) + dtheta_4*(dtheta_4*l_1*l_4*m_5*np.sin(theta_4 - theta_1) + dtheta_4*l_1*m_4*r_4*np.sin(theta_4 - theta_1)) + dtheta_3*(dtheta_3*l_1*l_3*m_4*np.sin(theta_3 - theta_1) + dtheta_3*l_1*l_3*m_5*np.sin(theta_3 - theta_1) + dtheta_3*l_1*m_3*r_3*np.sin(theta_3 - theta_1)) + dtheta_2*mu_2 + dtheta_2*(dtheta_2*l_1*l_2*m_3*np.sin(theta_2 - theta_1) + dtheta_2*l_1*l_2*m_4*np.sin(theta_2 - theta_1) + dtheta_2*l_1*l_2*m_5*np.sin(theta_2 - theta_1) + dtheta_2*l_1*m_2*r_2*np.sin(theta_2 - theta_1)) - dtheta_1*(mu_1 + mu_2) - ddx_c*(-l_1*m_2 - l_1*m_3 - l_1*m_4 - l_1*m_5 - m_1*r_1)*np.cos(theta_1) - g*(-l_1*m_2 - l_1*m_3 - l_1*m_4 - l_1*m_5 - m_1*r_1)*np.sin(theta_1))*np.cos(theta_5 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) - (-l_1**2*m_5*r_5*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_5 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*m_5*r_5*np.cos(theta_5 - theta_2))*(dtheta_5**2*l_2*m_5*r_5*np.sin(theta_5 - theta_2) + dtheta_4*(dtheta_4*l_2*l_4*m_5*np.sin(theta_4 - theta_2) + dtheta_4*l_2*m_4*r_4*np.sin(theta_4 - theta_2)) + dtheta_3*mu_3 + dtheta_3*(dtheta_3*l_2*l_3*m_4*np.sin(theta_3 - theta_2) + dtheta_3*l_2*l_3*m_5*np.sin(theta_3 - theta_2) + dtheta_3*l_2*m_3*r_3*np.sin(theta_3 - theta_2)) - dtheta_2*(mu_2 + mu_3) + dtheta_1*mu_2 + dtheta_1*(-dtheta_1*l_1*l_2*m_3*np.sin(theta_2 - theta_1) - dtheta_1*l_1*l_2*m_4*np.sin(theta_2 - theta_1) - dtheta_1*l_1*l_2*m_5*np.sin(theta_2 - theta_1) - dtheta_1*l_1*m_2*r_2*np.sin(theta_2 - theta_1)) - ddx_c*(-l_2*m_3 - l_2*m_4 - l_2*m_5 - m_2*r_2)*np.cos(theta_2) - g*(-l_2*m_3 - l_2*m_4 - l_2*m_5 - m_2*r_2)*np.sin(theta_2) - l_1*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*(dtheta_5**2*l_1*m_5*r_5*np.sin(theta_5 - theta_1) + dtheta_4*(dtheta_4*l_1*l_4*m_5*np.sin(theta_4 - theta_1) + dtheta_4*l_1*m_4*r_4*np.sin(theta_4 - theta_1)) + dtheta_3*(dtheta_3*l_1*l_3*m_4*np.sin(theta_3 - theta_1) + dtheta_3*l_1*l_3*m_5*np.sin(theta_3 - theta_1) + dtheta_3*l_1*m_3*r_3*np.sin(theta_3 - theta_1)) + dtheta_2*mu_2 + dtheta_2*(dtheta_2*l_1*l_2*m_3*np.sin(theta_2 - theta_1) + dtheta_2*l_1*l_2*m_4*np.sin(theta_2 - theta_1) + dtheta_2*l_1*l_2*m_5*np.sin(theta_2 - theta_1) + dtheta_2*l_1*m_2*r_2*np.sin(theta_2 - theta_1)) - dtheta_1*(mu_1 + mu_2) - ddx_c*(-l_1*m_2 - l_1*m_3 - l_1*m_4 - l_1*m_5 - m_1*r_1)*np.cos(theta_1) - g*(-l_1*m_2 - l_1*m_3 - l_1*m_4 - l_1*m_5 - m_1*r_1)*np.sin(theta_1))*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2))/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2) - (-l_1**2*m_5*r_5*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_5 - theta_1)*np.cos(theta_3 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_3*m_5*r_5*np.cos(theta_5 - theta_3) - (-l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_3 - theta_2))*(-l_1**2*m_5*r_5*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_5 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*m_5*r_5*np.cos(theta_5 - theta_2))/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2))*(dtheta_5**2*l_3*m_5*r_5*np.sin(theta_5 - theta_3) + dtheta_4*mu_4 + dtheta_4*(dtheta_4*l_3*l_4*m_5*np.sin(theta_4 - theta_3) + dtheta_4*l_3*m_4*r_4*np.sin(theta_4 - theta_3)) - dtheta_3*(mu_3 + mu_4) + dtheta_2*mu_3 + dtheta_2*(-dtheta_2*l_2*l_3*m_4*np.sin(theta_3 - theta_2) - dtheta_2*l_2*l_3*m_5*np.sin(theta_3 - theta_2) - dtheta_2*l_2*m_3*r_3*np.sin(theta_3 - theta_2)) + dtheta_1*(-dtheta_1*l_1*l_3*m_4*np.sin(theta_3 - theta_1) - dtheta_1*l_1*l_3*m_5*np.sin(theta_3 - theta_1) - dtheta_1*l_1*m_3*r_3*np.sin(theta_3 - theta_1)) - ddx_c*(-l_3*m_4 - l_3*m_5 - m_3*r_3)*np.cos(theta_3) - g*(-l_3*m_4 - l_3*m_5 - m_3*r_3)*np.sin(theta_3) - l_1*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(dtheta_5**2*l_1*m_5*r_5*np.sin(theta_5 - theta_1) + dtheta_4*(dtheta_4*l_1*l_4*m_5*np.sin(theta_4 - theta_1) + dtheta_4*l_1*m_4*r_4*np.sin(theta_4 - theta_1)) + dtheta_3*(dtheta_3*l_1*l_3*m_4*np.sin(theta_3 - theta_1) + dtheta_3*l_1*l_3*m_5*np.sin(theta_3 - theta_1) + dtheta_3*l_1*m_3*r_3*np.sin(theta_3 - theta_1)) + dtheta_2*mu_2 + dtheta_2*(dtheta_2*l_1*l_2*m_3*np.sin(theta_2 - theta_1) + dtheta_2*l_1*l_2*m_4*np.sin(theta_2 - theta_1) + dtheta_2*l_1*l_2*m_5*np.sin(theta_2 - theta_1) + dtheta_2*l_1*m_2*r_2*np.sin(theta_2 - theta_1)) - dtheta_1*(mu_1 + mu_2) - ddx_c*(-l_1*m_2 - l_1*m_3 - l_1*m_4 - l_1*m_5 - m_1*r_1)*np.cos(theta_1) - g*(-l_1*m_2 - l_1*m_3 - l_1*m_4 - l_1*m_5 - m_1*r_1)*np.sin(theta_1))*np.cos(theta_3 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) - (-l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_3 - theta_2))*(dtheta_5**2*l_2*m_5*r_5*np.sin(theta_5 - theta_2) + dtheta_4*(dtheta_4*l_2*l_4*m_5*np.sin(theta_4 - theta_2) + dtheta_4*l_2*m_4*r_4*np.sin(theta_4 - theta_2)) + dtheta_3*mu_3 + dtheta_3*(dtheta_3*l_2*l_3*m_4*np.sin(theta_3 - theta_2) + dtheta_3*l_2*l_3*m_5*np.sin(theta_3 - theta_2) + dtheta_3*l_2*m_3*r_3*np.sin(theta_3 - theta_2)) - dtheta_2*(mu_2 + mu_3) + dtheta_1*mu_2 + dtheta_1*(-dtheta_1*l_1*l_2*m_3*np.sin(theta_2 - theta_1) - dtheta_1*l_1*l_2*m_4*np.sin(theta_2 - theta_1) - dtheta_1*l_1*l_2*m_5*np.sin(theta_2 - theta_1) - dtheta_1*l_1*m_2*r_2*np.sin(theta_2 - theta_1)) - ddx_c*(-l_2*m_3 - l_2*m_4 - l_2*m_5 - m_2*r_2)*np.cos(theta_2) - g*(-l_2*m_3 - l_2*m_4 - l_2*m_5 - m_2*r_2)*np.sin(theta_2) - l_1*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*(dtheta_5**2*l_1*m_5*r_5*np.sin(theta_5 - theta_1) + dtheta_4*(dtheta_4*l_1*l_4*m_5*np.sin(theta_4 - theta_1) + dtheta_4*l_1*m_4*r_4*np.sin(theta_4 - theta_1)) + dtheta_3*(dtheta_3*l_1*l_3*m_4*np.sin(theta_3 - theta_1) + dtheta_3*l_1*l_3*m_5*np.sin(theta_3 - theta_1) + dtheta_3*l_1*m_3*r_3*np.sin(theta_3 - theta_1)) + dtheta_2*mu_2 + dtheta_2*(dtheta_2*l_1*l_2*m_3*np.sin(theta_2 - theta_1) + dtheta_2*l_1*l_2*m_4*np.sin(theta_2 - theta_1) + dtheta_2*l_1*l_2*m_5*np.sin(theta_2 - theta_1) + dtheta_2*l_1*m_2*r_2*np.sin(theta_2 - theta_1)) - dtheta_1*(mu_1 + mu_2) - ddx_c*(-l_1*m_2 - l_1*m_3 - l_1*m_4 - l_1*m_5 - m_1*r_1)*np.cos(theta_1) - g*(-l_1*m_2 - l_1*m_3 - l_1*m_4 - l_1*m_5 - m_1*r_1)*np.sin(theta_1))*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2))/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2))/(I_3 - l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)**2*np.cos(theta_3 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_3**2*m_4 + l_3**2*m_5 + m_3*r_3**2 - (-l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_3 - theta_2))**2/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2)) - (-l_1**2*m_5*r_5*(l_4*m_5 + m_4*r_4)*np.cos(theta_5 - theta_1)*np.cos(theta_4 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_4*m_5*r_5*np.cos(theta_5 - theta_4) - (-l_1**2*(l_4*m_5 + m_4*r_4)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_4 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_4*m_5 + m_4*r_4)*np.cos(theta_4 - theta_2))*(-l_1**2*m_5*r_5*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_5 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*m_5*r_5*np.cos(theta_5 - theta_2))/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2) - (-l_1**2*(l_4*m_5 + m_4*r_4)*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_4 - theta_1)*np.cos(theta_3 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_3*(l_4*m_5 + m_4*r_4)*np.cos(theta_4 - theta_3) - (-l_1**2*(l_4*m_5 + m_4*r_4)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_4 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_4*m_5 + m_4*r_4)*np.cos(theta_4 - theta_2))*(-l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_3 - theta_2))/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2))*(-l_1**2*m_5*r_5*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_5 - theta_1)*np.cos(theta_3 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_3*m_5*r_5*np.cos(theta_5 - theta_3) - (-l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_3 - theta_2))*(-l_1**2*m_5*r_5*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_5 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*m_5*r_5*np.cos(theta_5 - theta_2))/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2))/(I_3 - l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)**2*np.cos(theta_3 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_3**2*m_4 + l_3**2*m_5 + m_3*r_3**2 - (-l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_3 - theta_2))**2/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2)))*(dtheta_5**2*l_4*m_5*r_5*np.sin(theta_5 - theta_4) + dtheta_5*mu_5 - dtheta_4*(mu_4 + mu_5) + dtheta_3*mu_4 + dtheta_3*(-dtheta_3*l_3*l_4*m_5*np.sin(theta_4 - theta_3) - dtheta_3*l_3*m_4*r_4*np.sin(theta_4 - theta_3)) + dtheta_2*(-dtheta_2*l_2*l_4*m_5*np.sin(theta_4 - theta_2) - dtheta_2*l_2*m_4*r_4*np.sin(theta_4 - theta_2)) + dtheta_1*(-dtheta_1*l_1*l_4*m_5*np.sin(theta_4 - theta_1) - dtheta_1*l_1*m_4*r_4*np.sin(theta_4 - theta_1)) - ddx_c*(-l_4*m_5 - m_4*r_4)*np.cos(theta_4) - g*(-l_4*m_5 - m_4*r_4)*np.sin(theta_4) - l_1*(l_4*m_5 + m_4*r_4)*(dtheta_5**2*l_1*m_5*r_5*np.sin(theta_5 - theta_1) + dtheta_4*(dtheta_4*l_1*l_4*m_5*np.sin(theta_4 - theta_1) + dtheta_4*l_1*m_4*r_4*np.sin(theta_4 - theta_1)) + dtheta_3*(dtheta_3*l_1*l_3*m_4*np.sin(theta_3 - theta_1) + dtheta_3*l_1*l_3*m_5*np.sin(theta_3 - theta_1) + dtheta_3*l_1*m_3*r_3*np.sin(theta_3 - theta_1)) + dtheta_2*mu_2 + dtheta_2*(dtheta_2*l_1*l_2*m_3*np.sin(theta_2 - theta_1) + dtheta_2*l_1*l_2*m_4*np.sin(theta_2 - theta_1) + dtheta_2*l_1*l_2*m_5*np.sin(theta_2 - theta_1) + dtheta_2*l_1*m_2*r_2*np.sin(theta_2 - theta_1)) - dtheta_1*(mu_1 + mu_2) - ddx_c*(-l_1*m_2 - l_1*m_3 - l_1*m_4 - l_1*m_5 - m_1*r_1)*np.cos(theta_1) - g*(-l_1*m_2 - l_1*m_3 - l_1*m_4 - l_1*m_5 - m_1*r_1)*np.sin(theta_1))*np.cos(theta_4 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) - (-l_1**2*(l_4*m_5 + m_4*r_4)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_4 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_4*m_5 + m_4*r_4)*np.cos(theta_4 - theta_2))*(dtheta_5**2*l_2*m_5*r_5*np.sin(theta_5 - theta_2) + dtheta_4*(dtheta_4*l_2*l_4*m_5*np.sin(theta_4 - theta_2) + dtheta_4*l_2*m_4*r_4*np.sin(theta_4 - theta_2)) + dtheta_3*mu_3 + dtheta_3*(dtheta_3*l_2*l_3*m_4*np.sin(theta_3 - theta_2) + dtheta_3*l_2*l_3*m_5*np.sin(theta_3 - theta_2) + dtheta_3*l_2*m_3*r_3*np.sin(theta_3 - theta_2)) - dtheta_2*(mu_2 + mu_3) + dtheta_1*mu_2 + dtheta_1*(-dtheta_1*l_1*l_2*m_3*np.sin(theta_2 - theta_1) - dtheta_1*l_1*l_2*m_4*np.sin(theta_2 - theta_1) - dtheta_1*l_1*l_2*m_5*np.sin(theta_2 - theta_1) - dtheta_1*l_1*m_2*r_2*np.sin(theta_2 - theta_1)) - ddx_c*(-l_2*m_3 - l_2*m_4 - l_2*m_5 - m_2*r_2)*np.cos(theta_2) - g*(-l_2*m_3 - l_2*m_4 - l_2*m_5 - m_2*r_2)*np.sin(theta_2) - l_1*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*(dtheta_5**2*l_1*m_5*r_5*np.sin(theta_5 - theta_1) + dtheta_4*(dtheta_4*l_1*l_4*m_5*np.sin(theta_4 - theta_1) + dtheta_4*l_1*m_4*r_4*np.sin(theta_4 - theta_1)) + dtheta_3*(dtheta_3*l_1*l_3*m_4*np.sin(theta_3 - theta_1) + dtheta_3*l_1*l_3*m_5*np.sin(theta_3 - theta_1) + dtheta_3*l_1*m_3*r_3*np.sin(theta_3 - theta_1)) + dtheta_2*mu_2 + dtheta_2*(dtheta_2*l_1*l_2*m_3*np.sin(theta_2 - theta_1) + dtheta_2*l_1*l_2*m_4*np.sin(theta_2 - theta_1) + dtheta_2*l_1*l_2*m_5*np.sin(theta_2 - theta_1) + dtheta_2*l_1*m_2*r_2*np.sin(theta_2 - theta_1)) - dtheta_1*(mu_1 + mu_2) - ddx_c*(-l_1*m_2 - l_1*m_3 - l_1*m_4 - l_1*m_5 - m_1*r_1)*np.cos(theta_1) - g*(-l_1*m_2 - l_1*m_3 - l_1*m_4 - l_1*m_5 - m_1*r_1)*np.sin(theta_1))*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2))/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2) - (-l_1**2*(l_4*m_5 + m_4*r_4)*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_4 - theta_1)*np.cos(theta_3 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_3*(l_4*m_5 + m_4*r_4)*np.cos(theta_4 - theta_3) - (-l_1**2*(l_4*m_5 + m_4*r_4)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_4 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_4*m_5 + m_4*r_4)*np.cos(theta_4 - theta_2))*(-l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_3 - theta_2))/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2))*(dtheta_5**2*l_3*m_5*r_5*np.sin(theta_5 - theta_3) + dtheta_4*mu_4 + dtheta_4*(dtheta_4*l_3*l_4*m_5*np.sin(theta_4 - theta_3) + dtheta_4*l_3*m_4*r_4*np.sin(theta_4 - theta_3)) - dtheta_3*(mu_3 + mu_4) + dtheta_2*mu_3 + dtheta_2*(-dtheta_2*l_2*l_3*m_4*np.sin(theta_3 - theta_2) - dtheta_2*l_2*l_3*m_5*np.sin(theta_3 - theta_2) - dtheta_2*l_2*m_3*r_3*np.sin(theta_3 - theta_2)) + dtheta_1*(-dtheta_1*l_1*l_3*m_4*np.sin(theta_3 - theta_1) - dtheta_1*l_1*l_3*m_5*np.sin(theta_3 - theta_1) - dtheta_1*l_1*m_3*r_3*np.sin(theta_3 - theta_1)) - ddx_c*(-l_3*m_4 - l_3*m_5 - m_3*r_3)*np.cos(theta_3) - g*(-l_3*m_4 - l_3*m_5 - m_3*r_3)*np.sin(theta_3) - l_1*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(dtheta_5**2*l_1*m_5*r_5*np.sin(theta_5 - theta_1) + dtheta_4*(dtheta_4*l_1*l_4*m_5*np.sin(theta_4 - theta_1) + dtheta_4*l_1*m_4*r_4*np.sin(theta_4 - theta_1)) + dtheta_3*(dtheta_3*l_1*l_3*m_4*np.sin(theta_3 - theta_1) + dtheta_3*l_1*l_3*m_5*np.sin(theta_3 - theta_1) + dtheta_3*l_1*m_3*r_3*np.sin(theta_3 - theta_1)) + dtheta_2*mu_2 + dtheta_2*(dtheta_2*l_1*l_2*m_3*np.sin(theta_2 - theta_1) + dtheta_2*l_1*l_2*m_4*np.sin(theta_2 - theta_1) + dtheta_2*l_1*l_2*m_5*np.sin(theta_2 - theta_1) + dtheta_2*l_1*m_2*r_2*np.sin(theta_2 - theta_1)) - dtheta_1*(mu_1 + mu_2) - ddx_c*(-l_1*m_2 - l_1*m_3 - l_1*m_4 - l_1*m_5 - m_1*r_1)*np.cos(theta_1) - g*(-l_1*m_2 - l_1*m_3 - l_1*m_4 - l_1*m_5 - m_1*r_1)*np.sin(theta_1))*np.cos(theta_3 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) - (-l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_3 - theta_2))*(dtheta_5**2*l_2*m_5*r_5*np.sin(theta_5 - theta_2) + dtheta_4*(dtheta_4*l_2*l_4*m_5*np.sin(theta_4 - theta_2) + dtheta_4*l_2*m_4*r_4*np.sin(theta_4 - theta_2)) + dtheta_3*mu_3 + dtheta_3*(dtheta_3*l_2*l_3*m_4*np.sin(theta_3 - theta_2) + dtheta_3*l_2*l_3*m_5*np.sin(theta_3 - theta_2) + dtheta_3*l_2*m_3*r_3*np.sin(theta_3 - theta_2)) - dtheta_2*(mu_2 + mu_3) + dtheta_1*mu_2 + dtheta_1*(-dtheta_1*l_1*l_2*m_3*np.sin(theta_2 - theta_1) - dtheta_1*l_1*l_2*m_4*np.sin(theta_2 - theta_1) - dtheta_1*l_1*l_2*m_5*np.sin(theta_2 - theta_1) - dtheta_1*l_1*m_2*r_2*np.sin(theta_2 - theta_1)) - ddx_c*(-l_2*m_3 - l_2*m_4 - l_2*m_5 - m_2*r_2)*np.cos(theta_2) - g*(-l_2*m_3 - l_2*m_4 - l_2*m_5 - m_2*r_2)*np.sin(theta_2) - l_1*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*(dtheta_5**2*l_1*m_5*r_5*np.sin(theta_5 - theta_1) + dtheta_4*(dtheta_4*l_1*l_4*m_5*np.sin(theta_4 - theta_1) + dtheta_4*l_1*m_4*r_4*np.sin(theta_4 - theta_1)) + dtheta_3*(dtheta_3*l_1*l_3*m_4*np.sin(theta_3 - theta_1) + dtheta_3*l_1*l_3*m_5*np.sin(theta_3 - theta_1) + dtheta_3*l_1*m_3*r_3*np.sin(theta_3 - theta_1)) + dtheta_2*mu_2 + dtheta_2*(dtheta_2*l_1*l_2*m_3*np.sin(theta_2 - theta_1) + dtheta_2*l_1*l_2*m_4*np.sin(theta_2 - theta_1) + dtheta_2*l_1*l_2*m_5*np.sin(theta_2 - theta_1) + dtheta_2*l_1*m_2*r_2*np.sin(theta_2 - theta_1)) - dtheta_1*(mu_1 + mu_2) - ddx_c*(-l_1*m_2 - l_1*m_3 - l_1*m_4 - l_1*m_5 - m_1*r_1)*np.cos(theta_1) - g*(-l_1*m_2 - l_1*m_3 - l_1*m_4 - l_1*m_5 - m_1*r_1)*np.sin(theta_1))*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2))/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2))/(I_3 - l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)**2*np.cos(theta_3 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_3**2*m_4 + l_3**2*m_5 + m_3*r_3**2 - (-l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_3 - theta_2))**2/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2)))/(I_4 - l_1**2*(l_4*m_5 + m_4*r_4)**2*np.cos(theta_4 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_4**2*m_5 + m_4*r_4**2 - (-l_1**2*(l_4*m_5 + m_4*r_4)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_4 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_4*m_5 + m_4*r_4)*np.cos(theta_4 - theta_2))**2/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2) - (-l_1**2*(l_4*m_5 + m_4*r_4)*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_4 - theta_1)*np.cos(theta_3 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_3*(l_4*m_5 + m_4*r_4)*np.cos(theta_4 - theta_3) - (-l_1**2*(l_4*m_5 + m_4*r_4)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_4 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_4*m_5 + m_4*r_4)*np.cos(theta_4 - theta_2))*(-l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_3 - theta_2))/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2))**2/(I_3 - l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)**2*np.cos(theta_3 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_3**2*m_4 + l_3**2*m_5 + m_3*r_3**2 - (-l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_3 - theta_2))**2/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2))))*np.cos(theta_5 - theta_1)/(I_5 - l_1**2*m_5**2*r_5**2*np.cos(theta_5 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + m_5*r_5**2 - (-l_1**2*m_5*r_5*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_5 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*m_5*r_5*np.cos(theta_5 - theta_2))**2/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2) - (-l_1**2*m_5*r_5*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_5 - theta_1)*np.cos(theta_3 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_3*m_5*r_5*np.cos(theta_5 - theta_3) - (-l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_3 - theta_2))*(-l_1**2*m_5*r_5*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_5 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*m_5*r_5*np.cos(theta_5 - theta_2))/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2))**2/(I_3 - l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)**2*np.cos(theta_3 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_3**2*m_4 + l_3**2*m_5 + m_3*r_3**2 - (-l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_3 - theta_2))**2/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2)) - (-l_1**2*m_5*r_5*(l_4*m_5 + m_4*r_4)*np.cos(theta_5 - theta_1)*np.cos(theta_4 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_4*m_5*r_5*np.cos(theta_5 - theta_4) - (-l_1**2*(l_4*m_5 + m_4*r_4)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_4 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_4*m_5 + m_4*r_4)*np.cos(theta_4 - theta_2))*(-l_1**2*m_5*r_5*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_5 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*m_5*r_5*np.cos(theta_5 - theta_2))/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2) - (-l_1**2*(l_4*m_5 + m_4*r_4)*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_4 - theta_1)*np.cos(theta_3 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_3*(l_4*m_5 + m_4*r_4)*np.cos(theta_4 - theta_3) - (-l_1**2*(l_4*m_5 + m_4*r_4)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_4 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_4*m_5 + m_4*r_4)*np.cos(theta_4 - theta_2))*(-l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_3 - theta_2))/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2))*(-l_1**2*m_5*r_5*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_5 - theta_1)*np.cos(theta_3 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_3*m_5*r_5*np.cos(theta_5 - theta_3) - (-l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_3 - theta_2))*(-l_1**2*m_5*r_5*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_5 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*m_5*r_5*np.cos(theta_5 - theta_2))/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2))/(I_3 - l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)**2*np.cos(theta_3 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_3**2*m_4 + l_3**2*m_5 + m_3*r_3**2 - (-l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_3 - theta_2))**2/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2)))**2/(I_4 - l_1**2*(l_4*m_5 + m_4*r_4)**2*np.cos(theta_4 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_4**2*m_5 + m_4*r_4**2 - (-l_1**2*(l_4*m_5 + m_4*r_4)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_4 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_4*m_5 + m_4*r_4)*np.cos(theta_4 - theta_2))**2/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2) - (-l_1**2*(l_4*m_5 + m_4*r_4)*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_4 - theta_1)*np.cos(theta_3 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_3*(l_4*m_5 + m_4*r_4)*np.cos(theta_4 - theta_3) - (-l_1**2*(l_4*m_5 + m_4*r_4)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_4 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_4*m_5 + m_4*r_4)*np.cos(theta_4 - theta_2))*(-l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_3 - theta_2))/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2))**2/(I_3 - l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)**2*np.cos(theta_3 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_3**2*m_4 + l_3**2*m_5 + m_3*r_3**2 - (-l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_3 - theta_2))**2/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2)))) - l_1*(l_4*m_5 + m_4*r_4)*(dtheta_5**2*l_4*m_5*r_5*np.sin(theta_5 - theta_4) + dtheta_5*mu_5 - dtheta_4*(mu_4 + mu_5) + dtheta_3*mu_4 + dtheta_3*(-dtheta_3*l_3*l_4*m_5*np.sin(theta_4 - theta_3) - dtheta_3*l_3*m_4*r_4*np.sin(theta_4 - theta_3)) + dtheta_2*(-dtheta_2*l_2*l_4*m_5*np.sin(theta_4 - theta_2) - dtheta_2*l_2*m_4*r_4*np.sin(theta_4 - theta_2)) + dtheta_1*(-dtheta_1*l_1*l_4*m_5*np.sin(theta_4 - theta_1) - dtheta_1*l_1*m_4*r_4*np.sin(theta_4 - theta_1)) - ddx_c*(-l_4*m_5 - m_4*r_4)*np.cos(theta_4) - g*(-l_4*m_5 - m_4*r_4)*np.sin(theta_4) - l_1*(l_4*m_5 + m_4*r_4)*(dtheta_5**2*l_1*m_5*r_5*np.sin(theta_5 - theta_1) + dtheta_4*(dtheta_4*l_1*l_4*m_5*np.sin(theta_4 - theta_1) + dtheta_4*l_1*m_4*r_4*np.sin(theta_4 - theta_1)) + dtheta_3*(dtheta_3*l_1*l_3*m_4*np.sin(theta_3 - theta_1) + dtheta_3*l_1*l_3*m_5*np.sin(theta_3 - theta_1) + dtheta_3*l_1*m_3*r_3*np.sin(theta_3 - theta_1)) + dtheta_2*mu_2 + dtheta_2*(dtheta_2*l_1*l_2*m_3*np.sin(theta_2 - theta_1) + dtheta_2*l_1*l_2*m_4*np.sin(theta_2 - theta_1) + dtheta_2*l_1*l_2*m_5*np.sin(theta_2 - theta_1) + dtheta_2*l_1*m_2*r_2*np.sin(theta_2 - theta_1)) - dtheta_1*(mu_1 + mu_2) - ddx_c*(-l_1*m_2 - l_1*m_3 - l_1*m_4 - l_1*m_5 - m_1*r_1)*np.cos(theta_1) - g*(-l_1*m_2 - l_1*m_3 - l_1*m_4 - l_1*m_5 - m_1*r_1)*np.sin(theta_1))*np.cos(theta_4 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) - (-l_1**2*(l_4*m_5 + m_4*r_4)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_4 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_4*m_5 + m_4*r_4)*np.cos(theta_4 - theta_2))*(dtheta_5**2*l_2*m_5*r_5*np.sin(theta_5 - theta_2) + dtheta_4*(dtheta_4*l_2*l_4*m_5*np.sin(theta_4 - theta_2) + dtheta_4*l_2*m_4*r_4*np.sin(theta_4 - theta_2)) + dtheta_3*mu_3 + dtheta_3*(dtheta_3*l_2*l_3*m_4*np.sin(theta_3 - theta_2) + dtheta_3*l_2*l_3*m_5*np.sin(theta_3 - theta_2) + dtheta_3*l_2*m_3*r_3*np.sin(theta_3 - theta_2)) - dtheta_2*(mu_2 + mu_3) + dtheta_1*mu_2 + dtheta_1*(-dtheta_1*l_1*l_2*m_3*np.sin(theta_2 - theta_1) - dtheta_1*l_1*l_2*m_4*np.sin(theta_2 - theta_1) - dtheta_1*l_1*l_2*m_5*np.sin(theta_2 - theta_1) - dtheta_1*l_1*m_2*r_2*np.sin(theta_2 - theta_1)) - ddx_c*(-l_2*m_3 - l_2*m_4 - l_2*m_5 - m_2*r_2)*np.cos(theta_2) - g*(-l_2*m_3 - l_2*m_4 - l_2*m_5 - m_2*r_2)*np.sin(theta_2) - l_1*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*(dtheta_5**2*l_1*m_5*r_5*np.sin(theta_5 - theta_1) + dtheta_4*(dtheta_4*l_1*l_4*m_5*np.sin(theta_4 - theta_1) + dtheta_4*l_1*m_4*r_4*np.sin(theta_4 - theta_1)) + dtheta_3*(dtheta_3*l_1*l_3*m_4*np.sin(theta_3 - theta_1) + dtheta_3*l_1*l_3*m_5*np.sin(theta_3 - theta_1) + dtheta_3*l_1*m_3*r_3*np.sin(theta_3 - theta_1)) + dtheta_2*mu_2 + dtheta_2*(dtheta_2*l_1*l_2*m_3*np.sin(theta_2 - theta_1) + dtheta_2*l_1*l_2*m_4*np.sin(theta_2 - theta_1) + dtheta_2*l_1*l_2*m_5*np.sin(theta_2 - theta_1) + dtheta_2*l_1*m_2*r_2*np.sin(theta_2 - theta_1)) - dtheta_1*(mu_1 + mu_2) - ddx_c*(-l_1*m_2 - l_1*m_3 - l_1*m_4 - l_1*m_5 - m_1*r_1)*np.cos(theta_1) - g*(-l_1*m_2 - l_1*m_3 - l_1*m_4 - l_1*m_5 - m_1*r_1)*np.sin(theta_1))*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2))/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2) - (-l_1**2*(l_4*m_5 + m_4*r_4)*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_4 - theta_1)*np.cos(theta_3 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_3*(l_4*m_5 + m_4*r_4)*np.cos(theta_4 - theta_3) - (-l_1**2*(l_4*m_5 + m_4*r_4)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_4 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_4*m_5 + m_4*r_4)*np.cos(theta_4 - theta_2))*(-l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_3 - theta_2))/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2))*(dtheta_5**2*l_3*m_5*r_5*np.sin(theta_5 - theta_3) + dtheta_4*mu_4 + dtheta_4*(dtheta_4*l_3*l_4*m_5*np.sin(theta_4 - theta_3) + dtheta_4*l_3*m_4*r_4*np.sin(theta_4 - theta_3)) - dtheta_3*(mu_3 + mu_4) + dtheta_2*mu_3 + dtheta_2*(-dtheta_2*l_2*l_3*m_4*np.sin(theta_3 - theta_2) - dtheta_2*l_2*l_3*m_5*np.sin(theta_3 - theta_2) - dtheta_2*l_2*m_3*r_3*np.sin(theta_3 - theta_2)) + dtheta_1*(-dtheta_1*l_1*l_3*m_4*np.sin(theta_3 - theta_1) - dtheta_1*l_1*l_3*m_5*np.sin(theta_3 - theta_1) - dtheta_1*l_1*m_3*r_3*np.sin(theta_3 - theta_1)) - ddx_c*(-l_3*m_4 - l_3*m_5 - m_3*r_3)*np.cos(theta_3) - g*(-l_3*m_4 - l_3*m_5 - m_3*r_3)*np.sin(theta_3) - l_1*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(dtheta_5**2*l_1*m_5*r_5*np.sin(theta_5 - theta_1) + dtheta_4*(dtheta_4*l_1*l_4*m_5*np.sin(theta_4 - theta_1) + dtheta_4*l_1*m_4*r_4*np.sin(theta_4 - theta_1)) + dtheta_3*(dtheta_3*l_1*l_3*m_4*np.sin(theta_3 - theta_1) + dtheta_3*l_1*l_3*m_5*np.sin(theta_3 - theta_1) + dtheta_3*l_1*m_3*r_3*np.sin(theta_3 - theta_1)) + dtheta_2*mu_2 + dtheta_2*(dtheta_2*l_1*l_2*m_3*np.sin(theta_2 - theta_1) + dtheta_2*l_1*l_2*m_4*np.sin(theta_2 - theta_1) + dtheta_2*l_1*l_2*m_5*np.sin(theta_2 - theta_1) + dtheta_2*l_1*m_2*r_2*np.sin(theta_2 - theta_1)) - dtheta_1*(mu_1 + mu_2) - ddx_c*(-l_1*m_2 - l_1*m_3 - l_1*m_4 - l_1*m_5 - m_1*r_1)*np.cos(theta_1) - g*(-l_1*m_2 - l_1*m_3 - l_1*m_4 - l_1*m_5 - m_1*r_1)*np.sin(theta_1))*np.cos(theta_3 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) - (-l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_3 - theta_2))*(dtheta_5**2*l_2*m_5*r_5*np.sin(theta_5 - theta_2) + dtheta_4*(dtheta_4*l_2*l_4*m_5*np.sin(theta_4 - theta_2) + dtheta_4*l_2*m_4*r_4*np.sin(theta_4 - theta_2)) + dtheta_3*mu_3 + dtheta_3*(dtheta_3*l_2*l_3*m_4*np.sin(theta_3 - theta_2) + dtheta_3*l_2*l_3*m_5*np.sin(theta_3 - theta_2) + dtheta_3*l_2*m_3*r_3*np.sin(theta_3 - theta_2)) - dtheta_2*(mu_2 + mu_3) + dtheta_1*mu_2 + dtheta_1*(-dtheta_1*l_1*l_2*m_3*np.sin(theta_2 - theta_1) - dtheta_1*l_1*l_2*m_4*np.sin(theta_2 - theta_1) - dtheta_1*l_1*l_2*m_5*np.sin(theta_2 - theta_1) - dtheta_1*l_1*m_2*r_2*np.sin(theta_2 - theta_1)) - ddx_c*(-l_2*m_3 - l_2*m_4 - l_2*m_5 - m_2*r_2)*np.cos(theta_2) - g*(-l_2*m_3 - l_2*m_4 - l_2*m_5 - m_2*r_2)*np.sin(theta_2) - l_1*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*(dtheta_5**2*l_1*m_5*r_5*np.sin(theta_5 - theta_1) + dtheta_4*(dtheta_4*l_1*l_4*m_5*np.sin(theta_4 - theta_1) + dtheta_4*l_1*m_4*r_4*np.sin(theta_4 - theta_1)) + dtheta_3*(dtheta_3*l_1*l_3*m_4*np.sin(theta_3 - theta_1) + dtheta_3*l_1*l_3*m_5*np.sin(theta_3 - theta_1) + dtheta_3*l_1*m_3*r_3*np.sin(theta_3 - theta_1)) + dtheta_2*mu_2 + dtheta_2*(dtheta_2*l_1*l_2*m_3*np.sin(theta_2 - theta_1) + dtheta_2*l_1*l_2*m_4*np.sin(theta_2 - theta_1) + dtheta_2*l_1*l_2*m_5*np.sin(theta_2 - theta_1) + dtheta_2*l_1*m_2*r_2*np.sin(theta_2 - theta_1)) - dtheta_1*(mu_1 + mu_2) - ddx_c*(-l_1*m_2 - l_1*m_3 - l_1*m_4 - l_1*m_5 - m_1*r_1)*np.cos(theta_1) - g*(-l_1*m_2 - l_1*m_3 - l_1*m_4 - l_1*m_5 - m_1*r_1)*np.sin(theta_1))*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2))/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2))/(I_3 - l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)**2*np.cos(theta_3 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_3**2*m_4 + l_3**2*m_5 + m_3*r_3**2 - (-l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_3 - theta_2))**2/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2)) - (-l_1**2*m_5*r_5*(l_4*m_5 + m_4*r_4)*np.cos(theta_5 - theta_1)*np.cos(theta_4 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_4*m_5*r_5*np.cos(theta_5 - theta_4) - (-l_1**2*(l_4*m_5 + m_4*r_4)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_4 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_4*m_5 + m_4*r_4)*np.cos(theta_4 - theta_2))*(-l_1**2*m_5*r_5*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_5 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*m_5*r_5*np.cos(theta_5 - theta_2))/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2) - (-l_1**2*(l_4*m_5 + m_4*r_4)*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_4 - theta_1)*np.cos(theta_3 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_3*(l_4*m_5 + m_4*r_4)*np.cos(theta_4 - theta_3) - (-l_1**2*(l_4*m_5 + m_4*r_4)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_4 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_4*m_5 + m_4*r_4)*np.cos(theta_4 - theta_2))*(-l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_3 - theta_2))/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2))*(-l_1**2*m_5*r_5*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_5 - theta_1)*np.cos(theta_3 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_3*m_5*r_5*np.cos(theta_5 - theta_3) - (-l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_3 - theta_2))*(-l_1**2*m_5*r_5*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_5 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*m_5*r_5*np.cos(theta_5 - theta_2))/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2))/(I_3 - l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)**2*np.cos(theta_3 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_3**2*m_4 + l_3**2*m_5 + m_3*r_3**2 - (-l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_3 - theta_2))**2/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2)))*(-dtheta_5*mu_5 - dtheta_4**2*l_4*m_5*r_5*np.sin(theta_5 - theta_4) + dtheta_4*mu_5 - dtheta_3**2*l_3*m_5*r_5*np.sin(theta_5 - theta_3) - dtheta_2**2*l_2*m_5*r_5*np.sin(theta_5 - theta_2) - dtheta_1**2*l_1*m_5*r_5*np.sin(theta_5 - theta_1) + ddx_c*m_5*r_5*np.cos(theta_5) + g*m_5*r_5*np.sin(theta_5) - l_1*m_5*r_5*(dtheta_5**2*l_1*m_5*r_5*np.sin(theta_5 - theta_1) + dtheta_4*(dtheta_4*l_1*l_4*m_5*np.sin(theta_4 - theta_1) + dtheta_4*l_1*m_4*r_4*np.sin(theta_4 - theta_1)) + dtheta_3*(dtheta_3*l_1*l_3*m_4*np.sin(theta_3 - theta_1) + dtheta_3*l_1*l_3*m_5*np.sin(theta_3 - theta_1) + dtheta_3*l_1*m_3*r_3*np.sin(theta_3 - theta_1)) + dtheta_2*mu_2 + dtheta_2*(dtheta_2*l_1*l_2*m_3*np.sin(theta_2 - theta_1) + dtheta_2*l_1*l_2*m_4*np.sin(theta_2 - theta_1) + dtheta_2*l_1*l_2*m_5*np.sin(theta_2 - theta_1) + dtheta_2*l_1*m_2*r_2*np.sin(theta_2 - theta_1)) - dtheta_1*(mu_1 + mu_2) - ddx_c*(-l_1*m_2 - l_1*m_3 - l_1*m_4 - l_1*m_5 - m_1*r_1)*np.cos(theta_1) - g*(-l_1*m_2 - l_1*m_3 - l_1*m_4 - l_1*m_5 - m_1*r_1)*np.sin(theta_1))*np.cos(theta_5 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) - (-l_1**2*m_5*r_5*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_5 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*m_5*r_5*np.cos(theta_5 - theta_2))*(dtheta_5**2*l_2*m_5*r_5*np.sin(theta_5 - theta_2) + dtheta_4*(dtheta_4*l_2*l_4*m_5*np.sin(theta_4 - theta_2) + dtheta_4*l_2*m_4*r_4*np.sin(theta_4 - theta_2)) + dtheta_3*mu_3 + dtheta_3*(dtheta_3*l_2*l_3*m_4*np.sin(theta_3 - theta_2) + dtheta_3*l_2*l_3*m_5*np.sin(theta_3 - theta_2) + dtheta_3*l_2*m_3*r_3*np.sin(theta_3 - theta_2)) - dtheta_2*(mu_2 + mu_3) + dtheta_1*mu_2 + dtheta_1*(-dtheta_1*l_1*l_2*m_3*np.sin(theta_2 - theta_1) - dtheta_1*l_1*l_2*m_4*np.sin(theta_2 - theta_1) - dtheta_1*l_1*l_2*m_5*np.sin(theta_2 - theta_1) - dtheta_1*l_1*m_2*r_2*np.sin(theta_2 - theta_1)) - ddx_c*(-l_2*m_3 - l_2*m_4 - l_2*m_5 - m_2*r_2)*np.cos(theta_2) - g*(-l_2*m_3 - l_2*m_4 - l_2*m_5 - m_2*r_2)*np.sin(theta_2) - l_1*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*(dtheta_5**2*l_1*m_5*r_5*np.sin(theta_5 - theta_1) + dtheta_4*(dtheta_4*l_1*l_4*m_5*np.sin(theta_4 - theta_1) + dtheta_4*l_1*m_4*r_4*np.sin(theta_4 - theta_1)) + dtheta_3*(dtheta_3*l_1*l_3*m_4*np.sin(theta_3 - theta_1) + dtheta_3*l_1*l_3*m_5*np.sin(theta_3 - theta_1) + dtheta_3*l_1*m_3*r_3*np.sin(theta_3 - theta_1)) + dtheta_2*mu_2 + dtheta_2*(dtheta_2*l_1*l_2*m_3*np.sin(theta_2 - theta_1) + dtheta_2*l_1*l_2*m_4*np.sin(theta_2 - theta_1) + dtheta_2*l_1*l_2*m_5*np.sin(theta_2 - theta_1) + dtheta_2*l_1*m_2*r_2*np.sin(theta_2 - theta_1)) - dtheta_1*(mu_1 + mu_2) - ddx_c*(-l_1*m_2 - l_1*m_3 - l_1*m_4 - l_1*m_5 - m_1*r_1)*np.cos(theta_1) - g*(-l_1*m_2 - l_1*m_3 - l_1*m_4 - l_1*m_5 - m_1*r_1)*np.sin(theta_1))*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2))/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2) - (-l_1**2*m_5*r_5*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_5 - theta_1)*np.cos(theta_3 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_3*m_5*r_5*np.cos(theta_5 - theta_3) - (-l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_3 - theta_2))*(-l_1**2*m_5*r_5*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_5 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*m_5*r_5*np.cos(theta_5 - theta_2))/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2))*(dtheta_5**2*l_3*m_5*r_5*np.sin(theta_5 - theta_3) + dtheta_4*mu_4 + dtheta_4*(dtheta_4*l_3*l_4*m_5*np.sin(theta_4 - theta_3) + dtheta_4*l_3*m_4*r_4*np.sin(theta_4 - theta_3)) - dtheta_3*(mu_3 + mu_4) + dtheta_2*mu_3 + dtheta_2*(-dtheta_2*l_2*l_3*m_4*np.sin(theta_3 - theta_2) - dtheta_2*l_2*l_3*m_5*np.sin(theta_3 - theta_2) - dtheta_2*l_2*m_3*r_3*np.sin(theta_3 - theta_2)) + dtheta_1*(-dtheta_1*l_1*l_3*m_4*np.sin(theta_3 - theta_1) - dtheta_1*l_1*l_3*m_5*np.sin(theta_3 - theta_1) - dtheta_1*l_1*m_3*r_3*np.sin(theta_3 - theta_1)) - ddx_c*(-l_3*m_4 - l_3*m_5 - m_3*r_3)*np.cos(theta_3) - g*(-l_3*m_4 - l_3*m_5 - m_3*r_3)*np.sin(theta_3) - l_1*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(dtheta_5**2*l_1*m_5*r_5*np.sin(theta_5 - theta_1) + dtheta_4*(dtheta_4*l_1*l_4*m_5*np.sin(theta_4 - theta_1) + dtheta_4*l_1*m_4*r_4*np.sin(theta_4 - theta_1)) + dtheta_3*(dtheta_3*l_1*l_3*m_4*np.sin(theta_3 - theta_1) + dtheta_3*l_1*l_3*m_5*np.sin(theta_3 - theta_1) + dtheta_3*l_1*m_3*r_3*np.sin(theta_3 - theta_1)) + dtheta_2*mu_2 + dtheta_2*(dtheta_2*l_1*l_2*m_3*np.sin(theta_2 - theta_1) + dtheta_2*l_1*l_2*m_4*np.sin(theta_2 - theta_1) + dtheta_2*l_1*l_2*m_5*np.sin(theta_2 - theta_1) + dtheta_2*l_1*m_2*r_2*np.sin(theta_2 - theta_1)) - dtheta_1*(mu_1 + mu_2) - ddx_c*(-l_1*m_2 - l_1*m_3 - l_1*m_4 - l_1*m_5 - m_1*r_1)*np.cos(theta_1) - g*(-l_1*m_2 - l_1*m_3 - l_1*m_4 - l_1*m_5 - m_1*r_1)*np.sin(theta_1))*np.cos(theta_3 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) - (-l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_3 - theta_2))*(dtheta_5**2*l_2*m_5*r_5*np.sin(theta_5 - theta_2) + dtheta_4*(dtheta_4*l_2*l_4*m_5*np.sin(theta_4 - theta_2) + dtheta_4*l_2*m_4*r_4*np.sin(theta_4 - theta_2)) + dtheta_3*mu_3 + dtheta_3*(dtheta_3*l_2*l_3*m_4*np.sin(theta_3 - theta_2) + dtheta_3*l_2*l_3*m_5*np.sin(theta_3 - theta_2) + dtheta_3*l_2*m_3*r_3*np.sin(theta_3 - theta_2)) - dtheta_2*(mu_2 + mu_3) + dtheta_1*mu_2 + dtheta_1*(-dtheta_1*l_1*l_2*m_3*np.sin(theta_2 - theta_1) - dtheta_1*l_1*l_2*m_4*np.sin(theta_2 - theta_1) - dtheta_1*l_1*l_2*m_5*np.sin(theta_2 - theta_1) - dtheta_1*l_1*m_2*r_2*np.sin(theta_2 - theta_1)) - ddx_c*(-l_2*m_3 - l_2*m_4 - l_2*m_5 - m_2*r_2)*np.cos(theta_2) - g*(-l_2*m_3 - l_2*m_4 - l_2*m_5 - m_2*r_2)*np.sin(theta_2) - l_1*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*(dtheta_5**2*l_1*m_5*r_5*np.sin(theta_5 - theta_1) + dtheta_4*(dtheta_4*l_1*l_4*m_5*np.sin(theta_4 - theta_1) + dtheta_4*l_1*m_4*r_4*np.sin(theta_4 - theta_1)) + dtheta_3*(dtheta_3*l_1*l_3*m_4*np.sin(theta_3 - theta_1) + dtheta_3*l_1*l_3*m_5*np.sin(theta_3 - theta_1) + dtheta_3*l_1*m_3*r_3*np.sin(theta_3 - theta_1)) + dtheta_2*mu_2 + dtheta_2*(dtheta_2*l_1*l_2*m_3*np.sin(theta_2 - theta_1) + dtheta_2*l_1*l_2*m_4*np.sin(theta_2 - theta_1) + dtheta_2*l_1*l_2*m_5*np.sin(theta_2 - theta_1) + dtheta_2*l_1*m_2*r_2*np.sin(theta_2 - theta_1)) - dtheta_1*(mu_1 + mu_2) - ddx_c*(-l_1*m_2 - l_1*m_3 - l_1*m_4 - l_1*m_5 - m_1*r_1)*np.cos(theta_1) - g*(-l_1*m_2 - l_1*m_3 - l_1*m_4 - l_1*m_5 - m_1*r_1)*np.sin(theta_1))*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2))/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2))/(I_3 - l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)**2*np.cos(theta_3 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_3**2*m_4 + l_3**2*m_5 + m_3*r_3**2 - (-l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_3 - theta_2))**2/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2)) - (-l_1**2*m_5*r_5*(l_4*m_5 + m_4*r_4)*np.cos(theta_5 - theta_1)*np.cos(theta_4 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_4*m_5*r_5*np.cos(theta_5 - theta_4) - (-l_1**2*(l_4*m_5 + m_4*r_4)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_4 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_4*m_5 + m_4*r_4)*np.cos(theta_4 - theta_2))*(-l_1**2*m_5*r_5*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_5 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*m_5*r_5*np.cos(theta_5 - theta_2))/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2) - (-l_1**2*(l_4*m_5 + m_4*r_4)*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_4 - theta_1)*np.cos(theta_3 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_3*(l_4*m_5 + m_4*r_4)*np.cos(theta_4 - theta_3) - (-l_1**2*(l_4*m_5 + m_4*r_4)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_4 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_4*m_5 + m_4*r_4)*np.cos(theta_4 - theta_2))*(-l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_3 - theta_2))/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2))*(-l_1**2*m_5*r_5*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_5 - theta_1)*np.cos(theta_3 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_3*m_5*r_5*np.cos(theta_5 - theta_3) - (-l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_3 - theta_2))*(-l_1**2*m_5*r_5*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_5 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*m_5*r_5*np.cos(theta_5 - theta_2))/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2))/(I_3 - l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)**2*np.cos(theta_3 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_3**2*m_4 + l_3**2*m_5 + m_3*r_3**2 - (-l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_3 - theta_2))**2/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2)))*(dtheta_5**2*l_4*m_5*r_5*np.sin(theta_5 - theta_4) + dtheta_5*mu_5 - dtheta_4*(mu_4 + mu_5) + dtheta_3*mu_4 + dtheta_3*(-dtheta_3*l_3*l_4*m_5*np.sin(theta_4 - theta_3) - dtheta_3*l_3*m_4*r_4*np.sin(theta_4 - theta_3)) + dtheta_2*(-dtheta_2*l_2*l_4*m_5*np.sin(theta_4 - theta_2) - dtheta_2*l_2*m_4*r_4*np.sin(theta_4 - theta_2)) + dtheta_1*(-dtheta_1*l_1*l_4*m_5*np.sin(theta_4 - theta_1) - dtheta_1*l_1*m_4*r_4*np.sin(theta_4 - theta_1)) - ddx_c*(-l_4*m_5 - m_4*r_4)*np.cos(theta_4) - g*(-l_4*m_5 - m_4*r_4)*np.sin(theta_4) - l_1*(l_4*m_5 + m_4*r_4)*(dtheta_5**2*l_1*m_5*r_5*np.sin(theta_5 - theta_1) + dtheta_4*(dtheta_4*l_1*l_4*m_5*np.sin(theta_4 - theta_1) + dtheta_4*l_1*m_4*r_4*np.sin(theta_4 - theta_1)) + dtheta_3*(dtheta_3*l_1*l_3*m_4*np.sin(theta_3 - theta_1) + dtheta_3*l_1*l_3*m_5*np.sin(theta_3 - theta_1) + dtheta_3*l_1*m_3*r_3*np.sin(theta_3 - theta_1)) + dtheta_2*mu_2 + dtheta_2*(dtheta_2*l_1*l_2*m_3*np.sin(theta_2 - theta_1) + dtheta_2*l_1*l_2*m_4*np.sin(theta_2 - theta_1) + dtheta_2*l_1*l_2*m_5*np.sin(theta_2 - theta_1) + dtheta_2*l_1*m_2*r_2*np.sin(theta_2 - theta_1)) - dtheta_1*(mu_1 + mu_2) - ddx_c*(-l_1*m_2 - l_1*m_3 - l_1*m_4 - l_1*m_5 - m_1*r_1)*np.cos(theta_1) - g*(-l_1*m_2 - l_1*m_3 - l_1*m_4 - l_1*m_5 - m_1*r_1)*np.sin(theta_1))*np.cos(theta_4 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) - (-l_1**2*(l_4*m_5 + m_4*r_4)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_4 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_4*m_5 + m_4*r_4)*np.cos(theta_4 - theta_2))*(dtheta_5**2*l_2*m_5*r_5*np.sin(theta_5 - theta_2) + dtheta_4*(dtheta_4*l_2*l_4*m_5*np.sin(theta_4 - theta_2) + dtheta_4*l_2*m_4*r_4*np.sin(theta_4 - theta_2)) + dtheta_3*mu_3 + dtheta_3*(dtheta_3*l_2*l_3*m_4*np.sin(theta_3 - theta_2) + dtheta_3*l_2*l_3*m_5*np.sin(theta_3 - theta_2) + dtheta_3*l_2*m_3*r_3*np.sin(theta_3 - theta_2)) - dtheta_2*(mu_2 + mu_3) + dtheta_1*mu_2 + dtheta_1*(-dtheta_1*l_1*l_2*m_3*np.sin(theta_2 - theta_1) - dtheta_1*l_1*l_2*m_4*np.sin(theta_2 - theta_1) - dtheta_1*l_1*l_2*m_5*np.sin(theta_2 - theta_1) - dtheta_1*l_1*m_2*r_2*np.sin(theta_2 - theta_1)) - ddx_c*(-l_2*m_3 - l_2*m_4 - l_2*m_5 - m_2*r_2)*np.cos(theta_2) - g*(-l_2*m_3 - l_2*m_4 - l_2*m_5 - m_2*r_2)*np.sin(theta_2) - l_1*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*(dtheta_5**2*l_1*m_5*r_5*np.sin(theta_5 - theta_1) + dtheta_4*(dtheta_4*l_1*l_4*m_5*np.sin(theta_4 - theta_1) + dtheta_4*l_1*m_4*r_4*np.sin(theta_4 - theta_1)) + dtheta_3*(dtheta_3*l_1*l_3*m_4*np.sin(theta_3 - theta_1) + dtheta_3*l_1*l_3*m_5*np.sin(theta_3 - theta_1) + dtheta_3*l_1*m_3*r_3*np.sin(theta_3 - theta_1)) + dtheta_2*mu_2 + dtheta_2*(dtheta_2*l_1*l_2*m_3*np.sin(theta_2 - theta_1) + dtheta_2*l_1*l_2*m_4*np.sin(theta_2 - theta_1) + dtheta_2*l_1*l_2*m_5*np.sin(theta_2 - theta_1) + dtheta_2*l_1*m_2*r_2*np.sin(theta_2 - theta_1)) - dtheta_1*(mu_1 + mu_2) - ddx_c*(-l_1*m_2 - l_1*m_3 - l_1*m_4 - l_1*m_5 - m_1*r_1)*np.cos(theta_1) - g*(-l_1*m_2 - l_1*m_3 - l_1*m_4 - l_1*m_5 - m_1*r_1)*np.sin(theta_1))*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2))/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2) - (-l_1**2*(l_4*m_5 + m_4*r_4)*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_4 - theta_1)*np.cos(theta_3 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_3*(l_4*m_5 + m_4*r_4)*np.cos(theta_4 - theta_3) - (-l_1**2*(l_4*m_5 + m_4*r_4)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_4 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_4*m_5 + m_4*r_4)*np.cos(theta_4 - theta_2))*(-l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_3 - theta_2))/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2))*(dtheta_5**2*l_3*m_5*r_5*np.sin(theta_5 - theta_3) + dtheta_4*mu_4 + dtheta_4*(dtheta_4*l_3*l_4*m_5*np.sin(theta_4 - theta_3) + dtheta_4*l_3*m_4*r_4*np.sin(theta_4 - theta_3)) - dtheta_3*(mu_3 + mu_4) + dtheta_2*mu_3 + dtheta_2*(-dtheta_2*l_2*l_3*m_4*np.sin(theta_3 - theta_2) - dtheta_2*l_2*l_3*m_5*np.sin(theta_3 - theta_2) - dtheta_2*l_2*m_3*r_3*np.sin(theta_3 - theta_2)) + dtheta_1*(-dtheta_1*l_1*l_3*m_4*np.sin(theta_3 - theta_1) - dtheta_1*l_1*l_3*m_5*np.sin(theta_3 - theta_1) - dtheta_1*l_1*m_3*r_3*np.sin(theta_3 - theta_1)) - ddx_c*(-l_3*m_4 - l_3*m_5 - m_3*r_3)*np.cos(theta_3) - g*(-l_3*m_4 - l_3*m_5 - m_3*r_3)*np.sin(theta_3) - l_1*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(dtheta_5**2*l_1*m_5*r_5*np.sin(theta_5 - theta_1) + dtheta_4*(dtheta_4*l_1*l_4*m_5*np.sin(theta_4 - theta_1) + dtheta_4*l_1*m_4*r_4*np.sin(theta_4 - theta_1)) + dtheta_3*(dtheta_3*l_1*l_3*m_4*np.sin(theta_3 - theta_1) + dtheta_3*l_1*l_3*m_5*np.sin(theta_3 - theta_1) + dtheta_3*l_1*m_3*r_3*np.sin(theta_3 - theta_1)) + dtheta_2*mu_2 + dtheta_2*(dtheta_2*l_1*l_2*m_3*np.sin(theta_2 - theta_1) + dtheta_2*l_1*l_2*m_4*np.sin(theta_2 - theta_1) + dtheta_2*l_1*l_2*m_5*np.sin(theta_2 - theta_1) + dtheta_2*l_1*m_2*r_2*np.sin(theta_2 - theta_1)) - dtheta_1*(mu_1 + mu_2) - ddx_c*(-l_1*m_2 - l_1*m_3 - l_1*m_4 - l_1*m_5 - m_1*r_1)*np.cos(theta_1) - g*(-l_1*m_2 - l_1*m_3 - l_1*m_4 - l_1*m_5 - m_1*r_1)*np.sin(theta_1))*np.cos(theta_3 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) - (-l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_3 - theta_2))*(dtheta_5**2*l_2*m_5*r_5*np.sin(theta_5 - theta_2) + dtheta_4*(dtheta_4*l_2*l_4*m_5*np.sin(theta_4 - theta_2) + dtheta_4*l_2*m_4*r_4*np.sin(theta_4 - theta_2)) + dtheta_3*mu_3 + dtheta_3*(dtheta_3*l_2*l_3*m_4*np.sin(theta_3 - theta_2) + dtheta_3*l_2*l_3*m_5*np.sin(theta_3 - theta_2) + dtheta_3*l_2*m_3*r_3*np.sin(theta_3 - theta_2)) - dtheta_2*(mu_2 + mu_3) + dtheta_1*mu_2 + dtheta_1*(-dtheta_1*l_1*l_2*m_3*np.sin(theta_2 - theta_1) - dtheta_1*l_1*l_2*m_4*np.sin(theta_2 - theta_1) - dtheta_1*l_1*l_2*m_5*np.sin(theta_2 - theta_1) - dtheta_1*l_1*m_2*r_2*np.sin(theta_2 - theta_1)) - ddx_c*(-l_2*m_3 - l_2*m_4 - l_2*m_5 - m_2*r_2)*np.cos(theta_2) - g*(-l_2*m_3 - l_2*m_4 - l_2*m_5 - m_2*r_2)*np.sin(theta_2) - l_1*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*(dtheta_5**2*l_1*m_5*r_5*np.sin(theta_5 - theta_1) + dtheta_4*(dtheta_4*l_1*l_4*m_5*np.sin(theta_4 - theta_1) + dtheta_4*l_1*m_4*r_4*np.sin(theta_4 - theta_1)) + dtheta_3*(dtheta_3*l_1*l_3*m_4*np.sin(theta_3 - theta_1) + dtheta_3*l_1*l_3*m_5*np.sin(theta_3 - theta_1) + dtheta_3*l_1*m_3*r_3*np.sin(theta_3 - theta_1)) + dtheta_2*mu_2 + dtheta_2*(dtheta_2*l_1*l_2*m_3*np.sin(theta_2 - theta_1) + dtheta_2*l_1*l_2*m_4*np.sin(theta_2 - theta_1) + dtheta_2*l_1*l_2*m_5*np.sin(theta_2 - theta_1) + dtheta_2*l_1*m_2*r_2*np.sin(theta_2 - theta_1)) - dtheta_1*(mu_1 + mu_2) - ddx_c*(-l_1*m_2 - l_1*m_3 - l_1*m_4 - l_1*m_5 - m_1*r_1)*np.cos(theta_1) - g*(-l_1*m_2 - l_1*m_3 - l_1*m_4 - l_1*m_5 - m_1*r_1)*np.sin(theta_1))*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2))/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2))/(I_3 - l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)**2*np.cos(theta_3 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_3**2*m_4 + l_3**2*m_5 + m_3*r_3**2 - (-l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_3 - theta_2))**2/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2)))/(I_4 - l_1**2*(l_4*m_5 + m_4*r_4)**2*np.cos(theta_4 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_4**2*m_5 + m_4*r_4**2 - (-l_1**2*(l_4*m_5 + m_4*r_4)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_4 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_4*m_5 + m_4*r_4)*np.cos(theta_4 - theta_2))**2/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2) - (-l_1**2*(l_4*m_5 + m_4*r_4)*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_4 - theta_1)*np.cos(theta_3 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_3*(l_4*m_5 + m_4*r_4)*np.cos(theta_4 - theta_3) - (-l_1**2*(l_4*m_5 + m_4*r_4)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_4 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_4*m_5 + m_4*r_4)*np.cos(theta_4 - theta_2))*(-l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_3 - theta_2))/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2))**2/(I_3 - l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)**2*np.cos(theta_3 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_3**2*m_4 + l_3**2*m_5 + m_3*r_3**2 - (-l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_3 - theta_2))**2/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2))))/(I_5 - l_1**2*m_5**2*r_5**2*np.cos(theta_5 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + m_5*r_5**2 - (-l_1**2*m_5*r_5*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_5 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*m_5*r_5*np.cos(theta_5 - theta_2))**2/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2) - (-l_1**2*m_5*r_5*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_5 - theta_1)*np.cos(theta_3 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_3*m_5*r_5*np.cos(theta_5 - theta_3) - (-l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_3 - theta_2))*(-l_1**2*m_5*r_5*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_5 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*m_5*r_5*np.cos(theta_5 - theta_2))/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2))**2/(I_3 - l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)**2*np.cos(theta_3 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_3**2*m_4 + l_3**2*m_5 + m_3*r_3**2 - (-l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_3 - theta_2))**2/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2)) - (-l_1**2*m_5*r_5*(l_4*m_5 + m_4*r_4)*np.cos(theta_5 - theta_1)*np.cos(theta_4 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_4*m_5*r_5*np.cos(theta_5 - theta_4) - (-l_1**2*(l_4*m_5 + m_4*r_4)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_4 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_4*m_5 + m_4*r_4)*np.cos(theta_4 - theta_2))*(-l_1**2*m_5*r_5*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_5 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*m_5*r_5*np.cos(theta_5 - theta_2))/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2) - (-l_1**2*(l_4*m_5 + m_4*r_4)*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_4 - theta_1)*np.cos(theta_3 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_3*(l_4*m_5 + m_4*r_4)*np.cos(theta_4 - theta_3) - (-l_1**2*(l_4*m_5 + m_4*r_4)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_4 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_4*m_5 + m_4*r_4)*np.cos(theta_4 - theta_2))*(-l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_3 - theta_2))/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2))*(-l_1**2*m_5*r_5*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_5 - theta_1)*np.cos(theta_3 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_3*m_5*r_5*np.cos(theta_5 - theta_3) - (-l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_3 - theta_2))*(-l_1**2*m_5*r_5*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_5 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*m_5*r_5*np.cos(theta_5 - theta_2))/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2))/(I_3 - l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)**2*np.cos(theta_3 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_3**2*m_4 + l_3**2*m_5 + m_3*r_3**2 - (-l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_3 - theta_2))**2/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2)))**2/(I_4 - l_1**2*(l_4*m_5 + m_4*r_4)**2*np.cos(theta_4 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_4**2*m_5 + m_4*r_4**2 - (-l_1**2*(l_4*m_5 + m_4*r_4)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_4 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_4*m_5 + m_4*r_4)*np.cos(theta_4 - theta_2))**2/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2) - (-l_1**2*(l_4*m_5 + m_4*r_4)*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_4 - theta_1)*np.cos(theta_3 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_3*(l_4*m_5 + m_4*r_4)*np.cos(theta_4 - theta_3) - (-l_1**2*(l_4*m_5 + m_4*r_4)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_4 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_4*m_5 + m_4*r_4)*np.cos(theta_4 - theta_2))*(-l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_3 - theta_2))/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2))**2/(I_3 - l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)**2*np.cos(theta_3 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_3**2*m_4 + l_3**2*m_5 + m_3*r_3**2 - (-l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_3 - theta_2))**2/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2)))))*np.cos(theta_4 - theta_1)/(I_4 - l_1**2*(l_4*m_5 + m_4*r_4)**2*np.cos(theta_4 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_4**2*m_5 + m_4*r_4**2 - (-l_1**2*(l_4*m_5 + m_4*r_4)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_4 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_4*m_5 + m_4*r_4)*np.cos(theta_4 - theta_2))**2/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2) - (-l_1**2*(l_4*m_5 + m_4*r_4)*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_4 - theta_1)*np.cos(theta_3 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_3*(l_4*m_5 + m_4*r_4)*np.cos(theta_4 - theta_3) - (-l_1**2*(l_4*m_5 + m_4*r_4)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_4 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_4*m_5 + m_4*r_4)*np.cos(theta_4 - theta_2))*(-l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_3 - theta_2))/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2))**2/(I_3 - l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)**2*np.cos(theta_3 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_3**2*m_4 + l_3**2*m_5 + m_3*r_3**2 - (-l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_3 - theta_2))**2/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2))) - l_1*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(dtheta_5**2*l_3*m_5*r_5*np.sin(theta_5 - theta_3) + dtheta_4*mu_4 + dtheta_4*(dtheta_4*l_3*l_4*m_5*np.sin(theta_4 - theta_3) + dtheta_4*l_3*m_4*r_4*np.sin(theta_4 - theta_3)) - dtheta_3*(mu_3 + mu_4) + dtheta_2*mu_3 + dtheta_2*(-dtheta_2*l_2*l_3*m_4*np.sin(theta_3 - theta_2) - dtheta_2*l_2*l_3*m_5*np.sin(theta_3 - theta_2) - dtheta_2*l_2*m_3*r_3*np.sin(theta_3 - theta_2)) + dtheta_1*(-dtheta_1*l_1*l_3*m_4*np.sin(theta_3 - theta_1) - dtheta_1*l_1*l_3*m_5*np.sin(theta_3 - theta_1) - dtheta_1*l_1*m_3*r_3*np.sin(theta_3 - theta_1)) - ddx_c*(-l_3*m_4 - l_3*m_5 - m_3*r_3)*np.cos(theta_3) - g*(-l_3*m_4 - l_3*m_5 - m_3*r_3)*np.sin(theta_3) - l_1*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(dtheta_5**2*l_1*m_5*r_5*np.sin(theta_5 - theta_1) + dtheta_4*(dtheta_4*l_1*l_4*m_5*np.sin(theta_4 - theta_1) + dtheta_4*l_1*m_4*r_4*np.sin(theta_4 - theta_1)) + dtheta_3*(dtheta_3*l_1*l_3*m_4*np.sin(theta_3 - theta_1) + dtheta_3*l_1*l_3*m_5*np.sin(theta_3 - theta_1) + dtheta_3*l_1*m_3*r_3*np.sin(theta_3 - theta_1)) + dtheta_2*mu_2 + dtheta_2*(dtheta_2*l_1*l_2*m_3*np.sin(theta_2 - theta_1) + dtheta_2*l_1*l_2*m_4*np.sin(theta_2 - theta_1) + dtheta_2*l_1*l_2*m_5*np.sin(theta_2 - theta_1) + dtheta_2*l_1*m_2*r_2*np.sin(theta_2 - theta_1)) - dtheta_1*(mu_1 + mu_2) - ddx_c*(-l_1*m_2 - l_1*m_3 - l_1*m_4 - l_1*m_5 - m_1*r_1)*np.cos(theta_1) - g*(-l_1*m_2 - l_1*m_3 - l_1*m_4 - l_1*m_5 - m_1*r_1)*np.sin(theta_1))*np.cos(theta_3 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) - (-l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_3 - theta_2))*(dtheta_5**2*l_2*m_5*r_5*np.sin(theta_5 - theta_2) + dtheta_4*(dtheta_4*l_2*l_4*m_5*np.sin(theta_4 - theta_2) + dtheta_4*l_2*m_4*r_4*np.sin(theta_4 - theta_2)) + dtheta_3*mu_3 + dtheta_3*(dtheta_3*l_2*l_3*m_4*np.sin(theta_3 - theta_2) + dtheta_3*l_2*l_3*m_5*np.sin(theta_3 - theta_2) + dtheta_3*l_2*m_3*r_3*np.sin(theta_3 - theta_2)) - dtheta_2*(mu_2 + mu_3) + dtheta_1*mu_2 + dtheta_1*(-dtheta_1*l_1*l_2*m_3*np.sin(theta_2 - theta_1) - dtheta_1*l_1*l_2*m_4*np.sin(theta_2 - theta_1) - dtheta_1*l_1*l_2*m_5*np.sin(theta_2 - theta_1) - dtheta_1*l_1*m_2*r_2*np.sin(theta_2 - theta_1)) - ddx_c*(-l_2*m_3 - l_2*m_4 - l_2*m_5 - m_2*r_2)*np.cos(theta_2) - g*(-l_2*m_3 - l_2*m_4 - l_2*m_5 - m_2*r_2)*np.sin(theta_2) - l_1*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*(dtheta_5**2*l_1*m_5*r_5*np.sin(theta_5 - theta_1) + dtheta_4*(dtheta_4*l_1*l_4*m_5*np.sin(theta_4 - theta_1) + dtheta_4*l_1*m_4*r_4*np.sin(theta_4 - theta_1)) + dtheta_3*(dtheta_3*l_1*l_3*m_4*np.sin(theta_3 - theta_1) + dtheta_3*l_1*l_3*m_5*np.sin(theta_3 - theta_1) + dtheta_3*l_1*m_3*r_3*np.sin(theta_3 - theta_1)) + dtheta_2*mu_2 + dtheta_2*(dtheta_2*l_1*l_2*m_3*np.sin(theta_2 - theta_1) + dtheta_2*l_1*l_2*m_4*np.sin(theta_2 - theta_1) + dtheta_2*l_1*l_2*m_5*np.sin(theta_2 - theta_1) + dtheta_2*l_1*m_2*r_2*np.sin(theta_2 - theta_1)) - dtheta_1*(mu_1 + mu_2) - ddx_c*(-l_1*m_2 - l_1*m_3 - l_1*m_4 - l_1*m_5 - m_1*r_1)*np.cos(theta_1) - g*(-l_1*m_2 - l_1*m_3 - l_1*m_4 - l_1*m_5 - m_1*r_1)*np.sin(theta_1))*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2))/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2) - (-l_1**2*(l_4*m_5 + m_4*r_4)*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_4 - theta_1)*np.cos(theta_3 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_3*(l_4*m_5 + m_4*r_4)*np.cos(theta_4 - theta_3) - (-l_1**2*(l_4*m_5 + m_4*r_4)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_4 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_4*m_5 + m_4*r_4)*np.cos(theta_4 - theta_2))*(-l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_3 - theta_2))/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2))*(dtheta_5**2*l_4*m_5*r_5*np.sin(theta_5 - theta_4) + dtheta_5*mu_5 - dtheta_4*(mu_4 + mu_5) + dtheta_3*mu_4 + dtheta_3*(-dtheta_3*l_3*l_4*m_5*np.sin(theta_4 - theta_3) - dtheta_3*l_3*m_4*r_4*np.sin(theta_4 - theta_3)) + dtheta_2*(-dtheta_2*l_2*l_4*m_5*np.sin(theta_4 - theta_2) - dtheta_2*l_2*m_4*r_4*np.sin(theta_4 - theta_2)) + dtheta_1*(-dtheta_1*l_1*l_4*m_5*np.sin(theta_4 - theta_1) - dtheta_1*l_1*m_4*r_4*np.sin(theta_4 - theta_1)) - ddx_c*(-l_4*m_5 - m_4*r_4)*np.cos(theta_4) - g*(-l_4*m_5 - m_4*r_4)*np.sin(theta_4) - l_1*(l_4*m_5 + m_4*r_4)*(dtheta_5**2*l_1*m_5*r_5*np.sin(theta_5 - theta_1) + dtheta_4*(dtheta_4*l_1*l_4*m_5*np.sin(theta_4 - theta_1) + dtheta_4*l_1*m_4*r_4*np.sin(theta_4 - theta_1)) + dtheta_3*(dtheta_3*l_1*l_3*m_4*np.sin(theta_3 - theta_1) + dtheta_3*l_1*l_3*m_5*np.sin(theta_3 - theta_1) + dtheta_3*l_1*m_3*r_3*np.sin(theta_3 - theta_1)) + dtheta_2*mu_2 + dtheta_2*(dtheta_2*l_1*l_2*m_3*np.sin(theta_2 - theta_1) + dtheta_2*l_1*l_2*m_4*np.sin(theta_2 - theta_1) + dtheta_2*l_1*l_2*m_5*np.sin(theta_2 - theta_1) + dtheta_2*l_1*m_2*r_2*np.sin(theta_2 - theta_1)) - dtheta_1*(mu_1 + mu_2) - ddx_c*(-l_1*m_2 - l_1*m_3 - l_1*m_4 - l_1*m_5 - m_1*r_1)*np.cos(theta_1) - g*(-l_1*m_2 - l_1*m_3 - l_1*m_4 - l_1*m_5 - m_1*r_1)*np.sin(theta_1))*np.cos(theta_4 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) - (-l_1**2*(l_4*m_5 + m_4*r_4)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_4 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_4*m_5 + m_4*r_4)*np.cos(theta_4 - theta_2))*(dtheta_5**2*l_2*m_5*r_5*np.sin(theta_5 - theta_2) + dtheta_4*(dtheta_4*l_2*l_4*m_5*np.sin(theta_4 - theta_2) + dtheta_4*l_2*m_4*r_4*np.sin(theta_4 - theta_2)) + dtheta_3*mu_3 + dtheta_3*(dtheta_3*l_2*l_3*m_4*np.sin(theta_3 - theta_2) + dtheta_3*l_2*l_3*m_5*np.sin(theta_3 - theta_2) + dtheta_3*l_2*m_3*r_3*np.sin(theta_3 - theta_2)) - dtheta_2*(mu_2 + mu_3) + dtheta_1*mu_2 + dtheta_1*(-dtheta_1*l_1*l_2*m_3*np.sin(theta_2 - theta_1) - dtheta_1*l_1*l_2*m_4*np.sin(theta_2 - theta_1) - dtheta_1*l_1*l_2*m_5*np.sin(theta_2 - theta_1) - dtheta_1*l_1*m_2*r_2*np.sin(theta_2 - theta_1)) - ddx_c*(-l_2*m_3 - l_2*m_4 - l_2*m_5 - m_2*r_2)*np.cos(theta_2) - g*(-l_2*m_3 - l_2*m_4 - l_2*m_5 - m_2*r_2)*np.sin(theta_2) - l_1*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*(dtheta_5**2*l_1*m_5*r_5*np.sin(theta_5 - theta_1) + dtheta_4*(dtheta_4*l_1*l_4*m_5*np.sin(theta_4 - theta_1) + dtheta_4*l_1*m_4*r_4*np.sin(theta_4 - theta_1)) + dtheta_3*(dtheta_3*l_1*l_3*m_4*np.sin(theta_3 - theta_1) + dtheta_3*l_1*l_3*m_5*np.sin(theta_3 - theta_1) + dtheta_3*l_1*m_3*r_3*np.sin(theta_3 - theta_1)) + dtheta_2*mu_2 + dtheta_2*(dtheta_2*l_1*l_2*m_3*np.sin(theta_2 - theta_1) + dtheta_2*l_1*l_2*m_4*np.sin(theta_2 - theta_1) + dtheta_2*l_1*l_2*m_5*np.sin(theta_2 - theta_1) + dtheta_2*l_1*m_2*r_2*np.sin(theta_2 - theta_1)) - dtheta_1*(mu_1 + mu_2) - ddx_c*(-l_1*m_2 - l_1*m_3 - l_1*m_4 - l_1*m_5 - m_1*r_1)*np.cos(theta_1) - g*(-l_1*m_2 - l_1*m_3 - l_1*m_4 - l_1*m_5 - m_1*r_1)*np.sin(theta_1))*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2))/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2) - (-l_1**2*(l_4*m_5 + m_4*r_4)*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_4 - theta_1)*np.cos(theta_3 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_3*(l_4*m_5 + m_4*r_4)*np.cos(theta_4 - theta_3) - (-l_1**2*(l_4*m_5 + m_4*r_4)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_4 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_4*m_5 + m_4*r_4)*np.cos(theta_4 - theta_2))*(-l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_3 - theta_2))/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2))*(dtheta_5**2*l_3*m_5*r_5*np.sin(theta_5 - theta_3) + dtheta_4*mu_4 + dtheta_4*(dtheta_4*l_3*l_4*m_5*np.sin(theta_4 - theta_3) + dtheta_4*l_3*m_4*r_4*np.sin(theta_4 - theta_3)) - dtheta_3*(mu_3 + mu_4) + dtheta_2*mu_3 + dtheta_2*(-dtheta_2*l_2*l_3*m_4*np.sin(theta_3 - theta_2) - dtheta_2*l_2*l_3*m_5*np.sin(theta_3 - theta_2) - dtheta_2*l_2*m_3*r_3*np.sin(theta_3 - theta_2)) + dtheta_1*(-dtheta_1*l_1*l_3*m_4*np.sin(theta_3 - theta_1) - dtheta_1*l_1*l_3*m_5*np.sin(theta_3 - theta_1) - dtheta_1*l_1*m_3*r_3*np.sin(theta_3 - theta_1)) - ddx_c*(-l_3*m_4 - l_3*m_5 - m_3*r_3)*np.cos(theta_3) - g*(-l_3*m_4 - l_3*m_5 - m_3*r_3)*np.sin(theta_3) - l_1*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(dtheta_5**2*l_1*m_5*r_5*np.sin(theta_5 - theta_1) + dtheta_4*(dtheta_4*l_1*l_4*m_5*np.sin(theta_4 - theta_1) + dtheta_4*l_1*m_4*r_4*np.sin(theta_4 - theta_1)) + dtheta_3*(dtheta_3*l_1*l_3*m_4*np.sin(theta_3 - theta_1) + dtheta_3*l_1*l_3*m_5*np.sin(theta_3 - theta_1) + dtheta_3*l_1*m_3*r_3*np.sin(theta_3 - theta_1)) + dtheta_2*mu_2 + dtheta_2*(dtheta_2*l_1*l_2*m_3*np.sin(theta_2 - theta_1) + dtheta_2*l_1*l_2*m_4*np.sin(theta_2 - theta_1) + dtheta_2*l_1*l_2*m_5*np.sin(theta_2 - theta_1) + dtheta_2*l_1*m_2*r_2*np.sin(theta_2 - theta_1)) - dtheta_1*(mu_1 + mu_2) - ddx_c*(-l_1*m_2 - l_1*m_3 - l_1*m_4 - l_1*m_5 - m_1*r_1)*np.cos(theta_1) - g*(-l_1*m_2 - l_1*m_3 - l_1*m_4 - l_1*m_5 - m_1*r_1)*np.sin(theta_1))*np.cos(theta_3 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) - (-l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_3 - theta_2))*(dtheta_5**2*l_2*m_5*r_5*np.sin(theta_5 - theta_2) + dtheta_4*(dtheta_4*l_2*l_4*m_5*np.sin(theta_4 - theta_2) + dtheta_4*l_2*m_4*r_4*np.sin(theta_4 - theta_2)) + dtheta_3*mu_3 + dtheta_3*(dtheta_3*l_2*l_3*m_4*np.sin(theta_3 - theta_2) + dtheta_3*l_2*l_3*m_5*np.sin(theta_3 - theta_2) + dtheta_3*l_2*m_3*r_3*np.sin(theta_3 - theta_2)) - dtheta_2*(mu_2 + mu_3) + dtheta_1*mu_2 + dtheta_1*(-dtheta_1*l_1*l_2*m_3*np.sin(theta_2 - theta_1) - dtheta_1*l_1*l_2*m_4*np.sin(theta_2 - theta_1) - dtheta_1*l_1*l_2*m_5*np.sin(theta_2 - theta_1) - dtheta_1*l_1*m_2*r_2*np.sin(theta_2 - theta_1)) - ddx_c*(-l_2*m_3 - l_2*m_4 - l_2*m_5 - m_2*r_2)*np.cos(theta_2) - g*(-l_2*m_3 - l_2*m_4 - l_2*m_5 - m_2*r_2)*np.sin(theta_2) - l_1*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*(dtheta_5**2*l_1*m_5*r_5*np.sin(theta_5 - theta_1) + dtheta_4*(dtheta_4*l_1*l_4*m_5*np.sin(theta_4 - theta_1) + dtheta_4*l_1*m_4*r_4*np.sin(theta_4 - theta_1)) + dtheta_3*(dtheta_3*l_1*l_3*m_4*np.sin(theta_3 - theta_1) + dtheta_3*l_1*l_3*m_5*np.sin(theta_3 - theta_1) + dtheta_3*l_1*m_3*r_3*np.sin(theta_3 - theta_1)) + dtheta_2*mu_2 + dtheta_2*(dtheta_2*l_1*l_2*m_3*np.sin(theta_2 - theta_1) + dtheta_2*l_1*l_2*m_4*np.sin(theta_2 - theta_1) + dtheta_2*l_1*l_2*m_5*np.sin(theta_2 - theta_1) + dtheta_2*l_1*m_2*r_2*np.sin(theta_2 - theta_1)) - dtheta_1*(mu_1 + mu_2) - ddx_c*(-l_1*m_2 - l_1*m_3 - l_1*m_4 - l_1*m_5 - m_1*r_1)*np.cos(theta_1) - g*(-l_1*m_2 - l_1*m_3 - l_1*m_4 - l_1*m_5 - m_1*r_1)*np.sin(theta_1))*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2))/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2))/(I_3 - l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)**2*np.cos(theta_3 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_3**2*m_4 + l_3**2*m_5 + m_3*r_3**2 - (-l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_3 - theta_2))**2/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2)) - (-l_1**2*m_5*r_5*(l_4*m_5 + m_4*r_4)*np.cos(theta_5 - theta_1)*np.cos(theta_4 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_4*m_5*r_5*np.cos(theta_5 - theta_4) - (-l_1**2*(l_4*m_5 + m_4*r_4)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_4 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_4*m_5 + m_4*r_4)*np.cos(theta_4 - theta_2))*(-l_1**2*m_5*r_5*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_5 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*m_5*r_5*np.cos(theta_5 - theta_2))/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2) - (-l_1**2*(l_4*m_5 + m_4*r_4)*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_4 - theta_1)*np.cos(theta_3 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_3*(l_4*m_5 + m_4*r_4)*np.cos(theta_4 - theta_3) - (-l_1**2*(l_4*m_5 + m_4*r_4)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_4 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_4*m_5 + m_4*r_4)*np.cos(theta_4 - theta_2))*(-l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_3 - theta_2))/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2))*(-l_1**2*m_5*r_5*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_5 - theta_1)*np.cos(theta_3 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_3*m_5*r_5*np.cos(theta_5 - theta_3) - (-l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_3 - theta_2))*(-l_1**2*m_5*r_5*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_5 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*m_5*r_5*np.cos(theta_5 - theta_2))/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2))/(I_3 - l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)**2*np.cos(theta_3 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_3**2*m_4 + l_3**2*m_5 + m_3*r_3**2 - (-l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_3 - theta_2))**2/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2)))*(-dtheta_5*mu_5 - dtheta_4**2*l_4*m_5*r_5*np.sin(theta_5 - theta_4) + dtheta_4*mu_5 - dtheta_3**2*l_3*m_5*r_5*np.sin(theta_5 - theta_3) - dtheta_2**2*l_2*m_5*r_5*np.sin(theta_5 - theta_2) - dtheta_1**2*l_1*m_5*r_5*np.sin(theta_5 - theta_1) + ddx_c*m_5*r_5*np.cos(theta_5) + g*m_5*r_5*np.sin(theta_5) - l_1*m_5*r_5*(dtheta_5**2*l_1*m_5*r_5*np.sin(theta_5 - theta_1) + dtheta_4*(dtheta_4*l_1*l_4*m_5*np.sin(theta_4 - theta_1) + dtheta_4*l_1*m_4*r_4*np.sin(theta_4 - theta_1)) + dtheta_3*(dtheta_3*l_1*l_3*m_4*np.sin(theta_3 - theta_1) + dtheta_3*l_1*l_3*m_5*np.sin(theta_3 - theta_1) + dtheta_3*l_1*m_3*r_3*np.sin(theta_3 - theta_1)) + dtheta_2*mu_2 + dtheta_2*(dtheta_2*l_1*l_2*m_3*np.sin(theta_2 - theta_1) + dtheta_2*l_1*l_2*m_4*np.sin(theta_2 - theta_1) + dtheta_2*l_1*l_2*m_5*np.sin(theta_2 - theta_1) + dtheta_2*l_1*m_2*r_2*np.sin(theta_2 - theta_1)) - dtheta_1*(mu_1 + mu_2) - ddx_c*(-l_1*m_2 - l_1*m_3 - l_1*m_4 - l_1*m_5 - m_1*r_1)*np.cos(theta_1) - g*(-l_1*m_2 - l_1*m_3 - l_1*m_4 - l_1*m_5 - m_1*r_1)*np.sin(theta_1))*np.cos(theta_5 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) - (-l_1**2*m_5*r_5*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_5 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*m_5*r_5*np.cos(theta_5 - theta_2))*(dtheta_5**2*l_2*m_5*r_5*np.sin(theta_5 - theta_2) + dtheta_4*(dtheta_4*l_2*l_4*m_5*np.sin(theta_4 - theta_2) + dtheta_4*l_2*m_4*r_4*np.sin(theta_4 - theta_2)) + dtheta_3*mu_3 + dtheta_3*(dtheta_3*l_2*l_3*m_4*np.sin(theta_3 - theta_2) + dtheta_3*l_2*l_3*m_5*np.sin(theta_3 - theta_2) + dtheta_3*l_2*m_3*r_3*np.sin(theta_3 - theta_2)) - dtheta_2*(mu_2 + mu_3) + dtheta_1*mu_2 + dtheta_1*(-dtheta_1*l_1*l_2*m_3*np.sin(theta_2 - theta_1) - dtheta_1*l_1*l_2*m_4*np.sin(theta_2 - theta_1) - dtheta_1*l_1*l_2*m_5*np.sin(theta_2 - theta_1) - dtheta_1*l_1*m_2*r_2*np.sin(theta_2 - theta_1)) - ddx_c*(-l_2*m_3 - l_2*m_4 - l_2*m_5 - m_2*r_2)*np.cos(theta_2) - g*(-l_2*m_3 - l_2*m_4 - l_2*m_5 - m_2*r_2)*np.sin(theta_2) - l_1*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*(dtheta_5**2*l_1*m_5*r_5*np.sin(theta_5 - theta_1) + dtheta_4*(dtheta_4*l_1*l_4*m_5*np.sin(theta_4 - theta_1) + dtheta_4*l_1*m_4*r_4*np.sin(theta_4 - theta_1)) + dtheta_3*(dtheta_3*l_1*l_3*m_4*np.sin(theta_3 - theta_1) + dtheta_3*l_1*l_3*m_5*np.sin(theta_3 - theta_1) + dtheta_3*l_1*m_3*r_3*np.sin(theta_3 - theta_1)) + dtheta_2*mu_2 + dtheta_2*(dtheta_2*l_1*l_2*m_3*np.sin(theta_2 - theta_1) + dtheta_2*l_1*l_2*m_4*np.sin(theta_2 - theta_1) + dtheta_2*l_1*l_2*m_5*np.sin(theta_2 - theta_1) + dtheta_2*l_1*m_2*r_2*np.sin(theta_2 - theta_1)) - dtheta_1*(mu_1 + mu_2) - ddx_c*(-l_1*m_2 - l_1*m_3 - l_1*m_4 - l_1*m_5 - m_1*r_1)*np.cos(theta_1) - g*(-l_1*m_2 - l_1*m_3 - l_1*m_4 - l_1*m_5 - m_1*r_1)*np.sin(theta_1))*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2))/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2) - (-l_1**2*m_5*r_5*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_5 - theta_1)*np.cos(theta_3 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_3*m_5*r_5*np.cos(theta_5 - theta_3) - (-l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_3 - theta_2))*(-l_1**2*m_5*r_5*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_5 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*m_5*r_5*np.cos(theta_5 - theta_2))/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2))*(dtheta_5**2*l_3*m_5*r_5*np.sin(theta_5 - theta_3) + dtheta_4*mu_4 + dtheta_4*(dtheta_4*l_3*l_4*m_5*np.sin(theta_4 - theta_3) + dtheta_4*l_3*m_4*r_4*np.sin(theta_4 - theta_3)) - dtheta_3*(mu_3 + mu_4) + dtheta_2*mu_3 + dtheta_2*(-dtheta_2*l_2*l_3*m_4*np.sin(theta_3 - theta_2) - dtheta_2*l_2*l_3*m_5*np.sin(theta_3 - theta_2) - dtheta_2*l_2*m_3*r_3*np.sin(theta_3 - theta_2)) + dtheta_1*(-dtheta_1*l_1*l_3*m_4*np.sin(theta_3 - theta_1) - dtheta_1*l_1*l_3*m_5*np.sin(theta_3 - theta_1) - dtheta_1*l_1*m_3*r_3*np.sin(theta_3 - theta_1)) - ddx_c*(-l_3*m_4 - l_3*m_5 - m_3*r_3)*np.cos(theta_3) - g*(-l_3*m_4 - l_3*m_5 - m_3*r_3)*np.sin(theta_3) - l_1*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(dtheta_5**2*l_1*m_5*r_5*np.sin(theta_5 - theta_1) + dtheta_4*(dtheta_4*l_1*l_4*m_5*np.sin(theta_4 - theta_1) + dtheta_4*l_1*m_4*r_4*np.sin(theta_4 - theta_1)) + dtheta_3*(dtheta_3*l_1*l_3*m_4*np.sin(theta_3 - theta_1) + dtheta_3*l_1*l_3*m_5*np.sin(theta_3 - theta_1) + dtheta_3*l_1*m_3*r_3*np.sin(theta_3 - theta_1)) + dtheta_2*mu_2 + dtheta_2*(dtheta_2*l_1*l_2*m_3*np.sin(theta_2 - theta_1) + dtheta_2*l_1*l_2*m_4*np.sin(theta_2 - theta_1) + dtheta_2*l_1*l_2*m_5*np.sin(theta_2 - theta_1) + dtheta_2*l_1*m_2*r_2*np.sin(theta_2 - theta_1)) - dtheta_1*(mu_1 + mu_2) - ddx_c*(-l_1*m_2 - l_1*m_3 - l_1*m_4 - l_1*m_5 - m_1*r_1)*np.cos(theta_1) - g*(-l_1*m_2 - l_1*m_3 - l_1*m_4 - l_1*m_5 - m_1*r_1)*np.sin(theta_1))*np.cos(theta_3 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) - (-l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_3 - theta_2))*(dtheta_5**2*l_2*m_5*r_5*np.sin(theta_5 - theta_2) + dtheta_4*(dtheta_4*l_2*l_4*m_5*np.sin(theta_4 - theta_2) + dtheta_4*l_2*m_4*r_4*np.sin(theta_4 - theta_2)) + dtheta_3*mu_3 + dtheta_3*(dtheta_3*l_2*l_3*m_4*np.sin(theta_3 - theta_2) + dtheta_3*l_2*l_3*m_5*np.sin(theta_3 - theta_2) + dtheta_3*l_2*m_3*r_3*np.sin(theta_3 - theta_2)) - dtheta_2*(mu_2 + mu_3) + dtheta_1*mu_2 + dtheta_1*(-dtheta_1*l_1*l_2*m_3*np.sin(theta_2 - theta_1) - dtheta_1*l_1*l_2*m_4*np.sin(theta_2 - theta_1) - dtheta_1*l_1*l_2*m_5*np.sin(theta_2 - theta_1) - dtheta_1*l_1*m_2*r_2*np.sin(theta_2 - theta_1)) - ddx_c*(-l_2*m_3 - l_2*m_4 - l_2*m_5 - m_2*r_2)*np.cos(theta_2) - g*(-l_2*m_3 - l_2*m_4 - l_2*m_5 - m_2*r_2)*np.sin(theta_2) - l_1*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*(dtheta_5**2*l_1*m_5*r_5*np.sin(theta_5 - theta_1) + dtheta_4*(dtheta_4*l_1*l_4*m_5*np.sin(theta_4 - theta_1) + dtheta_4*l_1*m_4*r_4*np.sin(theta_4 - theta_1)) + dtheta_3*(dtheta_3*l_1*l_3*m_4*np.sin(theta_3 - theta_1) + dtheta_3*l_1*l_3*m_5*np.sin(theta_3 - theta_1) + dtheta_3*l_1*m_3*r_3*np.sin(theta_3 - theta_1)) + dtheta_2*mu_2 + dtheta_2*(dtheta_2*l_1*l_2*m_3*np.sin(theta_2 - theta_1) + dtheta_2*l_1*l_2*m_4*np.sin(theta_2 - theta_1) + dtheta_2*l_1*l_2*m_5*np.sin(theta_2 - theta_1) + dtheta_2*l_1*m_2*r_2*np.sin(theta_2 - theta_1)) - dtheta_1*(mu_1 + mu_2) - ddx_c*(-l_1*m_2 - l_1*m_3 - l_1*m_4 - l_1*m_5 - m_1*r_1)*np.cos(theta_1) - g*(-l_1*m_2 - l_1*m_3 - l_1*m_4 - l_1*m_5 - m_1*r_1)*np.sin(theta_1))*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2))/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2))/(I_3 - l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)**2*np.cos(theta_3 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_3**2*m_4 + l_3**2*m_5 + m_3*r_3**2 - (-l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_3 - theta_2))**2/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2)) - (-l_1**2*m_5*r_5*(l_4*m_5 + m_4*r_4)*np.cos(theta_5 - theta_1)*np.cos(theta_4 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_4*m_5*r_5*np.cos(theta_5 - theta_4) - (-l_1**2*(l_4*m_5 + m_4*r_4)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_4 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_4*m_5 + m_4*r_4)*np.cos(theta_4 - theta_2))*(-l_1**2*m_5*r_5*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_5 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*m_5*r_5*np.cos(theta_5 - theta_2))/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2) - (-l_1**2*(l_4*m_5 + m_4*r_4)*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_4 - theta_1)*np.cos(theta_3 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_3*(l_4*m_5 + m_4*r_4)*np.cos(theta_4 - theta_3) - (-l_1**2*(l_4*m_5 + m_4*r_4)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_4 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_4*m_5 + m_4*r_4)*np.cos(theta_4 - theta_2))*(-l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_3 - theta_2))/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2))*(-l_1**2*m_5*r_5*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_5 - theta_1)*np.cos(theta_3 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_3*m_5*r_5*np.cos(theta_5 - theta_3) - (-l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_3 - theta_2))*(-l_1**2*m_5*r_5*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_5 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*m_5*r_5*np.cos(theta_5 - theta_2))/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2))/(I_3 - l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)**2*np.cos(theta_3 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_3**2*m_4 + l_3**2*m_5 + m_3*r_3**2 - (-l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_3 - theta_2))**2/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2)))*(dtheta_5**2*l_4*m_5*r_5*np.sin(theta_5 - theta_4) + dtheta_5*mu_5 - dtheta_4*(mu_4 + mu_5) + dtheta_3*mu_4 + dtheta_3*(-dtheta_3*l_3*l_4*m_5*np.sin(theta_4 - theta_3) - dtheta_3*l_3*m_4*r_4*np.sin(theta_4 - theta_3)) + dtheta_2*(-dtheta_2*l_2*l_4*m_5*np.sin(theta_4 - theta_2) - dtheta_2*l_2*m_4*r_4*np.sin(theta_4 - theta_2)) + dtheta_1*(-dtheta_1*l_1*l_4*m_5*np.sin(theta_4 - theta_1) - dtheta_1*l_1*m_4*r_4*np.sin(theta_4 - theta_1)) - ddx_c*(-l_4*m_5 - m_4*r_4)*np.cos(theta_4) - g*(-l_4*m_5 - m_4*r_4)*np.sin(theta_4) - l_1*(l_4*m_5 + m_4*r_4)*(dtheta_5**2*l_1*m_5*r_5*np.sin(theta_5 - theta_1) + dtheta_4*(dtheta_4*l_1*l_4*m_5*np.sin(theta_4 - theta_1) + dtheta_4*l_1*m_4*r_4*np.sin(theta_4 - theta_1)) + dtheta_3*(dtheta_3*l_1*l_3*m_4*np.sin(theta_3 - theta_1) + dtheta_3*l_1*l_3*m_5*np.sin(theta_3 - theta_1) + dtheta_3*l_1*m_3*r_3*np.sin(theta_3 - theta_1)) + dtheta_2*mu_2 + dtheta_2*(dtheta_2*l_1*l_2*m_3*np.sin(theta_2 - theta_1) + dtheta_2*l_1*l_2*m_4*np.sin(theta_2 - theta_1) + dtheta_2*l_1*l_2*m_5*np.sin(theta_2 - theta_1) + dtheta_2*l_1*m_2*r_2*np.sin(theta_2 - theta_1)) - dtheta_1*(mu_1 + mu_2) - ddx_c*(-l_1*m_2 - l_1*m_3 - l_1*m_4 - l_1*m_5 - m_1*r_1)*np.cos(theta_1) - g*(-l_1*m_2 - l_1*m_3 - l_1*m_4 - l_1*m_5 - m_1*r_1)*np.sin(theta_1))*np.cos(theta_4 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) - (-l_1**2*(l_4*m_5 + m_4*r_4)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_4 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_4*m_5 + m_4*r_4)*np.cos(theta_4 - theta_2))*(dtheta_5**2*l_2*m_5*r_5*np.sin(theta_5 - theta_2) + dtheta_4*(dtheta_4*l_2*l_4*m_5*np.sin(theta_4 - theta_2) + dtheta_4*l_2*m_4*r_4*np.sin(theta_4 - theta_2)) + dtheta_3*mu_3 + dtheta_3*(dtheta_3*l_2*l_3*m_4*np.sin(theta_3 - theta_2) + dtheta_3*l_2*l_3*m_5*np.sin(theta_3 - theta_2) + dtheta_3*l_2*m_3*r_3*np.sin(theta_3 - theta_2)) - dtheta_2*(mu_2 + mu_3) + dtheta_1*mu_2 + dtheta_1*(-dtheta_1*l_1*l_2*m_3*np.sin(theta_2 - theta_1) - dtheta_1*l_1*l_2*m_4*np.sin(theta_2 - theta_1) - dtheta_1*l_1*l_2*m_5*np.sin(theta_2 - theta_1) - dtheta_1*l_1*m_2*r_2*np.sin(theta_2 - theta_1)) - ddx_c*(-l_2*m_3 - l_2*m_4 - l_2*m_5 - m_2*r_2)*np.cos(theta_2) - g*(-l_2*m_3 - l_2*m_4 - l_2*m_5 - m_2*r_2)*np.sin(theta_2) - l_1*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*(dtheta_5**2*l_1*m_5*r_5*np.sin(theta_5 - theta_1) + dtheta_4*(dtheta_4*l_1*l_4*m_5*np.sin(theta_4 - theta_1) + dtheta_4*l_1*m_4*r_4*np.sin(theta_4 - theta_1)) + dtheta_3*(dtheta_3*l_1*l_3*m_4*np.sin(theta_3 - theta_1) + dtheta_3*l_1*l_3*m_5*np.sin(theta_3 - theta_1) + dtheta_3*l_1*m_3*r_3*np.sin(theta_3 - theta_1)) + dtheta_2*mu_2 + dtheta_2*(dtheta_2*l_1*l_2*m_3*np.sin(theta_2 - theta_1) + dtheta_2*l_1*l_2*m_4*np.sin(theta_2 - theta_1) + dtheta_2*l_1*l_2*m_5*np.sin(theta_2 - theta_1) + dtheta_2*l_1*m_2*r_2*np.sin(theta_2 - theta_1)) - dtheta_1*(mu_1 + mu_2) - ddx_c*(-l_1*m_2 - l_1*m_3 - l_1*m_4 - l_1*m_5 - m_1*r_1)*np.cos(theta_1) - g*(-l_1*m_2 - l_1*m_3 - l_1*m_4 - l_1*m_5 - m_1*r_1)*np.sin(theta_1))*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2))/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2) - (-l_1**2*(l_4*m_5 + m_4*r_4)*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_4 - theta_1)*np.cos(theta_3 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_3*(l_4*m_5 + m_4*r_4)*np.cos(theta_4 - theta_3) - (-l_1**2*(l_4*m_5 + m_4*r_4)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_4 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_4*m_5 + m_4*r_4)*np.cos(theta_4 - theta_2))*(-l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_3 - theta_2))/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2))*(dtheta_5**2*l_3*m_5*r_5*np.sin(theta_5 - theta_3) + dtheta_4*mu_4 + dtheta_4*(dtheta_4*l_3*l_4*m_5*np.sin(theta_4 - theta_3) + dtheta_4*l_3*m_4*r_4*np.sin(theta_4 - theta_3)) - dtheta_3*(mu_3 + mu_4) + dtheta_2*mu_3 + dtheta_2*(-dtheta_2*l_2*l_3*m_4*np.sin(theta_3 - theta_2) - dtheta_2*l_2*l_3*m_5*np.sin(theta_3 - theta_2) - dtheta_2*l_2*m_3*r_3*np.sin(theta_3 - theta_2)) + dtheta_1*(-dtheta_1*l_1*l_3*m_4*np.sin(theta_3 - theta_1) - dtheta_1*l_1*l_3*m_5*np.sin(theta_3 - theta_1) - dtheta_1*l_1*m_3*r_3*np.sin(theta_3 - theta_1)) - ddx_c*(-l_3*m_4 - l_3*m_5 - m_3*r_3)*np.cos(theta_3) - g*(-l_3*m_4 - l_3*m_5 - m_3*r_3)*np.sin(theta_3) - l_1*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(dtheta_5**2*l_1*m_5*r_5*np.sin(theta_5 - theta_1) + dtheta_4*(dtheta_4*l_1*l_4*m_5*np.sin(theta_4 - theta_1) + dtheta_4*l_1*m_4*r_4*np.sin(theta_4 - theta_1)) + dtheta_3*(dtheta_3*l_1*l_3*m_4*np.sin(theta_3 - theta_1) + dtheta_3*l_1*l_3*m_5*np.sin(theta_3 - theta_1) + dtheta_3*l_1*m_3*r_3*np.sin(theta_3 - theta_1)) + dtheta_2*mu_2 + dtheta_2*(dtheta_2*l_1*l_2*m_3*np.sin(theta_2 - theta_1) + dtheta_2*l_1*l_2*m_4*np.sin(theta_2 - theta_1) + dtheta_2*l_1*l_2*m_5*np.sin(theta_2 - theta_1) + dtheta_2*l_1*m_2*r_2*np.sin(theta_2 - theta_1)) - dtheta_1*(mu_1 + mu_2) - ddx_c*(-l_1*m_2 - l_1*m_3 - l_1*m_4 - l_1*m_5 - m_1*r_1)*np.cos(theta_1) - g*(-l_1*m_2 - l_1*m_3 - l_1*m_4 - l_1*m_5 - m_1*r_1)*np.sin(theta_1))*np.cos(theta_3 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) - (-l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_3 - theta_2))*(dtheta_5**2*l_2*m_5*r_5*np.sin(theta_5 - theta_2) + dtheta_4*(dtheta_4*l_2*l_4*m_5*np.sin(theta_4 - theta_2) + dtheta_4*l_2*m_4*r_4*np.sin(theta_4 - theta_2)) + dtheta_3*mu_3 + dtheta_3*(dtheta_3*l_2*l_3*m_4*np.sin(theta_3 - theta_2) + dtheta_3*l_2*l_3*m_5*np.sin(theta_3 - theta_2) + dtheta_3*l_2*m_3*r_3*np.sin(theta_3 - theta_2)) - dtheta_2*(mu_2 + mu_3) + dtheta_1*mu_2 + dtheta_1*(-dtheta_1*l_1*l_2*m_3*np.sin(theta_2 - theta_1) - dtheta_1*l_1*l_2*m_4*np.sin(theta_2 - theta_1) - dtheta_1*l_1*l_2*m_5*np.sin(theta_2 - theta_1) - dtheta_1*l_1*m_2*r_2*np.sin(theta_2 - theta_1)) - ddx_c*(-l_2*m_3 - l_2*m_4 - l_2*m_5 - m_2*r_2)*np.cos(theta_2) - g*(-l_2*m_3 - l_2*m_4 - l_2*m_5 - m_2*r_2)*np.sin(theta_2) - l_1*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*(dtheta_5**2*l_1*m_5*r_5*np.sin(theta_5 - theta_1) + dtheta_4*(dtheta_4*l_1*l_4*m_5*np.sin(theta_4 - theta_1) + dtheta_4*l_1*m_4*r_4*np.sin(theta_4 - theta_1)) + dtheta_3*(dtheta_3*l_1*l_3*m_4*np.sin(theta_3 - theta_1) + dtheta_3*l_1*l_3*m_5*np.sin(theta_3 - theta_1) + dtheta_3*l_1*m_3*r_3*np.sin(theta_3 - theta_1)) + dtheta_2*mu_2 + dtheta_2*(dtheta_2*l_1*l_2*m_3*np.sin(theta_2 - theta_1) + dtheta_2*l_1*l_2*m_4*np.sin(theta_2 - theta_1) + dtheta_2*l_1*l_2*m_5*np.sin(theta_2 - theta_1) + dtheta_2*l_1*m_2*r_2*np.sin(theta_2 - theta_1)) - dtheta_1*(mu_1 + mu_2) - ddx_c*(-l_1*m_2 - l_1*m_3 - l_1*m_4 - l_1*m_5 - m_1*r_1)*np.cos(theta_1) - g*(-l_1*m_2 - l_1*m_3 - l_1*m_4 - l_1*m_5 - m_1*r_1)*np.sin(theta_1))*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2))/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2))/(I_3 - l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)**2*np.cos(theta_3 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_3**2*m_4 + l_3**2*m_5 + m_3*r_3**2 - (-l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_3 - theta_2))**2/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2)))/(I_4 - l_1**2*(l_4*m_5 + m_4*r_4)**2*np.cos(theta_4 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_4**2*m_5 + m_4*r_4**2 - (-l_1**2*(l_4*m_5 + m_4*r_4)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_4 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_4*m_5 + m_4*r_4)*np.cos(theta_4 - theta_2))**2/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2) - (-l_1**2*(l_4*m_5 + m_4*r_4)*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_4 - theta_1)*np.cos(theta_3 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_3*(l_4*m_5 + m_4*r_4)*np.cos(theta_4 - theta_3) - (-l_1**2*(l_4*m_5 + m_4*r_4)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_4 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_4*m_5 + m_4*r_4)*np.cos(theta_4 - theta_2))*(-l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_3 - theta_2))/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2))**2/(I_3 - l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)**2*np.cos(theta_3 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_3**2*m_4 + l_3**2*m_5 + m_3*r_3**2 - (-l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_3 - theta_2))**2/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2))))/(I_5 - l_1**2*m_5**2*r_5**2*np.cos(theta_5 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + m_5*r_5**2 - (-l_1**2*m_5*r_5*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_5 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*m_5*r_5*np.cos(theta_5 - theta_2))**2/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2) - (-l_1**2*m_5*r_5*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_5 - theta_1)*np.cos(theta_3 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_3*m_5*r_5*np.cos(theta_5 - theta_3) - (-l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_3 - theta_2))*(-l_1**2*m_5*r_5*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_5 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*m_5*r_5*np.cos(theta_5 - theta_2))/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2))**2/(I_3 - l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)**2*np.cos(theta_3 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_3**2*m_4 + l_3**2*m_5 + m_3*r_3**2 - (-l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_3 - theta_2))**2/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2)) - (-l_1**2*m_5*r_5*(l_4*m_5 + m_4*r_4)*np.cos(theta_5 - theta_1)*np.cos(theta_4 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_4*m_5*r_5*np.cos(theta_5 - theta_4) - (-l_1**2*(l_4*m_5 + m_4*r_4)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_4 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_4*m_5 + m_4*r_4)*np.cos(theta_4 - theta_2))*(-l_1**2*m_5*r_5*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_5 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*m_5*r_5*np.cos(theta_5 - theta_2))/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2) - (-l_1**2*(l_4*m_5 + m_4*r_4)*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_4 - theta_1)*np.cos(theta_3 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_3*(l_4*m_5 + m_4*r_4)*np.cos(theta_4 - theta_3) - (-l_1**2*(l_4*m_5 + m_4*r_4)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_4 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_4*m_5 + m_4*r_4)*np.cos(theta_4 - theta_2))*(-l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_3 - theta_2))/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2))*(-l_1**2*m_5*r_5*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_5 - theta_1)*np.cos(theta_3 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_3*m_5*r_5*np.cos(theta_5 - theta_3) - (-l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_3 - theta_2))*(-l_1**2*m_5*r_5*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_5 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*m_5*r_5*np.cos(theta_5 - theta_2))/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2))/(I_3 - l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)**2*np.cos(theta_3 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_3**2*m_4 + l_3**2*m_5 + m_3*r_3**2 - (-l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_3 - theta_2))**2/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2)))**2/(I_4 - l_1**2*(l_4*m_5 + m_4*r_4)**2*np.cos(theta_4 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_4**2*m_5 + m_4*r_4**2 - (-l_1**2*(l_4*m_5 + m_4*r_4)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_4 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_4*m_5 + m_4*r_4)*np.cos(theta_4 - theta_2))**2/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2) - (-l_1**2*(l_4*m_5 + m_4*r_4)*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_4 - theta_1)*np.cos(theta_3 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_3*(l_4*m_5 + m_4*r_4)*np.cos(theta_4 - theta_3) - (-l_1**2*(l_4*m_5 + m_4*r_4)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_4 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_4*m_5 + m_4*r_4)*np.cos(theta_4 - theta_2))*(-l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_3 - theta_2))/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2))**2/(I_3 - l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)**2*np.cos(theta_3 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_3**2*m_4 + l_3**2*m_5 + m_3*r_3**2 - (-l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_3 - theta_2))**2/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2)))))/(I_4 - l_1**2*(l_4*m_5 + m_4*r_4)**2*np.cos(theta_4 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_4**2*m_5 + m_4*r_4**2 - (-l_1**2*(l_4*m_5 + m_4*r_4)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_4 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_4*m_5 + m_4*r_4)*np.cos(theta_4 - theta_2))**2/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2) - (-l_1**2*(l_4*m_5 + m_4*r_4)*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_4 - theta_1)*np.cos(theta_3 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_3*(l_4*m_5 + m_4*r_4)*np.cos(theta_4 - theta_3) - (-l_1**2*(l_4*m_5 + m_4*r_4)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_4 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_4*m_5 + m_4*r_4)*np.cos(theta_4 - theta_2))*(-l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_3 - theta_2))/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2))**2/(I_3 - l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)**2*np.cos(theta_3 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_3**2*m_4 + l_3**2*m_5 + m_3*r_3**2 - (-l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_3 - theta_2))**2/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2))) - (-l_1**2*m_5*r_5*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_5 - theta_1)*np.cos(theta_3 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_3*m_5*r_5*np.cos(theta_5 - theta_3) - (-l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_3 - theta_2))*(-l_1**2*m_5*r_5*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_5 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*m_5*r_5*np.cos(theta_5 - theta_2))/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2))*(-dtheta_5*mu_5 - dtheta_4**2*l_4*m_5*r_5*np.sin(theta_5 - theta_4) + dtheta_4*mu_5 - dtheta_3**2*l_3*m_5*r_5*np.sin(theta_5 - theta_3) - dtheta_2**2*l_2*m_5*r_5*np.sin(theta_5 - theta_2) - dtheta_1**2*l_1*m_5*r_5*np.sin(theta_5 - theta_1) + ddx_c*m_5*r_5*np.cos(theta_5) + g*m_5*r_5*np.sin(theta_5) - l_1*m_5*r_5*(dtheta_5**2*l_1*m_5*r_5*np.sin(theta_5 - theta_1) + dtheta_4*(dtheta_4*l_1*l_4*m_5*np.sin(theta_4 - theta_1) + dtheta_4*l_1*m_4*r_4*np.sin(theta_4 - theta_1)) + dtheta_3*(dtheta_3*l_1*l_3*m_4*np.sin(theta_3 - theta_1) + dtheta_3*l_1*l_3*m_5*np.sin(theta_3 - theta_1) + dtheta_3*l_1*m_3*r_3*np.sin(theta_3 - theta_1)) + dtheta_2*mu_2 + dtheta_2*(dtheta_2*l_1*l_2*m_3*np.sin(theta_2 - theta_1) + dtheta_2*l_1*l_2*m_4*np.sin(theta_2 - theta_1) + dtheta_2*l_1*l_2*m_5*np.sin(theta_2 - theta_1) + dtheta_2*l_1*m_2*r_2*np.sin(theta_2 - theta_1)) - dtheta_1*(mu_1 + mu_2) - ddx_c*(-l_1*m_2 - l_1*m_3 - l_1*m_4 - l_1*m_5 - m_1*r_1)*np.cos(theta_1) - g*(-l_1*m_2 - l_1*m_3 - l_1*m_4 - l_1*m_5 - m_1*r_1)*np.sin(theta_1))*np.cos(theta_5 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) - (-l_1**2*m_5*r_5*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_5 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*m_5*r_5*np.cos(theta_5 - theta_2))*(dtheta_5**2*l_2*m_5*r_5*np.sin(theta_5 - theta_2) + dtheta_4*(dtheta_4*l_2*l_4*m_5*np.sin(theta_4 - theta_2) + dtheta_4*l_2*m_4*r_4*np.sin(theta_4 - theta_2)) + dtheta_3*mu_3 + dtheta_3*(dtheta_3*l_2*l_3*m_4*np.sin(theta_3 - theta_2) + dtheta_3*l_2*l_3*m_5*np.sin(theta_3 - theta_2) + dtheta_3*l_2*m_3*r_3*np.sin(theta_3 - theta_2)) - dtheta_2*(mu_2 + mu_3) + dtheta_1*mu_2 + dtheta_1*(-dtheta_1*l_1*l_2*m_3*np.sin(theta_2 - theta_1) - dtheta_1*l_1*l_2*m_4*np.sin(theta_2 - theta_1) - dtheta_1*l_1*l_2*m_5*np.sin(theta_2 - theta_1) - dtheta_1*l_1*m_2*r_2*np.sin(theta_2 - theta_1)) - ddx_c*(-l_2*m_3 - l_2*m_4 - l_2*m_5 - m_2*r_2)*np.cos(theta_2) - g*(-l_2*m_3 - l_2*m_4 - l_2*m_5 - m_2*r_2)*np.sin(theta_2) - l_1*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*(dtheta_5**2*l_1*m_5*r_5*np.sin(theta_5 - theta_1) + dtheta_4*(dtheta_4*l_1*l_4*m_5*np.sin(theta_4 - theta_1) + dtheta_4*l_1*m_4*r_4*np.sin(theta_4 - theta_1)) + dtheta_3*(dtheta_3*l_1*l_3*m_4*np.sin(theta_3 - theta_1) + dtheta_3*l_1*l_3*m_5*np.sin(theta_3 - theta_1) + dtheta_3*l_1*m_3*r_3*np.sin(theta_3 - theta_1)) + dtheta_2*mu_2 + dtheta_2*(dtheta_2*l_1*l_2*m_3*np.sin(theta_2 - theta_1) + dtheta_2*l_1*l_2*m_4*np.sin(theta_2 - theta_1) + dtheta_2*l_1*l_2*m_5*np.sin(theta_2 - theta_1) + dtheta_2*l_1*m_2*r_2*np.sin(theta_2 - theta_1)) - dtheta_1*(mu_1 + mu_2) - ddx_c*(-l_1*m_2 - l_1*m_3 - l_1*m_4 - l_1*m_5 - m_1*r_1)*np.cos(theta_1) - g*(-l_1*m_2 - l_1*m_3 - l_1*m_4 - l_1*m_5 - m_1*r_1)*np.sin(theta_1))*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2))/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2) - (-l_1**2*m_5*r_5*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_5 - theta_1)*np.cos(theta_3 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_3*m_5*r_5*np.cos(theta_5 - theta_3) - (-l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_3 - theta_2))*(-l_1**2*m_5*r_5*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_5 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*m_5*r_5*np.cos(theta_5 - theta_2))/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2))*(dtheta_5**2*l_3*m_5*r_5*np.sin(theta_5 - theta_3) + dtheta_4*mu_4 + dtheta_4*(dtheta_4*l_3*l_4*m_5*np.sin(theta_4 - theta_3) + dtheta_4*l_3*m_4*r_4*np.sin(theta_4 - theta_3)) - dtheta_3*(mu_3 + mu_4) + dtheta_2*mu_3 + dtheta_2*(-dtheta_2*l_2*l_3*m_4*np.sin(theta_3 - theta_2) - dtheta_2*l_2*l_3*m_5*np.sin(theta_3 - theta_2) - dtheta_2*l_2*m_3*r_3*np.sin(theta_3 - theta_2)) + dtheta_1*(-dtheta_1*l_1*l_3*m_4*np.sin(theta_3 - theta_1) - dtheta_1*l_1*l_3*m_5*np.sin(theta_3 - theta_1) - dtheta_1*l_1*m_3*r_3*np.sin(theta_3 - theta_1)) - ddx_c*(-l_3*m_4 - l_3*m_5 - m_3*r_3)*np.cos(theta_3) - g*(-l_3*m_4 - l_3*m_5 - m_3*r_3)*np.sin(theta_3) - l_1*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(dtheta_5**2*l_1*m_5*r_5*np.sin(theta_5 - theta_1) + dtheta_4*(dtheta_4*l_1*l_4*m_5*np.sin(theta_4 - theta_1) + dtheta_4*l_1*m_4*r_4*np.sin(theta_4 - theta_1)) + dtheta_3*(dtheta_3*l_1*l_3*m_4*np.sin(theta_3 - theta_1) + dtheta_3*l_1*l_3*m_5*np.sin(theta_3 - theta_1) + dtheta_3*l_1*m_3*r_3*np.sin(theta_3 - theta_1)) + dtheta_2*mu_2 + dtheta_2*(dtheta_2*l_1*l_2*m_3*np.sin(theta_2 - theta_1) + dtheta_2*l_1*l_2*m_4*np.sin(theta_2 - theta_1) + dtheta_2*l_1*l_2*m_5*np.sin(theta_2 - theta_1) + dtheta_2*l_1*m_2*r_2*np.sin(theta_2 - theta_1)) - dtheta_1*(mu_1 + mu_2) - ddx_c*(-l_1*m_2 - l_1*m_3 - l_1*m_4 - l_1*m_5 - m_1*r_1)*np.cos(theta_1) - g*(-l_1*m_2 - l_1*m_3 - l_1*m_4 - l_1*m_5 - m_1*r_1)*np.sin(theta_1))*np.cos(theta_3 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) - (-l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_3 - theta_2))*(dtheta_5**2*l_2*m_5*r_5*np.sin(theta_5 - theta_2) + dtheta_4*(dtheta_4*l_2*l_4*m_5*np.sin(theta_4 - theta_2) + dtheta_4*l_2*m_4*r_4*np.sin(theta_4 - theta_2)) + dtheta_3*mu_3 + dtheta_3*(dtheta_3*l_2*l_3*m_4*np.sin(theta_3 - theta_2) + dtheta_3*l_2*l_3*m_5*np.sin(theta_3 - theta_2) + dtheta_3*l_2*m_3*r_3*np.sin(theta_3 - theta_2)) - dtheta_2*(mu_2 + mu_3) + dtheta_1*mu_2 + dtheta_1*(-dtheta_1*l_1*l_2*m_3*np.sin(theta_2 - theta_1) - dtheta_1*l_1*l_2*m_4*np.sin(theta_2 - theta_1) - dtheta_1*l_1*l_2*m_5*np.sin(theta_2 - theta_1) - dtheta_1*l_1*m_2*r_2*np.sin(theta_2 - theta_1)) - ddx_c*(-l_2*m_3 - l_2*m_4 - l_2*m_5 - m_2*r_2)*np.cos(theta_2) - g*(-l_2*m_3 - l_2*m_4 - l_2*m_5 - m_2*r_2)*np.sin(theta_2) - l_1*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*(dtheta_5**2*l_1*m_5*r_5*np.sin(theta_5 - theta_1) + dtheta_4*(dtheta_4*l_1*l_4*m_5*np.sin(theta_4 - theta_1) + dtheta_4*l_1*m_4*r_4*np.sin(theta_4 - theta_1)) + dtheta_3*(dtheta_3*l_1*l_3*m_4*np.sin(theta_3 - theta_1) + dtheta_3*l_1*l_3*m_5*np.sin(theta_3 - theta_1) + dtheta_3*l_1*m_3*r_3*np.sin(theta_3 - theta_1)) + dtheta_2*mu_2 + dtheta_2*(dtheta_2*l_1*l_2*m_3*np.sin(theta_2 - theta_1) + dtheta_2*l_1*l_2*m_4*np.sin(theta_2 - theta_1) + dtheta_2*l_1*l_2*m_5*np.sin(theta_2 - theta_1) + dtheta_2*l_1*m_2*r_2*np.sin(theta_2 - theta_1)) - dtheta_1*(mu_1 + mu_2) - ddx_c*(-l_1*m_2 - l_1*m_3 - l_1*m_4 - l_1*m_5 - m_1*r_1)*np.cos(theta_1) - g*(-l_1*m_2 - l_1*m_3 - l_1*m_4 - l_1*m_5 - m_1*r_1)*np.sin(theta_1))*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2))/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2))/(I_3 - l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)**2*np.cos(theta_3 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_3**2*m_4 + l_3**2*m_5 + m_3*r_3**2 - (-l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_3 - theta_2))**2/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2)) - (-l_1**2*m_5*r_5*(l_4*m_5 + m_4*r_4)*np.cos(theta_5 - theta_1)*np.cos(theta_4 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_4*m_5*r_5*np.cos(theta_5 - theta_4) - (-l_1**2*(l_4*m_5 + m_4*r_4)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_4 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_4*m_5 + m_4*r_4)*np.cos(theta_4 - theta_2))*(-l_1**2*m_5*r_5*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_5 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*m_5*r_5*np.cos(theta_5 - theta_2))/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2) - (-l_1**2*(l_4*m_5 + m_4*r_4)*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_4 - theta_1)*np.cos(theta_3 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_3*(l_4*m_5 + m_4*r_4)*np.cos(theta_4 - theta_3) - (-l_1**2*(l_4*m_5 + m_4*r_4)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_4 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_4*m_5 + m_4*r_4)*np.cos(theta_4 - theta_2))*(-l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_3 - theta_2))/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2))*(-l_1**2*m_5*r_5*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_5 - theta_1)*np.cos(theta_3 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_3*m_5*r_5*np.cos(theta_5 - theta_3) - (-l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_3 - theta_2))*(-l_1**2*m_5*r_5*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_5 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*m_5*r_5*np.cos(theta_5 - theta_2))/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2))/(I_3 - l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)**2*np.cos(theta_3 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_3**2*m_4 + l_3**2*m_5 + m_3*r_3**2 - (-l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_3 - theta_2))**2/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2)))*(dtheta_5**2*l_4*m_5*r_5*np.sin(theta_5 - theta_4) + dtheta_5*mu_5 - dtheta_4*(mu_4 + mu_5) + dtheta_3*mu_4 + dtheta_3*(-dtheta_3*l_3*l_4*m_5*np.sin(theta_4 - theta_3) - dtheta_3*l_3*m_4*r_4*np.sin(theta_4 - theta_3)) + dtheta_2*(-dtheta_2*l_2*l_4*m_5*np.sin(theta_4 - theta_2) - dtheta_2*l_2*m_4*r_4*np.sin(theta_4 - theta_2)) + dtheta_1*(-dtheta_1*l_1*l_4*m_5*np.sin(theta_4 - theta_1) - dtheta_1*l_1*m_4*r_4*np.sin(theta_4 - theta_1)) - ddx_c*(-l_4*m_5 - m_4*r_4)*np.cos(theta_4) - g*(-l_4*m_5 - m_4*r_4)*np.sin(theta_4) - l_1*(l_4*m_5 + m_4*r_4)*(dtheta_5**2*l_1*m_5*r_5*np.sin(theta_5 - theta_1) + dtheta_4*(dtheta_4*l_1*l_4*m_5*np.sin(theta_4 - theta_1) + dtheta_4*l_1*m_4*r_4*np.sin(theta_4 - theta_1)) + dtheta_3*(dtheta_3*l_1*l_3*m_4*np.sin(theta_3 - theta_1) + dtheta_3*l_1*l_3*m_5*np.sin(theta_3 - theta_1) + dtheta_3*l_1*m_3*r_3*np.sin(theta_3 - theta_1)) + dtheta_2*mu_2 + dtheta_2*(dtheta_2*l_1*l_2*m_3*np.sin(theta_2 - theta_1) + dtheta_2*l_1*l_2*m_4*np.sin(theta_2 - theta_1) + dtheta_2*l_1*l_2*m_5*np.sin(theta_2 - theta_1) + dtheta_2*l_1*m_2*r_2*np.sin(theta_2 - theta_1)) - dtheta_1*(mu_1 + mu_2) - ddx_c*(-l_1*m_2 - l_1*m_3 - l_1*m_4 - l_1*m_5 - m_1*r_1)*np.cos(theta_1) - g*(-l_1*m_2 - l_1*m_3 - l_1*m_4 - l_1*m_5 - m_1*r_1)*np.sin(theta_1))*np.cos(theta_4 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) - (-l_1**2*(l_4*m_5 + m_4*r_4)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_4 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_4*m_5 + m_4*r_4)*np.cos(theta_4 - theta_2))*(dtheta_5**2*l_2*m_5*r_5*np.sin(theta_5 - theta_2) + dtheta_4*(dtheta_4*l_2*l_4*m_5*np.sin(theta_4 - theta_2) + dtheta_4*l_2*m_4*r_4*np.sin(theta_4 - theta_2)) + dtheta_3*mu_3 + dtheta_3*(dtheta_3*l_2*l_3*m_4*np.sin(theta_3 - theta_2) + dtheta_3*l_2*l_3*m_5*np.sin(theta_3 - theta_2) + dtheta_3*l_2*m_3*r_3*np.sin(theta_3 - theta_2)) - dtheta_2*(mu_2 + mu_3) + dtheta_1*mu_2 + dtheta_1*(-dtheta_1*l_1*l_2*m_3*np.sin(theta_2 - theta_1) - dtheta_1*l_1*l_2*m_4*np.sin(theta_2 - theta_1) - dtheta_1*l_1*l_2*m_5*np.sin(theta_2 - theta_1) - dtheta_1*l_1*m_2*r_2*np.sin(theta_2 - theta_1)) - ddx_c*(-l_2*m_3 - l_2*m_4 - l_2*m_5 - m_2*r_2)*np.cos(theta_2) - g*(-l_2*m_3 - l_2*m_4 - l_2*m_5 - m_2*r_2)*np.sin(theta_2) - l_1*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*(dtheta_5**2*l_1*m_5*r_5*np.sin(theta_5 - theta_1) + dtheta_4*(dtheta_4*l_1*l_4*m_5*np.sin(theta_4 - theta_1) + dtheta_4*l_1*m_4*r_4*np.sin(theta_4 - theta_1)) + dtheta_3*(dtheta_3*l_1*l_3*m_4*np.sin(theta_3 - theta_1) + dtheta_3*l_1*l_3*m_5*np.sin(theta_3 - theta_1) + dtheta_3*l_1*m_3*r_3*np.sin(theta_3 - theta_1)) + dtheta_2*mu_2 + dtheta_2*(dtheta_2*l_1*l_2*m_3*np.sin(theta_2 - theta_1) + dtheta_2*l_1*l_2*m_4*np.sin(theta_2 - theta_1) + dtheta_2*l_1*l_2*m_5*np.sin(theta_2 - theta_1) + dtheta_2*l_1*m_2*r_2*np.sin(theta_2 - theta_1)) - dtheta_1*(mu_1 + mu_2) - ddx_c*(-l_1*m_2 - l_1*m_3 - l_1*m_4 - l_1*m_5 - m_1*r_1)*np.cos(theta_1) - g*(-l_1*m_2 - l_1*m_3 - l_1*m_4 - l_1*m_5 - m_1*r_1)*np.sin(theta_1))*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2))/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2) - (-l_1**2*(l_4*m_5 + m_4*r_4)*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_4 - theta_1)*np.cos(theta_3 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_3*(l_4*m_5 + m_4*r_4)*np.cos(theta_4 - theta_3) - (-l_1**2*(l_4*m_5 + m_4*r_4)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_4 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_4*m_5 + m_4*r_4)*np.cos(theta_4 - theta_2))*(-l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_3 - theta_2))/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2))*(dtheta_5**2*l_3*m_5*r_5*np.sin(theta_5 - theta_3) + dtheta_4*mu_4 + dtheta_4*(dtheta_4*l_3*l_4*m_5*np.sin(theta_4 - theta_3) + dtheta_4*l_3*m_4*r_4*np.sin(theta_4 - theta_3)) - dtheta_3*(mu_3 + mu_4) + dtheta_2*mu_3 + dtheta_2*(-dtheta_2*l_2*l_3*m_4*np.sin(theta_3 - theta_2) - dtheta_2*l_2*l_3*m_5*np.sin(theta_3 - theta_2) - dtheta_2*l_2*m_3*r_3*np.sin(theta_3 - theta_2)) + dtheta_1*(-dtheta_1*l_1*l_3*m_4*np.sin(theta_3 - theta_1) - dtheta_1*l_1*l_3*m_5*np.sin(theta_3 - theta_1) - dtheta_1*l_1*m_3*r_3*np.sin(theta_3 - theta_1)) - ddx_c*(-l_3*m_4 - l_3*m_5 - m_3*r_3)*np.cos(theta_3) - g*(-l_3*m_4 - l_3*m_5 - m_3*r_3)*np.sin(theta_3) - l_1*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(dtheta_5**2*l_1*m_5*r_5*np.sin(theta_5 - theta_1) + dtheta_4*(dtheta_4*l_1*l_4*m_5*np.sin(theta_4 - theta_1) + dtheta_4*l_1*m_4*r_4*np.sin(theta_4 - theta_1)) + dtheta_3*(dtheta_3*l_1*l_3*m_4*np.sin(theta_3 - theta_1) + dtheta_3*l_1*l_3*m_5*np.sin(theta_3 - theta_1) + dtheta_3*l_1*m_3*r_3*np.sin(theta_3 - theta_1)) + dtheta_2*mu_2 + dtheta_2*(dtheta_2*l_1*l_2*m_3*np.sin(theta_2 - theta_1) + dtheta_2*l_1*l_2*m_4*np.sin(theta_2 - theta_1) + dtheta_2*l_1*l_2*m_5*np.sin(theta_2 - theta_1) + dtheta_2*l_1*m_2*r_2*np.sin(theta_2 - theta_1)) - dtheta_1*(mu_1 + mu_2) - ddx_c*(-l_1*m_2 - l_1*m_3 - l_1*m_4 - l_1*m_5 - m_1*r_1)*np.cos(theta_1) - g*(-l_1*m_2 - l_1*m_3 - l_1*m_4 - l_1*m_5 - m_1*r_1)*np.sin(theta_1))*np.cos(theta_3 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) - (-l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_3 - theta_2))*(dtheta_5**2*l_2*m_5*r_5*np.sin(theta_5 - theta_2) + dtheta_4*(dtheta_4*l_2*l_4*m_5*np.sin(theta_4 - theta_2) + dtheta_4*l_2*m_4*r_4*np.sin(theta_4 - theta_2)) + dtheta_3*mu_3 + dtheta_3*(dtheta_3*l_2*l_3*m_4*np.sin(theta_3 - theta_2) + dtheta_3*l_2*l_3*m_5*np.sin(theta_3 - theta_2) + dtheta_3*l_2*m_3*r_3*np.sin(theta_3 - theta_2)) - dtheta_2*(mu_2 + mu_3) + dtheta_1*mu_2 + dtheta_1*(-dtheta_1*l_1*l_2*m_3*np.sin(theta_2 - theta_1) - dtheta_1*l_1*l_2*m_4*np.sin(theta_2 - theta_1) - dtheta_1*l_1*l_2*m_5*np.sin(theta_2 - theta_1) - dtheta_1*l_1*m_2*r_2*np.sin(theta_2 - theta_1)) - ddx_c*(-l_2*m_3 - l_2*m_4 - l_2*m_5 - m_2*r_2)*np.cos(theta_2) - g*(-l_2*m_3 - l_2*m_4 - l_2*m_5 - m_2*r_2)*np.sin(theta_2) - l_1*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*(dtheta_5**2*l_1*m_5*r_5*np.sin(theta_5 - theta_1) + dtheta_4*(dtheta_4*l_1*l_4*m_5*np.sin(theta_4 - theta_1) + dtheta_4*l_1*m_4*r_4*np.sin(theta_4 - theta_1)) + dtheta_3*(dtheta_3*l_1*l_3*m_4*np.sin(theta_3 - theta_1) + dtheta_3*l_1*l_3*m_5*np.sin(theta_3 - theta_1) + dtheta_3*l_1*m_3*r_3*np.sin(theta_3 - theta_1)) + dtheta_2*mu_2 + dtheta_2*(dtheta_2*l_1*l_2*m_3*np.sin(theta_2 - theta_1) + dtheta_2*l_1*l_2*m_4*np.sin(theta_2 - theta_1) + dtheta_2*l_1*l_2*m_5*np.sin(theta_2 - theta_1) + dtheta_2*l_1*m_2*r_2*np.sin(theta_2 - theta_1)) - dtheta_1*(mu_1 + mu_2) - ddx_c*(-l_1*m_2 - l_1*m_3 - l_1*m_4 - l_1*m_5 - m_1*r_1)*np.cos(theta_1) - g*(-l_1*m_2 - l_1*m_3 - l_1*m_4 - l_1*m_5 - m_1*r_1)*np.sin(theta_1))*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2))/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2))/(I_3 - l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)**2*np.cos(theta_3 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_3**2*m_4 + l_3**2*m_5 + m_3*r_3**2 - (-l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_3 - theta_2))**2/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2)))/(I_4 - l_1**2*(l_4*m_5 + m_4*r_4)**2*np.cos(theta_4 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_4**2*m_5 + m_4*r_4**2 - (-l_1**2*(l_4*m_5 + m_4*r_4)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_4 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_4*m_5 + m_4*r_4)*np.cos(theta_4 - theta_2))**2/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2) - (-l_1**2*(l_4*m_5 + m_4*r_4)*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_4 - theta_1)*np.cos(theta_3 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_3*(l_4*m_5 + m_4*r_4)*np.cos(theta_4 - theta_3) - (-l_1**2*(l_4*m_5 + m_4*r_4)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_4 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_4*m_5 + m_4*r_4)*np.cos(theta_4 - theta_2))*(-l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_3 - theta_2))/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2))**2/(I_3 - l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)**2*np.cos(theta_3 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_3**2*m_4 + l_3**2*m_5 + m_3*r_3**2 - (-l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_3 - theta_2))**2/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2))))/(I_5 - l_1**2*m_5**2*r_5**2*np.cos(theta_5 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + m_5*r_5**2 - (-l_1**2*m_5*r_5*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_5 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*m_5*r_5*np.cos(theta_5 - theta_2))**2/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2) - (-l_1**2*m_5*r_5*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_5 - theta_1)*np.cos(theta_3 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_3*m_5*r_5*np.cos(theta_5 - theta_3) - (-l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_3 - theta_2))*(-l_1**2*m_5*r_5*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_5 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*m_5*r_5*np.cos(theta_5 - theta_2))/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2))**2/(I_3 - l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)**2*np.cos(theta_3 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_3**2*m_4 + l_3**2*m_5 + m_3*r_3**2 - (-l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_3 - theta_2))**2/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2)) - (-l_1**2*m_5*r_5*(l_4*m_5 + m_4*r_4)*np.cos(theta_5 - theta_1)*np.cos(theta_4 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_4*m_5*r_5*np.cos(theta_5 - theta_4) - (-l_1**2*(l_4*m_5 + m_4*r_4)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_4 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_4*m_5 + m_4*r_4)*np.cos(theta_4 - theta_2))*(-l_1**2*m_5*r_5*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_5 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*m_5*r_5*np.cos(theta_5 - theta_2))/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2) - (-l_1**2*(l_4*m_5 + m_4*r_4)*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_4 - theta_1)*np.cos(theta_3 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_3*(l_4*m_5 + m_4*r_4)*np.cos(theta_4 - theta_3) - (-l_1**2*(l_4*m_5 + m_4*r_4)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_4 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_4*m_5 + m_4*r_4)*np.cos(theta_4 - theta_2))*(-l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_3 - theta_2))/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2))*(-l_1**2*m_5*r_5*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_5 - theta_1)*np.cos(theta_3 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_3*m_5*r_5*np.cos(theta_5 - theta_3) - (-l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_3 - theta_2))*(-l_1**2*m_5*r_5*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_5 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*m_5*r_5*np.cos(theta_5 - theta_2))/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2))/(I_3 - l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)**2*np.cos(theta_3 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_3**2*m_4 + l_3**2*m_5 + m_3*r_3**2 - (-l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_3 - theta_2))**2/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2)))**2/(I_4 - l_1**2*(l_4*m_5 + m_4*r_4)**2*np.cos(theta_4 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_4**2*m_5 + m_4*r_4**2 - (-l_1**2*(l_4*m_5 + m_4*r_4)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_4 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_4*m_5 + m_4*r_4)*np.cos(theta_4 - theta_2))**2/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2) - (-l_1**2*(l_4*m_5 + m_4*r_4)*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_4 - theta_1)*np.cos(theta_3 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_3*(l_4*m_5 + m_4*r_4)*np.cos(theta_4 - theta_3) - (-l_1**2*(l_4*m_5 + m_4*r_4)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_4 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_4*m_5 + m_4*r_4)*np.cos(theta_4 - theta_2))*(-l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_3 - theta_2))/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2))**2/(I_3 - l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)**2*np.cos(theta_3 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_3**2*m_4 + l_3**2*m_5 + m_3*r_3**2 - (-l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_3 - theta_2))**2/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2)))))*np.cos(theta_3 - theta_1)/(I_3 - l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)**2*np.cos(theta_3 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_3**2*m_4 + l_3**2*m_5 + m_3*r_3**2 - (-l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_3 - theta_2))**2/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2)) - l_1*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*(dtheta_5**2*l_2*m_5*r_5*np.sin(theta_5 - theta_2) + dtheta_4*(dtheta_4*l_2*l_4*m_5*np.sin(theta_4 - theta_2) + dtheta_4*l_2*m_4*r_4*np.sin(theta_4 - theta_2)) + dtheta_3*mu_3 + dtheta_3*(dtheta_3*l_2*l_3*m_4*np.sin(theta_3 - theta_2) + dtheta_3*l_2*l_3*m_5*np.sin(theta_3 - theta_2) + dtheta_3*l_2*m_3*r_3*np.sin(theta_3 - theta_2)) - dtheta_2*(mu_2 + mu_3) + dtheta_1*mu_2 + dtheta_1*(-dtheta_1*l_1*l_2*m_3*np.sin(theta_2 - theta_1) - dtheta_1*l_1*l_2*m_4*np.sin(theta_2 - theta_1) - dtheta_1*l_1*l_2*m_5*np.sin(theta_2 - theta_1) - dtheta_1*l_1*m_2*r_2*np.sin(theta_2 - theta_1)) - ddx_c*(-l_2*m_3 - l_2*m_4 - l_2*m_5 - m_2*r_2)*np.cos(theta_2) - g*(-l_2*m_3 - l_2*m_4 - l_2*m_5 - m_2*r_2)*np.sin(theta_2) - l_1*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*(dtheta_5**2*l_1*m_5*r_5*np.sin(theta_5 - theta_1) + dtheta_4*(dtheta_4*l_1*l_4*m_5*np.sin(theta_4 - theta_1) + dtheta_4*l_1*m_4*r_4*np.sin(theta_4 - theta_1)) + dtheta_3*(dtheta_3*l_1*l_3*m_4*np.sin(theta_3 - theta_1) + dtheta_3*l_1*l_3*m_5*np.sin(theta_3 - theta_1) + dtheta_3*l_1*m_3*r_3*np.sin(theta_3 - theta_1)) + dtheta_2*mu_2 + dtheta_2*(dtheta_2*l_1*l_2*m_3*np.sin(theta_2 - theta_1) + dtheta_2*l_1*l_2*m_4*np.sin(theta_2 - theta_1) + dtheta_2*l_1*l_2*m_5*np.sin(theta_2 - theta_1) + dtheta_2*l_1*m_2*r_2*np.sin(theta_2 - theta_1)) - dtheta_1*(mu_1 + mu_2) - ddx_c*(-l_1*m_2 - l_1*m_3 - l_1*m_4 - l_1*m_5 - m_1*r_1)*np.cos(theta_1) - g*(-l_1*m_2 - l_1*m_3 - l_1*m_4 - l_1*m_5 - m_1*r_1)*np.sin(theta_1))*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) - (-l_1**2*(l_4*m_5 + m_4*r_4)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_4 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_4*m_5 + m_4*r_4)*np.cos(theta_4 - theta_2))*(dtheta_5**2*l_4*m_5*r_5*np.sin(theta_5 - theta_4) + dtheta_5*mu_5 - dtheta_4*(mu_4 + mu_5) + dtheta_3*mu_4 + dtheta_3*(-dtheta_3*l_3*l_4*m_5*np.sin(theta_4 - theta_3) - dtheta_3*l_3*m_4*r_4*np.sin(theta_4 - theta_3)) + dtheta_2*(-dtheta_2*l_2*l_4*m_5*np.sin(theta_4 - theta_2) - dtheta_2*l_2*m_4*r_4*np.sin(theta_4 - theta_2)) + dtheta_1*(-dtheta_1*l_1*l_4*m_5*np.sin(theta_4 - theta_1) - dtheta_1*l_1*m_4*r_4*np.sin(theta_4 - theta_1)) - ddx_c*(-l_4*m_5 - m_4*r_4)*np.cos(theta_4) - g*(-l_4*m_5 - m_4*r_4)*np.sin(theta_4) - l_1*(l_4*m_5 + m_4*r_4)*(dtheta_5**2*l_1*m_5*r_5*np.sin(theta_5 - theta_1) + dtheta_4*(dtheta_4*l_1*l_4*m_5*np.sin(theta_4 - theta_1) + dtheta_4*l_1*m_4*r_4*np.sin(theta_4 - theta_1)) + dtheta_3*(dtheta_3*l_1*l_3*m_4*np.sin(theta_3 - theta_1) + dtheta_3*l_1*l_3*m_5*np.sin(theta_3 - theta_1) + dtheta_3*l_1*m_3*r_3*np.sin(theta_3 - theta_1)) + dtheta_2*mu_2 + dtheta_2*(dtheta_2*l_1*l_2*m_3*np.sin(theta_2 - theta_1) + dtheta_2*l_1*l_2*m_4*np.sin(theta_2 - theta_1) + dtheta_2*l_1*l_2*m_5*np.sin(theta_2 - theta_1) + dtheta_2*l_1*m_2*r_2*np.sin(theta_2 - theta_1)) - dtheta_1*(mu_1 + mu_2) - ddx_c*(-l_1*m_2 - l_1*m_3 - l_1*m_4 - l_1*m_5 - m_1*r_1)*np.cos(theta_1) - g*(-l_1*m_2 - l_1*m_3 - l_1*m_4 - l_1*m_5 - m_1*r_1)*np.sin(theta_1))*np.cos(theta_4 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) - (-l_1**2*(l_4*m_5 + m_4*r_4)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_4 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_4*m_5 + m_4*r_4)*np.cos(theta_4 - theta_2))*(dtheta_5**2*l_2*m_5*r_5*np.sin(theta_5 - theta_2) + dtheta_4*(dtheta_4*l_2*l_4*m_5*np.sin(theta_4 - theta_2) + dtheta_4*l_2*m_4*r_4*np.sin(theta_4 - theta_2)) + dtheta_3*mu_3 + dtheta_3*(dtheta_3*l_2*l_3*m_4*np.sin(theta_3 - theta_2) + dtheta_3*l_2*l_3*m_5*np.sin(theta_3 - theta_2) + dtheta_3*l_2*m_3*r_3*np.sin(theta_3 - theta_2)) - dtheta_2*(mu_2 + mu_3) + dtheta_1*mu_2 + dtheta_1*(-dtheta_1*l_1*l_2*m_3*np.sin(theta_2 - theta_1) - dtheta_1*l_1*l_2*m_4*np.sin(theta_2 - theta_1) - dtheta_1*l_1*l_2*m_5*np.sin(theta_2 - theta_1) - dtheta_1*l_1*m_2*r_2*np.sin(theta_2 - theta_1)) - ddx_c*(-l_2*m_3 - l_2*m_4 - l_2*m_5 - m_2*r_2)*np.cos(theta_2) - g*(-l_2*m_3 - l_2*m_4 - l_2*m_5 - m_2*r_2)*np.sin(theta_2) - l_1*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*(dtheta_5**2*l_1*m_5*r_5*np.sin(theta_5 - theta_1) + dtheta_4*(dtheta_4*l_1*l_4*m_5*np.sin(theta_4 - theta_1) + dtheta_4*l_1*m_4*r_4*np.sin(theta_4 - theta_1)) + dtheta_3*(dtheta_3*l_1*l_3*m_4*np.sin(theta_3 - theta_1) + dtheta_3*l_1*l_3*m_5*np.sin(theta_3 - theta_1) + dtheta_3*l_1*m_3*r_3*np.sin(theta_3 - theta_1)) + dtheta_2*mu_2 + dtheta_2*(dtheta_2*l_1*l_2*m_3*np.sin(theta_2 - theta_1) + dtheta_2*l_1*l_2*m_4*np.sin(theta_2 - theta_1) + dtheta_2*l_1*l_2*m_5*np.sin(theta_2 - theta_1) + dtheta_2*l_1*m_2*r_2*np.sin(theta_2 - theta_1)) - dtheta_1*(mu_1 + mu_2) - ddx_c*(-l_1*m_2 - l_1*m_3 - l_1*m_4 - l_1*m_5 - m_1*r_1)*np.cos(theta_1) - g*(-l_1*m_2 - l_1*m_3 - l_1*m_4 - l_1*m_5 - m_1*r_1)*np.sin(theta_1))*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2))/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2) - (-l_1**2*(l_4*m_5 + m_4*r_4)*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_4 - theta_1)*np.cos(theta_3 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_3*(l_4*m_5 + m_4*r_4)*np.cos(theta_4 - theta_3) - (-l_1**2*(l_4*m_5 + m_4*r_4)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_4 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_4*m_5 + m_4*r_4)*np.cos(theta_4 - theta_2))*(-l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_3 - theta_2))/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2))*(dtheta_5**2*l_3*m_5*r_5*np.sin(theta_5 - theta_3) + dtheta_4*mu_4 + dtheta_4*(dtheta_4*l_3*l_4*m_5*np.sin(theta_4 - theta_3) + dtheta_4*l_3*m_4*r_4*np.sin(theta_4 - theta_3)) - dtheta_3*(mu_3 + mu_4) + dtheta_2*mu_3 + dtheta_2*(-dtheta_2*l_2*l_3*m_4*np.sin(theta_3 - theta_2) - dtheta_2*l_2*l_3*m_5*np.sin(theta_3 - theta_2) - dtheta_2*l_2*m_3*r_3*np.sin(theta_3 - theta_2)) + dtheta_1*(-dtheta_1*l_1*l_3*m_4*np.sin(theta_3 - theta_1) - dtheta_1*l_1*l_3*m_5*np.sin(theta_3 - theta_1) - dtheta_1*l_1*m_3*r_3*np.sin(theta_3 - theta_1)) - ddx_c*(-l_3*m_4 - l_3*m_5 - m_3*r_3)*np.cos(theta_3) - g*(-l_3*m_4 - l_3*m_5 - m_3*r_3)*np.sin(theta_3) - l_1*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(dtheta_5**2*l_1*m_5*r_5*np.sin(theta_5 - theta_1) + dtheta_4*(dtheta_4*l_1*l_4*m_5*np.sin(theta_4 - theta_1) + dtheta_4*l_1*m_4*r_4*np.sin(theta_4 - theta_1)) + dtheta_3*(dtheta_3*l_1*l_3*m_4*np.sin(theta_3 - theta_1) + dtheta_3*l_1*l_3*m_5*np.sin(theta_3 - theta_1) + dtheta_3*l_1*m_3*r_3*np.sin(theta_3 - theta_1)) + dtheta_2*mu_2 + dtheta_2*(dtheta_2*l_1*l_2*m_3*np.sin(theta_2 - theta_1) + dtheta_2*l_1*l_2*m_4*np.sin(theta_2 - theta_1) + dtheta_2*l_1*l_2*m_5*np.sin(theta_2 - theta_1) + dtheta_2*l_1*m_2*r_2*np.sin(theta_2 - theta_1)) - dtheta_1*(mu_1 + mu_2) - ddx_c*(-l_1*m_2 - l_1*m_3 - l_1*m_4 - l_1*m_5 - m_1*r_1)*np.cos(theta_1) - g*(-l_1*m_2 - l_1*m_3 - l_1*m_4 - l_1*m_5 - m_1*r_1)*np.sin(theta_1))*np.cos(theta_3 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) - (-l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_3 - theta_2))*(dtheta_5**2*l_2*m_5*r_5*np.sin(theta_5 - theta_2) + dtheta_4*(dtheta_4*l_2*l_4*m_5*np.sin(theta_4 - theta_2) + dtheta_4*l_2*m_4*r_4*np.sin(theta_4 - theta_2)) + dtheta_3*mu_3 + dtheta_3*(dtheta_3*l_2*l_3*m_4*np.sin(theta_3 - theta_2) + dtheta_3*l_2*l_3*m_5*np.sin(theta_3 - theta_2) + dtheta_3*l_2*m_3*r_3*np.sin(theta_3 - theta_2)) - dtheta_2*(mu_2 + mu_3) + dtheta_1*mu_2 + dtheta_1*(-dtheta_1*l_1*l_2*m_3*np.sin(theta_2 - theta_1) - dtheta_1*l_1*l_2*m_4*np.sin(theta_2 - theta_1) - dtheta_1*l_1*l_2*m_5*np.sin(theta_2 - theta_1) - dtheta_1*l_1*m_2*r_2*np.sin(theta_2 - theta_1)) - ddx_c*(-l_2*m_3 - l_2*m_4 - l_2*m_5 - m_2*r_2)*np.cos(theta_2) - g*(-l_2*m_3 - l_2*m_4 - l_2*m_5 - m_2*r_2)*np.sin(theta_2) - l_1*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*(dtheta_5**2*l_1*m_5*r_5*np.sin(theta_5 - theta_1) + dtheta_4*(dtheta_4*l_1*l_4*m_5*np.sin(theta_4 - theta_1) + dtheta_4*l_1*m_4*r_4*np.sin(theta_4 - theta_1)) + dtheta_3*(dtheta_3*l_1*l_3*m_4*np.sin(theta_3 - theta_1) + dtheta_3*l_1*l_3*m_5*np.sin(theta_3 - theta_1) + dtheta_3*l_1*m_3*r_3*np.sin(theta_3 - theta_1)) + dtheta_2*mu_2 + dtheta_2*(dtheta_2*l_1*l_2*m_3*np.sin(theta_2 - theta_1) + dtheta_2*l_1*l_2*m_4*np.sin(theta_2 - theta_1) + dtheta_2*l_1*l_2*m_5*np.sin(theta_2 - theta_1) + dtheta_2*l_1*m_2*r_2*np.sin(theta_2 - theta_1)) - dtheta_1*(mu_1 + mu_2) - ddx_c*(-l_1*m_2 - l_1*m_3 - l_1*m_4 - l_1*m_5 - m_1*r_1)*np.cos(theta_1) - g*(-l_1*m_2 - l_1*m_3 - l_1*m_4 - l_1*m_5 - m_1*r_1)*np.sin(theta_1))*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2))/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2))/(I_3 - l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)**2*np.cos(theta_3 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_3**2*m_4 + l_3**2*m_5 + m_3*r_3**2 - (-l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_3 - theta_2))**2/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2)) - (-l_1**2*m_5*r_5*(l_4*m_5 + m_4*r_4)*np.cos(theta_5 - theta_1)*np.cos(theta_4 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_4*m_5*r_5*np.cos(theta_5 - theta_4) - (-l_1**2*(l_4*m_5 + m_4*r_4)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_4 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_4*m_5 + m_4*r_4)*np.cos(theta_4 - theta_2))*(-l_1**2*m_5*r_5*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_5 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*m_5*r_5*np.cos(theta_5 - theta_2))/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2) - (-l_1**2*(l_4*m_5 + m_4*r_4)*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_4 - theta_1)*np.cos(theta_3 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_3*(l_4*m_5 + m_4*r_4)*np.cos(theta_4 - theta_3) - (-l_1**2*(l_4*m_5 + m_4*r_4)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_4 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_4*m_5 + m_4*r_4)*np.cos(theta_4 - theta_2))*(-l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_3 - theta_2))/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2))*(-l_1**2*m_5*r_5*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_5 - theta_1)*np.cos(theta_3 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_3*m_5*r_5*np.cos(theta_5 - theta_3) - (-l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_3 - theta_2))*(-l_1**2*m_5*r_5*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_5 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*m_5*r_5*np.cos(theta_5 - theta_2))/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2))/(I_3 - l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)**2*np.cos(theta_3 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_3**2*m_4 + l_3**2*m_5 + m_3*r_3**2 - (-l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_3 - theta_2))**2/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2)))*(-dtheta_5*mu_5 - dtheta_4**2*l_4*m_5*r_5*np.sin(theta_5 - theta_4) + dtheta_4*mu_5 - dtheta_3**2*l_3*m_5*r_5*np.sin(theta_5 - theta_3) - dtheta_2**2*l_2*m_5*r_5*np.sin(theta_5 - theta_2) - dtheta_1**2*l_1*m_5*r_5*np.sin(theta_5 - theta_1) + ddx_c*m_5*r_5*np.cos(theta_5) + g*m_5*r_5*np.sin(theta_5) - l_1*m_5*r_5*(dtheta_5**2*l_1*m_5*r_5*np.sin(theta_5 - theta_1) + dtheta_4*(dtheta_4*l_1*l_4*m_5*np.sin(theta_4 - theta_1) + dtheta_4*l_1*m_4*r_4*np.sin(theta_4 - theta_1)) + dtheta_3*(dtheta_3*l_1*l_3*m_4*np.sin(theta_3 - theta_1) + dtheta_3*l_1*l_3*m_5*np.sin(theta_3 - theta_1) + dtheta_3*l_1*m_3*r_3*np.sin(theta_3 - theta_1)) + dtheta_2*mu_2 + dtheta_2*(dtheta_2*l_1*l_2*m_3*np.sin(theta_2 - theta_1) + dtheta_2*l_1*l_2*m_4*np.sin(theta_2 - theta_1) + dtheta_2*l_1*l_2*m_5*np.sin(theta_2 - theta_1) + dtheta_2*l_1*m_2*r_2*np.sin(theta_2 - theta_1)) - dtheta_1*(mu_1 + mu_2) - ddx_c*(-l_1*m_2 - l_1*m_3 - l_1*m_4 - l_1*m_5 - m_1*r_1)*np.cos(theta_1) - g*(-l_1*m_2 - l_1*m_3 - l_1*m_4 - l_1*m_5 - m_1*r_1)*np.sin(theta_1))*np.cos(theta_5 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) - (-l_1**2*m_5*r_5*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_5 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*m_5*r_5*np.cos(theta_5 - theta_2))*(dtheta_5**2*l_2*m_5*r_5*np.sin(theta_5 - theta_2) + dtheta_4*(dtheta_4*l_2*l_4*m_5*np.sin(theta_4 - theta_2) + dtheta_4*l_2*m_4*r_4*np.sin(theta_4 - theta_2)) + dtheta_3*mu_3 + dtheta_3*(dtheta_3*l_2*l_3*m_4*np.sin(theta_3 - theta_2) + dtheta_3*l_2*l_3*m_5*np.sin(theta_3 - theta_2) + dtheta_3*l_2*m_3*r_3*np.sin(theta_3 - theta_2)) - dtheta_2*(mu_2 + mu_3) + dtheta_1*mu_2 + dtheta_1*(-dtheta_1*l_1*l_2*m_3*np.sin(theta_2 - theta_1) - dtheta_1*l_1*l_2*m_4*np.sin(theta_2 - theta_1) - dtheta_1*l_1*l_2*m_5*np.sin(theta_2 - theta_1) - dtheta_1*l_1*m_2*r_2*np.sin(theta_2 - theta_1)) - ddx_c*(-l_2*m_3 - l_2*m_4 - l_2*m_5 - m_2*r_2)*np.cos(theta_2) - g*(-l_2*m_3 - l_2*m_4 - l_2*m_5 - m_2*r_2)*np.sin(theta_2) - l_1*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*(dtheta_5**2*l_1*m_5*r_5*np.sin(theta_5 - theta_1) + dtheta_4*(dtheta_4*l_1*l_4*m_5*np.sin(theta_4 - theta_1) + dtheta_4*l_1*m_4*r_4*np.sin(theta_4 - theta_1)) + dtheta_3*(dtheta_3*l_1*l_3*m_4*np.sin(theta_3 - theta_1) + dtheta_3*l_1*l_3*m_5*np.sin(theta_3 - theta_1) + dtheta_3*l_1*m_3*r_3*np.sin(theta_3 - theta_1)) + dtheta_2*mu_2 + dtheta_2*(dtheta_2*l_1*l_2*m_3*np.sin(theta_2 - theta_1) + dtheta_2*l_1*l_2*m_4*np.sin(theta_2 - theta_1) + dtheta_2*l_1*l_2*m_5*np.sin(theta_2 - theta_1) + dtheta_2*l_1*m_2*r_2*np.sin(theta_2 - theta_1)) - dtheta_1*(mu_1 + mu_2) - ddx_c*(-l_1*m_2 - l_1*m_3 - l_1*m_4 - l_1*m_5 - m_1*r_1)*np.cos(theta_1) - g*(-l_1*m_2 - l_1*m_3 - l_1*m_4 - l_1*m_5 - m_1*r_1)*np.sin(theta_1))*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2))/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2) - (-l_1**2*m_5*r_5*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_5 - theta_1)*np.cos(theta_3 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_3*m_5*r_5*np.cos(theta_5 - theta_3) - (-l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_3 - theta_2))*(-l_1**2*m_5*r_5*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_5 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*m_5*r_5*np.cos(theta_5 - theta_2))/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2))*(dtheta_5**2*l_3*m_5*r_5*np.sin(theta_5 - theta_3) + dtheta_4*mu_4 + dtheta_4*(dtheta_4*l_3*l_4*m_5*np.sin(theta_4 - theta_3) + dtheta_4*l_3*m_4*r_4*np.sin(theta_4 - theta_3)) - dtheta_3*(mu_3 + mu_4) + dtheta_2*mu_3 + dtheta_2*(-dtheta_2*l_2*l_3*m_4*np.sin(theta_3 - theta_2) - dtheta_2*l_2*l_3*m_5*np.sin(theta_3 - theta_2) - dtheta_2*l_2*m_3*r_3*np.sin(theta_3 - theta_2)) + dtheta_1*(-dtheta_1*l_1*l_3*m_4*np.sin(theta_3 - theta_1) - dtheta_1*l_1*l_3*m_5*np.sin(theta_3 - theta_1) - dtheta_1*l_1*m_3*r_3*np.sin(theta_3 - theta_1)) - ddx_c*(-l_3*m_4 - l_3*m_5 - m_3*r_3)*np.cos(theta_3) - g*(-l_3*m_4 - l_3*m_5 - m_3*r_3)*np.sin(theta_3) - l_1*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(dtheta_5**2*l_1*m_5*r_5*np.sin(theta_5 - theta_1) + dtheta_4*(dtheta_4*l_1*l_4*m_5*np.sin(theta_4 - theta_1) + dtheta_4*l_1*m_4*r_4*np.sin(theta_4 - theta_1)) + dtheta_3*(dtheta_3*l_1*l_3*m_4*np.sin(theta_3 - theta_1) + dtheta_3*l_1*l_3*m_5*np.sin(theta_3 - theta_1) + dtheta_3*l_1*m_3*r_3*np.sin(theta_3 - theta_1)) + dtheta_2*mu_2 + dtheta_2*(dtheta_2*l_1*l_2*m_3*np.sin(theta_2 - theta_1) + dtheta_2*l_1*l_2*m_4*np.sin(theta_2 - theta_1) + dtheta_2*l_1*l_2*m_5*np.sin(theta_2 - theta_1) + dtheta_2*l_1*m_2*r_2*np.sin(theta_2 - theta_1)) - dtheta_1*(mu_1 + mu_2) - ddx_c*(-l_1*m_2 - l_1*m_3 - l_1*m_4 - l_1*m_5 - m_1*r_1)*np.cos(theta_1) - g*(-l_1*m_2 - l_1*m_3 - l_1*m_4 - l_1*m_5 - m_1*r_1)*np.sin(theta_1))*np.cos(theta_3 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) - (-l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_3 - theta_2))*(dtheta_5**2*l_2*m_5*r_5*np.sin(theta_5 - theta_2) + dtheta_4*(dtheta_4*l_2*l_4*m_5*np.sin(theta_4 - theta_2) + dtheta_4*l_2*m_4*r_4*np.sin(theta_4 - theta_2)) + dtheta_3*mu_3 + dtheta_3*(dtheta_3*l_2*l_3*m_4*np.sin(theta_3 - theta_2) + dtheta_3*l_2*l_3*m_5*np.sin(theta_3 - theta_2) + dtheta_3*l_2*m_3*r_3*np.sin(theta_3 - theta_2)) - dtheta_2*(mu_2 + mu_3) + dtheta_1*mu_2 + dtheta_1*(-dtheta_1*l_1*l_2*m_3*np.sin(theta_2 - theta_1) - dtheta_1*l_1*l_2*m_4*np.sin(theta_2 - theta_1) - dtheta_1*l_1*l_2*m_5*np.sin(theta_2 - theta_1) - dtheta_1*l_1*m_2*r_2*np.sin(theta_2 - theta_1)) - ddx_c*(-l_2*m_3 - l_2*m_4 - l_2*m_5 - m_2*r_2)*np.cos(theta_2) - g*(-l_2*m_3 - l_2*m_4 - l_2*m_5 - m_2*r_2)*np.sin(theta_2) - l_1*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*(dtheta_5**2*l_1*m_5*r_5*np.sin(theta_5 - theta_1) + dtheta_4*(dtheta_4*l_1*l_4*m_5*np.sin(theta_4 - theta_1) + dtheta_4*l_1*m_4*r_4*np.sin(theta_4 - theta_1)) + dtheta_3*(dtheta_3*l_1*l_3*m_4*np.sin(theta_3 - theta_1) + dtheta_3*l_1*l_3*m_5*np.sin(theta_3 - theta_1) + dtheta_3*l_1*m_3*r_3*np.sin(theta_3 - theta_1)) + dtheta_2*mu_2 + dtheta_2*(dtheta_2*l_1*l_2*m_3*np.sin(theta_2 - theta_1) + dtheta_2*l_1*l_2*m_4*np.sin(theta_2 - theta_1) + dtheta_2*l_1*l_2*m_5*np.sin(theta_2 - theta_1) + dtheta_2*l_1*m_2*r_2*np.sin(theta_2 - theta_1)) - dtheta_1*(mu_1 + mu_2) - ddx_c*(-l_1*m_2 - l_1*m_3 - l_1*m_4 - l_1*m_5 - m_1*r_1)*np.cos(theta_1) - g*(-l_1*m_2 - l_1*m_3 - l_1*m_4 - l_1*m_5 - m_1*r_1)*np.sin(theta_1))*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2))/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2))/(I_3 - l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)**2*np.cos(theta_3 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_3**2*m_4 + l_3**2*m_5 + m_3*r_3**2 - (-l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_3 - theta_2))**2/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2)) - (-l_1**2*m_5*r_5*(l_4*m_5 + m_4*r_4)*np.cos(theta_5 - theta_1)*np.cos(theta_4 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_4*m_5*r_5*np.cos(theta_5 - theta_4) - (-l_1**2*(l_4*m_5 + m_4*r_4)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_4 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_4*m_5 + m_4*r_4)*np.cos(theta_4 - theta_2))*(-l_1**2*m_5*r_5*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_5 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*m_5*r_5*np.cos(theta_5 - theta_2))/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2) - (-l_1**2*(l_4*m_5 + m_4*r_4)*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_4 - theta_1)*np.cos(theta_3 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_3*(l_4*m_5 + m_4*r_4)*np.cos(theta_4 - theta_3) - (-l_1**2*(l_4*m_5 + m_4*r_4)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_4 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_4*m_5 + m_4*r_4)*np.cos(theta_4 - theta_2))*(-l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_3 - theta_2))/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2))*(-l_1**2*m_5*r_5*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_5 - theta_1)*np.cos(theta_3 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_3*m_5*r_5*np.cos(theta_5 - theta_3) - (-l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_3 - theta_2))*(-l_1**2*m_5*r_5*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_5 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*m_5*r_5*np.cos(theta_5 - theta_2))/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2))/(I_3 - l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)**2*np.cos(theta_3 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_3**2*m_4 + l_3**2*m_5 + m_3*r_3**2 - (-l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_3 - theta_2))**2/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2)))*(dtheta_5**2*l_4*m_5*r_5*np.sin(theta_5 - theta_4) + dtheta_5*mu_5 - dtheta_4*(mu_4 + mu_5) + dtheta_3*mu_4 + dtheta_3*(-dtheta_3*l_3*l_4*m_5*np.sin(theta_4 - theta_3) - dtheta_3*l_3*m_4*r_4*np.sin(theta_4 - theta_3)) + dtheta_2*(-dtheta_2*l_2*l_4*m_5*np.sin(theta_4 - theta_2) - dtheta_2*l_2*m_4*r_4*np.sin(theta_4 - theta_2)) + dtheta_1*(-dtheta_1*l_1*l_4*m_5*np.sin(theta_4 - theta_1) - dtheta_1*l_1*m_4*r_4*np.sin(theta_4 - theta_1)) - ddx_c*(-l_4*m_5 - m_4*r_4)*np.cos(theta_4) - g*(-l_4*m_5 - m_4*r_4)*np.sin(theta_4) - l_1*(l_4*m_5 + m_4*r_4)*(dtheta_5**2*l_1*m_5*r_5*np.sin(theta_5 - theta_1) + dtheta_4*(dtheta_4*l_1*l_4*m_5*np.sin(theta_4 - theta_1) + dtheta_4*l_1*m_4*r_4*np.sin(theta_4 - theta_1)) + dtheta_3*(dtheta_3*l_1*l_3*m_4*np.sin(theta_3 - theta_1) + dtheta_3*l_1*l_3*m_5*np.sin(theta_3 - theta_1) + dtheta_3*l_1*m_3*r_3*np.sin(theta_3 - theta_1)) + dtheta_2*mu_2 + dtheta_2*(dtheta_2*l_1*l_2*m_3*np.sin(theta_2 - theta_1) + dtheta_2*l_1*l_2*m_4*np.sin(theta_2 - theta_1) + dtheta_2*l_1*l_2*m_5*np.sin(theta_2 - theta_1) + dtheta_2*l_1*m_2*r_2*np.sin(theta_2 - theta_1)) - dtheta_1*(mu_1 + mu_2) - ddx_c*(-l_1*m_2 - l_1*m_3 - l_1*m_4 - l_1*m_5 - m_1*r_1)*np.cos(theta_1) - g*(-l_1*m_2 - l_1*m_3 - l_1*m_4 - l_1*m_5 - m_1*r_1)*np.sin(theta_1))*np.cos(theta_4 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) - (-l_1**2*(l_4*m_5 + m_4*r_4)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_4 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_4*m_5 + m_4*r_4)*np.cos(theta_4 - theta_2))*(dtheta_5**2*l_2*m_5*r_5*np.sin(theta_5 - theta_2) + dtheta_4*(dtheta_4*l_2*l_4*m_5*np.sin(theta_4 - theta_2) + dtheta_4*l_2*m_4*r_4*np.sin(theta_4 - theta_2)) + dtheta_3*mu_3 + dtheta_3*(dtheta_3*l_2*l_3*m_4*np.sin(theta_3 - theta_2) + dtheta_3*l_2*l_3*m_5*np.sin(theta_3 - theta_2) + dtheta_3*l_2*m_3*r_3*np.sin(theta_3 - theta_2)) - dtheta_2*(mu_2 + mu_3) + dtheta_1*mu_2 + dtheta_1*(-dtheta_1*l_1*l_2*m_3*np.sin(theta_2 - theta_1) - dtheta_1*l_1*l_2*m_4*np.sin(theta_2 - theta_1) - dtheta_1*l_1*l_2*m_5*np.sin(theta_2 - theta_1) - dtheta_1*l_1*m_2*r_2*np.sin(theta_2 - theta_1)) - ddx_c*(-l_2*m_3 - l_2*m_4 - l_2*m_5 - m_2*r_2)*np.cos(theta_2) - g*(-l_2*m_3 - l_2*m_4 - l_2*m_5 - m_2*r_2)*np.sin(theta_2) - l_1*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*(dtheta_5**2*l_1*m_5*r_5*np.sin(theta_5 - theta_1) + dtheta_4*(dtheta_4*l_1*l_4*m_5*np.sin(theta_4 - theta_1) + dtheta_4*l_1*m_4*r_4*np.sin(theta_4 - theta_1)) + dtheta_3*(dtheta_3*l_1*l_3*m_4*np.sin(theta_3 - theta_1) + dtheta_3*l_1*l_3*m_5*np.sin(theta_3 - theta_1) + dtheta_3*l_1*m_3*r_3*np.sin(theta_3 - theta_1)) + dtheta_2*mu_2 + dtheta_2*(dtheta_2*l_1*l_2*m_3*np.sin(theta_2 - theta_1) + dtheta_2*l_1*l_2*m_4*np.sin(theta_2 - theta_1) + dtheta_2*l_1*l_2*m_5*np.sin(theta_2 - theta_1) + dtheta_2*l_1*m_2*r_2*np.sin(theta_2 - theta_1)) - dtheta_1*(mu_1 + mu_2) - ddx_c*(-l_1*m_2 - l_1*m_3 - l_1*m_4 - l_1*m_5 - m_1*r_1)*np.cos(theta_1) - g*(-l_1*m_2 - l_1*m_3 - l_1*m_4 - l_1*m_5 - m_1*r_1)*np.sin(theta_1))*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2))/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2) - (-l_1**2*(l_4*m_5 + m_4*r_4)*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_4 - theta_1)*np.cos(theta_3 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_3*(l_4*m_5 + m_4*r_4)*np.cos(theta_4 - theta_3) - (-l_1**2*(l_4*m_5 + m_4*r_4)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_4 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_4*m_5 + m_4*r_4)*np.cos(theta_4 - theta_2))*(-l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_3 - theta_2))/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2))*(dtheta_5**2*l_3*m_5*r_5*np.sin(theta_5 - theta_3) + dtheta_4*mu_4 + dtheta_4*(dtheta_4*l_3*l_4*m_5*np.sin(theta_4 - theta_3) + dtheta_4*l_3*m_4*r_4*np.sin(theta_4 - theta_3)) - dtheta_3*(mu_3 + mu_4) + dtheta_2*mu_3 + dtheta_2*(-dtheta_2*l_2*l_3*m_4*np.sin(theta_3 - theta_2) - dtheta_2*l_2*l_3*m_5*np.sin(theta_3 - theta_2) - dtheta_2*l_2*m_3*r_3*np.sin(theta_3 - theta_2)) + dtheta_1*(-dtheta_1*l_1*l_3*m_4*np.sin(theta_3 - theta_1) - dtheta_1*l_1*l_3*m_5*np.sin(theta_3 - theta_1) - dtheta_1*l_1*m_3*r_3*np.sin(theta_3 - theta_1)) - ddx_c*(-l_3*m_4 - l_3*m_5 - m_3*r_3)*np.cos(theta_3) - g*(-l_3*m_4 - l_3*m_5 - m_3*r_3)*np.sin(theta_3) - l_1*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(dtheta_5**2*l_1*m_5*r_5*np.sin(theta_5 - theta_1) + dtheta_4*(dtheta_4*l_1*l_4*m_5*np.sin(theta_4 - theta_1) + dtheta_4*l_1*m_4*r_4*np.sin(theta_4 - theta_1)) + dtheta_3*(dtheta_3*l_1*l_3*m_4*np.sin(theta_3 - theta_1) + dtheta_3*l_1*l_3*m_5*np.sin(theta_3 - theta_1) + dtheta_3*l_1*m_3*r_3*np.sin(theta_3 - theta_1)) + dtheta_2*mu_2 + dtheta_2*(dtheta_2*l_1*l_2*m_3*np.sin(theta_2 - theta_1) + dtheta_2*l_1*l_2*m_4*np.sin(theta_2 - theta_1) + dtheta_2*l_1*l_2*m_5*np.sin(theta_2 - theta_1) + dtheta_2*l_1*m_2*r_2*np.sin(theta_2 - theta_1)) - dtheta_1*(mu_1 + mu_2) - ddx_c*(-l_1*m_2 - l_1*m_3 - l_1*m_4 - l_1*m_5 - m_1*r_1)*np.cos(theta_1) - g*(-l_1*m_2 - l_1*m_3 - l_1*m_4 - l_1*m_5 - m_1*r_1)*np.sin(theta_1))*np.cos(theta_3 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) - (-l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_3 - theta_2))*(dtheta_5**2*l_2*m_5*r_5*np.sin(theta_5 - theta_2) + dtheta_4*(dtheta_4*l_2*l_4*m_5*np.sin(theta_4 - theta_2) + dtheta_4*l_2*m_4*r_4*np.sin(theta_4 - theta_2)) + dtheta_3*mu_3 + dtheta_3*(dtheta_3*l_2*l_3*m_4*np.sin(theta_3 - theta_2) + dtheta_3*l_2*l_3*m_5*np.sin(theta_3 - theta_2) + dtheta_3*l_2*m_3*r_3*np.sin(theta_3 - theta_2)) - dtheta_2*(mu_2 + mu_3) + dtheta_1*mu_2 + dtheta_1*(-dtheta_1*l_1*l_2*m_3*np.sin(theta_2 - theta_1) - dtheta_1*l_1*l_2*m_4*np.sin(theta_2 - theta_1) - dtheta_1*l_1*l_2*m_5*np.sin(theta_2 - theta_1) - dtheta_1*l_1*m_2*r_2*np.sin(theta_2 - theta_1)) - ddx_c*(-l_2*m_3 - l_2*m_4 - l_2*m_5 - m_2*r_2)*np.cos(theta_2) - g*(-l_2*m_3 - l_2*m_4 - l_2*m_5 - m_2*r_2)*np.sin(theta_2) - l_1*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*(dtheta_5**2*l_1*m_5*r_5*np.sin(theta_5 - theta_1) + dtheta_4*(dtheta_4*l_1*l_4*m_5*np.sin(theta_4 - theta_1) + dtheta_4*l_1*m_4*r_4*np.sin(theta_4 - theta_1)) + dtheta_3*(dtheta_3*l_1*l_3*m_4*np.sin(theta_3 - theta_1) + dtheta_3*l_1*l_3*m_5*np.sin(theta_3 - theta_1) + dtheta_3*l_1*m_3*r_3*np.sin(theta_3 - theta_1)) + dtheta_2*mu_2 + dtheta_2*(dtheta_2*l_1*l_2*m_3*np.sin(theta_2 - theta_1) + dtheta_2*l_1*l_2*m_4*np.sin(theta_2 - theta_1) + dtheta_2*l_1*l_2*m_5*np.sin(theta_2 - theta_1) + dtheta_2*l_1*m_2*r_2*np.sin(theta_2 - theta_1)) - dtheta_1*(mu_1 + mu_2) - ddx_c*(-l_1*m_2 - l_1*m_3 - l_1*m_4 - l_1*m_5 - m_1*r_1)*np.cos(theta_1) - g*(-l_1*m_2 - l_1*m_3 - l_1*m_4 - l_1*m_5 - m_1*r_1)*np.sin(theta_1))*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2))/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2))/(I_3 - l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)**2*np.cos(theta_3 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_3**2*m_4 + l_3**2*m_5 + m_3*r_3**2 - (-l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_3 - theta_2))**2/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2)))/(I_4 - l_1**2*(l_4*m_5 + m_4*r_4)**2*np.cos(theta_4 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_4**2*m_5 + m_4*r_4**2 - (-l_1**2*(l_4*m_5 + m_4*r_4)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_4 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_4*m_5 + m_4*r_4)*np.cos(theta_4 - theta_2))**2/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2) - (-l_1**2*(l_4*m_5 + m_4*r_4)*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_4 - theta_1)*np.cos(theta_3 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_3*(l_4*m_5 + m_4*r_4)*np.cos(theta_4 - theta_3) - (-l_1**2*(l_4*m_5 + m_4*r_4)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_4 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_4*m_5 + m_4*r_4)*np.cos(theta_4 - theta_2))*(-l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_3 - theta_2))/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2))**2/(I_3 - l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)**2*np.cos(theta_3 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_3**2*m_4 + l_3**2*m_5 + m_3*r_3**2 - (-l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_3 - theta_2))**2/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2))))/(I_5 - l_1**2*m_5**2*r_5**2*np.cos(theta_5 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + m_5*r_5**2 - (-l_1**2*m_5*r_5*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_5 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*m_5*r_5*np.cos(theta_5 - theta_2))**2/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2) - (-l_1**2*m_5*r_5*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_5 - theta_1)*np.cos(theta_3 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_3*m_5*r_5*np.cos(theta_5 - theta_3) - (-l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_3 - theta_2))*(-l_1**2*m_5*r_5*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_5 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*m_5*r_5*np.cos(theta_5 - theta_2))/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2))**2/(I_3 - l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)**2*np.cos(theta_3 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_3**2*m_4 + l_3**2*m_5 + m_3*r_3**2 - (-l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_3 - theta_2))**2/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2)) - (-l_1**2*m_5*r_5*(l_4*m_5 + m_4*r_4)*np.cos(theta_5 - theta_1)*np.cos(theta_4 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_4*m_5*r_5*np.cos(theta_5 - theta_4) - (-l_1**2*(l_4*m_5 + m_4*r_4)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_4 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_4*m_5 + m_4*r_4)*np.cos(theta_4 - theta_2))*(-l_1**2*m_5*r_5*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_5 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*m_5*r_5*np.cos(theta_5 - theta_2))/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2) - (-l_1**2*(l_4*m_5 + m_4*r_4)*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_4 - theta_1)*np.cos(theta_3 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_3*(l_4*m_5 + m_4*r_4)*np.cos(theta_4 - theta_3) - (-l_1**2*(l_4*m_5 + m_4*r_4)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_4 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_4*m_5 + m_4*r_4)*np.cos(theta_4 - theta_2))*(-l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_3 - theta_2))/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2))*(-l_1**2*m_5*r_5*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_5 - theta_1)*np.cos(theta_3 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_3*m_5*r_5*np.cos(theta_5 - theta_3) - (-l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_3 - theta_2))*(-l_1**2*m_5*r_5*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_5 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*m_5*r_5*np.cos(theta_5 - theta_2))/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2))/(I_3 - l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)**2*np.cos(theta_3 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_3**2*m_4 + l_3**2*m_5 + m_3*r_3**2 - (-l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_3 - theta_2))**2/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2)))**2/(I_4 - l_1**2*(l_4*m_5 + m_4*r_4)**2*np.cos(theta_4 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_4**2*m_5 + m_4*r_4**2 - (-l_1**2*(l_4*m_5 + m_4*r_4)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_4 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_4*m_5 + m_4*r_4)*np.cos(theta_4 - theta_2))**2/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2) - (-l_1**2*(l_4*m_5 + m_4*r_4)*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_4 - theta_1)*np.cos(theta_3 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_3*(l_4*m_5 + m_4*r_4)*np.cos(theta_4 - theta_3) - (-l_1**2*(l_4*m_5 + m_4*r_4)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_4 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_4*m_5 + m_4*r_4)*np.cos(theta_4 - theta_2))*(-l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_3 - theta_2))/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2))**2/(I_3 - l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)**2*np.cos(theta_3 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_3**2*m_4 + l_3**2*m_5 + m_3*r_3**2 - (-l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_3 - theta_2))**2/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2)))))/(I_4 - l_1**2*(l_4*m_5 + m_4*r_4)**2*np.cos(theta_4 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_4**2*m_5 + m_4*r_4**2 - (-l_1**2*(l_4*m_5 + m_4*r_4)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_4 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_4*m_5 + m_4*r_4)*np.cos(theta_4 - theta_2))**2/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2) - (-l_1**2*(l_4*m_5 + m_4*r_4)*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_4 - theta_1)*np.cos(theta_3 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_3*(l_4*m_5 + m_4*r_4)*np.cos(theta_4 - theta_3) - (-l_1**2*(l_4*m_5 + m_4*r_4)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_4 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_4*m_5 + m_4*r_4)*np.cos(theta_4 - theta_2))*(-l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_3 - theta_2))/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2))**2/(I_3 - l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)**2*np.cos(theta_3 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_3**2*m_4 + l_3**2*m_5 + m_3*r_3**2 - (-l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_3 - theta_2))**2/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2))) - (-l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_3 - theta_2))*(dtheta_5**2*l_3*m_5*r_5*np.sin(theta_5 - theta_3) + dtheta_4*mu_4 + dtheta_4*(dtheta_4*l_3*l_4*m_5*np.sin(theta_4 - theta_3) + dtheta_4*l_3*m_4*r_4*np.sin(theta_4 - theta_3)) - dtheta_3*(mu_3 + mu_4) + dtheta_2*mu_3 + dtheta_2*(-dtheta_2*l_2*l_3*m_4*np.sin(theta_3 - theta_2) - dtheta_2*l_2*l_3*m_5*np.sin(theta_3 - theta_2) - dtheta_2*l_2*m_3*r_3*np.sin(theta_3 - theta_2)) + dtheta_1*(-dtheta_1*l_1*l_3*m_4*np.sin(theta_3 - theta_1) - dtheta_1*l_1*l_3*m_5*np.sin(theta_3 - theta_1) - dtheta_1*l_1*m_3*r_3*np.sin(theta_3 - theta_1)) - ddx_c*(-l_3*m_4 - l_3*m_5 - m_3*r_3)*np.cos(theta_3) - g*(-l_3*m_4 - l_3*m_5 - m_3*r_3)*np.sin(theta_3) - l_1*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(dtheta_5**2*l_1*m_5*r_5*np.sin(theta_5 - theta_1) + dtheta_4*(dtheta_4*l_1*l_4*m_5*np.sin(theta_4 - theta_1) + dtheta_4*l_1*m_4*r_4*np.sin(theta_4 - theta_1)) + dtheta_3*(dtheta_3*l_1*l_3*m_4*np.sin(theta_3 - theta_1) + dtheta_3*l_1*l_3*m_5*np.sin(theta_3 - theta_1) + dtheta_3*l_1*m_3*r_3*np.sin(theta_3 - theta_1)) + dtheta_2*mu_2 + dtheta_2*(dtheta_2*l_1*l_2*m_3*np.sin(theta_2 - theta_1) + dtheta_2*l_1*l_2*m_4*np.sin(theta_2 - theta_1) + dtheta_2*l_1*l_2*m_5*np.sin(theta_2 - theta_1) + dtheta_2*l_1*m_2*r_2*np.sin(theta_2 - theta_1)) - dtheta_1*(mu_1 + mu_2) - ddx_c*(-l_1*m_2 - l_1*m_3 - l_1*m_4 - l_1*m_5 - m_1*r_1)*np.cos(theta_1) - g*(-l_1*m_2 - l_1*m_3 - l_1*m_4 - l_1*m_5 - m_1*r_1)*np.sin(theta_1))*np.cos(theta_3 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) - (-l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_3 - theta_2))*(dtheta_5**2*l_2*m_5*r_5*np.sin(theta_5 - theta_2) + dtheta_4*(dtheta_4*l_2*l_4*m_5*np.sin(theta_4 - theta_2) + dtheta_4*l_2*m_4*r_4*np.sin(theta_4 - theta_2)) + dtheta_3*mu_3 + dtheta_3*(dtheta_3*l_2*l_3*m_4*np.sin(theta_3 - theta_2) + dtheta_3*l_2*l_3*m_5*np.sin(theta_3 - theta_2) + dtheta_3*l_2*m_3*r_3*np.sin(theta_3 - theta_2)) - dtheta_2*(mu_2 + mu_3) + dtheta_1*mu_2 + dtheta_1*(-dtheta_1*l_1*l_2*m_3*np.sin(theta_2 - theta_1) - dtheta_1*l_1*l_2*m_4*np.sin(theta_2 - theta_1) - dtheta_1*l_1*l_2*m_5*np.sin(theta_2 - theta_1) - dtheta_1*l_1*m_2*r_2*np.sin(theta_2 - theta_1)) - ddx_c*(-l_2*m_3 - l_2*m_4 - l_2*m_5 - m_2*r_2)*np.cos(theta_2) - g*(-l_2*m_3 - l_2*m_4 - l_2*m_5 - m_2*r_2)*np.sin(theta_2) - l_1*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*(dtheta_5**2*l_1*m_5*r_5*np.sin(theta_5 - theta_1) + dtheta_4*(dtheta_4*l_1*l_4*m_5*np.sin(theta_4 - theta_1) + dtheta_4*l_1*m_4*r_4*np.sin(theta_4 - theta_1)) + dtheta_3*(dtheta_3*l_1*l_3*m_4*np.sin(theta_3 - theta_1) + dtheta_3*l_1*l_3*m_5*np.sin(theta_3 - theta_1) + dtheta_3*l_1*m_3*r_3*np.sin(theta_3 - theta_1)) + dtheta_2*mu_2 + dtheta_2*(dtheta_2*l_1*l_2*m_3*np.sin(theta_2 - theta_1) + dtheta_2*l_1*l_2*m_4*np.sin(theta_2 - theta_1) + dtheta_2*l_1*l_2*m_5*np.sin(theta_2 - theta_1) + dtheta_2*l_1*m_2*r_2*np.sin(theta_2 - theta_1)) - dtheta_1*(mu_1 + mu_2) - ddx_c*(-l_1*m_2 - l_1*m_3 - l_1*m_4 - l_1*m_5 - m_1*r_1)*np.cos(theta_1) - g*(-l_1*m_2 - l_1*m_3 - l_1*m_4 - l_1*m_5 - m_1*r_1)*np.sin(theta_1))*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2))/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2) - (-l_1**2*(l_4*m_5 + m_4*r_4)*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_4 - theta_1)*np.cos(theta_3 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_3*(l_4*m_5 + m_4*r_4)*np.cos(theta_4 - theta_3) - (-l_1**2*(l_4*m_5 + m_4*r_4)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_4 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_4*m_5 + m_4*r_4)*np.cos(theta_4 - theta_2))*(-l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_3 - theta_2))/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2))*(dtheta_5**2*l_4*m_5*r_5*np.sin(theta_5 - theta_4) + dtheta_5*mu_5 - dtheta_4*(mu_4 + mu_5) + dtheta_3*mu_4 + dtheta_3*(-dtheta_3*l_3*l_4*m_5*np.sin(theta_4 - theta_3) - dtheta_3*l_3*m_4*r_4*np.sin(theta_4 - theta_3)) + dtheta_2*(-dtheta_2*l_2*l_4*m_5*np.sin(theta_4 - theta_2) - dtheta_2*l_2*m_4*r_4*np.sin(theta_4 - theta_2)) + dtheta_1*(-dtheta_1*l_1*l_4*m_5*np.sin(theta_4 - theta_1) - dtheta_1*l_1*m_4*r_4*np.sin(theta_4 - theta_1)) - ddx_c*(-l_4*m_5 - m_4*r_4)*np.cos(theta_4) - g*(-l_4*m_5 - m_4*r_4)*np.sin(theta_4) - l_1*(l_4*m_5 + m_4*r_4)*(dtheta_5**2*l_1*m_5*r_5*np.sin(theta_5 - theta_1) + dtheta_4*(dtheta_4*l_1*l_4*m_5*np.sin(theta_4 - theta_1) + dtheta_4*l_1*m_4*r_4*np.sin(theta_4 - theta_1)) + dtheta_3*(dtheta_3*l_1*l_3*m_4*np.sin(theta_3 - theta_1) + dtheta_3*l_1*l_3*m_5*np.sin(theta_3 - theta_1) + dtheta_3*l_1*m_3*r_3*np.sin(theta_3 - theta_1)) + dtheta_2*mu_2 + dtheta_2*(dtheta_2*l_1*l_2*m_3*np.sin(theta_2 - theta_1) + dtheta_2*l_1*l_2*m_4*np.sin(theta_2 - theta_1) + dtheta_2*l_1*l_2*m_5*np.sin(theta_2 - theta_1) + dtheta_2*l_1*m_2*r_2*np.sin(theta_2 - theta_1)) - dtheta_1*(mu_1 + mu_2) - ddx_c*(-l_1*m_2 - l_1*m_3 - l_1*m_4 - l_1*m_5 - m_1*r_1)*np.cos(theta_1) - g*(-l_1*m_2 - l_1*m_3 - l_1*m_4 - l_1*m_5 - m_1*r_1)*np.sin(theta_1))*np.cos(theta_4 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) - (-l_1**2*(l_4*m_5 + m_4*r_4)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_4 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_4*m_5 + m_4*r_4)*np.cos(theta_4 - theta_2))*(dtheta_5**2*l_2*m_5*r_5*np.sin(theta_5 - theta_2) + dtheta_4*(dtheta_4*l_2*l_4*m_5*np.sin(theta_4 - theta_2) + dtheta_4*l_2*m_4*r_4*np.sin(theta_4 - theta_2)) + dtheta_3*mu_3 + dtheta_3*(dtheta_3*l_2*l_3*m_4*np.sin(theta_3 - theta_2) + dtheta_3*l_2*l_3*m_5*np.sin(theta_3 - theta_2) + dtheta_3*l_2*m_3*r_3*np.sin(theta_3 - theta_2)) - dtheta_2*(mu_2 + mu_3) + dtheta_1*mu_2 + dtheta_1*(-dtheta_1*l_1*l_2*m_3*np.sin(theta_2 - theta_1) - dtheta_1*l_1*l_2*m_4*np.sin(theta_2 - theta_1) - dtheta_1*l_1*l_2*m_5*np.sin(theta_2 - theta_1) - dtheta_1*l_1*m_2*r_2*np.sin(theta_2 - theta_1)) - ddx_c*(-l_2*m_3 - l_2*m_4 - l_2*m_5 - m_2*r_2)*np.cos(theta_2) - g*(-l_2*m_3 - l_2*m_4 - l_2*m_5 - m_2*r_2)*np.sin(theta_2) - l_1*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*(dtheta_5**2*l_1*m_5*r_5*np.sin(theta_5 - theta_1) + dtheta_4*(dtheta_4*l_1*l_4*m_5*np.sin(theta_4 - theta_1) + dtheta_4*l_1*m_4*r_4*np.sin(theta_4 - theta_1)) + dtheta_3*(dtheta_3*l_1*l_3*m_4*np.sin(theta_3 - theta_1) + dtheta_3*l_1*l_3*m_5*np.sin(theta_3 - theta_1) + dtheta_3*l_1*m_3*r_3*np.sin(theta_3 - theta_1)) + dtheta_2*mu_2 + dtheta_2*(dtheta_2*l_1*l_2*m_3*np.sin(theta_2 - theta_1) + dtheta_2*l_1*l_2*m_4*np.sin(theta_2 - theta_1) + dtheta_2*l_1*l_2*m_5*np.sin(theta_2 - theta_1) + dtheta_2*l_1*m_2*r_2*np.sin(theta_2 - theta_1)) - dtheta_1*(mu_1 + mu_2) - ddx_c*(-l_1*m_2 - l_1*m_3 - l_1*m_4 - l_1*m_5 - m_1*r_1)*np.cos(theta_1) - g*(-l_1*m_2 - l_1*m_3 - l_1*m_4 - l_1*m_5 - m_1*r_1)*np.sin(theta_1))*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2))/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2) - (-l_1**2*(l_4*m_5 + m_4*r_4)*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_4 - theta_1)*np.cos(theta_3 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_3*(l_4*m_5 + m_4*r_4)*np.cos(theta_4 - theta_3) - (-l_1**2*(l_4*m_5 + m_4*r_4)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_4 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_4*m_5 + m_4*r_4)*np.cos(theta_4 - theta_2))*(-l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_3 - theta_2))/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2))*(dtheta_5**2*l_3*m_5*r_5*np.sin(theta_5 - theta_3) + dtheta_4*mu_4 + dtheta_4*(dtheta_4*l_3*l_4*m_5*np.sin(theta_4 - theta_3) + dtheta_4*l_3*m_4*r_4*np.sin(theta_4 - theta_3)) - dtheta_3*(mu_3 + mu_4) + dtheta_2*mu_3 + dtheta_2*(-dtheta_2*l_2*l_3*m_4*np.sin(theta_3 - theta_2) - dtheta_2*l_2*l_3*m_5*np.sin(theta_3 - theta_2) - dtheta_2*l_2*m_3*r_3*np.sin(theta_3 - theta_2)) + dtheta_1*(-dtheta_1*l_1*l_3*m_4*np.sin(theta_3 - theta_1) - dtheta_1*l_1*l_3*m_5*np.sin(theta_3 - theta_1) - dtheta_1*l_1*m_3*r_3*np.sin(theta_3 - theta_1)) - ddx_c*(-l_3*m_4 - l_3*m_5 - m_3*r_3)*np.cos(theta_3) - g*(-l_3*m_4 - l_3*m_5 - m_3*r_3)*np.sin(theta_3) - l_1*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(dtheta_5**2*l_1*m_5*r_5*np.sin(theta_5 - theta_1) + dtheta_4*(dtheta_4*l_1*l_4*m_5*np.sin(theta_4 - theta_1) + dtheta_4*l_1*m_4*r_4*np.sin(theta_4 - theta_1)) + dtheta_3*(dtheta_3*l_1*l_3*m_4*np.sin(theta_3 - theta_1) + dtheta_3*l_1*l_3*m_5*np.sin(theta_3 - theta_1) + dtheta_3*l_1*m_3*r_3*np.sin(theta_3 - theta_1)) + dtheta_2*mu_2 + dtheta_2*(dtheta_2*l_1*l_2*m_3*np.sin(theta_2 - theta_1) + dtheta_2*l_1*l_2*m_4*np.sin(theta_2 - theta_1) + dtheta_2*l_1*l_2*m_5*np.sin(theta_2 - theta_1) + dtheta_2*l_1*m_2*r_2*np.sin(theta_2 - theta_1)) - dtheta_1*(mu_1 + mu_2) - ddx_c*(-l_1*m_2 - l_1*m_3 - l_1*m_4 - l_1*m_5 - m_1*r_1)*np.cos(theta_1) - g*(-l_1*m_2 - l_1*m_3 - l_1*m_4 - l_1*m_5 - m_1*r_1)*np.sin(theta_1))*np.cos(theta_3 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) - (-l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_3 - theta_2))*(dtheta_5**2*l_2*m_5*r_5*np.sin(theta_5 - theta_2) + dtheta_4*(dtheta_4*l_2*l_4*m_5*np.sin(theta_4 - theta_2) + dtheta_4*l_2*m_4*r_4*np.sin(theta_4 - theta_2)) + dtheta_3*mu_3 + dtheta_3*(dtheta_3*l_2*l_3*m_4*np.sin(theta_3 - theta_2) + dtheta_3*l_2*l_3*m_5*np.sin(theta_3 - theta_2) + dtheta_3*l_2*m_3*r_3*np.sin(theta_3 - theta_2)) - dtheta_2*(mu_2 + mu_3) + dtheta_1*mu_2 + dtheta_1*(-dtheta_1*l_1*l_2*m_3*np.sin(theta_2 - theta_1) - dtheta_1*l_1*l_2*m_4*np.sin(theta_2 - theta_1) - dtheta_1*l_1*l_2*m_5*np.sin(theta_2 - theta_1) - dtheta_1*l_1*m_2*r_2*np.sin(theta_2 - theta_1)) - ddx_c*(-l_2*m_3 - l_2*m_4 - l_2*m_5 - m_2*r_2)*np.cos(theta_2) - g*(-l_2*m_3 - l_2*m_4 - l_2*m_5 - m_2*r_2)*np.sin(theta_2) - l_1*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*(dtheta_5**2*l_1*m_5*r_5*np.sin(theta_5 - theta_1) + dtheta_4*(dtheta_4*l_1*l_4*m_5*np.sin(theta_4 - theta_1) + dtheta_4*l_1*m_4*r_4*np.sin(theta_4 - theta_1)) + dtheta_3*(dtheta_3*l_1*l_3*m_4*np.sin(theta_3 - theta_1) + dtheta_3*l_1*l_3*m_5*np.sin(theta_3 - theta_1) + dtheta_3*l_1*m_3*r_3*np.sin(theta_3 - theta_1)) + dtheta_2*mu_2 + dtheta_2*(dtheta_2*l_1*l_2*m_3*np.sin(theta_2 - theta_1) + dtheta_2*l_1*l_2*m_4*np.sin(theta_2 - theta_1) + dtheta_2*l_1*l_2*m_5*np.sin(theta_2 - theta_1) + dtheta_2*l_1*m_2*r_2*np.sin(theta_2 - theta_1)) - dtheta_1*(mu_1 + mu_2) - ddx_c*(-l_1*m_2 - l_1*m_3 - l_1*m_4 - l_1*m_5 - m_1*r_1)*np.cos(theta_1) - g*(-l_1*m_2 - l_1*m_3 - l_1*m_4 - l_1*m_5 - m_1*r_1)*np.sin(theta_1))*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2))/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2))/(I_3 - l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)**2*np.cos(theta_3 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_3**2*m_4 + l_3**2*m_5 + m_3*r_3**2 - (-l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_3 - theta_2))**2/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2)) - (-l_1**2*m_5*r_5*(l_4*m_5 + m_4*r_4)*np.cos(theta_5 - theta_1)*np.cos(theta_4 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_4*m_5*r_5*np.cos(theta_5 - theta_4) - (-l_1**2*(l_4*m_5 + m_4*r_4)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_4 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_4*m_5 + m_4*r_4)*np.cos(theta_4 - theta_2))*(-l_1**2*m_5*r_5*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_5 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*m_5*r_5*np.cos(theta_5 - theta_2))/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2) - (-l_1**2*(l_4*m_5 + m_4*r_4)*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_4 - theta_1)*np.cos(theta_3 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_3*(l_4*m_5 + m_4*r_4)*np.cos(theta_4 - theta_3) - (-l_1**2*(l_4*m_5 + m_4*r_4)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_4 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_4*m_5 + m_4*r_4)*np.cos(theta_4 - theta_2))*(-l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_3 - theta_2))/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2))*(-l_1**2*m_5*r_5*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_5 - theta_1)*np.cos(theta_3 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_3*m_5*r_5*np.cos(theta_5 - theta_3) - (-l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_3 - theta_2))*(-l_1**2*m_5*r_5*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_5 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*m_5*r_5*np.cos(theta_5 - theta_2))/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2))/(I_3 - l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)**2*np.cos(theta_3 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_3**2*m_4 + l_3**2*m_5 + m_3*r_3**2 - (-l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_3 - theta_2))**2/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2)))*(-dtheta_5*mu_5 - dtheta_4**2*l_4*m_5*r_5*np.sin(theta_5 - theta_4) + dtheta_4*mu_5 - dtheta_3**2*l_3*m_5*r_5*np.sin(theta_5 - theta_3) - dtheta_2**2*l_2*m_5*r_5*np.sin(theta_5 - theta_2) - dtheta_1**2*l_1*m_5*r_5*np.sin(theta_5 - theta_1) + ddx_c*m_5*r_5*np.cos(theta_5) + g*m_5*r_5*np.sin(theta_5) - l_1*m_5*r_5*(dtheta_5**2*l_1*m_5*r_5*np.sin(theta_5 - theta_1) + dtheta_4*(dtheta_4*l_1*l_4*m_5*np.sin(theta_4 - theta_1) + dtheta_4*l_1*m_4*r_4*np.sin(theta_4 - theta_1)) + dtheta_3*(dtheta_3*l_1*l_3*m_4*np.sin(theta_3 - theta_1) + dtheta_3*l_1*l_3*m_5*np.sin(theta_3 - theta_1) + dtheta_3*l_1*m_3*r_3*np.sin(theta_3 - theta_1)) + dtheta_2*mu_2 + dtheta_2*(dtheta_2*l_1*l_2*m_3*np.sin(theta_2 - theta_1) + dtheta_2*l_1*l_2*m_4*np.sin(theta_2 - theta_1) + dtheta_2*l_1*l_2*m_5*np.sin(theta_2 - theta_1) + dtheta_2*l_1*m_2*r_2*np.sin(theta_2 - theta_1)) - dtheta_1*(mu_1 + mu_2) - ddx_c*(-l_1*m_2 - l_1*m_3 - l_1*m_4 - l_1*m_5 - m_1*r_1)*np.cos(theta_1) - g*(-l_1*m_2 - l_1*m_3 - l_1*m_4 - l_1*m_5 - m_1*r_1)*np.sin(theta_1))*np.cos(theta_5 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) - (-l_1**2*m_5*r_5*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_5 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*m_5*r_5*np.cos(theta_5 - theta_2))*(dtheta_5**2*l_2*m_5*r_5*np.sin(theta_5 - theta_2) + dtheta_4*(dtheta_4*l_2*l_4*m_5*np.sin(theta_4 - theta_2) + dtheta_4*l_2*m_4*r_4*np.sin(theta_4 - theta_2)) + dtheta_3*mu_3 + dtheta_3*(dtheta_3*l_2*l_3*m_4*np.sin(theta_3 - theta_2) + dtheta_3*l_2*l_3*m_5*np.sin(theta_3 - theta_2) + dtheta_3*l_2*m_3*r_3*np.sin(theta_3 - theta_2)) - dtheta_2*(mu_2 + mu_3) + dtheta_1*mu_2 + dtheta_1*(-dtheta_1*l_1*l_2*m_3*np.sin(theta_2 - theta_1) - dtheta_1*l_1*l_2*m_4*np.sin(theta_2 - theta_1) - dtheta_1*l_1*l_2*m_5*np.sin(theta_2 - theta_1) - dtheta_1*l_1*m_2*r_2*np.sin(theta_2 - theta_1)) - ddx_c*(-l_2*m_3 - l_2*m_4 - l_2*m_5 - m_2*r_2)*np.cos(theta_2) - g*(-l_2*m_3 - l_2*m_4 - l_2*m_5 - m_2*r_2)*np.sin(theta_2) - l_1*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*(dtheta_5**2*l_1*m_5*r_5*np.sin(theta_5 - theta_1) + dtheta_4*(dtheta_4*l_1*l_4*m_5*np.sin(theta_4 - theta_1) + dtheta_4*l_1*m_4*r_4*np.sin(theta_4 - theta_1)) + dtheta_3*(dtheta_3*l_1*l_3*m_4*np.sin(theta_3 - theta_1) + dtheta_3*l_1*l_3*m_5*np.sin(theta_3 - theta_1) + dtheta_3*l_1*m_3*r_3*np.sin(theta_3 - theta_1)) + dtheta_2*mu_2 + dtheta_2*(dtheta_2*l_1*l_2*m_3*np.sin(theta_2 - theta_1) + dtheta_2*l_1*l_2*m_4*np.sin(theta_2 - theta_1) + dtheta_2*l_1*l_2*m_5*np.sin(theta_2 - theta_1) + dtheta_2*l_1*m_2*r_2*np.sin(theta_2 - theta_1)) - dtheta_1*(mu_1 + mu_2) - ddx_c*(-l_1*m_2 - l_1*m_3 - l_1*m_4 - l_1*m_5 - m_1*r_1)*np.cos(theta_1) - g*(-l_1*m_2 - l_1*m_3 - l_1*m_4 - l_1*m_5 - m_1*r_1)*np.sin(theta_1))*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2))/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2) - (-l_1**2*m_5*r_5*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_5 - theta_1)*np.cos(theta_3 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_3*m_5*r_5*np.cos(theta_5 - theta_3) - (-l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_3 - theta_2))*(-l_1**2*m_5*r_5*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_5 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*m_5*r_5*np.cos(theta_5 - theta_2))/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2))*(dtheta_5**2*l_3*m_5*r_5*np.sin(theta_5 - theta_3) + dtheta_4*mu_4 + dtheta_4*(dtheta_4*l_3*l_4*m_5*np.sin(theta_4 - theta_3) + dtheta_4*l_3*m_4*r_4*np.sin(theta_4 - theta_3)) - dtheta_3*(mu_3 + mu_4) + dtheta_2*mu_3 + dtheta_2*(-dtheta_2*l_2*l_3*m_4*np.sin(theta_3 - theta_2) - dtheta_2*l_2*l_3*m_5*np.sin(theta_3 - theta_2) - dtheta_2*l_2*m_3*r_3*np.sin(theta_3 - theta_2)) + dtheta_1*(-dtheta_1*l_1*l_3*m_4*np.sin(theta_3 - theta_1) - dtheta_1*l_1*l_3*m_5*np.sin(theta_3 - theta_1) - dtheta_1*l_1*m_3*r_3*np.sin(theta_3 - theta_1)) - ddx_c*(-l_3*m_4 - l_3*m_5 - m_3*r_3)*np.cos(theta_3) - g*(-l_3*m_4 - l_3*m_5 - m_3*r_3)*np.sin(theta_3) - l_1*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(dtheta_5**2*l_1*m_5*r_5*np.sin(theta_5 - theta_1) + dtheta_4*(dtheta_4*l_1*l_4*m_5*np.sin(theta_4 - theta_1) + dtheta_4*l_1*m_4*r_4*np.sin(theta_4 - theta_1)) + dtheta_3*(dtheta_3*l_1*l_3*m_4*np.sin(theta_3 - theta_1) + dtheta_3*l_1*l_3*m_5*np.sin(theta_3 - theta_1) + dtheta_3*l_1*m_3*r_3*np.sin(theta_3 - theta_1)) + dtheta_2*mu_2 + dtheta_2*(dtheta_2*l_1*l_2*m_3*np.sin(theta_2 - theta_1) + dtheta_2*l_1*l_2*m_4*np.sin(theta_2 - theta_1) + dtheta_2*l_1*l_2*m_5*np.sin(theta_2 - theta_1) + dtheta_2*l_1*m_2*r_2*np.sin(theta_2 - theta_1)) - dtheta_1*(mu_1 + mu_2) - ddx_c*(-l_1*m_2 - l_1*m_3 - l_1*m_4 - l_1*m_5 - m_1*r_1)*np.cos(theta_1) - g*(-l_1*m_2 - l_1*m_3 - l_1*m_4 - l_1*m_5 - m_1*r_1)*np.sin(theta_1))*np.cos(theta_3 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) - (-l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_3 - theta_2))*(dtheta_5**2*l_2*m_5*r_5*np.sin(theta_5 - theta_2) + dtheta_4*(dtheta_4*l_2*l_4*m_5*np.sin(theta_4 - theta_2) + dtheta_4*l_2*m_4*r_4*np.sin(theta_4 - theta_2)) + dtheta_3*mu_3 + dtheta_3*(dtheta_3*l_2*l_3*m_4*np.sin(theta_3 - theta_2) + dtheta_3*l_2*l_3*m_5*np.sin(theta_3 - theta_2) + dtheta_3*l_2*m_3*r_3*np.sin(theta_3 - theta_2)) - dtheta_2*(mu_2 + mu_3) + dtheta_1*mu_2 + dtheta_1*(-dtheta_1*l_1*l_2*m_3*np.sin(theta_2 - theta_1) - dtheta_1*l_1*l_2*m_4*np.sin(theta_2 - theta_1) - dtheta_1*l_1*l_2*m_5*np.sin(theta_2 - theta_1) - dtheta_1*l_1*m_2*r_2*np.sin(theta_2 - theta_1)) - ddx_c*(-l_2*m_3 - l_2*m_4 - l_2*m_5 - m_2*r_2)*np.cos(theta_2) - g*(-l_2*m_3 - l_2*m_4 - l_2*m_5 - m_2*r_2)*np.sin(theta_2) - l_1*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*(dtheta_5**2*l_1*m_5*r_5*np.sin(theta_5 - theta_1) + dtheta_4*(dtheta_4*l_1*l_4*m_5*np.sin(theta_4 - theta_1) + dtheta_4*l_1*m_4*r_4*np.sin(theta_4 - theta_1)) + dtheta_3*(dtheta_3*l_1*l_3*m_4*np.sin(theta_3 - theta_1) + dtheta_3*l_1*l_3*m_5*np.sin(theta_3 - theta_1) + dtheta_3*l_1*m_3*r_3*np.sin(theta_3 - theta_1)) + dtheta_2*mu_2 + dtheta_2*(dtheta_2*l_1*l_2*m_3*np.sin(theta_2 - theta_1) + dtheta_2*l_1*l_2*m_4*np.sin(theta_2 - theta_1) + dtheta_2*l_1*l_2*m_5*np.sin(theta_2 - theta_1) + dtheta_2*l_1*m_2*r_2*np.sin(theta_2 - theta_1)) - dtheta_1*(mu_1 + mu_2) - ddx_c*(-l_1*m_2 - l_1*m_3 - l_1*m_4 - l_1*m_5 - m_1*r_1)*np.cos(theta_1) - g*(-l_1*m_2 - l_1*m_3 - l_1*m_4 - l_1*m_5 - m_1*r_1)*np.sin(theta_1))*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2))/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2))/(I_3 - l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)**2*np.cos(theta_3 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_3**2*m_4 + l_3**2*m_5 + m_3*r_3**2 - (-l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_3 - theta_2))**2/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2)) - (-l_1**2*m_5*r_5*(l_4*m_5 + m_4*r_4)*np.cos(theta_5 - theta_1)*np.cos(theta_4 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_4*m_5*r_5*np.cos(theta_5 - theta_4) - (-l_1**2*(l_4*m_5 + m_4*r_4)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_4 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_4*m_5 + m_4*r_4)*np.cos(theta_4 - theta_2))*(-l_1**2*m_5*r_5*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_5 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*m_5*r_5*np.cos(theta_5 - theta_2))/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2) - (-l_1**2*(l_4*m_5 + m_4*r_4)*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_4 - theta_1)*np.cos(theta_3 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_3*(l_4*m_5 + m_4*r_4)*np.cos(theta_4 - theta_3) - (-l_1**2*(l_4*m_5 + m_4*r_4)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_4 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_4*m_5 + m_4*r_4)*np.cos(theta_4 - theta_2))*(-l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_3 - theta_2))/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2))*(-l_1**2*m_5*r_5*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_5 - theta_1)*np.cos(theta_3 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_3*m_5*r_5*np.cos(theta_5 - theta_3) - (-l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_3 - theta_2))*(-l_1**2*m_5*r_5*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_5 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*m_5*r_5*np.cos(theta_5 - theta_2))/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2))/(I_3 - l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)**2*np.cos(theta_3 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_3**2*m_4 + l_3**2*m_5 + m_3*r_3**2 - (-l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_3 - theta_2))**2/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2)))*(dtheta_5**2*l_4*m_5*r_5*np.sin(theta_5 - theta_4) + dtheta_5*mu_5 - dtheta_4*(mu_4 + mu_5) + dtheta_3*mu_4 + dtheta_3*(-dtheta_3*l_3*l_4*m_5*np.sin(theta_4 - theta_3) - dtheta_3*l_3*m_4*r_4*np.sin(theta_4 - theta_3)) + dtheta_2*(-dtheta_2*l_2*l_4*m_5*np.sin(theta_4 - theta_2) - dtheta_2*l_2*m_4*r_4*np.sin(theta_4 - theta_2)) + dtheta_1*(-dtheta_1*l_1*l_4*m_5*np.sin(theta_4 - theta_1) - dtheta_1*l_1*m_4*r_4*np.sin(theta_4 - theta_1)) - ddx_c*(-l_4*m_5 - m_4*r_4)*np.cos(theta_4) - g*(-l_4*m_5 - m_4*r_4)*np.sin(theta_4) - l_1*(l_4*m_5 + m_4*r_4)*(dtheta_5**2*l_1*m_5*r_5*np.sin(theta_5 - theta_1) + dtheta_4*(dtheta_4*l_1*l_4*m_5*np.sin(theta_4 - theta_1) + dtheta_4*l_1*m_4*r_4*np.sin(theta_4 - theta_1)) + dtheta_3*(dtheta_3*l_1*l_3*m_4*np.sin(theta_3 - theta_1) + dtheta_3*l_1*l_3*m_5*np.sin(theta_3 - theta_1) + dtheta_3*l_1*m_3*r_3*np.sin(theta_3 - theta_1)) + dtheta_2*mu_2 + dtheta_2*(dtheta_2*l_1*l_2*m_3*np.sin(theta_2 - theta_1) + dtheta_2*l_1*l_2*m_4*np.sin(theta_2 - theta_1) + dtheta_2*l_1*l_2*m_5*np.sin(theta_2 - theta_1) + dtheta_2*l_1*m_2*r_2*np.sin(theta_2 - theta_1)) - dtheta_1*(mu_1 + mu_2) - ddx_c*(-l_1*m_2 - l_1*m_3 - l_1*m_4 - l_1*m_5 - m_1*r_1)*np.cos(theta_1) - g*(-l_1*m_2 - l_1*m_3 - l_1*m_4 - l_1*m_5 - m_1*r_1)*np.sin(theta_1))*np.cos(theta_4 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) - (-l_1**2*(l_4*m_5 + m_4*r_4)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_4 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_4*m_5 + m_4*r_4)*np.cos(theta_4 - theta_2))*(dtheta_5**2*l_2*m_5*r_5*np.sin(theta_5 - theta_2) + dtheta_4*(dtheta_4*l_2*l_4*m_5*np.sin(theta_4 - theta_2) + dtheta_4*l_2*m_4*r_4*np.sin(theta_4 - theta_2)) + dtheta_3*mu_3 + dtheta_3*(dtheta_3*l_2*l_3*m_4*np.sin(theta_3 - theta_2) + dtheta_3*l_2*l_3*m_5*np.sin(theta_3 - theta_2) + dtheta_3*l_2*m_3*r_3*np.sin(theta_3 - theta_2)) - dtheta_2*(mu_2 + mu_3) + dtheta_1*mu_2 + dtheta_1*(-dtheta_1*l_1*l_2*m_3*np.sin(theta_2 - theta_1) - dtheta_1*l_1*l_2*m_4*np.sin(theta_2 - theta_1) - dtheta_1*l_1*l_2*m_5*np.sin(theta_2 - theta_1) - dtheta_1*l_1*m_2*r_2*np.sin(theta_2 - theta_1)) - ddx_c*(-l_2*m_3 - l_2*m_4 - l_2*m_5 - m_2*r_2)*np.cos(theta_2) - g*(-l_2*m_3 - l_2*m_4 - l_2*m_5 - m_2*r_2)*np.sin(theta_2) - l_1*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*(dtheta_5**2*l_1*m_5*r_5*np.sin(theta_5 - theta_1) + dtheta_4*(dtheta_4*l_1*l_4*m_5*np.sin(theta_4 - theta_1) + dtheta_4*l_1*m_4*r_4*np.sin(theta_4 - theta_1)) + dtheta_3*(dtheta_3*l_1*l_3*m_4*np.sin(theta_3 - theta_1) + dtheta_3*l_1*l_3*m_5*np.sin(theta_3 - theta_1) + dtheta_3*l_1*m_3*r_3*np.sin(theta_3 - theta_1)) + dtheta_2*mu_2 + dtheta_2*(dtheta_2*l_1*l_2*m_3*np.sin(theta_2 - theta_1) + dtheta_2*l_1*l_2*m_4*np.sin(theta_2 - theta_1) + dtheta_2*l_1*l_2*m_5*np.sin(theta_2 - theta_1) + dtheta_2*l_1*m_2*r_2*np.sin(theta_2 - theta_1)) - dtheta_1*(mu_1 + mu_2) - ddx_c*(-l_1*m_2 - l_1*m_3 - l_1*m_4 - l_1*m_5 - m_1*r_1)*np.cos(theta_1) - g*(-l_1*m_2 - l_1*m_3 - l_1*m_4 - l_1*m_5 - m_1*r_1)*np.sin(theta_1))*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2))/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2) - (-l_1**2*(l_4*m_5 + m_4*r_4)*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_4 - theta_1)*np.cos(theta_3 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_3*(l_4*m_5 + m_4*r_4)*np.cos(theta_4 - theta_3) - (-l_1**2*(l_4*m_5 + m_4*r_4)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_4 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_4*m_5 + m_4*r_4)*np.cos(theta_4 - theta_2))*(-l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_3 - theta_2))/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2))*(dtheta_5**2*l_3*m_5*r_5*np.sin(theta_5 - theta_3) + dtheta_4*mu_4 + dtheta_4*(dtheta_4*l_3*l_4*m_5*np.sin(theta_4 - theta_3) + dtheta_4*l_3*m_4*r_4*np.sin(theta_4 - theta_3)) - dtheta_3*(mu_3 + mu_4) + dtheta_2*mu_3 + dtheta_2*(-dtheta_2*l_2*l_3*m_4*np.sin(theta_3 - theta_2) - dtheta_2*l_2*l_3*m_5*np.sin(theta_3 - theta_2) - dtheta_2*l_2*m_3*r_3*np.sin(theta_3 - theta_2)) + dtheta_1*(-dtheta_1*l_1*l_3*m_4*np.sin(theta_3 - theta_1) - dtheta_1*l_1*l_3*m_5*np.sin(theta_3 - theta_1) - dtheta_1*l_1*m_3*r_3*np.sin(theta_3 - theta_1)) - ddx_c*(-l_3*m_4 - l_3*m_5 - m_3*r_3)*np.cos(theta_3) - g*(-l_3*m_4 - l_3*m_5 - m_3*r_3)*np.sin(theta_3) - l_1*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(dtheta_5**2*l_1*m_5*r_5*np.sin(theta_5 - theta_1) + dtheta_4*(dtheta_4*l_1*l_4*m_5*np.sin(theta_4 - theta_1) + dtheta_4*l_1*m_4*r_4*np.sin(theta_4 - theta_1)) + dtheta_3*(dtheta_3*l_1*l_3*m_4*np.sin(theta_3 - theta_1) + dtheta_3*l_1*l_3*m_5*np.sin(theta_3 - theta_1) + dtheta_3*l_1*m_3*r_3*np.sin(theta_3 - theta_1)) + dtheta_2*mu_2 + dtheta_2*(dtheta_2*l_1*l_2*m_3*np.sin(theta_2 - theta_1) + dtheta_2*l_1*l_2*m_4*np.sin(theta_2 - theta_1) + dtheta_2*l_1*l_2*m_5*np.sin(theta_2 - theta_1) + dtheta_2*l_1*m_2*r_2*np.sin(theta_2 - theta_1)) - dtheta_1*(mu_1 + mu_2) - ddx_c*(-l_1*m_2 - l_1*m_3 - l_1*m_4 - l_1*m_5 - m_1*r_1)*np.cos(theta_1) - g*(-l_1*m_2 - l_1*m_3 - l_1*m_4 - l_1*m_5 - m_1*r_1)*np.sin(theta_1))*np.cos(theta_3 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) - (-l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_3 - theta_2))*(dtheta_5**2*l_2*m_5*r_5*np.sin(theta_5 - theta_2) + dtheta_4*(dtheta_4*l_2*l_4*m_5*np.sin(theta_4 - theta_2) + dtheta_4*l_2*m_4*r_4*np.sin(theta_4 - theta_2)) + dtheta_3*mu_3 + dtheta_3*(dtheta_3*l_2*l_3*m_4*np.sin(theta_3 - theta_2) + dtheta_3*l_2*l_3*m_5*np.sin(theta_3 - theta_2) + dtheta_3*l_2*m_3*r_3*np.sin(theta_3 - theta_2)) - dtheta_2*(mu_2 + mu_3) + dtheta_1*mu_2 + dtheta_1*(-dtheta_1*l_1*l_2*m_3*np.sin(theta_2 - theta_1) - dtheta_1*l_1*l_2*m_4*np.sin(theta_2 - theta_1) - dtheta_1*l_1*l_2*m_5*np.sin(theta_2 - theta_1) - dtheta_1*l_1*m_2*r_2*np.sin(theta_2 - theta_1)) - ddx_c*(-l_2*m_3 - l_2*m_4 - l_2*m_5 - m_2*r_2)*np.cos(theta_2) - g*(-l_2*m_3 - l_2*m_4 - l_2*m_5 - m_2*r_2)*np.sin(theta_2) - l_1*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*(dtheta_5**2*l_1*m_5*r_5*np.sin(theta_5 - theta_1) + dtheta_4*(dtheta_4*l_1*l_4*m_5*np.sin(theta_4 - theta_1) + dtheta_4*l_1*m_4*r_4*np.sin(theta_4 - theta_1)) + dtheta_3*(dtheta_3*l_1*l_3*m_4*np.sin(theta_3 - theta_1) + dtheta_3*l_1*l_3*m_5*np.sin(theta_3 - theta_1) + dtheta_3*l_1*m_3*r_3*np.sin(theta_3 - theta_1)) + dtheta_2*mu_2 + dtheta_2*(dtheta_2*l_1*l_2*m_3*np.sin(theta_2 - theta_1) + dtheta_2*l_1*l_2*m_4*np.sin(theta_2 - theta_1) + dtheta_2*l_1*l_2*m_5*np.sin(theta_2 - theta_1) + dtheta_2*l_1*m_2*r_2*np.sin(theta_2 - theta_1)) - dtheta_1*(mu_1 + mu_2) - ddx_c*(-l_1*m_2 - l_1*m_3 - l_1*m_4 - l_1*m_5 - m_1*r_1)*np.cos(theta_1) - g*(-l_1*m_2 - l_1*m_3 - l_1*m_4 - l_1*m_5 - m_1*r_1)*np.sin(theta_1))*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2))/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2))/(I_3 - l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)**2*np.cos(theta_3 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_3**2*m_4 + l_3**2*m_5 + m_3*r_3**2 - (-l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_3 - theta_2))**2/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2)))/(I_4 - l_1**2*(l_4*m_5 + m_4*r_4)**2*np.cos(theta_4 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_4**2*m_5 + m_4*r_4**2 - (-l_1**2*(l_4*m_5 + m_4*r_4)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_4 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_4*m_5 + m_4*r_4)*np.cos(theta_4 - theta_2))**2/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2) - (-l_1**2*(l_4*m_5 + m_4*r_4)*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_4 - theta_1)*np.cos(theta_3 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_3*(l_4*m_5 + m_4*r_4)*np.cos(theta_4 - theta_3) - (-l_1**2*(l_4*m_5 + m_4*r_4)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_4 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_4*m_5 + m_4*r_4)*np.cos(theta_4 - theta_2))*(-l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_3 - theta_2))/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2))**2/(I_3 - l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)**2*np.cos(theta_3 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_3**2*m_4 + l_3**2*m_5 + m_3*r_3**2 - (-l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_3 - theta_2))**2/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2))))/(I_5 - l_1**2*m_5**2*r_5**2*np.cos(theta_5 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + m_5*r_5**2 - (-l_1**2*m_5*r_5*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_5 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*m_5*r_5*np.cos(theta_5 - theta_2))**2/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2) - (-l_1**2*m_5*r_5*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_5 - theta_1)*np.cos(theta_3 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_3*m_5*r_5*np.cos(theta_5 - theta_3) - (-l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_3 - theta_2))*(-l_1**2*m_5*r_5*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_5 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*m_5*r_5*np.cos(theta_5 - theta_2))/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2))**2/(I_3 - l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)**2*np.cos(theta_3 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_3**2*m_4 + l_3**2*m_5 + m_3*r_3**2 - (-l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_3 - theta_2))**2/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2)) - (-l_1**2*m_5*r_5*(l_4*m_5 + m_4*r_4)*np.cos(theta_5 - theta_1)*np.cos(theta_4 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_4*m_5*r_5*np.cos(theta_5 - theta_4) - (-l_1**2*(l_4*m_5 + m_4*r_4)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_4 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_4*m_5 + m_4*r_4)*np.cos(theta_4 - theta_2))*(-l_1**2*m_5*r_5*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_5 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*m_5*r_5*np.cos(theta_5 - theta_2))/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2) - (-l_1**2*(l_4*m_5 + m_4*r_4)*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_4 - theta_1)*np.cos(theta_3 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_3*(l_4*m_5 + m_4*r_4)*np.cos(theta_4 - theta_3) - (-l_1**2*(l_4*m_5 + m_4*r_4)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_4 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_4*m_5 + m_4*r_4)*np.cos(theta_4 - theta_2))*(-l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_3 - theta_2))/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2))*(-l_1**2*m_5*r_5*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_5 - theta_1)*np.cos(theta_3 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_3*m_5*r_5*np.cos(theta_5 - theta_3) - (-l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_3 - theta_2))*(-l_1**2*m_5*r_5*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_5 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*m_5*r_5*np.cos(theta_5 - theta_2))/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2))/(I_3 - l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)**2*np.cos(theta_3 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_3**2*m_4 + l_3**2*m_5 + m_3*r_3**2 - (-l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_3 - theta_2))**2/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2)))**2/(I_4 - l_1**2*(l_4*m_5 + m_4*r_4)**2*np.cos(theta_4 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_4**2*m_5 + m_4*r_4**2 - (-l_1**2*(l_4*m_5 + m_4*r_4)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_4 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_4*m_5 + m_4*r_4)*np.cos(theta_4 - theta_2))**2/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2) - (-l_1**2*(l_4*m_5 + m_4*r_4)*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_4 - theta_1)*np.cos(theta_3 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_3*(l_4*m_5 + m_4*r_4)*np.cos(theta_4 - theta_3) - (-l_1**2*(l_4*m_5 + m_4*r_4)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_4 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_4*m_5 + m_4*r_4)*np.cos(theta_4 - theta_2))*(-l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_3 - theta_2))/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2))**2/(I_3 - l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)**2*np.cos(theta_3 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_3**2*m_4 + l_3**2*m_5 + m_3*r_3**2 - (-l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_3 - theta_2))**2/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2)))))/(I_4 - l_1**2*(l_4*m_5 + m_4*r_4)**2*np.cos(theta_4 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_4**2*m_5 + m_4*r_4**2 - (-l_1**2*(l_4*m_5 + m_4*r_4)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_4 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_4*m_5 + m_4*r_4)*np.cos(theta_4 - theta_2))**2/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2) - (-l_1**2*(l_4*m_5 + m_4*r_4)*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_4 - theta_1)*np.cos(theta_3 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_3*(l_4*m_5 + m_4*r_4)*np.cos(theta_4 - theta_3) - (-l_1**2*(l_4*m_5 + m_4*r_4)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_4 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_4*m_5 + m_4*r_4)*np.cos(theta_4 - theta_2))*(-l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_3 - theta_2))/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2))**2/(I_3 - l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)**2*np.cos(theta_3 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_3**2*m_4 + l_3**2*m_5 + m_3*r_3**2 - (-l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_3 - theta_2))**2/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2))) - (-l_1**2*m_5*r_5*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_5 - theta_1)*np.cos(theta_3 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_3*m_5*r_5*np.cos(theta_5 - theta_3) - (-l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_3 - theta_2))*(-l_1**2*m_5*r_5*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_5 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*m_5*r_5*np.cos(theta_5 - theta_2))/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2))*(-dtheta_5*mu_5 - dtheta_4**2*l_4*m_5*r_5*np.sin(theta_5 - theta_4) + dtheta_4*mu_5 - dtheta_3**2*l_3*m_5*r_5*np.sin(theta_5 - theta_3) - dtheta_2**2*l_2*m_5*r_5*np.sin(theta_5 - theta_2) - dtheta_1**2*l_1*m_5*r_5*np.sin(theta_5 - theta_1) + ddx_c*m_5*r_5*np.cos(theta_5) + g*m_5*r_5*np.sin(theta_5) - l_1*m_5*r_5*(dtheta_5**2*l_1*m_5*r_5*np.sin(theta_5 - theta_1) + dtheta_4*(dtheta_4*l_1*l_4*m_5*np.sin(theta_4 - theta_1) + dtheta_4*l_1*m_4*r_4*np.sin(theta_4 - theta_1)) + dtheta_3*(dtheta_3*l_1*l_3*m_4*np.sin(theta_3 - theta_1) + dtheta_3*l_1*l_3*m_5*np.sin(theta_3 - theta_1) + dtheta_3*l_1*m_3*r_3*np.sin(theta_3 - theta_1)) + dtheta_2*mu_2 + dtheta_2*(dtheta_2*l_1*l_2*m_3*np.sin(theta_2 - theta_1) + dtheta_2*l_1*l_2*m_4*np.sin(theta_2 - theta_1) + dtheta_2*l_1*l_2*m_5*np.sin(theta_2 - theta_1) + dtheta_2*l_1*m_2*r_2*np.sin(theta_2 - theta_1)) - dtheta_1*(mu_1 + mu_2) - ddx_c*(-l_1*m_2 - l_1*m_3 - l_1*m_4 - l_1*m_5 - m_1*r_1)*np.cos(theta_1) - g*(-l_1*m_2 - l_1*m_3 - l_1*m_4 - l_1*m_5 - m_1*r_1)*np.sin(theta_1))*np.cos(theta_5 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) - (-l_1**2*m_5*r_5*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_5 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*m_5*r_5*np.cos(theta_5 - theta_2))*(dtheta_5**2*l_2*m_5*r_5*np.sin(theta_5 - theta_2) + dtheta_4*(dtheta_4*l_2*l_4*m_5*np.sin(theta_4 - theta_2) + dtheta_4*l_2*m_4*r_4*np.sin(theta_4 - theta_2)) + dtheta_3*mu_3 + dtheta_3*(dtheta_3*l_2*l_3*m_4*np.sin(theta_3 - theta_2) + dtheta_3*l_2*l_3*m_5*np.sin(theta_3 - theta_2) + dtheta_3*l_2*m_3*r_3*np.sin(theta_3 - theta_2)) - dtheta_2*(mu_2 + mu_3) + dtheta_1*mu_2 + dtheta_1*(-dtheta_1*l_1*l_2*m_3*np.sin(theta_2 - theta_1) - dtheta_1*l_1*l_2*m_4*np.sin(theta_2 - theta_1) - dtheta_1*l_1*l_2*m_5*np.sin(theta_2 - theta_1) - dtheta_1*l_1*m_2*r_2*np.sin(theta_2 - theta_1)) - ddx_c*(-l_2*m_3 - l_2*m_4 - l_2*m_5 - m_2*r_2)*np.cos(theta_2) - g*(-l_2*m_3 - l_2*m_4 - l_2*m_5 - m_2*r_2)*np.sin(theta_2) - l_1*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*(dtheta_5**2*l_1*m_5*r_5*np.sin(theta_5 - theta_1) + dtheta_4*(dtheta_4*l_1*l_4*m_5*np.sin(theta_4 - theta_1) + dtheta_4*l_1*m_4*r_4*np.sin(theta_4 - theta_1)) + dtheta_3*(dtheta_3*l_1*l_3*m_4*np.sin(theta_3 - theta_1) + dtheta_3*l_1*l_3*m_5*np.sin(theta_3 - theta_1) + dtheta_3*l_1*m_3*r_3*np.sin(theta_3 - theta_1)) + dtheta_2*mu_2 + dtheta_2*(dtheta_2*l_1*l_2*m_3*np.sin(theta_2 - theta_1) + dtheta_2*l_1*l_2*m_4*np.sin(theta_2 - theta_1) + dtheta_2*l_1*l_2*m_5*np.sin(theta_2 - theta_1) + dtheta_2*l_1*m_2*r_2*np.sin(theta_2 - theta_1)) - dtheta_1*(mu_1 + mu_2) - ddx_c*(-l_1*m_2 - l_1*m_3 - l_1*m_4 - l_1*m_5 - m_1*r_1)*np.cos(theta_1) - g*(-l_1*m_2 - l_1*m_3 - l_1*m_4 - l_1*m_5 - m_1*r_1)*np.sin(theta_1))*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2))/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2) - (-l_1**2*m_5*r_5*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_5 - theta_1)*np.cos(theta_3 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_3*m_5*r_5*np.cos(theta_5 - theta_3) - (-l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_3 - theta_2))*(-l_1**2*m_5*r_5*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_5 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*m_5*r_5*np.cos(theta_5 - theta_2))/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2))*(dtheta_5**2*l_3*m_5*r_5*np.sin(theta_5 - theta_3) + dtheta_4*mu_4 + dtheta_4*(dtheta_4*l_3*l_4*m_5*np.sin(theta_4 - theta_3) + dtheta_4*l_3*m_4*r_4*np.sin(theta_4 - theta_3)) - dtheta_3*(mu_3 + mu_4) + dtheta_2*mu_3 + dtheta_2*(-dtheta_2*l_2*l_3*m_4*np.sin(theta_3 - theta_2) - dtheta_2*l_2*l_3*m_5*np.sin(theta_3 - theta_2) - dtheta_2*l_2*m_3*r_3*np.sin(theta_3 - theta_2)) + dtheta_1*(-dtheta_1*l_1*l_3*m_4*np.sin(theta_3 - theta_1) - dtheta_1*l_1*l_3*m_5*np.sin(theta_3 - theta_1) - dtheta_1*l_1*m_3*r_3*np.sin(theta_3 - theta_1)) - ddx_c*(-l_3*m_4 - l_3*m_5 - m_3*r_3)*np.cos(theta_3) - g*(-l_3*m_4 - l_3*m_5 - m_3*r_3)*np.sin(theta_3) - l_1*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(dtheta_5**2*l_1*m_5*r_5*np.sin(theta_5 - theta_1) + dtheta_4*(dtheta_4*l_1*l_4*m_5*np.sin(theta_4 - theta_1) + dtheta_4*l_1*m_4*r_4*np.sin(theta_4 - theta_1)) + dtheta_3*(dtheta_3*l_1*l_3*m_4*np.sin(theta_3 - theta_1) + dtheta_3*l_1*l_3*m_5*np.sin(theta_3 - theta_1) + dtheta_3*l_1*m_3*r_3*np.sin(theta_3 - theta_1)) + dtheta_2*mu_2 + dtheta_2*(dtheta_2*l_1*l_2*m_3*np.sin(theta_2 - theta_1) + dtheta_2*l_1*l_2*m_4*np.sin(theta_2 - theta_1) + dtheta_2*l_1*l_2*m_5*np.sin(theta_2 - theta_1) + dtheta_2*l_1*m_2*r_2*np.sin(theta_2 - theta_1)) - dtheta_1*(mu_1 + mu_2) - ddx_c*(-l_1*m_2 - l_1*m_3 - l_1*m_4 - l_1*m_5 - m_1*r_1)*np.cos(theta_1) - g*(-l_1*m_2 - l_1*m_3 - l_1*m_4 - l_1*m_5 - m_1*r_1)*np.sin(theta_1))*np.cos(theta_3 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) - (-l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_3 - theta_2))*(dtheta_5**2*l_2*m_5*r_5*np.sin(theta_5 - theta_2) + dtheta_4*(dtheta_4*l_2*l_4*m_5*np.sin(theta_4 - theta_2) + dtheta_4*l_2*m_4*r_4*np.sin(theta_4 - theta_2)) + dtheta_3*mu_3 + dtheta_3*(dtheta_3*l_2*l_3*m_4*np.sin(theta_3 - theta_2) + dtheta_3*l_2*l_3*m_5*np.sin(theta_3 - theta_2) + dtheta_3*l_2*m_3*r_3*np.sin(theta_3 - theta_2)) - dtheta_2*(mu_2 + mu_3) + dtheta_1*mu_2 + dtheta_1*(-dtheta_1*l_1*l_2*m_3*np.sin(theta_2 - theta_1) - dtheta_1*l_1*l_2*m_4*np.sin(theta_2 - theta_1) - dtheta_1*l_1*l_2*m_5*np.sin(theta_2 - theta_1) - dtheta_1*l_1*m_2*r_2*np.sin(theta_2 - theta_1)) - ddx_c*(-l_2*m_3 - l_2*m_4 - l_2*m_5 - m_2*r_2)*np.cos(theta_2) - g*(-l_2*m_3 - l_2*m_4 - l_2*m_5 - m_2*r_2)*np.sin(theta_2) - l_1*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*(dtheta_5**2*l_1*m_5*r_5*np.sin(theta_5 - theta_1) + dtheta_4*(dtheta_4*l_1*l_4*m_5*np.sin(theta_4 - theta_1) + dtheta_4*l_1*m_4*r_4*np.sin(theta_4 - theta_1)) + dtheta_3*(dtheta_3*l_1*l_3*m_4*np.sin(theta_3 - theta_1) + dtheta_3*l_1*l_3*m_5*np.sin(theta_3 - theta_1) + dtheta_3*l_1*m_3*r_3*np.sin(theta_3 - theta_1)) + dtheta_2*mu_2 + dtheta_2*(dtheta_2*l_1*l_2*m_3*np.sin(theta_2 - theta_1) + dtheta_2*l_1*l_2*m_4*np.sin(theta_2 - theta_1) + dtheta_2*l_1*l_2*m_5*np.sin(theta_2 - theta_1) + dtheta_2*l_1*m_2*r_2*np.sin(theta_2 - theta_1)) - dtheta_1*(mu_1 + mu_2) - ddx_c*(-l_1*m_2 - l_1*m_3 - l_1*m_4 - l_1*m_5 - m_1*r_1)*np.cos(theta_1) - g*(-l_1*m_2 - l_1*m_3 - l_1*m_4 - l_1*m_5 - m_1*r_1)*np.sin(theta_1))*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2))/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2))/(I_3 - l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)**2*np.cos(theta_3 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_3**2*m_4 + l_3**2*m_5 + m_3*r_3**2 - (-l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_3 - theta_2))**2/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2)) - (-l_1**2*m_5*r_5*(l_4*m_5 + m_4*r_4)*np.cos(theta_5 - theta_1)*np.cos(theta_4 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_4*m_5*r_5*np.cos(theta_5 - theta_4) - (-l_1**2*(l_4*m_5 + m_4*r_4)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_4 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_4*m_5 + m_4*r_4)*np.cos(theta_4 - theta_2))*(-l_1**2*m_5*r_5*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_5 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*m_5*r_5*np.cos(theta_5 - theta_2))/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2) - (-l_1**2*(l_4*m_5 + m_4*r_4)*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_4 - theta_1)*np.cos(theta_3 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_3*(l_4*m_5 + m_4*r_4)*np.cos(theta_4 - theta_3) - (-l_1**2*(l_4*m_5 + m_4*r_4)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_4 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_4*m_5 + m_4*r_4)*np.cos(theta_4 - theta_2))*(-l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_3 - theta_2))/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2))*(-l_1**2*m_5*r_5*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_5 - theta_1)*np.cos(theta_3 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_3*m_5*r_5*np.cos(theta_5 - theta_3) - (-l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_3 - theta_2))*(-l_1**2*m_5*r_5*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_5 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*m_5*r_5*np.cos(theta_5 - theta_2))/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2))/(I_3 - l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)**2*np.cos(theta_3 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_3**2*m_4 + l_3**2*m_5 + m_3*r_3**2 - (-l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_3 - theta_2))**2/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2)))*(dtheta_5**2*l_4*m_5*r_5*np.sin(theta_5 - theta_4) + dtheta_5*mu_5 - dtheta_4*(mu_4 + mu_5) + dtheta_3*mu_4 + dtheta_3*(-dtheta_3*l_3*l_4*m_5*np.sin(theta_4 - theta_3) - dtheta_3*l_3*m_4*r_4*np.sin(theta_4 - theta_3)) + dtheta_2*(-dtheta_2*l_2*l_4*m_5*np.sin(theta_4 - theta_2) - dtheta_2*l_2*m_4*r_4*np.sin(theta_4 - theta_2)) + dtheta_1*(-dtheta_1*l_1*l_4*m_5*np.sin(theta_4 - theta_1) - dtheta_1*l_1*m_4*r_4*np.sin(theta_4 - theta_1)) - ddx_c*(-l_4*m_5 - m_4*r_4)*np.cos(theta_4) - g*(-l_4*m_5 - m_4*r_4)*np.sin(theta_4) - l_1*(l_4*m_5 + m_4*r_4)*(dtheta_5**2*l_1*m_5*r_5*np.sin(theta_5 - theta_1) + dtheta_4*(dtheta_4*l_1*l_4*m_5*np.sin(theta_4 - theta_1) + dtheta_4*l_1*m_4*r_4*np.sin(theta_4 - theta_1)) + dtheta_3*(dtheta_3*l_1*l_3*m_4*np.sin(theta_3 - theta_1) + dtheta_3*l_1*l_3*m_5*np.sin(theta_3 - theta_1) + dtheta_3*l_1*m_3*r_3*np.sin(theta_3 - theta_1)) + dtheta_2*mu_2 + dtheta_2*(dtheta_2*l_1*l_2*m_3*np.sin(theta_2 - theta_1) + dtheta_2*l_1*l_2*m_4*np.sin(theta_2 - theta_1) + dtheta_2*l_1*l_2*m_5*np.sin(theta_2 - theta_1) + dtheta_2*l_1*m_2*r_2*np.sin(theta_2 - theta_1)) - dtheta_1*(mu_1 + mu_2) - ddx_c*(-l_1*m_2 - l_1*m_3 - l_1*m_4 - l_1*m_5 - m_1*r_1)*np.cos(theta_1) - g*(-l_1*m_2 - l_1*m_3 - l_1*m_4 - l_1*m_5 - m_1*r_1)*np.sin(theta_1))*np.cos(theta_4 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) - (-l_1**2*(l_4*m_5 + m_4*r_4)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_4 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_4*m_5 + m_4*r_4)*np.cos(theta_4 - theta_2))*(dtheta_5**2*l_2*m_5*r_5*np.sin(theta_5 - theta_2) + dtheta_4*(dtheta_4*l_2*l_4*m_5*np.sin(theta_4 - theta_2) + dtheta_4*l_2*m_4*r_4*np.sin(theta_4 - theta_2)) + dtheta_3*mu_3 + dtheta_3*(dtheta_3*l_2*l_3*m_4*np.sin(theta_3 - theta_2) + dtheta_3*l_2*l_3*m_5*np.sin(theta_3 - theta_2) + dtheta_3*l_2*m_3*r_3*np.sin(theta_3 - theta_2)) - dtheta_2*(mu_2 + mu_3) + dtheta_1*mu_2 + dtheta_1*(-dtheta_1*l_1*l_2*m_3*np.sin(theta_2 - theta_1) - dtheta_1*l_1*l_2*m_4*np.sin(theta_2 - theta_1) - dtheta_1*l_1*l_2*m_5*np.sin(theta_2 - theta_1) - dtheta_1*l_1*m_2*r_2*np.sin(theta_2 - theta_1)) - ddx_c*(-l_2*m_3 - l_2*m_4 - l_2*m_5 - m_2*r_2)*np.cos(theta_2) - g*(-l_2*m_3 - l_2*m_4 - l_2*m_5 - m_2*r_2)*np.sin(theta_2) - l_1*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*(dtheta_5**2*l_1*m_5*r_5*np.sin(theta_5 - theta_1) + dtheta_4*(dtheta_4*l_1*l_4*m_5*np.sin(theta_4 - theta_1) + dtheta_4*l_1*m_4*r_4*np.sin(theta_4 - theta_1)) + dtheta_3*(dtheta_3*l_1*l_3*m_4*np.sin(theta_3 - theta_1) + dtheta_3*l_1*l_3*m_5*np.sin(theta_3 - theta_1) + dtheta_3*l_1*m_3*r_3*np.sin(theta_3 - theta_1)) + dtheta_2*mu_2 + dtheta_2*(dtheta_2*l_1*l_2*m_3*np.sin(theta_2 - theta_1) + dtheta_2*l_1*l_2*m_4*np.sin(theta_2 - theta_1) + dtheta_2*l_1*l_2*m_5*np.sin(theta_2 - theta_1) + dtheta_2*l_1*m_2*r_2*np.sin(theta_2 - theta_1)) - dtheta_1*(mu_1 + mu_2) - ddx_c*(-l_1*m_2 - l_1*m_3 - l_1*m_4 - l_1*m_5 - m_1*r_1)*np.cos(theta_1) - g*(-l_1*m_2 - l_1*m_3 - l_1*m_4 - l_1*m_5 - m_1*r_1)*np.sin(theta_1))*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2))/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2) - (-l_1**2*(l_4*m_5 + m_4*r_4)*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_4 - theta_1)*np.cos(theta_3 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_3*(l_4*m_5 + m_4*r_4)*np.cos(theta_4 - theta_3) - (-l_1**2*(l_4*m_5 + m_4*r_4)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_4 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_4*m_5 + m_4*r_4)*np.cos(theta_4 - theta_2))*(-l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_3 - theta_2))/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2))*(dtheta_5**2*l_3*m_5*r_5*np.sin(theta_5 - theta_3) + dtheta_4*mu_4 + dtheta_4*(dtheta_4*l_3*l_4*m_5*np.sin(theta_4 - theta_3) + dtheta_4*l_3*m_4*r_4*np.sin(theta_4 - theta_3)) - dtheta_3*(mu_3 + mu_4) + dtheta_2*mu_3 + dtheta_2*(-dtheta_2*l_2*l_3*m_4*np.sin(theta_3 - theta_2) - dtheta_2*l_2*l_3*m_5*np.sin(theta_3 - theta_2) - dtheta_2*l_2*m_3*r_3*np.sin(theta_3 - theta_2)) + dtheta_1*(-dtheta_1*l_1*l_3*m_4*np.sin(theta_3 - theta_1) - dtheta_1*l_1*l_3*m_5*np.sin(theta_3 - theta_1) - dtheta_1*l_1*m_3*r_3*np.sin(theta_3 - theta_1)) - ddx_c*(-l_3*m_4 - l_3*m_5 - m_3*r_3)*np.cos(theta_3) - g*(-l_3*m_4 - l_3*m_5 - m_3*r_3)*np.sin(theta_3) - l_1*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(dtheta_5**2*l_1*m_5*r_5*np.sin(theta_5 - theta_1) + dtheta_4*(dtheta_4*l_1*l_4*m_5*np.sin(theta_4 - theta_1) + dtheta_4*l_1*m_4*r_4*np.sin(theta_4 - theta_1)) + dtheta_3*(dtheta_3*l_1*l_3*m_4*np.sin(theta_3 - theta_1) + dtheta_3*l_1*l_3*m_5*np.sin(theta_3 - theta_1) + dtheta_3*l_1*m_3*r_3*np.sin(theta_3 - theta_1)) + dtheta_2*mu_2 + dtheta_2*(dtheta_2*l_1*l_2*m_3*np.sin(theta_2 - theta_1) + dtheta_2*l_1*l_2*m_4*np.sin(theta_2 - theta_1) + dtheta_2*l_1*l_2*m_5*np.sin(theta_2 - theta_1) + dtheta_2*l_1*m_2*r_2*np.sin(theta_2 - theta_1)) - dtheta_1*(mu_1 + mu_2) - ddx_c*(-l_1*m_2 - l_1*m_3 - l_1*m_4 - l_1*m_5 - m_1*r_1)*np.cos(theta_1) - g*(-l_1*m_2 - l_1*m_3 - l_1*m_4 - l_1*m_5 - m_1*r_1)*np.sin(theta_1))*np.cos(theta_3 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) - (-l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_3 - theta_2))*(dtheta_5**2*l_2*m_5*r_5*np.sin(theta_5 - theta_2) + dtheta_4*(dtheta_4*l_2*l_4*m_5*np.sin(theta_4 - theta_2) + dtheta_4*l_2*m_4*r_4*np.sin(theta_4 - theta_2)) + dtheta_3*mu_3 + dtheta_3*(dtheta_3*l_2*l_3*m_4*np.sin(theta_3 - theta_2) + dtheta_3*l_2*l_3*m_5*np.sin(theta_3 - theta_2) + dtheta_3*l_2*m_3*r_3*np.sin(theta_3 - theta_2)) - dtheta_2*(mu_2 + mu_3) + dtheta_1*mu_2 + dtheta_1*(-dtheta_1*l_1*l_2*m_3*np.sin(theta_2 - theta_1) - dtheta_1*l_1*l_2*m_4*np.sin(theta_2 - theta_1) - dtheta_1*l_1*l_2*m_5*np.sin(theta_2 - theta_1) - dtheta_1*l_1*m_2*r_2*np.sin(theta_2 - theta_1)) - ddx_c*(-l_2*m_3 - l_2*m_4 - l_2*m_5 - m_2*r_2)*np.cos(theta_2) - g*(-l_2*m_3 - l_2*m_4 - l_2*m_5 - m_2*r_2)*np.sin(theta_2) - l_1*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*(dtheta_5**2*l_1*m_5*r_5*np.sin(theta_5 - theta_1) + dtheta_4*(dtheta_4*l_1*l_4*m_5*np.sin(theta_4 - theta_1) + dtheta_4*l_1*m_4*r_4*np.sin(theta_4 - theta_1)) + dtheta_3*(dtheta_3*l_1*l_3*m_4*np.sin(theta_3 - theta_1) + dtheta_3*l_1*l_3*m_5*np.sin(theta_3 - theta_1) + dtheta_3*l_1*m_3*r_3*np.sin(theta_3 - theta_1)) + dtheta_2*mu_2 + dtheta_2*(dtheta_2*l_1*l_2*m_3*np.sin(theta_2 - theta_1) + dtheta_2*l_1*l_2*m_4*np.sin(theta_2 - theta_1) + dtheta_2*l_1*l_2*m_5*np.sin(theta_2 - theta_1) + dtheta_2*l_1*m_2*r_2*np.sin(theta_2 - theta_1)) - dtheta_1*(mu_1 + mu_2) - ddx_c*(-l_1*m_2 - l_1*m_3 - l_1*m_4 - l_1*m_5 - m_1*r_1)*np.cos(theta_1) - g*(-l_1*m_2 - l_1*m_3 - l_1*m_4 - l_1*m_5 - m_1*r_1)*np.sin(theta_1))*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2))/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2))/(I_3 - l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)**2*np.cos(theta_3 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_3**2*m_4 + l_3**2*m_5 + m_3*r_3**2 - (-l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_3 - theta_2))**2/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2)))/(I_4 - l_1**2*(l_4*m_5 + m_4*r_4)**2*np.cos(theta_4 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_4**2*m_5 + m_4*r_4**2 - (-l_1**2*(l_4*m_5 + m_4*r_4)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_4 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_4*m_5 + m_4*r_4)*np.cos(theta_4 - theta_2))**2/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2) - (-l_1**2*(l_4*m_5 + m_4*r_4)*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_4 - theta_1)*np.cos(theta_3 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_3*(l_4*m_5 + m_4*r_4)*np.cos(theta_4 - theta_3) - (-l_1**2*(l_4*m_5 + m_4*r_4)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_4 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_4*m_5 + m_4*r_4)*np.cos(theta_4 - theta_2))*(-l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_3 - theta_2))/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2))**2/(I_3 - l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)**2*np.cos(theta_3 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_3**2*m_4 + l_3**2*m_5 + m_3*r_3**2 - (-l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_3 - theta_2))**2/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2))))/(I_5 - l_1**2*m_5**2*r_5**2*np.cos(theta_5 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + m_5*r_5**2 - (-l_1**2*m_5*r_5*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_5 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*m_5*r_5*np.cos(theta_5 - theta_2))**2/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2) - (-l_1**2*m_5*r_5*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_5 - theta_1)*np.cos(theta_3 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_3*m_5*r_5*np.cos(theta_5 - theta_3) - (-l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_3 - theta_2))*(-l_1**2*m_5*r_5*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_5 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*m_5*r_5*np.cos(theta_5 - theta_2))/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2))**2/(I_3 - l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)**2*np.cos(theta_3 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_3**2*m_4 + l_3**2*m_5 + m_3*r_3**2 - (-l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_3 - theta_2))**2/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2)) - (-l_1**2*m_5*r_5*(l_4*m_5 + m_4*r_4)*np.cos(theta_5 - theta_1)*np.cos(theta_4 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_4*m_5*r_5*np.cos(theta_5 - theta_4) - (-l_1**2*(l_4*m_5 + m_4*r_4)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_4 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_4*m_5 + m_4*r_4)*np.cos(theta_4 - theta_2))*(-l_1**2*m_5*r_5*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_5 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*m_5*r_5*np.cos(theta_5 - theta_2))/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2) - (-l_1**2*(l_4*m_5 + m_4*r_4)*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_4 - theta_1)*np.cos(theta_3 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_3*(l_4*m_5 + m_4*r_4)*np.cos(theta_4 - theta_3) - (-l_1**2*(l_4*m_5 + m_4*r_4)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_4 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_4*m_5 + m_4*r_4)*np.cos(theta_4 - theta_2))*(-l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_3 - theta_2))/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2))*(-l_1**2*m_5*r_5*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_5 - theta_1)*np.cos(theta_3 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_3*m_5*r_5*np.cos(theta_5 - theta_3) - (-l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_3 - theta_2))*(-l_1**2*m_5*r_5*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_5 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*m_5*r_5*np.cos(theta_5 - theta_2))/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2))/(I_3 - l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)**2*np.cos(theta_3 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_3**2*m_4 + l_3**2*m_5 + m_3*r_3**2 - (-l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_3 - theta_2))**2/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2)))**2/(I_4 - l_1**2*(l_4*m_5 + m_4*r_4)**2*np.cos(theta_4 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_4**2*m_5 + m_4*r_4**2 - (-l_1**2*(l_4*m_5 + m_4*r_4)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_4 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_4*m_5 + m_4*r_4)*np.cos(theta_4 - theta_2))**2/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2) - (-l_1**2*(l_4*m_5 + m_4*r_4)*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_4 - theta_1)*np.cos(theta_3 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_3*(l_4*m_5 + m_4*r_4)*np.cos(theta_4 - theta_3) - (-l_1**2*(l_4*m_5 + m_4*r_4)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_4 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_4*m_5 + m_4*r_4)*np.cos(theta_4 - theta_2))*(-l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_3 - theta_2))/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2))**2/(I_3 - l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)**2*np.cos(theta_3 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_3**2*m_4 + l_3**2*m_5 + m_3*r_3**2 - (-l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_3 - theta_2))**2/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2)))))/(I_3 - l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)**2*np.cos(theta_3 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_3**2*m_4 + l_3**2*m_5 + m_3*r_3**2 - (-l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_3 - theta_2))**2/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2)) - (-l_1**2*m_5*r_5*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_5 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*m_5*r_5*np.cos(theta_5 - theta_2))*(-dtheta_5*mu_5 - dtheta_4**2*l_4*m_5*r_5*np.sin(theta_5 - theta_4) + dtheta_4*mu_5 - dtheta_3**2*l_3*m_5*r_5*np.sin(theta_5 - theta_3) - dtheta_2**2*l_2*m_5*r_5*np.sin(theta_5 - theta_2) - dtheta_1**2*l_1*m_5*r_5*np.sin(theta_5 - theta_1) + ddx_c*m_5*r_5*np.cos(theta_5) + g*m_5*r_5*np.sin(theta_5) - l_1*m_5*r_5*(dtheta_5**2*l_1*m_5*r_5*np.sin(theta_5 - theta_1) + dtheta_4*(dtheta_4*l_1*l_4*m_5*np.sin(theta_4 - theta_1) + dtheta_4*l_1*m_4*r_4*np.sin(theta_4 - theta_1)) + dtheta_3*(dtheta_3*l_1*l_3*m_4*np.sin(theta_3 - theta_1) + dtheta_3*l_1*l_3*m_5*np.sin(theta_3 - theta_1) + dtheta_3*l_1*m_3*r_3*np.sin(theta_3 - theta_1)) + dtheta_2*mu_2 + dtheta_2*(dtheta_2*l_1*l_2*m_3*np.sin(theta_2 - theta_1) + dtheta_2*l_1*l_2*m_4*np.sin(theta_2 - theta_1) + dtheta_2*l_1*l_2*m_5*np.sin(theta_2 - theta_1) + dtheta_2*l_1*m_2*r_2*np.sin(theta_2 - theta_1)) - dtheta_1*(mu_1 + mu_2) - ddx_c*(-l_1*m_2 - l_1*m_3 - l_1*m_4 - l_1*m_5 - m_1*r_1)*np.cos(theta_1) - g*(-l_1*m_2 - l_1*m_3 - l_1*m_4 - l_1*m_5 - m_1*r_1)*np.sin(theta_1))*np.cos(theta_5 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) - (-l_1**2*m_5*r_5*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_5 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*m_5*r_5*np.cos(theta_5 - theta_2))*(dtheta_5**2*l_2*m_5*r_5*np.sin(theta_5 - theta_2) + dtheta_4*(dtheta_4*l_2*l_4*m_5*np.sin(theta_4 - theta_2) + dtheta_4*l_2*m_4*r_4*np.sin(theta_4 - theta_2)) + dtheta_3*mu_3 + dtheta_3*(dtheta_3*l_2*l_3*m_4*np.sin(theta_3 - theta_2) + dtheta_3*l_2*l_3*m_5*np.sin(theta_3 - theta_2) + dtheta_3*l_2*m_3*r_3*np.sin(theta_3 - theta_2)) - dtheta_2*(mu_2 + mu_3) + dtheta_1*mu_2 + dtheta_1*(-dtheta_1*l_1*l_2*m_3*np.sin(theta_2 - theta_1) - dtheta_1*l_1*l_2*m_4*np.sin(theta_2 - theta_1) - dtheta_1*l_1*l_2*m_5*np.sin(theta_2 - theta_1) - dtheta_1*l_1*m_2*r_2*np.sin(theta_2 - theta_1)) - ddx_c*(-l_2*m_3 - l_2*m_4 - l_2*m_5 - m_2*r_2)*np.cos(theta_2) - g*(-l_2*m_3 - l_2*m_4 - l_2*m_5 - m_2*r_2)*np.sin(theta_2) - l_1*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*(dtheta_5**2*l_1*m_5*r_5*np.sin(theta_5 - theta_1) + dtheta_4*(dtheta_4*l_1*l_4*m_5*np.sin(theta_4 - theta_1) + dtheta_4*l_1*m_4*r_4*np.sin(theta_4 - theta_1)) + dtheta_3*(dtheta_3*l_1*l_3*m_4*np.sin(theta_3 - theta_1) + dtheta_3*l_1*l_3*m_5*np.sin(theta_3 - theta_1) + dtheta_3*l_1*m_3*r_3*np.sin(theta_3 - theta_1)) + dtheta_2*mu_2 + dtheta_2*(dtheta_2*l_1*l_2*m_3*np.sin(theta_2 - theta_1) + dtheta_2*l_1*l_2*m_4*np.sin(theta_2 - theta_1) + dtheta_2*l_1*l_2*m_5*np.sin(theta_2 - theta_1) + dtheta_2*l_1*m_2*r_2*np.sin(theta_2 - theta_1)) - dtheta_1*(mu_1 + mu_2) - ddx_c*(-l_1*m_2 - l_1*m_3 - l_1*m_4 - l_1*m_5 - m_1*r_1)*np.cos(theta_1) - g*(-l_1*m_2 - l_1*m_3 - l_1*m_4 - l_1*m_5 - m_1*r_1)*np.sin(theta_1))*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2))/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2) - (-l_1**2*m_5*r_5*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_5 - theta_1)*np.cos(theta_3 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_3*m_5*r_5*np.cos(theta_5 - theta_3) - (-l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_3 - theta_2))*(-l_1**2*m_5*r_5*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_5 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*m_5*r_5*np.cos(theta_5 - theta_2))/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2))*(dtheta_5**2*l_3*m_5*r_5*np.sin(theta_5 - theta_3) + dtheta_4*mu_4 + dtheta_4*(dtheta_4*l_3*l_4*m_5*np.sin(theta_4 - theta_3) + dtheta_4*l_3*m_4*r_4*np.sin(theta_4 - theta_3)) - dtheta_3*(mu_3 + mu_4) + dtheta_2*mu_3 + dtheta_2*(-dtheta_2*l_2*l_3*m_4*np.sin(theta_3 - theta_2) - dtheta_2*l_2*l_3*m_5*np.sin(theta_3 - theta_2) - dtheta_2*l_2*m_3*r_3*np.sin(theta_3 - theta_2)) + dtheta_1*(-dtheta_1*l_1*l_3*m_4*np.sin(theta_3 - theta_1) - dtheta_1*l_1*l_3*m_5*np.sin(theta_3 - theta_1) - dtheta_1*l_1*m_3*r_3*np.sin(theta_3 - theta_1)) - ddx_c*(-l_3*m_4 - l_3*m_5 - m_3*r_3)*np.cos(theta_3) - g*(-l_3*m_4 - l_3*m_5 - m_3*r_3)*np.sin(theta_3) - l_1*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(dtheta_5**2*l_1*m_5*r_5*np.sin(theta_5 - theta_1) + dtheta_4*(dtheta_4*l_1*l_4*m_5*np.sin(theta_4 - theta_1) + dtheta_4*l_1*m_4*r_4*np.sin(theta_4 - theta_1)) + dtheta_3*(dtheta_3*l_1*l_3*m_4*np.sin(theta_3 - theta_1) + dtheta_3*l_1*l_3*m_5*np.sin(theta_3 - theta_1) + dtheta_3*l_1*m_3*r_3*np.sin(theta_3 - theta_1)) + dtheta_2*mu_2 + dtheta_2*(dtheta_2*l_1*l_2*m_3*np.sin(theta_2 - theta_1) + dtheta_2*l_1*l_2*m_4*np.sin(theta_2 - theta_1) + dtheta_2*l_1*l_2*m_5*np.sin(theta_2 - theta_1) + dtheta_2*l_1*m_2*r_2*np.sin(theta_2 - theta_1)) - dtheta_1*(mu_1 + mu_2) - ddx_c*(-l_1*m_2 - l_1*m_3 - l_1*m_4 - l_1*m_5 - m_1*r_1)*np.cos(theta_1) - g*(-l_1*m_2 - l_1*m_3 - l_1*m_4 - l_1*m_5 - m_1*r_1)*np.sin(theta_1))*np.cos(theta_3 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) - (-l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_3 - theta_2))*(dtheta_5**2*l_2*m_5*r_5*np.sin(theta_5 - theta_2) + dtheta_4*(dtheta_4*l_2*l_4*m_5*np.sin(theta_4 - theta_2) + dtheta_4*l_2*m_4*r_4*np.sin(theta_4 - theta_2)) + dtheta_3*mu_3 + dtheta_3*(dtheta_3*l_2*l_3*m_4*np.sin(theta_3 - theta_2) + dtheta_3*l_2*l_3*m_5*np.sin(theta_3 - theta_2) + dtheta_3*l_2*m_3*r_3*np.sin(theta_3 - theta_2)) - dtheta_2*(mu_2 + mu_3) + dtheta_1*mu_2 + dtheta_1*(-dtheta_1*l_1*l_2*m_3*np.sin(theta_2 - theta_1) - dtheta_1*l_1*l_2*m_4*np.sin(theta_2 - theta_1) - dtheta_1*l_1*l_2*m_5*np.sin(theta_2 - theta_1) - dtheta_1*l_1*m_2*r_2*np.sin(theta_2 - theta_1)) - ddx_c*(-l_2*m_3 - l_2*m_4 - l_2*m_5 - m_2*r_2)*np.cos(theta_2) - g*(-l_2*m_3 - l_2*m_4 - l_2*m_5 - m_2*r_2)*np.sin(theta_2) - l_1*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*(dtheta_5**2*l_1*m_5*r_5*np.sin(theta_5 - theta_1) + dtheta_4*(dtheta_4*l_1*l_4*m_5*np.sin(theta_4 - theta_1) + dtheta_4*l_1*m_4*r_4*np.sin(theta_4 - theta_1)) + dtheta_3*(dtheta_3*l_1*l_3*m_4*np.sin(theta_3 - theta_1) + dtheta_3*l_1*l_3*m_5*np.sin(theta_3 - theta_1) + dtheta_3*l_1*m_3*r_3*np.sin(theta_3 - theta_1)) + dtheta_2*mu_2 + dtheta_2*(dtheta_2*l_1*l_2*m_3*np.sin(theta_2 - theta_1) + dtheta_2*l_1*l_2*m_4*np.sin(theta_2 - theta_1) + dtheta_2*l_1*l_2*m_5*np.sin(theta_2 - theta_1) + dtheta_2*l_1*m_2*r_2*np.sin(theta_2 - theta_1)) - dtheta_1*(mu_1 + mu_2) - ddx_c*(-l_1*m_2 - l_1*m_3 - l_1*m_4 - l_1*m_5 - m_1*r_1)*np.cos(theta_1) - g*(-l_1*m_2 - l_1*m_3 - l_1*m_4 - l_1*m_5 - m_1*r_1)*np.sin(theta_1))*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2))/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2))/(I_3 - l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)**2*np.cos(theta_3 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_3**2*m_4 + l_3**2*m_5 + m_3*r_3**2 - (-l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_3 - theta_2))**2/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2)) - (-l_1**2*m_5*r_5*(l_4*m_5 + m_4*r_4)*np.cos(theta_5 - theta_1)*np.cos(theta_4 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_4*m_5*r_5*np.cos(theta_5 - theta_4) - (-l_1**2*(l_4*m_5 + m_4*r_4)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_4 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_4*m_5 + m_4*r_4)*np.cos(theta_4 - theta_2))*(-l_1**2*m_5*r_5*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_5 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*m_5*r_5*np.cos(theta_5 - theta_2))/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2) - (-l_1**2*(l_4*m_5 + m_4*r_4)*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_4 - theta_1)*np.cos(theta_3 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_3*(l_4*m_5 + m_4*r_4)*np.cos(theta_4 - theta_3) - (-l_1**2*(l_4*m_5 + m_4*r_4)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_4 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_4*m_5 + m_4*r_4)*np.cos(theta_4 - theta_2))*(-l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_3 - theta_2))/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2))*(-l_1**2*m_5*r_5*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_5 - theta_1)*np.cos(theta_3 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_3*m_5*r_5*np.cos(theta_5 - theta_3) - (-l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_3 - theta_2))*(-l_1**2*m_5*r_5*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_5 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*m_5*r_5*np.cos(theta_5 - theta_2))/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2))/(I_3 - l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)**2*np.cos(theta_3 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_3**2*m_4 + l_3**2*m_5 + m_3*r_3**2 - (-l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_3 - theta_2))**2/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2)))*(dtheta_5**2*l_4*m_5*r_5*np.sin(theta_5 - theta_4) + dtheta_5*mu_5 - dtheta_4*(mu_4 + mu_5) + dtheta_3*mu_4 + dtheta_3*(-dtheta_3*l_3*l_4*m_5*np.sin(theta_4 - theta_3) - dtheta_3*l_3*m_4*r_4*np.sin(theta_4 - theta_3)) + dtheta_2*(-dtheta_2*l_2*l_4*m_5*np.sin(theta_4 - theta_2) - dtheta_2*l_2*m_4*r_4*np.sin(theta_4 - theta_2)) + dtheta_1*(-dtheta_1*l_1*l_4*m_5*np.sin(theta_4 - theta_1) - dtheta_1*l_1*m_4*r_4*np.sin(theta_4 - theta_1)) - ddx_c*(-l_4*m_5 - m_4*r_4)*np.cos(theta_4) - g*(-l_4*m_5 - m_4*r_4)*np.sin(theta_4) - l_1*(l_4*m_5 + m_4*r_4)*(dtheta_5**2*l_1*m_5*r_5*np.sin(theta_5 - theta_1) + dtheta_4*(dtheta_4*l_1*l_4*m_5*np.sin(theta_4 - theta_1) + dtheta_4*l_1*m_4*r_4*np.sin(theta_4 - theta_1)) + dtheta_3*(dtheta_3*l_1*l_3*m_4*np.sin(theta_3 - theta_1) + dtheta_3*l_1*l_3*m_5*np.sin(theta_3 - theta_1) + dtheta_3*l_1*m_3*r_3*np.sin(theta_3 - theta_1)) + dtheta_2*mu_2 + dtheta_2*(dtheta_2*l_1*l_2*m_3*np.sin(theta_2 - theta_1) + dtheta_2*l_1*l_2*m_4*np.sin(theta_2 - theta_1) + dtheta_2*l_1*l_2*m_5*np.sin(theta_2 - theta_1) + dtheta_2*l_1*m_2*r_2*np.sin(theta_2 - theta_1)) - dtheta_1*(mu_1 + mu_2) - ddx_c*(-l_1*m_2 - l_1*m_3 - l_1*m_4 - l_1*m_5 - m_1*r_1)*np.cos(theta_1) - g*(-l_1*m_2 - l_1*m_3 - l_1*m_4 - l_1*m_5 - m_1*r_1)*np.sin(theta_1))*np.cos(theta_4 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) - (-l_1**2*(l_4*m_5 + m_4*r_4)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_4 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_4*m_5 + m_4*r_4)*np.cos(theta_4 - theta_2))*(dtheta_5**2*l_2*m_5*r_5*np.sin(theta_5 - theta_2) + dtheta_4*(dtheta_4*l_2*l_4*m_5*np.sin(theta_4 - theta_2) + dtheta_4*l_2*m_4*r_4*np.sin(theta_4 - theta_2)) + dtheta_3*mu_3 + dtheta_3*(dtheta_3*l_2*l_3*m_4*np.sin(theta_3 - theta_2) + dtheta_3*l_2*l_3*m_5*np.sin(theta_3 - theta_2) + dtheta_3*l_2*m_3*r_3*np.sin(theta_3 - theta_2)) - dtheta_2*(mu_2 + mu_3) + dtheta_1*mu_2 + dtheta_1*(-dtheta_1*l_1*l_2*m_3*np.sin(theta_2 - theta_1) - dtheta_1*l_1*l_2*m_4*np.sin(theta_2 - theta_1) - dtheta_1*l_1*l_2*m_5*np.sin(theta_2 - theta_1) - dtheta_1*l_1*m_2*r_2*np.sin(theta_2 - theta_1)) - ddx_c*(-l_2*m_3 - l_2*m_4 - l_2*m_5 - m_2*r_2)*np.cos(theta_2) - g*(-l_2*m_3 - l_2*m_4 - l_2*m_5 - m_2*r_2)*np.sin(theta_2) - l_1*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*(dtheta_5**2*l_1*m_5*r_5*np.sin(theta_5 - theta_1) + dtheta_4*(dtheta_4*l_1*l_4*m_5*np.sin(theta_4 - theta_1) + dtheta_4*l_1*m_4*r_4*np.sin(theta_4 - theta_1)) + dtheta_3*(dtheta_3*l_1*l_3*m_4*np.sin(theta_3 - theta_1) + dtheta_3*l_1*l_3*m_5*np.sin(theta_3 - theta_1) + dtheta_3*l_1*m_3*r_3*np.sin(theta_3 - theta_1)) + dtheta_2*mu_2 + dtheta_2*(dtheta_2*l_1*l_2*m_3*np.sin(theta_2 - theta_1) + dtheta_2*l_1*l_2*m_4*np.sin(theta_2 - theta_1) + dtheta_2*l_1*l_2*m_5*np.sin(theta_2 - theta_1) + dtheta_2*l_1*m_2*r_2*np.sin(theta_2 - theta_1)) - dtheta_1*(mu_1 + mu_2) - ddx_c*(-l_1*m_2 - l_1*m_3 - l_1*m_4 - l_1*m_5 - m_1*r_1)*np.cos(theta_1) - g*(-l_1*m_2 - l_1*m_3 - l_1*m_4 - l_1*m_5 - m_1*r_1)*np.sin(theta_1))*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2))/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2) - (-l_1**2*(l_4*m_5 + m_4*r_4)*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_4 - theta_1)*np.cos(theta_3 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_3*(l_4*m_5 + m_4*r_4)*np.cos(theta_4 - theta_3) - (-l_1**2*(l_4*m_5 + m_4*r_4)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_4 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_4*m_5 + m_4*r_4)*np.cos(theta_4 - theta_2))*(-l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_3 - theta_2))/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2))*(dtheta_5**2*l_3*m_5*r_5*np.sin(theta_5 - theta_3) + dtheta_4*mu_4 + dtheta_4*(dtheta_4*l_3*l_4*m_5*np.sin(theta_4 - theta_3) + dtheta_4*l_3*m_4*r_4*np.sin(theta_4 - theta_3)) - dtheta_3*(mu_3 + mu_4) + dtheta_2*mu_3 + dtheta_2*(-dtheta_2*l_2*l_3*m_4*np.sin(theta_3 - theta_2) - dtheta_2*l_2*l_3*m_5*np.sin(theta_3 - theta_2) - dtheta_2*l_2*m_3*r_3*np.sin(theta_3 - theta_2)) + dtheta_1*(-dtheta_1*l_1*l_3*m_4*np.sin(theta_3 - theta_1) - dtheta_1*l_1*l_3*m_5*np.sin(theta_3 - theta_1) - dtheta_1*l_1*m_3*r_3*np.sin(theta_3 - theta_1)) - ddx_c*(-l_3*m_4 - l_3*m_5 - m_3*r_3)*np.cos(theta_3) - g*(-l_3*m_4 - l_3*m_5 - m_3*r_3)*np.sin(theta_3) - l_1*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(dtheta_5**2*l_1*m_5*r_5*np.sin(theta_5 - theta_1) + dtheta_4*(dtheta_4*l_1*l_4*m_5*np.sin(theta_4 - theta_1) + dtheta_4*l_1*m_4*r_4*np.sin(theta_4 - theta_1)) + dtheta_3*(dtheta_3*l_1*l_3*m_4*np.sin(theta_3 - theta_1) + dtheta_3*l_1*l_3*m_5*np.sin(theta_3 - theta_1) + dtheta_3*l_1*m_3*r_3*np.sin(theta_3 - theta_1)) + dtheta_2*mu_2 + dtheta_2*(dtheta_2*l_1*l_2*m_3*np.sin(theta_2 - theta_1) + dtheta_2*l_1*l_2*m_4*np.sin(theta_2 - theta_1) + dtheta_2*l_1*l_2*m_5*np.sin(theta_2 - theta_1) + dtheta_2*l_1*m_2*r_2*np.sin(theta_2 - theta_1)) - dtheta_1*(mu_1 + mu_2) - ddx_c*(-l_1*m_2 - l_1*m_3 - l_1*m_4 - l_1*m_5 - m_1*r_1)*np.cos(theta_1) - g*(-l_1*m_2 - l_1*m_3 - l_1*m_4 - l_1*m_5 - m_1*r_1)*np.sin(theta_1))*np.cos(theta_3 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) - (-l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_3 - theta_2))*(dtheta_5**2*l_2*m_5*r_5*np.sin(theta_5 - theta_2) + dtheta_4*(dtheta_4*l_2*l_4*m_5*np.sin(theta_4 - theta_2) + dtheta_4*l_2*m_4*r_4*np.sin(theta_4 - theta_2)) + dtheta_3*mu_3 + dtheta_3*(dtheta_3*l_2*l_3*m_4*np.sin(theta_3 - theta_2) + dtheta_3*l_2*l_3*m_5*np.sin(theta_3 - theta_2) + dtheta_3*l_2*m_3*r_3*np.sin(theta_3 - theta_2)) - dtheta_2*(mu_2 + mu_3) + dtheta_1*mu_2 + dtheta_1*(-dtheta_1*l_1*l_2*m_3*np.sin(theta_2 - theta_1) - dtheta_1*l_1*l_2*m_4*np.sin(theta_2 - theta_1) - dtheta_1*l_1*l_2*m_5*np.sin(theta_2 - theta_1) - dtheta_1*l_1*m_2*r_2*np.sin(theta_2 - theta_1)) - ddx_c*(-l_2*m_3 - l_2*m_4 - l_2*m_5 - m_2*r_2)*np.cos(theta_2) - g*(-l_2*m_3 - l_2*m_4 - l_2*m_5 - m_2*r_2)*np.sin(theta_2) - l_1*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*(dtheta_5**2*l_1*m_5*r_5*np.sin(theta_5 - theta_1) + dtheta_4*(dtheta_4*l_1*l_4*m_5*np.sin(theta_4 - theta_1) + dtheta_4*l_1*m_4*r_4*np.sin(theta_4 - theta_1)) + dtheta_3*(dtheta_3*l_1*l_3*m_4*np.sin(theta_3 - theta_1) + dtheta_3*l_1*l_3*m_5*np.sin(theta_3 - theta_1) + dtheta_3*l_1*m_3*r_3*np.sin(theta_3 - theta_1)) + dtheta_2*mu_2 + dtheta_2*(dtheta_2*l_1*l_2*m_3*np.sin(theta_2 - theta_1) + dtheta_2*l_1*l_2*m_4*np.sin(theta_2 - theta_1) + dtheta_2*l_1*l_2*m_5*np.sin(theta_2 - theta_1) + dtheta_2*l_1*m_2*r_2*np.sin(theta_2 - theta_1)) - dtheta_1*(mu_1 + mu_2) - ddx_c*(-l_1*m_2 - l_1*m_3 - l_1*m_4 - l_1*m_5 - m_1*r_1)*np.cos(theta_1) - g*(-l_1*m_2 - l_1*m_3 - l_1*m_4 - l_1*m_5 - m_1*r_1)*np.sin(theta_1))*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2))/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2))/(I_3 - l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)**2*np.cos(theta_3 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_3**2*m_4 + l_3**2*m_5 + m_3*r_3**2 - (-l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_3 - theta_2))**2/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2)))/(I_4 - l_1**2*(l_4*m_5 + m_4*r_4)**2*np.cos(theta_4 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_4**2*m_5 + m_4*r_4**2 - (-l_1**2*(l_4*m_5 + m_4*r_4)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_4 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_4*m_5 + m_4*r_4)*np.cos(theta_4 - theta_2))**2/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2) - (-l_1**2*(l_4*m_5 + m_4*r_4)*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_4 - theta_1)*np.cos(theta_3 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_3*(l_4*m_5 + m_4*r_4)*np.cos(theta_4 - theta_3) - (-l_1**2*(l_4*m_5 + m_4*r_4)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_4 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_4*m_5 + m_4*r_4)*np.cos(theta_4 - theta_2))*(-l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_3 - theta_2))/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2))**2/(I_3 - l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)**2*np.cos(theta_3 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_3**2*m_4 + l_3**2*m_5 + m_3*r_3**2 - (-l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_3 - theta_2))**2/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2))))/(I_5 - l_1**2*m_5**2*r_5**2*np.cos(theta_5 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + m_5*r_5**2 - (-l_1**2*m_5*r_5*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_5 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*m_5*r_5*np.cos(theta_5 - theta_2))**2/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2) - (-l_1**2*m_5*r_5*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_5 - theta_1)*np.cos(theta_3 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_3*m_5*r_5*np.cos(theta_5 - theta_3) - (-l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_3 - theta_2))*(-l_1**2*m_5*r_5*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_5 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*m_5*r_5*np.cos(theta_5 - theta_2))/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2))**2/(I_3 - l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)**2*np.cos(theta_3 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_3**2*m_4 + l_3**2*m_5 + m_3*r_3**2 - (-l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_3 - theta_2))**2/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2)) - (-l_1**2*m_5*r_5*(l_4*m_5 + m_4*r_4)*np.cos(theta_5 - theta_1)*np.cos(theta_4 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_4*m_5*r_5*np.cos(theta_5 - theta_4) - (-l_1**2*(l_4*m_5 + m_4*r_4)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_4 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_4*m_5 + m_4*r_4)*np.cos(theta_4 - theta_2))*(-l_1**2*m_5*r_5*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_5 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*m_5*r_5*np.cos(theta_5 - theta_2))/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2) - (-l_1**2*(l_4*m_5 + m_4*r_4)*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_4 - theta_1)*np.cos(theta_3 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_3*(l_4*m_5 + m_4*r_4)*np.cos(theta_4 - theta_3) - (-l_1**2*(l_4*m_5 + m_4*r_4)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_4 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_4*m_5 + m_4*r_4)*np.cos(theta_4 - theta_2))*(-l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_3 - theta_2))/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2))*(-l_1**2*m_5*r_5*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_5 - theta_1)*np.cos(theta_3 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_3*m_5*r_5*np.cos(theta_5 - theta_3) - (-l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_3 - theta_2))*(-l_1**2*m_5*r_5*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_5 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*m_5*r_5*np.cos(theta_5 - theta_2))/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2))/(I_3 - l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)**2*np.cos(theta_3 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_3**2*m_4 + l_3**2*m_5 + m_3*r_3**2 - (-l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_3 - theta_2))**2/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2)))**2/(I_4 - l_1**2*(l_4*m_5 + m_4*r_4)**2*np.cos(theta_4 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_4**2*m_5 + m_4*r_4**2 - (-l_1**2*(l_4*m_5 + m_4*r_4)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_4 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_4*m_5 + m_4*r_4)*np.cos(theta_4 - theta_2))**2/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2) - (-l_1**2*(l_4*m_5 + m_4*r_4)*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_4 - theta_1)*np.cos(theta_3 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_3*(l_4*m_5 + m_4*r_4)*np.cos(theta_4 - theta_3) - (-l_1**2*(l_4*m_5 + m_4*r_4)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_4 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_4*m_5 + m_4*r_4)*np.cos(theta_4 - theta_2))*(-l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_3 - theta_2))/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2))**2/(I_3 - l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)**2*np.cos(theta_3 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_3**2*m_4 + l_3**2*m_5 + m_3*r_3**2 - (-l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_3 - theta_2))**2/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2)))))*np.cos(theta_2 - theta_1)/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2))/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2),



                      (dtheta_5**2*l_2*m_5*r_5*np.sin(theta_5 - theta_2) + dtheta_4*(dtheta_4*l_2*l_4*m_5*np.sin(theta_4 - theta_2) + dtheta_4*l_2*m_4*r_4*np.sin(theta_4 - theta_2)) + dtheta_3*mu_3 + dtheta_3*(dtheta_3*l_2*l_3*m_4*np.sin(theta_3 - theta_2) + dtheta_3*l_2*l_3*m_5*np.sin(theta_3 - theta_2) + dtheta_3*l_2*m_3*r_3*np.sin(theta_3 - theta_2)) - dtheta_2*(mu_2 + mu_3) + dtheta_1*mu_2 + dtheta_1*(-dtheta_1*l_1*l_2*m_3*np.sin(theta_2 - theta_1) - dtheta_1*l_1*l_2*m_4*np.sin(theta_2 - theta_1) - dtheta_1*l_1*l_2*m_5*np.sin(theta_2 - theta_1) - dtheta_1*l_1*m_2*r_2*np.sin(theta_2 - theta_1)) - ddx_c*(-l_2*m_3 - l_2*m_4 - l_2*m_5 - m_2*r_2)*np.cos(theta_2) - g*(-l_2*m_3 - l_2*m_4 - l_2*m_5 - m_2*r_2)*np.sin(theta_2) - l_1*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*(dtheta_5**2*l_1*m_5*r_5*np.sin(theta_5 - theta_1) + dtheta_4*(dtheta_4*l_1*l_4*m_5*np.sin(theta_4 - theta_1) + dtheta_4*l_1*m_4*r_4*np.sin(theta_4 - theta_1)) + dtheta_3*(dtheta_3*l_1*l_3*m_4*np.sin(theta_3 - theta_1) + dtheta_3*l_1*l_3*m_5*np.sin(theta_3 - theta_1) + dtheta_3*l_1*m_3*r_3*np.sin(theta_3 - theta_1)) + dtheta_2*mu_2 + dtheta_2*(dtheta_2*l_1*l_2*m_3*np.sin(theta_2 - theta_1) + dtheta_2*l_1*l_2*m_4*np.sin(theta_2 - theta_1) + dtheta_2*l_1*l_2*m_5*np.sin(theta_2 - theta_1) + dtheta_2*l_1*m_2*r_2*np.sin(theta_2 - theta_1)) - dtheta_1*(mu_1 + mu_2) - ddx_c*(-l_1*m_2 - l_1*m_3 - l_1*m_4 - l_1*m_5 - m_1*r_1)*np.cos(theta_1) - g*(-l_1*m_2 - l_1*m_3 - l_1*m_4 - l_1*m_5 - m_1*r_1)*np.sin(theta_1))*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) - (-l_1**2*(l_4*m_5 + m_4*r_4)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_4 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_4*m_5 + m_4*r_4)*np.cos(theta_4 - theta_2))*(dtheta_5**2*l_4*m_5*r_5*np.sin(theta_5 - theta_4) + dtheta_5*mu_5 - dtheta_4*(mu_4 + mu_5) + dtheta_3*mu_4 + dtheta_3*(-dtheta_3*l_3*l_4*m_5*np.sin(theta_4 - theta_3) - dtheta_3*l_3*m_4*r_4*np.sin(theta_4 - theta_3)) + dtheta_2*(-dtheta_2*l_2*l_4*m_5*np.sin(theta_4 - theta_2) - dtheta_2*l_2*m_4*r_4*np.sin(theta_4 - theta_2)) + dtheta_1*(-dtheta_1*l_1*l_4*m_5*np.sin(theta_4 - theta_1) - dtheta_1*l_1*m_4*r_4*np.sin(theta_4 - theta_1)) - ddx_c*(-l_4*m_5 - m_4*r_4)*np.cos(theta_4) - g*(-l_4*m_5 - m_4*r_4)*np.sin(theta_4) - l_1*(l_4*m_5 + m_4*r_4)*(dtheta_5**2*l_1*m_5*r_5*np.sin(theta_5 - theta_1) + dtheta_4*(dtheta_4*l_1*l_4*m_5*np.sin(theta_4 - theta_1) + dtheta_4*l_1*m_4*r_4*np.sin(theta_4 - theta_1)) + dtheta_3*(dtheta_3*l_1*l_3*m_4*np.sin(theta_3 - theta_1) + dtheta_3*l_1*l_3*m_5*np.sin(theta_3 - theta_1) + dtheta_3*l_1*m_3*r_3*np.sin(theta_3 - theta_1)) + dtheta_2*mu_2 + dtheta_2*(dtheta_2*l_1*l_2*m_3*np.sin(theta_2 - theta_1) + dtheta_2*l_1*l_2*m_4*np.sin(theta_2 - theta_1) + dtheta_2*l_1*l_2*m_5*np.sin(theta_2 - theta_1) + dtheta_2*l_1*m_2*r_2*np.sin(theta_2 - theta_1)) - dtheta_1*(mu_1 + mu_2) - ddx_c*(-l_1*m_2 - l_1*m_3 - l_1*m_4 - l_1*m_5 - m_1*r_1)*np.cos(theta_1) - g*(-l_1*m_2 - l_1*m_3 - l_1*m_4 - l_1*m_5 - m_1*r_1)*np.sin(theta_1))*np.cos(theta_4 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) - (-l_1**2*(l_4*m_5 + m_4*r_4)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_4 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_4*m_5 + m_4*r_4)*np.cos(theta_4 - theta_2))*(dtheta_5**2*l_2*m_5*r_5*np.sin(theta_5 - theta_2) + dtheta_4*(dtheta_4*l_2*l_4*m_5*np.sin(theta_4 - theta_2) + dtheta_4*l_2*m_4*r_4*np.sin(theta_4 - theta_2)) + dtheta_3*mu_3 + dtheta_3*(dtheta_3*l_2*l_3*m_4*np.sin(theta_3 - theta_2) + dtheta_3*l_2*l_3*m_5*np.sin(theta_3 - theta_2) + dtheta_3*l_2*m_3*r_3*np.sin(theta_3 - theta_2)) - dtheta_2*(mu_2 + mu_3) + dtheta_1*mu_2 + dtheta_1*(-dtheta_1*l_1*l_2*m_3*np.sin(theta_2 - theta_1) - dtheta_1*l_1*l_2*m_4*np.sin(theta_2 - theta_1) - dtheta_1*l_1*l_2*m_5*np.sin(theta_2 - theta_1) - dtheta_1*l_1*m_2*r_2*np.sin(theta_2 - theta_1)) - ddx_c*(-l_2*m_3 - l_2*m_4 - l_2*m_5 - m_2*r_2)*np.cos(theta_2) - g*(-l_2*m_3 - l_2*m_4 - l_2*m_5 - m_2*r_2)*np.sin(theta_2) - l_1*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*(dtheta_5**2*l_1*m_5*r_5*np.sin(theta_5 - theta_1) + dtheta_4*(dtheta_4*l_1*l_4*m_5*np.sin(theta_4 - theta_1) + dtheta_4*l_1*m_4*r_4*np.sin(theta_4 - theta_1)) + dtheta_3*(dtheta_3*l_1*l_3*m_4*np.sin(theta_3 - theta_1) + dtheta_3*l_1*l_3*m_5*np.sin(theta_3 - theta_1) + dtheta_3*l_1*m_3*r_3*np.sin(theta_3 - theta_1)) + dtheta_2*mu_2 + dtheta_2*(dtheta_2*l_1*l_2*m_3*np.sin(theta_2 - theta_1) + dtheta_2*l_1*l_2*m_4*np.sin(theta_2 - theta_1) + dtheta_2*l_1*l_2*m_5*np.sin(theta_2 - theta_1) + dtheta_2*l_1*m_2*r_2*np.sin(theta_2 - theta_1)) - dtheta_1*(mu_1 + mu_2) - ddx_c*(-l_1*m_2 - l_1*m_3 - l_1*m_4 - l_1*m_5 - m_1*r_1)*np.cos(theta_1) - g*(-l_1*m_2 - l_1*m_3 - l_1*m_4 - l_1*m_5 - m_1*r_1)*np.sin(theta_1))*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2))/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2) - (-l_1**2*(l_4*m_5 + m_4*r_4)*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_4 - theta_1)*np.cos(theta_3 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_3*(l_4*m_5 + m_4*r_4)*np.cos(theta_4 - theta_3) - (-l_1**2*(l_4*m_5 + m_4*r_4)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_4 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_4*m_5 + m_4*r_4)*np.cos(theta_4 - theta_2))*(-l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_3 - theta_2))/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2))*(dtheta_5**2*l_3*m_5*r_5*np.sin(theta_5 - theta_3) + dtheta_4*mu_4 + dtheta_4*(dtheta_4*l_3*l_4*m_5*np.sin(theta_4 - theta_3) + dtheta_4*l_3*m_4*r_4*np.sin(theta_4 - theta_3)) - dtheta_3*(mu_3 + mu_4) + dtheta_2*mu_3 + dtheta_2*(-dtheta_2*l_2*l_3*m_4*np.sin(theta_3 - theta_2) - dtheta_2*l_2*l_3*m_5*np.sin(theta_3 - theta_2) - dtheta_2*l_2*m_3*r_3*np.sin(theta_3 - theta_2)) + dtheta_1*(-dtheta_1*l_1*l_3*m_4*np.sin(theta_3 - theta_1) - dtheta_1*l_1*l_3*m_5*np.sin(theta_3 - theta_1) - dtheta_1*l_1*m_3*r_3*np.sin(theta_3 - theta_1)) - ddx_c*(-l_3*m_4 - l_3*m_5 - m_3*r_3)*np.cos(theta_3) - g*(-l_3*m_4 - l_3*m_5 - m_3*r_3)*np.sin(theta_3) - l_1*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(dtheta_5**2*l_1*m_5*r_5*np.sin(theta_5 - theta_1) + dtheta_4*(dtheta_4*l_1*l_4*m_5*np.sin(theta_4 - theta_1) + dtheta_4*l_1*m_4*r_4*np.sin(theta_4 - theta_1)) + dtheta_3*(dtheta_3*l_1*l_3*m_4*np.sin(theta_3 - theta_1) + dtheta_3*l_1*l_3*m_5*np.sin(theta_3 - theta_1) + dtheta_3*l_1*m_3*r_3*np.sin(theta_3 - theta_1)) + dtheta_2*mu_2 + dtheta_2*(dtheta_2*l_1*l_2*m_3*np.sin(theta_2 - theta_1) + dtheta_2*l_1*l_2*m_4*np.sin(theta_2 - theta_1) + dtheta_2*l_1*l_2*m_5*np.sin(theta_2 - theta_1) + dtheta_2*l_1*m_2*r_2*np.sin(theta_2 - theta_1)) - dtheta_1*(mu_1 + mu_2) - ddx_c*(-l_1*m_2 - l_1*m_3 - l_1*m_4 - l_1*m_5 - m_1*r_1)*np.cos(theta_1) - g*(-l_1*m_2 - l_1*m_3 - l_1*m_4 - l_1*m_5 - m_1*r_1)*np.sin(theta_1))*np.cos(theta_3 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) - (-l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_3 - theta_2))*(dtheta_5**2*l_2*m_5*r_5*np.sin(theta_5 - theta_2) + dtheta_4*(dtheta_4*l_2*l_4*m_5*np.sin(theta_4 - theta_2) + dtheta_4*l_2*m_4*r_4*np.sin(theta_4 - theta_2)) + dtheta_3*mu_3 + dtheta_3*(dtheta_3*l_2*l_3*m_4*np.sin(theta_3 - theta_2) + dtheta_3*l_2*l_3*m_5*np.sin(theta_3 - theta_2) + dtheta_3*l_2*m_3*r_3*np.sin(theta_3 - theta_2)) - dtheta_2*(mu_2 + mu_3) + dtheta_1*mu_2 + dtheta_1*(-dtheta_1*l_1*l_2*m_3*np.sin(theta_2 - theta_1) - dtheta_1*l_1*l_2*m_4*np.sin(theta_2 - theta_1) - dtheta_1*l_1*l_2*m_5*np.sin(theta_2 - theta_1) - dtheta_1*l_1*m_2*r_2*np.sin(theta_2 - theta_1)) - ddx_c*(-l_2*m_3 - l_2*m_4 - l_2*m_5 - m_2*r_2)*np.cos(theta_2) - g*(-l_2*m_3 - l_2*m_4 - l_2*m_5 - m_2*r_2)*np.sin(theta_2) - l_1*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*(dtheta_5**2*l_1*m_5*r_5*np.sin(theta_5 - theta_1) + dtheta_4*(dtheta_4*l_1*l_4*m_5*np.sin(theta_4 - theta_1) + dtheta_4*l_1*m_4*r_4*np.sin(theta_4 - theta_1)) + dtheta_3*(dtheta_3*l_1*l_3*m_4*np.sin(theta_3 - theta_1) + dtheta_3*l_1*l_3*m_5*np.sin(theta_3 - theta_1) + dtheta_3*l_1*m_3*r_3*np.sin(theta_3 - theta_1)) + dtheta_2*mu_2 + dtheta_2*(dtheta_2*l_1*l_2*m_3*np.sin(theta_2 - theta_1) + dtheta_2*l_1*l_2*m_4*np.sin(theta_2 - theta_1) + dtheta_2*l_1*l_2*m_5*np.sin(theta_2 - theta_1) + dtheta_2*l_1*m_2*r_2*np.sin(theta_2 - theta_1)) - dtheta_1*(mu_1 + mu_2) - ddx_c*(-l_1*m_2 - l_1*m_3 - l_1*m_4 - l_1*m_5 - m_1*r_1)*np.cos(theta_1) - g*(-l_1*m_2 - l_1*m_3 - l_1*m_4 - l_1*m_5 - m_1*r_1)*np.sin(theta_1))*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2))/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2))/(I_3 - l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)**2*np.cos(theta_3 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_3**2*m_4 + l_3**2*m_5 + m_3*r_3**2 - (-l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_3 - theta_2))**2/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2)) - (-l_1**2*m_5*r_5*(l_4*m_5 + m_4*r_4)*np.cos(theta_5 - theta_1)*np.cos(theta_4 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_4*m_5*r_5*np.cos(theta_5 - theta_4) - (-l_1**2*(l_4*m_5 + m_4*r_4)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_4 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_4*m_5 + m_4*r_4)*np.cos(theta_4 - theta_2))*(-l_1**2*m_5*r_5*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_5 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*m_5*r_5*np.cos(theta_5 - theta_2))/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2) - (-l_1**2*(l_4*m_5 + m_4*r_4)*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_4 - theta_1)*np.cos(theta_3 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_3*(l_4*m_5 + m_4*r_4)*np.cos(theta_4 - theta_3) - (-l_1**2*(l_4*m_5 + m_4*r_4)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_4 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_4*m_5 + m_4*r_4)*np.cos(theta_4 - theta_2))*(-l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_3 - theta_2))/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2))*(-l_1**2*m_5*r_5*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_5 - theta_1)*np.cos(theta_3 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_3*m_5*r_5*np.cos(theta_5 - theta_3) - (-l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_3 - theta_2))*(-l_1**2*m_5*r_5*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_5 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*m_5*r_5*np.cos(theta_5 - theta_2))/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2))/(I_3 - l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)**2*np.cos(theta_3 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_3**2*m_4 + l_3**2*m_5 + m_3*r_3**2 - (-l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_3 - theta_2))**2/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2)))*(-dtheta_5*mu_5 - dtheta_4**2*l_4*m_5*r_5*np.sin(theta_5 - theta_4) + dtheta_4*mu_5 - dtheta_3**2*l_3*m_5*r_5*np.sin(theta_5 - theta_3) - dtheta_2**2*l_2*m_5*r_5*np.sin(theta_5 - theta_2) - dtheta_1**2*l_1*m_5*r_5*np.sin(theta_5 - theta_1) + ddx_c*m_5*r_5*np.cos(theta_5) + g*m_5*r_5*np.sin(theta_5) - l_1*m_5*r_5*(dtheta_5**2*l_1*m_5*r_5*np.sin(theta_5 - theta_1) + dtheta_4*(dtheta_4*l_1*l_4*m_5*np.sin(theta_4 - theta_1) + dtheta_4*l_1*m_4*r_4*np.sin(theta_4 - theta_1)) + dtheta_3*(dtheta_3*l_1*l_3*m_4*np.sin(theta_3 - theta_1) + dtheta_3*l_1*l_3*m_5*np.sin(theta_3 - theta_1) + dtheta_3*l_1*m_3*r_3*np.sin(theta_3 - theta_1)) + dtheta_2*mu_2 + dtheta_2*(dtheta_2*l_1*l_2*m_3*np.sin(theta_2 - theta_1) + dtheta_2*l_1*l_2*m_4*np.sin(theta_2 - theta_1) + dtheta_2*l_1*l_2*m_5*np.sin(theta_2 - theta_1) + dtheta_2*l_1*m_2*r_2*np.sin(theta_2 - theta_1)) - dtheta_1*(mu_1 + mu_2) - ddx_c*(-l_1*m_2 - l_1*m_3 - l_1*m_4 - l_1*m_5 - m_1*r_1)*np.cos(theta_1) - g*(-l_1*m_2 - l_1*m_3 - l_1*m_4 - l_1*m_5 - m_1*r_1)*np.sin(theta_1))*np.cos(theta_5 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) - (-l_1**2*m_5*r_5*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_5 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*m_5*r_5*np.cos(theta_5 - theta_2))*(dtheta_5**2*l_2*m_5*r_5*np.sin(theta_5 - theta_2) + dtheta_4*(dtheta_4*l_2*l_4*m_5*np.sin(theta_4 - theta_2) + dtheta_4*l_2*m_4*r_4*np.sin(theta_4 - theta_2)) + dtheta_3*mu_3 + dtheta_3*(dtheta_3*l_2*l_3*m_4*np.sin(theta_3 - theta_2) + dtheta_3*l_2*l_3*m_5*np.sin(theta_3 - theta_2) + dtheta_3*l_2*m_3*r_3*np.sin(theta_3 - theta_2)) - dtheta_2*(mu_2 + mu_3) + dtheta_1*mu_2 + dtheta_1*(-dtheta_1*l_1*l_2*m_3*np.sin(theta_2 - theta_1) - dtheta_1*l_1*l_2*m_4*np.sin(theta_2 - theta_1) - dtheta_1*l_1*l_2*m_5*np.sin(theta_2 - theta_1) - dtheta_1*l_1*m_2*r_2*np.sin(theta_2 - theta_1)) - ddx_c*(-l_2*m_3 - l_2*m_4 - l_2*m_5 - m_2*r_2)*np.cos(theta_2) - g*(-l_2*m_3 - l_2*m_4 - l_2*m_5 - m_2*r_2)*np.sin(theta_2) - l_1*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*(dtheta_5**2*l_1*m_5*r_5*np.sin(theta_5 - theta_1) + dtheta_4*(dtheta_4*l_1*l_4*m_5*np.sin(theta_4 - theta_1) + dtheta_4*l_1*m_4*r_4*np.sin(theta_4 - theta_1)) + dtheta_3*(dtheta_3*l_1*l_3*m_4*np.sin(theta_3 - theta_1) + dtheta_3*l_1*l_3*m_5*np.sin(theta_3 - theta_1) + dtheta_3*l_1*m_3*r_3*np.sin(theta_3 - theta_1)) + dtheta_2*mu_2 + dtheta_2*(dtheta_2*l_1*l_2*m_3*np.sin(theta_2 - theta_1) + dtheta_2*l_1*l_2*m_4*np.sin(theta_2 - theta_1) + dtheta_2*l_1*l_2*m_5*np.sin(theta_2 - theta_1) + dtheta_2*l_1*m_2*r_2*np.sin(theta_2 - theta_1)) - dtheta_1*(mu_1 + mu_2) - ddx_c*(-l_1*m_2 - l_1*m_3 - l_1*m_4 - l_1*m_5 - m_1*r_1)*np.cos(theta_1) - g*(-l_1*m_2 - l_1*m_3 - l_1*m_4 - l_1*m_5 - m_1*r_1)*np.sin(theta_1))*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2))/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2) - (-l_1**2*m_5*r_5*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_5 - theta_1)*np.cos(theta_3 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_3*m_5*r_5*np.cos(theta_5 - theta_3) - (-l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_3 - theta_2))*(-l_1**2*m_5*r_5*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_5 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*m_5*r_5*np.cos(theta_5 - theta_2))/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2))*(dtheta_5**2*l_3*m_5*r_5*np.sin(theta_5 - theta_3) + dtheta_4*mu_4 + dtheta_4*(dtheta_4*l_3*l_4*m_5*np.sin(theta_4 - theta_3) + dtheta_4*l_3*m_4*r_4*np.sin(theta_4 - theta_3)) - dtheta_3*(mu_3 + mu_4) + dtheta_2*mu_3 + dtheta_2*(-dtheta_2*l_2*l_3*m_4*np.sin(theta_3 - theta_2) - dtheta_2*l_2*l_3*m_5*np.sin(theta_3 - theta_2) - dtheta_2*l_2*m_3*r_3*np.sin(theta_3 - theta_2)) + dtheta_1*(-dtheta_1*l_1*l_3*m_4*np.sin(theta_3 - theta_1) - dtheta_1*l_1*l_3*m_5*np.sin(theta_3 - theta_1) - dtheta_1*l_1*m_3*r_3*np.sin(theta_3 - theta_1)) - ddx_c*(-l_3*m_4 - l_3*m_5 - m_3*r_3)*np.cos(theta_3) - g*(-l_3*m_4 - l_3*m_5 - m_3*r_3)*np.sin(theta_3) - l_1*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(dtheta_5**2*l_1*m_5*r_5*np.sin(theta_5 - theta_1) + dtheta_4*(dtheta_4*l_1*l_4*m_5*np.sin(theta_4 - theta_1) + dtheta_4*l_1*m_4*r_4*np.sin(theta_4 - theta_1)) + dtheta_3*(dtheta_3*l_1*l_3*m_4*np.sin(theta_3 - theta_1) + dtheta_3*l_1*l_3*m_5*np.sin(theta_3 - theta_1) + dtheta_3*l_1*m_3*r_3*np.sin(theta_3 - theta_1)) + dtheta_2*mu_2 + dtheta_2*(dtheta_2*l_1*l_2*m_3*np.sin(theta_2 - theta_1) + dtheta_2*l_1*l_2*m_4*np.sin(theta_2 - theta_1) + dtheta_2*l_1*l_2*m_5*np.sin(theta_2 - theta_1) + dtheta_2*l_1*m_2*r_2*np.sin(theta_2 - theta_1)) - dtheta_1*(mu_1 + mu_2) - ddx_c*(-l_1*m_2 - l_1*m_3 - l_1*m_4 - l_1*m_5 - m_1*r_1)*np.cos(theta_1) - g*(-l_1*m_2 - l_1*m_3 - l_1*m_4 - l_1*m_5 - m_1*r_1)*np.sin(theta_1))*np.cos(theta_3 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) - (-l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_3 - theta_2))*(dtheta_5**2*l_2*m_5*r_5*np.sin(theta_5 - theta_2) + dtheta_4*(dtheta_4*l_2*l_4*m_5*np.sin(theta_4 - theta_2) + dtheta_4*l_2*m_4*r_4*np.sin(theta_4 - theta_2)) + dtheta_3*mu_3 + dtheta_3*(dtheta_3*l_2*l_3*m_4*np.sin(theta_3 - theta_2) + dtheta_3*l_2*l_3*m_5*np.sin(theta_3 - theta_2) + dtheta_3*l_2*m_3*r_3*np.sin(theta_3 - theta_2)) - dtheta_2*(mu_2 + mu_3) + dtheta_1*mu_2 + dtheta_1*(-dtheta_1*l_1*l_2*m_3*np.sin(theta_2 - theta_1) - dtheta_1*l_1*l_2*m_4*np.sin(theta_2 - theta_1) - dtheta_1*l_1*l_2*m_5*np.sin(theta_2 - theta_1) - dtheta_1*l_1*m_2*r_2*np.sin(theta_2 - theta_1)) - ddx_c*(-l_2*m_3 - l_2*m_4 - l_2*m_5 - m_2*r_2)*np.cos(theta_2) - g*(-l_2*m_3 - l_2*m_4 - l_2*m_5 - m_2*r_2)*np.sin(theta_2) - l_1*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*(dtheta_5**2*l_1*m_5*r_5*np.sin(theta_5 - theta_1) + dtheta_4*(dtheta_4*l_1*l_4*m_5*np.sin(theta_4 - theta_1) + dtheta_4*l_1*m_4*r_4*np.sin(theta_4 - theta_1)) + dtheta_3*(dtheta_3*l_1*l_3*m_4*np.sin(theta_3 - theta_1) + dtheta_3*l_1*l_3*m_5*np.sin(theta_3 - theta_1) + dtheta_3*l_1*m_3*r_3*np.sin(theta_3 - theta_1)) + dtheta_2*mu_2 + dtheta_2*(dtheta_2*l_1*l_2*m_3*np.sin(theta_2 - theta_1) + dtheta_2*l_1*l_2*m_4*np.sin(theta_2 - theta_1) + dtheta_2*l_1*l_2*m_5*np.sin(theta_2 - theta_1) + dtheta_2*l_1*m_2*r_2*np.sin(theta_2 - theta_1)) - dtheta_1*(mu_1 + mu_2) - ddx_c*(-l_1*m_2 - l_1*m_3 - l_1*m_4 - l_1*m_5 - m_1*r_1)*np.cos(theta_1) - g*(-l_1*m_2 - l_1*m_3 - l_1*m_4 - l_1*m_5 - m_1*r_1)*np.sin(theta_1))*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2))/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2))/(I_3 - l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)**2*np.cos(theta_3 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_3**2*m_4 + l_3**2*m_5 + m_3*r_3**2 - (-l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_3 - theta_2))**2/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2)) - (-l_1**2*m_5*r_5*(l_4*m_5 + m_4*r_4)*np.cos(theta_5 - theta_1)*np.cos(theta_4 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_4*m_5*r_5*np.cos(theta_5 - theta_4) - (-l_1**2*(l_4*m_5 + m_4*r_4)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_4 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_4*m_5 + m_4*r_4)*np.cos(theta_4 - theta_2))*(-l_1**2*m_5*r_5*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_5 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*m_5*r_5*np.cos(theta_5 - theta_2))/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2) - (-l_1**2*(l_4*m_5 + m_4*r_4)*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_4 - theta_1)*np.cos(theta_3 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_3*(l_4*m_5 + m_4*r_4)*np.cos(theta_4 - theta_3) - (-l_1**2*(l_4*m_5 + m_4*r_4)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_4 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_4*m_5 + m_4*r_4)*np.cos(theta_4 - theta_2))*(-l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_3 - theta_2))/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2))*(-l_1**2*m_5*r_5*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_5 - theta_1)*np.cos(theta_3 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_3*m_5*r_5*np.cos(theta_5 - theta_3) - (-l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_3 - theta_2))*(-l_1**2*m_5*r_5*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_5 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*m_5*r_5*np.cos(theta_5 - theta_2))/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2))/(I_3 - l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)**2*np.cos(theta_3 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_3**2*m_4 + l_3**2*m_5 + m_3*r_3**2 - (-l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_3 - theta_2))**2/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2)))*(dtheta_5**2*l_4*m_5*r_5*np.sin(theta_5 - theta_4) + dtheta_5*mu_5 - dtheta_4*(mu_4 + mu_5) + dtheta_3*mu_4 + dtheta_3*(-dtheta_3*l_3*l_4*m_5*np.sin(theta_4 - theta_3) - dtheta_3*l_3*m_4*r_4*np.sin(theta_4 - theta_3)) + dtheta_2*(-dtheta_2*l_2*l_4*m_5*np.sin(theta_4 - theta_2) - dtheta_2*l_2*m_4*r_4*np.sin(theta_4 - theta_2)) + dtheta_1*(-dtheta_1*l_1*l_4*m_5*np.sin(theta_4 - theta_1) - dtheta_1*l_1*m_4*r_4*np.sin(theta_4 - theta_1)) - ddx_c*(-l_4*m_5 - m_4*r_4)*np.cos(theta_4) - g*(-l_4*m_5 - m_4*r_4)*np.sin(theta_4) - l_1*(l_4*m_5 + m_4*r_4)*(dtheta_5**2*l_1*m_5*r_5*np.sin(theta_5 - theta_1) + dtheta_4*(dtheta_4*l_1*l_4*m_5*np.sin(theta_4 - theta_1) + dtheta_4*l_1*m_4*r_4*np.sin(theta_4 - theta_1)) + dtheta_3*(dtheta_3*l_1*l_3*m_4*np.sin(theta_3 - theta_1) + dtheta_3*l_1*l_3*m_5*np.sin(theta_3 - theta_1) + dtheta_3*l_1*m_3*r_3*np.sin(theta_3 - theta_1)) + dtheta_2*mu_2 + dtheta_2*(dtheta_2*l_1*l_2*m_3*np.sin(theta_2 - theta_1) + dtheta_2*l_1*l_2*m_4*np.sin(theta_2 - theta_1) + dtheta_2*l_1*l_2*m_5*np.sin(theta_2 - theta_1) + dtheta_2*l_1*m_2*r_2*np.sin(theta_2 - theta_1)) - dtheta_1*(mu_1 + mu_2) - ddx_c*(-l_1*m_2 - l_1*m_3 - l_1*m_4 - l_1*m_5 - m_1*r_1)*np.cos(theta_1) - g*(-l_1*m_2 - l_1*m_3 - l_1*m_4 - l_1*m_5 - m_1*r_1)*np.sin(theta_1))*np.cos(theta_4 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) - (-l_1**2*(l_4*m_5 + m_4*r_4)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_4 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_4*m_5 + m_4*r_4)*np.cos(theta_4 - theta_2))*(dtheta_5**2*l_2*m_5*r_5*np.sin(theta_5 - theta_2) + dtheta_4*(dtheta_4*l_2*l_4*m_5*np.sin(theta_4 - theta_2) + dtheta_4*l_2*m_4*r_4*np.sin(theta_4 - theta_2)) + dtheta_3*mu_3 + dtheta_3*(dtheta_3*l_2*l_3*m_4*np.sin(theta_3 - theta_2) + dtheta_3*l_2*l_3*m_5*np.sin(theta_3 - theta_2) + dtheta_3*l_2*m_3*r_3*np.sin(theta_3 - theta_2)) - dtheta_2*(mu_2 + mu_3) + dtheta_1*mu_2 + dtheta_1*(-dtheta_1*l_1*l_2*m_3*np.sin(theta_2 - theta_1) - dtheta_1*l_1*l_2*m_4*np.sin(theta_2 - theta_1) - dtheta_1*l_1*l_2*m_5*np.sin(theta_2 - theta_1) - dtheta_1*l_1*m_2*r_2*np.sin(theta_2 - theta_1)) - ddx_c*(-l_2*m_3 - l_2*m_4 - l_2*m_5 - m_2*r_2)*np.cos(theta_2) - g*(-l_2*m_3 - l_2*m_4 - l_2*m_5 - m_2*r_2)*np.sin(theta_2) - l_1*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*(dtheta_5**2*l_1*m_5*r_5*np.sin(theta_5 - theta_1) + dtheta_4*(dtheta_4*l_1*l_4*m_5*np.sin(theta_4 - theta_1) + dtheta_4*l_1*m_4*r_4*np.sin(theta_4 - theta_1)) + dtheta_3*(dtheta_3*l_1*l_3*m_4*np.sin(theta_3 - theta_1) + dtheta_3*l_1*l_3*m_5*np.sin(theta_3 - theta_1) + dtheta_3*l_1*m_3*r_3*np.sin(theta_3 - theta_1)) + dtheta_2*mu_2 + dtheta_2*(dtheta_2*l_1*l_2*m_3*np.sin(theta_2 - theta_1) + dtheta_2*l_1*l_2*m_4*np.sin(theta_2 - theta_1) + dtheta_2*l_1*l_2*m_5*np.sin(theta_2 - theta_1) + dtheta_2*l_1*m_2*r_2*np.sin(theta_2 - theta_1)) - dtheta_1*(mu_1 + mu_2) - ddx_c*(-l_1*m_2 - l_1*m_3 - l_1*m_4 - l_1*m_5 - m_1*r_1)*np.cos(theta_1) - g*(-l_1*m_2 - l_1*m_3 - l_1*m_4 - l_1*m_5 - m_1*r_1)*np.sin(theta_1))*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2))/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2) - (-l_1**2*(l_4*m_5 + m_4*r_4)*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_4 - theta_1)*np.cos(theta_3 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_3*(l_4*m_5 + m_4*r_4)*np.cos(theta_4 - theta_3) - (-l_1**2*(l_4*m_5 + m_4*r_4)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_4 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_4*m_5 + m_4*r_4)*np.cos(theta_4 - theta_2))*(-l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_3 - theta_2))/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2))*(dtheta_5**2*l_3*m_5*r_5*np.sin(theta_5 - theta_3) + dtheta_4*mu_4 + dtheta_4*(dtheta_4*l_3*l_4*m_5*np.sin(theta_4 - theta_3) + dtheta_4*l_3*m_4*r_4*np.sin(theta_4 - theta_3)) - dtheta_3*(mu_3 + mu_4) + dtheta_2*mu_3 + dtheta_2*(-dtheta_2*l_2*l_3*m_4*np.sin(theta_3 - theta_2) - dtheta_2*l_2*l_3*m_5*np.sin(theta_3 - theta_2) - dtheta_2*l_2*m_3*r_3*np.sin(theta_3 - theta_2)) + dtheta_1*(-dtheta_1*l_1*l_3*m_4*np.sin(theta_3 - theta_1) - dtheta_1*l_1*l_3*m_5*np.sin(theta_3 - theta_1) - dtheta_1*l_1*m_3*r_3*np.sin(theta_3 - theta_1)) - ddx_c*(-l_3*m_4 - l_3*m_5 - m_3*r_3)*np.cos(theta_3) - g*(-l_3*m_4 - l_3*m_5 - m_3*r_3)*np.sin(theta_3) - l_1*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(dtheta_5**2*l_1*m_5*r_5*np.sin(theta_5 - theta_1) + dtheta_4*(dtheta_4*l_1*l_4*m_5*np.sin(theta_4 - theta_1) + dtheta_4*l_1*m_4*r_4*np.sin(theta_4 - theta_1)) + dtheta_3*(dtheta_3*l_1*l_3*m_4*np.sin(theta_3 - theta_1) + dtheta_3*l_1*l_3*m_5*np.sin(theta_3 - theta_1) + dtheta_3*l_1*m_3*r_3*np.sin(theta_3 - theta_1)) + dtheta_2*mu_2 + dtheta_2*(dtheta_2*l_1*l_2*m_3*np.sin(theta_2 - theta_1) + dtheta_2*l_1*l_2*m_4*np.sin(theta_2 - theta_1) + dtheta_2*l_1*l_2*m_5*np.sin(theta_2 - theta_1) + dtheta_2*l_1*m_2*r_2*np.sin(theta_2 - theta_1)) - dtheta_1*(mu_1 + mu_2) - ddx_c*(-l_1*m_2 - l_1*m_3 - l_1*m_4 - l_1*m_5 - m_1*r_1)*np.cos(theta_1) - g*(-l_1*m_2 - l_1*m_3 - l_1*m_4 - l_1*m_5 - m_1*r_1)*np.sin(theta_1))*np.cos(theta_3 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) - (-l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_3 - theta_2))*(dtheta_5**2*l_2*m_5*r_5*np.sin(theta_5 - theta_2) + dtheta_4*(dtheta_4*l_2*l_4*m_5*np.sin(theta_4 - theta_2) + dtheta_4*l_2*m_4*r_4*np.sin(theta_4 - theta_2)) + dtheta_3*mu_3 + dtheta_3*(dtheta_3*l_2*l_3*m_4*np.sin(theta_3 - theta_2) + dtheta_3*l_2*l_3*m_5*np.sin(theta_3 - theta_2) + dtheta_3*l_2*m_3*r_3*np.sin(theta_3 - theta_2)) - dtheta_2*(mu_2 + mu_3) + dtheta_1*mu_2 + dtheta_1*(-dtheta_1*l_1*l_2*m_3*np.sin(theta_2 - theta_1) - dtheta_1*l_1*l_2*m_4*np.sin(theta_2 - theta_1) - dtheta_1*l_1*l_2*m_5*np.sin(theta_2 - theta_1) - dtheta_1*l_1*m_2*r_2*np.sin(theta_2 - theta_1)) - ddx_c*(-l_2*m_3 - l_2*m_4 - l_2*m_5 - m_2*r_2)*np.cos(theta_2) - g*(-l_2*m_3 - l_2*m_4 - l_2*m_5 - m_2*r_2)*np.sin(theta_2) - l_1*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*(dtheta_5**2*l_1*m_5*r_5*np.sin(theta_5 - theta_1) + dtheta_4*(dtheta_4*l_1*l_4*m_5*np.sin(theta_4 - theta_1) + dtheta_4*l_1*m_4*r_4*np.sin(theta_4 - theta_1)) + dtheta_3*(dtheta_3*l_1*l_3*m_4*np.sin(theta_3 - theta_1) + dtheta_3*l_1*l_3*m_5*np.sin(theta_3 - theta_1) + dtheta_3*l_1*m_3*r_3*np.sin(theta_3 - theta_1)) + dtheta_2*mu_2 + dtheta_2*(dtheta_2*l_1*l_2*m_3*np.sin(theta_2 - theta_1) + dtheta_2*l_1*l_2*m_4*np.sin(theta_2 - theta_1) + dtheta_2*l_1*l_2*m_5*np.sin(theta_2 - theta_1) + dtheta_2*l_1*m_2*r_2*np.sin(theta_2 - theta_1)) - dtheta_1*(mu_1 + mu_2) - ddx_c*(-l_1*m_2 - l_1*m_3 - l_1*m_4 - l_1*m_5 - m_1*r_1)*np.cos(theta_1) - g*(-l_1*m_2 - l_1*m_3 - l_1*m_4 - l_1*m_5 - m_1*r_1)*np.sin(theta_1))*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2))/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2))/(I_3 - l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)**2*np.cos(theta_3 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_3**2*m_4 + l_3**2*m_5 + m_3*r_3**2 - (-l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_3 - theta_2))**2/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2)))/(I_4 - l_1**2*(l_4*m_5 + m_4*r_4)**2*np.cos(theta_4 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_4**2*m_5 + m_4*r_4**2 - (-l_1**2*(l_4*m_5 + m_4*r_4)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_4 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_4*m_5 + m_4*r_4)*np.cos(theta_4 - theta_2))**2/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2) - (-l_1**2*(l_4*m_5 + m_4*r_4)*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_4 - theta_1)*np.cos(theta_3 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_3*(l_4*m_5 + m_4*r_4)*np.cos(theta_4 - theta_3) - (-l_1**2*(l_4*m_5 + m_4*r_4)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_4 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_4*m_5 + m_4*r_4)*np.cos(theta_4 - theta_2))*(-l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_3 - theta_2))/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2))**2/(I_3 - l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)**2*np.cos(theta_3 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_3**2*m_4 + l_3**2*m_5 + m_3*r_3**2 - (-l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_3 - theta_2))**2/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2))))/(I_5 - l_1**2*m_5**2*r_5**2*np.cos(theta_5 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + m_5*r_5**2 - (-l_1**2*m_5*r_5*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_5 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*m_5*r_5*np.cos(theta_5 - theta_2))**2/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2) - (-l_1**2*m_5*r_5*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_5 - theta_1)*np.cos(theta_3 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_3*m_5*r_5*np.cos(theta_5 - theta_3) - (-l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_3 - theta_2))*(-l_1**2*m_5*r_5*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_5 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*m_5*r_5*np.cos(theta_5 - theta_2))/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2))**2/(I_3 - l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)**2*np.cos(theta_3 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_3**2*m_4 + l_3**2*m_5 + m_3*r_3**2 - (-l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_3 - theta_2))**2/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2)) - (-l_1**2*m_5*r_5*(l_4*m_5 + m_4*r_4)*np.cos(theta_5 - theta_1)*np.cos(theta_4 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_4*m_5*r_5*np.cos(theta_5 - theta_4) - (-l_1**2*(l_4*m_5 + m_4*r_4)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_4 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_4*m_5 + m_4*r_4)*np.cos(theta_4 - theta_2))*(-l_1**2*m_5*r_5*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_5 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*m_5*r_5*np.cos(theta_5 - theta_2))/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2) - (-l_1**2*(l_4*m_5 + m_4*r_4)*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_4 - theta_1)*np.cos(theta_3 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_3*(l_4*m_5 + m_4*r_4)*np.cos(theta_4 - theta_3) - (-l_1**2*(l_4*m_5 + m_4*r_4)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_4 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_4*m_5 + m_4*r_4)*np.cos(theta_4 - theta_2))*(-l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_3 - theta_2))/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2))*(-l_1**2*m_5*r_5*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_5 - theta_1)*np.cos(theta_3 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_3*m_5*r_5*np.cos(theta_5 - theta_3) - (-l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_3 - theta_2))*(-l_1**2*m_5*r_5*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_5 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*m_5*r_5*np.cos(theta_5 - theta_2))/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2))/(I_3 - l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)**2*np.cos(theta_3 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_3**2*m_4 + l_3**2*m_5 + m_3*r_3**2 - (-l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_3 - theta_2))**2/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2)))**2/(I_4 - l_1**2*(l_4*m_5 + m_4*r_4)**2*np.cos(theta_4 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_4**2*m_5 + m_4*r_4**2 - (-l_1**2*(l_4*m_5 + m_4*r_4)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_4 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_4*m_5 + m_4*r_4)*np.cos(theta_4 - theta_2))**2/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2) - (-l_1**2*(l_4*m_5 + m_4*r_4)*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_4 - theta_1)*np.cos(theta_3 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_3*(l_4*m_5 + m_4*r_4)*np.cos(theta_4 - theta_3) - (-l_1**2*(l_4*m_5 + m_4*r_4)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_4 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_4*m_5 + m_4*r_4)*np.cos(theta_4 - theta_2))*(-l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_3 - theta_2))/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2))**2/(I_3 - l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)**2*np.cos(theta_3 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_3**2*m_4 + l_3**2*m_5 + m_3*r_3**2 - (-l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_3 - theta_2))**2/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2)))))/(I_4 - l_1**2*(l_4*m_5 + m_4*r_4)**2*np.cos(theta_4 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_4**2*m_5 + m_4*r_4**2 - (-l_1**2*(l_4*m_5 + m_4*r_4)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_4 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_4*m_5 + m_4*r_4)*np.cos(theta_4 - theta_2))**2/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2) - (-l_1**2*(l_4*m_5 + m_4*r_4)*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_4 - theta_1)*np.cos(theta_3 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_3*(l_4*m_5 + m_4*r_4)*np.cos(theta_4 - theta_3) - (-l_1**2*(l_4*m_5 + m_4*r_4)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_4 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_4*m_5 + m_4*r_4)*np.cos(theta_4 - theta_2))*(-l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_3 - theta_2))/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2))**2/(I_3 - l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)**2*np.cos(theta_3 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_3**2*m_4 + l_3**2*m_5 + m_3*r_3**2 - (-l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_3 - theta_2))**2/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2))) - (-l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_3 - theta_2))*(dtheta_5**2*l_3*m_5*r_5*np.sin(theta_5 - theta_3) + dtheta_4*mu_4 + dtheta_4*(dtheta_4*l_3*l_4*m_5*np.sin(theta_4 - theta_3) + dtheta_4*l_3*m_4*r_4*np.sin(theta_4 - theta_3)) - dtheta_3*(mu_3 + mu_4) + dtheta_2*mu_3 + dtheta_2*(-dtheta_2*l_2*l_3*m_4*np.sin(theta_3 - theta_2) - dtheta_2*l_2*l_3*m_5*np.sin(theta_3 - theta_2) - dtheta_2*l_2*m_3*r_3*np.sin(theta_3 - theta_2)) + dtheta_1*(-dtheta_1*l_1*l_3*m_4*np.sin(theta_3 - theta_1) - dtheta_1*l_1*l_3*m_5*np.sin(theta_3 - theta_1) - dtheta_1*l_1*m_3*r_3*np.sin(theta_3 - theta_1)) - ddx_c*(-l_3*m_4 - l_3*m_5 - m_3*r_3)*np.cos(theta_3) - g*(-l_3*m_4 - l_3*m_5 - m_3*r_3)*np.sin(theta_3) - l_1*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(dtheta_5**2*l_1*m_5*r_5*np.sin(theta_5 - theta_1) + dtheta_4*(dtheta_4*l_1*l_4*m_5*np.sin(theta_4 - theta_1) + dtheta_4*l_1*m_4*r_4*np.sin(theta_4 - theta_1)) + dtheta_3*(dtheta_3*l_1*l_3*m_4*np.sin(theta_3 - theta_1) + dtheta_3*l_1*l_3*m_5*np.sin(theta_3 - theta_1) + dtheta_3*l_1*m_3*r_3*np.sin(theta_3 - theta_1)) + dtheta_2*mu_2 + dtheta_2*(dtheta_2*l_1*l_2*m_3*np.sin(theta_2 - theta_1) + dtheta_2*l_1*l_2*m_4*np.sin(theta_2 - theta_1) + dtheta_2*l_1*l_2*m_5*np.sin(theta_2 - theta_1) + dtheta_2*l_1*m_2*r_2*np.sin(theta_2 - theta_1)) - dtheta_1*(mu_1 + mu_2) - ddx_c*(-l_1*m_2 - l_1*m_3 - l_1*m_4 - l_1*m_5 - m_1*r_1)*np.cos(theta_1) - g*(-l_1*m_2 - l_1*m_3 - l_1*m_4 - l_1*m_5 - m_1*r_1)*np.sin(theta_1))*np.cos(theta_3 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) - (-l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_3 - theta_2))*(dtheta_5**2*l_2*m_5*r_5*np.sin(theta_5 - theta_2) + dtheta_4*(dtheta_4*l_2*l_4*m_5*np.sin(theta_4 - theta_2) + dtheta_4*l_2*m_4*r_4*np.sin(theta_4 - theta_2)) + dtheta_3*mu_3 + dtheta_3*(dtheta_3*l_2*l_3*m_4*np.sin(theta_3 - theta_2) + dtheta_3*l_2*l_3*m_5*np.sin(theta_3 - theta_2) + dtheta_3*l_2*m_3*r_3*np.sin(theta_3 - theta_2)) - dtheta_2*(mu_2 + mu_3) + dtheta_1*mu_2 + dtheta_1*(-dtheta_1*l_1*l_2*m_3*np.sin(theta_2 - theta_1) - dtheta_1*l_1*l_2*m_4*np.sin(theta_2 - theta_1) - dtheta_1*l_1*l_2*m_5*np.sin(theta_2 - theta_1) - dtheta_1*l_1*m_2*r_2*np.sin(theta_2 - theta_1)) - ddx_c*(-l_2*m_3 - l_2*m_4 - l_2*m_5 - m_2*r_2)*np.cos(theta_2) - g*(-l_2*m_3 - l_2*m_4 - l_2*m_5 - m_2*r_2)*np.sin(theta_2) - l_1*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*(dtheta_5**2*l_1*m_5*r_5*np.sin(theta_5 - theta_1) + dtheta_4*(dtheta_4*l_1*l_4*m_5*np.sin(theta_4 - theta_1) + dtheta_4*l_1*m_4*r_4*np.sin(theta_4 - theta_1)) + dtheta_3*(dtheta_3*l_1*l_3*m_4*np.sin(theta_3 - theta_1) + dtheta_3*l_1*l_3*m_5*np.sin(theta_3 - theta_1) + dtheta_3*l_1*m_3*r_3*np.sin(theta_3 - theta_1)) + dtheta_2*mu_2 + dtheta_2*(dtheta_2*l_1*l_2*m_3*np.sin(theta_2 - theta_1) + dtheta_2*l_1*l_2*m_4*np.sin(theta_2 - theta_1) + dtheta_2*l_1*l_2*m_5*np.sin(theta_2 - theta_1) + dtheta_2*l_1*m_2*r_2*np.sin(theta_2 - theta_1)) - dtheta_1*(mu_1 + mu_2) - ddx_c*(-l_1*m_2 - l_1*m_3 - l_1*m_4 - l_1*m_5 - m_1*r_1)*np.cos(theta_1) - g*(-l_1*m_2 - l_1*m_3 - l_1*m_4 - l_1*m_5 - m_1*r_1)*np.sin(theta_1))*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2))/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2) - (-l_1**2*(l_4*m_5 + m_4*r_4)*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_4 - theta_1)*np.cos(theta_3 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_3*(l_4*m_5 + m_4*r_4)*np.cos(theta_4 - theta_3) - (-l_1**2*(l_4*m_5 + m_4*r_4)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_4 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_4*m_5 + m_4*r_4)*np.cos(theta_4 - theta_2))*(-l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_3 - theta_2))/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2))*(dtheta_5**2*l_4*m_5*r_5*np.sin(theta_5 - theta_4) + dtheta_5*mu_5 - dtheta_4*(mu_4 + mu_5) + dtheta_3*mu_4 + dtheta_3*(-dtheta_3*l_3*l_4*m_5*np.sin(theta_4 - theta_3) - dtheta_3*l_3*m_4*r_4*np.sin(theta_4 - theta_3)) + dtheta_2*(-dtheta_2*l_2*l_4*m_5*np.sin(theta_4 - theta_2) - dtheta_2*l_2*m_4*r_4*np.sin(theta_4 - theta_2)) + dtheta_1*(-dtheta_1*l_1*l_4*m_5*np.sin(theta_4 - theta_1) - dtheta_1*l_1*m_4*r_4*np.sin(theta_4 - theta_1)) - ddx_c*(-l_4*m_5 - m_4*r_4)*np.cos(theta_4) - g*(-l_4*m_5 - m_4*r_4)*np.sin(theta_4) - l_1*(l_4*m_5 + m_4*r_4)*(dtheta_5**2*l_1*m_5*r_5*np.sin(theta_5 - theta_1) + dtheta_4*(dtheta_4*l_1*l_4*m_5*np.sin(theta_4 - theta_1) + dtheta_4*l_1*m_4*r_4*np.sin(theta_4 - theta_1)) + dtheta_3*(dtheta_3*l_1*l_3*m_4*np.sin(theta_3 - theta_1) + dtheta_3*l_1*l_3*m_5*np.sin(theta_3 - theta_1) + dtheta_3*l_1*m_3*r_3*np.sin(theta_3 - theta_1)) + dtheta_2*mu_2 + dtheta_2*(dtheta_2*l_1*l_2*m_3*np.sin(theta_2 - theta_1) + dtheta_2*l_1*l_2*m_4*np.sin(theta_2 - theta_1) + dtheta_2*l_1*l_2*m_5*np.sin(theta_2 - theta_1) + dtheta_2*l_1*m_2*r_2*np.sin(theta_2 - theta_1)) - dtheta_1*(mu_1 + mu_2) - ddx_c*(-l_1*m_2 - l_1*m_3 - l_1*m_4 - l_1*m_5 - m_1*r_1)*np.cos(theta_1) - g*(-l_1*m_2 - l_1*m_3 - l_1*m_4 - l_1*m_5 - m_1*r_1)*np.sin(theta_1))*np.cos(theta_4 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) - (-l_1**2*(l_4*m_5 + m_4*r_4)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_4 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_4*m_5 + m_4*r_4)*np.cos(theta_4 - theta_2))*(dtheta_5**2*l_2*m_5*r_5*np.sin(theta_5 - theta_2) + dtheta_4*(dtheta_4*l_2*l_4*m_5*np.sin(theta_4 - theta_2) + dtheta_4*l_2*m_4*r_4*np.sin(theta_4 - theta_2)) + dtheta_3*mu_3 + dtheta_3*(dtheta_3*l_2*l_3*m_4*np.sin(theta_3 - theta_2) + dtheta_3*l_2*l_3*m_5*np.sin(theta_3 - theta_2) + dtheta_3*l_2*m_3*r_3*np.sin(theta_3 - theta_2)) - dtheta_2*(mu_2 + mu_3) + dtheta_1*mu_2 + dtheta_1*(-dtheta_1*l_1*l_2*m_3*np.sin(theta_2 - theta_1) - dtheta_1*l_1*l_2*m_4*np.sin(theta_2 - theta_1) - dtheta_1*l_1*l_2*m_5*np.sin(theta_2 - theta_1) - dtheta_1*l_1*m_2*r_2*np.sin(theta_2 - theta_1)) - ddx_c*(-l_2*m_3 - l_2*m_4 - l_2*m_5 - m_2*r_2)*np.cos(theta_2) - g*(-l_2*m_3 - l_2*m_4 - l_2*m_5 - m_2*r_2)*np.sin(theta_2) - l_1*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*(dtheta_5**2*l_1*m_5*r_5*np.sin(theta_5 - theta_1) + dtheta_4*(dtheta_4*l_1*l_4*m_5*np.sin(theta_4 - theta_1) + dtheta_4*l_1*m_4*r_4*np.sin(theta_4 - theta_1)) + dtheta_3*(dtheta_3*l_1*l_3*m_4*np.sin(theta_3 - theta_1) + dtheta_3*l_1*l_3*m_5*np.sin(theta_3 - theta_1) + dtheta_3*l_1*m_3*r_3*np.sin(theta_3 - theta_1)) + dtheta_2*mu_2 + dtheta_2*(dtheta_2*l_1*l_2*m_3*np.sin(theta_2 - theta_1) + dtheta_2*l_1*l_2*m_4*np.sin(theta_2 - theta_1) + dtheta_2*l_1*l_2*m_5*np.sin(theta_2 - theta_1) + dtheta_2*l_1*m_2*r_2*np.sin(theta_2 - theta_1)) - dtheta_1*(mu_1 + mu_2) - ddx_c*(-l_1*m_2 - l_1*m_3 - l_1*m_4 - l_1*m_5 - m_1*r_1)*np.cos(theta_1) - g*(-l_1*m_2 - l_1*m_3 - l_1*m_4 - l_1*m_5 - m_1*r_1)*np.sin(theta_1))*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2))/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2) - (-l_1**2*(l_4*m_5 + m_4*r_4)*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_4 - theta_1)*np.cos(theta_3 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_3*(l_4*m_5 + m_4*r_4)*np.cos(theta_4 - theta_3) - (-l_1**2*(l_4*m_5 + m_4*r_4)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_4 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_4*m_5 + m_4*r_4)*np.cos(theta_4 - theta_2))*(-l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_3 - theta_2))/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2))*(dtheta_5**2*l_3*m_5*r_5*np.sin(theta_5 - theta_3) + dtheta_4*mu_4 + dtheta_4*(dtheta_4*l_3*l_4*m_5*np.sin(theta_4 - theta_3) + dtheta_4*l_3*m_4*r_4*np.sin(theta_4 - theta_3)) - dtheta_3*(mu_3 + mu_4) + dtheta_2*mu_3 + dtheta_2*(-dtheta_2*l_2*l_3*m_4*np.sin(theta_3 - theta_2) - dtheta_2*l_2*l_3*m_5*np.sin(theta_3 - theta_2) - dtheta_2*l_2*m_3*r_3*np.sin(theta_3 - theta_2)) + dtheta_1*(-dtheta_1*l_1*l_3*m_4*np.sin(theta_3 - theta_1) - dtheta_1*l_1*l_3*m_5*np.sin(theta_3 - theta_1) - dtheta_1*l_1*m_3*r_3*np.sin(theta_3 - theta_1)) - ddx_c*(-l_3*m_4 - l_3*m_5 - m_3*r_3)*np.cos(theta_3) - g*(-l_3*m_4 - l_3*m_5 - m_3*r_3)*np.sin(theta_3) - l_1*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(dtheta_5**2*l_1*m_5*r_5*np.sin(theta_5 - theta_1) + dtheta_4*(dtheta_4*l_1*l_4*m_5*np.sin(theta_4 - theta_1) + dtheta_4*l_1*m_4*r_4*np.sin(theta_4 - theta_1)) + dtheta_3*(dtheta_3*l_1*l_3*m_4*np.sin(theta_3 - theta_1) + dtheta_3*l_1*l_3*m_5*np.sin(theta_3 - theta_1) + dtheta_3*l_1*m_3*r_3*np.sin(theta_3 - theta_1)) + dtheta_2*mu_2 + dtheta_2*(dtheta_2*l_1*l_2*m_3*np.sin(theta_2 - theta_1) + dtheta_2*l_1*l_2*m_4*np.sin(theta_2 - theta_1) + dtheta_2*l_1*l_2*m_5*np.sin(theta_2 - theta_1) + dtheta_2*l_1*m_2*r_2*np.sin(theta_2 - theta_1)) - dtheta_1*(mu_1 + mu_2) - ddx_c*(-l_1*m_2 - l_1*m_3 - l_1*m_4 - l_1*m_5 - m_1*r_1)*np.cos(theta_1) - g*(-l_1*m_2 - l_1*m_3 - l_1*m_4 - l_1*m_5 - m_1*r_1)*np.sin(theta_1))*np.cos(theta_3 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) - (-l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_3 - theta_2))*(dtheta_5**2*l_2*m_5*r_5*np.sin(theta_5 - theta_2) + dtheta_4*(dtheta_4*l_2*l_4*m_5*np.sin(theta_4 - theta_2) + dtheta_4*l_2*m_4*r_4*np.sin(theta_4 - theta_2)) + dtheta_3*mu_3 + dtheta_3*(dtheta_3*l_2*l_3*m_4*np.sin(theta_3 - theta_2) + dtheta_3*l_2*l_3*m_5*np.sin(theta_3 - theta_2) + dtheta_3*l_2*m_3*r_3*np.sin(theta_3 - theta_2)) - dtheta_2*(mu_2 + mu_3) + dtheta_1*mu_2 + dtheta_1*(-dtheta_1*l_1*l_2*m_3*np.sin(theta_2 - theta_1) - dtheta_1*l_1*l_2*m_4*np.sin(theta_2 - theta_1) - dtheta_1*l_1*l_2*m_5*np.sin(theta_2 - theta_1) - dtheta_1*l_1*m_2*r_2*np.sin(theta_2 - theta_1)) - ddx_c*(-l_2*m_3 - l_2*m_4 - l_2*m_5 - m_2*r_2)*np.cos(theta_2) - g*(-l_2*m_3 - l_2*m_4 - l_2*m_5 - m_2*r_2)*np.sin(theta_2) - l_1*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*(dtheta_5**2*l_1*m_5*r_5*np.sin(theta_5 - theta_1) + dtheta_4*(dtheta_4*l_1*l_4*m_5*np.sin(theta_4 - theta_1) + dtheta_4*l_1*m_4*r_4*np.sin(theta_4 - theta_1)) + dtheta_3*(dtheta_3*l_1*l_3*m_4*np.sin(theta_3 - theta_1) + dtheta_3*l_1*l_3*m_5*np.sin(theta_3 - theta_1) + dtheta_3*l_1*m_3*r_3*np.sin(theta_3 - theta_1)) + dtheta_2*mu_2 + dtheta_2*(dtheta_2*l_1*l_2*m_3*np.sin(theta_2 - theta_1) + dtheta_2*l_1*l_2*m_4*np.sin(theta_2 - theta_1) + dtheta_2*l_1*l_2*m_5*np.sin(theta_2 - theta_1) + dtheta_2*l_1*m_2*r_2*np.sin(theta_2 - theta_1)) - dtheta_1*(mu_1 + mu_2) - ddx_c*(-l_1*m_2 - l_1*m_3 - l_1*m_4 - l_1*m_5 - m_1*r_1)*np.cos(theta_1) - g*(-l_1*m_2 - l_1*m_3 - l_1*m_4 - l_1*m_5 - m_1*r_1)*np.sin(theta_1))*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2))/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2))/(I_3 - l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)**2*np.cos(theta_3 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_3**2*m_4 + l_3**2*m_5 + m_3*r_3**2 - (-l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_3 - theta_2))**2/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2)) - (-l_1**2*m_5*r_5*(l_4*m_5 + m_4*r_4)*np.cos(theta_5 - theta_1)*np.cos(theta_4 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_4*m_5*r_5*np.cos(theta_5 - theta_4) - (-l_1**2*(l_4*m_5 + m_4*r_4)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_4 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_4*m_5 + m_4*r_4)*np.cos(theta_4 - theta_2))*(-l_1**2*m_5*r_5*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_5 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*m_5*r_5*np.cos(theta_5 - theta_2))/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2) - (-l_1**2*(l_4*m_5 + m_4*r_4)*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_4 - theta_1)*np.cos(theta_3 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_3*(l_4*m_5 + m_4*r_4)*np.cos(theta_4 - theta_3) - (-l_1**2*(l_4*m_5 + m_4*r_4)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_4 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_4*m_5 + m_4*r_4)*np.cos(theta_4 - theta_2))*(-l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_3 - theta_2))/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2))*(-l_1**2*m_5*r_5*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_5 - theta_1)*np.cos(theta_3 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_3*m_5*r_5*np.cos(theta_5 - theta_3) - (-l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_3 - theta_2))*(-l_1**2*m_5*r_5*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_5 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*m_5*r_5*np.cos(theta_5 - theta_2))/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2))/(I_3 - l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)**2*np.cos(theta_3 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_3**2*m_4 + l_3**2*m_5 + m_3*r_3**2 - (-l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_3 - theta_2))**2/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2)))*(-dtheta_5*mu_5 - dtheta_4**2*l_4*m_5*r_5*np.sin(theta_5 - theta_4) + dtheta_4*mu_5 - dtheta_3**2*l_3*m_5*r_5*np.sin(theta_5 - theta_3) - dtheta_2**2*l_2*m_5*r_5*np.sin(theta_5 - theta_2) - dtheta_1**2*l_1*m_5*r_5*np.sin(theta_5 - theta_1) + ddx_c*m_5*r_5*np.cos(theta_5) + g*m_5*r_5*np.sin(theta_5) - l_1*m_5*r_5*(dtheta_5**2*l_1*m_5*r_5*np.sin(theta_5 - theta_1) + dtheta_4*(dtheta_4*l_1*l_4*m_5*np.sin(theta_4 - theta_1) + dtheta_4*l_1*m_4*r_4*np.sin(theta_4 - theta_1)) + dtheta_3*(dtheta_3*l_1*l_3*m_4*np.sin(theta_3 - theta_1) + dtheta_3*l_1*l_3*m_5*np.sin(theta_3 - theta_1) + dtheta_3*l_1*m_3*r_3*np.sin(theta_3 - theta_1)) + dtheta_2*mu_2 + dtheta_2*(dtheta_2*l_1*l_2*m_3*np.sin(theta_2 - theta_1) + dtheta_2*l_1*l_2*m_4*np.sin(theta_2 - theta_1) + dtheta_2*l_1*l_2*m_5*np.sin(theta_2 - theta_1) + dtheta_2*l_1*m_2*r_2*np.sin(theta_2 - theta_1)) - dtheta_1*(mu_1 + mu_2) - ddx_c*(-l_1*m_2 - l_1*m_3 - l_1*m_4 - l_1*m_5 - m_1*r_1)*np.cos(theta_1) - g*(-l_1*m_2 - l_1*m_3 - l_1*m_4 - l_1*m_5 - m_1*r_1)*np.sin(theta_1))*np.cos(theta_5 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) - (-l_1**2*m_5*r_5*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_5 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*m_5*r_5*np.cos(theta_5 - theta_2))*(dtheta_5**2*l_2*m_5*r_5*np.sin(theta_5 - theta_2) + dtheta_4*(dtheta_4*l_2*l_4*m_5*np.sin(theta_4 - theta_2) + dtheta_4*l_2*m_4*r_4*np.sin(theta_4 - theta_2)) + dtheta_3*mu_3 + dtheta_3*(dtheta_3*l_2*l_3*m_4*np.sin(theta_3 - theta_2) + dtheta_3*l_2*l_3*m_5*np.sin(theta_3 - theta_2) + dtheta_3*l_2*m_3*r_3*np.sin(theta_3 - theta_2)) - dtheta_2*(mu_2 + mu_3) + dtheta_1*mu_2 + dtheta_1*(-dtheta_1*l_1*l_2*m_3*np.sin(theta_2 - theta_1) - dtheta_1*l_1*l_2*m_4*np.sin(theta_2 - theta_1) - dtheta_1*l_1*l_2*m_5*np.sin(theta_2 - theta_1) - dtheta_1*l_1*m_2*r_2*np.sin(theta_2 - theta_1)) - ddx_c*(-l_2*m_3 - l_2*m_4 - l_2*m_5 - m_2*r_2)*np.cos(theta_2) - g*(-l_2*m_3 - l_2*m_4 - l_2*m_5 - m_2*r_2)*np.sin(theta_2) - l_1*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*(dtheta_5**2*l_1*m_5*r_5*np.sin(theta_5 - theta_1) + dtheta_4*(dtheta_4*l_1*l_4*m_5*np.sin(theta_4 - theta_1) + dtheta_4*l_1*m_4*r_4*np.sin(theta_4 - theta_1)) + dtheta_3*(dtheta_3*l_1*l_3*m_4*np.sin(theta_3 - theta_1) + dtheta_3*l_1*l_3*m_5*np.sin(theta_3 - theta_1) + dtheta_3*l_1*m_3*r_3*np.sin(theta_3 - theta_1)) + dtheta_2*mu_2 + dtheta_2*(dtheta_2*l_1*l_2*m_3*np.sin(theta_2 - theta_1) + dtheta_2*l_1*l_2*m_4*np.sin(theta_2 - theta_1) + dtheta_2*l_1*l_2*m_5*np.sin(theta_2 - theta_1) + dtheta_2*l_1*m_2*r_2*np.sin(theta_2 - theta_1)) - dtheta_1*(mu_1 + mu_2) - ddx_c*(-l_1*m_2 - l_1*m_3 - l_1*m_4 - l_1*m_5 - m_1*r_1)*np.cos(theta_1) - g*(-l_1*m_2 - l_1*m_3 - l_1*m_4 - l_1*m_5 - m_1*r_1)*np.sin(theta_1))*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2))/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2) - (-l_1**2*m_5*r_5*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_5 - theta_1)*np.cos(theta_3 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_3*m_5*r_5*np.cos(theta_5 - theta_3) - (-l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_3 - theta_2))*(-l_1**2*m_5*r_5*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_5 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*m_5*r_5*np.cos(theta_5 - theta_2))/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2))*(dtheta_5**2*l_3*m_5*r_5*np.sin(theta_5 - theta_3) + dtheta_4*mu_4 + dtheta_4*(dtheta_4*l_3*l_4*m_5*np.sin(theta_4 - theta_3) + dtheta_4*l_3*m_4*r_4*np.sin(theta_4 - theta_3)) - dtheta_3*(mu_3 + mu_4) + dtheta_2*mu_3 + dtheta_2*(-dtheta_2*l_2*l_3*m_4*np.sin(theta_3 - theta_2) - dtheta_2*l_2*l_3*m_5*np.sin(theta_3 - theta_2) - dtheta_2*l_2*m_3*r_3*np.sin(theta_3 - theta_2)) + dtheta_1*(-dtheta_1*l_1*l_3*m_4*np.sin(theta_3 - theta_1) - dtheta_1*l_1*l_3*m_5*np.sin(theta_3 - theta_1) - dtheta_1*l_1*m_3*r_3*np.sin(theta_3 - theta_1)) - ddx_c*(-l_3*m_4 - l_3*m_5 - m_3*r_3)*np.cos(theta_3) - g*(-l_3*m_4 - l_3*m_5 - m_3*r_3)*np.sin(theta_3) - l_1*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(dtheta_5**2*l_1*m_5*r_5*np.sin(theta_5 - theta_1) + dtheta_4*(dtheta_4*l_1*l_4*m_5*np.sin(theta_4 - theta_1) + dtheta_4*l_1*m_4*r_4*np.sin(theta_4 - theta_1)) + dtheta_3*(dtheta_3*l_1*l_3*m_4*np.sin(theta_3 - theta_1) + dtheta_3*l_1*l_3*m_5*np.sin(theta_3 - theta_1) + dtheta_3*l_1*m_3*r_3*np.sin(theta_3 - theta_1)) + dtheta_2*mu_2 + dtheta_2*(dtheta_2*l_1*l_2*m_3*np.sin(theta_2 - theta_1) + dtheta_2*l_1*l_2*m_4*np.sin(theta_2 - theta_1) + dtheta_2*l_1*l_2*m_5*np.sin(theta_2 - theta_1) + dtheta_2*l_1*m_2*r_2*np.sin(theta_2 - theta_1)) - dtheta_1*(mu_1 + mu_2) - ddx_c*(-l_1*m_2 - l_1*m_3 - l_1*m_4 - l_1*m_5 - m_1*r_1)*np.cos(theta_1) - g*(-l_1*m_2 - l_1*m_3 - l_1*m_4 - l_1*m_5 - m_1*r_1)*np.sin(theta_1))*np.cos(theta_3 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) - (-l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_3 - theta_2))*(dtheta_5**2*l_2*m_5*r_5*np.sin(theta_5 - theta_2) + dtheta_4*(dtheta_4*l_2*l_4*m_5*np.sin(theta_4 - theta_2) + dtheta_4*l_2*m_4*r_4*np.sin(theta_4 - theta_2)) + dtheta_3*mu_3 + dtheta_3*(dtheta_3*l_2*l_3*m_4*np.sin(theta_3 - theta_2) + dtheta_3*l_2*l_3*m_5*np.sin(theta_3 - theta_2) + dtheta_3*l_2*m_3*r_3*np.sin(theta_3 - theta_2)) - dtheta_2*(mu_2 + mu_3) + dtheta_1*mu_2 + dtheta_1*(-dtheta_1*l_1*l_2*m_3*np.sin(theta_2 - theta_1) - dtheta_1*l_1*l_2*m_4*np.sin(theta_2 - theta_1) - dtheta_1*l_1*l_2*m_5*np.sin(theta_2 - theta_1) - dtheta_1*l_1*m_2*r_2*np.sin(theta_2 - theta_1)) - ddx_c*(-l_2*m_3 - l_2*m_4 - l_2*m_5 - m_2*r_2)*np.cos(theta_2) - g*(-l_2*m_3 - l_2*m_4 - l_2*m_5 - m_2*r_2)*np.sin(theta_2) - l_1*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*(dtheta_5**2*l_1*m_5*r_5*np.sin(theta_5 - theta_1) + dtheta_4*(dtheta_4*l_1*l_4*m_5*np.sin(theta_4 - theta_1) + dtheta_4*l_1*m_4*r_4*np.sin(theta_4 - theta_1)) + dtheta_3*(dtheta_3*l_1*l_3*m_4*np.sin(theta_3 - theta_1) + dtheta_3*l_1*l_3*m_5*np.sin(theta_3 - theta_1) + dtheta_3*l_1*m_3*r_3*np.sin(theta_3 - theta_1)) + dtheta_2*mu_2 + dtheta_2*(dtheta_2*l_1*l_2*m_3*np.sin(theta_2 - theta_1) + dtheta_2*l_1*l_2*m_4*np.sin(theta_2 - theta_1) + dtheta_2*l_1*l_2*m_5*np.sin(theta_2 - theta_1) + dtheta_2*l_1*m_2*r_2*np.sin(theta_2 - theta_1)) - dtheta_1*(mu_1 + mu_2) - ddx_c*(-l_1*m_2 - l_1*m_3 - l_1*m_4 - l_1*m_5 - m_1*r_1)*np.cos(theta_1) - g*(-l_1*m_2 - l_1*m_3 - l_1*m_4 - l_1*m_5 - m_1*r_1)*np.sin(theta_1))*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2))/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2))/(I_3 - l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)**2*np.cos(theta_3 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_3**2*m_4 + l_3**2*m_5 + m_3*r_3**2 - (-l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_3 - theta_2))**2/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2)) - (-l_1**2*m_5*r_5*(l_4*m_5 + m_4*r_4)*np.cos(theta_5 - theta_1)*np.cos(theta_4 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_4*m_5*r_5*np.cos(theta_5 - theta_4) - (-l_1**2*(l_4*m_5 + m_4*r_4)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_4 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_4*m_5 + m_4*r_4)*np.cos(theta_4 - theta_2))*(-l_1**2*m_5*r_5*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_5 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*m_5*r_5*np.cos(theta_5 - theta_2))/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2) - (-l_1**2*(l_4*m_5 + m_4*r_4)*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_4 - theta_1)*np.cos(theta_3 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_3*(l_4*m_5 + m_4*r_4)*np.cos(theta_4 - theta_3) - (-l_1**2*(l_4*m_5 + m_4*r_4)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_4 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_4*m_5 + m_4*r_4)*np.cos(theta_4 - theta_2))*(-l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_3 - theta_2))/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2))*(-l_1**2*m_5*r_5*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_5 - theta_1)*np.cos(theta_3 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_3*m_5*r_5*np.cos(theta_5 - theta_3) - (-l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_3 - theta_2))*(-l_1**2*m_5*r_5*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_5 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*m_5*r_5*np.cos(theta_5 - theta_2))/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2))/(I_3 - l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)**2*np.cos(theta_3 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_3**2*m_4 + l_3**2*m_5 + m_3*r_3**2 - (-l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_3 - theta_2))**2/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2)))*(dtheta_5**2*l_4*m_5*r_5*np.sin(theta_5 - theta_4) + dtheta_5*mu_5 - dtheta_4*(mu_4 + mu_5) + dtheta_3*mu_4 + dtheta_3*(-dtheta_3*l_3*l_4*m_5*np.sin(theta_4 - theta_3) - dtheta_3*l_3*m_4*r_4*np.sin(theta_4 - theta_3)) + dtheta_2*(-dtheta_2*l_2*l_4*m_5*np.sin(theta_4 - theta_2) - dtheta_2*l_2*m_4*r_4*np.sin(theta_4 - theta_2)) + dtheta_1*(-dtheta_1*l_1*l_4*m_5*np.sin(theta_4 - theta_1) - dtheta_1*l_1*m_4*r_4*np.sin(theta_4 - theta_1)) - ddx_c*(-l_4*m_5 - m_4*r_4)*np.cos(theta_4) - g*(-l_4*m_5 - m_4*r_4)*np.sin(theta_4) - l_1*(l_4*m_5 + m_4*r_4)*(dtheta_5**2*l_1*m_5*r_5*np.sin(theta_5 - theta_1) + dtheta_4*(dtheta_4*l_1*l_4*m_5*np.sin(theta_4 - theta_1) + dtheta_4*l_1*m_4*r_4*np.sin(theta_4 - theta_1)) + dtheta_3*(dtheta_3*l_1*l_3*m_4*np.sin(theta_3 - theta_1) + dtheta_3*l_1*l_3*m_5*np.sin(theta_3 - theta_1) + dtheta_3*l_1*m_3*r_3*np.sin(theta_3 - theta_1)) + dtheta_2*mu_2 + dtheta_2*(dtheta_2*l_1*l_2*m_3*np.sin(theta_2 - theta_1) + dtheta_2*l_1*l_2*m_4*np.sin(theta_2 - theta_1) + dtheta_2*l_1*l_2*m_5*np.sin(theta_2 - theta_1) + dtheta_2*l_1*m_2*r_2*np.sin(theta_2 - theta_1)) - dtheta_1*(mu_1 + mu_2) - ddx_c*(-l_1*m_2 - l_1*m_3 - l_1*m_4 - l_1*m_5 - m_1*r_1)*np.cos(theta_1) - g*(-l_1*m_2 - l_1*m_3 - l_1*m_4 - l_1*m_5 - m_1*r_1)*np.sin(theta_1))*np.cos(theta_4 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) - (-l_1**2*(l_4*m_5 + m_4*r_4)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_4 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_4*m_5 + m_4*r_4)*np.cos(theta_4 - theta_2))*(dtheta_5**2*l_2*m_5*r_5*np.sin(theta_5 - theta_2) + dtheta_4*(dtheta_4*l_2*l_4*m_5*np.sin(theta_4 - theta_2) + dtheta_4*l_2*m_4*r_4*np.sin(theta_4 - theta_2)) + dtheta_3*mu_3 + dtheta_3*(dtheta_3*l_2*l_3*m_4*np.sin(theta_3 - theta_2) + dtheta_3*l_2*l_3*m_5*np.sin(theta_3 - theta_2) + dtheta_3*l_2*m_3*r_3*np.sin(theta_3 - theta_2)) - dtheta_2*(mu_2 + mu_3) + dtheta_1*mu_2 + dtheta_1*(-dtheta_1*l_1*l_2*m_3*np.sin(theta_2 - theta_1) - dtheta_1*l_1*l_2*m_4*np.sin(theta_2 - theta_1) - dtheta_1*l_1*l_2*m_5*np.sin(theta_2 - theta_1) - dtheta_1*l_1*m_2*r_2*np.sin(theta_2 - theta_1)) - ddx_c*(-l_2*m_3 - l_2*m_4 - l_2*m_5 - m_2*r_2)*np.cos(theta_2) - g*(-l_2*m_3 - l_2*m_4 - l_2*m_5 - m_2*r_2)*np.sin(theta_2) - l_1*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*(dtheta_5**2*l_1*m_5*r_5*np.sin(theta_5 - theta_1) + dtheta_4*(dtheta_4*l_1*l_4*m_5*np.sin(theta_4 - theta_1) + dtheta_4*l_1*m_4*r_4*np.sin(theta_4 - theta_1)) + dtheta_3*(dtheta_3*l_1*l_3*m_4*np.sin(theta_3 - theta_1) + dtheta_3*l_1*l_3*m_5*np.sin(theta_3 - theta_1) + dtheta_3*l_1*m_3*r_3*np.sin(theta_3 - theta_1)) + dtheta_2*mu_2 + dtheta_2*(dtheta_2*l_1*l_2*m_3*np.sin(theta_2 - theta_1) + dtheta_2*l_1*l_2*m_4*np.sin(theta_2 - theta_1) + dtheta_2*l_1*l_2*m_5*np.sin(theta_2 - theta_1) + dtheta_2*l_1*m_2*r_2*np.sin(theta_2 - theta_1)) - dtheta_1*(mu_1 + mu_2) - ddx_c*(-l_1*m_2 - l_1*m_3 - l_1*m_4 - l_1*m_5 - m_1*r_1)*np.cos(theta_1) - g*(-l_1*m_2 - l_1*m_3 - l_1*m_4 - l_1*m_5 - m_1*r_1)*np.sin(theta_1))*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2))/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2) - (-l_1**2*(l_4*m_5 + m_4*r_4)*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_4 - theta_1)*np.cos(theta_3 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_3*(l_4*m_5 + m_4*r_4)*np.cos(theta_4 - theta_3) - (-l_1**2*(l_4*m_5 + m_4*r_4)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_4 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_4*m_5 + m_4*r_4)*np.cos(theta_4 - theta_2))*(-l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_3 - theta_2))/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2))*(dtheta_5**2*l_3*m_5*r_5*np.sin(theta_5 - theta_3) + dtheta_4*mu_4 + dtheta_4*(dtheta_4*l_3*l_4*m_5*np.sin(theta_4 - theta_3) + dtheta_4*l_3*m_4*r_4*np.sin(theta_4 - theta_3)) - dtheta_3*(mu_3 + mu_4) + dtheta_2*mu_3 + dtheta_2*(-dtheta_2*l_2*l_3*m_4*np.sin(theta_3 - theta_2) - dtheta_2*l_2*l_3*m_5*np.sin(theta_3 - theta_2) - dtheta_2*l_2*m_3*r_3*np.sin(theta_3 - theta_2)) + dtheta_1*(-dtheta_1*l_1*l_3*m_4*np.sin(theta_3 - theta_1) - dtheta_1*l_1*l_3*m_5*np.sin(theta_3 - theta_1) - dtheta_1*l_1*m_3*r_3*np.sin(theta_3 - theta_1)) - ddx_c*(-l_3*m_4 - l_3*m_5 - m_3*r_3)*np.cos(theta_3) - g*(-l_3*m_4 - l_3*m_5 - m_3*r_3)*np.sin(theta_3) - l_1*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(dtheta_5**2*l_1*m_5*r_5*np.sin(theta_5 - theta_1) + dtheta_4*(dtheta_4*l_1*l_4*m_5*np.sin(theta_4 - theta_1) + dtheta_4*l_1*m_4*r_4*np.sin(theta_4 - theta_1)) + dtheta_3*(dtheta_3*l_1*l_3*m_4*np.sin(theta_3 - theta_1) + dtheta_3*l_1*l_3*m_5*np.sin(theta_3 - theta_1) + dtheta_3*l_1*m_3*r_3*np.sin(theta_3 - theta_1)) + dtheta_2*mu_2 + dtheta_2*(dtheta_2*l_1*l_2*m_3*np.sin(theta_2 - theta_1) + dtheta_2*l_1*l_2*m_4*np.sin(theta_2 - theta_1) + dtheta_2*l_1*l_2*m_5*np.sin(theta_2 - theta_1) + dtheta_2*l_1*m_2*r_2*np.sin(theta_2 - theta_1)) - dtheta_1*(mu_1 + mu_2) - ddx_c*(-l_1*m_2 - l_1*m_3 - l_1*m_4 - l_1*m_5 - m_1*r_1)*np.cos(theta_1) - g*(-l_1*m_2 - l_1*m_3 - l_1*m_4 - l_1*m_5 - m_1*r_1)*np.sin(theta_1))*np.cos(theta_3 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) - (-l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_3 - theta_2))*(dtheta_5**2*l_2*m_5*r_5*np.sin(theta_5 - theta_2) + dtheta_4*(dtheta_4*l_2*l_4*m_5*np.sin(theta_4 - theta_2) + dtheta_4*l_2*m_4*r_4*np.sin(theta_4 - theta_2)) + dtheta_3*mu_3 + dtheta_3*(dtheta_3*l_2*l_3*m_4*np.sin(theta_3 - theta_2) + dtheta_3*l_2*l_3*m_5*np.sin(theta_3 - theta_2) + dtheta_3*l_2*m_3*r_3*np.sin(theta_3 - theta_2)) - dtheta_2*(mu_2 + mu_3) + dtheta_1*mu_2 + dtheta_1*(-dtheta_1*l_1*l_2*m_3*np.sin(theta_2 - theta_1) - dtheta_1*l_1*l_2*m_4*np.sin(theta_2 - theta_1) - dtheta_1*l_1*l_2*m_5*np.sin(theta_2 - theta_1) - dtheta_1*l_1*m_2*r_2*np.sin(theta_2 - theta_1)) - ddx_c*(-l_2*m_3 - l_2*m_4 - l_2*m_5 - m_2*r_2)*np.cos(theta_2) - g*(-l_2*m_3 - l_2*m_4 - l_2*m_5 - m_2*r_2)*np.sin(theta_2) - l_1*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*(dtheta_5**2*l_1*m_5*r_5*np.sin(theta_5 - theta_1) + dtheta_4*(dtheta_4*l_1*l_4*m_5*np.sin(theta_4 - theta_1) + dtheta_4*l_1*m_4*r_4*np.sin(theta_4 - theta_1)) + dtheta_3*(dtheta_3*l_1*l_3*m_4*np.sin(theta_3 - theta_1) + dtheta_3*l_1*l_3*m_5*np.sin(theta_3 - theta_1) + dtheta_3*l_1*m_3*r_3*np.sin(theta_3 - theta_1)) + dtheta_2*mu_2 + dtheta_2*(dtheta_2*l_1*l_2*m_3*np.sin(theta_2 - theta_1) + dtheta_2*l_1*l_2*m_4*np.sin(theta_2 - theta_1) + dtheta_2*l_1*l_2*m_5*np.sin(theta_2 - theta_1) + dtheta_2*l_1*m_2*r_2*np.sin(theta_2 - theta_1)) - dtheta_1*(mu_1 + mu_2) - ddx_c*(-l_1*m_2 - l_1*m_3 - l_1*m_4 - l_1*m_5 - m_1*r_1)*np.cos(theta_1) - g*(-l_1*m_2 - l_1*m_3 - l_1*m_4 - l_1*m_5 - m_1*r_1)*np.sin(theta_1))*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2))/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2))/(I_3 - l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)**2*np.cos(theta_3 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_3**2*m_4 + l_3**2*m_5 + m_3*r_3**2 - (-l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_3 - theta_2))**2/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2)))/(I_4 - l_1**2*(l_4*m_5 + m_4*r_4)**2*np.cos(theta_4 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_4**2*m_5 + m_4*r_4**2 - (-l_1**2*(l_4*m_5 + m_4*r_4)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_4 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_4*m_5 + m_4*r_4)*np.cos(theta_4 - theta_2))**2/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2) - (-l_1**2*(l_4*m_5 + m_4*r_4)*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_4 - theta_1)*np.cos(theta_3 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_3*(l_4*m_5 + m_4*r_4)*np.cos(theta_4 - theta_3) - (-l_1**2*(l_4*m_5 + m_4*r_4)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_4 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_4*m_5 + m_4*r_4)*np.cos(theta_4 - theta_2))*(-l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_3 - theta_2))/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2))**2/(I_3 - l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)**2*np.cos(theta_3 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_3**2*m_4 + l_3**2*m_5 + m_3*r_3**2 - (-l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_3 - theta_2))**2/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2))))/(I_5 - l_1**2*m_5**2*r_5**2*np.cos(theta_5 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + m_5*r_5**2 - (-l_1**2*m_5*r_5*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_5 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*m_5*r_5*np.cos(theta_5 - theta_2))**2/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2) - (-l_1**2*m_5*r_5*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_5 - theta_1)*np.cos(theta_3 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_3*m_5*r_5*np.cos(theta_5 - theta_3) - (-l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_3 - theta_2))*(-l_1**2*m_5*r_5*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_5 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*m_5*r_5*np.cos(theta_5 - theta_2))/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2))**2/(I_3 - l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)**2*np.cos(theta_3 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_3**2*m_4 + l_3**2*m_5 + m_3*r_3**2 - (-l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_3 - theta_2))**2/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2)) - (-l_1**2*m_5*r_5*(l_4*m_5 + m_4*r_4)*np.cos(theta_5 - theta_1)*np.cos(theta_4 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_4*m_5*r_5*np.cos(theta_5 - theta_4) - (-l_1**2*(l_4*m_5 + m_4*r_4)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_4 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_4*m_5 + m_4*r_4)*np.cos(theta_4 - theta_2))*(-l_1**2*m_5*r_5*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_5 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*m_5*r_5*np.cos(theta_5 - theta_2))/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2) - (-l_1**2*(l_4*m_5 + m_4*r_4)*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_4 - theta_1)*np.cos(theta_3 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_3*(l_4*m_5 + m_4*r_4)*np.cos(theta_4 - theta_3) - (-l_1**2*(l_4*m_5 + m_4*r_4)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_4 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_4*m_5 + m_4*r_4)*np.cos(theta_4 - theta_2))*(-l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_3 - theta_2))/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2))*(-l_1**2*m_5*r_5*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_5 - theta_1)*np.cos(theta_3 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_3*m_5*r_5*np.cos(theta_5 - theta_3) - (-l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_3 - theta_2))*(-l_1**2*m_5*r_5*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_5 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*m_5*r_5*np.cos(theta_5 - theta_2))/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2))/(I_3 - l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)**2*np.cos(theta_3 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_3**2*m_4 + l_3**2*m_5 + m_3*r_3**2 - (-l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_3 - theta_2))**2/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2)))**2/(I_4 - l_1**2*(l_4*m_5 + m_4*r_4)**2*np.cos(theta_4 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_4**2*m_5 + m_4*r_4**2 - (-l_1**2*(l_4*m_5 + m_4*r_4)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_4 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_4*m_5 + m_4*r_4)*np.cos(theta_4 - theta_2))**2/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2) - (-l_1**2*(l_4*m_5 + m_4*r_4)*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_4 - theta_1)*np.cos(theta_3 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_3*(l_4*m_5 + m_4*r_4)*np.cos(theta_4 - theta_3) - (-l_1**2*(l_4*m_5 + m_4*r_4)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_4 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_4*m_5 + m_4*r_4)*np.cos(theta_4 - theta_2))*(-l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_3 - theta_2))/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2))**2/(I_3 - l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)**2*np.cos(theta_3 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_3**2*m_4 + l_3**2*m_5 + m_3*r_3**2 - (-l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_3 - theta_2))**2/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2)))))/(I_4 - l_1**2*(l_4*m_5 + m_4*r_4)**2*np.cos(theta_4 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_4**2*m_5 + m_4*r_4**2 - (-l_1**2*(l_4*m_5 + m_4*r_4)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_4 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_4*m_5 + m_4*r_4)*np.cos(theta_4 - theta_2))**2/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2) - (-l_1**2*(l_4*m_5 + m_4*r_4)*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_4 - theta_1)*np.cos(theta_3 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_3*(l_4*m_5 + m_4*r_4)*np.cos(theta_4 - theta_3) - (-l_1**2*(l_4*m_5 + m_4*r_4)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_4 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_4*m_5 + m_4*r_4)*np.cos(theta_4 - theta_2))*(-l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_3 - theta_2))/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2))**2/(I_3 - l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)**2*np.cos(theta_3 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_3**2*m_4 + l_3**2*m_5 + m_3*r_3**2 - (-l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_3 - theta_2))**2/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2))) - (-l_1**2*m_5*r_5*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_5 - theta_1)*np.cos(theta_3 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_3*m_5*r_5*np.cos(theta_5 - theta_3) - (-l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_3 - theta_2))*(-l_1**2*m_5*r_5*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_5 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*m_5*r_5*np.cos(theta_5 - theta_2))/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2))*(-dtheta_5*mu_5 - dtheta_4**2*l_4*m_5*r_5*np.sin(theta_5 - theta_4) + dtheta_4*mu_5 - dtheta_3**2*l_3*m_5*r_5*np.sin(theta_5 - theta_3) - dtheta_2**2*l_2*m_5*r_5*np.sin(theta_5 - theta_2) - dtheta_1**2*l_1*m_5*r_5*np.sin(theta_5 - theta_1) + ddx_c*m_5*r_5*np.cos(theta_5) + g*m_5*r_5*np.sin(theta_5) - l_1*m_5*r_5*(dtheta_5**2*l_1*m_5*r_5*np.sin(theta_5 - theta_1) + dtheta_4*(dtheta_4*l_1*l_4*m_5*np.sin(theta_4 - theta_1) + dtheta_4*l_1*m_4*r_4*np.sin(theta_4 - theta_1)) + dtheta_3*(dtheta_3*l_1*l_3*m_4*np.sin(theta_3 - theta_1) + dtheta_3*l_1*l_3*m_5*np.sin(theta_3 - theta_1) + dtheta_3*l_1*m_3*r_3*np.sin(theta_3 - theta_1)) + dtheta_2*mu_2 + dtheta_2*(dtheta_2*l_1*l_2*m_3*np.sin(theta_2 - theta_1) + dtheta_2*l_1*l_2*m_4*np.sin(theta_2 - theta_1) + dtheta_2*l_1*l_2*m_5*np.sin(theta_2 - theta_1) + dtheta_2*l_1*m_2*r_2*np.sin(theta_2 - theta_1)) - dtheta_1*(mu_1 + mu_2) - ddx_c*(-l_1*m_2 - l_1*m_3 - l_1*m_4 - l_1*m_5 - m_1*r_1)*np.cos(theta_1) - g*(-l_1*m_2 - l_1*m_3 - l_1*m_4 - l_1*m_5 - m_1*r_1)*np.sin(theta_1))*np.cos(theta_5 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) - (-l_1**2*m_5*r_5*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_5 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*m_5*r_5*np.cos(theta_5 - theta_2))*(dtheta_5**2*l_2*m_5*r_5*np.sin(theta_5 - theta_2) + dtheta_4*(dtheta_4*l_2*l_4*m_5*np.sin(theta_4 - theta_2) + dtheta_4*l_2*m_4*r_4*np.sin(theta_4 - theta_2)) + dtheta_3*mu_3 + dtheta_3*(dtheta_3*l_2*l_3*m_4*np.sin(theta_3 - theta_2) + dtheta_3*l_2*l_3*m_5*np.sin(theta_3 - theta_2) + dtheta_3*l_2*m_3*r_3*np.sin(theta_3 - theta_2)) - dtheta_2*(mu_2 + mu_3) + dtheta_1*mu_2 + dtheta_1*(-dtheta_1*l_1*l_2*m_3*np.sin(theta_2 - theta_1) - dtheta_1*l_1*l_2*m_4*np.sin(theta_2 - theta_1) - dtheta_1*l_1*l_2*m_5*np.sin(theta_2 - theta_1) - dtheta_1*l_1*m_2*r_2*np.sin(theta_2 - theta_1)) - ddx_c*(-l_2*m_3 - l_2*m_4 - l_2*m_5 - m_2*r_2)*np.cos(theta_2) - g*(-l_2*m_3 - l_2*m_4 - l_2*m_5 - m_2*r_2)*np.sin(theta_2) - l_1*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*(dtheta_5**2*l_1*m_5*r_5*np.sin(theta_5 - theta_1) + dtheta_4*(dtheta_4*l_1*l_4*m_5*np.sin(theta_4 - theta_1) + dtheta_4*l_1*m_4*r_4*np.sin(theta_4 - theta_1)) + dtheta_3*(dtheta_3*l_1*l_3*m_4*np.sin(theta_3 - theta_1) + dtheta_3*l_1*l_3*m_5*np.sin(theta_3 - theta_1) + dtheta_3*l_1*m_3*r_3*np.sin(theta_3 - theta_1)) + dtheta_2*mu_2 + dtheta_2*(dtheta_2*l_1*l_2*m_3*np.sin(theta_2 - theta_1) + dtheta_2*l_1*l_2*m_4*np.sin(theta_2 - theta_1) + dtheta_2*l_1*l_2*m_5*np.sin(theta_2 - theta_1) + dtheta_2*l_1*m_2*r_2*np.sin(theta_2 - theta_1)) - dtheta_1*(mu_1 + mu_2) - ddx_c*(-l_1*m_2 - l_1*m_3 - l_1*m_4 - l_1*m_5 - m_1*r_1)*np.cos(theta_1) - g*(-l_1*m_2 - l_1*m_3 - l_1*m_4 - l_1*m_5 - m_1*r_1)*np.sin(theta_1))*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2))/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2) - (-l_1**2*m_5*r_5*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_5 - theta_1)*np.cos(theta_3 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_3*m_5*r_5*np.cos(theta_5 - theta_3) - (-l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_3 - theta_2))*(-l_1**2*m_5*r_5*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_5 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*m_5*r_5*np.cos(theta_5 - theta_2))/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2))*(dtheta_5**2*l_3*m_5*r_5*np.sin(theta_5 - theta_3) + dtheta_4*mu_4 + dtheta_4*(dtheta_4*l_3*l_4*m_5*np.sin(theta_4 - theta_3) + dtheta_4*l_3*m_4*r_4*np.sin(theta_4 - theta_3)) - dtheta_3*(mu_3 + mu_4) + dtheta_2*mu_3 + dtheta_2*(-dtheta_2*l_2*l_3*m_4*np.sin(theta_3 - theta_2) - dtheta_2*l_2*l_3*m_5*np.sin(theta_3 - theta_2) - dtheta_2*l_2*m_3*r_3*np.sin(theta_3 - theta_2)) + dtheta_1*(-dtheta_1*l_1*l_3*m_4*np.sin(theta_3 - theta_1) - dtheta_1*l_1*l_3*m_5*np.sin(theta_3 - theta_1) - dtheta_1*l_1*m_3*r_3*np.sin(theta_3 - theta_1)) - ddx_c*(-l_3*m_4 - l_3*m_5 - m_3*r_3)*np.cos(theta_3) - g*(-l_3*m_4 - l_3*m_5 - m_3*r_3)*np.sin(theta_3) - l_1*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(dtheta_5**2*l_1*m_5*r_5*np.sin(theta_5 - theta_1) + dtheta_4*(dtheta_4*l_1*l_4*m_5*np.sin(theta_4 - theta_1) + dtheta_4*l_1*m_4*r_4*np.sin(theta_4 - theta_1)) + dtheta_3*(dtheta_3*l_1*l_3*m_4*np.sin(theta_3 - theta_1) + dtheta_3*l_1*l_3*m_5*np.sin(theta_3 - theta_1) + dtheta_3*l_1*m_3*r_3*np.sin(theta_3 - theta_1)) + dtheta_2*mu_2 + dtheta_2*(dtheta_2*l_1*l_2*m_3*np.sin(theta_2 - theta_1) + dtheta_2*l_1*l_2*m_4*np.sin(theta_2 - theta_1) + dtheta_2*l_1*l_2*m_5*np.sin(theta_2 - theta_1) + dtheta_2*l_1*m_2*r_2*np.sin(theta_2 - theta_1)) - dtheta_1*(mu_1 + mu_2) - ddx_c*(-l_1*m_2 - l_1*m_3 - l_1*m_4 - l_1*m_5 - m_1*r_1)*np.cos(theta_1) - g*(-l_1*m_2 - l_1*m_3 - l_1*m_4 - l_1*m_5 - m_1*r_1)*np.sin(theta_1))*np.cos(theta_3 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) - (-l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_3 - theta_2))*(dtheta_5**2*l_2*m_5*r_5*np.sin(theta_5 - theta_2) + dtheta_4*(dtheta_4*l_2*l_4*m_5*np.sin(theta_4 - theta_2) + dtheta_4*l_2*m_4*r_4*np.sin(theta_4 - theta_2)) + dtheta_3*mu_3 + dtheta_3*(dtheta_3*l_2*l_3*m_4*np.sin(theta_3 - theta_2) + dtheta_3*l_2*l_3*m_5*np.sin(theta_3 - theta_2) + dtheta_3*l_2*m_3*r_3*np.sin(theta_3 - theta_2)) - dtheta_2*(mu_2 + mu_3) + dtheta_1*mu_2 + dtheta_1*(-dtheta_1*l_1*l_2*m_3*np.sin(theta_2 - theta_1) - dtheta_1*l_1*l_2*m_4*np.sin(theta_2 - theta_1) - dtheta_1*l_1*l_2*m_5*np.sin(theta_2 - theta_1) - dtheta_1*l_1*m_2*r_2*np.sin(theta_2 - theta_1)) - ddx_c*(-l_2*m_3 - l_2*m_4 - l_2*m_5 - m_2*r_2)*np.cos(theta_2) - g*(-l_2*m_3 - l_2*m_4 - l_2*m_5 - m_2*r_2)*np.sin(theta_2) - l_1*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*(dtheta_5**2*l_1*m_5*r_5*np.sin(theta_5 - theta_1) + dtheta_4*(dtheta_4*l_1*l_4*m_5*np.sin(theta_4 - theta_1) + dtheta_4*l_1*m_4*r_4*np.sin(theta_4 - theta_1)) + dtheta_3*(dtheta_3*l_1*l_3*m_4*np.sin(theta_3 - theta_1) + dtheta_3*l_1*l_3*m_5*np.sin(theta_3 - theta_1) + dtheta_3*l_1*m_3*r_3*np.sin(theta_3 - theta_1)) + dtheta_2*mu_2 + dtheta_2*(dtheta_2*l_1*l_2*m_3*np.sin(theta_2 - theta_1) + dtheta_2*l_1*l_2*m_4*np.sin(theta_2 - theta_1) + dtheta_2*l_1*l_2*m_5*np.sin(theta_2 - theta_1) + dtheta_2*l_1*m_2*r_2*np.sin(theta_2 - theta_1)) - dtheta_1*(mu_1 + mu_2) - ddx_c*(-l_1*m_2 - l_1*m_3 - l_1*m_4 - l_1*m_5 - m_1*r_1)*np.cos(theta_1) - g*(-l_1*m_2 - l_1*m_3 - l_1*m_4 - l_1*m_5 - m_1*r_1)*np.sin(theta_1))*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2))/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2))/(I_3 - l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)**2*np.cos(theta_3 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_3**2*m_4 + l_3**2*m_5 + m_3*r_3**2 - (-l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_3 - theta_2))**2/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2)) - (-l_1**2*m_5*r_5*(l_4*m_5 + m_4*r_4)*np.cos(theta_5 - theta_1)*np.cos(theta_4 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_4*m_5*r_5*np.cos(theta_5 - theta_4) - (-l_1**2*(l_4*m_5 + m_4*r_4)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_4 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_4*m_5 + m_4*r_4)*np.cos(theta_4 - theta_2))*(-l_1**2*m_5*r_5*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_5 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*m_5*r_5*np.cos(theta_5 - theta_2))/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2) - (-l_1**2*(l_4*m_5 + m_4*r_4)*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_4 - theta_1)*np.cos(theta_3 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_3*(l_4*m_5 + m_4*r_4)*np.cos(theta_4 - theta_3) - (-l_1**2*(l_4*m_5 + m_4*r_4)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_4 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_4*m_5 + m_4*r_4)*np.cos(theta_4 - theta_2))*(-l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_3 - theta_2))/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2))*(-l_1**2*m_5*r_5*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_5 - theta_1)*np.cos(theta_3 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_3*m_5*r_5*np.cos(theta_5 - theta_3) - (-l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_3 - theta_2))*(-l_1**2*m_5*r_5*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_5 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*m_5*r_5*np.cos(theta_5 - theta_2))/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2))/(I_3 - l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)**2*np.cos(theta_3 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_3**2*m_4 + l_3**2*m_5 + m_3*r_3**2 - (-l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_3 - theta_2))**2/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2)))*(dtheta_5**2*l_4*m_5*r_5*np.sin(theta_5 - theta_4) + dtheta_5*mu_5 - dtheta_4*(mu_4 + mu_5) + dtheta_3*mu_4 + dtheta_3*(-dtheta_3*l_3*l_4*m_5*np.sin(theta_4 - theta_3) - dtheta_3*l_3*m_4*r_4*np.sin(theta_4 - theta_3)) + dtheta_2*(-dtheta_2*l_2*l_4*m_5*np.sin(theta_4 - theta_2) - dtheta_2*l_2*m_4*r_4*np.sin(theta_4 - theta_2)) + dtheta_1*(-dtheta_1*l_1*l_4*m_5*np.sin(theta_4 - theta_1) - dtheta_1*l_1*m_4*r_4*np.sin(theta_4 - theta_1)) - ddx_c*(-l_4*m_5 - m_4*r_4)*np.cos(theta_4) - g*(-l_4*m_5 - m_4*r_4)*np.sin(theta_4) - l_1*(l_4*m_5 + m_4*r_4)*(dtheta_5**2*l_1*m_5*r_5*np.sin(theta_5 - theta_1) + dtheta_4*(dtheta_4*l_1*l_4*m_5*np.sin(theta_4 - theta_1) + dtheta_4*l_1*m_4*r_4*np.sin(theta_4 - theta_1)) + dtheta_3*(dtheta_3*l_1*l_3*m_4*np.sin(theta_3 - theta_1) + dtheta_3*l_1*l_3*m_5*np.sin(theta_3 - theta_1) + dtheta_3*l_1*m_3*r_3*np.sin(theta_3 - theta_1)) + dtheta_2*mu_2 + dtheta_2*(dtheta_2*l_1*l_2*m_3*np.sin(theta_2 - theta_1) + dtheta_2*l_1*l_2*m_4*np.sin(theta_2 - theta_1) + dtheta_2*l_1*l_2*m_5*np.sin(theta_2 - theta_1) + dtheta_2*l_1*m_2*r_2*np.sin(theta_2 - theta_1)) - dtheta_1*(mu_1 + mu_2) - ddx_c*(-l_1*m_2 - l_1*m_3 - l_1*m_4 - l_1*m_5 - m_1*r_1)*np.cos(theta_1) - g*(-l_1*m_2 - l_1*m_3 - l_1*m_4 - l_1*m_5 - m_1*r_1)*np.sin(theta_1))*np.cos(theta_4 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) - (-l_1**2*(l_4*m_5 + m_4*r_4)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_4 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_4*m_5 + m_4*r_4)*np.cos(theta_4 - theta_2))*(dtheta_5**2*l_2*m_5*r_5*np.sin(theta_5 - theta_2) + dtheta_4*(dtheta_4*l_2*l_4*m_5*np.sin(theta_4 - theta_2) + dtheta_4*l_2*m_4*r_4*np.sin(theta_4 - theta_2)) + dtheta_3*mu_3 + dtheta_3*(dtheta_3*l_2*l_3*m_4*np.sin(theta_3 - theta_2) + dtheta_3*l_2*l_3*m_5*np.sin(theta_3 - theta_2) + dtheta_3*l_2*m_3*r_3*np.sin(theta_3 - theta_2)) - dtheta_2*(mu_2 + mu_3) + dtheta_1*mu_2 + dtheta_1*(-dtheta_1*l_1*l_2*m_3*np.sin(theta_2 - theta_1) - dtheta_1*l_1*l_2*m_4*np.sin(theta_2 - theta_1) - dtheta_1*l_1*l_2*m_5*np.sin(theta_2 - theta_1) - dtheta_1*l_1*m_2*r_2*np.sin(theta_2 - theta_1)) - ddx_c*(-l_2*m_3 - l_2*m_4 - l_2*m_5 - m_2*r_2)*np.cos(theta_2) - g*(-l_2*m_3 - l_2*m_4 - l_2*m_5 - m_2*r_2)*np.sin(theta_2) - l_1*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*(dtheta_5**2*l_1*m_5*r_5*np.sin(theta_5 - theta_1) + dtheta_4*(dtheta_4*l_1*l_4*m_5*np.sin(theta_4 - theta_1) + dtheta_4*l_1*m_4*r_4*np.sin(theta_4 - theta_1)) + dtheta_3*(dtheta_3*l_1*l_3*m_4*np.sin(theta_3 - theta_1) + dtheta_3*l_1*l_3*m_5*np.sin(theta_3 - theta_1) + dtheta_3*l_1*m_3*r_3*np.sin(theta_3 - theta_1)) + dtheta_2*mu_2 + dtheta_2*(dtheta_2*l_1*l_2*m_3*np.sin(theta_2 - theta_1) + dtheta_2*l_1*l_2*m_4*np.sin(theta_2 - theta_1) + dtheta_2*l_1*l_2*m_5*np.sin(theta_2 - theta_1) + dtheta_2*l_1*m_2*r_2*np.sin(theta_2 - theta_1)) - dtheta_1*(mu_1 + mu_2) - ddx_c*(-l_1*m_2 - l_1*m_3 - l_1*m_4 - l_1*m_5 - m_1*r_1)*np.cos(theta_1) - g*(-l_1*m_2 - l_1*m_3 - l_1*m_4 - l_1*m_5 - m_1*r_1)*np.sin(theta_1))*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2))/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2) - (-l_1**2*(l_4*m_5 + m_4*r_4)*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_4 - theta_1)*np.cos(theta_3 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_3*(l_4*m_5 + m_4*r_4)*np.cos(theta_4 - theta_3) - (-l_1**2*(l_4*m_5 + m_4*r_4)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_4 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_4*m_5 + m_4*r_4)*np.cos(theta_4 - theta_2))*(-l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_3 - theta_2))/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2))*(dtheta_5**2*l_3*m_5*r_5*np.sin(theta_5 - theta_3) + dtheta_4*mu_4 + dtheta_4*(dtheta_4*l_3*l_4*m_5*np.sin(theta_4 - theta_3) + dtheta_4*l_3*m_4*r_4*np.sin(theta_4 - theta_3)) - dtheta_3*(mu_3 + mu_4) + dtheta_2*mu_3 + dtheta_2*(-dtheta_2*l_2*l_3*m_4*np.sin(theta_3 - theta_2) - dtheta_2*l_2*l_3*m_5*np.sin(theta_3 - theta_2) - dtheta_2*l_2*m_3*r_3*np.sin(theta_3 - theta_2)) + dtheta_1*(-dtheta_1*l_1*l_3*m_4*np.sin(theta_3 - theta_1) - dtheta_1*l_1*l_3*m_5*np.sin(theta_3 - theta_1) - dtheta_1*l_1*m_3*r_3*np.sin(theta_3 - theta_1)) - ddx_c*(-l_3*m_4 - l_3*m_5 - m_3*r_3)*np.cos(theta_3) - g*(-l_3*m_4 - l_3*m_5 - m_3*r_3)*np.sin(theta_3) - l_1*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(dtheta_5**2*l_1*m_5*r_5*np.sin(theta_5 - theta_1) + dtheta_4*(dtheta_4*l_1*l_4*m_5*np.sin(theta_4 - theta_1) + dtheta_4*l_1*m_4*r_4*np.sin(theta_4 - theta_1)) + dtheta_3*(dtheta_3*l_1*l_3*m_4*np.sin(theta_3 - theta_1) + dtheta_3*l_1*l_3*m_5*np.sin(theta_3 - theta_1) + dtheta_3*l_1*m_3*r_3*np.sin(theta_3 - theta_1)) + dtheta_2*mu_2 + dtheta_2*(dtheta_2*l_1*l_2*m_3*np.sin(theta_2 - theta_1) + dtheta_2*l_1*l_2*m_4*np.sin(theta_2 - theta_1) + dtheta_2*l_1*l_2*m_5*np.sin(theta_2 - theta_1) + dtheta_2*l_1*m_2*r_2*np.sin(theta_2 - theta_1)) - dtheta_1*(mu_1 + mu_2) - ddx_c*(-l_1*m_2 - l_1*m_3 - l_1*m_4 - l_1*m_5 - m_1*r_1)*np.cos(theta_1) - g*(-l_1*m_2 - l_1*m_3 - l_1*m_4 - l_1*m_5 - m_1*r_1)*np.sin(theta_1))*np.cos(theta_3 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) - (-l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_3 - theta_2))*(dtheta_5**2*l_2*m_5*r_5*np.sin(theta_5 - theta_2) + dtheta_4*(dtheta_4*l_2*l_4*m_5*np.sin(theta_4 - theta_2) + dtheta_4*l_2*m_4*r_4*np.sin(theta_4 - theta_2)) + dtheta_3*mu_3 + dtheta_3*(dtheta_3*l_2*l_3*m_4*np.sin(theta_3 - theta_2) + dtheta_3*l_2*l_3*m_5*np.sin(theta_3 - theta_2) + dtheta_3*l_2*m_3*r_3*np.sin(theta_3 - theta_2)) - dtheta_2*(mu_2 + mu_3) + dtheta_1*mu_2 + dtheta_1*(-dtheta_1*l_1*l_2*m_3*np.sin(theta_2 - theta_1) - dtheta_1*l_1*l_2*m_4*np.sin(theta_2 - theta_1) - dtheta_1*l_1*l_2*m_5*np.sin(theta_2 - theta_1) - dtheta_1*l_1*m_2*r_2*np.sin(theta_2 - theta_1)) - ddx_c*(-l_2*m_3 - l_2*m_4 - l_2*m_5 - m_2*r_2)*np.cos(theta_2) - g*(-l_2*m_3 - l_2*m_4 - l_2*m_5 - m_2*r_2)*np.sin(theta_2) - l_1*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*(dtheta_5**2*l_1*m_5*r_5*np.sin(theta_5 - theta_1) + dtheta_4*(dtheta_4*l_1*l_4*m_5*np.sin(theta_4 - theta_1) + dtheta_4*l_1*m_4*r_4*np.sin(theta_4 - theta_1)) + dtheta_3*(dtheta_3*l_1*l_3*m_4*np.sin(theta_3 - theta_1) + dtheta_3*l_1*l_3*m_5*np.sin(theta_3 - theta_1) + dtheta_3*l_1*m_3*r_3*np.sin(theta_3 - theta_1)) + dtheta_2*mu_2 + dtheta_2*(dtheta_2*l_1*l_2*m_3*np.sin(theta_2 - theta_1) + dtheta_2*l_1*l_2*m_4*np.sin(theta_2 - theta_1) + dtheta_2*l_1*l_2*m_5*np.sin(theta_2 - theta_1) + dtheta_2*l_1*m_2*r_2*np.sin(theta_2 - theta_1)) - dtheta_1*(mu_1 + mu_2) - ddx_c*(-l_1*m_2 - l_1*m_3 - l_1*m_4 - l_1*m_5 - m_1*r_1)*np.cos(theta_1) - g*(-l_1*m_2 - l_1*m_3 - l_1*m_4 - l_1*m_5 - m_1*r_1)*np.sin(theta_1))*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2))/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2))/(I_3 - l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)**2*np.cos(theta_3 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_3**2*m_4 + l_3**2*m_5 + m_3*r_3**2 - (-l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_3 - theta_2))**2/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2)))/(I_4 - l_1**2*(l_4*m_5 + m_4*r_4)**2*np.cos(theta_4 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_4**2*m_5 + m_4*r_4**2 - (-l_1**2*(l_4*m_5 + m_4*r_4)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_4 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_4*m_5 + m_4*r_4)*np.cos(theta_4 - theta_2))**2/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2) - (-l_1**2*(l_4*m_5 + m_4*r_4)*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_4 - theta_1)*np.cos(theta_3 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_3*(l_4*m_5 + m_4*r_4)*np.cos(theta_4 - theta_3) - (-l_1**2*(l_4*m_5 + m_4*r_4)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_4 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_4*m_5 + m_4*r_4)*np.cos(theta_4 - theta_2))*(-l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_3 - theta_2))/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2))**2/(I_3 - l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)**2*np.cos(theta_3 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_3**2*m_4 + l_3**2*m_5 + m_3*r_3**2 - (-l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_3 - theta_2))**2/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2))))/(I_5 - l_1**2*m_5**2*r_5**2*np.cos(theta_5 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + m_5*r_5**2 - (-l_1**2*m_5*r_5*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_5 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*m_5*r_5*np.cos(theta_5 - theta_2))**2/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2) - (-l_1**2*m_5*r_5*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_5 - theta_1)*np.cos(theta_3 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_3*m_5*r_5*np.cos(theta_5 - theta_3) - (-l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_3 - theta_2))*(-l_1**2*m_5*r_5*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_5 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*m_5*r_5*np.cos(theta_5 - theta_2))/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2))**2/(I_3 - l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)**2*np.cos(theta_3 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_3**2*m_4 + l_3**2*m_5 + m_3*r_3**2 - (-l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_3 - theta_2))**2/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2)) - (-l_1**2*m_5*r_5*(l_4*m_5 + m_4*r_4)*np.cos(theta_5 - theta_1)*np.cos(theta_4 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_4*m_5*r_5*np.cos(theta_5 - theta_4) - (-l_1**2*(l_4*m_5 + m_4*r_4)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_4 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_4*m_5 + m_4*r_4)*np.cos(theta_4 - theta_2))*(-l_1**2*m_5*r_5*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_5 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*m_5*r_5*np.cos(theta_5 - theta_2))/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2) - (-l_1**2*(l_4*m_5 + m_4*r_4)*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_4 - theta_1)*np.cos(theta_3 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_3*(l_4*m_5 + m_4*r_4)*np.cos(theta_4 - theta_3) - (-l_1**2*(l_4*m_5 + m_4*r_4)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_4 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_4*m_5 + m_4*r_4)*np.cos(theta_4 - theta_2))*(-l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_3 - theta_2))/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2))*(-l_1**2*m_5*r_5*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_5 - theta_1)*np.cos(theta_3 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_3*m_5*r_5*np.cos(theta_5 - theta_3) - (-l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_3 - theta_2))*(-l_1**2*m_5*r_5*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_5 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*m_5*r_5*np.cos(theta_5 - theta_2))/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2))/(I_3 - l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)**2*np.cos(theta_3 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_3**2*m_4 + l_3**2*m_5 + m_3*r_3**2 - (-l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_3 - theta_2))**2/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2)))**2/(I_4 - l_1**2*(l_4*m_5 + m_4*r_4)**2*np.cos(theta_4 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_4**2*m_5 + m_4*r_4**2 - (-l_1**2*(l_4*m_5 + m_4*r_4)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_4 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_4*m_5 + m_4*r_4)*np.cos(theta_4 - theta_2))**2/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2) - (-l_1**2*(l_4*m_5 + m_4*r_4)*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_4 - theta_1)*np.cos(theta_3 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_3*(l_4*m_5 + m_4*r_4)*np.cos(theta_4 - theta_3) - (-l_1**2*(l_4*m_5 + m_4*r_4)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_4 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_4*m_5 + m_4*r_4)*np.cos(theta_4 - theta_2))*(-l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_3 - theta_2))/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2))**2/(I_3 - l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)**2*np.cos(theta_3 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_3**2*m_4 + l_3**2*m_5 + m_3*r_3**2 - (-l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_3 - theta_2))**2/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2)))))/(I_3 - l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)**2*np.cos(theta_3 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_3**2*m_4 + l_3**2*m_5 + m_3*r_3**2 - (-l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_3 - theta_2))**2/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2)) - (-l_1**2*m_5*r_5*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_5 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*m_5*r_5*np.cos(theta_5 - theta_2))*(-dtheta_5*mu_5 - dtheta_4**2*l_4*m_5*r_5*np.sin(theta_5 - theta_4) + dtheta_4*mu_5 - dtheta_3**2*l_3*m_5*r_5*np.sin(theta_5 - theta_3) - dtheta_2**2*l_2*m_5*r_5*np.sin(theta_5 - theta_2) - dtheta_1**2*l_1*m_5*r_5*np.sin(theta_5 - theta_1) + ddx_c*m_5*r_5*np.cos(theta_5) + g*m_5*r_5*np.sin(theta_5) - l_1*m_5*r_5*(dtheta_5**2*l_1*m_5*r_5*np.sin(theta_5 - theta_1) + dtheta_4*(dtheta_4*l_1*l_4*m_5*np.sin(theta_4 - theta_1) + dtheta_4*l_1*m_4*r_4*np.sin(theta_4 - theta_1)) + dtheta_3*(dtheta_3*l_1*l_3*m_4*np.sin(theta_3 - theta_1) + dtheta_3*l_1*l_3*m_5*np.sin(theta_3 - theta_1) + dtheta_3*l_1*m_3*r_3*np.sin(theta_3 - theta_1)) + dtheta_2*mu_2 + dtheta_2*(dtheta_2*l_1*l_2*m_3*np.sin(theta_2 - theta_1) + dtheta_2*l_1*l_2*m_4*np.sin(theta_2 - theta_1) + dtheta_2*l_1*l_2*m_5*np.sin(theta_2 - theta_1) + dtheta_2*l_1*m_2*r_2*np.sin(theta_2 - theta_1)) - dtheta_1*(mu_1 + mu_2) - ddx_c*(-l_1*m_2 - l_1*m_3 - l_1*m_4 - l_1*m_5 - m_1*r_1)*np.cos(theta_1) - g*(-l_1*m_2 - l_1*m_3 - l_1*m_4 - l_1*m_5 - m_1*r_1)*np.sin(theta_1))*np.cos(theta_5 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) - (-l_1**2*m_5*r_5*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_5 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*m_5*r_5*np.cos(theta_5 - theta_2))*(dtheta_5**2*l_2*m_5*r_5*np.sin(theta_5 - theta_2) + dtheta_4*(dtheta_4*l_2*l_4*m_5*np.sin(theta_4 - theta_2) + dtheta_4*l_2*m_4*r_4*np.sin(theta_4 - theta_2)) + dtheta_3*mu_3 + dtheta_3*(dtheta_3*l_2*l_3*m_4*np.sin(theta_3 - theta_2) + dtheta_3*l_2*l_3*m_5*np.sin(theta_3 - theta_2) + dtheta_3*l_2*m_3*r_3*np.sin(theta_3 - theta_2)) - dtheta_2*(mu_2 + mu_3) + dtheta_1*mu_2 + dtheta_1*(-dtheta_1*l_1*l_2*m_3*np.sin(theta_2 - theta_1) - dtheta_1*l_1*l_2*m_4*np.sin(theta_2 - theta_1) - dtheta_1*l_1*l_2*m_5*np.sin(theta_2 - theta_1) - dtheta_1*l_1*m_2*r_2*np.sin(theta_2 - theta_1)) - ddx_c*(-l_2*m_3 - l_2*m_4 - l_2*m_5 - m_2*r_2)*np.cos(theta_2) - g*(-l_2*m_3 - l_2*m_4 - l_2*m_5 - m_2*r_2)*np.sin(theta_2) - l_1*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*(dtheta_5**2*l_1*m_5*r_5*np.sin(theta_5 - theta_1) + dtheta_4*(dtheta_4*l_1*l_4*m_5*np.sin(theta_4 - theta_1) + dtheta_4*l_1*m_4*r_4*np.sin(theta_4 - theta_1)) + dtheta_3*(dtheta_3*l_1*l_3*m_4*np.sin(theta_3 - theta_1) + dtheta_3*l_1*l_3*m_5*np.sin(theta_3 - theta_1) + dtheta_3*l_1*m_3*r_3*np.sin(theta_3 - theta_1)) + dtheta_2*mu_2 + dtheta_2*(dtheta_2*l_1*l_2*m_3*np.sin(theta_2 - theta_1) + dtheta_2*l_1*l_2*m_4*np.sin(theta_2 - theta_1) + dtheta_2*l_1*l_2*m_5*np.sin(theta_2 - theta_1) + dtheta_2*l_1*m_2*r_2*np.sin(theta_2 - theta_1)) - dtheta_1*(mu_1 + mu_2) - ddx_c*(-l_1*m_2 - l_1*m_3 - l_1*m_4 - l_1*m_5 - m_1*r_1)*np.cos(theta_1) - g*(-l_1*m_2 - l_1*m_3 - l_1*m_4 - l_1*m_5 - m_1*r_1)*np.sin(theta_1))*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2))/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2) - (-l_1**2*m_5*r_5*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_5 - theta_1)*np.cos(theta_3 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_3*m_5*r_5*np.cos(theta_5 - theta_3) - (-l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_3 - theta_2))*(-l_1**2*m_5*r_5*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_5 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*m_5*r_5*np.cos(theta_5 - theta_2))/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2))*(dtheta_5**2*l_3*m_5*r_5*np.sin(theta_5 - theta_3) + dtheta_4*mu_4 + dtheta_4*(dtheta_4*l_3*l_4*m_5*np.sin(theta_4 - theta_3) + dtheta_4*l_3*m_4*r_4*np.sin(theta_4 - theta_3)) - dtheta_3*(mu_3 + mu_4) + dtheta_2*mu_3 + dtheta_2*(-dtheta_2*l_2*l_3*m_4*np.sin(theta_3 - theta_2) - dtheta_2*l_2*l_3*m_5*np.sin(theta_3 - theta_2) - dtheta_2*l_2*m_3*r_3*np.sin(theta_3 - theta_2)) + dtheta_1*(-dtheta_1*l_1*l_3*m_4*np.sin(theta_3 - theta_1) - dtheta_1*l_1*l_3*m_5*np.sin(theta_3 - theta_1) - dtheta_1*l_1*m_3*r_3*np.sin(theta_3 - theta_1)) - ddx_c*(-l_3*m_4 - l_3*m_5 - m_3*r_3)*np.cos(theta_3) - g*(-l_3*m_4 - l_3*m_5 - m_3*r_3)*np.sin(theta_3) - l_1*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(dtheta_5**2*l_1*m_5*r_5*np.sin(theta_5 - theta_1) + dtheta_4*(dtheta_4*l_1*l_4*m_5*np.sin(theta_4 - theta_1) + dtheta_4*l_1*m_4*r_4*np.sin(theta_4 - theta_1)) + dtheta_3*(dtheta_3*l_1*l_3*m_4*np.sin(theta_3 - theta_1) + dtheta_3*l_1*l_3*m_5*np.sin(theta_3 - theta_1) + dtheta_3*l_1*m_3*r_3*np.sin(theta_3 - theta_1)) + dtheta_2*mu_2 + dtheta_2*(dtheta_2*l_1*l_2*m_3*np.sin(theta_2 - theta_1) + dtheta_2*l_1*l_2*m_4*np.sin(theta_2 - theta_1) + dtheta_2*l_1*l_2*m_5*np.sin(theta_2 - theta_1) + dtheta_2*l_1*m_2*r_2*np.sin(theta_2 - theta_1)) - dtheta_1*(mu_1 + mu_2) - ddx_c*(-l_1*m_2 - l_1*m_3 - l_1*m_4 - l_1*m_5 - m_1*r_1)*np.cos(theta_1) - g*(-l_1*m_2 - l_1*m_3 - l_1*m_4 - l_1*m_5 - m_1*r_1)*np.sin(theta_1))*np.cos(theta_3 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) - (-l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_3 - theta_2))*(dtheta_5**2*l_2*m_5*r_5*np.sin(theta_5 - theta_2) + dtheta_4*(dtheta_4*l_2*l_4*m_5*np.sin(theta_4 - theta_2) + dtheta_4*l_2*m_4*r_4*np.sin(theta_4 - theta_2)) + dtheta_3*mu_3 + dtheta_3*(dtheta_3*l_2*l_3*m_4*np.sin(theta_3 - theta_2) + dtheta_3*l_2*l_3*m_5*np.sin(theta_3 - theta_2) + dtheta_3*l_2*m_3*r_3*np.sin(theta_3 - theta_2)) - dtheta_2*(mu_2 + mu_3) + dtheta_1*mu_2 + dtheta_1*(-dtheta_1*l_1*l_2*m_3*np.sin(theta_2 - theta_1) - dtheta_1*l_1*l_2*m_4*np.sin(theta_2 - theta_1) - dtheta_1*l_1*l_2*m_5*np.sin(theta_2 - theta_1) - dtheta_1*l_1*m_2*r_2*np.sin(theta_2 - theta_1)) - ddx_c*(-l_2*m_3 - l_2*m_4 - l_2*m_5 - m_2*r_2)*np.cos(theta_2) - g*(-l_2*m_3 - l_2*m_4 - l_2*m_5 - m_2*r_2)*np.sin(theta_2) - l_1*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*(dtheta_5**2*l_1*m_5*r_5*np.sin(theta_5 - theta_1) + dtheta_4*(dtheta_4*l_1*l_4*m_5*np.sin(theta_4 - theta_1) + dtheta_4*l_1*m_4*r_4*np.sin(theta_4 - theta_1)) + dtheta_3*(dtheta_3*l_1*l_3*m_4*np.sin(theta_3 - theta_1) + dtheta_3*l_1*l_3*m_5*np.sin(theta_3 - theta_1) + dtheta_3*l_1*m_3*r_3*np.sin(theta_3 - theta_1)) + dtheta_2*mu_2 + dtheta_2*(dtheta_2*l_1*l_2*m_3*np.sin(theta_2 - theta_1) + dtheta_2*l_1*l_2*m_4*np.sin(theta_2 - theta_1) + dtheta_2*l_1*l_2*m_5*np.sin(theta_2 - theta_1) + dtheta_2*l_1*m_2*r_2*np.sin(theta_2 - theta_1)) - dtheta_1*(mu_1 + mu_2) - ddx_c*(-l_1*m_2 - l_1*m_3 - l_1*m_4 - l_1*m_5 - m_1*r_1)*np.cos(theta_1) - g*(-l_1*m_2 - l_1*m_3 - l_1*m_4 - l_1*m_5 - m_1*r_1)*np.sin(theta_1))*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2))/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2))/(I_3 - l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)**2*np.cos(theta_3 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_3**2*m_4 + l_3**2*m_5 + m_3*r_3**2 - (-l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_3 - theta_2))**2/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2)) - (-l_1**2*m_5*r_5*(l_4*m_5 + m_4*r_4)*np.cos(theta_5 - theta_1)*np.cos(theta_4 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_4*m_5*r_5*np.cos(theta_5 - theta_4) - (-l_1**2*(l_4*m_5 + m_4*r_4)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_4 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_4*m_5 + m_4*r_4)*np.cos(theta_4 - theta_2))*(-l_1**2*m_5*r_5*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_5 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*m_5*r_5*np.cos(theta_5 - theta_2))/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2) - (-l_1**2*(l_4*m_5 + m_4*r_4)*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_4 - theta_1)*np.cos(theta_3 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_3*(l_4*m_5 + m_4*r_4)*np.cos(theta_4 - theta_3) - (-l_1**2*(l_4*m_5 + m_4*r_4)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_4 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_4*m_5 + m_4*r_4)*np.cos(theta_4 - theta_2))*(-l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_3 - theta_2))/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2))*(-l_1**2*m_5*r_5*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_5 - theta_1)*np.cos(theta_3 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_3*m_5*r_5*np.cos(theta_5 - theta_3) - (-l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_3 - theta_2))*(-l_1**2*m_5*r_5*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_5 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*m_5*r_5*np.cos(theta_5 - theta_2))/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2))/(I_3 - l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)**2*np.cos(theta_3 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_3**2*m_4 + l_3**2*m_5 + m_3*r_3**2 - (-l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_3 - theta_2))**2/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2)))*(dtheta_5**2*l_4*m_5*r_5*np.sin(theta_5 - theta_4) + dtheta_5*mu_5 - dtheta_4*(mu_4 + mu_5) + dtheta_3*mu_4 + dtheta_3*(-dtheta_3*l_3*l_4*m_5*np.sin(theta_4 - theta_3) - dtheta_3*l_3*m_4*r_4*np.sin(theta_4 - theta_3)) + dtheta_2*(-dtheta_2*l_2*l_4*m_5*np.sin(theta_4 - theta_2) - dtheta_2*l_2*m_4*r_4*np.sin(theta_4 - theta_2)) + dtheta_1*(-dtheta_1*l_1*l_4*m_5*np.sin(theta_4 - theta_1) - dtheta_1*l_1*m_4*r_4*np.sin(theta_4 - theta_1)) - ddx_c*(-l_4*m_5 - m_4*r_4)*np.cos(theta_4) - g*(-l_4*m_5 - m_4*r_4)*np.sin(theta_4) - l_1*(l_4*m_5 + m_4*r_4)*(dtheta_5**2*l_1*m_5*r_5*np.sin(theta_5 - theta_1) + dtheta_4*(dtheta_4*l_1*l_4*m_5*np.sin(theta_4 - theta_1) + dtheta_4*l_1*m_4*r_4*np.sin(theta_4 - theta_1)) + dtheta_3*(dtheta_3*l_1*l_3*m_4*np.sin(theta_3 - theta_1) + dtheta_3*l_1*l_3*m_5*np.sin(theta_3 - theta_1) + dtheta_3*l_1*m_3*r_3*np.sin(theta_3 - theta_1)) + dtheta_2*mu_2 + dtheta_2*(dtheta_2*l_1*l_2*m_3*np.sin(theta_2 - theta_1) + dtheta_2*l_1*l_2*m_4*np.sin(theta_2 - theta_1) + dtheta_2*l_1*l_2*m_5*np.sin(theta_2 - theta_1) + dtheta_2*l_1*m_2*r_2*np.sin(theta_2 - theta_1)) - dtheta_1*(mu_1 + mu_2) - ddx_c*(-l_1*m_2 - l_1*m_3 - l_1*m_4 - l_1*m_5 - m_1*r_1)*np.cos(theta_1) - g*(-l_1*m_2 - l_1*m_3 - l_1*m_4 - l_1*m_5 - m_1*r_1)*np.sin(theta_1))*np.cos(theta_4 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) - (-l_1**2*(l_4*m_5 + m_4*r_4)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_4 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_4*m_5 + m_4*r_4)*np.cos(theta_4 - theta_2))*(dtheta_5**2*l_2*m_5*r_5*np.sin(theta_5 - theta_2) + dtheta_4*(dtheta_4*l_2*l_4*m_5*np.sin(theta_4 - theta_2) + dtheta_4*l_2*m_4*r_4*np.sin(theta_4 - theta_2)) + dtheta_3*mu_3 + dtheta_3*(dtheta_3*l_2*l_3*m_4*np.sin(theta_3 - theta_2) + dtheta_3*l_2*l_3*m_5*np.sin(theta_3 - theta_2) + dtheta_3*l_2*m_3*r_3*np.sin(theta_3 - theta_2)) - dtheta_2*(mu_2 + mu_3) + dtheta_1*mu_2 + dtheta_1*(-dtheta_1*l_1*l_2*m_3*np.sin(theta_2 - theta_1) - dtheta_1*l_1*l_2*m_4*np.sin(theta_2 - theta_1) - dtheta_1*l_1*l_2*m_5*np.sin(theta_2 - theta_1) - dtheta_1*l_1*m_2*r_2*np.sin(theta_2 - theta_1)) - ddx_c*(-l_2*m_3 - l_2*m_4 - l_2*m_5 - m_2*r_2)*np.cos(theta_2) - g*(-l_2*m_3 - l_2*m_4 - l_2*m_5 - m_2*r_2)*np.sin(theta_2) - l_1*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*(dtheta_5**2*l_1*m_5*r_5*np.sin(theta_5 - theta_1) + dtheta_4*(dtheta_4*l_1*l_4*m_5*np.sin(theta_4 - theta_1) + dtheta_4*l_1*m_4*r_4*np.sin(theta_4 - theta_1)) + dtheta_3*(dtheta_3*l_1*l_3*m_4*np.sin(theta_3 - theta_1) + dtheta_3*l_1*l_3*m_5*np.sin(theta_3 - theta_1) + dtheta_3*l_1*m_3*r_3*np.sin(theta_3 - theta_1)) + dtheta_2*mu_2 + dtheta_2*(dtheta_2*l_1*l_2*m_3*np.sin(theta_2 - theta_1) + dtheta_2*l_1*l_2*m_4*np.sin(theta_2 - theta_1) + dtheta_2*l_1*l_2*m_5*np.sin(theta_2 - theta_1) + dtheta_2*l_1*m_2*r_2*np.sin(theta_2 - theta_1)) - dtheta_1*(mu_1 + mu_2) - ddx_c*(-l_1*m_2 - l_1*m_3 - l_1*m_4 - l_1*m_5 - m_1*r_1)*np.cos(theta_1) - g*(-l_1*m_2 - l_1*m_3 - l_1*m_4 - l_1*m_5 - m_1*r_1)*np.sin(theta_1))*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2))/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2) - (-l_1**2*(l_4*m_5 + m_4*r_4)*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_4 - theta_1)*np.cos(theta_3 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_3*(l_4*m_5 + m_4*r_4)*np.cos(theta_4 - theta_3) - (-l_1**2*(l_4*m_5 + m_4*r_4)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_4 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_4*m_5 + m_4*r_4)*np.cos(theta_4 - theta_2))*(-l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_3 - theta_2))/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2))*(dtheta_5**2*l_3*m_5*r_5*np.sin(theta_5 - theta_3) + dtheta_4*mu_4 + dtheta_4*(dtheta_4*l_3*l_4*m_5*np.sin(theta_4 - theta_3) + dtheta_4*l_3*m_4*r_4*np.sin(theta_4 - theta_3)) - dtheta_3*(mu_3 + mu_4) + dtheta_2*mu_3 + dtheta_2*(-dtheta_2*l_2*l_3*m_4*np.sin(theta_3 - theta_2) - dtheta_2*l_2*l_3*m_5*np.sin(theta_3 - theta_2) - dtheta_2*l_2*m_3*r_3*np.sin(theta_3 - theta_2)) + dtheta_1*(-dtheta_1*l_1*l_3*m_4*np.sin(theta_3 - theta_1) - dtheta_1*l_1*l_3*m_5*np.sin(theta_3 - theta_1) - dtheta_1*l_1*m_3*r_3*np.sin(theta_3 - theta_1)) - ddx_c*(-l_3*m_4 - l_3*m_5 - m_3*r_3)*np.cos(theta_3) - g*(-l_3*m_4 - l_3*m_5 - m_3*r_3)*np.sin(theta_3) - l_1*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(dtheta_5**2*l_1*m_5*r_5*np.sin(theta_5 - theta_1) + dtheta_4*(dtheta_4*l_1*l_4*m_5*np.sin(theta_4 - theta_1) + dtheta_4*l_1*m_4*r_4*np.sin(theta_4 - theta_1)) + dtheta_3*(dtheta_3*l_1*l_3*m_4*np.sin(theta_3 - theta_1) + dtheta_3*l_1*l_3*m_5*np.sin(theta_3 - theta_1) + dtheta_3*l_1*m_3*r_3*np.sin(theta_3 - theta_1)) + dtheta_2*mu_2 + dtheta_2*(dtheta_2*l_1*l_2*m_3*np.sin(theta_2 - theta_1) + dtheta_2*l_1*l_2*m_4*np.sin(theta_2 - theta_1) + dtheta_2*l_1*l_2*m_5*np.sin(theta_2 - theta_1) + dtheta_2*l_1*m_2*r_2*np.sin(theta_2 - theta_1)) - dtheta_1*(mu_1 + mu_2) - ddx_c*(-l_1*m_2 - l_1*m_3 - l_1*m_4 - l_1*m_5 - m_1*r_1)*np.cos(theta_1) - g*(-l_1*m_2 - l_1*m_3 - l_1*m_4 - l_1*m_5 - m_1*r_1)*np.sin(theta_1))*np.cos(theta_3 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) - (-l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_3 - theta_2))*(dtheta_5**2*l_2*m_5*r_5*np.sin(theta_5 - theta_2) + dtheta_4*(dtheta_4*l_2*l_4*m_5*np.sin(theta_4 - theta_2) + dtheta_4*l_2*m_4*r_4*np.sin(theta_4 - theta_2)) + dtheta_3*mu_3 + dtheta_3*(dtheta_3*l_2*l_3*m_4*np.sin(theta_3 - theta_2) + dtheta_3*l_2*l_3*m_5*np.sin(theta_3 - theta_2) + dtheta_3*l_2*m_3*r_3*np.sin(theta_3 - theta_2)) - dtheta_2*(mu_2 + mu_3) + dtheta_1*mu_2 + dtheta_1*(-dtheta_1*l_1*l_2*m_3*np.sin(theta_2 - theta_1) - dtheta_1*l_1*l_2*m_4*np.sin(theta_2 - theta_1) - dtheta_1*l_1*l_2*m_5*np.sin(theta_2 - theta_1) - dtheta_1*l_1*m_2*r_2*np.sin(theta_2 - theta_1)) - ddx_c*(-l_2*m_3 - l_2*m_4 - l_2*m_5 - m_2*r_2)*np.cos(theta_2) - g*(-l_2*m_3 - l_2*m_4 - l_2*m_5 - m_2*r_2)*np.sin(theta_2) - l_1*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*(dtheta_5**2*l_1*m_5*r_5*np.sin(theta_5 - theta_1) + dtheta_4*(dtheta_4*l_1*l_4*m_5*np.sin(theta_4 - theta_1) + dtheta_4*l_1*m_4*r_4*np.sin(theta_4 - theta_1)) + dtheta_3*(dtheta_3*l_1*l_3*m_4*np.sin(theta_3 - theta_1) + dtheta_3*l_1*l_3*m_5*np.sin(theta_3 - theta_1) + dtheta_3*l_1*m_3*r_3*np.sin(theta_3 - theta_1)) + dtheta_2*mu_2 + dtheta_2*(dtheta_2*l_1*l_2*m_3*np.sin(theta_2 - theta_1) + dtheta_2*l_1*l_2*m_4*np.sin(theta_2 - theta_1) + dtheta_2*l_1*l_2*m_5*np.sin(theta_2 - theta_1) + dtheta_2*l_1*m_2*r_2*np.sin(theta_2 - theta_1)) - dtheta_1*(mu_1 + mu_2) - ddx_c*(-l_1*m_2 - l_1*m_3 - l_1*m_4 - l_1*m_5 - m_1*r_1)*np.cos(theta_1) - g*(-l_1*m_2 - l_1*m_3 - l_1*m_4 - l_1*m_5 - m_1*r_1)*np.sin(theta_1))*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2))/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2))/(I_3 - l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)**2*np.cos(theta_3 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_3**2*m_4 + l_3**2*m_5 + m_3*r_3**2 - (-l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_3 - theta_2))**2/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2)))/(I_4 - l_1**2*(l_4*m_5 + m_4*r_4)**2*np.cos(theta_4 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_4**2*m_5 + m_4*r_4**2 - (-l_1**2*(l_4*m_5 + m_4*r_4)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_4 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_4*m_5 + m_4*r_4)*np.cos(theta_4 - theta_2))**2/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2) - (-l_1**2*(l_4*m_5 + m_4*r_4)*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_4 - theta_1)*np.cos(theta_3 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_3*(l_4*m_5 + m_4*r_4)*np.cos(theta_4 - theta_3) - (-l_1**2*(l_4*m_5 + m_4*r_4)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_4 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_4*m_5 + m_4*r_4)*np.cos(theta_4 - theta_2))*(-l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_3 - theta_2))/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2))**2/(I_3 - l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)**2*np.cos(theta_3 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_3**2*m_4 + l_3**2*m_5 + m_3*r_3**2 - (-l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_3 - theta_2))**2/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2))))/(I_5 - l_1**2*m_5**2*r_5**2*np.cos(theta_5 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + m_5*r_5**2 - (-l_1**2*m_5*r_5*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_5 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*m_5*r_5*np.cos(theta_5 - theta_2))**2/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2) - (-l_1**2*m_5*r_5*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_5 - theta_1)*np.cos(theta_3 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_3*m_5*r_5*np.cos(theta_5 - theta_3) - (-l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_3 - theta_2))*(-l_1**2*m_5*r_5*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_5 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*m_5*r_5*np.cos(theta_5 - theta_2))/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2))**2/(I_3 - l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)**2*np.cos(theta_3 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_3**2*m_4 + l_3**2*m_5 + m_3*r_3**2 - (-l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_3 - theta_2))**2/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2)) - (-l_1**2*m_5*r_5*(l_4*m_5 + m_4*r_4)*np.cos(theta_5 - theta_1)*np.cos(theta_4 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_4*m_5*r_5*np.cos(theta_5 - theta_4) - (-l_1**2*(l_4*m_5 + m_4*r_4)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_4 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_4*m_5 + m_4*r_4)*np.cos(theta_4 - theta_2))*(-l_1**2*m_5*r_5*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_5 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*m_5*r_5*np.cos(theta_5 - theta_2))/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2) - (-l_1**2*(l_4*m_5 + m_4*r_4)*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_4 - theta_1)*np.cos(theta_3 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_3*(l_4*m_5 + m_4*r_4)*np.cos(theta_4 - theta_3) - (-l_1**2*(l_4*m_5 + m_4*r_4)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_4 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_4*m_5 + m_4*r_4)*np.cos(theta_4 - theta_2))*(-l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_3 - theta_2))/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2))*(-l_1**2*m_5*r_5*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_5 - theta_1)*np.cos(theta_3 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_3*m_5*r_5*np.cos(theta_5 - theta_3) - (-l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_3 - theta_2))*(-l_1**2*m_5*r_5*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_5 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*m_5*r_5*np.cos(theta_5 - theta_2))/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2))/(I_3 - l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)**2*np.cos(theta_3 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_3**2*m_4 + l_3**2*m_5 + m_3*r_3**2 - (-l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_3 - theta_2))**2/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2)))**2/(I_4 - l_1**2*(l_4*m_5 + m_4*r_4)**2*np.cos(theta_4 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_4**2*m_5 + m_4*r_4**2 - (-l_1**2*(l_4*m_5 + m_4*r_4)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_4 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_4*m_5 + m_4*r_4)*np.cos(theta_4 - theta_2))**2/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2) - (-l_1**2*(l_4*m_5 + m_4*r_4)*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_4 - theta_1)*np.cos(theta_3 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_3*(l_4*m_5 + m_4*r_4)*np.cos(theta_4 - theta_3) - (-l_1**2*(l_4*m_5 + m_4*r_4)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_4 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_4*m_5 + m_4*r_4)*np.cos(theta_4 - theta_2))*(-l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_3 - theta_2))/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2))**2/(I_3 - l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)**2*np.cos(theta_3 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_3**2*m_4 + l_3**2*m_5 + m_3*r_3**2 - (-l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_3 - theta_2))**2/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2)))))/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2),


                      (dtheta_5**2*l_3*m_5*r_5*np.sin(theta_5 - theta_3) + dtheta_4*mu_4 + dtheta_4*(dtheta_4*l_3*l_4*m_5*np.sin(theta_4 - theta_3) + dtheta_4*l_3*m_4*r_4*np.sin(theta_4 - theta_3)) - dtheta_3*(mu_3 + mu_4) + dtheta_2*mu_3 + dtheta_2*(-dtheta_2*l_2*l_3*m_4*np.sin(theta_3 - theta_2) - dtheta_2*l_2*l_3*m_5*np.sin(theta_3 - theta_2) - dtheta_2*l_2*m_3*r_3*np.sin(theta_3 - theta_2)) + dtheta_1*(-dtheta_1*l_1*l_3*m_4*np.sin(theta_3 - theta_1) - dtheta_1*l_1*l_3*m_5*np.sin(theta_3 - theta_1) - dtheta_1*l_1*m_3*r_3*np.sin(theta_3 - theta_1)) - ddx_c*(-l_3*m_4 - l_3*m_5 - m_3*r_3)*np.cos(theta_3) - g*(-l_3*m_4 - l_3*m_5 - m_3*r_3)*np.sin(theta_3) - l_1*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(dtheta_5**2*l_1*m_5*r_5*np.sin(theta_5 - theta_1) + dtheta_4*(dtheta_4*l_1*l_4*m_5*np.sin(theta_4 - theta_1) + dtheta_4*l_1*m_4*r_4*np.sin(theta_4 - theta_1)) + dtheta_3*(dtheta_3*l_1*l_3*m_4*np.sin(theta_3 - theta_1) + dtheta_3*l_1*l_3*m_5*np.sin(theta_3 - theta_1) + dtheta_3*l_1*m_3*r_3*np.sin(theta_3 - theta_1)) + dtheta_2*mu_2 + dtheta_2*(dtheta_2*l_1*l_2*m_3*np.sin(theta_2 - theta_1) + dtheta_2*l_1*l_2*m_4*np.sin(theta_2 - theta_1) + dtheta_2*l_1*l_2*m_5*np.sin(theta_2 - theta_1) + dtheta_2*l_1*m_2*r_2*np.sin(theta_2 - theta_1)) - dtheta_1*(mu_1 + mu_2) - ddx_c*(-l_1*m_2 - l_1*m_3 - l_1*m_4 - l_1*m_5 - m_1*r_1)*np.cos(theta_1) - g*(-l_1*m_2 - l_1*m_3 - l_1*m_4 - l_1*m_5 - m_1*r_1)*np.sin(theta_1))*np.cos(theta_3 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) - (-l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_3 - theta_2))*(dtheta_5**2*l_2*m_5*r_5*np.sin(theta_5 - theta_2) + dtheta_4*(dtheta_4*l_2*l_4*m_5*np.sin(theta_4 - theta_2) + dtheta_4*l_2*m_4*r_4*np.sin(theta_4 - theta_2)) + dtheta_3*mu_3 + dtheta_3*(dtheta_3*l_2*l_3*m_4*np.sin(theta_3 - theta_2) + dtheta_3*l_2*l_3*m_5*np.sin(theta_3 - theta_2) + dtheta_3*l_2*m_3*r_3*np.sin(theta_3 - theta_2)) - dtheta_2*(mu_2 + mu_3) + dtheta_1*mu_2 + dtheta_1*(-dtheta_1*l_1*l_2*m_3*np.sin(theta_2 - theta_1) - dtheta_1*l_1*l_2*m_4*np.sin(theta_2 - theta_1) - dtheta_1*l_1*l_2*m_5*np.sin(theta_2 - theta_1) - dtheta_1*l_1*m_2*r_2*np.sin(theta_2 - theta_1)) - ddx_c*(-l_2*m_3 - l_2*m_4 - l_2*m_5 - m_2*r_2)*np.cos(theta_2) - g*(-l_2*m_3 - l_2*m_4 - l_2*m_5 - m_2*r_2)*np.sin(theta_2) - l_1*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*(dtheta_5**2*l_1*m_5*r_5*np.sin(theta_5 - theta_1) + dtheta_4*(dtheta_4*l_1*l_4*m_5*np.sin(theta_4 - theta_1) + dtheta_4*l_1*m_4*r_4*np.sin(theta_4 - theta_1)) + dtheta_3*(dtheta_3*l_1*l_3*m_4*np.sin(theta_3 - theta_1) + dtheta_3*l_1*l_3*m_5*np.sin(theta_3 - theta_1) + dtheta_3*l_1*m_3*r_3*np.sin(theta_3 - theta_1)) + dtheta_2*mu_2 + dtheta_2*(dtheta_2*l_1*l_2*m_3*np.sin(theta_2 - theta_1) + dtheta_2*l_1*l_2*m_4*np.sin(theta_2 - theta_1) + dtheta_2*l_1*l_2*m_5*np.sin(theta_2 - theta_1) + dtheta_2*l_1*m_2*r_2*np.sin(theta_2 - theta_1)) - dtheta_1*(mu_1 + mu_2) - ddx_c*(-l_1*m_2 - l_1*m_3 - l_1*m_4 - l_1*m_5 - m_1*r_1)*np.cos(theta_1) - g*(-l_1*m_2 - l_1*m_3 - l_1*m_4 - l_1*m_5 - m_1*r_1)*np.sin(theta_1))*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2))/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2) - (-l_1**2*(l_4*m_5 + m_4*r_4)*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_4 - theta_1)*np.cos(theta_3 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_3*(l_4*m_5 + m_4*r_4)*np.cos(theta_4 - theta_3) - (-l_1**2*(l_4*m_5 + m_4*r_4)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_4 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_4*m_5 + m_4*r_4)*np.cos(theta_4 - theta_2))*(-l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_3 - theta_2))/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2))*(dtheta_5**2*l_4*m_5*r_5*np.sin(theta_5 - theta_4) + dtheta_5*mu_5 - dtheta_4*(mu_4 + mu_5) + dtheta_3*mu_4 + dtheta_3*(-dtheta_3*l_3*l_4*m_5*np.sin(theta_4 - theta_3) - dtheta_3*l_3*m_4*r_4*np.sin(theta_4 - theta_3)) + dtheta_2*(-dtheta_2*l_2*l_4*m_5*np.sin(theta_4 - theta_2) - dtheta_2*l_2*m_4*r_4*np.sin(theta_4 - theta_2)) + dtheta_1*(-dtheta_1*l_1*l_4*m_5*np.sin(theta_4 - theta_1) - dtheta_1*l_1*m_4*r_4*np.sin(theta_4 - theta_1)) - ddx_c*(-l_4*m_5 - m_4*r_4)*np.cos(theta_4) - g*(-l_4*m_5 - m_4*r_4)*np.sin(theta_4) - l_1*(l_4*m_5 + m_4*r_4)*(dtheta_5**2*l_1*m_5*r_5*np.sin(theta_5 - theta_1) + dtheta_4*(dtheta_4*l_1*l_4*m_5*np.sin(theta_4 - theta_1) + dtheta_4*l_1*m_4*r_4*np.sin(theta_4 - theta_1)) + dtheta_3*(dtheta_3*l_1*l_3*m_4*np.sin(theta_3 - theta_1) + dtheta_3*l_1*l_3*m_5*np.sin(theta_3 - theta_1) + dtheta_3*l_1*m_3*r_3*np.sin(theta_3 - theta_1)) + dtheta_2*mu_2 + dtheta_2*(dtheta_2*l_1*l_2*m_3*np.sin(theta_2 - theta_1) + dtheta_2*l_1*l_2*m_4*np.sin(theta_2 - theta_1) + dtheta_2*l_1*l_2*m_5*np.sin(theta_2 - theta_1) + dtheta_2*l_1*m_2*r_2*np.sin(theta_2 - theta_1)) - dtheta_1*(mu_1 + mu_2) - ddx_c*(-l_1*m_2 - l_1*m_3 - l_1*m_4 - l_1*m_5 - m_1*r_1)*np.cos(theta_1) - g*(-l_1*m_2 - l_1*m_3 - l_1*m_4 - l_1*m_5 - m_1*r_1)*np.sin(theta_1))*np.cos(theta_4 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) - (-l_1**2*(l_4*m_5 + m_4*r_4)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_4 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_4*m_5 + m_4*r_4)*np.cos(theta_4 - theta_2))*(dtheta_5**2*l_2*m_5*r_5*np.sin(theta_5 - theta_2) + dtheta_4*(dtheta_4*l_2*l_4*m_5*np.sin(theta_4 - theta_2) + dtheta_4*l_2*m_4*r_4*np.sin(theta_4 - theta_2)) + dtheta_3*mu_3 + dtheta_3*(dtheta_3*l_2*l_3*m_4*np.sin(theta_3 - theta_2) + dtheta_3*l_2*l_3*m_5*np.sin(theta_3 - theta_2) + dtheta_3*l_2*m_3*r_3*np.sin(theta_3 - theta_2)) - dtheta_2*(mu_2 + mu_3) + dtheta_1*mu_2 + dtheta_1*(-dtheta_1*l_1*l_2*m_3*np.sin(theta_2 - theta_1) - dtheta_1*l_1*l_2*m_4*np.sin(theta_2 - theta_1) - dtheta_1*l_1*l_2*m_5*np.sin(theta_2 - theta_1) - dtheta_1*l_1*m_2*r_2*np.sin(theta_2 - theta_1)) - ddx_c*(-l_2*m_3 - l_2*m_4 - l_2*m_5 - m_2*r_2)*np.cos(theta_2) - g*(-l_2*m_3 - l_2*m_4 - l_2*m_5 - m_2*r_2)*np.sin(theta_2) - l_1*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*(dtheta_5**2*l_1*m_5*r_5*np.sin(theta_5 - theta_1) + dtheta_4*(dtheta_4*l_1*l_4*m_5*np.sin(theta_4 - theta_1) + dtheta_4*l_1*m_4*r_4*np.sin(theta_4 - theta_1)) + dtheta_3*(dtheta_3*l_1*l_3*m_4*np.sin(theta_3 - theta_1) + dtheta_3*l_1*l_3*m_5*np.sin(theta_3 - theta_1) + dtheta_3*l_1*m_3*r_3*np.sin(theta_3 - theta_1)) + dtheta_2*mu_2 + dtheta_2*(dtheta_2*l_1*l_2*m_3*np.sin(theta_2 - theta_1) + dtheta_2*l_1*l_2*m_4*np.sin(theta_2 - theta_1) + dtheta_2*l_1*l_2*m_5*np.sin(theta_2 - theta_1) + dtheta_2*l_1*m_2*r_2*np.sin(theta_2 - theta_1)) - dtheta_1*(mu_1 + mu_2) - ddx_c*(-l_1*m_2 - l_1*m_3 - l_1*m_4 - l_1*m_5 - m_1*r_1)*np.cos(theta_1) - g*(-l_1*m_2 - l_1*m_3 - l_1*m_4 - l_1*m_5 - m_1*r_1)*np.sin(theta_1))*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2))/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2) - (-l_1**2*(l_4*m_5 + m_4*r_4)*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_4 - theta_1)*np.cos(theta_3 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_3*(l_4*m_5 + m_4*r_4)*np.cos(theta_4 - theta_3) - (-l_1**2*(l_4*m_5 + m_4*r_4)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_4 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_4*m_5 + m_4*r_4)*np.cos(theta_4 - theta_2))*(-l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_3 - theta_2))/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2))*(dtheta_5**2*l_3*m_5*r_5*np.sin(theta_5 - theta_3) + dtheta_4*mu_4 + dtheta_4*(dtheta_4*l_3*l_4*m_5*np.sin(theta_4 - theta_3) + dtheta_4*l_3*m_4*r_4*np.sin(theta_4 - theta_3)) - dtheta_3*(mu_3 + mu_4) + dtheta_2*mu_3 + dtheta_2*(-dtheta_2*l_2*l_3*m_4*np.sin(theta_3 - theta_2) - dtheta_2*l_2*l_3*m_5*np.sin(theta_3 - theta_2) - dtheta_2*l_2*m_3*r_3*np.sin(theta_3 - theta_2)) + dtheta_1*(-dtheta_1*l_1*l_3*m_4*np.sin(theta_3 - theta_1) - dtheta_1*l_1*l_3*m_5*np.sin(theta_3 - theta_1) - dtheta_1*l_1*m_3*r_3*np.sin(theta_3 - theta_1)) - ddx_c*(-l_3*m_4 - l_3*m_5 - m_3*r_3)*np.cos(theta_3) - g*(-l_3*m_4 - l_3*m_5 - m_3*r_3)*np.sin(theta_3) - l_1*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(dtheta_5**2*l_1*m_5*r_5*np.sin(theta_5 - theta_1) + dtheta_4*(dtheta_4*l_1*l_4*m_5*np.sin(theta_4 - theta_1) + dtheta_4*l_1*m_4*r_4*np.sin(theta_4 - theta_1)) + dtheta_3*(dtheta_3*l_1*l_3*m_4*np.sin(theta_3 - theta_1) + dtheta_3*l_1*l_3*m_5*np.sin(theta_3 - theta_1) + dtheta_3*l_1*m_3*r_3*np.sin(theta_3 - theta_1)) + dtheta_2*mu_2 + dtheta_2*(dtheta_2*l_1*l_2*m_3*np.sin(theta_2 - theta_1) + dtheta_2*l_1*l_2*m_4*np.sin(theta_2 - theta_1) + dtheta_2*l_1*l_2*m_5*np.sin(theta_2 - theta_1) + dtheta_2*l_1*m_2*r_2*np.sin(theta_2 - theta_1)) - dtheta_1*(mu_1 + mu_2) - ddx_c*(-l_1*m_2 - l_1*m_3 - l_1*m_4 - l_1*m_5 - m_1*r_1)*np.cos(theta_1) - g*(-l_1*m_2 - l_1*m_3 - l_1*m_4 - l_1*m_5 - m_1*r_1)*np.sin(theta_1))*np.cos(theta_3 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) - (-l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_3 - theta_2))*(dtheta_5**2*l_2*m_5*r_5*np.sin(theta_5 - theta_2) + dtheta_4*(dtheta_4*l_2*l_4*m_5*np.sin(theta_4 - theta_2) + dtheta_4*l_2*m_4*r_4*np.sin(theta_4 - theta_2)) + dtheta_3*mu_3 + dtheta_3*(dtheta_3*l_2*l_3*m_4*np.sin(theta_3 - theta_2) + dtheta_3*l_2*l_3*m_5*np.sin(theta_3 - theta_2) + dtheta_3*l_2*m_3*r_3*np.sin(theta_3 - theta_2)) - dtheta_2*(mu_2 + mu_3) + dtheta_1*mu_2 + dtheta_1*(-dtheta_1*l_1*l_2*m_3*np.sin(theta_2 - theta_1) - dtheta_1*l_1*l_2*m_4*np.sin(theta_2 - theta_1) - dtheta_1*l_1*l_2*m_5*np.sin(theta_2 - theta_1) - dtheta_1*l_1*m_2*r_2*np.sin(theta_2 - theta_1)) - ddx_c*(-l_2*m_3 - l_2*m_4 - l_2*m_5 - m_2*r_2)*np.cos(theta_2) - g*(-l_2*m_3 - l_2*m_4 - l_2*m_5 - m_2*r_2)*np.sin(theta_2) - l_1*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*(dtheta_5**2*l_1*m_5*r_5*np.sin(theta_5 - theta_1) + dtheta_4*(dtheta_4*l_1*l_4*m_5*np.sin(theta_4 - theta_1) + dtheta_4*l_1*m_4*r_4*np.sin(theta_4 - theta_1)) + dtheta_3*(dtheta_3*l_1*l_3*m_4*np.sin(theta_3 - theta_1) + dtheta_3*l_1*l_3*m_5*np.sin(theta_3 - theta_1) + dtheta_3*l_1*m_3*r_3*np.sin(theta_3 - theta_1)) + dtheta_2*mu_2 + dtheta_2*(dtheta_2*l_1*l_2*m_3*np.sin(theta_2 - theta_1) + dtheta_2*l_1*l_2*m_4*np.sin(theta_2 - theta_1) + dtheta_2*l_1*l_2*m_5*np.sin(theta_2 - theta_1) + dtheta_2*l_1*m_2*r_2*np.sin(theta_2 - theta_1)) - dtheta_1*(mu_1 + mu_2) - ddx_c*(-l_1*m_2 - l_1*m_3 - l_1*m_4 - l_1*m_5 - m_1*r_1)*np.cos(theta_1) - g*(-l_1*m_2 - l_1*m_3 - l_1*m_4 - l_1*m_5 - m_1*r_1)*np.sin(theta_1))*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2))/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2))/(I_3 - l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)**2*np.cos(theta_3 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_3**2*m_4 + l_3**2*m_5 + m_3*r_3**2 - (-l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_3 - theta_2))**2/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2)) - (-l_1**2*m_5*r_5*(l_4*m_5 + m_4*r_4)*np.cos(theta_5 - theta_1)*np.cos(theta_4 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_4*m_5*r_5*np.cos(theta_5 - theta_4) - (-l_1**2*(l_4*m_5 + m_4*r_4)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_4 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_4*m_5 + m_4*r_4)*np.cos(theta_4 - theta_2))*(-l_1**2*m_5*r_5*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_5 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*m_5*r_5*np.cos(theta_5 - theta_2))/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2) - (-l_1**2*(l_4*m_5 + m_4*r_4)*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_4 - theta_1)*np.cos(theta_3 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_3*(l_4*m_5 + m_4*r_4)*np.cos(theta_4 - theta_3) - (-l_1**2*(l_4*m_5 + m_4*r_4)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_4 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_4*m_5 + m_4*r_4)*np.cos(theta_4 - theta_2))*(-l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_3 - theta_2))/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2))*(-l_1**2*m_5*r_5*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_5 - theta_1)*np.cos(theta_3 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_3*m_5*r_5*np.cos(theta_5 - theta_3) - (-l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_3 - theta_2))*(-l_1**2*m_5*r_5*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_5 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*m_5*r_5*np.cos(theta_5 - theta_2))/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2))/(I_3 - l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)**2*np.cos(theta_3 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_3**2*m_4 + l_3**2*m_5 + m_3*r_3**2 - (-l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_3 - theta_2))**2/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2)))*(-dtheta_5*mu_5 - dtheta_4**2*l_4*m_5*r_5*np.sin(theta_5 - theta_4) + dtheta_4*mu_5 - dtheta_3**2*l_3*m_5*r_5*np.sin(theta_5 - theta_3) - dtheta_2**2*l_2*m_5*r_5*np.sin(theta_5 - theta_2) - dtheta_1**2*l_1*m_5*r_5*np.sin(theta_5 - theta_1) + ddx_c*m_5*r_5*np.cos(theta_5) + g*m_5*r_5*np.sin(theta_5) - l_1*m_5*r_5*(dtheta_5**2*l_1*m_5*r_5*np.sin(theta_5 - theta_1) + dtheta_4*(dtheta_4*l_1*l_4*m_5*np.sin(theta_4 - theta_1) + dtheta_4*l_1*m_4*r_4*np.sin(theta_4 - theta_1)) + dtheta_3*(dtheta_3*l_1*l_3*m_4*np.sin(theta_3 - theta_1) + dtheta_3*l_1*l_3*m_5*np.sin(theta_3 - theta_1) + dtheta_3*l_1*m_3*r_3*np.sin(theta_3 - theta_1)) + dtheta_2*mu_2 + dtheta_2*(dtheta_2*l_1*l_2*m_3*np.sin(theta_2 - theta_1) + dtheta_2*l_1*l_2*m_4*np.sin(theta_2 - theta_1) + dtheta_2*l_1*l_2*m_5*np.sin(theta_2 - theta_1) + dtheta_2*l_1*m_2*r_2*np.sin(theta_2 - theta_1)) - dtheta_1*(mu_1 + mu_2) - ddx_c*(-l_1*m_2 - l_1*m_3 - l_1*m_4 - l_1*m_5 - m_1*r_1)*np.cos(theta_1) - g*(-l_1*m_2 - l_1*m_3 - l_1*m_4 - l_1*m_5 - m_1*r_1)*np.sin(theta_1))*np.cos(theta_5 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) - (-l_1**2*m_5*r_5*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_5 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*m_5*r_5*np.cos(theta_5 - theta_2))*(dtheta_5**2*l_2*m_5*r_5*np.sin(theta_5 - theta_2) + dtheta_4*(dtheta_4*l_2*l_4*m_5*np.sin(theta_4 - theta_2) + dtheta_4*l_2*m_4*r_4*np.sin(theta_4 - theta_2)) + dtheta_3*mu_3 + dtheta_3*(dtheta_3*l_2*l_3*m_4*np.sin(theta_3 - theta_2) + dtheta_3*l_2*l_3*m_5*np.sin(theta_3 - theta_2) + dtheta_3*l_2*m_3*r_3*np.sin(theta_3 - theta_2)) - dtheta_2*(mu_2 + mu_3) + dtheta_1*mu_2 + dtheta_1*(-dtheta_1*l_1*l_2*m_3*np.sin(theta_2 - theta_1) - dtheta_1*l_1*l_2*m_4*np.sin(theta_2 - theta_1) - dtheta_1*l_1*l_2*m_5*np.sin(theta_2 - theta_1) - dtheta_1*l_1*m_2*r_2*np.sin(theta_2 - theta_1)) - ddx_c*(-l_2*m_3 - l_2*m_4 - l_2*m_5 - m_2*r_2)*np.cos(theta_2) - g*(-l_2*m_3 - l_2*m_4 - l_2*m_5 - m_2*r_2)*np.sin(theta_2) - l_1*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*(dtheta_5**2*l_1*m_5*r_5*np.sin(theta_5 - theta_1) + dtheta_4*(dtheta_4*l_1*l_4*m_5*np.sin(theta_4 - theta_1) + dtheta_4*l_1*m_4*r_4*np.sin(theta_4 - theta_1)) + dtheta_3*(dtheta_3*l_1*l_3*m_4*np.sin(theta_3 - theta_1) + dtheta_3*l_1*l_3*m_5*np.sin(theta_3 - theta_1) + dtheta_3*l_1*m_3*r_3*np.sin(theta_3 - theta_1)) + dtheta_2*mu_2 + dtheta_2*(dtheta_2*l_1*l_2*m_3*np.sin(theta_2 - theta_1) + dtheta_2*l_1*l_2*m_4*np.sin(theta_2 - theta_1) + dtheta_2*l_1*l_2*m_5*np.sin(theta_2 - theta_1) + dtheta_2*l_1*m_2*r_2*np.sin(theta_2 - theta_1)) - dtheta_1*(mu_1 + mu_2) - ddx_c*(-l_1*m_2 - l_1*m_3 - l_1*m_4 - l_1*m_5 - m_1*r_1)*np.cos(theta_1) - g*(-l_1*m_2 - l_1*m_3 - l_1*m_4 - l_1*m_5 - m_1*r_1)*np.sin(theta_1))*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2))/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2) - (-l_1**2*m_5*r_5*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_5 - theta_1)*np.cos(theta_3 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_3*m_5*r_5*np.cos(theta_5 - theta_3) - (-l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_3 - theta_2))*(-l_1**2*m_5*r_5*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_5 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*m_5*r_5*np.cos(theta_5 - theta_2))/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2))*(dtheta_5**2*l_3*m_5*r_5*np.sin(theta_5 - theta_3) + dtheta_4*mu_4 + dtheta_4*(dtheta_4*l_3*l_4*m_5*np.sin(theta_4 - theta_3) + dtheta_4*l_3*m_4*r_4*np.sin(theta_4 - theta_3)) - dtheta_3*(mu_3 + mu_4) + dtheta_2*mu_3 + dtheta_2*(-dtheta_2*l_2*l_3*m_4*np.sin(theta_3 - theta_2) - dtheta_2*l_2*l_3*m_5*np.sin(theta_3 - theta_2) - dtheta_2*l_2*m_3*r_3*np.sin(theta_3 - theta_2)) + dtheta_1*(-dtheta_1*l_1*l_3*m_4*np.sin(theta_3 - theta_1) - dtheta_1*l_1*l_3*m_5*np.sin(theta_3 - theta_1) - dtheta_1*l_1*m_3*r_3*np.sin(theta_3 - theta_1)) - ddx_c*(-l_3*m_4 - l_3*m_5 - m_3*r_3)*np.cos(theta_3) - g*(-l_3*m_4 - l_3*m_5 - m_3*r_3)*np.sin(theta_3) - l_1*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(dtheta_5**2*l_1*m_5*r_5*np.sin(theta_5 - theta_1) + dtheta_4*(dtheta_4*l_1*l_4*m_5*np.sin(theta_4 - theta_1) + dtheta_4*l_1*m_4*r_4*np.sin(theta_4 - theta_1)) + dtheta_3*(dtheta_3*l_1*l_3*m_4*np.sin(theta_3 - theta_1) + dtheta_3*l_1*l_3*m_5*np.sin(theta_3 - theta_1) + dtheta_3*l_1*m_3*r_3*np.sin(theta_3 - theta_1)) + dtheta_2*mu_2 + dtheta_2*(dtheta_2*l_1*l_2*m_3*np.sin(theta_2 - theta_1) + dtheta_2*l_1*l_2*m_4*np.sin(theta_2 - theta_1) + dtheta_2*l_1*l_2*m_5*np.sin(theta_2 - theta_1) + dtheta_2*l_1*m_2*r_2*np.sin(theta_2 - theta_1)) - dtheta_1*(mu_1 + mu_2) - ddx_c*(-l_1*m_2 - l_1*m_3 - l_1*m_4 - l_1*m_5 - m_1*r_1)*np.cos(theta_1) - g*(-l_1*m_2 - l_1*m_3 - l_1*m_4 - l_1*m_5 - m_1*r_1)*np.sin(theta_1))*np.cos(theta_3 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) - (-l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_3 - theta_2))*(dtheta_5**2*l_2*m_5*r_5*np.sin(theta_5 - theta_2) + dtheta_4*(dtheta_4*l_2*l_4*m_5*np.sin(theta_4 - theta_2) + dtheta_4*l_2*m_4*r_4*np.sin(theta_4 - theta_2)) + dtheta_3*mu_3 + dtheta_3*(dtheta_3*l_2*l_3*m_4*np.sin(theta_3 - theta_2) + dtheta_3*l_2*l_3*m_5*np.sin(theta_3 - theta_2) + dtheta_3*l_2*m_3*r_3*np.sin(theta_3 - theta_2)) - dtheta_2*(mu_2 + mu_3) + dtheta_1*mu_2 + dtheta_1*(-dtheta_1*l_1*l_2*m_3*np.sin(theta_2 - theta_1) - dtheta_1*l_1*l_2*m_4*np.sin(theta_2 - theta_1) - dtheta_1*l_1*l_2*m_5*np.sin(theta_2 - theta_1) - dtheta_1*l_1*m_2*r_2*np.sin(theta_2 - theta_1)) - ddx_c*(-l_2*m_3 - l_2*m_4 - l_2*m_5 - m_2*r_2)*np.cos(theta_2) - g*(-l_2*m_3 - l_2*m_4 - l_2*m_5 - m_2*r_2)*np.sin(theta_2) - l_1*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*(dtheta_5**2*l_1*m_5*r_5*np.sin(theta_5 - theta_1) + dtheta_4*(dtheta_4*l_1*l_4*m_5*np.sin(theta_4 - theta_1) + dtheta_4*l_1*m_4*r_4*np.sin(theta_4 - theta_1)) + dtheta_3*(dtheta_3*l_1*l_3*m_4*np.sin(theta_3 - theta_1) + dtheta_3*l_1*l_3*m_5*np.sin(theta_3 - theta_1) + dtheta_3*l_1*m_3*r_3*np.sin(theta_3 - theta_1)) + dtheta_2*mu_2 + dtheta_2*(dtheta_2*l_1*l_2*m_3*np.sin(theta_2 - theta_1) + dtheta_2*l_1*l_2*m_4*np.sin(theta_2 - theta_1) + dtheta_2*l_1*l_2*m_5*np.sin(theta_2 - theta_1) + dtheta_2*l_1*m_2*r_2*np.sin(theta_2 - theta_1)) - dtheta_1*(mu_1 + mu_2) - ddx_c*(-l_1*m_2 - l_1*m_3 - l_1*m_4 - l_1*m_5 - m_1*r_1)*np.cos(theta_1) - g*(-l_1*m_2 - l_1*m_3 - l_1*m_4 - l_1*m_5 - m_1*r_1)*np.sin(theta_1))*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2))/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2))/(I_3 - l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)**2*np.cos(theta_3 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_3**2*m_4 + l_3**2*m_5 + m_3*r_3**2 - (-l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_3 - theta_2))**2/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2)) - (-l_1**2*m_5*r_5*(l_4*m_5 + m_4*r_4)*np.cos(theta_5 - theta_1)*np.cos(theta_4 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_4*m_5*r_5*np.cos(theta_5 - theta_4) - (-l_1**2*(l_4*m_5 + m_4*r_4)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_4 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_4*m_5 + m_4*r_4)*np.cos(theta_4 - theta_2))*(-l_1**2*m_5*r_5*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_5 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*m_5*r_5*np.cos(theta_5 - theta_2))/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2) - (-l_1**2*(l_4*m_5 + m_4*r_4)*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_4 - theta_1)*np.cos(theta_3 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_3*(l_4*m_5 + m_4*r_4)*np.cos(theta_4 - theta_3) - (-l_1**2*(l_4*m_5 + m_4*r_4)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_4 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_4*m_5 + m_4*r_4)*np.cos(theta_4 - theta_2))*(-l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_3 - theta_2))/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2))*(-l_1**2*m_5*r_5*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_5 - theta_1)*np.cos(theta_3 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_3*m_5*r_5*np.cos(theta_5 - theta_3) - (-l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_3 - theta_2))*(-l_1**2*m_5*r_5*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_5 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*m_5*r_5*np.cos(theta_5 - theta_2))/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2))/(I_3 - l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)**2*np.cos(theta_3 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_3**2*m_4 + l_3**2*m_5 + m_3*r_3**2 - (-l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_3 - theta_2))**2/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2)))*(dtheta_5**2*l_4*m_5*r_5*np.sin(theta_5 - theta_4) + dtheta_5*mu_5 - dtheta_4*(mu_4 + mu_5) + dtheta_3*mu_4 + dtheta_3*(-dtheta_3*l_3*l_4*m_5*np.sin(theta_4 - theta_3) - dtheta_3*l_3*m_4*r_4*np.sin(theta_4 - theta_3)) + dtheta_2*(-dtheta_2*l_2*l_4*m_5*np.sin(theta_4 - theta_2) - dtheta_2*l_2*m_4*r_4*np.sin(theta_4 - theta_2)) + dtheta_1*(-dtheta_1*l_1*l_4*m_5*np.sin(theta_4 - theta_1) - dtheta_1*l_1*m_4*r_4*np.sin(theta_4 - theta_1)) - ddx_c*(-l_4*m_5 - m_4*r_4)*np.cos(theta_4) - g*(-l_4*m_5 - m_4*r_4)*np.sin(theta_4) - l_1*(l_4*m_5 + m_4*r_4)*(dtheta_5**2*l_1*m_5*r_5*np.sin(theta_5 - theta_1) + dtheta_4*(dtheta_4*l_1*l_4*m_5*np.sin(theta_4 - theta_1) + dtheta_4*l_1*m_4*r_4*np.sin(theta_4 - theta_1)) + dtheta_3*(dtheta_3*l_1*l_3*m_4*np.sin(theta_3 - theta_1) + dtheta_3*l_1*l_3*m_5*np.sin(theta_3 - theta_1) + dtheta_3*l_1*m_3*r_3*np.sin(theta_3 - theta_1)) + dtheta_2*mu_2 + dtheta_2*(dtheta_2*l_1*l_2*m_3*np.sin(theta_2 - theta_1) + dtheta_2*l_1*l_2*m_4*np.sin(theta_2 - theta_1) + dtheta_2*l_1*l_2*m_5*np.sin(theta_2 - theta_1) + dtheta_2*l_1*m_2*r_2*np.sin(theta_2 - theta_1)) - dtheta_1*(mu_1 + mu_2) - ddx_c*(-l_1*m_2 - l_1*m_3 - l_1*m_4 - l_1*m_5 - m_1*r_1)*np.cos(theta_1) - g*(-l_1*m_2 - l_1*m_3 - l_1*m_4 - l_1*m_5 - m_1*r_1)*np.sin(theta_1))*np.cos(theta_4 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) - (-l_1**2*(l_4*m_5 + m_4*r_4)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_4 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_4*m_5 + m_4*r_4)*np.cos(theta_4 - theta_2))*(dtheta_5**2*l_2*m_5*r_5*np.sin(theta_5 - theta_2) + dtheta_4*(dtheta_4*l_2*l_4*m_5*np.sin(theta_4 - theta_2) + dtheta_4*l_2*m_4*r_4*np.sin(theta_4 - theta_2)) + dtheta_3*mu_3 + dtheta_3*(dtheta_3*l_2*l_3*m_4*np.sin(theta_3 - theta_2) + dtheta_3*l_2*l_3*m_5*np.sin(theta_3 - theta_2) + dtheta_3*l_2*m_3*r_3*np.sin(theta_3 - theta_2)) - dtheta_2*(mu_2 + mu_3) + dtheta_1*mu_2 + dtheta_1*(-dtheta_1*l_1*l_2*m_3*np.sin(theta_2 - theta_1) - dtheta_1*l_1*l_2*m_4*np.sin(theta_2 - theta_1) - dtheta_1*l_1*l_2*m_5*np.sin(theta_2 - theta_1) - dtheta_1*l_1*m_2*r_2*np.sin(theta_2 - theta_1)) - ddx_c*(-l_2*m_3 - l_2*m_4 - l_2*m_5 - m_2*r_2)*np.cos(theta_2) - g*(-l_2*m_3 - l_2*m_4 - l_2*m_5 - m_2*r_2)*np.sin(theta_2) - l_1*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*(dtheta_5**2*l_1*m_5*r_5*np.sin(theta_5 - theta_1) + dtheta_4*(dtheta_4*l_1*l_4*m_5*np.sin(theta_4 - theta_1) + dtheta_4*l_1*m_4*r_4*np.sin(theta_4 - theta_1)) + dtheta_3*(dtheta_3*l_1*l_3*m_4*np.sin(theta_3 - theta_1) + dtheta_3*l_1*l_3*m_5*np.sin(theta_3 - theta_1) + dtheta_3*l_1*m_3*r_3*np.sin(theta_3 - theta_1)) + dtheta_2*mu_2 + dtheta_2*(dtheta_2*l_1*l_2*m_3*np.sin(theta_2 - theta_1) + dtheta_2*l_1*l_2*m_4*np.sin(theta_2 - theta_1) + dtheta_2*l_1*l_2*m_5*np.sin(theta_2 - theta_1) + dtheta_2*l_1*m_2*r_2*np.sin(theta_2 - theta_1)) - dtheta_1*(mu_1 + mu_2) - ddx_c*(-l_1*m_2 - l_1*m_3 - l_1*m_4 - l_1*m_5 - m_1*r_1)*np.cos(theta_1) - g*(-l_1*m_2 - l_1*m_3 - l_1*m_4 - l_1*m_5 - m_1*r_1)*np.sin(theta_1))*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2))/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2) - (-l_1**2*(l_4*m_5 + m_4*r_4)*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_4 - theta_1)*np.cos(theta_3 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_3*(l_4*m_5 + m_4*r_4)*np.cos(theta_4 - theta_3) - (-l_1**2*(l_4*m_5 + m_4*r_4)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_4 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_4*m_5 + m_4*r_4)*np.cos(theta_4 - theta_2))*(-l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_3 - theta_2))/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2))*(dtheta_5**2*l_3*m_5*r_5*np.sin(theta_5 - theta_3) + dtheta_4*mu_4 + dtheta_4*(dtheta_4*l_3*l_4*m_5*np.sin(theta_4 - theta_3) + dtheta_4*l_3*m_4*r_4*np.sin(theta_4 - theta_3)) - dtheta_3*(mu_3 + mu_4) + dtheta_2*mu_3 + dtheta_2*(-dtheta_2*l_2*l_3*m_4*np.sin(theta_3 - theta_2) - dtheta_2*l_2*l_3*m_5*np.sin(theta_3 - theta_2) - dtheta_2*l_2*m_3*r_3*np.sin(theta_3 - theta_2)) + dtheta_1*(-dtheta_1*l_1*l_3*m_4*np.sin(theta_3 - theta_1) - dtheta_1*l_1*l_3*m_5*np.sin(theta_3 - theta_1) - dtheta_1*l_1*m_3*r_3*np.sin(theta_3 - theta_1)) - ddx_c*(-l_3*m_4 - l_3*m_5 - m_3*r_3)*np.cos(theta_3) - g*(-l_3*m_4 - l_3*m_5 - m_3*r_3)*np.sin(theta_3) - l_1*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(dtheta_5**2*l_1*m_5*r_5*np.sin(theta_5 - theta_1) + dtheta_4*(dtheta_4*l_1*l_4*m_5*np.sin(theta_4 - theta_1) + dtheta_4*l_1*m_4*r_4*np.sin(theta_4 - theta_1)) + dtheta_3*(dtheta_3*l_1*l_3*m_4*np.sin(theta_3 - theta_1) + dtheta_3*l_1*l_3*m_5*np.sin(theta_3 - theta_1) + dtheta_3*l_1*m_3*r_3*np.sin(theta_3 - theta_1)) + dtheta_2*mu_2 + dtheta_2*(dtheta_2*l_1*l_2*m_3*np.sin(theta_2 - theta_1) + dtheta_2*l_1*l_2*m_4*np.sin(theta_2 - theta_1) + dtheta_2*l_1*l_2*m_5*np.sin(theta_2 - theta_1) + dtheta_2*l_1*m_2*r_2*np.sin(theta_2 - theta_1)) - dtheta_1*(mu_1 + mu_2) - ddx_c*(-l_1*m_2 - l_1*m_3 - l_1*m_4 - l_1*m_5 - m_1*r_1)*np.cos(theta_1) - g*(-l_1*m_2 - l_1*m_3 - l_1*m_4 - l_1*m_5 - m_1*r_1)*np.sin(theta_1))*np.cos(theta_3 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) - (-l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_3 - theta_2))*(dtheta_5**2*l_2*m_5*r_5*np.sin(theta_5 - theta_2) + dtheta_4*(dtheta_4*l_2*l_4*m_5*np.sin(theta_4 - theta_2) + dtheta_4*l_2*m_4*r_4*np.sin(theta_4 - theta_2)) + dtheta_3*mu_3 + dtheta_3*(dtheta_3*l_2*l_3*m_4*np.sin(theta_3 - theta_2) + dtheta_3*l_2*l_3*m_5*np.sin(theta_3 - theta_2) + dtheta_3*l_2*m_3*r_3*np.sin(theta_3 - theta_2)) - dtheta_2*(mu_2 + mu_3) + dtheta_1*mu_2 + dtheta_1*(-dtheta_1*l_1*l_2*m_3*np.sin(theta_2 - theta_1) - dtheta_1*l_1*l_2*m_4*np.sin(theta_2 - theta_1) - dtheta_1*l_1*l_2*m_5*np.sin(theta_2 - theta_1) - dtheta_1*l_1*m_2*r_2*np.sin(theta_2 - theta_1)) - ddx_c*(-l_2*m_3 - l_2*m_4 - l_2*m_5 - m_2*r_2)*np.cos(theta_2) - g*(-l_2*m_3 - l_2*m_4 - l_2*m_5 - m_2*r_2)*np.sin(theta_2) - l_1*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*(dtheta_5**2*l_1*m_5*r_5*np.sin(theta_5 - theta_1) + dtheta_4*(dtheta_4*l_1*l_4*m_5*np.sin(theta_4 - theta_1) + dtheta_4*l_1*m_4*r_4*np.sin(theta_4 - theta_1)) + dtheta_3*(dtheta_3*l_1*l_3*m_4*np.sin(theta_3 - theta_1) + dtheta_3*l_1*l_3*m_5*np.sin(theta_3 - theta_1) + dtheta_3*l_1*m_3*r_3*np.sin(theta_3 - theta_1)) + dtheta_2*mu_2 + dtheta_2*(dtheta_2*l_1*l_2*m_3*np.sin(theta_2 - theta_1) + dtheta_2*l_1*l_2*m_4*np.sin(theta_2 - theta_1) + dtheta_2*l_1*l_2*m_5*np.sin(theta_2 - theta_1) + dtheta_2*l_1*m_2*r_2*np.sin(theta_2 - theta_1)) - dtheta_1*(mu_1 + mu_2) - ddx_c*(-l_1*m_2 - l_1*m_3 - l_1*m_4 - l_1*m_5 - m_1*r_1)*np.cos(theta_1) - g*(-l_1*m_2 - l_1*m_3 - l_1*m_4 - l_1*m_5 - m_1*r_1)*np.sin(theta_1))*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2))/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2))/(I_3 - l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)**2*np.cos(theta_3 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_3**2*m_4 + l_3**2*m_5 + m_3*r_3**2 - (-l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_3 - theta_2))**2/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2)))/(I_4 - l_1**2*(l_4*m_5 + m_4*r_4)**2*np.cos(theta_4 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_4**2*m_5 + m_4*r_4**2 - (-l_1**2*(l_4*m_5 + m_4*r_4)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_4 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_4*m_5 + m_4*r_4)*np.cos(theta_4 - theta_2))**2/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2) - (-l_1**2*(l_4*m_5 + m_4*r_4)*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_4 - theta_1)*np.cos(theta_3 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_3*(l_4*m_5 + m_4*r_4)*np.cos(theta_4 - theta_3) - (-l_1**2*(l_4*m_5 + m_4*r_4)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_4 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_4*m_5 + m_4*r_4)*np.cos(theta_4 - theta_2))*(-l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_3 - theta_2))/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2))**2/(I_3 - l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)**2*np.cos(theta_3 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_3**2*m_4 + l_3**2*m_5 + m_3*r_3**2 - (-l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_3 - theta_2))**2/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2))))/(I_5 - l_1**2*m_5**2*r_5**2*np.cos(theta_5 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + m_5*r_5**2 - (-l_1**2*m_5*r_5*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_5 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*m_5*r_5*np.cos(theta_5 - theta_2))**2/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2) - (-l_1**2*m_5*r_5*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_5 - theta_1)*np.cos(theta_3 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_3*m_5*r_5*np.cos(theta_5 - theta_3) - (-l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_3 - theta_2))*(-l_1**2*m_5*r_5*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_5 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*m_5*r_5*np.cos(theta_5 - theta_2))/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2))**2/(I_3 - l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)**2*np.cos(theta_3 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_3**2*m_4 + l_3**2*m_5 + m_3*r_3**2 - (-l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_3 - theta_2))**2/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2)) - (-l_1**2*m_5*r_5*(l_4*m_5 + m_4*r_4)*np.cos(theta_5 - theta_1)*np.cos(theta_4 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_4*m_5*r_5*np.cos(theta_5 - theta_4) - (-l_1**2*(l_4*m_5 + m_4*r_4)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_4 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_4*m_5 + m_4*r_4)*np.cos(theta_4 - theta_2))*(-l_1**2*m_5*r_5*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_5 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*m_5*r_5*np.cos(theta_5 - theta_2))/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2) - (-l_1**2*(l_4*m_5 + m_4*r_4)*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_4 - theta_1)*np.cos(theta_3 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_3*(l_4*m_5 + m_4*r_4)*np.cos(theta_4 - theta_3) - (-l_1**2*(l_4*m_5 + m_4*r_4)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_4 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_4*m_5 + m_4*r_4)*np.cos(theta_4 - theta_2))*(-l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_3 - theta_2))/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2))*(-l_1**2*m_5*r_5*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_5 - theta_1)*np.cos(theta_3 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_3*m_5*r_5*np.cos(theta_5 - theta_3) - (-l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_3 - theta_2))*(-l_1**2*m_5*r_5*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_5 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*m_5*r_5*np.cos(theta_5 - theta_2))/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2))/(I_3 - l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)**2*np.cos(theta_3 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_3**2*m_4 + l_3**2*m_5 + m_3*r_3**2 - (-l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_3 - theta_2))**2/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2)))**2/(I_4 - l_1**2*(l_4*m_5 + m_4*r_4)**2*np.cos(theta_4 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_4**2*m_5 + m_4*r_4**2 - (-l_1**2*(l_4*m_5 + m_4*r_4)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_4 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_4*m_5 + m_4*r_4)*np.cos(theta_4 - theta_2))**2/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2) - (-l_1**2*(l_4*m_5 + m_4*r_4)*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_4 - theta_1)*np.cos(theta_3 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_3*(l_4*m_5 + m_4*r_4)*np.cos(theta_4 - theta_3) - (-l_1**2*(l_4*m_5 + m_4*r_4)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_4 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_4*m_5 + m_4*r_4)*np.cos(theta_4 - theta_2))*(-l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_3 - theta_2))/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2))**2/(I_3 - l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)**2*np.cos(theta_3 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_3**2*m_4 + l_3**2*m_5 + m_3*r_3**2 - (-l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_3 - theta_2))**2/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2)))))/(I_4 - l_1**2*(l_4*m_5 + m_4*r_4)**2*np.cos(theta_4 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_4**2*m_5 + m_4*r_4**2 - (-l_1**2*(l_4*m_5 + m_4*r_4)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_4 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_4*m_5 + m_4*r_4)*np.cos(theta_4 - theta_2))**2/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2) - (-l_1**2*(l_4*m_5 + m_4*r_4)*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_4 - theta_1)*np.cos(theta_3 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_3*(l_4*m_5 + m_4*r_4)*np.cos(theta_4 - theta_3) - (-l_1**2*(l_4*m_5 + m_4*r_4)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_4 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_4*m_5 + m_4*r_4)*np.cos(theta_4 - theta_2))*(-l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_3 - theta_2))/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2))**2/(I_3 - l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)**2*np.cos(theta_3 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_3**2*m_4 + l_3**2*m_5 + m_3*r_3**2 - (-l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_3 - theta_2))**2/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2))) - (-l_1**2*m_5*r_5*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_5 - theta_1)*np.cos(theta_3 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_3*m_5*r_5*np.cos(theta_5 - theta_3) - (-l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_3 - theta_2))*(-l_1**2*m_5*r_5*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_5 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*m_5*r_5*np.cos(theta_5 - theta_2))/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2))*(-dtheta_5*mu_5 - dtheta_4**2*l_4*m_5*r_5*np.sin(theta_5 - theta_4) + dtheta_4*mu_5 - dtheta_3**2*l_3*m_5*r_5*np.sin(theta_5 - theta_3) - dtheta_2**2*l_2*m_5*r_5*np.sin(theta_5 - theta_2) - dtheta_1**2*l_1*m_5*r_5*np.sin(theta_5 - theta_1) + ddx_c*m_5*r_5*np.cos(theta_5) + g*m_5*r_5*np.sin(theta_5) - l_1*m_5*r_5*(dtheta_5**2*l_1*m_5*r_5*np.sin(theta_5 - theta_1) + dtheta_4*(dtheta_4*l_1*l_4*m_5*np.sin(theta_4 - theta_1) + dtheta_4*l_1*m_4*r_4*np.sin(theta_4 - theta_1)) + dtheta_3*(dtheta_3*l_1*l_3*m_4*np.sin(theta_3 - theta_1) + dtheta_3*l_1*l_3*m_5*np.sin(theta_3 - theta_1) + dtheta_3*l_1*m_3*r_3*np.sin(theta_3 - theta_1)) + dtheta_2*mu_2 + dtheta_2*(dtheta_2*l_1*l_2*m_3*np.sin(theta_2 - theta_1) + dtheta_2*l_1*l_2*m_4*np.sin(theta_2 - theta_1) + dtheta_2*l_1*l_2*m_5*np.sin(theta_2 - theta_1) + dtheta_2*l_1*m_2*r_2*np.sin(theta_2 - theta_1)) - dtheta_1*(mu_1 + mu_2) - ddx_c*(-l_1*m_2 - l_1*m_3 - l_1*m_4 - l_1*m_5 - m_1*r_1)*np.cos(theta_1) - g*(-l_1*m_2 - l_1*m_3 - l_1*m_4 - l_1*m_5 - m_1*r_1)*np.sin(theta_1))*np.cos(theta_5 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) - (-l_1**2*m_5*r_5*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_5 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*m_5*r_5*np.cos(theta_5 - theta_2))*(dtheta_5**2*l_2*m_5*r_5*np.sin(theta_5 - theta_2) + dtheta_4*(dtheta_4*l_2*l_4*m_5*np.sin(theta_4 - theta_2) + dtheta_4*l_2*m_4*r_4*np.sin(theta_4 - theta_2)) + dtheta_3*mu_3 + dtheta_3*(dtheta_3*l_2*l_3*m_4*np.sin(theta_3 - theta_2) + dtheta_3*l_2*l_3*m_5*np.sin(theta_3 - theta_2) + dtheta_3*l_2*m_3*r_3*np.sin(theta_3 - theta_2)) - dtheta_2*(mu_2 + mu_3) + dtheta_1*mu_2 + dtheta_1*(-dtheta_1*l_1*l_2*m_3*np.sin(theta_2 - theta_1) - dtheta_1*l_1*l_2*m_4*np.sin(theta_2 - theta_1) - dtheta_1*l_1*l_2*m_5*np.sin(theta_2 - theta_1) - dtheta_1*l_1*m_2*r_2*np.sin(theta_2 - theta_1)) - ddx_c*(-l_2*m_3 - l_2*m_4 - l_2*m_5 - m_2*r_2)*np.cos(theta_2) - g*(-l_2*m_3 - l_2*m_4 - l_2*m_5 - m_2*r_2)*np.sin(theta_2) - l_1*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*(dtheta_5**2*l_1*m_5*r_5*np.sin(theta_5 - theta_1) + dtheta_4*(dtheta_4*l_1*l_4*m_5*np.sin(theta_4 - theta_1) + dtheta_4*l_1*m_4*r_4*np.sin(theta_4 - theta_1)) + dtheta_3*(dtheta_3*l_1*l_3*m_4*np.sin(theta_3 - theta_1) + dtheta_3*l_1*l_3*m_5*np.sin(theta_3 - theta_1) + dtheta_3*l_1*m_3*r_3*np.sin(theta_3 - theta_1)) + dtheta_2*mu_2 + dtheta_2*(dtheta_2*l_1*l_2*m_3*np.sin(theta_2 - theta_1) + dtheta_2*l_1*l_2*m_4*np.sin(theta_2 - theta_1) + dtheta_2*l_1*l_2*m_5*np.sin(theta_2 - theta_1) + dtheta_2*l_1*m_2*r_2*np.sin(theta_2 - theta_1)) - dtheta_1*(mu_1 + mu_2) - ddx_c*(-l_1*m_2 - l_1*m_3 - l_1*m_4 - l_1*m_5 - m_1*r_1)*np.cos(theta_1) - g*(-l_1*m_2 - l_1*m_3 - l_1*m_4 - l_1*m_5 - m_1*r_1)*np.sin(theta_1))*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2))/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2) - (-l_1**2*m_5*r_5*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_5 - theta_1)*np.cos(theta_3 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_3*m_5*r_5*np.cos(theta_5 - theta_3) - (-l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_3 - theta_2))*(-l_1**2*m_5*r_5*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_5 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*m_5*r_5*np.cos(theta_5 - theta_2))/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2))*(dtheta_5**2*l_3*m_5*r_5*np.sin(theta_5 - theta_3) + dtheta_4*mu_4 + dtheta_4*(dtheta_4*l_3*l_4*m_5*np.sin(theta_4 - theta_3) + dtheta_4*l_3*m_4*r_4*np.sin(theta_4 - theta_3)) - dtheta_3*(mu_3 + mu_4) + dtheta_2*mu_3 + dtheta_2*(-dtheta_2*l_2*l_3*m_4*np.sin(theta_3 - theta_2) - dtheta_2*l_2*l_3*m_5*np.sin(theta_3 - theta_2) - dtheta_2*l_2*m_3*r_3*np.sin(theta_3 - theta_2)) + dtheta_1*(-dtheta_1*l_1*l_3*m_4*np.sin(theta_3 - theta_1) - dtheta_1*l_1*l_3*m_5*np.sin(theta_3 - theta_1) - dtheta_1*l_1*m_3*r_3*np.sin(theta_3 - theta_1)) - ddx_c*(-l_3*m_4 - l_3*m_5 - m_3*r_3)*np.cos(theta_3) - g*(-l_3*m_4 - l_3*m_5 - m_3*r_3)*np.sin(theta_3) - l_1*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(dtheta_5**2*l_1*m_5*r_5*np.sin(theta_5 - theta_1) + dtheta_4*(dtheta_4*l_1*l_4*m_5*np.sin(theta_4 - theta_1) + dtheta_4*l_1*m_4*r_4*np.sin(theta_4 - theta_1)) + dtheta_3*(dtheta_3*l_1*l_3*m_4*np.sin(theta_3 - theta_1) + dtheta_3*l_1*l_3*m_5*np.sin(theta_3 - theta_1) + dtheta_3*l_1*m_3*r_3*np.sin(theta_3 - theta_1)) + dtheta_2*mu_2 + dtheta_2*(dtheta_2*l_1*l_2*m_3*np.sin(theta_2 - theta_1) + dtheta_2*l_1*l_2*m_4*np.sin(theta_2 - theta_1) + dtheta_2*l_1*l_2*m_5*np.sin(theta_2 - theta_1) + dtheta_2*l_1*m_2*r_2*np.sin(theta_2 - theta_1)) - dtheta_1*(mu_1 + mu_2) - ddx_c*(-l_1*m_2 - l_1*m_3 - l_1*m_4 - l_1*m_5 - m_1*r_1)*np.cos(theta_1) - g*(-l_1*m_2 - l_1*m_3 - l_1*m_4 - l_1*m_5 - m_1*r_1)*np.sin(theta_1))*np.cos(theta_3 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) - (-l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_3 - theta_2))*(dtheta_5**2*l_2*m_5*r_5*np.sin(theta_5 - theta_2) + dtheta_4*(dtheta_4*l_2*l_4*m_5*np.sin(theta_4 - theta_2) + dtheta_4*l_2*m_4*r_4*np.sin(theta_4 - theta_2)) + dtheta_3*mu_3 + dtheta_3*(dtheta_3*l_2*l_3*m_4*np.sin(theta_3 - theta_2) + dtheta_3*l_2*l_3*m_5*np.sin(theta_3 - theta_2) + dtheta_3*l_2*m_3*r_3*np.sin(theta_3 - theta_2)) - dtheta_2*(mu_2 + mu_3) + dtheta_1*mu_2 + dtheta_1*(-dtheta_1*l_1*l_2*m_3*np.sin(theta_2 - theta_1) - dtheta_1*l_1*l_2*m_4*np.sin(theta_2 - theta_1) - dtheta_1*l_1*l_2*m_5*np.sin(theta_2 - theta_1) - dtheta_1*l_1*m_2*r_2*np.sin(theta_2 - theta_1)) - ddx_c*(-l_2*m_3 - l_2*m_4 - l_2*m_5 - m_2*r_2)*np.cos(theta_2) - g*(-l_2*m_3 - l_2*m_4 - l_2*m_5 - m_2*r_2)*np.sin(theta_2) - l_1*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*(dtheta_5**2*l_1*m_5*r_5*np.sin(theta_5 - theta_1) + dtheta_4*(dtheta_4*l_1*l_4*m_5*np.sin(theta_4 - theta_1) + dtheta_4*l_1*m_4*r_4*np.sin(theta_4 - theta_1)) + dtheta_3*(dtheta_3*l_1*l_3*m_4*np.sin(theta_3 - theta_1) + dtheta_3*l_1*l_3*m_5*np.sin(theta_3 - theta_1) + dtheta_3*l_1*m_3*r_3*np.sin(theta_3 - theta_1)) + dtheta_2*mu_2 + dtheta_2*(dtheta_2*l_1*l_2*m_3*np.sin(theta_2 - theta_1) + dtheta_2*l_1*l_2*m_4*np.sin(theta_2 - theta_1) + dtheta_2*l_1*l_2*m_5*np.sin(theta_2 - theta_1) + dtheta_2*l_1*m_2*r_2*np.sin(theta_2 - theta_1)) - dtheta_1*(mu_1 + mu_2) - ddx_c*(-l_1*m_2 - l_1*m_3 - l_1*m_4 - l_1*m_5 - m_1*r_1)*np.cos(theta_1) - g*(-l_1*m_2 - l_1*m_3 - l_1*m_4 - l_1*m_5 - m_1*r_1)*np.sin(theta_1))*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2))/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2))/(I_3 - l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)**2*np.cos(theta_3 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_3**2*m_4 + l_3**2*m_5 + m_3*r_3**2 - (-l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_3 - theta_2))**2/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2)) - (-l_1**2*m_5*r_5*(l_4*m_5 + m_4*r_4)*np.cos(theta_5 - theta_1)*np.cos(theta_4 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_4*m_5*r_5*np.cos(theta_5 - theta_4) - (-l_1**2*(l_4*m_5 + m_4*r_4)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_4 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_4*m_5 + m_4*r_4)*np.cos(theta_4 - theta_2))*(-l_1**2*m_5*r_5*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_5 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*m_5*r_5*np.cos(theta_5 - theta_2))/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2) - (-l_1**2*(l_4*m_5 + m_4*r_4)*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_4 - theta_1)*np.cos(theta_3 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_3*(l_4*m_5 + m_4*r_4)*np.cos(theta_4 - theta_3) - (-l_1**2*(l_4*m_5 + m_4*r_4)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_4 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_4*m_5 + m_4*r_4)*np.cos(theta_4 - theta_2))*(-l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_3 - theta_2))/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2))*(-l_1**2*m_5*r_5*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_5 - theta_1)*np.cos(theta_3 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_3*m_5*r_5*np.cos(theta_5 - theta_3) - (-l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_3 - theta_2))*(-l_1**2*m_5*r_5*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_5 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*m_5*r_5*np.cos(theta_5 - theta_2))/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2))/(I_3 - l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)**2*np.cos(theta_3 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_3**2*m_4 + l_3**2*m_5 + m_3*r_3**2 - (-l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_3 - theta_2))**2/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2)))*(dtheta_5**2*l_4*m_5*r_5*np.sin(theta_5 - theta_4) + dtheta_5*mu_5 - dtheta_4*(mu_4 + mu_5) + dtheta_3*mu_4 + dtheta_3*(-dtheta_3*l_3*l_4*m_5*np.sin(theta_4 - theta_3) - dtheta_3*l_3*m_4*r_4*np.sin(theta_4 - theta_3)) + dtheta_2*(-dtheta_2*l_2*l_4*m_5*np.sin(theta_4 - theta_2) - dtheta_2*l_2*m_4*r_4*np.sin(theta_4 - theta_2)) + dtheta_1*(-dtheta_1*l_1*l_4*m_5*np.sin(theta_4 - theta_1) - dtheta_1*l_1*m_4*r_4*np.sin(theta_4 - theta_1)) - ddx_c*(-l_4*m_5 - m_4*r_4)*np.cos(theta_4) - g*(-l_4*m_5 - m_4*r_4)*np.sin(theta_4) - l_1*(l_4*m_5 + m_4*r_4)*(dtheta_5**2*l_1*m_5*r_5*np.sin(theta_5 - theta_1) + dtheta_4*(dtheta_4*l_1*l_4*m_5*np.sin(theta_4 - theta_1) + dtheta_4*l_1*m_4*r_4*np.sin(theta_4 - theta_1)) + dtheta_3*(dtheta_3*l_1*l_3*m_4*np.sin(theta_3 - theta_1) + dtheta_3*l_1*l_3*m_5*np.sin(theta_3 - theta_1) + dtheta_3*l_1*m_3*r_3*np.sin(theta_3 - theta_1)) + dtheta_2*mu_2 + dtheta_2*(dtheta_2*l_1*l_2*m_3*np.sin(theta_2 - theta_1) + dtheta_2*l_1*l_2*m_4*np.sin(theta_2 - theta_1) + dtheta_2*l_1*l_2*m_5*np.sin(theta_2 - theta_1) + dtheta_2*l_1*m_2*r_2*np.sin(theta_2 - theta_1)) - dtheta_1*(mu_1 + mu_2) - ddx_c*(-l_1*m_2 - l_1*m_3 - l_1*m_4 - l_1*m_5 - m_1*r_1)*np.cos(theta_1) - g*(-l_1*m_2 - l_1*m_3 - l_1*m_4 - l_1*m_5 - m_1*r_1)*np.sin(theta_1))*np.cos(theta_4 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) - (-l_1**2*(l_4*m_5 + m_4*r_4)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_4 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_4*m_5 + m_4*r_4)*np.cos(theta_4 - theta_2))*(dtheta_5**2*l_2*m_5*r_5*np.sin(theta_5 - theta_2) + dtheta_4*(dtheta_4*l_2*l_4*m_5*np.sin(theta_4 - theta_2) + dtheta_4*l_2*m_4*r_4*np.sin(theta_4 - theta_2)) + dtheta_3*mu_3 + dtheta_3*(dtheta_3*l_2*l_3*m_4*np.sin(theta_3 - theta_2) + dtheta_3*l_2*l_3*m_5*np.sin(theta_3 - theta_2) + dtheta_3*l_2*m_3*r_3*np.sin(theta_3 - theta_2)) - dtheta_2*(mu_2 + mu_3) + dtheta_1*mu_2 + dtheta_1*(-dtheta_1*l_1*l_2*m_3*np.sin(theta_2 - theta_1) - dtheta_1*l_1*l_2*m_4*np.sin(theta_2 - theta_1) - dtheta_1*l_1*l_2*m_5*np.sin(theta_2 - theta_1) - dtheta_1*l_1*m_2*r_2*np.sin(theta_2 - theta_1)) - ddx_c*(-l_2*m_3 - l_2*m_4 - l_2*m_5 - m_2*r_2)*np.cos(theta_2) - g*(-l_2*m_3 - l_2*m_4 - l_2*m_5 - m_2*r_2)*np.sin(theta_2) - l_1*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*(dtheta_5**2*l_1*m_5*r_5*np.sin(theta_5 - theta_1) + dtheta_4*(dtheta_4*l_1*l_4*m_5*np.sin(theta_4 - theta_1) + dtheta_4*l_1*m_4*r_4*np.sin(theta_4 - theta_1)) + dtheta_3*(dtheta_3*l_1*l_3*m_4*np.sin(theta_3 - theta_1) + dtheta_3*l_1*l_3*m_5*np.sin(theta_3 - theta_1) + dtheta_3*l_1*m_3*r_3*np.sin(theta_3 - theta_1)) + dtheta_2*mu_2 + dtheta_2*(dtheta_2*l_1*l_2*m_3*np.sin(theta_2 - theta_1) + dtheta_2*l_1*l_2*m_4*np.sin(theta_2 - theta_1) + dtheta_2*l_1*l_2*m_5*np.sin(theta_2 - theta_1) + dtheta_2*l_1*m_2*r_2*np.sin(theta_2 - theta_1)) - dtheta_1*(mu_1 + mu_2) - ddx_c*(-l_1*m_2 - l_1*m_3 - l_1*m_4 - l_1*m_5 - m_1*r_1)*np.cos(theta_1) - g*(-l_1*m_2 - l_1*m_3 - l_1*m_4 - l_1*m_5 - m_1*r_1)*np.sin(theta_1))*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2))/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2) - (-l_1**2*(l_4*m_5 + m_4*r_4)*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_4 - theta_1)*np.cos(theta_3 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_3*(l_4*m_5 + m_4*r_4)*np.cos(theta_4 - theta_3) - (-l_1**2*(l_4*m_5 + m_4*r_4)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_4 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_4*m_5 + m_4*r_4)*np.cos(theta_4 - theta_2))*(-l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_3 - theta_2))/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2))*(dtheta_5**2*l_3*m_5*r_5*np.sin(theta_5 - theta_3) + dtheta_4*mu_4 + dtheta_4*(dtheta_4*l_3*l_4*m_5*np.sin(theta_4 - theta_3) + dtheta_4*l_3*m_4*r_4*np.sin(theta_4 - theta_3)) - dtheta_3*(mu_3 + mu_4) + dtheta_2*mu_3 + dtheta_2*(-dtheta_2*l_2*l_3*m_4*np.sin(theta_3 - theta_2) - dtheta_2*l_2*l_3*m_5*np.sin(theta_3 - theta_2) - dtheta_2*l_2*m_3*r_3*np.sin(theta_3 - theta_2)) + dtheta_1*(-dtheta_1*l_1*l_3*m_4*np.sin(theta_3 - theta_1) - dtheta_1*l_1*l_3*m_5*np.sin(theta_3 - theta_1) - dtheta_1*l_1*m_3*r_3*np.sin(theta_3 - theta_1)) - ddx_c*(-l_3*m_4 - l_3*m_5 - m_3*r_3)*np.cos(theta_3) - g*(-l_3*m_4 - l_3*m_5 - m_3*r_3)*np.sin(theta_3) - l_1*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(dtheta_5**2*l_1*m_5*r_5*np.sin(theta_5 - theta_1) + dtheta_4*(dtheta_4*l_1*l_4*m_5*np.sin(theta_4 - theta_1) + dtheta_4*l_1*m_4*r_4*np.sin(theta_4 - theta_1)) + dtheta_3*(dtheta_3*l_1*l_3*m_4*np.sin(theta_3 - theta_1) + dtheta_3*l_1*l_3*m_5*np.sin(theta_3 - theta_1) + dtheta_3*l_1*m_3*r_3*np.sin(theta_3 - theta_1)) + dtheta_2*mu_2 + dtheta_2*(dtheta_2*l_1*l_2*m_3*np.sin(theta_2 - theta_1) + dtheta_2*l_1*l_2*m_4*np.sin(theta_2 - theta_1) + dtheta_2*l_1*l_2*m_5*np.sin(theta_2 - theta_1) + dtheta_2*l_1*m_2*r_2*np.sin(theta_2 - theta_1)) - dtheta_1*(mu_1 + mu_2) - ddx_c*(-l_1*m_2 - l_1*m_3 - l_1*m_4 - l_1*m_5 - m_1*r_1)*np.cos(theta_1) - g*(-l_1*m_2 - l_1*m_3 - l_1*m_4 - l_1*m_5 - m_1*r_1)*np.sin(theta_1))*np.cos(theta_3 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) - (-l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_3 - theta_2))*(dtheta_5**2*l_2*m_5*r_5*np.sin(theta_5 - theta_2) + dtheta_4*(dtheta_4*l_2*l_4*m_5*np.sin(theta_4 - theta_2) + dtheta_4*l_2*m_4*r_4*np.sin(theta_4 - theta_2)) + dtheta_3*mu_3 + dtheta_3*(dtheta_3*l_2*l_3*m_4*np.sin(theta_3 - theta_2) + dtheta_3*l_2*l_3*m_5*np.sin(theta_3 - theta_2) + dtheta_3*l_2*m_3*r_3*np.sin(theta_3 - theta_2)) - dtheta_2*(mu_2 + mu_3) + dtheta_1*mu_2 + dtheta_1*(-dtheta_1*l_1*l_2*m_3*np.sin(theta_2 - theta_1) - dtheta_1*l_1*l_2*m_4*np.sin(theta_2 - theta_1) - dtheta_1*l_1*l_2*m_5*np.sin(theta_2 - theta_1) - dtheta_1*l_1*m_2*r_2*np.sin(theta_2 - theta_1)) - ddx_c*(-l_2*m_3 - l_2*m_4 - l_2*m_5 - m_2*r_2)*np.cos(theta_2) - g*(-l_2*m_3 - l_2*m_4 - l_2*m_5 - m_2*r_2)*np.sin(theta_2) - l_1*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*(dtheta_5**2*l_1*m_5*r_5*np.sin(theta_5 - theta_1) + dtheta_4*(dtheta_4*l_1*l_4*m_5*np.sin(theta_4 - theta_1) + dtheta_4*l_1*m_4*r_4*np.sin(theta_4 - theta_1)) + dtheta_3*(dtheta_3*l_1*l_3*m_4*np.sin(theta_3 - theta_1) + dtheta_3*l_1*l_3*m_5*np.sin(theta_3 - theta_1) + dtheta_3*l_1*m_3*r_3*np.sin(theta_3 - theta_1)) + dtheta_2*mu_2 + dtheta_2*(dtheta_2*l_1*l_2*m_3*np.sin(theta_2 - theta_1) + dtheta_2*l_1*l_2*m_4*np.sin(theta_2 - theta_1) + dtheta_2*l_1*l_2*m_5*np.sin(theta_2 - theta_1) + dtheta_2*l_1*m_2*r_2*np.sin(theta_2 - theta_1)) - dtheta_1*(mu_1 + mu_2) - ddx_c*(-l_1*m_2 - l_1*m_3 - l_1*m_4 - l_1*m_5 - m_1*r_1)*np.cos(theta_1) - g*(-l_1*m_2 - l_1*m_3 - l_1*m_4 - l_1*m_5 - m_1*r_1)*np.sin(theta_1))*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2))/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2))/(I_3 - l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)**2*np.cos(theta_3 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_3**2*m_4 + l_3**2*m_5 + m_3*r_3**2 - (-l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_3 - theta_2))**2/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2)))/(I_4 - l_1**2*(l_4*m_5 + m_4*r_4)**2*np.cos(theta_4 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_4**2*m_5 + m_4*r_4**2 - (-l_1**2*(l_4*m_5 + m_4*r_4)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_4 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_4*m_5 + m_4*r_4)*np.cos(theta_4 - theta_2))**2/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2) - (-l_1**2*(l_4*m_5 + m_4*r_4)*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_4 - theta_1)*np.cos(theta_3 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_3*(l_4*m_5 + m_4*r_4)*np.cos(theta_4 - theta_3) - (-l_1**2*(l_4*m_5 + m_4*r_4)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_4 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_4*m_5 + m_4*r_4)*np.cos(theta_4 - theta_2))*(-l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_3 - theta_2))/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2))**2/(I_3 - l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)**2*np.cos(theta_3 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_3**2*m_4 + l_3**2*m_5 + m_3*r_3**2 - (-l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_3 - theta_2))**2/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2))))/(I_5 - l_1**2*m_5**2*r_5**2*np.cos(theta_5 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + m_5*r_5**2 - (-l_1**2*m_5*r_5*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_5 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*m_5*r_5*np.cos(theta_5 - theta_2))**2/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2) - (-l_1**2*m_5*r_5*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_5 - theta_1)*np.cos(theta_3 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_3*m_5*r_5*np.cos(theta_5 - theta_3) - (-l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_3 - theta_2))*(-l_1**2*m_5*r_5*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_5 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*m_5*r_5*np.cos(theta_5 - theta_2))/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2))**2/(I_3 - l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)**2*np.cos(theta_3 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_3**2*m_4 + l_3**2*m_5 + m_3*r_3**2 - (-l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_3 - theta_2))**2/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2)) - (-l_1**2*m_5*r_5*(l_4*m_5 + m_4*r_4)*np.cos(theta_5 - theta_1)*np.cos(theta_4 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_4*m_5*r_5*np.cos(theta_5 - theta_4) - (-l_1**2*(l_4*m_5 + m_4*r_4)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_4 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_4*m_5 + m_4*r_4)*np.cos(theta_4 - theta_2))*(-l_1**2*m_5*r_5*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_5 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*m_5*r_5*np.cos(theta_5 - theta_2))/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2) - (-l_1**2*(l_4*m_5 + m_4*r_4)*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_4 - theta_1)*np.cos(theta_3 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_3*(l_4*m_5 + m_4*r_4)*np.cos(theta_4 - theta_3) - (-l_1**2*(l_4*m_5 + m_4*r_4)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_4 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_4*m_5 + m_4*r_4)*np.cos(theta_4 - theta_2))*(-l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_3 - theta_2))/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2))*(-l_1**2*m_5*r_5*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_5 - theta_1)*np.cos(theta_3 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_3*m_5*r_5*np.cos(theta_5 - theta_3) - (-l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_3 - theta_2))*(-l_1**2*m_5*r_5*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_5 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*m_5*r_5*np.cos(theta_5 - theta_2))/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2))/(I_3 - l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)**2*np.cos(theta_3 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_3**2*m_4 + l_3**2*m_5 + m_3*r_3**2 - (-l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_3 - theta_2))**2/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2)))**2/(I_4 - l_1**2*(l_4*m_5 + m_4*r_4)**2*np.cos(theta_4 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_4**2*m_5 + m_4*r_4**2 - (-l_1**2*(l_4*m_5 + m_4*r_4)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_4 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_4*m_5 + m_4*r_4)*np.cos(theta_4 - theta_2))**2/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2) - (-l_1**2*(l_4*m_5 + m_4*r_4)*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_4 - theta_1)*np.cos(theta_3 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_3*(l_4*m_5 + m_4*r_4)*np.cos(theta_4 - theta_3) - (-l_1**2*(l_4*m_5 + m_4*r_4)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_4 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_4*m_5 + m_4*r_4)*np.cos(theta_4 - theta_2))*(-l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_3 - theta_2))/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2))**2/(I_3 - l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)**2*np.cos(theta_3 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_3**2*m_4 + l_3**2*m_5 + m_3*r_3**2 - (-l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_3 - theta_2))**2/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2)))))/(I_3 - l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)**2*np.cos(theta_3 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_3**2*m_4 + l_3**2*m_5 + m_3*r_3**2 - (-l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_3 - theta_2))**2/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2)),


                      (dtheta_5**2*l_4*m_5*r_5*np.sin(theta_5 - theta_4) + dtheta_5*mu_5 - dtheta_4*(mu_4 + mu_5) + dtheta_3*mu_4 + dtheta_3*(-dtheta_3*l_3*l_4*m_5*np.sin(theta_4 - theta_3) - dtheta_3*l_3*m_4*r_4*np.sin(theta_4 - theta_3)) + dtheta_2*(-dtheta_2*l_2*l_4*m_5*np.sin(theta_4 - theta_2) - dtheta_2*l_2*m_4*r_4*np.sin(theta_4 - theta_2)) + dtheta_1*(-dtheta_1*l_1*l_4*m_5*np.sin(theta_4 - theta_1) - dtheta_1*l_1*m_4*r_4*np.sin(theta_4 - theta_1)) - ddx_c*(-l_4*m_5 - m_4*r_4)*np.cos(theta_4) - g*(-l_4*m_5 - m_4*r_4)*np.sin(theta_4) - l_1*(l_4*m_5 + m_4*r_4)*(dtheta_5**2*l_1*m_5*r_5*np.sin(theta_5 - theta_1) + dtheta_4*(dtheta_4*l_1*l_4*m_5*np.sin(theta_4 - theta_1) + dtheta_4*l_1*m_4*r_4*np.sin(theta_4 - theta_1)) + dtheta_3*(dtheta_3*l_1*l_3*m_4*np.sin(theta_3 - theta_1) + dtheta_3*l_1*l_3*m_5*np.sin(theta_3 - theta_1) + dtheta_3*l_1*m_3*r_3*np.sin(theta_3 - theta_1)) + dtheta_2*mu_2 + dtheta_2*(dtheta_2*l_1*l_2*m_3*np.sin(theta_2 - theta_1) + dtheta_2*l_1*l_2*m_4*np.sin(theta_2 - theta_1) + dtheta_2*l_1*l_2*m_5*np.sin(theta_2 - theta_1) + dtheta_2*l_1*m_2*r_2*np.sin(theta_2 - theta_1)) - dtheta_1*(mu_1 + mu_2) - ddx_c*(-l_1*m_2 - l_1*m_3 - l_1*m_4 - l_1*m_5 - m_1*r_1)*np.cos(theta_1) - g*(-l_1*m_2 - l_1*m_3 - l_1*m_4 - l_1*m_5 - m_1*r_1)*np.sin(theta_1))*np.cos(theta_4 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) - (-l_1**2*(l_4*m_5 + m_4*r_4)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_4 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_4*m_5 + m_4*r_4)*np.cos(theta_4 - theta_2))*(dtheta_5**2*l_2*m_5*r_5*np.sin(theta_5 - theta_2) + dtheta_4*(dtheta_4*l_2*l_4*m_5*np.sin(theta_4 - theta_2) + dtheta_4*l_2*m_4*r_4*np.sin(theta_4 - theta_2)) + dtheta_3*mu_3 + dtheta_3*(dtheta_3*l_2*l_3*m_4*np.sin(theta_3 - theta_2) + dtheta_3*l_2*l_3*m_5*np.sin(theta_3 - theta_2) + dtheta_3*l_2*m_3*r_3*np.sin(theta_3 - theta_2)) - dtheta_2*(mu_2 + mu_3) + dtheta_1*mu_2 + dtheta_1*(-dtheta_1*l_1*l_2*m_3*np.sin(theta_2 - theta_1) - dtheta_1*l_1*l_2*m_4*np.sin(theta_2 - theta_1) - dtheta_1*l_1*l_2*m_5*np.sin(theta_2 - theta_1) - dtheta_1*l_1*m_2*r_2*np.sin(theta_2 - theta_1)) - ddx_c*(-l_2*m_3 - l_2*m_4 - l_2*m_5 - m_2*r_2)*np.cos(theta_2) - g*(-l_2*m_3 - l_2*m_4 - l_2*m_5 - m_2*r_2)*np.sin(theta_2) - l_1*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*(dtheta_5**2*l_1*m_5*r_5*np.sin(theta_5 - theta_1) + dtheta_4*(dtheta_4*l_1*l_4*m_5*np.sin(theta_4 - theta_1) + dtheta_4*l_1*m_4*r_4*np.sin(theta_4 - theta_1)) + dtheta_3*(dtheta_3*l_1*l_3*m_4*np.sin(theta_3 - theta_1) + dtheta_3*l_1*l_3*m_5*np.sin(theta_3 - theta_1) + dtheta_3*l_1*m_3*r_3*np.sin(theta_3 - theta_1)) + dtheta_2*mu_2 + dtheta_2*(dtheta_2*l_1*l_2*m_3*np.sin(theta_2 - theta_1) + dtheta_2*l_1*l_2*m_4*np.sin(theta_2 - theta_1) + dtheta_2*l_1*l_2*m_5*np.sin(theta_2 - theta_1) + dtheta_2*l_1*m_2*r_2*np.sin(theta_2 - theta_1)) - dtheta_1*(mu_1 + mu_2) - ddx_c*(-l_1*m_2 - l_1*m_3 - l_1*m_4 - l_1*m_5 - m_1*r_1)*np.cos(theta_1) - g*(-l_1*m_2 - l_1*m_3 - l_1*m_4 - l_1*m_5 - m_1*r_1)*np.sin(theta_1))*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2))/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2) - (-l_1**2*(l_4*m_5 + m_4*r_4)*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_4 - theta_1)*np.cos(theta_3 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_3*(l_4*m_5 + m_4*r_4)*np.cos(theta_4 - theta_3) - (-l_1**2*(l_4*m_5 + m_4*r_4)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_4 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_4*m_5 + m_4*r_4)*np.cos(theta_4 - theta_2))*(-l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_3 - theta_2))/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2))*(dtheta_5**2*l_3*m_5*r_5*np.sin(theta_5 - theta_3) + dtheta_4*mu_4 + dtheta_4*(dtheta_4*l_3*l_4*m_5*np.sin(theta_4 - theta_3) + dtheta_4*l_3*m_4*r_4*np.sin(theta_4 - theta_3)) - dtheta_3*(mu_3 + mu_4) + dtheta_2*mu_3 + dtheta_2*(-dtheta_2*l_2*l_3*m_4*np.sin(theta_3 - theta_2) - dtheta_2*l_2*l_3*m_5*np.sin(theta_3 - theta_2) - dtheta_2*l_2*m_3*r_3*np.sin(theta_3 - theta_2)) + dtheta_1*(-dtheta_1*l_1*l_3*m_4*np.sin(theta_3 - theta_1) - dtheta_1*l_1*l_3*m_5*np.sin(theta_3 - theta_1) - dtheta_1*l_1*m_3*r_3*np.sin(theta_3 - theta_1)) - ddx_c*(-l_3*m_4 - l_3*m_5 - m_3*r_3)*np.cos(theta_3) - g*(-l_3*m_4 - l_3*m_5 - m_3*r_3)*np.sin(theta_3) - l_1*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(dtheta_5**2*l_1*m_5*r_5*np.sin(theta_5 - theta_1) + dtheta_4*(dtheta_4*l_1*l_4*m_5*np.sin(theta_4 - theta_1) + dtheta_4*l_1*m_4*r_4*np.sin(theta_4 - theta_1)) + dtheta_3*(dtheta_3*l_1*l_3*m_4*np.sin(theta_3 - theta_1) + dtheta_3*l_1*l_3*m_5*np.sin(theta_3 - theta_1) + dtheta_3*l_1*m_3*r_3*np.sin(theta_3 - theta_1)) + dtheta_2*mu_2 + dtheta_2*(dtheta_2*l_1*l_2*m_3*np.sin(theta_2 - theta_1) + dtheta_2*l_1*l_2*m_4*np.sin(theta_2 - theta_1) + dtheta_2*l_1*l_2*m_5*np.sin(theta_2 - theta_1) + dtheta_2*l_1*m_2*r_2*np.sin(theta_2 - theta_1)) - dtheta_1*(mu_1 + mu_2) - ddx_c*(-l_1*m_2 - l_1*m_3 - l_1*m_4 - l_1*m_5 - m_1*r_1)*np.cos(theta_1) - g*(-l_1*m_2 - l_1*m_3 - l_1*m_4 - l_1*m_5 - m_1*r_1)*np.sin(theta_1))*np.cos(theta_3 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) - (-l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_3 - theta_2))*(dtheta_5**2*l_2*m_5*r_5*np.sin(theta_5 - theta_2) + dtheta_4*(dtheta_4*l_2*l_4*m_5*np.sin(theta_4 - theta_2) + dtheta_4*l_2*m_4*r_4*np.sin(theta_4 - theta_2)) + dtheta_3*mu_3 + dtheta_3*(dtheta_3*l_2*l_3*m_4*np.sin(theta_3 - theta_2) + dtheta_3*l_2*l_3*m_5*np.sin(theta_3 - theta_2) + dtheta_3*l_2*m_3*r_3*np.sin(theta_3 - theta_2)) - dtheta_2*(mu_2 + mu_3) + dtheta_1*mu_2 + dtheta_1*(-dtheta_1*l_1*l_2*m_3*np.sin(theta_2 - theta_1) - dtheta_1*l_1*l_2*m_4*np.sin(theta_2 - theta_1) - dtheta_1*l_1*l_2*m_5*np.sin(theta_2 - theta_1) - dtheta_1*l_1*m_2*r_2*np.sin(theta_2 - theta_1)) - ddx_c*(-l_2*m_3 - l_2*m_4 - l_2*m_5 - m_2*r_2)*np.cos(theta_2) - g*(-l_2*m_3 - l_2*m_4 - l_2*m_5 - m_2*r_2)*np.sin(theta_2) - l_1*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*(dtheta_5**2*l_1*m_5*r_5*np.sin(theta_5 - theta_1) + dtheta_4*(dtheta_4*l_1*l_4*m_5*np.sin(theta_4 - theta_1) + dtheta_4*l_1*m_4*r_4*np.sin(theta_4 - theta_1)) + dtheta_3*(dtheta_3*l_1*l_3*m_4*np.sin(theta_3 - theta_1) + dtheta_3*l_1*l_3*m_5*np.sin(theta_3 - theta_1) + dtheta_3*l_1*m_3*r_3*np.sin(theta_3 - theta_1)) + dtheta_2*mu_2 + dtheta_2*(dtheta_2*l_1*l_2*m_3*np.sin(theta_2 - theta_1) + dtheta_2*l_1*l_2*m_4*np.sin(theta_2 - theta_1) + dtheta_2*l_1*l_2*m_5*np.sin(theta_2 - theta_1) + dtheta_2*l_1*m_2*r_2*np.sin(theta_2 - theta_1)) - dtheta_1*(mu_1 + mu_2) - ddx_c*(-l_1*m_2 - l_1*m_3 - l_1*m_4 - l_1*m_5 - m_1*r_1)*np.cos(theta_1) - g*(-l_1*m_2 - l_1*m_3 - l_1*m_4 - l_1*m_5 - m_1*r_1)*np.sin(theta_1))*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2))/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2))/(I_3 - l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)**2*np.cos(theta_3 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_3**2*m_4 + l_3**2*m_5 + m_3*r_3**2 - (-l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_3 - theta_2))**2/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2)) - (-l_1**2*m_5*r_5*(l_4*m_5 + m_4*r_4)*np.cos(theta_5 - theta_1)*np.cos(theta_4 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_4*m_5*r_5*np.cos(theta_5 - theta_4) - (-l_1**2*(l_4*m_5 + m_4*r_4)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_4 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_4*m_5 + m_4*r_4)*np.cos(theta_4 - theta_2))*(-l_1**2*m_5*r_5*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_5 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*m_5*r_5*np.cos(theta_5 - theta_2))/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2) - (-l_1**2*(l_4*m_5 + m_4*r_4)*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_4 - theta_1)*np.cos(theta_3 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_3*(l_4*m_5 + m_4*r_4)*np.cos(theta_4 - theta_3) - (-l_1**2*(l_4*m_5 + m_4*r_4)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_4 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_4*m_5 + m_4*r_4)*np.cos(theta_4 - theta_2))*(-l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_3 - theta_2))/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2))*(-l_1**2*m_5*r_5*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_5 - theta_1)*np.cos(theta_3 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_3*m_5*r_5*np.cos(theta_5 - theta_3) - (-l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_3 - theta_2))*(-l_1**2*m_5*r_5*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_5 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*m_5*r_5*np.cos(theta_5 - theta_2))/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2))/(I_3 - l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)**2*np.cos(theta_3 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_3**2*m_4 + l_3**2*m_5 + m_3*r_3**2 - (-l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_3 - theta_2))**2/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2)))*(-dtheta_5*mu_5 - dtheta_4**2*l_4*m_5*r_5*np.sin(theta_5 - theta_4) + dtheta_4*mu_5 - dtheta_3**2*l_3*m_5*r_5*np.sin(theta_5 - theta_3) - dtheta_2**2*l_2*m_5*r_5*np.sin(theta_5 - theta_2) - dtheta_1**2*l_1*m_5*r_5*np.sin(theta_5 - theta_1) + ddx_c*m_5*r_5*np.cos(theta_5) + g*m_5*r_5*np.sin(theta_5) - l_1*m_5*r_5*(dtheta_5**2*l_1*m_5*r_5*np.sin(theta_5 - theta_1) + dtheta_4*(dtheta_4*l_1*l_4*m_5*np.sin(theta_4 - theta_1) + dtheta_4*l_1*m_4*r_4*np.sin(theta_4 - theta_1)) + dtheta_3*(dtheta_3*l_1*l_3*m_4*np.sin(theta_3 - theta_1) + dtheta_3*l_1*l_3*m_5*np.sin(theta_3 - theta_1) + dtheta_3*l_1*m_3*r_3*np.sin(theta_3 - theta_1)) + dtheta_2*mu_2 + dtheta_2*(dtheta_2*l_1*l_2*m_3*np.sin(theta_2 - theta_1) + dtheta_2*l_1*l_2*m_4*np.sin(theta_2 - theta_1) + dtheta_2*l_1*l_2*m_5*np.sin(theta_2 - theta_1) + dtheta_2*l_1*m_2*r_2*np.sin(theta_2 - theta_1)) - dtheta_1*(mu_1 + mu_2) - ddx_c*(-l_1*m_2 - l_1*m_3 - l_1*m_4 - l_1*m_5 - m_1*r_1)*np.cos(theta_1) - g*(-l_1*m_2 - l_1*m_3 - l_1*m_4 - l_1*m_5 - m_1*r_1)*np.sin(theta_1))*np.cos(theta_5 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) - (-l_1**2*m_5*r_5*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_5 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*m_5*r_5*np.cos(theta_5 - theta_2))*(dtheta_5**2*l_2*m_5*r_5*np.sin(theta_5 - theta_2) + dtheta_4*(dtheta_4*l_2*l_4*m_5*np.sin(theta_4 - theta_2) + dtheta_4*l_2*m_4*r_4*np.sin(theta_4 - theta_2)) + dtheta_3*mu_3 + dtheta_3*(dtheta_3*l_2*l_3*m_4*np.sin(theta_3 - theta_2) + dtheta_3*l_2*l_3*m_5*np.sin(theta_3 - theta_2) + dtheta_3*l_2*m_3*r_3*np.sin(theta_3 - theta_2)) - dtheta_2*(mu_2 + mu_3) + dtheta_1*mu_2 + dtheta_1*(-dtheta_1*l_1*l_2*m_3*np.sin(theta_2 - theta_1) - dtheta_1*l_1*l_2*m_4*np.sin(theta_2 - theta_1) - dtheta_1*l_1*l_2*m_5*np.sin(theta_2 - theta_1) - dtheta_1*l_1*m_2*r_2*np.sin(theta_2 - theta_1)) - ddx_c*(-l_2*m_3 - l_2*m_4 - l_2*m_5 - m_2*r_2)*np.cos(theta_2) - g*(-l_2*m_3 - l_2*m_4 - l_2*m_5 - m_2*r_2)*np.sin(theta_2) - l_1*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*(dtheta_5**2*l_1*m_5*r_5*np.sin(theta_5 - theta_1) + dtheta_4*(dtheta_4*l_1*l_4*m_5*np.sin(theta_4 - theta_1) + dtheta_4*l_1*m_4*r_4*np.sin(theta_4 - theta_1)) + dtheta_3*(dtheta_3*l_1*l_3*m_4*np.sin(theta_3 - theta_1) + dtheta_3*l_1*l_3*m_5*np.sin(theta_3 - theta_1) + dtheta_3*l_1*m_3*r_3*np.sin(theta_3 - theta_1)) + dtheta_2*mu_2 + dtheta_2*(dtheta_2*l_1*l_2*m_3*np.sin(theta_2 - theta_1) + dtheta_2*l_1*l_2*m_4*np.sin(theta_2 - theta_1) + dtheta_2*l_1*l_2*m_5*np.sin(theta_2 - theta_1) + dtheta_2*l_1*m_2*r_2*np.sin(theta_2 - theta_1)) - dtheta_1*(mu_1 + mu_2) - ddx_c*(-l_1*m_2 - l_1*m_3 - l_1*m_4 - l_1*m_5 - m_1*r_1)*np.cos(theta_1) - g*(-l_1*m_2 - l_1*m_3 - l_1*m_4 - l_1*m_5 - m_1*r_1)*np.sin(theta_1))*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2))/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2) - (-l_1**2*m_5*r_5*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_5 - theta_1)*np.cos(theta_3 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_3*m_5*r_5*np.cos(theta_5 - theta_3) - (-l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_3 - theta_2))*(-l_1**2*m_5*r_5*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_5 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*m_5*r_5*np.cos(theta_5 - theta_2))/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2))*(dtheta_5**2*l_3*m_5*r_5*np.sin(theta_5 - theta_3) + dtheta_4*mu_4 + dtheta_4*(dtheta_4*l_3*l_4*m_5*np.sin(theta_4 - theta_3) + dtheta_4*l_3*m_4*r_4*np.sin(theta_4 - theta_3)) - dtheta_3*(mu_3 + mu_4) + dtheta_2*mu_3 + dtheta_2*(-dtheta_2*l_2*l_3*m_4*np.sin(theta_3 - theta_2) - dtheta_2*l_2*l_3*m_5*np.sin(theta_3 - theta_2) - dtheta_2*l_2*m_3*r_3*np.sin(theta_3 - theta_2)) + dtheta_1*(-dtheta_1*l_1*l_3*m_4*np.sin(theta_3 - theta_1) - dtheta_1*l_1*l_3*m_5*np.sin(theta_3 - theta_1) - dtheta_1*l_1*m_3*r_3*np.sin(theta_3 - theta_1)) - ddx_c*(-l_3*m_4 - l_3*m_5 - m_3*r_3)*np.cos(theta_3) - g*(-l_3*m_4 - l_3*m_5 - m_3*r_3)*np.sin(theta_3) - l_1*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(dtheta_5**2*l_1*m_5*r_5*np.sin(theta_5 - theta_1) + dtheta_4*(dtheta_4*l_1*l_4*m_5*np.sin(theta_4 - theta_1) + dtheta_4*l_1*m_4*r_4*np.sin(theta_4 - theta_1)) + dtheta_3*(dtheta_3*l_1*l_3*m_4*np.sin(theta_3 - theta_1) + dtheta_3*l_1*l_3*m_5*np.sin(theta_3 - theta_1) + dtheta_3*l_1*m_3*r_3*np.sin(theta_3 - theta_1)) + dtheta_2*mu_2 + dtheta_2*(dtheta_2*l_1*l_2*m_3*np.sin(theta_2 - theta_1) + dtheta_2*l_1*l_2*m_4*np.sin(theta_2 - theta_1) + dtheta_2*l_1*l_2*m_5*np.sin(theta_2 - theta_1) + dtheta_2*l_1*m_2*r_2*np.sin(theta_2 - theta_1)) - dtheta_1*(mu_1 + mu_2) - ddx_c*(-l_1*m_2 - l_1*m_3 - l_1*m_4 - l_1*m_5 - m_1*r_1)*np.cos(theta_1) - g*(-l_1*m_2 - l_1*m_3 - l_1*m_4 - l_1*m_5 - m_1*r_1)*np.sin(theta_1))*np.cos(theta_3 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) - (-l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_3 - theta_2))*(dtheta_5**2*l_2*m_5*r_5*np.sin(theta_5 - theta_2) + dtheta_4*(dtheta_4*l_2*l_4*m_5*np.sin(theta_4 - theta_2) + dtheta_4*l_2*m_4*r_4*np.sin(theta_4 - theta_2)) + dtheta_3*mu_3 + dtheta_3*(dtheta_3*l_2*l_3*m_4*np.sin(theta_3 - theta_2) + dtheta_3*l_2*l_3*m_5*np.sin(theta_3 - theta_2) + dtheta_3*l_2*m_3*r_3*np.sin(theta_3 - theta_2)) - dtheta_2*(mu_2 + mu_3) + dtheta_1*mu_2 + dtheta_1*(-dtheta_1*l_1*l_2*m_3*np.sin(theta_2 - theta_1) - dtheta_1*l_1*l_2*m_4*np.sin(theta_2 - theta_1) - dtheta_1*l_1*l_2*m_5*np.sin(theta_2 - theta_1) - dtheta_1*l_1*m_2*r_2*np.sin(theta_2 - theta_1)) - ddx_c*(-l_2*m_3 - l_2*m_4 - l_2*m_5 - m_2*r_2)*np.cos(theta_2) - g*(-l_2*m_3 - l_2*m_4 - l_2*m_5 - m_2*r_2)*np.sin(theta_2) - l_1*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*(dtheta_5**2*l_1*m_5*r_5*np.sin(theta_5 - theta_1) + dtheta_4*(dtheta_4*l_1*l_4*m_5*np.sin(theta_4 - theta_1) + dtheta_4*l_1*m_4*r_4*np.sin(theta_4 - theta_1)) + dtheta_3*(dtheta_3*l_1*l_3*m_4*np.sin(theta_3 - theta_1) + dtheta_3*l_1*l_3*m_5*np.sin(theta_3 - theta_1) + dtheta_3*l_1*m_3*r_3*np.sin(theta_3 - theta_1)) + dtheta_2*mu_2 + dtheta_2*(dtheta_2*l_1*l_2*m_3*np.sin(theta_2 - theta_1) + dtheta_2*l_1*l_2*m_4*np.sin(theta_2 - theta_1) + dtheta_2*l_1*l_2*m_5*np.sin(theta_2 - theta_1) + dtheta_2*l_1*m_2*r_2*np.sin(theta_2 - theta_1)) - dtheta_1*(mu_1 + mu_2) - ddx_c*(-l_1*m_2 - l_1*m_3 - l_1*m_4 - l_1*m_5 - m_1*r_1)*np.cos(theta_1) - g*(-l_1*m_2 - l_1*m_3 - l_1*m_4 - l_1*m_5 - m_1*r_1)*np.sin(theta_1))*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2))/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2))/(I_3 - l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)**2*np.cos(theta_3 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_3**2*m_4 + l_3**2*m_5 + m_3*r_3**2 - (-l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_3 - theta_2))**2/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2)) - (-l_1**2*m_5*r_5*(l_4*m_5 + m_4*r_4)*np.cos(theta_5 - theta_1)*np.cos(theta_4 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_4*m_5*r_5*np.cos(theta_5 - theta_4) - (-l_1**2*(l_4*m_5 + m_4*r_4)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_4 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_4*m_5 + m_4*r_4)*np.cos(theta_4 - theta_2))*(-l_1**2*m_5*r_5*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_5 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*m_5*r_5*np.cos(theta_5 - theta_2))/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2) - (-l_1**2*(l_4*m_5 + m_4*r_4)*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_4 - theta_1)*np.cos(theta_3 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_3*(l_4*m_5 + m_4*r_4)*np.cos(theta_4 - theta_3) - (-l_1**2*(l_4*m_5 + m_4*r_4)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_4 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_4*m_5 + m_4*r_4)*np.cos(theta_4 - theta_2))*(-l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_3 - theta_2))/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2))*(-l_1**2*m_5*r_5*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_5 - theta_1)*np.cos(theta_3 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_3*m_5*r_5*np.cos(theta_5 - theta_3) - (-l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_3 - theta_2))*(-l_1**2*m_5*r_5*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_5 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*m_5*r_5*np.cos(theta_5 - theta_2))/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2))/(I_3 - l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)**2*np.cos(theta_3 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_3**2*m_4 + l_3**2*m_5 + m_3*r_3**2 - (-l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_3 - theta_2))**2/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2)))*(dtheta_5**2*l_4*m_5*r_5*np.sin(theta_5 - theta_4) + dtheta_5*mu_5 - dtheta_4*(mu_4 + mu_5) + dtheta_3*mu_4 + dtheta_3*(-dtheta_3*l_3*l_4*m_5*np.sin(theta_4 - theta_3) - dtheta_3*l_3*m_4*r_4*np.sin(theta_4 - theta_3)) + dtheta_2*(-dtheta_2*l_2*l_4*m_5*np.sin(theta_4 - theta_2) - dtheta_2*l_2*m_4*r_4*np.sin(theta_4 - theta_2)) + dtheta_1*(-dtheta_1*l_1*l_4*m_5*np.sin(theta_4 - theta_1) - dtheta_1*l_1*m_4*r_4*np.sin(theta_4 - theta_1)) - ddx_c*(-l_4*m_5 - m_4*r_4)*np.cos(theta_4) - g*(-l_4*m_5 - m_4*r_4)*np.sin(theta_4) - l_1*(l_4*m_5 + m_4*r_4)*(dtheta_5**2*l_1*m_5*r_5*np.sin(theta_5 - theta_1) + dtheta_4*(dtheta_4*l_1*l_4*m_5*np.sin(theta_4 - theta_1) + dtheta_4*l_1*m_4*r_4*np.sin(theta_4 - theta_1)) + dtheta_3*(dtheta_3*l_1*l_3*m_4*np.sin(theta_3 - theta_1) + dtheta_3*l_1*l_3*m_5*np.sin(theta_3 - theta_1) + dtheta_3*l_1*m_3*r_3*np.sin(theta_3 - theta_1)) + dtheta_2*mu_2 + dtheta_2*(dtheta_2*l_1*l_2*m_3*np.sin(theta_2 - theta_1) + dtheta_2*l_1*l_2*m_4*np.sin(theta_2 - theta_1) + dtheta_2*l_1*l_2*m_5*np.sin(theta_2 - theta_1) + dtheta_2*l_1*m_2*r_2*np.sin(theta_2 - theta_1)) - dtheta_1*(mu_1 + mu_2) - ddx_c*(-l_1*m_2 - l_1*m_3 - l_1*m_4 - l_1*m_5 - m_1*r_1)*np.cos(theta_1) - g*(-l_1*m_2 - l_1*m_3 - l_1*m_4 - l_1*m_5 - m_1*r_1)*np.sin(theta_1))*np.cos(theta_4 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) - (-l_1**2*(l_4*m_5 + m_4*r_4)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_4 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_4*m_5 + m_4*r_4)*np.cos(theta_4 - theta_2))*(dtheta_5**2*l_2*m_5*r_5*np.sin(theta_5 - theta_2) + dtheta_4*(dtheta_4*l_2*l_4*m_5*np.sin(theta_4 - theta_2) + dtheta_4*l_2*m_4*r_4*np.sin(theta_4 - theta_2)) + dtheta_3*mu_3 + dtheta_3*(dtheta_3*l_2*l_3*m_4*np.sin(theta_3 - theta_2) + dtheta_3*l_2*l_3*m_5*np.sin(theta_3 - theta_2) + dtheta_3*l_2*m_3*r_3*np.sin(theta_3 - theta_2)) - dtheta_2*(mu_2 + mu_3) + dtheta_1*mu_2 + dtheta_1*(-dtheta_1*l_1*l_2*m_3*np.sin(theta_2 - theta_1) - dtheta_1*l_1*l_2*m_4*np.sin(theta_2 - theta_1) - dtheta_1*l_1*l_2*m_5*np.sin(theta_2 - theta_1) - dtheta_1*l_1*m_2*r_2*np.sin(theta_2 - theta_1)) - ddx_c*(-l_2*m_3 - l_2*m_4 - l_2*m_5 - m_2*r_2)*np.cos(theta_2) - g*(-l_2*m_3 - l_2*m_4 - l_2*m_5 - m_2*r_2)*np.sin(theta_2) - l_1*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*(dtheta_5**2*l_1*m_5*r_5*np.sin(theta_5 - theta_1) + dtheta_4*(dtheta_4*l_1*l_4*m_5*np.sin(theta_4 - theta_1) + dtheta_4*l_1*m_4*r_4*np.sin(theta_4 - theta_1)) + dtheta_3*(dtheta_3*l_1*l_3*m_4*np.sin(theta_3 - theta_1) + dtheta_3*l_1*l_3*m_5*np.sin(theta_3 - theta_1) + dtheta_3*l_1*m_3*r_3*np.sin(theta_3 - theta_1)) + dtheta_2*mu_2 + dtheta_2*(dtheta_2*l_1*l_2*m_3*np.sin(theta_2 - theta_1) + dtheta_2*l_1*l_2*m_4*np.sin(theta_2 - theta_1) + dtheta_2*l_1*l_2*m_5*np.sin(theta_2 - theta_1) + dtheta_2*l_1*m_2*r_2*np.sin(theta_2 - theta_1)) - dtheta_1*(mu_1 + mu_2) - ddx_c*(-l_1*m_2 - l_1*m_3 - l_1*m_4 - l_1*m_5 - m_1*r_1)*np.cos(theta_1) - g*(-l_1*m_2 - l_1*m_3 - l_1*m_4 - l_1*m_5 - m_1*r_1)*np.sin(theta_1))*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2))/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2) - (-l_1**2*(l_4*m_5 + m_4*r_4)*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_4 - theta_1)*np.cos(theta_3 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_3*(l_4*m_5 + m_4*r_4)*np.cos(theta_4 - theta_3) - (-l_1**2*(l_4*m_5 + m_4*r_4)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_4 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_4*m_5 + m_4*r_4)*np.cos(theta_4 - theta_2))*(-l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_3 - theta_2))/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2))*(dtheta_5**2*l_3*m_5*r_5*np.sin(theta_5 - theta_3) + dtheta_4*mu_4 + dtheta_4*(dtheta_4*l_3*l_4*m_5*np.sin(theta_4 - theta_3) + dtheta_4*l_3*m_4*r_4*np.sin(theta_4 - theta_3)) - dtheta_3*(mu_3 + mu_4) + dtheta_2*mu_3 + dtheta_2*(-dtheta_2*l_2*l_3*m_4*np.sin(theta_3 - theta_2) - dtheta_2*l_2*l_3*m_5*np.sin(theta_3 - theta_2) - dtheta_2*l_2*m_3*r_3*np.sin(theta_3 - theta_2)) + dtheta_1*(-dtheta_1*l_1*l_3*m_4*np.sin(theta_3 - theta_1) - dtheta_1*l_1*l_3*m_5*np.sin(theta_3 - theta_1) - dtheta_1*l_1*m_3*r_3*np.sin(theta_3 - theta_1)) - ddx_c*(-l_3*m_4 - l_3*m_5 - m_3*r_3)*np.cos(theta_3) - g*(-l_3*m_4 - l_3*m_5 - m_3*r_3)*np.sin(theta_3) - l_1*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(dtheta_5**2*l_1*m_5*r_5*np.sin(theta_5 - theta_1) + dtheta_4*(dtheta_4*l_1*l_4*m_5*np.sin(theta_4 - theta_1) + dtheta_4*l_1*m_4*r_4*np.sin(theta_4 - theta_1)) + dtheta_3*(dtheta_3*l_1*l_3*m_4*np.sin(theta_3 - theta_1) + dtheta_3*l_1*l_3*m_5*np.sin(theta_3 - theta_1) + dtheta_3*l_1*m_3*r_3*np.sin(theta_3 - theta_1)) + dtheta_2*mu_2 + dtheta_2*(dtheta_2*l_1*l_2*m_3*np.sin(theta_2 - theta_1) + dtheta_2*l_1*l_2*m_4*np.sin(theta_2 - theta_1) + dtheta_2*l_1*l_2*m_5*np.sin(theta_2 - theta_1) + dtheta_2*l_1*m_2*r_2*np.sin(theta_2 - theta_1)) - dtheta_1*(mu_1 + mu_2) - ddx_c*(-l_1*m_2 - l_1*m_3 - l_1*m_4 - l_1*m_5 - m_1*r_1)*np.cos(theta_1) - g*(-l_1*m_2 - l_1*m_3 - l_1*m_4 - l_1*m_5 - m_1*r_1)*np.sin(theta_1))*np.cos(theta_3 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) - (-l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_3 - theta_2))*(dtheta_5**2*l_2*m_5*r_5*np.sin(theta_5 - theta_2) + dtheta_4*(dtheta_4*l_2*l_4*m_5*np.sin(theta_4 - theta_2) + dtheta_4*l_2*m_4*r_4*np.sin(theta_4 - theta_2)) + dtheta_3*mu_3 + dtheta_3*(dtheta_3*l_2*l_3*m_4*np.sin(theta_3 - theta_2) + dtheta_3*l_2*l_3*m_5*np.sin(theta_3 - theta_2) + dtheta_3*l_2*m_3*r_3*np.sin(theta_3 - theta_2)) - dtheta_2*(mu_2 + mu_3) + dtheta_1*mu_2 + dtheta_1*(-dtheta_1*l_1*l_2*m_3*np.sin(theta_2 - theta_1) - dtheta_1*l_1*l_2*m_4*np.sin(theta_2 - theta_1) - dtheta_1*l_1*l_2*m_5*np.sin(theta_2 - theta_1) - dtheta_1*l_1*m_2*r_2*np.sin(theta_2 - theta_1)) - ddx_c*(-l_2*m_3 - l_2*m_4 - l_2*m_5 - m_2*r_2)*np.cos(theta_2) - g*(-l_2*m_3 - l_2*m_4 - l_2*m_5 - m_2*r_2)*np.sin(theta_2) - l_1*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*(dtheta_5**2*l_1*m_5*r_5*np.sin(theta_5 - theta_1) + dtheta_4*(dtheta_4*l_1*l_4*m_5*np.sin(theta_4 - theta_1) + dtheta_4*l_1*m_4*r_4*np.sin(theta_4 - theta_1)) + dtheta_3*(dtheta_3*l_1*l_3*m_4*np.sin(theta_3 - theta_1) + dtheta_3*l_1*l_3*m_5*np.sin(theta_3 - theta_1) + dtheta_3*l_1*m_3*r_3*np.sin(theta_3 - theta_1)) + dtheta_2*mu_2 + dtheta_2*(dtheta_2*l_1*l_2*m_3*np.sin(theta_2 - theta_1) + dtheta_2*l_1*l_2*m_4*np.sin(theta_2 - theta_1) + dtheta_2*l_1*l_2*m_5*np.sin(theta_2 - theta_1) + dtheta_2*l_1*m_2*r_2*np.sin(theta_2 - theta_1)) - dtheta_1*(mu_1 + mu_2) - ddx_c*(-l_1*m_2 - l_1*m_3 - l_1*m_4 - l_1*m_5 - m_1*r_1)*np.cos(theta_1) - g*(-l_1*m_2 - l_1*m_3 - l_1*m_4 - l_1*m_5 - m_1*r_1)*np.sin(theta_1))*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2))/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2))/(I_3 - l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)**2*np.cos(theta_3 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_3**2*m_4 + l_3**2*m_5 + m_3*r_3**2 - (-l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_3 - theta_2))**2/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2)))/(I_4 - l_1**2*(l_4*m_5 + m_4*r_4)**2*np.cos(theta_4 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_4**2*m_5 + m_4*r_4**2 - (-l_1**2*(l_4*m_5 + m_4*r_4)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_4 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_4*m_5 + m_4*r_4)*np.cos(theta_4 - theta_2))**2/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2) - (-l_1**2*(l_4*m_5 + m_4*r_4)*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_4 - theta_1)*np.cos(theta_3 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_3*(l_4*m_5 + m_4*r_4)*np.cos(theta_4 - theta_3) - (-l_1**2*(l_4*m_5 + m_4*r_4)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_4 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_4*m_5 + m_4*r_4)*np.cos(theta_4 - theta_2))*(-l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_3 - theta_2))/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2))**2/(I_3 - l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)**2*np.cos(theta_3 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_3**2*m_4 + l_3**2*m_5 + m_3*r_3**2 - (-l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_3 - theta_2))**2/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2))))/(I_5 - l_1**2*m_5**2*r_5**2*np.cos(theta_5 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + m_5*r_5**2 - (-l_1**2*m_5*r_5*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_5 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*m_5*r_5*np.cos(theta_5 - theta_2))**2/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2) - (-l_1**2*m_5*r_5*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_5 - theta_1)*np.cos(theta_3 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_3*m_5*r_5*np.cos(theta_5 - theta_3) - (-l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_3 - theta_2))*(-l_1**2*m_5*r_5*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_5 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*m_5*r_5*np.cos(theta_5 - theta_2))/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2))**2/(I_3 - l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)**2*np.cos(theta_3 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_3**2*m_4 + l_3**2*m_5 + m_3*r_3**2 - (-l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_3 - theta_2))**2/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2)) - (-l_1**2*m_5*r_5*(l_4*m_5 + m_4*r_4)*np.cos(theta_5 - theta_1)*np.cos(theta_4 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_4*m_5*r_5*np.cos(theta_5 - theta_4) - (-l_1**2*(l_4*m_5 + m_4*r_4)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_4 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_4*m_5 + m_4*r_4)*np.cos(theta_4 - theta_2))*(-l_1**2*m_5*r_5*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_5 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*m_5*r_5*np.cos(theta_5 - theta_2))/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2) - (-l_1**2*(l_4*m_5 + m_4*r_4)*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_4 - theta_1)*np.cos(theta_3 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_3*(l_4*m_5 + m_4*r_4)*np.cos(theta_4 - theta_3) - (-l_1**2*(l_4*m_5 + m_4*r_4)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_4 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_4*m_5 + m_4*r_4)*np.cos(theta_4 - theta_2))*(-l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_3 - theta_2))/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2))*(-l_1**2*m_5*r_5*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_5 - theta_1)*np.cos(theta_3 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_3*m_5*r_5*np.cos(theta_5 - theta_3) - (-l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_3 - theta_2))*(-l_1**2*m_5*r_5*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_5 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*m_5*r_5*np.cos(theta_5 - theta_2))/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2))/(I_3 - l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)**2*np.cos(theta_3 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_3**2*m_4 + l_3**2*m_5 + m_3*r_3**2 - (-l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_3 - theta_2))**2/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2)))**2/(I_4 - l_1**2*(l_4*m_5 + m_4*r_4)**2*np.cos(theta_4 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_4**2*m_5 + m_4*r_4**2 - (-l_1**2*(l_4*m_5 + m_4*r_4)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_4 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_4*m_5 + m_4*r_4)*np.cos(theta_4 - theta_2))**2/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2) - (-l_1**2*(l_4*m_5 + m_4*r_4)*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_4 - theta_1)*np.cos(theta_3 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_3*(l_4*m_5 + m_4*r_4)*np.cos(theta_4 - theta_3) - (-l_1**2*(l_4*m_5 + m_4*r_4)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_4 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_4*m_5 + m_4*r_4)*np.cos(theta_4 - theta_2))*(-l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_3 - theta_2))/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2))**2/(I_3 - l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)**2*np.cos(theta_3 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_3**2*m_4 + l_3**2*m_5 + m_3*r_3**2 - (-l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_3 - theta_2))**2/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2)))))/(I_4 - l_1**2*(l_4*m_5 + m_4*r_4)**2*np.cos(theta_4 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_4**2*m_5 + m_4*r_4**2 - (-l_1**2*(l_4*m_5 + m_4*r_4)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_4 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_4*m_5 + m_4*r_4)*np.cos(theta_4 - theta_2))**2/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2) - (-l_1**2*(l_4*m_5 + m_4*r_4)*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_4 - theta_1)*np.cos(theta_3 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_3*(l_4*m_5 + m_4*r_4)*np.cos(theta_4 - theta_3) - (-l_1**2*(l_4*m_5 + m_4*r_4)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_4 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_4*m_5 + m_4*r_4)*np.cos(theta_4 - theta_2))*(-l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_3 - theta_2))/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2))**2/(I_3 - l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)**2*np.cos(theta_3 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_3**2*m_4 + l_3**2*m_5 + m_3*r_3**2 - (-l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_3 - theta_2))**2/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2))),


                      (-dtheta_5*mu_5 - dtheta_4**2*l_4*m_5*r_5*np.sin(theta_5 - theta_4) + dtheta_4*mu_5 - dtheta_3**2*l_3*m_5*r_5*np.sin(theta_5 - theta_3) - dtheta_2**2*l_2*m_5*r_5*np.sin(theta_5 - theta_2) - dtheta_1**2*l_1*m_5*r_5*np.sin(theta_5 - theta_1) + ddx_c*m_5*r_5*np.cos(theta_5) + g*m_5*r_5*np.sin(theta_5) - l_1*m_5*r_5*(dtheta_5**2*l_1*m_5*r_5*np.sin(theta_5 - theta_1) + dtheta_4*(dtheta_4*l_1*l_4*m_5*np.sin(theta_4 - theta_1) + dtheta_4*l_1*m_4*r_4*np.sin(theta_4 - theta_1)) + dtheta_3*(dtheta_3*l_1*l_3*m_4*np.sin(theta_3 - theta_1) + dtheta_3*l_1*l_3*m_5*np.sin(theta_3 - theta_1) + dtheta_3*l_1*m_3*r_3*np.sin(theta_3 - theta_1)) + dtheta_2*mu_2 + dtheta_2*(dtheta_2*l_1*l_2*m_3*np.sin(theta_2 - theta_1) + dtheta_2*l_1*l_2*m_4*np.sin(theta_2 - theta_1) + dtheta_2*l_1*l_2*m_5*np.sin(theta_2 - theta_1) + dtheta_2*l_1*m_2*r_2*np.sin(theta_2 - theta_1)) - dtheta_1*(mu_1 + mu_2) - ddx_c*(-l_1*m_2 - l_1*m_3 - l_1*m_4 - l_1*m_5 - m_1*r_1)*np.cos(theta_1) - g*(-l_1*m_2 - l_1*m_3 - l_1*m_4 - l_1*m_5 - m_1*r_1)*np.sin(theta_1))*np.cos(theta_5 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) - (-l_1**2*m_5*r_5*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_5 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*m_5*r_5*np.cos(theta_5 - theta_2))*(dtheta_5**2*l_2*m_5*r_5*np.sin(theta_5 - theta_2) + dtheta_4*(dtheta_4*l_2*l_4*m_5*np.sin(theta_4 - theta_2) + dtheta_4*l_2*m_4*r_4*np.sin(theta_4 - theta_2)) + dtheta_3*mu_3 + dtheta_3*(dtheta_3*l_2*l_3*m_4*np.sin(theta_3 - theta_2) + dtheta_3*l_2*l_3*m_5*np.sin(theta_3 - theta_2) + dtheta_3*l_2*m_3*r_3*np.sin(theta_3 - theta_2)) - dtheta_2*(mu_2 + mu_3) + dtheta_1*mu_2 + dtheta_1*(-dtheta_1*l_1*l_2*m_3*np.sin(theta_2 - theta_1) - dtheta_1*l_1*l_2*m_4*np.sin(theta_2 - theta_1) - dtheta_1*l_1*l_2*m_5*np.sin(theta_2 - theta_1) - dtheta_1*l_1*m_2*r_2*np.sin(theta_2 - theta_1)) - ddx_c*(-l_2*m_3 - l_2*m_4 - l_2*m_5 - m_2*r_2)*np.cos(theta_2) - g*(-l_2*m_3 - l_2*m_4 - l_2*m_5 - m_2*r_2)*np.sin(theta_2) - l_1*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*(dtheta_5**2*l_1*m_5*r_5*np.sin(theta_5 - theta_1) + dtheta_4*(dtheta_4*l_1*l_4*m_5*np.sin(theta_4 - theta_1) + dtheta_4*l_1*m_4*r_4*np.sin(theta_4 - theta_1)) + dtheta_3*(dtheta_3*l_1*l_3*m_4*np.sin(theta_3 - theta_1) + dtheta_3*l_1*l_3*m_5*np.sin(theta_3 - theta_1) + dtheta_3*l_1*m_3*r_3*np.sin(theta_3 - theta_1)) + dtheta_2*mu_2 + dtheta_2*(dtheta_2*l_1*l_2*m_3*np.sin(theta_2 - theta_1) + dtheta_2*l_1*l_2*m_4*np.sin(theta_2 - theta_1) + dtheta_2*l_1*l_2*m_5*np.sin(theta_2 - theta_1) + dtheta_2*l_1*m_2*r_2*np.sin(theta_2 - theta_1)) - dtheta_1*(mu_1 + mu_2) - ddx_c*(-l_1*m_2 - l_1*m_3 - l_1*m_4 - l_1*m_5 - m_1*r_1)*np.cos(theta_1) - g*(-l_1*m_2 - l_1*m_3 - l_1*m_4 - l_1*m_5 - m_1*r_1)*np.sin(theta_1))*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2))/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2) - (-l_1**2*m_5*r_5*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_5 - theta_1)*np.cos(theta_3 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_3*m_5*r_5*np.cos(theta_5 - theta_3) - (-l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_3 - theta_2))*(-l_1**2*m_5*r_5*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_5 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*m_5*r_5*np.cos(theta_5 - theta_2))/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2))*(dtheta_5**2*l_3*m_5*r_5*np.sin(theta_5 - theta_3) + dtheta_4*mu_4 + dtheta_4*(dtheta_4*l_3*l_4*m_5*np.sin(theta_4 - theta_3) + dtheta_4*l_3*m_4*r_4*np.sin(theta_4 - theta_3)) - dtheta_3*(mu_3 + mu_4) + dtheta_2*mu_3 + dtheta_2*(-dtheta_2*l_2*l_3*m_4*np.sin(theta_3 - theta_2) - dtheta_2*l_2*l_3*m_5*np.sin(theta_3 - theta_2) - dtheta_2*l_2*m_3*r_3*np.sin(theta_3 - theta_2)) + dtheta_1*(-dtheta_1*l_1*l_3*m_4*np.sin(theta_3 - theta_1) - dtheta_1*l_1*l_3*m_5*np.sin(theta_3 - theta_1) - dtheta_1*l_1*m_3*r_3*np.sin(theta_3 - theta_1)) - ddx_c*(-l_3*m_4 - l_3*m_5 - m_3*r_3)*np.cos(theta_3) - g*(-l_3*m_4 - l_3*m_5 - m_3*r_3)*np.sin(theta_3) - l_1*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(dtheta_5**2*l_1*m_5*r_5*np.sin(theta_5 - theta_1) + dtheta_4*(dtheta_4*l_1*l_4*m_5*np.sin(theta_4 - theta_1) + dtheta_4*l_1*m_4*r_4*np.sin(theta_4 - theta_1)) + dtheta_3*(dtheta_3*l_1*l_3*m_4*np.sin(theta_3 - theta_1) + dtheta_3*l_1*l_3*m_5*np.sin(theta_3 - theta_1) + dtheta_3*l_1*m_3*r_3*np.sin(theta_3 - theta_1)) + dtheta_2*mu_2 + dtheta_2*(dtheta_2*l_1*l_2*m_3*np.sin(theta_2 - theta_1) + dtheta_2*l_1*l_2*m_4*np.sin(theta_2 - theta_1) + dtheta_2*l_1*l_2*m_5*np.sin(theta_2 - theta_1) + dtheta_2*l_1*m_2*r_2*np.sin(theta_2 - theta_1)) - dtheta_1*(mu_1 + mu_2) - ddx_c*(-l_1*m_2 - l_1*m_3 - l_1*m_4 - l_1*m_5 - m_1*r_1)*np.cos(theta_1) - g*(-l_1*m_2 - l_1*m_3 - l_1*m_4 - l_1*m_5 - m_1*r_1)*np.sin(theta_1))*np.cos(theta_3 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) - (-l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_3 - theta_2))*(dtheta_5**2*l_2*m_5*r_5*np.sin(theta_5 - theta_2) + dtheta_4*(dtheta_4*l_2*l_4*m_5*np.sin(theta_4 - theta_2) + dtheta_4*l_2*m_4*r_4*np.sin(theta_4 - theta_2)) + dtheta_3*mu_3 + dtheta_3*(dtheta_3*l_2*l_3*m_4*np.sin(theta_3 - theta_2) + dtheta_3*l_2*l_3*m_5*np.sin(theta_3 - theta_2) + dtheta_3*l_2*m_3*r_3*np.sin(theta_3 - theta_2)) - dtheta_2*(mu_2 + mu_3) + dtheta_1*mu_2 + dtheta_1*(-dtheta_1*l_1*l_2*m_3*np.sin(theta_2 - theta_1) - dtheta_1*l_1*l_2*m_4*np.sin(theta_2 - theta_1) - dtheta_1*l_1*l_2*m_5*np.sin(theta_2 - theta_1) - dtheta_1*l_1*m_2*r_2*np.sin(theta_2 - theta_1)) - ddx_c*(-l_2*m_3 - l_2*m_4 - l_2*m_5 - m_2*r_2)*np.cos(theta_2) - g*(-l_2*m_3 - l_2*m_4 - l_2*m_5 - m_2*r_2)*np.sin(theta_2) - l_1*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*(dtheta_5**2*l_1*m_5*r_5*np.sin(theta_5 - theta_1) + dtheta_4*(dtheta_4*l_1*l_4*m_5*np.sin(theta_4 - theta_1) + dtheta_4*l_1*m_4*r_4*np.sin(theta_4 - theta_1)) + dtheta_3*(dtheta_3*l_1*l_3*m_4*np.sin(theta_3 - theta_1) + dtheta_3*l_1*l_3*m_5*np.sin(theta_3 - theta_1) + dtheta_3*l_1*m_3*r_3*np.sin(theta_3 - theta_1)) + dtheta_2*mu_2 + dtheta_2*(dtheta_2*l_1*l_2*m_3*np.sin(theta_2 - theta_1) + dtheta_2*l_1*l_2*m_4*np.sin(theta_2 - theta_1) + dtheta_2*l_1*l_2*m_5*np.sin(theta_2 - theta_1) + dtheta_2*l_1*m_2*r_2*np.sin(theta_2 - theta_1)) - dtheta_1*(mu_1 + mu_2) - ddx_c*(-l_1*m_2 - l_1*m_3 - l_1*m_4 - l_1*m_5 - m_1*r_1)*np.cos(theta_1) - g*(-l_1*m_2 - l_1*m_3 - l_1*m_4 - l_1*m_5 - m_1*r_1)*np.sin(theta_1))*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2))/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2))/(I_3 - l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)**2*np.cos(theta_3 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_3**2*m_4 + l_3**2*m_5 + m_3*r_3**2 - (-l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_3 - theta_2))**2/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2)) - (-l_1**2*m_5*r_5*(l_4*m_5 + m_4*r_4)*np.cos(theta_5 - theta_1)*np.cos(theta_4 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_4*m_5*r_5*np.cos(theta_5 - theta_4) - (-l_1**2*(l_4*m_5 + m_4*r_4)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_4 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_4*m_5 + m_4*r_4)*np.cos(theta_4 - theta_2))*(-l_1**2*m_5*r_5*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_5 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*m_5*r_5*np.cos(theta_5 - theta_2))/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2) - (-l_1**2*(l_4*m_5 + m_4*r_4)*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_4 - theta_1)*np.cos(theta_3 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_3*(l_4*m_5 + m_4*r_4)*np.cos(theta_4 - theta_3) - (-l_1**2*(l_4*m_5 + m_4*r_4)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_4 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_4*m_5 + m_4*r_4)*np.cos(theta_4 - theta_2))*(-l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_3 - theta_2))/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2))*(-l_1**2*m_5*r_5*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_5 - theta_1)*np.cos(theta_3 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_3*m_5*r_5*np.cos(theta_5 - theta_3) - (-l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_3 - theta_2))*(-l_1**2*m_5*r_5*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_5 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*m_5*r_5*np.cos(theta_5 - theta_2))/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2))/(I_3 - l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)**2*np.cos(theta_3 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_3**2*m_4 + l_3**2*m_5 + m_3*r_3**2 - (-l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_3 - theta_2))**2/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2)))*(dtheta_5**2*l_4*m_5*r_5*np.sin(theta_5 - theta_4) + dtheta_5*mu_5 - dtheta_4*(mu_4 + mu_5) + dtheta_3*mu_4 + dtheta_3*(-dtheta_3*l_3*l_4*m_5*np.sin(theta_4 - theta_3) - dtheta_3*l_3*m_4*r_4*np.sin(theta_4 - theta_3)) + dtheta_2*(-dtheta_2*l_2*l_4*m_5*np.sin(theta_4 - theta_2) - dtheta_2*l_2*m_4*r_4*np.sin(theta_4 - theta_2)) + dtheta_1*(-dtheta_1*l_1*l_4*m_5*np.sin(theta_4 - theta_1) - dtheta_1*l_1*m_4*r_4*np.sin(theta_4 - theta_1)) - ddx_c*(-l_4*m_5 - m_4*r_4)*np.cos(theta_4) - g*(-l_4*m_5 - m_4*r_4)*np.sin(theta_4) - l_1*(l_4*m_5 + m_4*r_4)*(dtheta_5**2*l_1*m_5*r_5*np.sin(theta_5 - theta_1) + dtheta_4*(dtheta_4*l_1*l_4*m_5*np.sin(theta_4 - theta_1) + dtheta_4*l_1*m_4*r_4*np.sin(theta_4 - theta_1)) + dtheta_3*(dtheta_3*l_1*l_3*m_4*np.sin(theta_3 - theta_1) + dtheta_3*l_1*l_3*m_5*np.sin(theta_3 - theta_1) + dtheta_3*l_1*m_3*r_3*np.sin(theta_3 - theta_1)) + dtheta_2*mu_2 + dtheta_2*(dtheta_2*l_1*l_2*m_3*np.sin(theta_2 - theta_1) + dtheta_2*l_1*l_2*m_4*np.sin(theta_2 - theta_1) + dtheta_2*l_1*l_2*m_5*np.sin(theta_2 - theta_1) + dtheta_2*l_1*m_2*r_2*np.sin(theta_2 - theta_1)) - dtheta_1*(mu_1 + mu_2) - ddx_c*(-l_1*m_2 - l_1*m_3 - l_1*m_4 - l_1*m_5 - m_1*r_1)*np.cos(theta_1) - g*(-l_1*m_2 - l_1*m_3 - l_1*m_4 - l_1*m_5 - m_1*r_1)*np.sin(theta_1))*np.cos(theta_4 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) - (-l_1**2*(l_4*m_5 + m_4*r_4)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_4 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_4*m_5 + m_4*r_4)*np.cos(theta_4 - theta_2))*(dtheta_5**2*l_2*m_5*r_5*np.sin(theta_5 - theta_2) + dtheta_4*(dtheta_4*l_2*l_4*m_5*np.sin(theta_4 - theta_2) + dtheta_4*l_2*m_4*r_4*np.sin(theta_4 - theta_2)) + dtheta_3*mu_3 + dtheta_3*(dtheta_3*l_2*l_3*m_4*np.sin(theta_3 - theta_2) + dtheta_3*l_2*l_3*m_5*np.sin(theta_3 - theta_2) + dtheta_3*l_2*m_3*r_3*np.sin(theta_3 - theta_2)) - dtheta_2*(mu_2 + mu_3) + dtheta_1*mu_2 + dtheta_1*(-dtheta_1*l_1*l_2*m_3*np.sin(theta_2 - theta_1) - dtheta_1*l_1*l_2*m_4*np.sin(theta_2 - theta_1) - dtheta_1*l_1*l_2*m_5*np.sin(theta_2 - theta_1) - dtheta_1*l_1*m_2*r_2*np.sin(theta_2 - theta_1)) - ddx_c*(-l_2*m_3 - l_2*m_4 - l_2*m_5 - m_2*r_2)*np.cos(theta_2) - g*(-l_2*m_3 - l_2*m_4 - l_2*m_5 - m_2*r_2)*np.sin(theta_2) - l_1*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*(dtheta_5**2*l_1*m_5*r_5*np.sin(theta_5 - theta_1) + dtheta_4*(dtheta_4*l_1*l_4*m_5*np.sin(theta_4 - theta_1) + dtheta_4*l_1*m_4*r_4*np.sin(theta_4 - theta_1)) + dtheta_3*(dtheta_3*l_1*l_3*m_4*np.sin(theta_3 - theta_1) + dtheta_3*l_1*l_3*m_5*np.sin(theta_3 - theta_1) + dtheta_3*l_1*m_3*r_3*np.sin(theta_3 - theta_1)) + dtheta_2*mu_2 + dtheta_2*(dtheta_2*l_1*l_2*m_3*np.sin(theta_2 - theta_1) + dtheta_2*l_1*l_2*m_4*np.sin(theta_2 - theta_1) + dtheta_2*l_1*l_2*m_5*np.sin(theta_2 - theta_1) + dtheta_2*l_1*m_2*r_2*np.sin(theta_2 - theta_1)) - dtheta_1*(mu_1 + mu_2) - ddx_c*(-l_1*m_2 - l_1*m_3 - l_1*m_4 - l_1*m_5 - m_1*r_1)*np.cos(theta_1) - g*(-l_1*m_2 - l_1*m_3 - l_1*m_4 - l_1*m_5 - m_1*r_1)*np.sin(theta_1))*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2))/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2) - (-l_1**2*(l_4*m_5 + m_4*r_4)*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_4 - theta_1)*np.cos(theta_3 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_3*(l_4*m_5 + m_4*r_4)*np.cos(theta_4 - theta_3) - (-l_1**2*(l_4*m_5 + m_4*r_4)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_4 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_4*m_5 + m_4*r_4)*np.cos(theta_4 - theta_2))*(-l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_3 - theta_2))/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2))*(dtheta_5**2*l_3*m_5*r_5*np.sin(theta_5 - theta_3) + dtheta_4*mu_4 + dtheta_4*(dtheta_4*l_3*l_4*m_5*np.sin(theta_4 - theta_3) + dtheta_4*l_3*m_4*r_4*np.sin(theta_4 - theta_3)) - dtheta_3*(mu_3 + mu_4) + dtheta_2*mu_3 + dtheta_2*(-dtheta_2*l_2*l_3*m_4*np.sin(theta_3 - theta_2) - dtheta_2*l_2*l_3*m_5*np.sin(theta_3 - theta_2) - dtheta_2*l_2*m_3*r_3*np.sin(theta_3 - theta_2)) + dtheta_1*(-dtheta_1*l_1*l_3*m_4*np.sin(theta_3 - theta_1) - dtheta_1*l_1*l_3*m_5*np.sin(theta_3 - theta_1) - dtheta_1*l_1*m_3*r_3*np.sin(theta_3 - theta_1)) - ddx_c*(-l_3*m_4 - l_3*m_5 - m_3*r_3)*np.cos(theta_3) - g*(-l_3*m_4 - l_3*m_5 - m_3*r_3)*np.sin(theta_3) - l_1*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(dtheta_5**2*l_1*m_5*r_5*np.sin(theta_5 - theta_1) + dtheta_4*(dtheta_4*l_1*l_4*m_5*np.sin(theta_4 - theta_1) + dtheta_4*l_1*m_4*r_4*np.sin(theta_4 - theta_1)) + dtheta_3*(dtheta_3*l_1*l_3*m_4*np.sin(theta_3 - theta_1) + dtheta_3*l_1*l_3*m_5*np.sin(theta_3 - theta_1) + dtheta_3*l_1*m_3*r_3*np.sin(theta_3 - theta_1)) + dtheta_2*mu_2 + dtheta_2*(dtheta_2*l_1*l_2*m_3*np.sin(theta_2 - theta_1) + dtheta_2*l_1*l_2*m_4*np.sin(theta_2 - theta_1) + dtheta_2*l_1*l_2*m_5*np.sin(theta_2 - theta_1) + dtheta_2*l_1*m_2*r_2*np.sin(theta_2 - theta_1)) - dtheta_1*(mu_1 + mu_2) - ddx_c*(-l_1*m_2 - l_1*m_3 - l_1*m_4 - l_1*m_5 - m_1*r_1)*np.cos(theta_1) - g*(-l_1*m_2 - l_1*m_3 - l_1*m_4 - l_1*m_5 - m_1*r_1)*np.sin(theta_1))*np.cos(theta_3 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) - (-l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_3 - theta_2))*(dtheta_5**2*l_2*m_5*r_5*np.sin(theta_5 - theta_2) + dtheta_4*(dtheta_4*l_2*l_4*m_5*np.sin(theta_4 - theta_2) + dtheta_4*l_2*m_4*r_4*np.sin(theta_4 - theta_2)) + dtheta_3*mu_3 + dtheta_3*(dtheta_3*l_2*l_3*m_4*np.sin(theta_3 - theta_2) + dtheta_3*l_2*l_3*m_5*np.sin(theta_3 - theta_2) + dtheta_3*l_2*m_3*r_3*np.sin(theta_3 - theta_2)) - dtheta_2*(mu_2 + mu_3) + dtheta_1*mu_2 + dtheta_1*(-dtheta_1*l_1*l_2*m_3*np.sin(theta_2 - theta_1) - dtheta_1*l_1*l_2*m_4*np.sin(theta_2 - theta_1) - dtheta_1*l_1*l_2*m_5*np.sin(theta_2 - theta_1) - dtheta_1*l_1*m_2*r_2*np.sin(theta_2 - theta_1)) - ddx_c*(-l_2*m_3 - l_2*m_4 - l_2*m_5 - m_2*r_2)*np.cos(theta_2) - g*(-l_2*m_3 - l_2*m_4 - l_2*m_5 - m_2*r_2)*np.sin(theta_2) - l_1*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*(dtheta_5**2*l_1*m_5*r_5*np.sin(theta_5 - theta_1) + dtheta_4*(dtheta_4*l_1*l_4*m_5*np.sin(theta_4 - theta_1) + dtheta_4*l_1*m_4*r_4*np.sin(theta_4 - theta_1)) + dtheta_3*(dtheta_3*l_1*l_3*m_4*np.sin(theta_3 - theta_1) + dtheta_3*l_1*l_3*m_5*np.sin(theta_3 - theta_1) + dtheta_3*l_1*m_3*r_3*np.sin(theta_3 - theta_1)) + dtheta_2*mu_2 + dtheta_2*(dtheta_2*l_1*l_2*m_3*np.sin(theta_2 - theta_1) + dtheta_2*l_1*l_2*m_4*np.sin(theta_2 - theta_1) + dtheta_2*l_1*l_2*m_5*np.sin(theta_2 - theta_1) + dtheta_2*l_1*m_2*r_2*np.sin(theta_2 - theta_1)) - dtheta_1*(mu_1 + mu_2) - ddx_c*(-l_1*m_2 - l_1*m_3 - l_1*m_4 - l_1*m_5 - m_1*r_1)*np.cos(theta_1) - g*(-l_1*m_2 - l_1*m_3 - l_1*m_4 - l_1*m_5 - m_1*r_1)*np.sin(theta_1))*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2))/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2))/(I_3 - l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)**2*np.cos(theta_3 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_3**2*m_4 + l_3**2*m_5 + m_3*r_3**2 - (-l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_3 - theta_2))**2/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2)))/(I_4 - l_1**2*(l_4*m_5 + m_4*r_4)**2*np.cos(theta_4 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_4**2*m_5 + m_4*r_4**2 - (-l_1**2*(l_4*m_5 + m_4*r_4)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_4 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_4*m_5 + m_4*r_4)*np.cos(theta_4 - theta_2))**2/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2) - (-l_1**2*(l_4*m_5 + m_4*r_4)*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_4 - theta_1)*np.cos(theta_3 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_3*(l_4*m_5 + m_4*r_4)*np.cos(theta_4 - theta_3) - (-l_1**2*(l_4*m_5 + m_4*r_4)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_4 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_4*m_5 + m_4*r_4)*np.cos(theta_4 - theta_2))*(-l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_3 - theta_2))/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2))**2/(I_3 - l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)**2*np.cos(theta_3 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_3**2*m_4 + l_3**2*m_5 + m_3*r_3**2 - (-l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_3 - theta_2))**2/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2))))/(I_5 - l_1**2*m_5**2*r_5**2*np.cos(theta_5 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + m_5*r_5**2 - (-l_1**2*m_5*r_5*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_5 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*m_5*r_5*np.cos(theta_5 - theta_2))**2/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2) - (-l_1**2*m_5*r_5*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_5 - theta_1)*np.cos(theta_3 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_3*m_5*r_5*np.cos(theta_5 - theta_3) - (-l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_3 - theta_2))*(-l_1**2*m_5*r_5*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_5 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*m_5*r_5*np.cos(theta_5 - theta_2))/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2))**2/(I_3 - l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)**2*np.cos(theta_3 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_3**2*m_4 + l_3**2*m_5 + m_3*r_3**2 - (-l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_3 - theta_2))**2/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2)) - (-l_1**2*m_5*r_5*(l_4*m_5 + m_4*r_4)*np.cos(theta_5 - theta_1)*np.cos(theta_4 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_4*m_5*r_5*np.cos(theta_5 - theta_4) - (-l_1**2*(l_4*m_5 + m_4*r_4)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_4 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_4*m_5 + m_4*r_4)*np.cos(theta_4 - theta_2))*(-l_1**2*m_5*r_5*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_5 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*m_5*r_5*np.cos(theta_5 - theta_2))/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2) - (-l_1**2*(l_4*m_5 + m_4*r_4)*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_4 - theta_1)*np.cos(theta_3 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_3*(l_4*m_5 + m_4*r_4)*np.cos(theta_4 - theta_3) - (-l_1**2*(l_4*m_5 + m_4*r_4)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_4 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_4*m_5 + m_4*r_4)*np.cos(theta_4 - theta_2))*(-l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_3 - theta_2))/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2))*(-l_1**2*m_5*r_5*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_5 - theta_1)*np.cos(theta_3 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_3*m_5*r_5*np.cos(theta_5 - theta_3) - (-l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_3 - theta_2))*(-l_1**2*m_5*r_5*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_5 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*m_5*r_5*np.cos(theta_5 - theta_2))/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2))/(I_3 - l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)**2*np.cos(theta_3 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_3**2*m_4 + l_3**2*m_5 + m_3*r_3**2 - (-l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_3 - theta_2))**2/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2)))**2/(I_4 - l_1**2*(l_4*m_5 + m_4*r_4)**2*np.cos(theta_4 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_4**2*m_5 + m_4*r_4**2 - (-l_1**2*(l_4*m_5 + m_4*r_4)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_4 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_4*m_5 + m_4*r_4)*np.cos(theta_4 - theta_2))**2/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2) - (-l_1**2*(l_4*m_5 + m_4*r_4)*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_4 - theta_1)*np.cos(theta_3 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_3*(l_4*m_5 + m_4*r_4)*np.cos(theta_4 - theta_3) - (-l_1**2*(l_4*m_5 + m_4*r_4)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_4 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_4*m_5 + m_4*r_4)*np.cos(theta_4 - theta_2))*(-l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_3 - theta_2))/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2))**2/(I_3 - l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)**2*np.cos(theta_3 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_3**2*m_4 + l_3**2*m_5 + m_3*r_3**2 - (-l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_3 - theta_2))**2/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2)))),
                      ddx_c
    ])

---

In [7]:
t = time.time()
math_env.math['str_to_sol_func']['ddq_f'](*[1, 2, 3, 4, 5, 6, 7, 8, 9, 0, 1, 9.81 , 0.3  , 0.3  , 0.3  , 0.3  , 0.2  , 0.2  , 0.2  , 0.2  ,
       0.2  , 0.8  , 0.8  , 0.8  , 0.8  , 0.8  , 0.015, 0.015, 0.015,
       0.015, 0.015, 0.011, 0.011, 0.011, 0.011, 0.011]); # CPU + numpy
time.time()-t

0.012139558792114258

In [12]:
# @numba.jit()
# def f(input_):
#     env_loaded.math['str_to_sol_func']['ddq_f'](input_) # does not work with numba or jax.jit: non-compilable code

In [16]:
# f(input_= [1, 2, 3, 4, 5, 6, 7, 8, 9, 0, 1, 9.81 , 0.3  , 0.3  , 0.3  , 0.3  , 0.2  , 0.2  , 0.2  , 0.2  ,
#        0.2  , 0.8  , 0.8  , 0.8  , 0.8  , 0.8  , 0.015, 0.015, 0.015,
#        0.015, 0.015, 0.011, 0.011, 0.011, 0.011, 0.011]) 

# error

---

In [10]:
t = time.time()
robotic_equation_n5_numpy(t=1, y=(1, 2, 3, 4, 1, 11, 22, 33, 44, 55, 1), args=[9.81 , 0.3  , 0.3  , 0.3  , 0.3  , 0.2  , 0.2  , 0.2  , 0.2  ,
       0.2  , 0.8  , 0.8  , 0.8  , 0.8  , 0.8  , 0.015, 0.015, 0.015,
       0.015, 0.015, 0.011, 0.011, 0.011, 0.011, 0.011]);
time.time()-t

0.012526988983154297

In [ ]:
# robotic_equation_n5_numpy_jit(*inputs_np_cpu) 
# error: numba can't compile a function type within a function

#unbearably slow, did not finish after 10 minutes

In [ ]:
# %%time
# robotic_equation_n5_numpy_jit(*inputs_np_cpu)

---

In [11]:
t = time.time()
func_jax(*inputs_jax_cpu).block_until_ready(); # jax cpu (no compiling)
time.time()-t

1.8567452430725098

In [12]:
t = time.time()
func_jax(*inputs_jax_gpu).block_until_ready(); # jax gpu (no compiling)
time.time()-t

23.831520795822144

---

In [13]:
t = time.time()
func_jax_jit(*inputs_jax_cpu).block_until_ready(); # jax cpu + compiling (first call)
time.time()-t

32.565789461135864

In [14]:
t = time.time()
func_jax_jit(*inputs_jax_cpu).block_until_ready(); # jax cpu + compiling (second call)
time.time()-t

0.0003943443298339844

---

In [16]:
t = time.time()
func_jax_jit(*inputs_jax_gpu).block_until_ready(); # jax gpu + compiling (first call)
time.time()-t

11.075194120407104

In [17]:
t = time.time()
func_jax_jit(*inputs_jax_gpu).block_until_ready(); # jax gpu + compiling (second call)
time.time()-t

0.002736806869506836

---

#### numba and the Pure function

In [27]:
# %%time
# robotic_equation_n5_numpy_jit(*inputs_np_cpu)

#unbearably slow, did not finish after 10 minutes

In [ ]:
# %%time
# robotic_equation_n5_numpy_jit(*inputs_np_cpu)

---

So the definite winner is jax, as the compiled code with 1 or 3 ms was way faster both on CPU or GPU than the uncompiled numpy code with 12 ms.
The compilation time is around 40 seconds for the CPU, while the GPU compiling takes around 10 s less.

### Extra Test on small functions

In [8]:
def func(x):
    return x+x**x

def func_(x):
    return x+x**x

@jax.jit
def func_jit(x):
    return x+x**x

In [9]:
gpu = jax.devices("gpu")[0]  # First GPU
cpu = jax.devices("cpu")[0]  # First CPU

inputs_np_cpu  = np.array([31,33,41,32,22,53.1])
inputs_jax_gpu = jnp.array([31,33,41,32,22,53.1], device=gpu)
inputs_jax_cpu = jnp.array([31,33,41,32,22,53.1], device=cpu)

---

In [10]:
t = time.time()
func(inputs_np_cpu)  # regular function with numpy on cpu (first call)
time.time()-t 

7.176399230957031e-05

In [11]:
t = time.time()
func(inputs_np_cpu)  # regular function with numpy on cpu (second call)
time.time()-t

6.079673767089844e-05

In [12]:
t = time.time()
func(inputs_jax_cpu).block_until_ready(); # regular function with jax on cpu (first call)
time.time()-t

0.02817225456237793

In [13]:
t = time.time()
func(inputs_jax_cpu).block_until_ready(); # regular function with jax on cpu (second call)
time.time()-t

0.0011484622955322266

In [14]:
t = time.time()
func_(inputs_jax_gpu).block_until_ready();  # regular function with jax on gpu (first call)
time.time()-t

0.05716300010681152

In [15]:
t = time.time()
func_(inputs_jax_gpu).block_until_ready();  # regular function with jax on gpu (second call)
time.time()-t

0.0027320384979248047

- Jax seems to cache functions and their computation graphs, as calling them the second time without jitting gives faster computation speed.

---

In [16]:
inputs_jax_gpu_ = jnp.array([31,33,41,32,22,53.1], device=gpu)*3.1
inputs_jax_cpu_ = jnp.array([31,33,41,32,22,53.1], device=cpu)*3.1

In [17]:
t = time.time()
func(inputs_jax_cpu_).block_until_ready(); # regular function with jax on cpu (third call with different inputs)
time.time()-t

0.0007443428039550781

In [18]:
t = time.time()
func(inputs_jax_gpu_).block_until_ready(); # regular function with jax on gpu (third call with different inputs)
time.time()-t

0.0026030540466308594

---

In [19]:
t = time.time()
func_jit(inputs_jax_cpu).block_until_ready(); # jax with cpu + first jit call
time.time()-t

0.015270471572875977

In [22]:
t = time.time()
func_jit(inputs_jax_cpu).block_until_ready(); # jax with cpu + second jit call
time.time()-t

0.0008111000061035156

---

In [23]:
t = time.time()
func_jit(inputs_jax_gpu).block_until_ready(); # jax with gpu + first jit call
time.time()-t

0.03332805633544922

In [24]:
t = time.time()
func_jit(inputs_jax_gpu).block_until_ready(); # jax with gpu + second jit call
time.time()-t

0.0020689964294433594

---

- JAX is not built for small and unbatched computations

In [27]:
inputs_np_cpu_10000  = np.array([31,33,41,32,22,53.1]*10000)
inputs_jax_gpu_10000 = jnp.array([31,33,41,32,22,53.1]*10000, device=gpu)
inputs_jax_cpu_10000 = jnp.array([31,33,41,32,22,53.1]*10000, device=cpu)

In [30]:
t = time.time()
func(inputs_np_cpu_10000) # regular function with numpy on cpu, and 10000 inputs
time.time()-t

0.0019044876098632812

In [31]:
t = time.time()
func(inputs_jax_cpu_10000).block_until_ready(); # regular function with jax on cpu, and 10000 inputs
time.time()-t

0.03542184829711914

In [32]:
t = time.time()
func(inputs_jax_gpu_10000).block_until_ready() # regular function with jax on gpu, and 10000 inputs
time.time()-t

0.05952930450439453

---

In [33]:
t = time.time()
func_jit(inputs_jax_cpu_10000).block_until_ready(); # jax with cpu + first jit call
time.time()-t

0.0160372257232666

In [34]:
t = time.time()
func_jit(inputs_jax_cpu_10000).block_until_ready(); # jax with cpu + second jit call
time.time()-t

0.0009236335754394531

---

In [35]:
t = time.time()
func_jit(inputs_jax_gpu_10000).block_until_ready(); # jax with gpu + first jit call
time.time()-t

0.03893256187438965

In [36]:
t = time.time()
func_jit(inputs_jax_gpu_10000).block_until_ready(); # jax with gpu + second jit call
time.time()-t

0.0021507740020751953

---

In [47]:
@jax.jit
def func_jax_jit(t, y, args):
    return robotic_equation_n5(t=t, # this is exactly robotic_equation_n5_numpy, but .np  is replaced with .jnp
                    y=y,
                    args=args)

def func_jax(t, y, args):
    return robotic_equation_n5(t=t, # this is exactly robotic_equation_n5_numpy, but .np  is replaced with .jnp
                    y=y,
                    args=args)

def robotic_equation_n5_numpy(t, y, args):
    theta_1, theta_2, theta_3, theta_4, theta_5, dtheta_1, dtheta_2, dtheta_3, dtheta_4, dtheta_5, ddx_c                           = y
    g, l_1, l_2, l_3, l_4, r_1, r_2, r_3, r_4, r_5, m_1, m_2, m_3, m_4, m_5, mu_1, mu_2, mu_3, mu_4, mu_5, I_1, I_2, I_3, I_4, I_5 = args

    return np.array([dtheta_1,
                      dtheta_2,
                      dtheta_3,
                      dtheta_4,
                      dtheta_5,

                      (dtheta_5**2*l_1*m_5*r_5*np.sin(theta_5 - theta_1) + dtheta_4*(dtheta_4*l_1*l_4*m_5*np.sin(theta_4 - theta_1) + dtheta_4*l_1*m_4*r_4*np.sin(theta_4 - theta_1)) + dtheta_3*(dtheta_3*l_1*l_3*m_4*np.sin(theta_3 - theta_1) + dtheta_3*l_1*l_3*m_5*np.sin(theta_3 - theta_1) + dtheta_3*l_1*m_3*r_3*np.sin(theta_3 - theta_1)) + dtheta_2*mu_2 + dtheta_2*(dtheta_2*l_1*l_2*m_3*np.sin(theta_2 - theta_1) + dtheta_2*l_1*l_2*m_4*np.sin(theta_2 - theta_1) + dtheta_2*l_1*l_2*m_5*np.sin(theta_2 - theta_1) + dtheta_2*l_1*m_2*r_2*np.sin(theta_2 - theta_1)) - dtheta_1*(mu_1 + mu_2) - ddx_c*(-l_1*m_2 - l_1*m_3 - l_1*m_4 - l_1*m_5 - m_1*r_1)*np.cos(theta_1) - g*(-l_1*m_2 - l_1*m_3 - l_1*m_4 - l_1*m_5 - m_1*r_1)*np.sin(theta_1) - l_1*m_5*r_5*(-dtheta_5*mu_5 - dtheta_4**2*l_4*m_5*r_5*np.sin(theta_5 - theta_4) + dtheta_4*mu_5 - dtheta_3**2*l_3*m_5*r_5*np.sin(theta_5 - theta_3) - dtheta_2**2*l_2*m_5*r_5*np.sin(theta_5 - theta_2) - dtheta_1**2*l_1*m_5*r_5*np.sin(theta_5 - theta_1) + ddx_c*m_5*r_5*np.cos(theta_5) + g*m_5*r_5*np.sin(theta_5) - l_1*m_5*r_5*(dtheta_5**2*l_1*m_5*r_5*np.sin(theta_5 - theta_1) + dtheta_4*(dtheta_4*l_1*l_4*m_5*np.sin(theta_4 - theta_1) + dtheta_4*l_1*m_4*r_4*np.sin(theta_4 - theta_1)) + dtheta_3*(dtheta_3*l_1*l_3*m_4*np.sin(theta_3 - theta_1) + dtheta_3*l_1*l_3*m_5*np.sin(theta_3 - theta_1) + dtheta_3*l_1*m_3*r_3*np.sin(theta_3 - theta_1)) + dtheta_2*mu_2 + dtheta_2*(dtheta_2*l_1*l_2*m_3*np.sin(theta_2 - theta_1) + dtheta_2*l_1*l_2*m_4*np.sin(theta_2 - theta_1) + dtheta_2*l_1*l_2*m_5*np.sin(theta_2 - theta_1) + dtheta_2*l_1*m_2*r_2*np.sin(theta_2 - theta_1)) - dtheta_1*(mu_1 + mu_2) - ddx_c*(-l_1*m_2 - l_1*m_3 - l_1*m_4 - l_1*m_5 - m_1*r_1)*np.cos(theta_1) - g*(-l_1*m_2 - l_1*m_3 - l_1*m_4 - l_1*m_5 - m_1*r_1)*np.sin(theta_1))*np.cos(theta_5 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) - (-l_1**2*m_5*r_5*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_5 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*m_5*r_5*np.cos(theta_5 - theta_2))*(dtheta_5**2*l_2*m_5*r_5*np.sin(theta_5 - theta_2) + dtheta_4*(dtheta_4*l_2*l_4*m_5*np.sin(theta_4 - theta_2) + dtheta_4*l_2*m_4*r_4*np.sin(theta_4 - theta_2)) + dtheta_3*mu_3 + dtheta_3*(dtheta_3*l_2*l_3*m_4*np.sin(theta_3 - theta_2) + dtheta_3*l_2*l_3*m_5*np.sin(theta_3 - theta_2) + dtheta_3*l_2*m_3*r_3*np.sin(theta_3 - theta_2)) - dtheta_2*(mu_2 + mu_3) + dtheta_1*mu_2 + dtheta_1*(-dtheta_1*l_1*l_2*m_3*np.sin(theta_2 - theta_1) - dtheta_1*l_1*l_2*m_4*np.sin(theta_2 - theta_1) - dtheta_1*l_1*l_2*m_5*np.sin(theta_2 - theta_1) - dtheta_1*l_1*m_2*r_2*np.sin(theta_2 - theta_1)) - ddx_c*(-l_2*m_3 - l_2*m_4 - l_2*m_5 - m_2*r_2)*np.cos(theta_2) - g*(-l_2*m_3 - l_2*m_4 - l_2*m_5 - m_2*r_2)*np.sin(theta_2) - l_1*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*(dtheta_5**2*l_1*m_5*r_5*np.sin(theta_5 - theta_1) + dtheta_4*(dtheta_4*l_1*l_4*m_5*np.sin(theta_4 - theta_1) + dtheta_4*l_1*m_4*r_4*np.sin(theta_4 - theta_1)) + dtheta_3*(dtheta_3*l_1*l_3*m_4*np.sin(theta_3 - theta_1) + dtheta_3*l_1*l_3*m_5*np.sin(theta_3 - theta_1) + dtheta_3*l_1*m_3*r_3*np.sin(theta_3 - theta_1)) + dtheta_2*mu_2 + dtheta_2*(dtheta_2*l_1*l_2*m_3*np.sin(theta_2 - theta_1) + dtheta_2*l_1*l_2*m_4*np.sin(theta_2 - theta_1) + dtheta_2*l_1*l_2*m_5*np.sin(theta_2 - theta_1) + dtheta_2*l_1*m_2*r_2*np.sin(theta_2 - theta_1)) - dtheta_1*(mu_1 + mu_2) - ddx_c*(-l_1*m_2 - l_1*m_3 - l_1*m_4 - l_1*m_5 - m_1*r_1)*np.cos(theta_1) - g*(-l_1*m_2 - l_1*m_3 - l_1*m_4 - l_1*m_5 - m_1*r_1)*np.sin(theta_1))*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2))/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2) - (-l_1**2*m_5*r_5*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_5 - theta_1)*np.cos(theta_3 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_3*m_5*r_5*np.cos(theta_5 - theta_3) - (-l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_3 - theta_2))*(-l_1**2*m_5*r_5*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_5 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*m_5*r_5*np.cos(theta_5 - theta_2))/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2))*(dtheta_5**2*l_3*m_5*r_5*np.sin(theta_5 - theta_3) + dtheta_4*mu_4 + dtheta_4*(dtheta_4*l_3*l_4*m_5*np.sin(theta_4 - theta_3) + dtheta_4*l_3*m_4*r_4*np.sin(theta_4 - theta_3)) - dtheta_3*(mu_3 + mu_4) + dtheta_2*mu_3 + dtheta_2*(-dtheta_2*l_2*l_3*m_4*np.sin(theta_3 - theta_2) - dtheta_2*l_2*l_3*m_5*np.sin(theta_3 - theta_2) - dtheta_2*l_2*m_3*r_3*np.sin(theta_3 - theta_2)) + dtheta_1*(-dtheta_1*l_1*l_3*m_4*np.sin(theta_3 - theta_1) - dtheta_1*l_1*l_3*m_5*np.sin(theta_3 - theta_1) - dtheta_1*l_1*m_3*r_3*np.sin(theta_3 - theta_1)) - ddx_c*(-l_3*m_4 - l_3*m_5 - m_3*r_3)*np.cos(theta_3) - g*(-l_3*m_4 - l_3*m_5 - m_3*r_3)*np.sin(theta_3) - l_1*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(dtheta_5**2*l_1*m_5*r_5*np.sin(theta_5 - theta_1) + dtheta_4*(dtheta_4*l_1*l_4*m_5*np.sin(theta_4 - theta_1) + dtheta_4*l_1*m_4*r_4*np.sin(theta_4 - theta_1)) + dtheta_3*(dtheta_3*l_1*l_3*m_4*np.sin(theta_3 - theta_1) + dtheta_3*l_1*l_3*m_5*np.sin(theta_3 - theta_1) + dtheta_3*l_1*m_3*r_3*np.sin(theta_3 - theta_1)) + dtheta_2*mu_2 + dtheta_2*(dtheta_2*l_1*l_2*m_3*np.sin(theta_2 - theta_1) + dtheta_2*l_1*l_2*m_4*np.sin(theta_2 - theta_1) + dtheta_2*l_1*l_2*m_5*np.sin(theta_2 - theta_1) + dtheta_2*l_1*m_2*r_2*np.sin(theta_2 - theta_1)) - dtheta_1*(mu_1 + mu_2) - ddx_c*(-l_1*m_2 - l_1*m_3 - l_1*m_4 - l_1*m_5 - m_1*r_1)*np.cos(theta_1) - g*(-l_1*m_2 - l_1*m_3 - l_1*m_4 - l_1*m_5 - m_1*r_1)*np.sin(theta_1))*np.cos(theta_3 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) - (-l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_3 - theta_2))*(dtheta_5**2*l_2*m_5*r_5*np.sin(theta_5 - theta_2) + dtheta_4*(dtheta_4*l_2*l_4*m_5*np.sin(theta_4 - theta_2) + dtheta_4*l_2*m_4*r_4*np.sin(theta_4 - theta_2)) + dtheta_3*mu_3 + dtheta_3*(dtheta_3*l_2*l_3*m_4*np.sin(theta_3 - theta_2) + dtheta_3*l_2*l_3*m_5*np.sin(theta_3 - theta_2) + dtheta_3*l_2*m_3*r_3*np.sin(theta_3 - theta_2)) - dtheta_2*(mu_2 + mu_3) + dtheta_1*mu_2 + dtheta_1*(-dtheta_1*l_1*l_2*m_3*np.sin(theta_2 - theta_1) - dtheta_1*l_1*l_2*m_4*np.sin(theta_2 - theta_1) - dtheta_1*l_1*l_2*m_5*np.sin(theta_2 - theta_1) - dtheta_1*l_1*m_2*r_2*np.sin(theta_2 - theta_1)) - ddx_c*(-l_2*m_3 - l_2*m_4 - l_2*m_5 - m_2*r_2)*np.cos(theta_2) - g*(-l_2*m_3 - l_2*m_4 - l_2*m_5 - m_2*r_2)*np.sin(theta_2) - l_1*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*(dtheta_5**2*l_1*m_5*r_5*np.sin(theta_5 - theta_1) + dtheta_4*(dtheta_4*l_1*l_4*m_5*np.sin(theta_4 - theta_1) + dtheta_4*l_1*m_4*r_4*np.sin(theta_4 - theta_1)) + dtheta_3*(dtheta_3*l_1*l_3*m_4*np.sin(theta_3 - theta_1) + dtheta_3*l_1*l_3*m_5*np.sin(theta_3 - theta_1) + dtheta_3*l_1*m_3*r_3*np.sin(theta_3 - theta_1)) + dtheta_2*mu_2 + dtheta_2*(dtheta_2*l_1*l_2*m_3*np.sin(theta_2 - theta_1) + dtheta_2*l_1*l_2*m_4*np.sin(theta_2 - theta_1) + dtheta_2*l_1*l_2*m_5*np.sin(theta_2 - theta_1) + dtheta_2*l_1*m_2*r_2*np.sin(theta_2 - theta_1)) - dtheta_1*(mu_1 + mu_2) - ddx_c*(-l_1*m_2 - l_1*m_3 - l_1*m_4 - l_1*m_5 - m_1*r_1)*np.cos(theta_1) - g*(-l_1*m_2 - l_1*m_3 - l_1*m_4 - l_1*m_5 - m_1*r_1)*np.sin(theta_1))*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2))/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2))/(I_3 - l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)**2*np.cos(theta_3 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_3**2*m_4 + l_3**2*m_5 + m_3*r_3**2 - (-l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_3 - theta_2))**2/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2)) - (-l_1**2*m_5*r_5*(l_4*m_5 + m_4*r_4)*np.cos(theta_5 - theta_1)*np.cos(theta_4 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_4*m_5*r_5*np.cos(theta_5 - theta_4) - (-l_1**2*(l_4*m_5 + m_4*r_4)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_4 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_4*m_5 + m_4*r_4)*np.cos(theta_4 - theta_2))*(-l_1**2*m_5*r_5*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_5 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*m_5*r_5*np.cos(theta_5 - theta_2))/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2) - (-l_1**2*(l_4*m_5 + m_4*r_4)*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_4 - theta_1)*np.cos(theta_3 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_3*(l_4*m_5 + m_4*r_4)*np.cos(theta_4 - theta_3) - (-l_1**2*(l_4*m_5 + m_4*r_4)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_4 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_4*m_5 + m_4*r_4)*np.cos(theta_4 - theta_2))*(-l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_3 - theta_2))/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2))*(-l_1**2*m_5*r_5*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_5 - theta_1)*np.cos(theta_3 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_3*m_5*r_5*np.cos(theta_5 - theta_3) - (-l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_3 - theta_2))*(-l_1**2*m_5*r_5*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_5 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*m_5*r_5*np.cos(theta_5 - theta_2))/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2))/(I_3 - l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)**2*np.cos(theta_3 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_3**2*m_4 + l_3**2*m_5 + m_3*r_3**2 - (-l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_3 - theta_2))**2/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2)))*(dtheta_5**2*l_4*m_5*r_5*np.sin(theta_5 - theta_4) + dtheta_5*mu_5 - dtheta_4*(mu_4 + mu_5) + dtheta_3*mu_4 + dtheta_3*(-dtheta_3*l_3*l_4*m_5*np.sin(theta_4 - theta_3) - dtheta_3*l_3*m_4*r_4*np.sin(theta_4 - theta_3)) + dtheta_2*(-dtheta_2*l_2*l_4*m_5*np.sin(theta_4 - theta_2) - dtheta_2*l_2*m_4*r_4*np.sin(theta_4 - theta_2)) + dtheta_1*(-dtheta_1*l_1*l_4*m_5*np.sin(theta_4 - theta_1) - dtheta_1*l_1*m_4*r_4*np.sin(theta_4 - theta_1)) - ddx_c*(-l_4*m_5 - m_4*r_4)*np.cos(theta_4) - g*(-l_4*m_5 - m_4*r_4)*np.sin(theta_4) - l_1*(l_4*m_5 + m_4*r_4)*(dtheta_5**2*l_1*m_5*r_5*np.sin(theta_5 - theta_1) + dtheta_4*(dtheta_4*l_1*l_4*m_5*np.sin(theta_4 - theta_1) + dtheta_4*l_1*m_4*r_4*np.sin(theta_4 - theta_1)) + dtheta_3*(dtheta_3*l_1*l_3*m_4*np.sin(theta_3 - theta_1) + dtheta_3*l_1*l_3*m_5*np.sin(theta_3 - theta_1) + dtheta_3*l_1*m_3*r_3*np.sin(theta_3 - theta_1)) + dtheta_2*mu_2 + dtheta_2*(dtheta_2*l_1*l_2*m_3*np.sin(theta_2 - theta_1) + dtheta_2*l_1*l_2*m_4*np.sin(theta_2 - theta_1) + dtheta_2*l_1*l_2*m_5*np.sin(theta_2 - theta_1) + dtheta_2*l_1*m_2*r_2*np.sin(theta_2 - theta_1)) - dtheta_1*(mu_1 + mu_2) - ddx_c*(-l_1*m_2 - l_1*m_3 - l_1*m_4 - l_1*m_5 - m_1*r_1)*np.cos(theta_1) - g*(-l_1*m_2 - l_1*m_3 - l_1*m_4 - l_1*m_5 - m_1*r_1)*np.sin(theta_1))*np.cos(theta_4 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) - (-l_1**2*(l_4*m_5 + m_4*r_4)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_4 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_4*m_5 + m_4*r_4)*np.cos(theta_4 - theta_2))*(dtheta_5**2*l_2*m_5*r_5*np.sin(theta_5 - theta_2) + dtheta_4*(dtheta_4*l_2*l_4*m_5*np.sin(theta_4 - theta_2) + dtheta_4*l_2*m_4*r_4*np.sin(theta_4 - theta_2)) + dtheta_3*mu_3 + dtheta_3*(dtheta_3*l_2*l_3*m_4*np.sin(theta_3 - theta_2) + dtheta_3*l_2*l_3*m_5*np.sin(theta_3 - theta_2) + dtheta_3*l_2*m_3*r_3*np.sin(theta_3 - theta_2)) - dtheta_2*(mu_2 + mu_3) + dtheta_1*mu_2 + dtheta_1*(-dtheta_1*l_1*l_2*m_3*np.sin(theta_2 - theta_1) - dtheta_1*l_1*l_2*m_4*np.sin(theta_2 - theta_1) - dtheta_1*l_1*l_2*m_5*np.sin(theta_2 - theta_1) - dtheta_1*l_1*m_2*r_2*np.sin(theta_2 - theta_1)) - ddx_c*(-l_2*m_3 - l_2*m_4 - l_2*m_5 - m_2*r_2)*np.cos(theta_2) - g*(-l_2*m_3 - l_2*m_4 - l_2*m_5 - m_2*r_2)*np.sin(theta_2) - l_1*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*(dtheta_5**2*l_1*m_5*r_5*np.sin(theta_5 - theta_1) + dtheta_4*(dtheta_4*l_1*l_4*m_5*np.sin(theta_4 - theta_1) + dtheta_4*l_1*m_4*r_4*np.sin(theta_4 - theta_1)) + dtheta_3*(dtheta_3*l_1*l_3*m_4*np.sin(theta_3 - theta_1) + dtheta_3*l_1*l_3*m_5*np.sin(theta_3 - theta_1) + dtheta_3*l_1*m_3*r_3*np.sin(theta_3 - theta_1)) + dtheta_2*mu_2 + dtheta_2*(dtheta_2*l_1*l_2*m_3*np.sin(theta_2 - theta_1) + dtheta_2*l_1*l_2*m_4*np.sin(theta_2 - theta_1) + dtheta_2*l_1*l_2*m_5*np.sin(theta_2 - theta_1) + dtheta_2*l_1*m_2*r_2*np.sin(theta_2 - theta_1)) - dtheta_1*(mu_1 + mu_2) - ddx_c*(-l_1*m_2 - l_1*m_3 - l_1*m_4 - l_1*m_5 - m_1*r_1)*np.cos(theta_1) - g*(-l_1*m_2 - l_1*m_3 - l_1*m_4 - l_1*m_5 - m_1*r_1)*np.sin(theta_1))*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2))/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2) - (-l_1**2*(l_4*m_5 + m_4*r_4)*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_4 - theta_1)*np.cos(theta_3 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_3*(l_4*m_5 + m_4*r_4)*np.cos(theta_4 - theta_3) - (-l_1**2*(l_4*m_5 + m_4*r_4)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_4 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_4*m_5 + m_4*r_4)*np.cos(theta_4 - theta_2))*(-l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_3 - theta_2))/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2))*(dtheta_5**2*l_3*m_5*r_5*np.sin(theta_5 - theta_3) + dtheta_4*mu_4 + dtheta_4*(dtheta_4*l_3*l_4*m_5*np.sin(theta_4 - theta_3) + dtheta_4*l_3*m_4*r_4*np.sin(theta_4 - theta_3)) - dtheta_3*(mu_3 + mu_4) + dtheta_2*mu_3 + dtheta_2*(-dtheta_2*l_2*l_3*m_4*np.sin(theta_3 - theta_2) - dtheta_2*l_2*l_3*m_5*np.sin(theta_3 - theta_2) - dtheta_2*l_2*m_3*r_3*np.sin(theta_3 - theta_2)) + dtheta_1*(-dtheta_1*l_1*l_3*m_4*np.sin(theta_3 - theta_1) - dtheta_1*l_1*l_3*m_5*np.sin(theta_3 - theta_1) - dtheta_1*l_1*m_3*r_3*np.sin(theta_3 - theta_1)) - ddx_c*(-l_3*m_4 - l_3*m_5 - m_3*r_3)*np.cos(theta_3) - g*(-l_3*m_4 - l_3*m_5 - m_3*r_3)*np.sin(theta_3) - l_1*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(dtheta_5**2*l_1*m_5*r_5*np.sin(theta_5 - theta_1) + dtheta_4*(dtheta_4*l_1*l_4*m_5*np.sin(theta_4 - theta_1) + dtheta_4*l_1*m_4*r_4*np.sin(theta_4 - theta_1)) + dtheta_3*(dtheta_3*l_1*l_3*m_4*np.sin(theta_3 - theta_1) + dtheta_3*l_1*l_3*m_5*np.sin(theta_3 - theta_1) + dtheta_3*l_1*m_3*r_3*np.sin(theta_3 - theta_1)) + dtheta_2*mu_2 + dtheta_2*(dtheta_2*l_1*l_2*m_3*np.sin(theta_2 - theta_1) + dtheta_2*l_1*l_2*m_4*np.sin(theta_2 - theta_1) + dtheta_2*l_1*l_2*m_5*np.sin(theta_2 - theta_1) + dtheta_2*l_1*m_2*r_2*np.sin(theta_2 - theta_1)) - dtheta_1*(mu_1 + mu_2) - ddx_c*(-l_1*m_2 - l_1*m_3 - l_1*m_4 - l_1*m_5 - m_1*r_1)*np.cos(theta_1) - g*(-l_1*m_2 - l_1*m_3 - l_1*m_4 - l_1*m_5 - m_1*r_1)*np.sin(theta_1))*np.cos(theta_3 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) - (-l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_3 - theta_2))*(dtheta_5**2*l_2*m_5*r_5*np.sin(theta_5 - theta_2) + dtheta_4*(dtheta_4*l_2*l_4*m_5*np.sin(theta_4 - theta_2) + dtheta_4*l_2*m_4*r_4*np.sin(theta_4 - theta_2)) + dtheta_3*mu_3 + dtheta_3*(dtheta_3*l_2*l_3*m_4*np.sin(theta_3 - theta_2) + dtheta_3*l_2*l_3*m_5*np.sin(theta_3 - theta_2) + dtheta_3*l_2*m_3*r_3*np.sin(theta_3 - theta_2)) - dtheta_2*(mu_2 + mu_3) + dtheta_1*mu_2 + dtheta_1*(-dtheta_1*l_1*l_2*m_3*np.sin(theta_2 - theta_1) - dtheta_1*l_1*l_2*m_4*np.sin(theta_2 - theta_1) - dtheta_1*l_1*l_2*m_5*np.sin(theta_2 - theta_1) - dtheta_1*l_1*m_2*r_2*np.sin(theta_2 - theta_1)) - ddx_c*(-l_2*m_3 - l_2*m_4 - l_2*m_5 - m_2*r_2)*np.cos(theta_2) - g*(-l_2*m_3 - l_2*m_4 - l_2*m_5 - m_2*r_2)*np.sin(theta_2) - l_1*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*(dtheta_5**2*l_1*m_5*r_5*np.sin(theta_5 - theta_1) + dtheta_4*(dtheta_4*l_1*l_4*m_5*np.sin(theta_4 - theta_1) + dtheta_4*l_1*m_4*r_4*np.sin(theta_4 - theta_1)) + dtheta_3*(dtheta_3*l_1*l_3*m_4*np.sin(theta_3 - theta_1) + dtheta_3*l_1*l_3*m_5*np.sin(theta_3 - theta_1) + dtheta_3*l_1*m_3*r_3*np.sin(theta_3 - theta_1)) + dtheta_2*mu_2 + dtheta_2*(dtheta_2*l_1*l_2*m_3*np.sin(theta_2 - theta_1) + dtheta_2*l_1*l_2*m_4*np.sin(theta_2 - theta_1) + dtheta_2*l_1*l_2*m_5*np.sin(theta_2 - theta_1) + dtheta_2*l_1*m_2*r_2*np.sin(theta_2 - theta_1)) - dtheta_1*(mu_1 + mu_2) - ddx_c*(-l_1*m_2 - l_1*m_3 - l_1*m_4 - l_1*m_5 - m_1*r_1)*np.cos(theta_1) - g*(-l_1*m_2 - l_1*m_3 - l_1*m_4 - l_1*m_5 - m_1*r_1)*np.sin(theta_1))*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2))/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2))/(I_3 - l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)**2*np.cos(theta_3 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_3**2*m_4 + l_3**2*m_5 + m_3*r_3**2 - (-l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_3 - theta_2))**2/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2)))/(I_4 - l_1**2*(l_4*m_5 + m_4*r_4)**2*np.cos(theta_4 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_4**2*m_5 + m_4*r_4**2 - (-l_1**2*(l_4*m_5 + m_4*r_4)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_4 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_4*m_5 + m_4*r_4)*np.cos(theta_4 - theta_2))**2/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2) - (-l_1**2*(l_4*m_5 + m_4*r_4)*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_4 - theta_1)*np.cos(theta_3 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_3*(l_4*m_5 + m_4*r_4)*np.cos(theta_4 - theta_3) - (-l_1**2*(l_4*m_5 + m_4*r_4)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_4 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_4*m_5 + m_4*r_4)*np.cos(theta_4 - theta_2))*(-l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_3 - theta_2))/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2))**2/(I_3 - l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)**2*np.cos(theta_3 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_3**2*m_4 + l_3**2*m_5 + m_3*r_3**2 - (-l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_3 - theta_2))**2/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2))))*np.cos(theta_5 - theta_1)/(I_5 - l_1**2*m_5**2*r_5**2*np.cos(theta_5 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + m_5*r_5**2 - (-l_1**2*m_5*r_5*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_5 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*m_5*r_5*np.cos(theta_5 - theta_2))**2/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2) - (-l_1**2*m_5*r_5*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_5 - theta_1)*np.cos(theta_3 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_3*m_5*r_5*np.cos(theta_5 - theta_3) - (-l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_3 - theta_2))*(-l_1**2*m_5*r_5*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_5 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*m_5*r_5*np.cos(theta_5 - theta_2))/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2))**2/(I_3 - l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)**2*np.cos(theta_3 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_3**2*m_4 + l_3**2*m_5 + m_3*r_3**2 - (-l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_3 - theta_2))**2/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2)) - (-l_1**2*m_5*r_5*(l_4*m_5 + m_4*r_4)*np.cos(theta_5 - theta_1)*np.cos(theta_4 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_4*m_5*r_5*np.cos(theta_5 - theta_4) - (-l_1**2*(l_4*m_5 + m_4*r_4)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_4 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_4*m_5 + m_4*r_4)*np.cos(theta_4 - theta_2))*(-l_1**2*m_5*r_5*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_5 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*m_5*r_5*np.cos(theta_5 - theta_2))/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2) - (-l_1**2*(l_4*m_5 + m_4*r_4)*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_4 - theta_1)*np.cos(theta_3 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_3*(l_4*m_5 + m_4*r_4)*np.cos(theta_4 - theta_3) - (-l_1**2*(l_4*m_5 + m_4*r_4)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_4 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_4*m_5 + m_4*r_4)*np.cos(theta_4 - theta_2))*(-l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_3 - theta_2))/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2))*(-l_1**2*m_5*r_5*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_5 - theta_1)*np.cos(theta_3 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_3*m_5*r_5*np.cos(theta_5 - theta_3) - (-l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_3 - theta_2))*(-l_1**2*m_5*r_5*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_5 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*m_5*r_5*np.cos(theta_5 - theta_2))/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2))/(I_3 - l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)**2*np.cos(theta_3 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_3**2*m_4 + l_3**2*m_5 + m_3*r_3**2 - (-l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_3 - theta_2))**2/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2)))**2/(I_4 - l_1**2*(l_4*m_5 + m_4*r_4)**2*np.cos(theta_4 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_4**2*m_5 + m_4*r_4**2 - (-l_1**2*(l_4*m_5 + m_4*r_4)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_4 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_4*m_5 + m_4*r_4)*np.cos(theta_4 - theta_2))**2/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2) - (-l_1**2*(l_4*m_5 + m_4*r_4)*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_4 - theta_1)*np.cos(theta_3 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_3*(l_4*m_5 + m_4*r_4)*np.cos(theta_4 - theta_3) - (-l_1**2*(l_4*m_5 + m_4*r_4)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_4 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_4*m_5 + m_4*r_4)*np.cos(theta_4 - theta_2))*(-l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_3 - theta_2))/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2))**2/(I_3 - l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)**2*np.cos(theta_3 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_3**2*m_4 + l_3**2*m_5 + m_3*r_3**2 - (-l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_3 - theta_2))**2/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2)))) - l_1*(l_4*m_5 + m_4*r_4)*(dtheta_5**2*l_4*m_5*r_5*np.sin(theta_5 - theta_4) + dtheta_5*mu_5 - dtheta_4*(mu_4 + mu_5) + dtheta_3*mu_4 + dtheta_3*(-dtheta_3*l_3*l_4*m_5*np.sin(theta_4 - theta_3) - dtheta_3*l_3*m_4*r_4*np.sin(theta_4 - theta_3)) + dtheta_2*(-dtheta_2*l_2*l_4*m_5*np.sin(theta_4 - theta_2) - dtheta_2*l_2*m_4*r_4*np.sin(theta_4 - theta_2)) + dtheta_1*(-dtheta_1*l_1*l_4*m_5*np.sin(theta_4 - theta_1) - dtheta_1*l_1*m_4*r_4*np.sin(theta_4 - theta_1)) - ddx_c*(-l_4*m_5 - m_4*r_4)*np.cos(theta_4) - g*(-l_4*m_5 - m_4*r_4)*np.sin(theta_4) - l_1*(l_4*m_5 + m_4*r_4)*(dtheta_5**2*l_1*m_5*r_5*np.sin(theta_5 - theta_1) + dtheta_4*(dtheta_4*l_1*l_4*m_5*np.sin(theta_4 - theta_1) + dtheta_4*l_1*m_4*r_4*np.sin(theta_4 - theta_1)) + dtheta_3*(dtheta_3*l_1*l_3*m_4*np.sin(theta_3 - theta_1) + dtheta_3*l_1*l_3*m_5*np.sin(theta_3 - theta_1) + dtheta_3*l_1*m_3*r_3*np.sin(theta_3 - theta_1)) + dtheta_2*mu_2 + dtheta_2*(dtheta_2*l_1*l_2*m_3*np.sin(theta_2 - theta_1) + dtheta_2*l_1*l_2*m_4*np.sin(theta_2 - theta_1) + dtheta_2*l_1*l_2*m_5*np.sin(theta_2 - theta_1) + dtheta_2*l_1*m_2*r_2*np.sin(theta_2 - theta_1)) - dtheta_1*(mu_1 + mu_2) - ddx_c*(-l_1*m_2 - l_1*m_3 - l_1*m_4 - l_1*m_5 - m_1*r_1)*np.cos(theta_1) - g*(-l_1*m_2 - l_1*m_3 - l_1*m_4 - l_1*m_5 - m_1*r_1)*np.sin(theta_1))*np.cos(theta_4 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) - (-l_1**2*(l_4*m_5 + m_4*r_4)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_4 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_4*m_5 + m_4*r_4)*np.cos(theta_4 - theta_2))*(dtheta_5**2*l_2*m_5*r_5*np.sin(theta_5 - theta_2) + dtheta_4*(dtheta_4*l_2*l_4*m_5*np.sin(theta_4 - theta_2) + dtheta_4*l_2*m_4*r_4*np.sin(theta_4 - theta_2)) + dtheta_3*mu_3 + dtheta_3*(dtheta_3*l_2*l_3*m_4*np.sin(theta_3 - theta_2) + dtheta_3*l_2*l_3*m_5*np.sin(theta_3 - theta_2) + dtheta_3*l_2*m_3*r_3*np.sin(theta_3 - theta_2)) - dtheta_2*(mu_2 + mu_3) + dtheta_1*mu_2 + dtheta_1*(-dtheta_1*l_1*l_2*m_3*np.sin(theta_2 - theta_1) - dtheta_1*l_1*l_2*m_4*np.sin(theta_2 - theta_1) - dtheta_1*l_1*l_2*m_5*np.sin(theta_2 - theta_1) - dtheta_1*l_1*m_2*r_2*np.sin(theta_2 - theta_1)) - ddx_c*(-l_2*m_3 - l_2*m_4 - l_2*m_5 - m_2*r_2)*np.cos(theta_2) - g*(-l_2*m_3 - l_2*m_4 - l_2*m_5 - m_2*r_2)*np.sin(theta_2) - l_1*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*(dtheta_5**2*l_1*m_5*r_5*np.sin(theta_5 - theta_1) + dtheta_4*(dtheta_4*l_1*l_4*m_5*np.sin(theta_4 - theta_1) + dtheta_4*l_1*m_4*r_4*np.sin(theta_4 - theta_1)) + dtheta_3*(dtheta_3*l_1*l_3*m_4*np.sin(theta_3 - theta_1) + dtheta_3*l_1*l_3*m_5*np.sin(theta_3 - theta_1) + dtheta_3*l_1*m_3*r_3*np.sin(theta_3 - theta_1)) + dtheta_2*mu_2 + dtheta_2*(dtheta_2*l_1*l_2*m_3*np.sin(theta_2 - theta_1) + dtheta_2*l_1*l_2*m_4*np.sin(theta_2 - theta_1) + dtheta_2*l_1*l_2*m_5*np.sin(theta_2 - theta_1) + dtheta_2*l_1*m_2*r_2*np.sin(theta_2 - theta_1)) - dtheta_1*(mu_1 + mu_2) - ddx_c*(-l_1*m_2 - l_1*m_3 - l_1*m_4 - l_1*m_5 - m_1*r_1)*np.cos(theta_1) - g*(-l_1*m_2 - l_1*m_3 - l_1*m_4 - l_1*m_5 - m_1*r_1)*np.sin(theta_1))*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2))/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2) - (-l_1**2*(l_4*m_5 + m_4*r_4)*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_4 - theta_1)*np.cos(theta_3 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_3*(l_4*m_5 + m_4*r_4)*np.cos(theta_4 - theta_3) - (-l_1**2*(l_4*m_5 + m_4*r_4)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_4 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_4*m_5 + m_4*r_4)*np.cos(theta_4 - theta_2))*(-l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_3 - theta_2))/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2))*(dtheta_5**2*l_3*m_5*r_5*np.sin(theta_5 - theta_3) + dtheta_4*mu_4 + dtheta_4*(dtheta_4*l_3*l_4*m_5*np.sin(theta_4 - theta_3) + dtheta_4*l_3*m_4*r_4*np.sin(theta_4 - theta_3)) - dtheta_3*(mu_3 + mu_4) + dtheta_2*mu_3 + dtheta_2*(-dtheta_2*l_2*l_3*m_4*np.sin(theta_3 - theta_2) - dtheta_2*l_2*l_3*m_5*np.sin(theta_3 - theta_2) - dtheta_2*l_2*m_3*r_3*np.sin(theta_3 - theta_2)) + dtheta_1*(-dtheta_1*l_1*l_3*m_4*np.sin(theta_3 - theta_1) - dtheta_1*l_1*l_3*m_5*np.sin(theta_3 - theta_1) - dtheta_1*l_1*m_3*r_3*np.sin(theta_3 - theta_1)) - ddx_c*(-l_3*m_4 - l_3*m_5 - m_3*r_3)*np.cos(theta_3) - g*(-l_3*m_4 - l_3*m_5 - m_3*r_3)*np.sin(theta_3) - l_1*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(dtheta_5**2*l_1*m_5*r_5*np.sin(theta_5 - theta_1) + dtheta_4*(dtheta_4*l_1*l_4*m_5*np.sin(theta_4 - theta_1) + dtheta_4*l_1*m_4*r_4*np.sin(theta_4 - theta_1)) + dtheta_3*(dtheta_3*l_1*l_3*m_4*np.sin(theta_3 - theta_1) + dtheta_3*l_1*l_3*m_5*np.sin(theta_3 - theta_1) + dtheta_3*l_1*m_3*r_3*np.sin(theta_3 - theta_1)) + dtheta_2*mu_2 + dtheta_2*(dtheta_2*l_1*l_2*m_3*np.sin(theta_2 - theta_1) + dtheta_2*l_1*l_2*m_4*np.sin(theta_2 - theta_1) + dtheta_2*l_1*l_2*m_5*np.sin(theta_2 - theta_1) + dtheta_2*l_1*m_2*r_2*np.sin(theta_2 - theta_1)) - dtheta_1*(mu_1 + mu_2) - ddx_c*(-l_1*m_2 - l_1*m_3 - l_1*m_4 - l_1*m_5 - m_1*r_1)*np.cos(theta_1) - g*(-l_1*m_2 - l_1*m_3 - l_1*m_4 - l_1*m_5 - m_1*r_1)*np.sin(theta_1))*np.cos(theta_3 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) - (-l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_3 - theta_2))*(dtheta_5**2*l_2*m_5*r_5*np.sin(theta_5 - theta_2) + dtheta_4*(dtheta_4*l_2*l_4*m_5*np.sin(theta_4 - theta_2) + dtheta_4*l_2*m_4*r_4*np.sin(theta_4 - theta_2)) + dtheta_3*mu_3 + dtheta_3*(dtheta_3*l_2*l_3*m_4*np.sin(theta_3 - theta_2) + dtheta_3*l_2*l_3*m_5*np.sin(theta_3 - theta_2) + dtheta_3*l_2*m_3*r_3*np.sin(theta_3 - theta_2)) - dtheta_2*(mu_2 + mu_3) + dtheta_1*mu_2 + dtheta_1*(-dtheta_1*l_1*l_2*m_3*np.sin(theta_2 - theta_1) - dtheta_1*l_1*l_2*m_4*np.sin(theta_2 - theta_1) - dtheta_1*l_1*l_2*m_5*np.sin(theta_2 - theta_1) - dtheta_1*l_1*m_2*r_2*np.sin(theta_2 - theta_1)) - ddx_c*(-l_2*m_3 - l_2*m_4 - l_2*m_5 - m_2*r_2)*np.cos(theta_2) - g*(-l_2*m_3 - l_2*m_4 - l_2*m_5 - m_2*r_2)*np.sin(theta_2) - l_1*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*(dtheta_5**2*l_1*m_5*r_5*np.sin(theta_5 - theta_1) + dtheta_4*(dtheta_4*l_1*l_4*m_5*np.sin(theta_4 - theta_1) + dtheta_4*l_1*m_4*r_4*np.sin(theta_4 - theta_1)) + dtheta_3*(dtheta_3*l_1*l_3*m_4*np.sin(theta_3 - theta_1) + dtheta_3*l_1*l_3*m_5*np.sin(theta_3 - theta_1) + dtheta_3*l_1*m_3*r_3*np.sin(theta_3 - theta_1)) + dtheta_2*mu_2 + dtheta_2*(dtheta_2*l_1*l_2*m_3*np.sin(theta_2 - theta_1) + dtheta_2*l_1*l_2*m_4*np.sin(theta_2 - theta_1) + dtheta_2*l_1*l_2*m_5*np.sin(theta_2 - theta_1) + dtheta_2*l_1*m_2*r_2*np.sin(theta_2 - theta_1)) - dtheta_1*(mu_1 + mu_2) - ddx_c*(-l_1*m_2 - l_1*m_3 - l_1*m_4 - l_1*m_5 - m_1*r_1)*np.cos(theta_1) - g*(-l_1*m_2 - l_1*m_3 - l_1*m_4 - l_1*m_5 - m_1*r_1)*np.sin(theta_1))*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2))/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2))/(I_3 - l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)**2*np.cos(theta_3 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_3**2*m_4 + l_3**2*m_5 + m_3*r_3**2 - (-l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_3 - theta_2))**2/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2)) - (-l_1**2*m_5*r_5*(l_4*m_5 + m_4*r_4)*np.cos(theta_5 - theta_1)*np.cos(theta_4 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_4*m_5*r_5*np.cos(theta_5 - theta_4) - (-l_1**2*(l_4*m_5 + m_4*r_4)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_4 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_4*m_5 + m_4*r_4)*np.cos(theta_4 - theta_2))*(-l_1**2*m_5*r_5*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_5 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*m_5*r_5*np.cos(theta_5 - theta_2))/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2) - (-l_1**2*(l_4*m_5 + m_4*r_4)*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_4 - theta_1)*np.cos(theta_3 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_3*(l_4*m_5 + m_4*r_4)*np.cos(theta_4 - theta_3) - (-l_1**2*(l_4*m_5 + m_4*r_4)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_4 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_4*m_5 + m_4*r_4)*np.cos(theta_4 - theta_2))*(-l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_3 - theta_2))/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2))*(-l_1**2*m_5*r_5*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_5 - theta_1)*np.cos(theta_3 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_3*m_5*r_5*np.cos(theta_5 - theta_3) - (-l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_3 - theta_2))*(-l_1**2*m_5*r_5*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_5 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*m_5*r_5*np.cos(theta_5 - theta_2))/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2))/(I_3 - l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)**2*np.cos(theta_3 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_3**2*m_4 + l_3**2*m_5 + m_3*r_3**2 - (-l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_3 - theta_2))**2/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2)))*(-dtheta_5*mu_5 - dtheta_4**2*l_4*m_5*r_5*np.sin(theta_5 - theta_4) + dtheta_4*mu_5 - dtheta_3**2*l_3*m_5*r_5*np.sin(theta_5 - theta_3) - dtheta_2**2*l_2*m_5*r_5*np.sin(theta_5 - theta_2) - dtheta_1**2*l_1*m_5*r_5*np.sin(theta_5 - theta_1) + ddx_c*m_5*r_5*np.cos(theta_5) + g*m_5*r_5*np.sin(theta_5) - l_1*m_5*r_5*(dtheta_5**2*l_1*m_5*r_5*np.sin(theta_5 - theta_1) + dtheta_4*(dtheta_4*l_1*l_4*m_5*np.sin(theta_4 - theta_1) + dtheta_4*l_1*m_4*r_4*np.sin(theta_4 - theta_1)) + dtheta_3*(dtheta_3*l_1*l_3*m_4*np.sin(theta_3 - theta_1) + dtheta_3*l_1*l_3*m_5*np.sin(theta_3 - theta_1) + dtheta_3*l_1*m_3*r_3*np.sin(theta_3 - theta_1)) + dtheta_2*mu_2 + dtheta_2*(dtheta_2*l_1*l_2*m_3*np.sin(theta_2 - theta_1) + dtheta_2*l_1*l_2*m_4*np.sin(theta_2 - theta_1) + dtheta_2*l_1*l_2*m_5*np.sin(theta_2 - theta_1) + dtheta_2*l_1*m_2*r_2*np.sin(theta_2 - theta_1)) - dtheta_1*(mu_1 + mu_2) - ddx_c*(-l_1*m_2 - l_1*m_3 - l_1*m_4 - l_1*m_5 - m_1*r_1)*np.cos(theta_1) - g*(-l_1*m_2 - l_1*m_3 - l_1*m_4 - l_1*m_5 - m_1*r_1)*np.sin(theta_1))*np.cos(theta_5 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) - (-l_1**2*m_5*r_5*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_5 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*m_5*r_5*np.cos(theta_5 - theta_2))*(dtheta_5**2*l_2*m_5*r_5*np.sin(theta_5 - theta_2) + dtheta_4*(dtheta_4*l_2*l_4*m_5*np.sin(theta_4 - theta_2) + dtheta_4*l_2*m_4*r_4*np.sin(theta_4 - theta_2)) + dtheta_3*mu_3 + dtheta_3*(dtheta_3*l_2*l_3*m_4*np.sin(theta_3 - theta_2) + dtheta_3*l_2*l_3*m_5*np.sin(theta_3 - theta_2) + dtheta_3*l_2*m_3*r_3*np.sin(theta_3 - theta_2)) - dtheta_2*(mu_2 + mu_3) + dtheta_1*mu_2 + dtheta_1*(-dtheta_1*l_1*l_2*m_3*np.sin(theta_2 - theta_1) - dtheta_1*l_1*l_2*m_4*np.sin(theta_2 - theta_1) - dtheta_1*l_1*l_2*m_5*np.sin(theta_2 - theta_1) - dtheta_1*l_1*m_2*r_2*np.sin(theta_2 - theta_1)) - ddx_c*(-l_2*m_3 - l_2*m_4 - l_2*m_5 - m_2*r_2)*np.cos(theta_2) - g*(-l_2*m_3 - l_2*m_4 - l_2*m_5 - m_2*r_2)*np.sin(theta_2) - l_1*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*(dtheta_5**2*l_1*m_5*r_5*np.sin(theta_5 - theta_1) + dtheta_4*(dtheta_4*l_1*l_4*m_5*np.sin(theta_4 - theta_1) + dtheta_4*l_1*m_4*r_4*np.sin(theta_4 - theta_1)) + dtheta_3*(dtheta_3*l_1*l_3*m_4*np.sin(theta_3 - theta_1) + dtheta_3*l_1*l_3*m_5*np.sin(theta_3 - theta_1) + dtheta_3*l_1*m_3*r_3*np.sin(theta_3 - theta_1)) + dtheta_2*mu_2 + dtheta_2*(dtheta_2*l_1*l_2*m_3*np.sin(theta_2 - theta_1) + dtheta_2*l_1*l_2*m_4*np.sin(theta_2 - theta_1) + dtheta_2*l_1*l_2*m_5*np.sin(theta_2 - theta_1) + dtheta_2*l_1*m_2*r_2*np.sin(theta_2 - theta_1)) - dtheta_1*(mu_1 + mu_2) - ddx_c*(-l_1*m_2 - l_1*m_3 - l_1*m_4 - l_1*m_5 - m_1*r_1)*np.cos(theta_1) - g*(-l_1*m_2 - l_1*m_3 - l_1*m_4 - l_1*m_5 - m_1*r_1)*np.sin(theta_1))*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2))/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2) - (-l_1**2*m_5*r_5*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_5 - theta_1)*np.cos(theta_3 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_3*m_5*r_5*np.cos(theta_5 - theta_3) - (-l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_3 - theta_2))*(-l_1**2*m_5*r_5*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_5 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*m_5*r_5*np.cos(theta_5 - theta_2))/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2))*(dtheta_5**2*l_3*m_5*r_5*np.sin(theta_5 - theta_3) + dtheta_4*mu_4 + dtheta_4*(dtheta_4*l_3*l_4*m_5*np.sin(theta_4 - theta_3) + dtheta_4*l_3*m_4*r_4*np.sin(theta_4 - theta_3)) - dtheta_3*(mu_3 + mu_4) + dtheta_2*mu_3 + dtheta_2*(-dtheta_2*l_2*l_3*m_4*np.sin(theta_3 - theta_2) - dtheta_2*l_2*l_3*m_5*np.sin(theta_3 - theta_2) - dtheta_2*l_2*m_3*r_3*np.sin(theta_3 - theta_2)) + dtheta_1*(-dtheta_1*l_1*l_3*m_4*np.sin(theta_3 - theta_1) - dtheta_1*l_1*l_3*m_5*np.sin(theta_3 - theta_1) - dtheta_1*l_1*m_3*r_3*np.sin(theta_3 - theta_1)) - ddx_c*(-l_3*m_4 - l_3*m_5 - m_3*r_3)*np.cos(theta_3) - g*(-l_3*m_4 - l_3*m_5 - m_3*r_3)*np.sin(theta_3) - l_1*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(dtheta_5**2*l_1*m_5*r_5*np.sin(theta_5 - theta_1) + dtheta_4*(dtheta_4*l_1*l_4*m_5*np.sin(theta_4 - theta_1) + dtheta_4*l_1*m_4*r_4*np.sin(theta_4 - theta_1)) + dtheta_3*(dtheta_3*l_1*l_3*m_4*np.sin(theta_3 - theta_1) + dtheta_3*l_1*l_3*m_5*np.sin(theta_3 - theta_1) + dtheta_3*l_1*m_3*r_3*np.sin(theta_3 - theta_1)) + dtheta_2*mu_2 + dtheta_2*(dtheta_2*l_1*l_2*m_3*np.sin(theta_2 - theta_1) + dtheta_2*l_1*l_2*m_4*np.sin(theta_2 - theta_1) + dtheta_2*l_1*l_2*m_5*np.sin(theta_2 - theta_1) + dtheta_2*l_1*m_2*r_2*np.sin(theta_2 - theta_1)) - dtheta_1*(mu_1 + mu_2) - ddx_c*(-l_1*m_2 - l_1*m_3 - l_1*m_4 - l_1*m_5 - m_1*r_1)*np.cos(theta_1) - g*(-l_1*m_2 - l_1*m_3 - l_1*m_4 - l_1*m_5 - m_1*r_1)*np.sin(theta_1))*np.cos(theta_3 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) - (-l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_3 - theta_2))*(dtheta_5**2*l_2*m_5*r_5*np.sin(theta_5 - theta_2) + dtheta_4*(dtheta_4*l_2*l_4*m_5*np.sin(theta_4 - theta_2) + dtheta_4*l_2*m_4*r_4*np.sin(theta_4 - theta_2)) + dtheta_3*mu_3 + dtheta_3*(dtheta_3*l_2*l_3*m_4*np.sin(theta_3 - theta_2) + dtheta_3*l_2*l_3*m_5*np.sin(theta_3 - theta_2) + dtheta_3*l_2*m_3*r_3*np.sin(theta_3 - theta_2)) - dtheta_2*(mu_2 + mu_3) + dtheta_1*mu_2 + dtheta_1*(-dtheta_1*l_1*l_2*m_3*np.sin(theta_2 - theta_1) - dtheta_1*l_1*l_2*m_4*np.sin(theta_2 - theta_1) - dtheta_1*l_1*l_2*m_5*np.sin(theta_2 - theta_1) - dtheta_1*l_1*m_2*r_2*np.sin(theta_2 - theta_1)) - ddx_c*(-l_2*m_3 - l_2*m_4 - l_2*m_5 - m_2*r_2)*np.cos(theta_2) - g*(-l_2*m_3 - l_2*m_4 - l_2*m_5 - m_2*r_2)*np.sin(theta_2) - l_1*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*(dtheta_5**2*l_1*m_5*r_5*np.sin(theta_5 - theta_1) + dtheta_4*(dtheta_4*l_1*l_4*m_5*np.sin(theta_4 - theta_1) + dtheta_4*l_1*m_4*r_4*np.sin(theta_4 - theta_1)) + dtheta_3*(dtheta_3*l_1*l_3*m_4*np.sin(theta_3 - theta_1) + dtheta_3*l_1*l_3*m_5*np.sin(theta_3 - theta_1) + dtheta_3*l_1*m_3*r_3*np.sin(theta_3 - theta_1)) + dtheta_2*mu_2 + dtheta_2*(dtheta_2*l_1*l_2*m_3*np.sin(theta_2 - theta_1) + dtheta_2*l_1*l_2*m_4*np.sin(theta_2 - theta_1) + dtheta_2*l_1*l_2*m_5*np.sin(theta_2 - theta_1) + dtheta_2*l_1*m_2*r_2*np.sin(theta_2 - theta_1)) - dtheta_1*(mu_1 + mu_2) - ddx_c*(-l_1*m_2 - l_1*m_3 - l_1*m_4 - l_1*m_5 - m_1*r_1)*np.cos(theta_1) - g*(-l_1*m_2 - l_1*m_3 - l_1*m_4 - l_1*m_5 - m_1*r_1)*np.sin(theta_1))*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2))/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2))/(I_3 - l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)**2*np.cos(theta_3 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_3**2*m_4 + l_3**2*m_5 + m_3*r_3**2 - (-l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_3 - theta_2))**2/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2)) - (-l_1**2*m_5*r_5*(l_4*m_5 + m_4*r_4)*np.cos(theta_5 - theta_1)*np.cos(theta_4 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_4*m_5*r_5*np.cos(theta_5 - theta_4) - (-l_1**2*(l_4*m_5 + m_4*r_4)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_4 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_4*m_5 + m_4*r_4)*np.cos(theta_4 - theta_2))*(-l_1**2*m_5*r_5*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_5 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*m_5*r_5*np.cos(theta_5 - theta_2))/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2) - (-l_1**2*(l_4*m_5 + m_4*r_4)*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_4 - theta_1)*np.cos(theta_3 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_3*(l_4*m_5 + m_4*r_4)*np.cos(theta_4 - theta_3) - (-l_1**2*(l_4*m_5 + m_4*r_4)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_4 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_4*m_5 + m_4*r_4)*np.cos(theta_4 - theta_2))*(-l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_3 - theta_2))/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2))*(-l_1**2*m_5*r_5*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_5 - theta_1)*np.cos(theta_3 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_3*m_5*r_5*np.cos(theta_5 - theta_3) - (-l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_3 - theta_2))*(-l_1**2*m_5*r_5*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_5 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*m_5*r_5*np.cos(theta_5 - theta_2))/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2))/(I_3 - l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)**2*np.cos(theta_3 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_3**2*m_4 + l_3**2*m_5 + m_3*r_3**2 - (-l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_3 - theta_2))**2/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2)))*(dtheta_5**2*l_4*m_5*r_5*np.sin(theta_5 - theta_4) + dtheta_5*mu_5 - dtheta_4*(mu_4 + mu_5) + dtheta_3*mu_4 + dtheta_3*(-dtheta_3*l_3*l_4*m_5*np.sin(theta_4 - theta_3) - dtheta_3*l_3*m_4*r_4*np.sin(theta_4 - theta_3)) + dtheta_2*(-dtheta_2*l_2*l_4*m_5*np.sin(theta_4 - theta_2) - dtheta_2*l_2*m_4*r_4*np.sin(theta_4 - theta_2)) + dtheta_1*(-dtheta_1*l_1*l_4*m_5*np.sin(theta_4 - theta_1) - dtheta_1*l_1*m_4*r_4*np.sin(theta_4 - theta_1)) - ddx_c*(-l_4*m_5 - m_4*r_4)*np.cos(theta_4) - g*(-l_4*m_5 - m_4*r_4)*np.sin(theta_4) - l_1*(l_4*m_5 + m_4*r_4)*(dtheta_5**2*l_1*m_5*r_5*np.sin(theta_5 - theta_1) + dtheta_4*(dtheta_4*l_1*l_4*m_5*np.sin(theta_4 - theta_1) + dtheta_4*l_1*m_4*r_4*np.sin(theta_4 - theta_1)) + dtheta_3*(dtheta_3*l_1*l_3*m_4*np.sin(theta_3 - theta_1) + dtheta_3*l_1*l_3*m_5*np.sin(theta_3 - theta_1) + dtheta_3*l_1*m_3*r_3*np.sin(theta_3 - theta_1)) + dtheta_2*mu_2 + dtheta_2*(dtheta_2*l_1*l_2*m_3*np.sin(theta_2 - theta_1) + dtheta_2*l_1*l_2*m_4*np.sin(theta_2 - theta_1) + dtheta_2*l_1*l_2*m_5*np.sin(theta_2 - theta_1) + dtheta_2*l_1*m_2*r_2*np.sin(theta_2 - theta_1)) - dtheta_1*(mu_1 + mu_2) - ddx_c*(-l_1*m_2 - l_1*m_3 - l_1*m_4 - l_1*m_5 - m_1*r_1)*np.cos(theta_1) - g*(-l_1*m_2 - l_1*m_3 - l_1*m_4 - l_1*m_5 - m_1*r_1)*np.sin(theta_1))*np.cos(theta_4 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) - (-l_1**2*(l_4*m_5 + m_4*r_4)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_4 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_4*m_5 + m_4*r_4)*np.cos(theta_4 - theta_2))*(dtheta_5**2*l_2*m_5*r_5*np.sin(theta_5 - theta_2) + dtheta_4*(dtheta_4*l_2*l_4*m_5*np.sin(theta_4 - theta_2) + dtheta_4*l_2*m_4*r_4*np.sin(theta_4 - theta_2)) + dtheta_3*mu_3 + dtheta_3*(dtheta_3*l_2*l_3*m_4*np.sin(theta_3 - theta_2) + dtheta_3*l_2*l_3*m_5*np.sin(theta_3 - theta_2) + dtheta_3*l_2*m_3*r_3*np.sin(theta_3 - theta_2)) - dtheta_2*(mu_2 + mu_3) + dtheta_1*mu_2 + dtheta_1*(-dtheta_1*l_1*l_2*m_3*np.sin(theta_2 - theta_1) - dtheta_1*l_1*l_2*m_4*np.sin(theta_2 - theta_1) - dtheta_1*l_1*l_2*m_5*np.sin(theta_2 - theta_1) - dtheta_1*l_1*m_2*r_2*np.sin(theta_2 - theta_1)) - ddx_c*(-l_2*m_3 - l_2*m_4 - l_2*m_5 - m_2*r_2)*np.cos(theta_2) - g*(-l_2*m_3 - l_2*m_4 - l_2*m_5 - m_2*r_2)*np.sin(theta_2) - l_1*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*(dtheta_5**2*l_1*m_5*r_5*np.sin(theta_5 - theta_1) + dtheta_4*(dtheta_4*l_1*l_4*m_5*np.sin(theta_4 - theta_1) + dtheta_4*l_1*m_4*r_4*np.sin(theta_4 - theta_1)) + dtheta_3*(dtheta_3*l_1*l_3*m_4*np.sin(theta_3 - theta_1) + dtheta_3*l_1*l_3*m_5*np.sin(theta_3 - theta_1) + dtheta_3*l_1*m_3*r_3*np.sin(theta_3 - theta_1)) + dtheta_2*mu_2 + dtheta_2*(dtheta_2*l_1*l_2*m_3*np.sin(theta_2 - theta_1) + dtheta_2*l_1*l_2*m_4*np.sin(theta_2 - theta_1) + dtheta_2*l_1*l_2*m_5*np.sin(theta_2 - theta_1) + dtheta_2*l_1*m_2*r_2*np.sin(theta_2 - theta_1)) - dtheta_1*(mu_1 + mu_2) - ddx_c*(-l_1*m_2 - l_1*m_3 - l_1*m_4 - l_1*m_5 - m_1*r_1)*np.cos(theta_1) - g*(-l_1*m_2 - l_1*m_3 - l_1*m_4 - l_1*m_5 - m_1*r_1)*np.sin(theta_1))*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2))/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2) - (-l_1**2*(l_4*m_5 + m_4*r_4)*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_4 - theta_1)*np.cos(theta_3 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_3*(l_4*m_5 + m_4*r_4)*np.cos(theta_4 - theta_3) - (-l_1**2*(l_4*m_5 + m_4*r_4)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_4 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_4*m_5 + m_4*r_4)*np.cos(theta_4 - theta_2))*(-l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_3 - theta_2))/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2))*(dtheta_5**2*l_3*m_5*r_5*np.sin(theta_5 - theta_3) + dtheta_4*mu_4 + dtheta_4*(dtheta_4*l_3*l_4*m_5*np.sin(theta_4 - theta_3) + dtheta_4*l_3*m_4*r_4*np.sin(theta_4 - theta_3)) - dtheta_3*(mu_3 + mu_4) + dtheta_2*mu_3 + dtheta_2*(-dtheta_2*l_2*l_3*m_4*np.sin(theta_3 - theta_2) - dtheta_2*l_2*l_3*m_5*np.sin(theta_3 - theta_2) - dtheta_2*l_2*m_3*r_3*np.sin(theta_3 - theta_2)) + dtheta_1*(-dtheta_1*l_1*l_3*m_4*np.sin(theta_3 - theta_1) - dtheta_1*l_1*l_3*m_5*np.sin(theta_3 - theta_1) - dtheta_1*l_1*m_3*r_3*np.sin(theta_3 - theta_1)) - ddx_c*(-l_3*m_4 - l_3*m_5 - m_3*r_3)*np.cos(theta_3) - g*(-l_3*m_4 - l_3*m_5 - m_3*r_3)*np.sin(theta_3) - l_1*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(dtheta_5**2*l_1*m_5*r_5*np.sin(theta_5 - theta_1) + dtheta_4*(dtheta_4*l_1*l_4*m_5*np.sin(theta_4 - theta_1) + dtheta_4*l_1*m_4*r_4*np.sin(theta_4 - theta_1)) + dtheta_3*(dtheta_3*l_1*l_3*m_4*np.sin(theta_3 - theta_1) + dtheta_3*l_1*l_3*m_5*np.sin(theta_3 - theta_1) + dtheta_3*l_1*m_3*r_3*np.sin(theta_3 - theta_1)) + dtheta_2*mu_2 + dtheta_2*(dtheta_2*l_1*l_2*m_3*np.sin(theta_2 - theta_1) + dtheta_2*l_1*l_2*m_4*np.sin(theta_2 - theta_1) + dtheta_2*l_1*l_2*m_5*np.sin(theta_2 - theta_1) + dtheta_2*l_1*m_2*r_2*np.sin(theta_2 - theta_1)) - dtheta_1*(mu_1 + mu_2) - ddx_c*(-l_1*m_2 - l_1*m_3 - l_1*m_4 - l_1*m_5 - m_1*r_1)*np.cos(theta_1) - g*(-l_1*m_2 - l_1*m_3 - l_1*m_4 - l_1*m_5 - m_1*r_1)*np.sin(theta_1))*np.cos(theta_3 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) - (-l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_3 - theta_2))*(dtheta_5**2*l_2*m_5*r_5*np.sin(theta_5 - theta_2) + dtheta_4*(dtheta_4*l_2*l_4*m_5*np.sin(theta_4 - theta_2) + dtheta_4*l_2*m_4*r_4*np.sin(theta_4 - theta_2)) + dtheta_3*mu_3 + dtheta_3*(dtheta_3*l_2*l_3*m_4*np.sin(theta_3 - theta_2) + dtheta_3*l_2*l_3*m_5*np.sin(theta_3 - theta_2) + dtheta_3*l_2*m_3*r_3*np.sin(theta_3 - theta_2)) - dtheta_2*(mu_2 + mu_3) + dtheta_1*mu_2 + dtheta_1*(-dtheta_1*l_1*l_2*m_3*np.sin(theta_2 - theta_1) - dtheta_1*l_1*l_2*m_4*np.sin(theta_2 - theta_1) - dtheta_1*l_1*l_2*m_5*np.sin(theta_2 - theta_1) - dtheta_1*l_1*m_2*r_2*np.sin(theta_2 - theta_1)) - ddx_c*(-l_2*m_3 - l_2*m_4 - l_2*m_5 - m_2*r_2)*np.cos(theta_2) - g*(-l_2*m_3 - l_2*m_4 - l_2*m_5 - m_2*r_2)*np.sin(theta_2) - l_1*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*(dtheta_5**2*l_1*m_5*r_5*np.sin(theta_5 - theta_1) + dtheta_4*(dtheta_4*l_1*l_4*m_5*np.sin(theta_4 - theta_1) + dtheta_4*l_1*m_4*r_4*np.sin(theta_4 - theta_1)) + dtheta_3*(dtheta_3*l_1*l_3*m_4*np.sin(theta_3 - theta_1) + dtheta_3*l_1*l_3*m_5*np.sin(theta_3 - theta_1) + dtheta_3*l_1*m_3*r_3*np.sin(theta_3 - theta_1)) + dtheta_2*mu_2 + dtheta_2*(dtheta_2*l_1*l_2*m_3*np.sin(theta_2 - theta_1) + dtheta_2*l_1*l_2*m_4*np.sin(theta_2 - theta_1) + dtheta_2*l_1*l_2*m_5*np.sin(theta_2 - theta_1) + dtheta_2*l_1*m_2*r_2*np.sin(theta_2 - theta_1)) - dtheta_1*(mu_1 + mu_2) - ddx_c*(-l_1*m_2 - l_1*m_3 - l_1*m_4 - l_1*m_5 - m_1*r_1)*np.cos(theta_1) - g*(-l_1*m_2 - l_1*m_3 - l_1*m_4 - l_1*m_5 - m_1*r_1)*np.sin(theta_1))*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2))/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2))/(I_3 - l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)**2*np.cos(theta_3 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_3**2*m_4 + l_3**2*m_5 + m_3*r_3**2 - (-l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_3 - theta_2))**2/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2)))/(I_4 - l_1**2*(l_4*m_5 + m_4*r_4)**2*np.cos(theta_4 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_4**2*m_5 + m_4*r_4**2 - (-l_1**2*(l_4*m_5 + m_4*r_4)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_4 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_4*m_5 + m_4*r_4)*np.cos(theta_4 - theta_2))**2/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2) - (-l_1**2*(l_4*m_5 + m_4*r_4)*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_4 - theta_1)*np.cos(theta_3 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_3*(l_4*m_5 + m_4*r_4)*np.cos(theta_4 - theta_3) - (-l_1**2*(l_4*m_5 + m_4*r_4)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_4 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_4*m_5 + m_4*r_4)*np.cos(theta_4 - theta_2))*(-l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_3 - theta_2))/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2))**2/(I_3 - l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)**2*np.cos(theta_3 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_3**2*m_4 + l_3**2*m_5 + m_3*r_3**2 - (-l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_3 - theta_2))**2/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2))))/(I_5 - l_1**2*m_5**2*r_5**2*np.cos(theta_5 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + m_5*r_5**2 - (-l_1**2*m_5*r_5*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_5 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*m_5*r_5*np.cos(theta_5 - theta_2))**2/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2) - (-l_1**2*m_5*r_5*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_5 - theta_1)*np.cos(theta_3 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_3*m_5*r_5*np.cos(theta_5 - theta_3) - (-l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_3 - theta_2))*(-l_1**2*m_5*r_5*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_5 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*m_5*r_5*np.cos(theta_5 - theta_2))/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2))**2/(I_3 - l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)**2*np.cos(theta_3 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_3**2*m_4 + l_3**2*m_5 + m_3*r_3**2 - (-l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_3 - theta_2))**2/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2)) - (-l_1**2*m_5*r_5*(l_4*m_5 + m_4*r_4)*np.cos(theta_5 - theta_1)*np.cos(theta_4 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_4*m_5*r_5*np.cos(theta_5 - theta_4) - (-l_1**2*(l_4*m_5 + m_4*r_4)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_4 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_4*m_5 + m_4*r_4)*np.cos(theta_4 - theta_2))*(-l_1**2*m_5*r_5*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_5 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*m_5*r_5*np.cos(theta_5 - theta_2))/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2) - (-l_1**2*(l_4*m_5 + m_4*r_4)*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_4 - theta_1)*np.cos(theta_3 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_3*(l_4*m_5 + m_4*r_4)*np.cos(theta_4 - theta_3) - (-l_1**2*(l_4*m_5 + m_4*r_4)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_4 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_4*m_5 + m_4*r_4)*np.cos(theta_4 - theta_2))*(-l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_3 - theta_2))/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2))*(-l_1**2*m_5*r_5*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_5 - theta_1)*np.cos(theta_3 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_3*m_5*r_5*np.cos(theta_5 - theta_3) - (-l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_3 - theta_2))*(-l_1**2*m_5*r_5*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_5 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*m_5*r_5*np.cos(theta_5 - theta_2))/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2))/(I_3 - l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)**2*np.cos(theta_3 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_3**2*m_4 + l_3**2*m_5 + m_3*r_3**2 - (-l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_3 - theta_2))**2/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2)))**2/(I_4 - l_1**2*(l_4*m_5 + m_4*r_4)**2*np.cos(theta_4 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_4**2*m_5 + m_4*r_4**2 - (-l_1**2*(l_4*m_5 + m_4*r_4)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_4 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_4*m_5 + m_4*r_4)*np.cos(theta_4 - theta_2))**2/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2) - (-l_1**2*(l_4*m_5 + m_4*r_4)*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_4 - theta_1)*np.cos(theta_3 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_3*(l_4*m_5 + m_4*r_4)*np.cos(theta_4 - theta_3) - (-l_1**2*(l_4*m_5 + m_4*r_4)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_4 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_4*m_5 + m_4*r_4)*np.cos(theta_4 - theta_2))*(-l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_3 - theta_2))/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2))**2/(I_3 - l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)**2*np.cos(theta_3 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_3**2*m_4 + l_3**2*m_5 + m_3*r_3**2 - (-l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_3 - theta_2))**2/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2)))))*np.cos(theta_4 - theta_1)/(I_4 - l_1**2*(l_4*m_5 + m_4*r_4)**2*np.cos(theta_4 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_4**2*m_5 + m_4*r_4**2 - (-l_1**2*(l_4*m_5 + m_4*r_4)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_4 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_4*m_5 + m_4*r_4)*np.cos(theta_4 - theta_2))**2/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2) - (-l_1**2*(l_4*m_5 + m_4*r_4)*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_4 - theta_1)*np.cos(theta_3 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_3*(l_4*m_5 + m_4*r_4)*np.cos(theta_4 - theta_3) - (-l_1**2*(l_4*m_5 + m_4*r_4)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_4 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_4*m_5 + m_4*r_4)*np.cos(theta_4 - theta_2))*(-l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_3 - theta_2))/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2))**2/(I_3 - l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)**2*np.cos(theta_3 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_3**2*m_4 + l_3**2*m_5 + m_3*r_3**2 - (-l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_3 - theta_2))**2/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2))) - l_1*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(dtheta_5**2*l_3*m_5*r_5*np.sin(theta_5 - theta_3) + dtheta_4*mu_4 + dtheta_4*(dtheta_4*l_3*l_4*m_5*np.sin(theta_4 - theta_3) + dtheta_4*l_3*m_4*r_4*np.sin(theta_4 - theta_3)) - dtheta_3*(mu_3 + mu_4) + dtheta_2*mu_3 + dtheta_2*(-dtheta_2*l_2*l_3*m_4*np.sin(theta_3 - theta_2) - dtheta_2*l_2*l_3*m_5*np.sin(theta_3 - theta_2) - dtheta_2*l_2*m_3*r_3*np.sin(theta_3 - theta_2)) + dtheta_1*(-dtheta_1*l_1*l_3*m_4*np.sin(theta_3 - theta_1) - dtheta_1*l_1*l_3*m_5*np.sin(theta_3 - theta_1) - dtheta_1*l_1*m_3*r_3*np.sin(theta_3 - theta_1)) - ddx_c*(-l_3*m_4 - l_3*m_5 - m_3*r_3)*np.cos(theta_3) - g*(-l_3*m_4 - l_3*m_5 - m_3*r_3)*np.sin(theta_3) - l_1*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(dtheta_5**2*l_1*m_5*r_5*np.sin(theta_5 - theta_1) + dtheta_4*(dtheta_4*l_1*l_4*m_5*np.sin(theta_4 - theta_1) + dtheta_4*l_1*m_4*r_4*np.sin(theta_4 - theta_1)) + dtheta_3*(dtheta_3*l_1*l_3*m_4*np.sin(theta_3 - theta_1) + dtheta_3*l_1*l_3*m_5*np.sin(theta_3 - theta_1) + dtheta_3*l_1*m_3*r_3*np.sin(theta_3 - theta_1)) + dtheta_2*mu_2 + dtheta_2*(dtheta_2*l_1*l_2*m_3*np.sin(theta_2 - theta_1) + dtheta_2*l_1*l_2*m_4*np.sin(theta_2 - theta_1) + dtheta_2*l_1*l_2*m_5*np.sin(theta_2 - theta_1) + dtheta_2*l_1*m_2*r_2*np.sin(theta_2 - theta_1)) - dtheta_1*(mu_1 + mu_2) - ddx_c*(-l_1*m_2 - l_1*m_3 - l_1*m_4 - l_1*m_5 - m_1*r_1)*np.cos(theta_1) - g*(-l_1*m_2 - l_1*m_3 - l_1*m_4 - l_1*m_5 - m_1*r_1)*np.sin(theta_1))*np.cos(theta_3 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) - (-l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_3 - theta_2))*(dtheta_5**2*l_2*m_5*r_5*np.sin(theta_5 - theta_2) + dtheta_4*(dtheta_4*l_2*l_4*m_5*np.sin(theta_4 - theta_2) + dtheta_4*l_2*m_4*r_4*np.sin(theta_4 - theta_2)) + dtheta_3*mu_3 + dtheta_3*(dtheta_3*l_2*l_3*m_4*np.sin(theta_3 - theta_2) + dtheta_3*l_2*l_3*m_5*np.sin(theta_3 - theta_2) + dtheta_3*l_2*m_3*r_3*np.sin(theta_3 - theta_2)) - dtheta_2*(mu_2 + mu_3) + dtheta_1*mu_2 + dtheta_1*(-dtheta_1*l_1*l_2*m_3*np.sin(theta_2 - theta_1) - dtheta_1*l_1*l_2*m_4*np.sin(theta_2 - theta_1) - dtheta_1*l_1*l_2*m_5*np.sin(theta_2 - theta_1) - dtheta_1*l_1*m_2*r_2*np.sin(theta_2 - theta_1)) - ddx_c*(-l_2*m_3 - l_2*m_4 - l_2*m_5 - m_2*r_2)*np.cos(theta_2) - g*(-l_2*m_3 - l_2*m_4 - l_2*m_5 - m_2*r_2)*np.sin(theta_2) - l_1*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*(dtheta_5**2*l_1*m_5*r_5*np.sin(theta_5 - theta_1) + dtheta_4*(dtheta_4*l_1*l_4*m_5*np.sin(theta_4 - theta_1) + dtheta_4*l_1*m_4*r_4*np.sin(theta_4 - theta_1)) + dtheta_3*(dtheta_3*l_1*l_3*m_4*np.sin(theta_3 - theta_1) + dtheta_3*l_1*l_3*m_5*np.sin(theta_3 - theta_1) + dtheta_3*l_1*m_3*r_3*np.sin(theta_3 - theta_1)) + dtheta_2*mu_2 + dtheta_2*(dtheta_2*l_1*l_2*m_3*np.sin(theta_2 - theta_1) + dtheta_2*l_1*l_2*m_4*np.sin(theta_2 - theta_1) + dtheta_2*l_1*l_2*m_5*np.sin(theta_2 - theta_1) + dtheta_2*l_1*m_2*r_2*np.sin(theta_2 - theta_1)) - dtheta_1*(mu_1 + mu_2) - ddx_c*(-l_1*m_2 - l_1*m_3 - l_1*m_4 - l_1*m_5 - m_1*r_1)*np.cos(theta_1) - g*(-l_1*m_2 - l_1*m_3 - l_1*m_4 - l_1*m_5 - m_1*r_1)*np.sin(theta_1))*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2))/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2) - (-l_1**2*(l_4*m_5 + m_4*r_4)*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_4 - theta_1)*np.cos(theta_3 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_3*(l_4*m_5 + m_4*r_4)*np.cos(theta_4 - theta_3) - (-l_1**2*(l_4*m_5 + m_4*r_4)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_4 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_4*m_5 + m_4*r_4)*np.cos(theta_4 - theta_2))*(-l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_3 - theta_2))/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2))*(dtheta_5**2*l_4*m_5*r_5*np.sin(theta_5 - theta_4) + dtheta_5*mu_5 - dtheta_4*(mu_4 + mu_5) + dtheta_3*mu_4 + dtheta_3*(-dtheta_3*l_3*l_4*m_5*np.sin(theta_4 - theta_3) - dtheta_3*l_3*m_4*r_4*np.sin(theta_4 - theta_3)) + dtheta_2*(-dtheta_2*l_2*l_4*m_5*np.sin(theta_4 - theta_2) - dtheta_2*l_2*m_4*r_4*np.sin(theta_4 - theta_2)) + dtheta_1*(-dtheta_1*l_1*l_4*m_5*np.sin(theta_4 - theta_1) - dtheta_1*l_1*m_4*r_4*np.sin(theta_4 - theta_1)) - ddx_c*(-l_4*m_5 - m_4*r_4)*np.cos(theta_4) - g*(-l_4*m_5 - m_4*r_4)*np.sin(theta_4) - l_1*(l_4*m_5 + m_4*r_4)*(dtheta_5**2*l_1*m_5*r_5*np.sin(theta_5 - theta_1) + dtheta_4*(dtheta_4*l_1*l_4*m_5*np.sin(theta_4 - theta_1) + dtheta_4*l_1*m_4*r_4*np.sin(theta_4 - theta_1)) + dtheta_3*(dtheta_3*l_1*l_3*m_4*np.sin(theta_3 - theta_1) + dtheta_3*l_1*l_3*m_5*np.sin(theta_3 - theta_1) + dtheta_3*l_1*m_3*r_3*np.sin(theta_3 - theta_1)) + dtheta_2*mu_2 + dtheta_2*(dtheta_2*l_1*l_2*m_3*np.sin(theta_2 - theta_1) + dtheta_2*l_1*l_2*m_4*np.sin(theta_2 - theta_1) + dtheta_2*l_1*l_2*m_5*np.sin(theta_2 - theta_1) + dtheta_2*l_1*m_2*r_2*np.sin(theta_2 - theta_1)) - dtheta_1*(mu_1 + mu_2) - ddx_c*(-l_1*m_2 - l_1*m_3 - l_1*m_4 - l_1*m_5 - m_1*r_1)*np.cos(theta_1) - g*(-l_1*m_2 - l_1*m_3 - l_1*m_4 - l_1*m_5 - m_1*r_1)*np.sin(theta_1))*np.cos(theta_4 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) - (-l_1**2*(l_4*m_5 + m_4*r_4)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_4 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_4*m_5 + m_4*r_4)*np.cos(theta_4 - theta_2))*(dtheta_5**2*l_2*m_5*r_5*np.sin(theta_5 - theta_2) + dtheta_4*(dtheta_4*l_2*l_4*m_5*np.sin(theta_4 - theta_2) + dtheta_4*l_2*m_4*r_4*np.sin(theta_4 - theta_2)) + dtheta_3*mu_3 + dtheta_3*(dtheta_3*l_2*l_3*m_4*np.sin(theta_3 - theta_2) + dtheta_3*l_2*l_3*m_5*np.sin(theta_3 - theta_2) + dtheta_3*l_2*m_3*r_3*np.sin(theta_3 - theta_2)) - dtheta_2*(mu_2 + mu_3) + dtheta_1*mu_2 + dtheta_1*(-dtheta_1*l_1*l_2*m_3*np.sin(theta_2 - theta_1) - dtheta_1*l_1*l_2*m_4*np.sin(theta_2 - theta_1) - dtheta_1*l_1*l_2*m_5*np.sin(theta_2 - theta_1) - dtheta_1*l_1*m_2*r_2*np.sin(theta_2 - theta_1)) - ddx_c*(-l_2*m_3 - l_2*m_4 - l_2*m_5 - m_2*r_2)*np.cos(theta_2) - g*(-l_2*m_3 - l_2*m_4 - l_2*m_5 - m_2*r_2)*np.sin(theta_2) - l_1*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*(dtheta_5**2*l_1*m_5*r_5*np.sin(theta_5 - theta_1) + dtheta_4*(dtheta_4*l_1*l_4*m_5*np.sin(theta_4 - theta_1) + dtheta_4*l_1*m_4*r_4*np.sin(theta_4 - theta_1)) + dtheta_3*(dtheta_3*l_1*l_3*m_4*np.sin(theta_3 - theta_1) + dtheta_3*l_1*l_3*m_5*np.sin(theta_3 - theta_1) + dtheta_3*l_1*m_3*r_3*np.sin(theta_3 - theta_1)) + dtheta_2*mu_2 + dtheta_2*(dtheta_2*l_1*l_2*m_3*np.sin(theta_2 - theta_1) + dtheta_2*l_1*l_2*m_4*np.sin(theta_2 - theta_1) + dtheta_2*l_1*l_2*m_5*np.sin(theta_2 - theta_1) + dtheta_2*l_1*m_2*r_2*np.sin(theta_2 - theta_1)) - dtheta_1*(mu_1 + mu_2) - ddx_c*(-l_1*m_2 - l_1*m_3 - l_1*m_4 - l_1*m_5 - m_1*r_1)*np.cos(theta_1) - g*(-l_1*m_2 - l_1*m_3 - l_1*m_4 - l_1*m_5 - m_1*r_1)*np.sin(theta_1))*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2))/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2) - (-l_1**2*(l_4*m_5 + m_4*r_4)*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_4 - theta_1)*np.cos(theta_3 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_3*(l_4*m_5 + m_4*r_4)*np.cos(theta_4 - theta_3) - (-l_1**2*(l_4*m_5 + m_4*r_4)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_4 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_4*m_5 + m_4*r_4)*np.cos(theta_4 - theta_2))*(-l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_3 - theta_2))/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2))*(dtheta_5**2*l_3*m_5*r_5*np.sin(theta_5 - theta_3) + dtheta_4*mu_4 + dtheta_4*(dtheta_4*l_3*l_4*m_5*np.sin(theta_4 - theta_3) + dtheta_4*l_3*m_4*r_4*np.sin(theta_4 - theta_3)) - dtheta_3*(mu_3 + mu_4) + dtheta_2*mu_3 + dtheta_2*(-dtheta_2*l_2*l_3*m_4*np.sin(theta_3 - theta_2) - dtheta_2*l_2*l_3*m_5*np.sin(theta_3 - theta_2) - dtheta_2*l_2*m_3*r_3*np.sin(theta_3 - theta_2)) + dtheta_1*(-dtheta_1*l_1*l_3*m_4*np.sin(theta_3 - theta_1) - dtheta_1*l_1*l_3*m_5*np.sin(theta_3 - theta_1) - dtheta_1*l_1*m_3*r_3*np.sin(theta_3 - theta_1)) - ddx_c*(-l_3*m_4 - l_3*m_5 - m_3*r_3)*np.cos(theta_3) - g*(-l_3*m_4 - l_3*m_5 - m_3*r_3)*np.sin(theta_3) - l_1*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(dtheta_5**2*l_1*m_5*r_5*np.sin(theta_5 - theta_1) + dtheta_4*(dtheta_4*l_1*l_4*m_5*np.sin(theta_4 - theta_1) + dtheta_4*l_1*m_4*r_4*np.sin(theta_4 - theta_1)) + dtheta_3*(dtheta_3*l_1*l_3*m_4*np.sin(theta_3 - theta_1) + dtheta_3*l_1*l_3*m_5*np.sin(theta_3 - theta_1) + dtheta_3*l_1*m_3*r_3*np.sin(theta_3 - theta_1)) + dtheta_2*mu_2 + dtheta_2*(dtheta_2*l_1*l_2*m_3*np.sin(theta_2 - theta_1) + dtheta_2*l_1*l_2*m_4*np.sin(theta_2 - theta_1) + dtheta_2*l_1*l_2*m_5*np.sin(theta_2 - theta_1) + dtheta_2*l_1*m_2*r_2*np.sin(theta_2 - theta_1)) - dtheta_1*(mu_1 + mu_2) - ddx_c*(-l_1*m_2 - l_1*m_3 - l_1*m_4 - l_1*m_5 - m_1*r_1)*np.cos(theta_1) - g*(-l_1*m_2 - l_1*m_3 - l_1*m_4 - l_1*m_5 - m_1*r_1)*np.sin(theta_1))*np.cos(theta_3 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) - (-l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_3 - theta_2))*(dtheta_5**2*l_2*m_5*r_5*np.sin(theta_5 - theta_2) + dtheta_4*(dtheta_4*l_2*l_4*m_5*np.sin(theta_4 - theta_2) + dtheta_4*l_2*m_4*r_4*np.sin(theta_4 - theta_2)) + dtheta_3*mu_3 + dtheta_3*(dtheta_3*l_2*l_3*m_4*np.sin(theta_3 - theta_2) + dtheta_3*l_2*l_3*m_5*np.sin(theta_3 - theta_2) + dtheta_3*l_2*m_3*r_3*np.sin(theta_3 - theta_2)) - dtheta_2*(mu_2 + mu_3) + dtheta_1*mu_2 + dtheta_1*(-dtheta_1*l_1*l_2*m_3*np.sin(theta_2 - theta_1) - dtheta_1*l_1*l_2*m_4*np.sin(theta_2 - theta_1) - dtheta_1*l_1*l_2*m_5*np.sin(theta_2 - theta_1) - dtheta_1*l_1*m_2*r_2*np.sin(theta_2 - theta_1)) - ddx_c*(-l_2*m_3 - l_2*m_4 - l_2*m_5 - m_2*r_2)*np.cos(theta_2) - g*(-l_2*m_3 - l_2*m_4 - l_2*m_5 - m_2*r_2)*np.sin(theta_2) - l_1*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*(dtheta_5**2*l_1*m_5*r_5*np.sin(theta_5 - theta_1) + dtheta_4*(dtheta_4*l_1*l_4*m_5*np.sin(theta_4 - theta_1) + dtheta_4*l_1*m_4*r_4*np.sin(theta_4 - theta_1)) + dtheta_3*(dtheta_3*l_1*l_3*m_4*np.sin(theta_3 - theta_1) + dtheta_3*l_1*l_3*m_5*np.sin(theta_3 - theta_1) + dtheta_3*l_1*m_3*r_3*np.sin(theta_3 - theta_1)) + dtheta_2*mu_2 + dtheta_2*(dtheta_2*l_1*l_2*m_3*np.sin(theta_2 - theta_1) + dtheta_2*l_1*l_2*m_4*np.sin(theta_2 - theta_1) + dtheta_2*l_1*l_2*m_5*np.sin(theta_2 - theta_1) + dtheta_2*l_1*m_2*r_2*np.sin(theta_2 - theta_1)) - dtheta_1*(mu_1 + mu_2) - ddx_c*(-l_1*m_2 - l_1*m_3 - l_1*m_4 - l_1*m_5 - m_1*r_1)*np.cos(theta_1) - g*(-l_1*m_2 - l_1*m_3 - l_1*m_4 - l_1*m_5 - m_1*r_1)*np.sin(theta_1))*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2))/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2))/(I_3 - l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)**2*np.cos(theta_3 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_3**2*m_4 + l_3**2*m_5 + m_3*r_3**2 - (-l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_3 - theta_2))**2/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2)) - (-l_1**2*m_5*r_5*(l_4*m_5 + m_4*r_4)*np.cos(theta_5 - theta_1)*np.cos(theta_4 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_4*m_5*r_5*np.cos(theta_5 - theta_4) - (-l_1**2*(l_4*m_5 + m_4*r_4)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_4 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_4*m_5 + m_4*r_4)*np.cos(theta_4 - theta_2))*(-l_1**2*m_5*r_5*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_5 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*m_5*r_5*np.cos(theta_5 - theta_2))/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2) - (-l_1**2*(l_4*m_5 + m_4*r_4)*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_4 - theta_1)*np.cos(theta_3 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_3*(l_4*m_5 + m_4*r_4)*np.cos(theta_4 - theta_3) - (-l_1**2*(l_4*m_5 + m_4*r_4)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_4 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_4*m_5 + m_4*r_4)*np.cos(theta_4 - theta_2))*(-l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_3 - theta_2))/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2))*(-l_1**2*m_5*r_5*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_5 - theta_1)*np.cos(theta_3 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_3*m_5*r_5*np.cos(theta_5 - theta_3) - (-l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_3 - theta_2))*(-l_1**2*m_5*r_5*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_5 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*m_5*r_5*np.cos(theta_5 - theta_2))/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2))/(I_3 - l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)**2*np.cos(theta_3 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_3**2*m_4 + l_3**2*m_5 + m_3*r_3**2 - (-l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_3 - theta_2))**2/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2)))*(-dtheta_5*mu_5 - dtheta_4**2*l_4*m_5*r_5*np.sin(theta_5 - theta_4) + dtheta_4*mu_5 - dtheta_3**2*l_3*m_5*r_5*np.sin(theta_5 - theta_3) - dtheta_2**2*l_2*m_5*r_5*np.sin(theta_5 - theta_2) - dtheta_1**2*l_1*m_5*r_5*np.sin(theta_5 - theta_1) + ddx_c*m_5*r_5*np.cos(theta_5) + g*m_5*r_5*np.sin(theta_5) - l_1*m_5*r_5*(dtheta_5**2*l_1*m_5*r_5*np.sin(theta_5 - theta_1) + dtheta_4*(dtheta_4*l_1*l_4*m_5*np.sin(theta_4 - theta_1) + dtheta_4*l_1*m_4*r_4*np.sin(theta_4 - theta_1)) + dtheta_3*(dtheta_3*l_1*l_3*m_4*np.sin(theta_3 - theta_1) + dtheta_3*l_1*l_3*m_5*np.sin(theta_3 - theta_1) + dtheta_3*l_1*m_3*r_3*np.sin(theta_3 - theta_1)) + dtheta_2*mu_2 + dtheta_2*(dtheta_2*l_1*l_2*m_3*np.sin(theta_2 - theta_1) + dtheta_2*l_1*l_2*m_4*np.sin(theta_2 - theta_1) + dtheta_2*l_1*l_2*m_5*np.sin(theta_2 - theta_1) + dtheta_2*l_1*m_2*r_2*np.sin(theta_2 - theta_1)) - dtheta_1*(mu_1 + mu_2) - ddx_c*(-l_1*m_2 - l_1*m_3 - l_1*m_4 - l_1*m_5 - m_1*r_1)*np.cos(theta_1) - g*(-l_1*m_2 - l_1*m_3 - l_1*m_4 - l_1*m_5 - m_1*r_1)*np.sin(theta_1))*np.cos(theta_5 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) - (-l_1**2*m_5*r_5*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_5 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*m_5*r_5*np.cos(theta_5 - theta_2))*(dtheta_5**2*l_2*m_5*r_5*np.sin(theta_5 - theta_2) + dtheta_4*(dtheta_4*l_2*l_4*m_5*np.sin(theta_4 - theta_2) + dtheta_4*l_2*m_4*r_4*np.sin(theta_4 - theta_2)) + dtheta_3*mu_3 + dtheta_3*(dtheta_3*l_2*l_3*m_4*np.sin(theta_3 - theta_2) + dtheta_3*l_2*l_3*m_5*np.sin(theta_3 - theta_2) + dtheta_3*l_2*m_3*r_3*np.sin(theta_3 - theta_2)) - dtheta_2*(mu_2 + mu_3) + dtheta_1*mu_2 + dtheta_1*(-dtheta_1*l_1*l_2*m_3*np.sin(theta_2 - theta_1) - dtheta_1*l_1*l_2*m_4*np.sin(theta_2 - theta_1) - dtheta_1*l_1*l_2*m_5*np.sin(theta_2 - theta_1) - dtheta_1*l_1*m_2*r_2*np.sin(theta_2 - theta_1)) - ddx_c*(-l_2*m_3 - l_2*m_4 - l_2*m_5 - m_2*r_2)*np.cos(theta_2) - g*(-l_2*m_3 - l_2*m_4 - l_2*m_5 - m_2*r_2)*np.sin(theta_2) - l_1*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*(dtheta_5**2*l_1*m_5*r_5*np.sin(theta_5 - theta_1) + dtheta_4*(dtheta_4*l_1*l_4*m_5*np.sin(theta_4 - theta_1) + dtheta_4*l_1*m_4*r_4*np.sin(theta_4 - theta_1)) + dtheta_3*(dtheta_3*l_1*l_3*m_4*np.sin(theta_3 - theta_1) + dtheta_3*l_1*l_3*m_5*np.sin(theta_3 - theta_1) + dtheta_3*l_1*m_3*r_3*np.sin(theta_3 - theta_1)) + dtheta_2*mu_2 + dtheta_2*(dtheta_2*l_1*l_2*m_3*np.sin(theta_2 - theta_1) + dtheta_2*l_1*l_2*m_4*np.sin(theta_2 - theta_1) + dtheta_2*l_1*l_2*m_5*np.sin(theta_2 - theta_1) + dtheta_2*l_1*m_2*r_2*np.sin(theta_2 - theta_1)) - dtheta_1*(mu_1 + mu_2) - ddx_c*(-l_1*m_2 - l_1*m_3 - l_1*m_4 - l_1*m_5 - m_1*r_1)*np.cos(theta_1) - g*(-l_1*m_2 - l_1*m_3 - l_1*m_4 - l_1*m_5 - m_1*r_1)*np.sin(theta_1))*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2))/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2) - (-l_1**2*m_5*r_5*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_5 - theta_1)*np.cos(theta_3 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_3*m_5*r_5*np.cos(theta_5 - theta_3) - (-l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_3 - theta_2))*(-l_1**2*m_5*r_5*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_5 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*m_5*r_5*np.cos(theta_5 - theta_2))/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2))*(dtheta_5**2*l_3*m_5*r_5*np.sin(theta_5 - theta_3) + dtheta_4*mu_4 + dtheta_4*(dtheta_4*l_3*l_4*m_5*np.sin(theta_4 - theta_3) + dtheta_4*l_3*m_4*r_4*np.sin(theta_4 - theta_3)) - dtheta_3*(mu_3 + mu_4) + dtheta_2*mu_3 + dtheta_2*(-dtheta_2*l_2*l_3*m_4*np.sin(theta_3 - theta_2) - dtheta_2*l_2*l_3*m_5*np.sin(theta_3 - theta_2) - dtheta_2*l_2*m_3*r_3*np.sin(theta_3 - theta_2)) + dtheta_1*(-dtheta_1*l_1*l_3*m_4*np.sin(theta_3 - theta_1) - dtheta_1*l_1*l_3*m_5*np.sin(theta_3 - theta_1) - dtheta_1*l_1*m_3*r_3*np.sin(theta_3 - theta_1)) - ddx_c*(-l_3*m_4 - l_3*m_5 - m_3*r_3)*np.cos(theta_3) - g*(-l_3*m_4 - l_3*m_5 - m_3*r_3)*np.sin(theta_3) - l_1*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(dtheta_5**2*l_1*m_5*r_5*np.sin(theta_5 - theta_1) + dtheta_4*(dtheta_4*l_1*l_4*m_5*np.sin(theta_4 - theta_1) + dtheta_4*l_1*m_4*r_4*np.sin(theta_4 - theta_1)) + dtheta_3*(dtheta_3*l_1*l_3*m_4*np.sin(theta_3 - theta_1) + dtheta_3*l_1*l_3*m_5*np.sin(theta_3 - theta_1) + dtheta_3*l_1*m_3*r_3*np.sin(theta_3 - theta_1)) + dtheta_2*mu_2 + dtheta_2*(dtheta_2*l_1*l_2*m_3*np.sin(theta_2 - theta_1) + dtheta_2*l_1*l_2*m_4*np.sin(theta_2 - theta_1) + dtheta_2*l_1*l_2*m_5*np.sin(theta_2 - theta_1) + dtheta_2*l_1*m_2*r_2*np.sin(theta_2 - theta_1)) - dtheta_1*(mu_1 + mu_2) - ddx_c*(-l_1*m_2 - l_1*m_3 - l_1*m_4 - l_1*m_5 - m_1*r_1)*np.cos(theta_1) - g*(-l_1*m_2 - l_1*m_3 - l_1*m_4 - l_1*m_5 - m_1*r_1)*np.sin(theta_1))*np.cos(theta_3 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) - (-l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_3 - theta_2))*(dtheta_5**2*l_2*m_5*r_5*np.sin(theta_5 - theta_2) + dtheta_4*(dtheta_4*l_2*l_4*m_5*np.sin(theta_4 - theta_2) + dtheta_4*l_2*m_4*r_4*np.sin(theta_4 - theta_2)) + dtheta_3*mu_3 + dtheta_3*(dtheta_3*l_2*l_3*m_4*np.sin(theta_3 - theta_2) + dtheta_3*l_2*l_3*m_5*np.sin(theta_3 - theta_2) + dtheta_3*l_2*m_3*r_3*np.sin(theta_3 - theta_2)) - dtheta_2*(mu_2 + mu_3) + dtheta_1*mu_2 + dtheta_1*(-dtheta_1*l_1*l_2*m_3*np.sin(theta_2 - theta_1) - dtheta_1*l_1*l_2*m_4*np.sin(theta_2 - theta_1) - dtheta_1*l_1*l_2*m_5*np.sin(theta_2 - theta_1) - dtheta_1*l_1*m_2*r_2*np.sin(theta_2 - theta_1)) - ddx_c*(-l_2*m_3 - l_2*m_4 - l_2*m_5 - m_2*r_2)*np.cos(theta_2) - g*(-l_2*m_3 - l_2*m_4 - l_2*m_5 - m_2*r_2)*np.sin(theta_2) - l_1*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*(dtheta_5**2*l_1*m_5*r_5*np.sin(theta_5 - theta_1) + dtheta_4*(dtheta_4*l_1*l_4*m_5*np.sin(theta_4 - theta_1) + dtheta_4*l_1*m_4*r_4*np.sin(theta_4 - theta_1)) + dtheta_3*(dtheta_3*l_1*l_3*m_4*np.sin(theta_3 - theta_1) + dtheta_3*l_1*l_3*m_5*np.sin(theta_3 - theta_1) + dtheta_3*l_1*m_3*r_3*np.sin(theta_3 - theta_1)) + dtheta_2*mu_2 + dtheta_2*(dtheta_2*l_1*l_2*m_3*np.sin(theta_2 - theta_1) + dtheta_2*l_1*l_2*m_4*np.sin(theta_2 - theta_1) + dtheta_2*l_1*l_2*m_5*np.sin(theta_2 - theta_1) + dtheta_2*l_1*m_2*r_2*np.sin(theta_2 - theta_1)) - dtheta_1*(mu_1 + mu_2) - ddx_c*(-l_1*m_2 - l_1*m_3 - l_1*m_4 - l_1*m_5 - m_1*r_1)*np.cos(theta_1) - g*(-l_1*m_2 - l_1*m_3 - l_1*m_4 - l_1*m_5 - m_1*r_1)*np.sin(theta_1))*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2))/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2))/(I_3 - l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)**2*np.cos(theta_3 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_3**2*m_4 + l_3**2*m_5 + m_3*r_3**2 - (-l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_3 - theta_2))**2/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2)) - (-l_1**2*m_5*r_5*(l_4*m_5 + m_4*r_4)*np.cos(theta_5 - theta_1)*np.cos(theta_4 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_4*m_5*r_5*np.cos(theta_5 - theta_4) - (-l_1**2*(l_4*m_5 + m_4*r_4)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_4 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_4*m_5 + m_4*r_4)*np.cos(theta_4 - theta_2))*(-l_1**2*m_5*r_5*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_5 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*m_5*r_5*np.cos(theta_5 - theta_2))/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2) - (-l_1**2*(l_4*m_5 + m_4*r_4)*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_4 - theta_1)*np.cos(theta_3 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_3*(l_4*m_5 + m_4*r_4)*np.cos(theta_4 - theta_3) - (-l_1**2*(l_4*m_5 + m_4*r_4)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_4 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_4*m_5 + m_4*r_4)*np.cos(theta_4 - theta_2))*(-l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_3 - theta_2))/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2))*(-l_1**2*m_5*r_5*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_5 - theta_1)*np.cos(theta_3 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_3*m_5*r_5*np.cos(theta_5 - theta_3) - (-l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_3 - theta_2))*(-l_1**2*m_5*r_5*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_5 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*m_5*r_5*np.cos(theta_5 - theta_2))/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2))/(I_3 - l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)**2*np.cos(theta_3 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_3**2*m_4 + l_3**2*m_5 + m_3*r_3**2 - (-l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_3 - theta_2))**2/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2)))*(dtheta_5**2*l_4*m_5*r_5*np.sin(theta_5 - theta_4) + dtheta_5*mu_5 - dtheta_4*(mu_4 + mu_5) + dtheta_3*mu_4 + dtheta_3*(-dtheta_3*l_3*l_4*m_5*np.sin(theta_4 - theta_3) - dtheta_3*l_3*m_4*r_4*np.sin(theta_4 - theta_3)) + dtheta_2*(-dtheta_2*l_2*l_4*m_5*np.sin(theta_4 - theta_2) - dtheta_2*l_2*m_4*r_4*np.sin(theta_4 - theta_2)) + dtheta_1*(-dtheta_1*l_1*l_4*m_5*np.sin(theta_4 - theta_1) - dtheta_1*l_1*m_4*r_4*np.sin(theta_4 - theta_1)) - ddx_c*(-l_4*m_5 - m_4*r_4)*np.cos(theta_4) - g*(-l_4*m_5 - m_4*r_4)*np.sin(theta_4) - l_1*(l_4*m_5 + m_4*r_4)*(dtheta_5**2*l_1*m_5*r_5*np.sin(theta_5 - theta_1) + dtheta_4*(dtheta_4*l_1*l_4*m_5*np.sin(theta_4 - theta_1) + dtheta_4*l_1*m_4*r_4*np.sin(theta_4 - theta_1)) + dtheta_3*(dtheta_3*l_1*l_3*m_4*np.sin(theta_3 - theta_1) + dtheta_3*l_1*l_3*m_5*np.sin(theta_3 - theta_1) + dtheta_3*l_1*m_3*r_3*np.sin(theta_3 - theta_1)) + dtheta_2*mu_2 + dtheta_2*(dtheta_2*l_1*l_2*m_3*np.sin(theta_2 - theta_1) + dtheta_2*l_1*l_2*m_4*np.sin(theta_2 - theta_1) + dtheta_2*l_1*l_2*m_5*np.sin(theta_2 - theta_1) + dtheta_2*l_1*m_2*r_2*np.sin(theta_2 - theta_1)) - dtheta_1*(mu_1 + mu_2) - ddx_c*(-l_1*m_2 - l_1*m_3 - l_1*m_4 - l_1*m_5 - m_1*r_1)*np.cos(theta_1) - g*(-l_1*m_2 - l_1*m_3 - l_1*m_4 - l_1*m_5 - m_1*r_1)*np.sin(theta_1))*np.cos(theta_4 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) - (-l_1**2*(l_4*m_5 + m_4*r_4)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_4 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_4*m_5 + m_4*r_4)*np.cos(theta_4 - theta_2))*(dtheta_5**2*l_2*m_5*r_5*np.sin(theta_5 - theta_2) + dtheta_4*(dtheta_4*l_2*l_4*m_5*np.sin(theta_4 - theta_2) + dtheta_4*l_2*m_4*r_4*np.sin(theta_4 - theta_2)) + dtheta_3*mu_3 + dtheta_3*(dtheta_3*l_2*l_3*m_4*np.sin(theta_3 - theta_2) + dtheta_3*l_2*l_3*m_5*np.sin(theta_3 - theta_2) + dtheta_3*l_2*m_3*r_3*np.sin(theta_3 - theta_2)) - dtheta_2*(mu_2 + mu_3) + dtheta_1*mu_2 + dtheta_1*(-dtheta_1*l_1*l_2*m_3*np.sin(theta_2 - theta_1) - dtheta_1*l_1*l_2*m_4*np.sin(theta_2 - theta_1) - dtheta_1*l_1*l_2*m_5*np.sin(theta_2 - theta_1) - dtheta_1*l_1*m_2*r_2*np.sin(theta_2 - theta_1)) - ddx_c*(-l_2*m_3 - l_2*m_4 - l_2*m_5 - m_2*r_2)*np.cos(theta_2) - g*(-l_2*m_3 - l_2*m_4 - l_2*m_5 - m_2*r_2)*np.sin(theta_2) - l_1*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*(dtheta_5**2*l_1*m_5*r_5*np.sin(theta_5 - theta_1) + dtheta_4*(dtheta_4*l_1*l_4*m_5*np.sin(theta_4 - theta_1) + dtheta_4*l_1*m_4*r_4*np.sin(theta_4 - theta_1)) + dtheta_3*(dtheta_3*l_1*l_3*m_4*np.sin(theta_3 - theta_1) + dtheta_3*l_1*l_3*m_5*np.sin(theta_3 - theta_1) + dtheta_3*l_1*m_3*r_3*np.sin(theta_3 - theta_1)) + dtheta_2*mu_2 + dtheta_2*(dtheta_2*l_1*l_2*m_3*np.sin(theta_2 - theta_1) + dtheta_2*l_1*l_2*m_4*np.sin(theta_2 - theta_1) + dtheta_2*l_1*l_2*m_5*np.sin(theta_2 - theta_1) + dtheta_2*l_1*m_2*r_2*np.sin(theta_2 - theta_1)) - dtheta_1*(mu_1 + mu_2) - ddx_c*(-l_1*m_2 - l_1*m_3 - l_1*m_4 - l_1*m_5 - m_1*r_1)*np.cos(theta_1) - g*(-l_1*m_2 - l_1*m_3 - l_1*m_4 - l_1*m_5 - m_1*r_1)*np.sin(theta_1))*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2))/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2) - (-l_1**2*(l_4*m_5 + m_4*r_4)*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_4 - theta_1)*np.cos(theta_3 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_3*(l_4*m_5 + m_4*r_4)*np.cos(theta_4 - theta_3) - (-l_1**2*(l_4*m_5 + m_4*r_4)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_4 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_4*m_5 + m_4*r_4)*np.cos(theta_4 - theta_2))*(-l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_3 - theta_2))/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2))*(dtheta_5**2*l_3*m_5*r_5*np.sin(theta_5 - theta_3) + dtheta_4*mu_4 + dtheta_4*(dtheta_4*l_3*l_4*m_5*np.sin(theta_4 - theta_3) + dtheta_4*l_3*m_4*r_4*np.sin(theta_4 - theta_3)) - dtheta_3*(mu_3 + mu_4) + dtheta_2*mu_3 + dtheta_2*(-dtheta_2*l_2*l_3*m_4*np.sin(theta_3 - theta_2) - dtheta_2*l_2*l_3*m_5*np.sin(theta_3 - theta_2) - dtheta_2*l_2*m_3*r_3*np.sin(theta_3 - theta_2)) + dtheta_1*(-dtheta_1*l_1*l_3*m_4*np.sin(theta_3 - theta_1) - dtheta_1*l_1*l_3*m_5*np.sin(theta_3 - theta_1) - dtheta_1*l_1*m_3*r_3*np.sin(theta_3 - theta_1)) - ddx_c*(-l_3*m_4 - l_3*m_5 - m_3*r_3)*np.cos(theta_3) - g*(-l_3*m_4 - l_3*m_5 - m_3*r_3)*np.sin(theta_3) - l_1*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(dtheta_5**2*l_1*m_5*r_5*np.sin(theta_5 - theta_1) + dtheta_4*(dtheta_4*l_1*l_4*m_5*np.sin(theta_4 - theta_1) + dtheta_4*l_1*m_4*r_4*np.sin(theta_4 - theta_1)) + dtheta_3*(dtheta_3*l_1*l_3*m_4*np.sin(theta_3 - theta_1) + dtheta_3*l_1*l_3*m_5*np.sin(theta_3 - theta_1) + dtheta_3*l_1*m_3*r_3*np.sin(theta_3 - theta_1)) + dtheta_2*mu_2 + dtheta_2*(dtheta_2*l_1*l_2*m_3*np.sin(theta_2 - theta_1) + dtheta_2*l_1*l_2*m_4*np.sin(theta_2 - theta_1) + dtheta_2*l_1*l_2*m_5*np.sin(theta_2 - theta_1) + dtheta_2*l_1*m_2*r_2*np.sin(theta_2 - theta_1)) - dtheta_1*(mu_1 + mu_2) - ddx_c*(-l_1*m_2 - l_1*m_3 - l_1*m_4 - l_1*m_5 - m_1*r_1)*np.cos(theta_1) - g*(-l_1*m_2 - l_1*m_3 - l_1*m_4 - l_1*m_5 - m_1*r_1)*np.sin(theta_1))*np.cos(theta_3 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) - (-l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_3 - theta_2))*(dtheta_5**2*l_2*m_5*r_5*np.sin(theta_5 - theta_2) + dtheta_4*(dtheta_4*l_2*l_4*m_5*np.sin(theta_4 - theta_2) + dtheta_4*l_2*m_4*r_4*np.sin(theta_4 - theta_2)) + dtheta_3*mu_3 + dtheta_3*(dtheta_3*l_2*l_3*m_4*np.sin(theta_3 - theta_2) + dtheta_3*l_2*l_3*m_5*np.sin(theta_3 - theta_2) + dtheta_3*l_2*m_3*r_3*np.sin(theta_3 - theta_2)) - dtheta_2*(mu_2 + mu_3) + dtheta_1*mu_2 + dtheta_1*(-dtheta_1*l_1*l_2*m_3*np.sin(theta_2 - theta_1) - dtheta_1*l_1*l_2*m_4*np.sin(theta_2 - theta_1) - dtheta_1*l_1*l_2*m_5*np.sin(theta_2 - theta_1) - dtheta_1*l_1*m_2*r_2*np.sin(theta_2 - theta_1)) - ddx_c*(-l_2*m_3 - l_2*m_4 - l_2*m_5 - m_2*r_2)*np.cos(theta_2) - g*(-l_2*m_3 - l_2*m_4 - l_2*m_5 - m_2*r_2)*np.sin(theta_2) - l_1*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*(dtheta_5**2*l_1*m_5*r_5*np.sin(theta_5 - theta_1) + dtheta_4*(dtheta_4*l_1*l_4*m_5*np.sin(theta_4 - theta_1) + dtheta_4*l_1*m_4*r_4*np.sin(theta_4 - theta_1)) + dtheta_3*(dtheta_3*l_1*l_3*m_4*np.sin(theta_3 - theta_1) + dtheta_3*l_1*l_3*m_5*np.sin(theta_3 - theta_1) + dtheta_3*l_1*m_3*r_3*np.sin(theta_3 - theta_1)) + dtheta_2*mu_2 + dtheta_2*(dtheta_2*l_1*l_2*m_3*np.sin(theta_2 - theta_1) + dtheta_2*l_1*l_2*m_4*np.sin(theta_2 - theta_1) + dtheta_2*l_1*l_2*m_5*np.sin(theta_2 - theta_1) + dtheta_2*l_1*m_2*r_2*np.sin(theta_2 - theta_1)) - dtheta_1*(mu_1 + mu_2) - ddx_c*(-l_1*m_2 - l_1*m_3 - l_1*m_4 - l_1*m_5 - m_1*r_1)*np.cos(theta_1) - g*(-l_1*m_2 - l_1*m_3 - l_1*m_4 - l_1*m_5 - m_1*r_1)*np.sin(theta_1))*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2))/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2))/(I_3 - l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)**2*np.cos(theta_3 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_3**2*m_4 + l_3**2*m_5 + m_3*r_3**2 - (-l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_3 - theta_2))**2/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2)))/(I_4 - l_1**2*(l_4*m_5 + m_4*r_4)**2*np.cos(theta_4 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_4**2*m_5 + m_4*r_4**2 - (-l_1**2*(l_4*m_5 + m_4*r_4)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_4 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_4*m_5 + m_4*r_4)*np.cos(theta_4 - theta_2))**2/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2) - (-l_1**2*(l_4*m_5 + m_4*r_4)*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_4 - theta_1)*np.cos(theta_3 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_3*(l_4*m_5 + m_4*r_4)*np.cos(theta_4 - theta_3) - (-l_1**2*(l_4*m_5 + m_4*r_4)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_4 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_4*m_5 + m_4*r_4)*np.cos(theta_4 - theta_2))*(-l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_3 - theta_2))/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2))**2/(I_3 - l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)**2*np.cos(theta_3 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_3**2*m_4 + l_3**2*m_5 + m_3*r_3**2 - (-l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_3 - theta_2))**2/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2))))/(I_5 - l_1**2*m_5**2*r_5**2*np.cos(theta_5 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + m_5*r_5**2 - (-l_1**2*m_5*r_5*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_5 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*m_5*r_5*np.cos(theta_5 - theta_2))**2/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2) - (-l_1**2*m_5*r_5*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_5 - theta_1)*np.cos(theta_3 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_3*m_5*r_5*np.cos(theta_5 - theta_3) - (-l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_3 - theta_2))*(-l_1**2*m_5*r_5*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_5 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*m_5*r_5*np.cos(theta_5 - theta_2))/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2))**2/(I_3 - l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)**2*np.cos(theta_3 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_3**2*m_4 + l_3**2*m_5 + m_3*r_3**2 - (-l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_3 - theta_2))**2/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2)) - (-l_1**2*m_5*r_5*(l_4*m_5 + m_4*r_4)*np.cos(theta_5 - theta_1)*np.cos(theta_4 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_4*m_5*r_5*np.cos(theta_5 - theta_4) - (-l_1**2*(l_4*m_5 + m_4*r_4)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_4 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_4*m_5 + m_4*r_4)*np.cos(theta_4 - theta_2))*(-l_1**2*m_5*r_5*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_5 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*m_5*r_5*np.cos(theta_5 - theta_2))/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2) - (-l_1**2*(l_4*m_5 + m_4*r_4)*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_4 - theta_1)*np.cos(theta_3 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_3*(l_4*m_5 + m_4*r_4)*np.cos(theta_4 - theta_3) - (-l_1**2*(l_4*m_5 + m_4*r_4)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_4 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_4*m_5 + m_4*r_4)*np.cos(theta_4 - theta_2))*(-l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_3 - theta_2))/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2))*(-l_1**2*m_5*r_5*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_5 - theta_1)*np.cos(theta_3 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_3*m_5*r_5*np.cos(theta_5 - theta_3) - (-l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_3 - theta_2))*(-l_1**2*m_5*r_5*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_5 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*m_5*r_5*np.cos(theta_5 - theta_2))/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2))/(I_3 - l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)**2*np.cos(theta_3 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_3**2*m_4 + l_3**2*m_5 + m_3*r_3**2 - (-l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_3 - theta_2))**2/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2)))**2/(I_4 - l_1**2*(l_4*m_5 + m_4*r_4)**2*np.cos(theta_4 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_4**2*m_5 + m_4*r_4**2 - (-l_1**2*(l_4*m_5 + m_4*r_4)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_4 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_4*m_5 + m_4*r_4)*np.cos(theta_4 - theta_2))**2/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2) - (-l_1**2*(l_4*m_5 + m_4*r_4)*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_4 - theta_1)*np.cos(theta_3 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_3*(l_4*m_5 + m_4*r_4)*np.cos(theta_4 - theta_3) - (-l_1**2*(l_4*m_5 + m_4*r_4)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_4 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_4*m_5 + m_4*r_4)*np.cos(theta_4 - theta_2))*(-l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_3 - theta_2))/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2))**2/(I_3 - l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)**2*np.cos(theta_3 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_3**2*m_4 + l_3**2*m_5 + m_3*r_3**2 - (-l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_3 - theta_2))**2/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2)))))/(I_4 - l_1**2*(l_4*m_5 + m_4*r_4)**2*np.cos(theta_4 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_4**2*m_5 + m_4*r_4**2 - (-l_1**2*(l_4*m_5 + m_4*r_4)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_4 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_4*m_5 + m_4*r_4)*np.cos(theta_4 - theta_2))**2/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2) - (-l_1**2*(l_4*m_5 + m_4*r_4)*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_4 - theta_1)*np.cos(theta_3 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_3*(l_4*m_5 + m_4*r_4)*np.cos(theta_4 - theta_3) - (-l_1**2*(l_4*m_5 + m_4*r_4)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_4 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_4*m_5 + m_4*r_4)*np.cos(theta_4 - theta_2))*(-l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_3 - theta_2))/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2))**2/(I_3 - l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)**2*np.cos(theta_3 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_3**2*m_4 + l_3**2*m_5 + m_3*r_3**2 - (-l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_3 - theta_2))**2/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2))) - (-l_1**2*m_5*r_5*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_5 - theta_1)*np.cos(theta_3 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_3*m_5*r_5*np.cos(theta_5 - theta_3) - (-l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_3 - theta_2))*(-l_1**2*m_5*r_5*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_5 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*m_5*r_5*np.cos(theta_5 - theta_2))/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2))*(-dtheta_5*mu_5 - dtheta_4**2*l_4*m_5*r_5*np.sin(theta_5 - theta_4) + dtheta_4*mu_5 - dtheta_3**2*l_3*m_5*r_5*np.sin(theta_5 - theta_3) - dtheta_2**2*l_2*m_5*r_5*np.sin(theta_5 - theta_2) - dtheta_1**2*l_1*m_5*r_5*np.sin(theta_5 - theta_1) + ddx_c*m_5*r_5*np.cos(theta_5) + g*m_5*r_5*np.sin(theta_5) - l_1*m_5*r_5*(dtheta_5**2*l_1*m_5*r_5*np.sin(theta_5 - theta_1) + dtheta_4*(dtheta_4*l_1*l_4*m_5*np.sin(theta_4 - theta_1) + dtheta_4*l_1*m_4*r_4*np.sin(theta_4 - theta_1)) + dtheta_3*(dtheta_3*l_1*l_3*m_4*np.sin(theta_3 - theta_1) + dtheta_3*l_1*l_3*m_5*np.sin(theta_3 - theta_1) + dtheta_3*l_1*m_3*r_3*np.sin(theta_3 - theta_1)) + dtheta_2*mu_2 + dtheta_2*(dtheta_2*l_1*l_2*m_3*np.sin(theta_2 - theta_1) + dtheta_2*l_1*l_2*m_4*np.sin(theta_2 - theta_1) + dtheta_2*l_1*l_2*m_5*np.sin(theta_2 - theta_1) + dtheta_2*l_1*m_2*r_2*np.sin(theta_2 - theta_1)) - dtheta_1*(mu_1 + mu_2) - ddx_c*(-l_1*m_2 - l_1*m_3 - l_1*m_4 - l_1*m_5 - m_1*r_1)*np.cos(theta_1) - g*(-l_1*m_2 - l_1*m_3 - l_1*m_4 - l_1*m_5 - m_1*r_1)*np.sin(theta_1))*np.cos(theta_5 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) - (-l_1**2*m_5*r_5*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_5 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*m_5*r_5*np.cos(theta_5 - theta_2))*(dtheta_5**2*l_2*m_5*r_5*np.sin(theta_5 - theta_2) + dtheta_4*(dtheta_4*l_2*l_4*m_5*np.sin(theta_4 - theta_2) + dtheta_4*l_2*m_4*r_4*np.sin(theta_4 - theta_2)) + dtheta_3*mu_3 + dtheta_3*(dtheta_3*l_2*l_3*m_4*np.sin(theta_3 - theta_2) + dtheta_3*l_2*l_3*m_5*np.sin(theta_3 - theta_2) + dtheta_3*l_2*m_3*r_3*np.sin(theta_3 - theta_2)) - dtheta_2*(mu_2 + mu_3) + dtheta_1*mu_2 + dtheta_1*(-dtheta_1*l_1*l_2*m_3*np.sin(theta_2 - theta_1) - dtheta_1*l_1*l_2*m_4*np.sin(theta_2 - theta_1) - dtheta_1*l_1*l_2*m_5*np.sin(theta_2 - theta_1) - dtheta_1*l_1*m_2*r_2*np.sin(theta_2 - theta_1)) - ddx_c*(-l_2*m_3 - l_2*m_4 - l_2*m_5 - m_2*r_2)*np.cos(theta_2) - g*(-l_2*m_3 - l_2*m_4 - l_2*m_5 - m_2*r_2)*np.sin(theta_2) - l_1*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*(dtheta_5**2*l_1*m_5*r_5*np.sin(theta_5 - theta_1) + dtheta_4*(dtheta_4*l_1*l_4*m_5*np.sin(theta_4 - theta_1) + dtheta_4*l_1*m_4*r_4*np.sin(theta_4 - theta_1)) + dtheta_3*(dtheta_3*l_1*l_3*m_4*np.sin(theta_3 - theta_1) + dtheta_3*l_1*l_3*m_5*np.sin(theta_3 - theta_1) + dtheta_3*l_1*m_3*r_3*np.sin(theta_3 - theta_1)) + dtheta_2*mu_2 + dtheta_2*(dtheta_2*l_1*l_2*m_3*np.sin(theta_2 - theta_1) + dtheta_2*l_1*l_2*m_4*np.sin(theta_2 - theta_1) + dtheta_2*l_1*l_2*m_5*np.sin(theta_2 - theta_1) + dtheta_2*l_1*m_2*r_2*np.sin(theta_2 - theta_1)) - dtheta_1*(mu_1 + mu_2) - ddx_c*(-l_1*m_2 - l_1*m_3 - l_1*m_4 - l_1*m_5 - m_1*r_1)*np.cos(theta_1) - g*(-l_1*m_2 - l_1*m_3 - l_1*m_4 - l_1*m_5 - m_1*r_1)*np.sin(theta_1))*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2))/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2) - (-l_1**2*m_5*r_5*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_5 - theta_1)*np.cos(theta_3 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_3*m_5*r_5*np.cos(theta_5 - theta_3) - (-l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_3 - theta_2))*(-l_1**2*m_5*r_5*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_5 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*m_5*r_5*np.cos(theta_5 - theta_2))/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2))*(dtheta_5**2*l_3*m_5*r_5*np.sin(theta_5 - theta_3) + dtheta_4*mu_4 + dtheta_4*(dtheta_4*l_3*l_4*m_5*np.sin(theta_4 - theta_3) + dtheta_4*l_3*m_4*r_4*np.sin(theta_4 - theta_3)) - dtheta_3*(mu_3 + mu_4) + dtheta_2*mu_3 + dtheta_2*(-dtheta_2*l_2*l_3*m_4*np.sin(theta_3 - theta_2) - dtheta_2*l_2*l_3*m_5*np.sin(theta_3 - theta_2) - dtheta_2*l_2*m_3*r_3*np.sin(theta_3 - theta_2)) + dtheta_1*(-dtheta_1*l_1*l_3*m_4*np.sin(theta_3 - theta_1) - dtheta_1*l_1*l_3*m_5*np.sin(theta_3 - theta_1) - dtheta_1*l_1*m_3*r_3*np.sin(theta_3 - theta_1)) - ddx_c*(-l_3*m_4 - l_3*m_5 - m_3*r_3)*np.cos(theta_3) - g*(-l_3*m_4 - l_3*m_5 - m_3*r_3)*np.sin(theta_3) - l_1*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(dtheta_5**2*l_1*m_5*r_5*np.sin(theta_5 - theta_1) + dtheta_4*(dtheta_4*l_1*l_4*m_5*np.sin(theta_4 - theta_1) + dtheta_4*l_1*m_4*r_4*np.sin(theta_4 - theta_1)) + dtheta_3*(dtheta_3*l_1*l_3*m_4*np.sin(theta_3 - theta_1) + dtheta_3*l_1*l_3*m_5*np.sin(theta_3 - theta_1) + dtheta_3*l_1*m_3*r_3*np.sin(theta_3 - theta_1)) + dtheta_2*mu_2 + dtheta_2*(dtheta_2*l_1*l_2*m_3*np.sin(theta_2 - theta_1) + dtheta_2*l_1*l_2*m_4*np.sin(theta_2 - theta_1) + dtheta_2*l_1*l_2*m_5*np.sin(theta_2 - theta_1) + dtheta_2*l_1*m_2*r_2*np.sin(theta_2 - theta_1)) - dtheta_1*(mu_1 + mu_2) - ddx_c*(-l_1*m_2 - l_1*m_3 - l_1*m_4 - l_1*m_5 - m_1*r_1)*np.cos(theta_1) - g*(-l_1*m_2 - l_1*m_3 - l_1*m_4 - l_1*m_5 - m_1*r_1)*np.sin(theta_1))*np.cos(theta_3 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) - (-l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_3 - theta_2))*(dtheta_5**2*l_2*m_5*r_5*np.sin(theta_5 - theta_2) + dtheta_4*(dtheta_4*l_2*l_4*m_5*np.sin(theta_4 - theta_2) + dtheta_4*l_2*m_4*r_4*np.sin(theta_4 - theta_2)) + dtheta_3*mu_3 + dtheta_3*(dtheta_3*l_2*l_3*m_4*np.sin(theta_3 - theta_2) + dtheta_3*l_2*l_3*m_5*np.sin(theta_3 - theta_2) + dtheta_3*l_2*m_3*r_3*np.sin(theta_3 - theta_2)) - dtheta_2*(mu_2 + mu_3) + dtheta_1*mu_2 + dtheta_1*(-dtheta_1*l_1*l_2*m_3*np.sin(theta_2 - theta_1) - dtheta_1*l_1*l_2*m_4*np.sin(theta_2 - theta_1) - dtheta_1*l_1*l_2*m_5*np.sin(theta_2 - theta_1) - dtheta_1*l_1*m_2*r_2*np.sin(theta_2 - theta_1)) - ddx_c*(-l_2*m_3 - l_2*m_4 - l_2*m_5 - m_2*r_2)*np.cos(theta_2) - g*(-l_2*m_3 - l_2*m_4 - l_2*m_5 - m_2*r_2)*np.sin(theta_2) - l_1*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*(dtheta_5**2*l_1*m_5*r_5*np.sin(theta_5 - theta_1) + dtheta_4*(dtheta_4*l_1*l_4*m_5*np.sin(theta_4 - theta_1) + dtheta_4*l_1*m_4*r_4*np.sin(theta_4 - theta_1)) + dtheta_3*(dtheta_3*l_1*l_3*m_4*np.sin(theta_3 - theta_1) + dtheta_3*l_1*l_3*m_5*np.sin(theta_3 - theta_1) + dtheta_3*l_1*m_3*r_3*np.sin(theta_3 - theta_1)) + dtheta_2*mu_2 + dtheta_2*(dtheta_2*l_1*l_2*m_3*np.sin(theta_2 - theta_1) + dtheta_2*l_1*l_2*m_4*np.sin(theta_2 - theta_1) + dtheta_2*l_1*l_2*m_5*np.sin(theta_2 - theta_1) + dtheta_2*l_1*m_2*r_2*np.sin(theta_2 - theta_1)) - dtheta_1*(mu_1 + mu_2) - ddx_c*(-l_1*m_2 - l_1*m_3 - l_1*m_4 - l_1*m_5 - m_1*r_1)*np.cos(theta_1) - g*(-l_1*m_2 - l_1*m_3 - l_1*m_4 - l_1*m_5 - m_1*r_1)*np.sin(theta_1))*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2))/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2))/(I_3 - l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)**2*np.cos(theta_3 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_3**2*m_4 + l_3**2*m_5 + m_3*r_3**2 - (-l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_3 - theta_2))**2/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2)) - (-l_1**2*m_5*r_5*(l_4*m_5 + m_4*r_4)*np.cos(theta_5 - theta_1)*np.cos(theta_4 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_4*m_5*r_5*np.cos(theta_5 - theta_4) - (-l_1**2*(l_4*m_5 + m_4*r_4)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_4 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_4*m_5 + m_4*r_4)*np.cos(theta_4 - theta_2))*(-l_1**2*m_5*r_5*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_5 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*m_5*r_5*np.cos(theta_5 - theta_2))/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2) - (-l_1**2*(l_4*m_5 + m_4*r_4)*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_4 - theta_1)*np.cos(theta_3 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_3*(l_4*m_5 + m_4*r_4)*np.cos(theta_4 - theta_3) - (-l_1**2*(l_4*m_5 + m_4*r_4)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_4 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_4*m_5 + m_4*r_4)*np.cos(theta_4 - theta_2))*(-l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_3 - theta_2))/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2))*(-l_1**2*m_5*r_5*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_5 - theta_1)*np.cos(theta_3 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_3*m_5*r_5*np.cos(theta_5 - theta_3) - (-l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_3 - theta_2))*(-l_1**2*m_5*r_5*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_5 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*m_5*r_5*np.cos(theta_5 - theta_2))/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2))/(I_3 - l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)**2*np.cos(theta_3 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_3**2*m_4 + l_3**2*m_5 + m_3*r_3**2 - (-l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_3 - theta_2))**2/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2)))*(dtheta_5**2*l_4*m_5*r_5*np.sin(theta_5 - theta_4) + dtheta_5*mu_5 - dtheta_4*(mu_4 + mu_5) + dtheta_3*mu_4 + dtheta_3*(-dtheta_3*l_3*l_4*m_5*np.sin(theta_4 - theta_3) - dtheta_3*l_3*m_4*r_4*np.sin(theta_4 - theta_3)) + dtheta_2*(-dtheta_2*l_2*l_4*m_5*np.sin(theta_4 - theta_2) - dtheta_2*l_2*m_4*r_4*np.sin(theta_4 - theta_2)) + dtheta_1*(-dtheta_1*l_1*l_4*m_5*np.sin(theta_4 - theta_1) - dtheta_1*l_1*m_4*r_4*np.sin(theta_4 - theta_1)) - ddx_c*(-l_4*m_5 - m_4*r_4)*np.cos(theta_4) - g*(-l_4*m_5 - m_4*r_4)*np.sin(theta_4) - l_1*(l_4*m_5 + m_4*r_4)*(dtheta_5**2*l_1*m_5*r_5*np.sin(theta_5 - theta_1) + dtheta_4*(dtheta_4*l_1*l_4*m_5*np.sin(theta_4 - theta_1) + dtheta_4*l_1*m_4*r_4*np.sin(theta_4 - theta_1)) + dtheta_3*(dtheta_3*l_1*l_3*m_4*np.sin(theta_3 - theta_1) + dtheta_3*l_1*l_3*m_5*np.sin(theta_3 - theta_1) + dtheta_3*l_1*m_3*r_3*np.sin(theta_3 - theta_1)) + dtheta_2*mu_2 + dtheta_2*(dtheta_2*l_1*l_2*m_3*np.sin(theta_2 - theta_1) + dtheta_2*l_1*l_2*m_4*np.sin(theta_2 - theta_1) + dtheta_2*l_1*l_2*m_5*np.sin(theta_2 - theta_1) + dtheta_2*l_1*m_2*r_2*np.sin(theta_2 - theta_1)) - dtheta_1*(mu_1 + mu_2) - ddx_c*(-l_1*m_2 - l_1*m_3 - l_1*m_4 - l_1*m_5 - m_1*r_1)*np.cos(theta_1) - g*(-l_1*m_2 - l_1*m_3 - l_1*m_4 - l_1*m_5 - m_1*r_1)*np.sin(theta_1))*np.cos(theta_4 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) - (-l_1**2*(l_4*m_5 + m_4*r_4)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_4 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_4*m_5 + m_4*r_4)*np.cos(theta_4 - theta_2))*(dtheta_5**2*l_2*m_5*r_5*np.sin(theta_5 - theta_2) + dtheta_4*(dtheta_4*l_2*l_4*m_5*np.sin(theta_4 - theta_2) + dtheta_4*l_2*m_4*r_4*np.sin(theta_4 - theta_2)) + dtheta_3*mu_3 + dtheta_3*(dtheta_3*l_2*l_3*m_4*np.sin(theta_3 - theta_2) + dtheta_3*l_2*l_3*m_5*np.sin(theta_3 - theta_2) + dtheta_3*l_2*m_3*r_3*np.sin(theta_3 - theta_2)) - dtheta_2*(mu_2 + mu_3) + dtheta_1*mu_2 + dtheta_1*(-dtheta_1*l_1*l_2*m_3*np.sin(theta_2 - theta_1) - dtheta_1*l_1*l_2*m_4*np.sin(theta_2 - theta_1) - dtheta_1*l_1*l_2*m_5*np.sin(theta_2 - theta_1) - dtheta_1*l_1*m_2*r_2*np.sin(theta_2 - theta_1)) - ddx_c*(-l_2*m_3 - l_2*m_4 - l_2*m_5 - m_2*r_2)*np.cos(theta_2) - g*(-l_2*m_3 - l_2*m_4 - l_2*m_5 - m_2*r_2)*np.sin(theta_2) - l_1*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*(dtheta_5**2*l_1*m_5*r_5*np.sin(theta_5 - theta_1) + dtheta_4*(dtheta_4*l_1*l_4*m_5*np.sin(theta_4 - theta_1) + dtheta_4*l_1*m_4*r_4*np.sin(theta_4 - theta_1)) + dtheta_3*(dtheta_3*l_1*l_3*m_4*np.sin(theta_3 - theta_1) + dtheta_3*l_1*l_3*m_5*np.sin(theta_3 - theta_1) + dtheta_3*l_1*m_3*r_3*np.sin(theta_3 - theta_1)) + dtheta_2*mu_2 + dtheta_2*(dtheta_2*l_1*l_2*m_3*np.sin(theta_2 - theta_1) + dtheta_2*l_1*l_2*m_4*np.sin(theta_2 - theta_1) + dtheta_2*l_1*l_2*m_5*np.sin(theta_2 - theta_1) + dtheta_2*l_1*m_2*r_2*np.sin(theta_2 - theta_1)) - dtheta_1*(mu_1 + mu_2) - ddx_c*(-l_1*m_2 - l_1*m_3 - l_1*m_4 - l_1*m_5 - m_1*r_1)*np.cos(theta_1) - g*(-l_1*m_2 - l_1*m_3 - l_1*m_4 - l_1*m_5 - m_1*r_1)*np.sin(theta_1))*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2))/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2) - (-l_1**2*(l_4*m_5 + m_4*r_4)*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_4 - theta_1)*np.cos(theta_3 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_3*(l_4*m_5 + m_4*r_4)*np.cos(theta_4 - theta_3) - (-l_1**2*(l_4*m_5 + m_4*r_4)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_4 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_4*m_5 + m_4*r_4)*np.cos(theta_4 - theta_2))*(-l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_3 - theta_2))/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2))*(dtheta_5**2*l_3*m_5*r_5*np.sin(theta_5 - theta_3) + dtheta_4*mu_4 + dtheta_4*(dtheta_4*l_3*l_4*m_5*np.sin(theta_4 - theta_3) + dtheta_4*l_3*m_4*r_4*np.sin(theta_4 - theta_3)) - dtheta_3*(mu_3 + mu_4) + dtheta_2*mu_3 + dtheta_2*(-dtheta_2*l_2*l_3*m_4*np.sin(theta_3 - theta_2) - dtheta_2*l_2*l_3*m_5*np.sin(theta_3 - theta_2) - dtheta_2*l_2*m_3*r_3*np.sin(theta_3 - theta_2)) + dtheta_1*(-dtheta_1*l_1*l_3*m_4*np.sin(theta_3 - theta_1) - dtheta_1*l_1*l_3*m_5*np.sin(theta_3 - theta_1) - dtheta_1*l_1*m_3*r_3*np.sin(theta_3 - theta_1)) - ddx_c*(-l_3*m_4 - l_3*m_5 - m_3*r_3)*np.cos(theta_3) - g*(-l_3*m_4 - l_3*m_5 - m_3*r_3)*np.sin(theta_3) - l_1*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(dtheta_5**2*l_1*m_5*r_5*np.sin(theta_5 - theta_1) + dtheta_4*(dtheta_4*l_1*l_4*m_5*np.sin(theta_4 - theta_1) + dtheta_4*l_1*m_4*r_4*np.sin(theta_4 - theta_1)) + dtheta_3*(dtheta_3*l_1*l_3*m_4*np.sin(theta_3 - theta_1) + dtheta_3*l_1*l_3*m_5*np.sin(theta_3 - theta_1) + dtheta_3*l_1*m_3*r_3*np.sin(theta_3 - theta_1)) + dtheta_2*mu_2 + dtheta_2*(dtheta_2*l_1*l_2*m_3*np.sin(theta_2 - theta_1) + dtheta_2*l_1*l_2*m_4*np.sin(theta_2 - theta_1) + dtheta_2*l_1*l_2*m_5*np.sin(theta_2 - theta_1) + dtheta_2*l_1*m_2*r_2*np.sin(theta_2 - theta_1)) - dtheta_1*(mu_1 + mu_2) - ddx_c*(-l_1*m_2 - l_1*m_3 - l_1*m_4 - l_1*m_5 - m_1*r_1)*np.cos(theta_1) - g*(-l_1*m_2 - l_1*m_3 - l_1*m_4 - l_1*m_5 - m_1*r_1)*np.sin(theta_1))*np.cos(theta_3 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) - (-l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_3 - theta_2))*(dtheta_5**2*l_2*m_5*r_5*np.sin(theta_5 - theta_2) + dtheta_4*(dtheta_4*l_2*l_4*m_5*np.sin(theta_4 - theta_2) + dtheta_4*l_2*m_4*r_4*np.sin(theta_4 - theta_2)) + dtheta_3*mu_3 + dtheta_3*(dtheta_3*l_2*l_3*m_4*np.sin(theta_3 - theta_2) + dtheta_3*l_2*l_3*m_5*np.sin(theta_3 - theta_2) + dtheta_3*l_2*m_3*r_3*np.sin(theta_3 - theta_2)) - dtheta_2*(mu_2 + mu_3) + dtheta_1*mu_2 + dtheta_1*(-dtheta_1*l_1*l_2*m_3*np.sin(theta_2 - theta_1) - dtheta_1*l_1*l_2*m_4*np.sin(theta_2 - theta_1) - dtheta_1*l_1*l_2*m_5*np.sin(theta_2 - theta_1) - dtheta_1*l_1*m_2*r_2*np.sin(theta_2 - theta_1)) - ddx_c*(-l_2*m_3 - l_2*m_4 - l_2*m_5 - m_2*r_2)*np.cos(theta_2) - g*(-l_2*m_3 - l_2*m_4 - l_2*m_5 - m_2*r_2)*np.sin(theta_2) - l_1*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*(dtheta_5**2*l_1*m_5*r_5*np.sin(theta_5 - theta_1) + dtheta_4*(dtheta_4*l_1*l_4*m_5*np.sin(theta_4 - theta_1) + dtheta_4*l_1*m_4*r_4*np.sin(theta_4 - theta_1)) + dtheta_3*(dtheta_3*l_1*l_3*m_4*np.sin(theta_3 - theta_1) + dtheta_3*l_1*l_3*m_5*np.sin(theta_3 - theta_1) + dtheta_3*l_1*m_3*r_3*np.sin(theta_3 - theta_1)) + dtheta_2*mu_2 + dtheta_2*(dtheta_2*l_1*l_2*m_3*np.sin(theta_2 - theta_1) + dtheta_2*l_1*l_2*m_4*np.sin(theta_2 - theta_1) + dtheta_2*l_1*l_2*m_5*np.sin(theta_2 - theta_1) + dtheta_2*l_1*m_2*r_2*np.sin(theta_2 - theta_1)) - dtheta_1*(mu_1 + mu_2) - ddx_c*(-l_1*m_2 - l_1*m_3 - l_1*m_4 - l_1*m_5 - m_1*r_1)*np.cos(theta_1) - g*(-l_1*m_2 - l_1*m_3 - l_1*m_4 - l_1*m_5 - m_1*r_1)*np.sin(theta_1))*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2))/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2))/(I_3 - l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)**2*np.cos(theta_3 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_3**2*m_4 + l_3**2*m_5 + m_3*r_3**2 - (-l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_3 - theta_2))**2/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2)))/(I_4 - l_1**2*(l_4*m_5 + m_4*r_4)**2*np.cos(theta_4 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_4**2*m_5 + m_4*r_4**2 - (-l_1**2*(l_4*m_5 + m_4*r_4)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_4 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_4*m_5 + m_4*r_4)*np.cos(theta_4 - theta_2))**2/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2) - (-l_1**2*(l_4*m_5 + m_4*r_4)*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_4 - theta_1)*np.cos(theta_3 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_3*(l_4*m_5 + m_4*r_4)*np.cos(theta_4 - theta_3) - (-l_1**2*(l_4*m_5 + m_4*r_4)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_4 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_4*m_5 + m_4*r_4)*np.cos(theta_4 - theta_2))*(-l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_3 - theta_2))/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2))**2/(I_3 - l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)**2*np.cos(theta_3 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_3**2*m_4 + l_3**2*m_5 + m_3*r_3**2 - (-l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_3 - theta_2))**2/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2))))/(I_5 - l_1**2*m_5**2*r_5**2*np.cos(theta_5 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + m_5*r_5**2 - (-l_1**2*m_5*r_5*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_5 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*m_5*r_5*np.cos(theta_5 - theta_2))**2/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2) - (-l_1**2*m_5*r_5*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_5 - theta_1)*np.cos(theta_3 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_3*m_5*r_5*np.cos(theta_5 - theta_3) - (-l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_3 - theta_2))*(-l_1**2*m_5*r_5*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_5 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*m_5*r_5*np.cos(theta_5 - theta_2))/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2))**2/(I_3 - l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)**2*np.cos(theta_3 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_3**2*m_4 + l_3**2*m_5 + m_3*r_3**2 - (-l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_3 - theta_2))**2/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2)) - (-l_1**2*m_5*r_5*(l_4*m_5 + m_4*r_4)*np.cos(theta_5 - theta_1)*np.cos(theta_4 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_4*m_5*r_5*np.cos(theta_5 - theta_4) - (-l_1**2*(l_4*m_5 + m_4*r_4)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_4 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_4*m_5 + m_4*r_4)*np.cos(theta_4 - theta_2))*(-l_1**2*m_5*r_5*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_5 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*m_5*r_5*np.cos(theta_5 - theta_2))/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2) - (-l_1**2*(l_4*m_5 + m_4*r_4)*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_4 - theta_1)*np.cos(theta_3 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_3*(l_4*m_5 + m_4*r_4)*np.cos(theta_4 - theta_3) - (-l_1**2*(l_4*m_5 + m_4*r_4)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_4 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_4*m_5 + m_4*r_4)*np.cos(theta_4 - theta_2))*(-l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_3 - theta_2))/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2))*(-l_1**2*m_5*r_5*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_5 - theta_1)*np.cos(theta_3 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_3*m_5*r_5*np.cos(theta_5 - theta_3) - (-l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_3 - theta_2))*(-l_1**2*m_5*r_5*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_5 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*m_5*r_5*np.cos(theta_5 - theta_2))/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2))/(I_3 - l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)**2*np.cos(theta_3 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_3**2*m_4 + l_3**2*m_5 + m_3*r_3**2 - (-l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_3 - theta_2))**2/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2)))**2/(I_4 - l_1**2*(l_4*m_5 + m_4*r_4)**2*np.cos(theta_4 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_4**2*m_5 + m_4*r_4**2 - (-l_1**2*(l_4*m_5 + m_4*r_4)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_4 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_4*m_5 + m_4*r_4)*np.cos(theta_4 - theta_2))**2/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2) - (-l_1**2*(l_4*m_5 + m_4*r_4)*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_4 - theta_1)*np.cos(theta_3 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_3*(l_4*m_5 + m_4*r_4)*np.cos(theta_4 - theta_3) - (-l_1**2*(l_4*m_5 + m_4*r_4)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_4 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_4*m_5 + m_4*r_4)*np.cos(theta_4 - theta_2))*(-l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_3 - theta_2))/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2))**2/(I_3 - l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)**2*np.cos(theta_3 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_3**2*m_4 + l_3**2*m_5 + m_3*r_3**2 - (-l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_3 - theta_2))**2/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2)))))*np.cos(theta_3 - theta_1)/(I_3 - l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)**2*np.cos(theta_3 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_3**2*m_4 + l_3**2*m_5 + m_3*r_3**2 - (-l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_3 - theta_2))**2/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2)) - l_1*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*(dtheta_5**2*l_2*m_5*r_5*np.sin(theta_5 - theta_2) + dtheta_4*(dtheta_4*l_2*l_4*m_5*np.sin(theta_4 - theta_2) + dtheta_4*l_2*m_4*r_4*np.sin(theta_4 - theta_2)) + dtheta_3*mu_3 + dtheta_3*(dtheta_3*l_2*l_3*m_4*np.sin(theta_3 - theta_2) + dtheta_3*l_2*l_3*m_5*np.sin(theta_3 - theta_2) + dtheta_3*l_2*m_3*r_3*np.sin(theta_3 - theta_2)) - dtheta_2*(mu_2 + mu_3) + dtheta_1*mu_2 + dtheta_1*(-dtheta_1*l_1*l_2*m_3*np.sin(theta_2 - theta_1) - dtheta_1*l_1*l_2*m_4*np.sin(theta_2 - theta_1) - dtheta_1*l_1*l_2*m_5*np.sin(theta_2 - theta_1) - dtheta_1*l_1*m_2*r_2*np.sin(theta_2 - theta_1)) - ddx_c*(-l_2*m_3 - l_2*m_4 - l_2*m_5 - m_2*r_2)*np.cos(theta_2) - g*(-l_2*m_3 - l_2*m_4 - l_2*m_5 - m_2*r_2)*np.sin(theta_2) - l_1*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*(dtheta_5**2*l_1*m_5*r_5*np.sin(theta_5 - theta_1) + dtheta_4*(dtheta_4*l_1*l_4*m_5*np.sin(theta_4 - theta_1) + dtheta_4*l_1*m_4*r_4*np.sin(theta_4 - theta_1)) + dtheta_3*(dtheta_3*l_1*l_3*m_4*np.sin(theta_3 - theta_1) + dtheta_3*l_1*l_3*m_5*np.sin(theta_3 - theta_1) + dtheta_3*l_1*m_3*r_3*np.sin(theta_3 - theta_1)) + dtheta_2*mu_2 + dtheta_2*(dtheta_2*l_1*l_2*m_3*np.sin(theta_2 - theta_1) + dtheta_2*l_1*l_2*m_4*np.sin(theta_2 - theta_1) + dtheta_2*l_1*l_2*m_5*np.sin(theta_2 - theta_1) + dtheta_2*l_1*m_2*r_2*np.sin(theta_2 - theta_1)) - dtheta_1*(mu_1 + mu_2) - ddx_c*(-l_1*m_2 - l_1*m_3 - l_1*m_4 - l_1*m_5 - m_1*r_1)*np.cos(theta_1) - g*(-l_1*m_2 - l_1*m_3 - l_1*m_4 - l_1*m_5 - m_1*r_1)*np.sin(theta_1))*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) - (-l_1**2*(l_4*m_5 + m_4*r_4)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_4 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_4*m_5 + m_4*r_4)*np.cos(theta_4 - theta_2))*(dtheta_5**2*l_4*m_5*r_5*np.sin(theta_5 - theta_4) + dtheta_5*mu_5 - dtheta_4*(mu_4 + mu_5) + dtheta_3*mu_4 + dtheta_3*(-dtheta_3*l_3*l_4*m_5*np.sin(theta_4 - theta_3) - dtheta_3*l_3*m_4*r_4*np.sin(theta_4 - theta_3)) + dtheta_2*(-dtheta_2*l_2*l_4*m_5*np.sin(theta_4 - theta_2) - dtheta_2*l_2*m_4*r_4*np.sin(theta_4 - theta_2)) + dtheta_1*(-dtheta_1*l_1*l_4*m_5*np.sin(theta_4 - theta_1) - dtheta_1*l_1*m_4*r_4*np.sin(theta_4 - theta_1)) - ddx_c*(-l_4*m_5 - m_4*r_4)*np.cos(theta_4) - g*(-l_4*m_5 - m_4*r_4)*np.sin(theta_4) - l_1*(l_4*m_5 + m_4*r_4)*(dtheta_5**2*l_1*m_5*r_5*np.sin(theta_5 - theta_1) + dtheta_4*(dtheta_4*l_1*l_4*m_5*np.sin(theta_4 - theta_1) + dtheta_4*l_1*m_4*r_4*np.sin(theta_4 - theta_1)) + dtheta_3*(dtheta_3*l_1*l_3*m_4*np.sin(theta_3 - theta_1) + dtheta_3*l_1*l_3*m_5*np.sin(theta_3 - theta_1) + dtheta_3*l_1*m_3*r_3*np.sin(theta_3 - theta_1)) + dtheta_2*mu_2 + dtheta_2*(dtheta_2*l_1*l_2*m_3*np.sin(theta_2 - theta_1) + dtheta_2*l_1*l_2*m_4*np.sin(theta_2 - theta_1) + dtheta_2*l_1*l_2*m_5*np.sin(theta_2 - theta_1) + dtheta_2*l_1*m_2*r_2*np.sin(theta_2 - theta_1)) - dtheta_1*(mu_1 + mu_2) - ddx_c*(-l_1*m_2 - l_1*m_3 - l_1*m_4 - l_1*m_5 - m_1*r_1)*np.cos(theta_1) - g*(-l_1*m_2 - l_1*m_3 - l_1*m_4 - l_1*m_5 - m_1*r_1)*np.sin(theta_1))*np.cos(theta_4 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) - (-l_1**2*(l_4*m_5 + m_4*r_4)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_4 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_4*m_5 + m_4*r_4)*np.cos(theta_4 - theta_2))*(dtheta_5**2*l_2*m_5*r_5*np.sin(theta_5 - theta_2) + dtheta_4*(dtheta_4*l_2*l_4*m_5*np.sin(theta_4 - theta_2) + dtheta_4*l_2*m_4*r_4*np.sin(theta_4 - theta_2)) + dtheta_3*mu_3 + dtheta_3*(dtheta_3*l_2*l_3*m_4*np.sin(theta_3 - theta_2) + dtheta_3*l_2*l_3*m_5*np.sin(theta_3 - theta_2) + dtheta_3*l_2*m_3*r_3*np.sin(theta_3 - theta_2)) - dtheta_2*(mu_2 + mu_3) + dtheta_1*mu_2 + dtheta_1*(-dtheta_1*l_1*l_2*m_3*np.sin(theta_2 - theta_1) - dtheta_1*l_1*l_2*m_4*np.sin(theta_2 - theta_1) - dtheta_1*l_1*l_2*m_5*np.sin(theta_2 - theta_1) - dtheta_1*l_1*m_2*r_2*np.sin(theta_2 - theta_1)) - ddx_c*(-l_2*m_3 - l_2*m_4 - l_2*m_5 - m_2*r_2)*np.cos(theta_2) - g*(-l_2*m_3 - l_2*m_4 - l_2*m_5 - m_2*r_2)*np.sin(theta_2) - l_1*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*(dtheta_5**2*l_1*m_5*r_5*np.sin(theta_5 - theta_1) + dtheta_4*(dtheta_4*l_1*l_4*m_5*np.sin(theta_4 - theta_1) + dtheta_4*l_1*m_4*r_4*np.sin(theta_4 - theta_1)) + dtheta_3*(dtheta_3*l_1*l_3*m_4*np.sin(theta_3 - theta_1) + dtheta_3*l_1*l_3*m_5*np.sin(theta_3 - theta_1) + dtheta_3*l_1*m_3*r_3*np.sin(theta_3 - theta_1)) + dtheta_2*mu_2 + dtheta_2*(dtheta_2*l_1*l_2*m_3*np.sin(theta_2 - theta_1) + dtheta_2*l_1*l_2*m_4*np.sin(theta_2 - theta_1) + dtheta_2*l_1*l_2*m_5*np.sin(theta_2 - theta_1) + dtheta_2*l_1*m_2*r_2*np.sin(theta_2 - theta_1)) - dtheta_1*(mu_1 + mu_2) - ddx_c*(-l_1*m_2 - l_1*m_3 - l_1*m_4 - l_1*m_5 - m_1*r_1)*np.cos(theta_1) - g*(-l_1*m_2 - l_1*m_3 - l_1*m_4 - l_1*m_5 - m_1*r_1)*np.sin(theta_1))*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2))/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2) - (-l_1**2*(l_4*m_5 + m_4*r_4)*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_4 - theta_1)*np.cos(theta_3 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_3*(l_4*m_5 + m_4*r_4)*np.cos(theta_4 - theta_3) - (-l_1**2*(l_4*m_5 + m_4*r_4)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_4 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_4*m_5 + m_4*r_4)*np.cos(theta_4 - theta_2))*(-l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_3 - theta_2))/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2))*(dtheta_5**2*l_3*m_5*r_5*np.sin(theta_5 - theta_3) + dtheta_4*mu_4 + dtheta_4*(dtheta_4*l_3*l_4*m_5*np.sin(theta_4 - theta_3) + dtheta_4*l_3*m_4*r_4*np.sin(theta_4 - theta_3)) - dtheta_3*(mu_3 + mu_4) + dtheta_2*mu_3 + dtheta_2*(-dtheta_2*l_2*l_3*m_4*np.sin(theta_3 - theta_2) - dtheta_2*l_2*l_3*m_5*np.sin(theta_3 - theta_2) - dtheta_2*l_2*m_3*r_3*np.sin(theta_3 - theta_2)) + dtheta_1*(-dtheta_1*l_1*l_3*m_4*np.sin(theta_3 - theta_1) - dtheta_1*l_1*l_3*m_5*np.sin(theta_3 - theta_1) - dtheta_1*l_1*m_3*r_3*np.sin(theta_3 - theta_1)) - ddx_c*(-l_3*m_4 - l_3*m_5 - m_3*r_3)*np.cos(theta_3) - g*(-l_3*m_4 - l_3*m_5 - m_3*r_3)*np.sin(theta_3) - l_1*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(dtheta_5**2*l_1*m_5*r_5*np.sin(theta_5 - theta_1) + dtheta_4*(dtheta_4*l_1*l_4*m_5*np.sin(theta_4 - theta_1) + dtheta_4*l_1*m_4*r_4*np.sin(theta_4 - theta_1)) + dtheta_3*(dtheta_3*l_1*l_3*m_4*np.sin(theta_3 - theta_1) + dtheta_3*l_1*l_3*m_5*np.sin(theta_3 - theta_1) + dtheta_3*l_1*m_3*r_3*np.sin(theta_3 - theta_1)) + dtheta_2*mu_2 + dtheta_2*(dtheta_2*l_1*l_2*m_3*np.sin(theta_2 - theta_1) + dtheta_2*l_1*l_2*m_4*np.sin(theta_2 - theta_1) + dtheta_2*l_1*l_2*m_5*np.sin(theta_2 - theta_1) + dtheta_2*l_1*m_2*r_2*np.sin(theta_2 - theta_1)) - dtheta_1*(mu_1 + mu_2) - ddx_c*(-l_1*m_2 - l_1*m_3 - l_1*m_4 - l_1*m_5 - m_1*r_1)*np.cos(theta_1) - g*(-l_1*m_2 - l_1*m_3 - l_1*m_4 - l_1*m_5 - m_1*r_1)*np.sin(theta_1))*np.cos(theta_3 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) - (-l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_3 - theta_2))*(dtheta_5**2*l_2*m_5*r_5*np.sin(theta_5 - theta_2) + dtheta_4*(dtheta_4*l_2*l_4*m_5*np.sin(theta_4 - theta_2) + dtheta_4*l_2*m_4*r_4*np.sin(theta_4 - theta_2)) + dtheta_3*mu_3 + dtheta_3*(dtheta_3*l_2*l_3*m_4*np.sin(theta_3 - theta_2) + dtheta_3*l_2*l_3*m_5*np.sin(theta_3 - theta_2) + dtheta_3*l_2*m_3*r_3*np.sin(theta_3 - theta_2)) - dtheta_2*(mu_2 + mu_3) + dtheta_1*mu_2 + dtheta_1*(-dtheta_1*l_1*l_2*m_3*np.sin(theta_2 - theta_1) - dtheta_1*l_1*l_2*m_4*np.sin(theta_2 - theta_1) - dtheta_1*l_1*l_2*m_5*np.sin(theta_2 - theta_1) - dtheta_1*l_1*m_2*r_2*np.sin(theta_2 - theta_1)) - ddx_c*(-l_2*m_3 - l_2*m_4 - l_2*m_5 - m_2*r_2)*np.cos(theta_2) - g*(-l_2*m_3 - l_2*m_4 - l_2*m_5 - m_2*r_2)*np.sin(theta_2) - l_1*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*(dtheta_5**2*l_1*m_5*r_5*np.sin(theta_5 - theta_1) + dtheta_4*(dtheta_4*l_1*l_4*m_5*np.sin(theta_4 - theta_1) + dtheta_4*l_1*m_4*r_4*np.sin(theta_4 - theta_1)) + dtheta_3*(dtheta_3*l_1*l_3*m_4*np.sin(theta_3 - theta_1) + dtheta_3*l_1*l_3*m_5*np.sin(theta_3 - theta_1) + dtheta_3*l_1*m_3*r_3*np.sin(theta_3 - theta_1)) + dtheta_2*mu_2 + dtheta_2*(dtheta_2*l_1*l_2*m_3*np.sin(theta_2 - theta_1) + dtheta_2*l_1*l_2*m_4*np.sin(theta_2 - theta_1) + dtheta_2*l_1*l_2*m_5*np.sin(theta_2 - theta_1) + dtheta_2*l_1*m_2*r_2*np.sin(theta_2 - theta_1)) - dtheta_1*(mu_1 + mu_2) - ddx_c*(-l_1*m_2 - l_1*m_3 - l_1*m_4 - l_1*m_5 - m_1*r_1)*np.cos(theta_1) - g*(-l_1*m_2 - l_1*m_3 - l_1*m_4 - l_1*m_5 - m_1*r_1)*np.sin(theta_1))*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2))/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2))/(I_3 - l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)**2*np.cos(theta_3 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_3**2*m_4 + l_3**2*m_5 + m_3*r_3**2 - (-l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_3 - theta_2))**2/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2)) - (-l_1**2*m_5*r_5*(l_4*m_5 + m_4*r_4)*np.cos(theta_5 - theta_1)*np.cos(theta_4 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_4*m_5*r_5*np.cos(theta_5 - theta_4) - (-l_1**2*(l_4*m_5 + m_4*r_4)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_4 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_4*m_5 + m_4*r_4)*np.cos(theta_4 - theta_2))*(-l_1**2*m_5*r_5*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_5 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*m_5*r_5*np.cos(theta_5 - theta_2))/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2) - (-l_1**2*(l_4*m_5 + m_4*r_4)*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_4 - theta_1)*np.cos(theta_3 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_3*(l_4*m_5 + m_4*r_4)*np.cos(theta_4 - theta_3) - (-l_1**2*(l_4*m_5 + m_4*r_4)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_4 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_4*m_5 + m_4*r_4)*np.cos(theta_4 - theta_2))*(-l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_3 - theta_2))/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2))*(-l_1**2*m_5*r_5*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_5 - theta_1)*np.cos(theta_3 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_3*m_5*r_5*np.cos(theta_5 - theta_3) - (-l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_3 - theta_2))*(-l_1**2*m_5*r_5*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_5 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*m_5*r_5*np.cos(theta_5 - theta_2))/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2))/(I_3 - l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)**2*np.cos(theta_3 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_3**2*m_4 + l_3**2*m_5 + m_3*r_3**2 - (-l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_3 - theta_2))**2/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2)))*(-dtheta_5*mu_5 - dtheta_4**2*l_4*m_5*r_5*np.sin(theta_5 - theta_4) + dtheta_4*mu_5 - dtheta_3**2*l_3*m_5*r_5*np.sin(theta_5 - theta_3) - dtheta_2**2*l_2*m_5*r_5*np.sin(theta_5 - theta_2) - dtheta_1**2*l_1*m_5*r_5*np.sin(theta_5 - theta_1) + ddx_c*m_5*r_5*np.cos(theta_5) + g*m_5*r_5*np.sin(theta_5) - l_1*m_5*r_5*(dtheta_5**2*l_1*m_5*r_5*np.sin(theta_5 - theta_1) + dtheta_4*(dtheta_4*l_1*l_4*m_5*np.sin(theta_4 - theta_1) + dtheta_4*l_1*m_4*r_4*np.sin(theta_4 - theta_1)) + dtheta_3*(dtheta_3*l_1*l_3*m_4*np.sin(theta_3 - theta_1) + dtheta_3*l_1*l_3*m_5*np.sin(theta_3 - theta_1) + dtheta_3*l_1*m_3*r_3*np.sin(theta_3 - theta_1)) + dtheta_2*mu_2 + dtheta_2*(dtheta_2*l_1*l_2*m_3*np.sin(theta_2 - theta_1) + dtheta_2*l_1*l_2*m_4*np.sin(theta_2 - theta_1) + dtheta_2*l_1*l_2*m_5*np.sin(theta_2 - theta_1) + dtheta_2*l_1*m_2*r_2*np.sin(theta_2 - theta_1)) - dtheta_1*(mu_1 + mu_2) - ddx_c*(-l_1*m_2 - l_1*m_3 - l_1*m_4 - l_1*m_5 - m_1*r_1)*np.cos(theta_1) - g*(-l_1*m_2 - l_1*m_3 - l_1*m_4 - l_1*m_5 - m_1*r_1)*np.sin(theta_1))*np.cos(theta_5 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) - (-l_1**2*m_5*r_5*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_5 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*m_5*r_5*np.cos(theta_5 - theta_2))*(dtheta_5**2*l_2*m_5*r_5*np.sin(theta_5 - theta_2) + dtheta_4*(dtheta_4*l_2*l_4*m_5*np.sin(theta_4 - theta_2) + dtheta_4*l_2*m_4*r_4*np.sin(theta_4 - theta_2)) + dtheta_3*mu_3 + dtheta_3*(dtheta_3*l_2*l_3*m_4*np.sin(theta_3 - theta_2) + dtheta_3*l_2*l_3*m_5*np.sin(theta_3 - theta_2) + dtheta_3*l_2*m_3*r_3*np.sin(theta_3 - theta_2)) - dtheta_2*(mu_2 + mu_3) + dtheta_1*mu_2 + dtheta_1*(-dtheta_1*l_1*l_2*m_3*np.sin(theta_2 - theta_1) - dtheta_1*l_1*l_2*m_4*np.sin(theta_2 - theta_1) - dtheta_1*l_1*l_2*m_5*np.sin(theta_2 - theta_1) - dtheta_1*l_1*m_2*r_2*np.sin(theta_2 - theta_1)) - ddx_c*(-l_2*m_3 - l_2*m_4 - l_2*m_5 - m_2*r_2)*np.cos(theta_2) - g*(-l_2*m_3 - l_2*m_4 - l_2*m_5 - m_2*r_2)*np.sin(theta_2) - l_1*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*(dtheta_5**2*l_1*m_5*r_5*np.sin(theta_5 - theta_1) + dtheta_4*(dtheta_4*l_1*l_4*m_5*np.sin(theta_4 - theta_1) + dtheta_4*l_1*m_4*r_4*np.sin(theta_4 - theta_1)) + dtheta_3*(dtheta_3*l_1*l_3*m_4*np.sin(theta_3 - theta_1) + dtheta_3*l_1*l_3*m_5*np.sin(theta_3 - theta_1) + dtheta_3*l_1*m_3*r_3*np.sin(theta_3 - theta_1)) + dtheta_2*mu_2 + dtheta_2*(dtheta_2*l_1*l_2*m_3*np.sin(theta_2 - theta_1) + dtheta_2*l_1*l_2*m_4*np.sin(theta_2 - theta_1) + dtheta_2*l_1*l_2*m_5*np.sin(theta_2 - theta_1) + dtheta_2*l_1*m_2*r_2*np.sin(theta_2 - theta_1)) - dtheta_1*(mu_1 + mu_2) - ddx_c*(-l_1*m_2 - l_1*m_3 - l_1*m_4 - l_1*m_5 - m_1*r_1)*np.cos(theta_1) - g*(-l_1*m_2 - l_1*m_3 - l_1*m_4 - l_1*m_5 - m_1*r_1)*np.sin(theta_1))*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2))/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2) - (-l_1**2*m_5*r_5*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_5 - theta_1)*np.cos(theta_3 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_3*m_5*r_5*np.cos(theta_5 - theta_3) - (-l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_3 - theta_2))*(-l_1**2*m_5*r_5*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_5 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*m_5*r_5*np.cos(theta_5 - theta_2))/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2))*(dtheta_5**2*l_3*m_5*r_5*np.sin(theta_5 - theta_3) + dtheta_4*mu_4 + dtheta_4*(dtheta_4*l_3*l_4*m_5*np.sin(theta_4 - theta_3) + dtheta_4*l_3*m_4*r_4*np.sin(theta_4 - theta_3)) - dtheta_3*(mu_3 + mu_4) + dtheta_2*mu_3 + dtheta_2*(-dtheta_2*l_2*l_3*m_4*np.sin(theta_3 - theta_2) - dtheta_2*l_2*l_3*m_5*np.sin(theta_3 - theta_2) - dtheta_2*l_2*m_3*r_3*np.sin(theta_3 - theta_2)) + dtheta_1*(-dtheta_1*l_1*l_3*m_4*np.sin(theta_3 - theta_1) - dtheta_1*l_1*l_3*m_5*np.sin(theta_3 - theta_1) - dtheta_1*l_1*m_3*r_3*np.sin(theta_3 - theta_1)) - ddx_c*(-l_3*m_4 - l_3*m_5 - m_3*r_3)*np.cos(theta_3) - g*(-l_3*m_4 - l_3*m_5 - m_3*r_3)*np.sin(theta_3) - l_1*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(dtheta_5**2*l_1*m_5*r_5*np.sin(theta_5 - theta_1) + dtheta_4*(dtheta_4*l_1*l_4*m_5*np.sin(theta_4 - theta_1) + dtheta_4*l_1*m_4*r_4*np.sin(theta_4 - theta_1)) + dtheta_3*(dtheta_3*l_1*l_3*m_4*np.sin(theta_3 - theta_1) + dtheta_3*l_1*l_3*m_5*np.sin(theta_3 - theta_1) + dtheta_3*l_1*m_3*r_3*np.sin(theta_3 - theta_1)) + dtheta_2*mu_2 + dtheta_2*(dtheta_2*l_1*l_2*m_3*np.sin(theta_2 - theta_1) + dtheta_2*l_1*l_2*m_4*np.sin(theta_2 - theta_1) + dtheta_2*l_1*l_2*m_5*np.sin(theta_2 - theta_1) + dtheta_2*l_1*m_2*r_2*np.sin(theta_2 - theta_1)) - dtheta_1*(mu_1 + mu_2) - ddx_c*(-l_1*m_2 - l_1*m_3 - l_1*m_4 - l_1*m_5 - m_1*r_1)*np.cos(theta_1) - g*(-l_1*m_2 - l_1*m_3 - l_1*m_4 - l_1*m_5 - m_1*r_1)*np.sin(theta_1))*np.cos(theta_3 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) - (-l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_3 - theta_2))*(dtheta_5**2*l_2*m_5*r_5*np.sin(theta_5 - theta_2) + dtheta_4*(dtheta_4*l_2*l_4*m_5*np.sin(theta_4 - theta_2) + dtheta_4*l_2*m_4*r_4*np.sin(theta_4 - theta_2)) + dtheta_3*mu_3 + dtheta_3*(dtheta_3*l_2*l_3*m_4*np.sin(theta_3 - theta_2) + dtheta_3*l_2*l_3*m_5*np.sin(theta_3 - theta_2) + dtheta_3*l_2*m_3*r_3*np.sin(theta_3 - theta_2)) - dtheta_2*(mu_2 + mu_3) + dtheta_1*mu_2 + dtheta_1*(-dtheta_1*l_1*l_2*m_3*np.sin(theta_2 - theta_1) - dtheta_1*l_1*l_2*m_4*np.sin(theta_2 - theta_1) - dtheta_1*l_1*l_2*m_5*np.sin(theta_2 - theta_1) - dtheta_1*l_1*m_2*r_2*np.sin(theta_2 - theta_1)) - ddx_c*(-l_2*m_3 - l_2*m_4 - l_2*m_5 - m_2*r_2)*np.cos(theta_2) - g*(-l_2*m_3 - l_2*m_4 - l_2*m_5 - m_2*r_2)*np.sin(theta_2) - l_1*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*(dtheta_5**2*l_1*m_5*r_5*np.sin(theta_5 - theta_1) + dtheta_4*(dtheta_4*l_1*l_4*m_5*np.sin(theta_4 - theta_1) + dtheta_4*l_1*m_4*r_4*np.sin(theta_4 - theta_1)) + dtheta_3*(dtheta_3*l_1*l_3*m_4*np.sin(theta_3 - theta_1) + dtheta_3*l_1*l_3*m_5*np.sin(theta_3 - theta_1) + dtheta_3*l_1*m_3*r_3*np.sin(theta_3 - theta_1)) + dtheta_2*mu_2 + dtheta_2*(dtheta_2*l_1*l_2*m_3*np.sin(theta_2 - theta_1) + dtheta_2*l_1*l_2*m_4*np.sin(theta_2 - theta_1) + dtheta_2*l_1*l_2*m_5*np.sin(theta_2 - theta_1) + dtheta_2*l_1*m_2*r_2*np.sin(theta_2 - theta_1)) - dtheta_1*(mu_1 + mu_2) - ddx_c*(-l_1*m_2 - l_1*m_3 - l_1*m_4 - l_1*m_5 - m_1*r_1)*np.cos(theta_1) - g*(-l_1*m_2 - l_1*m_3 - l_1*m_4 - l_1*m_5 - m_1*r_1)*np.sin(theta_1))*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2))/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2))/(I_3 - l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)**2*np.cos(theta_3 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_3**2*m_4 + l_3**2*m_5 + m_3*r_3**2 - (-l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_3 - theta_2))**2/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2)) - (-l_1**2*m_5*r_5*(l_4*m_5 + m_4*r_4)*np.cos(theta_5 - theta_1)*np.cos(theta_4 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_4*m_5*r_5*np.cos(theta_5 - theta_4) - (-l_1**2*(l_4*m_5 + m_4*r_4)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_4 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_4*m_5 + m_4*r_4)*np.cos(theta_4 - theta_2))*(-l_1**2*m_5*r_5*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_5 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*m_5*r_5*np.cos(theta_5 - theta_2))/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2) - (-l_1**2*(l_4*m_5 + m_4*r_4)*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_4 - theta_1)*np.cos(theta_3 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_3*(l_4*m_5 + m_4*r_4)*np.cos(theta_4 - theta_3) - (-l_1**2*(l_4*m_5 + m_4*r_4)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_4 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_4*m_5 + m_4*r_4)*np.cos(theta_4 - theta_2))*(-l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_3 - theta_2))/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2))*(-l_1**2*m_5*r_5*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_5 - theta_1)*np.cos(theta_3 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_3*m_5*r_5*np.cos(theta_5 - theta_3) - (-l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_3 - theta_2))*(-l_1**2*m_5*r_5*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_5 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*m_5*r_5*np.cos(theta_5 - theta_2))/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2))/(I_3 - l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)**2*np.cos(theta_3 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_3**2*m_4 + l_3**2*m_5 + m_3*r_3**2 - (-l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_3 - theta_2))**2/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2)))*(dtheta_5**2*l_4*m_5*r_5*np.sin(theta_5 - theta_4) + dtheta_5*mu_5 - dtheta_4*(mu_4 + mu_5) + dtheta_3*mu_4 + dtheta_3*(-dtheta_3*l_3*l_4*m_5*np.sin(theta_4 - theta_3) - dtheta_3*l_3*m_4*r_4*np.sin(theta_4 - theta_3)) + dtheta_2*(-dtheta_2*l_2*l_4*m_5*np.sin(theta_4 - theta_2) - dtheta_2*l_2*m_4*r_4*np.sin(theta_4 - theta_2)) + dtheta_1*(-dtheta_1*l_1*l_4*m_5*np.sin(theta_4 - theta_1) - dtheta_1*l_1*m_4*r_4*np.sin(theta_4 - theta_1)) - ddx_c*(-l_4*m_5 - m_4*r_4)*np.cos(theta_4) - g*(-l_4*m_5 - m_4*r_4)*np.sin(theta_4) - l_1*(l_4*m_5 + m_4*r_4)*(dtheta_5**2*l_1*m_5*r_5*np.sin(theta_5 - theta_1) + dtheta_4*(dtheta_4*l_1*l_4*m_5*np.sin(theta_4 - theta_1) + dtheta_4*l_1*m_4*r_4*np.sin(theta_4 - theta_1)) + dtheta_3*(dtheta_3*l_1*l_3*m_4*np.sin(theta_3 - theta_1) + dtheta_3*l_1*l_3*m_5*np.sin(theta_3 - theta_1) + dtheta_3*l_1*m_3*r_3*np.sin(theta_3 - theta_1)) + dtheta_2*mu_2 + dtheta_2*(dtheta_2*l_1*l_2*m_3*np.sin(theta_2 - theta_1) + dtheta_2*l_1*l_2*m_4*np.sin(theta_2 - theta_1) + dtheta_2*l_1*l_2*m_5*np.sin(theta_2 - theta_1) + dtheta_2*l_1*m_2*r_2*np.sin(theta_2 - theta_1)) - dtheta_1*(mu_1 + mu_2) - ddx_c*(-l_1*m_2 - l_1*m_3 - l_1*m_4 - l_1*m_5 - m_1*r_1)*np.cos(theta_1) - g*(-l_1*m_2 - l_1*m_3 - l_1*m_4 - l_1*m_5 - m_1*r_1)*np.sin(theta_1))*np.cos(theta_4 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) - (-l_1**2*(l_4*m_5 + m_4*r_4)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_4 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_4*m_5 + m_4*r_4)*np.cos(theta_4 - theta_2))*(dtheta_5**2*l_2*m_5*r_5*np.sin(theta_5 - theta_2) + dtheta_4*(dtheta_4*l_2*l_4*m_5*np.sin(theta_4 - theta_2) + dtheta_4*l_2*m_4*r_4*np.sin(theta_4 - theta_2)) + dtheta_3*mu_3 + dtheta_3*(dtheta_3*l_2*l_3*m_4*np.sin(theta_3 - theta_2) + dtheta_3*l_2*l_3*m_5*np.sin(theta_3 - theta_2) + dtheta_3*l_2*m_3*r_3*np.sin(theta_3 - theta_2)) - dtheta_2*(mu_2 + mu_3) + dtheta_1*mu_2 + dtheta_1*(-dtheta_1*l_1*l_2*m_3*np.sin(theta_2 - theta_1) - dtheta_1*l_1*l_2*m_4*np.sin(theta_2 - theta_1) - dtheta_1*l_1*l_2*m_5*np.sin(theta_2 - theta_1) - dtheta_1*l_1*m_2*r_2*np.sin(theta_2 - theta_1)) - ddx_c*(-l_2*m_3 - l_2*m_4 - l_2*m_5 - m_2*r_2)*np.cos(theta_2) - g*(-l_2*m_3 - l_2*m_4 - l_2*m_5 - m_2*r_2)*np.sin(theta_2) - l_1*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*(dtheta_5**2*l_1*m_5*r_5*np.sin(theta_5 - theta_1) + dtheta_4*(dtheta_4*l_1*l_4*m_5*np.sin(theta_4 - theta_1) + dtheta_4*l_1*m_4*r_4*np.sin(theta_4 - theta_1)) + dtheta_3*(dtheta_3*l_1*l_3*m_4*np.sin(theta_3 - theta_1) + dtheta_3*l_1*l_3*m_5*np.sin(theta_3 - theta_1) + dtheta_3*l_1*m_3*r_3*np.sin(theta_3 - theta_1)) + dtheta_2*mu_2 + dtheta_2*(dtheta_2*l_1*l_2*m_3*np.sin(theta_2 - theta_1) + dtheta_2*l_1*l_2*m_4*np.sin(theta_2 - theta_1) + dtheta_2*l_1*l_2*m_5*np.sin(theta_2 - theta_1) + dtheta_2*l_1*m_2*r_2*np.sin(theta_2 - theta_1)) - dtheta_1*(mu_1 + mu_2) - ddx_c*(-l_1*m_2 - l_1*m_3 - l_1*m_4 - l_1*m_5 - m_1*r_1)*np.cos(theta_1) - g*(-l_1*m_2 - l_1*m_3 - l_1*m_4 - l_1*m_5 - m_1*r_1)*np.sin(theta_1))*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2))/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2) - (-l_1**2*(l_4*m_5 + m_4*r_4)*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_4 - theta_1)*np.cos(theta_3 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_3*(l_4*m_5 + m_4*r_4)*np.cos(theta_4 - theta_3) - (-l_1**2*(l_4*m_5 + m_4*r_4)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_4 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_4*m_5 + m_4*r_4)*np.cos(theta_4 - theta_2))*(-l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_3 - theta_2))/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2))*(dtheta_5**2*l_3*m_5*r_5*np.sin(theta_5 - theta_3) + dtheta_4*mu_4 + dtheta_4*(dtheta_4*l_3*l_4*m_5*np.sin(theta_4 - theta_3) + dtheta_4*l_3*m_4*r_4*np.sin(theta_4 - theta_3)) - dtheta_3*(mu_3 + mu_4) + dtheta_2*mu_3 + dtheta_2*(-dtheta_2*l_2*l_3*m_4*np.sin(theta_3 - theta_2) - dtheta_2*l_2*l_3*m_5*np.sin(theta_3 - theta_2) - dtheta_2*l_2*m_3*r_3*np.sin(theta_3 - theta_2)) + dtheta_1*(-dtheta_1*l_1*l_3*m_4*np.sin(theta_3 - theta_1) - dtheta_1*l_1*l_3*m_5*np.sin(theta_3 - theta_1) - dtheta_1*l_1*m_3*r_3*np.sin(theta_3 - theta_1)) - ddx_c*(-l_3*m_4 - l_3*m_5 - m_3*r_3)*np.cos(theta_3) - g*(-l_3*m_4 - l_3*m_5 - m_3*r_3)*np.sin(theta_3) - l_1*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(dtheta_5**2*l_1*m_5*r_5*np.sin(theta_5 - theta_1) + dtheta_4*(dtheta_4*l_1*l_4*m_5*np.sin(theta_4 - theta_1) + dtheta_4*l_1*m_4*r_4*np.sin(theta_4 - theta_1)) + dtheta_3*(dtheta_3*l_1*l_3*m_4*np.sin(theta_3 - theta_1) + dtheta_3*l_1*l_3*m_5*np.sin(theta_3 - theta_1) + dtheta_3*l_1*m_3*r_3*np.sin(theta_3 - theta_1)) + dtheta_2*mu_2 + dtheta_2*(dtheta_2*l_1*l_2*m_3*np.sin(theta_2 - theta_1) + dtheta_2*l_1*l_2*m_4*np.sin(theta_2 - theta_1) + dtheta_2*l_1*l_2*m_5*np.sin(theta_2 - theta_1) + dtheta_2*l_1*m_2*r_2*np.sin(theta_2 - theta_1)) - dtheta_1*(mu_1 + mu_2) - ddx_c*(-l_1*m_2 - l_1*m_3 - l_1*m_4 - l_1*m_5 - m_1*r_1)*np.cos(theta_1) - g*(-l_1*m_2 - l_1*m_3 - l_1*m_4 - l_1*m_5 - m_1*r_1)*np.sin(theta_1))*np.cos(theta_3 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) - (-l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_3 - theta_2))*(dtheta_5**2*l_2*m_5*r_5*np.sin(theta_5 - theta_2) + dtheta_4*(dtheta_4*l_2*l_4*m_5*np.sin(theta_4 - theta_2) + dtheta_4*l_2*m_4*r_4*np.sin(theta_4 - theta_2)) + dtheta_3*mu_3 + dtheta_3*(dtheta_3*l_2*l_3*m_4*np.sin(theta_3 - theta_2) + dtheta_3*l_2*l_3*m_5*np.sin(theta_3 - theta_2) + dtheta_3*l_2*m_3*r_3*np.sin(theta_3 - theta_2)) - dtheta_2*(mu_2 + mu_3) + dtheta_1*mu_2 + dtheta_1*(-dtheta_1*l_1*l_2*m_3*np.sin(theta_2 - theta_1) - dtheta_1*l_1*l_2*m_4*np.sin(theta_2 - theta_1) - dtheta_1*l_1*l_2*m_5*np.sin(theta_2 - theta_1) - dtheta_1*l_1*m_2*r_2*np.sin(theta_2 - theta_1)) - ddx_c*(-l_2*m_3 - l_2*m_4 - l_2*m_5 - m_2*r_2)*np.cos(theta_2) - g*(-l_2*m_3 - l_2*m_4 - l_2*m_5 - m_2*r_2)*np.sin(theta_2) - l_1*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*(dtheta_5**2*l_1*m_5*r_5*np.sin(theta_5 - theta_1) + dtheta_4*(dtheta_4*l_1*l_4*m_5*np.sin(theta_4 - theta_1) + dtheta_4*l_1*m_4*r_4*np.sin(theta_4 - theta_1)) + dtheta_3*(dtheta_3*l_1*l_3*m_4*np.sin(theta_3 - theta_1) + dtheta_3*l_1*l_3*m_5*np.sin(theta_3 - theta_1) + dtheta_3*l_1*m_3*r_3*np.sin(theta_3 - theta_1)) + dtheta_2*mu_2 + dtheta_2*(dtheta_2*l_1*l_2*m_3*np.sin(theta_2 - theta_1) + dtheta_2*l_1*l_2*m_4*np.sin(theta_2 - theta_1) + dtheta_2*l_1*l_2*m_5*np.sin(theta_2 - theta_1) + dtheta_2*l_1*m_2*r_2*np.sin(theta_2 - theta_1)) - dtheta_1*(mu_1 + mu_2) - ddx_c*(-l_1*m_2 - l_1*m_3 - l_1*m_4 - l_1*m_5 - m_1*r_1)*np.cos(theta_1) - g*(-l_1*m_2 - l_1*m_3 - l_1*m_4 - l_1*m_5 - m_1*r_1)*np.sin(theta_1))*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2))/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2))/(I_3 - l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)**2*np.cos(theta_3 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_3**2*m_4 + l_3**2*m_5 + m_3*r_3**2 - (-l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_3 - theta_2))**2/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2)))/(I_4 - l_1**2*(l_4*m_5 + m_4*r_4)**2*np.cos(theta_4 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_4**2*m_5 + m_4*r_4**2 - (-l_1**2*(l_4*m_5 + m_4*r_4)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_4 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_4*m_5 + m_4*r_4)*np.cos(theta_4 - theta_2))**2/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2) - (-l_1**2*(l_4*m_5 + m_4*r_4)*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_4 - theta_1)*np.cos(theta_3 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_3*(l_4*m_5 + m_4*r_4)*np.cos(theta_4 - theta_3) - (-l_1**2*(l_4*m_5 + m_4*r_4)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_4 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_4*m_5 + m_4*r_4)*np.cos(theta_4 - theta_2))*(-l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_3 - theta_2))/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2))**2/(I_3 - l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)**2*np.cos(theta_3 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_3**2*m_4 + l_3**2*m_5 + m_3*r_3**2 - (-l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_3 - theta_2))**2/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2))))/(I_5 - l_1**2*m_5**2*r_5**2*np.cos(theta_5 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + m_5*r_5**2 - (-l_1**2*m_5*r_5*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_5 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*m_5*r_5*np.cos(theta_5 - theta_2))**2/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2) - (-l_1**2*m_5*r_5*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_5 - theta_1)*np.cos(theta_3 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_3*m_5*r_5*np.cos(theta_5 - theta_3) - (-l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_3 - theta_2))*(-l_1**2*m_5*r_5*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_5 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*m_5*r_5*np.cos(theta_5 - theta_2))/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2))**2/(I_3 - l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)**2*np.cos(theta_3 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_3**2*m_4 + l_3**2*m_5 + m_3*r_3**2 - (-l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_3 - theta_2))**2/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2)) - (-l_1**2*m_5*r_5*(l_4*m_5 + m_4*r_4)*np.cos(theta_5 - theta_1)*np.cos(theta_4 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_4*m_5*r_5*np.cos(theta_5 - theta_4) - (-l_1**2*(l_4*m_5 + m_4*r_4)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_4 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_4*m_5 + m_4*r_4)*np.cos(theta_4 - theta_2))*(-l_1**2*m_5*r_5*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_5 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*m_5*r_5*np.cos(theta_5 - theta_2))/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2) - (-l_1**2*(l_4*m_5 + m_4*r_4)*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_4 - theta_1)*np.cos(theta_3 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_3*(l_4*m_5 + m_4*r_4)*np.cos(theta_4 - theta_3) - (-l_1**2*(l_4*m_5 + m_4*r_4)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_4 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_4*m_5 + m_4*r_4)*np.cos(theta_4 - theta_2))*(-l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_3 - theta_2))/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2))*(-l_1**2*m_5*r_5*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_5 - theta_1)*np.cos(theta_3 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_3*m_5*r_5*np.cos(theta_5 - theta_3) - (-l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_3 - theta_2))*(-l_1**2*m_5*r_5*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_5 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*m_5*r_5*np.cos(theta_5 - theta_2))/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2))/(I_3 - l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)**2*np.cos(theta_3 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_3**2*m_4 + l_3**2*m_5 + m_3*r_3**2 - (-l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_3 - theta_2))**2/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2)))**2/(I_4 - l_1**2*(l_4*m_5 + m_4*r_4)**2*np.cos(theta_4 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_4**2*m_5 + m_4*r_4**2 - (-l_1**2*(l_4*m_5 + m_4*r_4)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_4 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_4*m_5 + m_4*r_4)*np.cos(theta_4 - theta_2))**2/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2) - (-l_1**2*(l_4*m_5 + m_4*r_4)*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_4 - theta_1)*np.cos(theta_3 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_3*(l_4*m_5 + m_4*r_4)*np.cos(theta_4 - theta_3) - (-l_1**2*(l_4*m_5 + m_4*r_4)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_4 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_4*m_5 + m_4*r_4)*np.cos(theta_4 - theta_2))*(-l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_3 - theta_2))/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2))**2/(I_3 - l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)**2*np.cos(theta_3 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_3**2*m_4 + l_3**2*m_5 + m_3*r_3**2 - (-l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_3 - theta_2))**2/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2)))))/(I_4 - l_1**2*(l_4*m_5 + m_4*r_4)**2*np.cos(theta_4 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_4**2*m_5 + m_4*r_4**2 - (-l_1**2*(l_4*m_5 + m_4*r_4)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_4 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_4*m_5 + m_4*r_4)*np.cos(theta_4 - theta_2))**2/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2) - (-l_1**2*(l_4*m_5 + m_4*r_4)*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_4 - theta_1)*np.cos(theta_3 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_3*(l_4*m_5 + m_4*r_4)*np.cos(theta_4 - theta_3) - (-l_1**2*(l_4*m_5 + m_4*r_4)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_4 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_4*m_5 + m_4*r_4)*np.cos(theta_4 - theta_2))*(-l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_3 - theta_2))/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2))**2/(I_3 - l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)**2*np.cos(theta_3 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_3**2*m_4 + l_3**2*m_5 + m_3*r_3**2 - (-l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_3 - theta_2))**2/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2))) - (-l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_3 - theta_2))*(dtheta_5**2*l_3*m_5*r_5*np.sin(theta_5 - theta_3) + dtheta_4*mu_4 + dtheta_4*(dtheta_4*l_3*l_4*m_5*np.sin(theta_4 - theta_3) + dtheta_4*l_3*m_4*r_4*np.sin(theta_4 - theta_3)) - dtheta_3*(mu_3 + mu_4) + dtheta_2*mu_3 + dtheta_2*(-dtheta_2*l_2*l_3*m_4*np.sin(theta_3 - theta_2) - dtheta_2*l_2*l_3*m_5*np.sin(theta_3 - theta_2) - dtheta_2*l_2*m_3*r_3*np.sin(theta_3 - theta_2)) + dtheta_1*(-dtheta_1*l_1*l_3*m_4*np.sin(theta_3 - theta_1) - dtheta_1*l_1*l_3*m_5*np.sin(theta_3 - theta_1) - dtheta_1*l_1*m_3*r_3*np.sin(theta_3 - theta_1)) - ddx_c*(-l_3*m_4 - l_3*m_5 - m_3*r_3)*np.cos(theta_3) - g*(-l_3*m_4 - l_3*m_5 - m_3*r_3)*np.sin(theta_3) - l_1*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(dtheta_5**2*l_1*m_5*r_5*np.sin(theta_5 - theta_1) + dtheta_4*(dtheta_4*l_1*l_4*m_5*np.sin(theta_4 - theta_1) + dtheta_4*l_1*m_4*r_4*np.sin(theta_4 - theta_1)) + dtheta_3*(dtheta_3*l_1*l_3*m_4*np.sin(theta_3 - theta_1) + dtheta_3*l_1*l_3*m_5*np.sin(theta_3 - theta_1) + dtheta_3*l_1*m_3*r_3*np.sin(theta_3 - theta_1)) + dtheta_2*mu_2 + dtheta_2*(dtheta_2*l_1*l_2*m_3*np.sin(theta_2 - theta_1) + dtheta_2*l_1*l_2*m_4*np.sin(theta_2 - theta_1) + dtheta_2*l_1*l_2*m_5*np.sin(theta_2 - theta_1) + dtheta_2*l_1*m_2*r_2*np.sin(theta_2 - theta_1)) - dtheta_1*(mu_1 + mu_2) - ddx_c*(-l_1*m_2 - l_1*m_3 - l_1*m_4 - l_1*m_5 - m_1*r_1)*np.cos(theta_1) - g*(-l_1*m_2 - l_1*m_3 - l_1*m_4 - l_1*m_5 - m_1*r_1)*np.sin(theta_1))*np.cos(theta_3 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) - (-l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_3 - theta_2))*(dtheta_5**2*l_2*m_5*r_5*np.sin(theta_5 - theta_2) + dtheta_4*(dtheta_4*l_2*l_4*m_5*np.sin(theta_4 - theta_2) + dtheta_4*l_2*m_4*r_4*np.sin(theta_4 - theta_2)) + dtheta_3*mu_3 + dtheta_3*(dtheta_3*l_2*l_3*m_4*np.sin(theta_3 - theta_2) + dtheta_3*l_2*l_3*m_5*np.sin(theta_3 - theta_2) + dtheta_3*l_2*m_3*r_3*np.sin(theta_3 - theta_2)) - dtheta_2*(mu_2 + mu_3) + dtheta_1*mu_2 + dtheta_1*(-dtheta_1*l_1*l_2*m_3*np.sin(theta_2 - theta_1) - dtheta_1*l_1*l_2*m_4*np.sin(theta_2 - theta_1) - dtheta_1*l_1*l_2*m_5*np.sin(theta_2 - theta_1) - dtheta_1*l_1*m_2*r_2*np.sin(theta_2 - theta_1)) - ddx_c*(-l_2*m_3 - l_2*m_4 - l_2*m_5 - m_2*r_2)*np.cos(theta_2) - g*(-l_2*m_3 - l_2*m_4 - l_2*m_5 - m_2*r_2)*np.sin(theta_2) - l_1*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*(dtheta_5**2*l_1*m_5*r_5*np.sin(theta_5 - theta_1) + dtheta_4*(dtheta_4*l_1*l_4*m_5*np.sin(theta_4 - theta_1) + dtheta_4*l_1*m_4*r_4*np.sin(theta_4 - theta_1)) + dtheta_3*(dtheta_3*l_1*l_3*m_4*np.sin(theta_3 - theta_1) + dtheta_3*l_1*l_3*m_5*np.sin(theta_3 - theta_1) + dtheta_3*l_1*m_3*r_3*np.sin(theta_3 - theta_1)) + dtheta_2*mu_2 + dtheta_2*(dtheta_2*l_1*l_2*m_3*np.sin(theta_2 - theta_1) + dtheta_2*l_1*l_2*m_4*np.sin(theta_2 - theta_1) + dtheta_2*l_1*l_2*m_5*np.sin(theta_2 - theta_1) + dtheta_2*l_1*m_2*r_2*np.sin(theta_2 - theta_1)) - dtheta_1*(mu_1 + mu_2) - ddx_c*(-l_1*m_2 - l_1*m_3 - l_1*m_4 - l_1*m_5 - m_1*r_1)*np.cos(theta_1) - g*(-l_1*m_2 - l_1*m_3 - l_1*m_4 - l_1*m_5 - m_1*r_1)*np.sin(theta_1))*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2))/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2) - (-l_1**2*(l_4*m_5 + m_4*r_4)*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_4 - theta_1)*np.cos(theta_3 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_3*(l_4*m_5 + m_4*r_4)*np.cos(theta_4 - theta_3) - (-l_1**2*(l_4*m_5 + m_4*r_4)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_4 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_4*m_5 + m_4*r_4)*np.cos(theta_4 - theta_2))*(-l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_3 - theta_2))/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2))*(dtheta_5**2*l_4*m_5*r_5*np.sin(theta_5 - theta_4) + dtheta_5*mu_5 - dtheta_4*(mu_4 + mu_5) + dtheta_3*mu_4 + dtheta_3*(-dtheta_3*l_3*l_4*m_5*np.sin(theta_4 - theta_3) - dtheta_3*l_3*m_4*r_4*np.sin(theta_4 - theta_3)) + dtheta_2*(-dtheta_2*l_2*l_4*m_5*np.sin(theta_4 - theta_2) - dtheta_2*l_2*m_4*r_4*np.sin(theta_4 - theta_2)) + dtheta_1*(-dtheta_1*l_1*l_4*m_5*np.sin(theta_4 - theta_1) - dtheta_1*l_1*m_4*r_4*np.sin(theta_4 - theta_1)) - ddx_c*(-l_4*m_5 - m_4*r_4)*np.cos(theta_4) - g*(-l_4*m_5 - m_4*r_4)*np.sin(theta_4) - l_1*(l_4*m_5 + m_4*r_4)*(dtheta_5**2*l_1*m_5*r_5*np.sin(theta_5 - theta_1) + dtheta_4*(dtheta_4*l_1*l_4*m_5*np.sin(theta_4 - theta_1) + dtheta_4*l_1*m_4*r_4*np.sin(theta_4 - theta_1)) + dtheta_3*(dtheta_3*l_1*l_3*m_4*np.sin(theta_3 - theta_1) + dtheta_3*l_1*l_3*m_5*np.sin(theta_3 - theta_1) + dtheta_3*l_1*m_3*r_3*np.sin(theta_3 - theta_1)) + dtheta_2*mu_2 + dtheta_2*(dtheta_2*l_1*l_2*m_3*np.sin(theta_2 - theta_1) + dtheta_2*l_1*l_2*m_4*np.sin(theta_2 - theta_1) + dtheta_2*l_1*l_2*m_5*np.sin(theta_2 - theta_1) + dtheta_2*l_1*m_2*r_2*np.sin(theta_2 - theta_1)) - dtheta_1*(mu_1 + mu_2) - ddx_c*(-l_1*m_2 - l_1*m_3 - l_1*m_4 - l_1*m_5 - m_1*r_1)*np.cos(theta_1) - g*(-l_1*m_2 - l_1*m_3 - l_1*m_4 - l_1*m_5 - m_1*r_1)*np.sin(theta_1))*np.cos(theta_4 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) - (-l_1**2*(l_4*m_5 + m_4*r_4)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_4 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_4*m_5 + m_4*r_4)*np.cos(theta_4 - theta_2))*(dtheta_5**2*l_2*m_5*r_5*np.sin(theta_5 - theta_2) + dtheta_4*(dtheta_4*l_2*l_4*m_5*np.sin(theta_4 - theta_2) + dtheta_4*l_2*m_4*r_4*np.sin(theta_4 - theta_2)) + dtheta_3*mu_3 + dtheta_3*(dtheta_3*l_2*l_3*m_4*np.sin(theta_3 - theta_2) + dtheta_3*l_2*l_3*m_5*np.sin(theta_3 - theta_2) + dtheta_3*l_2*m_3*r_3*np.sin(theta_3 - theta_2)) - dtheta_2*(mu_2 + mu_3) + dtheta_1*mu_2 + dtheta_1*(-dtheta_1*l_1*l_2*m_3*np.sin(theta_2 - theta_1) - dtheta_1*l_1*l_2*m_4*np.sin(theta_2 - theta_1) - dtheta_1*l_1*l_2*m_5*np.sin(theta_2 - theta_1) - dtheta_1*l_1*m_2*r_2*np.sin(theta_2 - theta_1)) - ddx_c*(-l_2*m_3 - l_2*m_4 - l_2*m_5 - m_2*r_2)*np.cos(theta_2) - g*(-l_2*m_3 - l_2*m_4 - l_2*m_5 - m_2*r_2)*np.sin(theta_2) - l_1*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*(dtheta_5**2*l_1*m_5*r_5*np.sin(theta_5 - theta_1) + dtheta_4*(dtheta_4*l_1*l_4*m_5*np.sin(theta_4 - theta_1) + dtheta_4*l_1*m_4*r_4*np.sin(theta_4 - theta_1)) + dtheta_3*(dtheta_3*l_1*l_3*m_4*np.sin(theta_3 - theta_1) + dtheta_3*l_1*l_3*m_5*np.sin(theta_3 - theta_1) + dtheta_3*l_1*m_3*r_3*np.sin(theta_3 - theta_1)) + dtheta_2*mu_2 + dtheta_2*(dtheta_2*l_1*l_2*m_3*np.sin(theta_2 - theta_1) + dtheta_2*l_1*l_2*m_4*np.sin(theta_2 - theta_1) + dtheta_2*l_1*l_2*m_5*np.sin(theta_2 - theta_1) + dtheta_2*l_1*m_2*r_2*np.sin(theta_2 - theta_1)) - dtheta_1*(mu_1 + mu_2) - ddx_c*(-l_1*m_2 - l_1*m_3 - l_1*m_4 - l_1*m_5 - m_1*r_1)*np.cos(theta_1) - g*(-l_1*m_2 - l_1*m_3 - l_1*m_4 - l_1*m_5 - m_1*r_1)*np.sin(theta_1))*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2))/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2) - (-l_1**2*(l_4*m_5 + m_4*r_4)*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_4 - theta_1)*np.cos(theta_3 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_3*(l_4*m_5 + m_4*r_4)*np.cos(theta_4 - theta_3) - (-l_1**2*(l_4*m_5 + m_4*r_4)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_4 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_4*m_5 + m_4*r_4)*np.cos(theta_4 - theta_2))*(-l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_3 - theta_2))/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2))*(dtheta_5**2*l_3*m_5*r_5*np.sin(theta_5 - theta_3) + dtheta_4*mu_4 + dtheta_4*(dtheta_4*l_3*l_4*m_5*np.sin(theta_4 - theta_3) + dtheta_4*l_3*m_4*r_4*np.sin(theta_4 - theta_3)) - dtheta_3*(mu_3 + mu_4) + dtheta_2*mu_3 + dtheta_2*(-dtheta_2*l_2*l_3*m_4*np.sin(theta_3 - theta_2) - dtheta_2*l_2*l_3*m_5*np.sin(theta_3 - theta_2) - dtheta_2*l_2*m_3*r_3*np.sin(theta_3 - theta_2)) + dtheta_1*(-dtheta_1*l_1*l_3*m_4*np.sin(theta_3 - theta_1) - dtheta_1*l_1*l_3*m_5*np.sin(theta_3 - theta_1) - dtheta_1*l_1*m_3*r_3*np.sin(theta_3 - theta_1)) - ddx_c*(-l_3*m_4 - l_3*m_5 - m_3*r_3)*np.cos(theta_3) - g*(-l_3*m_4 - l_3*m_5 - m_3*r_3)*np.sin(theta_3) - l_1*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(dtheta_5**2*l_1*m_5*r_5*np.sin(theta_5 - theta_1) + dtheta_4*(dtheta_4*l_1*l_4*m_5*np.sin(theta_4 - theta_1) + dtheta_4*l_1*m_4*r_4*np.sin(theta_4 - theta_1)) + dtheta_3*(dtheta_3*l_1*l_3*m_4*np.sin(theta_3 - theta_1) + dtheta_3*l_1*l_3*m_5*np.sin(theta_3 - theta_1) + dtheta_3*l_1*m_3*r_3*np.sin(theta_3 - theta_1)) + dtheta_2*mu_2 + dtheta_2*(dtheta_2*l_1*l_2*m_3*np.sin(theta_2 - theta_1) + dtheta_2*l_1*l_2*m_4*np.sin(theta_2 - theta_1) + dtheta_2*l_1*l_2*m_5*np.sin(theta_2 - theta_1) + dtheta_2*l_1*m_2*r_2*np.sin(theta_2 - theta_1)) - dtheta_1*(mu_1 + mu_2) - ddx_c*(-l_1*m_2 - l_1*m_3 - l_1*m_4 - l_1*m_5 - m_1*r_1)*np.cos(theta_1) - g*(-l_1*m_2 - l_1*m_3 - l_1*m_4 - l_1*m_5 - m_1*r_1)*np.sin(theta_1))*np.cos(theta_3 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) - (-l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_3 - theta_2))*(dtheta_5**2*l_2*m_5*r_5*np.sin(theta_5 - theta_2) + dtheta_4*(dtheta_4*l_2*l_4*m_5*np.sin(theta_4 - theta_2) + dtheta_4*l_2*m_4*r_4*np.sin(theta_4 - theta_2)) + dtheta_3*mu_3 + dtheta_3*(dtheta_3*l_2*l_3*m_4*np.sin(theta_3 - theta_2) + dtheta_3*l_2*l_3*m_5*np.sin(theta_3 - theta_2) + dtheta_3*l_2*m_3*r_3*np.sin(theta_3 - theta_2)) - dtheta_2*(mu_2 + mu_3) + dtheta_1*mu_2 + dtheta_1*(-dtheta_1*l_1*l_2*m_3*np.sin(theta_2 - theta_1) - dtheta_1*l_1*l_2*m_4*np.sin(theta_2 - theta_1) - dtheta_1*l_1*l_2*m_5*np.sin(theta_2 - theta_1) - dtheta_1*l_1*m_2*r_2*np.sin(theta_2 - theta_1)) - ddx_c*(-l_2*m_3 - l_2*m_4 - l_2*m_5 - m_2*r_2)*np.cos(theta_2) - g*(-l_2*m_3 - l_2*m_4 - l_2*m_5 - m_2*r_2)*np.sin(theta_2) - l_1*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*(dtheta_5**2*l_1*m_5*r_5*np.sin(theta_5 - theta_1) + dtheta_4*(dtheta_4*l_1*l_4*m_5*np.sin(theta_4 - theta_1) + dtheta_4*l_1*m_4*r_4*np.sin(theta_4 - theta_1)) + dtheta_3*(dtheta_3*l_1*l_3*m_4*np.sin(theta_3 - theta_1) + dtheta_3*l_1*l_3*m_5*np.sin(theta_3 - theta_1) + dtheta_3*l_1*m_3*r_3*np.sin(theta_3 - theta_1)) + dtheta_2*mu_2 + dtheta_2*(dtheta_2*l_1*l_2*m_3*np.sin(theta_2 - theta_1) + dtheta_2*l_1*l_2*m_4*np.sin(theta_2 - theta_1) + dtheta_2*l_1*l_2*m_5*np.sin(theta_2 - theta_1) + dtheta_2*l_1*m_2*r_2*np.sin(theta_2 - theta_1)) - dtheta_1*(mu_1 + mu_2) - ddx_c*(-l_1*m_2 - l_1*m_3 - l_1*m_4 - l_1*m_5 - m_1*r_1)*np.cos(theta_1) - g*(-l_1*m_2 - l_1*m_3 - l_1*m_4 - l_1*m_5 - m_1*r_1)*np.sin(theta_1))*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2))/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2))/(I_3 - l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)**2*np.cos(theta_3 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_3**2*m_4 + l_3**2*m_5 + m_3*r_3**2 - (-l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_3 - theta_2))**2/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2)) - (-l_1**2*m_5*r_5*(l_4*m_5 + m_4*r_4)*np.cos(theta_5 - theta_1)*np.cos(theta_4 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_4*m_5*r_5*np.cos(theta_5 - theta_4) - (-l_1**2*(l_4*m_5 + m_4*r_4)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_4 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_4*m_5 + m_4*r_4)*np.cos(theta_4 - theta_2))*(-l_1**2*m_5*r_5*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_5 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*m_5*r_5*np.cos(theta_5 - theta_2))/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2) - (-l_1**2*(l_4*m_5 + m_4*r_4)*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_4 - theta_1)*np.cos(theta_3 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_3*(l_4*m_5 + m_4*r_4)*np.cos(theta_4 - theta_3) - (-l_1**2*(l_4*m_5 + m_4*r_4)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_4 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_4*m_5 + m_4*r_4)*np.cos(theta_4 - theta_2))*(-l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_3 - theta_2))/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2))*(-l_1**2*m_5*r_5*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_5 - theta_1)*np.cos(theta_3 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_3*m_5*r_5*np.cos(theta_5 - theta_3) - (-l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_3 - theta_2))*(-l_1**2*m_5*r_5*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_5 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*m_5*r_5*np.cos(theta_5 - theta_2))/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2))/(I_3 - l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)**2*np.cos(theta_3 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_3**2*m_4 + l_3**2*m_5 + m_3*r_3**2 - (-l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_3 - theta_2))**2/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2)))*(-dtheta_5*mu_5 - dtheta_4**2*l_4*m_5*r_5*np.sin(theta_5 - theta_4) + dtheta_4*mu_5 - dtheta_3**2*l_3*m_5*r_5*np.sin(theta_5 - theta_3) - dtheta_2**2*l_2*m_5*r_5*np.sin(theta_5 - theta_2) - dtheta_1**2*l_1*m_5*r_5*np.sin(theta_5 - theta_1) + ddx_c*m_5*r_5*np.cos(theta_5) + g*m_5*r_5*np.sin(theta_5) - l_1*m_5*r_5*(dtheta_5**2*l_1*m_5*r_5*np.sin(theta_5 - theta_1) + dtheta_4*(dtheta_4*l_1*l_4*m_5*np.sin(theta_4 - theta_1) + dtheta_4*l_1*m_4*r_4*np.sin(theta_4 - theta_1)) + dtheta_3*(dtheta_3*l_1*l_3*m_4*np.sin(theta_3 - theta_1) + dtheta_3*l_1*l_3*m_5*np.sin(theta_3 - theta_1) + dtheta_3*l_1*m_3*r_3*np.sin(theta_3 - theta_1)) + dtheta_2*mu_2 + dtheta_2*(dtheta_2*l_1*l_2*m_3*np.sin(theta_2 - theta_1) + dtheta_2*l_1*l_2*m_4*np.sin(theta_2 - theta_1) + dtheta_2*l_1*l_2*m_5*np.sin(theta_2 - theta_1) + dtheta_2*l_1*m_2*r_2*np.sin(theta_2 - theta_1)) - dtheta_1*(mu_1 + mu_2) - ddx_c*(-l_1*m_2 - l_1*m_3 - l_1*m_4 - l_1*m_5 - m_1*r_1)*np.cos(theta_1) - g*(-l_1*m_2 - l_1*m_3 - l_1*m_4 - l_1*m_5 - m_1*r_1)*np.sin(theta_1))*np.cos(theta_5 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) - (-l_1**2*m_5*r_5*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_5 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*m_5*r_5*np.cos(theta_5 - theta_2))*(dtheta_5**2*l_2*m_5*r_5*np.sin(theta_5 - theta_2) + dtheta_4*(dtheta_4*l_2*l_4*m_5*np.sin(theta_4 - theta_2) + dtheta_4*l_2*m_4*r_4*np.sin(theta_4 - theta_2)) + dtheta_3*mu_3 + dtheta_3*(dtheta_3*l_2*l_3*m_4*np.sin(theta_3 - theta_2) + dtheta_3*l_2*l_3*m_5*np.sin(theta_3 - theta_2) + dtheta_3*l_2*m_3*r_3*np.sin(theta_3 - theta_2)) - dtheta_2*(mu_2 + mu_3) + dtheta_1*mu_2 + dtheta_1*(-dtheta_1*l_1*l_2*m_3*np.sin(theta_2 - theta_1) - dtheta_1*l_1*l_2*m_4*np.sin(theta_2 - theta_1) - dtheta_1*l_1*l_2*m_5*np.sin(theta_2 - theta_1) - dtheta_1*l_1*m_2*r_2*np.sin(theta_2 - theta_1)) - ddx_c*(-l_2*m_3 - l_2*m_4 - l_2*m_5 - m_2*r_2)*np.cos(theta_2) - g*(-l_2*m_3 - l_2*m_4 - l_2*m_5 - m_2*r_2)*np.sin(theta_2) - l_1*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*(dtheta_5**2*l_1*m_5*r_5*np.sin(theta_5 - theta_1) + dtheta_4*(dtheta_4*l_1*l_4*m_5*np.sin(theta_4 - theta_1) + dtheta_4*l_1*m_4*r_4*np.sin(theta_4 - theta_1)) + dtheta_3*(dtheta_3*l_1*l_3*m_4*np.sin(theta_3 - theta_1) + dtheta_3*l_1*l_3*m_5*np.sin(theta_3 - theta_1) + dtheta_3*l_1*m_3*r_3*np.sin(theta_3 - theta_1)) + dtheta_2*mu_2 + dtheta_2*(dtheta_2*l_1*l_2*m_3*np.sin(theta_2 - theta_1) + dtheta_2*l_1*l_2*m_4*np.sin(theta_2 - theta_1) + dtheta_2*l_1*l_2*m_5*np.sin(theta_2 - theta_1) + dtheta_2*l_1*m_2*r_2*np.sin(theta_2 - theta_1)) - dtheta_1*(mu_1 + mu_2) - ddx_c*(-l_1*m_2 - l_1*m_3 - l_1*m_4 - l_1*m_5 - m_1*r_1)*np.cos(theta_1) - g*(-l_1*m_2 - l_1*m_3 - l_1*m_4 - l_1*m_5 - m_1*r_1)*np.sin(theta_1))*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2))/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2) - (-l_1**2*m_5*r_5*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_5 - theta_1)*np.cos(theta_3 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_3*m_5*r_5*np.cos(theta_5 - theta_3) - (-l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_3 - theta_2))*(-l_1**2*m_5*r_5*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_5 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*m_5*r_5*np.cos(theta_5 - theta_2))/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2))*(dtheta_5**2*l_3*m_5*r_5*np.sin(theta_5 - theta_3) + dtheta_4*mu_4 + dtheta_4*(dtheta_4*l_3*l_4*m_5*np.sin(theta_4 - theta_3) + dtheta_4*l_3*m_4*r_4*np.sin(theta_4 - theta_3)) - dtheta_3*(mu_3 + mu_4) + dtheta_2*mu_3 + dtheta_2*(-dtheta_2*l_2*l_3*m_4*np.sin(theta_3 - theta_2) - dtheta_2*l_2*l_3*m_5*np.sin(theta_3 - theta_2) - dtheta_2*l_2*m_3*r_3*np.sin(theta_3 - theta_2)) + dtheta_1*(-dtheta_1*l_1*l_3*m_4*np.sin(theta_3 - theta_1) - dtheta_1*l_1*l_3*m_5*np.sin(theta_3 - theta_1) - dtheta_1*l_1*m_3*r_3*np.sin(theta_3 - theta_1)) - ddx_c*(-l_3*m_4 - l_3*m_5 - m_3*r_3)*np.cos(theta_3) - g*(-l_3*m_4 - l_3*m_5 - m_3*r_3)*np.sin(theta_3) - l_1*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(dtheta_5**2*l_1*m_5*r_5*np.sin(theta_5 - theta_1) + dtheta_4*(dtheta_4*l_1*l_4*m_5*np.sin(theta_4 - theta_1) + dtheta_4*l_1*m_4*r_4*np.sin(theta_4 - theta_1)) + dtheta_3*(dtheta_3*l_1*l_3*m_4*np.sin(theta_3 - theta_1) + dtheta_3*l_1*l_3*m_5*np.sin(theta_3 - theta_1) + dtheta_3*l_1*m_3*r_3*np.sin(theta_3 - theta_1)) + dtheta_2*mu_2 + dtheta_2*(dtheta_2*l_1*l_2*m_3*np.sin(theta_2 - theta_1) + dtheta_2*l_1*l_2*m_4*np.sin(theta_2 - theta_1) + dtheta_2*l_1*l_2*m_5*np.sin(theta_2 - theta_1) + dtheta_2*l_1*m_2*r_2*np.sin(theta_2 - theta_1)) - dtheta_1*(mu_1 + mu_2) - ddx_c*(-l_1*m_2 - l_1*m_3 - l_1*m_4 - l_1*m_5 - m_1*r_1)*np.cos(theta_1) - g*(-l_1*m_2 - l_1*m_3 - l_1*m_4 - l_1*m_5 - m_1*r_1)*np.sin(theta_1))*np.cos(theta_3 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) - (-l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_3 - theta_2))*(dtheta_5**2*l_2*m_5*r_5*np.sin(theta_5 - theta_2) + dtheta_4*(dtheta_4*l_2*l_4*m_5*np.sin(theta_4 - theta_2) + dtheta_4*l_2*m_4*r_4*np.sin(theta_4 - theta_2)) + dtheta_3*mu_3 + dtheta_3*(dtheta_3*l_2*l_3*m_4*np.sin(theta_3 - theta_2) + dtheta_3*l_2*l_3*m_5*np.sin(theta_3 - theta_2) + dtheta_3*l_2*m_3*r_3*np.sin(theta_3 - theta_2)) - dtheta_2*(mu_2 + mu_3) + dtheta_1*mu_2 + dtheta_1*(-dtheta_1*l_1*l_2*m_3*np.sin(theta_2 - theta_1) - dtheta_1*l_1*l_2*m_4*np.sin(theta_2 - theta_1) - dtheta_1*l_1*l_2*m_5*np.sin(theta_2 - theta_1) - dtheta_1*l_1*m_2*r_2*np.sin(theta_2 - theta_1)) - ddx_c*(-l_2*m_3 - l_2*m_4 - l_2*m_5 - m_2*r_2)*np.cos(theta_2) - g*(-l_2*m_3 - l_2*m_4 - l_2*m_5 - m_2*r_2)*np.sin(theta_2) - l_1*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*(dtheta_5**2*l_1*m_5*r_5*np.sin(theta_5 - theta_1) + dtheta_4*(dtheta_4*l_1*l_4*m_5*np.sin(theta_4 - theta_1) + dtheta_4*l_1*m_4*r_4*np.sin(theta_4 - theta_1)) + dtheta_3*(dtheta_3*l_1*l_3*m_4*np.sin(theta_3 - theta_1) + dtheta_3*l_1*l_3*m_5*np.sin(theta_3 - theta_1) + dtheta_3*l_1*m_3*r_3*np.sin(theta_3 - theta_1)) + dtheta_2*mu_2 + dtheta_2*(dtheta_2*l_1*l_2*m_3*np.sin(theta_2 - theta_1) + dtheta_2*l_1*l_2*m_4*np.sin(theta_2 - theta_1) + dtheta_2*l_1*l_2*m_5*np.sin(theta_2 - theta_1) + dtheta_2*l_1*m_2*r_2*np.sin(theta_2 - theta_1)) - dtheta_1*(mu_1 + mu_2) - ddx_c*(-l_1*m_2 - l_1*m_3 - l_1*m_4 - l_1*m_5 - m_1*r_1)*np.cos(theta_1) - g*(-l_1*m_2 - l_1*m_3 - l_1*m_4 - l_1*m_5 - m_1*r_1)*np.sin(theta_1))*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2))/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2))/(I_3 - l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)**2*np.cos(theta_3 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_3**2*m_4 + l_3**2*m_5 + m_3*r_3**2 - (-l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_3 - theta_2))**2/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2)) - (-l_1**2*m_5*r_5*(l_4*m_5 + m_4*r_4)*np.cos(theta_5 - theta_1)*np.cos(theta_4 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_4*m_5*r_5*np.cos(theta_5 - theta_4) - (-l_1**2*(l_4*m_5 + m_4*r_4)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_4 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_4*m_5 + m_4*r_4)*np.cos(theta_4 - theta_2))*(-l_1**2*m_5*r_5*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_5 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*m_5*r_5*np.cos(theta_5 - theta_2))/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2) - (-l_1**2*(l_4*m_5 + m_4*r_4)*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_4 - theta_1)*np.cos(theta_3 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_3*(l_4*m_5 + m_4*r_4)*np.cos(theta_4 - theta_3) - (-l_1**2*(l_4*m_5 + m_4*r_4)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_4 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_4*m_5 + m_4*r_4)*np.cos(theta_4 - theta_2))*(-l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_3 - theta_2))/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2))*(-l_1**2*m_5*r_5*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_5 - theta_1)*np.cos(theta_3 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_3*m_5*r_5*np.cos(theta_5 - theta_3) - (-l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_3 - theta_2))*(-l_1**2*m_5*r_5*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_5 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*m_5*r_5*np.cos(theta_5 - theta_2))/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2))/(I_3 - l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)**2*np.cos(theta_3 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_3**2*m_4 + l_3**2*m_5 + m_3*r_3**2 - (-l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_3 - theta_2))**2/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2)))*(dtheta_5**2*l_4*m_5*r_5*np.sin(theta_5 - theta_4) + dtheta_5*mu_5 - dtheta_4*(mu_4 + mu_5) + dtheta_3*mu_4 + dtheta_3*(-dtheta_3*l_3*l_4*m_5*np.sin(theta_4 - theta_3) - dtheta_3*l_3*m_4*r_4*np.sin(theta_4 - theta_3)) + dtheta_2*(-dtheta_2*l_2*l_4*m_5*np.sin(theta_4 - theta_2) - dtheta_2*l_2*m_4*r_4*np.sin(theta_4 - theta_2)) + dtheta_1*(-dtheta_1*l_1*l_4*m_5*np.sin(theta_4 - theta_1) - dtheta_1*l_1*m_4*r_4*np.sin(theta_4 - theta_1)) - ddx_c*(-l_4*m_5 - m_4*r_4)*np.cos(theta_4) - g*(-l_4*m_5 - m_4*r_4)*np.sin(theta_4) - l_1*(l_4*m_5 + m_4*r_4)*(dtheta_5**2*l_1*m_5*r_5*np.sin(theta_5 - theta_1) + dtheta_4*(dtheta_4*l_1*l_4*m_5*np.sin(theta_4 - theta_1) + dtheta_4*l_1*m_4*r_4*np.sin(theta_4 - theta_1)) + dtheta_3*(dtheta_3*l_1*l_3*m_4*np.sin(theta_3 - theta_1) + dtheta_3*l_1*l_3*m_5*np.sin(theta_3 - theta_1) + dtheta_3*l_1*m_3*r_3*np.sin(theta_3 - theta_1)) + dtheta_2*mu_2 + dtheta_2*(dtheta_2*l_1*l_2*m_3*np.sin(theta_2 - theta_1) + dtheta_2*l_1*l_2*m_4*np.sin(theta_2 - theta_1) + dtheta_2*l_1*l_2*m_5*np.sin(theta_2 - theta_1) + dtheta_2*l_1*m_2*r_2*np.sin(theta_2 - theta_1)) - dtheta_1*(mu_1 + mu_2) - ddx_c*(-l_1*m_2 - l_1*m_3 - l_1*m_4 - l_1*m_5 - m_1*r_1)*np.cos(theta_1) - g*(-l_1*m_2 - l_1*m_3 - l_1*m_4 - l_1*m_5 - m_1*r_1)*np.sin(theta_1))*np.cos(theta_4 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) - (-l_1**2*(l_4*m_5 + m_4*r_4)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_4 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_4*m_5 + m_4*r_4)*np.cos(theta_4 - theta_2))*(dtheta_5**2*l_2*m_5*r_5*np.sin(theta_5 - theta_2) + dtheta_4*(dtheta_4*l_2*l_4*m_5*np.sin(theta_4 - theta_2) + dtheta_4*l_2*m_4*r_4*np.sin(theta_4 - theta_2)) + dtheta_3*mu_3 + dtheta_3*(dtheta_3*l_2*l_3*m_4*np.sin(theta_3 - theta_2) + dtheta_3*l_2*l_3*m_5*np.sin(theta_3 - theta_2) + dtheta_3*l_2*m_3*r_3*np.sin(theta_3 - theta_2)) - dtheta_2*(mu_2 + mu_3) + dtheta_1*mu_2 + dtheta_1*(-dtheta_1*l_1*l_2*m_3*np.sin(theta_2 - theta_1) - dtheta_1*l_1*l_2*m_4*np.sin(theta_2 - theta_1) - dtheta_1*l_1*l_2*m_5*np.sin(theta_2 - theta_1) - dtheta_1*l_1*m_2*r_2*np.sin(theta_2 - theta_1)) - ddx_c*(-l_2*m_3 - l_2*m_4 - l_2*m_5 - m_2*r_2)*np.cos(theta_2) - g*(-l_2*m_3 - l_2*m_4 - l_2*m_5 - m_2*r_2)*np.sin(theta_2) - l_1*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*(dtheta_5**2*l_1*m_5*r_5*np.sin(theta_5 - theta_1) + dtheta_4*(dtheta_4*l_1*l_4*m_5*np.sin(theta_4 - theta_1) + dtheta_4*l_1*m_4*r_4*np.sin(theta_4 - theta_1)) + dtheta_3*(dtheta_3*l_1*l_3*m_4*np.sin(theta_3 - theta_1) + dtheta_3*l_1*l_3*m_5*np.sin(theta_3 - theta_1) + dtheta_3*l_1*m_3*r_3*np.sin(theta_3 - theta_1)) + dtheta_2*mu_2 + dtheta_2*(dtheta_2*l_1*l_2*m_3*np.sin(theta_2 - theta_1) + dtheta_2*l_1*l_2*m_4*np.sin(theta_2 - theta_1) + dtheta_2*l_1*l_2*m_5*np.sin(theta_2 - theta_1) + dtheta_2*l_1*m_2*r_2*np.sin(theta_2 - theta_1)) - dtheta_1*(mu_1 + mu_2) - ddx_c*(-l_1*m_2 - l_1*m_3 - l_1*m_4 - l_1*m_5 - m_1*r_1)*np.cos(theta_1) - g*(-l_1*m_2 - l_1*m_3 - l_1*m_4 - l_1*m_5 - m_1*r_1)*np.sin(theta_1))*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2))/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2) - (-l_1**2*(l_4*m_5 + m_4*r_4)*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_4 - theta_1)*np.cos(theta_3 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_3*(l_4*m_5 + m_4*r_4)*np.cos(theta_4 - theta_3) - (-l_1**2*(l_4*m_5 + m_4*r_4)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_4 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_4*m_5 + m_4*r_4)*np.cos(theta_4 - theta_2))*(-l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_3 - theta_2))/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2))*(dtheta_5**2*l_3*m_5*r_5*np.sin(theta_5 - theta_3) + dtheta_4*mu_4 + dtheta_4*(dtheta_4*l_3*l_4*m_5*np.sin(theta_4 - theta_3) + dtheta_4*l_3*m_4*r_4*np.sin(theta_4 - theta_3)) - dtheta_3*(mu_3 + mu_4) + dtheta_2*mu_3 + dtheta_2*(-dtheta_2*l_2*l_3*m_4*np.sin(theta_3 - theta_2) - dtheta_2*l_2*l_3*m_5*np.sin(theta_3 - theta_2) - dtheta_2*l_2*m_3*r_3*np.sin(theta_3 - theta_2)) + dtheta_1*(-dtheta_1*l_1*l_3*m_4*np.sin(theta_3 - theta_1) - dtheta_1*l_1*l_3*m_5*np.sin(theta_3 - theta_1) - dtheta_1*l_1*m_3*r_3*np.sin(theta_3 - theta_1)) - ddx_c*(-l_3*m_4 - l_3*m_5 - m_3*r_3)*np.cos(theta_3) - g*(-l_3*m_4 - l_3*m_5 - m_3*r_3)*np.sin(theta_3) - l_1*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(dtheta_5**2*l_1*m_5*r_5*np.sin(theta_5 - theta_1) + dtheta_4*(dtheta_4*l_1*l_4*m_5*np.sin(theta_4 - theta_1) + dtheta_4*l_1*m_4*r_4*np.sin(theta_4 - theta_1)) + dtheta_3*(dtheta_3*l_1*l_3*m_4*np.sin(theta_3 - theta_1) + dtheta_3*l_1*l_3*m_5*np.sin(theta_3 - theta_1) + dtheta_3*l_1*m_3*r_3*np.sin(theta_3 - theta_1)) + dtheta_2*mu_2 + dtheta_2*(dtheta_2*l_1*l_2*m_3*np.sin(theta_2 - theta_1) + dtheta_2*l_1*l_2*m_4*np.sin(theta_2 - theta_1) + dtheta_2*l_1*l_2*m_5*np.sin(theta_2 - theta_1) + dtheta_2*l_1*m_2*r_2*np.sin(theta_2 - theta_1)) - dtheta_1*(mu_1 + mu_2) - ddx_c*(-l_1*m_2 - l_1*m_3 - l_1*m_4 - l_1*m_5 - m_1*r_1)*np.cos(theta_1) - g*(-l_1*m_2 - l_1*m_3 - l_1*m_4 - l_1*m_5 - m_1*r_1)*np.sin(theta_1))*np.cos(theta_3 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) - (-l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_3 - theta_2))*(dtheta_5**2*l_2*m_5*r_5*np.sin(theta_5 - theta_2) + dtheta_4*(dtheta_4*l_2*l_4*m_5*np.sin(theta_4 - theta_2) + dtheta_4*l_2*m_4*r_4*np.sin(theta_4 - theta_2)) + dtheta_3*mu_3 + dtheta_3*(dtheta_3*l_2*l_3*m_4*np.sin(theta_3 - theta_2) + dtheta_3*l_2*l_3*m_5*np.sin(theta_3 - theta_2) + dtheta_3*l_2*m_3*r_3*np.sin(theta_3 - theta_2)) - dtheta_2*(mu_2 + mu_3) + dtheta_1*mu_2 + dtheta_1*(-dtheta_1*l_1*l_2*m_3*np.sin(theta_2 - theta_1) - dtheta_1*l_1*l_2*m_4*np.sin(theta_2 - theta_1) - dtheta_1*l_1*l_2*m_5*np.sin(theta_2 - theta_1) - dtheta_1*l_1*m_2*r_2*np.sin(theta_2 - theta_1)) - ddx_c*(-l_2*m_3 - l_2*m_4 - l_2*m_5 - m_2*r_2)*np.cos(theta_2) - g*(-l_2*m_3 - l_2*m_4 - l_2*m_5 - m_2*r_2)*np.sin(theta_2) - l_1*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*(dtheta_5**2*l_1*m_5*r_5*np.sin(theta_5 - theta_1) + dtheta_4*(dtheta_4*l_1*l_4*m_5*np.sin(theta_4 - theta_1) + dtheta_4*l_1*m_4*r_4*np.sin(theta_4 - theta_1)) + dtheta_3*(dtheta_3*l_1*l_3*m_4*np.sin(theta_3 - theta_1) + dtheta_3*l_1*l_3*m_5*np.sin(theta_3 - theta_1) + dtheta_3*l_1*m_3*r_3*np.sin(theta_3 - theta_1)) + dtheta_2*mu_2 + dtheta_2*(dtheta_2*l_1*l_2*m_3*np.sin(theta_2 - theta_1) + dtheta_2*l_1*l_2*m_4*np.sin(theta_2 - theta_1) + dtheta_2*l_1*l_2*m_5*np.sin(theta_2 - theta_1) + dtheta_2*l_1*m_2*r_2*np.sin(theta_2 - theta_1)) - dtheta_1*(mu_1 + mu_2) - ddx_c*(-l_1*m_2 - l_1*m_3 - l_1*m_4 - l_1*m_5 - m_1*r_1)*np.cos(theta_1) - g*(-l_1*m_2 - l_1*m_3 - l_1*m_4 - l_1*m_5 - m_1*r_1)*np.sin(theta_1))*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2))/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2))/(I_3 - l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)**2*np.cos(theta_3 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_3**2*m_4 + l_3**2*m_5 + m_3*r_3**2 - (-l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_3 - theta_2))**2/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2)))/(I_4 - l_1**2*(l_4*m_5 + m_4*r_4)**2*np.cos(theta_4 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_4**2*m_5 + m_4*r_4**2 - (-l_1**2*(l_4*m_5 + m_4*r_4)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_4 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_4*m_5 + m_4*r_4)*np.cos(theta_4 - theta_2))**2/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2) - (-l_1**2*(l_4*m_5 + m_4*r_4)*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_4 - theta_1)*np.cos(theta_3 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_3*(l_4*m_5 + m_4*r_4)*np.cos(theta_4 - theta_3) - (-l_1**2*(l_4*m_5 + m_4*r_4)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_4 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_4*m_5 + m_4*r_4)*np.cos(theta_4 - theta_2))*(-l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_3 - theta_2))/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2))**2/(I_3 - l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)**2*np.cos(theta_3 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_3**2*m_4 + l_3**2*m_5 + m_3*r_3**2 - (-l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_3 - theta_2))**2/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2))))/(I_5 - l_1**2*m_5**2*r_5**2*np.cos(theta_5 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + m_5*r_5**2 - (-l_1**2*m_5*r_5*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_5 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*m_5*r_5*np.cos(theta_5 - theta_2))**2/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2) - (-l_1**2*m_5*r_5*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_5 - theta_1)*np.cos(theta_3 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_3*m_5*r_5*np.cos(theta_5 - theta_3) - (-l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_3 - theta_2))*(-l_1**2*m_5*r_5*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_5 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*m_5*r_5*np.cos(theta_5 - theta_2))/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2))**2/(I_3 - l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)**2*np.cos(theta_3 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_3**2*m_4 + l_3**2*m_5 + m_3*r_3**2 - (-l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_3 - theta_2))**2/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2)) - (-l_1**2*m_5*r_5*(l_4*m_5 + m_4*r_4)*np.cos(theta_5 - theta_1)*np.cos(theta_4 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_4*m_5*r_5*np.cos(theta_5 - theta_4) - (-l_1**2*(l_4*m_5 + m_4*r_4)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_4 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_4*m_5 + m_4*r_4)*np.cos(theta_4 - theta_2))*(-l_1**2*m_5*r_5*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_5 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*m_5*r_5*np.cos(theta_5 - theta_2))/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2) - (-l_1**2*(l_4*m_5 + m_4*r_4)*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_4 - theta_1)*np.cos(theta_3 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_3*(l_4*m_5 + m_4*r_4)*np.cos(theta_4 - theta_3) - (-l_1**2*(l_4*m_5 + m_4*r_4)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_4 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_4*m_5 + m_4*r_4)*np.cos(theta_4 - theta_2))*(-l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_3 - theta_2))/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2))*(-l_1**2*m_5*r_5*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_5 - theta_1)*np.cos(theta_3 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_3*m_5*r_5*np.cos(theta_5 - theta_3) - (-l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_3 - theta_2))*(-l_1**2*m_5*r_5*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_5 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*m_5*r_5*np.cos(theta_5 - theta_2))/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2))/(I_3 - l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)**2*np.cos(theta_3 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_3**2*m_4 + l_3**2*m_5 + m_3*r_3**2 - (-l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_3 - theta_2))**2/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2)))**2/(I_4 - l_1**2*(l_4*m_5 + m_4*r_4)**2*np.cos(theta_4 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_4**2*m_5 + m_4*r_4**2 - (-l_1**2*(l_4*m_5 + m_4*r_4)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_4 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_4*m_5 + m_4*r_4)*np.cos(theta_4 - theta_2))**2/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2) - (-l_1**2*(l_4*m_5 + m_4*r_4)*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_4 - theta_1)*np.cos(theta_3 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_3*(l_4*m_5 + m_4*r_4)*np.cos(theta_4 - theta_3) - (-l_1**2*(l_4*m_5 + m_4*r_4)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_4 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_4*m_5 + m_4*r_4)*np.cos(theta_4 - theta_2))*(-l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_3 - theta_2))/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2))**2/(I_3 - l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)**2*np.cos(theta_3 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_3**2*m_4 + l_3**2*m_5 + m_3*r_3**2 - (-l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_3 - theta_2))**2/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2)))))/(I_4 - l_1**2*(l_4*m_5 + m_4*r_4)**2*np.cos(theta_4 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_4**2*m_5 + m_4*r_4**2 - (-l_1**2*(l_4*m_5 + m_4*r_4)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_4 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_4*m_5 + m_4*r_4)*np.cos(theta_4 - theta_2))**2/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2) - (-l_1**2*(l_4*m_5 + m_4*r_4)*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_4 - theta_1)*np.cos(theta_3 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_3*(l_4*m_5 + m_4*r_4)*np.cos(theta_4 - theta_3) - (-l_1**2*(l_4*m_5 + m_4*r_4)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_4 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_4*m_5 + m_4*r_4)*np.cos(theta_4 - theta_2))*(-l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_3 - theta_2))/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2))**2/(I_3 - l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)**2*np.cos(theta_3 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_3**2*m_4 + l_3**2*m_5 + m_3*r_3**2 - (-l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_3 - theta_2))**2/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2))) - (-l_1**2*m_5*r_5*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_5 - theta_1)*np.cos(theta_3 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_3*m_5*r_5*np.cos(theta_5 - theta_3) - (-l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_3 - theta_2))*(-l_1**2*m_5*r_5*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_5 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*m_5*r_5*np.cos(theta_5 - theta_2))/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2))*(-dtheta_5*mu_5 - dtheta_4**2*l_4*m_5*r_5*np.sin(theta_5 - theta_4) + dtheta_4*mu_5 - dtheta_3**2*l_3*m_5*r_5*np.sin(theta_5 - theta_3) - dtheta_2**2*l_2*m_5*r_5*np.sin(theta_5 - theta_2) - dtheta_1**2*l_1*m_5*r_5*np.sin(theta_5 - theta_1) + ddx_c*m_5*r_5*np.cos(theta_5) + g*m_5*r_5*np.sin(theta_5) - l_1*m_5*r_5*(dtheta_5**2*l_1*m_5*r_5*np.sin(theta_5 - theta_1) + dtheta_4*(dtheta_4*l_1*l_4*m_5*np.sin(theta_4 - theta_1) + dtheta_4*l_1*m_4*r_4*np.sin(theta_4 - theta_1)) + dtheta_3*(dtheta_3*l_1*l_3*m_4*np.sin(theta_3 - theta_1) + dtheta_3*l_1*l_3*m_5*np.sin(theta_3 - theta_1) + dtheta_3*l_1*m_3*r_3*np.sin(theta_3 - theta_1)) + dtheta_2*mu_2 + dtheta_2*(dtheta_2*l_1*l_2*m_3*np.sin(theta_2 - theta_1) + dtheta_2*l_1*l_2*m_4*np.sin(theta_2 - theta_1) + dtheta_2*l_1*l_2*m_5*np.sin(theta_2 - theta_1) + dtheta_2*l_1*m_2*r_2*np.sin(theta_2 - theta_1)) - dtheta_1*(mu_1 + mu_2) - ddx_c*(-l_1*m_2 - l_1*m_3 - l_1*m_4 - l_1*m_5 - m_1*r_1)*np.cos(theta_1) - g*(-l_1*m_2 - l_1*m_3 - l_1*m_4 - l_1*m_5 - m_1*r_1)*np.sin(theta_1))*np.cos(theta_5 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) - (-l_1**2*m_5*r_5*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_5 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*m_5*r_5*np.cos(theta_5 - theta_2))*(dtheta_5**2*l_2*m_5*r_5*np.sin(theta_5 - theta_2) + dtheta_4*(dtheta_4*l_2*l_4*m_5*np.sin(theta_4 - theta_2) + dtheta_4*l_2*m_4*r_4*np.sin(theta_4 - theta_2)) + dtheta_3*mu_3 + dtheta_3*(dtheta_3*l_2*l_3*m_4*np.sin(theta_3 - theta_2) + dtheta_3*l_2*l_3*m_5*np.sin(theta_3 - theta_2) + dtheta_3*l_2*m_3*r_3*np.sin(theta_3 - theta_2)) - dtheta_2*(mu_2 + mu_3) + dtheta_1*mu_2 + dtheta_1*(-dtheta_1*l_1*l_2*m_3*np.sin(theta_2 - theta_1) - dtheta_1*l_1*l_2*m_4*np.sin(theta_2 - theta_1) - dtheta_1*l_1*l_2*m_5*np.sin(theta_2 - theta_1) - dtheta_1*l_1*m_2*r_2*np.sin(theta_2 - theta_1)) - ddx_c*(-l_2*m_3 - l_2*m_4 - l_2*m_5 - m_2*r_2)*np.cos(theta_2) - g*(-l_2*m_3 - l_2*m_4 - l_2*m_5 - m_2*r_2)*np.sin(theta_2) - l_1*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*(dtheta_5**2*l_1*m_5*r_5*np.sin(theta_5 - theta_1) + dtheta_4*(dtheta_4*l_1*l_4*m_5*np.sin(theta_4 - theta_1) + dtheta_4*l_1*m_4*r_4*np.sin(theta_4 - theta_1)) + dtheta_3*(dtheta_3*l_1*l_3*m_4*np.sin(theta_3 - theta_1) + dtheta_3*l_1*l_3*m_5*np.sin(theta_3 - theta_1) + dtheta_3*l_1*m_3*r_3*np.sin(theta_3 - theta_1)) + dtheta_2*mu_2 + dtheta_2*(dtheta_2*l_1*l_2*m_3*np.sin(theta_2 - theta_1) + dtheta_2*l_1*l_2*m_4*np.sin(theta_2 - theta_1) + dtheta_2*l_1*l_2*m_5*np.sin(theta_2 - theta_1) + dtheta_2*l_1*m_2*r_2*np.sin(theta_2 - theta_1)) - dtheta_1*(mu_1 + mu_2) - ddx_c*(-l_1*m_2 - l_1*m_3 - l_1*m_4 - l_1*m_5 - m_1*r_1)*np.cos(theta_1) - g*(-l_1*m_2 - l_1*m_3 - l_1*m_4 - l_1*m_5 - m_1*r_1)*np.sin(theta_1))*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2))/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2) - (-l_1**2*m_5*r_5*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_5 - theta_1)*np.cos(theta_3 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_3*m_5*r_5*np.cos(theta_5 - theta_3) - (-l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_3 - theta_2))*(-l_1**2*m_5*r_5*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_5 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*m_5*r_5*np.cos(theta_5 - theta_2))/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2))*(dtheta_5**2*l_3*m_5*r_5*np.sin(theta_5 - theta_3) + dtheta_4*mu_4 + dtheta_4*(dtheta_4*l_3*l_4*m_5*np.sin(theta_4 - theta_3) + dtheta_4*l_3*m_4*r_4*np.sin(theta_4 - theta_3)) - dtheta_3*(mu_3 + mu_4) + dtheta_2*mu_3 + dtheta_2*(-dtheta_2*l_2*l_3*m_4*np.sin(theta_3 - theta_2) - dtheta_2*l_2*l_3*m_5*np.sin(theta_3 - theta_2) - dtheta_2*l_2*m_3*r_3*np.sin(theta_3 - theta_2)) + dtheta_1*(-dtheta_1*l_1*l_3*m_4*np.sin(theta_3 - theta_1) - dtheta_1*l_1*l_3*m_5*np.sin(theta_3 - theta_1) - dtheta_1*l_1*m_3*r_3*np.sin(theta_3 - theta_1)) - ddx_c*(-l_3*m_4 - l_3*m_5 - m_3*r_3)*np.cos(theta_3) - g*(-l_3*m_4 - l_3*m_5 - m_3*r_3)*np.sin(theta_3) - l_1*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(dtheta_5**2*l_1*m_5*r_5*np.sin(theta_5 - theta_1) + dtheta_4*(dtheta_4*l_1*l_4*m_5*np.sin(theta_4 - theta_1) + dtheta_4*l_1*m_4*r_4*np.sin(theta_4 - theta_1)) + dtheta_3*(dtheta_3*l_1*l_3*m_4*np.sin(theta_3 - theta_1) + dtheta_3*l_1*l_3*m_5*np.sin(theta_3 - theta_1) + dtheta_3*l_1*m_3*r_3*np.sin(theta_3 - theta_1)) + dtheta_2*mu_2 + dtheta_2*(dtheta_2*l_1*l_2*m_3*np.sin(theta_2 - theta_1) + dtheta_2*l_1*l_2*m_4*np.sin(theta_2 - theta_1) + dtheta_2*l_1*l_2*m_5*np.sin(theta_2 - theta_1) + dtheta_2*l_1*m_2*r_2*np.sin(theta_2 - theta_1)) - dtheta_1*(mu_1 + mu_2) - ddx_c*(-l_1*m_2 - l_1*m_3 - l_1*m_4 - l_1*m_5 - m_1*r_1)*np.cos(theta_1) - g*(-l_1*m_2 - l_1*m_3 - l_1*m_4 - l_1*m_5 - m_1*r_1)*np.sin(theta_1))*np.cos(theta_3 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) - (-l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_3 - theta_2))*(dtheta_5**2*l_2*m_5*r_5*np.sin(theta_5 - theta_2) + dtheta_4*(dtheta_4*l_2*l_4*m_5*np.sin(theta_4 - theta_2) + dtheta_4*l_2*m_4*r_4*np.sin(theta_4 - theta_2)) + dtheta_3*mu_3 + dtheta_3*(dtheta_3*l_2*l_3*m_4*np.sin(theta_3 - theta_2) + dtheta_3*l_2*l_3*m_5*np.sin(theta_3 - theta_2) + dtheta_3*l_2*m_3*r_3*np.sin(theta_3 - theta_2)) - dtheta_2*(mu_2 + mu_3) + dtheta_1*mu_2 + dtheta_1*(-dtheta_1*l_1*l_2*m_3*np.sin(theta_2 - theta_1) - dtheta_1*l_1*l_2*m_4*np.sin(theta_2 - theta_1) - dtheta_1*l_1*l_2*m_5*np.sin(theta_2 - theta_1) - dtheta_1*l_1*m_2*r_2*np.sin(theta_2 - theta_1)) - ddx_c*(-l_2*m_3 - l_2*m_4 - l_2*m_5 - m_2*r_2)*np.cos(theta_2) - g*(-l_2*m_3 - l_2*m_4 - l_2*m_5 - m_2*r_2)*np.sin(theta_2) - l_1*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*(dtheta_5**2*l_1*m_5*r_5*np.sin(theta_5 - theta_1) + dtheta_4*(dtheta_4*l_1*l_4*m_5*np.sin(theta_4 - theta_1) + dtheta_4*l_1*m_4*r_4*np.sin(theta_4 - theta_1)) + dtheta_3*(dtheta_3*l_1*l_3*m_4*np.sin(theta_3 - theta_1) + dtheta_3*l_1*l_3*m_5*np.sin(theta_3 - theta_1) + dtheta_3*l_1*m_3*r_3*np.sin(theta_3 - theta_1)) + dtheta_2*mu_2 + dtheta_2*(dtheta_2*l_1*l_2*m_3*np.sin(theta_2 - theta_1) + dtheta_2*l_1*l_2*m_4*np.sin(theta_2 - theta_1) + dtheta_2*l_1*l_2*m_5*np.sin(theta_2 - theta_1) + dtheta_2*l_1*m_2*r_2*np.sin(theta_2 - theta_1)) - dtheta_1*(mu_1 + mu_2) - ddx_c*(-l_1*m_2 - l_1*m_3 - l_1*m_4 - l_1*m_5 - m_1*r_1)*np.cos(theta_1) - g*(-l_1*m_2 - l_1*m_3 - l_1*m_4 - l_1*m_5 - m_1*r_1)*np.sin(theta_1))*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2))/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2))/(I_3 - l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)**2*np.cos(theta_3 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_3**2*m_4 + l_3**2*m_5 + m_3*r_3**2 - (-l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_3 - theta_2))**2/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2)) - (-l_1**2*m_5*r_5*(l_4*m_5 + m_4*r_4)*np.cos(theta_5 - theta_1)*np.cos(theta_4 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_4*m_5*r_5*np.cos(theta_5 - theta_4) - (-l_1**2*(l_4*m_5 + m_4*r_4)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_4 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_4*m_5 + m_4*r_4)*np.cos(theta_4 - theta_2))*(-l_1**2*m_5*r_5*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_5 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*m_5*r_5*np.cos(theta_5 - theta_2))/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2) - (-l_1**2*(l_4*m_5 + m_4*r_4)*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_4 - theta_1)*np.cos(theta_3 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_3*(l_4*m_5 + m_4*r_4)*np.cos(theta_4 - theta_3) - (-l_1**2*(l_4*m_5 + m_4*r_4)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_4 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_4*m_5 + m_4*r_4)*np.cos(theta_4 - theta_2))*(-l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_3 - theta_2))/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2))*(-l_1**2*m_5*r_5*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_5 - theta_1)*np.cos(theta_3 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_3*m_5*r_5*np.cos(theta_5 - theta_3) - (-l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_3 - theta_2))*(-l_1**2*m_5*r_5*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_5 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*m_5*r_5*np.cos(theta_5 - theta_2))/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2))/(I_3 - l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)**2*np.cos(theta_3 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_3**2*m_4 + l_3**2*m_5 + m_3*r_3**2 - (-l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_3 - theta_2))**2/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2)))*(dtheta_5**2*l_4*m_5*r_5*np.sin(theta_5 - theta_4) + dtheta_5*mu_5 - dtheta_4*(mu_4 + mu_5) + dtheta_3*mu_4 + dtheta_3*(-dtheta_3*l_3*l_4*m_5*np.sin(theta_4 - theta_3) - dtheta_3*l_3*m_4*r_4*np.sin(theta_4 - theta_3)) + dtheta_2*(-dtheta_2*l_2*l_4*m_5*np.sin(theta_4 - theta_2) - dtheta_2*l_2*m_4*r_4*np.sin(theta_4 - theta_2)) + dtheta_1*(-dtheta_1*l_1*l_4*m_5*np.sin(theta_4 - theta_1) - dtheta_1*l_1*m_4*r_4*np.sin(theta_4 - theta_1)) - ddx_c*(-l_4*m_5 - m_4*r_4)*np.cos(theta_4) - g*(-l_4*m_5 - m_4*r_4)*np.sin(theta_4) - l_1*(l_4*m_5 + m_4*r_4)*(dtheta_5**2*l_1*m_5*r_5*np.sin(theta_5 - theta_1) + dtheta_4*(dtheta_4*l_1*l_4*m_5*np.sin(theta_4 - theta_1) + dtheta_4*l_1*m_4*r_4*np.sin(theta_4 - theta_1)) + dtheta_3*(dtheta_3*l_1*l_3*m_4*np.sin(theta_3 - theta_1) + dtheta_3*l_1*l_3*m_5*np.sin(theta_3 - theta_1) + dtheta_3*l_1*m_3*r_3*np.sin(theta_3 - theta_1)) + dtheta_2*mu_2 + dtheta_2*(dtheta_2*l_1*l_2*m_3*np.sin(theta_2 - theta_1) + dtheta_2*l_1*l_2*m_4*np.sin(theta_2 - theta_1) + dtheta_2*l_1*l_2*m_5*np.sin(theta_2 - theta_1) + dtheta_2*l_1*m_2*r_2*np.sin(theta_2 - theta_1)) - dtheta_1*(mu_1 + mu_2) - ddx_c*(-l_1*m_2 - l_1*m_3 - l_1*m_4 - l_1*m_5 - m_1*r_1)*np.cos(theta_1) - g*(-l_1*m_2 - l_1*m_3 - l_1*m_4 - l_1*m_5 - m_1*r_1)*np.sin(theta_1))*np.cos(theta_4 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) - (-l_1**2*(l_4*m_5 + m_4*r_4)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_4 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_4*m_5 + m_4*r_4)*np.cos(theta_4 - theta_2))*(dtheta_5**2*l_2*m_5*r_5*np.sin(theta_5 - theta_2) + dtheta_4*(dtheta_4*l_2*l_4*m_5*np.sin(theta_4 - theta_2) + dtheta_4*l_2*m_4*r_4*np.sin(theta_4 - theta_2)) + dtheta_3*mu_3 + dtheta_3*(dtheta_3*l_2*l_3*m_4*np.sin(theta_3 - theta_2) + dtheta_3*l_2*l_3*m_5*np.sin(theta_3 - theta_2) + dtheta_3*l_2*m_3*r_3*np.sin(theta_3 - theta_2)) - dtheta_2*(mu_2 + mu_3) + dtheta_1*mu_2 + dtheta_1*(-dtheta_1*l_1*l_2*m_3*np.sin(theta_2 - theta_1) - dtheta_1*l_1*l_2*m_4*np.sin(theta_2 - theta_1) - dtheta_1*l_1*l_2*m_5*np.sin(theta_2 - theta_1) - dtheta_1*l_1*m_2*r_2*np.sin(theta_2 - theta_1)) - ddx_c*(-l_2*m_3 - l_2*m_4 - l_2*m_5 - m_2*r_2)*np.cos(theta_2) - g*(-l_2*m_3 - l_2*m_4 - l_2*m_5 - m_2*r_2)*np.sin(theta_2) - l_1*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*(dtheta_5**2*l_1*m_5*r_5*np.sin(theta_5 - theta_1) + dtheta_4*(dtheta_4*l_1*l_4*m_5*np.sin(theta_4 - theta_1) + dtheta_4*l_1*m_4*r_4*np.sin(theta_4 - theta_1)) + dtheta_3*(dtheta_3*l_1*l_3*m_4*np.sin(theta_3 - theta_1) + dtheta_3*l_1*l_3*m_5*np.sin(theta_3 - theta_1) + dtheta_3*l_1*m_3*r_3*np.sin(theta_3 - theta_1)) + dtheta_2*mu_2 + dtheta_2*(dtheta_2*l_1*l_2*m_3*np.sin(theta_2 - theta_1) + dtheta_2*l_1*l_2*m_4*np.sin(theta_2 - theta_1) + dtheta_2*l_1*l_2*m_5*np.sin(theta_2 - theta_1) + dtheta_2*l_1*m_2*r_2*np.sin(theta_2 - theta_1)) - dtheta_1*(mu_1 + mu_2) - ddx_c*(-l_1*m_2 - l_1*m_3 - l_1*m_4 - l_1*m_5 - m_1*r_1)*np.cos(theta_1) - g*(-l_1*m_2 - l_1*m_3 - l_1*m_4 - l_1*m_5 - m_1*r_1)*np.sin(theta_1))*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2))/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2) - (-l_1**2*(l_4*m_5 + m_4*r_4)*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_4 - theta_1)*np.cos(theta_3 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_3*(l_4*m_5 + m_4*r_4)*np.cos(theta_4 - theta_3) - (-l_1**2*(l_4*m_5 + m_4*r_4)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_4 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_4*m_5 + m_4*r_4)*np.cos(theta_4 - theta_2))*(-l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_3 - theta_2))/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2))*(dtheta_5**2*l_3*m_5*r_5*np.sin(theta_5 - theta_3) + dtheta_4*mu_4 + dtheta_4*(dtheta_4*l_3*l_4*m_5*np.sin(theta_4 - theta_3) + dtheta_4*l_3*m_4*r_4*np.sin(theta_4 - theta_3)) - dtheta_3*(mu_3 + mu_4) + dtheta_2*mu_3 + dtheta_2*(-dtheta_2*l_2*l_3*m_4*np.sin(theta_3 - theta_2) - dtheta_2*l_2*l_3*m_5*np.sin(theta_3 - theta_2) - dtheta_2*l_2*m_3*r_3*np.sin(theta_3 - theta_2)) + dtheta_1*(-dtheta_1*l_1*l_3*m_4*np.sin(theta_3 - theta_1) - dtheta_1*l_1*l_3*m_5*np.sin(theta_3 - theta_1) - dtheta_1*l_1*m_3*r_3*np.sin(theta_3 - theta_1)) - ddx_c*(-l_3*m_4 - l_3*m_5 - m_3*r_3)*np.cos(theta_3) - g*(-l_3*m_4 - l_3*m_5 - m_3*r_3)*np.sin(theta_3) - l_1*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(dtheta_5**2*l_1*m_5*r_5*np.sin(theta_5 - theta_1) + dtheta_4*(dtheta_4*l_1*l_4*m_5*np.sin(theta_4 - theta_1) + dtheta_4*l_1*m_4*r_4*np.sin(theta_4 - theta_1)) + dtheta_3*(dtheta_3*l_1*l_3*m_4*np.sin(theta_3 - theta_1) + dtheta_3*l_1*l_3*m_5*np.sin(theta_3 - theta_1) + dtheta_3*l_1*m_3*r_3*np.sin(theta_3 - theta_1)) + dtheta_2*mu_2 + dtheta_2*(dtheta_2*l_1*l_2*m_3*np.sin(theta_2 - theta_1) + dtheta_2*l_1*l_2*m_4*np.sin(theta_2 - theta_1) + dtheta_2*l_1*l_2*m_5*np.sin(theta_2 - theta_1) + dtheta_2*l_1*m_2*r_2*np.sin(theta_2 - theta_1)) - dtheta_1*(mu_1 + mu_2) - ddx_c*(-l_1*m_2 - l_1*m_3 - l_1*m_4 - l_1*m_5 - m_1*r_1)*np.cos(theta_1) - g*(-l_1*m_2 - l_1*m_3 - l_1*m_4 - l_1*m_5 - m_1*r_1)*np.sin(theta_1))*np.cos(theta_3 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) - (-l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_3 - theta_2))*(dtheta_5**2*l_2*m_5*r_5*np.sin(theta_5 - theta_2) + dtheta_4*(dtheta_4*l_2*l_4*m_5*np.sin(theta_4 - theta_2) + dtheta_4*l_2*m_4*r_4*np.sin(theta_4 - theta_2)) + dtheta_3*mu_3 + dtheta_3*(dtheta_3*l_2*l_3*m_4*np.sin(theta_3 - theta_2) + dtheta_3*l_2*l_3*m_5*np.sin(theta_3 - theta_2) + dtheta_3*l_2*m_3*r_3*np.sin(theta_3 - theta_2)) - dtheta_2*(mu_2 + mu_3) + dtheta_1*mu_2 + dtheta_1*(-dtheta_1*l_1*l_2*m_3*np.sin(theta_2 - theta_1) - dtheta_1*l_1*l_2*m_4*np.sin(theta_2 - theta_1) - dtheta_1*l_1*l_2*m_5*np.sin(theta_2 - theta_1) - dtheta_1*l_1*m_2*r_2*np.sin(theta_2 - theta_1)) - ddx_c*(-l_2*m_3 - l_2*m_4 - l_2*m_5 - m_2*r_2)*np.cos(theta_2) - g*(-l_2*m_3 - l_2*m_4 - l_2*m_5 - m_2*r_2)*np.sin(theta_2) - l_1*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*(dtheta_5**2*l_1*m_5*r_5*np.sin(theta_5 - theta_1) + dtheta_4*(dtheta_4*l_1*l_4*m_5*np.sin(theta_4 - theta_1) + dtheta_4*l_1*m_4*r_4*np.sin(theta_4 - theta_1)) + dtheta_3*(dtheta_3*l_1*l_3*m_4*np.sin(theta_3 - theta_1) + dtheta_3*l_1*l_3*m_5*np.sin(theta_3 - theta_1) + dtheta_3*l_1*m_3*r_3*np.sin(theta_3 - theta_1)) + dtheta_2*mu_2 + dtheta_2*(dtheta_2*l_1*l_2*m_3*np.sin(theta_2 - theta_1) + dtheta_2*l_1*l_2*m_4*np.sin(theta_2 - theta_1) + dtheta_2*l_1*l_2*m_5*np.sin(theta_2 - theta_1) + dtheta_2*l_1*m_2*r_2*np.sin(theta_2 - theta_1)) - dtheta_1*(mu_1 + mu_2) - ddx_c*(-l_1*m_2 - l_1*m_3 - l_1*m_4 - l_1*m_5 - m_1*r_1)*np.cos(theta_1) - g*(-l_1*m_2 - l_1*m_3 - l_1*m_4 - l_1*m_5 - m_1*r_1)*np.sin(theta_1))*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2))/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2))/(I_3 - l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)**2*np.cos(theta_3 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_3**2*m_4 + l_3**2*m_5 + m_3*r_3**2 - (-l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_3 - theta_2))**2/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2)))/(I_4 - l_1**2*(l_4*m_5 + m_4*r_4)**2*np.cos(theta_4 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_4**2*m_5 + m_4*r_4**2 - (-l_1**2*(l_4*m_5 + m_4*r_4)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_4 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_4*m_5 + m_4*r_4)*np.cos(theta_4 - theta_2))**2/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2) - (-l_1**2*(l_4*m_5 + m_4*r_4)*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_4 - theta_1)*np.cos(theta_3 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_3*(l_4*m_5 + m_4*r_4)*np.cos(theta_4 - theta_3) - (-l_1**2*(l_4*m_5 + m_4*r_4)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_4 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_4*m_5 + m_4*r_4)*np.cos(theta_4 - theta_2))*(-l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_3 - theta_2))/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2))**2/(I_3 - l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)**2*np.cos(theta_3 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_3**2*m_4 + l_3**2*m_5 + m_3*r_3**2 - (-l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_3 - theta_2))**2/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2))))/(I_5 - l_1**2*m_5**2*r_5**2*np.cos(theta_5 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + m_5*r_5**2 - (-l_1**2*m_5*r_5*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_5 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*m_5*r_5*np.cos(theta_5 - theta_2))**2/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2) - (-l_1**2*m_5*r_5*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_5 - theta_1)*np.cos(theta_3 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_3*m_5*r_5*np.cos(theta_5 - theta_3) - (-l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_3 - theta_2))*(-l_1**2*m_5*r_5*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_5 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*m_5*r_5*np.cos(theta_5 - theta_2))/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2))**2/(I_3 - l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)**2*np.cos(theta_3 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_3**2*m_4 + l_3**2*m_5 + m_3*r_3**2 - (-l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_3 - theta_2))**2/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2)) - (-l_1**2*m_5*r_5*(l_4*m_5 + m_4*r_4)*np.cos(theta_5 - theta_1)*np.cos(theta_4 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_4*m_5*r_5*np.cos(theta_5 - theta_4) - (-l_1**2*(l_4*m_5 + m_4*r_4)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_4 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_4*m_5 + m_4*r_4)*np.cos(theta_4 - theta_2))*(-l_1**2*m_5*r_5*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_5 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*m_5*r_5*np.cos(theta_5 - theta_2))/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2) - (-l_1**2*(l_4*m_5 + m_4*r_4)*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_4 - theta_1)*np.cos(theta_3 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_3*(l_4*m_5 + m_4*r_4)*np.cos(theta_4 - theta_3) - (-l_1**2*(l_4*m_5 + m_4*r_4)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_4 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_4*m_5 + m_4*r_4)*np.cos(theta_4 - theta_2))*(-l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_3 - theta_2))/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2))*(-l_1**2*m_5*r_5*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_5 - theta_1)*np.cos(theta_3 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_3*m_5*r_5*np.cos(theta_5 - theta_3) - (-l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_3 - theta_2))*(-l_1**2*m_5*r_5*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_5 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*m_5*r_5*np.cos(theta_5 - theta_2))/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2))/(I_3 - l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)**2*np.cos(theta_3 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_3**2*m_4 + l_3**2*m_5 + m_3*r_3**2 - (-l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_3 - theta_2))**2/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2)))**2/(I_4 - l_1**2*(l_4*m_5 + m_4*r_4)**2*np.cos(theta_4 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_4**2*m_5 + m_4*r_4**2 - (-l_1**2*(l_4*m_5 + m_4*r_4)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_4 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_4*m_5 + m_4*r_4)*np.cos(theta_4 - theta_2))**2/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2) - (-l_1**2*(l_4*m_5 + m_4*r_4)*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_4 - theta_1)*np.cos(theta_3 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_3*(l_4*m_5 + m_4*r_4)*np.cos(theta_4 - theta_3) - (-l_1**2*(l_4*m_5 + m_4*r_4)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_4 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_4*m_5 + m_4*r_4)*np.cos(theta_4 - theta_2))*(-l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_3 - theta_2))/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2))**2/(I_3 - l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)**2*np.cos(theta_3 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_3**2*m_4 + l_3**2*m_5 + m_3*r_3**2 - (-l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_3 - theta_2))**2/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2)))))/(I_3 - l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)**2*np.cos(theta_3 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_3**2*m_4 + l_3**2*m_5 + m_3*r_3**2 - (-l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_3 - theta_2))**2/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2)) - (-l_1**2*m_5*r_5*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_5 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*m_5*r_5*np.cos(theta_5 - theta_2))*(-dtheta_5*mu_5 - dtheta_4**2*l_4*m_5*r_5*np.sin(theta_5 - theta_4) + dtheta_4*mu_5 - dtheta_3**2*l_3*m_5*r_5*np.sin(theta_5 - theta_3) - dtheta_2**2*l_2*m_5*r_5*np.sin(theta_5 - theta_2) - dtheta_1**2*l_1*m_5*r_5*np.sin(theta_5 - theta_1) + ddx_c*m_5*r_5*np.cos(theta_5) + g*m_5*r_5*np.sin(theta_5) - l_1*m_5*r_5*(dtheta_5**2*l_1*m_5*r_5*np.sin(theta_5 - theta_1) + dtheta_4*(dtheta_4*l_1*l_4*m_5*np.sin(theta_4 - theta_1) + dtheta_4*l_1*m_4*r_4*np.sin(theta_4 - theta_1)) + dtheta_3*(dtheta_3*l_1*l_3*m_4*np.sin(theta_3 - theta_1) + dtheta_3*l_1*l_3*m_5*np.sin(theta_3 - theta_1) + dtheta_3*l_1*m_3*r_3*np.sin(theta_3 - theta_1)) + dtheta_2*mu_2 + dtheta_2*(dtheta_2*l_1*l_2*m_3*np.sin(theta_2 - theta_1) + dtheta_2*l_1*l_2*m_4*np.sin(theta_2 - theta_1) + dtheta_2*l_1*l_2*m_5*np.sin(theta_2 - theta_1) + dtheta_2*l_1*m_2*r_2*np.sin(theta_2 - theta_1)) - dtheta_1*(mu_1 + mu_2) - ddx_c*(-l_1*m_2 - l_1*m_3 - l_1*m_4 - l_1*m_5 - m_1*r_1)*np.cos(theta_1) - g*(-l_1*m_2 - l_1*m_3 - l_1*m_4 - l_1*m_5 - m_1*r_1)*np.sin(theta_1))*np.cos(theta_5 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) - (-l_1**2*m_5*r_5*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_5 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*m_5*r_5*np.cos(theta_5 - theta_2))*(dtheta_5**2*l_2*m_5*r_5*np.sin(theta_5 - theta_2) + dtheta_4*(dtheta_4*l_2*l_4*m_5*np.sin(theta_4 - theta_2) + dtheta_4*l_2*m_4*r_4*np.sin(theta_4 - theta_2)) + dtheta_3*mu_3 + dtheta_3*(dtheta_3*l_2*l_3*m_4*np.sin(theta_3 - theta_2) + dtheta_3*l_2*l_3*m_5*np.sin(theta_3 - theta_2) + dtheta_3*l_2*m_3*r_3*np.sin(theta_3 - theta_2)) - dtheta_2*(mu_2 + mu_3) + dtheta_1*mu_2 + dtheta_1*(-dtheta_1*l_1*l_2*m_3*np.sin(theta_2 - theta_1) - dtheta_1*l_1*l_2*m_4*np.sin(theta_2 - theta_1) - dtheta_1*l_1*l_2*m_5*np.sin(theta_2 - theta_1) - dtheta_1*l_1*m_2*r_2*np.sin(theta_2 - theta_1)) - ddx_c*(-l_2*m_3 - l_2*m_4 - l_2*m_5 - m_2*r_2)*np.cos(theta_2) - g*(-l_2*m_3 - l_2*m_4 - l_2*m_5 - m_2*r_2)*np.sin(theta_2) - l_1*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*(dtheta_5**2*l_1*m_5*r_5*np.sin(theta_5 - theta_1) + dtheta_4*(dtheta_4*l_1*l_4*m_5*np.sin(theta_4 - theta_1) + dtheta_4*l_1*m_4*r_4*np.sin(theta_4 - theta_1)) + dtheta_3*(dtheta_3*l_1*l_3*m_4*np.sin(theta_3 - theta_1) + dtheta_3*l_1*l_3*m_5*np.sin(theta_3 - theta_1) + dtheta_3*l_1*m_3*r_3*np.sin(theta_3 - theta_1)) + dtheta_2*mu_2 + dtheta_2*(dtheta_2*l_1*l_2*m_3*np.sin(theta_2 - theta_1) + dtheta_2*l_1*l_2*m_4*np.sin(theta_2 - theta_1) + dtheta_2*l_1*l_2*m_5*np.sin(theta_2 - theta_1) + dtheta_2*l_1*m_2*r_2*np.sin(theta_2 - theta_1)) - dtheta_1*(mu_1 + mu_2) - ddx_c*(-l_1*m_2 - l_1*m_3 - l_1*m_4 - l_1*m_5 - m_1*r_1)*np.cos(theta_1) - g*(-l_1*m_2 - l_1*m_3 - l_1*m_4 - l_1*m_5 - m_1*r_1)*np.sin(theta_1))*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2))/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2) - (-l_1**2*m_5*r_5*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_5 - theta_1)*np.cos(theta_3 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_3*m_5*r_5*np.cos(theta_5 - theta_3) - (-l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_3 - theta_2))*(-l_1**2*m_5*r_5*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_5 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*m_5*r_5*np.cos(theta_5 - theta_2))/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2))*(dtheta_5**2*l_3*m_5*r_5*np.sin(theta_5 - theta_3) + dtheta_4*mu_4 + dtheta_4*(dtheta_4*l_3*l_4*m_5*np.sin(theta_4 - theta_3) + dtheta_4*l_3*m_4*r_4*np.sin(theta_4 - theta_3)) - dtheta_3*(mu_3 + mu_4) + dtheta_2*mu_3 + dtheta_2*(-dtheta_2*l_2*l_3*m_4*np.sin(theta_3 - theta_2) - dtheta_2*l_2*l_3*m_5*np.sin(theta_3 - theta_2) - dtheta_2*l_2*m_3*r_3*np.sin(theta_3 - theta_2)) + dtheta_1*(-dtheta_1*l_1*l_3*m_4*np.sin(theta_3 - theta_1) - dtheta_1*l_1*l_3*m_5*np.sin(theta_3 - theta_1) - dtheta_1*l_1*m_3*r_3*np.sin(theta_3 - theta_1)) - ddx_c*(-l_3*m_4 - l_3*m_5 - m_3*r_3)*np.cos(theta_3) - g*(-l_3*m_4 - l_3*m_5 - m_3*r_3)*np.sin(theta_3) - l_1*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(dtheta_5**2*l_1*m_5*r_5*np.sin(theta_5 - theta_1) + dtheta_4*(dtheta_4*l_1*l_4*m_5*np.sin(theta_4 - theta_1) + dtheta_4*l_1*m_4*r_4*np.sin(theta_4 - theta_1)) + dtheta_3*(dtheta_3*l_1*l_3*m_4*np.sin(theta_3 - theta_1) + dtheta_3*l_1*l_3*m_5*np.sin(theta_3 - theta_1) + dtheta_3*l_1*m_3*r_3*np.sin(theta_3 - theta_1)) + dtheta_2*mu_2 + dtheta_2*(dtheta_2*l_1*l_2*m_3*np.sin(theta_2 - theta_1) + dtheta_2*l_1*l_2*m_4*np.sin(theta_2 - theta_1) + dtheta_2*l_1*l_2*m_5*np.sin(theta_2 - theta_1) + dtheta_2*l_1*m_2*r_2*np.sin(theta_2 - theta_1)) - dtheta_1*(mu_1 + mu_2) - ddx_c*(-l_1*m_2 - l_1*m_3 - l_1*m_4 - l_1*m_5 - m_1*r_1)*np.cos(theta_1) - g*(-l_1*m_2 - l_1*m_3 - l_1*m_4 - l_1*m_5 - m_1*r_1)*np.sin(theta_1))*np.cos(theta_3 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) - (-l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_3 - theta_2))*(dtheta_5**2*l_2*m_5*r_5*np.sin(theta_5 - theta_2) + dtheta_4*(dtheta_4*l_2*l_4*m_5*np.sin(theta_4 - theta_2) + dtheta_4*l_2*m_4*r_4*np.sin(theta_4 - theta_2)) + dtheta_3*mu_3 + dtheta_3*(dtheta_3*l_2*l_3*m_4*np.sin(theta_3 - theta_2) + dtheta_3*l_2*l_3*m_5*np.sin(theta_3 - theta_2) + dtheta_3*l_2*m_3*r_3*np.sin(theta_3 - theta_2)) - dtheta_2*(mu_2 + mu_3) + dtheta_1*mu_2 + dtheta_1*(-dtheta_1*l_1*l_2*m_3*np.sin(theta_2 - theta_1) - dtheta_1*l_1*l_2*m_4*np.sin(theta_2 - theta_1) - dtheta_1*l_1*l_2*m_5*np.sin(theta_2 - theta_1) - dtheta_1*l_1*m_2*r_2*np.sin(theta_2 - theta_1)) - ddx_c*(-l_2*m_3 - l_2*m_4 - l_2*m_5 - m_2*r_2)*np.cos(theta_2) - g*(-l_2*m_3 - l_2*m_4 - l_2*m_5 - m_2*r_2)*np.sin(theta_2) - l_1*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*(dtheta_5**2*l_1*m_5*r_5*np.sin(theta_5 - theta_1) + dtheta_4*(dtheta_4*l_1*l_4*m_5*np.sin(theta_4 - theta_1) + dtheta_4*l_1*m_4*r_4*np.sin(theta_4 - theta_1)) + dtheta_3*(dtheta_3*l_1*l_3*m_4*np.sin(theta_3 - theta_1) + dtheta_3*l_1*l_3*m_5*np.sin(theta_3 - theta_1) + dtheta_3*l_1*m_3*r_3*np.sin(theta_3 - theta_1)) + dtheta_2*mu_2 + dtheta_2*(dtheta_2*l_1*l_2*m_3*np.sin(theta_2 - theta_1) + dtheta_2*l_1*l_2*m_4*np.sin(theta_2 - theta_1) + dtheta_2*l_1*l_2*m_5*np.sin(theta_2 - theta_1) + dtheta_2*l_1*m_2*r_2*np.sin(theta_2 - theta_1)) - dtheta_1*(mu_1 + mu_2) - ddx_c*(-l_1*m_2 - l_1*m_3 - l_1*m_4 - l_1*m_5 - m_1*r_1)*np.cos(theta_1) - g*(-l_1*m_2 - l_1*m_3 - l_1*m_4 - l_1*m_5 - m_1*r_1)*np.sin(theta_1))*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2))/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2))/(I_3 - l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)**2*np.cos(theta_3 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_3**2*m_4 + l_3**2*m_5 + m_3*r_3**2 - (-l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_3 - theta_2))**2/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2)) - (-l_1**2*m_5*r_5*(l_4*m_5 + m_4*r_4)*np.cos(theta_5 - theta_1)*np.cos(theta_4 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_4*m_5*r_5*np.cos(theta_5 - theta_4) - (-l_1**2*(l_4*m_5 + m_4*r_4)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_4 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_4*m_5 + m_4*r_4)*np.cos(theta_4 - theta_2))*(-l_1**2*m_5*r_5*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_5 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*m_5*r_5*np.cos(theta_5 - theta_2))/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2) - (-l_1**2*(l_4*m_5 + m_4*r_4)*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_4 - theta_1)*np.cos(theta_3 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_3*(l_4*m_5 + m_4*r_4)*np.cos(theta_4 - theta_3) - (-l_1**2*(l_4*m_5 + m_4*r_4)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_4 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_4*m_5 + m_4*r_4)*np.cos(theta_4 - theta_2))*(-l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_3 - theta_2))/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2))*(-l_1**2*m_5*r_5*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_5 - theta_1)*np.cos(theta_3 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_3*m_5*r_5*np.cos(theta_5 - theta_3) - (-l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_3 - theta_2))*(-l_1**2*m_5*r_5*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_5 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*m_5*r_5*np.cos(theta_5 - theta_2))/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2))/(I_3 - l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)**2*np.cos(theta_3 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_3**2*m_4 + l_3**2*m_5 + m_3*r_3**2 - (-l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_3 - theta_2))**2/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2)))*(dtheta_5**2*l_4*m_5*r_5*np.sin(theta_5 - theta_4) + dtheta_5*mu_5 - dtheta_4*(mu_4 + mu_5) + dtheta_3*mu_4 + dtheta_3*(-dtheta_3*l_3*l_4*m_5*np.sin(theta_4 - theta_3) - dtheta_3*l_3*m_4*r_4*np.sin(theta_4 - theta_3)) + dtheta_2*(-dtheta_2*l_2*l_4*m_5*np.sin(theta_4 - theta_2) - dtheta_2*l_2*m_4*r_4*np.sin(theta_4 - theta_2)) + dtheta_1*(-dtheta_1*l_1*l_4*m_5*np.sin(theta_4 - theta_1) - dtheta_1*l_1*m_4*r_4*np.sin(theta_4 - theta_1)) - ddx_c*(-l_4*m_5 - m_4*r_4)*np.cos(theta_4) - g*(-l_4*m_5 - m_4*r_4)*np.sin(theta_4) - l_1*(l_4*m_5 + m_4*r_4)*(dtheta_5**2*l_1*m_5*r_5*np.sin(theta_5 - theta_1) + dtheta_4*(dtheta_4*l_1*l_4*m_5*np.sin(theta_4 - theta_1) + dtheta_4*l_1*m_4*r_4*np.sin(theta_4 - theta_1)) + dtheta_3*(dtheta_3*l_1*l_3*m_4*np.sin(theta_3 - theta_1) + dtheta_3*l_1*l_3*m_5*np.sin(theta_3 - theta_1) + dtheta_3*l_1*m_3*r_3*np.sin(theta_3 - theta_1)) + dtheta_2*mu_2 + dtheta_2*(dtheta_2*l_1*l_2*m_3*np.sin(theta_2 - theta_1) + dtheta_2*l_1*l_2*m_4*np.sin(theta_2 - theta_1) + dtheta_2*l_1*l_2*m_5*np.sin(theta_2 - theta_1) + dtheta_2*l_1*m_2*r_2*np.sin(theta_2 - theta_1)) - dtheta_1*(mu_1 + mu_2) - ddx_c*(-l_1*m_2 - l_1*m_3 - l_1*m_4 - l_1*m_5 - m_1*r_1)*np.cos(theta_1) - g*(-l_1*m_2 - l_1*m_3 - l_1*m_4 - l_1*m_5 - m_1*r_1)*np.sin(theta_1))*np.cos(theta_4 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) - (-l_1**2*(l_4*m_5 + m_4*r_4)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_4 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_4*m_5 + m_4*r_4)*np.cos(theta_4 - theta_2))*(dtheta_5**2*l_2*m_5*r_5*np.sin(theta_5 - theta_2) + dtheta_4*(dtheta_4*l_2*l_4*m_5*np.sin(theta_4 - theta_2) + dtheta_4*l_2*m_4*r_4*np.sin(theta_4 - theta_2)) + dtheta_3*mu_3 + dtheta_3*(dtheta_3*l_2*l_3*m_4*np.sin(theta_3 - theta_2) + dtheta_3*l_2*l_3*m_5*np.sin(theta_3 - theta_2) + dtheta_3*l_2*m_3*r_3*np.sin(theta_3 - theta_2)) - dtheta_2*(mu_2 + mu_3) + dtheta_1*mu_2 + dtheta_1*(-dtheta_1*l_1*l_2*m_3*np.sin(theta_2 - theta_1) - dtheta_1*l_1*l_2*m_4*np.sin(theta_2 - theta_1) - dtheta_1*l_1*l_2*m_5*np.sin(theta_2 - theta_1) - dtheta_1*l_1*m_2*r_2*np.sin(theta_2 - theta_1)) - ddx_c*(-l_2*m_3 - l_2*m_4 - l_2*m_5 - m_2*r_2)*np.cos(theta_2) - g*(-l_2*m_3 - l_2*m_4 - l_2*m_5 - m_2*r_2)*np.sin(theta_2) - l_1*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*(dtheta_5**2*l_1*m_5*r_5*np.sin(theta_5 - theta_1) + dtheta_4*(dtheta_4*l_1*l_4*m_5*np.sin(theta_4 - theta_1) + dtheta_4*l_1*m_4*r_4*np.sin(theta_4 - theta_1)) + dtheta_3*(dtheta_3*l_1*l_3*m_4*np.sin(theta_3 - theta_1) + dtheta_3*l_1*l_3*m_5*np.sin(theta_3 - theta_1) + dtheta_3*l_1*m_3*r_3*np.sin(theta_3 - theta_1)) + dtheta_2*mu_2 + dtheta_2*(dtheta_2*l_1*l_2*m_3*np.sin(theta_2 - theta_1) + dtheta_2*l_1*l_2*m_4*np.sin(theta_2 - theta_1) + dtheta_2*l_1*l_2*m_5*np.sin(theta_2 - theta_1) + dtheta_2*l_1*m_2*r_2*np.sin(theta_2 - theta_1)) - dtheta_1*(mu_1 + mu_2) - ddx_c*(-l_1*m_2 - l_1*m_3 - l_1*m_4 - l_1*m_5 - m_1*r_1)*np.cos(theta_1) - g*(-l_1*m_2 - l_1*m_3 - l_1*m_4 - l_1*m_5 - m_1*r_1)*np.sin(theta_1))*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2))/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2) - (-l_1**2*(l_4*m_5 + m_4*r_4)*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_4 - theta_1)*np.cos(theta_3 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_3*(l_4*m_5 + m_4*r_4)*np.cos(theta_4 - theta_3) - (-l_1**2*(l_4*m_5 + m_4*r_4)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_4 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_4*m_5 + m_4*r_4)*np.cos(theta_4 - theta_2))*(-l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_3 - theta_2))/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2))*(dtheta_5**2*l_3*m_5*r_5*np.sin(theta_5 - theta_3) + dtheta_4*mu_4 + dtheta_4*(dtheta_4*l_3*l_4*m_5*np.sin(theta_4 - theta_3) + dtheta_4*l_3*m_4*r_4*np.sin(theta_4 - theta_3)) - dtheta_3*(mu_3 + mu_4) + dtheta_2*mu_3 + dtheta_2*(-dtheta_2*l_2*l_3*m_4*np.sin(theta_3 - theta_2) - dtheta_2*l_2*l_3*m_5*np.sin(theta_3 - theta_2) - dtheta_2*l_2*m_3*r_3*np.sin(theta_3 - theta_2)) + dtheta_1*(-dtheta_1*l_1*l_3*m_4*np.sin(theta_3 - theta_1) - dtheta_1*l_1*l_3*m_5*np.sin(theta_3 - theta_1) - dtheta_1*l_1*m_3*r_3*np.sin(theta_3 - theta_1)) - ddx_c*(-l_3*m_4 - l_3*m_5 - m_3*r_3)*np.cos(theta_3) - g*(-l_3*m_4 - l_3*m_5 - m_3*r_3)*np.sin(theta_3) - l_1*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(dtheta_5**2*l_1*m_5*r_5*np.sin(theta_5 - theta_1) + dtheta_4*(dtheta_4*l_1*l_4*m_5*np.sin(theta_4 - theta_1) + dtheta_4*l_1*m_4*r_4*np.sin(theta_4 - theta_1)) + dtheta_3*(dtheta_3*l_1*l_3*m_4*np.sin(theta_3 - theta_1) + dtheta_3*l_1*l_3*m_5*np.sin(theta_3 - theta_1) + dtheta_3*l_1*m_3*r_3*np.sin(theta_3 - theta_1)) + dtheta_2*mu_2 + dtheta_2*(dtheta_2*l_1*l_2*m_3*np.sin(theta_2 - theta_1) + dtheta_2*l_1*l_2*m_4*np.sin(theta_2 - theta_1) + dtheta_2*l_1*l_2*m_5*np.sin(theta_2 - theta_1) + dtheta_2*l_1*m_2*r_2*np.sin(theta_2 - theta_1)) - dtheta_1*(mu_1 + mu_2) - ddx_c*(-l_1*m_2 - l_1*m_3 - l_1*m_4 - l_1*m_5 - m_1*r_1)*np.cos(theta_1) - g*(-l_1*m_2 - l_1*m_3 - l_1*m_4 - l_1*m_5 - m_1*r_1)*np.sin(theta_1))*np.cos(theta_3 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) - (-l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_3 - theta_2))*(dtheta_5**2*l_2*m_5*r_5*np.sin(theta_5 - theta_2) + dtheta_4*(dtheta_4*l_2*l_4*m_5*np.sin(theta_4 - theta_2) + dtheta_4*l_2*m_4*r_4*np.sin(theta_4 - theta_2)) + dtheta_3*mu_3 + dtheta_3*(dtheta_3*l_2*l_3*m_4*np.sin(theta_3 - theta_2) + dtheta_3*l_2*l_3*m_5*np.sin(theta_3 - theta_2) + dtheta_3*l_2*m_3*r_3*np.sin(theta_3 - theta_2)) - dtheta_2*(mu_2 + mu_3) + dtheta_1*mu_2 + dtheta_1*(-dtheta_1*l_1*l_2*m_3*np.sin(theta_2 - theta_1) - dtheta_1*l_1*l_2*m_4*np.sin(theta_2 - theta_1) - dtheta_1*l_1*l_2*m_5*np.sin(theta_2 - theta_1) - dtheta_1*l_1*m_2*r_2*np.sin(theta_2 - theta_1)) - ddx_c*(-l_2*m_3 - l_2*m_4 - l_2*m_5 - m_2*r_2)*np.cos(theta_2) - g*(-l_2*m_3 - l_2*m_4 - l_2*m_5 - m_2*r_2)*np.sin(theta_2) - l_1*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*(dtheta_5**2*l_1*m_5*r_5*np.sin(theta_5 - theta_1) + dtheta_4*(dtheta_4*l_1*l_4*m_5*np.sin(theta_4 - theta_1) + dtheta_4*l_1*m_4*r_4*np.sin(theta_4 - theta_1)) + dtheta_3*(dtheta_3*l_1*l_3*m_4*np.sin(theta_3 - theta_1) + dtheta_3*l_1*l_3*m_5*np.sin(theta_3 - theta_1) + dtheta_3*l_1*m_3*r_3*np.sin(theta_3 - theta_1)) + dtheta_2*mu_2 + dtheta_2*(dtheta_2*l_1*l_2*m_3*np.sin(theta_2 - theta_1) + dtheta_2*l_1*l_2*m_4*np.sin(theta_2 - theta_1) + dtheta_2*l_1*l_2*m_5*np.sin(theta_2 - theta_1) + dtheta_2*l_1*m_2*r_2*np.sin(theta_2 - theta_1)) - dtheta_1*(mu_1 + mu_2) - ddx_c*(-l_1*m_2 - l_1*m_3 - l_1*m_4 - l_1*m_5 - m_1*r_1)*np.cos(theta_1) - g*(-l_1*m_2 - l_1*m_3 - l_1*m_4 - l_1*m_5 - m_1*r_1)*np.sin(theta_1))*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2))/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2))/(I_3 - l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)**2*np.cos(theta_3 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_3**2*m_4 + l_3**2*m_5 + m_3*r_3**2 - (-l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_3 - theta_2))**2/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2)))/(I_4 - l_1**2*(l_4*m_5 + m_4*r_4)**2*np.cos(theta_4 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_4**2*m_5 + m_4*r_4**2 - (-l_1**2*(l_4*m_5 + m_4*r_4)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_4 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_4*m_5 + m_4*r_4)*np.cos(theta_4 - theta_2))**2/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2) - (-l_1**2*(l_4*m_5 + m_4*r_4)*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_4 - theta_1)*np.cos(theta_3 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_3*(l_4*m_5 + m_4*r_4)*np.cos(theta_4 - theta_3) - (-l_1**2*(l_4*m_5 + m_4*r_4)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_4 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_4*m_5 + m_4*r_4)*np.cos(theta_4 - theta_2))*(-l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_3 - theta_2))/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2))**2/(I_3 - l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)**2*np.cos(theta_3 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_3**2*m_4 + l_3**2*m_5 + m_3*r_3**2 - (-l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_3 - theta_2))**2/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2))))/(I_5 - l_1**2*m_5**2*r_5**2*np.cos(theta_5 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + m_5*r_5**2 - (-l_1**2*m_5*r_5*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_5 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*m_5*r_5*np.cos(theta_5 - theta_2))**2/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2) - (-l_1**2*m_5*r_5*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_5 - theta_1)*np.cos(theta_3 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_3*m_5*r_5*np.cos(theta_5 - theta_3) - (-l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_3 - theta_2))*(-l_1**2*m_5*r_5*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_5 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*m_5*r_5*np.cos(theta_5 - theta_2))/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2))**2/(I_3 - l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)**2*np.cos(theta_3 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_3**2*m_4 + l_3**2*m_5 + m_3*r_3**2 - (-l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_3 - theta_2))**2/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2)) - (-l_1**2*m_5*r_5*(l_4*m_5 + m_4*r_4)*np.cos(theta_5 - theta_1)*np.cos(theta_4 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_4*m_5*r_5*np.cos(theta_5 - theta_4) - (-l_1**2*(l_4*m_5 + m_4*r_4)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_4 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_4*m_5 + m_4*r_4)*np.cos(theta_4 - theta_2))*(-l_1**2*m_5*r_5*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_5 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*m_5*r_5*np.cos(theta_5 - theta_2))/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2) - (-l_1**2*(l_4*m_5 + m_4*r_4)*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_4 - theta_1)*np.cos(theta_3 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_3*(l_4*m_5 + m_4*r_4)*np.cos(theta_4 - theta_3) - (-l_1**2*(l_4*m_5 + m_4*r_4)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_4 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_4*m_5 + m_4*r_4)*np.cos(theta_4 - theta_2))*(-l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_3 - theta_2))/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2))*(-l_1**2*m_5*r_5*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_5 - theta_1)*np.cos(theta_3 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_3*m_5*r_5*np.cos(theta_5 - theta_3) - (-l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_3 - theta_2))*(-l_1**2*m_5*r_5*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_5 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*m_5*r_5*np.cos(theta_5 - theta_2))/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2))/(I_3 - l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)**2*np.cos(theta_3 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_3**2*m_4 + l_3**2*m_5 + m_3*r_3**2 - (-l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_3 - theta_2))**2/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2)))**2/(I_4 - l_1**2*(l_4*m_5 + m_4*r_4)**2*np.cos(theta_4 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_4**2*m_5 + m_4*r_4**2 - (-l_1**2*(l_4*m_5 + m_4*r_4)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_4 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_4*m_5 + m_4*r_4)*np.cos(theta_4 - theta_2))**2/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2) - (-l_1**2*(l_4*m_5 + m_4*r_4)*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_4 - theta_1)*np.cos(theta_3 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_3*(l_4*m_5 + m_4*r_4)*np.cos(theta_4 - theta_3) - (-l_1**2*(l_4*m_5 + m_4*r_4)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_4 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_4*m_5 + m_4*r_4)*np.cos(theta_4 - theta_2))*(-l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_3 - theta_2))/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2))**2/(I_3 - l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)**2*np.cos(theta_3 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_3**2*m_4 + l_3**2*m_5 + m_3*r_3**2 - (-l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_3 - theta_2))**2/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2)))))*np.cos(theta_2 - theta_1)/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2))/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2),



                      (dtheta_5**2*l_2*m_5*r_5*np.sin(theta_5 - theta_2) + dtheta_4*(dtheta_4*l_2*l_4*m_5*np.sin(theta_4 - theta_2) + dtheta_4*l_2*m_4*r_4*np.sin(theta_4 - theta_2)) + dtheta_3*mu_3 + dtheta_3*(dtheta_3*l_2*l_3*m_4*np.sin(theta_3 - theta_2) + dtheta_3*l_2*l_3*m_5*np.sin(theta_3 - theta_2) + dtheta_3*l_2*m_3*r_3*np.sin(theta_3 - theta_2)) - dtheta_2*(mu_2 + mu_3) + dtheta_1*mu_2 + dtheta_1*(-dtheta_1*l_1*l_2*m_3*np.sin(theta_2 - theta_1) - dtheta_1*l_1*l_2*m_4*np.sin(theta_2 - theta_1) - dtheta_1*l_1*l_2*m_5*np.sin(theta_2 - theta_1) - dtheta_1*l_1*m_2*r_2*np.sin(theta_2 - theta_1)) - ddx_c*(-l_2*m_3 - l_2*m_4 - l_2*m_5 - m_2*r_2)*np.cos(theta_2) - g*(-l_2*m_3 - l_2*m_4 - l_2*m_5 - m_2*r_2)*np.sin(theta_2) - l_1*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*(dtheta_5**2*l_1*m_5*r_5*np.sin(theta_5 - theta_1) + dtheta_4*(dtheta_4*l_1*l_4*m_5*np.sin(theta_4 - theta_1) + dtheta_4*l_1*m_4*r_4*np.sin(theta_4 - theta_1)) + dtheta_3*(dtheta_3*l_1*l_3*m_4*np.sin(theta_3 - theta_1) + dtheta_3*l_1*l_3*m_5*np.sin(theta_3 - theta_1) + dtheta_3*l_1*m_3*r_3*np.sin(theta_3 - theta_1)) + dtheta_2*mu_2 + dtheta_2*(dtheta_2*l_1*l_2*m_3*np.sin(theta_2 - theta_1) + dtheta_2*l_1*l_2*m_4*np.sin(theta_2 - theta_1) + dtheta_2*l_1*l_2*m_5*np.sin(theta_2 - theta_1) + dtheta_2*l_1*m_2*r_2*np.sin(theta_2 - theta_1)) - dtheta_1*(mu_1 + mu_2) - ddx_c*(-l_1*m_2 - l_1*m_3 - l_1*m_4 - l_1*m_5 - m_1*r_1)*np.cos(theta_1) - g*(-l_1*m_2 - l_1*m_3 - l_1*m_4 - l_1*m_5 - m_1*r_1)*np.sin(theta_1))*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) - (-l_1**2*(l_4*m_5 + m_4*r_4)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_4 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_4*m_5 + m_4*r_4)*np.cos(theta_4 - theta_2))*(dtheta_5**2*l_4*m_5*r_5*np.sin(theta_5 - theta_4) + dtheta_5*mu_5 - dtheta_4*(mu_4 + mu_5) + dtheta_3*mu_4 + dtheta_3*(-dtheta_3*l_3*l_4*m_5*np.sin(theta_4 - theta_3) - dtheta_3*l_3*m_4*r_4*np.sin(theta_4 - theta_3)) + dtheta_2*(-dtheta_2*l_2*l_4*m_5*np.sin(theta_4 - theta_2) - dtheta_2*l_2*m_4*r_4*np.sin(theta_4 - theta_2)) + dtheta_1*(-dtheta_1*l_1*l_4*m_5*np.sin(theta_4 - theta_1) - dtheta_1*l_1*m_4*r_4*np.sin(theta_4 - theta_1)) - ddx_c*(-l_4*m_5 - m_4*r_4)*np.cos(theta_4) - g*(-l_4*m_5 - m_4*r_4)*np.sin(theta_4) - l_1*(l_4*m_5 + m_4*r_4)*(dtheta_5**2*l_1*m_5*r_5*np.sin(theta_5 - theta_1) + dtheta_4*(dtheta_4*l_1*l_4*m_5*np.sin(theta_4 - theta_1) + dtheta_4*l_1*m_4*r_4*np.sin(theta_4 - theta_1)) + dtheta_3*(dtheta_3*l_1*l_3*m_4*np.sin(theta_3 - theta_1) + dtheta_3*l_1*l_3*m_5*np.sin(theta_3 - theta_1) + dtheta_3*l_1*m_3*r_3*np.sin(theta_3 - theta_1)) + dtheta_2*mu_2 + dtheta_2*(dtheta_2*l_1*l_2*m_3*np.sin(theta_2 - theta_1) + dtheta_2*l_1*l_2*m_4*np.sin(theta_2 - theta_1) + dtheta_2*l_1*l_2*m_5*np.sin(theta_2 - theta_1) + dtheta_2*l_1*m_2*r_2*np.sin(theta_2 - theta_1)) - dtheta_1*(mu_1 + mu_2) - ddx_c*(-l_1*m_2 - l_1*m_3 - l_1*m_4 - l_1*m_5 - m_1*r_1)*np.cos(theta_1) - g*(-l_1*m_2 - l_1*m_3 - l_1*m_4 - l_1*m_5 - m_1*r_1)*np.sin(theta_1))*np.cos(theta_4 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) - (-l_1**2*(l_4*m_5 + m_4*r_4)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_4 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_4*m_5 + m_4*r_4)*np.cos(theta_4 - theta_2))*(dtheta_5**2*l_2*m_5*r_5*np.sin(theta_5 - theta_2) + dtheta_4*(dtheta_4*l_2*l_4*m_5*np.sin(theta_4 - theta_2) + dtheta_4*l_2*m_4*r_4*np.sin(theta_4 - theta_2)) + dtheta_3*mu_3 + dtheta_3*(dtheta_3*l_2*l_3*m_4*np.sin(theta_3 - theta_2) + dtheta_3*l_2*l_3*m_5*np.sin(theta_3 - theta_2) + dtheta_3*l_2*m_3*r_3*np.sin(theta_3 - theta_2)) - dtheta_2*(mu_2 + mu_3) + dtheta_1*mu_2 + dtheta_1*(-dtheta_1*l_1*l_2*m_3*np.sin(theta_2 - theta_1) - dtheta_1*l_1*l_2*m_4*np.sin(theta_2 - theta_1) - dtheta_1*l_1*l_2*m_5*np.sin(theta_2 - theta_1) - dtheta_1*l_1*m_2*r_2*np.sin(theta_2 - theta_1)) - ddx_c*(-l_2*m_3 - l_2*m_4 - l_2*m_5 - m_2*r_2)*np.cos(theta_2) - g*(-l_2*m_3 - l_2*m_4 - l_2*m_5 - m_2*r_2)*np.sin(theta_2) - l_1*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*(dtheta_5**2*l_1*m_5*r_5*np.sin(theta_5 - theta_1) + dtheta_4*(dtheta_4*l_1*l_4*m_5*np.sin(theta_4 - theta_1) + dtheta_4*l_1*m_4*r_4*np.sin(theta_4 - theta_1)) + dtheta_3*(dtheta_3*l_1*l_3*m_4*np.sin(theta_3 - theta_1) + dtheta_3*l_1*l_3*m_5*np.sin(theta_3 - theta_1) + dtheta_3*l_1*m_3*r_3*np.sin(theta_3 - theta_1)) + dtheta_2*mu_2 + dtheta_2*(dtheta_2*l_1*l_2*m_3*np.sin(theta_2 - theta_1) + dtheta_2*l_1*l_2*m_4*np.sin(theta_2 - theta_1) + dtheta_2*l_1*l_2*m_5*np.sin(theta_2 - theta_1) + dtheta_2*l_1*m_2*r_2*np.sin(theta_2 - theta_1)) - dtheta_1*(mu_1 + mu_2) - ddx_c*(-l_1*m_2 - l_1*m_3 - l_1*m_4 - l_1*m_5 - m_1*r_1)*np.cos(theta_1) - g*(-l_1*m_2 - l_1*m_3 - l_1*m_4 - l_1*m_5 - m_1*r_1)*np.sin(theta_1))*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2))/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2) - (-l_1**2*(l_4*m_5 + m_4*r_4)*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_4 - theta_1)*np.cos(theta_3 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_3*(l_4*m_5 + m_4*r_4)*np.cos(theta_4 - theta_3) - (-l_1**2*(l_4*m_5 + m_4*r_4)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_4 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_4*m_5 + m_4*r_4)*np.cos(theta_4 - theta_2))*(-l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_3 - theta_2))/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2))*(dtheta_5**2*l_3*m_5*r_5*np.sin(theta_5 - theta_3) + dtheta_4*mu_4 + dtheta_4*(dtheta_4*l_3*l_4*m_5*np.sin(theta_4 - theta_3) + dtheta_4*l_3*m_4*r_4*np.sin(theta_4 - theta_3)) - dtheta_3*(mu_3 + mu_4) + dtheta_2*mu_3 + dtheta_2*(-dtheta_2*l_2*l_3*m_4*np.sin(theta_3 - theta_2) - dtheta_2*l_2*l_3*m_5*np.sin(theta_3 - theta_2) - dtheta_2*l_2*m_3*r_3*np.sin(theta_3 - theta_2)) + dtheta_1*(-dtheta_1*l_1*l_3*m_4*np.sin(theta_3 - theta_1) - dtheta_1*l_1*l_3*m_5*np.sin(theta_3 - theta_1) - dtheta_1*l_1*m_3*r_3*np.sin(theta_3 - theta_1)) - ddx_c*(-l_3*m_4 - l_3*m_5 - m_3*r_3)*np.cos(theta_3) - g*(-l_3*m_4 - l_3*m_5 - m_3*r_3)*np.sin(theta_3) - l_1*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(dtheta_5**2*l_1*m_5*r_5*np.sin(theta_5 - theta_1) + dtheta_4*(dtheta_4*l_1*l_4*m_5*np.sin(theta_4 - theta_1) + dtheta_4*l_1*m_4*r_4*np.sin(theta_4 - theta_1)) + dtheta_3*(dtheta_3*l_1*l_3*m_4*np.sin(theta_3 - theta_1) + dtheta_3*l_1*l_3*m_5*np.sin(theta_3 - theta_1) + dtheta_3*l_1*m_3*r_3*np.sin(theta_3 - theta_1)) + dtheta_2*mu_2 + dtheta_2*(dtheta_2*l_1*l_2*m_3*np.sin(theta_2 - theta_1) + dtheta_2*l_1*l_2*m_4*np.sin(theta_2 - theta_1) + dtheta_2*l_1*l_2*m_5*np.sin(theta_2 - theta_1) + dtheta_2*l_1*m_2*r_2*np.sin(theta_2 - theta_1)) - dtheta_1*(mu_1 + mu_2) - ddx_c*(-l_1*m_2 - l_1*m_3 - l_1*m_4 - l_1*m_5 - m_1*r_1)*np.cos(theta_1) - g*(-l_1*m_2 - l_1*m_3 - l_1*m_4 - l_1*m_5 - m_1*r_1)*np.sin(theta_1))*np.cos(theta_3 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) - (-l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_3 - theta_2))*(dtheta_5**2*l_2*m_5*r_5*np.sin(theta_5 - theta_2) + dtheta_4*(dtheta_4*l_2*l_4*m_5*np.sin(theta_4 - theta_2) + dtheta_4*l_2*m_4*r_4*np.sin(theta_4 - theta_2)) + dtheta_3*mu_3 + dtheta_3*(dtheta_3*l_2*l_3*m_4*np.sin(theta_3 - theta_2) + dtheta_3*l_2*l_3*m_5*np.sin(theta_3 - theta_2) + dtheta_3*l_2*m_3*r_3*np.sin(theta_3 - theta_2)) - dtheta_2*(mu_2 + mu_3) + dtheta_1*mu_2 + dtheta_1*(-dtheta_1*l_1*l_2*m_3*np.sin(theta_2 - theta_1) - dtheta_1*l_1*l_2*m_4*np.sin(theta_2 - theta_1) - dtheta_1*l_1*l_2*m_5*np.sin(theta_2 - theta_1) - dtheta_1*l_1*m_2*r_2*np.sin(theta_2 - theta_1)) - ddx_c*(-l_2*m_3 - l_2*m_4 - l_2*m_5 - m_2*r_2)*np.cos(theta_2) - g*(-l_2*m_3 - l_2*m_4 - l_2*m_5 - m_2*r_2)*np.sin(theta_2) - l_1*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*(dtheta_5**2*l_1*m_5*r_5*np.sin(theta_5 - theta_1) + dtheta_4*(dtheta_4*l_1*l_4*m_5*np.sin(theta_4 - theta_1) + dtheta_4*l_1*m_4*r_4*np.sin(theta_4 - theta_1)) + dtheta_3*(dtheta_3*l_1*l_3*m_4*np.sin(theta_3 - theta_1) + dtheta_3*l_1*l_3*m_5*np.sin(theta_3 - theta_1) + dtheta_3*l_1*m_3*r_3*np.sin(theta_3 - theta_1)) + dtheta_2*mu_2 + dtheta_2*(dtheta_2*l_1*l_2*m_3*np.sin(theta_2 - theta_1) + dtheta_2*l_1*l_2*m_4*np.sin(theta_2 - theta_1) + dtheta_2*l_1*l_2*m_5*np.sin(theta_2 - theta_1) + dtheta_2*l_1*m_2*r_2*np.sin(theta_2 - theta_1)) - dtheta_1*(mu_1 + mu_2) - ddx_c*(-l_1*m_2 - l_1*m_3 - l_1*m_4 - l_1*m_5 - m_1*r_1)*np.cos(theta_1) - g*(-l_1*m_2 - l_1*m_3 - l_1*m_4 - l_1*m_5 - m_1*r_1)*np.sin(theta_1))*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2))/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2))/(I_3 - l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)**2*np.cos(theta_3 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_3**2*m_4 + l_3**2*m_5 + m_3*r_3**2 - (-l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_3 - theta_2))**2/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2)) - (-l_1**2*m_5*r_5*(l_4*m_5 + m_4*r_4)*np.cos(theta_5 - theta_1)*np.cos(theta_4 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_4*m_5*r_5*np.cos(theta_5 - theta_4) - (-l_1**2*(l_4*m_5 + m_4*r_4)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_4 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_4*m_5 + m_4*r_4)*np.cos(theta_4 - theta_2))*(-l_1**2*m_5*r_5*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_5 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*m_5*r_5*np.cos(theta_5 - theta_2))/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2) - (-l_1**2*(l_4*m_5 + m_4*r_4)*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_4 - theta_1)*np.cos(theta_3 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_3*(l_4*m_5 + m_4*r_4)*np.cos(theta_4 - theta_3) - (-l_1**2*(l_4*m_5 + m_4*r_4)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_4 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_4*m_5 + m_4*r_4)*np.cos(theta_4 - theta_2))*(-l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_3 - theta_2))/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2))*(-l_1**2*m_5*r_5*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_5 - theta_1)*np.cos(theta_3 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_3*m_5*r_5*np.cos(theta_5 - theta_3) - (-l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_3 - theta_2))*(-l_1**2*m_5*r_5*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_5 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*m_5*r_5*np.cos(theta_5 - theta_2))/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2))/(I_3 - l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)**2*np.cos(theta_3 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_3**2*m_4 + l_3**2*m_5 + m_3*r_3**2 - (-l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_3 - theta_2))**2/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2)))*(-dtheta_5*mu_5 - dtheta_4**2*l_4*m_5*r_5*np.sin(theta_5 - theta_4) + dtheta_4*mu_5 - dtheta_3**2*l_3*m_5*r_5*np.sin(theta_5 - theta_3) - dtheta_2**2*l_2*m_5*r_5*np.sin(theta_5 - theta_2) - dtheta_1**2*l_1*m_5*r_5*np.sin(theta_5 - theta_1) + ddx_c*m_5*r_5*np.cos(theta_5) + g*m_5*r_5*np.sin(theta_5) - l_1*m_5*r_5*(dtheta_5**2*l_1*m_5*r_5*np.sin(theta_5 - theta_1) + dtheta_4*(dtheta_4*l_1*l_4*m_5*np.sin(theta_4 - theta_1) + dtheta_4*l_1*m_4*r_4*np.sin(theta_4 - theta_1)) + dtheta_3*(dtheta_3*l_1*l_3*m_4*np.sin(theta_3 - theta_1) + dtheta_3*l_1*l_3*m_5*np.sin(theta_3 - theta_1) + dtheta_3*l_1*m_3*r_3*np.sin(theta_3 - theta_1)) + dtheta_2*mu_2 + dtheta_2*(dtheta_2*l_1*l_2*m_3*np.sin(theta_2 - theta_1) + dtheta_2*l_1*l_2*m_4*np.sin(theta_2 - theta_1) + dtheta_2*l_1*l_2*m_5*np.sin(theta_2 - theta_1) + dtheta_2*l_1*m_2*r_2*np.sin(theta_2 - theta_1)) - dtheta_1*(mu_1 + mu_2) - ddx_c*(-l_1*m_2 - l_1*m_3 - l_1*m_4 - l_1*m_5 - m_1*r_1)*np.cos(theta_1) - g*(-l_1*m_2 - l_1*m_3 - l_1*m_4 - l_1*m_5 - m_1*r_1)*np.sin(theta_1))*np.cos(theta_5 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) - (-l_1**2*m_5*r_5*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_5 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*m_5*r_5*np.cos(theta_5 - theta_2))*(dtheta_5**2*l_2*m_5*r_5*np.sin(theta_5 - theta_2) + dtheta_4*(dtheta_4*l_2*l_4*m_5*np.sin(theta_4 - theta_2) + dtheta_4*l_2*m_4*r_4*np.sin(theta_4 - theta_2)) + dtheta_3*mu_3 + dtheta_3*(dtheta_3*l_2*l_3*m_4*np.sin(theta_3 - theta_2) + dtheta_3*l_2*l_3*m_5*np.sin(theta_3 - theta_2) + dtheta_3*l_2*m_3*r_3*np.sin(theta_3 - theta_2)) - dtheta_2*(mu_2 + mu_3) + dtheta_1*mu_2 + dtheta_1*(-dtheta_1*l_1*l_2*m_3*np.sin(theta_2 - theta_1) - dtheta_1*l_1*l_2*m_4*np.sin(theta_2 - theta_1) - dtheta_1*l_1*l_2*m_5*np.sin(theta_2 - theta_1) - dtheta_1*l_1*m_2*r_2*np.sin(theta_2 - theta_1)) - ddx_c*(-l_2*m_3 - l_2*m_4 - l_2*m_5 - m_2*r_2)*np.cos(theta_2) - g*(-l_2*m_3 - l_2*m_4 - l_2*m_5 - m_2*r_2)*np.sin(theta_2) - l_1*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*(dtheta_5**2*l_1*m_5*r_5*np.sin(theta_5 - theta_1) + dtheta_4*(dtheta_4*l_1*l_4*m_5*np.sin(theta_4 - theta_1) + dtheta_4*l_1*m_4*r_4*np.sin(theta_4 - theta_1)) + dtheta_3*(dtheta_3*l_1*l_3*m_4*np.sin(theta_3 - theta_1) + dtheta_3*l_1*l_3*m_5*np.sin(theta_3 - theta_1) + dtheta_3*l_1*m_3*r_3*np.sin(theta_3 - theta_1)) + dtheta_2*mu_2 + dtheta_2*(dtheta_2*l_1*l_2*m_3*np.sin(theta_2 - theta_1) + dtheta_2*l_1*l_2*m_4*np.sin(theta_2 - theta_1) + dtheta_2*l_1*l_2*m_5*np.sin(theta_2 - theta_1) + dtheta_2*l_1*m_2*r_2*np.sin(theta_2 - theta_1)) - dtheta_1*(mu_1 + mu_2) - ddx_c*(-l_1*m_2 - l_1*m_3 - l_1*m_4 - l_1*m_5 - m_1*r_1)*np.cos(theta_1) - g*(-l_1*m_2 - l_1*m_3 - l_1*m_4 - l_1*m_5 - m_1*r_1)*np.sin(theta_1))*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2))/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2) - (-l_1**2*m_5*r_5*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_5 - theta_1)*np.cos(theta_3 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_3*m_5*r_5*np.cos(theta_5 - theta_3) - (-l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_3 - theta_2))*(-l_1**2*m_5*r_5*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_5 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*m_5*r_5*np.cos(theta_5 - theta_2))/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2))*(dtheta_5**2*l_3*m_5*r_5*np.sin(theta_5 - theta_3) + dtheta_4*mu_4 + dtheta_4*(dtheta_4*l_3*l_4*m_5*np.sin(theta_4 - theta_3) + dtheta_4*l_3*m_4*r_4*np.sin(theta_4 - theta_3)) - dtheta_3*(mu_3 + mu_4) + dtheta_2*mu_3 + dtheta_2*(-dtheta_2*l_2*l_3*m_4*np.sin(theta_3 - theta_2) - dtheta_2*l_2*l_3*m_5*np.sin(theta_3 - theta_2) - dtheta_2*l_2*m_3*r_3*np.sin(theta_3 - theta_2)) + dtheta_1*(-dtheta_1*l_1*l_3*m_4*np.sin(theta_3 - theta_1) - dtheta_1*l_1*l_3*m_5*np.sin(theta_3 - theta_1) - dtheta_1*l_1*m_3*r_3*np.sin(theta_3 - theta_1)) - ddx_c*(-l_3*m_4 - l_3*m_5 - m_3*r_3)*np.cos(theta_3) - g*(-l_3*m_4 - l_3*m_5 - m_3*r_3)*np.sin(theta_3) - l_1*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(dtheta_5**2*l_1*m_5*r_5*np.sin(theta_5 - theta_1) + dtheta_4*(dtheta_4*l_1*l_4*m_5*np.sin(theta_4 - theta_1) + dtheta_4*l_1*m_4*r_4*np.sin(theta_4 - theta_1)) + dtheta_3*(dtheta_3*l_1*l_3*m_4*np.sin(theta_3 - theta_1) + dtheta_3*l_1*l_3*m_5*np.sin(theta_3 - theta_1) + dtheta_3*l_1*m_3*r_3*np.sin(theta_3 - theta_1)) + dtheta_2*mu_2 + dtheta_2*(dtheta_2*l_1*l_2*m_3*np.sin(theta_2 - theta_1) + dtheta_2*l_1*l_2*m_4*np.sin(theta_2 - theta_1) + dtheta_2*l_1*l_2*m_5*np.sin(theta_2 - theta_1) + dtheta_2*l_1*m_2*r_2*np.sin(theta_2 - theta_1)) - dtheta_1*(mu_1 + mu_2) - ddx_c*(-l_1*m_2 - l_1*m_3 - l_1*m_4 - l_1*m_5 - m_1*r_1)*np.cos(theta_1) - g*(-l_1*m_2 - l_1*m_3 - l_1*m_4 - l_1*m_5 - m_1*r_1)*np.sin(theta_1))*np.cos(theta_3 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) - (-l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_3 - theta_2))*(dtheta_5**2*l_2*m_5*r_5*np.sin(theta_5 - theta_2) + dtheta_4*(dtheta_4*l_2*l_4*m_5*np.sin(theta_4 - theta_2) + dtheta_4*l_2*m_4*r_4*np.sin(theta_4 - theta_2)) + dtheta_3*mu_3 + dtheta_3*(dtheta_3*l_2*l_3*m_4*np.sin(theta_3 - theta_2) + dtheta_3*l_2*l_3*m_5*np.sin(theta_3 - theta_2) + dtheta_3*l_2*m_3*r_3*np.sin(theta_3 - theta_2)) - dtheta_2*(mu_2 + mu_3) + dtheta_1*mu_2 + dtheta_1*(-dtheta_1*l_1*l_2*m_3*np.sin(theta_2 - theta_1) - dtheta_1*l_1*l_2*m_4*np.sin(theta_2 - theta_1) - dtheta_1*l_1*l_2*m_5*np.sin(theta_2 - theta_1) - dtheta_1*l_1*m_2*r_2*np.sin(theta_2 - theta_1)) - ddx_c*(-l_2*m_3 - l_2*m_4 - l_2*m_5 - m_2*r_2)*np.cos(theta_2) - g*(-l_2*m_3 - l_2*m_4 - l_2*m_5 - m_2*r_2)*np.sin(theta_2) - l_1*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*(dtheta_5**2*l_1*m_5*r_5*np.sin(theta_5 - theta_1) + dtheta_4*(dtheta_4*l_1*l_4*m_5*np.sin(theta_4 - theta_1) + dtheta_4*l_1*m_4*r_4*np.sin(theta_4 - theta_1)) + dtheta_3*(dtheta_3*l_1*l_3*m_4*np.sin(theta_3 - theta_1) + dtheta_3*l_1*l_3*m_5*np.sin(theta_3 - theta_1) + dtheta_3*l_1*m_3*r_3*np.sin(theta_3 - theta_1)) + dtheta_2*mu_2 + dtheta_2*(dtheta_2*l_1*l_2*m_3*np.sin(theta_2 - theta_1) + dtheta_2*l_1*l_2*m_4*np.sin(theta_2 - theta_1) + dtheta_2*l_1*l_2*m_5*np.sin(theta_2 - theta_1) + dtheta_2*l_1*m_2*r_2*np.sin(theta_2 - theta_1)) - dtheta_1*(mu_1 + mu_2) - ddx_c*(-l_1*m_2 - l_1*m_3 - l_1*m_4 - l_1*m_5 - m_1*r_1)*np.cos(theta_1) - g*(-l_1*m_2 - l_1*m_3 - l_1*m_4 - l_1*m_5 - m_1*r_1)*np.sin(theta_1))*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2))/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2))/(I_3 - l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)**2*np.cos(theta_3 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_3**2*m_4 + l_3**2*m_5 + m_3*r_3**2 - (-l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_3 - theta_2))**2/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2)) - (-l_1**2*m_5*r_5*(l_4*m_5 + m_4*r_4)*np.cos(theta_5 - theta_1)*np.cos(theta_4 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_4*m_5*r_5*np.cos(theta_5 - theta_4) - (-l_1**2*(l_4*m_5 + m_4*r_4)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_4 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_4*m_5 + m_4*r_4)*np.cos(theta_4 - theta_2))*(-l_1**2*m_5*r_5*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_5 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*m_5*r_5*np.cos(theta_5 - theta_2))/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2) - (-l_1**2*(l_4*m_5 + m_4*r_4)*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_4 - theta_1)*np.cos(theta_3 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_3*(l_4*m_5 + m_4*r_4)*np.cos(theta_4 - theta_3) - (-l_1**2*(l_4*m_5 + m_4*r_4)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_4 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_4*m_5 + m_4*r_4)*np.cos(theta_4 - theta_2))*(-l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_3 - theta_2))/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2))*(-l_1**2*m_5*r_5*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_5 - theta_1)*np.cos(theta_3 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_3*m_5*r_5*np.cos(theta_5 - theta_3) - (-l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_3 - theta_2))*(-l_1**2*m_5*r_5*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_5 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*m_5*r_5*np.cos(theta_5 - theta_2))/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2))/(I_3 - l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)**2*np.cos(theta_3 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_3**2*m_4 + l_3**2*m_5 + m_3*r_3**2 - (-l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_3 - theta_2))**2/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2)))*(dtheta_5**2*l_4*m_5*r_5*np.sin(theta_5 - theta_4) + dtheta_5*mu_5 - dtheta_4*(mu_4 + mu_5) + dtheta_3*mu_4 + dtheta_3*(-dtheta_3*l_3*l_4*m_5*np.sin(theta_4 - theta_3) - dtheta_3*l_3*m_4*r_4*np.sin(theta_4 - theta_3)) + dtheta_2*(-dtheta_2*l_2*l_4*m_5*np.sin(theta_4 - theta_2) - dtheta_2*l_2*m_4*r_4*np.sin(theta_4 - theta_2)) + dtheta_1*(-dtheta_1*l_1*l_4*m_5*np.sin(theta_4 - theta_1) - dtheta_1*l_1*m_4*r_4*np.sin(theta_4 - theta_1)) - ddx_c*(-l_4*m_5 - m_4*r_4)*np.cos(theta_4) - g*(-l_4*m_5 - m_4*r_4)*np.sin(theta_4) - l_1*(l_4*m_5 + m_4*r_4)*(dtheta_5**2*l_1*m_5*r_5*np.sin(theta_5 - theta_1) + dtheta_4*(dtheta_4*l_1*l_4*m_5*np.sin(theta_4 - theta_1) + dtheta_4*l_1*m_4*r_4*np.sin(theta_4 - theta_1)) + dtheta_3*(dtheta_3*l_1*l_3*m_4*np.sin(theta_3 - theta_1) + dtheta_3*l_1*l_3*m_5*np.sin(theta_3 - theta_1) + dtheta_3*l_1*m_3*r_3*np.sin(theta_3 - theta_1)) + dtheta_2*mu_2 + dtheta_2*(dtheta_2*l_1*l_2*m_3*np.sin(theta_2 - theta_1) + dtheta_2*l_1*l_2*m_4*np.sin(theta_2 - theta_1) + dtheta_2*l_1*l_2*m_5*np.sin(theta_2 - theta_1) + dtheta_2*l_1*m_2*r_2*np.sin(theta_2 - theta_1)) - dtheta_1*(mu_1 + mu_2) - ddx_c*(-l_1*m_2 - l_1*m_3 - l_1*m_4 - l_1*m_5 - m_1*r_1)*np.cos(theta_1) - g*(-l_1*m_2 - l_1*m_3 - l_1*m_4 - l_1*m_5 - m_1*r_1)*np.sin(theta_1))*np.cos(theta_4 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) - (-l_1**2*(l_4*m_5 + m_4*r_4)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_4 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_4*m_5 + m_4*r_4)*np.cos(theta_4 - theta_2))*(dtheta_5**2*l_2*m_5*r_5*np.sin(theta_5 - theta_2) + dtheta_4*(dtheta_4*l_2*l_4*m_5*np.sin(theta_4 - theta_2) + dtheta_4*l_2*m_4*r_4*np.sin(theta_4 - theta_2)) + dtheta_3*mu_3 + dtheta_3*(dtheta_3*l_2*l_3*m_4*np.sin(theta_3 - theta_2) + dtheta_3*l_2*l_3*m_5*np.sin(theta_3 - theta_2) + dtheta_3*l_2*m_3*r_3*np.sin(theta_3 - theta_2)) - dtheta_2*(mu_2 + mu_3) + dtheta_1*mu_2 + dtheta_1*(-dtheta_1*l_1*l_2*m_3*np.sin(theta_2 - theta_1) - dtheta_1*l_1*l_2*m_4*np.sin(theta_2 - theta_1) - dtheta_1*l_1*l_2*m_5*np.sin(theta_2 - theta_1) - dtheta_1*l_1*m_2*r_2*np.sin(theta_2 - theta_1)) - ddx_c*(-l_2*m_3 - l_2*m_4 - l_2*m_5 - m_2*r_2)*np.cos(theta_2) - g*(-l_2*m_3 - l_2*m_4 - l_2*m_5 - m_2*r_2)*np.sin(theta_2) - l_1*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*(dtheta_5**2*l_1*m_5*r_5*np.sin(theta_5 - theta_1) + dtheta_4*(dtheta_4*l_1*l_4*m_5*np.sin(theta_4 - theta_1) + dtheta_4*l_1*m_4*r_4*np.sin(theta_4 - theta_1)) + dtheta_3*(dtheta_3*l_1*l_3*m_4*np.sin(theta_3 - theta_1) + dtheta_3*l_1*l_3*m_5*np.sin(theta_3 - theta_1) + dtheta_3*l_1*m_3*r_3*np.sin(theta_3 - theta_1)) + dtheta_2*mu_2 + dtheta_2*(dtheta_2*l_1*l_2*m_3*np.sin(theta_2 - theta_1) + dtheta_2*l_1*l_2*m_4*np.sin(theta_2 - theta_1) + dtheta_2*l_1*l_2*m_5*np.sin(theta_2 - theta_1) + dtheta_2*l_1*m_2*r_2*np.sin(theta_2 - theta_1)) - dtheta_1*(mu_1 + mu_2) - ddx_c*(-l_1*m_2 - l_1*m_3 - l_1*m_4 - l_1*m_5 - m_1*r_1)*np.cos(theta_1) - g*(-l_1*m_2 - l_1*m_3 - l_1*m_4 - l_1*m_5 - m_1*r_1)*np.sin(theta_1))*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2))/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2) - (-l_1**2*(l_4*m_5 + m_4*r_4)*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_4 - theta_1)*np.cos(theta_3 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_3*(l_4*m_5 + m_4*r_4)*np.cos(theta_4 - theta_3) - (-l_1**2*(l_4*m_5 + m_4*r_4)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_4 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_4*m_5 + m_4*r_4)*np.cos(theta_4 - theta_2))*(-l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_3 - theta_2))/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2))*(dtheta_5**2*l_3*m_5*r_5*np.sin(theta_5 - theta_3) + dtheta_4*mu_4 + dtheta_4*(dtheta_4*l_3*l_4*m_5*np.sin(theta_4 - theta_3) + dtheta_4*l_3*m_4*r_4*np.sin(theta_4 - theta_3)) - dtheta_3*(mu_3 + mu_4) + dtheta_2*mu_3 + dtheta_2*(-dtheta_2*l_2*l_3*m_4*np.sin(theta_3 - theta_2) - dtheta_2*l_2*l_3*m_5*np.sin(theta_3 - theta_2) - dtheta_2*l_2*m_3*r_3*np.sin(theta_3 - theta_2)) + dtheta_1*(-dtheta_1*l_1*l_3*m_4*np.sin(theta_3 - theta_1) - dtheta_1*l_1*l_3*m_5*np.sin(theta_3 - theta_1) - dtheta_1*l_1*m_3*r_3*np.sin(theta_3 - theta_1)) - ddx_c*(-l_3*m_4 - l_3*m_5 - m_3*r_3)*np.cos(theta_3) - g*(-l_3*m_4 - l_3*m_5 - m_3*r_3)*np.sin(theta_3) - l_1*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(dtheta_5**2*l_1*m_5*r_5*np.sin(theta_5 - theta_1) + dtheta_4*(dtheta_4*l_1*l_4*m_5*np.sin(theta_4 - theta_1) + dtheta_4*l_1*m_4*r_4*np.sin(theta_4 - theta_1)) + dtheta_3*(dtheta_3*l_1*l_3*m_4*np.sin(theta_3 - theta_1) + dtheta_3*l_1*l_3*m_5*np.sin(theta_3 - theta_1) + dtheta_3*l_1*m_3*r_3*np.sin(theta_3 - theta_1)) + dtheta_2*mu_2 + dtheta_2*(dtheta_2*l_1*l_2*m_3*np.sin(theta_2 - theta_1) + dtheta_2*l_1*l_2*m_4*np.sin(theta_2 - theta_1) + dtheta_2*l_1*l_2*m_5*np.sin(theta_2 - theta_1) + dtheta_2*l_1*m_2*r_2*np.sin(theta_2 - theta_1)) - dtheta_1*(mu_1 + mu_2) - ddx_c*(-l_1*m_2 - l_1*m_3 - l_1*m_4 - l_1*m_5 - m_1*r_1)*np.cos(theta_1) - g*(-l_1*m_2 - l_1*m_3 - l_1*m_4 - l_1*m_5 - m_1*r_1)*np.sin(theta_1))*np.cos(theta_3 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) - (-l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_3 - theta_2))*(dtheta_5**2*l_2*m_5*r_5*np.sin(theta_5 - theta_2) + dtheta_4*(dtheta_4*l_2*l_4*m_5*np.sin(theta_4 - theta_2) + dtheta_4*l_2*m_4*r_4*np.sin(theta_4 - theta_2)) + dtheta_3*mu_3 + dtheta_3*(dtheta_3*l_2*l_3*m_4*np.sin(theta_3 - theta_2) + dtheta_3*l_2*l_3*m_5*np.sin(theta_3 - theta_2) + dtheta_3*l_2*m_3*r_3*np.sin(theta_3 - theta_2)) - dtheta_2*(mu_2 + mu_3) + dtheta_1*mu_2 + dtheta_1*(-dtheta_1*l_1*l_2*m_3*np.sin(theta_2 - theta_1) - dtheta_1*l_1*l_2*m_4*np.sin(theta_2 - theta_1) - dtheta_1*l_1*l_2*m_5*np.sin(theta_2 - theta_1) - dtheta_1*l_1*m_2*r_2*np.sin(theta_2 - theta_1)) - ddx_c*(-l_2*m_3 - l_2*m_4 - l_2*m_5 - m_2*r_2)*np.cos(theta_2) - g*(-l_2*m_3 - l_2*m_4 - l_2*m_5 - m_2*r_2)*np.sin(theta_2) - l_1*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*(dtheta_5**2*l_1*m_5*r_5*np.sin(theta_5 - theta_1) + dtheta_4*(dtheta_4*l_1*l_4*m_5*np.sin(theta_4 - theta_1) + dtheta_4*l_1*m_4*r_4*np.sin(theta_4 - theta_1)) + dtheta_3*(dtheta_3*l_1*l_3*m_4*np.sin(theta_3 - theta_1) + dtheta_3*l_1*l_3*m_5*np.sin(theta_3 - theta_1) + dtheta_3*l_1*m_3*r_3*np.sin(theta_3 - theta_1)) + dtheta_2*mu_2 + dtheta_2*(dtheta_2*l_1*l_2*m_3*np.sin(theta_2 - theta_1) + dtheta_2*l_1*l_2*m_4*np.sin(theta_2 - theta_1) + dtheta_2*l_1*l_2*m_5*np.sin(theta_2 - theta_1) + dtheta_2*l_1*m_2*r_2*np.sin(theta_2 - theta_1)) - dtheta_1*(mu_1 + mu_2) - ddx_c*(-l_1*m_2 - l_1*m_3 - l_1*m_4 - l_1*m_5 - m_1*r_1)*np.cos(theta_1) - g*(-l_1*m_2 - l_1*m_3 - l_1*m_4 - l_1*m_5 - m_1*r_1)*np.sin(theta_1))*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2))/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2))/(I_3 - l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)**2*np.cos(theta_3 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_3**2*m_4 + l_3**2*m_5 + m_3*r_3**2 - (-l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_3 - theta_2))**2/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2)))/(I_4 - l_1**2*(l_4*m_5 + m_4*r_4)**2*np.cos(theta_4 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_4**2*m_5 + m_4*r_4**2 - (-l_1**2*(l_4*m_5 + m_4*r_4)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_4 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_4*m_5 + m_4*r_4)*np.cos(theta_4 - theta_2))**2/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2) - (-l_1**2*(l_4*m_5 + m_4*r_4)*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_4 - theta_1)*np.cos(theta_3 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_3*(l_4*m_5 + m_4*r_4)*np.cos(theta_4 - theta_3) - (-l_1**2*(l_4*m_5 + m_4*r_4)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_4 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_4*m_5 + m_4*r_4)*np.cos(theta_4 - theta_2))*(-l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_3 - theta_2))/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2))**2/(I_3 - l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)**2*np.cos(theta_3 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_3**2*m_4 + l_3**2*m_5 + m_3*r_3**2 - (-l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_3 - theta_2))**2/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2))))/(I_5 - l_1**2*m_5**2*r_5**2*np.cos(theta_5 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + m_5*r_5**2 - (-l_1**2*m_5*r_5*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_5 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*m_5*r_5*np.cos(theta_5 - theta_2))**2/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2) - (-l_1**2*m_5*r_5*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_5 - theta_1)*np.cos(theta_3 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_3*m_5*r_5*np.cos(theta_5 - theta_3) - (-l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_3 - theta_2))*(-l_1**2*m_5*r_5*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_5 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*m_5*r_5*np.cos(theta_5 - theta_2))/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2))**2/(I_3 - l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)**2*np.cos(theta_3 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_3**2*m_4 + l_3**2*m_5 + m_3*r_3**2 - (-l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_3 - theta_2))**2/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2)) - (-l_1**2*m_5*r_5*(l_4*m_5 + m_4*r_4)*np.cos(theta_5 - theta_1)*np.cos(theta_4 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_4*m_5*r_5*np.cos(theta_5 - theta_4) - (-l_1**2*(l_4*m_5 + m_4*r_4)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_4 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_4*m_5 + m_4*r_4)*np.cos(theta_4 - theta_2))*(-l_1**2*m_5*r_5*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_5 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*m_5*r_5*np.cos(theta_5 - theta_2))/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2) - (-l_1**2*(l_4*m_5 + m_4*r_4)*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_4 - theta_1)*np.cos(theta_3 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_3*(l_4*m_5 + m_4*r_4)*np.cos(theta_4 - theta_3) - (-l_1**2*(l_4*m_5 + m_4*r_4)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_4 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_4*m_5 + m_4*r_4)*np.cos(theta_4 - theta_2))*(-l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_3 - theta_2))/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2))*(-l_1**2*m_5*r_5*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_5 - theta_1)*np.cos(theta_3 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_3*m_5*r_5*np.cos(theta_5 - theta_3) - (-l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_3 - theta_2))*(-l_1**2*m_5*r_5*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_5 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*m_5*r_5*np.cos(theta_5 - theta_2))/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2))/(I_3 - l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)**2*np.cos(theta_3 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_3**2*m_4 + l_3**2*m_5 + m_3*r_3**2 - (-l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_3 - theta_2))**2/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2)))**2/(I_4 - l_1**2*(l_4*m_5 + m_4*r_4)**2*np.cos(theta_4 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_4**2*m_5 + m_4*r_4**2 - (-l_1**2*(l_4*m_5 + m_4*r_4)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_4 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_4*m_5 + m_4*r_4)*np.cos(theta_4 - theta_2))**2/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2) - (-l_1**2*(l_4*m_5 + m_4*r_4)*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_4 - theta_1)*np.cos(theta_3 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_3*(l_4*m_5 + m_4*r_4)*np.cos(theta_4 - theta_3) - (-l_1**2*(l_4*m_5 + m_4*r_4)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_4 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_4*m_5 + m_4*r_4)*np.cos(theta_4 - theta_2))*(-l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_3 - theta_2))/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2))**2/(I_3 - l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)**2*np.cos(theta_3 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_3**2*m_4 + l_3**2*m_5 + m_3*r_3**2 - (-l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_3 - theta_2))**2/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2)))))/(I_4 - l_1**2*(l_4*m_5 + m_4*r_4)**2*np.cos(theta_4 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_4**2*m_5 + m_4*r_4**2 - (-l_1**2*(l_4*m_5 + m_4*r_4)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_4 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_4*m_5 + m_4*r_4)*np.cos(theta_4 - theta_2))**2/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2) - (-l_1**2*(l_4*m_5 + m_4*r_4)*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_4 - theta_1)*np.cos(theta_3 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_3*(l_4*m_5 + m_4*r_4)*np.cos(theta_4 - theta_3) - (-l_1**2*(l_4*m_5 + m_4*r_4)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_4 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_4*m_5 + m_4*r_4)*np.cos(theta_4 - theta_2))*(-l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_3 - theta_2))/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2))**2/(I_3 - l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)**2*np.cos(theta_3 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_3**2*m_4 + l_3**2*m_5 + m_3*r_3**2 - (-l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_3 - theta_2))**2/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2))) - (-l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_3 - theta_2))*(dtheta_5**2*l_3*m_5*r_5*np.sin(theta_5 - theta_3) + dtheta_4*mu_4 + dtheta_4*(dtheta_4*l_3*l_4*m_5*np.sin(theta_4 - theta_3) + dtheta_4*l_3*m_4*r_4*np.sin(theta_4 - theta_3)) - dtheta_3*(mu_3 + mu_4) + dtheta_2*mu_3 + dtheta_2*(-dtheta_2*l_2*l_3*m_4*np.sin(theta_3 - theta_2) - dtheta_2*l_2*l_3*m_5*np.sin(theta_3 - theta_2) - dtheta_2*l_2*m_3*r_3*np.sin(theta_3 - theta_2)) + dtheta_1*(-dtheta_1*l_1*l_3*m_4*np.sin(theta_3 - theta_1) - dtheta_1*l_1*l_3*m_5*np.sin(theta_3 - theta_1) - dtheta_1*l_1*m_3*r_3*np.sin(theta_3 - theta_1)) - ddx_c*(-l_3*m_4 - l_3*m_5 - m_3*r_3)*np.cos(theta_3) - g*(-l_3*m_4 - l_3*m_5 - m_3*r_3)*np.sin(theta_3) - l_1*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(dtheta_5**2*l_1*m_5*r_5*np.sin(theta_5 - theta_1) + dtheta_4*(dtheta_4*l_1*l_4*m_5*np.sin(theta_4 - theta_1) + dtheta_4*l_1*m_4*r_4*np.sin(theta_4 - theta_1)) + dtheta_3*(dtheta_3*l_1*l_3*m_4*np.sin(theta_3 - theta_1) + dtheta_3*l_1*l_3*m_5*np.sin(theta_3 - theta_1) + dtheta_3*l_1*m_3*r_3*np.sin(theta_3 - theta_1)) + dtheta_2*mu_2 + dtheta_2*(dtheta_2*l_1*l_2*m_3*np.sin(theta_2 - theta_1) + dtheta_2*l_1*l_2*m_4*np.sin(theta_2 - theta_1) + dtheta_2*l_1*l_2*m_5*np.sin(theta_2 - theta_1) + dtheta_2*l_1*m_2*r_2*np.sin(theta_2 - theta_1)) - dtheta_1*(mu_1 + mu_2) - ddx_c*(-l_1*m_2 - l_1*m_3 - l_1*m_4 - l_1*m_5 - m_1*r_1)*np.cos(theta_1) - g*(-l_1*m_2 - l_1*m_3 - l_1*m_4 - l_1*m_5 - m_1*r_1)*np.sin(theta_1))*np.cos(theta_3 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) - (-l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_3 - theta_2))*(dtheta_5**2*l_2*m_5*r_5*np.sin(theta_5 - theta_2) + dtheta_4*(dtheta_4*l_2*l_4*m_5*np.sin(theta_4 - theta_2) + dtheta_4*l_2*m_4*r_4*np.sin(theta_4 - theta_2)) + dtheta_3*mu_3 + dtheta_3*(dtheta_3*l_2*l_3*m_4*np.sin(theta_3 - theta_2) + dtheta_3*l_2*l_3*m_5*np.sin(theta_3 - theta_2) + dtheta_3*l_2*m_3*r_3*np.sin(theta_3 - theta_2)) - dtheta_2*(mu_2 + mu_3) + dtheta_1*mu_2 + dtheta_1*(-dtheta_1*l_1*l_2*m_3*np.sin(theta_2 - theta_1) - dtheta_1*l_1*l_2*m_4*np.sin(theta_2 - theta_1) - dtheta_1*l_1*l_2*m_5*np.sin(theta_2 - theta_1) - dtheta_1*l_1*m_2*r_2*np.sin(theta_2 - theta_1)) - ddx_c*(-l_2*m_3 - l_2*m_4 - l_2*m_5 - m_2*r_2)*np.cos(theta_2) - g*(-l_2*m_3 - l_2*m_4 - l_2*m_5 - m_2*r_2)*np.sin(theta_2) - l_1*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*(dtheta_5**2*l_1*m_5*r_5*np.sin(theta_5 - theta_1) + dtheta_4*(dtheta_4*l_1*l_4*m_5*np.sin(theta_4 - theta_1) + dtheta_4*l_1*m_4*r_4*np.sin(theta_4 - theta_1)) + dtheta_3*(dtheta_3*l_1*l_3*m_4*np.sin(theta_3 - theta_1) + dtheta_3*l_1*l_3*m_5*np.sin(theta_3 - theta_1) + dtheta_3*l_1*m_3*r_3*np.sin(theta_3 - theta_1)) + dtheta_2*mu_2 + dtheta_2*(dtheta_2*l_1*l_2*m_3*np.sin(theta_2 - theta_1) + dtheta_2*l_1*l_2*m_4*np.sin(theta_2 - theta_1) + dtheta_2*l_1*l_2*m_5*np.sin(theta_2 - theta_1) + dtheta_2*l_1*m_2*r_2*np.sin(theta_2 - theta_1)) - dtheta_1*(mu_1 + mu_2) - ddx_c*(-l_1*m_2 - l_1*m_3 - l_1*m_4 - l_1*m_5 - m_1*r_1)*np.cos(theta_1) - g*(-l_1*m_2 - l_1*m_3 - l_1*m_4 - l_1*m_5 - m_1*r_1)*np.sin(theta_1))*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2))/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2) - (-l_1**2*(l_4*m_5 + m_4*r_4)*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_4 - theta_1)*np.cos(theta_3 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_3*(l_4*m_5 + m_4*r_4)*np.cos(theta_4 - theta_3) - (-l_1**2*(l_4*m_5 + m_4*r_4)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_4 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_4*m_5 + m_4*r_4)*np.cos(theta_4 - theta_2))*(-l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_3 - theta_2))/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2))*(dtheta_5**2*l_4*m_5*r_5*np.sin(theta_5 - theta_4) + dtheta_5*mu_5 - dtheta_4*(mu_4 + mu_5) + dtheta_3*mu_4 + dtheta_3*(-dtheta_3*l_3*l_4*m_5*np.sin(theta_4 - theta_3) - dtheta_3*l_3*m_4*r_4*np.sin(theta_4 - theta_3)) + dtheta_2*(-dtheta_2*l_2*l_4*m_5*np.sin(theta_4 - theta_2) - dtheta_2*l_2*m_4*r_4*np.sin(theta_4 - theta_2)) + dtheta_1*(-dtheta_1*l_1*l_4*m_5*np.sin(theta_4 - theta_1) - dtheta_1*l_1*m_4*r_4*np.sin(theta_4 - theta_1)) - ddx_c*(-l_4*m_5 - m_4*r_4)*np.cos(theta_4) - g*(-l_4*m_5 - m_4*r_4)*np.sin(theta_4) - l_1*(l_4*m_5 + m_4*r_4)*(dtheta_5**2*l_1*m_5*r_5*np.sin(theta_5 - theta_1) + dtheta_4*(dtheta_4*l_1*l_4*m_5*np.sin(theta_4 - theta_1) + dtheta_4*l_1*m_4*r_4*np.sin(theta_4 - theta_1)) + dtheta_3*(dtheta_3*l_1*l_3*m_4*np.sin(theta_3 - theta_1) + dtheta_3*l_1*l_3*m_5*np.sin(theta_3 - theta_1) + dtheta_3*l_1*m_3*r_3*np.sin(theta_3 - theta_1)) + dtheta_2*mu_2 + dtheta_2*(dtheta_2*l_1*l_2*m_3*np.sin(theta_2 - theta_1) + dtheta_2*l_1*l_2*m_4*np.sin(theta_2 - theta_1) + dtheta_2*l_1*l_2*m_5*np.sin(theta_2 - theta_1) + dtheta_2*l_1*m_2*r_2*np.sin(theta_2 - theta_1)) - dtheta_1*(mu_1 + mu_2) - ddx_c*(-l_1*m_2 - l_1*m_3 - l_1*m_4 - l_1*m_5 - m_1*r_1)*np.cos(theta_1) - g*(-l_1*m_2 - l_1*m_3 - l_1*m_4 - l_1*m_5 - m_1*r_1)*np.sin(theta_1))*np.cos(theta_4 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) - (-l_1**2*(l_4*m_5 + m_4*r_4)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_4 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_4*m_5 + m_4*r_4)*np.cos(theta_4 - theta_2))*(dtheta_5**2*l_2*m_5*r_5*np.sin(theta_5 - theta_2) + dtheta_4*(dtheta_4*l_2*l_4*m_5*np.sin(theta_4 - theta_2) + dtheta_4*l_2*m_4*r_4*np.sin(theta_4 - theta_2)) + dtheta_3*mu_3 + dtheta_3*(dtheta_3*l_2*l_3*m_4*np.sin(theta_3 - theta_2) + dtheta_3*l_2*l_3*m_5*np.sin(theta_3 - theta_2) + dtheta_3*l_2*m_3*r_3*np.sin(theta_3 - theta_2)) - dtheta_2*(mu_2 + mu_3) + dtheta_1*mu_2 + dtheta_1*(-dtheta_1*l_1*l_2*m_3*np.sin(theta_2 - theta_1) - dtheta_1*l_1*l_2*m_4*np.sin(theta_2 - theta_1) - dtheta_1*l_1*l_2*m_5*np.sin(theta_2 - theta_1) - dtheta_1*l_1*m_2*r_2*np.sin(theta_2 - theta_1)) - ddx_c*(-l_2*m_3 - l_2*m_4 - l_2*m_5 - m_2*r_2)*np.cos(theta_2) - g*(-l_2*m_3 - l_2*m_4 - l_2*m_5 - m_2*r_2)*np.sin(theta_2) - l_1*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*(dtheta_5**2*l_1*m_5*r_5*np.sin(theta_5 - theta_1) + dtheta_4*(dtheta_4*l_1*l_4*m_5*np.sin(theta_4 - theta_1) + dtheta_4*l_1*m_4*r_4*np.sin(theta_4 - theta_1)) + dtheta_3*(dtheta_3*l_1*l_3*m_4*np.sin(theta_3 - theta_1) + dtheta_3*l_1*l_3*m_5*np.sin(theta_3 - theta_1) + dtheta_3*l_1*m_3*r_3*np.sin(theta_3 - theta_1)) + dtheta_2*mu_2 + dtheta_2*(dtheta_2*l_1*l_2*m_3*np.sin(theta_2 - theta_1) + dtheta_2*l_1*l_2*m_4*np.sin(theta_2 - theta_1) + dtheta_2*l_1*l_2*m_5*np.sin(theta_2 - theta_1) + dtheta_2*l_1*m_2*r_2*np.sin(theta_2 - theta_1)) - dtheta_1*(mu_1 + mu_2) - ddx_c*(-l_1*m_2 - l_1*m_3 - l_1*m_4 - l_1*m_5 - m_1*r_1)*np.cos(theta_1) - g*(-l_1*m_2 - l_1*m_3 - l_1*m_4 - l_1*m_5 - m_1*r_1)*np.sin(theta_1))*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2))/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2) - (-l_1**2*(l_4*m_5 + m_4*r_4)*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_4 - theta_1)*np.cos(theta_3 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_3*(l_4*m_5 + m_4*r_4)*np.cos(theta_4 - theta_3) - (-l_1**2*(l_4*m_5 + m_4*r_4)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_4 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_4*m_5 + m_4*r_4)*np.cos(theta_4 - theta_2))*(-l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_3 - theta_2))/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2))*(dtheta_5**2*l_3*m_5*r_5*np.sin(theta_5 - theta_3) + dtheta_4*mu_4 + dtheta_4*(dtheta_4*l_3*l_4*m_5*np.sin(theta_4 - theta_3) + dtheta_4*l_3*m_4*r_4*np.sin(theta_4 - theta_3)) - dtheta_3*(mu_3 + mu_4) + dtheta_2*mu_3 + dtheta_2*(-dtheta_2*l_2*l_3*m_4*np.sin(theta_3 - theta_2) - dtheta_2*l_2*l_3*m_5*np.sin(theta_3 - theta_2) - dtheta_2*l_2*m_3*r_3*np.sin(theta_3 - theta_2)) + dtheta_1*(-dtheta_1*l_1*l_3*m_4*np.sin(theta_3 - theta_1) - dtheta_1*l_1*l_3*m_5*np.sin(theta_3 - theta_1) - dtheta_1*l_1*m_3*r_3*np.sin(theta_3 - theta_1)) - ddx_c*(-l_3*m_4 - l_3*m_5 - m_3*r_3)*np.cos(theta_3) - g*(-l_3*m_4 - l_3*m_5 - m_3*r_3)*np.sin(theta_3) - l_1*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(dtheta_5**2*l_1*m_5*r_5*np.sin(theta_5 - theta_1) + dtheta_4*(dtheta_4*l_1*l_4*m_5*np.sin(theta_4 - theta_1) + dtheta_4*l_1*m_4*r_4*np.sin(theta_4 - theta_1)) + dtheta_3*(dtheta_3*l_1*l_3*m_4*np.sin(theta_3 - theta_1) + dtheta_3*l_1*l_3*m_5*np.sin(theta_3 - theta_1) + dtheta_3*l_1*m_3*r_3*np.sin(theta_3 - theta_1)) + dtheta_2*mu_2 + dtheta_2*(dtheta_2*l_1*l_2*m_3*np.sin(theta_2 - theta_1) + dtheta_2*l_1*l_2*m_4*np.sin(theta_2 - theta_1) + dtheta_2*l_1*l_2*m_5*np.sin(theta_2 - theta_1) + dtheta_2*l_1*m_2*r_2*np.sin(theta_2 - theta_1)) - dtheta_1*(mu_1 + mu_2) - ddx_c*(-l_1*m_2 - l_1*m_3 - l_1*m_4 - l_1*m_5 - m_1*r_1)*np.cos(theta_1) - g*(-l_1*m_2 - l_1*m_3 - l_1*m_4 - l_1*m_5 - m_1*r_1)*np.sin(theta_1))*np.cos(theta_3 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) - (-l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_3 - theta_2))*(dtheta_5**2*l_2*m_5*r_5*np.sin(theta_5 - theta_2) + dtheta_4*(dtheta_4*l_2*l_4*m_5*np.sin(theta_4 - theta_2) + dtheta_4*l_2*m_4*r_4*np.sin(theta_4 - theta_2)) + dtheta_3*mu_3 + dtheta_3*(dtheta_3*l_2*l_3*m_4*np.sin(theta_3 - theta_2) + dtheta_3*l_2*l_3*m_5*np.sin(theta_3 - theta_2) + dtheta_3*l_2*m_3*r_3*np.sin(theta_3 - theta_2)) - dtheta_2*(mu_2 + mu_3) + dtheta_1*mu_2 + dtheta_1*(-dtheta_1*l_1*l_2*m_3*np.sin(theta_2 - theta_1) - dtheta_1*l_1*l_2*m_4*np.sin(theta_2 - theta_1) - dtheta_1*l_1*l_2*m_5*np.sin(theta_2 - theta_1) - dtheta_1*l_1*m_2*r_2*np.sin(theta_2 - theta_1)) - ddx_c*(-l_2*m_3 - l_2*m_4 - l_2*m_5 - m_2*r_2)*np.cos(theta_2) - g*(-l_2*m_3 - l_2*m_4 - l_2*m_5 - m_2*r_2)*np.sin(theta_2) - l_1*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*(dtheta_5**2*l_1*m_5*r_5*np.sin(theta_5 - theta_1) + dtheta_4*(dtheta_4*l_1*l_4*m_5*np.sin(theta_4 - theta_1) + dtheta_4*l_1*m_4*r_4*np.sin(theta_4 - theta_1)) + dtheta_3*(dtheta_3*l_1*l_3*m_4*np.sin(theta_3 - theta_1) + dtheta_3*l_1*l_3*m_5*np.sin(theta_3 - theta_1) + dtheta_3*l_1*m_3*r_3*np.sin(theta_3 - theta_1)) + dtheta_2*mu_2 + dtheta_2*(dtheta_2*l_1*l_2*m_3*np.sin(theta_2 - theta_1) + dtheta_2*l_1*l_2*m_4*np.sin(theta_2 - theta_1) + dtheta_2*l_1*l_2*m_5*np.sin(theta_2 - theta_1) + dtheta_2*l_1*m_2*r_2*np.sin(theta_2 - theta_1)) - dtheta_1*(mu_1 + mu_2) - ddx_c*(-l_1*m_2 - l_1*m_3 - l_1*m_4 - l_1*m_5 - m_1*r_1)*np.cos(theta_1) - g*(-l_1*m_2 - l_1*m_3 - l_1*m_4 - l_1*m_5 - m_1*r_1)*np.sin(theta_1))*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2))/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2))/(I_3 - l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)**2*np.cos(theta_3 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_3**2*m_4 + l_3**2*m_5 + m_3*r_3**2 - (-l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_3 - theta_2))**2/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2)) - (-l_1**2*m_5*r_5*(l_4*m_5 + m_4*r_4)*np.cos(theta_5 - theta_1)*np.cos(theta_4 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_4*m_5*r_5*np.cos(theta_5 - theta_4) - (-l_1**2*(l_4*m_5 + m_4*r_4)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_4 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_4*m_5 + m_4*r_4)*np.cos(theta_4 - theta_2))*(-l_1**2*m_5*r_5*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_5 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*m_5*r_5*np.cos(theta_5 - theta_2))/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2) - (-l_1**2*(l_4*m_5 + m_4*r_4)*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_4 - theta_1)*np.cos(theta_3 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_3*(l_4*m_5 + m_4*r_4)*np.cos(theta_4 - theta_3) - (-l_1**2*(l_4*m_5 + m_4*r_4)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_4 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_4*m_5 + m_4*r_4)*np.cos(theta_4 - theta_2))*(-l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_3 - theta_2))/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2))*(-l_1**2*m_5*r_5*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_5 - theta_1)*np.cos(theta_3 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_3*m_5*r_5*np.cos(theta_5 - theta_3) - (-l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_3 - theta_2))*(-l_1**2*m_5*r_5*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_5 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*m_5*r_5*np.cos(theta_5 - theta_2))/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2))/(I_3 - l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)**2*np.cos(theta_3 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_3**2*m_4 + l_3**2*m_5 + m_3*r_3**2 - (-l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_3 - theta_2))**2/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2)))*(-dtheta_5*mu_5 - dtheta_4**2*l_4*m_5*r_5*np.sin(theta_5 - theta_4) + dtheta_4*mu_5 - dtheta_3**2*l_3*m_5*r_5*np.sin(theta_5 - theta_3) - dtheta_2**2*l_2*m_5*r_5*np.sin(theta_5 - theta_2) - dtheta_1**2*l_1*m_5*r_5*np.sin(theta_5 - theta_1) + ddx_c*m_5*r_5*np.cos(theta_5) + g*m_5*r_5*np.sin(theta_5) - l_1*m_5*r_5*(dtheta_5**2*l_1*m_5*r_5*np.sin(theta_5 - theta_1) + dtheta_4*(dtheta_4*l_1*l_4*m_5*np.sin(theta_4 - theta_1) + dtheta_4*l_1*m_4*r_4*np.sin(theta_4 - theta_1)) + dtheta_3*(dtheta_3*l_1*l_3*m_4*np.sin(theta_3 - theta_1) + dtheta_3*l_1*l_3*m_5*np.sin(theta_3 - theta_1) + dtheta_3*l_1*m_3*r_3*np.sin(theta_3 - theta_1)) + dtheta_2*mu_2 + dtheta_2*(dtheta_2*l_1*l_2*m_3*np.sin(theta_2 - theta_1) + dtheta_2*l_1*l_2*m_4*np.sin(theta_2 - theta_1) + dtheta_2*l_1*l_2*m_5*np.sin(theta_2 - theta_1) + dtheta_2*l_1*m_2*r_2*np.sin(theta_2 - theta_1)) - dtheta_1*(mu_1 + mu_2) - ddx_c*(-l_1*m_2 - l_1*m_3 - l_1*m_4 - l_1*m_5 - m_1*r_1)*np.cos(theta_1) - g*(-l_1*m_2 - l_1*m_3 - l_1*m_4 - l_1*m_5 - m_1*r_1)*np.sin(theta_1))*np.cos(theta_5 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) - (-l_1**2*m_5*r_5*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_5 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*m_5*r_5*np.cos(theta_5 - theta_2))*(dtheta_5**2*l_2*m_5*r_5*np.sin(theta_5 - theta_2) + dtheta_4*(dtheta_4*l_2*l_4*m_5*np.sin(theta_4 - theta_2) + dtheta_4*l_2*m_4*r_4*np.sin(theta_4 - theta_2)) + dtheta_3*mu_3 + dtheta_3*(dtheta_3*l_2*l_3*m_4*np.sin(theta_3 - theta_2) + dtheta_3*l_2*l_3*m_5*np.sin(theta_3 - theta_2) + dtheta_3*l_2*m_3*r_3*np.sin(theta_3 - theta_2)) - dtheta_2*(mu_2 + mu_3) + dtheta_1*mu_2 + dtheta_1*(-dtheta_1*l_1*l_2*m_3*np.sin(theta_2 - theta_1) - dtheta_1*l_1*l_2*m_4*np.sin(theta_2 - theta_1) - dtheta_1*l_1*l_2*m_5*np.sin(theta_2 - theta_1) - dtheta_1*l_1*m_2*r_2*np.sin(theta_2 - theta_1)) - ddx_c*(-l_2*m_3 - l_2*m_4 - l_2*m_5 - m_2*r_2)*np.cos(theta_2) - g*(-l_2*m_3 - l_2*m_4 - l_2*m_5 - m_2*r_2)*np.sin(theta_2) - l_1*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*(dtheta_5**2*l_1*m_5*r_5*np.sin(theta_5 - theta_1) + dtheta_4*(dtheta_4*l_1*l_4*m_5*np.sin(theta_4 - theta_1) + dtheta_4*l_1*m_4*r_4*np.sin(theta_4 - theta_1)) + dtheta_3*(dtheta_3*l_1*l_3*m_4*np.sin(theta_3 - theta_1) + dtheta_3*l_1*l_3*m_5*np.sin(theta_3 - theta_1) + dtheta_3*l_1*m_3*r_3*np.sin(theta_3 - theta_1)) + dtheta_2*mu_2 + dtheta_2*(dtheta_2*l_1*l_2*m_3*np.sin(theta_2 - theta_1) + dtheta_2*l_1*l_2*m_4*np.sin(theta_2 - theta_1) + dtheta_2*l_1*l_2*m_5*np.sin(theta_2 - theta_1) + dtheta_2*l_1*m_2*r_2*np.sin(theta_2 - theta_1)) - dtheta_1*(mu_1 + mu_2) - ddx_c*(-l_1*m_2 - l_1*m_3 - l_1*m_4 - l_1*m_5 - m_1*r_1)*np.cos(theta_1) - g*(-l_1*m_2 - l_1*m_3 - l_1*m_4 - l_1*m_5 - m_1*r_1)*np.sin(theta_1))*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2))/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2) - (-l_1**2*m_5*r_5*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_5 - theta_1)*np.cos(theta_3 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_3*m_5*r_5*np.cos(theta_5 - theta_3) - (-l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_3 - theta_2))*(-l_1**2*m_5*r_5*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_5 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*m_5*r_5*np.cos(theta_5 - theta_2))/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2))*(dtheta_5**2*l_3*m_5*r_5*np.sin(theta_5 - theta_3) + dtheta_4*mu_4 + dtheta_4*(dtheta_4*l_3*l_4*m_5*np.sin(theta_4 - theta_3) + dtheta_4*l_3*m_4*r_4*np.sin(theta_4 - theta_3)) - dtheta_3*(mu_3 + mu_4) + dtheta_2*mu_3 + dtheta_2*(-dtheta_2*l_2*l_3*m_4*np.sin(theta_3 - theta_2) - dtheta_2*l_2*l_3*m_5*np.sin(theta_3 - theta_2) - dtheta_2*l_2*m_3*r_3*np.sin(theta_3 - theta_2)) + dtheta_1*(-dtheta_1*l_1*l_3*m_4*np.sin(theta_3 - theta_1) - dtheta_1*l_1*l_3*m_5*np.sin(theta_3 - theta_1) - dtheta_1*l_1*m_3*r_3*np.sin(theta_3 - theta_1)) - ddx_c*(-l_3*m_4 - l_3*m_5 - m_3*r_3)*np.cos(theta_3) - g*(-l_3*m_4 - l_3*m_5 - m_3*r_3)*np.sin(theta_3) - l_1*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(dtheta_5**2*l_1*m_5*r_5*np.sin(theta_5 - theta_1) + dtheta_4*(dtheta_4*l_1*l_4*m_5*np.sin(theta_4 - theta_1) + dtheta_4*l_1*m_4*r_4*np.sin(theta_4 - theta_1)) + dtheta_3*(dtheta_3*l_1*l_3*m_4*np.sin(theta_3 - theta_1) + dtheta_3*l_1*l_3*m_5*np.sin(theta_3 - theta_1) + dtheta_3*l_1*m_3*r_3*np.sin(theta_3 - theta_1)) + dtheta_2*mu_2 + dtheta_2*(dtheta_2*l_1*l_2*m_3*np.sin(theta_2 - theta_1) + dtheta_2*l_1*l_2*m_4*np.sin(theta_2 - theta_1) + dtheta_2*l_1*l_2*m_5*np.sin(theta_2 - theta_1) + dtheta_2*l_1*m_2*r_2*np.sin(theta_2 - theta_1)) - dtheta_1*(mu_1 + mu_2) - ddx_c*(-l_1*m_2 - l_1*m_3 - l_1*m_4 - l_1*m_5 - m_1*r_1)*np.cos(theta_1) - g*(-l_1*m_2 - l_1*m_3 - l_1*m_4 - l_1*m_5 - m_1*r_1)*np.sin(theta_1))*np.cos(theta_3 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) - (-l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_3 - theta_2))*(dtheta_5**2*l_2*m_5*r_5*np.sin(theta_5 - theta_2) + dtheta_4*(dtheta_4*l_2*l_4*m_5*np.sin(theta_4 - theta_2) + dtheta_4*l_2*m_4*r_4*np.sin(theta_4 - theta_2)) + dtheta_3*mu_3 + dtheta_3*(dtheta_3*l_2*l_3*m_4*np.sin(theta_3 - theta_2) + dtheta_3*l_2*l_3*m_5*np.sin(theta_3 - theta_2) + dtheta_3*l_2*m_3*r_3*np.sin(theta_3 - theta_2)) - dtheta_2*(mu_2 + mu_3) + dtheta_1*mu_2 + dtheta_1*(-dtheta_1*l_1*l_2*m_3*np.sin(theta_2 - theta_1) - dtheta_1*l_1*l_2*m_4*np.sin(theta_2 - theta_1) - dtheta_1*l_1*l_2*m_5*np.sin(theta_2 - theta_1) - dtheta_1*l_1*m_2*r_2*np.sin(theta_2 - theta_1)) - ddx_c*(-l_2*m_3 - l_2*m_4 - l_2*m_5 - m_2*r_2)*np.cos(theta_2) - g*(-l_2*m_3 - l_2*m_4 - l_2*m_5 - m_2*r_2)*np.sin(theta_2) - l_1*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*(dtheta_5**2*l_1*m_5*r_5*np.sin(theta_5 - theta_1) + dtheta_4*(dtheta_4*l_1*l_4*m_5*np.sin(theta_4 - theta_1) + dtheta_4*l_1*m_4*r_4*np.sin(theta_4 - theta_1)) + dtheta_3*(dtheta_3*l_1*l_3*m_4*np.sin(theta_3 - theta_1) + dtheta_3*l_1*l_3*m_5*np.sin(theta_3 - theta_1) + dtheta_3*l_1*m_3*r_3*np.sin(theta_3 - theta_1)) + dtheta_2*mu_2 + dtheta_2*(dtheta_2*l_1*l_2*m_3*np.sin(theta_2 - theta_1) + dtheta_2*l_1*l_2*m_4*np.sin(theta_2 - theta_1) + dtheta_2*l_1*l_2*m_5*np.sin(theta_2 - theta_1) + dtheta_2*l_1*m_2*r_2*np.sin(theta_2 - theta_1)) - dtheta_1*(mu_1 + mu_2) - ddx_c*(-l_1*m_2 - l_1*m_3 - l_1*m_4 - l_1*m_5 - m_1*r_1)*np.cos(theta_1) - g*(-l_1*m_2 - l_1*m_3 - l_1*m_4 - l_1*m_5 - m_1*r_1)*np.sin(theta_1))*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2))/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2))/(I_3 - l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)**2*np.cos(theta_3 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_3**2*m_4 + l_3**2*m_5 + m_3*r_3**2 - (-l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_3 - theta_2))**2/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2)) - (-l_1**2*m_5*r_5*(l_4*m_5 + m_4*r_4)*np.cos(theta_5 - theta_1)*np.cos(theta_4 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_4*m_5*r_5*np.cos(theta_5 - theta_4) - (-l_1**2*(l_4*m_5 + m_4*r_4)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_4 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_4*m_5 + m_4*r_4)*np.cos(theta_4 - theta_2))*(-l_1**2*m_5*r_5*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_5 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*m_5*r_5*np.cos(theta_5 - theta_2))/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2) - (-l_1**2*(l_4*m_5 + m_4*r_4)*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_4 - theta_1)*np.cos(theta_3 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_3*(l_4*m_5 + m_4*r_4)*np.cos(theta_4 - theta_3) - (-l_1**2*(l_4*m_5 + m_4*r_4)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_4 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_4*m_5 + m_4*r_4)*np.cos(theta_4 - theta_2))*(-l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_3 - theta_2))/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2))*(-l_1**2*m_5*r_5*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_5 - theta_1)*np.cos(theta_3 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_3*m_5*r_5*np.cos(theta_5 - theta_3) - (-l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_3 - theta_2))*(-l_1**2*m_5*r_5*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_5 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*m_5*r_5*np.cos(theta_5 - theta_2))/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2))/(I_3 - l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)**2*np.cos(theta_3 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_3**2*m_4 + l_3**2*m_5 + m_3*r_3**2 - (-l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_3 - theta_2))**2/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2)))*(dtheta_5**2*l_4*m_5*r_5*np.sin(theta_5 - theta_4) + dtheta_5*mu_5 - dtheta_4*(mu_4 + mu_5) + dtheta_3*mu_4 + dtheta_3*(-dtheta_3*l_3*l_4*m_5*np.sin(theta_4 - theta_3) - dtheta_3*l_3*m_4*r_4*np.sin(theta_4 - theta_3)) + dtheta_2*(-dtheta_2*l_2*l_4*m_5*np.sin(theta_4 - theta_2) - dtheta_2*l_2*m_4*r_4*np.sin(theta_4 - theta_2)) + dtheta_1*(-dtheta_1*l_1*l_4*m_5*np.sin(theta_4 - theta_1) - dtheta_1*l_1*m_4*r_4*np.sin(theta_4 - theta_1)) - ddx_c*(-l_4*m_5 - m_4*r_4)*np.cos(theta_4) - g*(-l_4*m_5 - m_4*r_4)*np.sin(theta_4) - l_1*(l_4*m_5 + m_4*r_4)*(dtheta_5**2*l_1*m_5*r_5*np.sin(theta_5 - theta_1) + dtheta_4*(dtheta_4*l_1*l_4*m_5*np.sin(theta_4 - theta_1) + dtheta_4*l_1*m_4*r_4*np.sin(theta_4 - theta_1)) + dtheta_3*(dtheta_3*l_1*l_3*m_4*np.sin(theta_3 - theta_1) + dtheta_3*l_1*l_3*m_5*np.sin(theta_3 - theta_1) + dtheta_3*l_1*m_3*r_3*np.sin(theta_3 - theta_1)) + dtheta_2*mu_2 + dtheta_2*(dtheta_2*l_1*l_2*m_3*np.sin(theta_2 - theta_1) + dtheta_2*l_1*l_2*m_4*np.sin(theta_2 - theta_1) + dtheta_2*l_1*l_2*m_5*np.sin(theta_2 - theta_1) + dtheta_2*l_1*m_2*r_2*np.sin(theta_2 - theta_1)) - dtheta_1*(mu_1 + mu_2) - ddx_c*(-l_1*m_2 - l_1*m_3 - l_1*m_4 - l_1*m_5 - m_1*r_1)*np.cos(theta_1) - g*(-l_1*m_2 - l_1*m_3 - l_1*m_4 - l_1*m_5 - m_1*r_1)*np.sin(theta_1))*np.cos(theta_4 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) - (-l_1**2*(l_4*m_5 + m_4*r_4)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_4 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_4*m_5 + m_4*r_4)*np.cos(theta_4 - theta_2))*(dtheta_5**2*l_2*m_5*r_5*np.sin(theta_5 - theta_2) + dtheta_4*(dtheta_4*l_2*l_4*m_5*np.sin(theta_4 - theta_2) + dtheta_4*l_2*m_4*r_4*np.sin(theta_4 - theta_2)) + dtheta_3*mu_3 + dtheta_3*(dtheta_3*l_2*l_3*m_4*np.sin(theta_3 - theta_2) + dtheta_3*l_2*l_3*m_5*np.sin(theta_3 - theta_2) + dtheta_3*l_2*m_3*r_3*np.sin(theta_3 - theta_2)) - dtheta_2*(mu_2 + mu_3) + dtheta_1*mu_2 + dtheta_1*(-dtheta_1*l_1*l_2*m_3*np.sin(theta_2 - theta_1) - dtheta_1*l_1*l_2*m_4*np.sin(theta_2 - theta_1) - dtheta_1*l_1*l_2*m_5*np.sin(theta_2 - theta_1) - dtheta_1*l_1*m_2*r_2*np.sin(theta_2 - theta_1)) - ddx_c*(-l_2*m_3 - l_2*m_4 - l_2*m_5 - m_2*r_2)*np.cos(theta_2) - g*(-l_2*m_3 - l_2*m_4 - l_2*m_5 - m_2*r_2)*np.sin(theta_2) - l_1*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*(dtheta_5**2*l_1*m_5*r_5*np.sin(theta_5 - theta_1) + dtheta_4*(dtheta_4*l_1*l_4*m_5*np.sin(theta_4 - theta_1) + dtheta_4*l_1*m_4*r_4*np.sin(theta_4 - theta_1)) + dtheta_3*(dtheta_3*l_1*l_3*m_4*np.sin(theta_3 - theta_1) + dtheta_3*l_1*l_3*m_5*np.sin(theta_3 - theta_1) + dtheta_3*l_1*m_3*r_3*np.sin(theta_3 - theta_1)) + dtheta_2*mu_2 + dtheta_2*(dtheta_2*l_1*l_2*m_3*np.sin(theta_2 - theta_1) + dtheta_2*l_1*l_2*m_4*np.sin(theta_2 - theta_1) + dtheta_2*l_1*l_2*m_5*np.sin(theta_2 - theta_1) + dtheta_2*l_1*m_2*r_2*np.sin(theta_2 - theta_1)) - dtheta_1*(mu_1 + mu_2) - ddx_c*(-l_1*m_2 - l_1*m_3 - l_1*m_4 - l_1*m_5 - m_1*r_1)*np.cos(theta_1) - g*(-l_1*m_2 - l_1*m_3 - l_1*m_4 - l_1*m_5 - m_1*r_1)*np.sin(theta_1))*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2))/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2) - (-l_1**2*(l_4*m_5 + m_4*r_4)*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_4 - theta_1)*np.cos(theta_3 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_3*(l_4*m_5 + m_4*r_4)*np.cos(theta_4 - theta_3) - (-l_1**2*(l_4*m_5 + m_4*r_4)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_4 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_4*m_5 + m_4*r_4)*np.cos(theta_4 - theta_2))*(-l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_3 - theta_2))/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2))*(dtheta_5**2*l_3*m_5*r_5*np.sin(theta_5 - theta_3) + dtheta_4*mu_4 + dtheta_4*(dtheta_4*l_3*l_4*m_5*np.sin(theta_4 - theta_3) + dtheta_4*l_3*m_4*r_4*np.sin(theta_4 - theta_3)) - dtheta_3*(mu_3 + mu_4) + dtheta_2*mu_3 + dtheta_2*(-dtheta_2*l_2*l_3*m_4*np.sin(theta_3 - theta_2) - dtheta_2*l_2*l_3*m_5*np.sin(theta_3 - theta_2) - dtheta_2*l_2*m_3*r_3*np.sin(theta_3 - theta_2)) + dtheta_1*(-dtheta_1*l_1*l_3*m_4*np.sin(theta_3 - theta_1) - dtheta_1*l_1*l_3*m_5*np.sin(theta_3 - theta_1) - dtheta_1*l_1*m_3*r_3*np.sin(theta_3 - theta_1)) - ddx_c*(-l_3*m_4 - l_3*m_5 - m_3*r_3)*np.cos(theta_3) - g*(-l_3*m_4 - l_3*m_5 - m_3*r_3)*np.sin(theta_3) - l_1*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(dtheta_5**2*l_1*m_5*r_5*np.sin(theta_5 - theta_1) + dtheta_4*(dtheta_4*l_1*l_4*m_5*np.sin(theta_4 - theta_1) + dtheta_4*l_1*m_4*r_4*np.sin(theta_4 - theta_1)) + dtheta_3*(dtheta_3*l_1*l_3*m_4*np.sin(theta_3 - theta_1) + dtheta_3*l_1*l_3*m_5*np.sin(theta_3 - theta_1) + dtheta_3*l_1*m_3*r_3*np.sin(theta_3 - theta_1)) + dtheta_2*mu_2 + dtheta_2*(dtheta_2*l_1*l_2*m_3*np.sin(theta_2 - theta_1) + dtheta_2*l_1*l_2*m_4*np.sin(theta_2 - theta_1) + dtheta_2*l_1*l_2*m_5*np.sin(theta_2 - theta_1) + dtheta_2*l_1*m_2*r_2*np.sin(theta_2 - theta_1)) - dtheta_1*(mu_1 + mu_2) - ddx_c*(-l_1*m_2 - l_1*m_3 - l_1*m_4 - l_1*m_5 - m_1*r_1)*np.cos(theta_1) - g*(-l_1*m_2 - l_1*m_3 - l_1*m_4 - l_1*m_5 - m_1*r_1)*np.sin(theta_1))*np.cos(theta_3 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) - (-l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_3 - theta_2))*(dtheta_5**2*l_2*m_5*r_5*np.sin(theta_5 - theta_2) + dtheta_4*(dtheta_4*l_2*l_4*m_5*np.sin(theta_4 - theta_2) + dtheta_4*l_2*m_4*r_4*np.sin(theta_4 - theta_2)) + dtheta_3*mu_3 + dtheta_3*(dtheta_3*l_2*l_3*m_4*np.sin(theta_3 - theta_2) + dtheta_3*l_2*l_3*m_5*np.sin(theta_3 - theta_2) + dtheta_3*l_2*m_3*r_3*np.sin(theta_3 - theta_2)) - dtheta_2*(mu_2 + mu_3) + dtheta_1*mu_2 + dtheta_1*(-dtheta_1*l_1*l_2*m_3*np.sin(theta_2 - theta_1) - dtheta_1*l_1*l_2*m_4*np.sin(theta_2 - theta_1) - dtheta_1*l_1*l_2*m_5*np.sin(theta_2 - theta_1) - dtheta_1*l_1*m_2*r_2*np.sin(theta_2 - theta_1)) - ddx_c*(-l_2*m_3 - l_2*m_4 - l_2*m_5 - m_2*r_2)*np.cos(theta_2) - g*(-l_2*m_3 - l_2*m_4 - l_2*m_5 - m_2*r_2)*np.sin(theta_2) - l_1*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*(dtheta_5**2*l_1*m_5*r_5*np.sin(theta_5 - theta_1) + dtheta_4*(dtheta_4*l_1*l_4*m_5*np.sin(theta_4 - theta_1) + dtheta_4*l_1*m_4*r_4*np.sin(theta_4 - theta_1)) + dtheta_3*(dtheta_3*l_1*l_3*m_4*np.sin(theta_3 - theta_1) + dtheta_3*l_1*l_3*m_5*np.sin(theta_3 - theta_1) + dtheta_3*l_1*m_3*r_3*np.sin(theta_3 - theta_1)) + dtheta_2*mu_2 + dtheta_2*(dtheta_2*l_1*l_2*m_3*np.sin(theta_2 - theta_1) + dtheta_2*l_1*l_2*m_4*np.sin(theta_2 - theta_1) + dtheta_2*l_1*l_2*m_5*np.sin(theta_2 - theta_1) + dtheta_2*l_1*m_2*r_2*np.sin(theta_2 - theta_1)) - dtheta_1*(mu_1 + mu_2) - ddx_c*(-l_1*m_2 - l_1*m_3 - l_1*m_4 - l_1*m_5 - m_1*r_1)*np.cos(theta_1) - g*(-l_1*m_2 - l_1*m_3 - l_1*m_4 - l_1*m_5 - m_1*r_1)*np.sin(theta_1))*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2))/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2))/(I_3 - l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)**2*np.cos(theta_3 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_3**2*m_4 + l_3**2*m_5 + m_3*r_3**2 - (-l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_3 - theta_2))**2/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2)))/(I_4 - l_1**2*(l_4*m_5 + m_4*r_4)**2*np.cos(theta_4 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_4**2*m_5 + m_4*r_4**2 - (-l_1**2*(l_4*m_5 + m_4*r_4)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_4 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_4*m_5 + m_4*r_4)*np.cos(theta_4 - theta_2))**2/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2) - (-l_1**2*(l_4*m_5 + m_4*r_4)*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_4 - theta_1)*np.cos(theta_3 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_3*(l_4*m_5 + m_4*r_4)*np.cos(theta_4 - theta_3) - (-l_1**2*(l_4*m_5 + m_4*r_4)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_4 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_4*m_5 + m_4*r_4)*np.cos(theta_4 - theta_2))*(-l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_3 - theta_2))/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2))**2/(I_3 - l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)**2*np.cos(theta_3 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_3**2*m_4 + l_3**2*m_5 + m_3*r_3**2 - (-l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_3 - theta_2))**2/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2))))/(I_5 - l_1**2*m_5**2*r_5**2*np.cos(theta_5 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + m_5*r_5**2 - (-l_1**2*m_5*r_5*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_5 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*m_5*r_5*np.cos(theta_5 - theta_2))**2/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2) - (-l_1**2*m_5*r_5*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_5 - theta_1)*np.cos(theta_3 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_3*m_5*r_5*np.cos(theta_5 - theta_3) - (-l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_3 - theta_2))*(-l_1**2*m_5*r_5*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_5 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*m_5*r_5*np.cos(theta_5 - theta_2))/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2))**2/(I_3 - l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)**2*np.cos(theta_3 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_3**2*m_4 + l_3**2*m_5 + m_3*r_3**2 - (-l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_3 - theta_2))**2/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2)) - (-l_1**2*m_5*r_5*(l_4*m_5 + m_4*r_4)*np.cos(theta_5 - theta_1)*np.cos(theta_4 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_4*m_5*r_5*np.cos(theta_5 - theta_4) - (-l_1**2*(l_4*m_5 + m_4*r_4)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_4 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_4*m_5 + m_4*r_4)*np.cos(theta_4 - theta_2))*(-l_1**2*m_5*r_5*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_5 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*m_5*r_5*np.cos(theta_5 - theta_2))/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2) - (-l_1**2*(l_4*m_5 + m_4*r_4)*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_4 - theta_1)*np.cos(theta_3 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_3*(l_4*m_5 + m_4*r_4)*np.cos(theta_4 - theta_3) - (-l_1**2*(l_4*m_5 + m_4*r_4)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_4 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_4*m_5 + m_4*r_4)*np.cos(theta_4 - theta_2))*(-l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_3 - theta_2))/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2))*(-l_1**2*m_5*r_5*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_5 - theta_1)*np.cos(theta_3 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_3*m_5*r_5*np.cos(theta_5 - theta_3) - (-l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_3 - theta_2))*(-l_1**2*m_5*r_5*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_5 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*m_5*r_5*np.cos(theta_5 - theta_2))/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2))/(I_3 - l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)**2*np.cos(theta_3 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_3**2*m_4 + l_3**2*m_5 + m_3*r_3**2 - (-l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_3 - theta_2))**2/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2)))**2/(I_4 - l_1**2*(l_4*m_5 + m_4*r_4)**2*np.cos(theta_4 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_4**2*m_5 + m_4*r_4**2 - (-l_1**2*(l_4*m_5 + m_4*r_4)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_4 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_4*m_5 + m_4*r_4)*np.cos(theta_4 - theta_2))**2/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2) - (-l_1**2*(l_4*m_5 + m_4*r_4)*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_4 - theta_1)*np.cos(theta_3 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_3*(l_4*m_5 + m_4*r_4)*np.cos(theta_4 - theta_3) - (-l_1**2*(l_4*m_5 + m_4*r_4)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_4 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_4*m_5 + m_4*r_4)*np.cos(theta_4 - theta_2))*(-l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_3 - theta_2))/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2))**2/(I_3 - l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)**2*np.cos(theta_3 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_3**2*m_4 + l_3**2*m_5 + m_3*r_3**2 - (-l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_3 - theta_2))**2/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2)))))/(I_4 - l_1**2*(l_4*m_5 + m_4*r_4)**2*np.cos(theta_4 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_4**2*m_5 + m_4*r_4**2 - (-l_1**2*(l_4*m_5 + m_4*r_4)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_4 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_4*m_5 + m_4*r_4)*np.cos(theta_4 - theta_2))**2/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2) - (-l_1**2*(l_4*m_5 + m_4*r_4)*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_4 - theta_1)*np.cos(theta_3 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_3*(l_4*m_5 + m_4*r_4)*np.cos(theta_4 - theta_3) - (-l_1**2*(l_4*m_5 + m_4*r_4)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_4 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_4*m_5 + m_4*r_4)*np.cos(theta_4 - theta_2))*(-l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_3 - theta_2))/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2))**2/(I_3 - l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)**2*np.cos(theta_3 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_3**2*m_4 + l_3**2*m_5 + m_3*r_3**2 - (-l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_3 - theta_2))**2/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2))) - (-l_1**2*m_5*r_5*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_5 - theta_1)*np.cos(theta_3 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_3*m_5*r_5*np.cos(theta_5 - theta_3) - (-l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_3 - theta_2))*(-l_1**2*m_5*r_5*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_5 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*m_5*r_5*np.cos(theta_5 - theta_2))/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2))*(-dtheta_5*mu_5 - dtheta_4**2*l_4*m_5*r_5*np.sin(theta_5 - theta_4) + dtheta_4*mu_5 - dtheta_3**2*l_3*m_5*r_5*np.sin(theta_5 - theta_3) - dtheta_2**2*l_2*m_5*r_5*np.sin(theta_5 - theta_2) - dtheta_1**2*l_1*m_5*r_5*np.sin(theta_5 - theta_1) + ddx_c*m_5*r_5*np.cos(theta_5) + g*m_5*r_5*np.sin(theta_5) - l_1*m_5*r_5*(dtheta_5**2*l_1*m_5*r_5*np.sin(theta_5 - theta_1) + dtheta_4*(dtheta_4*l_1*l_4*m_5*np.sin(theta_4 - theta_1) + dtheta_4*l_1*m_4*r_4*np.sin(theta_4 - theta_1)) + dtheta_3*(dtheta_3*l_1*l_3*m_4*np.sin(theta_3 - theta_1) + dtheta_3*l_1*l_3*m_5*np.sin(theta_3 - theta_1) + dtheta_3*l_1*m_3*r_3*np.sin(theta_3 - theta_1)) + dtheta_2*mu_2 + dtheta_2*(dtheta_2*l_1*l_2*m_3*np.sin(theta_2 - theta_1) + dtheta_2*l_1*l_2*m_4*np.sin(theta_2 - theta_1) + dtheta_2*l_1*l_2*m_5*np.sin(theta_2 - theta_1) + dtheta_2*l_1*m_2*r_2*np.sin(theta_2 - theta_1)) - dtheta_1*(mu_1 + mu_2) - ddx_c*(-l_1*m_2 - l_1*m_3 - l_1*m_4 - l_1*m_5 - m_1*r_1)*np.cos(theta_1) - g*(-l_1*m_2 - l_1*m_3 - l_1*m_4 - l_1*m_5 - m_1*r_1)*np.sin(theta_1))*np.cos(theta_5 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) - (-l_1**2*m_5*r_5*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_5 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*m_5*r_5*np.cos(theta_5 - theta_2))*(dtheta_5**2*l_2*m_5*r_5*np.sin(theta_5 - theta_2) + dtheta_4*(dtheta_4*l_2*l_4*m_5*np.sin(theta_4 - theta_2) + dtheta_4*l_2*m_4*r_4*np.sin(theta_4 - theta_2)) + dtheta_3*mu_3 + dtheta_3*(dtheta_3*l_2*l_3*m_4*np.sin(theta_3 - theta_2) + dtheta_3*l_2*l_3*m_5*np.sin(theta_3 - theta_2) + dtheta_3*l_2*m_3*r_3*np.sin(theta_3 - theta_2)) - dtheta_2*(mu_2 + mu_3) + dtheta_1*mu_2 + dtheta_1*(-dtheta_1*l_1*l_2*m_3*np.sin(theta_2 - theta_1) - dtheta_1*l_1*l_2*m_4*np.sin(theta_2 - theta_1) - dtheta_1*l_1*l_2*m_5*np.sin(theta_2 - theta_1) - dtheta_1*l_1*m_2*r_2*np.sin(theta_2 - theta_1)) - ddx_c*(-l_2*m_3 - l_2*m_4 - l_2*m_5 - m_2*r_2)*np.cos(theta_2) - g*(-l_2*m_3 - l_2*m_4 - l_2*m_5 - m_2*r_2)*np.sin(theta_2) - l_1*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*(dtheta_5**2*l_1*m_5*r_5*np.sin(theta_5 - theta_1) + dtheta_4*(dtheta_4*l_1*l_4*m_5*np.sin(theta_4 - theta_1) + dtheta_4*l_1*m_4*r_4*np.sin(theta_4 - theta_1)) + dtheta_3*(dtheta_3*l_1*l_3*m_4*np.sin(theta_3 - theta_1) + dtheta_3*l_1*l_3*m_5*np.sin(theta_3 - theta_1) + dtheta_3*l_1*m_3*r_3*np.sin(theta_3 - theta_1)) + dtheta_2*mu_2 + dtheta_2*(dtheta_2*l_1*l_2*m_3*np.sin(theta_2 - theta_1) + dtheta_2*l_1*l_2*m_4*np.sin(theta_2 - theta_1) + dtheta_2*l_1*l_2*m_5*np.sin(theta_2 - theta_1) + dtheta_2*l_1*m_2*r_2*np.sin(theta_2 - theta_1)) - dtheta_1*(mu_1 + mu_2) - ddx_c*(-l_1*m_2 - l_1*m_3 - l_1*m_4 - l_1*m_5 - m_1*r_1)*np.cos(theta_1) - g*(-l_1*m_2 - l_1*m_3 - l_1*m_4 - l_1*m_5 - m_1*r_1)*np.sin(theta_1))*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2))/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2) - (-l_1**2*m_5*r_5*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_5 - theta_1)*np.cos(theta_3 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_3*m_5*r_5*np.cos(theta_5 - theta_3) - (-l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_3 - theta_2))*(-l_1**2*m_5*r_5*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_5 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*m_5*r_5*np.cos(theta_5 - theta_2))/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2))*(dtheta_5**2*l_3*m_5*r_5*np.sin(theta_5 - theta_3) + dtheta_4*mu_4 + dtheta_4*(dtheta_4*l_3*l_4*m_5*np.sin(theta_4 - theta_3) + dtheta_4*l_3*m_4*r_4*np.sin(theta_4 - theta_3)) - dtheta_3*(mu_3 + mu_4) + dtheta_2*mu_3 + dtheta_2*(-dtheta_2*l_2*l_3*m_4*np.sin(theta_3 - theta_2) - dtheta_2*l_2*l_3*m_5*np.sin(theta_3 - theta_2) - dtheta_2*l_2*m_3*r_3*np.sin(theta_3 - theta_2)) + dtheta_1*(-dtheta_1*l_1*l_3*m_4*np.sin(theta_3 - theta_1) - dtheta_1*l_1*l_3*m_5*np.sin(theta_3 - theta_1) - dtheta_1*l_1*m_3*r_3*np.sin(theta_3 - theta_1)) - ddx_c*(-l_3*m_4 - l_3*m_5 - m_3*r_3)*np.cos(theta_3) - g*(-l_3*m_4 - l_3*m_5 - m_3*r_3)*np.sin(theta_3) - l_1*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(dtheta_5**2*l_1*m_5*r_5*np.sin(theta_5 - theta_1) + dtheta_4*(dtheta_4*l_1*l_4*m_5*np.sin(theta_4 - theta_1) + dtheta_4*l_1*m_4*r_4*np.sin(theta_4 - theta_1)) + dtheta_3*(dtheta_3*l_1*l_3*m_4*np.sin(theta_3 - theta_1) + dtheta_3*l_1*l_3*m_5*np.sin(theta_3 - theta_1) + dtheta_3*l_1*m_3*r_3*np.sin(theta_3 - theta_1)) + dtheta_2*mu_2 + dtheta_2*(dtheta_2*l_1*l_2*m_3*np.sin(theta_2 - theta_1) + dtheta_2*l_1*l_2*m_4*np.sin(theta_2 - theta_1) + dtheta_2*l_1*l_2*m_5*np.sin(theta_2 - theta_1) + dtheta_2*l_1*m_2*r_2*np.sin(theta_2 - theta_1)) - dtheta_1*(mu_1 + mu_2) - ddx_c*(-l_1*m_2 - l_1*m_3 - l_1*m_4 - l_1*m_5 - m_1*r_1)*np.cos(theta_1) - g*(-l_1*m_2 - l_1*m_3 - l_1*m_4 - l_1*m_5 - m_1*r_1)*np.sin(theta_1))*np.cos(theta_3 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) - (-l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_3 - theta_2))*(dtheta_5**2*l_2*m_5*r_5*np.sin(theta_5 - theta_2) + dtheta_4*(dtheta_4*l_2*l_4*m_5*np.sin(theta_4 - theta_2) + dtheta_4*l_2*m_4*r_4*np.sin(theta_4 - theta_2)) + dtheta_3*mu_3 + dtheta_3*(dtheta_3*l_2*l_3*m_4*np.sin(theta_3 - theta_2) + dtheta_3*l_2*l_3*m_5*np.sin(theta_3 - theta_2) + dtheta_3*l_2*m_3*r_3*np.sin(theta_3 - theta_2)) - dtheta_2*(mu_2 + mu_3) + dtheta_1*mu_2 + dtheta_1*(-dtheta_1*l_1*l_2*m_3*np.sin(theta_2 - theta_1) - dtheta_1*l_1*l_2*m_4*np.sin(theta_2 - theta_1) - dtheta_1*l_1*l_2*m_5*np.sin(theta_2 - theta_1) - dtheta_1*l_1*m_2*r_2*np.sin(theta_2 - theta_1)) - ddx_c*(-l_2*m_3 - l_2*m_4 - l_2*m_5 - m_2*r_2)*np.cos(theta_2) - g*(-l_2*m_3 - l_2*m_4 - l_2*m_5 - m_2*r_2)*np.sin(theta_2) - l_1*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*(dtheta_5**2*l_1*m_5*r_5*np.sin(theta_5 - theta_1) + dtheta_4*(dtheta_4*l_1*l_4*m_5*np.sin(theta_4 - theta_1) + dtheta_4*l_1*m_4*r_4*np.sin(theta_4 - theta_1)) + dtheta_3*(dtheta_3*l_1*l_3*m_4*np.sin(theta_3 - theta_1) + dtheta_3*l_1*l_3*m_5*np.sin(theta_3 - theta_1) + dtheta_3*l_1*m_3*r_3*np.sin(theta_3 - theta_1)) + dtheta_2*mu_2 + dtheta_2*(dtheta_2*l_1*l_2*m_3*np.sin(theta_2 - theta_1) + dtheta_2*l_1*l_2*m_4*np.sin(theta_2 - theta_1) + dtheta_2*l_1*l_2*m_5*np.sin(theta_2 - theta_1) + dtheta_2*l_1*m_2*r_2*np.sin(theta_2 - theta_1)) - dtheta_1*(mu_1 + mu_2) - ddx_c*(-l_1*m_2 - l_1*m_3 - l_1*m_4 - l_1*m_5 - m_1*r_1)*np.cos(theta_1) - g*(-l_1*m_2 - l_1*m_3 - l_1*m_4 - l_1*m_5 - m_1*r_1)*np.sin(theta_1))*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2))/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2))/(I_3 - l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)**2*np.cos(theta_3 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_3**2*m_4 + l_3**2*m_5 + m_3*r_3**2 - (-l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_3 - theta_2))**2/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2)) - (-l_1**2*m_5*r_5*(l_4*m_5 + m_4*r_4)*np.cos(theta_5 - theta_1)*np.cos(theta_4 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_4*m_5*r_5*np.cos(theta_5 - theta_4) - (-l_1**2*(l_4*m_5 + m_4*r_4)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_4 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_4*m_5 + m_4*r_4)*np.cos(theta_4 - theta_2))*(-l_1**2*m_5*r_5*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_5 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*m_5*r_5*np.cos(theta_5 - theta_2))/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2) - (-l_1**2*(l_4*m_5 + m_4*r_4)*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_4 - theta_1)*np.cos(theta_3 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_3*(l_4*m_5 + m_4*r_4)*np.cos(theta_4 - theta_3) - (-l_1**2*(l_4*m_5 + m_4*r_4)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_4 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_4*m_5 + m_4*r_4)*np.cos(theta_4 - theta_2))*(-l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_3 - theta_2))/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2))*(-l_1**2*m_5*r_5*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_5 - theta_1)*np.cos(theta_3 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_3*m_5*r_5*np.cos(theta_5 - theta_3) - (-l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_3 - theta_2))*(-l_1**2*m_5*r_5*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_5 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*m_5*r_5*np.cos(theta_5 - theta_2))/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2))/(I_3 - l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)**2*np.cos(theta_3 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_3**2*m_4 + l_3**2*m_5 + m_3*r_3**2 - (-l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_3 - theta_2))**2/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2)))*(dtheta_5**2*l_4*m_5*r_5*np.sin(theta_5 - theta_4) + dtheta_5*mu_5 - dtheta_4*(mu_4 + mu_5) + dtheta_3*mu_4 + dtheta_3*(-dtheta_3*l_3*l_4*m_5*np.sin(theta_4 - theta_3) - dtheta_3*l_3*m_4*r_4*np.sin(theta_4 - theta_3)) + dtheta_2*(-dtheta_2*l_2*l_4*m_5*np.sin(theta_4 - theta_2) - dtheta_2*l_2*m_4*r_4*np.sin(theta_4 - theta_2)) + dtheta_1*(-dtheta_1*l_1*l_4*m_5*np.sin(theta_4 - theta_1) - dtheta_1*l_1*m_4*r_4*np.sin(theta_4 - theta_1)) - ddx_c*(-l_4*m_5 - m_4*r_4)*np.cos(theta_4) - g*(-l_4*m_5 - m_4*r_4)*np.sin(theta_4) - l_1*(l_4*m_5 + m_4*r_4)*(dtheta_5**2*l_1*m_5*r_5*np.sin(theta_5 - theta_1) + dtheta_4*(dtheta_4*l_1*l_4*m_5*np.sin(theta_4 - theta_1) + dtheta_4*l_1*m_4*r_4*np.sin(theta_4 - theta_1)) + dtheta_3*(dtheta_3*l_1*l_3*m_4*np.sin(theta_3 - theta_1) + dtheta_3*l_1*l_3*m_5*np.sin(theta_3 - theta_1) + dtheta_3*l_1*m_3*r_3*np.sin(theta_3 - theta_1)) + dtheta_2*mu_2 + dtheta_2*(dtheta_2*l_1*l_2*m_3*np.sin(theta_2 - theta_1) + dtheta_2*l_1*l_2*m_4*np.sin(theta_2 - theta_1) + dtheta_2*l_1*l_2*m_5*np.sin(theta_2 - theta_1) + dtheta_2*l_1*m_2*r_2*np.sin(theta_2 - theta_1)) - dtheta_1*(mu_1 + mu_2) - ddx_c*(-l_1*m_2 - l_1*m_3 - l_1*m_4 - l_1*m_5 - m_1*r_1)*np.cos(theta_1) - g*(-l_1*m_2 - l_1*m_3 - l_1*m_4 - l_1*m_5 - m_1*r_1)*np.sin(theta_1))*np.cos(theta_4 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) - (-l_1**2*(l_4*m_5 + m_4*r_4)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_4 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_4*m_5 + m_4*r_4)*np.cos(theta_4 - theta_2))*(dtheta_5**2*l_2*m_5*r_5*np.sin(theta_5 - theta_2) + dtheta_4*(dtheta_4*l_2*l_4*m_5*np.sin(theta_4 - theta_2) + dtheta_4*l_2*m_4*r_4*np.sin(theta_4 - theta_2)) + dtheta_3*mu_3 + dtheta_3*(dtheta_3*l_2*l_3*m_4*np.sin(theta_3 - theta_2) + dtheta_3*l_2*l_3*m_5*np.sin(theta_3 - theta_2) + dtheta_3*l_2*m_3*r_3*np.sin(theta_3 - theta_2)) - dtheta_2*(mu_2 + mu_3) + dtheta_1*mu_2 + dtheta_1*(-dtheta_1*l_1*l_2*m_3*np.sin(theta_2 - theta_1) - dtheta_1*l_1*l_2*m_4*np.sin(theta_2 - theta_1) - dtheta_1*l_1*l_2*m_5*np.sin(theta_2 - theta_1) - dtheta_1*l_1*m_2*r_2*np.sin(theta_2 - theta_1)) - ddx_c*(-l_2*m_3 - l_2*m_4 - l_2*m_5 - m_2*r_2)*np.cos(theta_2) - g*(-l_2*m_3 - l_2*m_4 - l_2*m_5 - m_2*r_2)*np.sin(theta_2) - l_1*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*(dtheta_5**2*l_1*m_5*r_5*np.sin(theta_5 - theta_1) + dtheta_4*(dtheta_4*l_1*l_4*m_5*np.sin(theta_4 - theta_1) + dtheta_4*l_1*m_4*r_4*np.sin(theta_4 - theta_1)) + dtheta_3*(dtheta_3*l_1*l_3*m_4*np.sin(theta_3 - theta_1) + dtheta_3*l_1*l_3*m_5*np.sin(theta_3 - theta_1) + dtheta_3*l_1*m_3*r_3*np.sin(theta_3 - theta_1)) + dtheta_2*mu_2 + dtheta_2*(dtheta_2*l_1*l_2*m_3*np.sin(theta_2 - theta_1) + dtheta_2*l_1*l_2*m_4*np.sin(theta_2 - theta_1) + dtheta_2*l_1*l_2*m_5*np.sin(theta_2 - theta_1) + dtheta_2*l_1*m_2*r_2*np.sin(theta_2 - theta_1)) - dtheta_1*(mu_1 + mu_2) - ddx_c*(-l_1*m_2 - l_1*m_3 - l_1*m_4 - l_1*m_5 - m_1*r_1)*np.cos(theta_1) - g*(-l_1*m_2 - l_1*m_3 - l_1*m_4 - l_1*m_5 - m_1*r_1)*np.sin(theta_1))*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2))/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2) - (-l_1**2*(l_4*m_5 + m_4*r_4)*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_4 - theta_1)*np.cos(theta_3 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_3*(l_4*m_5 + m_4*r_4)*np.cos(theta_4 - theta_3) - (-l_1**2*(l_4*m_5 + m_4*r_4)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_4 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_4*m_5 + m_4*r_4)*np.cos(theta_4 - theta_2))*(-l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_3 - theta_2))/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2))*(dtheta_5**2*l_3*m_5*r_5*np.sin(theta_5 - theta_3) + dtheta_4*mu_4 + dtheta_4*(dtheta_4*l_3*l_4*m_5*np.sin(theta_4 - theta_3) + dtheta_4*l_3*m_4*r_4*np.sin(theta_4 - theta_3)) - dtheta_3*(mu_3 + mu_4) + dtheta_2*mu_3 + dtheta_2*(-dtheta_2*l_2*l_3*m_4*np.sin(theta_3 - theta_2) - dtheta_2*l_2*l_3*m_5*np.sin(theta_3 - theta_2) - dtheta_2*l_2*m_3*r_3*np.sin(theta_3 - theta_2)) + dtheta_1*(-dtheta_1*l_1*l_3*m_4*np.sin(theta_3 - theta_1) - dtheta_1*l_1*l_3*m_5*np.sin(theta_3 - theta_1) - dtheta_1*l_1*m_3*r_3*np.sin(theta_3 - theta_1)) - ddx_c*(-l_3*m_4 - l_3*m_5 - m_3*r_3)*np.cos(theta_3) - g*(-l_3*m_4 - l_3*m_5 - m_3*r_3)*np.sin(theta_3) - l_1*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(dtheta_5**2*l_1*m_5*r_5*np.sin(theta_5 - theta_1) + dtheta_4*(dtheta_4*l_1*l_4*m_5*np.sin(theta_4 - theta_1) + dtheta_4*l_1*m_4*r_4*np.sin(theta_4 - theta_1)) + dtheta_3*(dtheta_3*l_1*l_3*m_4*np.sin(theta_3 - theta_1) + dtheta_3*l_1*l_3*m_5*np.sin(theta_3 - theta_1) + dtheta_3*l_1*m_3*r_3*np.sin(theta_3 - theta_1)) + dtheta_2*mu_2 + dtheta_2*(dtheta_2*l_1*l_2*m_3*np.sin(theta_2 - theta_1) + dtheta_2*l_1*l_2*m_4*np.sin(theta_2 - theta_1) + dtheta_2*l_1*l_2*m_5*np.sin(theta_2 - theta_1) + dtheta_2*l_1*m_2*r_2*np.sin(theta_2 - theta_1)) - dtheta_1*(mu_1 + mu_2) - ddx_c*(-l_1*m_2 - l_1*m_3 - l_1*m_4 - l_1*m_5 - m_1*r_1)*np.cos(theta_1) - g*(-l_1*m_2 - l_1*m_3 - l_1*m_4 - l_1*m_5 - m_1*r_1)*np.sin(theta_1))*np.cos(theta_3 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) - (-l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_3 - theta_2))*(dtheta_5**2*l_2*m_5*r_5*np.sin(theta_5 - theta_2) + dtheta_4*(dtheta_4*l_2*l_4*m_5*np.sin(theta_4 - theta_2) + dtheta_4*l_2*m_4*r_4*np.sin(theta_4 - theta_2)) + dtheta_3*mu_3 + dtheta_3*(dtheta_3*l_2*l_3*m_4*np.sin(theta_3 - theta_2) + dtheta_3*l_2*l_3*m_5*np.sin(theta_3 - theta_2) + dtheta_3*l_2*m_3*r_3*np.sin(theta_3 - theta_2)) - dtheta_2*(mu_2 + mu_3) + dtheta_1*mu_2 + dtheta_1*(-dtheta_1*l_1*l_2*m_3*np.sin(theta_2 - theta_1) - dtheta_1*l_1*l_2*m_4*np.sin(theta_2 - theta_1) - dtheta_1*l_1*l_2*m_5*np.sin(theta_2 - theta_1) - dtheta_1*l_1*m_2*r_2*np.sin(theta_2 - theta_1)) - ddx_c*(-l_2*m_3 - l_2*m_4 - l_2*m_5 - m_2*r_2)*np.cos(theta_2) - g*(-l_2*m_3 - l_2*m_4 - l_2*m_5 - m_2*r_2)*np.sin(theta_2) - l_1*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*(dtheta_5**2*l_1*m_5*r_5*np.sin(theta_5 - theta_1) + dtheta_4*(dtheta_4*l_1*l_4*m_5*np.sin(theta_4 - theta_1) + dtheta_4*l_1*m_4*r_4*np.sin(theta_4 - theta_1)) + dtheta_3*(dtheta_3*l_1*l_3*m_4*np.sin(theta_3 - theta_1) + dtheta_3*l_1*l_3*m_5*np.sin(theta_3 - theta_1) + dtheta_3*l_1*m_3*r_3*np.sin(theta_3 - theta_1)) + dtheta_2*mu_2 + dtheta_2*(dtheta_2*l_1*l_2*m_3*np.sin(theta_2 - theta_1) + dtheta_2*l_1*l_2*m_4*np.sin(theta_2 - theta_1) + dtheta_2*l_1*l_2*m_5*np.sin(theta_2 - theta_1) + dtheta_2*l_1*m_2*r_2*np.sin(theta_2 - theta_1)) - dtheta_1*(mu_1 + mu_2) - ddx_c*(-l_1*m_2 - l_1*m_3 - l_1*m_4 - l_1*m_5 - m_1*r_1)*np.cos(theta_1) - g*(-l_1*m_2 - l_1*m_3 - l_1*m_4 - l_1*m_5 - m_1*r_1)*np.sin(theta_1))*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2))/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2))/(I_3 - l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)**2*np.cos(theta_3 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_3**2*m_4 + l_3**2*m_5 + m_3*r_3**2 - (-l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_3 - theta_2))**2/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2)))/(I_4 - l_1**2*(l_4*m_5 + m_4*r_4)**2*np.cos(theta_4 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_4**2*m_5 + m_4*r_4**2 - (-l_1**2*(l_4*m_5 + m_4*r_4)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_4 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_4*m_5 + m_4*r_4)*np.cos(theta_4 - theta_2))**2/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2) - (-l_1**2*(l_4*m_5 + m_4*r_4)*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_4 - theta_1)*np.cos(theta_3 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_3*(l_4*m_5 + m_4*r_4)*np.cos(theta_4 - theta_3) - (-l_1**2*(l_4*m_5 + m_4*r_4)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_4 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_4*m_5 + m_4*r_4)*np.cos(theta_4 - theta_2))*(-l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_3 - theta_2))/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2))**2/(I_3 - l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)**2*np.cos(theta_3 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_3**2*m_4 + l_3**2*m_5 + m_3*r_3**2 - (-l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_3 - theta_2))**2/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2))))/(I_5 - l_1**2*m_5**2*r_5**2*np.cos(theta_5 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + m_5*r_5**2 - (-l_1**2*m_5*r_5*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_5 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*m_5*r_5*np.cos(theta_5 - theta_2))**2/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2) - (-l_1**2*m_5*r_5*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_5 - theta_1)*np.cos(theta_3 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_3*m_5*r_5*np.cos(theta_5 - theta_3) - (-l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_3 - theta_2))*(-l_1**2*m_5*r_5*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_5 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*m_5*r_5*np.cos(theta_5 - theta_2))/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2))**2/(I_3 - l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)**2*np.cos(theta_3 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_3**2*m_4 + l_3**2*m_5 + m_3*r_3**2 - (-l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_3 - theta_2))**2/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2)) - (-l_1**2*m_5*r_5*(l_4*m_5 + m_4*r_4)*np.cos(theta_5 - theta_1)*np.cos(theta_4 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_4*m_5*r_5*np.cos(theta_5 - theta_4) - (-l_1**2*(l_4*m_5 + m_4*r_4)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_4 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_4*m_5 + m_4*r_4)*np.cos(theta_4 - theta_2))*(-l_1**2*m_5*r_5*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_5 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*m_5*r_5*np.cos(theta_5 - theta_2))/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2) - (-l_1**2*(l_4*m_5 + m_4*r_4)*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_4 - theta_1)*np.cos(theta_3 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_3*(l_4*m_5 + m_4*r_4)*np.cos(theta_4 - theta_3) - (-l_1**2*(l_4*m_5 + m_4*r_4)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_4 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_4*m_5 + m_4*r_4)*np.cos(theta_4 - theta_2))*(-l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_3 - theta_2))/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2))*(-l_1**2*m_5*r_5*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_5 - theta_1)*np.cos(theta_3 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_3*m_5*r_5*np.cos(theta_5 - theta_3) - (-l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_3 - theta_2))*(-l_1**2*m_5*r_5*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_5 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*m_5*r_5*np.cos(theta_5 - theta_2))/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2))/(I_3 - l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)**2*np.cos(theta_3 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_3**2*m_4 + l_3**2*m_5 + m_3*r_3**2 - (-l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_3 - theta_2))**2/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2)))**2/(I_4 - l_1**2*(l_4*m_5 + m_4*r_4)**2*np.cos(theta_4 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_4**2*m_5 + m_4*r_4**2 - (-l_1**2*(l_4*m_5 + m_4*r_4)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_4 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_4*m_5 + m_4*r_4)*np.cos(theta_4 - theta_2))**2/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2) - (-l_1**2*(l_4*m_5 + m_4*r_4)*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_4 - theta_1)*np.cos(theta_3 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_3*(l_4*m_5 + m_4*r_4)*np.cos(theta_4 - theta_3) - (-l_1**2*(l_4*m_5 + m_4*r_4)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_4 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_4*m_5 + m_4*r_4)*np.cos(theta_4 - theta_2))*(-l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_3 - theta_2))/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2))**2/(I_3 - l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)**2*np.cos(theta_3 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_3**2*m_4 + l_3**2*m_5 + m_3*r_3**2 - (-l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_3 - theta_2))**2/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2)))))/(I_3 - l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)**2*np.cos(theta_3 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_3**2*m_4 + l_3**2*m_5 + m_3*r_3**2 - (-l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_3 - theta_2))**2/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2)) - (-l_1**2*m_5*r_5*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_5 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*m_5*r_5*np.cos(theta_5 - theta_2))*(-dtheta_5*mu_5 - dtheta_4**2*l_4*m_5*r_5*np.sin(theta_5 - theta_4) + dtheta_4*mu_5 - dtheta_3**2*l_3*m_5*r_5*np.sin(theta_5 - theta_3) - dtheta_2**2*l_2*m_5*r_5*np.sin(theta_5 - theta_2) - dtheta_1**2*l_1*m_5*r_5*np.sin(theta_5 - theta_1) + ddx_c*m_5*r_5*np.cos(theta_5) + g*m_5*r_5*np.sin(theta_5) - l_1*m_5*r_5*(dtheta_5**2*l_1*m_5*r_5*np.sin(theta_5 - theta_1) + dtheta_4*(dtheta_4*l_1*l_4*m_5*np.sin(theta_4 - theta_1) + dtheta_4*l_1*m_4*r_4*np.sin(theta_4 - theta_1)) + dtheta_3*(dtheta_3*l_1*l_3*m_4*np.sin(theta_3 - theta_1) + dtheta_3*l_1*l_3*m_5*np.sin(theta_3 - theta_1) + dtheta_3*l_1*m_3*r_3*np.sin(theta_3 - theta_1)) + dtheta_2*mu_2 + dtheta_2*(dtheta_2*l_1*l_2*m_3*np.sin(theta_2 - theta_1) + dtheta_2*l_1*l_2*m_4*np.sin(theta_2 - theta_1) + dtheta_2*l_1*l_2*m_5*np.sin(theta_2 - theta_1) + dtheta_2*l_1*m_2*r_2*np.sin(theta_2 - theta_1)) - dtheta_1*(mu_1 + mu_2) - ddx_c*(-l_1*m_2 - l_1*m_3 - l_1*m_4 - l_1*m_5 - m_1*r_1)*np.cos(theta_1) - g*(-l_1*m_2 - l_1*m_3 - l_1*m_4 - l_1*m_5 - m_1*r_1)*np.sin(theta_1))*np.cos(theta_5 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) - (-l_1**2*m_5*r_5*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_5 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*m_5*r_5*np.cos(theta_5 - theta_2))*(dtheta_5**2*l_2*m_5*r_5*np.sin(theta_5 - theta_2) + dtheta_4*(dtheta_4*l_2*l_4*m_5*np.sin(theta_4 - theta_2) + dtheta_4*l_2*m_4*r_4*np.sin(theta_4 - theta_2)) + dtheta_3*mu_3 + dtheta_3*(dtheta_3*l_2*l_3*m_4*np.sin(theta_3 - theta_2) + dtheta_3*l_2*l_3*m_5*np.sin(theta_3 - theta_2) + dtheta_3*l_2*m_3*r_3*np.sin(theta_3 - theta_2)) - dtheta_2*(mu_2 + mu_3) + dtheta_1*mu_2 + dtheta_1*(-dtheta_1*l_1*l_2*m_3*np.sin(theta_2 - theta_1) - dtheta_1*l_1*l_2*m_4*np.sin(theta_2 - theta_1) - dtheta_1*l_1*l_2*m_5*np.sin(theta_2 - theta_1) - dtheta_1*l_1*m_2*r_2*np.sin(theta_2 - theta_1)) - ddx_c*(-l_2*m_3 - l_2*m_4 - l_2*m_5 - m_2*r_2)*np.cos(theta_2) - g*(-l_2*m_3 - l_2*m_4 - l_2*m_5 - m_2*r_2)*np.sin(theta_2) - l_1*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*(dtheta_5**2*l_1*m_5*r_5*np.sin(theta_5 - theta_1) + dtheta_4*(dtheta_4*l_1*l_4*m_5*np.sin(theta_4 - theta_1) + dtheta_4*l_1*m_4*r_4*np.sin(theta_4 - theta_1)) + dtheta_3*(dtheta_3*l_1*l_3*m_4*np.sin(theta_3 - theta_1) + dtheta_3*l_1*l_3*m_5*np.sin(theta_3 - theta_1) + dtheta_3*l_1*m_3*r_3*np.sin(theta_3 - theta_1)) + dtheta_2*mu_2 + dtheta_2*(dtheta_2*l_1*l_2*m_3*np.sin(theta_2 - theta_1) + dtheta_2*l_1*l_2*m_4*np.sin(theta_2 - theta_1) + dtheta_2*l_1*l_2*m_5*np.sin(theta_2 - theta_1) + dtheta_2*l_1*m_2*r_2*np.sin(theta_2 - theta_1)) - dtheta_1*(mu_1 + mu_2) - ddx_c*(-l_1*m_2 - l_1*m_3 - l_1*m_4 - l_1*m_5 - m_1*r_1)*np.cos(theta_1) - g*(-l_1*m_2 - l_1*m_3 - l_1*m_4 - l_1*m_5 - m_1*r_1)*np.sin(theta_1))*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2))/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2) - (-l_1**2*m_5*r_5*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_5 - theta_1)*np.cos(theta_3 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_3*m_5*r_5*np.cos(theta_5 - theta_3) - (-l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_3 - theta_2))*(-l_1**2*m_5*r_5*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_5 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*m_5*r_5*np.cos(theta_5 - theta_2))/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2))*(dtheta_5**2*l_3*m_5*r_5*np.sin(theta_5 - theta_3) + dtheta_4*mu_4 + dtheta_4*(dtheta_4*l_3*l_4*m_5*np.sin(theta_4 - theta_3) + dtheta_4*l_3*m_4*r_4*np.sin(theta_4 - theta_3)) - dtheta_3*(mu_3 + mu_4) + dtheta_2*mu_3 + dtheta_2*(-dtheta_2*l_2*l_3*m_4*np.sin(theta_3 - theta_2) - dtheta_2*l_2*l_3*m_5*np.sin(theta_3 - theta_2) - dtheta_2*l_2*m_3*r_3*np.sin(theta_3 - theta_2)) + dtheta_1*(-dtheta_1*l_1*l_3*m_4*np.sin(theta_3 - theta_1) - dtheta_1*l_1*l_3*m_5*np.sin(theta_3 - theta_1) - dtheta_1*l_1*m_3*r_3*np.sin(theta_3 - theta_1)) - ddx_c*(-l_3*m_4 - l_3*m_5 - m_3*r_3)*np.cos(theta_3) - g*(-l_3*m_4 - l_3*m_5 - m_3*r_3)*np.sin(theta_3) - l_1*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(dtheta_5**2*l_1*m_5*r_5*np.sin(theta_5 - theta_1) + dtheta_4*(dtheta_4*l_1*l_4*m_5*np.sin(theta_4 - theta_1) + dtheta_4*l_1*m_4*r_4*np.sin(theta_4 - theta_1)) + dtheta_3*(dtheta_3*l_1*l_3*m_4*np.sin(theta_3 - theta_1) + dtheta_3*l_1*l_3*m_5*np.sin(theta_3 - theta_1) + dtheta_3*l_1*m_3*r_3*np.sin(theta_3 - theta_1)) + dtheta_2*mu_2 + dtheta_2*(dtheta_2*l_1*l_2*m_3*np.sin(theta_2 - theta_1) + dtheta_2*l_1*l_2*m_4*np.sin(theta_2 - theta_1) + dtheta_2*l_1*l_2*m_5*np.sin(theta_2 - theta_1) + dtheta_2*l_1*m_2*r_2*np.sin(theta_2 - theta_1)) - dtheta_1*(mu_1 + mu_2) - ddx_c*(-l_1*m_2 - l_1*m_3 - l_1*m_4 - l_1*m_5 - m_1*r_1)*np.cos(theta_1) - g*(-l_1*m_2 - l_1*m_3 - l_1*m_4 - l_1*m_5 - m_1*r_1)*np.sin(theta_1))*np.cos(theta_3 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) - (-l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_3 - theta_2))*(dtheta_5**2*l_2*m_5*r_5*np.sin(theta_5 - theta_2) + dtheta_4*(dtheta_4*l_2*l_4*m_5*np.sin(theta_4 - theta_2) + dtheta_4*l_2*m_4*r_4*np.sin(theta_4 - theta_2)) + dtheta_3*mu_3 + dtheta_3*(dtheta_3*l_2*l_3*m_4*np.sin(theta_3 - theta_2) + dtheta_3*l_2*l_3*m_5*np.sin(theta_3 - theta_2) + dtheta_3*l_2*m_3*r_3*np.sin(theta_3 - theta_2)) - dtheta_2*(mu_2 + mu_3) + dtheta_1*mu_2 + dtheta_1*(-dtheta_1*l_1*l_2*m_3*np.sin(theta_2 - theta_1) - dtheta_1*l_1*l_2*m_4*np.sin(theta_2 - theta_1) - dtheta_1*l_1*l_2*m_5*np.sin(theta_2 - theta_1) - dtheta_1*l_1*m_2*r_2*np.sin(theta_2 - theta_1)) - ddx_c*(-l_2*m_3 - l_2*m_4 - l_2*m_5 - m_2*r_2)*np.cos(theta_2) - g*(-l_2*m_3 - l_2*m_4 - l_2*m_5 - m_2*r_2)*np.sin(theta_2) - l_1*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*(dtheta_5**2*l_1*m_5*r_5*np.sin(theta_5 - theta_1) + dtheta_4*(dtheta_4*l_1*l_4*m_5*np.sin(theta_4 - theta_1) + dtheta_4*l_1*m_4*r_4*np.sin(theta_4 - theta_1)) + dtheta_3*(dtheta_3*l_1*l_3*m_4*np.sin(theta_3 - theta_1) + dtheta_3*l_1*l_3*m_5*np.sin(theta_3 - theta_1) + dtheta_3*l_1*m_3*r_3*np.sin(theta_3 - theta_1)) + dtheta_2*mu_2 + dtheta_2*(dtheta_2*l_1*l_2*m_3*np.sin(theta_2 - theta_1) + dtheta_2*l_1*l_2*m_4*np.sin(theta_2 - theta_1) + dtheta_2*l_1*l_2*m_5*np.sin(theta_2 - theta_1) + dtheta_2*l_1*m_2*r_2*np.sin(theta_2 - theta_1)) - dtheta_1*(mu_1 + mu_2) - ddx_c*(-l_1*m_2 - l_1*m_3 - l_1*m_4 - l_1*m_5 - m_1*r_1)*np.cos(theta_1) - g*(-l_1*m_2 - l_1*m_3 - l_1*m_4 - l_1*m_5 - m_1*r_1)*np.sin(theta_1))*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2))/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2))/(I_3 - l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)**2*np.cos(theta_3 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_3**2*m_4 + l_3**2*m_5 + m_3*r_3**2 - (-l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_3 - theta_2))**2/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2)) - (-l_1**2*m_5*r_5*(l_4*m_5 + m_4*r_4)*np.cos(theta_5 - theta_1)*np.cos(theta_4 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_4*m_5*r_5*np.cos(theta_5 - theta_4) - (-l_1**2*(l_4*m_5 + m_4*r_4)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_4 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_4*m_5 + m_4*r_4)*np.cos(theta_4 - theta_2))*(-l_1**2*m_5*r_5*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_5 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*m_5*r_5*np.cos(theta_5 - theta_2))/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2) - (-l_1**2*(l_4*m_5 + m_4*r_4)*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_4 - theta_1)*np.cos(theta_3 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_3*(l_4*m_5 + m_4*r_4)*np.cos(theta_4 - theta_3) - (-l_1**2*(l_4*m_5 + m_4*r_4)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_4 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_4*m_5 + m_4*r_4)*np.cos(theta_4 - theta_2))*(-l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_3 - theta_2))/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2))*(-l_1**2*m_5*r_5*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_5 - theta_1)*np.cos(theta_3 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_3*m_5*r_5*np.cos(theta_5 - theta_3) - (-l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_3 - theta_2))*(-l_1**2*m_5*r_5*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_5 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*m_5*r_5*np.cos(theta_5 - theta_2))/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2))/(I_3 - l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)**2*np.cos(theta_3 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_3**2*m_4 + l_3**2*m_5 + m_3*r_3**2 - (-l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_3 - theta_2))**2/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2)))*(dtheta_5**2*l_4*m_5*r_5*np.sin(theta_5 - theta_4) + dtheta_5*mu_5 - dtheta_4*(mu_4 + mu_5) + dtheta_3*mu_4 + dtheta_3*(-dtheta_3*l_3*l_4*m_5*np.sin(theta_4 - theta_3) - dtheta_3*l_3*m_4*r_4*np.sin(theta_4 - theta_3)) + dtheta_2*(-dtheta_2*l_2*l_4*m_5*np.sin(theta_4 - theta_2) - dtheta_2*l_2*m_4*r_4*np.sin(theta_4 - theta_2)) + dtheta_1*(-dtheta_1*l_1*l_4*m_5*np.sin(theta_4 - theta_1) - dtheta_1*l_1*m_4*r_4*np.sin(theta_4 - theta_1)) - ddx_c*(-l_4*m_5 - m_4*r_4)*np.cos(theta_4) - g*(-l_4*m_5 - m_4*r_4)*np.sin(theta_4) - l_1*(l_4*m_5 + m_4*r_4)*(dtheta_5**2*l_1*m_5*r_5*np.sin(theta_5 - theta_1) + dtheta_4*(dtheta_4*l_1*l_4*m_5*np.sin(theta_4 - theta_1) + dtheta_4*l_1*m_4*r_4*np.sin(theta_4 - theta_1)) + dtheta_3*(dtheta_3*l_1*l_3*m_4*np.sin(theta_3 - theta_1) + dtheta_3*l_1*l_3*m_5*np.sin(theta_3 - theta_1) + dtheta_3*l_1*m_3*r_3*np.sin(theta_3 - theta_1)) + dtheta_2*mu_2 + dtheta_2*(dtheta_2*l_1*l_2*m_3*np.sin(theta_2 - theta_1) + dtheta_2*l_1*l_2*m_4*np.sin(theta_2 - theta_1) + dtheta_2*l_1*l_2*m_5*np.sin(theta_2 - theta_1) + dtheta_2*l_1*m_2*r_2*np.sin(theta_2 - theta_1)) - dtheta_1*(mu_1 + mu_2) - ddx_c*(-l_1*m_2 - l_1*m_3 - l_1*m_4 - l_1*m_5 - m_1*r_1)*np.cos(theta_1) - g*(-l_1*m_2 - l_1*m_3 - l_1*m_4 - l_1*m_5 - m_1*r_1)*np.sin(theta_1))*np.cos(theta_4 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) - (-l_1**2*(l_4*m_5 + m_4*r_4)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_4 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_4*m_5 + m_4*r_4)*np.cos(theta_4 - theta_2))*(dtheta_5**2*l_2*m_5*r_5*np.sin(theta_5 - theta_2) + dtheta_4*(dtheta_4*l_2*l_4*m_5*np.sin(theta_4 - theta_2) + dtheta_4*l_2*m_4*r_4*np.sin(theta_4 - theta_2)) + dtheta_3*mu_3 + dtheta_3*(dtheta_3*l_2*l_3*m_4*np.sin(theta_3 - theta_2) + dtheta_3*l_2*l_3*m_5*np.sin(theta_3 - theta_2) + dtheta_3*l_2*m_3*r_3*np.sin(theta_3 - theta_2)) - dtheta_2*(mu_2 + mu_3) + dtheta_1*mu_2 + dtheta_1*(-dtheta_1*l_1*l_2*m_3*np.sin(theta_2 - theta_1) - dtheta_1*l_1*l_2*m_4*np.sin(theta_2 - theta_1) - dtheta_1*l_1*l_2*m_5*np.sin(theta_2 - theta_1) - dtheta_1*l_1*m_2*r_2*np.sin(theta_2 - theta_1)) - ddx_c*(-l_2*m_3 - l_2*m_4 - l_2*m_5 - m_2*r_2)*np.cos(theta_2) - g*(-l_2*m_3 - l_2*m_4 - l_2*m_5 - m_2*r_2)*np.sin(theta_2) - l_1*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*(dtheta_5**2*l_1*m_5*r_5*np.sin(theta_5 - theta_1) + dtheta_4*(dtheta_4*l_1*l_4*m_5*np.sin(theta_4 - theta_1) + dtheta_4*l_1*m_4*r_4*np.sin(theta_4 - theta_1)) + dtheta_3*(dtheta_3*l_1*l_3*m_4*np.sin(theta_3 - theta_1) + dtheta_3*l_1*l_3*m_5*np.sin(theta_3 - theta_1) + dtheta_3*l_1*m_3*r_3*np.sin(theta_3 - theta_1)) + dtheta_2*mu_2 + dtheta_2*(dtheta_2*l_1*l_2*m_3*np.sin(theta_2 - theta_1) + dtheta_2*l_1*l_2*m_4*np.sin(theta_2 - theta_1) + dtheta_2*l_1*l_2*m_5*np.sin(theta_2 - theta_1) + dtheta_2*l_1*m_2*r_2*np.sin(theta_2 - theta_1)) - dtheta_1*(mu_1 + mu_2) - ddx_c*(-l_1*m_2 - l_1*m_3 - l_1*m_4 - l_1*m_5 - m_1*r_1)*np.cos(theta_1) - g*(-l_1*m_2 - l_1*m_3 - l_1*m_4 - l_1*m_5 - m_1*r_1)*np.sin(theta_1))*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2))/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2) - (-l_1**2*(l_4*m_5 + m_4*r_4)*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_4 - theta_1)*np.cos(theta_3 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_3*(l_4*m_5 + m_4*r_4)*np.cos(theta_4 - theta_3) - (-l_1**2*(l_4*m_5 + m_4*r_4)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_4 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_4*m_5 + m_4*r_4)*np.cos(theta_4 - theta_2))*(-l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_3 - theta_2))/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2))*(dtheta_5**2*l_3*m_5*r_5*np.sin(theta_5 - theta_3) + dtheta_4*mu_4 + dtheta_4*(dtheta_4*l_3*l_4*m_5*np.sin(theta_4 - theta_3) + dtheta_4*l_3*m_4*r_4*np.sin(theta_4 - theta_3)) - dtheta_3*(mu_3 + mu_4) + dtheta_2*mu_3 + dtheta_2*(-dtheta_2*l_2*l_3*m_4*np.sin(theta_3 - theta_2) - dtheta_2*l_2*l_3*m_5*np.sin(theta_3 - theta_2) - dtheta_2*l_2*m_3*r_3*np.sin(theta_3 - theta_2)) + dtheta_1*(-dtheta_1*l_1*l_3*m_4*np.sin(theta_3 - theta_1) - dtheta_1*l_1*l_3*m_5*np.sin(theta_3 - theta_1) - dtheta_1*l_1*m_3*r_3*np.sin(theta_3 - theta_1)) - ddx_c*(-l_3*m_4 - l_3*m_5 - m_3*r_3)*np.cos(theta_3) - g*(-l_3*m_4 - l_3*m_5 - m_3*r_3)*np.sin(theta_3) - l_1*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(dtheta_5**2*l_1*m_5*r_5*np.sin(theta_5 - theta_1) + dtheta_4*(dtheta_4*l_1*l_4*m_5*np.sin(theta_4 - theta_1) + dtheta_4*l_1*m_4*r_4*np.sin(theta_4 - theta_1)) + dtheta_3*(dtheta_3*l_1*l_3*m_4*np.sin(theta_3 - theta_1) + dtheta_3*l_1*l_3*m_5*np.sin(theta_3 - theta_1) + dtheta_3*l_1*m_3*r_3*np.sin(theta_3 - theta_1)) + dtheta_2*mu_2 + dtheta_2*(dtheta_2*l_1*l_2*m_3*np.sin(theta_2 - theta_1) + dtheta_2*l_1*l_2*m_4*np.sin(theta_2 - theta_1) + dtheta_2*l_1*l_2*m_5*np.sin(theta_2 - theta_1) + dtheta_2*l_1*m_2*r_2*np.sin(theta_2 - theta_1)) - dtheta_1*(mu_1 + mu_2) - ddx_c*(-l_1*m_2 - l_1*m_3 - l_1*m_4 - l_1*m_5 - m_1*r_1)*np.cos(theta_1) - g*(-l_1*m_2 - l_1*m_3 - l_1*m_4 - l_1*m_5 - m_1*r_1)*np.sin(theta_1))*np.cos(theta_3 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) - (-l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_3 - theta_2))*(dtheta_5**2*l_2*m_5*r_5*np.sin(theta_5 - theta_2) + dtheta_4*(dtheta_4*l_2*l_4*m_5*np.sin(theta_4 - theta_2) + dtheta_4*l_2*m_4*r_4*np.sin(theta_4 - theta_2)) + dtheta_3*mu_3 + dtheta_3*(dtheta_3*l_2*l_3*m_4*np.sin(theta_3 - theta_2) + dtheta_3*l_2*l_3*m_5*np.sin(theta_3 - theta_2) + dtheta_3*l_2*m_3*r_3*np.sin(theta_3 - theta_2)) - dtheta_2*(mu_2 + mu_3) + dtheta_1*mu_2 + dtheta_1*(-dtheta_1*l_1*l_2*m_3*np.sin(theta_2 - theta_1) - dtheta_1*l_1*l_2*m_4*np.sin(theta_2 - theta_1) - dtheta_1*l_1*l_2*m_5*np.sin(theta_2 - theta_1) - dtheta_1*l_1*m_2*r_2*np.sin(theta_2 - theta_1)) - ddx_c*(-l_2*m_3 - l_2*m_4 - l_2*m_5 - m_2*r_2)*np.cos(theta_2) - g*(-l_2*m_3 - l_2*m_4 - l_2*m_5 - m_2*r_2)*np.sin(theta_2) - l_1*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*(dtheta_5**2*l_1*m_5*r_5*np.sin(theta_5 - theta_1) + dtheta_4*(dtheta_4*l_1*l_4*m_5*np.sin(theta_4 - theta_1) + dtheta_4*l_1*m_4*r_4*np.sin(theta_4 - theta_1)) + dtheta_3*(dtheta_3*l_1*l_3*m_4*np.sin(theta_3 - theta_1) + dtheta_3*l_1*l_3*m_5*np.sin(theta_3 - theta_1) + dtheta_3*l_1*m_3*r_3*np.sin(theta_3 - theta_1)) + dtheta_2*mu_2 + dtheta_2*(dtheta_2*l_1*l_2*m_3*np.sin(theta_2 - theta_1) + dtheta_2*l_1*l_2*m_4*np.sin(theta_2 - theta_1) + dtheta_2*l_1*l_2*m_5*np.sin(theta_2 - theta_1) + dtheta_2*l_1*m_2*r_2*np.sin(theta_2 - theta_1)) - dtheta_1*(mu_1 + mu_2) - ddx_c*(-l_1*m_2 - l_1*m_3 - l_1*m_4 - l_1*m_5 - m_1*r_1)*np.cos(theta_1) - g*(-l_1*m_2 - l_1*m_3 - l_1*m_4 - l_1*m_5 - m_1*r_1)*np.sin(theta_1))*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2))/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2))/(I_3 - l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)**2*np.cos(theta_3 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_3**2*m_4 + l_3**2*m_5 + m_3*r_3**2 - (-l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_3 - theta_2))**2/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2)))/(I_4 - l_1**2*(l_4*m_5 + m_4*r_4)**2*np.cos(theta_4 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_4**2*m_5 + m_4*r_4**2 - (-l_1**2*(l_4*m_5 + m_4*r_4)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_4 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_4*m_5 + m_4*r_4)*np.cos(theta_4 - theta_2))**2/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2) - (-l_1**2*(l_4*m_5 + m_4*r_4)*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_4 - theta_1)*np.cos(theta_3 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_3*(l_4*m_5 + m_4*r_4)*np.cos(theta_4 - theta_3) - (-l_1**2*(l_4*m_5 + m_4*r_4)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_4 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_4*m_5 + m_4*r_4)*np.cos(theta_4 - theta_2))*(-l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_3 - theta_2))/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2))**2/(I_3 - l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)**2*np.cos(theta_3 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_3**2*m_4 + l_3**2*m_5 + m_3*r_3**2 - (-l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_3 - theta_2))**2/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2))))/(I_5 - l_1**2*m_5**2*r_5**2*np.cos(theta_5 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + m_5*r_5**2 - (-l_1**2*m_5*r_5*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_5 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*m_5*r_5*np.cos(theta_5 - theta_2))**2/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2) - (-l_1**2*m_5*r_5*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_5 - theta_1)*np.cos(theta_3 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_3*m_5*r_5*np.cos(theta_5 - theta_3) - (-l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_3 - theta_2))*(-l_1**2*m_5*r_5*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_5 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*m_5*r_5*np.cos(theta_5 - theta_2))/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2))**2/(I_3 - l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)**2*np.cos(theta_3 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_3**2*m_4 + l_3**2*m_5 + m_3*r_3**2 - (-l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_3 - theta_2))**2/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2)) - (-l_1**2*m_5*r_5*(l_4*m_5 + m_4*r_4)*np.cos(theta_5 - theta_1)*np.cos(theta_4 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_4*m_5*r_5*np.cos(theta_5 - theta_4) - (-l_1**2*(l_4*m_5 + m_4*r_4)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_4 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_4*m_5 + m_4*r_4)*np.cos(theta_4 - theta_2))*(-l_1**2*m_5*r_5*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_5 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*m_5*r_5*np.cos(theta_5 - theta_2))/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2) - (-l_1**2*(l_4*m_5 + m_4*r_4)*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_4 - theta_1)*np.cos(theta_3 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_3*(l_4*m_5 + m_4*r_4)*np.cos(theta_4 - theta_3) - (-l_1**2*(l_4*m_5 + m_4*r_4)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_4 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_4*m_5 + m_4*r_4)*np.cos(theta_4 - theta_2))*(-l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_3 - theta_2))/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2))*(-l_1**2*m_5*r_5*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_5 - theta_1)*np.cos(theta_3 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_3*m_5*r_5*np.cos(theta_5 - theta_3) - (-l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_3 - theta_2))*(-l_1**2*m_5*r_5*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_5 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*m_5*r_5*np.cos(theta_5 - theta_2))/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2))/(I_3 - l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)**2*np.cos(theta_3 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_3**2*m_4 + l_3**2*m_5 + m_3*r_3**2 - (-l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_3 - theta_2))**2/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2)))**2/(I_4 - l_1**2*(l_4*m_5 + m_4*r_4)**2*np.cos(theta_4 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_4**2*m_5 + m_4*r_4**2 - (-l_1**2*(l_4*m_5 + m_4*r_4)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_4 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_4*m_5 + m_4*r_4)*np.cos(theta_4 - theta_2))**2/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2) - (-l_1**2*(l_4*m_5 + m_4*r_4)*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_4 - theta_1)*np.cos(theta_3 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_3*(l_4*m_5 + m_4*r_4)*np.cos(theta_4 - theta_3) - (-l_1**2*(l_4*m_5 + m_4*r_4)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_4 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_4*m_5 + m_4*r_4)*np.cos(theta_4 - theta_2))*(-l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_3 - theta_2))/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2))**2/(I_3 - l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)**2*np.cos(theta_3 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_3**2*m_4 + l_3**2*m_5 + m_3*r_3**2 - (-l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_3 - theta_2))**2/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2)))))/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2),


                      (dtheta_5**2*l_3*m_5*r_5*np.sin(theta_5 - theta_3) + dtheta_4*mu_4 + dtheta_4*(dtheta_4*l_3*l_4*m_5*np.sin(theta_4 - theta_3) + dtheta_4*l_3*m_4*r_4*np.sin(theta_4 - theta_3)) - dtheta_3*(mu_3 + mu_4) + dtheta_2*mu_3 + dtheta_2*(-dtheta_2*l_2*l_3*m_4*np.sin(theta_3 - theta_2) - dtheta_2*l_2*l_3*m_5*np.sin(theta_3 - theta_2) - dtheta_2*l_2*m_3*r_3*np.sin(theta_3 - theta_2)) + dtheta_1*(-dtheta_1*l_1*l_3*m_4*np.sin(theta_3 - theta_1) - dtheta_1*l_1*l_3*m_5*np.sin(theta_3 - theta_1) - dtheta_1*l_1*m_3*r_3*np.sin(theta_3 - theta_1)) - ddx_c*(-l_3*m_4 - l_3*m_5 - m_3*r_3)*np.cos(theta_3) - g*(-l_3*m_4 - l_3*m_5 - m_3*r_3)*np.sin(theta_3) - l_1*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(dtheta_5**2*l_1*m_5*r_5*np.sin(theta_5 - theta_1) + dtheta_4*(dtheta_4*l_1*l_4*m_5*np.sin(theta_4 - theta_1) + dtheta_4*l_1*m_4*r_4*np.sin(theta_4 - theta_1)) + dtheta_3*(dtheta_3*l_1*l_3*m_4*np.sin(theta_3 - theta_1) + dtheta_3*l_1*l_3*m_5*np.sin(theta_3 - theta_1) + dtheta_3*l_1*m_3*r_3*np.sin(theta_3 - theta_1)) + dtheta_2*mu_2 + dtheta_2*(dtheta_2*l_1*l_2*m_3*np.sin(theta_2 - theta_1) + dtheta_2*l_1*l_2*m_4*np.sin(theta_2 - theta_1) + dtheta_2*l_1*l_2*m_5*np.sin(theta_2 - theta_1) + dtheta_2*l_1*m_2*r_2*np.sin(theta_2 - theta_1)) - dtheta_1*(mu_1 + mu_2) - ddx_c*(-l_1*m_2 - l_1*m_3 - l_1*m_4 - l_1*m_5 - m_1*r_1)*np.cos(theta_1) - g*(-l_1*m_2 - l_1*m_3 - l_1*m_4 - l_1*m_5 - m_1*r_1)*np.sin(theta_1))*np.cos(theta_3 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) - (-l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_3 - theta_2))*(dtheta_5**2*l_2*m_5*r_5*np.sin(theta_5 - theta_2) + dtheta_4*(dtheta_4*l_2*l_4*m_5*np.sin(theta_4 - theta_2) + dtheta_4*l_2*m_4*r_4*np.sin(theta_4 - theta_2)) + dtheta_3*mu_3 + dtheta_3*(dtheta_3*l_2*l_3*m_4*np.sin(theta_3 - theta_2) + dtheta_3*l_2*l_3*m_5*np.sin(theta_3 - theta_2) + dtheta_3*l_2*m_3*r_3*np.sin(theta_3 - theta_2)) - dtheta_2*(mu_2 + mu_3) + dtheta_1*mu_2 + dtheta_1*(-dtheta_1*l_1*l_2*m_3*np.sin(theta_2 - theta_1) - dtheta_1*l_1*l_2*m_4*np.sin(theta_2 - theta_1) - dtheta_1*l_1*l_2*m_5*np.sin(theta_2 - theta_1) - dtheta_1*l_1*m_2*r_2*np.sin(theta_2 - theta_1)) - ddx_c*(-l_2*m_3 - l_2*m_4 - l_2*m_5 - m_2*r_2)*np.cos(theta_2) - g*(-l_2*m_3 - l_2*m_4 - l_2*m_5 - m_2*r_2)*np.sin(theta_2) - l_1*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*(dtheta_5**2*l_1*m_5*r_5*np.sin(theta_5 - theta_1) + dtheta_4*(dtheta_4*l_1*l_4*m_5*np.sin(theta_4 - theta_1) + dtheta_4*l_1*m_4*r_4*np.sin(theta_4 - theta_1)) + dtheta_3*(dtheta_3*l_1*l_3*m_4*np.sin(theta_3 - theta_1) + dtheta_3*l_1*l_3*m_5*np.sin(theta_3 - theta_1) + dtheta_3*l_1*m_3*r_3*np.sin(theta_3 - theta_1)) + dtheta_2*mu_2 + dtheta_2*(dtheta_2*l_1*l_2*m_3*np.sin(theta_2 - theta_1) + dtheta_2*l_1*l_2*m_4*np.sin(theta_2 - theta_1) + dtheta_2*l_1*l_2*m_5*np.sin(theta_2 - theta_1) + dtheta_2*l_1*m_2*r_2*np.sin(theta_2 - theta_1)) - dtheta_1*(mu_1 + mu_2) - ddx_c*(-l_1*m_2 - l_1*m_3 - l_1*m_4 - l_1*m_5 - m_1*r_1)*np.cos(theta_1) - g*(-l_1*m_2 - l_1*m_3 - l_1*m_4 - l_1*m_5 - m_1*r_1)*np.sin(theta_1))*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2))/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2) - (-l_1**2*(l_4*m_5 + m_4*r_4)*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_4 - theta_1)*np.cos(theta_3 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_3*(l_4*m_5 + m_4*r_4)*np.cos(theta_4 - theta_3) - (-l_1**2*(l_4*m_5 + m_4*r_4)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_4 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_4*m_5 + m_4*r_4)*np.cos(theta_4 - theta_2))*(-l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_3 - theta_2))/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2))*(dtheta_5**2*l_4*m_5*r_5*np.sin(theta_5 - theta_4) + dtheta_5*mu_5 - dtheta_4*(mu_4 + mu_5) + dtheta_3*mu_4 + dtheta_3*(-dtheta_3*l_3*l_4*m_5*np.sin(theta_4 - theta_3) - dtheta_3*l_3*m_4*r_4*np.sin(theta_4 - theta_3)) + dtheta_2*(-dtheta_2*l_2*l_4*m_5*np.sin(theta_4 - theta_2) - dtheta_2*l_2*m_4*r_4*np.sin(theta_4 - theta_2)) + dtheta_1*(-dtheta_1*l_1*l_4*m_5*np.sin(theta_4 - theta_1) - dtheta_1*l_1*m_4*r_4*np.sin(theta_4 - theta_1)) - ddx_c*(-l_4*m_5 - m_4*r_4)*np.cos(theta_4) - g*(-l_4*m_5 - m_4*r_4)*np.sin(theta_4) - l_1*(l_4*m_5 + m_4*r_4)*(dtheta_5**2*l_1*m_5*r_5*np.sin(theta_5 - theta_1) + dtheta_4*(dtheta_4*l_1*l_4*m_5*np.sin(theta_4 - theta_1) + dtheta_4*l_1*m_4*r_4*np.sin(theta_4 - theta_1)) + dtheta_3*(dtheta_3*l_1*l_3*m_4*np.sin(theta_3 - theta_1) + dtheta_3*l_1*l_3*m_5*np.sin(theta_3 - theta_1) + dtheta_3*l_1*m_3*r_3*np.sin(theta_3 - theta_1)) + dtheta_2*mu_2 + dtheta_2*(dtheta_2*l_1*l_2*m_3*np.sin(theta_2 - theta_1) + dtheta_2*l_1*l_2*m_4*np.sin(theta_2 - theta_1) + dtheta_2*l_1*l_2*m_5*np.sin(theta_2 - theta_1) + dtheta_2*l_1*m_2*r_2*np.sin(theta_2 - theta_1)) - dtheta_1*(mu_1 + mu_2) - ddx_c*(-l_1*m_2 - l_1*m_3 - l_1*m_4 - l_1*m_5 - m_1*r_1)*np.cos(theta_1) - g*(-l_1*m_2 - l_1*m_3 - l_1*m_4 - l_1*m_5 - m_1*r_1)*np.sin(theta_1))*np.cos(theta_4 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) - (-l_1**2*(l_4*m_5 + m_4*r_4)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_4 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_4*m_5 + m_4*r_4)*np.cos(theta_4 - theta_2))*(dtheta_5**2*l_2*m_5*r_5*np.sin(theta_5 - theta_2) + dtheta_4*(dtheta_4*l_2*l_4*m_5*np.sin(theta_4 - theta_2) + dtheta_4*l_2*m_4*r_4*np.sin(theta_4 - theta_2)) + dtheta_3*mu_3 + dtheta_3*(dtheta_3*l_2*l_3*m_4*np.sin(theta_3 - theta_2) + dtheta_3*l_2*l_3*m_5*np.sin(theta_3 - theta_2) + dtheta_3*l_2*m_3*r_3*np.sin(theta_3 - theta_2)) - dtheta_2*(mu_2 + mu_3) + dtheta_1*mu_2 + dtheta_1*(-dtheta_1*l_1*l_2*m_3*np.sin(theta_2 - theta_1) - dtheta_1*l_1*l_2*m_4*np.sin(theta_2 - theta_1) - dtheta_1*l_1*l_2*m_5*np.sin(theta_2 - theta_1) - dtheta_1*l_1*m_2*r_2*np.sin(theta_2 - theta_1)) - ddx_c*(-l_2*m_3 - l_2*m_4 - l_2*m_5 - m_2*r_2)*np.cos(theta_2) - g*(-l_2*m_3 - l_2*m_4 - l_2*m_5 - m_2*r_2)*np.sin(theta_2) - l_1*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*(dtheta_5**2*l_1*m_5*r_5*np.sin(theta_5 - theta_1) + dtheta_4*(dtheta_4*l_1*l_4*m_5*np.sin(theta_4 - theta_1) + dtheta_4*l_1*m_4*r_4*np.sin(theta_4 - theta_1)) + dtheta_3*(dtheta_3*l_1*l_3*m_4*np.sin(theta_3 - theta_1) + dtheta_3*l_1*l_3*m_5*np.sin(theta_3 - theta_1) + dtheta_3*l_1*m_3*r_3*np.sin(theta_3 - theta_1)) + dtheta_2*mu_2 + dtheta_2*(dtheta_2*l_1*l_2*m_3*np.sin(theta_2 - theta_1) + dtheta_2*l_1*l_2*m_4*np.sin(theta_2 - theta_1) + dtheta_2*l_1*l_2*m_5*np.sin(theta_2 - theta_1) + dtheta_2*l_1*m_2*r_2*np.sin(theta_2 - theta_1)) - dtheta_1*(mu_1 + mu_2) - ddx_c*(-l_1*m_2 - l_1*m_3 - l_1*m_4 - l_1*m_5 - m_1*r_1)*np.cos(theta_1) - g*(-l_1*m_2 - l_1*m_3 - l_1*m_4 - l_1*m_5 - m_1*r_1)*np.sin(theta_1))*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2))/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2) - (-l_1**2*(l_4*m_5 + m_4*r_4)*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_4 - theta_1)*np.cos(theta_3 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_3*(l_4*m_5 + m_4*r_4)*np.cos(theta_4 - theta_3) - (-l_1**2*(l_4*m_5 + m_4*r_4)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_4 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_4*m_5 + m_4*r_4)*np.cos(theta_4 - theta_2))*(-l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_3 - theta_2))/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2))*(dtheta_5**2*l_3*m_5*r_5*np.sin(theta_5 - theta_3) + dtheta_4*mu_4 + dtheta_4*(dtheta_4*l_3*l_4*m_5*np.sin(theta_4 - theta_3) + dtheta_4*l_3*m_4*r_4*np.sin(theta_4 - theta_3)) - dtheta_3*(mu_3 + mu_4) + dtheta_2*mu_3 + dtheta_2*(-dtheta_2*l_2*l_3*m_4*np.sin(theta_3 - theta_2) - dtheta_2*l_2*l_3*m_5*np.sin(theta_3 - theta_2) - dtheta_2*l_2*m_3*r_3*np.sin(theta_3 - theta_2)) + dtheta_1*(-dtheta_1*l_1*l_3*m_4*np.sin(theta_3 - theta_1) - dtheta_1*l_1*l_3*m_5*np.sin(theta_3 - theta_1) - dtheta_1*l_1*m_3*r_3*np.sin(theta_3 - theta_1)) - ddx_c*(-l_3*m_4 - l_3*m_5 - m_3*r_3)*np.cos(theta_3) - g*(-l_3*m_4 - l_3*m_5 - m_3*r_3)*np.sin(theta_3) - l_1*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(dtheta_5**2*l_1*m_5*r_5*np.sin(theta_5 - theta_1) + dtheta_4*(dtheta_4*l_1*l_4*m_5*np.sin(theta_4 - theta_1) + dtheta_4*l_1*m_4*r_4*np.sin(theta_4 - theta_1)) + dtheta_3*(dtheta_3*l_1*l_3*m_4*np.sin(theta_3 - theta_1) + dtheta_3*l_1*l_3*m_5*np.sin(theta_3 - theta_1) + dtheta_3*l_1*m_3*r_3*np.sin(theta_3 - theta_1)) + dtheta_2*mu_2 + dtheta_2*(dtheta_2*l_1*l_2*m_3*np.sin(theta_2 - theta_1) + dtheta_2*l_1*l_2*m_4*np.sin(theta_2 - theta_1) + dtheta_2*l_1*l_2*m_5*np.sin(theta_2 - theta_1) + dtheta_2*l_1*m_2*r_2*np.sin(theta_2 - theta_1)) - dtheta_1*(mu_1 + mu_2) - ddx_c*(-l_1*m_2 - l_1*m_3 - l_1*m_4 - l_1*m_5 - m_1*r_1)*np.cos(theta_1) - g*(-l_1*m_2 - l_1*m_3 - l_1*m_4 - l_1*m_5 - m_1*r_1)*np.sin(theta_1))*np.cos(theta_3 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) - (-l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_3 - theta_2))*(dtheta_5**2*l_2*m_5*r_5*np.sin(theta_5 - theta_2) + dtheta_4*(dtheta_4*l_2*l_4*m_5*np.sin(theta_4 - theta_2) + dtheta_4*l_2*m_4*r_4*np.sin(theta_4 - theta_2)) + dtheta_3*mu_3 + dtheta_3*(dtheta_3*l_2*l_3*m_4*np.sin(theta_3 - theta_2) + dtheta_3*l_2*l_3*m_5*np.sin(theta_3 - theta_2) + dtheta_3*l_2*m_3*r_3*np.sin(theta_3 - theta_2)) - dtheta_2*(mu_2 + mu_3) + dtheta_1*mu_2 + dtheta_1*(-dtheta_1*l_1*l_2*m_3*np.sin(theta_2 - theta_1) - dtheta_1*l_1*l_2*m_4*np.sin(theta_2 - theta_1) - dtheta_1*l_1*l_2*m_5*np.sin(theta_2 - theta_1) - dtheta_1*l_1*m_2*r_2*np.sin(theta_2 - theta_1)) - ddx_c*(-l_2*m_3 - l_2*m_4 - l_2*m_5 - m_2*r_2)*np.cos(theta_2) - g*(-l_2*m_3 - l_2*m_4 - l_2*m_5 - m_2*r_2)*np.sin(theta_2) - l_1*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*(dtheta_5**2*l_1*m_5*r_5*np.sin(theta_5 - theta_1) + dtheta_4*(dtheta_4*l_1*l_4*m_5*np.sin(theta_4 - theta_1) + dtheta_4*l_1*m_4*r_4*np.sin(theta_4 - theta_1)) + dtheta_3*(dtheta_3*l_1*l_3*m_4*np.sin(theta_3 - theta_1) + dtheta_3*l_1*l_3*m_5*np.sin(theta_3 - theta_1) + dtheta_3*l_1*m_3*r_3*np.sin(theta_3 - theta_1)) + dtheta_2*mu_2 + dtheta_2*(dtheta_2*l_1*l_2*m_3*np.sin(theta_2 - theta_1) + dtheta_2*l_1*l_2*m_4*np.sin(theta_2 - theta_1) + dtheta_2*l_1*l_2*m_5*np.sin(theta_2 - theta_1) + dtheta_2*l_1*m_2*r_2*np.sin(theta_2 - theta_1)) - dtheta_1*(mu_1 + mu_2) - ddx_c*(-l_1*m_2 - l_1*m_3 - l_1*m_4 - l_1*m_5 - m_1*r_1)*np.cos(theta_1) - g*(-l_1*m_2 - l_1*m_3 - l_1*m_4 - l_1*m_5 - m_1*r_1)*np.sin(theta_1))*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2))/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2))/(I_3 - l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)**2*np.cos(theta_3 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_3**2*m_4 + l_3**2*m_5 + m_3*r_3**2 - (-l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_3 - theta_2))**2/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2)) - (-l_1**2*m_5*r_5*(l_4*m_5 + m_4*r_4)*np.cos(theta_5 - theta_1)*np.cos(theta_4 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_4*m_5*r_5*np.cos(theta_5 - theta_4) - (-l_1**2*(l_4*m_5 + m_4*r_4)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_4 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_4*m_5 + m_4*r_4)*np.cos(theta_4 - theta_2))*(-l_1**2*m_5*r_5*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_5 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*m_5*r_5*np.cos(theta_5 - theta_2))/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2) - (-l_1**2*(l_4*m_5 + m_4*r_4)*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_4 - theta_1)*np.cos(theta_3 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_3*(l_4*m_5 + m_4*r_4)*np.cos(theta_4 - theta_3) - (-l_1**2*(l_4*m_5 + m_4*r_4)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_4 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_4*m_5 + m_4*r_4)*np.cos(theta_4 - theta_2))*(-l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_3 - theta_2))/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2))*(-l_1**2*m_5*r_5*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_5 - theta_1)*np.cos(theta_3 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_3*m_5*r_5*np.cos(theta_5 - theta_3) - (-l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_3 - theta_2))*(-l_1**2*m_5*r_5*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_5 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*m_5*r_5*np.cos(theta_5 - theta_2))/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2))/(I_3 - l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)**2*np.cos(theta_3 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_3**2*m_4 + l_3**2*m_5 + m_3*r_3**2 - (-l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_3 - theta_2))**2/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2)))*(-dtheta_5*mu_5 - dtheta_4**2*l_4*m_5*r_5*np.sin(theta_5 - theta_4) + dtheta_4*mu_5 - dtheta_3**2*l_3*m_5*r_5*np.sin(theta_5 - theta_3) - dtheta_2**2*l_2*m_5*r_5*np.sin(theta_5 - theta_2) - dtheta_1**2*l_1*m_5*r_5*np.sin(theta_5 - theta_1) + ddx_c*m_5*r_5*np.cos(theta_5) + g*m_5*r_5*np.sin(theta_5) - l_1*m_5*r_5*(dtheta_5**2*l_1*m_5*r_5*np.sin(theta_5 - theta_1) + dtheta_4*(dtheta_4*l_1*l_4*m_5*np.sin(theta_4 - theta_1) + dtheta_4*l_1*m_4*r_4*np.sin(theta_4 - theta_1)) + dtheta_3*(dtheta_3*l_1*l_3*m_4*np.sin(theta_3 - theta_1) + dtheta_3*l_1*l_3*m_5*np.sin(theta_3 - theta_1) + dtheta_3*l_1*m_3*r_3*np.sin(theta_3 - theta_1)) + dtheta_2*mu_2 + dtheta_2*(dtheta_2*l_1*l_2*m_3*np.sin(theta_2 - theta_1) + dtheta_2*l_1*l_2*m_4*np.sin(theta_2 - theta_1) + dtheta_2*l_1*l_2*m_5*np.sin(theta_2 - theta_1) + dtheta_2*l_1*m_2*r_2*np.sin(theta_2 - theta_1)) - dtheta_1*(mu_1 + mu_2) - ddx_c*(-l_1*m_2 - l_1*m_3 - l_1*m_4 - l_1*m_5 - m_1*r_1)*np.cos(theta_1) - g*(-l_1*m_2 - l_1*m_3 - l_1*m_4 - l_1*m_5 - m_1*r_1)*np.sin(theta_1))*np.cos(theta_5 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) - (-l_1**2*m_5*r_5*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_5 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*m_5*r_5*np.cos(theta_5 - theta_2))*(dtheta_5**2*l_2*m_5*r_5*np.sin(theta_5 - theta_2) + dtheta_4*(dtheta_4*l_2*l_4*m_5*np.sin(theta_4 - theta_2) + dtheta_4*l_2*m_4*r_4*np.sin(theta_4 - theta_2)) + dtheta_3*mu_3 + dtheta_3*(dtheta_3*l_2*l_3*m_4*np.sin(theta_3 - theta_2) + dtheta_3*l_2*l_3*m_5*np.sin(theta_3 - theta_2) + dtheta_3*l_2*m_3*r_3*np.sin(theta_3 - theta_2)) - dtheta_2*(mu_2 + mu_3) + dtheta_1*mu_2 + dtheta_1*(-dtheta_1*l_1*l_2*m_3*np.sin(theta_2 - theta_1) - dtheta_1*l_1*l_2*m_4*np.sin(theta_2 - theta_1) - dtheta_1*l_1*l_2*m_5*np.sin(theta_2 - theta_1) - dtheta_1*l_1*m_2*r_2*np.sin(theta_2 - theta_1)) - ddx_c*(-l_2*m_3 - l_2*m_4 - l_2*m_5 - m_2*r_2)*np.cos(theta_2) - g*(-l_2*m_3 - l_2*m_4 - l_2*m_5 - m_2*r_2)*np.sin(theta_2) - l_1*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*(dtheta_5**2*l_1*m_5*r_5*np.sin(theta_5 - theta_1) + dtheta_4*(dtheta_4*l_1*l_4*m_5*np.sin(theta_4 - theta_1) + dtheta_4*l_1*m_4*r_4*np.sin(theta_4 - theta_1)) + dtheta_3*(dtheta_3*l_1*l_3*m_4*np.sin(theta_3 - theta_1) + dtheta_3*l_1*l_3*m_5*np.sin(theta_3 - theta_1) + dtheta_3*l_1*m_3*r_3*np.sin(theta_3 - theta_1)) + dtheta_2*mu_2 + dtheta_2*(dtheta_2*l_1*l_2*m_3*np.sin(theta_2 - theta_1) + dtheta_2*l_1*l_2*m_4*np.sin(theta_2 - theta_1) + dtheta_2*l_1*l_2*m_5*np.sin(theta_2 - theta_1) + dtheta_2*l_1*m_2*r_2*np.sin(theta_2 - theta_1)) - dtheta_1*(mu_1 + mu_2) - ddx_c*(-l_1*m_2 - l_1*m_3 - l_1*m_4 - l_1*m_5 - m_1*r_1)*np.cos(theta_1) - g*(-l_1*m_2 - l_1*m_3 - l_1*m_4 - l_1*m_5 - m_1*r_1)*np.sin(theta_1))*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2))/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2) - (-l_1**2*m_5*r_5*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_5 - theta_1)*np.cos(theta_3 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_3*m_5*r_5*np.cos(theta_5 - theta_3) - (-l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_3 - theta_2))*(-l_1**2*m_5*r_5*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_5 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*m_5*r_5*np.cos(theta_5 - theta_2))/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2))*(dtheta_5**2*l_3*m_5*r_5*np.sin(theta_5 - theta_3) + dtheta_4*mu_4 + dtheta_4*(dtheta_4*l_3*l_4*m_5*np.sin(theta_4 - theta_3) + dtheta_4*l_3*m_4*r_4*np.sin(theta_4 - theta_3)) - dtheta_3*(mu_3 + mu_4) + dtheta_2*mu_3 + dtheta_2*(-dtheta_2*l_2*l_3*m_4*np.sin(theta_3 - theta_2) - dtheta_2*l_2*l_3*m_5*np.sin(theta_3 - theta_2) - dtheta_2*l_2*m_3*r_3*np.sin(theta_3 - theta_2)) + dtheta_1*(-dtheta_1*l_1*l_3*m_4*np.sin(theta_3 - theta_1) - dtheta_1*l_1*l_3*m_5*np.sin(theta_3 - theta_1) - dtheta_1*l_1*m_3*r_3*np.sin(theta_3 - theta_1)) - ddx_c*(-l_3*m_4 - l_3*m_5 - m_3*r_3)*np.cos(theta_3) - g*(-l_3*m_4 - l_3*m_5 - m_3*r_3)*np.sin(theta_3) - l_1*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(dtheta_5**2*l_1*m_5*r_5*np.sin(theta_5 - theta_1) + dtheta_4*(dtheta_4*l_1*l_4*m_5*np.sin(theta_4 - theta_1) + dtheta_4*l_1*m_4*r_4*np.sin(theta_4 - theta_1)) + dtheta_3*(dtheta_3*l_1*l_3*m_4*np.sin(theta_3 - theta_1) + dtheta_3*l_1*l_3*m_5*np.sin(theta_3 - theta_1) + dtheta_3*l_1*m_3*r_3*np.sin(theta_3 - theta_1)) + dtheta_2*mu_2 + dtheta_2*(dtheta_2*l_1*l_2*m_3*np.sin(theta_2 - theta_1) + dtheta_2*l_1*l_2*m_4*np.sin(theta_2 - theta_1) + dtheta_2*l_1*l_2*m_5*np.sin(theta_2 - theta_1) + dtheta_2*l_1*m_2*r_2*np.sin(theta_2 - theta_1)) - dtheta_1*(mu_1 + mu_2) - ddx_c*(-l_1*m_2 - l_1*m_3 - l_1*m_4 - l_1*m_5 - m_1*r_1)*np.cos(theta_1) - g*(-l_1*m_2 - l_1*m_3 - l_1*m_4 - l_1*m_5 - m_1*r_1)*np.sin(theta_1))*np.cos(theta_3 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) - (-l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_3 - theta_2))*(dtheta_5**2*l_2*m_5*r_5*np.sin(theta_5 - theta_2) + dtheta_4*(dtheta_4*l_2*l_4*m_5*np.sin(theta_4 - theta_2) + dtheta_4*l_2*m_4*r_4*np.sin(theta_4 - theta_2)) + dtheta_3*mu_3 + dtheta_3*(dtheta_3*l_2*l_3*m_4*np.sin(theta_3 - theta_2) + dtheta_3*l_2*l_3*m_5*np.sin(theta_3 - theta_2) + dtheta_3*l_2*m_3*r_3*np.sin(theta_3 - theta_2)) - dtheta_2*(mu_2 + mu_3) + dtheta_1*mu_2 + dtheta_1*(-dtheta_1*l_1*l_2*m_3*np.sin(theta_2 - theta_1) - dtheta_1*l_1*l_2*m_4*np.sin(theta_2 - theta_1) - dtheta_1*l_1*l_2*m_5*np.sin(theta_2 - theta_1) - dtheta_1*l_1*m_2*r_2*np.sin(theta_2 - theta_1)) - ddx_c*(-l_2*m_3 - l_2*m_4 - l_2*m_5 - m_2*r_2)*np.cos(theta_2) - g*(-l_2*m_3 - l_2*m_4 - l_2*m_5 - m_2*r_2)*np.sin(theta_2) - l_1*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*(dtheta_5**2*l_1*m_5*r_5*np.sin(theta_5 - theta_1) + dtheta_4*(dtheta_4*l_1*l_4*m_5*np.sin(theta_4 - theta_1) + dtheta_4*l_1*m_4*r_4*np.sin(theta_4 - theta_1)) + dtheta_3*(dtheta_3*l_1*l_3*m_4*np.sin(theta_3 - theta_1) + dtheta_3*l_1*l_3*m_5*np.sin(theta_3 - theta_1) + dtheta_3*l_1*m_3*r_3*np.sin(theta_3 - theta_1)) + dtheta_2*mu_2 + dtheta_2*(dtheta_2*l_1*l_2*m_3*np.sin(theta_2 - theta_1) + dtheta_2*l_1*l_2*m_4*np.sin(theta_2 - theta_1) + dtheta_2*l_1*l_2*m_5*np.sin(theta_2 - theta_1) + dtheta_2*l_1*m_2*r_2*np.sin(theta_2 - theta_1)) - dtheta_1*(mu_1 + mu_2) - ddx_c*(-l_1*m_2 - l_1*m_3 - l_1*m_4 - l_1*m_5 - m_1*r_1)*np.cos(theta_1) - g*(-l_1*m_2 - l_1*m_3 - l_1*m_4 - l_1*m_5 - m_1*r_1)*np.sin(theta_1))*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2))/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2))/(I_3 - l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)**2*np.cos(theta_3 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_3**2*m_4 + l_3**2*m_5 + m_3*r_3**2 - (-l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_3 - theta_2))**2/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2)) - (-l_1**2*m_5*r_5*(l_4*m_5 + m_4*r_4)*np.cos(theta_5 - theta_1)*np.cos(theta_4 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_4*m_5*r_5*np.cos(theta_5 - theta_4) - (-l_1**2*(l_4*m_5 + m_4*r_4)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_4 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_4*m_5 + m_4*r_4)*np.cos(theta_4 - theta_2))*(-l_1**2*m_5*r_5*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_5 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*m_5*r_5*np.cos(theta_5 - theta_2))/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2) - (-l_1**2*(l_4*m_5 + m_4*r_4)*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_4 - theta_1)*np.cos(theta_3 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_3*(l_4*m_5 + m_4*r_4)*np.cos(theta_4 - theta_3) - (-l_1**2*(l_4*m_5 + m_4*r_4)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_4 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_4*m_5 + m_4*r_4)*np.cos(theta_4 - theta_2))*(-l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_3 - theta_2))/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2))*(-l_1**2*m_5*r_5*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_5 - theta_1)*np.cos(theta_3 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_3*m_5*r_5*np.cos(theta_5 - theta_3) - (-l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_3 - theta_2))*(-l_1**2*m_5*r_5*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_5 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*m_5*r_5*np.cos(theta_5 - theta_2))/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2))/(I_3 - l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)**2*np.cos(theta_3 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_3**2*m_4 + l_3**2*m_5 + m_3*r_3**2 - (-l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_3 - theta_2))**2/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2)))*(dtheta_5**2*l_4*m_5*r_5*np.sin(theta_5 - theta_4) + dtheta_5*mu_5 - dtheta_4*(mu_4 + mu_5) + dtheta_3*mu_4 + dtheta_3*(-dtheta_3*l_3*l_4*m_5*np.sin(theta_4 - theta_3) - dtheta_3*l_3*m_4*r_4*np.sin(theta_4 - theta_3)) + dtheta_2*(-dtheta_2*l_2*l_4*m_5*np.sin(theta_4 - theta_2) - dtheta_2*l_2*m_4*r_4*np.sin(theta_4 - theta_2)) + dtheta_1*(-dtheta_1*l_1*l_4*m_5*np.sin(theta_4 - theta_1) - dtheta_1*l_1*m_4*r_4*np.sin(theta_4 - theta_1)) - ddx_c*(-l_4*m_5 - m_4*r_4)*np.cos(theta_4) - g*(-l_4*m_5 - m_4*r_4)*np.sin(theta_4) - l_1*(l_4*m_5 + m_4*r_4)*(dtheta_5**2*l_1*m_5*r_5*np.sin(theta_5 - theta_1) + dtheta_4*(dtheta_4*l_1*l_4*m_5*np.sin(theta_4 - theta_1) + dtheta_4*l_1*m_4*r_4*np.sin(theta_4 - theta_1)) + dtheta_3*(dtheta_3*l_1*l_3*m_4*np.sin(theta_3 - theta_1) + dtheta_3*l_1*l_3*m_5*np.sin(theta_3 - theta_1) + dtheta_3*l_1*m_3*r_3*np.sin(theta_3 - theta_1)) + dtheta_2*mu_2 + dtheta_2*(dtheta_2*l_1*l_2*m_3*np.sin(theta_2 - theta_1) + dtheta_2*l_1*l_2*m_4*np.sin(theta_2 - theta_1) + dtheta_2*l_1*l_2*m_5*np.sin(theta_2 - theta_1) + dtheta_2*l_1*m_2*r_2*np.sin(theta_2 - theta_1)) - dtheta_1*(mu_1 + mu_2) - ddx_c*(-l_1*m_2 - l_1*m_3 - l_1*m_4 - l_1*m_5 - m_1*r_1)*np.cos(theta_1) - g*(-l_1*m_2 - l_1*m_3 - l_1*m_4 - l_1*m_5 - m_1*r_1)*np.sin(theta_1))*np.cos(theta_4 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) - (-l_1**2*(l_4*m_5 + m_4*r_4)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_4 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_4*m_5 + m_4*r_4)*np.cos(theta_4 - theta_2))*(dtheta_5**2*l_2*m_5*r_5*np.sin(theta_5 - theta_2) + dtheta_4*(dtheta_4*l_2*l_4*m_5*np.sin(theta_4 - theta_2) + dtheta_4*l_2*m_4*r_4*np.sin(theta_4 - theta_2)) + dtheta_3*mu_3 + dtheta_3*(dtheta_3*l_2*l_3*m_4*np.sin(theta_3 - theta_2) + dtheta_3*l_2*l_3*m_5*np.sin(theta_3 - theta_2) + dtheta_3*l_2*m_3*r_3*np.sin(theta_3 - theta_2)) - dtheta_2*(mu_2 + mu_3) + dtheta_1*mu_2 + dtheta_1*(-dtheta_1*l_1*l_2*m_3*np.sin(theta_2 - theta_1) - dtheta_1*l_1*l_2*m_4*np.sin(theta_2 - theta_1) - dtheta_1*l_1*l_2*m_5*np.sin(theta_2 - theta_1) - dtheta_1*l_1*m_2*r_2*np.sin(theta_2 - theta_1)) - ddx_c*(-l_2*m_3 - l_2*m_4 - l_2*m_5 - m_2*r_2)*np.cos(theta_2) - g*(-l_2*m_3 - l_2*m_4 - l_2*m_5 - m_2*r_2)*np.sin(theta_2) - l_1*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*(dtheta_5**2*l_1*m_5*r_5*np.sin(theta_5 - theta_1) + dtheta_4*(dtheta_4*l_1*l_4*m_5*np.sin(theta_4 - theta_1) + dtheta_4*l_1*m_4*r_4*np.sin(theta_4 - theta_1)) + dtheta_3*(dtheta_3*l_1*l_3*m_4*np.sin(theta_3 - theta_1) + dtheta_3*l_1*l_3*m_5*np.sin(theta_3 - theta_1) + dtheta_3*l_1*m_3*r_3*np.sin(theta_3 - theta_1)) + dtheta_2*mu_2 + dtheta_2*(dtheta_2*l_1*l_2*m_3*np.sin(theta_2 - theta_1) + dtheta_2*l_1*l_2*m_4*np.sin(theta_2 - theta_1) + dtheta_2*l_1*l_2*m_5*np.sin(theta_2 - theta_1) + dtheta_2*l_1*m_2*r_2*np.sin(theta_2 - theta_1)) - dtheta_1*(mu_1 + mu_2) - ddx_c*(-l_1*m_2 - l_1*m_3 - l_1*m_4 - l_1*m_5 - m_1*r_1)*np.cos(theta_1) - g*(-l_1*m_2 - l_1*m_3 - l_1*m_4 - l_1*m_5 - m_1*r_1)*np.sin(theta_1))*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2))/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2) - (-l_1**2*(l_4*m_5 + m_4*r_4)*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_4 - theta_1)*np.cos(theta_3 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_3*(l_4*m_5 + m_4*r_4)*np.cos(theta_4 - theta_3) - (-l_1**2*(l_4*m_5 + m_4*r_4)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_4 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_4*m_5 + m_4*r_4)*np.cos(theta_4 - theta_2))*(-l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_3 - theta_2))/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2))*(dtheta_5**2*l_3*m_5*r_5*np.sin(theta_5 - theta_3) + dtheta_4*mu_4 + dtheta_4*(dtheta_4*l_3*l_4*m_5*np.sin(theta_4 - theta_3) + dtheta_4*l_3*m_4*r_4*np.sin(theta_4 - theta_3)) - dtheta_3*(mu_3 + mu_4) + dtheta_2*mu_3 + dtheta_2*(-dtheta_2*l_2*l_3*m_4*np.sin(theta_3 - theta_2) - dtheta_2*l_2*l_3*m_5*np.sin(theta_3 - theta_2) - dtheta_2*l_2*m_3*r_3*np.sin(theta_3 - theta_2)) + dtheta_1*(-dtheta_1*l_1*l_3*m_4*np.sin(theta_3 - theta_1) - dtheta_1*l_1*l_3*m_5*np.sin(theta_3 - theta_1) - dtheta_1*l_1*m_3*r_3*np.sin(theta_3 - theta_1)) - ddx_c*(-l_3*m_4 - l_3*m_5 - m_3*r_3)*np.cos(theta_3) - g*(-l_3*m_4 - l_3*m_5 - m_3*r_3)*np.sin(theta_3) - l_1*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(dtheta_5**2*l_1*m_5*r_5*np.sin(theta_5 - theta_1) + dtheta_4*(dtheta_4*l_1*l_4*m_5*np.sin(theta_4 - theta_1) + dtheta_4*l_1*m_4*r_4*np.sin(theta_4 - theta_1)) + dtheta_3*(dtheta_3*l_1*l_3*m_4*np.sin(theta_3 - theta_1) + dtheta_3*l_1*l_3*m_5*np.sin(theta_3 - theta_1) + dtheta_3*l_1*m_3*r_3*np.sin(theta_3 - theta_1)) + dtheta_2*mu_2 + dtheta_2*(dtheta_2*l_1*l_2*m_3*np.sin(theta_2 - theta_1) + dtheta_2*l_1*l_2*m_4*np.sin(theta_2 - theta_1) + dtheta_2*l_1*l_2*m_5*np.sin(theta_2 - theta_1) + dtheta_2*l_1*m_2*r_2*np.sin(theta_2 - theta_1)) - dtheta_1*(mu_1 + mu_2) - ddx_c*(-l_1*m_2 - l_1*m_3 - l_1*m_4 - l_1*m_5 - m_1*r_1)*np.cos(theta_1) - g*(-l_1*m_2 - l_1*m_3 - l_1*m_4 - l_1*m_5 - m_1*r_1)*np.sin(theta_1))*np.cos(theta_3 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) - (-l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_3 - theta_2))*(dtheta_5**2*l_2*m_5*r_5*np.sin(theta_5 - theta_2) + dtheta_4*(dtheta_4*l_2*l_4*m_5*np.sin(theta_4 - theta_2) + dtheta_4*l_2*m_4*r_4*np.sin(theta_4 - theta_2)) + dtheta_3*mu_3 + dtheta_3*(dtheta_3*l_2*l_3*m_4*np.sin(theta_3 - theta_2) + dtheta_3*l_2*l_3*m_5*np.sin(theta_3 - theta_2) + dtheta_3*l_2*m_3*r_3*np.sin(theta_3 - theta_2)) - dtheta_2*(mu_2 + mu_3) + dtheta_1*mu_2 + dtheta_1*(-dtheta_1*l_1*l_2*m_3*np.sin(theta_2 - theta_1) - dtheta_1*l_1*l_2*m_4*np.sin(theta_2 - theta_1) - dtheta_1*l_1*l_2*m_5*np.sin(theta_2 - theta_1) - dtheta_1*l_1*m_2*r_2*np.sin(theta_2 - theta_1)) - ddx_c*(-l_2*m_3 - l_2*m_4 - l_2*m_5 - m_2*r_2)*np.cos(theta_2) - g*(-l_2*m_3 - l_2*m_4 - l_2*m_5 - m_2*r_2)*np.sin(theta_2) - l_1*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*(dtheta_5**2*l_1*m_5*r_5*np.sin(theta_5 - theta_1) + dtheta_4*(dtheta_4*l_1*l_4*m_5*np.sin(theta_4 - theta_1) + dtheta_4*l_1*m_4*r_4*np.sin(theta_4 - theta_1)) + dtheta_3*(dtheta_3*l_1*l_3*m_4*np.sin(theta_3 - theta_1) + dtheta_3*l_1*l_3*m_5*np.sin(theta_3 - theta_1) + dtheta_3*l_1*m_3*r_3*np.sin(theta_3 - theta_1)) + dtheta_2*mu_2 + dtheta_2*(dtheta_2*l_1*l_2*m_3*np.sin(theta_2 - theta_1) + dtheta_2*l_1*l_2*m_4*np.sin(theta_2 - theta_1) + dtheta_2*l_1*l_2*m_5*np.sin(theta_2 - theta_1) + dtheta_2*l_1*m_2*r_2*np.sin(theta_2 - theta_1)) - dtheta_1*(mu_1 + mu_2) - ddx_c*(-l_1*m_2 - l_1*m_3 - l_1*m_4 - l_1*m_5 - m_1*r_1)*np.cos(theta_1) - g*(-l_1*m_2 - l_1*m_3 - l_1*m_4 - l_1*m_5 - m_1*r_1)*np.sin(theta_1))*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2))/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2))/(I_3 - l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)**2*np.cos(theta_3 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_3**2*m_4 + l_3**2*m_5 + m_3*r_3**2 - (-l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_3 - theta_2))**2/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2)))/(I_4 - l_1**2*(l_4*m_5 + m_4*r_4)**2*np.cos(theta_4 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_4**2*m_5 + m_4*r_4**2 - (-l_1**2*(l_4*m_5 + m_4*r_4)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_4 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_4*m_5 + m_4*r_4)*np.cos(theta_4 - theta_2))**2/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2) - (-l_1**2*(l_4*m_5 + m_4*r_4)*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_4 - theta_1)*np.cos(theta_3 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_3*(l_4*m_5 + m_4*r_4)*np.cos(theta_4 - theta_3) - (-l_1**2*(l_4*m_5 + m_4*r_4)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_4 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_4*m_5 + m_4*r_4)*np.cos(theta_4 - theta_2))*(-l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_3 - theta_2))/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2))**2/(I_3 - l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)**2*np.cos(theta_3 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_3**2*m_4 + l_3**2*m_5 + m_3*r_3**2 - (-l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_3 - theta_2))**2/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2))))/(I_5 - l_1**2*m_5**2*r_5**2*np.cos(theta_5 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + m_5*r_5**2 - (-l_1**2*m_5*r_5*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_5 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*m_5*r_5*np.cos(theta_5 - theta_2))**2/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2) - (-l_1**2*m_5*r_5*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_5 - theta_1)*np.cos(theta_3 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_3*m_5*r_5*np.cos(theta_5 - theta_3) - (-l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_3 - theta_2))*(-l_1**2*m_5*r_5*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_5 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*m_5*r_5*np.cos(theta_5 - theta_2))/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2))**2/(I_3 - l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)**2*np.cos(theta_3 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_3**2*m_4 + l_3**2*m_5 + m_3*r_3**2 - (-l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_3 - theta_2))**2/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2)) - (-l_1**2*m_5*r_5*(l_4*m_5 + m_4*r_4)*np.cos(theta_5 - theta_1)*np.cos(theta_4 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_4*m_5*r_5*np.cos(theta_5 - theta_4) - (-l_1**2*(l_4*m_5 + m_4*r_4)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_4 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_4*m_5 + m_4*r_4)*np.cos(theta_4 - theta_2))*(-l_1**2*m_5*r_5*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_5 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*m_5*r_5*np.cos(theta_5 - theta_2))/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2) - (-l_1**2*(l_4*m_5 + m_4*r_4)*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_4 - theta_1)*np.cos(theta_3 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_3*(l_4*m_5 + m_4*r_4)*np.cos(theta_4 - theta_3) - (-l_1**2*(l_4*m_5 + m_4*r_4)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_4 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_4*m_5 + m_4*r_4)*np.cos(theta_4 - theta_2))*(-l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_3 - theta_2))/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2))*(-l_1**2*m_5*r_5*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_5 - theta_1)*np.cos(theta_3 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_3*m_5*r_5*np.cos(theta_5 - theta_3) - (-l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_3 - theta_2))*(-l_1**2*m_5*r_5*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_5 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*m_5*r_5*np.cos(theta_5 - theta_2))/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2))/(I_3 - l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)**2*np.cos(theta_3 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_3**2*m_4 + l_3**2*m_5 + m_3*r_3**2 - (-l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_3 - theta_2))**2/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2)))**2/(I_4 - l_1**2*(l_4*m_5 + m_4*r_4)**2*np.cos(theta_4 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_4**2*m_5 + m_4*r_4**2 - (-l_1**2*(l_4*m_5 + m_4*r_4)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_4 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_4*m_5 + m_4*r_4)*np.cos(theta_4 - theta_2))**2/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2) - (-l_1**2*(l_4*m_5 + m_4*r_4)*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_4 - theta_1)*np.cos(theta_3 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_3*(l_4*m_5 + m_4*r_4)*np.cos(theta_4 - theta_3) - (-l_1**2*(l_4*m_5 + m_4*r_4)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_4 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_4*m_5 + m_4*r_4)*np.cos(theta_4 - theta_2))*(-l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_3 - theta_2))/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2))**2/(I_3 - l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)**2*np.cos(theta_3 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_3**2*m_4 + l_3**2*m_5 + m_3*r_3**2 - (-l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_3 - theta_2))**2/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2)))))/(I_4 - l_1**2*(l_4*m_5 + m_4*r_4)**2*np.cos(theta_4 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_4**2*m_5 + m_4*r_4**2 - (-l_1**2*(l_4*m_5 + m_4*r_4)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_4 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_4*m_5 + m_4*r_4)*np.cos(theta_4 - theta_2))**2/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2) - (-l_1**2*(l_4*m_5 + m_4*r_4)*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_4 - theta_1)*np.cos(theta_3 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_3*(l_4*m_5 + m_4*r_4)*np.cos(theta_4 - theta_3) - (-l_1**2*(l_4*m_5 + m_4*r_4)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_4 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_4*m_5 + m_4*r_4)*np.cos(theta_4 - theta_2))*(-l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_3 - theta_2))/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2))**2/(I_3 - l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)**2*np.cos(theta_3 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_3**2*m_4 + l_3**2*m_5 + m_3*r_3**2 - (-l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_3 - theta_2))**2/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2))) - (-l_1**2*m_5*r_5*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_5 - theta_1)*np.cos(theta_3 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_3*m_5*r_5*np.cos(theta_5 - theta_3) - (-l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_3 - theta_2))*(-l_1**2*m_5*r_5*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_5 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*m_5*r_5*np.cos(theta_5 - theta_2))/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2))*(-dtheta_5*mu_5 - dtheta_4**2*l_4*m_5*r_5*np.sin(theta_5 - theta_4) + dtheta_4*mu_5 - dtheta_3**2*l_3*m_5*r_5*np.sin(theta_5 - theta_3) - dtheta_2**2*l_2*m_5*r_5*np.sin(theta_5 - theta_2) - dtheta_1**2*l_1*m_5*r_5*np.sin(theta_5 - theta_1) + ddx_c*m_5*r_5*np.cos(theta_5) + g*m_5*r_5*np.sin(theta_5) - l_1*m_5*r_5*(dtheta_5**2*l_1*m_5*r_5*np.sin(theta_5 - theta_1) + dtheta_4*(dtheta_4*l_1*l_4*m_5*np.sin(theta_4 - theta_1) + dtheta_4*l_1*m_4*r_4*np.sin(theta_4 - theta_1)) + dtheta_3*(dtheta_3*l_1*l_3*m_4*np.sin(theta_3 - theta_1) + dtheta_3*l_1*l_3*m_5*np.sin(theta_3 - theta_1) + dtheta_3*l_1*m_3*r_3*np.sin(theta_3 - theta_1)) + dtheta_2*mu_2 + dtheta_2*(dtheta_2*l_1*l_2*m_3*np.sin(theta_2 - theta_1) + dtheta_2*l_1*l_2*m_4*np.sin(theta_2 - theta_1) + dtheta_2*l_1*l_2*m_5*np.sin(theta_2 - theta_1) + dtheta_2*l_1*m_2*r_2*np.sin(theta_2 - theta_1)) - dtheta_1*(mu_1 + mu_2) - ddx_c*(-l_1*m_2 - l_1*m_3 - l_1*m_4 - l_1*m_5 - m_1*r_1)*np.cos(theta_1) - g*(-l_1*m_2 - l_1*m_3 - l_1*m_4 - l_1*m_5 - m_1*r_1)*np.sin(theta_1))*np.cos(theta_5 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) - (-l_1**2*m_5*r_5*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_5 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*m_5*r_5*np.cos(theta_5 - theta_2))*(dtheta_5**2*l_2*m_5*r_5*np.sin(theta_5 - theta_2) + dtheta_4*(dtheta_4*l_2*l_4*m_5*np.sin(theta_4 - theta_2) + dtheta_4*l_2*m_4*r_4*np.sin(theta_4 - theta_2)) + dtheta_3*mu_3 + dtheta_3*(dtheta_3*l_2*l_3*m_4*np.sin(theta_3 - theta_2) + dtheta_3*l_2*l_3*m_5*np.sin(theta_3 - theta_2) + dtheta_3*l_2*m_3*r_3*np.sin(theta_3 - theta_2)) - dtheta_2*(mu_2 + mu_3) + dtheta_1*mu_2 + dtheta_1*(-dtheta_1*l_1*l_2*m_3*np.sin(theta_2 - theta_1) - dtheta_1*l_1*l_2*m_4*np.sin(theta_2 - theta_1) - dtheta_1*l_1*l_2*m_5*np.sin(theta_2 - theta_1) - dtheta_1*l_1*m_2*r_2*np.sin(theta_2 - theta_1)) - ddx_c*(-l_2*m_3 - l_2*m_4 - l_2*m_5 - m_2*r_2)*np.cos(theta_2) - g*(-l_2*m_3 - l_2*m_4 - l_2*m_5 - m_2*r_2)*np.sin(theta_2) - l_1*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*(dtheta_5**2*l_1*m_5*r_5*np.sin(theta_5 - theta_1) + dtheta_4*(dtheta_4*l_1*l_4*m_5*np.sin(theta_4 - theta_1) + dtheta_4*l_1*m_4*r_4*np.sin(theta_4 - theta_1)) + dtheta_3*(dtheta_3*l_1*l_3*m_4*np.sin(theta_3 - theta_1) + dtheta_3*l_1*l_3*m_5*np.sin(theta_3 - theta_1) + dtheta_3*l_1*m_3*r_3*np.sin(theta_3 - theta_1)) + dtheta_2*mu_2 + dtheta_2*(dtheta_2*l_1*l_2*m_3*np.sin(theta_2 - theta_1) + dtheta_2*l_1*l_2*m_4*np.sin(theta_2 - theta_1) + dtheta_2*l_1*l_2*m_5*np.sin(theta_2 - theta_1) + dtheta_2*l_1*m_2*r_2*np.sin(theta_2 - theta_1)) - dtheta_1*(mu_1 + mu_2) - ddx_c*(-l_1*m_2 - l_1*m_3 - l_1*m_4 - l_1*m_5 - m_1*r_1)*np.cos(theta_1) - g*(-l_1*m_2 - l_1*m_3 - l_1*m_4 - l_1*m_5 - m_1*r_1)*np.sin(theta_1))*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2))/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2) - (-l_1**2*m_5*r_5*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_5 - theta_1)*np.cos(theta_3 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_3*m_5*r_5*np.cos(theta_5 - theta_3) - (-l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_3 - theta_2))*(-l_1**2*m_5*r_5*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_5 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*m_5*r_5*np.cos(theta_5 - theta_2))/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2))*(dtheta_5**2*l_3*m_5*r_5*np.sin(theta_5 - theta_3) + dtheta_4*mu_4 + dtheta_4*(dtheta_4*l_3*l_4*m_5*np.sin(theta_4 - theta_3) + dtheta_4*l_3*m_4*r_4*np.sin(theta_4 - theta_3)) - dtheta_3*(mu_3 + mu_4) + dtheta_2*mu_3 + dtheta_2*(-dtheta_2*l_2*l_3*m_4*np.sin(theta_3 - theta_2) - dtheta_2*l_2*l_3*m_5*np.sin(theta_3 - theta_2) - dtheta_2*l_2*m_3*r_3*np.sin(theta_3 - theta_2)) + dtheta_1*(-dtheta_1*l_1*l_3*m_4*np.sin(theta_3 - theta_1) - dtheta_1*l_1*l_3*m_5*np.sin(theta_3 - theta_1) - dtheta_1*l_1*m_3*r_3*np.sin(theta_3 - theta_1)) - ddx_c*(-l_3*m_4 - l_3*m_5 - m_3*r_3)*np.cos(theta_3) - g*(-l_3*m_4 - l_3*m_5 - m_3*r_3)*np.sin(theta_3) - l_1*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(dtheta_5**2*l_1*m_5*r_5*np.sin(theta_5 - theta_1) + dtheta_4*(dtheta_4*l_1*l_4*m_5*np.sin(theta_4 - theta_1) + dtheta_4*l_1*m_4*r_4*np.sin(theta_4 - theta_1)) + dtheta_3*(dtheta_3*l_1*l_3*m_4*np.sin(theta_3 - theta_1) + dtheta_3*l_1*l_3*m_5*np.sin(theta_3 - theta_1) + dtheta_3*l_1*m_3*r_3*np.sin(theta_3 - theta_1)) + dtheta_2*mu_2 + dtheta_2*(dtheta_2*l_1*l_2*m_3*np.sin(theta_2 - theta_1) + dtheta_2*l_1*l_2*m_4*np.sin(theta_2 - theta_1) + dtheta_2*l_1*l_2*m_5*np.sin(theta_2 - theta_1) + dtheta_2*l_1*m_2*r_2*np.sin(theta_2 - theta_1)) - dtheta_1*(mu_1 + mu_2) - ddx_c*(-l_1*m_2 - l_1*m_3 - l_1*m_4 - l_1*m_5 - m_1*r_1)*np.cos(theta_1) - g*(-l_1*m_2 - l_1*m_3 - l_1*m_4 - l_1*m_5 - m_1*r_1)*np.sin(theta_1))*np.cos(theta_3 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) - (-l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_3 - theta_2))*(dtheta_5**2*l_2*m_5*r_5*np.sin(theta_5 - theta_2) + dtheta_4*(dtheta_4*l_2*l_4*m_5*np.sin(theta_4 - theta_2) + dtheta_4*l_2*m_4*r_4*np.sin(theta_4 - theta_2)) + dtheta_3*mu_3 + dtheta_3*(dtheta_3*l_2*l_3*m_4*np.sin(theta_3 - theta_2) + dtheta_3*l_2*l_3*m_5*np.sin(theta_3 - theta_2) + dtheta_3*l_2*m_3*r_3*np.sin(theta_3 - theta_2)) - dtheta_2*(mu_2 + mu_3) + dtheta_1*mu_2 + dtheta_1*(-dtheta_1*l_1*l_2*m_3*np.sin(theta_2 - theta_1) - dtheta_1*l_1*l_2*m_4*np.sin(theta_2 - theta_1) - dtheta_1*l_1*l_2*m_5*np.sin(theta_2 - theta_1) - dtheta_1*l_1*m_2*r_2*np.sin(theta_2 - theta_1)) - ddx_c*(-l_2*m_3 - l_2*m_4 - l_2*m_5 - m_2*r_2)*np.cos(theta_2) - g*(-l_2*m_3 - l_2*m_4 - l_2*m_5 - m_2*r_2)*np.sin(theta_2) - l_1*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*(dtheta_5**2*l_1*m_5*r_5*np.sin(theta_5 - theta_1) + dtheta_4*(dtheta_4*l_1*l_4*m_5*np.sin(theta_4 - theta_1) + dtheta_4*l_1*m_4*r_4*np.sin(theta_4 - theta_1)) + dtheta_3*(dtheta_3*l_1*l_3*m_4*np.sin(theta_3 - theta_1) + dtheta_3*l_1*l_3*m_5*np.sin(theta_3 - theta_1) + dtheta_3*l_1*m_3*r_3*np.sin(theta_3 - theta_1)) + dtheta_2*mu_2 + dtheta_2*(dtheta_2*l_1*l_2*m_3*np.sin(theta_2 - theta_1) + dtheta_2*l_1*l_2*m_4*np.sin(theta_2 - theta_1) + dtheta_2*l_1*l_2*m_5*np.sin(theta_2 - theta_1) + dtheta_2*l_1*m_2*r_2*np.sin(theta_2 - theta_1)) - dtheta_1*(mu_1 + mu_2) - ddx_c*(-l_1*m_2 - l_1*m_3 - l_1*m_4 - l_1*m_5 - m_1*r_1)*np.cos(theta_1) - g*(-l_1*m_2 - l_1*m_3 - l_1*m_4 - l_1*m_5 - m_1*r_1)*np.sin(theta_1))*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2))/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2))/(I_3 - l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)**2*np.cos(theta_3 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_3**2*m_4 + l_3**2*m_5 + m_3*r_3**2 - (-l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_3 - theta_2))**2/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2)) - (-l_1**2*m_5*r_5*(l_4*m_5 + m_4*r_4)*np.cos(theta_5 - theta_1)*np.cos(theta_4 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_4*m_5*r_5*np.cos(theta_5 - theta_4) - (-l_1**2*(l_4*m_5 + m_4*r_4)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_4 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_4*m_5 + m_4*r_4)*np.cos(theta_4 - theta_2))*(-l_1**2*m_5*r_5*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_5 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*m_5*r_5*np.cos(theta_5 - theta_2))/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2) - (-l_1**2*(l_4*m_5 + m_4*r_4)*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_4 - theta_1)*np.cos(theta_3 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_3*(l_4*m_5 + m_4*r_4)*np.cos(theta_4 - theta_3) - (-l_1**2*(l_4*m_5 + m_4*r_4)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_4 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_4*m_5 + m_4*r_4)*np.cos(theta_4 - theta_2))*(-l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_3 - theta_2))/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2))*(-l_1**2*m_5*r_5*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_5 - theta_1)*np.cos(theta_3 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_3*m_5*r_5*np.cos(theta_5 - theta_3) - (-l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_3 - theta_2))*(-l_1**2*m_5*r_5*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_5 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*m_5*r_5*np.cos(theta_5 - theta_2))/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2))/(I_3 - l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)**2*np.cos(theta_3 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_3**2*m_4 + l_3**2*m_5 + m_3*r_3**2 - (-l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_3 - theta_2))**2/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2)))*(dtheta_5**2*l_4*m_5*r_5*np.sin(theta_5 - theta_4) + dtheta_5*mu_5 - dtheta_4*(mu_4 + mu_5) + dtheta_3*mu_4 + dtheta_3*(-dtheta_3*l_3*l_4*m_5*np.sin(theta_4 - theta_3) - dtheta_3*l_3*m_4*r_4*np.sin(theta_4 - theta_3)) + dtheta_2*(-dtheta_2*l_2*l_4*m_5*np.sin(theta_4 - theta_2) - dtheta_2*l_2*m_4*r_4*np.sin(theta_4 - theta_2)) + dtheta_1*(-dtheta_1*l_1*l_4*m_5*np.sin(theta_4 - theta_1) - dtheta_1*l_1*m_4*r_4*np.sin(theta_4 - theta_1)) - ddx_c*(-l_4*m_5 - m_4*r_4)*np.cos(theta_4) - g*(-l_4*m_5 - m_4*r_4)*np.sin(theta_4) - l_1*(l_4*m_5 + m_4*r_4)*(dtheta_5**2*l_1*m_5*r_5*np.sin(theta_5 - theta_1) + dtheta_4*(dtheta_4*l_1*l_4*m_5*np.sin(theta_4 - theta_1) + dtheta_4*l_1*m_4*r_4*np.sin(theta_4 - theta_1)) + dtheta_3*(dtheta_3*l_1*l_3*m_4*np.sin(theta_3 - theta_1) + dtheta_3*l_1*l_3*m_5*np.sin(theta_3 - theta_1) + dtheta_3*l_1*m_3*r_3*np.sin(theta_3 - theta_1)) + dtheta_2*mu_2 + dtheta_2*(dtheta_2*l_1*l_2*m_3*np.sin(theta_2 - theta_1) + dtheta_2*l_1*l_2*m_4*np.sin(theta_2 - theta_1) + dtheta_2*l_1*l_2*m_5*np.sin(theta_2 - theta_1) + dtheta_2*l_1*m_2*r_2*np.sin(theta_2 - theta_1)) - dtheta_1*(mu_1 + mu_2) - ddx_c*(-l_1*m_2 - l_1*m_3 - l_1*m_4 - l_1*m_5 - m_1*r_1)*np.cos(theta_1) - g*(-l_1*m_2 - l_1*m_3 - l_1*m_4 - l_1*m_5 - m_1*r_1)*np.sin(theta_1))*np.cos(theta_4 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) - (-l_1**2*(l_4*m_5 + m_4*r_4)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_4 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_4*m_5 + m_4*r_4)*np.cos(theta_4 - theta_2))*(dtheta_5**2*l_2*m_5*r_5*np.sin(theta_5 - theta_2) + dtheta_4*(dtheta_4*l_2*l_4*m_5*np.sin(theta_4 - theta_2) + dtheta_4*l_2*m_4*r_4*np.sin(theta_4 - theta_2)) + dtheta_3*mu_3 + dtheta_3*(dtheta_3*l_2*l_3*m_4*np.sin(theta_3 - theta_2) + dtheta_3*l_2*l_3*m_5*np.sin(theta_3 - theta_2) + dtheta_3*l_2*m_3*r_3*np.sin(theta_3 - theta_2)) - dtheta_2*(mu_2 + mu_3) + dtheta_1*mu_2 + dtheta_1*(-dtheta_1*l_1*l_2*m_3*np.sin(theta_2 - theta_1) - dtheta_1*l_1*l_2*m_4*np.sin(theta_2 - theta_1) - dtheta_1*l_1*l_2*m_5*np.sin(theta_2 - theta_1) - dtheta_1*l_1*m_2*r_2*np.sin(theta_2 - theta_1)) - ddx_c*(-l_2*m_3 - l_2*m_4 - l_2*m_5 - m_2*r_2)*np.cos(theta_2) - g*(-l_2*m_3 - l_2*m_4 - l_2*m_5 - m_2*r_2)*np.sin(theta_2) - l_1*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*(dtheta_5**2*l_1*m_5*r_5*np.sin(theta_5 - theta_1) + dtheta_4*(dtheta_4*l_1*l_4*m_5*np.sin(theta_4 - theta_1) + dtheta_4*l_1*m_4*r_4*np.sin(theta_4 - theta_1)) + dtheta_3*(dtheta_3*l_1*l_3*m_4*np.sin(theta_3 - theta_1) + dtheta_3*l_1*l_3*m_5*np.sin(theta_3 - theta_1) + dtheta_3*l_1*m_3*r_3*np.sin(theta_3 - theta_1)) + dtheta_2*mu_2 + dtheta_2*(dtheta_2*l_1*l_2*m_3*np.sin(theta_2 - theta_1) + dtheta_2*l_1*l_2*m_4*np.sin(theta_2 - theta_1) + dtheta_2*l_1*l_2*m_5*np.sin(theta_2 - theta_1) + dtheta_2*l_1*m_2*r_2*np.sin(theta_2 - theta_1)) - dtheta_1*(mu_1 + mu_2) - ddx_c*(-l_1*m_2 - l_1*m_3 - l_1*m_4 - l_1*m_5 - m_1*r_1)*np.cos(theta_1) - g*(-l_1*m_2 - l_1*m_3 - l_1*m_4 - l_1*m_5 - m_1*r_1)*np.sin(theta_1))*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2))/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2) - (-l_1**2*(l_4*m_5 + m_4*r_4)*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_4 - theta_1)*np.cos(theta_3 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_3*(l_4*m_5 + m_4*r_4)*np.cos(theta_4 - theta_3) - (-l_1**2*(l_4*m_5 + m_4*r_4)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_4 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_4*m_5 + m_4*r_4)*np.cos(theta_4 - theta_2))*(-l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_3 - theta_2))/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2))*(dtheta_5**2*l_3*m_5*r_5*np.sin(theta_5 - theta_3) + dtheta_4*mu_4 + dtheta_4*(dtheta_4*l_3*l_4*m_5*np.sin(theta_4 - theta_3) + dtheta_4*l_3*m_4*r_4*np.sin(theta_4 - theta_3)) - dtheta_3*(mu_3 + mu_4) + dtheta_2*mu_3 + dtheta_2*(-dtheta_2*l_2*l_3*m_4*np.sin(theta_3 - theta_2) - dtheta_2*l_2*l_3*m_5*np.sin(theta_3 - theta_2) - dtheta_2*l_2*m_3*r_3*np.sin(theta_3 - theta_2)) + dtheta_1*(-dtheta_1*l_1*l_3*m_4*np.sin(theta_3 - theta_1) - dtheta_1*l_1*l_3*m_5*np.sin(theta_3 - theta_1) - dtheta_1*l_1*m_3*r_3*np.sin(theta_3 - theta_1)) - ddx_c*(-l_3*m_4 - l_3*m_5 - m_3*r_3)*np.cos(theta_3) - g*(-l_3*m_4 - l_3*m_5 - m_3*r_3)*np.sin(theta_3) - l_1*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(dtheta_5**2*l_1*m_5*r_5*np.sin(theta_5 - theta_1) + dtheta_4*(dtheta_4*l_1*l_4*m_5*np.sin(theta_4 - theta_1) + dtheta_4*l_1*m_4*r_4*np.sin(theta_4 - theta_1)) + dtheta_3*(dtheta_3*l_1*l_3*m_4*np.sin(theta_3 - theta_1) + dtheta_3*l_1*l_3*m_5*np.sin(theta_3 - theta_1) + dtheta_3*l_1*m_3*r_3*np.sin(theta_3 - theta_1)) + dtheta_2*mu_2 + dtheta_2*(dtheta_2*l_1*l_2*m_3*np.sin(theta_2 - theta_1) + dtheta_2*l_1*l_2*m_4*np.sin(theta_2 - theta_1) + dtheta_2*l_1*l_2*m_5*np.sin(theta_2 - theta_1) + dtheta_2*l_1*m_2*r_2*np.sin(theta_2 - theta_1)) - dtheta_1*(mu_1 + mu_2) - ddx_c*(-l_1*m_2 - l_1*m_3 - l_1*m_4 - l_1*m_5 - m_1*r_1)*np.cos(theta_1) - g*(-l_1*m_2 - l_1*m_3 - l_1*m_4 - l_1*m_5 - m_1*r_1)*np.sin(theta_1))*np.cos(theta_3 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) - (-l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_3 - theta_2))*(dtheta_5**2*l_2*m_5*r_5*np.sin(theta_5 - theta_2) + dtheta_4*(dtheta_4*l_2*l_4*m_5*np.sin(theta_4 - theta_2) + dtheta_4*l_2*m_4*r_4*np.sin(theta_4 - theta_2)) + dtheta_3*mu_3 + dtheta_3*(dtheta_3*l_2*l_3*m_4*np.sin(theta_3 - theta_2) + dtheta_3*l_2*l_3*m_5*np.sin(theta_3 - theta_2) + dtheta_3*l_2*m_3*r_3*np.sin(theta_3 - theta_2)) - dtheta_2*(mu_2 + mu_3) + dtheta_1*mu_2 + dtheta_1*(-dtheta_1*l_1*l_2*m_3*np.sin(theta_2 - theta_1) - dtheta_1*l_1*l_2*m_4*np.sin(theta_2 - theta_1) - dtheta_1*l_1*l_2*m_5*np.sin(theta_2 - theta_1) - dtheta_1*l_1*m_2*r_2*np.sin(theta_2 - theta_1)) - ddx_c*(-l_2*m_3 - l_2*m_4 - l_2*m_5 - m_2*r_2)*np.cos(theta_2) - g*(-l_2*m_3 - l_2*m_4 - l_2*m_5 - m_2*r_2)*np.sin(theta_2) - l_1*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*(dtheta_5**2*l_1*m_5*r_5*np.sin(theta_5 - theta_1) + dtheta_4*(dtheta_4*l_1*l_4*m_5*np.sin(theta_4 - theta_1) + dtheta_4*l_1*m_4*r_4*np.sin(theta_4 - theta_1)) + dtheta_3*(dtheta_3*l_1*l_3*m_4*np.sin(theta_3 - theta_1) + dtheta_3*l_1*l_3*m_5*np.sin(theta_3 - theta_1) + dtheta_3*l_1*m_3*r_3*np.sin(theta_3 - theta_1)) + dtheta_2*mu_2 + dtheta_2*(dtheta_2*l_1*l_2*m_3*np.sin(theta_2 - theta_1) + dtheta_2*l_1*l_2*m_4*np.sin(theta_2 - theta_1) + dtheta_2*l_1*l_2*m_5*np.sin(theta_2 - theta_1) + dtheta_2*l_1*m_2*r_2*np.sin(theta_2 - theta_1)) - dtheta_1*(mu_1 + mu_2) - ddx_c*(-l_1*m_2 - l_1*m_3 - l_1*m_4 - l_1*m_5 - m_1*r_1)*np.cos(theta_1) - g*(-l_1*m_2 - l_1*m_3 - l_1*m_4 - l_1*m_5 - m_1*r_1)*np.sin(theta_1))*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2))/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2))/(I_3 - l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)**2*np.cos(theta_3 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_3**2*m_4 + l_3**2*m_5 + m_3*r_3**2 - (-l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_3 - theta_2))**2/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2)))/(I_4 - l_1**2*(l_4*m_5 + m_4*r_4)**2*np.cos(theta_4 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_4**2*m_5 + m_4*r_4**2 - (-l_1**2*(l_4*m_5 + m_4*r_4)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_4 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_4*m_5 + m_4*r_4)*np.cos(theta_4 - theta_2))**2/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2) - (-l_1**2*(l_4*m_5 + m_4*r_4)*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_4 - theta_1)*np.cos(theta_3 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_3*(l_4*m_5 + m_4*r_4)*np.cos(theta_4 - theta_3) - (-l_1**2*(l_4*m_5 + m_4*r_4)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_4 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_4*m_5 + m_4*r_4)*np.cos(theta_4 - theta_2))*(-l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_3 - theta_2))/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2))**2/(I_3 - l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)**2*np.cos(theta_3 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_3**2*m_4 + l_3**2*m_5 + m_3*r_3**2 - (-l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_3 - theta_2))**2/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2))))/(I_5 - l_1**2*m_5**2*r_5**2*np.cos(theta_5 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + m_5*r_5**2 - (-l_1**2*m_5*r_5*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_5 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*m_5*r_5*np.cos(theta_5 - theta_2))**2/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2) - (-l_1**2*m_5*r_5*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_5 - theta_1)*np.cos(theta_3 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_3*m_5*r_5*np.cos(theta_5 - theta_3) - (-l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_3 - theta_2))*(-l_1**2*m_5*r_5*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_5 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*m_5*r_5*np.cos(theta_5 - theta_2))/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2))**2/(I_3 - l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)**2*np.cos(theta_3 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_3**2*m_4 + l_3**2*m_5 + m_3*r_3**2 - (-l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_3 - theta_2))**2/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2)) - (-l_1**2*m_5*r_5*(l_4*m_5 + m_4*r_4)*np.cos(theta_5 - theta_1)*np.cos(theta_4 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_4*m_5*r_5*np.cos(theta_5 - theta_4) - (-l_1**2*(l_4*m_5 + m_4*r_4)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_4 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_4*m_5 + m_4*r_4)*np.cos(theta_4 - theta_2))*(-l_1**2*m_5*r_5*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_5 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*m_5*r_5*np.cos(theta_5 - theta_2))/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2) - (-l_1**2*(l_4*m_5 + m_4*r_4)*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_4 - theta_1)*np.cos(theta_3 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_3*(l_4*m_5 + m_4*r_4)*np.cos(theta_4 - theta_3) - (-l_1**2*(l_4*m_5 + m_4*r_4)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_4 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_4*m_5 + m_4*r_4)*np.cos(theta_4 - theta_2))*(-l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_3 - theta_2))/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2))*(-l_1**2*m_5*r_5*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_5 - theta_1)*np.cos(theta_3 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_3*m_5*r_5*np.cos(theta_5 - theta_3) - (-l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_3 - theta_2))*(-l_1**2*m_5*r_5*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_5 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*m_5*r_5*np.cos(theta_5 - theta_2))/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2))/(I_3 - l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)**2*np.cos(theta_3 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_3**2*m_4 + l_3**2*m_5 + m_3*r_3**2 - (-l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_3 - theta_2))**2/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2)))**2/(I_4 - l_1**2*(l_4*m_5 + m_4*r_4)**2*np.cos(theta_4 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_4**2*m_5 + m_4*r_4**2 - (-l_1**2*(l_4*m_5 + m_4*r_4)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_4 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_4*m_5 + m_4*r_4)*np.cos(theta_4 - theta_2))**2/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2) - (-l_1**2*(l_4*m_5 + m_4*r_4)*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_4 - theta_1)*np.cos(theta_3 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_3*(l_4*m_5 + m_4*r_4)*np.cos(theta_4 - theta_3) - (-l_1**2*(l_4*m_5 + m_4*r_4)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_4 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_4*m_5 + m_4*r_4)*np.cos(theta_4 - theta_2))*(-l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_3 - theta_2))/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2))**2/(I_3 - l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)**2*np.cos(theta_3 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_3**2*m_4 + l_3**2*m_5 + m_3*r_3**2 - (-l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_3 - theta_2))**2/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2)))))/(I_3 - l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)**2*np.cos(theta_3 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_3**2*m_4 + l_3**2*m_5 + m_3*r_3**2 - (-l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_3 - theta_2))**2/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2)),


                      (dtheta_5**2*l_4*m_5*r_5*np.sin(theta_5 - theta_4) + dtheta_5*mu_5 - dtheta_4*(mu_4 + mu_5) + dtheta_3*mu_4 + dtheta_3*(-dtheta_3*l_3*l_4*m_5*np.sin(theta_4 - theta_3) - dtheta_3*l_3*m_4*r_4*np.sin(theta_4 - theta_3)) + dtheta_2*(-dtheta_2*l_2*l_4*m_5*np.sin(theta_4 - theta_2) - dtheta_2*l_2*m_4*r_4*np.sin(theta_4 - theta_2)) + dtheta_1*(-dtheta_1*l_1*l_4*m_5*np.sin(theta_4 - theta_1) - dtheta_1*l_1*m_4*r_4*np.sin(theta_4 - theta_1)) - ddx_c*(-l_4*m_5 - m_4*r_4)*np.cos(theta_4) - g*(-l_4*m_5 - m_4*r_4)*np.sin(theta_4) - l_1*(l_4*m_5 + m_4*r_4)*(dtheta_5**2*l_1*m_5*r_5*np.sin(theta_5 - theta_1) + dtheta_4*(dtheta_4*l_1*l_4*m_5*np.sin(theta_4 - theta_1) + dtheta_4*l_1*m_4*r_4*np.sin(theta_4 - theta_1)) + dtheta_3*(dtheta_3*l_1*l_3*m_4*np.sin(theta_3 - theta_1) + dtheta_3*l_1*l_3*m_5*np.sin(theta_3 - theta_1) + dtheta_3*l_1*m_3*r_3*np.sin(theta_3 - theta_1)) + dtheta_2*mu_2 + dtheta_2*(dtheta_2*l_1*l_2*m_3*np.sin(theta_2 - theta_1) + dtheta_2*l_1*l_2*m_4*np.sin(theta_2 - theta_1) + dtheta_2*l_1*l_2*m_5*np.sin(theta_2 - theta_1) + dtheta_2*l_1*m_2*r_2*np.sin(theta_2 - theta_1)) - dtheta_1*(mu_1 + mu_2) - ddx_c*(-l_1*m_2 - l_1*m_3 - l_1*m_4 - l_1*m_5 - m_1*r_1)*np.cos(theta_1) - g*(-l_1*m_2 - l_1*m_3 - l_1*m_4 - l_1*m_5 - m_1*r_1)*np.sin(theta_1))*np.cos(theta_4 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) - (-l_1**2*(l_4*m_5 + m_4*r_4)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_4 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_4*m_5 + m_4*r_4)*np.cos(theta_4 - theta_2))*(dtheta_5**2*l_2*m_5*r_5*np.sin(theta_5 - theta_2) + dtheta_4*(dtheta_4*l_2*l_4*m_5*np.sin(theta_4 - theta_2) + dtheta_4*l_2*m_4*r_4*np.sin(theta_4 - theta_2)) + dtheta_3*mu_3 + dtheta_3*(dtheta_3*l_2*l_3*m_4*np.sin(theta_3 - theta_2) + dtheta_3*l_2*l_3*m_5*np.sin(theta_3 - theta_2) + dtheta_3*l_2*m_3*r_3*np.sin(theta_3 - theta_2)) - dtheta_2*(mu_2 + mu_3) + dtheta_1*mu_2 + dtheta_1*(-dtheta_1*l_1*l_2*m_3*np.sin(theta_2 - theta_1) - dtheta_1*l_1*l_2*m_4*np.sin(theta_2 - theta_1) - dtheta_1*l_1*l_2*m_5*np.sin(theta_2 - theta_1) - dtheta_1*l_1*m_2*r_2*np.sin(theta_2 - theta_1)) - ddx_c*(-l_2*m_3 - l_2*m_4 - l_2*m_5 - m_2*r_2)*np.cos(theta_2) - g*(-l_2*m_3 - l_2*m_4 - l_2*m_5 - m_2*r_2)*np.sin(theta_2) - l_1*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*(dtheta_5**2*l_1*m_5*r_5*np.sin(theta_5 - theta_1) + dtheta_4*(dtheta_4*l_1*l_4*m_5*np.sin(theta_4 - theta_1) + dtheta_4*l_1*m_4*r_4*np.sin(theta_4 - theta_1)) + dtheta_3*(dtheta_3*l_1*l_3*m_4*np.sin(theta_3 - theta_1) + dtheta_3*l_1*l_3*m_5*np.sin(theta_3 - theta_1) + dtheta_3*l_1*m_3*r_3*np.sin(theta_3 - theta_1)) + dtheta_2*mu_2 + dtheta_2*(dtheta_2*l_1*l_2*m_3*np.sin(theta_2 - theta_1) + dtheta_2*l_1*l_2*m_4*np.sin(theta_2 - theta_1) + dtheta_2*l_1*l_2*m_5*np.sin(theta_2 - theta_1) + dtheta_2*l_1*m_2*r_2*np.sin(theta_2 - theta_1)) - dtheta_1*(mu_1 + mu_2) - ddx_c*(-l_1*m_2 - l_1*m_3 - l_1*m_4 - l_1*m_5 - m_1*r_1)*np.cos(theta_1) - g*(-l_1*m_2 - l_1*m_3 - l_1*m_4 - l_1*m_5 - m_1*r_1)*np.sin(theta_1))*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2))/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2) - (-l_1**2*(l_4*m_5 + m_4*r_4)*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_4 - theta_1)*np.cos(theta_3 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_3*(l_4*m_5 + m_4*r_4)*np.cos(theta_4 - theta_3) - (-l_1**2*(l_4*m_5 + m_4*r_4)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_4 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_4*m_5 + m_4*r_4)*np.cos(theta_4 - theta_2))*(-l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_3 - theta_2))/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2))*(dtheta_5**2*l_3*m_5*r_5*np.sin(theta_5 - theta_3) + dtheta_4*mu_4 + dtheta_4*(dtheta_4*l_3*l_4*m_5*np.sin(theta_4 - theta_3) + dtheta_4*l_3*m_4*r_4*np.sin(theta_4 - theta_3)) - dtheta_3*(mu_3 + mu_4) + dtheta_2*mu_3 + dtheta_2*(-dtheta_2*l_2*l_3*m_4*np.sin(theta_3 - theta_2) - dtheta_2*l_2*l_3*m_5*np.sin(theta_3 - theta_2) - dtheta_2*l_2*m_3*r_3*np.sin(theta_3 - theta_2)) + dtheta_1*(-dtheta_1*l_1*l_3*m_4*np.sin(theta_3 - theta_1) - dtheta_1*l_1*l_3*m_5*np.sin(theta_3 - theta_1) - dtheta_1*l_1*m_3*r_3*np.sin(theta_3 - theta_1)) - ddx_c*(-l_3*m_4 - l_3*m_5 - m_3*r_3)*np.cos(theta_3) - g*(-l_3*m_4 - l_3*m_5 - m_3*r_3)*np.sin(theta_3) - l_1*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(dtheta_5**2*l_1*m_5*r_5*np.sin(theta_5 - theta_1) + dtheta_4*(dtheta_4*l_1*l_4*m_5*np.sin(theta_4 - theta_1) + dtheta_4*l_1*m_4*r_4*np.sin(theta_4 - theta_1)) + dtheta_3*(dtheta_3*l_1*l_3*m_4*np.sin(theta_3 - theta_1) + dtheta_3*l_1*l_3*m_5*np.sin(theta_3 - theta_1) + dtheta_3*l_1*m_3*r_3*np.sin(theta_3 - theta_1)) + dtheta_2*mu_2 + dtheta_2*(dtheta_2*l_1*l_2*m_3*np.sin(theta_2 - theta_1) + dtheta_2*l_1*l_2*m_4*np.sin(theta_2 - theta_1) + dtheta_2*l_1*l_2*m_5*np.sin(theta_2 - theta_1) + dtheta_2*l_1*m_2*r_2*np.sin(theta_2 - theta_1)) - dtheta_1*(mu_1 + mu_2) - ddx_c*(-l_1*m_2 - l_1*m_3 - l_1*m_4 - l_1*m_5 - m_1*r_1)*np.cos(theta_1) - g*(-l_1*m_2 - l_1*m_3 - l_1*m_4 - l_1*m_5 - m_1*r_1)*np.sin(theta_1))*np.cos(theta_3 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) - (-l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_3 - theta_2))*(dtheta_5**2*l_2*m_5*r_5*np.sin(theta_5 - theta_2) + dtheta_4*(dtheta_4*l_2*l_4*m_5*np.sin(theta_4 - theta_2) + dtheta_4*l_2*m_4*r_4*np.sin(theta_4 - theta_2)) + dtheta_3*mu_3 + dtheta_3*(dtheta_3*l_2*l_3*m_4*np.sin(theta_3 - theta_2) + dtheta_3*l_2*l_3*m_5*np.sin(theta_3 - theta_2) + dtheta_3*l_2*m_3*r_3*np.sin(theta_3 - theta_2)) - dtheta_2*(mu_2 + mu_3) + dtheta_1*mu_2 + dtheta_1*(-dtheta_1*l_1*l_2*m_3*np.sin(theta_2 - theta_1) - dtheta_1*l_1*l_2*m_4*np.sin(theta_2 - theta_1) - dtheta_1*l_1*l_2*m_5*np.sin(theta_2 - theta_1) - dtheta_1*l_1*m_2*r_2*np.sin(theta_2 - theta_1)) - ddx_c*(-l_2*m_3 - l_2*m_4 - l_2*m_5 - m_2*r_2)*np.cos(theta_2) - g*(-l_2*m_3 - l_2*m_4 - l_2*m_5 - m_2*r_2)*np.sin(theta_2) - l_1*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*(dtheta_5**2*l_1*m_5*r_5*np.sin(theta_5 - theta_1) + dtheta_4*(dtheta_4*l_1*l_4*m_5*np.sin(theta_4 - theta_1) + dtheta_4*l_1*m_4*r_4*np.sin(theta_4 - theta_1)) + dtheta_3*(dtheta_3*l_1*l_3*m_4*np.sin(theta_3 - theta_1) + dtheta_3*l_1*l_3*m_5*np.sin(theta_3 - theta_1) + dtheta_3*l_1*m_3*r_3*np.sin(theta_3 - theta_1)) + dtheta_2*mu_2 + dtheta_2*(dtheta_2*l_1*l_2*m_3*np.sin(theta_2 - theta_1) + dtheta_2*l_1*l_2*m_4*np.sin(theta_2 - theta_1) + dtheta_2*l_1*l_2*m_5*np.sin(theta_2 - theta_1) + dtheta_2*l_1*m_2*r_2*np.sin(theta_2 - theta_1)) - dtheta_1*(mu_1 + mu_2) - ddx_c*(-l_1*m_2 - l_1*m_3 - l_1*m_4 - l_1*m_5 - m_1*r_1)*np.cos(theta_1) - g*(-l_1*m_2 - l_1*m_3 - l_1*m_4 - l_1*m_5 - m_1*r_1)*np.sin(theta_1))*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2))/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2))/(I_3 - l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)**2*np.cos(theta_3 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_3**2*m_4 + l_3**2*m_5 + m_3*r_3**2 - (-l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_3 - theta_2))**2/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2)) - (-l_1**2*m_5*r_5*(l_4*m_5 + m_4*r_4)*np.cos(theta_5 - theta_1)*np.cos(theta_4 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_4*m_5*r_5*np.cos(theta_5 - theta_4) - (-l_1**2*(l_4*m_5 + m_4*r_4)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_4 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_4*m_5 + m_4*r_4)*np.cos(theta_4 - theta_2))*(-l_1**2*m_5*r_5*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_5 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*m_5*r_5*np.cos(theta_5 - theta_2))/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2) - (-l_1**2*(l_4*m_5 + m_4*r_4)*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_4 - theta_1)*np.cos(theta_3 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_3*(l_4*m_5 + m_4*r_4)*np.cos(theta_4 - theta_3) - (-l_1**2*(l_4*m_5 + m_4*r_4)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_4 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_4*m_5 + m_4*r_4)*np.cos(theta_4 - theta_2))*(-l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_3 - theta_2))/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2))*(-l_1**2*m_5*r_5*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_5 - theta_1)*np.cos(theta_3 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_3*m_5*r_5*np.cos(theta_5 - theta_3) - (-l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_3 - theta_2))*(-l_1**2*m_5*r_5*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_5 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*m_5*r_5*np.cos(theta_5 - theta_2))/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2))/(I_3 - l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)**2*np.cos(theta_3 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_3**2*m_4 + l_3**2*m_5 + m_3*r_3**2 - (-l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_3 - theta_2))**2/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2)))*(-dtheta_5*mu_5 - dtheta_4**2*l_4*m_5*r_5*np.sin(theta_5 - theta_4) + dtheta_4*mu_5 - dtheta_3**2*l_3*m_5*r_5*np.sin(theta_5 - theta_3) - dtheta_2**2*l_2*m_5*r_5*np.sin(theta_5 - theta_2) - dtheta_1**2*l_1*m_5*r_5*np.sin(theta_5 - theta_1) + ddx_c*m_5*r_5*np.cos(theta_5) + g*m_5*r_5*np.sin(theta_5) - l_1*m_5*r_5*(dtheta_5**2*l_1*m_5*r_5*np.sin(theta_5 - theta_1) + dtheta_4*(dtheta_4*l_1*l_4*m_5*np.sin(theta_4 - theta_1) + dtheta_4*l_1*m_4*r_4*np.sin(theta_4 - theta_1)) + dtheta_3*(dtheta_3*l_1*l_3*m_4*np.sin(theta_3 - theta_1) + dtheta_3*l_1*l_3*m_5*np.sin(theta_3 - theta_1) + dtheta_3*l_1*m_3*r_3*np.sin(theta_3 - theta_1)) + dtheta_2*mu_2 + dtheta_2*(dtheta_2*l_1*l_2*m_3*np.sin(theta_2 - theta_1) + dtheta_2*l_1*l_2*m_4*np.sin(theta_2 - theta_1) + dtheta_2*l_1*l_2*m_5*np.sin(theta_2 - theta_1) + dtheta_2*l_1*m_2*r_2*np.sin(theta_2 - theta_1)) - dtheta_1*(mu_1 + mu_2) - ddx_c*(-l_1*m_2 - l_1*m_3 - l_1*m_4 - l_1*m_5 - m_1*r_1)*np.cos(theta_1) - g*(-l_1*m_2 - l_1*m_3 - l_1*m_4 - l_1*m_5 - m_1*r_1)*np.sin(theta_1))*np.cos(theta_5 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) - (-l_1**2*m_5*r_5*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_5 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*m_5*r_5*np.cos(theta_5 - theta_2))*(dtheta_5**2*l_2*m_5*r_5*np.sin(theta_5 - theta_2) + dtheta_4*(dtheta_4*l_2*l_4*m_5*np.sin(theta_4 - theta_2) + dtheta_4*l_2*m_4*r_4*np.sin(theta_4 - theta_2)) + dtheta_3*mu_3 + dtheta_3*(dtheta_3*l_2*l_3*m_4*np.sin(theta_3 - theta_2) + dtheta_3*l_2*l_3*m_5*np.sin(theta_3 - theta_2) + dtheta_3*l_2*m_3*r_3*np.sin(theta_3 - theta_2)) - dtheta_2*(mu_2 + mu_3) + dtheta_1*mu_2 + dtheta_1*(-dtheta_1*l_1*l_2*m_3*np.sin(theta_2 - theta_1) - dtheta_1*l_1*l_2*m_4*np.sin(theta_2 - theta_1) - dtheta_1*l_1*l_2*m_5*np.sin(theta_2 - theta_1) - dtheta_1*l_1*m_2*r_2*np.sin(theta_2 - theta_1)) - ddx_c*(-l_2*m_3 - l_2*m_4 - l_2*m_5 - m_2*r_2)*np.cos(theta_2) - g*(-l_2*m_3 - l_2*m_4 - l_2*m_5 - m_2*r_2)*np.sin(theta_2) - l_1*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*(dtheta_5**2*l_1*m_5*r_5*np.sin(theta_5 - theta_1) + dtheta_4*(dtheta_4*l_1*l_4*m_5*np.sin(theta_4 - theta_1) + dtheta_4*l_1*m_4*r_4*np.sin(theta_4 - theta_1)) + dtheta_3*(dtheta_3*l_1*l_3*m_4*np.sin(theta_3 - theta_1) + dtheta_3*l_1*l_3*m_5*np.sin(theta_3 - theta_1) + dtheta_3*l_1*m_3*r_3*np.sin(theta_3 - theta_1)) + dtheta_2*mu_2 + dtheta_2*(dtheta_2*l_1*l_2*m_3*np.sin(theta_2 - theta_1) + dtheta_2*l_1*l_2*m_4*np.sin(theta_2 - theta_1) + dtheta_2*l_1*l_2*m_5*np.sin(theta_2 - theta_1) + dtheta_2*l_1*m_2*r_2*np.sin(theta_2 - theta_1)) - dtheta_1*(mu_1 + mu_2) - ddx_c*(-l_1*m_2 - l_1*m_3 - l_1*m_4 - l_1*m_5 - m_1*r_1)*np.cos(theta_1) - g*(-l_1*m_2 - l_1*m_3 - l_1*m_4 - l_1*m_5 - m_1*r_1)*np.sin(theta_1))*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2))/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2) - (-l_1**2*m_5*r_5*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_5 - theta_1)*np.cos(theta_3 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_3*m_5*r_5*np.cos(theta_5 - theta_3) - (-l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_3 - theta_2))*(-l_1**2*m_5*r_5*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_5 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*m_5*r_5*np.cos(theta_5 - theta_2))/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2))*(dtheta_5**2*l_3*m_5*r_5*np.sin(theta_5 - theta_3) + dtheta_4*mu_4 + dtheta_4*(dtheta_4*l_3*l_4*m_5*np.sin(theta_4 - theta_3) + dtheta_4*l_3*m_4*r_4*np.sin(theta_4 - theta_3)) - dtheta_3*(mu_3 + mu_4) + dtheta_2*mu_3 + dtheta_2*(-dtheta_2*l_2*l_3*m_4*np.sin(theta_3 - theta_2) - dtheta_2*l_2*l_3*m_5*np.sin(theta_3 - theta_2) - dtheta_2*l_2*m_3*r_3*np.sin(theta_3 - theta_2)) + dtheta_1*(-dtheta_1*l_1*l_3*m_4*np.sin(theta_3 - theta_1) - dtheta_1*l_1*l_3*m_5*np.sin(theta_3 - theta_1) - dtheta_1*l_1*m_3*r_3*np.sin(theta_3 - theta_1)) - ddx_c*(-l_3*m_4 - l_3*m_5 - m_3*r_3)*np.cos(theta_3) - g*(-l_3*m_4 - l_3*m_5 - m_3*r_3)*np.sin(theta_3) - l_1*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(dtheta_5**2*l_1*m_5*r_5*np.sin(theta_5 - theta_1) + dtheta_4*(dtheta_4*l_1*l_4*m_5*np.sin(theta_4 - theta_1) + dtheta_4*l_1*m_4*r_4*np.sin(theta_4 - theta_1)) + dtheta_3*(dtheta_3*l_1*l_3*m_4*np.sin(theta_3 - theta_1) + dtheta_3*l_1*l_3*m_5*np.sin(theta_3 - theta_1) + dtheta_3*l_1*m_3*r_3*np.sin(theta_3 - theta_1)) + dtheta_2*mu_2 + dtheta_2*(dtheta_2*l_1*l_2*m_3*np.sin(theta_2 - theta_1) + dtheta_2*l_1*l_2*m_4*np.sin(theta_2 - theta_1) + dtheta_2*l_1*l_2*m_5*np.sin(theta_2 - theta_1) + dtheta_2*l_1*m_2*r_2*np.sin(theta_2 - theta_1)) - dtheta_1*(mu_1 + mu_2) - ddx_c*(-l_1*m_2 - l_1*m_3 - l_1*m_4 - l_1*m_5 - m_1*r_1)*np.cos(theta_1) - g*(-l_1*m_2 - l_1*m_3 - l_1*m_4 - l_1*m_5 - m_1*r_1)*np.sin(theta_1))*np.cos(theta_3 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) - (-l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_3 - theta_2))*(dtheta_5**2*l_2*m_5*r_5*np.sin(theta_5 - theta_2) + dtheta_4*(dtheta_4*l_2*l_4*m_5*np.sin(theta_4 - theta_2) + dtheta_4*l_2*m_4*r_4*np.sin(theta_4 - theta_2)) + dtheta_3*mu_3 + dtheta_3*(dtheta_3*l_2*l_3*m_4*np.sin(theta_3 - theta_2) + dtheta_3*l_2*l_3*m_5*np.sin(theta_3 - theta_2) + dtheta_3*l_2*m_3*r_3*np.sin(theta_3 - theta_2)) - dtheta_2*(mu_2 + mu_3) + dtheta_1*mu_2 + dtheta_1*(-dtheta_1*l_1*l_2*m_3*np.sin(theta_2 - theta_1) - dtheta_1*l_1*l_2*m_4*np.sin(theta_2 - theta_1) - dtheta_1*l_1*l_2*m_5*np.sin(theta_2 - theta_1) - dtheta_1*l_1*m_2*r_2*np.sin(theta_2 - theta_1)) - ddx_c*(-l_2*m_3 - l_2*m_4 - l_2*m_5 - m_2*r_2)*np.cos(theta_2) - g*(-l_2*m_3 - l_2*m_4 - l_2*m_5 - m_2*r_2)*np.sin(theta_2) - l_1*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*(dtheta_5**2*l_1*m_5*r_5*np.sin(theta_5 - theta_1) + dtheta_4*(dtheta_4*l_1*l_4*m_5*np.sin(theta_4 - theta_1) + dtheta_4*l_1*m_4*r_4*np.sin(theta_4 - theta_1)) + dtheta_3*(dtheta_3*l_1*l_3*m_4*np.sin(theta_3 - theta_1) + dtheta_3*l_1*l_3*m_5*np.sin(theta_3 - theta_1) + dtheta_3*l_1*m_3*r_3*np.sin(theta_3 - theta_1)) + dtheta_2*mu_2 + dtheta_2*(dtheta_2*l_1*l_2*m_3*np.sin(theta_2 - theta_1) + dtheta_2*l_1*l_2*m_4*np.sin(theta_2 - theta_1) + dtheta_2*l_1*l_2*m_5*np.sin(theta_2 - theta_1) + dtheta_2*l_1*m_2*r_2*np.sin(theta_2 - theta_1)) - dtheta_1*(mu_1 + mu_2) - ddx_c*(-l_1*m_2 - l_1*m_3 - l_1*m_4 - l_1*m_5 - m_1*r_1)*np.cos(theta_1) - g*(-l_1*m_2 - l_1*m_3 - l_1*m_4 - l_1*m_5 - m_1*r_1)*np.sin(theta_1))*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2))/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2))/(I_3 - l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)**2*np.cos(theta_3 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_3**2*m_4 + l_3**2*m_5 + m_3*r_3**2 - (-l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_3 - theta_2))**2/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2)) - (-l_1**2*m_5*r_5*(l_4*m_5 + m_4*r_4)*np.cos(theta_5 - theta_1)*np.cos(theta_4 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_4*m_5*r_5*np.cos(theta_5 - theta_4) - (-l_1**2*(l_4*m_5 + m_4*r_4)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_4 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_4*m_5 + m_4*r_4)*np.cos(theta_4 - theta_2))*(-l_1**2*m_5*r_5*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_5 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*m_5*r_5*np.cos(theta_5 - theta_2))/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2) - (-l_1**2*(l_4*m_5 + m_4*r_4)*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_4 - theta_1)*np.cos(theta_3 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_3*(l_4*m_5 + m_4*r_4)*np.cos(theta_4 - theta_3) - (-l_1**2*(l_4*m_5 + m_4*r_4)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_4 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_4*m_5 + m_4*r_4)*np.cos(theta_4 - theta_2))*(-l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_3 - theta_2))/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2))*(-l_1**2*m_5*r_5*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_5 - theta_1)*np.cos(theta_3 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_3*m_5*r_5*np.cos(theta_5 - theta_3) - (-l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_3 - theta_2))*(-l_1**2*m_5*r_5*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_5 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*m_5*r_5*np.cos(theta_5 - theta_2))/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2))/(I_3 - l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)**2*np.cos(theta_3 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_3**2*m_4 + l_3**2*m_5 + m_3*r_3**2 - (-l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_3 - theta_2))**2/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2)))*(dtheta_5**2*l_4*m_5*r_5*np.sin(theta_5 - theta_4) + dtheta_5*mu_5 - dtheta_4*(mu_4 + mu_5) + dtheta_3*mu_4 + dtheta_3*(-dtheta_3*l_3*l_4*m_5*np.sin(theta_4 - theta_3) - dtheta_3*l_3*m_4*r_4*np.sin(theta_4 - theta_3)) + dtheta_2*(-dtheta_2*l_2*l_4*m_5*np.sin(theta_4 - theta_2) - dtheta_2*l_2*m_4*r_4*np.sin(theta_4 - theta_2)) + dtheta_1*(-dtheta_1*l_1*l_4*m_5*np.sin(theta_4 - theta_1) - dtheta_1*l_1*m_4*r_4*np.sin(theta_4 - theta_1)) - ddx_c*(-l_4*m_5 - m_4*r_4)*np.cos(theta_4) - g*(-l_4*m_5 - m_4*r_4)*np.sin(theta_4) - l_1*(l_4*m_5 + m_4*r_4)*(dtheta_5**2*l_1*m_5*r_5*np.sin(theta_5 - theta_1) + dtheta_4*(dtheta_4*l_1*l_4*m_5*np.sin(theta_4 - theta_1) + dtheta_4*l_1*m_4*r_4*np.sin(theta_4 - theta_1)) + dtheta_3*(dtheta_3*l_1*l_3*m_4*np.sin(theta_3 - theta_1) + dtheta_3*l_1*l_3*m_5*np.sin(theta_3 - theta_1) + dtheta_3*l_1*m_3*r_3*np.sin(theta_3 - theta_1)) + dtheta_2*mu_2 + dtheta_2*(dtheta_2*l_1*l_2*m_3*np.sin(theta_2 - theta_1) + dtheta_2*l_1*l_2*m_4*np.sin(theta_2 - theta_1) + dtheta_2*l_1*l_2*m_5*np.sin(theta_2 - theta_1) + dtheta_2*l_1*m_2*r_2*np.sin(theta_2 - theta_1)) - dtheta_1*(mu_1 + mu_2) - ddx_c*(-l_1*m_2 - l_1*m_3 - l_1*m_4 - l_1*m_5 - m_1*r_1)*np.cos(theta_1) - g*(-l_1*m_2 - l_1*m_3 - l_1*m_4 - l_1*m_5 - m_1*r_1)*np.sin(theta_1))*np.cos(theta_4 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) - (-l_1**2*(l_4*m_5 + m_4*r_4)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_4 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_4*m_5 + m_4*r_4)*np.cos(theta_4 - theta_2))*(dtheta_5**2*l_2*m_5*r_5*np.sin(theta_5 - theta_2) + dtheta_4*(dtheta_4*l_2*l_4*m_5*np.sin(theta_4 - theta_2) + dtheta_4*l_2*m_4*r_4*np.sin(theta_4 - theta_2)) + dtheta_3*mu_3 + dtheta_3*(dtheta_3*l_2*l_3*m_4*np.sin(theta_3 - theta_2) + dtheta_3*l_2*l_3*m_5*np.sin(theta_3 - theta_2) + dtheta_3*l_2*m_3*r_3*np.sin(theta_3 - theta_2)) - dtheta_2*(mu_2 + mu_3) + dtheta_1*mu_2 + dtheta_1*(-dtheta_1*l_1*l_2*m_3*np.sin(theta_2 - theta_1) - dtheta_1*l_1*l_2*m_4*np.sin(theta_2 - theta_1) - dtheta_1*l_1*l_2*m_5*np.sin(theta_2 - theta_1) - dtheta_1*l_1*m_2*r_2*np.sin(theta_2 - theta_1)) - ddx_c*(-l_2*m_3 - l_2*m_4 - l_2*m_5 - m_2*r_2)*np.cos(theta_2) - g*(-l_2*m_3 - l_2*m_4 - l_2*m_5 - m_2*r_2)*np.sin(theta_2) - l_1*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*(dtheta_5**2*l_1*m_5*r_5*np.sin(theta_5 - theta_1) + dtheta_4*(dtheta_4*l_1*l_4*m_5*np.sin(theta_4 - theta_1) + dtheta_4*l_1*m_4*r_4*np.sin(theta_4 - theta_1)) + dtheta_3*(dtheta_3*l_1*l_3*m_4*np.sin(theta_3 - theta_1) + dtheta_3*l_1*l_3*m_5*np.sin(theta_3 - theta_1) + dtheta_3*l_1*m_3*r_3*np.sin(theta_3 - theta_1)) + dtheta_2*mu_2 + dtheta_2*(dtheta_2*l_1*l_2*m_3*np.sin(theta_2 - theta_1) + dtheta_2*l_1*l_2*m_4*np.sin(theta_2 - theta_1) + dtheta_2*l_1*l_2*m_5*np.sin(theta_2 - theta_1) + dtheta_2*l_1*m_2*r_2*np.sin(theta_2 - theta_1)) - dtheta_1*(mu_1 + mu_2) - ddx_c*(-l_1*m_2 - l_1*m_3 - l_1*m_4 - l_1*m_5 - m_1*r_1)*np.cos(theta_1) - g*(-l_1*m_2 - l_1*m_3 - l_1*m_4 - l_1*m_5 - m_1*r_1)*np.sin(theta_1))*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2))/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2) - (-l_1**2*(l_4*m_5 + m_4*r_4)*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_4 - theta_1)*np.cos(theta_3 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_3*(l_4*m_5 + m_4*r_4)*np.cos(theta_4 - theta_3) - (-l_1**2*(l_4*m_5 + m_4*r_4)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_4 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_4*m_5 + m_4*r_4)*np.cos(theta_4 - theta_2))*(-l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_3 - theta_2))/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2))*(dtheta_5**2*l_3*m_5*r_5*np.sin(theta_5 - theta_3) + dtheta_4*mu_4 + dtheta_4*(dtheta_4*l_3*l_4*m_5*np.sin(theta_4 - theta_3) + dtheta_4*l_3*m_4*r_4*np.sin(theta_4 - theta_3)) - dtheta_3*(mu_3 + mu_4) + dtheta_2*mu_3 + dtheta_2*(-dtheta_2*l_2*l_3*m_4*np.sin(theta_3 - theta_2) - dtheta_2*l_2*l_3*m_5*np.sin(theta_3 - theta_2) - dtheta_2*l_2*m_3*r_3*np.sin(theta_3 - theta_2)) + dtheta_1*(-dtheta_1*l_1*l_3*m_4*np.sin(theta_3 - theta_1) - dtheta_1*l_1*l_3*m_5*np.sin(theta_3 - theta_1) - dtheta_1*l_1*m_3*r_3*np.sin(theta_3 - theta_1)) - ddx_c*(-l_3*m_4 - l_3*m_5 - m_3*r_3)*np.cos(theta_3) - g*(-l_3*m_4 - l_3*m_5 - m_3*r_3)*np.sin(theta_3) - l_1*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(dtheta_5**2*l_1*m_5*r_5*np.sin(theta_5 - theta_1) + dtheta_4*(dtheta_4*l_1*l_4*m_5*np.sin(theta_4 - theta_1) + dtheta_4*l_1*m_4*r_4*np.sin(theta_4 - theta_1)) + dtheta_3*(dtheta_3*l_1*l_3*m_4*np.sin(theta_3 - theta_1) + dtheta_3*l_1*l_3*m_5*np.sin(theta_3 - theta_1) + dtheta_3*l_1*m_3*r_3*np.sin(theta_3 - theta_1)) + dtheta_2*mu_2 + dtheta_2*(dtheta_2*l_1*l_2*m_3*np.sin(theta_2 - theta_1) + dtheta_2*l_1*l_2*m_4*np.sin(theta_2 - theta_1) + dtheta_2*l_1*l_2*m_5*np.sin(theta_2 - theta_1) + dtheta_2*l_1*m_2*r_2*np.sin(theta_2 - theta_1)) - dtheta_1*(mu_1 + mu_2) - ddx_c*(-l_1*m_2 - l_1*m_3 - l_1*m_4 - l_1*m_5 - m_1*r_1)*np.cos(theta_1) - g*(-l_1*m_2 - l_1*m_3 - l_1*m_4 - l_1*m_5 - m_1*r_1)*np.sin(theta_1))*np.cos(theta_3 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) - (-l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_3 - theta_2))*(dtheta_5**2*l_2*m_5*r_5*np.sin(theta_5 - theta_2) + dtheta_4*(dtheta_4*l_2*l_4*m_5*np.sin(theta_4 - theta_2) + dtheta_4*l_2*m_4*r_4*np.sin(theta_4 - theta_2)) + dtheta_3*mu_3 + dtheta_3*(dtheta_3*l_2*l_3*m_4*np.sin(theta_3 - theta_2) + dtheta_3*l_2*l_3*m_5*np.sin(theta_3 - theta_2) + dtheta_3*l_2*m_3*r_3*np.sin(theta_3 - theta_2)) - dtheta_2*(mu_2 + mu_3) + dtheta_1*mu_2 + dtheta_1*(-dtheta_1*l_1*l_2*m_3*np.sin(theta_2 - theta_1) - dtheta_1*l_1*l_2*m_4*np.sin(theta_2 - theta_1) - dtheta_1*l_1*l_2*m_5*np.sin(theta_2 - theta_1) - dtheta_1*l_1*m_2*r_2*np.sin(theta_2 - theta_1)) - ddx_c*(-l_2*m_3 - l_2*m_4 - l_2*m_5 - m_2*r_2)*np.cos(theta_2) - g*(-l_2*m_3 - l_2*m_4 - l_2*m_5 - m_2*r_2)*np.sin(theta_2) - l_1*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*(dtheta_5**2*l_1*m_5*r_5*np.sin(theta_5 - theta_1) + dtheta_4*(dtheta_4*l_1*l_4*m_5*np.sin(theta_4 - theta_1) + dtheta_4*l_1*m_4*r_4*np.sin(theta_4 - theta_1)) + dtheta_3*(dtheta_3*l_1*l_3*m_4*np.sin(theta_3 - theta_1) + dtheta_3*l_1*l_3*m_5*np.sin(theta_3 - theta_1) + dtheta_3*l_1*m_3*r_3*np.sin(theta_3 - theta_1)) + dtheta_2*mu_2 + dtheta_2*(dtheta_2*l_1*l_2*m_3*np.sin(theta_2 - theta_1) + dtheta_2*l_1*l_2*m_4*np.sin(theta_2 - theta_1) + dtheta_2*l_1*l_2*m_5*np.sin(theta_2 - theta_1) + dtheta_2*l_1*m_2*r_2*np.sin(theta_2 - theta_1)) - dtheta_1*(mu_1 + mu_2) - ddx_c*(-l_1*m_2 - l_1*m_3 - l_1*m_4 - l_1*m_5 - m_1*r_1)*np.cos(theta_1) - g*(-l_1*m_2 - l_1*m_3 - l_1*m_4 - l_1*m_5 - m_1*r_1)*np.sin(theta_1))*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2))/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2))/(I_3 - l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)**2*np.cos(theta_3 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_3**2*m_4 + l_3**2*m_5 + m_3*r_3**2 - (-l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_3 - theta_2))**2/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2)))/(I_4 - l_1**2*(l_4*m_5 + m_4*r_4)**2*np.cos(theta_4 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_4**2*m_5 + m_4*r_4**2 - (-l_1**2*(l_4*m_5 + m_4*r_4)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_4 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_4*m_5 + m_4*r_4)*np.cos(theta_4 - theta_2))**2/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2) - (-l_1**2*(l_4*m_5 + m_4*r_4)*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_4 - theta_1)*np.cos(theta_3 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_3*(l_4*m_5 + m_4*r_4)*np.cos(theta_4 - theta_3) - (-l_1**2*(l_4*m_5 + m_4*r_4)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_4 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_4*m_5 + m_4*r_4)*np.cos(theta_4 - theta_2))*(-l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_3 - theta_2))/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2))**2/(I_3 - l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)**2*np.cos(theta_3 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_3**2*m_4 + l_3**2*m_5 + m_3*r_3**2 - (-l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_3 - theta_2))**2/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2))))/(I_5 - l_1**2*m_5**2*r_5**2*np.cos(theta_5 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + m_5*r_5**2 - (-l_1**2*m_5*r_5*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_5 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*m_5*r_5*np.cos(theta_5 - theta_2))**2/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2) - (-l_1**2*m_5*r_5*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_5 - theta_1)*np.cos(theta_3 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_3*m_5*r_5*np.cos(theta_5 - theta_3) - (-l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_3 - theta_2))*(-l_1**2*m_5*r_5*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_5 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*m_5*r_5*np.cos(theta_5 - theta_2))/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2))**2/(I_3 - l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)**2*np.cos(theta_3 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_3**2*m_4 + l_3**2*m_5 + m_3*r_3**2 - (-l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_3 - theta_2))**2/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2)) - (-l_1**2*m_5*r_5*(l_4*m_5 + m_4*r_4)*np.cos(theta_5 - theta_1)*np.cos(theta_4 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_4*m_5*r_5*np.cos(theta_5 - theta_4) - (-l_1**2*(l_4*m_5 + m_4*r_4)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_4 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_4*m_5 + m_4*r_4)*np.cos(theta_4 - theta_2))*(-l_1**2*m_5*r_5*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_5 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*m_5*r_5*np.cos(theta_5 - theta_2))/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2) - (-l_1**2*(l_4*m_5 + m_4*r_4)*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_4 - theta_1)*np.cos(theta_3 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_3*(l_4*m_5 + m_4*r_4)*np.cos(theta_4 - theta_3) - (-l_1**2*(l_4*m_5 + m_4*r_4)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_4 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_4*m_5 + m_4*r_4)*np.cos(theta_4 - theta_2))*(-l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_3 - theta_2))/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2))*(-l_1**2*m_5*r_5*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_5 - theta_1)*np.cos(theta_3 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_3*m_5*r_5*np.cos(theta_5 - theta_3) - (-l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_3 - theta_2))*(-l_1**2*m_5*r_5*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_5 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*m_5*r_5*np.cos(theta_5 - theta_2))/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2))/(I_3 - l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)**2*np.cos(theta_3 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_3**2*m_4 + l_3**2*m_5 + m_3*r_3**2 - (-l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_3 - theta_2))**2/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2)))**2/(I_4 - l_1**2*(l_4*m_5 + m_4*r_4)**2*np.cos(theta_4 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_4**2*m_5 + m_4*r_4**2 - (-l_1**2*(l_4*m_5 + m_4*r_4)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_4 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_4*m_5 + m_4*r_4)*np.cos(theta_4 - theta_2))**2/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2) - (-l_1**2*(l_4*m_5 + m_4*r_4)*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_4 - theta_1)*np.cos(theta_3 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_3*(l_4*m_5 + m_4*r_4)*np.cos(theta_4 - theta_3) - (-l_1**2*(l_4*m_5 + m_4*r_4)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_4 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_4*m_5 + m_4*r_4)*np.cos(theta_4 - theta_2))*(-l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_3 - theta_2))/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2))**2/(I_3 - l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)**2*np.cos(theta_3 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_3**2*m_4 + l_3**2*m_5 + m_3*r_3**2 - (-l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_3 - theta_2))**2/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2)))))/(I_4 - l_1**2*(l_4*m_5 + m_4*r_4)**2*np.cos(theta_4 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_4**2*m_5 + m_4*r_4**2 - (-l_1**2*(l_4*m_5 + m_4*r_4)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_4 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_4*m_5 + m_4*r_4)*np.cos(theta_4 - theta_2))**2/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2) - (-l_1**2*(l_4*m_5 + m_4*r_4)*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_4 - theta_1)*np.cos(theta_3 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_3*(l_4*m_5 + m_4*r_4)*np.cos(theta_4 - theta_3) - (-l_1**2*(l_4*m_5 + m_4*r_4)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_4 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_4*m_5 + m_4*r_4)*np.cos(theta_4 - theta_2))*(-l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_3 - theta_2))/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2))**2/(I_3 - l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)**2*np.cos(theta_3 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_3**2*m_4 + l_3**2*m_5 + m_3*r_3**2 - (-l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_3 - theta_2))**2/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2))),


                      (-dtheta_5*mu_5 - dtheta_4**2*l_4*m_5*r_5*np.sin(theta_5 - theta_4) + dtheta_4*mu_5 - dtheta_3**2*l_3*m_5*r_5*np.sin(theta_5 - theta_3) - dtheta_2**2*l_2*m_5*r_5*np.sin(theta_5 - theta_2) - dtheta_1**2*l_1*m_5*r_5*np.sin(theta_5 - theta_1) + ddx_c*m_5*r_5*np.cos(theta_5) + g*m_5*r_5*np.sin(theta_5) - l_1*m_5*r_5*(dtheta_5**2*l_1*m_5*r_5*np.sin(theta_5 - theta_1) + dtheta_4*(dtheta_4*l_1*l_4*m_5*np.sin(theta_4 - theta_1) + dtheta_4*l_1*m_4*r_4*np.sin(theta_4 - theta_1)) + dtheta_3*(dtheta_3*l_1*l_3*m_4*np.sin(theta_3 - theta_1) + dtheta_3*l_1*l_3*m_5*np.sin(theta_3 - theta_1) + dtheta_3*l_1*m_3*r_3*np.sin(theta_3 - theta_1)) + dtheta_2*mu_2 + dtheta_2*(dtheta_2*l_1*l_2*m_3*np.sin(theta_2 - theta_1) + dtheta_2*l_1*l_2*m_4*np.sin(theta_2 - theta_1) + dtheta_2*l_1*l_2*m_5*np.sin(theta_2 - theta_1) + dtheta_2*l_1*m_2*r_2*np.sin(theta_2 - theta_1)) - dtheta_1*(mu_1 + mu_2) - ddx_c*(-l_1*m_2 - l_1*m_3 - l_1*m_4 - l_1*m_5 - m_1*r_1)*np.cos(theta_1) - g*(-l_1*m_2 - l_1*m_3 - l_1*m_4 - l_1*m_5 - m_1*r_1)*np.sin(theta_1))*np.cos(theta_5 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) - (-l_1**2*m_5*r_5*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_5 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*m_5*r_5*np.cos(theta_5 - theta_2))*(dtheta_5**2*l_2*m_5*r_5*np.sin(theta_5 - theta_2) + dtheta_4*(dtheta_4*l_2*l_4*m_5*np.sin(theta_4 - theta_2) + dtheta_4*l_2*m_4*r_4*np.sin(theta_4 - theta_2)) + dtheta_3*mu_3 + dtheta_3*(dtheta_3*l_2*l_3*m_4*np.sin(theta_3 - theta_2) + dtheta_3*l_2*l_3*m_5*np.sin(theta_3 - theta_2) + dtheta_3*l_2*m_3*r_3*np.sin(theta_3 - theta_2)) - dtheta_2*(mu_2 + mu_3) + dtheta_1*mu_2 + dtheta_1*(-dtheta_1*l_1*l_2*m_3*np.sin(theta_2 - theta_1) - dtheta_1*l_1*l_2*m_4*np.sin(theta_2 - theta_1) - dtheta_1*l_1*l_2*m_5*np.sin(theta_2 - theta_1) - dtheta_1*l_1*m_2*r_2*np.sin(theta_2 - theta_1)) - ddx_c*(-l_2*m_3 - l_2*m_4 - l_2*m_5 - m_2*r_2)*np.cos(theta_2) - g*(-l_2*m_3 - l_2*m_4 - l_2*m_5 - m_2*r_2)*np.sin(theta_2) - l_1*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*(dtheta_5**2*l_1*m_5*r_5*np.sin(theta_5 - theta_1) + dtheta_4*(dtheta_4*l_1*l_4*m_5*np.sin(theta_4 - theta_1) + dtheta_4*l_1*m_4*r_4*np.sin(theta_4 - theta_1)) + dtheta_3*(dtheta_3*l_1*l_3*m_4*np.sin(theta_3 - theta_1) + dtheta_3*l_1*l_3*m_5*np.sin(theta_3 - theta_1) + dtheta_3*l_1*m_3*r_3*np.sin(theta_3 - theta_1)) + dtheta_2*mu_2 + dtheta_2*(dtheta_2*l_1*l_2*m_3*np.sin(theta_2 - theta_1) + dtheta_2*l_1*l_2*m_4*np.sin(theta_2 - theta_1) + dtheta_2*l_1*l_2*m_5*np.sin(theta_2 - theta_1) + dtheta_2*l_1*m_2*r_2*np.sin(theta_2 - theta_1)) - dtheta_1*(mu_1 + mu_2) - ddx_c*(-l_1*m_2 - l_1*m_3 - l_1*m_4 - l_1*m_5 - m_1*r_1)*np.cos(theta_1) - g*(-l_1*m_2 - l_1*m_3 - l_1*m_4 - l_1*m_5 - m_1*r_1)*np.sin(theta_1))*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2))/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2) - (-l_1**2*m_5*r_5*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_5 - theta_1)*np.cos(theta_3 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_3*m_5*r_5*np.cos(theta_5 - theta_3) - (-l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_3 - theta_2))*(-l_1**2*m_5*r_5*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_5 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*m_5*r_5*np.cos(theta_5 - theta_2))/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2))*(dtheta_5**2*l_3*m_5*r_5*np.sin(theta_5 - theta_3) + dtheta_4*mu_4 + dtheta_4*(dtheta_4*l_3*l_4*m_5*np.sin(theta_4 - theta_3) + dtheta_4*l_3*m_4*r_4*np.sin(theta_4 - theta_3)) - dtheta_3*(mu_3 + mu_4) + dtheta_2*mu_3 + dtheta_2*(-dtheta_2*l_2*l_3*m_4*np.sin(theta_3 - theta_2) - dtheta_2*l_2*l_3*m_5*np.sin(theta_3 - theta_2) - dtheta_2*l_2*m_3*r_3*np.sin(theta_3 - theta_2)) + dtheta_1*(-dtheta_1*l_1*l_3*m_4*np.sin(theta_3 - theta_1) - dtheta_1*l_1*l_3*m_5*np.sin(theta_3 - theta_1) - dtheta_1*l_1*m_3*r_3*np.sin(theta_3 - theta_1)) - ddx_c*(-l_3*m_4 - l_3*m_5 - m_3*r_3)*np.cos(theta_3) - g*(-l_3*m_4 - l_3*m_5 - m_3*r_3)*np.sin(theta_3) - l_1*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(dtheta_5**2*l_1*m_5*r_5*np.sin(theta_5 - theta_1) + dtheta_4*(dtheta_4*l_1*l_4*m_5*np.sin(theta_4 - theta_1) + dtheta_4*l_1*m_4*r_4*np.sin(theta_4 - theta_1)) + dtheta_3*(dtheta_3*l_1*l_3*m_4*np.sin(theta_3 - theta_1) + dtheta_3*l_1*l_3*m_5*np.sin(theta_3 - theta_1) + dtheta_3*l_1*m_3*r_3*np.sin(theta_3 - theta_1)) + dtheta_2*mu_2 + dtheta_2*(dtheta_2*l_1*l_2*m_3*np.sin(theta_2 - theta_1) + dtheta_2*l_1*l_2*m_4*np.sin(theta_2 - theta_1) + dtheta_2*l_1*l_2*m_5*np.sin(theta_2 - theta_1) + dtheta_2*l_1*m_2*r_2*np.sin(theta_2 - theta_1)) - dtheta_1*(mu_1 + mu_2) - ddx_c*(-l_1*m_2 - l_1*m_3 - l_1*m_4 - l_1*m_5 - m_1*r_1)*np.cos(theta_1) - g*(-l_1*m_2 - l_1*m_3 - l_1*m_4 - l_1*m_5 - m_1*r_1)*np.sin(theta_1))*np.cos(theta_3 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) - (-l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_3 - theta_2))*(dtheta_5**2*l_2*m_5*r_5*np.sin(theta_5 - theta_2) + dtheta_4*(dtheta_4*l_2*l_4*m_5*np.sin(theta_4 - theta_2) + dtheta_4*l_2*m_4*r_4*np.sin(theta_4 - theta_2)) + dtheta_3*mu_3 + dtheta_3*(dtheta_3*l_2*l_3*m_4*np.sin(theta_3 - theta_2) + dtheta_3*l_2*l_3*m_5*np.sin(theta_3 - theta_2) + dtheta_3*l_2*m_3*r_3*np.sin(theta_3 - theta_2)) - dtheta_2*(mu_2 + mu_3) + dtheta_1*mu_2 + dtheta_1*(-dtheta_1*l_1*l_2*m_3*np.sin(theta_2 - theta_1) - dtheta_1*l_1*l_2*m_4*np.sin(theta_2 - theta_1) - dtheta_1*l_1*l_2*m_5*np.sin(theta_2 - theta_1) - dtheta_1*l_1*m_2*r_2*np.sin(theta_2 - theta_1)) - ddx_c*(-l_2*m_3 - l_2*m_4 - l_2*m_5 - m_2*r_2)*np.cos(theta_2) - g*(-l_2*m_3 - l_2*m_4 - l_2*m_5 - m_2*r_2)*np.sin(theta_2) - l_1*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*(dtheta_5**2*l_1*m_5*r_5*np.sin(theta_5 - theta_1) + dtheta_4*(dtheta_4*l_1*l_4*m_5*np.sin(theta_4 - theta_1) + dtheta_4*l_1*m_4*r_4*np.sin(theta_4 - theta_1)) + dtheta_3*(dtheta_3*l_1*l_3*m_4*np.sin(theta_3 - theta_1) + dtheta_3*l_1*l_3*m_5*np.sin(theta_3 - theta_1) + dtheta_3*l_1*m_3*r_3*np.sin(theta_3 - theta_1)) + dtheta_2*mu_2 + dtheta_2*(dtheta_2*l_1*l_2*m_3*np.sin(theta_2 - theta_1) + dtheta_2*l_1*l_2*m_4*np.sin(theta_2 - theta_1) + dtheta_2*l_1*l_2*m_5*np.sin(theta_2 - theta_1) + dtheta_2*l_1*m_2*r_2*np.sin(theta_2 - theta_1)) - dtheta_1*(mu_1 + mu_2) - ddx_c*(-l_1*m_2 - l_1*m_3 - l_1*m_4 - l_1*m_5 - m_1*r_1)*np.cos(theta_1) - g*(-l_1*m_2 - l_1*m_3 - l_1*m_4 - l_1*m_5 - m_1*r_1)*np.sin(theta_1))*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2))/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2))/(I_3 - l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)**2*np.cos(theta_3 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_3**2*m_4 + l_3**2*m_5 + m_3*r_3**2 - (-l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_3 - theta_2))**2/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2)) - (-l_1**2*m_5*r_5*(l_4*m_5 + m_4*r_4)*np.cos(theta_5 - theta_1)*np.cos(theta_4 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_4*m_5*r_5*np.cos(theta_5 - theta_4) - (-l_1**2*(l_4*m_5 + m_4*r_4)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_4 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_4*m_5 + m_4*r_4)*np.cos(theta_4 - theta_2))*(-l_1**2*m_5*r_5*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_5 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*m_5*r_5*np.cos(theta_5 - theta_2))/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2) - (-l_1**2*(l_4*m_5 + m_4*r_4)*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_4 - theta_1)*np.cos(theta_3 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_3*(l_4*m_5 + m_4*r_4)*np.cos(theta_4 - theta_3) - (-l_1**2*(l_4*m_5 + m_4*r_4)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_4 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_4*m_5 + m_4*r_4)*np.cos(theta_4 - theta_2))*(-l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_3 - theta_2))/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2))*(-l_1**2*m_5*r_5*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_5 - theta_1)*np.cos(theta_3 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_3*m_5*r_5*np.cos(theta_5 - theta_3) - (-l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_3 - theta_2))*(-l_1**2*m_5*r_5*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_5 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*m_5*r_5*np.cos(theta_5 - theta_2))/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2))/(I_3 - l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)**2*np.cos(theta_3 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_3**2*m_4 + l_3**2*m_5 + m_3*r_3**2 - (-l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_3 - theta_2))**2/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2)))*(dtheta_5**2*l_4*m_5*r_5*np.sin(theta_5 - theta_4) + dtheta_5*mu_5 - dtheta_4*(mu_4 + mu_5) + dtheta_3*mu_4 + dtheta_3*(-dtheta_3*l_3*l_4*m_5*np.sin(theta_4 - theta_3) - dtheta_3*l_3*m_4*r_4*np.sin(theta_4 - theta_3)) + dtheta_2*(-dtheta_2*l_2*l_4*m_5*np.sin(theta_4 - theta_2) - dtheta_2*l_2*m_4*r_4*np.sin(theta_4 - theta_2)) + dtheta_1*(-dtheta_1*l_1*l_4*m_5*np.sin(theta_4 - theta_1) - dtheta_1*l_1*m_4*r_4*np.sin(theta_4 - theta_1)) - ddx_c*(-l_4*m_5 - m_4*r_4)*np.cos(theta_4) - g*(-l_4*m_5 - m_4*r_4)*np.sin(theta_4) - l_1*(l_4*m_5 + m_4*r_4)*(dtheta_5**2*l_1*m_5*r_5*np.sin(theta_5 - theta_1) + dtheta_4*(dtheta_4*l_1*l_4*m_5*np.sin(theta_4 - theta_1) + dtheta_4*l_1*m_4*r_4*np.sin(theta_4 - theta_1)) + dtheta_3*(dtheta_3*l_1*l_3*m_4*np.sin(theta_3 - theta_1) + dtheta_3*l_1*l_3*m_5*np.sin(theta_3 - theta_1) + dtheta_3*l_1*m_3*r_3*np.sin(theta_3 - theta_1)) + dtheta_2*mu_2 + dtheta_2*(dtheta_2*l_1*l_2*m_3*np.sin(theta_2 - theta_1) + dtheta_2*l_1*l_2*m_4*np.sin(theta_2 - theta_1) + dtheta_2*l_1*l_2*m_5*np.sin(theta_2 - theta_1) + dtheta_2*l_1*m_2*r_2*np.sin(theta_2 - theta_1)) - dtheta_1*(mu_1 + mu_2) - ddx_c*(-l_1*m_2 - l_1*m_3 - l_1*m_4 - l_1*m_5 - m_1*r_1)*np.cos(theta_1) - g*(-l_1*m_2 - l_1*m_3 - l_1*m_4 - l_1*m_5 - m_1*r_1)*np.sin(theta_1))*np.cos(theta_4 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) - (-l_1**2*(l_4*m_5 + m_4*r_4)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_4 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_4*m_5 + m_4*r_4)*np.cos(theta_4 - theta_2))*(dtheta_5**2*l_2*m_5*r_5*np.sin(theta_5 - theta_2) + dtheta_4*(dtheta_4*l_2*l_4*m_5*np.sin(theta_4 - theta_2) + dtheta_4*l_2*m_4*r_4*np.sin(theta_4 - theta_2)) + dtheta_3*mu_3 + dtheta_3*(dtheta_3*l_2*l_3*m_4*np.sin(theta_3 - theta_2) + dtheta_3*l_2*l_3*m_5*np.sin(theta_3 - theta_2) + dtheta_3*l_2*m_3*r_3*np.sin(theta_3 - theta_2)) - dtheta_2*(mu_2 + mu_3) + dtheta_1*mu_2 + dtheta_1*(-dtheta_1*l_1*l_2*m_3*np.sin(theta_2 - theta_1) - dtheta_1*l_1*l_2*m_4*np.sin(theta_2 - theta_1) - dtheta_1*l_1*l_2*m_5*np.sin(theta_2 - theta_1) - dtheta_1*l_1*m_2*r_2*np.sin(theta_2 - theta_1)) - ddx_c*(-l_2*m_3 - l_2*m_4 - l_2*m_5 - m_2*r_2)*np.cos(theta_2) - g*(-l_2*m_3 - l_2*m_4 - l_2*m_5 - m_2*r_2)*np.sin(theta_2) - l_1*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*(dtheta_5**2*l_1*m_5*r_5*np.sin(theta_5 - theta_1) + dtheta_4*(dtheta_4*l_1*l_4*m_5*np.sin(theta_4 - theta_1) + dtheta_4*l_1*m_4*r_4*np.sin(theta_4 - theta_1)) + dtheta_3*(dtheta_3*l_1*l_3*m_4*np.sin(theta_3 - theta_1) + dtheta_3*l_1*l_3*m_5*np.sin(theta_3 - theta_1) + dtheta_3*l_1*m_3*r_3*np.sin(theta_3 - theta_1)) + dtheta_2*mu_2 + dtheta_2*(dtheta_2*l_1*l_2*m_3*np.sin(theta_2 - theta_1) + dtheta_2*l_1*l_2*m_4*np.sin(theta_2 - theta_1) + dtheta_2*l_1*l_2*m_5*np.sin(theta_2 - theta_1) + dtheta_2*l_1*m_2*r_2*np.sin(theta_2 - theta_1)) - dtheta_1*(mu_1 + mu_2) - ddx_c*(-l_1*m_2 - l_1*m_3 - l_1*m_4 - l_1*m_5 - m_1*r_1)*np.cos(theta_1) - g*(-l_1*m_2 - l_1*m_3 - l_1*m_4 - l_1*m_5 - m_1*r_1)*np.sin(theta_1))*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2))/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2) - (-l_1**2*(l_4*m_5 + m_4*r_4)*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_4 - theta_1)*np.cos(theta_3 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_3*(l_4*m_5 + m_4*r_4)*np.cos(theta_4 - theta_3) - (-l_1**2*(l_4*m_5 + m_4*r_4)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_4 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_4*m_5 + m_4*r_4)*np.cos(theta_4 - theta_2))*(-l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_3 - theta_2))/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2))*(dtheta_5**2*l_3*m_5*r_5*np.sin(theta_5 - theta_3) + dtheta_4*mu_4 + dtheta_4*(dtheta_4*l_3*l_4*m_5*np.sin(theta_4 - theta_3) + dtheta_4*l_3*m_4*r_4*np.sin(theta_4 - theta_3)) - dtheta_3*(mu_3 + mu_4) + dtheta_2*mu_3 + dtheta_2*(-dtheta_2*l_2*l_3*m_4*np.sin(theta_3 - theta_2) - dtheta_2*l_2*l_3*m_5*np.sin(theta_3 - theta_2) - dtheta_2*l_2*m_3*r_3*np.sin(theta_3 - theta_2)) + dtheta_1*(-dtheta_1*l_1*l_3*m_4*np.sin(theta_3 - theta_1) - dtheta_1*l_1*l_3*m_5*np.sin(theta_3 - theta_1) - dtheta_1*l_1*m_3*r_3*np.sin(theta_3 - theta_1)) - ddx_c*(-l_3*m_4 - l_3*m_5 - m_3*r_3)*np.cos(theta_3) - g*(-l_3*m_4 - l_3*m_5 - m_3*r_3)*np.sin(theta_3) - l_1*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(dtheta_5**2*l_1*m_5*r_5*np.sin(theta_5 - theta_1) + dtheta_4*(dtheta_4*l_1*l_4*m_5*np.sin(theta_4 - theta_1) + dtheta_4*l_1*m_4*r_4*np.sin(theta_4 - theta_1)) + dtheta_3*(dtheta_3*l_1*l_3*m_4*np.sin(theta_3 - theta_1) + dtheta_3*l_1*l_3*m_5*np.sin(theta_3 - theta_1) + dtheta_3*l_1*m_3*r_3*np.sin(theta_3 - theta_1)) + dtheta_2*mu_2 + dtheta_2*(dtheta_2*l_1*l_2*m_3*np.sin(theta_2 - theta_1) + dtheta_2*l_1*l_2*m_4*np.sin(theta_2 - theta_1) + dtheta_2*l_1*l_2*m_5*np.sin(theta_2 - theta_1) + dtheta_2*l_1*m_2*r_2*np.sin(theta_2 - theta_1)) - dtheta_1*(mu_1 + mu_2) - ddx_c*(-l_1*m_2 - l_1*m_3 - l_1*m_4 - l_1*m_5 - m_1*r_1)*np.cos(theta_1) - g*(-l_1*m_2 - l_1*m_3 - l_1*m_4 - l_1*m_5 - m_1*r_1)*np.sin(theta_1))*np.cos(theta_3 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) - (-l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_3 - theta_2))*(dtheta_5**2*l_2*m_5*r_5*np.sin(theta_5 - theta_2) + dtheta_4*(dtheta_4*l_2*l_4*m_5*np.sin(theta_4 - theta_2) + dtheta_4*l_2*m_4*r_4*np.sin(theta_4 - theta_2)) + dtheta_3*mu_3 + dtheta_3*(dtheta_3*l_2*l_3*m_4*np.sin(theta_3 - theta_2) + dtheta_3*l_2*l_3*m_5*np.sin(theta_3 - theta_2) + dtheta_3*l_2*m_3*r_3*np.sin(theta_3 - theta_2)) - dtheta_2*(mu_2 + mu_3) + dtheta_1*mu_2 + dtheta_1*(-dtheta_1*l_1*l_2*m_3*np.sin(theta_2 - theta_1) - dtheta_1*l_1*l_2*m_4*np.sin(theta_2 - theta_1) - dtheta_1*l_1*l_2*m_5*np.sin(theta_2 - theta_1) - dtheta_1*l_1*m_2*r_2*np.sin(theta_2 - theta_1)) - ddx_c*(-l_2*m_3 - l_2*m_4 - l_2*m_5 - m_2*r_2)*np.cos(theta_2) - g*(-l_2*m_3 - l_2*m_4 - l_2*m_5 - m_2*r_2)*np.sin(theta_2) - l_1*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*(dtheta_5**2*l_1*m_5*r_5*np.sin(theta_5 - theta_1) + dtheta_4*(dtheta_4*l_1*l_4*m_5*np.sin(theta_4 - theta_1) + dtheta_4*l_1*m_4*r_4*np.sin(theta_4 - theta_1)) + dtheta_3*(dtheta_3*l_1*l_3*m_4*np.sin(theta_3 - theta_1) + dtheta_3*l_1*l_3*m_5*np.sin(theta_3 - theta_1) + dtheta_3*l_1*m_3*r_3*np.sin(theta_3 - theta_1)) + dtheta_2*mu_2 + dtheta_2*(dtheta_2*l_1*l_2*m_3*np.sin(theta_2 - theta_1) + dtheta_2*l_1*l_2*m_4*np.sin(theta_2 - theta_1) + dtheta_2*l_1*l_2*m_5*np.sin(theta_2 - theta_1) + dtheta_2*l_1*m_2*r_2*np.sin(theta_2 - theta_1)) - dtheta_1*(mu_1 + mu_2) - ddx_c*(-l_1*m_2 - l_1*m_3 - l_1*m_4 - l_1*m_5 - m_1*r_1)*np.cos(theta_1) - g*(-l_1*m_2 - l_1*m_3 - l_1*m_4 - l_1*m_5 - m_1*r_1)*np.sin(theta_1))*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2))/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2))/(I_3 - l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)**2*np.cos(theta_3 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_3**2*m_4 + l_3**2*m_5 + m_3*r_3**2 - (-l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_3 - theta_2))**2/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2)))/(I_4 - l_1**2*(l_4*m_5 + m_4*r_4)**2*np.cos(theta_4 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_4**2*m_5 + m_4*r_4**2 - (-l_1**2*(l_4*m_5 + m_4*r_4)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_4 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_4*m_5 + m_4*r_4)*np.cos(theta_4 - theta_2))**2/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2) - (-l_1**2*(l_4*m_5 + m_4*r_4)*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_4 - theta_1)*np.cos(theta_3 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_3*(l_4*m_5 + m_4*r_4)*np.cos(theta_4 - theta_3) - (-l_1**2*(l_4*m_5 + m_4*r_4)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_4 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_4*m_5 + m_4*r_4)*np.cos(theta_4 - theta_2))*(-l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_3 - theta_2))/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2))**2/(I_3 - l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)**2*np.cos(theta_3 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_3**2*m_4 + l_3**2*m_5 + m_3*r_3**2 - (-l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_3 - theta_2))**2/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2))))/(I_5 - l_1**2*m_5**2*r_5**2*np.cos(theta_5 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + m_5*r_5**2 - (-l_1**2*m_5*r_5*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_5 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*m_5*r_5*np.cos(theta_5 - theta_2))**2/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2) - (-l_1**2*m_5*r_5*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_5 - theta_1)*np.cos(theta_3 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_3*m_5*r_5*np.cos(theta_5 - theta_3) - (-l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_3 - theta_2))*(-l_1**2*m_5*r_5*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_5 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*m_5*r_5*np.cos(theta_5 - theta_2))/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2))**2/(I_3 - l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)**2*np.cos(theta_3 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_3**2*m_4 + l_3**2*m_5 + m_3*r_3**2 - (-l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_3 - theta_2))**2/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2)) - (-l_1**2*m_5*r_5*(l_4*m_5 + m_4*r_4)*np.cos(theta_5 - theta_1)*np.cos(theta_4 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_4*m_5*r_5*np.cos(theta_5 - theta_4) - (-l_1**2*(l_4*m_5 + m_4*r_4)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_4 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_4*m_5 + m_4*r_4)*np.cos(theta_4 - theta_2))*(-l_1**2*m_5*r_5*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_5 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*m_5*r_5*np.cos(theta_5 - theta_2))/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2) - (-l_1**2*(l_4*m_5 + m_4*r_4)*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_4 - theta_1)*np.cos(theta_3 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_3*(l_4*m_5 + m_4*r_4)*np.cos(theta_4 - theta_3) - (-l_1**2*(l_4*m_5 + m_4*r_4)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_4 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_4*m_5 + m_4*r_4)*np.cos(theta_4 - theta_2))*(-l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_3 - theta_2))/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2))*(-l_1**2*m_5*r_5*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_5 - theta_1)*np.cos(theta_3 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_3*m_5*r_5*np.cos(theta_5 - theta_3) - (-l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_3 - theta_2))*(-l_1**2*m_5*r_5*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_5 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*m_5*r_5*np.cos(theta_5 - theta_2))/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2))/(I_3 - l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)**2*np.cos(theta_3 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_3**2*m_4 + l_3**2*m_5 + m_3*r_3**2 - (-l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_3 - theta_2))**2/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2)))**2/(I_4 - l_1**2*(l_4*m_5 + m_4*r_4)**2*np.cos(theta_4 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_4**2*m_5 + m_4*r_4**2 - (-l_1**2*(l_4*m_5 + m_4*r_4)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_4 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_4*m_5 + m_4*r_4)*np.cos(theta_4 - theta_2))**2/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2) - (-l_1**2*(l_4*m_5 + m_4*r_4)*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_4 - theta_1)*np.cos(theta_3 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_3*(l_4*m_5 + m_4*r_4)*np.cos(theta_4 - theta_3) - (-l_1**2*(l_4*m_5 + m_4*r_4)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_4 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_4*m_5 + m_4*r_4)*np.cos(theta_4 - theta_2))*(-l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_3 - theta_2))/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2))**2/(I_3 - l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)**2*np.cos(theta_3 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_3**2*m_4 + l_3**2*m_5 + m_3*r_3**2 - (-l_1**2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)*np.cos(theta_3 - theta_1)*np.cos(theta_2 - theta_1)/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2*(l_3*m_4 + l_3*m_5 + m_3*r_3)*np.cos(theta_3 - theta_2))**2/(I_2 - l_1**2*(l_2*m_3 + l_2*m_4 + l_2*m_5 + m_2*r_2)**2*np.cos(theta_2 - theta_1)**2/(I_1 + l_1**2*m_2 + l_1**2*m_3 + l_1**2*m_4 + l_1**2*m_5 + m_1*r_1**2) + l_2**2*m_3 + l_2**2*m_4 + l_2**2*m_5 + m_2*r_2**2)))),
                      ddx_c
    ])

In [39]:
t_1000    = np.array([[0]]*1000)
y_1000    = np.array([[[1, 2, 3, 4, 1, 11, 22, 33, 44, 55, 1]]*1000])
args_1000 = np.array([[[9.81 , 0.3  , 0.3  , 0.3  , 0.3  , 0.2  , 0.2  , 0.2  , 0.2  ,
       0.2  , 0.8  , 0.8  , 0.8  , 0.8  , 0.8  , 0.015, 0.015, 0.015,
       0.015, 0.015, 0.011, 0.011, 0.011, 0.011, 0.011]]*1000])


inputs_np_cpu_1000  = t_1000, y_1000.T, args_1000.T
inputs_jax_gpu_1000 = jnp.array(t_1000, device=gpu),jnp.array(y_1000.T, device=gpu), jnp.array(args_1000.T, device=gpu)
inputs_jax_cpu_1000 = [jnp.array(t_1000, device=cpu),jnp.array(y_1000.T, device=cpu), jnp.array(args_1000.T, device=cpu)]

In [46]:
t = time.time()
robotic_equation_n5_numpy(*inputs_np_cpu_1000);
time.time()-t

0.2716941833496094

In [42]:
t = time.time()
func_jax_jit(*inputs_jax_cpu_1000).block_until_ready(); # jax with cpu + first jit call
time.time()-t

33.43120241165161

In [43]:
t = time.time()
func_jax_jit(*inputs_jax_cpu_1000).block_until_ready(); # jax with cpu + second jit call
time.time()-t

0.0009970664978027344

---

In [50]:
t = time.time()
func_jax_jit(*inputs_jax_gpu_1000).block_until_ready(); # jax with gpu + first jit call
time.time()-t

33.88152074813843

In [51]:
t = time.time()
func_jax_jit(*inputs_jax_gpu_1000).block_until_ready(); # jax with gpu + second jit call
time.time()-t

0.0022706985473632812

---

- Running 1000 Environments on Numpy in parallel is a lot slower than compiled JAX. 
- For Jax, the time needed with CPU increases slightly, and the time GPU remains the same with increased batch size.

In [53]:
t_100000    = np.array([[0]]*100000)
y_100000    = np.array([[[1, 2, 3, 4, 1, 11, 22, 33, 44, 55, 1]]*100000])
args_100000 = np.array([[[9.81 , 0.3  , 0.3  , 0.3  , 0.3  , 0.2  , 0.2  , 0.2  , 0.2  ,
       0.2  , 0.8  , 0.8  , 0.8  , 0.8  , 0.8  , 0.015, 0.015, 0.015,
       0.015, 0.015, 0.011, 0.011, 0.011, 0.011, 0.011]]*100000])


inputs_np_cpu_100000  = t_100000, y_100000.T, args_100000.T
inputs_jax_gpu_100000 = jnp.array(t_100000, device=gpu),jnp.array(y_100000.T, device=gpu), jnp.array(args_100000.T, device=gpu)
inputs_jax_cpu_100000 = [jnp.array(t_100000, device=cpu),jnp.array(y_100000.T, device=cpu), jnp.array(args_100000.T, device=cpu)]

In [54]:
t = time.time()
robotic_equation_n5_numpy(t=t_100000,y=y_100000.T,args=args_100000.T);
time.time()-t

47.59974241256714

In [55]:
t = time.time()
func_jax_jit(*inputs_jax_cpu_100000).block_until_ready(); # jax with cpu + first jit call
time.time()-t

35.42853379249573

In [56]:
t = time.time()
func_jax_jit(*inputs_jax_cpu_100000).block_until_ready(); # jax with cpu + second jit call
time.time()-t

0.03613758087158203

---

In [57]:
t = time.time()
func_jax_jit(*inputs_jax_gpu_100000).block_until_ready(); # jax with gpu + first jit call
time.time()-t

20.69680953025818

In [58]:
t = time.time()
func_jax_jit(*inputs_jax_gpu_100000).block_until_ready(); # jax with gpu + second jit call
time.time()-t

0.007390022277832031

---

- Running 100000 Environments on Numpy in parallel is **extremely slow** and took nearly a minute to compute.
- For Jax, the time needed with CPU increases slightly, and the time GPU remains the same with increased batch-size again.
    - The JAX CPU code scaled better than numpy, because of XLA optimization of JAX, and JAX's optimized kernel operations from compiling.
- The GPU in now faster than the CPU as of the large batch size.